# Finding Fact-checkable Tweets with Machine Learning


## Credits

This notebook was based on one originally created by Jeremy Howard and the other folks at [fast.ai](https://fast.ai) as part of [this fantastic class](https://course.fast.ai/). Specifically, it comes from Lesson 4. You can [see the lession video](https://course.fast.ai/videos/?lesson=4) and [the original class notebook](https://github.com/fastai/course-v3/blob/master/nbs/dl1/lesson3-imdb.ipynb). 

The idea for the project came from Dan Keemahill at the Austin American-Statesman newspaper. Dan, Madlin Mekelburg, and others at the paper hand-coded the tweets used for the classificaiton training.

For more information about this project, and details about how to use this work in the wild, check out our [Quartz AI Studio blog post about the checkable-tweets project](https://qz.ai/?p=89).

-- John Keefe, [Quartz](https://qz.com), October 2019

## Setup

### For those using Google Colaboratory ...

Be aware that Google Colab instances are ephemeral -- they vanish *Poof* when you close them, or after a period of sitting idle (currently 90 minutes), or if you use one for more than 12 hours.

If you're using Google Colaboratory, you don't need a GPU for this notebook.

Just run this cell:

In [1]:
## ALL GOOGLE COLAB USERS RUN THIS CELL

## This runs a script that installs fast.ai
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


### For those _not_ using Google Colaboratory ...

This section is just for people who decide to use one of the notebooks on a system other than Google Colaboartory. 

Those people should run the cell below.

In [0]:
## NON-COLABORATORY USERS SHOULD RUN THIS CELL
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Everybody do this ...

Everyone needs to run the next cell, which initializes the Python libraries we'll use in this notebook.

In [2]:
## AND *EVERYBODY* SHOULD RUN THIS CELL
import warnings
warnings.filterwarnings('ignore')
from fastai.text import *
import fastai
print(f'fastai: {fastai.__version__}')
print(f'cuda: {torch.cuda.is_available()}')

fastai: 1.0.61
cuda: False


## The Data

We're going to be using two sets of tweets for this project:

- A CSV (comma-separated values file) containing a bunch of #txlege tweets
- A CSV of #txlege tweets that have been hand-coded as "fact-checkable" or "not fact-checkable"


In [3]:
# Run this cell to download the data we'll use for this exercise
# !wget -N https://s3.amazonaws.com/media.johnkeefe.net/newmark-investigations/unclassified_tweets.zip --quiet
# !unzip -q unclassified_tweets.zip

# !wget -N https://s3.amazonaws.com/media.johnkeefe.net/newmark-investigations/TWITTER_CSV_EXPORTS.zip --quiet
# !unzip -q TWITTER_CSV_EXPORTS.zip

!wget -N https://www.dropbox.com/s/rwt23g38d2krdae/output.zip
!unzip -q output.zip

print('Done!')

--2020-05-11 13:37:51--  https://www.dropbox.com/s/rwt23g38d2krdae/output.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rwt23g38d2krdae/output.zip [following]
--2020-05-11 13:37:51--  https://www.dropbox.com/s/raw/rwt23g38d2krdae/output.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc36bfa8c04f6cbbb255914d3d7b.dl.dropboxusercontent.com/cd/0/inline/A3ihaiwfDq-kMFBNBwR7PkVluBFgQpXZj2CdCQb-YHqvEOwGcPGqqK678SEeiGib8XwGexA2rdDOYogPk6vAQY5aorkVa6THroaV8C7cjUhUSg/file# [following]
--2020-05-11 13:37:52--  https://uc36bfa8c04f6cbbb255914d3d7b.dl.dropboxusercontent.com/cd/0/inline/A3ihaiwfDq-kMFBNBwR7PkVluBFgQpXZj2CdCQb-YHqvEOwGcPGqqK678SEeiGib8XwGexA2rdDOYogPk6vAQY5aorkVa6THroaV8C7cjUhUSg/file
Resolving uc36bfa8c04f6cb

Let's take a look.

In [4]:
%ls

data@  models@  output/  output.zip


In [5]:
import glob
import pandas as pd

# file_list = sorted(glob.glob('./unclassified_tweets/*.csv'))
file_list = sorted(glob.glob('./output/*.csv'))
# file_list2 = sorted(glob.glob('./unclassified_tweets/*.csv'))
# file_list.extend(file_list2)
file_list

['./output/A. Donald McEachin.csv',
 './output/A. Drew Ferguson IV.csv',
 './output/Abby Finkenauer.csv',
 './output/Abigail Davis Spanberger.csv',
 './output/Adam B. Schiff.csv',
 './output/Adam Kinzinger.csv',
 './output/Adam Smith.csv',
 './output/Adrian Smith.csv',
 './output/Adriano Espaillat.csv',
 './output/Al Green.csv',
 './output/Al Lawson, Jr..csv',
 './output/Alan S. Lowenthal.csv',
 './output/Albio Sires.csv',
 './output/Alcee L. Hastings.csv',
 './output/Alexander X. Mooney.csv',
 './output/Alexandria Ocasio-Cortez.csv',
 './output/Alma S. Adams.csv',
 './output/Ami Bera.csv',
 './output/Amy Klobuchar.csv',
 './output/André Carson.csv',
 './output/Andy Barr.csv',
 './output/Andy Biggs.csv',
 './output/Andy Harris.csv',
 './output/Andy Kim.csv',
 './output/Andy Levin.csv',
 './output/Angie Craig.csv',
 './output/Angus S. King, Jr..csv',
 './output/Ann Kirkpatrick.csv',
 './output/Ann M. Kuster.csv',
 './output/Ann Wagner.csv',
 './output/Anna G. Eshoo.csv',
 './output/Anth

In [9]:
!head './output/Yvette D. Clarke.csv'

tweet,dates
"Every May, we recognize our incredible mothers. We are indebted to their sacrifice, inspired by their wisdom and emboldened by their love.
 
This #MothersDay, let us remember to hold our loved ones extra close during this difficult time. https://t.co/IBLSQn1Jyw",2020-05-10 14:03:36
"A Saturday well-spent delivering masks to constituents in #Brownsville. Thanks for helping our community #flattenthecurve, #NY09! 💪🏾✨ https://t.co/D4qn979vzJ",2020-05-09 21:09:56
"In an ideal world, we'd honor teachers 7 days a week, 365 days a year for the enormous impact they have on our children. Too often, they’re the one person a child living in difficult circumstances can count on for support and an encouraging word. Happy #TeacherAppreciationWeek ✨",2020-05-08 18:15:45
Join me and @RepSusanWild tonight for this important conversation on #racial health disparities within #COVID19 .  Register here: https://t.co/IWEQmHAwqA https://t.co/0tjItGgyqh,2020-05-07 20:57:25
"This global pandemic ha

## Using our saved model

In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'ai-workshop/candidate_tweets/'
save_path = Path(base_dir)
save_path.mkdir(parents=True, exist_ok=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
# load the model from the 'export.pkl' file on your Google Drive
my_model = load_learner(save_path, file="export-tweetmodel.pkl")  

## Getting every candidate's stats

In [0]:
import csv
import os

In [0]:
# with open(file_list[0], newline='') as csvfile:
#   reader = csv.DictReader(csvfile)

#   for row in reader:
#     row['test'] = "hello"
#     summary_data.append(row)
#     print (row)

In [0]:
# ## Testing code
# file = file_list[0]

In [10]:
summary_data = []

# # loop through all the file names
for file in file_list: 

  # open csv
  with open(file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)

    # loop through all the rows in the csv
    for row in reader:

      # skip this row if there's no content
      if row['tweet'] == "":
        continue 

      # make the prediction
      fear = my_model.predict(row['tweet'])

      # Note: .item() here turns the float tensor into an actual float
      pct_true = fear[2][1].item()

      if pct_true > 0.50:

        row['file_name'] = os.path.basename(file)
        row['pct_true'] = pct_true

        summary_data.append(row)

        print(row)

 

Streaming output truncated to the last 5000 lines.
OrderedDict([('tweet', 'Roger Stone was found guilty of lying to Congress and threatening a witness.\n\nHe did it to cover up for Trump. His sentence is justified.\n\nIt should go without saying, but to pardon Stone when his crimes were committed to protect Trump would be a breathtaking act of corruption.'), ('dates', '2020-02-20 17:50:04'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.7303450107574463)])
OrderedDict([('tweet', 'Taxpayers have a right to know how much of their money is lining the President’s pockets.\n\nTrump shouldn’t have the option to hide it.\n\nI’ve introduced legislation to require disclosure of government spending that directly benefits presidents and their businesses.\n\nLet’s find out https://t.co/JUNW3Pmh7r'), ('dates', '2020-02-20 15:51:12'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.5741750001907349)])
OrderedDict([('tweet', 'There are 175,000 federal prisoners. Many are serving harsh sente

In [0]:
summary_data = [Streaming output truncated to the last 5000 lines.
OrderedDict([('tweet', 'Roger Stone was found guilty of lying to Congress and threatening a witness.\n\nHe did it to cover up for Trump. His sentence is justified.\n\nIt should go without saying, but to pardon Stone when his crimes were committed to protect Trump would be a breathtaking act of corruption.'), ('dates', '2020-02-20 17:50:04'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.7303450107574463)])
OrderedDict([('tweet', 'Taxpayers have a right to know how much of their money is lining the President’s pockets.\n\nTrump shouldn’t have the option to hide it.\n\nI’ve introduced legislation to require disclosure of government spending that directly benefits presidents and their businesses.\n\nLet’s find out https://t.co/JUNW3Pmh7r'), ('dates', '2020-02-20 15:51:12'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.5741750001907349)])
OrderedDict([('tweet', 'There are 175,000 federal prisoners. Many are serving harsh sentences and deserve mercy.\n\nYet, Trump uses pardons almost exclusively for:\n\n1) Political supporters\n2) Wealthy individuals\n3) Contestants on his TV show\n\nAnd worse abuses are almost surely to come.'), ('dates', '2020-02-18 20:43:44'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.6422843933105469)])
OrderedDict([('tweet', 'Barr admits he intervened in the sentencing of a man who lied to Congress to cover up for the President.\n\nHe’s only upset that Trump’s tweets made the political nature of his intervention obvious.\n\nBarr fools no one. He’s a witting accomplice to Trump’s attack on the rule of law. https://t.co/Kz5tVMnlPo'), ('dates', '2020-02-14 16:27:12'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.9471877217292786)])
OrderedDict([('tweet', "Trump abused his power to coerce Ukraine into announcing investigations for his personal benefit by freezing military aid.\n\nNow, he's using his powers to coerce states to stop investigations into him and his businesses.\n\nDifferent corrupt purpose, same corrupt President. https://t.co/4pBqY8gAnI"), ('dates', '2020-02-13 20:44:38'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.8272378444671631)])
OrderedDict([('tweet', 'For those who were convicted of lying to cover up his crimes, Trump urges lighter sentences and dangles pardons.\n\nHe demands his enemies be investigated and prosecuted.\n\nHe retaliates against witnesses and public servants.\n\nRepublicans’ response?\n\nSilence.'), ('dates', '2020-02-12 16:10:58'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.9234827756881714)])
OrderedDict([('tweet', 'Sentencing recommendations countermanded, prosecutors punished, and enemies investigated.\n\nHow do we measure the damage Bill Barr is inflicting on the independence, integrity and reputation of the Department of Justice? \n\nWe can’t. https://t.co/d4b7z49EOp'), ('dates', '2020-02-12 02:30:33'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.5341445803642273)])
OrderedDict([('tweet', 'Roger Stone lied to Congress and threatened a witness to cover up Trump campaign contacts with Wikileaks.\n\nHe was found guilty on all charges.\n\nBarr overruling career prosecutors at Trump’s urging is a disgraceful attack on the rule of law.\n\nHas DOJ no independence left? https://t.co/eMDGGmXXFi'), ('dates', '2020-02-11 23:28:52'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.807056725025177)])
OrderedDict([('tweet', 'At trial, we held up these redacted emails and asked Senators if they wanted to know what Trump was hiding.\n\nThey didn’t. Now we know why:\n\nThey confirm OMB misled Congress about DOD’s objections to the illegal hold on Ukraine aid.\n\nThe truth will come out. https://t.co/xjTGtAK5wc'), ('dates', '2020-02-11 19:50:01'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.9213363528251648)])
OrderedDict([('tweet', "Budgets reflect values, and those behind Trump's budget are clear:\n\nCuts to Medicaid. Cuts to children's health care. Cuts to Medicare. Cuts to the EPA. Cuts to the Department of Education. Cuts to the CDC. Cuts to affordable housing programs.\n\nThese are not our values. https://t.co/72OTxvFvwM"), ('dates', '2020-02-10 18:54:06'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.5468636155128479)])
OrderedDict([('tweet', 'President Trump is exacting his retribution, removing those who complied with subpoenas, came forward, and testified about his misconduct.\n\nThese are the actions of a man who believes he is above the law —\n\nPrecisely the kind of conduct Congressional Republicans enabled.'), ('dates', '2020-02-08 01:21:31'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.6334176659584045)])
OrderedDict([('tweet', 'The House proved overwhelmingly that the President abused his power.\n\nNow the verdict is in:\n\nOne Republican Senator had the courage to do impartial justice.\n\nTo put country over party.\n\nAnd vindicate the Founders’ faith in self-governance.\n\nAnd show us, \n\nRight still matters.'), ('dates', '2020-02-05 23:27:17'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.7358623147010803)])
OrderedDict([('tweet', 'Having proven Trump guilty, I asked if there was just one Republican Senator who would say “enough”\n\nWho would stand up against this dangerously immoral president\n\nWho would display moral courage\n\nWho would do impartial justice as their oath required and convict\n\nAnd there is. https://t.co/xNHi2CP6oF'), ('dates', '2020-02-05 19:29:43'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.7763552069664001)])
OrderedDict([('tweet', 'Trump claims he will protect those with preexisting conditions, while trying to take away their coverage.\n\nHe says he is taking on the drug companies, but does nothing to lower Rx costs.\n\nThere is a word for those who say one thing and do another.\n\nToday, that word is President.'), ('dates', '2020-02-05 04:13:05'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.7991685271263123)])
OrderedDict([('tweet', 'Trump’s defense has retreated to their last defense:\n\nHe’s guilty as sin, but the voters should clean up this mess.\n\nWe must ask ourselves:\n\nCan we trust the President will not continue to try to cheat in that very election?\n\nThe sad and incontestable answer is:\n\nNo, we can’t. https://t.co/qbFR4YNyMj'), ('dates', '2020-02-04 19:10:32'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.8203239440917969)])
OrderedDict([('tweet', 'Senators should have insisted on a fair trial with documents and witnesses.\n\nThey chose not to.\n \nSome argue, let the election decide — \n\nBut that argument is unpersuasive when the President is trying to cheat in 2020. https://t.co/o6qFHW2qcJ'), ('dates', '2020-02-02 21:55:51'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.9008287191390991)])
OrderedDict([('tweet', 'We proved our case. Republican Senators now admit Trump is guilty as charged.\n\nStill, they blocked witnesses from testifying, and deprived the American people of a full accounting of Trump’s misconduct.\n \nSenators will now render judgement – and be held accountable for it. https://t.co/VpSbBDofYL'), ('dates', '2020-02-02 20:47:37'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.9837696552276611)])
OrderedDict([('tweet', 'When we have a President who so egregiously abuses and corrupts his office, unlike other countries, we have a remedy:\n\nImpeachment.\n\nThis trial is now part of our Constitutional heritage.\n \nBut if we refuse a fair trial, it diminishes the example we set for the rest of world. https://t.co/0BgC9IaPZO'), ('dates', '2020-01-31 00:06:46'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.8202789425849915)])
OrderedDict([('tweet', "It's clear what the President's lawyers are saying:\n\nIf you want a fair trial, we're going to make this endless. We're going to make this painful.\n \nBut a fair trial isn't a threat. It's a cornerstone of American democracy.\n\nAnd you can't have a fair trial without basic fairness. https://t.co/eK43g4Fggc"), ('dates', '2020-01-30 02:57:03'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.6003367900848389)])
OrderedDict([('tweet', "The idea that abuse of power isn’t impeachable would’ve terrified our Founders.\n\nYou can’t write off a consensus of Constitutional scholars by calling them “Never Trumpers,”\n\nUnless we're prepared to accept that Article II really does allow the President to do whatever he wants. https://t.co/hY5hBv9AJ4"), ('dates', '2020-01-30 00:23:49'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.8609966039657593)])
OrderedDict([('tweet', "As Trump said, they “have all the material.” And there’s a reason they won’t provide it.\n\nThe White House is blocking all witnesses and documents, and Senators can infer why.\n\nBut Senators don't have to infer. They can demand to see it.\n\nAnd they must. https://t.co/anKQH9CMoR"), ('dates', '2020-01-29 23:42:19'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.8400590419769287)])
OrderedDict([('tweet', 'Here are my takeaways:\n \nTrump’s lawyers cannot, and did not, defend him on the facts.\n \nTheir defense has evolved to this: he did it, so what?\n \nHis lawyers strengthened our case that the Senate must hear from Bolton.\n \nA fair trial requires key witnesses. Will America have one?'), ('dates', '2020-01-28 20:06:38'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.8572421073913574)])
OrderedDict([('tweet', 'The President’s lawyers say this is about a “policy debate.”\n\nIt is not.\n\nUnless that policy provides that a president can be as corrupt as he chooses and there is nothing Congress can do about it.\n\nThat must never be the policy of the United States.'), ('dates', '2020-01-28 19:35:29'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.8887270092964172)])
OrderedDict([('tweet', "Trump's defense boils down to this:\n\nSure, the President did it.\n\nWe all know he did it.\n \nWe just don't want the American people to see even more evidence that he did it.\n\nBesides, he is allowed to do whatever he wants.\n\nGet over it.\n\nWell, the American people know better. https://t.co/r92J8ILk20"), ('dates', '2020-01-28 16:33:34'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.7143102288246155)])
OrderedDict([('tweet', 'A few takeaways:\n\nThe President’s lawyers made the case for calling John Bolton, falsely claiming the President never told anyone the aid was conditioned on investigations. \n\nThey don’t deny Trump sought foreign interference in 2020.\n\nThey smear the Bidens.\n\nThey have no defense.'), ('dates', '2020-01-28 02:32:38'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.9692364931106567)])
OrderedDict([('tweet', 'We already have strong and direct evidence that Trump withheld military aid to demand political investigations.\n\nBolton further corroborates it.\n\nSenators have taken an oath to be impartial. Don’t they want to hear the full truth?\n\nBolton must testify. https://t.co/UXl9b2x2ZX'), ('dates', '2020-01-27 14:55:27'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.8617543578147888)])
OrderedDict([('tweet', 'The President blocked our request for Bolton’s testimony.\n\nNow we see why:\n\nBolton directly contradicts the heart of the President’s defense.\n\nIf the trial is to be fair, Senators must insist that Mr. Bolton be called as a witness, and provide his notes and other documents. https://t.co/go9DJdRDEf'), ('dates', '2020-01-27 00:15:47'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.6286671161651611)])
OrderedDict([('tweet', "It requires moral courage to stand up to this President.\n\nThere's no doubt that he is vindictive — his threatening tweet about me today is proof positive.\n\nBut we all have an obligation to do the right thing.\n\nMoral courage is rare. But it is never more essential than now. https://t.co/sNDdd4Ii9t"), ('dates', '2020-01-26 18:39:26'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.5334674119949341)])
OrderedDict([('tweet', "The President's legal defense boils down to this:\n\nDeprive Americans of a fair trial.\n\nBut Americans know what a fair trial looks like. They know it requires witnesses and documents.\n\nIf Trump's team succeeds in orchestrating a trial without them,\n\nThere will be no exoneration. https://t.co/WEJkFMZBbo"), ('dates', '2020-01-26 17:41:41'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.897325873374939)])
OrderedDict([('tweet', 'After listening to the President’s lawyers opening arguments, I have three observations:\n\nThey don’t contest the facts of Trump’s scheme.\n\nThey’re trying to deflect, distract from, and distort the truth.\n\nAnd they are continuing to cover it up by blocking documents and witnesses.'), ('dates', '2020-01-25 17:06:03'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.7297675013542175)])
OrderedDict([('tweet', "Americans know about Trump's Ukraine scheme because people with a sense of duty were willing to step forward and speak out.\n\nPeople like Ambassadors Yovanovitch and Taylor, Lt. Col. Vindman, Dr. Hill.\n\nThey were willing to risk their careers. Can Members of Congress do the same? https://t.co/ac3WvdYULR"), ('dates', '2020-01-23 03:38:27'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.5074416399002075)])
OrderedDict([('tweet', 'Time and time again, President Trump has shown he will continue to abuse his power, unless we hold him accountable.\n\nThe only appropriate remedy for a President of the United States so unwilling to be constrained by the rule of law?\n\nImpeachment, and removal from office. https://t.co/JOlxRLllUB'), ('dates', '2020-01-23 00:01:33'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.5507080554962158)])
OrderedDict([('tweet', "Senators get 16 hours to ask questions. Wouldn't they like to ask about documents?\n\nIf McConnell gets his way, they won't see any documents before their questions.\n\nEven if documents are allowed later, they can't ask a single question about them.\n\nAmericans deserve a fair trial. https://t.co/wF5SBs9xV8"), ('dates', '2020-01-22 00:50:04'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.553456723690033)])
OrderedDict([('tweet', "Trump's lawyer claimed the House isn't ready to present our case.\n\nWe’re ready.\n\nThe House calls John Bolton to testify. The House calls Mick Mulvaney.\n\nWe’re ready to present our case, ready to call witnesses, ready to see the documents.\n\nThe question: Will the Senate let us? https://t.co/MD5d1bqsxU"), ('dates', '2020-01-21 21:55:08'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.8154789209365845)])
OrderedDict([('tweet', "Americans want a fair trial.\n\nThey want to believe their government is still capable of rising to the occasion.\n\nThey want to believe we can rise above party, to do what’s best for our country.\n\nSadly, a great many Americans doubt this is still possible.\n\nLet's prove them wrong. https://t.co/VGor4FSQdS"), ('dates', '2020-01-21 20:06:53'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.5062677264213562)])
OrderedDict([('tweet', 'Americans are watching, and they overwhelmingly support a fair trial. One that is fair to the President but also fair to the American people.\n \nIf Senators do not provide a fair trial, with documents and witnesses, history — and their constituents — will judge them harshly. https://t.co/AUsBOXNbrT'), ('dates', '2020-01-19 19:13:35'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.9158247113227844)])
OrderedDict([('tweet', 'Americans overwhelmingly want to see a fair trial.  That is the foundational issue on which everything else rests.\n\nBut will Senators allow witnesses and subpoena documents?\n \nIf Senators are true to the oath they have taken to impartially administer justice, they must do so. https://t.co/9cHAOODgow'), ('dates', '2020-01-19 17:23:57'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.9170161485671997)])
OrderedDict([('tweet', 'If the Vice President believes all of the witnesses who testified before the House contradict Mr. Parnas, why won’t he let the public see the supplemental testimony of his own staffer, Jennifer Williams?\n\nThere is no reason it should be classified. Release it, Mr. Vice President. https://t.co/tfQ7uIHOdG'), ('dates', '2020-01-16 21:02:31'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.6391708850860596)])
OrderedDict([('tweet', 'The nonpartisan Government Accountability Office has found the Trump Administration broke the law by withholding military aid from Ukraine.\n\nTrump’s willingness to break the law to coerce Ukraine to help him cheat in the next election shows the magnitude of his abuse of power. https://t.co/WgrJa3spcx'), ('dates', '2020-01-16 16:13:14'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.5019274950027466)])
OrderedDict([('tweet', 'Today, we take the next step to #DefendOurDemocracy, sending articles of impeachment to the Senate.\n \nMcConnell will have a choice: stand with the Constitution, or with Trump’s cover-up.\n \nUphold the rule of law, or help Trump undermine it.'), ('dates', '2020-01-15 13:37:50'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.5492963790893555)])
OrderedDict([('tweet', 'McConnell says the Senate shouldn’t hear witnesses or see documents, as if it’s not a trial, only an appeal.\n\nBut the trial has yet to begin:\n\nThese new materials show exactly why we need a fair trial — with documents and witnesses.\n\nNot a coverup by the President and his men. https://t.co/KFgHlDdocf'), ('dates', '2020-01-15 02:38:31'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.778032660484314)])
OrderedDict([('tweet', "The overwhelming majority of the American people want to see a fair trial — fair to the president, and fair to the public.\n\nBut Mitch McConnell wants to block witnesses and documents, an unprecedented attempt to obscure the truth.\n\nThat's not a fair trial. It's a cover up. https://t.co/zy2LBHWKrM"), ('dates', '2020-01-14 17:47:46'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.7236496806144714)])
OrderedDict([('tweet', 'Trump just tweeted:\n\n“I stand stronger than anyone in protecting your Healthcare with Pre- Existing Conditions.”\n\nHow absurd.\n\nHis Administration is in court—right now—trying to repeal the entire ACA, including protections for pre-existing conditions.\n\nNo one is fooled.'), ('dates', '2020-01-14 00:53:43'), ('file_name', 'Adam B. Schiff.csv'), ('pct_true', 0.7387624979019165)])
OrderedDict([('tweet', 'President Bush is a statesman, and has remained generally silent on politics.  Our differences as humans seem small when we face disasters, and @GWBLibrary and the President are 100% correct.  It’s not about you, it’s about us. Fear destroys people, and nations. https://t.co/6GXJ1O4Tun'), ('dates', '2020-05-03 14:09:09'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.700552761554718)])
OrderedDict([('tweet', 'The extreme restrictions and nonsense excuses by the Infinity @CelebrityCruise have become deeply troubling &amp; frankly unacceptable.  My staff and I will continue to work with the Whitcomb family to bring Julia home safely &amp; soon. https://t.co/bkRgZLrClX'), ('dates', '2020-04-30 00:44:40'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.5748410224914551)])
OrderedDict([('tweet', 'What kind of insane, Communist sympathy is this garbage @CNN ?  Really?  I’m speechless.  \n\nhttps://t.co/7Ix0SuGiaU'), ('dates', '2020-04-29 13:56:43'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.6476377248764038)])
OrderedDict([('tweet', 'When the Chinese gov’t tells me they “implemented control measures,” does that include the control of information; shutting out reporters &amp; silencing doctors? As the rest of the world works to fight this pandemic, China needs to come clean &amp; be honest about the #coronavirus. https://t.co/z8KJRC9WqJ'), ('dates', '2020-04-20 17:24:40'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.8095052242279053)])
OrderedDict([('tweet', 'Remember:  it was not until AFTER March 11 that @SpeakerPelosi began ANY restrictions or distancing in the Capitol despite calls to do so from some.  On March 11, we did get a recommendation on “mingling.”  She was not ahead of this. \n\nhttps://t.co/nAYv91h9yU'), ('dates', '2020-04-20 13:03:21'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.5297982096672058)])
OrderedDict([('tweet', 'We must get a full accounting from @chineseEmbinUS  on this, and answers to why for weeks  #chinahidthevirus  The Communist Party of China failed its own people, and the world. https://t.co/uVnbyGOJbb'), ('dates', '2020-04-14 12:55:47'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.6918944120407104)])
OrderedDict([('tweet', 'While the rest of the world comes together to take action &amp; help stop this global pandemic, the Communist Party of China continues to cover-up its origin &amp; spread conspiracy theories rather than step up &amp; share with the world what they know about the virus.'), ('dates', '2020-04-02 12:21:40'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.9262062311172485)])
OrderedDict([('tweet', 'American workers need relief—and not being able to access the #IDES website 💻 or get through to their hotline ☎️ is unacceptable. While I understand the surge happening, the state needs to get a handle on this with the resources they’ve received from the federal government.'), ('dates', '2020-03-26 16:48:23'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.5341280698776245)])
OrderedDict([('tweet', 'No family should have to endure what the Levinson’s have these last 13 years. Bob Levinson was an American hero &amp; it’s shameful that he was left behind, wrongfully held by the Iranian regime. I stand w/ Bob’s family in seeking justice, starting w/ the swift return of his body. https://t.co/ECTvYIJzVh'), ('dates', '2020-03-26 01:20:50'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.7357141971588135)])
OrderedDict([('tweet', '“Do Democrats even care?” That’s a good question, @WSJ. Hard to say when you’ve got @SpeakerPelosi holding a bipartisan #COVID19 relief bill as her own political hostage. https://t.co/JKiOcclDZN'), ('dates', '2020-03-24 15:40:48'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.6470605134963989)])
OrderedDict([('tweet', 'Dec 10: first COVID19 victim\nDec 31: Docs post on internet, promptly arrested by China. China orders destruction of samples.\nJan 21: First mention by China in China Daily newspaper\nJan 31: US travel restrictions\n\nFor two months, #Chinahidthevirus'), ('dates', '2020-03-19 15:09:01'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.8651502728462219)])
OrderedDict([('tweet', 'Absolutely.  China hid this virus for months and allowed their people to fly with it.  China doesn’t have a right to be outraged or offended, and they should pay a price. https://t.co/7fYLaA5X2M'), ('dates', '2020-03-18 13:18:56'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.9843883514404297)])
OrderedDict([('tweet', 'Right now, we need to focus on our people and get through this crisis.  When this is over, China must be held accountable for hiding this for so long.  Communism fails every time.'), ('dates', '2020-03-15 22:00:45'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.8513967394828796)])
OrderedDict([('tweet', 'Watched the President’s address.  Immediately after #cnn about Trump’s delivery and litigating his words.  #foxnews? “swine flu killed many and the media wasn’t panicked.”  Here’s an idea: let’s put the election aside, stop the division, and focus on the American people.'), ('dates', '2020-03-12 01:41:38'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.6926929950714111)])
OrderedDict([('tweet', "This is awful, &amp; entirely unacceptable. The decision by @GovPritzker to protect violent felons facing deportation is just wrong--&amp; it's clearly putting the people of #Illinois in danger. This reckless policy change should be reversed immediately. https://t.co/eYypRwJxSy"), ('dates', '2020-03-06 18:16:31'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.5569822192192078)])
OrderedDict([('tweet', "The Iranian people are united--they're united against the corrupt Iranian regime that violates their human rights &amp; funds terrorism around the world. I'm proud to cosponsor this resolution in support of the #IranProtests &amp; to stand with all who seek freedom. https://t.co/VgbzM8We9F"), ('dates', '2020-01-14 22:42:01'), ('file_name', 'Adam Kinzinger.csv'), ('pct_true', 0.7849080562591553)])
OrderedDict([('tweet', 'Michael Flynn pleaded guilty to lying to the FBI. \n\nThe Justice Dept compromised its integrity and independence by dropping the case. \n\nBarr has managed to sink even lower, politicizing the judicial system and further undermining the rule of law.\nhttps://t.co/Yc2VfY5376'), ('dates', '2020-05-08 21:40:21'), ('file_name', 'Adam Smith.csv'), ('pct_true', 0.7259652018547058)])
OrderedDict([('tweet', 'This Asian Pacific American Heritage Month, I stand in solidarity with my @CAPAC colleagues in denouncing xenophobia &amp; celebrating the many contributions of our #AAPI communities. It’s more important than ever to combat the rise in anti-Asian bigotry due to #COVID19. #APAHM2020'), ('dates', '2020-05-01 19:31:00'), ('file_name', 'Adam Smith.csv'), ('pct_true', 0.6081767082214355)])
OrderedDict([('tweet', 'The president attacked IGs for telling the truth and criticized them for following the law, putting politics over accountability and undermining a functioning fed gov. The unjustified retaliation against IGs for their oversight and honesty is appalling. Proud to join @RepMaloney. https://t.co/0hgUBr37eA'), ('dates', '2020-04-10 15:58:57'), ('file_name', 'Adam Smith.csv'), ('pct_true', 0.7226552367210388)])
OrderedDict([('tweet', 'Disruptions caused by #COVIDー19 will deny people across the country the opportunity to engage with agencies on policy revisions and rulemakings.\n\nAgencies should postpone and reschedule public hearings and extend public comment periods to when it becomes feasible again. https://t.co/O66EXl9vC0'), ('dates', '2020-04-01 22:37:02'), ('file_name', 'Adam Smith.csv'), ('pct_true', 0.5471487641334534)])
OrderedDict([('tweet', "Asian Americans like others across the country worry about #COVID19 but also fear the dramatic increase of threats against those of Asian descent. Proud to cosponsor @RepGraceMeng's resolution and take a stand against intolerance, bigotry, and discrimination during this pandemic. https://t.co/Px7yA3xTrE"), ('dates', '2020-03-26 17:31:17'), ('file_name', 'Adam Smith.csv'), ('pct_true', 0.7816590070724487)])
OrderedDict([('tweet', 'Misinformation has led to the stigmatization of those of Asian descent based on fear and xenophobia, and this threatens the wellbeing of our communities and public health. \n\nWe must remain unified and compassionate to combat this epidemic and to protect our diverse communities. https://t.co/0oiIFk5NmB'), ('dates', '2020-03-25 21:30:38'), ('file_name', 'Adam Smith.csv'), ('pct_true', 0.7175086736679077)])
OrderedDict([('tweet', 'Much has changed in the four months since President Trump signed #USMCA into law back in December – not the least of which is the ongoing COVID-19 pandemic. However, what has not changed is the importance of this agreement. My column here: https://t.co/rC8NoGsPmG'), ('dates', '2020-05-04 21:55:21'), ('file_name', 'Adrian Smith.csv'), ('pct_true', 0.535073459148407)])
OrderedDict([('tweet', 'The decision to eliminate the terrorist Soleimani yesterday sends the message while our goal must be peace, the United States will not tolerate Iranian provocations or further attacks on Americans. My statement here: https://t.co/VghOL9u0K0'), ('dates', '2020-01-03 19:27:20'), ('file_name', 'Adrian Smith.csv'), ('pct_true', 0.5015901923179626)])
OrderedDict([('tweet', 'While parts of the proposal look positive, the 2020 #RFS does not sufficiently address the continued abuse of small refinery exemptions. Once again, @EPAAWheeler left in place a significant loophole.'), ('dates', '2019-12-19 19:05:44'), ('file_name', 'Adrian Smith.csv'), ('pct_true', 0.5443532466888428)])
OrderedDict([('tweet', 'I voted against Speaker Pelosi’s socialist #HR3 today.  H.R. 19, which I support, contains bipartisan provisions to lower prescription drug prices and spur innovation. #MoreCures'), ('dates', '2019-12-12 19:49:15'), ('file_name', 'Adrian Smith.csv'), ('pct_true', 0.8156720399856567)])
OrderedDict([('tweet', 'Speaker Pelosi’s #FewerCures bill is an outright attempt to socialize American medicine. We have bipartisan solutions to lower prescription drug costs – this isn’t one of them.'), ('dates', '2019-12-11 18:59:57'), ('file_name', 'Adrian Smith.csv'), ('pct_true', 0.6336276531219482)])
OrderedDict([('tweet', 'Speaker Pelosi’s attempt to socialize American medicine undoes bipartisan work on lowering drug pricing policies which was well under way. My full statement on #HR3 here: https://t.co/cjr23ZRvy4'), ('dates', '2019-12-10 22:25:51'), ('file_name', 'Adrian Smith.csv'), ('pct_true', 0.5644857883453369)])
OrderedDict([('tweet', 'EPA’s proposed waiver rule leaves loopholes which would diminish RFS. I ask all of those affected by this policy to make their voices heard during the 30-day public comment period. My statement here: https://t.co/PJK9tVTgTH'), ('dates', '2019-10-30 21:09:06'), ('file_name', 'Adrian Smith.csv'), ('pct_true', 0.5562916398048401)])
OrderedDict([('tweet', 'We will not stand idly by while minority businesses, families &amp; our communities suffer!\n\nRead my letter to NY State Attorney @TishJames calling for an investigation into the ineffectual rollout of the loans &amp; grants provided by the #CARESAct and impact to minority businesses. https://t.co/xu2EAaYZ2z'), ('dates', '2020-04-25 20:03:17'), ('file_name', 'Adriano Espaillat.csv'), ('pct_true', 0.6966474056243896)])
OrderedDict([('tweet', 'Did anybody expect @realdonaldtrump to push dangerous unsubstantiated medical advice like a third rate charlatan? Yes. \n\nBut we must never accept that our President is comfortable purporting that life threatening snake oil remedies can end a pandemic. #Horrible #COVID19'), ('dates', '2020-04-23 23:53:10'), ('file_name', 'Adriano Espaillat.csv'), ('pct_true', 0.786663830280304)])
OrderedDict([('tweet', '#MitchAntoinette would rather deny aid to firefighters, police &amp; EMS than deliver the relief they need to keep fighting the coronavirus. This is a disgusting remark to make when American lives are on the line nationwide. https://t.co/7EJp3SOHPZ'), ('dates', '2020-04-23 18:03:05'), ('file_name', 'Adriano Espaillat.csv'), ('pct_true', 0.6375146508216858)])
OrderedDict([('tweet', "To whom it may concern:\nIf we know that African-American communities have a disproportionate concentration of deaths due to coronavirus, why aren't we concentrating more of our response on combating coronavirus in African-American communities? #BlackLivesMatter"), ('dates', '2020-04-28 13:49:58'), ('file_name', 'Al Green.csv'), ('pct_true', 0.605857789516449)])
OrderedDict([('tweet', '.@realDonaldTrump, in the midst of the #COVID19 pandemic, America needs leadership now more than ever during your presidency. Refusing to work with and then firing the Inspector General just adds to your track record of dishonorable conduct. \n\nDo the honorable thing - RESIGN.'), ('dates', '2020-04-07 22:04:20'), ('file_name', 'Al Green.csv'), ('pct_true', 0.7487598061561584)])
OrderedDict([('tweet', 'Today, I condemn Texas Governor Greg Abbott’s failure to protect innocent lives. Watch live here:\n\nhttps://t.co/ptuadZB1kL'), ('dates', '2020-04-05 19:09:46'), ('file_name', 'Al Green.csv'), ('pct_true', 0.513530969619751)])
OrderedDict([('tweet', "An impeached president who's been exposed to #coronavirus, but does not get tested, while encouraging others to be tested for the virus is worse than a bad example. He is unfit and constantly shows us why he should have been removed from office by the Senate. #CoronaVirusUpdates"), ('dates', '2020-03-14 13:23:20'), ('file_name', 'Al Green.csv'), ('pct_true', 0.8520116806030273)])
OrderedDict([('tweet', 'Of all of the top banks - the so called ‘too big to fail’ banks, there has never been a CEO or top officer of any of these banks to go to jail. It seems that they are not only ‘too big to fail,’ they’re also too big to jail.'), ('dates', '2020-03-10 14:29:21'), ('file_name', 'Al Green.csv'), ('pct_true', 0.5652007460594177)])
OrderedDict([('tweet', "In a righteous world, Attorney General Barr would resign for corrupting the Justice Department. \nIn President Trump’s world, Mr. Barr is likely a candidate for the Medal of Freedom. \n\nWake up America! Don't be hoodwinked by Mr. Barr's latest comments. #BarrResign"), ('dates', '2020-02-14 14:23:09'), ('file_name', 'Al Green.csv'), ('pct_true', 0.8789929747581482)])
OrderedDict([('tweet', 'In a righteous world, Attorney General Barr would resign to avoid impeachment for corrupting the Justice Department. \nIn President Trump’s world, Mr. Barr is likely a candidate for the Medal of Freedom. \n\nWake up America, before it’s too late!!!'), ('dates', '2020-02-13 14:18:09'), ('file_name', 'Al Green.csv'), ('pct_true', 0.839581310749054)])
OrderedDict([('tweet', "The President's reprehensible behavior was on full display in last night's #SOTU speech. His dastardly words and deeds continue to divide our Nation. Today, Senators must hold this #ImpeachedPresident accountable or be held accountable on Election Day.\n\n#ImpeachmentDay"), ('dates', '2020-02-05 18:14:23'), ('file_name', 'Al Green.csv'), ('pct_true', 0.9762394428253174)])
OrderedDict([('tweet', 'Because of an impeached, reckless, ruthless, lawless, shameless, corrupt, &amp; unapologetically bigoted president - who is still engaging in a coverup, the state of the House, the state of the Senate, and the #StateOfTheUnion are divided. \n\nI will NOT attend #SOTU2020.'), ('dates', '2020-02-04 14:26:32'), ('file_name', 'Al Green.csv'), ('pct_true', 0.9866454005241394)])
OrderedDict([('tweet', 'It has become glaringly apparent that Mr. Bolton has information that could be essential to the fate of not only the Senate impeachment trial but also our Nation. Choosing not to subpoena him puts party above politics and this President above the law.\n\n#JohnBoltonMustTestify'), ('dates', '2020-01-28 13:41:23'), ('file_name', 'Al Green.csv'), ('pct_true', 0.7013936638832092)])
OrderedDict([('tweet', 'We need a Senate impeachment trial NOT a Senate impeachment briefing. \n\nThe people of this country will have the last word. \nhttps://t.co/gB5sw9juN5'), ('dates', '2020-01-15 16:31:51'), ('file_name', 'Al Green.csv'), ('pct_true', 0.757067859172821)])
OrderedDict([('tweet', 'The people deserve a fair not a fake impeachment trial in the Senate. The hands of history will not be kind to those who impose a fake trial, and the hands of voters will not forget them.'), ('dates', '2019-12-31 19:04:31'), ('file_name', 'Al Green.csv'), ('pct_true', 0.8857728242874146)])
OrderedDict([('tweet', 'We must not allow the gamesmanship of politics to overshadow the statesmanship of our moral imperative.'), ('dates', '2019-12-22 14:35:09'), ('file_name', 'Al Green.csv'), ('pct_true', 0.6522321105003357)])
OrderedDict([('tweet', 'The Chief Justice of the Supreme Court should not legitimize an impeachment trial where it is perceived that the Republican President and the Republican Senate Majority Leader are engaging in collusion and coverups. \n\n#McConnellTrumpCoverup'), ('dates', '2019-12-16 15:03:24'), ('file_name', 'Al Green.csv'), ('pct_true', 0.9031007289886475)])
OrderedDict([('tweet', "The question isn't whether the House will impeach, the question is whether the Senate will uphold its constitutional duty to convict and remove a constitutionally unfit president from office.\n\n#ImpeachAndConvictTrump"), ('dates', '2019-12-11 13:47:06'), ('file_name', 'Al Green.csv'), ('pct_true', 0.9813298583030701)])
OrderedDict([('tweet', 'Those who discredit the #impeachment of Andrew Johnson in 1868 discredit impeachment rooted in racism. \n\n#Impeach President Trump for his racism, anti-Semitism, Islamophobia, transphobia, and xenophobia.'), ('dates', '2019-12-05 13:14:10'), ('file_name', 'Al Green.csv'), ('pct_true', 0.9248895645141602)])
OrderedDict([('tweet', 'Mr. President, @SpeakerPelosi isn’t crazy. We are not trying to change the voting system. We are defending the democracy from your corruption and coverups, bigotry and bribery. You are unfit and you will be impeached. #impeachment https://t.co/eULlBZd9Eo'), ('dates', '2019-11-19 16:30:32'), ('file_name', 'Al Green.csv'), ('pct_true', 0.8213923573493958)])
OrderedDict([('tweet', 'Mr. President, your partner in shame Roger Stone has been found #guilty for his attempts to prevent a House investigation into Russian interference in the 2016 election. \n\nIf you pardon him, it will only increase the likelihood of the inevitable - your impeachment.\n\n#ImpeachNow'), ('dates', '2019-11-15 18:53:13'), ('file_name', 'Al Green.csv'), ('pct_true', 0.7379946708679199)])
OrderedDict([('tweet', 'We may be #socialdistancing, but that does not stop Congress from legislating. For the economic relief package, I advocated on behalf of #smallbusinesses. Through these efforts, we secured $375 billion to ensure small businesses continue to thrive &amp; serve our local communities. https://t.co/AKVXjqzjPq'), ('dates', '2020-03-31 20:51:18'), ('file_name', 'Al Lawson, Jr..csv'), ('pct_true', 0.5008923411369324)])
OrderedDict([('tweet', 'The president mentioned how the “strong economy is lifting everyone up.” But who is really benefiting? It is not a “#BlueCollarBoom,” but a white collar boom. We bailed out Wall Street, but we still haven’t bailed out Main Street. All Americans deserve a livable\xa0wage. #SOTU https://t.co/AHgAz07oje'), ('dates', '2020-02-05 04:48:23'), ('file_name', 'Al Lawson, Jr..csv'), ('pct_true', 0.8397634625434875)])
OrderedDict([('tweet', 'President Trump again repeated that he will try to destroy Obamacare in the middle of the worst public health crisis in a century. This position is totally divorced from reality. It’s profoundly dangerous. https://t.co/UwQW4bQW3o'), ('dates', '2020-05-09 17:06:00'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.5031145811080933)])
OrderedDict([('tweet', 'Without aid to states and localities, millions could lose Medicaid coverage in the middle of a pandemic. Countless others who have lost their jobs may be left ineligible. We can’t let that happen. https://t.co/tXjfe0eWNu'), ('dates', '2020-05-07 16:40:26'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.6561061143875122)])
OrderedDict([('tweet', 'Workers are losing their jobs—and with it, their health insurance. This is profoundly dangerous. We need to think bigger to respond to this crisis. I’m proud to support the Medicare Crisis Program Act to cover unemployed workers, expand Medicaid and limit costs for everyone.'), ('dates', '2020-05-02 19:41:00'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.6526986956596375)])
OrderedDict([('tweet', 'The President is trying to hold a congressionally authorized loan for the USPS hostage for his own political purposes. This is outrageous. The USPS’s critical work is already endangered by this crisis—they need resources immediately. https://t.co/yoBvKTiXQB'), ('dates', '2020-04-27 15:02:00'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.6607774496078491)])
OrderedDict([('tweet', 'This disgusting bigotry has no place in our government. Michael Caputo should resign immediately. https://t.co/LqaYGlkUsP'), ('dates', '2020-04-24 13:23:30'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.7653000950813293)])
OrderedDict([('tweet', 'Xenophobia won’t save us. Immigrants aren’t to blame for coronavirus. It’s dangerous to try to implement sweeping policies in late-night tweets. And, this is an affront to the immigrant healthcare providers on the front lines of this crisis. https://t.co/3WC9YKu8Gi'), ('dates', '2020-04-22 15:21:00'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.7681087851524353)])
OrderedDict([('tweet', 'We will not allow a bailout of the biggest polluters. Hospitals are still desperate for PPE. Testing is inadequate. Small businesses are still waiting on relief. This is unacceptable. https://t.co/zOZgT3XCQi'), ('dates', '2020-04-22 13:00:31'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.59527188539505)])
OrderedDict([('tweet', 'The US Congress spoke clearly when we passed H. Res. 326. We oppose unilateral annexation of any West Bank territory as an unacceptable threat to peace and security. Only a two state solution can resolve the Israeli-Palestinian conflict.'), ('dates', '2020-04-20 22:31:00'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.5402138233184814)])
OrderedDict([('tweet', 'We must understand: there’s no way to sustainably reopen the economy without infrastructure to track and control the spread of the disease. If we lift restrictions too soon, all our efforts would be for nothing. Many would die, lockdowns would return. We need careful strategies.'), ('dates', '2020-04-17 18:31:00'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.5296822786331177)])
OrderedDict([('tweet', 'I’m glad the administration reconsidered their disastrous plan to make elderly Americans file tax forms in order to receive their CARES act funding—but they must do more to get payments out immediately. The American people are hurting.'), ('dates', '2020-04-04 20:01:00'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.6153985857963562)])
OrderedDict([('tweet', 'This is no time for the administration to move ahead with radical measures to weaken emissions standards. The threat of climate change remains as real as ever. This rule change is bad for manufacturers, consumers, and the public. https://t.co/rnv9mbgZTZ'), ('dates', '2020-04-01 17:27:00'), ('file_name', 'Alan S. Lowenthal.csv'), ('pct_true', 0.5400851964950562)])
OrderedDict([('tweet', '#Coronavirus affects everyone, but it does not affect everyone equally. Latinos have the highest uninsured rate of any racial or ethnic group in the US &amp; suffer from a persistent gap in health care quality &amp; access.'), ('dates', '2020-04-28 19:13:42'), ('file_name', 'Albio Sires.csv'), ('pct_true', 0.5802577137947083)])
OrderedDict([('tweet', 'Yesterday, Senate Majority Leader McConnell called for states to turn to bankruptcy rather than the federal government as they face the economic impact of the COVID-19 pandemic. This is an insult to those who are working to combat the virus and its financial toll.'), ('dates', '2020-04-23 16:29:35'), ('file_name', 'Albio Sires.csv'), ('pct_true', 0.8666121363639832)])
OrderedDict([('tweet', 'The CDC is awarding New Jersey an additional $5.6 million to support the state’s response to the COVID-19 pandemic. This funding will supplement an existing cooperative agreement.'), ('dates', '2020-04-07 13:58:48'), ('file_name', 'Albio Sires.csv'), ('pct_true', 0.6296770572662354)])
OrderedDict([('tweet', '.@BetsyDeVosED fails our students &amp; sexual assault survivors by changing #TitleIX protections to now benefit the accused &amp; enablers. This is an unconscionable reversal of decades of work to protect survivors &amp; end sexual harassment, misconduct, &amp; assault on campuses. #HandsOffIX https://t.co/mzs9UUPca9'), ('dates', '2020-05-07 17:30:50'), ('file_name', 'Alcee L. Hastings.csv'), ('pct_true', 0.5016043186187744)])
OrderedDict([('tweet', 'This #WorkersMemorialDay, we find ourselves in the midst of a pandemic where far too many workers are falling ill &amp; dying from #COVID19. Yet, @OSHA_DOL is still refusing to issue an emergency standard to keep our workers safe. We must recommit to workplace safety. #WhereIsOSHA? https://t.co/v1kS85lPk8'), ('dates', '2020-04-29 00:00:09'), ('file_name', 'Alcee L. Hastings.csv'), ('pct_true', 0.5571920871734619)])
OrderedDict([('tweet', "It’s been 10 years since the devastating BP #DeepwaterHorizon oil spill. @POTUS isn’t protecting our oceans, the Americans who live on our coasts, or the wildlife who need clean environments to live. We must #ProtectOurCoast, not Big Oil's profits at the cost of human life. https://t.co/ZcTv4Ks5Yt"), ('dates', '2020-04-20 19:08:16'), ('file_name', 'Alcee L. Hastings.csv'), ('pct_true', 0.8074461221694946)])
OrderedDict([('tweet', 'Small farms are small businesses &amp; they are feeding our nation. They deserve access to the same programs as other Main Street firms. @senatemajldr’s bill leaves them out in the cold. @housedemocrats are working to ensure they are eligible for #SmallBizRelief! https://t.co/0c9BlzmCUb'), ('dates', '2020-04-15 15:27:00'), ('file_name', 'Alcee L. Hastings.csv'), ('pct_true', 0.5596235990524292)])
OrderedDict([('tweet', '.@naehomelessness has proven that homeless Americans are especially vulnerable to #COVIDー19. @RepEBJ &amp; I led 91 Members in calling for funding to protect homeless Americans in the next COVID-19 aid bill. We have a responsibility to remember our most vulnerable in times of crisis https://t.co/h99X2NSjpm'), ('dates', '2020-04-11 01:43:46'), ('file_name', 'Alcee L. Hastings.csv'), ('pct_true', 0.527208685874939)])
OrderedDict([('tweet', 'The Little Sisters of the Poor today are arguing their case for religious liberty before #SCOTUS. I’m proud they continue to stand resilient in the fight for religious freedom against unconstitutional government mandates. #LetThemServe'), ('dates', '2020-05-06 15:19:41'), ('file_name', 'Alexander X. Mooney.csv'), ('pct_true', 0.5518935322761536)])
OrderedDict([('tweet', "The world should pay attention to the illegal actions taken by Russia to take greater control in the sovereign nation of Georgia during the coronavirus pandemic. Continued aggression by Russia has violated Georgia's border in the South Ossetia region. https://t.co/gOA2qPr65u"), ('dates', '2020-04-24 16:20:05'), ('file_name', 'Alexander X. Mooney.csv'), ('pct_true', 0.5059731602668762)])
OrderedDict([('tweet', "I spoke w/ @HoppyKercheval from Congress this morning.\n\nI'm here to vote for today's bill that will enable small businesses to keep their employees on payroll.\n\nThe #PaycheckProtectionProgram is working. Businesses have been shut down by the gov't through no fault of their own. https://t.co/pK6a3V6kUl"), ('dates', '2020-04-23 15:06:27'), ('file_name', 'Alexander X. Mooney.csv'), ('pct_true', 0.532139778137207)])
OrderedDict([('tweet', "For misrepresenting &amp; suppressing information related to #COVID19, China must submit itself to a fair trial in the Int'l Court of Justice.\n\nMy colleagues &amp; I urge @StateDept &amp; @TheJusticeDept to hold #CCP accountable for violating Int'l Health Regulations.\nhttps://t.co/D8EOUMyaJD"), ('dates', '2020-04-21 13:44:07'), ('file_name', 'Alexander X. Mooney.csv'), ('pct_true', 0.5783726572990417)])
OrderedDict([('tweet', '“The longer Democrats refuse to provide financing for small businesses after government cut off their revenue, the more Americans will have every right to conclude that Mrs. Pelosi &amp; Mr. Schumer are responsible for the worsening economic destruction.” @WSJ https://t.co/x5ec9WJMyx'), ('dates', '2020-04-17 16:20:12'), ('file_name', 'Alexander X. Mooney.csv'), ('pct_true', 0.9750983119010925)])
OrderedDict([('tweet', '.@stevenmnuchin1 &amp; @SBAJovita urging Congress to approve more #PPP funding:\n\n“The high demand we have seen underscores the need for hardworking Americans to have access to relief as soon as possible."\n\nDemocrats have turned their backs on small businesses &amp; Americans nationwide. https://t.co/cL9beGr5Mu'), ('dates', '2020-04-16 17:38:50'), ('file_name', 'Alexander X. Mooney.csv'), ('pct_true', 0.6335233449935913)])
OrderedDict([('tweet', 'Democrats continue to block additional funding for our small businesses that have been required to close through no fault of their own. If you are unable to apply for #PPPloans today, please share your story here 🚨⬇️ https://t.co/Wbkt5ry0xW https://t.co/VfTuddUzoH'), ('dates', '2020-04-16 16:02:39'), ('file_name', 'Alexander X. Mooney.csv'), ('pct_true', 0.733394205570221)])
OrderedDict([('tweet', 'President Trump is engaging in arms deals with Modi while his administration is ethnically cleansing the country’s religious minorities. We must not enable this rise in sectarian violence. https://t.co/WwUyf9cNCN'), ('dates', '2020-03-02 21:38:25'), ('file_name', 'Alexandria Ocasio-Cortez.csv'), ('pct_true', 0.7244254946708679)])
OrderedDict([('tweet', 'Over 800,000 #Dreamers have benefited from #DACA, but the Trump administration has left the future of the program unclear. If your status expires before 2020 you should renew now! #HomeIsHere https://t.co/e4fay4WKbM'), ('dates', '2020-02-28 19:43:27'), ('file_name', 'Alexandria Ocasio-Cortez.csv'), ('pct_true', 0.6450093984603882)])
OrderedDict([('tweet', 'It’s a disgrace that the Trump Admin is treating migrants &amp; asylum seekers like this -  1 year+ of #RemainInMexico has proven dangerous, and in some cases deadly. We must end this horrific policy. #DontLookAway\nhttps://t.co/wq5Q1wY13f'), ('dates', '2020-02-27 14:33:23'), ('file_name', 'Alexandria Ocasio-Cortez.csv'), ('pct_true', 0.7582195997238159)])
OrderedDict([('tweet', "Punishing the poor does not create a safer environment. Instead it threatens the very foundation of our community. That is why my New York colleagues and I wrote a letter to @NYGovCuomo asking him to help put an end to MTA's dangerous policing policy. Check it out here: https://t.co/g1AOWHA7Lh"), ('dates', '2019-12-17 17:42:29'), ('file_name', 'Alexandria Ocasio-Cortez.csv'), ('pct_true', 0.7263956069946289)])
OrderedDict([('tweet', "I agree. This is not the time for half-measures. Half-measures threaten the fate of our planet and the future of our kids and grandchildren. The Green New Deal for Public Housing Act is not radical. Radical is keeping us on track to climate catastrophe. Let's pass this bill! https://t.co/ANmDLaoQQy"), ('dates', '2019-11-14 18:48:19'), ('file_name', 'Alexandria Ocasio-Cortez.csv'), ('pct_true', 0.6129524111747742)])
OrderedDict([('tweet', '"We the public, we the people, developed this drug, we paid for this drug. There\'s no reason this should be $2,000 a month. People are dying because of it.” - AOC\n\nhttps://t.co/I4kuObbLqM'), ('dates', '2019-05-17 00:54:16'), ('file_name', 'Alexandria Ocasio-Cortez.csv'), ('pct_true', 0.631993293762207)])
OrderedDict([('tweet', 'Meat processing plants should not reopen without unprecedented protections and safeguards for workers and livestock. On #WorkersMemorialDay, POTUS shouldn’t decide which workers will be safe and which workers will be in mortal danger. https://t.co/rXUPkIx7p8'), ('dates', '2020-04-28 20:39:58'), ('file_name', 'Alma S. Adams.csv'), ('pct_true', 0.6171040534973145)])
OrderedDict([('tweet', 'On #WorkersMemorialDay, we remember countless workers who lost their lives braving unsafe and unhealthy conditions. As we face this worldwide crisis that threatens frontline workers, know this: if a worker is essential, then their safety is essential. https://t.co/Rd5IwVXrpa'), ('dates', '2020-04-28 13:10:04'), ('file_name', 'Alma S. Adams.csv'), ('pct_true', 0.5069792866706848)])
OrderedDict([('tweet', 'Man-made climate change is an existential threat that puts our lives at risk, and those of all future generations. This #EarthDay, we should recommit to addressing the climate crisis with all deliberate speed.'), ('dates', '2020-04-22 13:03:52'), ('file_name', 'Alma S. Adams.csv'), ('pct_true', 0.6636999249458313)])
OrderedDict([('tweet', 'Thousands of Americans are dying everyday and the President is trying to shoo away the doctors and scientists who are fighting to keep us safe.\n\nPlease make it make sense. https://t.co/VIkNDz1Oqb'), ('dates', '2020-05-06 02:51:32'), ('file_name', 'Ami Bera.csv'), ('pct_true', 0.6506562829017639)])
OrderedDict([('tweet', "#COVID19 isn't a Republican or Democratic virus. It doesn't recognize the color of your skin, the country of your origin, or the god you worship. \n\nWe're all in this fight together. https://t.co/0tkgUgU5Ar"), ('dates', '2020-04-23 20:16:45'), ('file_name', 'Ami Bera.csv'), ('pct_true', 0.5568627119064331)])
OrderedDict([('tweet', "We're months into this pandemic and we still have no national testing strategy needed to safely reopen the country. https://t.co/eo9qlArFRQ"), ('dates', '2020-04-20 18:26:49'), ('file_name', 'Ami Bera.csv'), ('pct_true', 0.5287377834320068)])
OrderedDict([('tweet', 'Americans are facing unprecedented disruptions to their daily lives and we need to make sure that in the midst of this pandemic, Americans don’t also lose their ability to vote.-AK @RonWyden #coronavirus #COVID19 https://t.co/av8VIVveoS'), ('dates', '2020-03-14 00:48:39'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.6394031643867493)])
OrderedDict([('tweet', 'We have a major #monopoly problem in this country, which harms consumers and threatens free and fair competition across our economy. Companies need to be put on notice that exclusionary behavior that threatens #competition cannot continue.-AK #antitrust https://t.co/sOk7laEF7K'), ('dates', '2020-03-10 15:57:42'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.9349250793457031)])
OrderedDict([('tweet', 'It is wrong to take money from political campaigns in exchange for disseminating blatant lies to the American people. It is also wrong that @Facebook is immune from any liability for the reckless political ads they sell.-AK https://t.co/bvhCvTRxiQ'), ('dates', '2020-01-09 19:19:15'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.7422585487365723)])
OrderedDict([('tweet', 'The Admin caused this prob by making exemptions for big oil companies,&amp; its failure to uphold the RFS has already led to the closure or idling of more than 35 ethanol &amp; biodiesel plants, leaving #rural America further behind. Enough is enough.-AK #BioFuels https://t.co/xKZvGDay3q'), ('dates', '2019-12-19 16:25:32'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.8098776340484619)])
OrderedDict([('tweet', 'Senators Klobuchar and @SenTinaSmith are seeking answers from @HUDgov after five Minnesotans were killed and four were injured in a tragic fire in a high-rise public housing building. https://t.co/S0jhD3ztmg'), ('dates', '2019-12-13 19:26:33'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.5877350568771362)])
OrderedDict([('tweet', "At a Sen Judiciary Committee hearing w/@JusticeOIG Horowitz yesterday, Sen Klobuchar highlighted conclusive evidence in the IG's report that political bias did not affect the FBI’s decision to investigate ppl associated w/the Trump campaign &amp; that the investigation was warranted. https://t.co/yWYW6LBDrW"), ('dates', '2019-12-12 21:30:39'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.6244627833366394)])
OrderedDict([('tweet', 'Americans have a right to know who is paying to influence our democracy. Rolling back transparency standards is wrong &amp; it undermines our political system, which rests on the idea that each voter has an equal say in our democracy.-AK #Disclosure #DarkMoney https://t.co/DQ6mI3k2D5'), ('dates', '2019-12-12 17:13:34'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.8177815675735474)])
OrderedDict([('tweet', 'Today, the Trump Administration notified the U.N. that the U.S. is withdrawing from the Paris climate agreement, which Sen. Klobuchar has opposed. She joined @SenatorCardin in introducing a resolution calling for the U.S. to remain in the pact. https://t.co/s0SAAkrByn'), ('dates', '2019-11-04 22:15:57'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.665542483329773)])
OrderedDict([('tweet', 'Sen Klobuchar led a group of senators in urging @SecPompeo and @SecElaineChao to reverse the Trump Administration’s decision to prohibit U.S. commercial air travel from the United States to Cuban cities other than Havana. #Cuba @StateDept https://t.co/51nj0f6nca'), ('dates', '2019-11-01 15:52:08'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.5895217061042786)])
OrderedDict([('tweet', 'We need to update our laws so that we have more people in public office who understand the daily realities of taking care of family and working a regular job. That’s why I have introduced the Help America Run Act.-AK #CampaignFinance https://t.co/emVfxErWId'), ('dates', '2019-10-30 18:20:13'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.5010549426078796)])
OrderedDict([('tweet', 'This is a fix for a prob. that the WH caused, &amp; it comes too late for rural communities that have been hurt. The real solution is to stop the President from giving exemptions to his friends in big oil &amp;cease issuing further improper refinery exemptions.-AK https://t.co/342BqYem6o'), ('dates', '2019-10-04 20:06:47'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.708856999874115)])
OrderedDict([('tweet', 'Farmers &amp;rural communities are struggling w/low prices, unstable weather &amp;uncertainty w/export markets. Yet in Aug, the EPA granted another 31 waivers, bringing the total issued by the Trump Admin to 85 &amp; reducing demand for over 4 bn. gal. of biofuels.-AK https://t.co/HInXKUqYV2'), ('dates', '2019-10-03 16:06:11'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.6040295362472534)])
OrderedDict([('tweet', 'The record of this call is deeply disturbing—the President solicited a foreign leader to investigate a political rival and his family, prioritizing his personal gain over the national security interests of the United States. The President has violated the public’s trust.'), ('dates', '2019-09-25 16:09:55'), ('file_name', 'Amy Klobuchar.csv'), ('pct_true', 0.5246387720108032)])
OrderedDict([('tweet', 'President Trump claims the #COVID19 pandemic is improving and we can begin to reopen. Yet he also claims it’s so bad we must close our nation’s doors to all immigrants? His handling of this crisis isn’t based in logic. It’s based in his own reckless, dangerous agenda.'), ('dates', '2020-04-21 13:39:57'), ('file_name', 'André Carson.csv'), ('pct_true', 0.6477296948432922)])
OrderedDict([('tweet', "Let's set the record straight: Dems aren't blocking #PPPloans for small businesses. We are fighting to make sure these funds get into the hands of small business owners being shut out by big lenders. GOP: Let's work together to get this relief to the folks who really need it."), ('dates', '2020-04-16 16:32:12'), ('file_name', 'André Carson.csv'), ('pct_true', 0.644582986831665)])
OrderedDict([('tweet', 'The student loan debt crisis is out of control, and the economic devastation of #COVID19 makes that even clearer. That\'s why I have cosponsored the “Student Debt Emergency Relief Act," which cancels at least $30K in loan debt per borrower.'), ('dates', '2020-03-23 18:02:55'), ('file_name', 'André Carson.csv'), ('pct_true', 0.5307308435440063)])
OrderedDict([('tweet', 'Instead of downplaying #COVID19 to protect his Wall Street buddies, President Trump should be supporting legislation to guarantee that all employees have paid sick leave. Nobody should be forced to make the choice between getting sick and paying the bills.'), ('dates', '2020-03-09 15:55:26'), ('file_name', 'André Carson.csv'), ('pct_true', 0.5508041977882385)])
OrderedDict([('tweet', 'A reminder that if Republicans succeed in repealing the #ACA:\n\n- Nearly 500,000 Hoosiers would lose their health insurance\n\n- 2.7 million Hoosiers with preexisting conditions could once again be denied coverage\n\n- Indiana loses $3 billion in health care funding\n\n#ProtectOurCare https://t.co/ZSbTcWcCaL'), ('dates', '2020-03-02 19:44:31'), ('file_name', 'André Carson.csv'), ('pct_true', 0.6905345320701599)])
OrderedDict([('tweet', "A year ago today, the House passed #HR8, the Bipartisan Background Checks Act, but @senatemajldr has refused to put this smart #gunsafety measure up for a vote in the Senate. In that time, 36,000 Americans have died from #gunviolence. Retweet if you think it's time for a vote!"), ('dates', '2020-02-27 20:55:24'), ('file_name', 'André Carson.csv'), ('pct_true', 0.7356276512145996)])
OrderedDict([('tweet', 'Meanwhile, the President claims the virus is "under control" while also planning to cut @CDCgov\'s budget by nearly 20%.\n\nThis administration is dangerously unprepared to handle a potential #CoronavirusOutbreak, and the American people will pay the price. https://t.co/QLVrdhEqRZ'), ('dates', '2020-02-25 21:30:55'), ('file_name', 'André Carson.csv'), ('pct_true', 0.7423593997955322)])
OrderedDict([('tweet', 'What do we have to lose under President Trump? Security, stability, and much more. \n\nHis latest budget proposal cuts #SNAP, #Medicaid, #SocialSecurity benefits, and loan forgiveness programs -- just to name a few. Under the #TrumpBudget, working families everywhere will suffer.'), ('dates', '2020-02-12 21:26:25'), ('file_name', 'André Carson.csv'), ('pct_true', 0.7553411722183228)])
OrderedDict([('tweet', 'President Trump may have claimed he will protect programs like #SocialSecurity and #Medicare, but his actions speak louder than words. His latest budget proposal, which guts  these lifelines, shows he is not being honest with Americans. https://t.co/Ah5IUyxJU2'), ('dates', '2020-02-11 18:21:21'), ('file_name', 'André Carson.csv'), ('pct_true', 0.7035419344902039)])
OrderedDict([('tweet', "We must do much more to respond to the challenge of China. In our modern world, data is money -- &amp; power. Allowing other nations to steal Americans' private info is trading away our liberty, &amp; giving them an advantage in emerging technology, such as #ArtificialInteligence. https://t.co/NiC7XVPpYM"), ('dates', '2020-02-10 19:07:59'), ('file_name', 'André Carson.csv'), ('pct_true', 0.7559828758239746)])
OrderedDict([('tweet', "An acquittal means very little if the trial was rigged. \n\nDespite today's outcome in the Senate, the facts that we uncovered in the House remain the same. This President betrayed the American people and violated his oath of office. \n\nAmericans won't soon forget that."), ('dates', '2020-02-05 22:53:38'), ('file_name', 'André Carson.csv'), ('pct_true', 0.9477438926696777)])
OrderedDict([('tweet', 'President Trump was simply not being honest with the American people in his #SOTU address. His pledge to protect Americans’ health care, and vital lifelines like Medicare and Social Security, don’t line up with his record. We’re headed down the wrong path under his presidency.'), ('dates', '2020-02-05 03:31:41'), ('file_name', 'André Carson.csv'), ('pct_true', 0.7901552319526672)])
OrderedDict([('tweet', '(1/2) While China was downplaying the virus and human-to-human transmission, they were in the market vacuuming up 2 billion masks even though they already manufacture half of the world medical masks. Congress needs to investigate China over COVID-19.\xa0 https://t.co/2l1SbtKGQ1'), ('dates', '2020-04-25 20:44:27'), ('file_name', 'Andy Barr.csv'), ('pct_true', 0.9166727662086487)])
OrderedDict([('tweet', 'It is a travesty that the Paycheck Protection Program #PPP has run out of funding. The PPP is saving millions of jobs and helping thousands of American small businesses stay open since its rollout. This outcome was completely and absolutely avoidable. https://t.co/sHrkV3yKu3'), ('dates', '2020-04-16 16:08:39'), ('file_name', 'Andy Barr.csv'), ('pct_true', 0.6026636362075806)])
OrderedDict([('tweet', 'The Paycheck Protection Program is a critical lifeline to our small businesses. Sadly, Democrats blocked legislation to continue funding the PPP yesterday. We need to come together and fund the PPP to ensure that no small business is left behind.  https://t.co/GkvgJj6kSm'), ('dates', '2020-04-10 19:59:21'), ('file_name', 'Andy Barr.csv'), ('pct_true', 0.7664851546287537)])
OrderedDict([('tweet', 'I represent people who are sick, exhausted health care workers, workers without paychecks, frightened and lonely seniors who need help, and small businesses whose dreams have been crushed. And this is what the Democrat majority offers us as a solution? https://t.co/9tp37obKtG'), ('dates', '2020-03-24 02:33:03'), ('file_name', 'Andy Barr.csv'), ('pct_true', 0.8526380658149719)])
OrderedDict([('tweet', 'Speaker Pelosi just released her own bill - an 1,100 page left-wing wish list of socialist policies that have nothing to do with immediate crisis and current economic shutdown.'), ('dates', '2020-03-23 16:16:46'), ('file_name', 'Andy Barr.csv'), ('pct_true', 0.8740155696868896)])
OrderedDict([('tweet', 'Speaker Pelosi has turned this institution into an oligarchy by keeping the House at bay, allowing her to negotiate whatever she wants in these spending packages.\n\nIf you like a representative republic, Speaker Pelosi’s actions have been disastrous. https://t.co/l9dF7G7Cgn'), ('dates', '2020-05-06 20:47:47'), ('file_name', 'Andy Biggs.csv'), ('pct_true', 0.6333770751953125)])
OrderedDict([('tweet', 'This is a national security threat. Congress must take immediate action to bring our spending under control and eliminate our debt. https://t.co/ex0jPziJsw'), ('dates', '2020-05-06 20:40:26'), ('file_name', 'Andy Biggs.csv'), ('pct_true', 0.6225804686546326)])
OrderedDict([('tweet', 'The pro-growth policies that President @realDonaldTrump championed to help our economy prosper will assist him to lead the GREAT AMERICAN COMEBACK.\n\nRegulatory reforms, tax cuts, and Americans as the captains of their own economic destinies will allow for tremendous success.'), ('dates', '2020-05-05 05:40:38'), ('file_name', 'Andy Biggs.csv'), ('pct_true', 0.7466486692428589)])
OrderedDict([('tweet', 'Good news! The Administration has announced that Social Security recipients do not need to take any action or file a tax return to receive their stimulus check. IRS officials will use information on existing 1099 forms (the forms that are normally mailed every year'), ('dates', '2020-04-02 14:53:49'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.6798455119132996)])
OrderedDict([('tweet', 'This is really unbelievable. It’s on page 803 of the Speaker’s bill.  If Speaker Pelosi destroys the economy, she won’t have to worry about getting the airline emissions part of the Green New Deal into the COVID-19 bill. That’s why we need #singleissuebills https://t.co/wBjYAOdX4I'), ('dates', '2020-03-23 22:34:42'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.6396205425262451)])
OrderedDict([('tweet', 'What does nullifying the President’s exec order on federal employee union issues have to do with COVID-19?  We have small businesses going out of business, and this is a priority for the coronavirus crisis? It’s on page 737 of Speaker Pelosi’s bill! #singleissuebills #skipthepork https://t.co/9diWCMyZVu'), ('dates', '2020-03-23 21:41:16'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.5600728988647461)])
OrderedDict([('tweet', 'I still have serious concerns with the Senate bailout bill being considered-and now its possible that if the bill comes to the House there may not even be debate on a $2 trillion package!  Not the way to do the nation’s business. Why not break up such a large bill? #skipthepork'), ('dates', '2020-03-23 00:40:58'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.6061453819274902)])
OrderedDict([('tweet', 'My parents fled communist Eastern Europe. Their experience, and the suffering and repression of millions like them, are why I never want to see America become a socialist country. Hear my chat with @CongressmanHice about socialism on the Freedom Caucus Podcast. LIBERTY&gt;SOCIALISM https://t.co/ij0vQYBhmj'), ('dates', '2020-03-10 01:48:10'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.6498127579689026)])
OrderedDict([('tweet', "After Soviet-style secret proceedings, Judiciary hearings where the rights of the minority weren’t recognized, and after hearing from thousands of my constituents, it remains clear to me that Democrats' impeachment of President Trump is a partisan sham! https://t.co/6hlt0NPC36"), ('dates', '2019-12-18 23:59:45'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.5993064045906067)])
OrderedDict([('tweet', 'This issue is bipartisan: newborns who survive an abortion deserve lifesaving treatment.\n\nIt’s time for Nancy Pelosi and the House majority to #LetUsVote #ProtectLife #EndInfanticide https://t.co/pPIGA19eLv'), ('dates', '2019-09-10 18:08:38'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.6300898790359497)])
OrderedDict([('tweet', 'This is why sanctuary cities must end! @realdonaldtrump is right, sanctuary cities are dangerous to our safety and security. These MS13 members were released by Prince George’s County after ICE placed a detainer on them. https://t.co/DQNXZflnNF'), ('dates', '2019-05-21 21:39:35'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.8973937630653381)])
OrderedDict([('tweet', "The House is holding its first ever hearing today on #MedicareForAll, which we all know provides Medicare-for-None, and it looks like this:\n-New taxes costing trillions\n-Losing your private insurance (even if you like it)\n-Long waits &amp; rationed care from doctors you didn't choose"), ('dates', '2019-04-30 14:40:34'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.6381622552871704)])
OrderedDict([('tweet', 'Allowing criminals like the Boston bomber to vote? Creating executive orders to confiscate guns — tearing away our Second Amendment rights? Is this really the future that the left wants? Because it’s certainly un-American. Let’s get back on the right track.'), ('dates', '2019-04-24 18:51:44'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.6565234065055847)])
OrderedDict([('tweet', 'Because of the majority’s inability to pass a budget, they attempted to sneak in a major spending increase through a rule today. However, I voted against the majority’s attempt to increase discretionary spending levels to an outrageous $1.3 trillion.'), ('dates', '2019-04-09 22:08:09'), ('file_name', 'Andy Harris.csv'), ('pct_true', 0.6395056843757629)])
OrderedDict([('tweet', 'The PPP has had serious problems from day one, and the refusal of the SBA to fix them has hurt a lot of small business owners and the employees that depend on them. https://t.co/ViwrBhJvXF'), ('dates', '2020-05-09 15:16:23'), ('file_name', 'Andy Kim.csv'), ('pct_true', 0.6598373055458069)])
OrderedDict([('tweet', "We passed money to help small businesses - but too often that money is going to big corporations that don't need it, or has been held back by bureaucratic rules. We need to make sure our small businesses get what they need, when they need it. https://t.co/yCUBo57LQL"), ('dates', '2020-05-01 12:47:14'), ('file_name', 'Andy Kim.csv'), ('pct_true', 0.844768762588501)])
OrderedDict([('tweet', "The money we'll vote to spend tomorrow isn't Congress' money, it's the people's money; and we need to make sure it's going to the people and main street small businesses who need it most. https://t.co/XjTfljQheB"), ('dates', '2020-04-22 22:30:52'), ('file_name', 'Andy Kim.csv'), ('pct_true', 0.5642434358596802)])
OrderedDict([('tweet', "We can't end this public health crisis and fully address the financial crisis our families are facing until we get more testing. Our counties are working hard, but we need more help from @fema right now. https://t.co/WU25jZvzCg"), ('dates', '2020-04-09 15:38:45'), ('file_name', 'Andy Kim.csv'), ('pct_true', 0.5269457101821899)])
OrderedDict([('tweet', 'These numbers are absolutely shocking, but we should remember what each one represents: a neighbor who now has to worry about paying bills without a job while staying safe, a small business owner who might not be able to reopen after the pandemic is over. https://t.co/6vFto9Tk88'), ('dates', '2020-04-02 14:18:14'), ('file_name', 'Andy Kim.csv'), ('pct_true', 0.5217211842536926)])
OrderedDict([('tweet', "It's still not enough. We still don't have enough tests, masks and gloves. New Jersey can't open for business until we solve this crisis, and we can't solve this crisis until we have the equipment we need. https://t.co/7u2aYQ2g3M"), ('dates', '2020-03-31 02:24:43'), ('file_name', 'Andy Kim.csv'), ('pct_true', 0.664682149887085)])
OrderedDict([('tweet', "We must harness the same imagination FDR demonstrated a century ago. Let’s put many tens of thousands of unemployed folks to work contact tracing and helping people isolate safely. That's what @SenWarren's &amp; my Coronavirus Containment Corps is all about. https://t.co/LZhCkInvzH"), ('dates', '2020-05-08 17:10:10'), ('file_name', 'Andy Levin.csv'), ('pct_true', 0.6118466258049011)])
OrderedDict([('tweet', 'The coronavirus pandemic has exposed the problems of structural racism, of not having national health insurance, of disinvestment in public health systems. We must meet this moment by turning a corner towards a more progressive and inclusive vision for America.'), ('dates', '2020-04-17 14:41:20'), ('file_name', 'Andy Levin.csv'), ('pct_true', 0.658001184463501)])
OrderedDict([('tweet', 'Last night, the president did what he does best in a crisis: blame somebody else. Cutting off funding to the @WHO does not erase the president’s failure to respond quickly and effectively to the #COVID19 pandemic, nor does it undo his lies that have endangered Americans’ health.'), ('dates', '2020-04-15 18:08:59'), ('file_name', 'Andy Levin.csv'), ('pct_true', 0.888518214225769)])
OrderedDict([('tweet', 'I have been calling repeatedly on the president to use the Defense Production Act to address the COVID-19 pandemic. However, the #DefenseProductionAct must be used as a tool to work with the private sector—not as a cudgel against individual companies.'), ('dates', '2020-03-27 22:44:59'), ('file_name', 'Andy Levin.csv'), ('pct_true', 0.7610463500022888)])
OrderedDict([('tweet', ".@SavannahGuthrie asked @Surgeon_General why not invoke the Defense Production Act now &amp; ensure we have the supplies we need to fight #COVID19? I agree. That's why dozens of my colleagues &amp; I called on @realDonaldTrump to do this, before it's too late. https://t.co/LIsRekwhFD"), ('dates', '2020-03-18 13:33:53'), ('file_name', 'Andy Levin.csv'), ('pct_true', 0.5226413011550903)])
OrderedDict([('tweet', 'We have our first #COVID19 case in a senior living center in Oakland County. The pandemic has transformed the Admin’s effort to relax nursing home oversight from a public health menace to downright life-threatening. They must reverse course immediately.\nhttps://t.co/BtNS8w6oN7'), ('dates', '2020-03-17 16:20:00'), ('file_name', 'Andy Levin.csv'), ('pct_true', 0.5573810935020447)])
OrderedDict([('tweet', 'A year ago yesterday, @HouseDemocrats made history by passing #HR1, the #ForThePeople Act – a transformative anti-corruption and clean elections bill. But to maintain the status quo in Washington, @senatemajldr continues to block the bill. It’s time for a Senate vote on #HR1! https://t.co/ZhS7F8iB0n'), ('dates', '2020-03-09 18:16:31'), ('file_name', 'Andy Levin.csv'), ('pct_true', 0.5870209336280823)])
OrderedDict([('tweet', 'The climate of white nationalism and anti-Semitism @realDonaldTrump and the @GOP have cultivated and emboldened is showing up at a Jewish candidate’s rally, just as it’s shown up in Charlottesville and so many other places. https://t.co/DVWqaf1LKV'), ('dates', '2020-03-06 19:39:35'), ('file_name', 'Andy Levin.csv'), ('pct_true', 0.5765022039413452)])
OrderedDict([('tweet', 'I am devastated by the violence that has consumed Delhi. This is what the ethno-nationalism of Narendra Modi has wrought—with President Trump’s stamp of approval. https://t.co/aV5ve17paB'), ('dates', '2020-02-29 15:06:34'), ('file_name', 'Andy Levin.csv'), ('pct_true', 0.519546389579773)])
OrderedDict([('tweet', 'Trying to “terminate” healthcare protections for Americans with pre-existing conditions is terrible; doubling down on this effort, amidst a global pandemic that has already killed tens of thousands of Americans, is even worse. We must protect the ACA. https://t.co/EFPKbExnHG'), ('dates', '2020-05-07 15:10:53'), ('file_name', 'Angus S. King, Jr..csv'), ('pct_true', 0.8254337906837463)])
OrderedDict([('tweet', 'The only way we can beat coronavirus is with facts and data – so it’s not helpful when the President removes people from their positions for telling the truth. We need our leaders to be seeking out all available information, not just cherry-picking the pieces they want to hear. https://t.co/rEbhdO5v1w'), ('dates', '2020-05-05 13:41:27'), ('file_name', 'Angus S. King, Jr..csv'), ('pct_true', 0.5226852297782898)])
OrderedDict([('tweet', 'In this crisis, we need to help those who need it most – but unfortunately, the CARES Act included massive tax breaks for a wealthy few. I’ve cosponsored a bill to repeal this so we can direct relief efforts to struggling families and small businesses. https://t.co/Kc108qa2nm'), ('dates', '2020-04-28 14:49:24'), ('file_name', 'Angus S. King, Jr..csv'), ('pct_true', 0.5301070213317871)])
OrderedDict([('tweet', 'From @IgnatiusPost: we’re seeing the consequences of being unprepared for threats that can disrupt our society, yet America is still woefully unprepared for cyberattacks. We need to bolster our defenses – and the @CyberSolarium has ideas on where to start. https://t.co/S0JFndXy0J'), ('dates', '2020-03-13 16:16:28'), ('file_name', 'Angus S. King, Jr..csv'), ('pct_true', 0.6149146556854248)])
OrderedDict([('tweet', 'Today is the latest of this Administration’s efforts to sabotage the ACA. There are no two-ways about this: the people who support this lawsuit are trying to destroy protections for millions of Americans with pre-existing conditions. \nhttps://t.co/WIaAyOdULD'), ('dates', '2020-03-02 19:07:03'), ('file_name', 'Angus S. King, Jr..csv'), ('pct_true', 0.7934410572052002)])
OrderedDict([('tweet', 'The Constitution makes it clear that Congress holds the power to declare war – but over the last 70 years, we’ve too often abdicated that responsibility. Today I joined @EdHenry to discuss the Senate’s bipartisan effort to reassert our Constitutionally-designated responsibility. https://t.co/KuQRX8Rs3B'), ('dates', '2020-02-13 17:55:12'), ('file_name', 'Angus S. King, Jr..csv'), ('pct_true', 0.6412627696990967)])
OrderedDict([('tweet', 'It’s wrong for a president – any president – to taint the independence of our judicial system by bending the rules for a political ally. In America, we have one set of laws, and no one is above them – if we abandon that principle, we abandon who we are.'), ('dates', '2020-02-12 22:30:56'), ('file_name', 'Angus S. King, Jr..csv'), ('pct_true', 0.8710421323776245)])
OrderedDict([('tweet', 'The COVID crisis has exacerbated inequities that Native American communities face, and has left them increasingly vulnerable. We must expand assistance and aid to these communities.\n\nhttps://t.co/QWVEBb0dc7'), ('dates', '2020-05-09 00:00:17'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.7018905282020569)])
OrderedDict([('tweet', "Trump's proposed payroll tax cut will harm Social Security and greatly benefit large corporations. I believe we need to help our most vulnerable, seniors, and #ExpandSocialSecurity."), ('dates', '2020-05-07 18:28:00'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.6069102883338928)])
OrderedDict([('tweet', 'Arizonans have a right to know if their loved ones are in danger. We have the right to decide to save more lives by taking our family members out of assisted living.\n\nhttps://t.co/OxiWUX5izR'), ('dates', '2020-05-01 02:15:03'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.67506343126297)])
OrderedDict([('tweet', "The preservation of our democracy should not be a partisan issue — it should be an American issue. If President Trump wants to 'make America great,' he should let Americans vote like he does, by mail. \nhttps://t.co/EVU4NQmanV"), ('dates', '2020-04-28 00:00:21'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.6473041772842407)])
OrderedDict([('tweet', "The Trump administration's policies are limiting migrant safety and further risking exposure. Not only is this inhumane and alters our U.S. obligation under international human rights law, it actually sets us back from fighting this global pandemic. \n\nhttps://t.co/vyPeSbW4Wg"), ('dates', '2020-04-27 16:58:33'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.8224602937698364)])
OrderedDict([('tweet', "Tucsonans shouldn't have to rely on 'Go Fund Me' pages to survive this-no one should have to choose between paying rent or bills and putting food on the table. We need White House and Senate to step up and put our blue collar workforce before Main Street\n\nhttps://t.co/r0jmllI3R2"), ('dates', '2020-04-27 16:56:52'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.6275179982185364)])
OrderedDict([('tweet', 'On Thursday, I voted to strengthen the Paycheck Protection Program with $310 billion in additional funding. Small businesses must survive this public health crisis in order to revive our economy.'), ('dates', '2020-04-25 19:00:32'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.6244673728942871)])
OrderedDict([('tweet', "The President's tweet about suspending immigration isn't a long term strategy, it's a distraction. Instead of creating divisions, he should be advocating for more testing and ensuring our small businesses and unemployed individuals are receiving the relief aid passed by Congress."), ('dates', '2020-04-21 20:19:08'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.9284025430679321)])
OrderedDict([('tweet', "The hospitals, state and local governments on the front lines of this crisis are already running out of the money provided in the CARES Act. The Democrat's interim emergency spending bill and provide our healthcare workers with the support and equipment they desperately need."), ('dates', '2020-04-16 22:00:23'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.6632530093193054)])
OrderedDict([('tweet', 'Why is it so hard for Mitch McConnell to put #FamiliesFirst? We know how to fix the Paycheck Protection Program and make sure small businesses have critical support. Time for him to get out of the way and make American people the priority.'), ('dates', '2020-04-16 00:00:17'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.7488202452659607)])
OrderedDict([('tweet', 'The Paycheck Protection Program is critical to protecting working people – but problems are causing many small businesses to fall through the cracks. Democrats have proposed essential fixes to get money flowing. Leader McConnell must stop blocking these solutions! #FamiliesFirst'), ('dates', '2020-04-15 18:00:18'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.9520384669303894)])
OrderedDict([('tweet', "I stand with the House Chairs who have condemned the President's attack on the Inspector Generals. He has criticized them for following the law, conducting oversight, and retaliated against them for telling the truth. This is detrimental to the integrity of our democracy."), ('dates', '2020-04-14 21:00:23'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.8089730739593506)])
OrderedDict([('tweet', '2,000 American lives were lost yesterday, and our president is focused on his tv ratings. Without dependable federal leadership, many governors are treating this crisis with the gravity it deserves. Gov. Ducey needs to do the same.\n\nhttps://t.co/6OPTLSTDb7'), ('dates', '2020-04-08 21:15:09'), ('file_name', 'Ann Kirkpatrick.csv'), ('pct_true', 0.7802280783653259)])
OrderedDict([('tweet', '"Fear is not new. Nor is danger and uncertainty. Americans have a long history of navigating both. The solution is to take back control, very carefully."\n\nThis is a thoughtful piece with tips for navigating through this time of stress, fear, &amp; uncertainty that we are living in. https://t.co/hEgEX2tc2E'), ('dates', '2020-04-08 13:15:14'), ('file_name', 'Ann M. Kuster.csv'), ('pct_true', 0.5719954371452332)])
OrderedDict([('tweet', "Deeply disappointed in the Trump Admin's irresponsible &amp; dangerous decision not to create a special ACA open enrollment period during the #COVID19 crisis.\n\nThis virus has disrupted our health care system &amp; economy. Uninsured Granite Staters need an additional enrollment period. https://t.co/JtFRRsqaBO"), ('dates', '2020-04-01 17:04:55'), ('file_name', 'Ann M. Kuster.csv'), ('pct_true', 0.6184394359588623)])
OrderedDict([('tweet', 'Three years ago, President Trump reinstated the misguided &amp; harmful #GlobalGagRule, putting the health &amp; lives of people around the world at risk. I’m proud to support the #GlobalHERAct to repeal this dangerous rule &amp; help ensure everyone can access the care they need. #Fight4HER'), ('dates', '2020-04-01 15:13:44'), ('file_name', 'Ann M. Kuster.csv'), ('pct_true', 0.7986890077590942)])
OrderedDict([('tweet', "I'm calling on Speaker Pelosi &amp; Leader McConnell to act immediately w/ a robust COVID-19 response to support NH's health care system, workers, families &amp; small businesses. We need quick action to provide certainty for communities in these uncertain times. Delay is not an option. https://t.co/uL8HJD9Uvj"), ('dates', '2020-03-21 21:54:16'), ('file_name', 'Ann M. Kuster.csv'), ('pct_true', 0.5065551996231079)])
OrderedDict([('tweet', 'It is clear that fear of offending the Communist Party of China (CCP) contributed to the World Health Organization’s mismanagement of the initial COVID-19 response.'), ('dates', '2020-04-17 16:52:24'), ('file_name', 'Ann Wagner.csv'), ('pct_true', 0.9006888270378113)])
OrderedDict([('tweet', 'The Paycheck Protection Program should have bipartisan support for renewed funding, but Democrats in the House &amp; Senate are blocking this emergency funding for small businesses &amp; refusing to let it pass unless their demands are met. \n\nRead my statement:\n\nhttps://t.co/p73tAtOYlz'), ('dates', '2020-04-16 17:25:45'), ('file_name', 'Ann Wagner.csv'), ('pct_true', 0.759830892086029)])
OrderedDict([('tweet', 'I joined my colleagues on both sides of the aisle to call on House and Senate leadership to reauthorize the Violence Against Women Act (VAWA) in our next tranche of COVID-19 legislation...'), ('dates', '2020-04-12 00:01:53'), ('file_name', 'Ann Wagner.csv'), ('pct_true', 0.572504460811615)])
OrderedDict([('tweet', 'Last week, 6.6 million Americans filed for unemployment, adding to the already unconscionable number of Americans who have lost their jobs due to this pandemic.  Small businesses are the backbone of our economy, and Congress worked in a bipartisan fashion...'), ('dates', '2020-04-09 20:28:26'), ('file_name', 'Ann Wagner.csv'), ('pct_true', 0.833880603313446)])
OrderedDict([('tweet', 'Democrats AGAIN blocked my bill requiring lifesaving medical care for babies who are born-alive.  \n\nIt breaks my heart to find that we must defend lifesaving care for newborn babies.\n\nI will not stop until we get a vote on this critically important bill.\n\nhttps://t.co/Q5LGLVtJfv'), ('dates', '2020-02-28 17:56:13'), ('file_name', 'Ann Wagner.csv'), ('pct_true', 0.598230242729187)])
OrderedDict([('tweet', 'Read my statement w/ @SteveScalise after Senate Dems  blocked the #BornAlive Act from advancing in the Senate.  \n\nI will not stop fighting for the rights of the born &amp; unborn &amp; I am grateful that @senatemajldr brought this legislation to the floor.\n\nhttps://t.co/u5NJ7hjdIM'), ('dates', '2020-02-25 22:12:23'), ('file_name', 'Ann Wagner.csv'), ('pct_true', 0.7646132111549377)])
OrderedDict([('tweet', 'Two weeks ago, President Trump ended Qasem Soleimani’s brutal reign of terror that killed and maimed countless Americans and coalition forces and threatened many more to come.'), ('dates', '2020-01-15 19:29:11'), ('file_name', 'Ann Wagner.csv'), ('pct_true', 0.6126366853713989)])
OrderedDict([('tweet', 'I spoke on the House Floor in opposition to the Democrats’ dangerous resolution to undermine our national defense and allow unchecked Iranian aggression against the United States and our allies.'), ('dates', '2020-01-09 21:43:27'), ('file_name', 'Ann Wagner.csv'), ('pct_true', 0.5914071798324585)])
OrderedDict([('tweet', 'Today I chose the Constitution over partisanship and cast my vote against the articles of impeachment. This is a crime-less impeachment and today we saw Democrats vote to undermine the results of an election without a single Republican joining them. \n\nhttps://t.co/rPR2wniLlI'), ('dates', '2019-12-19 01:57:06'), ('file_name', 'Ann Wagner.csv'), ('pct_true', 0.7520954608917236)])
OrderedDict([('tweet', 'This public health crisis underscores a simple truth: where you live shouldn’t determine your access to the internet. Our students, workers, and families need Congress to expand high-speed broadband access immediately! #FamiliesFirst'), ('dates', '2020-04-14 18:31:46'), ('file_name', 'Anna G. Eshoo.csv'), ('pct_true', 0.6474975347518921)])
OrderedDict([('tweet', 'The President claims the United States has the #coronavirus outbreak “very much under control.” His own policies could threaten that promise. https://t.co/YbyzUjgkkX'), ('dates', '2020-02-26 17:05:56'), ('file_name', 'Anna G. Eshoo.csv'), ('pct_true', 0.5149280428886414)])
OrderedDict([('tweet', 'I’m appalled by the President’s decision to once again put the interests of foreign dictators ahead of U.S. national security and our foreign allies. Allowing Turkey to launch a military offensive in Syria is reckless and further confirms that the President does not understand...'), ('dates', '2019-10-11 20:09:39'), ('file_name', 'Anna G. Eshoo.csv'), ('pct_true', 0.979042649269104)])
OrderedDict([('tweet', 'Today the Trump Administration barred Ambassador Gordon Sondland from appearing before investigators in Congress. The White House’s interference is an obstruction of Congress’ impeachment inquiry.'), ('dates', '2019-10-08 23:28:11'), ('file_name', 'Anna G. Eshoo.csv'), ('pct_true', 0.7576502561569214)])
OrderedDict([('tweet', 'Too many Americans are left w/ unaffordable bills for medications they need to survive. House Democrats are moving to rein in out-of-control drug prices for all Americans. I look forward to chairing next week’s hearing to advance the Lower Drug Costs Now Act https://t.co/tyW564XUgl'), ('dates', '2019-09-19 15:35:37'), ('file_name', 'Anna G. Eshoo.csv'), ('pct_true', 0.8851656913757324)])
OrderedDict([('tweet', "The Trump Admin’s #PublicCharge rule punishes immigrants who lawfully use Medicaid, housing vouchers, &amp; food stamps, forcing millions of immigrant families to choose b/w their legal status and health. That's why I signed the #TriCaucus amicus brief to oppose this unfair rule."), ('dates', '2019-09-17 17:26:37'), ('file_name', 'Anna G. Eshoo.csv'), ('pct_true', 0.8007828593254089)])
OrderedDict([('tweet', 'The Trump Administration is reaping the discord they sow. By forcing a gag rule on health care providers that use #TitleX funding, the Administration is robbing nearly 1.5 million women of affordable, quality, and lifesaving care.'), ('dates', '2019-08-20 00:03:31'), ('file_name', 'Anna G. Eshoo.csv'), ('pct_true', 0.8242111206054688)])
OrderedDict([('tweet', "We fought to get the SPOONSS Act signed into law because American troops should be using America-made products.\n\nI’m proud that #NY22's Sherrill Manufacturing is helping in the fight against #COVID19. https://t.co/vfcujvNyDN"), ('dates', '2020-04-20 18:39:00'), ('file_name', 'Anthony Brindisi.csv'), ('pct_true', 0.5421688556671143)])
OrderedDict([('tweet', 'Shameful! Big corporations taking all the money in the PPP while small mom and pop restaurants in my district can’t get a nickel. Large  corporations should not be double dipping this program at expense of small businesses across the nation: https://t.co/Vec311tKb6 #FixPPP'), ('dates', '2020-04-17 16:58:05'), ('file_name', 'Anthony Brindisi.csv'), ('pct_true', 0.6034757494926453)])
OrderedDict([('tweet', 'This is Washington politics at its worst. Families &amp; small businesses are struggling, and our frontline workers need more support. Both parties must work together to secure more funding for the Paycheck Protection Program &amp; other aid for our communities. https://t.co/MhcQj7UruZ'), ('dates', '2020-04-16 16:37:26'), ('file_name', 'Anthony Brindisi.csv'), ('pct_true', 0.636125922203064)])
OrderedDict([('tweet', 'Corporations like Spectrum are raising prices at a time when families need internet access to do their jobs and their school work. This has to stop. \n \nThat’s why I’m calling on the @FCC to do more to protect consumers during the coronavirus outbreak. https://t.co/qbQqMJCnhu'), ('dates', '2020-04-14 14:44:38'), ('file_name', 'Anthony Brindisi.csv'), ('pct_true', 0.5283727049827576)])
OrderedDict([('tweet', 'ICYMI: we should never have to rely on China in our time of need. It’s time to start manufacturing emergency supplies on American soil. \n\nThat’s why @RepBrianFitz and I introduced the bipartisan Made in America Emergency Preparedness Act. https://t.co/h21DfUV4uE'), ('dates', '2020-04-10 14:09:17'), ('file_name', 'Anthony Brindisi.csv'), ('pct_true', 0.5030601024627686)])
OrderedDict([('tweet', 'Illegal scammers are trying to profit off the coronavirus crisis. That’s just plain wrong. More on how to protect yourself and your family on @NewsChannel9 ⬇️ https://t.co/kG9GeY9kq9'), ('dates', '2020-04-03 14:13:36'), ('file_name', 'Anthony Brindisi.csv'), ('pct_true', 0.53807532787323)])
OrderedDict([('tweet', 'Now is not the time for a wish list of D or R priorities. Lives are on the line. \n\nWe need a coronavirus relief bill that helps the American people, protects workers, &amp; provides oversight. Not a bailout of multi-million dollar corporations, partisan pet projects, or slush funds.'), ('dates', '2020-03-24 19:00:14'), ('file_name', 'Anthony Brindisi.csv'), ('pct_true', 0.6676633358001709)])
OrderedDict([('tweet', "Bills are due but too many MD families haven't been able to apply for unemployment, and even more haven't received their benefits. Without assistance, MDers are getting closer to financial disaster.\n \nGovernor Hogan is failing working families when they need help the most. https://t.co/jkoDcpWlgO"), ('dates', '2020-05-06 15:59:21'), ('file_name', 'Anthony G. Brown.csv'), ('pct_true', 0.5125678777694702)])
OrderedDict([('tweet', 'This ludicrous suggestion for McConnell and the GOP would threaten state pensions, unemployment benefits, education funding, transportation, etc.\n\nAt a time when Americans are calling out for their government to help, Mitch McConnell is telling them to get lost. Shameful. https://t.co/YfmTrgXOUp'), ('dates', '2020-04-24 00:04:00'), ('file_name', 'Anthony G. Brown.csv'), ('pct_true', 0.7270241379737854)])
OrderedDict([('tweet', 'Trump promised 4 million tests a week in early March, today he celebrated 3.5 million tests.\n\nUntil we can rapidly test everyone, we can’t safely lift restrictions that are working to stop the spread #COVID19. We can’t ask any American to take that risk. https://t.co/OWA85JIGBf'), ('dates', '2020-04-16 23:53:42'), ('file_name', 'Anthony G. Brown.csv'), ('pct_true', 0.7073021531105042)])
OrderedDict([('tweet', 'The Trump Administration must make clear that those who receive Social Security checks do not have to file a tax return to receive a stimulus check. Our seniors need aid, not additional paperwork.  \n\nWe’re in this together, we need to ensure that no one gets left behind. #COVID19 https://t.co/KnOpFU6w4I'), ('dates', '2020-04-01 22:50:02'), ('file_name', 'Anthony G. Brown.csv'), ('pct_true', 0.6362958550453186)])
OrderedDict([('tweet', '10-years ago, the #AffordableCareAct passed into law covering over 20 million Americans and protecting over 133 million Americans with pre-existing conditions.\n\nThe fight to #ProtectOurCare is more important than ever before. No one should go bankrupt just because they got sick. https://t.co/VQnzr7AlU7'), ('dates', '2020-03-23 16:41:00'), ('file_name', 'Anthony G. Brown.csv'), ('pct_true', 0.7651868462562561)])
OrderedDict([('tweet', 'Even in a public health crisis, Mitch McConnell is sticking with his playbook of delaying critical legislation that would help American families.\n\nHouse Democrats, House Republicans and the Trump Admin. have already agreed on a deal to put #FamiliesFirst. We need action now. https://t.co/K7gIYBUSMS'), ('dates', '2020-03-16 15:57:31'), ('file_name', 'Anthony G. Brown.csv'), ('pct_true', 0.7283846735954285)])
OrderedDict([('tweet', 'Trump promised that “anybody” could get a test, but this admin is far behind the testing of other countries.\n\nWe need a President willing to take responsibility for his admin’s mistakes. Instead we have one who’s made this situation worse. #COVID19\n\nhttps://t.co/z0NR3E71Wl'), ('dates', '2020-03-13 22:13:52'), ('file_name', 'Anthony G. Brown.csv'), ('pct_true', 0.7688010931015015)])
OrderedDict([('tweet', "Glad to see @CommerceGov issue new trade regulations on China to help keep U.S. technology out of the hands of the Chinese military. China's Communist Party has been stealing our intellectual property for far too long - we must push back! https://t.co/GcIJ2pQfFR"), ('dates', '2020-04-29 17:56:20'), ('file_name', 'Anthony Gonzalez.csv'), ('pct_true', 0.8437369465827942)])
OrderedDict([('tweet', "Small business owners across #OH16 are relying on PPP to survive. Blocking a refill of funding to this program is utterly irresponsible. Congress must act now to refill PPP and stop playing political games with people's livelihoods. https://t.co/F0mqx46dXS"), ('dates', '2020-04-17 16:05:52'), ('file_name', 'Anthony Gonzalez.csv'), ('pct_true', 0.8782731294631958)])
OrderedDict([('tweet', "Brutal policies like Fidel Castro's literacy program for re-education go against the American way of life, but they are innate to the implementation of socialist policies like those that tore down the Cuba of my grandparents’ childhood. WATCH HERE -&gt; https://t.co/neXjYYeFbO"), ('dates', '2020-02-27 22:27:45'), ('file_name', 'Anthony Gonzalez.csv'), ('pct_true', 0.6561857461929321)])
OrderedDict([('tweet', 'This constitutes an outright attack by the Chinese military on Americans through cyber warfare. This sort of attack cannot and should not go unchallenged. The U.S. must ensure American consumers and Equifax get the justice they deserve. https://t.co/EvFOipykyM'), ('dates', '2020-02-10 16:40:09'), ('file_name', 'Anthony Gonzalez.csv'), ('pct_true', 0.7283682227134705)])
OrderedDict([('tweet', 'Today marks the 75th anniversary of the liberation of Auschwitz and #HolocaustRemembranceDay. We must never forget the 6 million Jewish men, women, and children who perished during the Holocaust and make it our duty to end anti-Semitism and hatred wherever it exists. https://t.co/JGX7BU1C8k'), ('dates', '2020-01-27 18:44:06'), ('file_name', 'Anthony Gonzalez.csv'), ('pct_true', 0.5760428309440613)])
OrderedDict([('tweet', 'For months, House Democrats wasted time &amp; taxpayer resources on a partisan impeachment process against @realDonaldTrump fueled by their hatred for him. (1/2)'), ('dates', '2020-02-05 22:26:32'), ('file_name', 'Austin Scott.csv'), ('pct_true', 0.8076364398002625)])
OrderedDict([('tweet', 'If we want to have an honest debate about war powers, then we should have an honest debate about war powers. Why, then, did our colleagues on the other side of the aisle single out only Iran in this resolution? Why not Libya? Why not Syria? (1/4)'), ('dates', '2020-01-10 00:01:19'), ('file_name', 'Austin Scott.csv'), ('pct_true', 0.5471219420433044)])
OrderedDict([('tweet', 'For years, Iran has used proxies to attack and kill American soldiers and our partner forces. Qassem Soleimani was the head of the Islamic Revolutionary Guard Corps- Quds Force, a designated terrorist organization, and was responsible for these attacks. (1/2)'), ('dates', '2020-01-03 16:40:49'), ('file_name', 'Austin Scott.csv'), ('pct_true', 0.5177913904190063)])
OrderedDict([('tweet', 'House Democrats have used sequencing &amp; patterns to manufacture a sham case for impeachment of @realDonaldTrump. Their hatred for the President of the United States blinds them to the truth. I’ll be voting NO on articles of impeachment. #ImpeachmentSham https://t.co/jRTBj87jjK'), ('dates', '2019-12-18 19:23:30'), ('file_name', 'Austin Scott.csv'), ('pct_true', 0.8259403109550476)])
OrderedDict([('tweet', 'Profs without proof. Not a single one of yesterday’s legal professors had any personal knowledge of a single fact in Adam Schiff’s report. This has been a factless personal attack against President Trump from the start. #ShamImpeachment https://t.co/eg2gDksXnG'), ('dates', '2019-12-05 15:40:43'), ('file_name', 'Austin Scott.csv'), ('pct_true', 0.5664601922035217)])
OrderedDict([('tweet', 'This call is the basis of Speaker Pelosi &amp; Adam Schiff’s impeachment inquiry, &amp; Ambassador Yovanovitch was recalled a month &amp; a half before it even took place. The facts don’t matter to our Dem colleagues. They’ve been trying to impeach @realDonaldTrump since he was sworn in. https://t.co/1MYcHngDcs'), ('dates', '2019-11-15 17:32:42'), ('file_name', 'Austin Scott.csv'), ('pct_true', 0.6288350224494934)])
OrderedDict([('tweet', 'I just voted against @SpeakerPelosi’s resolution to continue her sham impeachment inquiry against @realDonaldTrump. Their hatred for him is not grounds for impeachment.'), ('dates', '2019-10-31 16:42:49'), ('file_name', 'Austin Scott.csv'), ('pct_true', 0.7867809534072876)])
OrderedDict([('tweet', 'Unfortunately, Speaker Pelosi has succumbed to the belligerent left &amp; their media allies to move forward w/ an impeachment inquiry against @POTUS. Dems continue to waste time &amp; taxpayer dollars on endless investigations while refusing to address the challenges our nation faces.'), ('dates', '2019-09-24 22:34:55'), ('file_name', 'Austin Scott.csv'), ('pct_true', 0.960523247718811)])
OrderedDict([('tweet', '18 years ago, our nation stood still as news came in of attacks on American soil. We will #NeverForget the more than 3,000 Americans who lost their lives that day in NYC, DC &amp; PA. Today, we honor their lives, the first responders on scene, &amp; our troops overseas. #NeverAgain https://t.co/wYCsK6TRLH'), ('dates', '2019-09-11 14:49:48'), ('file_name', 'Austin Scott.csv'), ('pct_true', 0.5661005973815918)])
OrderedDict([('tweet', 'Micro businesses are the backbones of our communities, but so far, too many have been left out &amp; left behind by our federal relief efforts. \n\n@SenKamalaHarris &amp; I are introducing the #SavingOurStreet Act to get these businesses the support they need.\n\nhttps://t.co/X5w4maBIRg'), ('dates', '2020-05-06 18:42:29'), ('file_name', 'Ayanna Pressley.csv'), ('pct_true', 0.5912534594535828)])
OrderedDict([('tweet', 'This is a crisis within a crisis. Minority-owned businesses have faced structural barriers to accessing capital for generations, and this crisis is only exacerbating those inequities. We need dedicated funding for #CDFIs and #MDIs now. https://t.co/1vLRhQBxpQ'), ('dates', '2020-05-01 21:25:00'), ('file_name', 'Ayanna Pressley.csv'), ('pct_true', 0.7153214812278748)])
OrderedDict([('tweet', 'As this admin implements draconian #immigration restrictions under the guise of public health, they continue to deport people to countries w/o the public health infrastructure to handle this virus, risking millions of lives. The hypocrisy is almost as astounding as the cruelty. https://t.co/fnRNU4jW8u'), ('dates', '2020-04-22 20:45:10'), ('file_name', 'Ayanna Pressley.csv'), ('pct_true', 0.5002251863479614)])
OrderedDict([('tweet', "Our immigrant neighbors are essential to our nation's wellbeing—protecting and supporting our communities. The occupant’s shameful attempt to use this public health emergency to advance his xenophobic agenda is disturbing, but not surprising. The cruelty is the point. https://t.co/EUDbw192KO"), ('dates', '2020-04-21 14:27:26'), ('file_name', 'Ayanna Pressley.csv'), ('pct_true', 0.5785708427429199)])
OrderedDict([('tweet', 'Systemic barriers that have long blocked communities of color from accessing credit are preventing borrowers from accessing #PaycheckProtectionProgram loans. Lenders must be required to disclose data on who’s getting PPP funds so we can ensure equitable access to relief programs. https://t.co/6vAN0BID4U'), ('dates', '2020-04-17 21:24:14'), ('file_name', 'Ayanna Pressley.csv'), ('pct_true', 0.6544377207756042)])
OrderedDict([('tweet', 'The #USPostalService provides an essential service to communities in every corner of our country, but @USPS is on the brink of collapse due to #COVID19 and decades of GOP attacks. We must #SaveTheUSPS before its too late!  https://t.co/Hx747gsxUs'), ('dates', '2020-04-13 20:21:06'), ('file_name', 'Ayanna Pressley.csv'), ('pct_true', 0.583111584186554)])
OrderedDict([('tweet', "We must fight to bail out the people—not corporations. No one should have to choose btwn paying bills, putting food on the table or keeping their families safe. Joining the @GetUpCrew969 @ 730am to discuss how I'm fighting for the Commonwealth in Congress https://t.co/kizmhTaeli"), ('dates', '2020-04-03 11:10:00'), ('file_name', 'Ayanna Pressley.csv'), ('pct_true', 0.5266943573951721)])
OrderedDict([('tweet', 'Dropping the Flynn case is corruption through and through. We have a lawless President and a lawless Administration overseeing our country. \n\nShameful.'), ('dates', '2020-05-08 15:25:58'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.7881223559379578)])
OrderedDict([('tweet', "Trump's failure to act has directly resulted in the worst economic crisis since the Great Depression.\n \nIt's time for big, bold, progressive policies 𝕨𝕖 𝕜𝕟𝕠𝕨 𝕨𝕠𝕣𝕜.\n \nOur country can't afford for us to delay any longer. https://t.co/mgjIUgj3I8"), ('dates', '2020-05-08 14:13:36'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.559447169303894)])
OrderedDict([('tweet', 'Predictable. Trump has repeatedly ignored the Constitution, and his war mongering ways still represent a clear and present danger to our national security. #StopEndlessWar https://t.co/kA5UCPQhfg'), ('dates', '2020-05-06 23:05:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.7894883155822754)])
OrderedDict([('tweet', 'If the GOP has their way, millions of people will lose their right to vote. \n\nThey’re using the pandemic to systemically suppress votes and steal elections.  \n\nWe need to #RestoreTheVote. https://t.co/8iXIcoMAh6'), ('dates', '2020-05-05 20:56:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.5942479372024536)])
OrderedDict([('tweet', 'Families like Michelle’s are falling through the cracks. \n\nTrump, McConnell, and the GOP need to stop prioritizing their wealthy friends and actually put #FamiliesFirst. https://t.co/UVitiwoaD6'), ('dates', '2020-05-01 00:03:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.5940197706222534)])
OrderedDict([('tweet', 'The GOP is using this pandemic to suppress votes.\n\nMake no mistake – we will fight this. No one should have to risk their life to vote. https://t.co/bR0Xfml0MJ'), ('dates', '2020-04-30 19:39:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.5887139439582825)])
OrderedDict([('tweet', 'There it is.\n\nTrump doesn’t care about health and safety – he just wants the country to reopen so his poll numbers will go up.\n\nDisgusting. #PeopleOverProfits #ForThePeople https://t.co/kiU0kclxfj'), ('dates', '2020-04-30 16:40:12'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.752606987953186)])
OrderedDict([('tweet', 'Trump and the GOP are still trying to legislate away our health care.\n\nThe only people who should be involved in health decisions are the patient and their doctor, not right-wing politicians. #HandsOffMyBC https://t.co/c6k3nZbxuF'), ('dates', '2020-04-29 21:51:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.6882935166358948)])
OrderedDict([('tweet', 'Asian Americans are still being targeted with racially charged Coronavirus epithets and physical attacks.\n \nThere is never an excuse for racism. https://t.co/a4pPSYPE8i'), ('dates', '2020-04-28 20:47:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.5999931693077087)])
OrderedDict([('tweet', 'Trump is using a global crisis to punish his political enemies. \n\nHe’s literally putting people’s lives in danger to score some points. \n\nThis isn’t leadership. #FamiliesFirst https://t.co/JMY3quS9lO'), ('dates', '2020-04-28 16:03:35'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.9023423194885254)])
OrderedDict([('tweet', 'It’s surprising that Leader McConnell is worried about the national deficit now, when he passed the $2 trillion tax cut for the 1% that exploded the deficit.\n \nCongress needs to prioritize families and small businesses – not the wealthy and well connected. https://t.co/o7gKUljcdI'), ('dates', '2020-04-25 23:21:30'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.6408791542053223)])
OrderedDict([('tweet', 'Trump is silencing government officials and experts who disagree with his claims.\n\nThis. Is. Not. Normal. https://t.co/UpHeYaDMr1'), ('dates', '2020-04-23 19:46:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.5212041139602661)])
OrderedDict([('tweet', 'Trump has dithered and mishandled this pandemic from the beginning.\n\nThere is NO REASON for it to take 2 weeks to get the bare minimum in assistance.\n\nWe need to pass this bill and get to work on a relief package that works for everyone. #ForThePeople #PeopleOverProfits https://t.co/oHU6x5QCCN'), ('dates', '2020-04-23 16:08:35'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.5720803141593933)])
OrderedDict([('tweet', '#NeverAgain is never enough without action.  \n\nToday on #YomHaShoah, we remember the 6 million Jews who were murdered in the Holocaust, and honor them by fighting anti-Semitism and hate. #HolocaustRemembranceDay'), ('dates', '2020-04-21 17:47:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.9323867559432983)])
OrderedDict([('tweet', 'Businesses, workers, and families across the country are struggling – but for WEEKS for Mitch McConnell has fought to protect the wealthy rather than put #FamiliesFirst. We need to focus on helping people, not playing politics with their health and safety. https://t.co/eoCHKIozEZ'), ('dates', '2020-04-20 17:08:35'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.7794795632362366)])
OrderedDict([('tweet', 'Voting in elections is the central tenet of our democracy. We can’t let Trump and his henchmen use this pandemic as an excuse to block access to the ballot box. #RestoreTheVote #VoteByMail'), ('dates', '2020-04-17 19:41:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.7495834827423096)])
OrderedDict([('tweet', 'Small and local businesses are the backbones of our communities and economy. It’s absolutely outrageous that Mitch McConnell is holding up increased SBA loans to play politics. \n\nFamilies need help and they need it NOW. #FamiliesFirst #PeopleOverProfits'), ('dates', '2020-04-17 16:28:38'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.9134473204612732)])
OrderedDict([('tweet', 'We are in the middle of a global crisis – and Trump is playing politics with people’s lives.\n \nScapegoating the WHO for his failed response to this global pandemic is dangerous and senseless but sadly predictable. https://t.co/cNseZZoAzC'), ('dates', '2020-04-15 17:08:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.8151870369911194)])
OrderedDict([('tweet', "In the middle of a global pandemic, people deserve the truth. Trump's actions and lies over the last few months have put lives on the line, and it’s time to call them out. https://t.co/muk9EfJE93"), ('dates', '2020-04-14 23:59:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.7259925007820129)])
OrderedDict([('tweet', 'We can’t let COVID increase the fault lines in our society. We need to #ReleaseTheData on racial disparities so that Congress can get help to communities of color. Glad to work with @RepPressley, @RepRobinKelly, @RepKarenBass, and @SenWarren on this bill. https://t.co/aueeH8nD3x'), ('dates', '2020-04-14 21:27:33'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.7645083069801331)])
OrderedDict([('tweet', 'People are dying. Families are struggling. Workers put their lives on the line every day.\n\nInstead of bringing the country together, Trump is using his press conferences to scream at reporters and push propaganda. Now is the time for leadership – not egos.'), ('dates', '2020-04-14 17:37:00'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.8627608418464661)])
OrderedDict([('tweet', "Glad we were able to get a provision in the Senate bill requiring CDC to report racial data on COVID. Shameful that Senate Republicans just blocked it.  \n \nIf Congress is going to help the country, we need to know who's being affected.  https://t.co/hgzjP2K5JF"), ('dates', '2020-04-09 15:54:31'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.6294500827789307)])
OrderedDict([('tweet', 'Decades of structural racism have prevented communities of color from accessing quality health care – and the coronavirus crisis is blowing these disparities wide open. \n \nCongress needs the racial disparity data to address these issues.\nhttps://t.co/WYGIyV5w72'), ('dates', '2020-04-08 17:21:19'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.6479076743125916)])
OrderedDict([('tweet', 'Trump is completely heartless – in the middle of a pandemic, his xenophobic Administration is planning to deport several Haitians tomorrow.\n \nThis is beyond outrageous! https://t.co/rKrsHUB4Yo'), ('dates', '2020-04-06 23:46:24'), ('file_name', 'Barbara Lee.csv'), ('pct_true', 0.7103415727615356)])
OrderedDict([('tweet', 'Regardless of when the medical community deems it okay for people to return to work, Dems will delay so they can say they were the ‘responsible’ party and listened to the medical community; but, in reality they ignored COVID-19, blasted @realDonaldTrump’s initial travel bans...'), ('dates', '2020-04-28 18:37:22'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.5658857226371765)])
OrderedDict([('tweet', 'Democrats need to stop blocking additional funding for the #PayCheckProtectionProgram. Millions of jobs have been saved due to this program and it’s vital that we allocate more money so we can save more businesses and jobs. #GetAmericaBackToWork'), ('dates', '2020-04-16 15:41:19'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.9697360992431641)])
OrderedDict([('tweet', 'The last time real immigration reform was passed, floppy disks were standard and Top Gun was the highest grossing movie. If Congress would do its job, administrations wouldn’t need to act unilaterally and the #SCOTUS wouldn’t need to get involved.\nhttps://t.co/UdPbnX9Bo3'), ('dates', '2020-02-27 12:43:50'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.6055505871772766)])
OrderedDict([('tweet', 'Democrats have again shown they will not let rules, laws, or the Constitution stand in their way of partisan games. Even Justice Ginsberg, a proponent of equal rights, has said the #ERA process should start over. https://t.co/QfIS2XEruc'), ('dates', '2020-02-13 16:41:03'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.9359333515167236)])
OrderedDict([('tweet', 'Speaker Pelosi once said, “show me your values, show me your budget.” But for the second year in a row, Democrats have failed to propose a budget...yet they criticize @realDonaldTrump’s?\n\nCould it be Speaker Pelosi is no longer sure if they are socialists or capitalists?'), ('dates', '2020-02-11 22:35:22'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.8725654482841492)])
OrderedDict([('tweet', 'Sanctuary cities are nothing more than a haven for criminal aliens cloaked in the left’s self-righteous indignation. Thankfully we have a president and AG willing to go after these corrupt policies.\n\nhttps://t.co/f2LWSHJIVN'), ('dates', '2020-02-11 20:58:43'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.7741617560386658)])
OrderedDict([('tweet', 'Lower taxes, deregulation, and better trade deals are key to a booming economy - not the #ProAct, which House Dems just passed...'), ('dates', '2020-02-07 02:26:39'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.6556709408760071)])
OrderedDict([('tweet', 'These are the faces of a Democratic party that supports free healthcare for illegal immigrants, but refuses to console/clap for the brother of a American citizen who was killed by an illegal immigrant. #OutofTouch https://t.co/sRc4WTtsyc'), ('dates', '2020-02-05 15:49:51'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.6619296669960022)])
OrderedDict([('tweet', '#Immigration: Despite obstruction by congressional Democrats, under @realDonaldTrump’s leadership, apprehensions at the U.S.-Mexico border are down more than 75% since May and Mexico is now doing its part and stopping dangerous migrant caravans from reaching our border.'), ('dates', '2020-02-05 02:47:08'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.8814663887023926)])
OrderedDict([('tweet', '#NationalSecurity: Under @realDonaldTrump’s leadership, the U.S. defeated the ISIS caliphate and killed two of the worlds top terrorists: Iranian General Soleimani and ISIS leader al-Baghdadi.'), ('dates', '2020-02-05 02:37:24'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.6398482918739319)])
OrderedDict([('tweet', '#ForeignAffairs: @realDonaldTrump has completely reshaped America’s 🇺🇸 trade deals - USMCA and China Phase 1 - so they work for us, not against us.'), ('dates', '2020-02-05 02:24:38'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.814454197883606)])
OrderedDict([('tweet', 'Last year, House Dems’ landmark achievement was #HR1, a complex and massive bill that puts the federal bureaucracy in charge of our elections.\n\nThey can’t count their own votes, but they want to tell everyone else how to run an election...?'), ('dates', '2020-02-04 20:43:16'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.7903169989585876)])
OrderedDict([('tweet', 'Given the #IowaCaucus debacle, it’s frightening House Dems passed a bill last year to take control of our nation’s elections process.'), ('dates', '2020-02-04 15:43:43'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.6246147751808167)])
OrderedDict([('tweet', 'Today’s vote in the Senate was not a vote to block witnesses; it was a vote showing that the American people have had enough of this #shamimpeachment. The Democrats inability to prove their case, let alone come together as a team, can be summed up by... #JerryJerryJerry'), ('dates', '2020-01-31 22:45:15'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.5426293015480042)])
OrderedDict([('tweet', 'In about 5 min, Patrick Philbin dismantled an entire article of impeachment (obstruction) when he calmly described how House Dems couldn’t even get the paper pushing part of impeaching @realDonaldTrump right. They ignored the law on how to file subpoenas, which made them invalid.'), ('dates', '2020-01-30 16:12:53'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.8634296655654907)])
OrderedDict([('tweet', 'Interesting to hear Schumer allege Republicans have something to hide because some GOP members asked to read Bolton’s manuscript in a SCIF...then what were Schiff and fellow House Dems hiding when they deposed 17 witnesses over the course of two weeks in a SCIF? #ShamImpeachment https://t.co/FqyKKPqOKw'), ('dates', '2020-01-28 21:06:54'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.9062075018882751)])
OrderedDict([('tweet', 'BREAKING: In 9 days, the temporary ban on fentanyl analogues will expire. House Dems have had since NOVEMBER to fix this deadly problem, but instead they have wasted their time on a #ShamImpeachment. Fentanyl is 100x more potent than morphine and killed 28,000+ people in ‘17.'), ('dates', '2020-01-28 18:36:47'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.6402305960655212)])
OrderedDict([('tweet', 'Remember, when Dems couldn’t prove ‘Russia collusion’ they moved to ‘obstruction.’ On Ukraine, they couldn’t prove ‘quid pro quo’ ‘extortion’ or ‘bribery’ bc not even one of these made it into the articles of impeachment. When you keep redefining the crime...there is no crime.'), ('dates', '2020-01-23 19:48:49'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.7603349089622498)])
OrderedDict([('tweet', 'Given Pelosi and Schiff rushed impeachment through the House and didn’t subpoena Bolton, it’s absurd to hear the Dems demand more time and more witnesses. Demanding full accountability from @realDonaldTrump while holding yourself to a lesser standard is the height of hypocrisy.'), ('dates', '2020-01-22 18:36:43'), ('file_name', 'Barry Loudermilk.csv'), ('pct_true', 0.8702325820922852)])
OrderedDict([('tweet', '75 years ago, the United States declared victory in Europe over Nazi Germany and the region’s Axis Powers. Let’s never forget the lives lost and honor the heroes who fought to liberate the world from oppression. https://t.co/kvgeWFIiwT'), ('dates', '2020-05-08 13:01:26'), ('file_name', 'Ben Cline.csv'), ('pct_true', 0.754218339920044)])
OrderedDict([('tweet', 'Colleges &amp; universities with billion dollar endowments should not be requesting funds from the CARES Act. The money should be reserved for businesses &amp; institutions in need, not entities that have unlimited resources through their foundations &amp; endowments. https://t.co/9zPS02n1ox'), ('dates', '2020-04-24 14:51:29'), ('file_name', 'Ben Cline.csv'), ('pct_true', 0.5418890118598938)])
OrderedDict([('tweet', 'China knew the danger COVID-19 posed to the world back in November and recklessly failed to warn the global health community. That is why I recently urged President Trump to evaluate the long-term threat China poses to the United States’ economic and national security. https://t.co/52mwHZBSm1'), ('dates', '2020-03-28 19:10:59'), ('file_name', 'Ben Cline.csv'), ('pct_true', 0.8980074524879456)])
OrderedDict([('tweet', 'The billions in small business loans approved by Congress is for Main Street businesses, not big corporations. It’s not right it’s and not fair to hardworking Utah families. Treasury and SBA need to show us the list of companies who got the money. #utpol https://t.co/qD6xcjchOt'), ('dates', '2020-04-29 19:14:24'), ('file_name', 'Ben McAdams.csv'), ('pct_true', 0.5499844551086426)])
OrderedDict([('tweet', 'I couldn’t agree more with the Fed Chair. Our nation’s debt and deficit is out of control. Republicans and Democrats must get serious about working together to tackle this problem. #utpol \n\nhttps://t.co/OzYOg6Rzld'), ('dates', '2020-02-11 22:53:36'), ('file_name', 'Ben McAdams.csv'), ('pct_true', 0.7061172723770142)])
OrderedDict([('tweet', 'On Yom HaShoah, we remember the millions of lives lost during the Holocaust and recommit ourselves to the fight against anti-Semitism. #NeverAgain'), ('dates', '2020-04-21 21:32:31'), ('file_name', 'Ben Ray Luján.csv'), ('pct_true', 0.5666311979293823)])
OrderedDict([('tweet', 'New Mexico small businesses deserve their fair share. That’s why @SenatorTomUdall, @MartinHeinrich, @RepDebHaaland, @RepTorresSmall &amp; I are calling on SBA to fix their implementation problems so that struggling small businesses and nonprofits in can get the assistance they need.'), ('dates', '2020-04-15 00:32:09'), ('file_name', 'Ben Ray Luján.csv'), ('pct_true', 0.7978810667991638)])
OrderedDict([('tweet', 'It’s reprehensible that companies would try to profit from the fear caused by this pandemic. Today, My colleagues + I introduced legislation to stop the greed, and protect New Mexicans. https://t.co/zBJJDAAQB4'), ('dates', '2020-04-09 02:22:00'), ('file_name', 'Ben Ray Luján.csv'), ('pct_true', 0.5228796601295471)])
OrderedDict([('tweet', 'Small businesses are the lifeblood of New Mexico and have been hit hard by the coronavirus outbreak.'), ('dates', '2020-04-03 23:41:33'), ('file_name', 'Ben Ray Luján.csv'), ('pct_true', 0.6784731149673462)])
OrderedDict([('tweet', 'We cannot let these families suffer. The Trump administration must take steps to address these delays immediately, so New Mexican families can receive the economic relief they deserve. https://t.co/obehdusmTd'), ('dates', '2020-04-02 21:26:44'), ('file_name', 'Ben Ray Luján.csv'), ('pct_true', 0.575124204158783)])
OrderedDict([('tweet', "For every $1 a man makes, a woman makes 81 cents. For black women, it's 62 cents; for Native American women, it's 57 cents; for Asian American and Pacific Islander women it's 92 cents; and for Latina women it's 54 cents. It's unacceptable."), ('dates', '2020-03-31 22:46:58'), ('file_name', 'Ben Ray Luján.csv'), ('pct_true', 0.6764063835144043)])
OrderedDict([('tweet', 'This tweet is simply drunk partisanship. Gen. Soleimani has killed hundreds and hundreds of Americans, and was actively plotting more. This commander-in-chief — any C-in-C. — has an obligation to defend America by killing this bastard. https://t.co/eQmiCgegxS'), ('dates', '2020-01-03 03:28:21'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.8446169495582581)])
OrderedDict([('tweet', '.@KingJames — you’re parroting communist propaganda. China is running torture camps and you know it. #GoogleUyghurs https://t.co/BExRpi02EB'), ('dates', '2019-10-15 03:00:43'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.7539629340171814)])
OrderedDict([('tweet', '"Earmarks are a crummy way to govern and they have no business in Congress...This is an important fight and I’m glad that my Republican colleagues agreed with my rules change to make the earmark ban permanent." https://t.co/BluMqZI83i'), ('dates', '2019-05-23 16:48:42'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.5085970163345337)])
OrderedDict([('tweet', 'Thousands of Americans come to DC for the #MarchForLife. This should fill us with hope. Our cause is pro-woman, pro-baby, pro-science, and pro-human dignity. We’re pro-truth, and truth is winning. https://t.co/kl4EnFNf4m'), ('dates', '2019-01-18 21:22:17'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.5412480235099792)])
OrderedDict([('tweet', "The U.S. intelligence community knows that the Russian government attacked the U.S. This is not a Republican or a Democrat view -- it is simply the reality. All patriotic Americans should understand that Putin is not America's friend, and he is not the President's buddy... https://t.co/GfgtDKKtSF"), ('dates', '2018-07-13 17:26:13'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.8194558620452881)])
OrderedDict([('tweet', 'About those new tariffs: Europe, Canada &amp; Mexico aren\'t China. You don’t treat allies the same way you treat opponents. Blanket protectionism is a big part of why we had a Great Depression. "Make America Great Again" shouldn’t mean "Make America 1929 Again."'), ('dates', '2018-05-31 15:30:07'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.6291422247886658)])
OrderedDict([('tweet', 'Let’s get this straight: Equifax put millions of Americans at risk, got a no-bid contract from the feds, and at least one employee turned a personal profit from trading on the misfortune of the victims? \n\nSomeone needs to answer for this. \n\nhttps://t.co/Rz3vsJF9Qa'), ('dates', '2018-03-14 16:30:37'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.5707879662513733)])
OrderedDict([('tweet', 'This is national security 101: Washington should be hunting down terrorists who fund global violence with domestic drug money. https://t.co/XGUMbrWGIK'), ('dates', '2018-01-11 16:02:59'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.728144645690918)])
OrderedDict([('tweet', "The ABA isn't a neutral vetter of judges but a liberal organization that has publicly &amp; consistently advocated for left-of-center positions. https://t.co/ZTMrJVEAjw"), ('dates', '2017-11-16 23:57:06'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.7517316341400146)])
OrderedDict([('tweet', 'Great news: The EPA’s unconstitutional water rule will die. Nebraska’s farmers &amp; ranchers won’t shed a single tear. https://t.co/urfVyVq1Q8 https://t.co/x3qDAJ9NAS'), ('dates', '2017-02-28 19:58:07'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.685077965259552)])
OrderedDict([('tweet', 'This should not be viewed through partisan labels. Congress should investigate Russia’s influence campaign in the US https://t.co/B0utCny1sm'), ('dates', '2017-02-17 15:27:43'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.6376398205757141)])
OrderedDict([('tweet', "Reintroduced Sarah's Law in the new Congress. If an illegal alien kills an American &amp; they are detained, they should be held until trial. https://t.co/PHR6hnv8TE"), ('dates', '2017-02-03 01:36:01'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.5082905292510986)])
OrderedDict([('tweet', 'Sen. Schumer is about to tell Americans that Judge Gorsuch kicks puppies &amp; heckles piano recitals. That’s hogwash. https://t.co/YABOHdtznT'), ('dates', '2017-02-01 02:08:02'), ('file_name', 'Ben Sasse.csv'), ('pct_true', 0.5042273998260498)])
OrderedDict([('tweet', "Arresting Ahmaud Arbery's killers is the right thing to do. What's wrong is that it took 74 days &amp; public outcry to do it; that Ahmaud couldn't go jogging without his life being taken. His family should be celebrating his birthday w him today, not demanding justice for his murder https://t.co/qiuoqI6h4q"), ('dates', '2020-05-09 00:35:59'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.5311705470085144)])
OrderedDict([('tweet', 'Flynn pled guilty to lying to the FBI - twice. AG Barr dropping criminal charges against him damages the credibility of the DoJ and rule of law in this country.\nhttps://t.co/QenMwFAl50'), ('dates', '2020-05-08 20:17:22'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.7685623168945312)])
OrderedDict([('tweet', 'Deflecting blame is not the way to save lives and get our nation and the world through this global public health crisis. Yet again, @POTUS chastises others for perceived mistakes while falsely claiming his response to the COVID-19 outbreak was perfect. https://t.co/BX5XpERvqu'), ('dates', '2020-04-15 01:40:51'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.8896145820617676)])
OrderedDict([('tweet', "Small businesses need capital now so they can retain workers and stay in business. Merely adding more money to PPP won't help #SmallBiz being locked out by big banks. We have a chance to fix the program and add $. We need to do both. @senatemajldr's stunt fails to address this. https://t.co/81jLyutH5x"), ('dates', '2020-04-09 15:03:13'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.6827257871627808)])
OrderedDict([('tweet', '.@FreedomHouse’s annual report on political rights and civil liberties shows a global backslide of democratic values for 14 straight years. \n\nMost alarming: the U.S. has now fallen behind its traditional democratic peers. https://t.co/Pdq8qRvR3a'), ('dates', '2020-03-04 22:50:11'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.5458718538284302)])
OrderedDict([('tweet', 'Violence in #Cameroon has killed more than 3,000 since 2017. Most victims are civilians and children. The violence must stop; Pres. Biya must follow through on his pledge to investigate last month’s clash in Ngarbu. https://t.co/trMIsQnYvd'), ('dates', '2020-03-03 15:18:33'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.5009493231773376)])
OrderedDict([('tweet', "Recent airstrikes in Syria have escalated conflict btwn Turkey &amp; Russian-backed Assad regime leaving innocent civilians in the crossfire. \n\nThe int'l community must seek an immediate ceasefire in Idlib &amp; help get humanitarian supplies &amp; assistance to displaced communities in need https://t.co/lTU1Phjysu"), ('dates', '2020-02-28 22:30:06'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.507041335105896)])
OrderedDict([('tweet', 'Trump has already harmed our ability to handle the Coronavirus by cutting funds &amp; workforces of agencies that respond to public health emergencies. \n\nHiring freezes, budget cuts, and his anti-health &amp; science agenda threaten efforts to keep Americans safe. https://t.co/07eaHqKTIb'), ('dates', '2020-02-28 19:30:55'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.8594276905059814)])
OrderedDict([('tweet', "5 years ago Boris Nemtsov was killed by Putin's regime for calling out corruption. Today political prisoners are still threatened with the same fate. We must honor Nemtsov's courage by standing for justice &amp; condemning Russia’s on-going human rights abuses https://t.co/5QvywPU6Tx"), ('dates', '2020-02-27 22:59:32'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.7562148571014404)])
OrderedDict([('tweet', 'In the year since #HR8 passed the House, gun violence has taken countless lives with 418 mass shootings in 2019 alone. #BackgroundChecksSaveLives by closing loopholes to keep guns out of the wrong hands. Yet @senatemajldr continues to block this bill in the Senate #EnoughisEnough'), ('dates', '2020-02-27 19:08:21'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.7086063027381897)])
OrderedDict([('tweet', '"Never again" is the promise we made to stand up &amp; fight against hate, violence &amp; genocide – that’s what’s happening in Burma. \n\nThe Burma Human Rights and Freedom Act makes it clear that the US will stand behind the #Rohingya people &amp; hold human rights violators accountable. https://t.co/7wkVihWdBc'), ('dates', '2020-02-25 18:44:55'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.7222151756286621)])
OrderedDict([('tweet', 'Attorney General Barr has been the personal attorney for the president rather than America’s Attorney General. He’s crossed the line, and in doing so has soiled the reputation of the DOJ and trust in its independence. He needs to resign. https://t.co/Ts8HW4fm6R'), ('dates', '2020-02-18 18:41:35'), ('file_name', 'Benjamin L. Cardin.csv'), ('pct_true', 0.5888654589653015)])
OrderedDict([('tweet', 'Today, I joined @RepBobbyRush in calling on @EPA to secure equal rights to clean air protections, because COVID-19 and climate change prove that #CleanAirMatters.'), ('dates', '2020-04-22 18:50:18'), ('file_name', 'Bennie G. Thompson.csv'), ('pct_true', 0.6581540703773499)])
OrderedDict([('tweet', 'ICYMI: The current bill up for consideration is in the Senate. The House is working on our own bill.'), ('dates', '2020-03-23 17:21:02'), ('file_name', 'Bennie G. Thompson.csv'), ('pct_true', 0.6617999076843262)])
OrderedDict([('tweet', 'Democrats are not supporting any bill that only protects the rich and corporations. We want to expand unemployment benefits, include student loan debt, and protections for the working class. Let us keep working until hardworking Americans are taken care of during this time.'), ('dates', '2020-03-23 16:48:30'), ('file_name', 'Bennie G. Thompson.csv'), ('pct_true', 0.928234875202179)])
OrderedDict([('tweet', '...but there’s no reason to panic. Pence leading this is enough reason to do so. https://t.co/JjvqnXVkci'), ('dates', '2020-03-07 00:25:27'), ('file_name', 'Bennie G. Thompson.csv'), ('pct_true', 0.5252840518951416)])
OrderedDict([('tweet', 'Trump’s budget hurts Black students – Proposes $170 billion cut to student loan programs over 10 years and eliminates the Public Service Loan Forgiveness program. #TrumpBudget'), ('dates', '2020-02-13 01:25:28'), ('file_name', 'Bennie G. Thompson.csv'), ('pct_true', 0.9794202446937561)])
OrderedDict([('tweet', '20 million people lost their jobs last month. \n\nThe American people do not want their health care tied to their employment.\n\nThey want Medicare for All. https://t.co/Y8Zb2BOZDU'), ('dates', '2020-05-09 13:32:50'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.6890442967414856)])
OrderedDict([('tweet', "This is cronyism at its worst: A Postmaster General who has no experience delivering mail—just huge campaign contributions to Trump.\n\nWe cannot let this president exploit a horrific pandemic to privatize the Postal Service or bust postal workers' unions. https://t.co/dJcNZdBZps"), ('dates', '2020-05-07 14:44:49'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5379186868667603)])
OrderedDict([('tweet', 'Trump said “great nations do not fight endless wars,” and yet he has now vetoed two bipartisan War Powers Resolutions to keep us in those wars.\n\nCongress must continue its efforts to rein in this lawless president. https://t.co/ISC32gS6UO'), ('dates', '2020-05-07 13:31:31'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.6347529292106628)])
OrderedDict([('tweet', 'Outrageous. Instead of spending hundreds of millions of dollars on dividends to enrich wealthy shareholders, Caterpillar, Black &amp; Decker, Levi Strauss and other corporations should be using this money to compensate the thousands of workers they laid off. https://t.co/MUArk4FefD'), ('dates', '2020-05-06 13:02:00'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.6970964670181274)])
OrderedDict([('tweet', "There is no excuse for medical workers in the richest country on Earth to lack masks, gloves, gowns and tests.\n\nIf Trump won't act, Congress must. @RepRoKhanna and I are introducing legislation to rapidly manufacture all the medical equipment we need: https://t.co/nQESBMcRXW"), ('dates', '2020-05-04 15:03:12'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5141239166259766)])
OrderedDict([('tweet', 'There are legitimate questions we must ask about the Chinese government’s inability to contain the virus. But Pompeo’s incitement is a blatant effort to distract us from questions about Trump’s own disastrous failure. https://t.co/ucuic8qB2G'), ('dates', '2020-05-04 12:43:01'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.837813675403595)])
OrderedDict([('tweet', 'More than 30 million Americans have lost their jobs because of this horrific pandemic, yet they are still expected to cobble up enough income to pay the rent that is due today. That is unacceptable.\n\nIt is time to #CancelRent and \n#CancelMortgages until this crisis is over.'), ('dates', '2020-05-01 22:35:00'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5684230327606201)])
OrderedDict([('tweet', 'If the White House Gift Shop is going to produce $100 COVID-19 coins, Trump can sure as hell utilize the Defense Production Act to manufacture the gloves, gowns, and masks our medical workers desperately need. https://t.co/Wr6Y1uwG0i'), ('dates', '2020-04-29 21:26:10'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5074599385261536)])
OrderedDict([('tweet', 'We have a president forcing hazardous meat plants to reopen, threatening workers\' health.\n\nWe have a Labor Department siding with corporations over workers\' safety.\n\nDisgusting. Maybe this is too "radical," but we need a White House that protects public health during a pandemic. https://t.co/g3DnKcETqf'), ('dates', '2020-04-29 16:33:32'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.9267951250076294)])
OrderedDict([('tweet', 'It is disgusting that Trump would exploit this pandemic to bust postal unions and privatize the most popular government agency in America.\n\nIf we can bail out large corporations, we can damn well save the Postal Service and protect these essential jobs. https://t.co/ObYKqnqPsS'), ('dates', '2020-04-24 17:17:38'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.8062108159065247)])
OrderedDict([('tweet', 'Leader McConnell: You cannot give $500 billion to bail out large corporations and then tell state governments to go bankrupt fighting this deadly pandemic.\n\nWhatever that is, it is certainly not the leadership the American people want. https://t.co/FfZJCZSm2S'), ('dates', '2020-04-24 12:01:00'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5119757056236267)])
OrderedDict([('tweet', 'American taxpayers put tens of millions of dollars into developing remdesivir. Any coronavirus treatment or vaccine must be made free for everybody.\n\nWe will not tolerate profiteering by Gilead or any other pharmaceutical corporation right now. Human life comes first. https://t.co/3lvDKZUKMA'), ('dates', '2020-04-19 15:00:04'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5518400073051453)])
OrderedDict([('tweet', 'Trump’s decision to cut funding to the World Health Organization—in the middle of a pandemic—endangers American lives. And his own officials know it.\n\nThis is a global crisis. We have got to strongly support multilateral institutions like the WHO to confront it. https://t.co/027gbIMRd2'), ('dates', '2020-04-16 15:36:09'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.7249047160148621)])
OrderedDict([('tweet', 'This is an economic meltdown. Millions of Americans have no food in the pantry to feed their kids. No money for rent. Can\'t pay their mortgage, student debt, or medical bills.\n\nWe cannot say, "Let\'s wait and see." We need to get help to our people now and protect their wellbeing.'), ('dates', '2020-04-05 19:58:00'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5818167328834534)])
OrderedDict([('tweet', 'The White House wanted to give Indian Country $0 to deal with the coronavirus. Democrats fought for and got $10 billion, but Native American communities need far more. The next coronavirus package must prioritize tribal health care and economic support. https://t.co/cylzxDSQma'), ('dates', '2020-04-02 21:01:44'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.6066160202026367)])
OrderedDict([('tweet', "Amazon's warehouse workers protested because people are getting sick on the job.\n\nTheir demands were not radical: a safer workplace, protective gear and paid sick leave.\n\nAmazon’s response? Retaliate by firing a worker who helped organize the walk-out. That is absolutely immoral. https://t.co/qBfGrDx53l"), ('dates', '2020-04-01 21:43:41'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.625621497631073)])
OrderedDict([('tweet', "Coronavirus has killed 2,600 in Iran, but U.S. sanctions are obstructing medicine and aid from getting in.\n\n@SecPompeo and @stevenmnuchin1: Pandemics know no borders. Let us put aside our countries' disputes and lift sanctions to reduce human suffering. https://t.co/fRFndKSP39"), ('dates', '2020-03-31 19:15:33'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.71552973985672)])
OrderedDict([('tweet', 'I stand with GE’s workers who are calling on the company to convert its factories to produce ventilators.\n \nNo more layoffs, outsourcing, and tax dodging, @generalelectric. Put Americans to work manufacturing the life-saving ventilators we need. https://t.co/wCbDrCmWZ9'), ('dates', '2020-03-30 19:21:34'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5976656079292297)])
OrderedDict([('tweet', 'The American people increasingly understand that now is the time to think boldly and act decisively.\n\nWe must move rapidly to a Medicare for All, single-payer system that treats health care as a human right. https://t.co/mCe3ehQlNW'), ('dates', '2020-03-30 15:49:28'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.6391709446907043)])
OrderedDict([('tweet', 'How do the geniuses at Trump’s EPA respond to Americans dealing with a deadly respiratory disease? They allow power plants and factories to spew even more air pollution! Absolutely obscene. https://t.co/V2sUFPDtva'), ('dates', '2020-03-29 16:30:00'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5464673638343811)])
OrderedDict([('tweet', 'It is an outrage that debt collectors are garnishing wages and Social Security checks for unpaid medical bills and student loans.\n\nWe must cancel all medical and student debt—and stop all debt collection right now.\n\nA pandemic is not a time for greed. https://t.co/vTJDnxEpvo'), ('dates', '2020-03-28 21:02:35'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.7395497560501099)])
OrderedDict([('tweet', 'This bill was far from perfect, and we must go much further. But Democrats did a good job standing up to Trump and McConnell to make sure this relief package got help to workers, small businesses, hospitals, and states and cities that desperately need it. https://t.co/9GsMc7GTId'), ('dates', '2020-03-28 19:01:00'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.62504643201828)])
OrderedDict([('tweet', 'We will not tolerate our people going bankrupt to get treated for coronavirus or any other damn illness.\n\nWe need Medicare for All. https://t.co/n8zKK4WXVo'), ('dates', '2020-03-28 17:47:48'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.7420027256011963)])
OrderedDict([('tweet', "We must not forget that 27,000 young Dreamers work as doctors, nurses, paramedics, pharmacists, and other health providers as we are fighting the coronavirus pandemic.\n\nWe have got to stand with them and block Trump's insane effort to end the DACA program.\nhttps://t.co/C5QfAnEsFF"), ('dates', '2020-03-27 21:54:03'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.521432638168335)])
OrderedDict([('tweet', 'My Republican colleagues had no problem voting for a trillion dollars in tax breaks for billionaires and large corporations.\n\nBut—horrors of horrors!—poor workers might get a few bucks more in unemployment in the midst of a terrible crisis. Oh my god, the universe is collapsing! https://t.co/KQF1I5frWM'), ('dates', '2020-03-27 01:12:49'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.6348916292190552)])
OrderedDict([('tweet', 'In this crisis, what the American people want is for us to use our taxpayer dollars in every way that we can to protect the working families of this country and the 50 percent of Americans who are living paycheck to paycheck, not just the billionaires and large corporations. https://t.co/U3BF9lP9UK'), ('dates', '2020-03-26 02:08:39'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5826224088668823)])
OrderedDict([('tweet', 'Unless Republican Senators drop their objections to the coronavirus legislation, I am prepared to put a hold on this bill until stronger conditions are imposed on the $500 billion corporate welfare fund. https://t.co/7X0o9C4BoO'), ('dates', '2020-03-25 19:29:46'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.5590280890464783)])
OrderedDict([('tweet', 'It is insane and unacceptable that the Trump Administration has given the Gilead pharmaceutical corporation a seven-year monopoly on a potential coronavirus treatment.\n\nWe will not tolerate profiteering. Any treatment or vaccine must be made free for all. https://t.co/VCdgBSZJw6'), ('dates', '2020-03-24 15:37:09'), ('file_name', 'Bernard Sanders.csv'), ('pct_true', 0.6260678768157959)])
OrderedDict([('tweet', 'In the midst of a generational public health crisis, the Trump admin is still urging SCOTUS to strike down the ACA. Even as millions of Americans lose coverage, @realDonaldTrump would rather upend our health system just to score cheap political points. https://t.co/MxTwnS3jUQ'), ('dates', '2020-05-06 22:34:00'), ('file_name', 'Betty McCollum.csv'), ('pct_true', 0.8927194476127625)])
OrderedDict([('tweet', 'ICYMI: If President Trump had Americans’ health &amp; safety top of mind, he’d strengthen air protections during a global pandemic, not weaken them. \n\nOur health – and the health of our environment – depends on us sounding the alarm &amp; demanding better.\nhttps://t.co/9aj8IXFG52'), ('dates', '2020-05-01 16:32:00'), ('file_name', 'Betty McCollum.csv'), ('pct_true', 0.5056968927383423)])
OrderedDict([('tweet', 'Pres. Trump was repeatedly briefed &amp; warned by U.S. officials, yet he chose to ignore #COVID19, holding political rallies instead – and now America suffers.'), ('dates', '2020-04-14 23:30:18'), ('file_name', 'Betty McCollum.csv'), ('pct_true', 0.8963811993598938)])
OrderedDict([('tweet', 'Imagine we put all this money into keeping a restaurant open, but we don’t have sanitation workers and garbage builds up in front of the restaurant and rats are running around the garbage, no one is going to eat at that restaurant.\n\nhttps://t.co/LKqiEPckB6'), ('dates', '2020-05-08 18:47:55'), ('file_name', 'Bill Cassidy.csv'), ('pct_true', 0.6161835193634033)])
OrderedDict([('tweet', 'Businesses are suffering &amp; state &amp; local govts are watching their revenue dry up, leaving them scrambling to find ways to pay police, fire &amp; sanitation workers.\n\nWe need those workers on the job if we’re going to have a recovery, &amp; that’s why we need the $500 billion SMART Fund. https://t.co/9AbFnkVOJw'), ('dates', '2020-05-06 16:47:03'), ('file_name', 'Bill Cassidy.csv'), ('pct_true', 0.5673599243164062)])
OrderedDict([('tweet', 'Congress need to pass the $500B SMART Fund proposed by @SenatorMenendez and me. Our economy won’t reopen if small businesses are still shuttered because our cities are bankrupt due to being unable to maintain garbage pickup and policing. Let’s put our small businesses first. https://t.co/BLh2wgJZ92'), ('dates', '2020-04-28 15:30:13'), ('file_name', 'Bill Cassidy.csv'), ('pct_true', 0.6280341148376465)])
OrderedDict([('tweet', 'The Senate has approved more money for the #PPP, which ran out days ago but was not refilled because of Washington politics. I’m glad @SenSchumer and @SpeakerPelosi were finally forced to put aside politics to help rescue small businesses, churches and charities from bankruptcy.'), ('dates', '2020-04-21 22:09:40'), ('file_name', 'Bill Cassidy.csv'), ('pct_true', 0.5625717639923096)])
OrderedDict([('tweet', 'Millions of Americans are losing their jobs, meanwhile @SpeakerPelosi is bragging about blocking more funding for the PPP while eating ice cream from her $24,000 fridge. How low are you willing to go, Nancy? https://t.co/JOHaJY3SgB'), ('dates', '2020-04-21 18:23:41'), ('file_name', 'Bill Cassidy.csv'), ('pct_true', 0.7966691255569458)])
OrderedDict([('tweet', 'Ten years ago today, the #DeepwaterHorizon oil rig exploded in the Gulf of Mexico, tragically killing 11 workers and spilling nearly 5 million barrels of oil into the Gulf of Mexico in the largest oil spill in U.S. history. #BP10'), ('dates', '2020-04-20 15:06:48'), ('file_name', 'Bill Cassidy.csv'), ('pct_true', 0.7723561525344849)])
OrderedDict([('tweet', 'I’m receiving text messages, phone calls &amp; emails from #smallbiz owners who wonder if more money will be put into the #PaycheckProtectionProgram. Republicans have been trying to do so for a week.\n\n@SpeakerPelosi &amp; @SenSchumer are holding it hostage, trying to get other priorities https://t.co/rpAOh68Jcu'), ('dates', '2020-04-16 23:41:18'), ('file_name', 'Bill Cassidy.csv'), ('pct_true', 0.6973585486412048)])
OrderedDict([('tweet', '.@SBAgov can no longer accept new applications for the #PaycheckProtectionProgram because money has run out. \n\nRepublicans have tried to replenish this fund for weeks, but @SenSchumer and @SpeakerPelosi blocked it.'), ('dates', '2020-04-16 15:28:06'), ('file_name', 'Bill Cassidy.csv'), ('pct_true', 0.6343634724617004)])
OrderedDict([('tweet', 'We strongly suspect that once somebody has contracted #coronavirus, he or she is now immune. That immunity would allow them to return to work — if they’re a nurse, they can go see a #COVIDー19 patient without fear of being infected or infecting others.\n\nhttps://t.co/lMXInuA0kq'), ('dates', '2020-04-13 14:23:14'), ('file_name', 'Bill Cassidy.csv'), ('pct_true', 0.6209058165550232)])
OrderedDict([('tweet', 'Just like @SenSchumer and @Speaker Pelosi delayed Phase 3 of the #coronavirus response, they’re now blocking a bipartisan bill that would provide billions more that Americans need to get through this epidemic. This is not the time for their political games. https://t.co/7CX0BEatvQ'), ('dates', '2020-04-09 16:36:53'), ('file_name', 'Bill Cassidy.csv'), ('pct_true', 0.6349799633026123)])
OrderedDict([('tweet', 'Instead of trying to "squeeze" Republicans by demanding billions of dollars on liberal fantasies that don\'t address the crisis, Democrats could actually support policies that help the millions of Americans hurting as a result of this pandemic.\nhttps://t.co/g49GMHFfP4'), ('dates', '2020-05-05 16:22:18'), ('file_name', 'Bill Flores.csv'), ('pct_true', 0.7910515666007996)])
OrderedDict([('tweet', 'Texas had more applicants for the Paycheck Protection Program than any other state.  This is good news for the millions of small businesses that were hurt by Democrats refusing to fund the program two weeks ago.\nhttps://t.co/NglaEvmPGj'), ('dates', '2020-04-24 20:29:25'), ('file_name', 'Bill Flores.csv'), ('pct_true', 0.5243462920188904)])
OrderedDict([('tweet', 'Today I voted to increase funding for the Paycheck Protection Program, an important lifeline for small businesses around the country hardest hit by the Coronavirus.  It is shameful that House Democrats made these businesses wait two weeks for this funding.'), ('dates', '2020-04-23 21:10:17'), ('file_name', 'Bill Flores.csv'), ('pct_true', 0.5191500186920166)])
OrderedDict([('tweet', 'The Paycheck Protection Program has helped millions of small businesses stay afloat and pay their employees.  It is shameful that @SpeakerPelosi is holding American workers and businesses hostage, doing even further damage to our economy.\nhttps://t.co/meefquPWgK'), ('dates', '2020-04-16 14:48:00'), ('file_name', 'Bill Flores.csv'), ('pct_true', 0.5424506664276123)])
OrderedDict([('tweet', 'Our small businesses are relying on the Paycheck Protection Program to keep employees on payroll and stay afloat.  It is absolutely shameful that @SpeakerPelosi and @SenSchumer have decided to block desperately needed funding for partisan reasons.'), ('dates', '2020-04-09 14:50:42'), ('file_name', 'Bill Flores.csv'), ('pct_true', 0.6395999193191528)])
OrderedDict([('tweet', 'Trump may have abandoned his responsibility to lead in the midst of this pandemic, but our local governments are on the front lines. Why won’t @senatemajldr put #FamiliesFirst and let Congress make sure communities nationwide have the resources they need to keep up the fight?'), ('dates', '2020-05-06 19:15:21'), ('file_name', 'Bill Foster.csv'), ('pct_true', 0.9248433113098145)])
OrderedDict([('tweet', "All due respect @GOPLeader, but a pandemic is an outbreak of a dangerous disease affecting a high proportion of the population - like the current one we're facing that has already killed more Americans than the Vietnam War.\n\nComparing it to litigation is offensive. https://t.co/JbN9v5KZX3"), ('dates', '2020-05-02 23:05:10'), ('file_name', 'Bill Foster.csv'), ('pct_true', 0.6115697622299194)])
OrderedDict([('tweet', "It's been a month since the House passed the Families First #Coronavirus Response Act which guarantees free diagnostic and antibody testing for everyone - yet the Trump administration still has no procedures in place?\n\nUnacceptable. https://t.co/fAlPiu2frw"), ('dates', '2020-04-14 18:20:31'), ('file_name', 'Bill Foster.csv'), ('pct_true', 0.5222064852714539)])
OrderedDict([('tweet', "President Trump's shameful attempt to use the distraction of #COVID19 to dismantle fundamental checks and balances of government will not go unnoticed.\n\nHis late-night firing of the IG is an assault on oversight, accountability, and the rule of law.\n\nhttps://t.co/YBKKL19F9J"), ('dates', '2020-04-04 18:47:06'), ('file_name', 'Bill Foster.csv'), ('pct_true', 0.6888123154640198)])
OrderedDict([('tweet', '.@POTUS has signed legislation restoring critical funding for the #PaycheckProtectionProgram. The #PPP has been a lifeline for #WestMichigan small businesses and the thousands upon thousands of individuals they employ! Now it’s time for Congress to focus on the #HEROESAct!'), ('dates', '2020-04-24 16:48:34'), ('file_name', 'Bill Huizenga.csv'), ('pct_true', 0.5987537503242493)])
OrderedDict([('tweet', ".@wwmtnews: #WestMichigan businesses stuck after Paycheck Protection Program funding runs out. \n\nThis shouldn't be the case. Please remember Senate Democrats blocked a clean increase in #SmallBizRelief via the #PPP over a week ago. \nhttps://t.co/VcGdfHYuVg"), ('dates', '2020-04-17 21:06:48'), ('file_name', 'Bill Huizenga.csv'), ('pct_true', 0.7229934334754944)])
OrderedDict([('tweet', 'And now we have a 1,200 page “Pelosi only” bill being hawked in the House! They have truly managed to turn this health crisis into a economic fiasco. https://t.co/qC1zPAXKG7'), ('dates', '2020-03-23 16:42:00'), ('file_name', 'Bill Huizenga.csv'), ('pct_true', 0.5025278925895691)])
OrderedDict([('tweet', 'This is why the Senate needs to pass a COVID19 response, not a Christmas tree bill with everyone’s pet projects, ASAP! Keeping people on a payroll is far less expensive than putting people on unemployment and then trying to get them back into the workforce. Support small biz! https://t.co/e2K9ehqJhU'), ('dates', '2020-03-23 14:36:31'), ('file_name', 'Bill Huizenga.csv'), ('pct_true', 0.6800108551979065)])
OrderedDict([('tweet', 'As Chinese officials continue to promote conspiracy theories regarding #COVID19 it is utterly ridiculous for China to try and draw parity on this issue. It is crystal clear what the communist regime is doing.  (3/3)'), ('dates', '2020-03-17 17:56:46'), ('file_name', 'Bill Huizenga.csv'), ('pct_true', 0.8792261481285095)])
OrderedDict([('tweet', "This isn't reciprocity, it's about control. American journalists work for independent organizations. The Chinese outlets here in America typically act as an extension of China's Communist Government.  (2/3)"), ('dates', '2020-03-17 17:56:17'), ('file_name', 'Bill Huizenga.csv'), ('pct_true', 0.9779517650604248)])
OrderedDict([('tweet', 'Banning American journalists from the @nytimes, @WSJ, and @washingtonpost from reporting in China is not only wrong, it is dangerous. This is how the communist regime in China controls the news during a pandemic.  (1/3)'), ('dates', '2020-03-17 17:55:46'), ('file_name', 'Bill Huizenga.csv'), ('pct_true', 0.8297401070594788)])
OrderedDict([('tweet', 'What’s outrageous, @RepJerryNadler , is that he was even prosecuted in the first place. Storm clouds are brewing for those who pushed the Russian Collusion delusion. https://t.co/pFBVsbrzHM'), ('dates', '2020-05-07 22:03:31'), ('file_name', 'Bill Johnson.csv'), ('pct_true', 0.6443820595741272)])
OrderedDict([('tweet', 'I’m headed back to Washington to finally vote to replenish the #PPP program. It’s been out of money for too long because Democrats chose to block funding for this program to push their political agendas. \n\nAmericans need help now, and we must deliver for them.'), ('dates', '2020-04-22 17:29:45'), ('file_name', 'Bill Johnson.csv'), ('pct_true', 0.6124695539474487)])
OrderedDict([('tweet', 'This could have all been avoided if @SenSchumer and @SpeakerPelosi prioritized American small businesses and families over their own political agendas. https://t.co/FOeCg2BNVi'), ('dates', '2020-04-16 14:56:48'), ('file_name', 'Bill Johnson.csv'), ('pct_true', 0.9617733955383301)])
OrderedDict([('tweet', 'The @WHO was wrong about #Covid_19 from the beginning. They bought China’s blatant lies. This cost the world precious time and many people their lives. It’s important that we hold this organization accountable, and freezing their funding, for now, is an appropriate step.'), ('dates', '2020-04-15 13:41:27'), ('file_name', 'Bill Johnson.csv'), ('pct_true', 0.9897719025611877)])
OrderedDict([('tweet', 'The coronavirus has been a nuclear bomb to our economy. \n\nWorkers are hurting. \nSmall businesses are hurting. \nThousands of Americans are sick.\n\nAmericans are pitching in to help and I will vote yes on the #CARESAct to provide real relief NOW!\n\nhttps://t.co/yEmpeGrrFp'), ('dates', '2020-03-27 16:20:41'), ('file_name', 'Bill Johnson.csv'), ('pct_true', 0.6264033913612366)])
OrderedDict([('tweet', 'We need to hold China accountable for mismanaging the #COVID19 outbreak, which is why I am an original cosponsor of this bill. https://t.co/2dcu4XYzu0'), ('dates', '2020-03-24 16:25:43'), ('file_name', 'Bill Johnson.csv'), ('pct_true', 0.9144193530082703)])
OrderedDict([('tweet', 'Democrats are treating this national emergency as a golden political opportunity.  Its disgusting. Here’s where we stand, and why: https://t.co/EOo3sWnHqn'), ('dates', '2020-03-23 19:08:20'), ('file_name', 'Bill Johnson.csv'), ('pct_true', 0.8899883031845093)])
OrderedDict([('tweet', 'Nancy Pelosi rode back into Washington and derailed a bipartisan deal that Senate Majority Leader Mitch Mitch McConnell and Minority Leader Chuck Schumer had agreed to.  A deal that would have provided cash for struggling and frightened American workers...'), ('dates', '2020-03-23 02:25:23'), ('file_name', 'Bill Johnson.csv'), ('pct_true', 0.9607334733009338)])
OrderedDict([('tweet', 'This virus is not the fault of Republicans.\n \nThis virus is not the fault of Democrats. \n \nThis virus is not the fault of any demographic here in America.\n \nThere is only one entity at fault: communist China who refused to tell the world about the virus until it was too late.'), ('dates', '2020-03-19 14:53:55'), ('file_name', 'Bill Johnson.csv'), ('pct_true', 0.9756672978401184)])
OrderedDict([('tweet', 'The House is scheduled to leave today and return home for a district work period. \n\n@SpeakerPelosi should not put the House in recess until there is bipartisan support for serious legislation that helps to address the #coronavirus and provide a boost to the economy.'), ('dates', '2020-03-12 20:39:08'), ('file_name', 'Bill Johnson.csv'), ('pct_true', 0.6523634195327759)])
OrderedDict([('tweet', 'Trump’s attacks on voting aren’t funny because taken to their logical conclusion his endless lies about voter “fraud” threaten the legitimacy of every election and democracy in America. https://t.co/BxxtYw2tdG'), ('dates', '2020-05-09 19:37:23'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7131643891334534)])
OrderedDict([('tweet', 'Because of the pandemic and the trump administration’s total failure Americans are enduring the worst crisis since the 1930s. \n\nThe suffering is unfathomable.\n\nSubstantial federal aid to our cities, states, and unemployed must continue. Now isn’t the time to pump brakes. https://t.co/fSY3OfjaAS'), ('dates', '2020-05-08 13:48:07'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.6680821776390076)])
OrderedDict([('tweet', 'A revealing remark. To barr + other trump stooges there is no right or wrong or truth or lie, there is just power: those who wield it and those beneath it. \n\nThis poisonous worldview is how *your* AG turns his office into a corrupt exec’s personal lawyer. #ImpeachBarr #DisbarBarr https://t.co/W052VrFfpg'), ('dates', '2020-05-08 00:05:23'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.8610312342643738)])
OrderedDict([('tweet', 'Mike Flynn is a convicted felon. Flynn confessed. Flynn was on Putin’s payroll. A federal judge said Flynn ‘sold out’ America and all but called him a traitor.\n\nAG Barr has converted DOJ into trump’s personal law firm and has put trump and his flunkies above all laws. https://t.co/jGKKjXk0Rl'), ('dates', '2020-05-07 20:15:06'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7190855741500854)])
OrderedDict([('tweet', 'As trump and republicans increasingly deflect their failures onto China do not forget that trump has spent over three years sucking up to the Chinese communist party and parroting their propaganda. https://t.co/tsvFsF9tnQ'), ('dates', '2020-05-07 15:01:02'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.9574217796325684)])
OrderedDict([('tweet', "These latest allegations by the whistleblower Dr. Bright paint a picture of astonishing incompetence, rampant self-dealing, and outright corruption. Trump's government is completely and totally out of control and Americans will continue to die from its unchecked failures. https://t.co/bdObmeLaj2"), ('dates', '2020-05-05 22:59:27'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7222444415092468)])
OrderedDict([('tweet', "America is #1 in the world in deaths and infections because of trump's catastrophic failures and malicious stupidity. This is the rambling of a con artist who has no idea what he's doing and doesn't care. https://t.co/qCXscQiJ14"), ('dates', '2020-05-05 15:31:37'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.6726943850517273)])
OrderedDict([('tweet', 'Trump’s vile attacks on honest experts is the fruit of decades of republican attacks to destroy the civil service &amp; enable corruption. \n\nFailures to meet the covid challenge are part a product of the gop’s vandalism against functional govt. Quoting Pres Obama, the gop built that! https://t.co/iq04aEPUPj'), ('dates', '2020-05-02 17:55:48'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5031617283821106)])
OrderedDict([('tweet', 'This virus has landed on Jersey like a sledgehammer. Bc of a screwy formula the 1st allocation of CARES Act money was a bad joke and our delegation raised hell.\n\nThis evening’s allocation of $1.7B to our hospitals finally prioritizes states like #NJ carrying the heaviest burdens. https://t.co/LTaIf56D8f'), ('dates', '2020-05-01 23:28:46'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5713850855827332)])
OrderedDict([('tweet', "Trump's Tom Hagen AG Barr continues to move heaven and earth to keep Congress from seeing the complete Mueller Report chronicling trump's treachery.\n\nWhat on earth are they hiding? https://t.co/P7Z4mui6sy"), ('dates', '2020-05-01 21:38:42'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5703344345092773)])
OrderedDict([('tweet', 'Donald trump has never had the slightest interest in being leader to people he thinks didn’t vote for him. In fact he goes out of his way to divide the nation and hurt people, even in a pandemic.\n\nYou can’t get much more un-American than that. https://t.co/hnwLzyXPPO'), ('dates', '2020-05-01 00:24:16'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7024644017219543)])
OrderedDict([('tweet', 'Corrupt trump was impeached for trying to extort a foreign nation. He’s doing the same thing only now to tens of millions of Americans during a pandemic.\n\nEvery republican in Congress but one voted to keep this disgrace in office invited this. https://t.co/sHeIsr7U53'), ('dates', '2020-04-30 22:11:48'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7864047884941101)])
OrderedDict([('tweet', 'Trump and his flunkies are lying about Mike Flynn.\n\nMichael Flynn is a convicted felon. Here’s his confession. Flynn was on Putin’s payroll. A federal judge said Flynn “sold out” America and all but called him a traitor.\n\nThis is who trump and republicans want to rehabilitate. https://t.co/LIgtqjO4Et https://t.co/a5kbPKVW7r'), ('dates', '2020-04-30 21:13:51'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.770878791809082)])
OrderedDict([('tweet', "The hostility trump and his goons have for regular Americans has no bottom. Now they're trying to make it easier for Americans to fall prey to predatory payday lenders. https://t.co/w1CBBN8Cx1"), ('dates', '2020-04-29 19:49:40'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7701973915100098)])
OrderedDict([('tweet', "The Chinese communist party is using the cover of the pandemic to continue its violent crackdown on Hong Kong through a power grab to crush HK's democratic hopes. I #StandWithHongKong against the Chinese communist party and so do the American people.\nhttps://t.co/2PHkooouX0"), ('dates', '2020-04-25 16:01:55'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.9476550221443176)])
OrderedDict([('tweet', "After brazenly lying about the Mueller Report's findings, AG Barr continues to view trump as a king and keeps moving heaven and earth to cover up the full report from Congress. https://t.co/JdPyyyvkMU"), ('dates', '2020-04-24 21:04:07'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.8772475123405457)])
OrderedDict([('tweet', 'Americans rank the post office maybe the most popular arm of government, it employs 600,000 Americans, and has been around since Benjamin Franklin.\n\nTrump wants to destroy it during a pandemic while bailing out Big Oil billionaires and cruise ships. No. https://t.co/0ibQrMBiMG'), ('dates', '2020-04-24 17:11:24'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.833823561668396)])
OrderedDict([('tweet', 'Trump and his goons want the states and the post office to go bankrupt but want you to pay for a Big Oil Bailout for his billionaire buddies. Really.\nhttps://t.co/i9ZilxWDwz'), ('dates', '2020-04-24 13:05:37'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.8545002341270447)])
OrderedDict([('tweet', '🚨 🚨 This is a major scandal. \n\nTrump &amp; his stooges ousted 1 of the nation’s leading vaccine experts bc he refused to pimp trump’s miracle drug &amp; insisted on “science not politics or cronyism.”\n\nThere are now 46,900 dead Americans. Trump is intent to make that number even higher https://t.co/1we8NZozSZ'), ('dates', '2020-04-22 20:47:11'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5891028642654419)])
OrderedDict([('tweet', "Mcconnell's wretched slur is slathered in the trumpian nihilism that targets Americans they think don't vote republican. Democracy can't survive a pandemic if one party is at open war w/Americans.\n\nI never forgot those who refused to help NJ after Hurr. Sandy. I won't forget this https://t.co/Ski9XQy92E"), ('dates', '2020-04-22 19:46:11'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.8209229707717896)])
OrderedDict([('tweet', 'This is what the republican party thinks of every American. To the gop, helping communities, hospitals, and suffering Americans in a pandemic is a luxury but bailing out big oil and airlines and packing the courts are the first and only priorities. https://t.co/xoMPDDcgrn'), ('dates', '2020-04-22 16:29:50'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7744774222373962)])
OrderedDict([('tweet', "Under trump the government has been overrun by unqualified hacks whose only qualification is dog loyalty to trump. \n\nAmerica's stunning inability to be guided by experts in this crisis is a direct result of trump and republicans' deliberate vandalism of our institutions. https://t.co/xcpr8N98mb"), ('dates', '2020-04-22 14:08:24'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.8097108006477356)])
OrderedDict([('tweet', 'Yet more hard evidence these tiny tiny “protests” are trump political rallies funded by right wing billionaires and promoted by right wing propaganda organs. https://t.co/FFn9A24XOY'), ('dates', '2020-04-22 01:25:07'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5720005631446838)])
OrderedDict([('tweet', 'There’s a reason I’ve been calling for Barr’s legal disbarment and impeachment for almost a year. https://t.co/cBBWI0Hm8v'), ('dates', '2020-04-21 17:59:06'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5486226081848145)])
OrderedDict([('tweet', 'Just so you know corrupt trump is again demanding a Big Oil Bailout for his billionaire buddies that he wants *you* to pay for. #NoBigOilBailout https://t.co/CY2L6gFOLC'), ('dates', '2020-04-21 14:12:40'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7038472294807434)])
OrderedDict([('tweet', 'This new “announcement” by trump is not about a policy, or stopping covid, or protecting your life. It’s just another stunt to use some ‘other’ to blame to distract from his own catastrophic failures that are killing Americans. https://t.co/BgJJaZgO85'), ('dates', '2020-04-21 13:45:35'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7399823069572449)])
OrderedDict([('tweet', 'Tonight we have crossed 790,000 infections and 42,000 dead. This corrupt buffoon will will try any poisonous distraction and blame anyone to deflect from his failures that are killing our fellow Americans. https://t.co/m3IFGEhwf7'), ('dates', '2020-04-21 03:47:18'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5583139061927795)])
OrderedDict([('tweet', '"The White House pledged over 2 weeks ago to cover coronavirus treatment for uninsured Americans but the administration still doesn\'t have a plan for how to do it."\n\nNever forget: trump &amp; republicans remain in court *right now* trying to destroy the ACA &amp; steal your health care. https://t.co/lHCHCcnc75'), ('dates', '2020-04-20 17:45:11'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7696295380592346)])
OrderedDict([('tweet', 'Trump’s been sucking up to the Chinese communist party for over three years. https://t.co/MiaIU0VWgb https://t.co/iVmiwPyYmI'), ('dates', '2020-04-20 02:17:29'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5198276042938232)])
OrderedDict([('tweet', 'The deadbeat trump campaign is refusing to pay back $500,000 to yet another city that hosted one of his violence-prone hate rallies.\n\nTrump’s stiffing our cities and states is esp harmful as they need every cent to fight the virus.\nhttps://t.co/5BPgwWhTXx'), ('dates', '2020-04-19 21:59:04'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7099762558937073)])
OrderedDict([('tweet', 'The gop doesn’t give a rat’s tail about deficits.\n\nRepublicans’ only principle is that deficits are intolerable under a Democratic President and perfectly fine when a republican is in office. \n\nNo one should ever fall for this bait-and-switch lie again.\nhttps://t.co/HCuBaLhO0T'), ('dates', '2020-04-19 19:38:59'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.7985809445381165)])
OrderedDict([('tweet', 'Never forget that trump has spent his entire time in office groveling to the ruling dictator of the Chinese communist party. His China-bashing is cynical lies to distract you from his catastrophic failure. https://t.co/mTqlliw77A'), ('dates', '2020-04-18 22:52:46'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.9319773316383362)])
OrderedDict([('tweet', 'Most Americans recognize what needs to be done to stop the virus and are making sacrifices.\n\nThese tiny zombie “protests” are rent-a-crowds coordinated by right wing billionaires to support trump’s call for mob rule and coverup his catastrophic failure. https://t.co/bwKL40UWO9 https://t.co/EfAazYyEMd'), ('dates', '2020-04-18 21:29:13'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.6128280162811279)])
OrderedDict([('tweet', 'As the pandemic grew trump sent millions of masks to his buddies in the Chinese communist party against warnings we would need them here. Americans are dying because our hospitals can’t get masks thanks to trump.\nhttps://t.co/brEm4Uu0OD'), ('dates', '2020-04-18 20:08:00'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.646857738494873)])
OrderedDict([('tweet', 'Two weeks ago @RepJimmyPanetta and I demanded the trump admin cut its absurd paperwork that is delaying stimulus checks to veterans. Today Treasury announced they’re heeding our calls and will start delivering checks to our vets directly. https://t.co/AVgQQUv35n'), ('dates', '2020-04-17 19:33:56'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5913370251655579)])
OrderedDict([('tweet', 'Ticketmaster is refusing to give refunds for indefinitely postponed shows. @RepKatiePorter and I are calling on the heads of @Ticketmaster @LiveNation to stop confiscating fans’ money and demanding they issue full refunds to *all* customers who request them. https://t.co/H8azggOIYD'), ('dates', '2020-04-17 15:55:16'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5245129466056824)])
OrderedDict([('tweet', 'Every word of this statement @SpeakerPelosi just sent to the House caucus membership. We have to be brutally honest about Donald trump’s catastrophic failures that have left America wide open to this pandemic. https://t.co/eSNaYI3XwV'), ('dates', '2020-04-15 01:05:52'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5281575918197632)])
OrderedDict([('tweet', 'America’s public safety officers are risking their lives to protect us. They deserve peace of mind that if they get sick their families won’t be left in the cold. New legislation @RepJerryNadler @RepMaxRose and I just unveiled will do exactly that. https://t.co/WJOYCZSy8O'), ('dates', '2020-04-14 21:15:38'), ('file_name', 'Bill Pascrell, Jr..csv'), ('pct_true', 0.5643519163131714)])
OrderedDict([('tweet', '(1/2) While we are unsure of #COVID19’s exact origins, it is clear that China’s gross negligence and deception has led to the deaths of hundreds of thousands around the world. We must hold them accountable.'), ('dates', '2020-05-08 20:11:15'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.5639170408248901)])
OrderedDict([('tweet', '4.4 million Americans lost their jobs last week because Dems put their political agendas first &amp; let the #PPP run out of money. \n\nMany of these lay-offs could’ve been avoided if Dems had been as concerned w/ the well-being of American workers as they were w/ losing their leverage https://t.co/gnaVWiCVtd'), ('dates', '2020-04-23 14:36:04'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.9570605754852295)])
OrderedDict([('tweet', 'Countless Americans lost their jobs because Democrats blocked replenishing #PPP, &amp; Pelosi applauded their efforts. \n\nSmall businesses suffered for two weeks because Dems were more focused on leveraging this crisis to promote their political agendas; that’s on Pelosi and Schumer. https://t.co/MgUT5lL9T3'), ('dates', '2020-04-22 17:09:38'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.9852244853973389)])
OrderedDict([('tweet', 'I’m so glad that we’re replenishing #PPP, but small businesses shouldn’t have been forced to wait b/c Dems didn’t want to lose their “leverage” or squander a “tremendous opportunity.” \n\nSmall businesses &amp; workers have sustained irreparable damage b/c Dems wanted to play politics. https://t.co/lHyN3uHvbk'), ('dates', '2020-04-21 21:35:53'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.8801152110099792)])
OrderedDict([('tweet', '30 MILLION JOBS have been saved thanks to #PPP, but Speaker Pelosi says there’s “no data as to why we need” to keep funding it. Maybe I’m missing something, but saving millions of American jobs seems like a really good reason to replenish #PPP to me. https://t.co/P0gzuCUcuk'), ('dates', '2020-04-20 22:26:51'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.889162003993988)])
OrderedDict([('tweet', 'Democrats and the media have made inflammatory, hyperbolic, and downright false claims that @realDonaldTrump didn’t respond fast enough to #COVID19, but that’s simply not true. Watch @RepDanCrenshaw debunk this reckless narrative. https://t.co/WJUiKs1rok'), ('dates', '2020-04-20 21:43:29'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.9614615440368652)])
OrderedDict([('tweet', 'It’s been 11 days since Dems blocked a CLEAN bill to replenish the #PPP and 4 days since its funding ran dry. \n\nEnough is enough, @SpeakerPelosi and @SenSchumer. 22 million Americans are now unemployed, and countless more are at risk. Our small businesses and workers can’t wait!'), ('dates', '2020-04-20 20:12:08'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.9318451285362244)])
OrderedDict([('tweet', 'Republicans sounded the alarm over a week ago that the #PPP was running out of funds, &amp; Dems let it happen. \n\nPelosi says there’s “no data as to why” we need to keep funding #PPP. @SteveScalise &amp; I disagree. Share your story here: https://t.co/2np9ofVlAE'), ('dates', '2020-04-17 16:32:42'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.8969500660896301)])
OrderedDict([('tweet', 'Speaker Pelosi says there’s “no data as to why we need” to keep funding #PPP? Tell that to the 34k Missourians &amp; 1.6M small businesses across the nation this program has helped. \n\nThousands of small businesses are still in dire need. We need to replenish this BIPARTISAN fund NOW!'), ('dates', '2020-04-16 16:41:02'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.9245356917381287)])
OrderedDict([('tweet', 'This is the real American Horror Story. \n\nThe fact that the Democrats are playing partisan politics while millions of Americans are jobless proves we’ve reached a new low. https://t.co/LajM0775ZY'), ('dates', '2020-04-16 15:41:06'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.8836185336112976)])
OrderedDict([('tweet', 'The #PPP is officially out of funds, and the Dems let it happen. The fact that small business owners in need have been turned away because Democrats put partisan politics over replenishing this vital program is disheartening and enraging. \n\nOur small businesses deserve better! https://t.co/TQRwXKXHk0'), ('dates', '2020-04-16 15:23:27'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.9239062070846558)])
OrderedDict([('tweet', "17M Americans have been laid off; last week, 6.6M filed for unemployment. Replenishing the #PaycheckProtectionProgram is critical for our small businesses. Americans are suffering &amp; can't wait; now is NOT the time for Dems to play politics. \n\n@GOPLeader is right; it is disgusting https://t.co/ZD9e6NkFq0"), ('dates', '2020-04-09 17:12:05'), ('file_name', 'Billy Long.csv'), ('pct_true', 0.6627551317214966)])
OrderedDict([('tweet', "Instead of holding China accountable for its role in this pandemic, Democrats are using Congress' time &amp; resources for yet another witch hunt &amp; attack on @realDonaldTrump. I look forward serving on the Coronavirus Oversight Committee to hold my Democratic colleagues accountable. https://t.co/5jsLvMVDHM"), ('dates', '2020-05-07 17:41:13'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.9618945121765137)])
OrderedDict([('tweet', "MO businesses like our favorite restaurants, barber shops &amp; florists have been devastated by COVID-19's economic impact &amp; need help weathering this storm. Congress has passed a bill to provide more funding to the PPP &amp; our incredible medical community. https://t.co/GrSDXTMJTf"), ('dates', '2020-04-24 13:34:00'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.6715707778930664)])
OrderedDict([('tweet', "Last week, Democrats blocked a clean bill that would have added much needed funding to the Paycheck Protection Program and yesterday the PPP's funding ran out. Their political games need to stop. American businesses and workers can’t wait.\n \nhttps://t.co/qLeRjWnJuS"), ('dates', '2020-04-17 17:55:10'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.8743025064468384)])
OrderedDict([('tweet', 'Congress is considering drastic - but necessary - measures to stabilize the economy &amp; provide relief to businesses &amp; families. One FREE way to provide relief is to eliminate #CECL, an ill-advised accounting standard restricting lending &amp; sucking billions of $ out of the economy.'), ('dates', '2020-03-19 17:10:22'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.5986346006393433)])
OrderedDict([('tweet', 'In the past few years, MO has exported over $300 million of goods &amp; services worldwide by utilizing the #EXIM Bank. Thankfully, last week Congress reauthorized this important resource that allows small biz to compete in the global marketplace &amp; keeps jobs in America.'), ('dates', '2019-12-27 15:15:53'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.5795495510101318)])
OrderedDict([('tweet', 'House Democrats have achieved nothing more than a massive political divide in America and long-lasting damage to our democracy.\nhttps://t.co/ftoZ3dacOT'), ('dates', '2019-12-19 02:02:55'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.6534479260444641)])
OrderedDict([('tweet', 'Speaker Pelosi said it herself: Democrats started the impeachment of @realDonaldTrump 2.5 years ago. https://t.co/s8a4ak9k57'), ('dates', '2019-12-09 20:20:05'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.5290950536727905)])
OrderedDict([('tweet', "Seeing as Democrats have been trying to impeach @realDonaldTrump since the day he was sworn in, Speaker Pelosi's announcement that the House will proceed with articles of impeachment is not news, just another political stunt in their unconstitutional impeachment “investigation.”"), ('dates', '2019-12-05 18:08:36'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.9627895355224609)])
OrderedDict([('tweet', 'House Dems keep changing the narrative hoping to make even an ounce of progress on their impeachment quest. \n\nFirst they accused @POTUS @realDonaldTrump of "quid pro quo."\nNext, “extortion.”\nNow, it’s “bribery.” \n\nTheir investigation is unraveling. It’s time for the sham to end.'), ('dates', '2019-11-20 14:42:09'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.633538544178009)])
OrderedDict([('tweet', 'We’re now over two hours into the so-called “open hearing” and not a single Republican has been allowed to ask a question.'), ('dates', '2019-11-13 17:22:09'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.5845714807510376)])
OrderedDict([('tweet', 'House Democrats’ tainted impeachment charade drags on today and continues to prevent Congress from legislating. Republicans are ready to pass #USMCA, give our troops a pay raise, and fund the government. Let’s get back to work for the American people.'), ('dates', '2019-11-13 16:46:32'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.7448089718818665)])
OrderedDict([('tweet', "Democrats are relentlessly pursuing impeachment of @realDonaldTrump, who they've been trying to undermine since his 1st day in office. Today’s vote &amp; this investigation are simply political stunts from Democrats who are still aching from the 2016 election https://t.co/OVaXD196EV"), ('dates', '2019-10-31 19:04:55'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.9512327909469604)])
OrderedDict([('tweet', 'The #USMCA will benefit farmers, manufacturers and businesses across the country. House Republicans are ready to pass it, but Democrats continue to use this so-called “impeachment inquiry” as a stall tactic.'), ('dates', '2019-10-29 21:11:00'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.921819806098938)])
OrderedDict([('tweet', 'I am proud to cosponsor @RepAndyBiggsAZ’s resolution to condemn and censure Rep. Adam Schiff. He should be held accountable for his reckless pursuit of impeachment and for continuously misleading the American people.'), ('dates', '2019-10-09 18:54:39'), ('file_name', 'Blaine Luetkemeyer.csv'), ('pct_true', 0.8822183012962341)])
OrderedDict([('tweet', 'How many jobs were lost while Nancy Pelosi slowed down important relief for Americans trying to leverage this crisis for political gain 𝗳𝗼𝗿 𝘁𝗵𝗲 𝘀𝗲𝗰𝗼𝗻𝗱 𝘁𝗶𝗺𝗲? https://t.co/nylmnMgHY3'), ('dates', '2020-04-22 19:01:04'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.6550511121749878)])
OrderedDict([('tweet', 'This could have been avoided but Schumer and Pelosi are blocking a simple, clean funding bill to replenish the #PaycheckProtectionProgram, a lifeline for small businesses in these dire times. https://t.co/2ZzoMnNcwF'), ('dates', '2020-04-16 14:50:10'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.7784008383750916)])
OrderedDict([('tweet', "This should surprise no one. It's why I signed on to support @RepJimBanks' resolution calling China out for, among many things: denying COVID19's person-to-person transmission capability &amp; waging an outrageous &amp; false propaganda war that blames the US for creating the virus. https://t.co/UkDot74K26"), ('dates', '2020-04-01 12:22:34'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.7329288721084595)])
OrderedDict([('tweet', "The @US_FDA decision to limit @Battelle's PPE sterilization process is outrageous and another example of an unaccountable bureaucracy run amok. The FDA’s restrictions cripple Battelle to little more than 6 percent of their capacity."), ('dates', '2020-03-29 17:15:31'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.5440124869346619)])
OrderedDict([('tweet', 'So @SpeakerPelosi is holding up CRITICAL AID for American small businesses, workers, &amp; healthcare providers over completely unrelated items from their progressive wishlist? Shameful &amp; outrageous. @HouseDemocrats &amp; @SenateDems are putting politics over people. https://t.co/ZWzP650J4R'), ('dates', '2020-03-23 19:43:26'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.7990917563438416)])
OrderedDict([('tweet', "It should be as simple as passing bipartisan legislation to provide relief for those affected by #Coronavirus. But Nancy Pelosi has been wasting time trying to stuff a relief bill w/ items from liberal wish lists. It's time to get serious and put the American people over politics https://t.co/M6XemN1c3Y"), ('dates', '2020-03-12 20:18:43'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.6728466153144836)])
OrderedDict([('tweet', 'Now is not the time for political posturing. Dems need to stop trying to sneak in poison pills to coronavirus funding.  It’s time to put up a CLEAN funding bill. Time to put public health first, not politics. https://t.co/JQCxAo8V7F'), ('dates', '2020-03-04 17:26:07'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.866356372833252)])
OrderedDict([('tweet', 'While Congressional Dems keep pushing their #ShamImpeachment, \u2066@realDonaldTrump\u2069 is killing terrorists and signing trade deals. Why and how eliminating Soleimani was a good thing: https://t.co/DZDDLdZYZL'), ('dates', '2020-01-24 18:45:27'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.9079226851463318)])
OrderedDict([('tweet', 'Do Dems have a slam dunk case or do they need to call witnesses during the Senate trial? Can’t have it both ways. Speaker Pelosi &amp; Schiff wouldn’t allow House Republicans to call witnesses in their #ShamImpeachment but now want the same privilege they denied @HouseGOP https://t.co/j7EZUct8FU'), ('dates', '2020-01-22 20:36:41'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.9361416697502136)])
OrderedDict([('tweet', 'The killing of Qassem Soleimani was justified and necessary in sending a clear signal to Iran and their terrorist proxies that America will not stand by while our forces, civilian personnel, assets, and allies are threatened or attacked. https://t.co/bFTwtZbyFp'), ('dates', '2020-01-08 17:17:48'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.6573398113250732)])
OrderedDict([('tweet', 'Spoke w @KyleMidura from @graydcnews about today’s sham impeachment. House Dems pushing forward w no evidence, only conjecture, presumptions, &amp; hearsay. They didn’t just lower the bar, they eliminated it to fulfill a political vendetta they’ve pursued since Trump was sworn in. https://t.co/BUBqqD4tfA'), ('dates', '2019-12-18 19:24:32'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.925560474395752)])
OrderedDict([('tweet', 'This impeachment is not just ridiculous on the merits, the very process Chairman Nadler &amp; Speaker Pelosi are running is a sham. That’s why I joined \u2066@RepAndyBiggsAZ\u2069 to demand an evidentiary hearing we are entitled to per House rules. https://t.co/KRXoDcdFuk'), ('dates', '2019-12-12 21:21:22'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.5168446898460388)])
OrderedDict([('tweet', 'The so-called offenses Congressional Democrats unveiled today in their articles of impeachment is an attack to politically injure President Trump for the 2020 election.'), ('dates', '2019-12-10 16:29:24'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.8512650728225708)])
OrderedDict([('tweet', 'Ambassador Sondland’s testimony did more damage to the House Democrats’ allegations than they realize. Once again, they’re relying on presumptions and hearsay. https://t.co/soqQg1qMb2'), ('dates', '2019-11-20 21:50:27'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.5409067273139954)])
OrderedDict([('tweet', 'Think this sham #impeachment is really about Ukraine? Democrats have been trying to reverse the 2016 election since the day @realDonaldTrump was sworn in. https://t.co/Rie2icPqOd'), ('dates', '2019-11-13 15:20:17'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.8649365901947021)])
OrderedDict([('tweet', 'Democrats’ dishonest desire to impeach @realDonaldTrump has nothing to do w Ukraine or some imagined abuse of power. They’ve been trying to impeach since day one as revenge for winning the 2016 election. https://t.co/0QE8wgshRs'), ('dates', '2019-10-31 18:00:23'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.9111054539680481)])
OrderedDict([('tweet', 'Tonight Democrats blocked @RepAndyBiggsAZ resolution to censure @RepAdamSchiff for conducting a secretive impeachment witch hunt, misleading the American people, and withholding documents from GOP members of @HouseIntel.'), ('dates', '2019-10-22 01:56:01'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.9966359734535217)])
OrderedDict([('tweet', 'A complete lack of transparency, selectively leaking testimony, no vote on an impeachment inquiry, and refusing to answer even the most basic questions. @HouseDemocrats’ impeachment is a partisan stunt. The American people deserve better. https://t.co/MakvIaPRW9'), ('dates', '2019-10-16 15:40:46'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.6385961174964905)])
OrderedDict([('tweet', 'While Democrats and the media have been chasing a phony scandal, an announcement was made yesterday that  benefits our nation’s farmers and ranchers! https://t.co/Y1cnMtbPfK'), ('dates', '2019-09-26 16:47:56'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.5417923927307129)])
OrderedDict([('tweet', 'Yesterday, Nancy Pelosi announced impeachment proceedings before the facts even came out. Today, we have the unredacted transcript showing no wrongdoing and confirmation by the President of Ukraine that he felt no pressure. https://t.co/mVoHtYSKE9'), ('dates', '2019-09-25 20:33:28'), ('file_name', 'Bob Gibbs.csv'), ('pct_true', 0.8415361046791077)])
OrderedDict([('tweet', 'Rep. Underwood is right. That is why I encourage all members to support my bill, the COVID-19 TRACE Act, which would create a $100 BILLION grant program for local organizations to hire, train, and pay individuals to operate mobile testing units and conduct contact tracing. https://t.co/AbhvrGcAwe'), ('dates', '2020-05-09 18:36:46'), ('file_name', 'Bobby L. Rush.csv'), ('pct_true', 0.6477462649345398)])
OrderedDict([('tweet', 'You know what else happened yesterday? The murderers who LYNCHED #AhmaudAubrey were finally arrested—three months later. \n\nBut to this President, the “justice” that occurred yesterday was the politically motivated dismissal of charges against a man who pleaded guilty—TWICE. https://t.co/FUk5ZZBlwj'), ('dates', '2020-05-08 18:23:08'), ('file_name', 'Bobby L. Rush.csv'), ('pct_true', 0.5633806586265564)])
OrderedDict([('tweet', 'Two days after the first COVID-19 case was confirmed in the U.S., @realDonaldTrump said he wasn\'t worried about a pandemic and that he had the situation "totally under control."\n\nToday, the U.S. has over one million confirmed cases of coronavirus and over 60k Americans have died. https://t.co/hFk4v6o1F2'), ('dates', '2020-04-29 21:41:22'), ('file_name', 'Bobby L. Rush.csv'), ('pct_true', 0.6703922748565674)])
OrderedDict([('tweet', 'There is no “eternal lockdown crowd,” @jaredkushner. There are people who put human health over the health of our economy, and then there are people like you. https://t.co/GZjZUjKmz0'), ('dates', '2020-04-29 12:42:55'), ('file_name', 'Bobby L. Rush.csv'), ('pct_true', 0.7133892178535461)])
OrderedDict([('tweet', 'For the 2018–19 flu season, @CDCgov estimates that influenza killed 34k Americans.\n\nCOVID-19 has so far killed over 50k Americans &amp; the head of CDC predicts it will be back this fall with a vengeance.\n\nMaybe if @FoxNews brought on actual doctors, @IngrahamAngle would understand. https://t.co/c1OFpjXx72'), ('dates', '2020-04-24 13:14:19'), ('file_name', 'Bobby L. Rush.csv'), ('pct_true', 0.5256030559539795)])
OrderedDict([('tweet', 'Where are the Black Republicans today? Will @SenatorTimScott condemn this injustice, @SecretaryCarson are you pro life now? Or was Audre Lorde right when she said ”The Master’s tool will never dismantle the Master’s house” https://t.co/e8NtDhOkCh'), ('dates', '2020-05-07 18:01:27'), ('file_name', 'Bonnie Watson Coleman.csv'), ('pct_true', 0.8579175472259521)])
OrderedDict([('tweet', 'SHOCKER: The Trump Admin seems to be putting the priorities of Big Business over the American People.\U0001f92f\n\nTheir own report says to expect the death rate from coronavirus to double to 3,000 per day by June. https://t.co/QmqxtoCMKk \n\nYet they are going ahead with plans to reopen...'), ('dates', '2020-05-06 13:29:58'), ('file_name', 'Bonnie Watson Coleman.csv'), ('pct_true', 0.5359485149383545)])
OrderedDict([('tweet', "The White House has blocked my subcommittee's request for Dr. Fauci to appear next week. This is yet another move to block any and all oversight of the president as he conducts business that impacts millions of American lives. \nhttps://t.co/ayv2g6prEM"), ('dates', '2020-05-01 22:31:19'), ('file_name', 'Bonnie Watson Coleman.csv'), ('pct_true', 0.6455315947532654)])
OrderedDict([('tweet', 'This is exactly the kind of inherent, systemic and built in bias that Black women always face, exactly the kind of concern @CBWGCAUCUS raised when we talked about what COVID-19 will mean in our communities, and exactly why resources need to be targeted where cases are highest. https://t.co/EoOAHFW0yo'), ('dates', '2020-04-29 23:42:28'), ('file_name', 'Bonnie Watson Coleman.csv'), ('pct_true', 0.5160840749740601)])
OrderedDict([('tweet', ".@senatemajldr McConnell's suggestion that states and localities go bankrupt while a Senate loophole allowed $750 million dollars for small businesses to go to large companies to pay for bonuses and corporate consolidation, says all you need to to know about who he represents"), ('dates', '2020-04-27 16:09:09'), ('file_name', 'Bonnie Watson Coleman.csv'), ('pct_true', 0.8408721089363098)])
OrderedDict([('tweet', ".@GovMurphy is right! @senatemajldr McConnell's suggestion that states not receive the aid they need but instead go bankrupt is an unacceptable suggestion and sad political ploy. https://t.co/cFkTlhQ2VR"), ('dates', '2020-04-22 19:38:03'), ('file_name', 'Bonnie Watson Coleman.csv'), ('pct_true', 0.6847128868103027)])
OrderedDict([('tweet', "The President's behavior is reminiscent of dictators. He demands ZERO oversight for the money he wants to give to big corporations.\n\nTrump’s corrupt, immoral, incompetent, reverse Robin Hood behavior continues.\n\nhttps://t.co/30LtHy12Ej"), ('dates', '2020-04-07 18:12:58'), ('file_name', 'Bonnie Watson Coleman.csv'), ('pct_true', 0.8392650485038757)])
OrderedDict([('tweet', "The next few weeks are going to be tough but there are no tougher people than New Jerseyans and there's nobody I'd rather get through this with than my fellow Garden Staters. \n\nWe'll get through this if we stick together and come out of it stronger than ever. \n\n#JerseyStrong https://t.co/hP6BO5dpRy"), ('dates', '2020-04-06 17:29:31'), ('file_name', 'Bonnie Watson Coleman.csv'), ('pct_true', 0.5150379538536072)])
OrderedDict([('tweet', '75 years ago, Nazi Germany surrendered to the Allied forces, effectively ending WWII in Europe. Today’s anniversary marks the end of one of our world’s darkest chapters, thanks to the bravery &amp; sacrifice of allied soldiers &amp; fueled by the commitment to justice &amp; freedom for all. https://t.co/44V4IPUYWi'), ('dates', '2020-05-08 15:23:11'), ('file_name', 'Brad R. Wenstrup.csv'), ('pct_true', 0.7886749505996704)])
OrderedDict([('tweet', "Another day has passed and Speaker Pelosi and Leader Schumer are still holding #PPP hostage.\n\nAmerica's small businesses need Congress to approve more funding ASAP! https://t.co/TguptDgah9"), ('dates', '2020-04-17 15:28:30'), ('file_name', 'Brad R. Wenstrup.csv'), ('pct_true', 0.6031615734100342)])
OrderedDict([('tweet', 'Speaker Pelosi and Leader Schumer must stop holding the Paycheck Protection Program hostage. \n\nAmericans are counting on this program to keep their businesses open and employees on payroll. https://t.co/U5spoaTiGb'), ('dates', '2020-04-16 15:24:00'), ('file_name', 'Brad R. Wenstrup.csv'), ('pct_true', 0.5890674591064453)])
OrderedDict([('tweet', "Economic Impact Payments have already begun hitting people’s bank accounts &amp; will continue to do so over the coming days. Most people don't need to take any action at this time, &amp; the IRS has created a tool for those who want to check their payment status: https://t.co/IFeN0kBLxo"), ('dates', '2020-04-15 18:46:30'), ('file_name', 'Brad R. Wenstrup.csv'), ('pct_true', 0.6087148189544678)])
OrderedDict([('tweet', 'Small businesses around the country are already utilizing the Paycheck Protection Program (PPP). Certain members must stop stalling &amp; let Congress fulfill President Trump’s request for additional funding to help small businesses cover their bills &amp; keep employees on the payroll. https://t.co/vnapilOPgG'), ('dates', '2020-04-11 21:20:41'), ('file_name', 'Brad R. Wenstrup.csv'), ('pct_true', 0.8127683997154236)])
OrderedDict([('tweet', "It’s clear that the Chinese Communist regime withheld information that could've helped us respond to COVID-19 earlier. The World Health Organization owes us an explanation for why they’ve seemingly covered for China’s lies, &amp; the President is right to question their credibility. https://t.co/KXPs3aYOHO"), ('dates', '2020-04-11 18:26:20'), ('file_name', 'Brad R. Wenstrup.csv'), ('pct_true', 0.9841662645339966)])
OrderedDict([('tweet', '(1/2) The @TheJusticeDept is acting outrageously with respect to Flynn’s #obstructionofjustice plea. Also being ignored is how #Flynn admitted to lying about being a paid agent of the Turkish govt and false filings under the FARA Act (all while an advisor to the Trump transition)'), ('dates', '2020-05-07 22:40:00'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.6107134222984314)])
OrderedDict([('tweet', 'What we need from China is not denials. What we need is access for U.S. and international researchers. Unfortunately China’s misleading statements and lack of transparency in December gives these charges credibility, unless and until such access is granted.https://t.co/gL3L20lj33'), ('dates', '2020-04-21 14:36:25'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.9526804089546204)])
OrderedDict([('tweet', '(1/2) Today @LacyClayMO1 and I urged the Trump administration to make renters and homeowners aware of their rights under the CARES Act. Roughly half of all mortgage borrowers and renters have protections from foreclosure and eviction that they are likely not aware of.'), ('dates', '2020-04-16 20:49:39'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.5018886923789978)])
OrderedDict([('tweet', "Asian Americans are increasingly the target of racist attacks as #COVID19 spreads. Asian Americans are not to blame, &amp; many are on the frontlines fighting it as doctors, researchers &amp; 1st responders. I signed @RepGraceMeng's resolution to condemn this hate &amp; protect all Americans"), ('dates', '2020-03-25 17:45:53'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.721817135810852)])
OrderedDict([('tweet', '(1/2) Trump tells us America is great, so we are handling the crisis better than anyone. South Korea, Japan, Hong Kong &amp; Singapore are closer to China and have far more travelers from China, per capita. Asian countries acted faster and were better prepared. [Image by @FT] https://t.co/acaL79azFB'), ('dates', '2020-03-19 21:26:01'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.5026012659072876)])
OrderedDict([('tweet', '"The topic of share buybacks...came up during a House Financial Services Hearing [on March 10th] where Rep. Brad Sherman asked Wells Fargo CEO...if he could commit to suspending the bank’s $31.4 billion stock buyback plan amid the economic uncertainty caused by the virus." https://t.co/FQtpYrhfDG'), ('dates', '2020-03-17 16:20:57'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.5227880477905273)])
OrderedDict([('tweet', 'Trump prevents the flow of intelligence to Congress and appoints an inexperienced sycophant to head the DNI all while Republicans keep blocking election security bills in the Senate. \nhttps://t.co/x1ACxLTxzR'), ('dates', '2020-02-21 18:00:07'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.5710867047309875)])
OrderedDict([('tweet', "Once again, the President is putting his own political interests ahead of America's national security interests and concerns by choosing an acting director whose sole qualification is complete loyalty to Trump. \nhttps://t.co/oYYyp22vn5"), ('dates', '2020-02-20 20:29:50'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.7735394239425659)])
OrderedDict([('tweet', 'Back in November: Bill O\'Reilly asks Trump if he directed Rudy to go to Ukraine on Trump\'s behalf.\n\nTrump: "No, I didn\'t direct him." \n\nYesterday: Geraldo Rivera asks Trump if he\'s sorry he sent Rudy to Ukraine. \n\nTrump: "No, not at all." https://t.co/h4soTkLd06'), ('dates', '2020-02-14 17:34:53'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.5627621412277222)])
OrderedDict([('tweet', 'More than a year ago, the House passed #HR8 and Mitch McConnell still hasn’t brought it to the Senate floor for a vote. RT to tell Leader #McConnell that it’s time to stop protecting the #NRA and #EndGunViolence.'), ('dates', '2020-02-07 18:15:59'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.65768963098526)])
OrderedDict([('tweet', 'A sad day for our republic, as the Senate did not remove a President who abused his power to benefit himself personally. This #ImpeachmentTrial, without hearing witnesses or subpoenaing docs, could not even be called a trial at all. Therefore, Trump has not been exonerated. (1/3)'), ('dates', '2020-02-06 04:09:08'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.7312979698181152)])
OrderedDict([('tweet', "Despite Trump's empty rhetoric at #SOTU2020, his administration has tried to strip people of their healthcare, gravely degraded environmental standards &amp; has given huge giveaways to big pharma. Trump has failed hard working Americans everywhere."), ('dates', '2020-02-05 04:06:59'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.7151941061019897)])
OrderedDict([('tweet', 'If you believe Trump will protect your Social Security, you probably believe he was faithful to all three of his wives. #SOTU2020'), ('dates', '2020-02-05 03:16:32'), ('file_name', 'Brad Sherman.csv'), ('pct_true', 0.6996488571166992)])
OrderedDict([('tweet', 'Nancy Pelosi and House Democrats have again said the House won’t be called back into session until a bill is ready for a vote. What about hearings? What about markups? What about bipartisan negotiations? Pelosi wants to keep the House out of session to consolidate her own power!'), ('dates', '2020-05-06 16:25:19'), ('file_name', 'Bradley Byrne.csv'), ('pct_true', 0.8921546339988708)])
OrderedDict([('tweet', "The federal government has cushioned the blow of #covid19, but we can't spend our way out of this. We must begin reopening our economy to create real value and restore lost personal liberty. The @realDonaldTrump admin has provided states a blueprint. https://t.co/6DerczXiYA"), ('dates', '2020-05-05 23:15:49'), ('file_name', 'Bradley Byrne.csv'), ('pct_true', 0.6516310572624207)])
OrderedDict([('tweet', 'Without lawmakers there to speak up for their districts and influence the legislative process, @SpeakerPelosi has made herself the sole voice and negotiator for the House, as it passes massive funding and regulatory bills. https://t.co/jpbuUlQM5M'), ('dates', '2020-05-04 14:13:26'), ('file_name', 'Bradley Byrne.csv'), ('pct_true', 0.5091124773025513)])
OrderedDict([('tweet', 'This is all because @SpeakerPelosi and @SenSchumer keep blocking the funding. While small businesses across the country struggle, Democrats are trying to use these #smallbiz as pawns in a political game. https://t.co/iy0wmYSXys'), ('dates', '2020-04-16 14:40:32'), ('file_name', 'Bradley Byrne.csv'), ('pct_true', 0.9615316390991211)])
OrderedDict([('tweet', 'Nancy Pelosi is CONGRATULATING Senate Democrats for blocking critical aid to small businesses. This is totally absurd. People are hurting and @SpeakerPelosi is trying to advance her Leftist agenda. https://t.co/EByftEBZtv'), ('dates', '2020-04-15 14:49:46'), ('file_name', 'Bradley Byrne.csv'), ('pct_true', 0.9407640695571899)])
OrderedDict([('tweet', 'These protesters are exposing their vile bigotry and small mindedness with hateful signs and chants. \n\nComparing the current fight to bend the curve of the pandemic and save the lives of our family members and neighbors to Nazis and the Holocaust is simply repugnant. https://t.co/U8I4YlDTfM'), ('dates', '2020-05-02 15:32:12'), ('file_name', 'Bradley Scott Schneider.csv'), ('pct_true', 0.7529537677764893)])
OrderedDict([('tweet', 'Our small businesses are hanging on by a thread, and the Trump Administration’s failings are preventing them from getting relief.\nhttps://t.co/WboVeUBKEx'), ('dates', '2020-04-27 17:32:19'), ('file_name', 'Bradley Scott Schneider.csv'), ('pct_true', 0.6487952470779419)])
OrderedDict([('tweet', 'Michael Caputo’s antisemitic and racist views have no place in government, nor in America. President Trump should not have appointed him as spokesman for HHS and must immediately remove him from the position and from his administration.  https://t.co/XaufVcZ6C8'), ('dates', '2020-04-24 17:45:05'), ('file_name', 'Bradley Scott Schneider.csv'), ('pct_true', 0.9254210591316223)])
OrderedDict([('tweet', "It's outrageous that the President prioritizes his ego &amp; personal interests over his responsibility to the American people. Rather than getting these funds to struggling families as quickly as possible, President Trump will make them wait in order to print his name on each check. https://t.co/SgOQDuaDLp"), ('dates', '2020-04-15 14:32:30'), ('file_name', 'Bradley Scott Schneider.csv'), ('pct_true', 0.8549928665161133)])
OrderedDict([('tweet', 'This is the latest in a long pattern by this President to avoid oversight.\n\nCongress will continue to uphold its Constitutional responsibility to ensure these funds are administered properly and that the relief is getting to the workers, families, and small businesses who need it https://t.co/AgH7w5kbZ6'), ('dates', '2020-04-07 20:27:04'), ('file_name', 'Bradley Scott Schneider.csv'), ('pct_true', 0.7431309819221497)])
OrderedDict([('tweet', 'Who is in charge here? Our communities desperately need more ventilators and the Pentagon has surplus ready to donate, but the Trump Administration still hasn\'t instructed where to send them. \n\nThis is case in point why Administration needs to appoint a "supply chain czar" ASAP. https://t.co/VdEjcCxwWF'), ('dates', '2020-04-01 14:43:52'), ('file_name', 'Bradley Scott Schneider.csv'), ('pct_true', 0.5173436403274536)])
OrderedDict([('tweet', 'With the coronavirus we are experiencing the consequences of an Administration failing to take seriously the warnings of experts and prepare. President Trump continues to chart the same path of dangerous ignorance on the climate crisis. https://t.co/e6wNaBN7on'), ('dates', '2020-03-31 16:05:30'), ('file_name', 'Bradley Scott Schneider.csv'), ('pct_true', 0.7231349349021912)])
OrderedDict([('tweet', 'Millions are sheltered in place &amp; thousands who will become infected will die because the Trump Admin was too slow to take coronavirus seriously, too reluctant to take the urgent actions necessary to secure critical medical supplies, &amp; totally unwilling to communicate honestly.'), ('dates', '2020-03-30 22:25:31'), ('file_name', 'Bradley Scott Schneider.csv'), ('pct_true', 0.7860831618309021)])
OrderedDict([('tweet', 'IMPORTANT: We are still seeking details on the 30-day restriction on entry from Europe announced by President Trump last night, but the Administration has since clarified it applies to foreign nationals, NOT U.S. citizens or permanent residents. \n\nhttps://t.co/r2tgGeLsay'), ('dates', '2020-03-12 15:29:18'), ('file_name', 'Bradley Scott Schneider.csv'), ('pct_true', 0.5760924816131592)])
OrderedDict([('tweet', 'Thousands of Americans have died, and millions more have lost their jobs. Yet the President is trying to distract the public from his complete mismanagement of this crisis by shutting down immigration. Let us not forget that immigrants - the doctors, nurses, home health aides,'), ('dates', '2020-04-21 19:31:45'), ('file_name', 'Brenda L. Lawrence.csv'), ('pct_true', 0.6505983471870422)])
OrderedDict([('tweet', '.@senatemajldr is saying he would rather see states go bankrupt than deliver lifesaving relief. How does this help our firefighters, police &amp; EMS fight #COVID19? Lives are at risk. Proud to have joined this effort to fight for more funding for local &amp; state gov. in the next bill. https://t.co/poq4ylaEVd'), ('dates', '2020-04-24 15:53:40'), ('file_name', 'Brendan F. Boyle.csv'), ('pct_true', 0.5357511639595032)])
OrderedDict([('tweet', "We can't let the rise of racism and violence toward AAPIs  go unaddressed. That's why I am an original cosponsor of a resolution condemning all forms of anti-Asian sentiment related to #COVID19 with my @CAPAC colleagues. Together, we must end these attacks on AAPI communities."), ('dates', '2020-03-25 19:55:47'), ('file_name', 'Brendan F. Boyle.csv'), ('pct_true', 0.751076877117157)])
OrderedDict([('tweet', 'You can’t fight an enemy you can’t see. That’s why the #FamiliesFirst Act makes #coronavirus testing free to all. Congress must pass this legislation immediately. https://t.co/cjbt6W3leU'), ('dates', '2020-03-18 17:43:53'), ('file_name', 'Brendan F. Boyle.csv'), ('pct_true', 0.6414932012557983)])
OrderedDict([('tweet', 'One year ago, @HouseDemocrats made history by passing #HR1, the #ForThePeople Act – a transformative anti-corruption and clean elections bill. But to maintain the status quo in Washington, @senatemajldr continues to block the bill. It’s time for a Senate vote on #HR1! https://t.co/mruxhBmpqi'), ('dates', '2020-03-08 18:25:00'), ('file_name', 'Brendan F. Boyle.csv'), ('pct_true', 0.6181745529174805)])
OrderedDict([('tweet', 'Did you know that Social Security will never demand an immediate payment from you or require payment by cash or wire transfer?  Scammers are out there, unfortunately. \n\nProtect yourself and your family members from Social Security Scams. https://t.co/p0wrr96dD3'), ('dates', '2020-03-03 14:53:09'), ('file_name', 'Brendan F. Boyle.csv'), ('pct_true', 0.5356779098510742)])
OrderedDict([('tweet', 'Now is the time for all of us to put politics aside and deliver relief to small businesses &amp; Americans. Instead, Speaker Pelosi is pushing a partisan agenda instead of working with us on bipartisan solutions. We need to be #InItTogether - our health and safety are on the line.'), ('dates', '2020-03-24 01:02:48'), ('file_name', 'Brett Guthrie.csv'), ('pct_true', 0.6436413526535034)])
OrderedDict([('tweet', 'Illnesses caused by e-cigarettes &amp; rising drug prices are two major public health crises – yet Dems on @HouseCommerce scheduled hearings on them at the same time. Both of these issues deserve our undivided attention, &amp; I am committed to working with my colleagues to address them. https://t.co/LhSz0ksEZF'), ('dates', '2019-09-25 15:20:05'), ('file_name', 'Brett Guthrie.csv'), ('pct_true', 0.6979540586471558)])
OrderedDict([('tweet', 'Democrats are shamelessly #ForThePolitics and have turned a bipartisan bill to lower drug prices into a partisan fight. I’ll be talking on the House floor shortly about my commitment to lowering drug prices in a bipartisan way. Watch: https://t.co/3u6WxwKF2h'), ('dates', '2019-05-16 16:37:41'), ('file_name', 'Brett Guthrie.csv'), ('pct_true', 0.7008249163627625)])
OrderedDict([('tweet', 'Democrats are all about their Medicare for All proposal – until it comes time to actually legislating. Why won’t they hold a hearing at @HouseCommerce instead of at the Rules Committee? Because they know socializing medicine won’t actually work.'), ('dates', '2019-04-30 18:24:14'), ('file_name', 'Brett Guthrie.csv'), ('pct_true', 0.9761160016059875)])
OrderedDict([('tweet', 'The Democrats’ Medicare for All plan will force millions of Americans off of their health care plans and will stifle innovation. I join my @HouseCommerce colleagues in calling for the committee to hold a hearing on this disastrous bill. https://t.co/AvpMzKMgtB'), ('dates', '2019-02-27 22:25:54'), ('file_name', 'Brett Guthrie.csv'), ('pct_true', 0.6740611791610718)])
OrderedDict([('tweet', 'We must protect all of the unborn as well as babies who survive abortion attempts. Sadly, Democrats in the Senate don’t agree. I support the Born-Alive Abortion Survivors Protection Act and will do everything I can to get this bill passed in the House.  https://t.co/sdYfT7NgkZ'), ('dates', '2019-02-26 16:27:48'), ('file_name', 'Brett Guthrie.csv'), ('pct_true', 0.6403780579566956)])
OrderedDict([('tweet', 'If Democrats are serious about their Medicare for All proposal, they need to get serious about holding congressional hearings on it. I want to know how this will impact Kentuckians. https://t.co/5MVhB6y7fv'), ('dates', '2019-02-11 22:38:10'), ('file_name', 'Brett Guthrie.csv'), ('pct_true', 0.6433402895927429)])
OrderedDict([('tweet', "America should never trust or depend on the People's Republic of China ever again! This communist regime is responsible for the death of thousands. We must hold them accountable for their heinous actions. https://t.co/GyROasWRgl"), ('dates', '2020-05-08 23:50:49'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.9137772917747498)])
OrderedDict([('tweet', 'Today marks 75 yrs since the Allies formally accepted Nazi Germany’s unconditional surrender – marking the end of WWII in Europe. \n\nOur brave WWII veterans have forever earned a heartfelt thank you from every American. #VictoryInEuropeDay #VEDay75 #GreatestGeneration https://t.co/q148aqavJD'), ('dates', '2020-05-08 18:46:45'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.7793203592300415)])
OrderedDict([('tweet', "We live in the greatest nation on Earth &amp; we’ll never leave the medical fate of Americans in the hands of bad actors like the Communist Party of China ever again. China's control &amp; monopoly must end. Powerful &amp; informative words from @GOPLeader McCarthy! https://t.co/yiWkrNNWut"), ('dates', '2020-05-01 21:40:33'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.930759608745575)])
OrderedDict([('tweet', "Americans deserve to know the truth &amp; see justice come to those who violated our laws in their attempt to undo the 2016 election! @Jim_Jordan's exactly right – if they can do this to one of the highest ranking officials in our gov, imagine what they can do to you &amp; your family! https://t.co/zCokwKHOvO"), ('dates', '2020-04-30 20:03:51'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.5046983361244202)])
OrderedDict([('tweet', 'For years, U.S. tax $$ have gone to Wuhan, China’s state-run bio-agent lab for dangerous &amp; careless coronavirus research. @realdonaldtrump’s cutting those grants. Last wk, I joined many colleagues urging leadership to ensure no #COVID19 relief funds end up at this notorious lab. https://t.co/LTzlG77dXI'), ('dates', '2020-04-28 23:04:27'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.6450684666633606)])
OrderedDict([('tweet', 'So the person who heavily scrutinized the President, calling him a racist and xenophobic, for halting travel from China months ago now says he didn’t ban travel enough? The lengths that the Left will go to blame @realDonaldTrump is RIDICULOUS. https://t.co/sZU4Utofxf'), ('dates', '2020-04-27 22:22:54'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.8479239344596863)])
OrderedDict([('tweet', 'It’s absurd to think that ANY elected official would exploit a national crisis to push for policies that would endanger the lives of American citizens. Sadly, many Democrats in Congress did just that. They should be incredibly ashamed of themselves. https://t.co/ugKQYHClMf'), ('dates', '2020-04-22 20:32:44'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.7705658674240112)])
OrderedDict([('tweet', 'It’s incredible that Democrats and the mainstream media would rather lay false blame on @realDonaldTrump than point a finger at the TRUE culprit behind this pandemic - China. The deceptive actions of China must have consequences. https://t.co/8ApmRLYdSp'), ('dates', '2020-04-22 18:13:29'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.9185317754745483)])
OrderedDict([('tweet', 'China’s Communist Regime violated the 2005 Int’l Health Regs when they deliberately lied to the world about #COVID19. I joined @RepJimBanks’ letter asking the @StateDept and @TheJusticeDept to sue China for the harm they caused millions with their deceit.\nhttps://t.co/qQr55p2H4x'), ('dates', '2020-04-22 15:54:56'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.9574103355407715)])
OrderedDict([('tweet', 'This took too long. D’s knew PPP was going to run out of funding 12 days ago. Instead of immediately working w/ R’s to appropriate more money for this vital program, they let it run dry which only further hurt Americans and their businesses unnecessarily. https://t.co/UQ1OJ79C5r'), ('dates', '2020-04-21 23:12:48'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.5877182483673096)])
OrderedDict([('tweet', 'It’s despicable that socialist Democrats like AOC would ever take pleasure in seeing millions of US energy workers losing their jobs. I promise to fight until my last breath against their disastrous radical agenda, which would absolutely destroy our economy &amp; bankrupt our nation. https://t.co/U159EuLctO'), ('dates', '2020-04-21 15:37:22'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.9552542567253113)])
OrderedDict([('tweet', '.@RepDanCrenshaw sets the record straight. The Left continues to promote a false narrative w/ the sole purpose of making @realDonaldTrump out to be a villain.\n\n@POTUS and his Administration have worked tirelessly to protect our nation and the American people. https://t.co/jRZfdgl1cP'), ('dates', '2020-04-20 23:03:46'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.767802894115448)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram is out of money, but Pelosi/Schumer are so consumed w/ political games that they don’t care! Pleasing their radical base and pushing their far-left agenda is more of a priority to them than the MILLIONS of hurting Americans. https://t.co/viFxr8npFb'), ('dates', '2020-04-16 17:40:10'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.9765547513961792)])
OrderedDict([('tweet', '.@realDonaldTrump made the right decision to halt U.S. taxpayer funding to the @WHO while a review is conducted. The #WHO was dishonest and tried to cover up the spread of #COVID19, in the interest of China. These actions cost us many lives. https://t.co/xZX951cwVc'), ('dates', '2020-04-15 19:56:14'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.6038884520530701)])
OrderedDict([('tweet', "Disastrous “sanctuary city” policies are STILL allowing illegal criminal aliens to be released into our communities scot free. We've enough to worry about w/ #COVID19, yet some state/local officials willfully choose to continuously put Americans in danger. https://t.co/sjZuBakMh3"), ('dates', '2020-04-13 21:28:26'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.7400184869766235)])
OrderedDict([('tweet', "Let's be clear: The same Democratic Party who allows/urges illegal aliens to vote in US elections &amp; obtain driver’s licenses now wonders why Republicans oppose mail-in voting? \n\nD's are using a health pandemic to try &amp; control Nov election results. Period.https://t.co/TcInbly8vn"), ('dates', '2020-04-09 20:53:11'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.6881414651870728)])
OrderedDict([('tweet', 'Last week, 6.6M Americans filed for unemployment. Securing an additional $250B for the #PaycheckProtectionProgram is critical for our #smallbiz. Dems continue to exploit the national crisis to carry out their political agenda, and it’s absolutely shameful.\nhttps://t.co/kMx1M36C6g'), ('dates', '2020-04-09 16:25:18'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.657157838344574)])
OrderedDict([('tweet', 'The loss of life is always tragic – no matter the # – but America and its people will endure.\n\nChina’s lied about nearly every aspect of this virus from the start. Why does the media cont to make this Communist Regime out to be a hero in this crisis? https://t.co/KnatBqmGus'), ('dates', '2020-04-07 22:23:55'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.9202781915664673)])
OrderedDict([('tweet', "Had China been truthful about #COVID19 from the start it might've been contained/limited, lives would've been saved &amp; economic damage we’re seeing might've been much less. Proud to join @RepJimBanks' bipartisan res to hold the Communist regime accountable.https://t.co/fNXqteMEL2"), ('dates', '2020-03-25 19:19:32'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.8985744714736938)])
OrderedDict([('tweet', "We must stop the spread of #COVID19, but we can't do that with unknown individuals illegally crossing our southern (or northern) border. @realDonaldTrump and his Administration continue to do what is needed to ensure our country is safe. https://t.co/YQs5bRkTmw"), ('dates', '2020-03-19 15:54:09'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.5939903259277344)])
OrderedDict([('tweet', "Crimes by criminal illegal aliens don't stop because we're social distancing. Each day #sanctuarycity policies cont, Americans, like this 3-year-old girl, are at risk. When will state/local officials take the blame for allowing these atrocities to happen! https://t.co/610qpmpOC6"), ('dates', '2020-03-17 18:13:22'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.6634541153907776)])
OrderedDict([('tweet', 'This is just one of the 200+ criminal illegal aliens @ICEgov has arrested in the New England area recently. More than half of them had convictions/pending criminal charges for rape, drug trafficking and sexual assault. \n\nAmericans are safer because of ICE. https://t.co/k4ktQ1PC2S'), ('dates', '2020-03-12 19:40:12'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.5988941788673401)])
OrderedDict([('tweet', 'The novel #coronavirus originated in Wuhan, China. Lyme disease originated in Lyme, Connecticut. We could do this all day. Referring to this as the "Chinese coronavirus" is NOT racist. Democrats never seem to miss an opportunity to politicize something to make a headline. https://t.co/bMpwDWnZAF'), ('dates', '2020-03-11 22:27:00'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.676723062992096)])
OrderedDict([('tweet', "#Sanctuarycity policies have allowed innocent Americans to be robbed, raped &amp; killed. Rogue, liberal elected officials should be held responsible. We’re supposed to be a nation of laws but apparently it's👌🏼to be above the law as long as you’re doing it under the name “sanctuary.” https://t.co/nRaG5WC0j4"), ('dates', '2020-03-11 21:23:59'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.8264023065567017)])
OrderedDict([('tweet', 'Every day that we allow our borders to remain unsecured and reckless sanctuary city policies to continue, American lives are in danger. This is absolutely the greatest threat facing our nation. I went into depth on this important issue with @Heritage and @DailySignal below! https://t.co/t4EZFFy3TM'), ('dates', '2020-03-10 18:20:41'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.8599344491958618)])
OrderedDict([('tweet', "The $93T #GreenNewDeal will absolutely bankrupt our nation and American families. There's nothing affordable or realistic about this insane bill.\n\nAlso, the vast majority of Americans don't have an extra $75K lying around each year to pay for it - sorry. https://t.co/HuNQYszMiT"), ('dates', '2020-03-05 17:26:05'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.6119765639305115)])
OrderedDict([('tweet', '.@SenSchumer knows no shame - absolutely disgraceful. His threats toward @Scotus Justices Kavanaugh &amp; Gorsuch are inexcusable &amp; dangerous. Justice Roberts was right to call him out - all Dems &amp; the media should. Not holding my breath for that to happen. https://t.co/ox4mCf1u0m'), ('dates', '2020-03-05 16:00:05'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.9713000655174255)])
OrderedDict([('tweet', "Venezuela is a prime example of what really happens under socialism. What was once the wealthiest nation in Latin America, is now impoverished &amp; its people starved. In the last 3 yrs, 15% of Venezuela's pop has fled the country. \n\nDon't let socialism win. https://t.co/osBRyeHxFX"), ('dates', '2020-03-02 22:28:03'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.5617802739143372)])
OrderedDict([('tweet', 'Thanks to @realDonaldTrump, Americans are living under the best economy our nation has seen in half a century. Knowing this, why would ANYONE put their trust in Liberal socialist policies that would reverse every ounce of this success? https://t.co/aTsA9tXC6o'), ('dates', '2020-02-27 23:10:19'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.8608004450798035)])
OrderedDict([('tweet', 'Once again, American citizens are being placed at risk and put secondary to criminal illegal aliens by Liberal Democrat elected officials. It’s a damn shame! #TX36 https://t.co/CgnsiDffvA'), ('dates', '2020-02-27 02:48:00'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.8623251914978027)])
OrderedDict([('tweet', 'This is shocking &amp; shameful. Local elected officials are repeatedly breaking federal law &amp; getting away w/ it scot free. This program endangers the very people who are being forced to pay for it! Illegal aliens continue to be prioritized while #AngelFamilies are forgotten. #TX36 https://t.co/ECvqdNYsV6'), ('dates', '2020-02-26 23:56:53'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.8316273093223572)])
OrderedDict([('tweet', 'This is a no-brainer. Congrats to @POTUS &amp; the Admin for enforcing the public charge rule after several unwarranted roadblocks. This rule is NOT anti-immigrant, it’s pro-conserving our assistance programs for the truly needy &amp; pro-hardworking US taxpayers. https://t.co/1TLFmqCVFW'), ('dates', '2020-02-25 23:14:11'), ('file_name', 'Brian Babin.csv'), ('pct_true', 0.7694770097732544)])
OrderedDict([('tweet', '7 years ago the #RanaPlaza factory collapse killed 1,134 workers. Today women &amp; girls in Bangladesh’s garment industry bear the brunt of #COVID19 &amp; worker rights in the U.S. continue to be under attack by the Trump Administration. We fight on for worker safety &amp; a living wage. https://t.co/N3H8vtPRe5'), ('dates', '2020-04-24 23:54:18'), ('file_name', 'Brian Higgins.csv'), ('pct_true', 0.6408218741416931)])
OrderedDict([('tweet', "Seniors have enough to worry about -- they shouldn't have to fight the bureaucracy too. We told the federal govt that requiring social security recipients to complete forms for #COVID19 relief payments created an unnecessary delay. Thankfully, they listened. More: https://t.co/RW9vcPqZ47 https://t.co/RzE600anBy"), ('dates', '2020-04-02 16:16:50'), ('file_name', 'Brian Higgins.csv'), ('pct_true', 0.5834665894508362)])
OrderedDict([('tweet', '75 years ago, the Nazis surrendered to the Allied Powers in Europe, ending WWII. Today we honor those who answered the call to service &amp; salute their sacrifices for our freedom. These men &amp; women changed the course of history by proving that good will always triumph over evil! https://t.co/YRM0hj0wS0'), ('dates', '2020-05-08 14:01:27'), ('file_name', 'Brian J. Mast.csv'), ('pct_true', 0.6865526437759399)])
OrderedDict([('tweet', 'ICYMI → I introduced a resolution this week to hold China accountable for their intentional deception regarding the coronavirus that has cost our country so much. Read more:\nhttps://t.co/DhwNXpNrfq'), ('dates', '2020-05-01 14:15:00'), ('file_name', 'Brian J. Mast.csv'), ('pct_true', 0.7178599238395691)])
OrderedDict([('tweet', 'China’s intentional deception has cost thousands of lives and millions of jobs. I introduced a resolution this week to hold China accountable for their cover-up and force them to pay back the taxpayer dollars that have been spent as a result. Read more:\nhttps://t.co/BMr53G2O7y'), ('dates', '2020-04-30 19:00:00'), ('file_name', 'Brian J. Mast.csv'), ('pct_true', 0.9785876274108887)])
OrderedDict([('tweet', 'The US must hold China accountable for their total failure &amp; deception in managing this outbreak that has taken so many American lives, cost so many jobs &amp; caused so many businesses to close! I introduced a bill to make China pay for all costs incurred because of their duplicity: https://t.co/ISgoK2opx3'), ('dates', '2020-04-28 16:29:02'), ('file_name', 'Brian J. Mast.csv'), ('pct_true', 0.9897157549858093)])
OrderedDict([('tweet', 'China’s total lack of transparency and mishandling of the coronavirus outbreak has cost tens of thousands of lives, millions of jobs and left untold economic destruction.'), ('dates', '2020-04-20 21:04:06'), ('file_name', 'Brian J. Mast.csv'), ('pct_true', 0.9162540435791016)])
OrderedDict([('tweet', 'Nearly 89,000 PPP loans for more than $17.8 billion have been approved in FL to help keep employees on payroll, but there are still so many small businesses that need this support and haven’t gotten it yet. That’s why I’m fighting for additional funding for this program ASAP!'), ('dates', '2020-04-20 19:30:00'), ('file_name', 'Brian J. Mast.csv'), ('pct_true', 0.5205662846565247)])
OrderedDict([('tweet', 'This is COWARDLY "leadership." @SpeakerPelosi do your damn job and bring the House of Representatives into session so the other 434 Members of Congress can do theirs also.  Stop negotiating behind closed doors without anybody else and let\'s let the American people SEE THE DEBATE! https://t.co/R5UCMjebEG'), ('dates', '2020-04-17 19:19:25'), ('file_name', 'Brian J. Mast.csv'), ('pct_true', 0.5577208399772644)])
OrderedDict([('tweet', 'I have never been more embarrassed in my life than @SpeakerPelosi having us members of Congress home, not working even as PPP loans predictably ran out of $$. @realDonaldTrump push her to show some American grit like our first responders/nurses/doctors. #getCongressToWork'), ('dates', '2020-04-17 13:26:08'), ('file_name', 'Brian J. Mast.csv'), ('pct_true', 0.5867201685905457)])
OrderedDict([('tweet', 'THREAD: At a time of unprecedented crisis for our nation, Speaker Pelosi has kept Congress out of session — not even debating new ways to help Americans — for weeks.  She just announced she won’t re-open the House until at least May 4th…'), ('dates', '2020-04-14 14:27:44'), ('file_name', 'Brian J. Mast.csv'), ('pct_true', 0.611914873123169)])
OrderedDict([('tweet', "The wet markets in China are disgusting. To think that taxpayer dollars were spent to buy animals from these markets for cruel experiments is insane. We fought to end this practice &amp; I'll keep fighting to ensure taxpayer dollars never go to them again!\nhttps://t.co/hf6OOPckMa"), ('dates', '2020-04-13 17:44:42'), ('file_name', 'Brian J. Mast.csv'), ('pct_true', 0.7361420392990112)])
OrderedDict([('tweet', 'Small businesses across our country are suffering through no fault of their own. We must provide assurance to them that in the case of future national emergencies that they will never have to go through such excruciating uncertainty ever again. https://t.co/tYKxWeW6KA'), ('dates', '2020-05-07 18:00:29'), ('file_name', 'Brian K. Fitzpatrick.csv'), ('pct_true', 0.8722892999649048)])
OrderedDict([('tweet', 'We simply cannot outsource our public safety and national security to foreign nations. We must reconstitute our healthcare and public safety supply chain back to the United States. My bill with @RepBrindisi does just that. https://t.co/Fbbrgx1Rmz'), ('dates', '2020-04-28 22:08:55'), ('file_name', 'Brian K. Fitzpatrick.csv'), ('pct_true', 0.6825615763664246)])
OrderedDict([('tweet', '50 years ago today, #Apollo13 launched in Florida. These men went where most of us can only dream of and gave this country something that we can all use right now; hope. #InItTogether https://t.co/UKRlOZ1jy9'), ('dates', '2020-04-11 21:07:00'), ('file_name', 'Brian K. Fitzpatrick.csv'), ('pct_true', 0.5612412691116333)])
OrderedDict([('tweet', 'We simply cannot outsource our public safety and national security to foreign nations. We must reconstitute our healthcare and public safety supply chain back to the United States. My landmark, bipartisan legislation does just that.\n\nhttps://t.co/cDrHWgSSZj'), ('dates', '2020-04-11 16:33:00'), ('file_name', 'Brian K. Fitzpatrick.csv'), ('pct_true', 0.6107385158538818)])
OrderedDict([('tweet', 'We simply cannot outsource our public safety and national security to foreign nations. We must reconstitute our healthcare and public safety supply chain back to the United States. My landmark, bipartisan legislation does just that. https://t.co/aP1LLmWw4S'), ('dates', '2020-04-07 23:00:00'), ('file_name', 'Brian K. Fitzpatrick.csv'), ('pct_true', 0.6083393096923828)])
OrderedDict([('tweet', 'We simply cannot outsource our public safety and national security to foreign nations. We must reconstitute our healthcare and public safety supply chain back to the United States. This landmark, bipartisan legislation does just that. https://t.co/yxEozJbLNZ'), ('dates', '2020-04-04 14:04:15'), ('file_name', 'Brian K. Fitzpatrick.csv'), ('pct_true', 0.6545795202255249)])
OrderedDict([('tweet', 'By ignoring science &amp; experts, President Trump has caused mass preventable deaths. His impatience to reopen the country will cause even more. \n\nWe’re introducing a bill to create an independent commission to put politics aside when determining how &amp; when to reopen the country. https://t.co/YGx1H5LGc2'), ('dates', '2020-04-16 16:20:06'), ('file_name', 'Brian Schatz.csv'), ('pct_true', 0.6628803014755249)])
OrderedDict([('tweet', "The Trump administration's response to the #Coronavirus is unacceptable. They have communicated poorly with state officials, left key positions unfilled &amp; want to cut preparedness &amp; response funding.\n \nToday I demanded answers from Trump official @SecAzar.\nhttps://t.co/2fxcuPD3gU"), ('dates', '2020-02-26 01:01:00'), ('file_name', 'Brian Schatz.csv'), ('pct_true', 0.5360440611839294)])
OrderedDict([('tweet', 'I asked Julie Brill of @Microsoft if people have any idea that online companies exploit their personal information by hiding behind complex legal loopholes.\n\nShe said no.\n\nThat’s unacceptable – people deserve a federal privacy law that protects them, not Big Tech’s profits. https://t.co/Vt2baoVacC'), ('dates', '2019-12-04 22:46:27'), ('file_name', 'Brian Schatz.csv'), ('pct_true', 0.8110190033912659)])
OrderedDict([('tweet', 'When we let companies use our personal information, we should be able to trust that it will be protected &amp; won’t be used to harm us.\n\nOur new bill lays out a set of strong data privacy rights &amp; provides the tools needed to hold Big Tech accountable.\n\nhttps://t.co/29l8RwleQM'), ('dates', '2019-11-26 23:21:00'), ('file_name', 'Brian Schatz.csv'), ('pct_true', 0.5141967535018921)])
OrderedDict([('tweet', 'When Kauai lost cell &amp; internet service, residents were left in the dark and didn’t have the information they needed.\n\n@GetSpectrum, @Verizon &amp; @CenturyLink owe us an explanation for the outage on Kauai &amp; a plan to prevent future outages. https://t.co/fIthGX4xjB'), ('dates', '2019-11-13 04:36:00'), ('file_name', 'Brian Schatz.csv'), ('pct_true', 0.6124861836433411)])
OrderedDict([('tweet', "Trump's decision to officially begin withdrawing from the Paris Climate Accord today is disappointing &amp; short-sighted – but we aren't deterred.\n\nWe know the future is clean energy &amp; innovation. We are committed to climate action. And we will let the world know that #WeAreStillIn https://t.co/sMdbhFN8Qh"), ('dates', '2019-11-05 01:40:37'), ('file_name', 'Brian Schatz.csv'), ('pct_true', 0.8004591464996338)])
OrderedDict([('tweet', 'The Pacific Ocean is warming at an alarming rate &amp; bleaching Hawaii’s coral reefs. Dying reefs could mean the collapse of entire aquatic ecosystems &amp; a threat to our coastal areas.\n \nWe don’t have time to waste. We need to #ActOnClimate now.\n\nhttps://t.co/dOHdUww0Cv'), ('dates', '2019-10-22 20:00:00'), ('file_name', 'Brian Schatz.csv'), ('pct_true', 0.5360286831855774)])
OrderedDict([('tweet', "Trump's proposal to roll back the Clean Power Plan is full-fledged climate denial. We can't give coal companies a green light to pollute the air we breathe and the water we drink.\n\nThat's why I voted against this dirty energy plan on the floor today."), ('dates', '2019-10-17 23:07:35'), ('file_name', 'Brian Schatz.csv'), ('pct_true', 0.8839256167411804)])
OrderedDict([('tweet', "This is why measured climate solutions are so important. I agree we should take action, but not by gut punching our economy.\n\nPlus, banning fracking does nothing to solve the problem of carbon that's already in the atmosphere, hence why we need more trees! https://t.co/Oj8dI2BB75"), ('dates', '2020-02-27 17:45:45'), ('file_name', 'Bruce Westerman.csv'), ('pct_true', 0.5057713985443115)])
OrderedDict([('tweet', 'The House should be in session now, getting answers for the American public:\n\nHow do we bring the medical supply chain back to the US?\n\nWhat did China know and when did they know it?'), ('dates', '2020-05-05 21:36:47'), ('file_name', 'Bryan Steil.csv'), ('pct_true', 0.802081286907196)])
OrderedDict([('tweet', 'We cannot take our eyes off Iran. Iran is using the coronavirus pandemic to work to avoid our sanctions. I have a plan to stop that if we act now.\n\nhttps://t.co/Ta1taypP6R'), ('dates', '2020-04-23 16:21:22'), ('file_name', 'Bryan Steil.csv'), ('pct_true', 0.6348916888237)])
OrderedDict([('tweet', 'Glad to see SBA address my concerns. Religious nonprofit &amp; religious organization workers are suffering from the economic fallout of coronavirus. Great that these workers can receive a paycheck &amp; these groups can have fed aid to keep their doors open. https://t.co/NnE4AkDEIc'), ('dates', '2020-04-06 16:35:36'), ('file_name', 'Bryan Steil.csv'), ('pct_true', 0.5179500579833984)])
OrderedDict([('tweet', '“She said the government had banned drinking, and one thing she wanted when she got home was a Spotted Cow, a popular New Glarus Brewing product that is not sold outside Wisconsin.\n\nScott said they found two Spotted Cow six-packs on the porch.”\n\nhttps://t.co/ffUbJ9il1t'), ('dates', '2020-04-02 19:05:51'), ('file_name', 'Bryan Steil.csv'), ('pct_true', 0.5141075253486633)])
OrderedDict([('tweet', 'The more I think about it, the more questions I have. Foremost: why is @realDonaldTrump  spending his time evening political scores – firing IC Inspector General Atkinson for what was a textbook response to a corroborated whistleblower – amid an unprecedented pandemic?'), ('dates', '2020-04-06 17:33:54'), ('file_name', 'C. A. Dutch Ruppersberger.csv'), ('pct_true', 0.7135036587715149)])
OrderedDict([('tweet', 'Just voted YES on a bill that will save lives and livelihoods. This bill addresses the immediate concerns of treating sick people and making sure our nation’s economy can bounce back when the pandemic is over. #familiesfirst More: https://t.co/EnPcdxaFRZ'), ('dates', '2020-03-27 18:13:38'), ('file_name', 'C. A. Dutch Ruppersberger.csv'), ('pct_true', 0.578849196434021)])
OrderedDict([('tweet', 'American families – not Wall Street -- need cash now. Economists agree it’s time to consider direct and immediate payments to middle class Americans. Nothing should be off the table. #Coronavirus'), ('dates', '2020-03-17 19:13:33'), ('file_name', 'C. A. Dutch Ruppersberger.csv'), ('pct_true', 0.5258209705352783)])
OrderedDict([('tweet', 'You can’t have security without privacy, and you can’t have privacy without security. Today, the House passed bipartisan #FISA #fisareform that will keep us safe, while holding true to American values. More: https://t.co/rku2XqoxpY'), ('dates', '2020-03-11 21:44:49'), ('file_name', 'C. A. Dutch Ruppersberger.csv'), ('pct_true', 0.5873775482177734)])
OrderedDict([('tweet', 'In the year #HR8 has waited on a vote in the Senate, 2,887 children &amp; teens died from gun violence and twice as many were wounded. It’s time for @senatemajldr to do his job and bring this life-saving bill to a vote in the Senate. #EndGunViolence'), ('dates', '2020-02-27 14:25:34'), ('file_name', 'C. A. Dutch Ruppersberger.csv'), ('pct_true', 0.6719735860824585)])
OrderedDict([('tweet', 'Most Americans pay into Social Security all year long. But as of today, millionaires will stop contributing to Social Security. Let’s #ScrapTheCap that unnecessarily protects our wealthiest citizens and #ExpandSocialSecurity. https://t.co/s20z6poQl0'), ('dates', '2020-02-19 15:48:15'), ('file_name', 'C. A. Dutch Ruppersberger.csv'), ('pct_true', 0.5918752551078796)])
OrderedDict([('tweet', 'Two years ago today, 17 people were murdered in just over 6 minutes in the deadliest high school shooting in US history. May we always remember the lives lost in this horrific tragedy + those who must carry on without their loved ones forever. #MSDstrong https://t.co/QXTbu4m0P8'), ('dates', '2020-02-14 15:05:45'), ('file_name', 'C. A. Dutch Ruppersberger.csv'), ('pct_true', 0.5531666278839111)])
OrderedDict([('tweet', 'I’ve been saying this for nearly a decade - that we can’t let companies beholden to the Chinese Communist Party control the backbone of Western communications. Pleased to see our IC is sharing as much info as possible to spread the word. https://t.co/mYgv2JiVsl'), ('dates', '2020-02-12 02:08:28'), ('file_name', 'C. A. Dutch Ruppersberger.csv'), ('pct_true', 0.5798364877700806)])
OrderedDict([('tweet', 'The outrageous #TrumpBudget includes more than $77M in funding cuts to the @chesbayprogram @chesapeakebay. This is yet ANOTHER assault on not just clean air + water efforts + endangered species within + around the Bay, but also on our state’s livelihood, culture + economy.'), ('dates', '2020-02-11 19:55:44'), ('file_name', 'C. A. Dutch Ruppersberger.csv'), ('pct_true', 0.5495927929878235)])
OrderedDict([('tweet', 'It has been 4 days since the #PaycheckProtectionProgram ran out of funding, putting millions of jobs at risk. \n\nSpeaker Pelosi, when are you going to put Americans first and allow us to vote on more funding for this crucial program? https://t.co/kQU7SrHbjw'), ('dates', '2020-04-20 19:21:38'), ('file_name', 'Carol D. Miller.csv'), ('pct_true', 0.5383425951004028)])
OrderedDict([('tweet', 'Washington Democrats are once again playing political games and blocking crucial economic relief. There is no time for this. \n\n17 million Americans have lost their jobs in the past three weeks. We need to act now! https://t.co/gDoacPoqwe'), ('dates', '2020-04-09 18:09:34'), ('file_name', 'Carol D. Miller.csv'), ('pct_true', 0.9390437006950378)])
OrderedDict([('tweet', 'Speaker Pelosi is once again stalling important #COVID19 legislation by attempting to add requirements for all airlines to end carbon emissions by 2025 &amp; funding sanctuary cities.\n\nWhen will Democrats put politics aside and help the American people get through this crisis?'), ('dates', '2020-03-24 02:42:12'), ('file_name', 'Carol D. Miller.csv'), ('pct_true', 0.8477042317390442)])
OrderedDict([('tweet', 'Yet again, House Democrats defeated @RepAnnWagner’s legislation that would protect the rights of children that are #BornAlive following an abortion attempt. This is heartbreaking. We are better than this, we can do better, and we need to do better.'), ('dates', '2020-02-28 18:04:54'), ('file_name', 'Carol D. Miller.csv'), ('pct_true', 0.6462759971618652)])
OrderedDict([('tweet', 'We must protect the lives of ALL people, especially those who survive abortions and are #BornAlive. \n\nI asked that this bill be voted on in the House &amp; Democrats blocked it. Today, @senatemajldr brought it up in the Senate.\n\nAll Americans deserve the right to life. 👶🏻'), ('dates', '2020-02-25 21:31:17'), ('file_name', 'Carol D. Miller.csv'), ('pct_true', 0.8105823397636414)])
OrderedDict([('tweet', 'Since @POTUS declared a crisis on our Southern Border, illegal immigration has decreased significantly, but there is still so much to be done. It is imperative that we work to secure our borders, and crack down on Sanctuary Cities. #SOTU'), ('dates', '2020-02-05 02:59:30'), ('file_name', 'Carol D. Miller.csv'), ('pct_true', 0.753473699092865)])
OrderedDict([('tweet', 'DYK: Under the Trump Administration we are swiftly moving towards American energy independence and American energy dominance! #SOTU'), ('dates', '2020-02-05 02:23:16'), ('file_name', 'Carol D. Miller.csv'), ('pct_true', 0.7076196074485779)])
OrderedDict([('tweet', 'Our economy is the best it has ever been. In the past month, @POTUS has signed #USMCA and the Phase 1 Trade Agreement with China. These deals will level America’s playing field and create endless opportunities for our businesses, farmers, manufacturers, and innovators. #SOTU'), ('dates', '2020-02-05 02:13:25'), ('file_name', 'Carol D. Miller.csv'), ('pct_true', 0.829432487487793)])
OrderedDict([('tweet', 'Violence rates against indigenous women and girls are 10X higher than the national average &amp; their disappearances and murders often slip through cracks in our legal system. We must address the #MMIW crisis to protect indigenous women. That’s why I support the #NotInvisible Act. https://t.co/YwU8iibpBQ'), ('dates', '2020-05-05 23:55:06'), ('file_name', 'Carolyn B. Maloney.csv'), ('pct_true', 0.7167425155639648)])
OrderedDict([('tweet', 'Assisted living facilities share many of the same risk factors as nursing homes, but are largely unregulated with no national reporting requirements for #COVID19 cases.\n \n@SenWarren, @SenMarkey, and I are demanding answers &amp; mitigation plans.\nhttps://t.co/GzL37TV9WZ'), ('dates', '2020-05-01 18:25:04'), ('file_name', 'Carolyn B. Maloney.csv'), ('pct_true', 0.5128609538078308)])
OrderedDict([('tweet', "The Executive Branch has encroached on Congress' power of the purse &amp; spent federal dollars in the interest of the President instead of the American people.\n\nMy bill with @RepJohnYarmuth &amp; @NitaLowey makes clear Congress has sole authority to decide how federal dollars are spent. https://t.co/z1WOt1FhED"), ('dates', '2020-04-30 19:35:04'), ('file_name', 'Carolyn B. Maloney.csv'), ('pct_true', 0.6812375783920288)])
OrderedDict([('tweet', "Trump's failed response to the #COVID19 crisis has already caused unimaginable suffering and chaos. The Trump Admin needs to stop dragging its feet, be honest with the American people, and devise a national plan to get the resources we need.\nhttps://t.co/HHoe9kDFx5"), ('dates', '2020-04-29 17:30:06'), ('file_name', 'Carolyn B. Maloney.csv'), ('pct_true', 0.8362735509872437)])
OrderedDict([('tweet', 'The President has no shame. Exploiting a public health crisis to peddle his racist, anti-immigrant agenda is a new low.\n\nNew York is, and will always be, proud to be a sanctuary state that protects our immigrant communities. This bullying won’t work. \nhttps://t.co/rANsGX2OsM'), ('dates', '2020-04-29 16:00:38'), ('file_name', 'Carolyn B. Maloney.csv'), ('pct_true', 0.8729291558265686)])
OrderedDict([('tweet', 'Today is #WorkersMemorialDay. Our nation is suffering the worst worker safety crisis in recent history &amp; thousands are dying from preventable #COVID19 infections. \n\nBut @OSHA_DOL still refuses to take meaningful action to protect America’s workers. #WhereisOSHA? https://t.co/mUF1Z8Jhl5'), ('dates', '2020-04-28 19:05:46'), ('file_name', 'Carolyn B. Maloney.csv'), ('pct_true', 0.6707738637924194)])
OrderedDict([('tweet', 'For the 6 million Jews &amp; the millions more murdered by the Nazis, on #YomHaShoah we renew our vow; #NeverAgain.\n\nTo make that promise true, we must #NeverForget. We must teach children the horrors of the Holocaust &amp; the dangers of antisemitism and hate. #HolocaustMemorialDay https://t.co/Ijy1LctQ7P'), ('dates', '2020-04-21 14:46:21'), ('file_name', 'Carolyn B. Maloney.csv'), ('pct_true', 0.5189302563667297)])
OrderedDict([('tweet', 'This move will not protect Americans from #COVID19- it’s nothing but a pathetic excuse to further a racist, anti-immigrant agenda. Yet again, @realDonaldTrump resorts to demonizing and attacking others to shift attention away from his own failures. https://t.co/ii4pkpgm3R'), ('dates', '2020-04-21 13:00:11'), ('file_name', 'Carolyn B. Maloney.csv'), ('pct_true', 0.9040468335151672)])
OrderedDict([('tweet', 'As Nevadans confront the #coronavirus pandemic, families with mixed immigration status or who are undocumented may be concerned that they might not have access to important info and resources.\n\nI want to change that.'), ('dates', '2020-04-27 14:24:36'), ('file_name', 'Catherine Cortez Masto.csv'), ('pct_true', 0.7271708250045776)])
OrderedDict([('tweet', '.@SenJackyRosen&amp; I are proud to have fought for new legislation—which the Senate just passed—to give health care providers support to keep our communities healthy&amp; inject additional funding into the PPP&amp; EIDL programs so that #SmallBiz can weather this crisis&amp; reopen their doors.'), ('dates', '2020-04-21 22:33:45'), ('file_name', 'Catherine Cortez Masto.csv'), ('pct_true', 0.5743451118469238)])
OrderedDict([('tweet', "Recent reports have stated that extreme limitations and a lack of guidance have kept SBA loans from being properly administered to small business owners. We're in a midst of a global pandemic--this is unacceptable."), ('dates', '2020-04-20 11:52:38'), ('file_name', 'Catherine Cortez Masto.csv'), ('pct_true', 0.8744487762451172)])
OrderedDict([('tweet', 'SBA relief programs have been incredibly important for hundreds of small businesses across Nevada, but the SBA’s discriminatory rules have prohibited hundreds of companies from getting access to this needed relief. @SenJackyRosen and I are working to change that.'), ('dates', '2020-04-18 18:02:07'), ('file_name', 'Catherine Cortez Masto.csv'), ('pct_true', 0.5895249843597412)])
OrderedDict([('tweet', 'We should win the future with🇺🇸nuclear innovation. It’s dangerous to our security &amp; global competitiveness that China &amp; Russia are exporting their nuclear tech to other countries. For our energy independence, @POTUS’ strategy to restore our energy leadership must be a priority. https://t.co/marH89XGeu'), ('dates', '2020-04-24 16:03:11'), ('file_name', 'Cathy McMorris Rodgers.csv'), ('pct_true', 0.5671169757843018)])
OrderedDict([('tweet', 'We need a thorough review of how @WHO spread misinformation from China and how it shaped the global response to COVID-19. For strong American global leadership, @POTUS is right to demand accountability &amp; transparency here.'), ('dates', '2020-04-15 01:04:40'), ('file_name', 'Cathy McMorris Rodgers.csv'), ('pct_true', 0.6555571556091309)])
OrderedDict([('tweet', '.@realDonaldTrump is right to demand more transparency from @WHO. America should hold @WHO accountable for its relationship with China, and the rest of the world should follow our leadership. Why? \n\nBecause #TBT.... https://t.co/JRRTzYS83j'), ('dates', '2020-04-09 17:20:00'), ('file_name', 'Cathy McMorris Rodgers.csv'), ('pct_true', 0.8954885601997375)])
OrderedDict([('tweet', 'Insane. China— which is currently oppressing at least a million Uighurs in concentration camps— is appointed to a UN panel to oversee arbitrary detention &amp; freedom of speech. That’s a million reasons alone (w/ plenty more) why the CCP can’t be trusted to oversee human rights. https://t.co/uwXRkC42Yj'), ('dates', '2020-04-06 20:03:00'), ('file_name', 'Cathy McMorris Rodgers.csv'), ('pct_true', 0.918095052242279)])
OrderedDict([('tweet', 'Don\'t be fooled by the Chinese Communist Party\'s propaganda.  \n\n🚨U.S. intel has confirmed we cannot trust China’s data on the coronavirus.🚨 \n\n"China has concealed the extent of the coronavirus outbreak in its country, under-reporting both total cases &amp; deaths..." https://t.co/j7gJflkoie'), ('dates', '2020-04-01 19:27:25'), ('file_name', 'Cathy McMorris Rodgers.csv'), ('pct_true', 0.8727946281433105)])
OrderedDict([('tweet', "China won't waste any chance to smear America, freedom, &amp; all that we stand for.\n \nHere's a look at their propaganda machine to blame us for the #coronavirus. They're using a global crisis to get ahead &amp; make America look weak. Don't be fooled.\n \nhttps://t.co/agfaZsyatG"), ('dates', '2020-03-13 15:00:01'), ('file_name', 'Cathy McMorris Rodgers.csv'), ('pct_true', 0.8700712323188782)])
OrderedDict([('tweet', 'China is the #1 source for unsafe &amp; fake products. To keep people safe and beat China, we should harness &amp; encourage ways the private sector is cracking down on counterfeit packages. WATCH to learn how one innovative company in #WA05 is leading. https://t.co/ZJ0vd5625e'), ('dates', '2020-03-04 18:25:00'), ('file_name', 'Cathy McMorris Rodgers.csv'), ('pct_true', 0.557920515537262)])
OrderedDict([('tweet', 'Completely agree with this. To beat China, the EU should follow America’s leadership for a AI regulatory framework that promotes our shared values for civil liberties, freedom, and innovation. https://t.co/sKB5zTjqPB'), ('dates', '2020-02-25 21:59:00'), ('file_name', 'Cathy McMorris Rodgers.csv'), ('pct_true', 0.8084756135940552)])
OrderedDict([('tweet', 'This is a damn shame and this President wants Billions to build a wall! We can to better.  https://t.co/2yLeWbIj8K'), ('dates', '2020-02-05 23:59:47'), ('file_name', 'Cedric L. Richmond.csv'), ('pct_true', 0.5662260055541992)])
OrderedDict([('tweet', 'We just had the worst job losses since the Great Depression.\n\nBut President Trump and Senator Mitch McConnell say that it’s time to pause on helping Americans across this country?\n\nNo.\n\nWe need big, bold action, and we need it now. https://t.co/txm94UN6Fo'), ('dates', '2020-05-08 22:25:44'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5500642657279968)])
OrderedDict([('tweet', 'No one could look at today’s jobs report, the highest unemployment since the Great Depression, and say we should hit the pause button on further government action, as Leader McConnell, Leader McCarthy and the Trump White House have said. https://t.co/Er8H2KB9c8'), ('dates', '2020-05-08 15:53:50'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5341553092002869)])
OrderedDict([('tweet', 'In February—a bipartisan majority of Senators sent a clear message saying Pres. Trump cannot wage war with Iran without Congress’s approval\n\nThe House passed the same resolution\n\nBut Pres. Trump vetoed it\n\nThe Senate should vote to overturn the veto today\n\nNo more endless wars!'), ('dates', '2020-05-07 18:15:27'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.9527292847633362)])
OrderedDict([('tweet', 'Several big corporations that received small business loans they never should have gotten to begin with are returning them to the Treasury.\n\nThe SBA should re-direct those returned loans to a set-aside for truly small and underserved businesses using community-based lenders NOW.'), ('dates', '2020-05-07 16:39:01'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.9111527800559998)])
OrderedDict([('tweet', "It's May 6th.\n\nThat means it’s been 2 months since President Trump lied to Americans and said that:\n\n“Anybody that needs a test gets a test.”\n\nBut President Trump still doesn’t have an adequate national testing strategy."), ('dates', '2020-05-06 20:16:27'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.6745867133140564)])
OrderedDict([('tweet', 'Legal briefs are due in the SCOTUS case this week to determine the future of our healthcare law\n\nBut in a global pandemic—@SenateGOP wants to jam through a judge who believes it should all end\n\nTens of millions would lose health insurance. Pre-existing condition protections gone.'), ('dates', '2020-05-06 18:31:46'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5024863481521606)])
OrderedDict([('tweet', "Why isn't the @SenateGOP focused on responding to COVID-19?\n\nSenator McConnell hasn’t held a single vote here on the Senate floor this week related to COVID-19.\n\nInstead, Republicans on the Judiciary Committee are wasting time on the nomination of McConnell protégé Justin Walker."), ('dates', '2020-05-06 18:00:58'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.8699554204940796)])
OrderedDict([('tweet', 'When asked about why we weren’t prepared for COVID-19:\n\nPresident Trump said he “had a lot of things going on”\n\nLike watching TV, attacking the WHO, attacking the press, or attacking his political opponents.\n\nPresident Trump: It’s not about you. You need to get to work. https://t.co/1CEYoO7exk'), ('dates', '2020-05-06 16:48:21'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.8245668411254883)])
OrderedDict([('tweet', "2 months ago today, President Trump told Americans:\n\n“Anybody that needs a test gets a test.”\n\nIt was a lie then.\nIt’s still a lie now.\n\nPresident Trump: We've seen 1.2 million cases of COVID-19 and 70,000 deaths.\n\nWhy do you still not have an adequate national testing strategy? https://t.co/hqDurlwEsH"), ('dates', '2020-05-06 14:04:32'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.6720554828643799)])
OrderedDict([('tweet', "This is a national emergency.\n\nBut Sen. McConnell has scheduled no floor votes on COVID-19 bills.\n\nInstead the Senate GOP's focused on nominations and unqualified, right-wing judges.\n\nThat includes Justin Walker, a McConnell protégé who argued against our health care law."), ('dates', '2020-05-05 19:40:38'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.8375017046928406)])
OrderedDict([('tweet', "You know how the White House knows that it’s safe to hold a press conference?\n\nThey test the reporters\n\nWhat do they do before Pres. Trump meets business leaders?\n\nThey take temperatures &amp; administer coronavirus tests\n\nWhy isn't there a plan to do the same thing for the country?"), ('dates', '2020-05-05 02:22:31'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.8365793824195862)])
OrderedDict([('tweet', 'President Trump should learn that by muzzling science and the truth—he will only prolong this crisis.\n\nThe president’s failure to accept the truth, and then his desire to hide it, is one of the chief reasons we are lagging behind so many other countries in beating this scourge. https://t.co/faejgZGmfV'), ('dates', '2020-05-05 01:09:49'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7300042510032654)])
OrderedDict([('tweet', 'President Trump continues to pressure states and businesses to re-open.\n\nBut he refuses take responsibility for the one thing that will allow them to do it safely:\n\nTesting.'), ('dates', '2020-05-04 22:28:28'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5352799296379089)])
OrderedDict([('tweet', "Sen. McConnell's red line in a Coronavirus bill is not testing, help for small biz, housing assistance for families, or rescuing our healthcare system.\n\nSen. McConnell won’t support a new Coronavirus bill unless it gives big corporations legal immunity.\n\nHow does that make sense?"), ('dates', '2020-05-04 21:53:27'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.8853397369384766)])
OrderedDict([('tweet', 'Senator McConnell and Senate Republicans must concentrate on helping Americans recover from COVID-19\n\nnot confirming right-wing judges or protecting big businesses that threaten to put workers at serious risk.'), ('dates', '2020-05-04 21:13:48'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5485253930091858)])
OrderedDict([('tweet', 'It’s good that Ashford, Inc. is returning the money. But it shouldn’t have taken my asking to do so—they never should’ve gotten it to begin with.\n\nOther big corporations should take the same step, and the SBA must make sure this doesn’t happen again.\n\nhttps://t.co/g7Apza7ubG'), ('dates', '2020-05-03 03:30:17'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7481119632720947)])
OrderedDict([('tweet', 'The SBA needs to review PPP loans to Ashford, Inc. and its subsidiaries NOW!\n\nThis is a big, publicly traded company—not a small business struggling with this crisis.\n\nReal small businesses need help!'), ('dates', '2020-05-01 23:14:07'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.8170425891876221)])
OrderedDict([('tweet', "Families are hurting\n\nUndocumented immigrants &amp; their families are especially vulnerable\n\nSo are US citizens married to them who aren't eligible for stimulus payments—and think of their kids\n\nWe're fighting this. GOP is discriminating against taxpayers because of who they love. https://t.co/LwwZgAV7KS"), ('dates', '2020-05-01 22:50:35'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5738810300827026)])
OrderedDict([('tweet', 'Senate Republicans should be laser focused on the health and economic crises caused by COVID-19, not confirming right-wing judges or protecting big business from legal liability.'), ('dates', '2020-05-01 21:58:56'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.6295948028564453)])
OrderedDict([('tweet', "Americans deserve to immediately hear from top Trump admin officials on why we don’t have enough tests, why small biz's are having trouble accessing SBA help, why unemployment insurance checks are failing to get to workers, how to protect frontline workers with needed equipment."), ('dates', '2020-05-01 20:53:14'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.9523763656616211)])
OrderedDict([('tweet', "Pres. Trump's rushing to bail out big oil after its CEOs donated millions to his campaign—while Main Street businesses are still waiting for help\n\nOutrageous—Small biz's are fighting for limited funds while the Trump admin showers big oil with taxpayer $$\n\nhttps://t.co/x7SGLd6CZU"), ('dates', '2020-04-30 23:22:17'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7659230828285217)])
OrderedDict([('tweet', 'There still aren’t enough tests\n\nHealthcare workers still don’t have the PPE they need\n\nWe need aggressive oversight of the Trump admin’s coronavirus response\n\nRepublicans should be focused on fighting for Americans in this pandemic, not confirming unqualified, right-wing judges https://t.co/0rHH31TLT8'), ('dates', '2020-04-30 20:32:13'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5337151885032654)])
OrderedDict([('tweet', 'The American people are demanding answers and solutions.\n\nSen. McConnell should focus the Senate’s work on the crises caused by COVID-19—not right-wing judges or fulfilling his “pre-existing partisan wish-list” of protecting big business from any harm done to the American people.'), ('dates', '2020-04-29 17:58:00'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7661029100418091)])
OrderedDict([('tweet', "When Senate returns next week, Dems demand tough oversight of the Trump admin’s dreadful response to coronavirus, with Trump officials answering tough questions on why we don't have adequate testing, what is being done for workers &amp; people, where the small business money is going"), ('dates', '2020-04-29 17:22:48'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7071985602378845)])
OrderedDict([('tweet', 'President Trump’s failed leadership has left our frontline healthcare workers without needed PPE, testing, &amp; medical equipment\n\nBIG—We’re announcing the Medical Supply Transparency &amp; Delivery Act to force him to use the full authority of the Defense Production Act for this effort'), ('dates', '2020-04-29 16:20:55'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5398128032684326)])
OrderedDict([('tweet', 'Senator McConnell:\n\nWhy is there NO legislative or committee business related to the COVID-19 public health and economic emergencies scheduled for when the Senate returns? https://t.co/DPDlc48Dub'), ('dates', '2020-04-28 22:49:31'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.6599651575088501)])
OrderedDict([('tweet', 'I’m joining forces with @SenatorDurbin, @SenatorMenendez, &amp; immigration advocates across America to demand the Trump admin extend work authorizations for the over 130,000 TPS holders and over 200,000 DACA recipients—many who are on the frontlines of the fight against coronavirus'), ('dates', '2020-04-28 17:32:19'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5706764459609985)])
OrderedDict([('tweet', "Our first job when the Senate comes back should be demanding answers from the Trump admin\n\nWhere are the tests?\nWhere is the small business money going?\n\nBut Sen. McConnell's focused on jamming through more ideological judges who want to take health care &amp; worker protections away"), ('dates', '2020-04-28 15:44:18'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7123538255691528)])
OrderedDict([('tweet', "Americans are dying. They're losing jobs. We don’t have adequate testing.\n\nHow does Senator McConnell respond?\n\nBy rushing to confirm his protégé—who called a Supreme Court decision to protect healthcare “indefensible”—to a lifetime appointment to the 2nd highest court in America https://t.co/rzrgdWxNKw"), ('dates', '2020-04-28 02:08:03'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7506420016288757)])
OrderedDict([('tweet', "Now that Sen. McConnell's decided the Senate will reconvene next week—he should instruct his Committee Chairs to immediately begin oversight hearings of the Trump admin on why we still don't have adequate testing, why some PPP lenders prioritized larger, wealthier clients, &amp; more https://t.co/zoOuRr2svG"), ('dates', '2020-04-27 20:06:58'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.908696174621582)])
OrderedDict([('tweet', 'It is long past time for President Trump to focus on TESTING.\n\nSenate Democrats fought to make sure there were major new investments in a NATIONAL TESTING STRATEGY.\n\nPresident Trump needs to use them IMMEDIATELY.'), ('dates', '2020-04-27 18:24:50'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.6208149194717407)])
OrderedDict([('tweet', "The Trump administration bungled the first round of PPP funding and prevented too many small businesses even having access.\n\nWe're fighting to hold them accountable.\n\nAnd we fought for a new round of dedicated funding for truly small, underserved businesses—not big shots."), ('dates', '2020-04-27 16:32:53'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7898690104484558)])
OrderedDict([('tweet', "Do GOP Senators support Senator McConnell saying states should go bankrupt?\n\nThis isn't abstract.\n\nWe’re talking about jobs for police officers, firefighters, teachers, bus drivers, people who keep our food safe, and more.\n\nTheir jobs are at stake if the government doesn’t act."), ('dates', '2020-04-27 02:10:25'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.6515559554100037)])
OrderedDict([('tweet', "It's hypocrisy from Senator McConnell\n\nHis $2T tax cut for the wealthiest few went to corporate stock buybacks and exploded the deficit\n\nBut now he suggests bankrupting states because of the deficit when jobs for police officers, firefighters, teachers, and more are on the line?"), ('dates', '2020-04-24 16:37:16'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5120065808296204)])
OrderedDict([('tweet', 'Not all heroes wear capes, some wear masks and scrubs.\n\nThese Americans are the true heroes of this pandemic, and we need to make sure they are taken care of.\n\nThat’s why Senate Democrats are fighting for a Heroes Fund.\n\nhttps://t.co/oIoH8vPTx3'), ('dates', '2020-04-24 00:36:36'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7569304704666138)])
OrderedDict([('tweet', 'We need a national testing strategy NOW.\n\nAnd because Senate Democrats fought for it, help is on the way to bolster the supply and manufacturing chain and to expand free testing for all, reporting, and contact tracing.'), ('dates', '2020-04-23 22:21:26'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.6232782006263733)])
OrderedDict([('tweet', 'From the Wall Street Journal:\n\n‘Immigrants in this pandemic are...helping our country to endure’\n\n‘Nearly all of the economic evidence shows that immigrants enhance American growth and jobs’\n\nStop the distractions, President Trump.\n\nPut our health first.\n\nhttps://t.co/FYGKPkjfsy'), ('dates', '2020-04-22 21:38:00'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.563742458820343)])
OrderedDict([('tweet', 'It’s been 50 years since we celebrated the first #EarthDay.\n\nAnd this year like every year, we all have to come together for our planet.\n\nBecause there is no Planet B. We have to fight climate change. And we have to act now.\n\n#EarthDay50'), ('dates', '2020-04-22 16:47:04'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5694730281829834)])
OrderedDict([('tweet', 'States, localities, tribal governments need support. So does the Postal Service. Working Americans need rental assistance. Frontline workers deserve hazard pay. We must ensure our 2020 elections are conducted fairly.\n\nDemocrats will not stop fighting for these priorities &amp; more.'), ('dates', '2020-04-21 23:09:26'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5404688119888306)])
OrderedDict([('tweet', "The bill before us is an interim measure.\n\nIt has hard-won provisions Democrats fought for, but it's a building block.\n\nIn the weeks ahead, Congress must prepare another major bill—big, bold, and ambitious."), ('dates', '2020-04-21 23:04:13'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.6282955408096313)])
OrderedDict([('tweet', "Democrats secured an agreement that's been transformed to provide real support for lives and livelihoods.\n\nIt's unfortunate the GOP refusal to negotiate delayed an agreement.\n\nAs soon as it passes the House, we'll advance CARES 2—which must be transformative and far-reaching. https://t.co/IBEd6MFKMh"), ('dates', '2020-04-21 22:02:16'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5635615587234497)])
OrderedDict([('tweet', 'On testing, we need major new investments to bolster the supply and manufacturing chain and significantly expand free testing for all, reporting, and contact tracing.\n\nThat’s why Democrats are fighting for our $30 billion plan for a comprehensive national testing strategy.'), ('dates', '2020-04-21 02:21:09'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5192967653274536)])
OrderedDict([('tweet', 'Democrats are fighting for small businesses, and hospitals and healthcare providers, and frontline workers, and state and local governments across the country. They need more help from the federal government, and they need it fast.'), ('dates', '2020-04-19 17:34:24'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.6650170683860779)])
OrderedDict([('tweet', 'Bigger companies with long-term relationships with bigger banks are getting access to the federal dollars but smaller businesses, minority owned businesses, businesses in more rural areas are not getting the dollars they need.\n\nWe’re fighting to help ALL small businesses.'), ('dates', '2020-04-19 14:55:13'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5701571106910706)])
OrderedDict([('tweet', 'Senate Democrats are fighting for the nail salons, the barbershops, the delis.\n\nWhy are Senator McConnell and Republicans only trying to help bigger businesses that already have connections to bigger banks?\n\nhttps://t.co/0ETwtIxAoh'), ('dates', '2020-04-17 19:03:45'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7756018042564392)])
OrderedDict([('tweet', 'We need a strong testing regime NOW\n\nDemocrats are fighting for our $30 billion plan for a comprehensive national testing strategy\n\nand major new investments to bolster the supply &amp; manufacturing chain, and significantly expand free testing for all, reporting, and contact tracing'), ('dates', '2020-04-17 16:20:35'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5304766893386841)])
OrderedDict([('tweet', 'This is big:\n\nDemocrats are laying out our $30 billion plan for a comprehensive national testing strategy\n\nand major new investments that bolster the supply and manufacturing chain, and significantly expand free testing for all, reporting, and contact tracing.'), ('dates', '2020-04-17 01:10:27'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7216815948486328)])
OrderedDict([('tweet', 'Democrats will not stop fighting for money for small businesses, and hospitals and healthcare providers, and testing, and our state and local governments, and more to help fight coronavirus.'), ('dates', '2020-04-16 21:53:26'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.9651767611503601)])
OrderedDict([('tweet', 'Democrats are fighting for funding for state and local governments.\n\nBecause they’re facing choices like laying off TEACHERS and POLICE and FIREFIGHTERS and MORE.\n\nThey need help. And fast.\n\nhttps://t.co/naQIwVKNDX'), ('dates', '2020-04-16 15:20:52'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.5524677634239197)])
OrderedDict([('tweet', "Our small biz programs need more money now\n\nWe all agree\n\nBut Dems actually want to get it into the hands of all who need it\n\nThat's not what's happening when websites are crashing, mom &amp; pop shops and minority businesses are shut out, bigger banks are loaning to fav customers"), ('dates', '2020-04-15 23:17:34'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7491000294685364)])
OrderedDict([('tweet', 'Democrats are presenting an ACCURATE picture of the challenges our country is facing with testing—not issuing false and misleading claims like President Trump\n\nThis isn’t the Chinese Communist Party. This is America. We need our government to be truthful.\n\nhttps://t.co/PwRYboK1fT'), ('dates', '2020-04-15 21:50:46'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.9794488549232483)])
OrderedDict([('tweet', 'To President Trump:\n\nYou need to get coronavirus testing under control. Now. Immediately.\n\nDemocrats fought for free coronavirus testing.\n\nAnd we won’t stop fighting to ensure that Americans are getting tested so we can beat this pandemic.'), ('dates', '2020-04-15 03:00:46'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7683780789375305)])
OrderedDict([('tweet', 'Senate Democrats fought to support the USPS.\n\nBut Republicans refused to include anything that would have given the USPS real help.\n\nWe won’t stop fighting for them.\n\nhttps://t.co/FulUpdSeBS'), ('dates', '2020-04-13 23:47:46'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.6552492380142212)])
OrderedDict([('tweet', 'Democrats will not stop fighting for money for small businesses, and hospitals, and testing, and our state and local governments, and more to help fight coronavirus.'), ('dates', '2020-04-13 15:04:27'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.9717211127281189)])
OrderedDict([('tweet', "Small businesses across America are telling us the @SBAgov Paycheck Protection Program isn't working.\n\nThat loan applications are not being processed and businesses are not getting the money in time.\n\nNot acceptable. President Trump and his admin need to fix these issues NOW."), ('dates', '2020-04-13 12:26:33'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7343972325325012)])
OrderedDict([('tweet', 'It is long past time for President Trump get coronavirus testing under control. This needs to happen immediately.\n\nDemocrats fought for free coronavirus testing. And we won’t stop fighting to ensure that Americans are getting tested so we can beat this pandemic.'), ('dates', '2020-04-13 11:42:56'), ('file_name', 'Charles E. Schumer.csv'), ('pct_true', 0.7797524333000183)])
OrderedDict([('tweet', 'The evidence continues to pile up that China withheld critical information from the world, allowing this deadly disease to spread. \n\nChina WILL be held accountable. \nhttps://t.co/TGsLUb1IiB'), ('dates', '2020-05-03 22:48:11'), ('file_name', 'Charles J. "Chuck" Fleischmann.csv'), ('pct_true', 0.6692530512809753)])
OrderedDict([('tweet', 'Extremely concerning report.\n\nMalign foreign actors (including China) are instigating cyber attacks against American hospitals, research labs, health care providers,  pharmaceutical companies, &amp; federal government agencies.\n\nWe must keep our research safe.\nhttps://t.co/S8TVBNuHpT'), ('dates', '2020-04-27 18:17:24'), ('file_name', 'Charles J. "Chuck" Fleischmann.csv'), ('pct_true', 0.5468549728393555)])
OrderedDict([('tweet', 'The U.S. must lead in holding China accountable.\n\nThe Chinese Communist Party lied &amp; tried to cover up their failure to contain COVID-19.\n\nTheir failure has now cost thousands of American lives &amp; millions of American jobs.\n\nVia @jeff_poor @BreitbartNews\nhttps://t.co/Zhyd9CVyXI'), ('dates', '2020-04-21 13:33:18'), ('file_name', 'Charles J. "Chuck" Fleischmann.csv'), ('pct_true', 0.9879646301269531)])
OrderedDict([('tweet', 'Concerning news from Hong Kong this weekend.\n\nThe arrests of pro-democracy advocates in Hong Kong is shameful. \n\nThe message must be clear: the Chinese Community Party CANNOT suppress the rising tide of democracy and the desire for freedom.\nhttps://t.co/NizPdnVHBD'), ('dates', '2020-04-19 18:48:29'), ('file_name', 'Charles J. "Chuck" Fleischmann.csv'), ('pct_true', 0.5231546759605408)])
OrderedDict([('tweet', 'All Americans, especially elected officials must stay vigilant.\n\nThe Chinese Communist Party will continue to try use propaganda to  pressure &amp; disinform in order to divide our nation.\n\nChina lied to the world and they WILL be held accountable.\nhttps://t.co/JVSEKO1mtq'), ('dates', '2020-04-17 13:05:28'), ('file_name', 'Charles J. "Chuck" Fleischmann.csv'), ('pct_true', 0.9209635257720947)])
OrderedDict([('tweet', '#Smallbiz in my district are not only struggling w/ the effects of COVID-19 but also a deadly tornado.\n\nThe American people have had enough of these partisan games by Senate &amp; House Democrats.\n\nWe NEED a clean funding bill for the Paycheck Protection Program &amp; we need it NOW. https://t.co/qb6IuSwF0q'), ('dates', '2020-04-16 14:55:56'), ('file_name', 'Charles J. "Chuck" Fleischmann.csv'), ('pct_true', 0.9271472096443176)])
OrderedDict([('tweet', "It's appalling that Trump is using a deadly pandemic to advance a hateful, bigoted agenda. While millions of immigrants are doing essential jobs (many w/o PPE) this admin is trying to divert attn from its failure to respond swiftly to #COVID19, costing 40k lives+millions of jobs. https://t.co/A4wCanaqe4"), ('dates', '2020-04-21 13:19:14'), ('file_name', 'Chellie Pingree.csv'), ('pct_true', 0.8706899881362915)])
OrderedDict([('tweet', '#COVID19 has put a strain on every state’s health care system. The Trump admin should have responded months ago by ramping up PPE production + coordinating the transparent distribution of critical medical supplies rather than leaving states + providers to fend for themselves. https://t.co/9YxOlg58ze'), ('dates', '2020-04-13 14:04:08'), ('file_name', 'Chellie Pingree.csv'), ('pct_true', 0.5261133313179016)])
OrderedDict([('tweet', 'People are hurting from #COVID19 - including our family farms, small businesses &amp;seniors. They cannot be overlooked. That is why today I am meeting with the @AppropsDems Labor-HHS Subcommittee to investigate #COVID19 response &amp; ensure districts like ours aren’t left overwhelmed. https://t.co/N5DuiOlKr2'), ('dates', '2020-05-06 13:48:19'), ('file_name', 'Cheri Bustos.csv'), ('pct_true', 0.6024728417396545)])
OrderedDict([('tweet', 'THREAD: "If China retaliates against the United States at this time, it will also announce strategic control over medical products and ban exports to the United States...the US will be caught in the ocean of the new coronaviruses," Xinhua, a commie state-run media agency.'), ('dates', '2020-05-04 19:30:37'), ('file_name', 'Chip Roy.csv'), ('pct_true', 0.8304452896118164)])
OrderedDict([('tweet', 'Catch @CBedfordDC and I discuss how restaurants (and small businesses) are being hammered and how the bill Congress just passed won’t save them. \n\nhttps://t.co/9bi5peQIdD'), ('dates', '2020-04-24 22:15:18'), ('file_name', 'Chip Roy.csv'), ('pct_true', 0.532764196395874)])
OrderedDict([('tweet', "What are Congress and state / local governments not factoring in with the current #coronavirus response? \n\nHow many folks are developing cancer but can't get screening? How many women aren't able to get #mammograms right now bc it isn't deemed essential? \n\n#CongressDoesNotKnow"), ('dates', '2020-04-23 19:08:02'), ('file_name', 'Chip Roy.csv'), ('pct_true', 0.6156436800956726)])
OrderedDict([('tweet', '"Where Has Congress Been? 43% of American households have had a significant or total pay cut. 26.4 million Americans have filed for unemployment. ENOUGH. No More HALF-ASSED Legislation." \n\nCongress should be here and should have been here fixing the mess we created. #DoOurJob https://t.co/cSuyfDmVzd'), ('dates', '2020-04-23 17:39:34'), ('file_name', 'Chip Roy.csv'), ('pct_true', 0.5843700170516968)])
OrderedDict([('tweet', 'No more playing politics with the lives and livelihoods of the American people, @HouseDemocrats @SenateDems. \n\nOur small businesses -- the backbone of the American economy -- are hurting. They need this money. They needed it yesterday \n\nIn what world is this OK?'), ('dates', '2020-04-17 15:22:19'), ('file_name', 'Chip Roy.csv'), ('pct_true', 0.9491673111915588)])
OrderedDict([('tweet', 'Serious Question: Why are @HouseDemocrats, @SenateDems more interested in supporting communist China than getting US businesses restarted? Think about that. https://t.co/2cbuvLXt0D'), ('dates', '2020-04-16 11:54:21'), ('file_name', 'Chip Roy.csv'), ('pct_true', 0.7694864273071289)])
OrderedDict([('tweet', 'It is honestly unbelievable that @HouseDemocrats, @SenateDems are holding money hostage meant to provide small businesses support and keep workers in jobs. No more politics. We are dealing with lives and livelihoods. It’s sick. \n\n@foxandfriends \n\nhttps://t.co/94fxzREX85'), ('dates', '2020-04-16 11:52:21'), ('file_name', 'Chip Roy.csv'), ('pct_true', 0.724853515625)])
OrderedDict([('tweet', 'Did you know I introduced the #BEATCHINA Act last week? \n\nAs we are seeing play out in real time, the US does not have nearly enough control of the pharmaceutical supply chain. We need to be looking for options today to alleviate our dependence on China for pharmaceuticals. https://t.co/i3WVMlwzt6'), ('dates', '2020-03-24 23:47:09'), ('file_name', 'Chip Roy.csv'), ('pct_true', 0.5381713509559631)])
OrderedDict([('tweet', "Democrats are stopping Americans from getting relief over aid for abortion providers, federalized elections, + a bailout of the postal service. \n\nWhen we need to work together, Democrats can only think about themselves. You've seen their true colors. \n\nStop playing politics!"), ('dates', '2020-03-24 23:15:48'), ('file_name', 'Chip Roy.csv'), ('pct_true', 0.908091127872467)])
OrderedDict([('tweet', 'As Congress adds money for loan programs for our small businesses, it’s imperative that publicly-traded companies with tremendous resources aren’t able to get SBA help. Main Street businesses are devastated and that’s where our support must go. https://t.co/QJitfVCSu0'), ('dates', '2020-04-23 21:23:47'), ('file_name', 'Chris Pappas.csv'), ('pct_true', 0.5541328191757202)])
OrderedDict([('tweet', 'It’s ridiculous that @SpeakerPelosi has now held up both COVID-19 relief bills. And for what? \n\nHow many business went under because of her? How many Americans lost their job because of her?'), ('dates', '2020-04-21 19:58:50'), ('file_name', 'Chris Stewart.csv'), ('pct_true', 0.9747427701950073)])
OrderedDict([('tweet', 'This is just wrong. Small businesses are now the pawns in @SpeakerPelosi’s political game. https://t.co/DcTMwBOt8j'), ('dates', '2020-04-16 16:03:14'), ('file_name', 'Chris Stewart.csv'), ('pct_true', 0.6007452011108398)])
OrderedDict([('tweet', 'Absolutely the right thing to do. China has a LONG list of questions they need to answer. So does Director-General @DrTedros\xa0and other leadership at the @WHO. \n\nhttps://t.co/GVOT0AP2iX'), ('dates', '2020-04-14 23:47:37'), ('file_name', 'Chris Stewart.csv'), ('pct_true', 0.7200648188591003)])
OrderedDict([('tweet', "I agree with Senator Scott. We must understand China's lies &amp; deceptions and how the @WHO facilitated them. How many more people will die because of this? \n\nhttps://t.co/la2X4GKLet"), ('dates', '2020-04-01 21:27:56'), ('file_name', 'Chris Stewart.csv'), ('pct_true', 0.7566213607788086)])
OrderedDict([('tweet', 'This is an unprecedented time in our nation’s history. We’re facing an invisible enemy that has affected every American in one way or another. Millions have lost their jobs and millions are worried about their health and livelihood.'), ('dates', '2020-03-27 18:55:53'), ('file_name', 'Chris Stewart.csv'), ('pct_true', 0.7928974628448486)])
OrderedDict([('tweet', "The surveillance status quo ends today.\n\nToday, the House will vote on the much improved USA Freedom Act reauthorization. This bill includes most of the reforms I've put forth in my own FISA Improvements Act:"), ('dates', '2020-03-11 16:29:22'), ('file_name', 'Chris Stewart.csv'), ('pct_true', 0.714832603931427)])
OrderedDict([('tweet', "House Democrats don't have a budget. They haven't done the work. They're not in the arena saying what they would do. Yet they continue to sit on the sidelines, criticizing the efforts of the President's budget. https://t.co/9v8Igcymok"), ('dates', '2020-02-12 22:32:35'), ('file_name', 'Chris Stewart.csv'), ('pct_true', 0.9330081343650818)])
OrderedDict([('tweet', 'The next time Trump boasts about the stock market going up, remember stocks surged yesterday on one of worst days ever for American workers and U.S. economy. How he measures success tells us who he really cares about.'), ('dates', '2020-05-09 17:16:00'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5567959547042847)])
OrderedDict([('tweet', 'As a White House lawyer, Brian Miller personally participated in stonewalling the GAO inquiry that found Trump broke the law by withholding aid from Ukraine. Covering up is apparently the criteria for IG nominations by Trump. The IGs who actually do their jobs get fired. https://t.co/rZIzIWJKh3'), ('dates', '2020-05-05 23:37:32'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5094541907310486)])
OrderedDict([('tweet', 'Now Trump wants to impose a gag order to stop Fauci from testifying to Congress. His excuse? That it would be “counterproductive." Translation: he doesn\'t want the public to hear the full truth, like “don’t inject disinfectants” or “we need more rapid testing." Free Fauci! https://t.co/NYBqE592LT'), ('dates', '2020-05-01 22:38:37'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.8213058710098267)])
OrderedDict([('tweet', 'This is exactly why I’ve pressed @USGSA for details on what rent relief the Trump Hotel is requesting from them. Just see how American taxpayers are already footing the bill at Trump’s luxury hotel and lining his pockets. https://t.co/nCEHlcLXLd'), ('dates', '2020-05-01 14:10:24'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.6235861778259277)])
OrderedDict([('tweet', 'The American people deserve to know that the President isn’t giving any secret favors to Deutsche Bank. It’s entirely unacceptable that the bank’s executives are refusing to disclose their recent deals with him. https://t.co/kTu8c2yZfN'), ('dates', '2020-04-29 13:06:37'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.8748464584350586)])
OrderedDict([('tweet', 'Never knew we would have to say this to a president, but there is no place for "sarcastic" comments on drinking or injecting bleach to treat COVID-19.\n \nAmericans are dying. Health care workers don’t have enough protective equipment. We don’t have enough tests. https://t.co/spfshS9MOT'), ('dates', '2020-04-24 17:51:14'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.744996964931488)])
OrderedDict([('tweet', 'Now the Trump Organization is asking the Trump Administration to give the Trump Hotel a break? This is a recipe for abuse of power and undue influence. We must shine a light to stop Trump from again using the government to advance his financial interests. https://t.co/dCLTpNNmdh'), ('dates', '2020-04-24 00:21:45'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.7017729878425598)])
OrderedDict([('tweet', 'This #EarthDay, we face a serious threat to progress: Trump. We must stop him from using this pandemic to weaken environmental laws and help Big Oil.\n\nLet’s unite the American people behind stopping climate change with the same spirit that created Earth Day 50 years ago. https://t.co/OLh8ZAmQke'), ('dates', '2020-04-22 21:21:52'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.8648698329925537)])
OrderedDict([('tweet', 'Trump has been warned by public health officials and businesses that a rush to end social distancing, without dramatically expanding testing, will have grave consequences. He must put the health of our country ahead of his desire to engage in an unconstitutional power grab.'), ('dates', '2020-04-16 19:44:57'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.7212186455726624)])
OrderedDict([('tweet', 'Millions of Americans in desperate financial straits will have to wait longer to get their individual payment checks because Donald Trump wants to add his name to them.\n \nNo other American President has done this.\n \nHis COVID-19 strategy: Trump First.'), ('dates', '2020-04-15 14:52:33'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.7182328701019287)])
OrderedDict([('tweet', "How does a stable genius deal with a global pandemic? \n\nFirst threaten to fire Dr. Fauci, the nation's top public health expert.\n\nAnd then cut off funds for the World Health Organization. \n\nWhy didn’t someone else think of that?"), ('dates', '2020-04-14 23:03:32'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.8164370059967041)])
OrderedDict([('tweet', 'Ross said the pandemic would be good for American jobs and Kushner said the federal stockpile isn’t intended to help states.\n\nThis is who Trump chooses to be on his recovery council? \n\nNo public health experts, real economists, workers, or small business leaders. https://t.co/HXOJHEX8sA'), ('dates', '2020-04-14 00:41:25'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.6313741207122803)])
OrderedDict([('tweet', "It's ridiculous the number of hoops that small businesses must jump through to receive an Economic Injury Disaster Loan. Congress should act to address this if the Administration won’t. https://t.co/Tzzcyv46Ip"), ('dates', '2020-04-13 22:56:22'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.6409226059913635)])
OrderedDict([('tweet', 'At the White House briefing, Trump needs to correct the record—not give more mixed messages. Bullying @WHO is the last thing he should be doing in the midst of a global pandemic. We cannot politicize the coronavirus away. We need to work together to save lives.'), ('dates', '2020-04-08 22:08:27'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.608106255531311)])
OrderedDict([('tweet', "Trump just can't stop playing politics even when lives are at stake. \n\nWe need an independent watchdog to force accountability on this Administration for the $2 trillion emergency relief package. \n\nAmerican families and businesses are counting on it. https://t.co/IPbbGpetiN"), ('dates', '2020-04-07 19:37:17'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.6812909841537476)])
OrderedDict([('tweet', 'Modly should be fired and @DoD_IG needs to immediately investigate the Administration’s disturbing decision to relieve Capt. Crozier. Navy leadership can\'t put politics over the safety of sailors or have Modly\'s all-out focus on avoiding getting “crossways with the President." https://t.co/qYDWJ8VC1v https://t.co/hqHmOH4B6P'), ('dates', '2020-04-06 21:48:34'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5457594394683838)])
OrderedDict([('tweet', 'As we suspend many of our normal daily activities, WE MUST NOT SUSPEND OUR DEMOCRACY.\n\nYet the Republican Party is seeking to block efforts to make it easier to vote safely—like the option of voting by mail. \n\nElections should be about addition, not subtraction!'), ('dates', '2020-04-05 17:31:46'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5243262648582458)])
OrderedDict([('tweet', 'Mr. President, stop worrying about your TV ratings and address this shortage in Maryland, our region, and throughout the country. Our delegation has been urging FEMA to help get these supplies for weeks. Stop playing the President on TV and start actually being one. https://t.co/IZNKFd8WXq'), ('dates', '2020-04-02 17:58:51'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5125206112861633)])
OrderedDict([('tweet', 'Amazing how the GOP, McConnell, and Trump always try to blame Nancy Pelosi for their own screw-ups. The Phase 3 emergency bill was developed and delayed in the Senate, not the House. Remember, Pelosi, steered Phase 1 and 2 through House without a hitch. https://t.co/BqR1Ufvbld https://t.co/QoBgH3Cm4q'), ('dates', '2020-03-30 19:46:42'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5037572979927063)])
OrderedDict([('tweet', 'Here’s the thing: McConnell is pointing fingers, but Democrats are at the table. Rather than playing politics, he could have delayed the vote until later today and committed to a bipartisan bill. This was all for a partisan show while our country suffers. https://t.co/PNka5nmhcS'), ('dates', '2020-03-23 20:05:25'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.7924962043762207)])
OrderedDict([('tweet', 'We are all paying the price for this gross negligence: it turns out that after Trump axed the White House office created to fight pandemics they also eliminated a key American CDC scientist assigned to China to detect outbreaks of the disease. https://t.co/ygQyDduvxc'), ('dates', '2020-03-23 14:03:22'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.8416392207145691)])
OrderedDict([('tweet', 'Bad news. After saying they wanted to join us in helping workers, families, and small + midsized businesses that are going under, Trump and McConnell have taken a total U-turn. They just want to bail out big corporate cronies at everyone else’s expense. Unacceptable.'), ('dates', '2020-03-22 18:05:27'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.7402752637863159)])
OrderedDict([('tweet', "Here we are in the midst of a national health emergency and Trump's political cronies are using the moment to purge the National Counterterrorism Center of its nonpartisan professionals who work to keep us safe. This will come back to haunt us. https://t.co/hgd0ckFVYk"), ('dates', '2020-03-20 14:20:55'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5034894347190857)])
OrderedDict([('tweet', 'After leaving the briefing on the coronavirus and hearing the President’s misdirected speech last night, we urgently need to pass the House legislation to further bolster the response and support those who may be out of work because of the crisis. Together, let’s get this done.'), ('dates', '2020-03-12 15:33:07'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5117142796516418)])
OrderedDict([('tweet', 'We are dealing with a global pandemic and the Trump Administration still wants to cut billions from CDC, HHS, and the Infectious Diseases Rapid Response Reserve Fund. Not only does this defy logic when these agencies are fighting the coronavirus, but it’s downright dangerous. https://t.co/uOikRi9bhr'), ('dates', '2020-03-12 02:40:29'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.6565631628036499)])
OrderedDict([('tweet', 'Over a year ago, the House passed #HR1, the most sweeping anti-corruption and campaign finance bill in history.\n\nIt includes my bills to require disclosure of dark money, reform the FEC, and fix the presidential public financing system.\n\nCall on McConnell to hold a vote.'), ('dates', '2020-03-11 19:35:30'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.6157732009887695)])
OrderedDict([('tweet', "We must protect consumers who are in financial danger due to the coronavirus leaving them out of work. Right now, 40% of American households would not be able to cover a $400 emergency expense; they'd be unable to pay their rent, car payments, and credit card bills. https://t.co/iNvd4BLVrY"), ('dates', '2020-03-10 22:59:27'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5195795893669128)])
OrderedDict([('tweet', 'The Senate is considering a bipartisan energy package this week. Despite McConnell’s efforts to block amendments, @SenMarkey and I have filed a proposal to study the creation of a National Climate Bank—which would accelerate the deployment of clean energy to fight climate change. https://t.co/poLuz95F2k'), ('dates', '2020-03-10 14:22:55'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.7242535948753357)])
OrderedDict([('tweet', 'This is why active congressional oversight is so important. \n\nIt shines a light on wrongdoing. I salute @RepMaxineWaters and @FSCDems for doing their job. \n\nToo bad Mitch McConnell and too many Senate Committee chairs refuse to do theirs. https://t.co/7nJ6Sqw2fC'), ('dates', '2020-03-09 18:27:58'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.7243719100952148)])
OrderedDict([('tweet', 'Today the Supreme Court heard oral arguments on a Louisiana law that directly challenges Roe v. Wade. This case highlights the risk of putting an anti-choice majority on the bench. We must continue to fight efforts to strip away constitutional rights. https://t.co/RiNIaxnowc'), ('dates', '2020-03-05 01:44:24'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5749213099479675)])
OrderedDict([('tweet', "Bernhardt continues to act like BP's lawyer, not the Secretary of the Interior.\n\nIf we’re ever going to make progress on the Migratory Bird Treaty Act, then we should be able to have a civil conversation.\n\nWe need straight answers about the impacts of Interior’s proposed rule. https://t.co/11kWsaVVME"), ('dates', '2020-03-04 16:31:45'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.5291826725006104)])
OrderedDict([('tweet', 'Even in the midst of the coronavirus crisis, the Trump Administration seeks to promote junk plans that endanger access to care and is asking the courts to rip away protections from millions with pre-existing health conditions. Republicans need to end their partisan lawsuit. https://t.co/KOakqdMmCY'), ('dates', '2020-03-02 19:10:47'), ('file_name', 'Chris Van Hollen.csv'), ('pct_true', 0.7927370071411133)])
OrderedDict([('tweet', "Workers are being laid off through no fault of their own and are losing access to lifesaving health insurance during this #coronavirus pandemic. I stand with @GoIUPAT and am working in Congress to make sure these workers don't lose their coverage. https://t.co/lFTzicH1bz"), ('dates', '2020-04-06 12:54:45'), ('file_name', 'Chrissy Houlahan.csv'), ('pct_true', 0.5382943153381348)])
OrderedDict([('tweet', "We are in crisis - a literal pandemic. The President &amp; Congressional Leadership need to act like it. We don't have hours. We don't even have minutes. Knock it off &amp; pass the bill. I'm ready to vote to start delivering relief to Pennsylvanians &amp; Americans who desperately need it."), ('dates', '2020-03-25 21:05:36'), ('file_name', 'Chrissy Houlahan.csv'), ('pct_true', 0.9274182915687561)])
OrderedDict([('tweet', 'Small businesses that have contracts w/ our federal government need expedited payments. I led a bipartisan effort w/ @RepSlotkin &amp; @RepBalderson to provide that. Small businesses are the backbone of our economy, &amp; we need to do whatever we can to lift them up right now. https://t.co/NI78eWHFLW'), ('dates', '2020-03-23 16:44:32'), ('file_name', 'Chrissy Houlahan.csv'), ('pct_true', 0.691895067691803)])
OrderedDict([('tweet', 'We need to activate American manufacturing so that the American supply chain can efficiently deliver PPE, diagnostic equipment, and – eventually – a COVID-19 vaccine. https://t.co/luW5HeEl7G'), ('dates', '2020-05-07 22:13:29'), ('file_name', 'Christopher A. Coons.csv'), ('pct_true', 0.576367974281311)])
OrderedDict([('tweet', "I'm a Democrat and @BillKristol is a lifelong conservative, but we agree that coronavirus denialism from President Trump and too many Republicans in Congress will hurt our economy and cost American lives. @usatodayopinion https://t.co/yWQvwCyEzQ"), ('dates', '2020-05-07 16:57:00'), ('file_name', 'Christopher A. Coons.csv'), ('pct_true', 0.5485556125640869)])
OrderedDict([('tweet', 'Tomorrow – instead of deliberating on a COVID-19 response – Mitch McConnell has set a Senate agenda that focuses on rushing a hearing on an inexperienced and partisan nominee to the D.C. Circuit.'), ('dates', '2020-05-05 15:11:14'), ('file_name', 'Christopher A. Coons.csv'), ('pct_true', 0.9092636108398438)])
OrderedDict([('tweet', 'Amid a pandemic, Leader McConnell has brought the Senate into session – not to deliberate on oversight &amp; the federal response to COVID-19, but to rush through an under-qualified judicial nomination for a seat that won’t even be open until September.  \nhttps://t.co/sWSTlyg2sv'), ('dates', '2020-05-05 01:52:09'), ('file_name', 'Christopher A. Coons.csv'), ('pct_true', 0.8298426270484924)])
OrderedDict([('tweet', 'There is absolutely no place for racism, hatred, and bigotry in our politics – I condemn this cowardly action in the strongest possible terms. \nDelaware’s diversity is our strength &amp; I’m proud to stand with the Delaware Legislative Black Caucus &amp; @RepLBR.\nhttps://t.co/SLKk4YSriA'), ('dates', '2020-04-30 20:23:41'), ('file_name', 'Christopher A. Coons.csv'), ('pct_true', 0.5677686929702759)])
OrderedDict([('tweet', 'In the middle of a pandemic, Mitch McConnell is forcing senators – &amp; hundreds more in staff &amp; Capitol employees – to travel to the Capitol, not to work on the #COVID19 response, but simply to force another right-wing judicial nominee to the federal bench. https://t.co/aicdNoygYp'), ('dates', '2020-04-30 16:20:32'), ('file_name', 'Christopher A. Coons.csv'), ('pct_true', 0.7306808829307556)])
OrderedDict([('tweet', 'Without immediate action, SBA’s flagship lending program will shut down, denying small businesses struggling from the pandemic the ability to access an emergency relief provision I authored as part of the CARES Act. My colleagues &amp; I are urging @SBAgov to swiftly resolve this. https://t.co/2xaYDbmVWf'), ('dates', '2020-04-30 13:56:01'), ('file_name', 'Christopher A. Coons.csv'), ('pct_true', 0.6309712529182434)])
OrderedDict([('tweet', "Here's the problem: Other countries are prepared to manufacture a #COVID19 vaccine quickly &amp; at scale; the US is not.\n\nWe cannot rely solely on countries like China for ingredients, supplies, or vaccine production. We need to improve our manufacturing capacity here at home."), ('dates', '2020-04-25 15:01:34'), ('file_name', 'Christopher A. Coons.csv'), ('pct_true', 0.7943706512451172)])
OrderedDict([('tweet', 'When @senatemajldr says he doesn’t want to support state &amp; local governments, he’s saying that the federal government shouldn’t support firefighters, unemployment offices, teachers, health officials, trash collectors &amp; more. That’s what state &amp; local governments need funding for. https://t.co/cxd0F7zSeu'), ('dates', '2020-04-23 20:00:00'), ('file_name', 'Christopher A. Coons.csv'), ('pct_true', 0.5786256790161133)])
OrderedDict([('tweet', 'The #COVID19 pandemic is a crisis facing the entire country, &amp; part of the federal response must be to help states prepare to hold elections in this new environment.'), ('dates', '2020-04-23 15:35:26'), ('file_name', 'Christopher A. Coons.csv'), ('pct_true', 0.5446106791496277)])
OrderedDict([('tweet', 'ICYMI: @SSAPress has a new campaign to fight scammers. If u/someone u know receives a suspicious call stating there is a problem w/their SSN/account. Hang up. Never give the caller money/personal info. Ppl should then report the scam to https://t.co/Mp0WRgDICh or 📞1.800.772.1213 https://t.co/PVdpI9pisG'), ('dates', '2020-01-28 19:25:40'), ('file_name', 'Christopher H. Smith.csv'), ('pct_true', 0.5370178818702698)])
OrderedDict([('tweet', 'Ride-share companies continue to fail to protect passengers from fake drivers &amp;/or sexual predators. It is time to pass Sami’s Law-HR4686-legislation to protect passengers-especially women-who use ride-share services from #assault. @uber @lyft \n\nhttps://t.co/B8bO4MfjqQ https://t.co/LWQvrN9Yay'), ('dates', '2019-12-06 15:10:37'), ('file_name', 'Christopher H. Smith.csv'), ('pct_true', 0.5164858102798462)])
OrderedDict([('tweet', "#China must be held accountable for human rights abuses against millions of Muslim-majority Uyghurs. We cannot be silent. Tuesday's passage of the #UyghurHumanRightsPolicyAct takes the next step in demanding accountability from the #Chinesegovernment. \n\nhttps://t.co/RT8F5tlUYz https://t.co/snrNCM8626"), ('dates', '2019-12-05 20:25:48'), ('file_name', 'Christopher H. Smith.csv'), ('pct_true', 0.8388659954071045)])
OrderedDict([('tweet', 'Enactment of #HongKongHumanRightsandDemocracyAct makes it abundantly clear that the Trump Administration, US Congress, &amp; the American people stand in solidarity with the people of #HongKong. Beating, torturing, jailing of democracy activists is wrong.'), ('dates', '2019-11-28 02:17:35'), ('file_name', 'Christopher H. Smith.csv'), ('pct_true', 0.8372755646705627)])
OrderedDict([('tweet', '5 years ago—in 2014—joined by @SpeakerPelosi, 1st introduced the bipartisan #HongKongHumanRightsandDemocracyAct, &amp; I reintroduced it in 2015, 2017 &amp; again this year. The current version, #HR3289—cosponsored by @RepMcGovern—is a blueprint for meaningful action. Pass it today.'), ('dates', '2019-10-15 19:50:21'), ('file_name', 'Christopher H. Smith.csv'), ('pct_true', 0.9012759923934937)])
OrderedDict([('tweet', 'The internet should not enable human trafficking, &amp; those complicit must be held accountable. I was proud to co-sponsor The Allow States and Victims to Fight Online Sex Trafficking Act of 2017 to crack down on this abuse. The Senate should act quickly to pass this critical bill.'), ('dates', '2018-02-28 21:28:54'), ('file_name', 'Christopher H. Smith.csv'), ('pct_true', 0.790148138999939)])
OrderedDict([('tweet', 'Today is #HumanTraffickingAwarenessDay. The suffering of more than 25 million trafficking victims worldwide—who are mostly women and children—must spur us to work even harder for their rescue, for the prosecution of traffickers, and to end this cruelty.'), ('dates', '2018-01-11 20:06:27'), ('file_name', 'Christopher H. Smith.csv'), ('pct_true', 0.5933398604393005)])
OrderedDict([('tweet', 'Letting #LiuXiaobo out of prison does not mean he is free. #China must allow him the freedom to seek medical care in any country. https://t.co/Zcpghw7n8J'), ('dates', '2017-06-30 20:43:45'), ('file_name', 'Christopher H. Smith.csv'), ('pct_true', 0.6029203534126282)])
OrderedDict([('tweet', 'To all incensed ppl who dislike Barr setting Flynn free and say it is never done. THINK / most prosecutions r not as corrupt unusual unconstitutional and illegal as used on Flynn'), ('dates', '2020-05-08 21:43:46'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.642044186592102)])
OrderedDict([('tweet', 'Gr8 to see Pres @realDonaldTrump pushing 2 investigate cause of price disparity in beef supply chain Producers must sell cattle 4loss yet prices at meat counter r sky high I called for USDA/DOJ 2examine possible violations of antitrust law Now USDA is on the case What abt DOJ?'), ('dates', '2020-05-08 21:35:21'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.7929121255874634)])
OrderedDict([('tweet', '2day marks 30 yrs since the Declaration on the Restoration of Independence of the Republic of Latvia was adopted expressing the will of the Latvian ppl to be free of the Soviet occupation Many Latvians fled the Soviets some settled in Iowa'), ('dates', '2020-05-04 16:52:53'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.7113686203956604)])
OrderedDict([('tweet', 'AttGen Barr: it’s so obvious the unconstitutional punishment that GenFlynn has had for four long years u need to intervene and #FREE FLYNN'), ('dates', '2020-04-30 15:51:09'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.7626628279685974)])
OrderedDict([('tweet', "Many of my colleagues r rightfully calling for prosecution of ppl who violated Flynn's constitutional rights MORE IMPORTANTLY the whole country needs to know how his rights were violated TRUTH getting out is most important so this FBI abuse wont be repeated again on someone else"), ('dates', '2020-04-30 15:46:59'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.6262623071670532)])
OrderedDict([('tweet', 'Thx SCOTUS for correctly applying my recent updates to my Anti-Terrorism Act +forcing DC &amp; 2nd Circuits to reconsider their flawed decisions against terror victims Time for lower courts to GET IT RIGHT &amp;protect justice for American victims of PLO terrorism!'), ('dates', '2020-04-28 01:53:21'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.5364035367965698)])
OrderedDict([('tweet', 'Watched Iowa Press &amp;they talked abt remarkable flow of liquidity thru SBA paycheck protection program &amp;its impact Iowa needs more resources 2meet demand Iowans r calling on Congress 2act Dems pls don’t stand in way of small biz! PPP is wrking in Iowa 4.3B$ loaned to nearly 30kbiz'), ('dates', '2020-04-18 16:56:15'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.9448371529579163)])
OrderedDict([('tweet', 'ironic how FBI investigation into Russian disinformation/collusion w political campaign was built on flawed Steele dossier that was full of anti-Trump Russian disnformation+paid for by Clinton/DNC\xa0 FBI tasked with guarding national security WE NEED YOU TO DO BETTER!!'), ('dates', '2020-04-10 23:19:11'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.6174421906471252)])
OrderedDict([('tweet', 'These notes beg question of when FBI learned of Russian disinfo in dossier+what did they do about it?? Why did they keep renewing spying authority with this info? What did Mueller know and when? Did we even need Mueller?\n\xa0\nMore unmasking to come'), ('dates', '2020-04-10 23:11:00'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.8370288610458374)])
OrderedDict([('tweet', 'shldnt hv taken 3 yrs+$30 million +special counsel 2debunk Trump collusion theory if FBI did their homework/followed rules We know there was no collusion &amp; FISAs were bogus Thx 2declassification by Barr/Grenell/ @realDonaldTrump we now know the Russians were playing FBI frm start'), ('dates', '2020-04-10 23:07:44'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.9652072191238403)])
OrderedDict([('tweet', 'After asking USDA to investigate price fixing by beef packers they announced today they will Farmers need to see USDA report as soon as possible Beef producers need help! Thx Scty Perdue &amp; Pres Trump for hearing their concerns'), ('dates', '2020-04-08 21:23:34'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.5504314303398132)])
OrderedDict([('tweet', 'Dept of homeland security made the right decision by not expanding h2b worker visas American workers sidelined by coronavirus shldnt hv 2compete w cheap foreign labor/temp foreign workers for job Now is no time to increase visas'), ('dates', '2020-04-02 19:43:45'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.5777280330657959)])
OrderedDict([('tweet', "Transparency brings acctability but DoD wants neither Ridiculous proposal 2classify entire DoD future spending plan which has been unclassified since 1989 Shld b roundly rejected by Congress! Ppl deserve to know what Pentagon is planning to spend + it's the ppl’s biz/taxpayer$$"), ('dates', '2020-03-30 22:42:35'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.9375842809677124)])
OrderedDict([('tweet', 'Beef is flying off grocery shelves but farmers are seeing prices go down If packers are illegally manipulating markets during crisis we need USDA &amp; DOJ &amp; CFTC to investigate + help farmers. 4 companies control 80% of market &amp;theyre taking advantage'), ('dates', '2020-03-27 15:38:55'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.9453691244125366)])
OrderedDict([('tweet', 'Finally after 3 lost days during a period of gr8 anxiety for American workers all due to Democrat obstacles we r passing a bill that’ll jumpstart US economy that’s shutdown bc of pandemic &amp;resulting biz closures GOOD NEWS for wrkers families +esp small biz BACK 2 WORK USA ECONOMY'), ('dates', '2020-03-26 03:52:24'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.5630180239677429)])
OrderedDict([('tweet', 'Pres @realDonaldTrump’s declaring a major disaster exists in Iowa bc of the coronavirus will give the state govt led by Gov Reynolds more resources+federal $$$ to combat the spread of the disease/respond to the impact it’s having on the state/Iowans'), ('dates', '2020-03-25 01:51:14'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.5595678687095642)])
OrderedDict([('tweet', 'Pres @realDonaldTrump is right Vote on coronavirus relief legis 2DAY! We shldve passed our bipartisan bill yesterday Needless delays by Democrats are hurting the ppl who need help most Ppl who are unemployed bc of the virus &amp; gov’t shutting down biz bc of no fault of their own'), ('dates', '2020-03-24 13:48:43'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.6777819395065308)])
OrderedDict([('tweet', "Dems are criticizing Trump admin handling of coronavirus but TWICE blocked a bipartisan bill to fund fed govt response They're holding taxpayers &amp; workers hostage to push unrelated Dem priorities This is a national emergency not a time for politics/partisanship!"), ('dates', '2020-03-23 21:34:31'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.9846425652503967)])
OrderedDict([('tweet', 'Im calling on Senate+House Democrats 2stop the politics! House Dem majority led phase 2 Now SenateGOP majority leading phase 3 negotiating w Dems for days NO time to waste 4 badly needed relief for American ppl Rs joined Dems for passage of phase 2 Dems shld join Rs for phase 3!'), ('dates', '2020-03-22 21:46:47'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.7490007877349854)])
OrderedDict([('tweet', 'Heard rumors that White House is trying 2move a surprise billing bill + health care extenders w/out moving Grassley-Wyden rx drug bill If that happens it will b much harder to get Grassley-Wyden. @realdonaldtrump badly wants Rx prices lowered! Plus it SAVES $95B'), ('dates', '2020-03-21 23:53:56'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.8976718783378601)])
OrderedDict([('tweet', 'I don’t understand why China gets upset bc we refer to the virus that originated there the ”Chinese virus” Spain never got upset when we referred to the Spanish flu in 1918&amp;1919'), ('dates', '2020-03-18 11:54:43'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.9894569516181946)])
OrderedDict([('tweet', 'China’s decision to kick out even more American journalists is a disgrace. Freedom of press promotes transparency &amp; accountability especially in times of crisis like now w the spread of coronavirus from Wuhan. More evidence that Xi has something to hide'), ('dates', '2020-03-17 20:50:41'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.9431963562965393)])
OrderedDict([('tweet', '2day at 948AM we just received the bill from the House the “Families First Coronavirus Response Act” What business do democrats hv blaming McConnell all wknd for not calling a vote when we can’t do anything until the House sends it over? They know better Stop the politics!'), ('dates', '2020-03-17 13:52:46'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.5501635670661926)])
OrderedDict([('tweet', 'I dont understand why China wants 2b so secretive abt the virus General Scty Xi owes it to the world 2b totally transparent so we know what happened The disappearance of Mr Ren is evidence Gen Scty Xi is hiding something 2/2'), ('dates', '2020-03-17 13:13:37'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.910021185874939)])
OrderedDict([('tweet', 'Why wld Congress cede constitutional oversight responsibility bc its an election yr? Unaccountable faceless bureaucrats shldnt get pass every 4yrs Holding agencies accountable is job of Congress not exec branch esp FBI Ive overseen GOP+Dem admins for 40 yrs Not going 2stop now'), ('dates', '2020-03-06 15:41:41'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.8810545206069946)])
OrderedDict([('tweet', 'New study shows drug prices increased 3.5x faster than inflation 2007-18 My bill stops drug cos charging taxpayers for increases more than inflation. For ex. last yr 1.7%. Big Pharma opposes but American ppl support Congress has to pick a side. Pres Trump supports Grassley-Wyden'), ('dates', '2020-03-05 01:17:34'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.7719102501869202)])
OrderedDict([('tweet', 'Sen Schumers threats 2day against Supreme Court Justices are wrong &amp;dangerous he shld apologize This is just another example of Schumer campaign 2politicize courts: Estrada, 2007 ACS speech &amp; Kavanaugh hearings to name a few Stop attacking independent Judiciary!'), ('dates', '2020-03-04 22:22:19'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.7724104523658752)])
OrderedDict([('tweet', 'This wk thousands attended the AIPAC policy conference I went to the cong reception &amp; had many Iowans in my office 2day to discuss grassroots support for the US-Israel relationship. Israel is the only democracy in the region + a strong US ally'), ('dates', '2020-03-03 22:56:22'), ('file_name', 'Chuck Grassley.csv'), ('pct_true', 0.5376780033111572)])
OrderedDict([('tweet', 'We must be diligent to ensure #COVID19 pandemic isn’t used to infringe on our constitutional Second Amendment rights, whether that means wrongly denying @sbagov relief for law-abiding #firearms businesses or undue background check delays. #2A Read more: https://t.co/a59ytoysPz https://t.co/EeZtBD3cZm'), ('dates', '2020-05-04 20:18:30'), ('file_name', 'Cindy Hyde-Smith.csv'), ('pct_true', 0.7407886981964111)])
OrderedDict([('tweet', 'Today @SBAgov announced officially that #PaycheckProtectionProgram is out of funds. Every Senate Republican was ready to act today, but Democrats won’t let us do more now to help workers &amp; small businesses hurt by #COVID19 shutdowns. https://t.co/uW54fxrHx7'), ('dates', '2020-04-16 20:04:06'), ('file_name', 'Cindy Hyde-Smith.csv'), ('pct_true', 0.5503829717636108)])
OrderedDict([('tweet', 'There is no good reason for Senate Democrats to choose this dire time to play politics. The American people need help, and it’s simply irresponsible to jeopardize that support. https://t.co/ewciEXBfok'), ('dates', '2020-04-09 17:52:53'), ('file_name', 'Cindy Hyde-Smith.csv'), ('pct_true', 0.7282190918922424)])
OrderedDict([('tweet', "You are either helping right now, or hurting. Pelosi's handpicked partisan ploys are helping no one. She's hurting American families by prioritizing airplane emissions, same day voter registration &amp; wind energy tax credits over #coronavirus relief. #StopTheGamesNancy #PasstheBill"), ('dates', '2020-03-24 14:37:24'), ('file_name', 'Cindy Hyde-Smith.csv'), ('pct_true', 0.985322117805481)])
OrderedDict([('tweet', 'We’ve been pushing the Governor to release Louisiana’s economy and restore individual freedoms. Today, Louisiana’s House Republican delegation again called on the Governor to reopen Louisiana. https://t.co/MWM8izgBGo'), ('dates', '2020-05-07 23:18:49'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.5733670592308044)])
OrderedDict([('tweet', '.@realDonaldTrump and the Republican-controlled Senate are both working today. However, Speaker Pelosi refuses to open the People’s House for business.'), ('dates', '2020-05-04 19:40:41'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.5386762619018555)])
OrderedDict([('tweet', "We must expand funding for the Paycheck Protection Program now. Democrats' partisan delay is hurting small businesses and millions of American families.\n\nhttps://t.co/ZA2Xxs4ZEl"), ('dates', '2020-04-17 14:56:01'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.5182987451553345)])
OrderedDict([('tweet', 'The Paycheck Protection Program has run out of funding. We tried expanding the program by $250B earlier this week, but Speaker Pelosi and Chuck Schumer blocked that effort. Small businesses and millions of American families are being injured by this delay. https://t.co/O5ryfJaYhv'), ('dates', '2020-04-16 16:00:18'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.7549981474876404)])
OrderedDict([('tweet', 'Speaker Pelosi, stop holding small businesses hostage. Expand the necessary funding now. American families are being injured by this delay. https://t.co/GMqiDuNyCQ'), ('dates', '2020-04-14 15:49:11'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.6062981486320496)])
OrderedDict([('tweet', 'Sanctuary cities should be barred from receiving federal grants if they continue to harbor criminal aliens and ignore the law. Sanctuary policies are a threat to law enforcement officers and American citizens.\n\nStand up America.\n\nhttps://t.co/p2k7uYQfdR'), ('dates', '2020-02-26 19:48:52'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.5262636542320251)])
OrderedDict([('tweet', 'The partisan impeachment has been defeated. @realDonaldTrump is acquitted. https://t.co/Fo9D6zhjJ6'), ('dates', '2020-02-05 21:32:21'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.5385189056396484)])
OrderedDict([('tweet', 'Under @realDonaldTrump, America is winning the fight against terror. No longer will we sit idle while American interests are threatened and attacked. #SOTU'), ('dates', '2020-02-05 03:10:51'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.8561179041862488)])
OrderedDict([('tweet', '.@realDonaldTrump: "As a result of our unprecedented efforts, illegal crossings are down 75 percent since May -- dropping 8 straight months in a row.  And as the wall goes up, drug seizures rise, and border crossings go down." #SOTU'), ('dates', '2020-02-05 03:05:42'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.6511136889457703)])
OrderedDict([('tweet', '.@realDonaldTrump: "Whether we are Republican, Democrat, or Independent, surely we must all agree that every human life is a sacred gift from God!" #SOTU #ProLife'), ('dates', '2020-02-05 02:55:46'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.5969648957252502)])
OrderedDict([('tweet', '.@realDonaldTrump: "Our agenda is relentlessly pro-worker, pro-family, pro-growth, and more significantly, pro-American." #SOTU https://t.co/gmAKKBi1un'), ('dates', '2020-02-05 02:21:46'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.668106198310852)])
OrderedDict([('tweet', '.@realDonaldTrump has kept his promise to redefine the WOTUS rule. The new rule rolls back federal regulatory overreach, eliminating the Obama admin’s ridiculous interpretation that puddles, ditches, tractor tire ruts, and farm ponds could be considered navigable waterways.'), ('dates', '2020-01-23 20:22:59'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.7106940150260925)])
OrderedDict([('tweet', 'USMCA was an easy win for America, and yet it was delayed by Speaker Pelosi for more than a year.  Democrats waited until #USMCA could be used to offset their one-sided, weaponized impeachment.'), ('dates', '2019-12-19 21:40:53'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.5554097294807434)])
OrderedDict([('tweet', 'Democrats have moved forward, absent of evidence and with hate in their hearts, to impeach @realDonaldTrump. The American people will see this for what it is...an attempt by one party to weaponize impeachment and overturn an election.\n\nMy full statement ⬇️ https://t.co/EB5Tl5hHSY'), ('dates', '2019-12-19 01:57:48'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.9478949904441833)])
OrderedDict([('tweet', 'This is what the Democrats fear, they fear the true will of We, the People....They fear our faith, they fear our strength, they fear our unity, they fear our vote, and they fear @realDonaldTrump. https://t.co/d3ghSRir3i'), ('dates', '2019-12-18 19:06:58'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.9878732562065125)])
OrderedDict([('tweet', 'Speaker Pelosi’s drug plan is bad policy, designed to move America closer to fully socialized government-controlled health care. The included provisions attack the free market and would lead to higher drug prices and fewer cures.'), ('dates', '2019-12-12 20:05:46'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.8587648868560791)])
OrderedDict([('tweet', 'Democrats have been charging toward impeachment since President Trump was first elected. This was an investigation in search of a crime. The outcome was predetermined long ago.'), ('dates', '2019-12-10 19:40:32'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.9769163727760315)])
OrderedDict([('tweet', "The Founding Fathers did not intend impeachment to be used as a political weapon. However, that's exactly what Democrats are doing just 11 months before the 2020 Presidential election. https://t.co/gqMF0uxGrV"), ('dates', '2019-12-04 15:25:39'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.7316256761550903)])
OrderedDict([('tweet', 'The @AmericanPress is correct. Speaker Pelosi should allow the House to vote on #USMCA. This would deliver a major win for our economy. We should not allow the #USMCA trade deal to become a victim of Democrats’ sham impeachment.\n\nhttps://t.co/1aVRUhf6lp'), ('dates', '2019-11-14 20:33:36'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.6190625429153442)])
OrderedDict([('tweet', 'The two Schiff-Pelosi screened and schooled Democrat witnesses admit that they do not possess first-hand knowledge of @realDonaldTrump’s phone call. Their testimony today... orchestrated by Schiff... is based on hearsay, presumption, and the opinions of unelected DC bureaucrats.'), ('dates', '2019-11-13 17:13:25'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.8415964841842651)])
OrderedDict([('tweet', 'House Democrats are conducting a partisan operation, disguised as “impeachment proceedings.” Selected and screened witnesses. Carefully leaked segments of testimony. No due process. Presumption of guilt.\n\nI shared my thoughts during an interview with Gray TV... https://t.co/PkvXhSOJOj'), ('dates', '2019-10-24 17:41:48'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.5617637038230896)])
OrderedDict([('tweet', 'Adam Schiff continues to mislead the American people with his secret investigation disguised as an impeachment inquiry. He should be held to account. https://t.co/bzCCJGmLnI'), ('dates', '2019-10-21 22:24:45'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.6204712986946106)])
OrderedDict([('tweet', 'Democrats are barreling toward impeachment with no facts and zero evidence to support Constitutionally impeachable behavior. Their probable cause is solely a burning hatred for President Trump.\n\nMy full statement → https://t.co/4wmXZ8dpTo'), ('dates', '2019-09-24 21:56:59'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.9553331136703491)])
OrderedDict([('tweet', 'Many presidents have campaigned on moving the US Embassy and recognizing Jerusalem as the capital of Israel, but only @realDonaldTrump followed through and kept his promise. https://t.co/rp6X9DiAVJ'), ('dates', '2019-08-14 18:29:25'), ('file_name', 'Clay Higgins.csv'), ('pct_true', 0.5435770153999329)])
OrderedDict([('tweet', 'We have a responsibility to every Vietnam era veteran who faced Agent Orange exposure. The President must take action now to ensure these veterans get the VA care they need, there is no time to waste. #PA17 @VetAffairsDems https://t.co/raDr527Rh1'), ('dates', '2020-02-13 23:18:12'), ('file_name', 'Conor Lamb.csv'), ('pct_true', 0.5403328537940979)])
OrderedDict([('tweet', 'Today, the House passed the broad election protection package SHIELD Act which included my bill the DEFEND Act. Foreign governments want to lie to the American people. Let’s keep them out of our elections and debate these things ourselves. #PA17 https://t.co/4zm7LdHgLn'), ('dates', '2019-10-24 00:19:06'), ('file_name', 'Conor Lamb.csv'), ('pct_true', 0.5711139440536499)])
OrderedDict([('tweet', 'We cannot allow even the possibility to exist that our President used the immense power of that office to protect his own selfish interests, rather than to protect the American people. #PA17 https://t.co/1mFXC22SId https://t.co/awHGlDESsi'), ('dates', '2019-09-23 20:39:29'), ('file_name', 'Conor Lamb.csv'), ('pct_true', 0.7464430928230286)])
OrderedDict([('tweet', 'No one likes robocalls. That’s why I voted for the “Stopping Bad Robocalls Act”.   Tell the Senate to stop phoning it in and pass this bill!  #PA17 https://t.co/JTSAhhLgUv'), ('dates', '2019-07-25 15:02:32'), ('file_name', 'Conor Lamb.csv'), ('pct_true', 0.5085740685462952)])
OrderedDict([('tweet', 'Under the cover of a global pandemic, Trump’s DOJ is trying to drop the charges against a top Trump associate who pled guilty multiple times in open court. This reeks of corruption. The DOJ’s IG must investigate. https://t.co/2nHkMEYmXR'), ('dates', '2020-05-07 22:52:51'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.9055713415145874)])
OrderedDict([('tweet', 'Over 2 million Americans are behind bars living under conditions that make them particularly susceptible to the spread of #COVID19. We must do more to protect them including passing my legislation that would move vulnerable people out of prison to supervision. #AnswerTheirCall https://t.co/pyahylTMXe'), ('dates', '2020-05-06 18:37:10'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.5452271699905396)])
OrderedDict([('tweet', "During a crisis that's taken nearly 70,000 American lives &amp; continues to threaten the livelihood of everyone in this country, Senate Republicans’ priority tomorrow isn’t how to stop #COVID19—it’s a hearing on a judicial nominee for an opening that won’t exist for 4 more months."), ('dates', '2020-05-05 18:59:06'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.9253220558166504)])
OrderedDict([('tweet', 'Thousands of DACA recipients in NJ are on the frontlines of this crisis saving lives. If the Trump administration follows through on its cruel threat to deport these hardworking Americans in the middle of a pandemic, our country will suffer. We must stand up &amp; protect dreamers. https://t.co/kCjMLOJyqq'), ('dates', '2020-05-03 21:29:18'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.6177424192428589)])
OrderedDict([('tweet', '#COVID19 is spreading like wildfire in our prisons and Congress hasn’t done nearly enough to stop it. We need more testing and to move vulnerable people in jail to supervision to save lives. I have legislation that does this—Congress must act. https://t.co/JNHe5Gf47x'), ('dates', '2020-05-02 18:09:14'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.6266810894012451)])
OrderedDict([('tweet', 'Amazon must ensure the safety of its employees—yet there’s a clear disconnect between what it says it’s doing to protect its workers &amp; what they face on the job every day. I won’t stop fighting for them until Amazon takes this responsibility seriously. https://t.co/xVmHxf5VOR'), ('dates', '2020-04-23 14:38:53'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.6417360901832581)])
OrderedDict([('tweet', 'It’s not enough to legalize marijuana—we must expunge arrest records &amp; repair the damage done to those communities that have been most harmed by the failed War on Drugs. My Marijuana Justice Act would do this &amp; more. The end we seek is not just legalization—it’s justice.'), ('dates', '2020-04-20 22:26:43'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.6798220276832581)])
OrderedDict([('tweet', 'Trump has repeatedly blamed his own failures on others &amp; he’s doing it again with the @WHO. Cutting off funding to the WHO right when we need international coordination to fight and beat #COVID19 only diminishes our ability to bring this pandemic under control. https://t.co/oAvmA2mXxz'), ('dates', '2020-04-16 15:48:49'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.9182502627372742)])
OrderedDict([('tweet', "In just 3 weeks, 17 million people have filed for unemployment. This doesn’t even include everyone who's been unable to file. 1 in 10 American workers is out of a job. Congress hasn't done enough to mitigate the impact of #COVID19 on lives &amp; our economy—we must get back to work. https://t.co/qikftHKwop"), ('dates', '2020-04-09 22:56:37'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.5630483627319336)])
OrderedDict([('tweet', "We're relying on our farmworkers now more than ever—yet so many are immigrants who have little to no worker protections, were left out of the emergency relief &amp; now have to deal with ICE raids. We can't leave our immigrant communities behind. https://t.co/3f3T0hQ8dv"), ('dates', '2020-04-04 16:02:30'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.7679638862609863)])
OrderedDict([('tweet', 'Loosening restrictions on public activity right now—when we’re in the midst of a public health crisis—would only get more Americans sick and add more time to our economic recovery. We can’t afford to ignore public health experts on this. https://t.co/ZPRdv4OqvS'), ('dates', '2020-03-24 00:06:24'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.5868374109268188)])
OrderedDict([('tweet', 'Any bill Congress puts forward must focus on ending the public health crisis and put people ahead of corporate bailouts—what are we doing to help workers, families, small business owners, patients, &amp; health care providers who are on the front lines of this #COVID19 pandemic?'), ('dates', '2020-03-23 21:21:41'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.8891319632530212)])
OrderedDict([('tweet', '10 years ago the #AffordableCareAct became law providing health care for over 20 million Americans &amp; protecting 135 million with preexisting conditions. Even in the #COVID19 crisis, Trump is still trying to terminate the ACA &amp; has no plan to replace it. We must keep up the fight. https://t.co/DLkygDADwi'), ('dates', '2020-03-23 17:32:50'), ('file_name', 'Cory A. Booker.csv'), ('pct_true', 0.7305542230606079)])
OrderedDict([('tweet', 'After well over a year of partisan obstruction, Chuck Schumer finally stopped blocking @POTUS’ nominee to serve on the @WHO Executive Board. I look forward to having a Senate-approved U.S. voice to advance our interests and push forward needed reforms at the @WHO.'), ('dates', '2020-05-08 21:02:29'), ('file_name', 'Cory Gardner.csv'), ('pct_true', 0.6619561910629272)])
OrderedDict([('tweet', 'The @WHO should be a reliable, independent purveyor of scientifically-based information around the globe. Its flawed handling of #COVID19 and deference to Chinese authorities who misled them warrants an independent investigation into @WHO leadership.\nhttps://t.co/40tduazxNt'), ('dates', '2020-04-29 15:24:27'), ('file_name', 'Cory Gardner.csv'), ('pct_true', 0.7630773186683655)])
OrderedDict([('tweet', "Taiwan continues to show why it's a beacon of hope for democracy &amp; for those seeking to escape the wrath of the Chinese Communist Party that cannot tolerate any dissent. Bravo, @iingwen – the US stands in solidarity with our friends in #Taiwan &amp; #HongKong.\nhttps://t.co/5UdpCMzd0k"), ('dates', '2020-04-25 19:27:05'), ('file_name', 'Cory Gardner.csv'), ('pct_true', 0.7571954727172852)])
OrderedDict([('tweet', 'Nancy Pelosi and Chuck Schumer’s obstruction of the #PaycheckProtectionProgram has cost:\n \n💰 $114 billion in aid to small businesses\n💼 Assistance for tens of thousands of small businesses\n👩\u200d💻 Millions of American jobs kept on payroll\n \nIt’s time to fund this program.'), ('dates', '2020-04-21 20:03:10'), ('file_name', 'Cory Gardner.csv'), ('pct_true', 0.8332498073577881)])
OrderedDict([('tweet', 'I condemn in the strongest terms the arrest of peaceful pro-democracy leaders in Hong Kong and call on the local authorities and their Chinese Communist Party handlers in Beijing to release them immediately.  #StandWithHongKong. #HongKongProtests\nhttps://t.co/hfw62Fncjo'), ('dates', '2020-04-18 17:06:07'), ('file_name', 'Cory Gardner.csv'), ('pct_true', 0.6034427881240845)])
OrderedDict([('tweet', 'This is why we need fundamental reform and strong US leadership at @WHO that will stand up to Chinese malign influence. Had @WHO responded to Taiwan’s inquiries in December, the world would have been better equipped to respond to #COVID19.\nhttps://t.co/7BN8Wu5LMf'), ('dates', '2020-04-14 15:40:09'), ('file_name', 'Cory Gardner.csv'), ('pct_true', 0.8024690747261047)])
OrderedDict([('tweet', 'If China is reporting honest and accurate information about #COVID19, why are people so desperate to leave Hubei province? https://t.co/q5VWGvs535'), ('dates', '2020-03-29 03:33:26'), ('file_name', 'Cory Gardner.csv'), ('pct_true', 0.9099043011665344)])
OrderedDict([('tweet', "The last thing Iowa farmers need is additional stress and uncertainty.\n\nAt @HouseAgDems this week, I asked @USDA's @SecretarySonny if he expects China to hold up its end of the 'Phase One' deal &amp; whether Iowa farmers will make gains after suffering through a prolonged trade war. https://t.co/tuo3kXB25j"), ('dates', '2020-03-05 21:15:05'), ('file_name', 'Cynthia Axne.csv'), ('pct_true', 0.5629011988639832)])
OrderedDict([('tweet', 'Speaker Pelosi &amp; Dems will say the only solution is to keep you locked up &amp; spend trillions of your taxpayer dollars on a partisan bill drafted in secret.\n\nIn truth, the solution is to reopen society &amp; get people back to work ASAP. Any politician saying otherwise is lying to you. https://t.co/YNt56l2jvD'), ('dates', '2020-05-08 14:58:39'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.5978500843048096)])
OrderedDict([('tweet', 'We must hold China accountable. Nefarious and despicable actions like these by the CCP are precisely why @SenTomCotton and I introduced legislation to make China pay. This behavior is inexcusable. https://t.co/qYNR4hNzGy'), ('dates', '2020-05-04 20:24:02'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.9838364720344543)])
OrderedDict([('tweet', '“Nothing can bring back the tens of thousands of American lives cut short by the coronavirus... Still, China should be held accountable for unleashing a global catastrophe.”\n\nGood read in @WashTimes on my bill with @SenTomCotton to hold China accountable.\nhttps://t.co/QTIQpxz0U0'), ('dates', '2020-05-03 20:56:17'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.762213945388794)])
OrderedDict([('tweet', 'Congress shouldn’t be hiding like cowards while the country is in crisis. We should be in Washington debating how to solve ongoing problems, one bill at a time.\n\nWe’re asking our essential workers to show up &amp; do their jobs. Congress is no different.\n\nTime to go back to work. https://t.co/ZFtgRylTAv'), ('dates', '2020-05-01 00:05:47'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6791156530380249)])
OrderedDict([('tweet', 'Blatant lack of leadership. The country is in crisis, and Congress is in hiding, scared.\n\nWe should be in DC debating and solving problems, not waiting every 3 weeks to spend huge sums of taxpayer money and expecting a pat on the back.\n\nCongress needs to get back to work. https://t.co/AxoHx7MKIG'), ('dates', '2020-04-28 19:55:16'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6688222885131836)])
OrderedDict([('tweet', 'Why was funding for small businesses delayed (again)? I go behind the scenes with @GOPLeader to discuss why Democrats keep playing politics with coronavirus relief.\n\nWe also asked @SpeakerPelosi to join but she was "unavailable."\nShe has an open invitation\nhttps://t.co/mJ8daILzJQ https://t.co/l5nJ6Gb1zV'), ('dates', '2020-04-24 22:22:07'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.7081446647644043)])
OrderedDict([('tweet', 'Evidence emerges each day that lays the blame for this global pandemic at the feet of the Chinese Communist Party. So how do we hold China’s government accountable? We can empower the American people to seek justice.\n\nMy latest with @SenTomCotton https://t.co/zLRV0AkrVU'), ('dates', '2020-04-24 18:28:06'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.947920024394989)])
OrderedDict([('tweet', 'The Chinese Communist Party allowed the coronavirus to spread across the world. Their lies and coverup cost countless American lives and livelihoods. We need to empower Americans to hold China accountable. https://t.co/GnCS6MCsGF'), ('dates', '2020-04-22 17:57:08'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.9836629629135132)])
OrderedDict([('tweet', 'We should have been in Washington a week ago for a non-controversial vote to replenish funding for the small business rescue program.\n\nDemocrats have purposely delayed this while millions lose jobs. \n\nThere’s no “other side” to this. This is a betrayal to America. Pure &amp; simple. https://t.co/PN8ke9QB0a'), ('dates', '2020-04-21 00:20:19'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6628919839859009)])
OrderedDict([('tweet', 'Opening our courts and allowing Americans to sue the Chinese government over COVID-19 is one way we can make the communist regime pay for their lies and coverup that allowed this pandemic to spread across the world.\n\nWATCH https://t.co/Ngsbp3SvDL'), ('dates', '2020-04-19 21:32:05'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6429831981658936)])
OrderedDict([('tweet', 'This is economically and environmentally irresponsible. It is  downright foolish. \n\nNatural gas is cleaner, and pipelines are safer.\n\nDemocrats would prefer to import dirtier Russian natural gas than use US gas.\n\nThis is a fundamentally anti-American policy proposal. https://t.co/QD5LAGmxR0'), ('dates', '2020-04-17 02:14:24'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6377679705619812)])
OrderedDict([('tweet', 'Every day more evidence emerges that places the blame for the global spread of coronavirus at the feet of President Xi and his communist regime. China’s malicious coverup and lies, parroted by far too many American journalists, cannot go without severe consequences. https://t.co/DaRtFQu3gh'), ('dates', '2020-04-14 20:18:11'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.7734744548797607)])
OrderedDict([('tweet', 'This is spot on from Dr. Gottlieb &amp; outlines facts much of the media seems to forget:\n\n- China lied &amp; tried to coverup their failure to contain coronavirus\n\n- WHO was validating the communist regime’s false claims\n\n- All of this made the global spread of this pandemic much worse https://t.co/jVNCmT7EgC'), ('dates', '2020-04-13 17:48:31'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.889121413230896)])
OrderedDict([('tweet', '“As Democrats look at this unprecedented economic crisis, they are unfortunately still channeling Rahm Emanuel’s old line that it would be a terrible thing to waste.”\n\nSpot on. Democrats have shown that “For the People” was nothing more than a slogan.\n\nhttps://t.co/oVjBymy5KV'), ('dates', '2020-04-10 14:51:25'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.7041459083557129)])
OrderedDict([('tweet', 'This is happening again. The media can cover for Democrats all they want, but the truth is clear as day: Democrats are blocking common-sense aid packages for no good reason.\n\nHold these people accountable. Call your representatives. https://t.co/aBuZkytseI'), ('dates', '2020-04-09 16:16:51'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.7824594974517822)])
OrderedDict([('tweet', 'House Democrats are obsessed with attacking Trump &amp; picking &amp; choosing which Americans get much-needed help. That’s going to be the priority of the COVID-19 “oversight” committee. Don’t let them fool you. \n\nWe should be focused on the strategy against the pandemic, not politics. https://t.co/MnApQlRIpw'), ('dates', '2020-04-04 23:38:44'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.8372977375984192)])
OrderedDict([('tweet', 'The smugness emanating from this tweet is precisely why Americans are becoming fed up with the media. \n\nThis isn’t a fact-check. This is a bad faith “told ya so!” off the backs of millions of people out of work. Just stop. https://t.co/uLmQKAP8So'), ('dates', '2020-04-04 21:56:14'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.72696453332901)])
OrderedDict([('tweet', 'New episode.\n\nI expose the bad-faith blame game played by Democrats and the media. Their disingenuous, petty finger pointing has been harmful to our ability to solve the problem. It must stop.\n\nDOWNLOAD NOW: https://t.co/MMDcsqqlA8 https://t.co/XdSzgjC18t'), ('dates', '2020-04-04 00:40:10'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6009281873703003)])
OrderedDict([('tweet', 'Sound the alarm. We cannot let this industry go under. American energy independence is at stake. Hundreds of thousands of jobs are at stake. Green New Deal fanatics have sold the American people a lie. If you care about emissions, then we must protect the American energy sector. https://t.co/6jwHoVpMyh'), ('dates', '2020-04-04 00:06:29'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6128835678100586)])
OrderedDict([('tweet', 'Shameless false reporting from @TexasMonthly.\n\nIn the last 5 days @GregAbbott_TX’s team has distributed 1.3M face masks, 165K face shields, 1.5M gloves, 155K gowns.\n\nThe media picked the worse time to abandon their primary role: report the truth. \n\nStop lying and do your job. https://t.co/EJuS22KKxX'), ('dates', '2020-04-03 14:34:09'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.5682011842727661)])
OrderedDict([('tweet', 'New episode is a must listen.\n\nChina’s malicious lies, coverup, and propaganda helped create this global pandemic. So how do we make the communist regime pay?\n\nDOWNLOAD NOW: https://t.co/rdis3UxH1n https://t.co/EKpnodeO3w'), ('dates', '2020-04-01 15:00:42'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.7747463583946228)])
OrderedDict([('tweet', 'Stop lying Joe. This isn’t true. \n\n1. We’re in the midst of a pandemic. We’ve asked people to stay home. As a result, jobs are suffering.\n\n2. Pelosi &amp; Schumer torpedoed a rescue bill to push a progressive wish list. That was more important to them than helping Americans in need. https://t.co/XAiYUgqkmY'), ('dates', '2020-03-26 22:24:21'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.9159013032913208)])
OrderedDict([('tweet', 'China’s coronavirus propaganda is just the latest lie from the communist regime. They prey on America’s divisions to spread lies &amp; distract from their coverup.\n\nWe’re in this together, and we must resist the urge to spread their talking points. My oped:\nhttps://t.co/7MVoNPENaA'), ('dates', '2020-03-25 21:56:28'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.977121889591217)])
OrderedDict([('tweet', 'In my limited time in Washington, I’ve never seen anything as shameless and cruel as what Democrats have done in the past 48 hours. \n\nThey’ve held American families, businesses, and patients hostage to fulfill a progressive wish list. \n\nFor the people? No. For the politics. https://t.co/QGqJQ1B11s'), ('dates', '2020-03-24 15:58:44'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.7989672422409058)])
OrderedDict([('tweet', 'Call the office of every Democrat in Congress. \n\nDo not let them get away with this. \n\nThey are holding America hostage for TOTALLY unrelated progressive policies, and lying to you about it. https://t.co/KBXckZk721'), ('dates', '2020-03-23 17:43:13'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.5370824933052063)])
OrderedDict([('tweet', 'This is the $500B provision Dems claim has “no oversight.”\n\nDirect Lending Limitations: executive total compensation may not exceed $425,000; prohibition of stock buybacks during the duration of the loan; borrowers must maintain existing payroll as of March 13.\n\nDems are lying.'), ('dates', '2020-03-23 15:47:37'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.8691188097000122)])
OrderedDict([('tweet', 'Democrats just tanked bipartisan negotiations on a bill that:\n\nProvides immediate relief to businesses large and small\n\nPuts cash in the hands of Americans in need\n\nBolsters unemployment benefits for people out of work\n\nWhy? Gross partisanship. We need action now. Call your reps https://t.co/MNsypN0aAa'), ('dates', '2020-03-23 00:57:09'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.7882585525512695)])
OrderedDict([('tweet', 'China hid the virus. Now they’re lying about it. This is a MUST download for how the Chinese Communist Party operates. https://t.co/9I08YzBWLK https://t.co/TTgJDhVF73'), ('dates', '2020-03-21 03:09:41'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.9774993062019348)])
OrderedDict([('tweet', 'The leading Dem candidate for president supports a bill that would destroy the gig economy. Young Americans want to sell their time &amp; labor on their own terms, &amp; the left wants to take away that right.\n\nAnother reason younger generations should walk away from the Democratic Party https://t.co/2dRXCRjBng'), ('dates', '2020-03-10 19:46:28'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.8331783413887024)])
OrderedDict([('tweet', 'The far left has hijacked the climate debate with crazed plans like the Green New Deal.\n\nBut it doesn’t have to be this way. We have fact-based ideas, like my #NewEnergyFrontier, to reduce emissions without destroying the economy. Read my latest in @NRO.\nhttps://t.co/fOEUypXv98'), ('dates', '2020-03-03 19:21:18'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.5078856945037842)])
OrderedDict([('tweet', '“By putting government in charge of every health care decision, Medicare for All would eliminate the adaptability of private innovation, which is an American advantage.”\n\nExactly. Medicare for All &amp; price controls would crush the innovation we are relying on to tackle coronavirus https://t.co/RiOFUaW82i'), ('dates', '2020-03-03 15:21:39'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.5981187224388123)])
OrderedDict([('tweet', 'There are Democrats outright lying about the President’s response to coronavirus. Bloomberg is actually running ads that say CDC funding was cut - &amp; it’s patently false.\n\nThere is no need to politicize this important issue, but they can’t help themselves. \nhttps://t.co/LGBEGButJ2'), ('dates', '2020-02-27 17:11:38'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.9577793478965759)])
OrderedDict([('tweet', 'While the Iranian regime targets their own people, their proxies in Gaza are busy targeting Israelis. Not the Israeli military, but women, children, the elderly, and anyone within range.\n\nAnd yet, Israel will continue to be condemned by the UN, the media, and members of Congress. https://t.co/7uSA5AE0Pp'), ('dates', '2020-02-24 23:55:31'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.8251013159751892)])
OrderedDict([('tweet', 'Exactly. If they wanted to get his point of view, have an informed journalist interview him. But don’t let this murderous terrorist have free reign to spread his propaganda in the pages of the New York Times. https://t.co/lW9bh104cL'), ('dates', '2020-02-22 00:59:36'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.5246614217758179)])
OrderedDict([('tweet', 'The question is: will this money be spent researching new carbon capture tech that shows promise? Fundamental science for CO2 use? Advanced nuclear? Trillion trees?\n\nOr wasted on the same ideological activism that results in slogans but no solutions?\n\nHoping Bezos figures it out. https://t.co/H9nIPzJl94'), ('dates', '2020-02-18 14:37:04'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.5024821162223816)])
OrderedDict([('tweet', 'Stop what you’re doing &amp; download my latest podcast w/ @KSoltisAnderson for a discussion on Millennial voters. We talk about her book “The Selfie Vote,” why social media is shaping Millennial politics, &amp; bringing young people into the conservative movement\nhttps://t.co/M5IoGEVWFb'), ('dates', '2020-02-17 16:17:44'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6268321871757507)])
OrderedDict([('tweet', 'We’ve been tracking shipments like this for a long time. We seize them when possible.\n\nYet 55 Senators just voted to limit the president’s power to take necessary actions against Iran, including these low-risk ops.\n\nCongress must consider 2nd &amp; 3rd order effects of its actions. https://t.co/2KRyHOCgDZ'), ('dates', '2020-02-15 17:41:29'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6918587684631348)])
OrderedDict([('tweet', 'Bernie Sanders never proposes solutions to climate change. He uses virtue signaling &amp; fear to cloak a socialist takeover of the entire economy. \n\nWe need real solutions.\n\n#NewEnergyFrontier is grounded in proven science about what works to reduce emissions without killing jobs. https://t.co/gpGfrEB90y'), ('dates', '2020-02-13 18:49:17'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6404035687446594)])
OrderedDict([('tweet', 'President Trump’s budget got a lot of criticism today from some Democrats who REFUSE to actually propose their own budget. This is hypocrisy at its finest.\n\nWhat’s worse, is the moral grandstanding and shaming that occurs every year with respect to spending. https://t.co/2pwFh9Z6SW'), ('dates', '2020-02-13 00:15:07'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6480656266212463)])
OrderedDict([('tweet', 'China’s cyber attacks on U.S. consumers, businesses, &amp; government are only going to ramp up.\n\nMy bill that passed the House today - HR 4753 - ensures that DHS isn’t  purchasing Chinese drone equipment that could give China back door access to our data. https://t.co/HS6a6FU9UW'), ('dates', '2020-02-11 00:17:10'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.9605697989463806)])
OrderedDict([('tweet', 'This acquittal is justified. Democrats rushed a partisan impeachment through the House to meet a political timeline, and failed to prove their weak case. It’s over.\n\nTime to move on and get back to work on the issues President Trump addressed in his State of the Union.'), ('dates', '2020-02-05 22:28:28'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.9705173969268799)])
OrderedDict([('tweet', 'Good news, but we can’t let up in the opioid crisis fight.\n\nFentanyl killed more than 30,000 Americans in one year &amp; is a scourge in Houston. That’s why I voted for a bill this week to give DEA tools to go after the criminals trafficking this deadly drug on our streets. https://t.co/O8P2OFdmqT'), ('dates', '2020-01-31 16:04:39'), ('file_name', 'Dan Crenshaw.csv'), ('pct_true', 0.6980359554290771)])
OrderedDict([('tweet', 'Unfortunately, scammers are using the COVID-19 pandemic to try to steal Medicare Numbers, personal information, and money. \n\nLearn how to protect yourself or your loved ones from Medicare fraud: https://t.co/cXrKJRFVDs'), ('dates', '2020-05-07 01:21:00'), ('file_name', 'Dan Newhouse.csv'), ('pct_true', 0.5608803629875183)])
OrderedDict([('tweet', 'Since the outbreak of #COVID19 and the Saudi-Russian price war, U.S. energy producers have been devastated by a simultaneous supply &amp; demand shock. Across the country energy workers are being laid off, rigs are shutting down and future investments are being drastically cut. [1/5]'), ('dates', '2020-05-09 00:02:43'), ('file_name', 'Dan Sullivan.csv'), ('pct_true', 0.6335957646369934)])
OrderedDict([('tweet', 'As small businesses suffer &amp; PPP $ to help has run dry, @SenSchumer spends his time character-assassinating an indigenous Alaska woman of impeccable integrity—as well as Alaska Native orgs &amp; people. Careful Chuck: Elite animus for Alaska Native people on full display. https://t.co/6HsuG4g1mg'), ('dates', '2020-04-16 19:57:41'), ('file_name', 'Dan Sullivan.csv'), ('pct_true', 0.8151967525482178)])
OrderedDict([('tweet', 'As the COVID-19 crisis goes on, most of Congress has been left on the sidelines. Still, the federal government has spent $2.5 trillion with more to follow. This is unacceptable in our democracy. If businesses can find a way to operate remotely, so can we. https://t.co/HBniRHMtd6'), ('dates', '2020-05-06 00:05:07'), ('file_name', 'Daniel Lipinski.csv'), ('pct_true', 0.5138922929763794)])
OrderedDict([('tweet', '105 years later, let us continue to remember those 1.5 million individuals who lost their lives in the Armenian Genocide and renew our resolve to prevent future mass atrocities. #ArmenianGenocide'), ('dates', '2020-04-24 23:23:02'), ('file_name', 'Daniel Lipinski.csv'), ('pct_true', 0.7481075525283813)])
OrderedDict([('tweet', 'A complex health system means bad actors can charge huge bills &amp; halt efforts to cut costs. That’s why I introduced the Transparency &amp; Accountability in #HealthCare Costs &amp; Prices Act to bring to light hidden data needed for price transparency &amp; other health reform tools #IL03 \U0001fa7a'), ('dates', '2020-03-03 01:08:53'), ('file_name', 'Daniel Lipinski.csv'), ('pct_true', 0.5463977456092834)])
OrderedDict([('tweet', "It's been 1 year since the House passed #HR8. 90% of U.S. supports background checks, including vast majorities of Republicans, Democrats, &amp; gun owners. On the heels of the tragedy in Milwaukee, it's time for the Senate to do the right thing &amp; vote on this bill #GunSenseMajority"), ('dates', '2020-02-27 21:27:38'), ('file_name', 'Daniel Lipinski.csv'), ('pct_true', 0.7054487466812134)])
OrderedDict([('tweet', '. @narprail is awarding Rep. Lipinski w/ its prestigious Golden Spike Award for his 15 yrs of work ensuring safe relationships between railroads &amp; the communities in which they operate. Previous winners include politicians, Walt Disney &amp; Johnny Cash! https://t.co/1XHopr9X6M #IL03'), ('dates', '2020-02-20 15:06:54'), ('file_name', 'Daniel Lipinski.csv'), ('pct_true', 0.7161726355552673)])
OrderedDict([('tweet', 'We may have a deal on PPP but Nancy Pelosi and Chuck Schumer delaying this caused existential disruption and unnecessary pain on thousands of businesses and tens of thousands of families.'), ('dates', '2020-04-20 00:29:26'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.9699060916900635)])
OrderedDict([('tweet', '(1/4) Even on Easter Sunday, the mainstream media was trying to convince the public that @realDonaldTrump is to blame for #COVID19. The facts tell a different story. On 1/23, the @who said #COVID19 was "not a global health threat."'), ('dates', '2020-04-13 16:26:18'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.743391215801239)])
OrderedDict([('tweet', '(1/2) This morning, Senate Democrats blocked an effort to increase funding for the Paycheck Protection Program. @stevenmnuchin1 has said that supplemental funding is absolutely essential -- Senate Democrats saying otherwise is not only damaging, it’s a dereliction of duty.'), ('dates', '2020-04-09 18:43:59'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.9331017136573792)])
OrderedDict([('tweet', '(1/2) On Tuesday, Senate Democrats voted down the Born Alive Abortion Survivors Protection Act.'), ('dates', '2020-02-27 14:36:40'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.6513652801513672)])
OrderedDict([('tweet', 'Impeachment rocked the people’s trust. Democrats’ political agenda consumed the resources of all three branches of government at the expense of countless other priorities. Let’s hope they now work to rebuild a semblance of trust &amp; bridge troubled waters. \n\nhttps://t.co/qWLynnxqC6'), ('dates', '2020-02-12 19:30:00'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.8168718218803406)])
OrderedDict([('tweet', 'A Budget for America’s Future doesn’t saddle future generations with debt. I introduced the BASIC Act to provide a full outlook on how new federal spending impacts national debt. This is commonsense. I am glad to see this approach in @realDonaldTrump’s budget request.'), ('dates', '2020-02-12 16:03:00'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.6227281093597412)])
OrderedDict([('tweet', "Since Day 1, Democrats obsessed over doing one thing: impeaching @realDonaldTrump. They consumed every branch of government's time for a political ploy.\n\nToday's acquittal shows how weak and political their case has been from the start.\n\nRead my statement: https://t.co/WDYlOjjsbi"), ('dates', '2020-02-05 21:51:50'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.7922201156616211)])
OrderedDict([('tweet', 'The 4 Democrat Senators running for President should #recuse themselves from voting today. They’re voting to remove an opponent from the race and get an advantage in an election.'), ('dates', '2020-02-05 16:11:00'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.7446396946907043)])
OrderedDict([('tweet', 'While Democrats obsess over impeaching him, @realDonaldTrump delivers results for the American people. \n\nImagine if all of the energy that went into impeaching the President had gone toward working with him to solve problems. https://t.co/0urseP8ER9'), ('dates', '2020-01-29 23:14:35'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.5819291472434998)])
OrderedDict([('tweet', 'The President’s peace proposal is a step toward ending tensions between Israel &amp; Palestine. Peace requires compromise but Israel can’t compromise its security. @realDonaldTrump and @netanyahu created an agreement that balances security and peaceful coexistence.'), ('dates', '2020-01-28 23:58:16'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.8720561265945435)])
OrderedDict([('tweet', 'The President’s Defense Team began their presentation today. It’s about time the American people get the facts in this case. There was no quid pro quo, aid was released, and Trump and Zelensky had their meeting. It’s about the facts and it’s time to move on.'), ('dates', '2020-01-25 15:53:20'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.567133367061615)])
OrderedDict([('tweet', 'As House Democrats present their case to the Senate, you won’t hear:\n \n1. Zelensky repeatedly said he felt no pressure &amp; there was no quid pro quo;\n2. Aid was released without investigations into Biden, Burisma, or 2016;\n3. Any firsthand evidence corroborating Democrats’ claims. https://t.co/0WnIrReCjL'), ('dates', '2020-01-23 18:44:41'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.8833632469177246)])
OrderedDict([('tweet', 'President Trump has the truth on his side &amp; selected an excellent legal team to present the facts. The Senate will conduct a fair trial and @realDonaldTrump will be acquitted based on the facts. Democrats failed to produce any evidence in House proceedings &amp; now want a do-over.'), ('dates', '2020-01-21 23:06:54'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.8927383422851562)])
OrderedDict([('tweet', '6 of the 7 Democrats presenting the case against President Trump voted to impeach him before the Ukraine call even happened. This was never about facts, it was always about overturning the result of the 2016 election and removing a duly elected President of the United States.'), ('dates', '2020-01-18 15:30:00'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.8712068200111389)])
OrderedDict([('tweet', "28 days after voting to impeach the President, Democrats formally appointed impeachment managers. In delaying this procedural step, Speaker Pelosi has shown her true motivation: trying to prevent the President's re-election.\n\nRead my full statement here:\n\nhttps://t.co/JYm2Z5hRGA"), ('dates', '2020-01-15 18:38:12'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.5203123092651367)])
OrderedDict([('tweet', '(1/3) After decades of unfair trade that only got worse in the last 10 years, President Trump kept his promise to improve our trade deals. @RealDonaldTrump took us from a trade war to a great trade deal with China.'), ('dates', '2020-01-15 16:44:39'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.6156452298164368)])
OrderedDict([('tweet', "Energy independence means national security. Because of @RealDonaldTrump's pro-domestic energy policies, Iran has lost its leverage over the United States. 4-5 years ago, Middle East turmoil would have spiked gas prices for American families. https://t.co/1OLcqTe5Sp"), ('dates', '2020-01-08 17:07:44'), ('file_name', 'Daniel Meuser.csv'), ('pct_true', 0.8084081411361694)])
OrderedDict([('tweet', "Mitch McConnell has no problem giving big corporations TRILLIONS of dollars in tax cuts, but wants to let states hit hard by the #coronavirus, like Michigan, go bankrupt.\n\n@senatemajldr's comments are insulting and show just how out of touch Republicans are with reality. https://t.co/pHmbjQeb2b"), ('dates', '2020-04-23 18:59:47'), ('file_name', 'Daniel T. Kildee.csv'), ('pct_true', 0.6363162398338318)])
OrderedDict([('tweet', "Who wants to tell the President that COVID-19 isn't on Twitter? https://t.co/lHu5Qz8dW1"), ('dates', '2020-04-17 15:50:46'), ('file_name', 'Daniel T. Kildee.csv'), ('pct_true', 0.5605884790420532)])
OrderedDict([('tweet', 'We can’t fully fight the #coronavirus worldwide if we are not collaborating with others. This virus knows no borders. The President’s decision to halt funding to a global health organization, @WHO, is dangerous &amp; will only make stopping the spread of this pandemic more difficult. https://t.co/M2QrF2QBGr'), ('dates', '2020-04-15 19:30:45'), ('file_name', 'Daniel T. Kildee.csv'), ('pct_true', 0.9151051640510559)])
OrderedDict([('tweet', 'We should not be delaying economic relief for struggling American workers so that @RealDonaldTrump can affix his signature to checks going out. This reeks of political posturing and narcissism. https://t.co/fC5YwgIwHJ'), ('dates', '2020-04-15 12:46:33'), ('file_name', 'Daniel T. Kildee.csv'), ('pct_true', 0.5244117975234985)])
OrderedDict([('tweet', 'Michiganders are just sick of the President’s finger-pointing and cheap political shots. People are DYING. Every American should want him to succeed in this moment of crisis. We need all leaders, especially POTUS, to politics aside and just work to solve the problem at hand.'), ('dates', '2020-03-27 23:07:12'), ('file_name', 'Daniel T. Kildee.csv'), ('pct_true', 0.8119592666625977)])
OrderedDict([('tweet', 'Gov. Whitmer is doing everything in her power to slow the spread. Instead of attacking the virus, @realDonaldTrump is focused on scoring petty political points. Michigan—with the 5th most cases in the U.S.—needs more masks, gloves &amp; ventilators from the federal government. NOW. https://t.co/OfYemQpGaT'), ('dates', '2020-03-27 22:58:16'), ('file_name', 'Daniel T. Kildee.csv'), ('pct_true', 0.6602346301078796)])
OrderedDict([('tweet', 'Last week, Republicans proposed a straightforward, clean bill that would add more money for these loan programs, so small businesses could keep people employed. I hope my Democratic colleagues will stop blocking funding.  Small businesses need help now. https://t.co/1L6EuNFafy'), ('dates', '2020-04-16 21:11:00'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.640484631061554)])
OrderedDict([('tweet', 'Incredibly frustrating that Democrats are choosing to push their partisan liberal agenda over providing relief to hurting, hardworking Americans. https://t.co/bqef6O3OP4'), ('dates', '2020-03-24 22:58:06'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.9321367144584656)])
OrderedDict([('tweet', 'I agree. Florida has 3 confirmed cases already. Democrats need to stop playing politics and allow a vote on a clean funding bill. https://t.co/JGoKp51yja'), ('dates', '2020-03-04 16:35:16'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.680687427520752)])
OrderedDict([('tweet', 'Blinded by their hatred of President @realDonaldTrump, a majority of House Democrats ignored the facts and the role of the judicial branch in Article III of the U.S. Constitution.  https://t.co/kjGaa124lk'), ('dates', '2019-12-19 16:30:00'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.6409354209899902)])
OrderedDict([('tweet', 'None of the witnesses or evidence confirmed Democrats’ allegations. \n\nThe articles of impeachment ignore facts and the role of the judicial branch. \n\nThis impeachment and today’s vote is not about facts or the constitution. \nhttps://t.co/W2kDE1rXvs'), ('dates', '2019-12-18 23:27:59'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.5949342250823975)])
OrderedDict([('tweet', 'This resolution is merely one more attempt to deceive Americans by claiming it provides transparency and due-process. In reality, this sham resolution does nothing to change the Democrats’ soviet-style impeachment process.  Full statement: https://t.co/VE49p38QFO'), ('dates', '2019-10-31 15:48:04'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.8077709078788757)])
OrderedDict([('tweet', 'Even the liberal Washington Post says Democrats should bring up and pass this trade deal. #USMCAnow @USTradeRep @realDonaldTrump https://t.co/lIfb3dGauR'), ('dates', '2019-09-12 13:03:59'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.5675644874572754)])
OrderedDict([('tweet', "For months Republicans have asked Democrats to hold a hearing &amp; vote on bill to protect babies born alive after an abortion. Despite 70+% support, they've refused. Today, Republicans are holding that hearing. Watch live: https://t.co/eFRV0lXIxF \n\n#EndInfanticide #LetUsVote https://t.co/dEtIlcthGm"), ('dates', '2019-09-10 13:19:42'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.8092505931854248)])
OrderedDict([('tweet', 'Today, Democrats once again chose to play politics over passing legislation that would invest in and support our Armed Services. In doing so they ended a nearly 50-year record of the #NDAA being developed and passed with bipartisan support.\nhttps://t.co/LCWnF7zWm1 https://t.co/iGI8J5bB4z'), ('dates', '2019-07-12 18:27:28'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.7811871767044067)])
OrderedDict([('tweet', "Democrats' proposal is not a serious fix, it’s an act of political lip-service to a very serious situation &amp; children in need of real help. If Democrats will join us &amp; cease playing politics, we can pass and send President Trump a clean, commonsense bill. https://t.co/hXVpQmBpL5"), ('dates', '2019-06-27 15:45:14'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.9001548886299133)])
OrderedDict([('tweet', "30 years ago today, brave Chinese students and civilians lost their lives at the hands of their brutal government. Despite China's attempt to erase this day and silence the voices of freedom, we remember them. We must not tolerate censorship and human rights abuses. #Tiananmen30 https://t.co/NmVIAjW5B4"), ('dates', '2019-06-04 13:59:55'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.9609875082969666)])
OrderedDict([('tweet', 'Incredible news in today’s jobs report. New data confirms that under President @realDonaldTrump’s pro-growth policies, hardworking Americans are benefit from a more prosperous economy with incredible job creation! We have the lowest unemployment rate in 49 years.'), ('dates', '2019-05-03 17:29:24'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.5056769847869873)])
OrderedDict([('tweet', "More than 8 million Floridians would lose their current insurance plans. Americans deserve better than the Democrats' one-size-fits-all, government-run health care system that will inevitably require rationing of care, higher taxes and an overall reduction in quality. https://t.co/jbsJJq720v"), ('dates', '2019-04-30 21:27:01'), ('file_name', 'Daniel Webster.csv'), ('pct_true', 0.7230328321456909)])
OrderedDict([('tweet', "Conservatives believe in welfare programs...don't let them tell you otherwise ... https://t.co/Uh46UDkNVo"), ('dates', '2020-05-03 23:45:41'), ('file_name', 'Danny K. Davis.csv'), ('pct_true', 0.651481032371521)])
OrderedDict([('tweet', "#COVID19 didn't create IL’s pension crisis &amp; taxpayers shouldn’t be on the hook to bail out decades of fiscal mismanagement. IL needs real reform &amp; Congress should focus on helping everyday Americans.\n\nMy conversation with @guypbenson on @GuyBensonShow  ⬇️\nhttps://t.co/KXPqVtl5LK"), ('dates', '2020-05-05 01:04:57'), ('file_name', 'Darin LaHood.csv'), ('pct_true', 0.5478428602218628)])
OrderedDict([('tweet', 'Decades of bad policy caused Illinois’ pension crisis, not #COVIDー19. I will continue to support funding for Illinois to combat this virus &amp; recover, but Congress shouldn’t bail out Illinois’ fiscal mismanagement. We should be helping everyday Americans.\nhttps://t.co/VEu3XT7VLu'), ('dates', '2020-04-21 19:12:44'), ('file_name', 'Darin LaHood.csv'), ('pct_true', 0.8407162427902222)])
OrderedDict([('tweet', 'Time for Senate Democrats to put politics aside &amp; support American workers. So many citizens who work in small businesses like hair salons, restaurants, as self employed painters, landscapers &amp; others are facing economic hardship. They need help. Act now. #SupportAmericanWorkers'), ('dates', '2020-03-24 12:47:20'), ('file_name', 'Darin LaHood.csv'), ('pct_true', 0.9029422402381897)])
OrderedDict([('tweet', 'Reckless.\n\nTwice in the last 24 hours, Senate Democrats voted to block the bipartisan coronavirus relief bill that will save jobs and give money to families in need. There is a health and economic crisis in our country. People need help. Save the politics for another day.'), ('dates', '2020-03-23 22:22:30'), ('file_name', 'Darin LaHood.csv'), ('pct_true', 0.9469165802001953)])
OrderedDict([('tweet', 'The pro-growth agenda championed by @WaysandMeansGOP, \n@HouseGOP &amp; @POTUS is lifting folks off the sidelines. Since enacting #TCJA:\n\n-Unemployment hit 50-year low\n-7 mil+ jobs created\n-Wages rising\n-7 mil+ people moved off food stamps\n\nWATCH my comments with @stevenmnuchin1 ⬇️ https://t.co/YMyjocLYRA'), ('dates', '2020-03-03 17:49:24'), ('file_name', 'Darin LaHood.csv'), ('pct_true', 0.5627691745758057)])
OrderedDict([('tweet', 'In the midst of a pandemic, @ProbSolveCaucus proves Republicans &amp; Democrats can come together to find new &amp; innovative ways to legislate while maintaining public health guidelines. While technologically imperfect, I’m proud of  our bipartisan efforts! https://t.co/0zh5ZbT3j1'), ('dates', '2020-05-08 15:06:50'), ('file_name', 'Darren Soto.csv'), ('pct_true', 0.5489162802696228)])
OrderedDict([('tweet', "A global pandemic has turned students' lives upside-down. US Dept of Education should be spending their time ensuring students have the support they need to succeed in school, not rolling back the rights of survivors. @usedgov https://t.co/yibN6h0DT4"), ('dates', '2020-05-07 14:59:42'), ('file_name', 'Darren Soto.csv'), ('pct_true', 0.5762996673583984)])
OrderedDict([('tweet', 'Pres Trump’s demand for payroll tax cut is rebuffed by Congress because it would put #SocialSecurity Trust Fund in jeopardy and mostly help those who already have jobs! #COVID19 https://t.co/8Vxm8OQDF5'), ('dates', '2020-05-06 13:26:18'), ('file_name', 'Darren Soto.csv'), ('pct_true', 0.7958251237869263)])
OrderedDict([('tweet', 'This is a statewide disgrace. Now 166,000 families can’t pay their bills, after enduring weeks of uncertainty. Many may still be eligible for federal benefits and were wrongly denied. We will fight this together. Constituents can call us at (407) 452-1171 https://t.co/88HUV1HJ8T'), ('dates', '2020-04-28 14:22:58'), ('file_name', 'Darren Soto.csv'), ('pct_true', 0.7125501036643982)])
OrderedDict([('tweet', 'Pres Trump refused \u2066@WHO\u2069 #coronavirus test and tasked @CDCgov\u2069 to make our own under his America First policy. Those CDC tests became contaminated and America lost weeks in our fight against the pandemic. https://t.co/0H3B7FTvxD'), ('dates', '2020-04-18 18:31:17'), ('file_name', 'Darren Soto.csv'), ('pct_true', 0.6852889657020569)])
OrderedDict([('tweet', 'Venezuela is undergoing a major human rights crisis which will be dramatically exacerbated by #COVID19. The Senate stalling on designating temporary protective status for Venezuelans in America is unacceptable. We need #VenezuelaTPSNow! https://t.co/cYhFf9IdBJ'), ('dates', '2020-04-15 16:37:32'), ('file_name', 'Darren Soto.csv'), ('pct_true', 0.5742345452308655)])
OrderedDict([('tweet', 'As predicted the @SBAgov ran out of funding for #PPP. This situation could have been avoided. There are other priorities that will need to be addressed but right now this is the urgent need. People’s livelihoods are at stake. Congress must put political games aside &amp; ACT NOW! https://t.co/svfUSu5G4p'), ('dates', '2020-04-16 15:59:21'), ('file_name', 'David B. McKinley.csv'), ('pct_true', 0.6967372298240662)])
OrderedDict([('tweet', '62+ million people benefit from the ACA’s birth control coverage. \n\nYet, today, #SCOTUS is hearing arguments on employer’s power to decide whether or not to cover birth control for employees. \n\nBirth control is BASIC healthcare.'), ('dates', '2020-05-06 15:32:00'), ('file_name', 'David E. Price.csv'), ('pct_true', 0.7258844971656799)])
OrderedDict([('tweet', 'Americans deserve quality health insurance now more than ever. Despite years of systematic, politicized attempts by Republicans to undermine the ACA &amp; take away insurance, the federal government has a LEGAL obligation to follow through on its commitments to insurers. https://t.co/ceMU131FCM'), ('dates', '2020-04-27 20:45:00'), ('file_name', 'David E. Price.csv'), ('pct_true', 0.8361005187034607)])
OrderedDict([('tweet', 'PPP ran out. EIDL ran out. Small businesses deserve better, but @senatemajldr is playing politics with their health &amp; their livelihood.\n\n@housedemocrats have a plan:\n\n- $250bil for small biz\n-$100 bil for community health centers\n-$150bil for state &amp; local govts.'), ('dates', '2020-04-16 21:01:00'), ('file_name', 'David E. Price.csv'), ('pct_true', 0.783105194568634)])
OrderedDict([('tweet', "Today, I joined my @HouseForeign colleagues for a briefing on COVID-19 global trends. We covered important topics including why we won't immediately see a vaccine and how U.S. leadership is pivotal during this global crisis. Global health = health and economic security at home."), ('dates', '2020-04-21 22:05:06'), ('file_name', 'David J. Trone.csv'), ('pct_true', 0.5311465859413147)])
OrderedDict([('tweet', ".@realDonaldTrump &amp; the Senate are working for the American people in Washington, while @SpeakerPelosi  won't let the House return.\n\nCongress should be working to:\n- get our economy running\n- help small businesses\n- assist those on the front lines\n\nWe can't allow any more delay!"), ('dates', '2020-05-05 17:51:52'), ('file_name', 'David Kustoff.csv'), ('pct_true', 0.8236875534057617)])
OrderedDict([('tweet', "I voted to replenish the #PPP, giving small businesses means to get back on their feet &amp; keep employees on payroll. While @SpeakerPelosi &amp; @SenSchumer's delay of this funding hurt Americans, I’m grateful this relief is on the way to those who need it. \nhttps://t.co/mD5KWLThQQ"), ('dates', '2020-04-23 23:01:38'), ('file_name', 'David Kustoff.csv'), ('pct_true', 0.6663397550582886)])
OrderedDict([('tweet', "The #PPP has been incredibly successful for small businesses across our nation. It's disappointing Democrats waited so long to replenish this program's funding. I will be voting YES this week to replenish this program's vital funding &amp; give our businesses much needed support. https://t.co/2oR4gfI5ld"), ('dates', '2020-04-22 15:52:18'), ('file_name', 'David Kustoff.csv'), ('pct_true', 0.517730712890625)])
OrderedDict([('tweet', "COVID-19 has hurt TN businesses, families, &amp; healthcare workers. That is why, along w/ my colleagues in the TN delegation, I sent a letter to @realDonaldTrump asking him to approve @GovBillLee's request for additional federal assistance to combat the effects of the #coronavirus. https://t.co/ag9GJYYi5J"), ('dates', '2020-03-31 14:51:36'), ('file_name', 'David Kustoff.csv'), ('pct_true', 0.576249897480011)])
OrderedDict([('tweet', 'Speaker Pelosi’s actions at the #SOTU were disgraceful and deeply disrespectful to the American heroes honored in @realDonaldTrump’s address. That is why I supported @RepKayGranger’s resolution to condemn the Speaker’s shameful behavior. Her action’s must be held accountable.'), ('dates', '2020-02-06 19:40:18'), ('file_name', 'David Kustoff.csv'), ('pct_true', 0.6437825560569763)])
OrderedDict([('tweet', 'I commend the Senate for acquitting @realDonaldTrump  today and putting an end to this partisan endeavor. The fact is, President Trump did nothing wrong, and today’s acquittal proves just that.'), ('dates', '2020-02-05 21:38:56'), ('file_name', 'David Kustoff.csv'), ('pct_true', 0.5593520998954773)])
OrderedDict([('tweet', 'We can’t leave our front-line fighters without the supplies they need to save American lives. The Medical Supply Chain Emergency Act will force the President to federalize American manufacturing &amp; end the supply shortage. We need the #DefenseProductionActNOW. https://t.co/bVCZJ2heMd'), ('dates', '2020-03-27 18:47:46'), ('file_name', 'David Loebsack.csv'), ('pct_true', 0.5203908681869507)])
OrderedDict([('tweet', 'Earlier this year, the 10th Circuit Court of Appeals ruled that @EPA had exceeded its authority in issuing SREs to oil companies who had not received waivers in the prior year. Today, I urged @EPAAWheeler to apply that decision nationally. #RFSworks https://t.co/dTJbVIYUeI'), ('dates', '2020-02-27 22:40:12'), ('file_name', 'David Loebsack.csv'), ('pct_true', 0.6216311454772949)])
OrderedDict([('tweet', 'Ten years ago, #CitizensUnited opened the floodgates to unlimited spending in our elections, giving undue influence to a handful of wealthy individuals and corporations. We must return power to the people by passing #HR1 and overturning the decision.  #ForThePeople https://t.co/CUlJjRVo5t'), ('dates', '2020-01-21 18:40:10'), ('file_name', 'David Loebsack.csv'), ('pct_true', 0.7961214184761047)])
OrderedDict([('tweet', 'Nothing in our democracy is more sacred than the right to vote. That’s why I voted to #RestoreTheVote and stop the most egregious voter suppression tactics before they go into place. The House just passed #HR4, the Senate must act NOW!'), ('dates', '2019-12-06 18:00:49'), ('file_name', 'David Loebsack.csv'), ('pct_true', 0.7847843766212463)])
OrderedDict([('tweet', '“It is outrageous that not even two weeks after the President and @EPA announced a ‘deal’ to offset the excessive number of SREs they granted to big, profitable oil companies, they fall back on their word." https://t.co/KlDpF8A2Bd'), ('dates', '2019-10-15 22:26:08'), ('file_name', 'David Loebsack.csv'), ('pct_true', 0.5932545065879822)])
OrderedDict([('tweet', 'Pres Trump’s decision to side w oil producers over Iowa’s farmers is another stab in the back to those working to feed &amp; fuel our nation. From his Twitter trade war to approving “hardship” waivers, when will his assault on the livelihood of Iowans end? https://t.co/VkkdR4raYw'), ('dates', '2019-08-16 18:05:09'), ('file_name', 'David Loebsack.csv'), ('pct_true', 0.6358015537261963)])
OrderedDict([('tweet', 'Israel has always been a strong democratic ally of America. Its decision to deny entry to 2 American congresswomen is a sign of weakness and is clearly harmful to the US-Israel relationship. Instead of barring persons with opposing views of (1/2)'), ('dates', '2019-08-15 20:04:32'), ('file_name', 'David Loebsack.csv'), ('pct_true', 0.5083935260772705)])
OrderedDict([('tweet', 'Yes, the President’s tweets were racist and I will join my colleagues in supporting the House resolution condemning his statements. https://t.co/b5wXhbKk3n'), ('dates', '2019-07-15 20:56:29'), ('file_name', 'David Loebsack.csv'), ('pct_true', 0.7194189429283142)])
OrderedDict([('tweet', 'Did you know that right now many servicemembers who are carrying debt are subject to unfair collection practices? \n\nEven worse: debt collectors are allowed to threaten our troops with punishment through the Uniform Code of Military Justice.'), ('dates', '2020-03-03 17:08:11'), ('file_name', 'David N. Cicilline.csv'), ('pct_true', 0.6310351490974426)])
OrderedDict([('tweet', 'Not only is @RealDonaldTrump’s request for emergency coronavirus funding woefully insufficient to protect Americans, he has also spent the past 2 years dismantling the federal agencies responsible for handling the coronavirus epidemic.⬇️\nhttps://t.co/sTVGvXRj7K'), ('dates', '2020-02-26 22:07:32'), ('file_name', 'David N. Cicilline.csv'), ('pct_true', 0.5028784871101379)])
OrderedDict([('tweet', 'Two weeks ago, @RealDonaldTrump brazenly broke his promise not to cut Social Security or Medicare. \n\nThe #TrumpBudget cuts:\n⛔$850 billion from Medicare\n⛔$30 million from Social Security  \n\n@HouseDemocrats will continue to fight for robust funding for these vital programs. https://t.co/XOzuk3upeU'), ('dates', '2020-02-25 22:39:49'), ('file_name', 'David N. Cicilline.csv'), ('pct_true', 0.9024519920349121)])
OrderedDict([('tweet', 'On #YomHaShoah we remember the millions of Jewish men, women &amp; children murdered during the Holocaust.\n\nToday &amp; everyday we must reaffirm our promise of #NeverAgain &amp; condemn anti-Semitism in the US &amp; across the globe. Together, we are stronger than hate. #HolocaustRemembranceDay https://t.co/V5VN3Y7ALa'), ('dates', '2020-04-22 00:29:39'), ('file_name', 'David P. Joyce.csv'), ('pct_true', 0.5522003769874573)])
OrderedDict([('tweet', "The #PaycheckProtectionProgram is now out of money. Congress' inability to work across party lines has forsaken #SmallBusinesses trying to pay their workers amid the #COVID19 crisis which has already cost 22 million pple their jobs.\n\nThat's unacceptable. Americans deserve better."), ('dates', '2020-04-16 16:17:02'), ('file_name', 'David P. Joyce.csv'), ('pct_true', 0.8396292924880981)])
OrderedDict([('tweet', 'The Democrats are literally holding American workers’ salaries hostage. 16 MILLION filed for unemployment in the last 3 weeks, but they think it’s an appropriate time to stall funding for businesses to re-hire employees. Shameful.'), ('dates', '2020-04-09 22:44:58'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.9837623834609985)])
OrderedDict([('tweet', "The measures required to defeat this virus are unlike anything we've ever experienced. Right now, people are scared, confused and frustrated as our economy comes to a standstill and widespread job losses are occurring. (1/3)"), ('dates', '2020-03-24 15:51:43'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.6715468764305115)])
OrderedDict([('tweet', 'There’s no such thing as a Republican or Democrat disease, so we can’t allow our response to coronavirus to be politicized. @realDonaldTrump and Congress will provide whatever resources are needed to protect the American people from this illness, period.'), ('dates', '2020-02-28 18:50:44'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.8538369536399841)])
OrderedDict([('tweet', "There's no question Iranian Gen. Qasem Soleimani was a bad actor and a danger to the U.S. as well as our allies. His plots threatened countless American lives. I applaud @realDonaldTrump and our servicemembers for this successful mission, making America safer as a result."), ('dates', '2020-01-03 18:49:24'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.6621838808059692)])
OrderedDict([('tweet', 'Impeaching @realDonaldTrump with no evidence is dangerous &amp; will create a new normal that impeachment is acceptable because two branches of government disagree. I hope my Democrat colleagues will consider the consequences of impeaching a duly-elected president based on politics. https://t.co/Q7c4dTOWIF'), ('dates', '2019-12-18 14:33:55'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.9360842704772949)])
OrderedDict([('tweet', 'Just finished reading the articles of impeachment against @realDonaldTrump\n \nMy conclusion? House Democrats have no facts and no proof, only a political agenda. https://t.co/GN2b4kGcWc'), ('dates', '2019-12-10 17:48:13'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.7033665776252747)])
OrderedDict([('tweet', 'House Democrats are so desperate, they had to bring in constitutional “experts” to try to fool the public into supporting their sham impeachment inquiry of @realDonaldTrump. (1/2)'), ('dates', '2019-12-04 23:03:05'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.749614953994751)])
OrderedDict([('tweet', '203,651 people voted for @realdonaldtrump in TN’s First District. That is 203,651 Tennessee voices House Democrats are trying to silence during this impeachment sham. This is wrong and I will continue to speak up on behalf of my constituents. #TNSpeaksUp https://t.co/Oc5iGbRT3L'), ('dates', '2019-11-20 14:39:13'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.9247276782989502)])
OrderedDict([('tweet', 'For months, Democrats have been saying President Trump is guilty of a ‘quid pro quo.’ Turns out a ‘quid pro quo’ isn’t actually a crime, so now Democrats are accusing the president of ‘bribery.’ Give me a break. This is a sham. https://t.co/33YwalOezC'), ('dates', '2019-11-15 21:41:44'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.9647601842880249)])
OrderedDict([('tweet', 'With Democrats’ public impeachment hearings starting tomorrow, it’s worth noting: holding public hearings in which witnesses are handpicked to help the Democrats’ narrative is not fair or transparent. This is a coordinated effort to remove a president by his political opponents.'), ('dates', '2019-11-12 22:44:06'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.9617824554443359)])
OrderedDict([('tweet', "In my 10+ years on HVAC, I've never seen this committee be conducted in such a partisan fashion. Dems put politics over veterans when they closed debate on an amendment to prohibit VA from paying child care to facilities that employ a registered sex-offender, among others. (1/2)"), ('dates', '2019-10-29 22:07:13'), ('file_name', 'David P. Roe.csv'), ('pct_true', 0.6549679040908813)])
OrderedDict([('tweet', 'American taxpayers shouldn’t be on the hook for China’s predatory lending practices. \n\n15 senators joined me in demanding transparency from China’s Belt &amp; Road Initiative as the U.S. leads efforts to stabilize the developing world. https://t.co/3GtXSXEhwW'), ('dates', '2020-05-04 23:30:48'), ('file_name', 'David Perdue.csv'), ('pct_true', 0.7978450059890747)])
OrderedDict([('tweet', "“Without a revenue stream, small businesses are just not going to make it. And when small businesses don't make it, a community doesn't thrive.” \n– Dara Buczynsky, Athens, GA\n\nThis is why replenishing the #PaycheckProtectionProgram is so critical. https://t.co/4owfQdrsl3"), ('dates', '2020-04-17 13:05:59'), ('file_name', 'David Perdue.csv'), ('pct_true', 0.5013847947120667)])
OrderedDict([('tweet', 'President Trump’s agenda created the greatest economic turnaround in U.S. history. However, the #COVID19 crisis and resulting shutdown of much of our country has taken a serious toll on our economy. (1/2)'), ('dates', '2020-04-16 17:02:26'), ('file_name', 'David Perdue.csv'), ('pct_true', 0.5450227856636047)])
OrderedDict([('tweet', 'America is facing an economic crisis.\n\nOur medical community is battling the worst viral outbreak in recent history.\n\nBusinesses are closing. \n\nJobs are being lost.\n\nBut Senate Democrats are choosing to use this moment as political leverage. Absolutely pathetic.'), ('dates', '2020-03-22 23:46:32'), ('file_name', 'David Perdue.csv'), ('pct_true', 0.9419811964035034)])
OrderedDict([('tweet', 'For over two decades, US forces have helped keep the peace between Kosovo &amp; Serbia. Now, with historic progress in sight, Kosovo must do its part &amp; abolish all duties imposed on Serbia. \n\nIf Kosovo is not fully committed to peace, then the US should reconsider its presence there.'), ('dates', '2020-03-10 02:07:58'), ('file_name', 'David Perdue.csv'), ('pct_true', 0.7382147908210754)])
OrderedDict([('tweet', 'This is low, even for Chuck Schumer.\n \nThreatening two sitting Supreme Court justices is not only disgraceful, it is dangerous. I hope the Senate will stand together and condemn this unacceptable behavior. https://t.co/aNBsxPXOg3'), ('dates', '2020-03-05 19:35:09'), ('file_name', 'David Perdue.csv'), ('pct_true', 0.5652456879615784)])
OrderedDict([('tweet', 'The Paycheck Protection Program has been a historic success, putting $350 billion into small businesses’ pockets. Speaker Pelosi and Senator Schumer need to set aside their demands and allow the $250 billion in funding to pass so every small business &amp; its employees can be helped'), ('dates', '2020-04-16 19:54:30'), ('file_name', 'David Rouzer.csv'), ('pct_true', 0.7944700121879578)])
OrderedDict([('tweet', 'This is no time for Pelosi and Schumer to hold a good bill hostage.  Senate bill is result of bi-partisan input.  Good for workers, small businesses &amp; health care professionals on the front lines!  Vote today!'), ('dates', '2020-03-23 13:47:31'), ('file_name', 'David Rouzer.csv'), ('pct_true', 0.9387161731719971)])
OrderedDict([('tweet', "#OTD 75 years ago, the Allied Forces accepted Nazi Germany's surrender, marking the end of #WWII. This #VEDay, we honor every veteran from the Greatest Generation who fought for freedom and thank them for their sacrifice. https://t.co/5gsNYReywd"), ('dates', '2020-05-08 16:12:17'), ('file_name', 'David Schweikert.csv'), ('pct_true', 0.6145903468132019)])
OrderedDict([('tweet', 'We could have avoided this problem last week. \n\nSmall business owners are still applying for the Paycheck Protection Program. My Democratic colleagues must stop blocking this critical funding and allow Congress to add additional resources for the #PPPloans. https://t.co/6oWWMr8wcJ'), ('dates', '2020-04-16 14:56:01'), ('file_name', 'David Schweikert.csv'), ('pct_true', 0.7885702252388)])
OrderedDict([('tweet', 'Tomorrow, China will institute another round of tariffs on American goods. This is not what winning a trade war looks like. Farmers in Georgia need the administration to figure out a way to get a long-term deal done because their livelihoods depend on it. #AgNeedsADeal https://t.co/PlsHomxWjI'), ('dates', '2019-05-31 16:57:45'), ('file_name', 'David Scott.csv'), ('pct_true', 0.787155270576477)])
OrderedDict([('tweet', "Without transparent oversight, businesses which need relief the least may continue to get $$ at the expense of those which need it most. That's not the spirit of our bipartisan legislation and I won't rest until EVERY MN small business has an equal shot. \nhttps://t.co/YO0xAZpWhv https://t.co/noNry3gBXG"), ('dates', '2020-04-27 17:33:11'), ('file_name', 'Dean Phillips.csv'), ('pct_true', 0.5164647698402405)])
OrderedDict([('tweet', 'THREAD: It’s reasonable and necessary to investigate if the Communist Party of China is guilty of lying to its own citizens and the world. But one thing is certain; a free press changes government decision calculus because they always uncover the truth. And truth saves lives. 1/2'), ('dates', '2020-04-19 15:51:51'), ('file_name', 'Dean Phillips.csv'), ('pct_true', 0.9662205576896667)])
OrderedDict([('tweet', 'The President: "...the things they had in there were crazy. Levels of voting that if you ever agreed to it you\'d never have a Republican elected in this country again."\n\nThe Country: Whether we live in a city, suburb or rural area, we wish to vote easily, safely and securely.'), ('dates', '2020-03-30 16:42:51'), ('file_name', 'Dean Phillips.csv'), ('pct_true', 0.6144551634788513)])
OrderedDict([('tweet', '#COVID19 is a national crisis which is why the Senate unanimously passed and @realdonaldtrump signed into law the CARES Act. \n\xa0\nRead in this week’s column how the bill will provide relief for Nebraska families, businesses, ag producers and hospitals. https://t.co/CLJ2RwIkPM'), ('dates', '2020-03-30 15:52:57'), ('file_name', 'Deb Fischer.csv'), ('pct_true', 0.515363335609436)])
OrderedDict([('tweet', "This is the right decision by the @EPA.\n \nI led a bipartisan group of senators urging the EPA to not appeal this decision which struck down harmful small refinery exemptions. This is a victory for #NE's economy, and our farmers and ethanol producers. https://t.co/f3v7E6hcP9"), ('dates', '2020-03-25 14:50:55'), ('file_name', 'Deb Fischer.csv'), ('pct_true', 0.8640215992927551)])
OrderedDict([('tweet', 'It’s troubling that other countries plan on deploying 5G using untrustworthy equipment that could give Beijing leverage. This is why I helped introduce the bipartisan Network Security Trade Act to ensure network security remains a top priority in our trade negotiations.'), ('dates', '2020-03-05 17:25:47'), ('file_name', 'Deb Fischer.csv'), ('pct_true', 0.556816041469574)])
OrderedDict([('tweet', "Coronavirus has raised serious concerns for workers safety.\n\nWe can't ask workers to make the impossible choice between missing work and potentially losing their job or working in an unsafe environment.\n\nWe need workplace guidelines that put workers first now. #WhereisOSHA"), ('dates', '2020-04-28 17:49:00'), ('file_name', 'Debbie Dingell.csv'), ('pct_true', 0.5626033544540405)])
OrderedDict([('tweet', "These are not normal times &amp; taking shots at the state &amp; local gov'ts, which are actually the fighters on the ground, isn't helpful to anybody. They don’t want to see partisan fighting, they want to see us continue to fight to keep them safe and to get the economy reopened again. https://t.co/o0UaEohFDS"), ('dates', '2020-04-28 16:37:13'), ('file_name', 'Debbie Dingell.csv'), ('pct_true', 0.6512209177017212)])
OrderedDict([('tweet', "There's no us verse them. There’s no room for name calling and partisan attacks. We need comfort and unity. Remember, it's all of us against this invisible enemy."), ('dates', '2020-04-26 15:43:19'), ('file_name', 'Debbie Dingell.csv'), ('pct_true', 0.5300962924957275)])
OrderedDict([('tweet', "During times of crisis, our nation turns to local media for life-saving information. But as local media helps us understand this current crisis, they too are suffering enormous losses in revenue. We're calling on the Trump Admin to direct Fed Gov't advertising funds to local news https://t.co/HIeiNtE5bj"), ('dates', '2020-04-20 18:13:49'), ('file_name', 'Debbie Dingell.csv'), ('pct_true', 0.7863672375679016)])
OrderedDict([('tweet', 'The coronavirus disrupted educational programs and left students without access to housing, meal plans, or classroom space. Under the CARES Act which was passed by Congress, colleges and universities in our district will receive more than $66 million.'), ('dates', '2020-04-10 21:19:09'), ('file_name', 'Debbie Dingell.csv'), ('pct_true', 0.525075376033783)])
OrderedDict([('tweet', 'The accelerated spread of coronavirus means Congress must provide additional support for community hospitals &amp; healthcare systems. They need more PPE &amp; tests. House Democrats are calling for an additional $100 billion to make this happen. Frontline workers need this help now.'), ('dates', '2020-04-08 17:34:29'), ('file_name', 'Debbie Dingell.csv'), ('pct_true', 0.55552738904953)])
OrderedDict([('tweet', "Right now we're seeing the effects of decades of policies that incentivized offshoring at the cost of domestic manufacturing. Our supply chains are so dependent on other countries that we don't have the ability to respond rapidly in a time of crisis."), ('dates', '2020-04-08 12:57:55'), ('file_name', 'Debbie Dingell.csv'), ('pct_true', 0.6407337784767151)])
OrderedDict([('tweet', "This week marks 1yr since the House reauthorized the Violence Against Women Act which the Senate refused to reauthorize. Now with stay at home orders, victims are forced to stay inside with their abusers. It shouldn't have gotten to this, the Senate should have acted a year ago."), ('dates', '2020-04-07 17:05:32'), ('file_name', 'Debbie Dingell.csv'), ('pct_true', 0.6903136372566223)])
OrderedDict([('tweet', '(1/2) The more we learn, the clearer the picture becomes—Democrats, top Obama officials, and apparently even President Obama himself, were going to stop at nothing to ambush @realDonaldTrump.'), ('dates', '2020-05-09 18:26:00'), ('file_name', 'Debbie Lesko.csv'), ('pct_true', 0.5949416160583496)])
OrderedDict([('tweet', 'The Democrats are at it again—taking any chance they can get to grab power and attack @realdonaldtrump. During this crisis, we should be focused on keeping the American people safe and getting them access to the resources they need, not playing political games. https://t.co/ccf6Lclsx1'), ('dates', '2020-04-22 23:45:09'), ('file_name', 'Debbie Lesko.csv'), ('pct_true', 0.6341696977615356)])
OrderedDict([('tweet', "I’m disappointed that @SenSchumer and @SpeakerPelosi just killed a bipartisan agreement to help Americans. All 47 Senate Democrats just voted no. Stop the partisan politics and let's help America! https://t.co/ez03CV0Oyv"), ('dates', '2020-03-23 01:42:39'), ('file_name', 'Debbie Lesko.csv'), ('pct_true', 0.8554291725158691)])
OrderedDict([('tweet', 'Why is Mitch McConnell convening Senate and put Members, staff, Capitol Police, and service workers at risk at the height of a pandemic?\n\nIt’s not to confront COVID-19. It’s to confirm yet another unqualified, partisan judge. \n\nShameless. Our constituents need to pay rent.'), ('dates', '2020-05-08 13:19:09'), ('file_name', 'Debbie Mucarsel-Powell.csv'), ('pct_true', 0.750114917755127)])
OrderedDict([('tweet', "This is shocking.\n\nGen. Flynn pleaded guilty to lying to the FBI...TWICE.\n\nWe need answers now from AG Barr about the DOJ's independence and his ability to serve the public.\n\nTrump must not be allowed to abuse the rule of law and disregard justice for the American people. https://t.co/N6r6wnlsW6"), ('dates', '2020-05-07 19:06:24'), ('file_name', 'Debbie Mucarsel-Powell.csv'), ('pct_true', 0.73882657289505)])
OrderedDict([('tweet', "Floridians are facing record unemployment through no fault of their own.\n\nThis #COVID19 crisis has exposed Florida’s maliciously designed unemployment system that's set up to deny benefits to workers.\n\n@GovRonDeSantis, how about you set up a step-by-step plan to fix THIS issue. https://t.co/l2YwMZP0rg"), ('dates', '2020-04-30 14:16:58'), ('file_name', 'Debbie Mucarsel-Powell.csv'), ('pct_true', 0.7385463118553162)])
OrderedDict([('tweet', 'When the nation needed him to invoke the DPA to produce PPE to save lives, the President refused.\n\nNow, he’s using it to put American lives at risk.\n\nWe need a president who values our workers &amp; fights for hazard pay. Not one who has shown from day 1 that lives are disposable. https://t.co/xYLvdSHgVk'), ('dates', '2020-04-29 13:07:30'), ('file_name', 'Debbie Mucarsel-Powell.csv'), ('pct_true', 0.7000733613967896)])
OrderedDict([('tweet', 'Since @GovRonDeSantis has blindly followed the President\'s "leadership" during this crisis, I\'m just going to get ahead of this one -\n\nFloridians, please don\'t inject yourselves with bleach or lysol.'), ('dates', '2020-04-24 16:34:45'), ('file_name', 'Debbie Mucarsel-Powell.csv'), ('pct_true', 0.5649133324623108)])
OrderedDict([('tweet', "South Florida is hurting.\n\nMy constituents call me in tears because they don't know how they will make payments or keep their businesses afloat.\n\nTo make matters worse, over a million Floridians can't access unemployment funds.\n\nThere's still work to do to help our communities. https://t.co/3cXSpcIX2M"), ('dates', '2020-04-23 20:15:26'), ('file_name', 'Debbie Mucarsel-Powell.csv'), ('pct_true', 0.5898603200912476)])
OrderedDict([('tweet', 'Weeks ago, I urged @GovRonDeSantis to fix the unemployment system &amp; start processing these applications.\n\nHundreds of Floridians are now left to fend for themselves due to an archaic system designed to keep unemployment claims low.\n\nThis is unacceptable. People need help now! https://t.co/I6TZgUp1UL'), ('dates', '2020-04-18 14:24:57'), ('file_name', 'Debbie Mucarsel-Powell.csv'), ('pct_true', 0.5105070471763611)])
OrderedDict([('tweet', "The #COVID19 pandemic has made the humanitarian, political &amp; economic crisis in Venezuela even more devastating.\n\nYet Venezuelans still don't have protection in the US.\n\nThe Senate must support the bipartisan bill passed by the House and pass #VenezuelaTPSNow."), ('dates', '2020-04-15 22:16:57'), ('file_name', 'Debbie Mucarsel-Powell.csv'), ('pct_true', 0.8536938428878784)])
OrderedDict([('tweet', 'The State of Florida trying to manipulate #COVID19 data is beyond concerning\n\nIn this crisis, glossing over the truth is a grave disservice that could lead to prematurely opening up the state &amp; squandering away our shared sacrifices\n\nLives are on the line.\nhttps://t.co/q1SnYmujSH'), ('dates', '2020-04-15 18:22:56'), ('file_name', 'Debbie Mucarsel-Powell.csv'), ('pct_true', 0.6127669215202332)])
OrderedDict([('tweet', 'POTUS is shamefully trying to shift blame to @WHO for his own inaction early in this pandemic. \n\nLives are on the line. \n\nNow is the time for principled leadership, not petty scapegoating.\nhttps://t.co/uuF0fv13MK'), ('dates', '2020-04-14 23:31:18'), ('file_name', 'Debbie Mucarsel-Powell.csv'), ('pct_true', 0.7217959761619568)])
OrderedDict([('tweet', "The Trump Administration's borrower defense rule puts the interests of predatory for-profit colleges over those of hardworking MI students. I voted to put #StudentsFirst &amp; I urge Sec. DeVos to provide thousands of defrauded MI student loan borrowers the relief that they deserve."), ('dates', '2020-03-11 19:57:02'), ('file_name', 'Debbie Stabenow.csv'), ('pct_true', 0.7469753623008728)])
OrderedDict([('tweet', "#HR1, the For the People Act, makes it easier for Americans to vote and limits the influence of dark money from special interests in our elections.\n\nThat's why Leader McConnell has kept #HR1 in his Legislative Graveyard for over a year now. It's time he work #ForThePeople. https://t.co/4N9ieNw9L3"), ('dates', '2020-03-10 23:54:53'), ('file_name', 'Debbie Stabenow.csv'), ('pct_true', 0.514177143573761)])
OrderedDict([('tweet', 'One year ago, the House passed #HR8, the Bipartisan Background Checks Act. But for an entire year, Mitch McConnell has blocked action on this commonsense solution to reduce gun violence &amp; save lives. It’s time for the Senate to act! https://t.co/cSSCrIrSOD'), ('dates', '2020-02-27 19:03:20'), ('file_name', 'Debbie Stabenow.csv'), ('pct_true', 0.7603984475135803)])
OrderedDict([('tweet', 'This is dangerous and irresponsible. Our Michigan National Guard needs this funding to protect our northern border, stop cyber security attacks, and train our men and women in uniform. https://t.co/Sp2INf6dfj'), ('dates', '2020-02-13 19:38:39'), ('file_name', 'Debbie Stabenow.csv'), ('pct_true', 0.6565488576889038)])
OrderedDict([('tweet', "The President promised to take care of the Great Lakes, but included no funding to move forward with the Brandon Road project, a critical choke point to stop Asian carp from entering the Lakes. \n \nThe #TrumpBudget fails our MI test for what's important.\nhttps://t.co/XzQ2CjAMIg"), ('dates', '2020-02-12 21:15:28'), ('file_name', 'Debbie Stabenow.csv'), ('pct_true', 0.5059675574302673)])
OrderedDict([('tweet', 'A fair trial requires all relevant witnesses and evidence. If President Trump’s staff has evidence of his innocence, he would have insisted that we hear from them, as we should. Senate Republicans have once again voted to hide the truth about the President’s actions.'), ('dates', '2020-01-31 22:59:17'), ('file_name', 'Debbie Stabenow.csv'), ('pct_true', 0.5536262392997742)])
OrderedDict([('tweet', 'This isn’t surprising. President Trump betrayed workers and families with his giant tax giveaway, blew a giant hole in the deficit, and tried gutting Medicare to pay for it.\n\nhttps://t.co/8RNlDoMVfw'), ('dates', '2020-01-24 03:47:19'), ('file_name', 'Debbie Stabenow.csv'), ('pct_true', 0.7871708869934082)])
OrderedDict([('tweet', 'Yesterday @HHSGov admitted they had no National Testing Strategy. That leads to states fending for themselves, corp. governance and wasteful supply competition &amp; shortages. \n\nWhy have a President or fed govt if neither will lead in a public health crisis?\n\nhttps://t.co/p1sdCtKqrT'), ('dates', '2020-05-02 21:12:21'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.665614664554596)])
OrderedDict([('tweet', 'Florida’s unemployment numbers are staggering, and our GOP-designed-to-fail unemployment system is collapsing.\n \nDeSantis can’t keep ignoring the 2 million out-of-work Floridians who must now live on the state’s basement-dwelling jobless benefits.\n \nhttps://t.co/mBq65jqvXe'), ('dates', '2020-04-30 14:41:19'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.6932587027549744)])
OrderedDict([('tweet', ".@realDonaldTrump never fails to scapegoat immigrants. Now he's using them to distract from his disastrous #coronavirus response. It will not heal our economy either. Ramping up testing &amp; protecting health care workers will truly keep Americans safe.\nhttps://t.co/6KaBGmHM8O"), ('dates', '2020-04-21 15:12:54'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.5455822348594666)])
OrderedDict([('tweet', "A decade after #DeepwaterHorizon dumped ~5 million barrels of oil into the Gulf, Trump still wants to expand drilling. Florida's environment, economy &amp; livelihood can't tolerate another disaster. We must shift to clean energy now to fight climate change. #ProtectOurCoasts https://t.co/kagMD2Uewf"), ('dates', '2020-04-20 19:45:51'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.672742486000061)])
OrderedDict([('tweet', "Maduro's regime inflicted brutal economic &amp; humanitarian pain on the people of #Venezuela - all before a pandemic made it worse.\n\nVZs in America need Temporary Protected Status more than ever. \n\nThe Senate must support the House's bipartisan bill and pass #VenezuelaTPSNow."), ('dates', '2020-04-16 01:03:24'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.5333390831947327)])
OrderedDict([('tweet', "People are hurting, and need help now. \n\nFor Trump to hold up checks to people who can't pay their bills – all to stroke his own ego – is immoral and obscene.\n\nAs always, it’s Trump first, and Americans last. https://t.co/ls0aJTnAW1"), ('dates', '2020-04-15 20:17:52'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.8450998067855835)])
OrderedDict([('tweet', "As @jkbjournalist reports, our judicial system betrayed #Epstein's victims again. This appalling decision ignores the #CVRA requirement that prosecutors notify and treat victims fairly. His co-conspirators' best ally: our legal system. My bill will prevent such future injustices. https://t.co/Yc3O5gdVdI"), ('dates', '2020-04-14 18:27:21'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.518657922744751)])
OrderedDict([('tweet', 'The Nicolas Maduro regime has betrayed the people of #Venezuela, and these narco-terrorists should face justice. \n\nhttps://t.co/sRGK9vi0wk'), ('dates', '2020-03-26 15:36:51'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.641975998878479)])
OrderedDict([('tweet', 'Amid this crisis, we cannot allow a @GOP lawsuit to rip healthcare away from millions. We are asking the President to do what’s right: drop his lawsuit against the #ACA &amp; work with us to get all Americans the care they need. #ProtectOurCare. https://t.co/Nkyd5AumrO'), ('dates', '2020-03-23 23:22:09'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.6103695631027222)])
OrderedDict([('tweet', 'At the #SOTU @realDonaldTrump promised support for the people of #Venezuela. Instead, he locks them up or banishes them to Mexico when they come here seeking refuge. Once again, Trump has chosen xenophobia over compassion. https://t.co/GCFigbiUu0'), ('dates', '2020-03-13 00:54:49'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.6707960963249207)])
OrderedDict([('tweet', "The court holds that MPP likely violates the Immigration &amp; Nationality Act &amp; UN Convention on Refugees. MPP has been a human rights disaster from the start. I’ve seen firsthand the pain it’s brought to migrant families. I'm relieved to see the courts halt it again. #DontLookAway https://t.co/xbsGVeEu0x"), ('dates', '2020-02-28 18:41:12'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.6954926252365112)])
OrderedDict([('tweet', 'A child that never starts vaping stays off a perilous path to nicotine addiction. Democrats will take action when this Republican Administration caves to Big Tobacco. https://t.co/iWWe7KDCZ1'), ('dates', '2020-02-28 13:40:44'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.6277469992637634)])
OrderedDict([('tweet', "Instead of working w/ the Tohono O’odham Nation, Trump officials desecrated their sacred lands to build a feckless border wall on this nat'l monument.\n\nI told @CBPMarkMorgan: stop assaulting tribal sovereignty &amp; enviro laws. \n\nThis wall will cost us more than money. #TalkIsCheap https://t.co/N9jm11lBMU"), ('dates', '2020-02-27 21:13:50'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.6178519129753113)])
OrderedDict([('tweet', 'One year ago, @HouseDemocrats took action to #EndGunViolence. Experts and Americans agree that the bipartisan background checks legislation passed by the House would help save lives, but Mitch McConnell has let it sit in the Senate without a vote. This inaction is unacceptable. https://t.co/geCBJNSLPW'), ('dates', '2020-02-27 19:34:03'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.6944774985313416)])
OrderedDict([('tweet', 'The vast majority of Americans oppose the GOP Admin’s theft of military funds to build an ineffective border wall. Trump has stolen billions from our servicemembers &amp; their families to fund this boondoggle and still won’t have his wall when he leaves office next January. https://t.co/pOeNsPfElz'), ('dates', '2020-02-26 19:25:00'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.8112377524375916)])
OrderedDict([('tweet', 'The Senate is supposed to be the world’s greatest deliberative body. But that reputation has crumbled under @SenateMajLdr McConnell’s reign of obstruction. \n\nWanting the Senate to live up to the vision outlined by our founders is not a partisan issue. \nhttps://t.co/W93SIE6brW'), ('dates', '2020-02-25 23:17:56'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.5046483278274536)])
OrderedDict([('tweet', 'Trump is mad. Not at Russia, but at Congress for finding out that the Kremlin is interfering in the 2020 campaign.\n\nCongressional leaders need blunt intelligence assessments, and Americans need a President who protects our elections. https://t.co/rZyTUlMmUa'), ('dates', '2020-02-20 23:44:07'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.6564250588417053)])
OrderedDict([('tweet', "It's been 13 days since Trump affirmed America's support for #Venezuela at the SOTU, but still no announcement of TPS for Venezuelans. We must ensure those in the U.S. who fled Maduro's brutal regime are not at risk of deportation."), ('dates', '2020-02-17 17:06:24'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.9473535418510437)])
OrderedDict([('tweet', 'The @realDonaldTrump admin wants to slash Medicare, Medicaid, and Social Security while eliminating protections for pre-existing conditions. Why are they bent on making more Americans uninsured? https://t.co/ZJShS8PBTB'), ('dates', '2020-02-15 15:31:28'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.9411552548408508)])
OrderedDict([('tweet', 'I strongly condemn the UN blacklisting of companies working in the West Bank. Singling out Israel furthers the discriminatory BDS movement a bipartisan House majority opposed. Rather than incite boycotts the UN should promote direct bilateral negotiations to achieve lasting peace'), ('dates', '2020-02-13 04:56:57'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.911062479019165)])
OrderedDict([('tweet', "Telling the truth to protect America's national security is what this President means by 'very insubordinate.'\n\nOne of these men is loyal to his nation and the Constitution.\n\nThe other wants to silence the truth and hide his misdeeds. https://t.co/TBKkjcnB2O"), ('dates', '2020-02-08 20:27:15'), ('file_name', 'Debbie Wasserman Schultz.csv'), ('pct_true', 0.787203311920166)])
OrderedDict([('tweet', 'For decades Indian Country has been left behind creating connectivity barriers that lead to elders not having access to telehealth, children not being able to login for lessons &amp; workers unable apply for unemployment during the #coronavirus pandemic. https://t.co/QbhUu3hmeo'), ('dates', '2020-05-08 19:41:19'), ('file_name', 'Debra A. Haaland.csv'), ('pct_true', 0.6789694428443909)])
OrderedDict([('tweet', '‘What a mess’ is right… small businesses are relying on these disaster loans to keep folks on the payrolls and survive this pandemic. Today is a new day – I’m urging the SBA to update their system so our communities can access the funding they need.\nhttps://t.co/BjPFBrtrgZ'), ('dates', '2020-04-28 14:17:13'), ('file_name', 'Debra A. Haaland.csv'), ('pct_true', 0.6776171922683716)])
OrderedDict([('tweet', 'We heard from credit unions in NM who were kept out of the disaster loan process limiting access to small biz relief. Credit unions have stepped up &amp; the revisions I fought for in this most recent bill will help keep workers on payrolls &amp; small biz afloat. https://t.co/s7mpBKuD0z'), ('dates', '2020-04-27 05:31:12'), ('file_name', 'Debra A. Haaland.csv'), ('pct_true', 0.5628240704536438)])
OrderedDict([('tweet', 'Our communities are relying on important funding to keep families safe and healthy, but Senate Republicans blocked important funding for our state, local, and tribal governments in this stimulus package. #coronavirus https://t.co/btHMlzl7oo'), ('dates', '2020-04-23 17:12:57'), ('file_name', 'Debra A. Haaland.csv'), ('pct_true', 0.6028804183006287)])
OrderedDict([('tweet', 'Today, we honor the 6 million lives lost to deep hatred &amp; bigotry. As we live in a time of great fear &amp; uncertainty, we must continue combatting anti-Semitism &amp; never forget the Holocaust. #NeverAgain #WeRemember #USHMM #PreventGenocide @HolocaustMuseum  https://t.co/MpJtkv9Tyj'), ('dates', '2020-04-21 23:01:29'), ('file_name', 'Debra A. Haaland.csv'), ('pct_true', 0.5272349119186401)])
OrderedDict([('tweet', "#OTD 10 years ago, the largest offshore oil spill in world history dumped 5 million barrels of oil into the Gulf.\n\n11 people died.\n \nThis can't happen again, but Trump's greedy agenda puts communities at risk. @NRDems are putting #PeopleOverPoluters. #BP10\nhttps://t.co/c6h7BzKxLt"), ('dates', '2020-04-20 19:01:58'), ('file_name', 'Debra A. Haaland.csv'), ('pct_true', 0.6767305135726929)])
OrderedDict([('tweet', 'Nearly 17 million Americans have filed for unemployment in the last three weeks. Congress has passed significantly expanded unemployment assistance. But we need to make sure these benefits are reaching the people that need them as quickly as possible. https://t.co/gnoVNQE4HY'), ('dates', '2020-04-10 18:09:16'), ('file_name', 'Denny Heck.csv'), ('pct_true', 0.6187196969985962)])
OrderedDict([('tweet', '.@RepMcGovern, House Democrats walked away from the opportunity to form a bipartisan task force. Take it from a threat expert and warfighter, China abuses human rights, lied about #COVID-19, and is a geopolitical and military threat. https://t.co/OhIyk90YdW'), ('dates', '2020-05-07 20:33:03'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.7603331208229065)])
OrderedDict([('tweet', 'It is an honor to be named to the China Task Force by @GOPLeader. Communist China has absolutely lied about #COVID_19 and their deceitful actions have cost American lives. I will work with @POTUS to hold China accountable. Read more here: https://t.co/BFqma90dfJ'), ('dates', '2020-05-07 19:56:03'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.7434090375900269)])
OrderedDict([('tweet', "It's an honor to be named to the China Task Force by @GOPLeader. Communist China has absolutely lied about #COVID19 and their deceitful actions have cost American lives. I'll work with @POTUS to hold China accountable. They need to be taken to task for their dangerous behavior."), ('dates', '2020-05-07 16:48:47'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.7802107930183411)])
OrderedDict([('tweet', "I'm cosponsoring a resolution to condemn the United Nations for their decision to place China on the Council on Human Rights as they continue to cover up the spread of #COVID_19 and abuse the Uighur community.  Read more here: https://t.co/JxhcAdp451"), ('dates', '2020-05-06 21:02:00'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.6089661717414856)])
OrderedDict([('tweet', '.@GOPLeader is right, we need to protect the health of American citizens. That is why I am cosponsoring a resolution to hold China accountable for hiding the truth about the #COVID_19 pandemic.\nhttps://t.co/Rvpm4PXZMY'), ('dates', '2020-04-30 17:40:34'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.8164427280426025)])
OrderedDict([('tweet', "I co-sponsored a resolution to hold China accountable for their failure in responding to the #COVID_19 pandemic.  The CDC needs to be allowed to investigate its origins.  And for China, maybe it's time you come clean, mate. https://t.co/IxSULyMe0S"), ('dates', '2020-04-28 23:44:45'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.6488223075866699)])
OrderedDict([('tweet', "The Chinese government didn't warn the public about the dangers of #COVID19 for six days, leading to a worldwide pandemic. I am proud to join @RepAndyBarr in asking Congress to create a committee to investigate China’s actions.  Read more here:  https://t.co/yVmEorMLlb"), ('dates', '2020-04-20 22:05:42'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.6907410621643066)])
OrderedDict([('tweet', '.@SpeakerPelosi and @HouseDemocrats are holding small businesses hostage during a time of national crisis.  Now the #PaymentProtectionProgram has run out of money and she still refuses to act.  Let us know if your business is being left out of the PPP: https://t.co/2jQl5rpU7q'), ('dates', '2020-04-16 17:38:23'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.7712551355361938)])
OrderedDict([('tweet', "The Chinese government didn't warn the public about the dangers of #COVIDー19 for SIX DAYS, causing thousands of unnecessary infections and leading to a worldwide pandemic. That's why I co-sponsored a resolution condemning their response.\xa0\nRead the bill:\xa0https://t.co/yS8b4HXwLQ"), ('dates', '2020-04-15 18:47:55'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.7640775442123413)])
OrderedDict([('tweet', 'While the country suffers during #COVID19, @SpeakerPelosi and House Democrats are playing politics with Americans’ jobs and livelihoods. The #PaycheckProtectionProgram needs to be renewed before funds run out on Friday, this will keep small businesses open and Americans employed.'), ('dates', '2020-04-14 14:27:27'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.8388206958770752)])
OrderedDict([('tweet', 'While jobless claims soar to their highest level in years, Democrats are playing politics with American lives.  This morning Democrats blocked a $250 billion dollar expansion of the #PaycheckProtectionProgram.  This program will save small businesses and American jobs.'), ('dates', '2020-04-09 16:50:41'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.9849543571472168)])
OrderedDict([('tweet', 'It is unbelievable to me that Senate Democrats would refuse to debate a #COVIDー19 relief package and @SpeakerPelosi  would walk away from negotiations.  During a time of national crisis, our leaders must take decisive action to fight this pandemic.'), ('dates', '2020-03-23 13:34:45'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.8386725187301636)])
OrderedDict([('tweet', 'Mnuchin and Pelosi have emerged as partners in the battle to protect Americans from the economic destruction wrought by #Covid_19.\n\n“Right now there is no one else,” said Rep. Denver Riggleman. He says of Mnuchin: “I’ve been very impressed with how he sticks to the facts.” https://t.co/rbpDdXboj7'), ('dates', '2020-03-20 13:19:14'), ('file_name', 'Denver Riggleman.csv'), ('pct_true', 0.5573487877845764)])
OrderedDict([('tweet', "While new programs are providing significant assistance to millions of small businesses, others are struggling to gain access. That's why I introduced a bill to give our  smallest small businesses a rebate to help retain employees and keep the lights on. https://t.co/rdSWxOVZ5T"), ('dates', '2020-04-29 21:09:28'), ('file_name', 'Derek Kilmer.csv'), ('pct_true', 0.5365553498268127)])
OrderedDict([('tweet', 'Today marks 10 yrs since the Deepwater Horizon spill in the Gulf of Mexico, which killed 11 &amp; spilled 210M gallons of oil into the ocean. In our region, Democrats &amp; Republicans have agreed for decades that opening the waters off our coast to drilling would be shortsighted &amp; wrong'), ('dates', '2020-04-20 20:23:43'), ('file_name', 'Derek Kilmer.csv'), ('pct_true', 0.8833703994750977)])
OrderedDict([('tweet', 'Before Speaker Pelosi introduced House Democrats’ disastrous drug pricing bill, Republicans and Democrats were working together to lower drug prices. Instead, Democrats are advancing a partisan bill that will take a sledgehammer to medical innovation. https://t.co/VDnN4BgeVR'), ('dates', '2019-10-17 21:19:07'), ('file_name', 'Devin Nunes.csv'), ('pct_true', 0.9694195985794067)])
OrderedDict([('tweet', "Seeing a lot of fake news from media elites and others who have no interest in violations of Americans' civil liberties via unmaskings."), ('dates', '2017-06-01 17:23:50'), ('file_name', 'Devin Nunes.csv'), ('pct_true', 0.6999979615211487)])
OrderedDict([('tweet', "70 days ago, President Trump claimed that the number of U.S. COVID-19 cases would soon be “close to zero.”\n\nWe now have over 1.2 million cases &amp; nearly 75,000 deaths.\n\nThe Trump admin's coronavirus response has been a colossal failure &amp; the American people are paying the price. https://t.co/AtsUql7JI7"), ('dates', '2020-05-07 17:53:30'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.7931877970695496)])
OrderedDict([('tweet', "We must ensure minority-owned small businesses are getting equal access to the #PaycheckProtectionProgram.\n\nWe're calling for greater transparency on who is getting these loans. And if certain businesses owners are being shut out, we need to take immediate steps to rectify it. https://t.co/flrA1jCVMz"), ('dates', '2020-04-21 20:40:38'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.5822337865829468)])
OrderedDict([('tweet', "President Trump just announced he's halting all funding to the World Health Organization.\n\nWe are in the midst of an unprecedented pandemic. We need to work with our global allies, not cast them aside.\n\nThis terrible decision will cost lives and we cannot let it stand."), ('dates', '2020-04-14 23:21:47'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.5092777013778687)])
OrderedDict([('tweet', "Increasingly concerned by President Trump's baseless attacks on our nation's independent inspector generals.\n\nToday, I'm calling on HHS Secretary Azar to ensure his agency continues to cooperate fully with the Office of Inspector General's audits and investigations.\n \nRead more: https://t.co/XDUCNNmdHp"), ('dates', '2020-04-14 19:45:08'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.512308657169342)])
OrderedDict([('tweet', "Colorado was set to receive 500 ventilators until FEMA blocked the shipment. Now, Trump says we will get 100 as a courtesy to Sen. Gardner. Because POTUS is playing politics w/ public health, we're still 400 short. His mismanagement of this crisis is costing lives &amp; livelihoods. https://t.co/wgKAgGgzvj"), ('dates', '2020-04-08 15:55:33'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.5348889827728271)])
OrderedDict([('tweet', 'We are in the midst of an unprecedented global health crisis. Yet, rather than focus on protecting public health, anti-choice politicians are exploiting this crisis to try and block patients’ access to abortion care. https://t.co/9hrywuu9pG'), ('dates', '2020-04-08 01:33:27'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.7109721302986145)])
OrderedDict([('tweet', "Unbelievable. In the midst of a global pandemic, the Trump admin just announced it will roll back our nation's auto pollution rules – which have been critical in our efforts to combat the climate crisis.\n\nThis is a direct attack on our public health and our environment. https://t.co/cQRmDvAsp7"), ('dates', '2020-03-31 17:21:16'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.571401059627533)])
OrderedDict([('tweet', 'While protecting Americans from the coronavirus is our top priority, we can’t lose sight of the ongoing attacks on our environment.\n \nDespite this crisis, the Trump admin has continued to auction off public land and restrict science.\n\nThis is unacceptable.\nhttps://t.co/VXsqArDsTX'), ('dates', '2020-03-25 19:39:06'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.5642586946487427)])
OrderedDict([('tweet', "NEWS: My bill to require health insurers to cover the FULL cost of coronavirus tests has been included in the coronavirus response bill that the House will be voting on soon.\n \nNo American should worry about how they're going to pay for a test if a doctor advises them to get one."), ('dates', '2020-03-13 22:22:29'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.5747963190078735)])
OrderedDict([('tweet', 'President Trump needs to stop misleading the American people. Saying this virus will simply “wash through” is dangerous.\n \nWe need to act now and pass our #FamiliesFirst Act to provide paid sick leave, food assistance, free testing, and other help to those affected by this virus.'), ('dates', '2020-03-13 21:37:16'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.5616547465324402)])
OrderedDict([('tweet', 'President Trump needs to stop misleading this country, and start doing whatever he can to protect it.\n \nWe need to listen to our scientists about the threat we face – not the president’s “hunches,” which experts agree are just flat-out false.\nhttps://t.co/guYuj7mWJc'), ('dates', '2020-03-05 19:24:24'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.6400625109672546)])
OrderedDict([('tweet', 'We need a real coronavirus czar, one who has command and control of the situation, the authority to direct the federal government’s overall response and budget, and makes decision based solely on what’s in the best interest of the public’s health.'), ('dates', '2020-02-27 01:55:16'), ('file_name', 'Diana DeGette.csv'), ('pct_true', 0.5979738235473633)])
OrderedDict([('tweet', 'Justin Walker should not be confirmed to the D.C. Circuit Court. His opposition to the ACA is disqualifying. He’s only 37 years old and has served just six months as a district court judge. Senate Republicans should not promote him to the second most powerful court in the nation. https://t.co/t6RdWpofUR'), ('dates', '2020-05-06 18:11:30'), ('file_name', 'Dianne Feinstein.csv'), ('pct_true', 0.5558986663818359)])
OrderedDict([('tweet', 'The Supreme Court is hearing arguments tomorrow on Republican-led attempts to roll back birth control protections under the ACA. This is unacceptable. The Supreme Court should uphold the ACA and protect women’s access to contraception. #HandsOffMyBC'), ('dates', '2020-05-05 19:19:03'), ('file_name', 'Dianne Feinstein.csv'), ('pct_true', 0.9588272571563721)])
OrderedDict([('tweet', 'When the Senate reconvenes next week, we should focus on the coronavirus and economic recovery. Rather than holding routine hearings, we need to address this public health emergency and the administration’s implementation of funding.\xa0\nhttps://t.co/VH3hLZGjwd'), ('dates', '2020-04-28 20:29:51'), ('file_name', 'Dianne Feinstein.csv'), ('pct_true', 0.5514891147613525)])
OrderedDict([('tweet', 'The president’s immigration proclamation is a distraction at a time when we should be focused on fighting the pandemic and economic recovery. We need to safely reopen the country, and adding unnecessary obstacles for immigrants won’t help us get there. https://t.co/Xz7SXtpKK5'), ('dates', '2020-04-23 17:43:09'), ('file_name', 'Dianne Feinstein.csv'), ('pct_true', 0.5432273149490356)])
OrderedDict([('tweet', 'It’s reprehensible that people are attacking Asian Americans because of racist, unfounded associations that these individuals are somehow linked to the coronavirus pandemic. We’re calling on President Trump to defend the Asian American and Pacific Islander community. https://t.co/haEaMY6biU'), ('dates', '2020-04-22 18:28:30'), ('file_name', 'Dianne Feinstein.csv'), ('pct_true', 0.9926486611366272)])
OrderedDict([('tweet', 'The Asian Pacific Policy and Planning Council reported more than 1,100 incidents of Asian Americans being harassed in the last two weeks. Bigotry and hate have no place here. https://t.co/YqNPzROuqI'), ('dates', '2020-04-17 15:55:09'), ('file_name', 'Dianne Feinstein.csv'), ('pct_true', 0.7124096155166626)])
OrderedDict([('tweet', 'Reporting by @ProPublica shows federal prosecutors were ready to pursue charges against Walmart for refusing to let its pharmacists block opioid prescriptions from suspected “pill mills.” But high-ranking DOJ officials prevented an indictment after being approached by Walmart.'), ('dates', '2020-04-03 19:52:00'), ('file_name', 'Dianne Feinstein.csv'), ('pct_true', 0.5858627557754517)])
OrderedDict([('tweet', 'Americans will go hungry because the USDA has not done enough to connect farmers to organizations around the country that are feeding those in need. The Trump Administration can and should do more.\nhttps://t.co/AwlgzshdeX'), ('dates', '2020-04-28 18:12:35'), ('file_name', 'Dina Titus.csv'), ('pct_true', 0.6786578893661499)])
OrderedDict([('tweet', "President Trump should've avoided this obvious conflict of interest by divesting from his business like every modern president before him. It’s wrong for the Trump Administration to use taxpayer resources to bail out Trump's D.C. hotel. https://t.co/J3SsZ5G29s"), ('dates', '2020-04-22 16:48:14'), ('file_name', 'Dina Titus.csv'), ('pct_true', 0.7937065958976746)])
OrderedDict([('tweet', 'On #HolocaustRemembranceDay, let us recommit to #NeverForget the 6 million Jews and millions of others who were murdered by the Nazi regime. We must share their stories and continue to fight against bigotry and hate in all its ugly forms.'), ('dates', '2020-04-21 19:09:40'), ('file_name', 'Dina Titus.csv'), ('pct_true', 0.5049203634262085)])
OrderedDict([('tweet', 'The reports of violence against Asian Americans in this country are simply unacceptable. I stand in solidarity with Nevada’s AAPI community. We must all do our part to stop the spread of bigotry and racism. #NoAAPIHate'), ('dates', '2020-04-14 18:26:18'), ('file_name', 'Dina Titus.csv'), ('pct_true', 0.834585964679718)])
OrderedDict([('tweet', 'On #EqualPayDay we are reminded that women lose an average of $10,000+ to the wage gap each year. Amidst the #COVID19 pandemic, too many families are struggling to pay rent and put food on the table. @Senatemajldr should allow a vote on the #PaycheckFairnessAct!'), ('dates', '2020-03-31 18:46:05'), ('file_name', 'Dina Titus.csv'), ('pct_true', 0.5417380928993225)])
OrderedDict([('tweet', "Instead of warning the world, China's Communist Govt lied while censoring doctors &amp; journalists delaying the global response by at least a month. This time could have reduced the spread by 2/3. The world is now racing to control the damage caused by China's irresponsible actions. https://t.co/MKAGkPocEb"), ('dates', '2020-04-22 20:20:09'), ('file_name', 'Don Bacon.csv'), ('pct_true', 0.9253159165382385)])
OrderedDict([('tweet', "Many businesses will never recover because of Speaker Pelosi and Sen. Schumer's delays in replenishing funds for the #PaycheckProtectionProgram. It's time to put personal agendas aside and put America's small businesses first. Let's act now!"), ('dates', '2020-04-21 15:05:04'), ('file_name', 'Don Bacon.csv'), ('pct_true', 0.9305700659751892)])
OrderedDict([('tweet', 'Nebraska’s small businesses &amp; non-profits got big-time support from the Paycheck Protection Program. Schumer &amp; Pelosi have repeatedly blocked efforts to replenish program that ran out of funds last week. When virus cloud lifts we want Small Biz to be open. https://t.co/rWNrbVG4d7'), ('dates', '2020-04-20 12:27:59'), ('file_name', 'Don Bacon.csv'), ('pct_true', 0.8960831761360168)])
OrderedDict([('tweet', 'Speaker Pelosi needs to stop playing politics and stop holding the #PaycheckProtectionProgram hostage. I have heard from countless small businesses and nonprofits who are waiting to apply so they can keep their doors open and keep workers on payroll. https://t.co/PD2lKrGPLX'), ('dates', '2020-04-17 22:50:02'), ('file_name', 'Don Bacon.csv'), ('pct_true', 0.6100830435752869)])
OrderedDict([('tweet', '#OTD 75 years ago, ruthless evil was defeated in Europe. We’ll never forget our brave soldiers who fought with our Allies against Hitler’s fascist aggression, and today I hope you’ll join me in keeping the brave soldiers who fought for freedom in your prayers. #VEDay75 https://t.co/8yXKIlOP0k'), ('dates', '2020-05-08 16:22:56'), ('file_name', 'Don Young.csv'), ('pct_true', 0.507780134677887)])
OrderedDict([('tweet', 'For too long, the federal government has stood in the way of states that have acted to set their own #cannabis policies. I’ve visited my state’s legal cannabis operations, and the fact is that these businesses contribute to our economy and create jobs for Alaskans. https://t.co/cX8zvsxtZa'), ('dates', '2020-04-20 15:44:43'), ('file_name', 'Don Young.csv'), ('pct_true', 0.7137267589569092)])
OrderedDict([('tweet', ".@ASIndianAffairs Sweeney has always advocated for ALL Indigenous peoples. @SenSchumer's attack on the first Alaska Native woman in her post is disgraceful and a fundamental misunderstanding of the very law he voted for. https://t.co/EwTsdwSBDS"), ('dates', '2020-04-16 20:20:21'), ('file_name', 'Don Young.csv'), ('pct_true', 0.6437646150588989)])
OrderedDict([('tweet', "75 years ago, Allied forces accepted Nazi Germany's unconditional surrender – ending #WWII in Europe.\n\nThankful for our servicemembers – &amp; the allies who fought alongside them – for their bravery &amp; sacrifice. #VEDay75 https://t.co/Q2U4lgltHA"), ('dates', '2020-05-08 20:06:03'), ('file_name', 'Donald Norcross.csv'), ('pct_true', 0.844976544380188)])
OrderedDict([('tweet', 'Tonight, we commemorate #YomHaShoah &amp; remember the 6 million Jews &amp; millions of others who were killed during the Holocaust 75 years ago.\n\nWe must #NeverForget the victims &amp; recommit to standing up anti-Semitism in all forms. #HolocaustRemembranceDay https://t.co/GvJfaT9oPs'), ('dates', '2020-04-20 22:27:58'), ('file_name', 'Donald Norcross.csv'), ('pct_true', 0.6241458058357239)])
OrderedDict([('tweet', 'Too many small businesses are missing out on #PPPLoans. \n\n@housedemocrats are working to ensure #smallbiz in underserved communities have access to this money. That’s why any #Smallbizrelief bill must empower community lenders. https://t.co/KdsROHOwHY'), ('dates', '2020-04-17 18:29:58'), ('file_name', 'Donald Norcross.csv'), ('pct_true', 0.6808899641036987)])
OrderedDict([('tweet', 'Tens of thousands of Americans – over 73,000 – have now died from a pandemic that has hit our country so much harder because this President ignored warnings from health officials over and over again. This is so dangerous. https://t.co/APQuJEq0Xl'), ('dates', '2020-05-08 00:46:24'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.8103347420692444)])
OrderedDict([('tweet', 'So many need help right now just to buy food or pay for housing. Congress must not fail them. This is not a time to hold back – leaving people at the mercy of this crisis would be immoral.\n\n@RepDean and I have a plan to help people with new additional, improved direct payments. https://t.co/yEyUsgoS0K'), ('dates', '2020-05-07 21:39:08'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.631554901599884)])
OrderedDict([('tweet', 'Whataboutism on a grand scale with the clear intent to distract the American people from the bumbling incompetence of a President who ignored repeated warnings and failed to keep them safe. https://t.co/549FV4JYuf'), ('dates', '2020-05-07 16:03:35'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.5532321929931641)])
OrderedDict([('tweet', "You might think that even this president would have enough sense not to try to kick millions of Americans off their health coverage and wreck our national health care system in the middle of the worst pandemic in a century. But you'd be wrong. https://t.co/unI6esuKtL"), ('dates', '2020-05-06 21:31:00'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.5640981197357178)])
OrderedDict([('tweet', 'Tens of thousands of Americans are dying from COVID-19 and the President is laser focused on stealing your money which Congress did not appropriate to spend on painting his giant symbol of bigotry a different color at an estimated cost of millions of dollars per mile. https://t.co/HYC2Stk3cH'), ('dates', '2020-05-06 17:33:00'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.9159688949584961)])
OrderedDict([('tweet', 'Trump keeps blaming China for his own failed pandemic response -- but a new whistleblower complaint says a top health official warned the Administration early on that the virus could spread to the U.S., and was met with "indifference which then developed into hostility." https://t.co/QuRezLH0bB'), ('dates', '2020-05-05 20:41:25'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.8897180557250977)])
OrderedDict([('tweet', "Trump is openly calling for tax cuts for the wealthy and big corporations as ransom for preventing layoffs of teachers, firefighters, and police, and averting a national depression. \n\nThat's what this means. https://t.co/W9D9WJmEUX"), ('dates', '2020-05-05 18:48:37'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.6929119229316711)])
OrderedDict([('tweet', 'Months into the crisis the President still has no idea what the country’s testing capacity is like.\n\nTesting problems contributed enormously to early failures to contain this virus, we still don’t have enough testing, and the President is just completely clueless about it. https://t.co/KbVuebdKw7'), ('dates', '2020-05-04 14:30:30'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.640277087688446)])
OrderedDict([('tweet', 'The American people need and deserve to hear the officials in charge of fighting this pandemic answer questions from their Representatives about their government’s response.\n\nBlocking Dr. Fauci’s testimony is an utterly damning indictment of the Trump Administration’s response. https://t.co/KBR22Av2C8'), ('dates', '2020-05-01 22:40:40'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.5840019583702087)])
OrderedDict([('tweet', 'According to Republicans, select committees to investigate heavily edited sting videos and shout about Benghazi for campaign purposes are good uses of taxpayer money, but oversight of the response to a pandemic that cost 60,000 American lives and 30 million jobs is "unnecessary." https://t.co/9nD8u7TPLx'), ('dates', '2020-04-30 18:46:54'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.6571601033210754)])
OrderedDict([('tweet', '30 million American workers have now lost their jobs in the past six weeks.\n\n1 in 5 Americans are unemployed.\n\nCongress must do all in its power to help them survive this crisis no matter how long it lasts.'), ('dates', '2020-04-30 12:43:14'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.6884443163871765)])
OrderedDict([('tweet', '60,000 Americans have died from COVID-19 and over a million are sick https://t.co/aytQx2N8YJ'), ('dates', '2020-04-30 12:20:40'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.5246356129646301)])
OrderedDict([('tweet', 'House Republicans call oversight of the coronavirus relief funds "unnecessary."\n\nIf they were in charge they\'d write the Trump Administration a blank check to make sure the wealthy and big corporations were taken care of while sweeping any wrongdoing under the rug. AKA "2017." https://t.co/vPlFjJmF2U'), ('dates', '2020-04-29 21:10:06'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.5003193616867065)])
OrderedDict([('tweet', 'The Vietnam Veterans Memorial has 58,320 names inscribed on it commemorating American lives lost in Vietnam during 19 years of war from June, 1956 to May, 1975.\n\nThe United States has now lost more lives in two months to COVID-19 than we lost in 228 months of the Vietnam War.'), ('dates', '2020-04-28 22:46:57'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.5292304158210754)])
OrderedDict([('tweet', 'This is the kind of blackmail that got Trump impeached, except in this case he proposes threatening American citizens with unemployment, sickness, and death. https://t.co/owY2rjFF6A'), ('dates', '2020-04-28 21:40:21'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.7806236147880554)])
OrderedDict([('tweet', "The Administration spent crucial weeks ignoring public health officials, and in consequence the U.S. has a third of the world's COVID-19 cases and a quarter of its deaths with only 4% of the global population.\n\nSuch stupid, dangerous thinking would risk repeating that disaster. https://t.co/BrKUzwhxb4"), ('dates', '2020-04-27 20:23:17'), ('file_name', 'Donald S. Beyer, Jr..csv'), ('pct_true', 0.5243188142776489)])
OrderedDict([('tweet', 'ICE is deporting COVID-19 positive patients back Haiti. This is callous and unconscionable.\n\nThe Trump administration should not jeopardize other countries and spread its failure to manage this crisis abroad. https://t.co/kpmboF1C7p'), ('dates', '2020-05-08 12:58:44'), ('file_name', 'Donna E. Shalala.csv'), ('pct_true', 0.5927042365074158)])
OrderedDict([('tweet', 'Why is it so hard for Mitch McConnell to put #FamiliesFirst?\n\nFirst, he holds up the PPP for two weeks – keeping resources from small businesses – and now he wants states to go bankrupt instead of delivering desperately needed support for police, fire, and EMS.'), ('dates', '2020-05-06 20:07:35'), ('file_name', 'Donna E. Shalala.csv'), ('pct_true', 0.802014172077179)])
OrderedDict([('tweet', "I don't know what's worse - Florida's inability to process the thousands of unemployment benefits claims that have been filed or that it keeps taking down the website for days at a time.\n\nPeople are suffering and this has been a total failure. \n\nhttps://t.co/DMSt2qOurG?"), ('dates', '2020-04-24 18:27:35'), ('file_name', 'Donna E. Shalala.csv'), ('pct_true', 0.5850666165351868)])
OrderedDict([('tweet', "Yesterday, the House passed a bill to extend billions more in emergency relief to small businesses.\n\nThat said, we're simply putting a blanket over the fire until this administration shows some leadership and gets the money to those who need it most.\n\nhttps://t.co/Q5JYI0q4cK"), ('dates', '2020-04-24 14:18:55'), ('file_name', 'Donna E. Shalala.csv'), ('pct_true', 0.5278216600418091)])
OrderedDict([('tweet', 'President Trump just suspended immigration to the US.\n\nPredictably, he made sure his ban didn’t apply to millionaire investors.\n\nThis is a new low, even for this administration.'), ('dates', '2020-04-22 23:37:09'), ('file_name', 'Donna E. Shalala.csv'), ('pct_true', 0.5038763880729675)])
OrderedDict([('tweet', '.@realDonaldTrump has failed every test of leadership since the COVID-19 pandemic began.\n\nSuspending immigration to the US is a xenophobic ploy to distract the public from his complete mismanagement of this crisis.\n\nAmericans will not be fooled or forget your failures so easily.'), ('dates', '2020-04-21 14:18:09'), ('file_name', 'Donna E. Shalala.csv'), ('pct_true', 0.6721988916397095)])
OrderedDict([('tweet', 'On #YomHaShoah we remember the millions of Jews who were murdered in the Holocaust, the millions who survived its horrors, and the heroes who resisted in the face of almost certain death.'), ('dates', '2020-04-20 23:12:33'), ('file_name', 'Donna E. Shalala.csv'), ('pct_true', 0.6654554605484009)])
OrderedDict([('tweet', '10 years after the #DeepwaterHorizon disaster, we need to #ProtectOurCoast more than ever.\n\nTrump isn’t helping protecting our oceans, the Americans who live on our coasts, or the wildlife who need clean environments to live. https://t.co/AP333EJMq2'), ('dates', '2020-04-20 18:30:00'), ('file_name', 'Donna E. Shalala.csv'), ('pct_true', 0.5441657900810242)])
OrderedDict([('tweet', 'What a disgrace.\n\n650,000 people have applied for unemployment in FL since March 15 and only 33,600 have been paid. \n\nPeople are suffering and Governor DeSantis clearly doesn’t have a plan to fix it.\n\nhttps://t.co/Hzh3SqZGH9'), ('dates', '2020-04-17 11:20:41'), ('file_name', 'Donna E. Shalala.csv'), ('pct_true', 0.7963179349899292)])
OrderedDict([('tweet', '.@realDonaldTrump will never take responsibility for his missteps. He always needs someone to blame.\n\nDefunding the @WHO in the middle of a pandemic is foolish and will have dire consequences. https://t.co/1HnjhrsDed'), ('dates', '2020-04-14 22:44:54'), ('file_name', 'Donna E. Shalala.csv'), ('pct_true', 0.6511701941490173)])
OrderedDict([('tweet', "Today, Secretary DeVos made the unconscionable decision to implement a rule that slashes Title IX protections for students &amp; protects those accused of sexual assault over victims. Amid a worldwide pandemic, now is not the time to upend survivors' rights. \nhttps://t.co/rI2koNcicP"), ('dates', '2020-05-06 20:46:55'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.5986002683639526)])
OrderedDict([('tweet', 'Today, the Trump Administration finalized Part Two of the so-called SAFE Vehicles Rule - significantly relaxing fuel efficiency standards and preventing many of the gains in greenhouse gas reductions that would have been achieved under the Obama-era standards.'), ('dates', '2020-03-31 23:26:26'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.6716340184211731)])
OrderedDict([('tweet', 'The Trump Administration’s cruel Public Charge rule goes into effect today. This is a blatant attempt to strike fear into immigrant communities. We must give families access to the right information so that they know their rights. https://t.co/uYHUZPy3yr https://t.co/1zi86YCyOj'), ('dates', '2020-02-24 20:13:07'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.7646852731704712)])
OrderedDict([('tweet', '78 years ago today, Executive Order 9066 incarcerated my family &amp; over 120,000 other Americans in an act of fear – simply because of our ethnic heritage. The internment of Japanese Americans was an act of bigotry, racism &amp; injustice that went against our founding principles.'), ('dates', '2020-02-19 22:23:31'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.7527099251747131)])
OrderedDict([('tweet', 'Today, I’m proud to stand with Democrats fighting back against the Trump Administration’s Medicaid block grant scheme, which threatens to increase drug and health care costs for millions of Americans. #ProtectOurCare https://t.co/inqPPMfynd'), ('dates', '2020-02-06 19:38:20'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.8044597506523132)])
OrderedDict([('tweet', "The updated Travel Ban continues the policy of separating families. We can't let these discriminatory rules continue under the guise of safety. That is why I am a co-sponsor of the No Ban Act, which would rescind all versions of @POTUS’ hateful Muslim Ban. https://t.co/Pov7dOCPTL"), ('dates', '2020-02-06 00:30:00'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.5515938401222229)])
OrderedDict([('tweet', 'The #StateOfHealthCare is: Democrats fighting to finally lower prescription drug costs and expand care, while Pres. Trump and Republicans try to rip care away. #SOTU https://t.co/omRK9VZdzh'), ('dates', '2020-02-04 22:00:00'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.5041406154632568)])
OrderedDict([('tweet', 'This National Gun Violence Survivors Week, we remember all of the American families &amp; communities that have suffered from the gun violence epidemic in this country. We need wholesale change to protect our families. We need to confront this crisis. #MomentsThatSurvive https://t.co/OVnjsathHf'), ('dates', '2020-02-03 22:24:25'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.5202775597572327)])
OrderedDict([('tweet', "Capping funding for Medicaid is not flexibility – it’s taking away people’s coverage and access to care. The Trump Administration's new Medicaid plan is a targeted attack on the health care of millions of Americans, plain and simple. https://t.co/ZXfAmM1NM9"), ('dates', '2020-01-30 19:44:03'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.5944759249687195)])
OrderedDict([('tweet', 'On #HolocaustRemembranceDay, #WeRemember the genocide of over six million Jews by the Nazi regime. We must never stop fighting against hate, bigotry &amp; other-ing in our society. Antisemitism, racism and all forms of prejudice must be defeated. #NeverAgain https://t.co/EpRzTuTJVX'), ('dates', '2020-01-27 23:00:07'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.9513060450553894)])
OrderedDict([('tweet', 'Three years ago today, the Trump administration reinstated the Global Gag Rule, forcing NGOs to choose between providing vital reproductive health services or accessing US assistance. https://t.co/tQUrrpHHrQ'), ('dates', '2020-01-23 23:52:37'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.5335745215415955)])
OrderedDict([('tweet', "Today, the #SCOTUS brought forward Trump's unlawful case against #DACA recipients. We cannot let hate, xenophobia, and fear dictate how we treat human beings. Dreamers are vital parts of our communities, and I will stand shoulder to shoulder you in this fight. #HomeIsHere https://t.co/5aUmQR0O2T"), ('dates', '2019-11-12 18:28:50'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.5283796787261963)])
OrderedDict([('tweet', "Climate change is our planet's greatest threat. That's why Trump pulling out of the Paris Climate Agreement will be remembered as a failure of American leadership. This catastrophic decision will have consequences for our planet and generations to come. https://t.co/GCyYCPxjrf"), ('dates', '2019-11-04 22:25:49'), ('file_name', 'Doris O. Matsui.csv'), ('pct_true', 0.7105439305305481)])
OrderedDict([('tweet', 'Wrong again, Comey.\n\nDOJ lost its way when self righteous “leaders” at the top of our justice system thought they could play Superman and dictate the will of the American people.\nhttps://t.co/RyTBn93Ikk'), ('dates', '2020-05-07 21:24:47'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.6525239944458008)])
OrderedDict([('tweet', 'We already knew the Chinese Communist Party covered up the severity of #COVIDー19.\n \nBut their lies allowed the world to suffer while they hoarded medical supplies.\n \nThey must be held accountable.\n \nhttps://t.co/8FCtEB22CG'), ('dates', '2020-05-04 20:21:33'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.9657635688781738)])
OrderedDict([('tweet', 'Contrary to what the Democrats want you to believe, General Flynn wasn’t charged with Russian collusion.\n\nHe was wrongly charged with lying to the corrupt cabal at the FBI that desperately tried to take down @realDonaldTrump in 2016. https://t.co/YzptjNOtG2'), ('dates', '2020-04-29 21:01:56'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.9297241568565369)])
OrderedDict([('tweet', "The World Health Organization was a willing accomplice to the Chinese Communist Party's coverup scheme, playing a critical role in spreading their lies and misinformation.\n \nAs the single largest contributor to the WHO, we MUST hold them accountable. https://t.co/7TAooImr43"), ('dates', '2020-04-24 19:05:27'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.7198551893234253)])
OrderedDict([('tweet', "Democrats delayed and delayed while small businesses suffered.\n \nBut as soon as we return to Washington, the first action they take is forming a Select Committee to investigate @realDonaldTrump's response to #COVIDー19.\n\nNow more than ever, it's clear where their priorities lie."), ('dates', '2020-04-23 17:30:18'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.9444764256477356)])
OrderedDict([('tweet', '.@realDonaldTrump requested additional funding for #PPP weeks ago, but Democrats have left American workers and small businesses waiting.\n \nCongress has the responsibility to look out for both the health of the American people and the economic health of our country. https://t.co/yy5X65JNoP'), ('dates', '2020-04-23 15:12:50'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.8981732726097107)])
OrderedDict([('tweet', "China's coverup of #COVIDー19 and their continued lies about its origin have shown us all firsthand that we cannot trust the word of the Chinese government.\n \nThat's why I'm calling on @USDA to reconsider allowing the U.S. to import citrus from China. https://t.co/VujHfmw6aE"), ('dates', '2020-04-20 19:10:03'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.9775501489639282)])
OrderedDict([('tweet', 'Where was Adam Schiff — the Chairman of the House Intelligence Committee — when China was lying about this pandemic?\n\nHe was impeaching the President. https://t.co/Dq9Yo9ciDM'), ('dates', '2020-04-15 19:29:18'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.5999349355697632)])
OrderedDict([('tweet', 'Big news from @AP → Chinese officials failed to alert the public of the likely pandemic for nearly a week.\n \nThink about how many lives could have been saved if they had warned the public six days sooner.\n\n@realDonaldTrump is right to hold China accountable. https://t.co/TNn7cniyDq'), ('dates', '2020-04-15 15:31:39'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.7567728757858276)])
OrderedDict([('tweet', '.@SpeakerPelosi would rather put small businesses at risk than pass additional funding for the #PaycheckProtectionProgram.\n \nAmericans are worried about their health and livelihoods while she’s worried about her political agenda. https://t.co/8wmmpHgb9X'), ('dates', '2020-04-15 00:27:47'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.5300270915031433)])
OrderedDict([('tweet', 'I’m glad to see @realDonaldTrump take this step. The @WHO has fiercely defended the Chinese Communist Party while they created a worldwide pandemic. American taxpayers shouldn’t have to give them another dime. https://t.co/wr5Q3v4ZIl'), ('dates', '2020-04-14 23:06:03'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.8817524313926697)])
OrderedDict([('tweet', 'It shouldn’t take a pandemic to recognize the real bad actor on the world stage — China.\n \nBy bringing our supply chain back to the US, we can control American access to pharmaceuticals, health care equipment, and military supplies. https://t.co/91zRqH39SC'), ('dates', '2020-04-13 19:51:28'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.5958305597305298)])
OrderedDict([('tweet', 'Congress must pass additional funding for the #PaycheckProtectionProgram immediately.\n \nDemocrats are dragging out negotiations, but we need to get this done NOW.\n \nOur small businesses are counting on us. https://t.co/MXW3X37zvY'), ('dates', '2020-04-13 14:04:39'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.5512401461601257)])
OrderedDict([('tweet', "Senate Democrats just blocked additional funding for small businesses.\n\nThese funds are absolutely critical to ensuring American workers keep their jobs and small businesses stay afloat.\n \nDemocrats should be ashamed for putting partisan politics over Americans' livelihoods."), ('dates', '2020-04-09 15:23:50'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.9542989134788513)])
OrderedDict([('tweet', '.@WHO is turning a blind eye to the corruption in China that created a worldwide pandemic, but has criticized @realDonaldTrump for putting American interests first.\n \nAn international organization that supports dictators over strong leaders doesn’t need America’s help. https://t.co/GVqmWyPY8p'), ('dates', '2020-04-08 22:55:07'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.9639723896980286)])
OrderedDict([('tweet', "Since day one, the Chinese Communist Party has lied about #COVIDー19 and put the rest of the world at risk.\n\nThe @WHO's response? Praise China's response while criticizing @realDonaldTrump's travel restrictions.\n\nProud to support @GReschenthaler's resolution to defund @WHO. https://t.co/R4ATIaajxH"), ('dates', '2020-04-08 20:06:26'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.9495527148246765)])
OrderedDict([('tweet', "Rather than calling for a commission to investigate @realDonaldTrump’s coronavirus response, Adam Schiff should focus on doing his job and look into China's role in the spread of #COVID19.\n\nMaybe if he wasn’t so obsessed with impeachment, Congress could have acted sooner. https://t.co/xlxSQtl77f"), ('dates', '2020-04-02 20:22:06'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.862388014793396)])
OrderedDict([('tweet', 'Think about this → the first #coronavirus case in the U.S. was confirmed on January 15. \n\nThe same day, @SpeakerPelosi delivered the articles of impeachment to the Senate.\n\nMaybe if she stopped playing political games, our country could focus on what really matters. https://t.co/GMTqa5gCNk'), ('dates', '2020-03-31 21:02:22'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.6842826008796692)])
OrderedDict([('tweet', 'American workers, families, small businesses and medical professionals are waiting on Congress to act.\n\nBut they can’t wait any longer. They need relief NOW.\n\nLet’s get this done. #CARESAct'), ('dates', '2020-03-27 15:31:28'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.5401353240013123)])
OrderedDict([('tweet', 'People are losing their jobs, and some are even losing their lives.\n\nSpeaker Pelosi’s plan to limit ICE and border patrol, implement same-day voter registration, and mandate carbon offsets isn’t going to help.\n\nWe are running out of time. The American people need our help NOW.'), ('dates', '2020-03-24 13:08:29'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.8141561150550842)])
OrderedDict([('tweet', 'Millions of Americans are losing their jobs. Meanwhile, Democrats are working to... offset airline carbon emissions.\n\nDemocrats are taking advantage of this pandemic to check items off their wishlist while watching the American people suffer, and it’s truly horrifying. https://t.co/lQATB20EzB'), ('dates', '2020-03-23 17:17:29'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.9019409418106079)])
OrderedDict([('tweet', 'While Americans are hurting, @SenSchumer and @SpeakerPelosi are holding #coronavirus relief hostage in an effort to further their liberal agenda.\n\nA global health crisis is not the time to play politics. We must provide American workers with the relief they desperately need.'), ('dates', '2020-03-23 16:01:56'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.971687376499176)])
OrderedDict([('tweet', 'As @realDonaldTrump said, we will come out of this crisis.\n \nBut right now, our farmers, truckers, and independent contractors are bearing the brunt of #COVIDー19 and they need our support. https://t.co/zyYNMtpQ4d'), ('dates', '2020-03-20 19:06:06'), ('file_name', 'Doug Collins.csv'), ('pct_true', 0.5541809797286987)])
OrderedDict([('tweet', 'We need a bill that does more for the working folks who are struggling the most, not no-strings bailouts for corporations. The Senate will get something done for the American people, but we need to stop wasting time on political games. https://t.co/bFTJ1Ab6IZ'), ('dates', '2020-03-23 00:33:35'), ('file_name', 'Doug Jones.csv'), ('pct_true', 0.8208149671554565)])
OrderedDict([('tweet', "I agree w/@SenToomey: @CommerceGov must comply w/the law and release this report. It has yet to explain how Alabama-made cars are a nat'l security threat. \n\nHINT: they're not. This is a delay tactic so they can continue to threaten tariffs on our allies.\n\nhttps://t.co/z1B5OavoXr"), ('dates', '2020-01-22 00:49:11'), ('file_name', 'Doug Jones.csv'), ('pct_true', 0.6090756058692932)])
OrderedDict([('tweet', "These proposed rules have this trial taking place in the middle of the night without any guarantee for relevant documents and witnesses. The American people deserve a fair trial, and so does President Trump -- but this ain't it, folks. https://t.co/RARBvxqz3b"), ('dates', '2020-01-21 17:31:56'), ('file_name', 'Doug Jones.csv'), ('pct_true', 0.562759518623352)])
OrderedDict([('tweet', 'The #WHO was dishonest about the coronavirus — echoing and praising the lies disseminating from China. \n\n@realDonaldTrump was right to hold #WHO accountable. The American taxpayers should not prop-up an organization that shows dangerous bias towards the Chinese government.'), ('dates', '2020-04-15 15:16:26'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.9926018118858337)])
OrderedDict([('tweet', "It's been 28 days since Democrats jammed Articles of Impeachment through the House\n\nSince then, @POTUS has:\n✅Signed phase 1 of a trade deal with China\n✅Killed an Iranian terrorist\n✅Modernized NEPA\n\nIt's clear who is really working for Americans."), ('dates', '2020-01-15 18:07:23'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.605372428894043)])
OrderedDict([('tweet', "House Democrats just blocked @GOPLeader's resolution to stand behind Iranian protesters who are protesting a regime that chants 'Death to America.'\n\nThese people are fighting for freedom and risking their lives in the process. They need our support."), ('dates', '2020-01-14 21:14:00'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.9601837396621704)])
OrderedDict([('tweet', "This resolution puts the Democrats' agenda ahead of U.S. national security by undermining @realDonaldTrump's  Article II powers. \n\nThis is another example of the political games Democrats continue to play, &amp; that's why Americans will continue to trust Trump with our military."), ('dates', '2020-01-09 23:15:10'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.9898993372917175)])
OrderedDict([('tweet', "Remember: it took less than 20 minutes into the Trump Presidency for Dems to decide @realDonaldTrump should be impeached.\n\n3 years later, they're still at it - &amp; their quest to remove @POTUS from office at any cost won't stop. https://t.co/Ay7CthL3ct"), ('dates', '2019-11-25 16:23:36'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.5767372846603394)])
OrderedDict([('tweet', "It appears the bipartisan #USMCA is the latest victim of @SpeakerPelosi &amp; @RepAdamSchiff's impeachment obsession.\n\nNot much productivity left for Congress in 2019 - only more theatrics &amp; hysteria from Dems. It's a shame. We should pass the #USMCAnow.\n\nhttps://t.co/xrIQ3l29gs"), ('dates', '2019-11-21 21:09:01'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.723060667514801)])
OrderedDict([('tweet', 'No quid pro quo. No "bribery." Nothing.\n\nThe reaction of @RepAdamSchiff in this clip says it all - Democrats\' entire fabricated #impeachment narrative is collapsing all around them. https://t.co/kUjBV3mlaJ'), ('dates', '2019-11-20 21:01:45'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.7827264070510864)])
OrderedDict([('tweet', 'While this CR avoids a govt shutdown for a few weeks, Dems have merely punted from Thanksgiving to Christmas. We need a long-term solution that provides our nation with stability, namely for our military &amp; border security, &amp; this measure falls well short.\n\nhttps://t.co/FD55mIdnBu https://t.co/3DojCLvD4N'), ('dates', '2019-11-19 21:03:14'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.5706540942192078)])
OrderedDict([('tweet', 'So, let me get this straight. Democrats did some focus testing and decided that "bribery" was a better buzzword than "quid pro quo," regardless of whether or not it actually applies (it doesn\'t). Now they\'re sticking with it because it better fits their narrative?\n\nGot it. https://t.co/tm58I0bV4k'), ('dates', '2019-11-19 19:39:50'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.5192946195602417)])
OrderedDict([('tweet', "Following this week, there are only 2 remaining weeks Congress is scheduled to be in session this year.\n\nWhile @SpeakerPelosi &amp; @RepAdamSchiff focus on their #ImpeachmentScam, we're unable to get anything real accomplished, like the bipartisan #USMCA. Americans deserve better."), ('dates', '2019-11-19 15:40:47'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.6246505379676819)])
OrderedDict([('tweet', 'Quid pro quo, extortion, bribery...while Dems are busy focus testing which buzzword makes their false accusations sound the best, Americans can see right through this political smear campaign on @realDonaldTrump.\n\nBut Dems never let the facts get in the way of a good narrative. https://t.co/KoFhfU2xy4'), ('dates', '2019-11-19 15:20:49'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.9536981582641602)])
OrderedDict([('tweet', "As I've said many times before - this bogus #Impeachment scam is all about politics and nothing else.\n\nDemocrats can't change the results of the last election, but that won't stop them from attempting to rig the next one. https://t.co/Ne62a61eZd"), ('dates', '2019-11-18 15:33:49'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.78376305103302)])
OrderedDict([('tweet', 'The 1st public #impeachment hearings took over the cable airwaves this week, &amp; I spoke on the House floor to offer my thoughts on this whole process. Nothing has changed - this is an ugly, partisan witch hunt against @POTUS that is dividing our country &amp; holding up real progress. https://t.co/jVys3pz0pu'), ('dates', '2019-11-15 18:50:23'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.7583940625190735)])
OrderedDict([('tweet', "Nearly 40% of the Dems who've been tasked with overseeing the impeachment investigation are already compromised.\n\nThey have such a strong vendetta against @POTUS that they voted to impeach @realDonaldTrump long before the call in question even took place.\n\nIt's a partisan sham. https://t.co/xMGi5or7MG"), ('dates', '2019-11-15 15:02:09'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.7091457843780518)])
OrderedDict([('tweet', 'Yesterday on impeachment, @RepAOC: "This is about preventing a potentially disastrous outcome from occurring next year."\n\n@RepAlGreen in May: "I\'m concerned if we don’t impeach this president, he will get re-elected."\n\nIt\'s clear Dems\' main focus/fear is @realDonaldTrump in 2020. https://t.co/VofMkTvESa'), ('dates', '2019-11-14 17:38:20'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.6125693321228027)])
OrderedDict([('tweet', "As if we couldn't figure it out for ourselves, Democrats are actually saying it out loud - the sole purpose of this entire impeachment charade is to prevent President @realDonaldTrump from being re-elected. \n\nThis is an incredible and dangerous abuse of power. https://t.co/cNxHRuLUSp"), ('dates', '2019-11-14 15:16:29'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.7316192984580994)])
OrderedDict([('tweet', '.@RepAdamSchiff &amp; House Dems finally allowed their weeks-long, closed-door impeachment proceedings to take place in the public eye. Nothing has changed – the cameras don’t suddenly legitimize this bogus process. We do not impeach @POTUS based on hearsay.\nhttps://t.co/d2TohUDYR0 https://t.co/gloHzhR4uY'), ('dates', '2019-11-13 22:51:14'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.740618109703064)])
OrderedDict([('tweet', '.@DevinNunes is right: Speaker Pelosi pledged not to move to impeach @realDonaldTrump w/o bipartisan support.\n\n-ZERO Republicans supported impeachment resolution.\n-2 Dems even opposed it.\n\n@SpeakerPelosi received bipartisan OPPOSITION, but has moved forward anyways. Partisanship. https://t.co/BnNzQ1MBs4'), ('dates', '2019-11-13 17:53:52'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.801899790763855)])
OrderedDict([('tweet', "Dems have brought their #impeachment charade out of the basement &amp; into the spotlight today, but it's still all smoke &amp; mirrors to distract Americans from the facts.\n\nWhile Dems attempt to justify their vendetta against @realDonaldTrump, keep in mind these key facts: https://t.co/5w9ReaQL9W"), ('dates', '2019-11-13 16:34:37'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.9507402181625366)])
OrderedDict([('tweet', "While Dems' partisan impeachment crusade goes public this week, nothing has changed: @HouseGOP are relying on facts. Dems are not.\n\nThey've been unable to present any actual evidence of an impeachable offense by @realDonaldTrump, &amp; you don't impeach the @POTUS based on hearsay."), ('dates', '2019-11-12 19:13:47'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.9855717420578003)])
OrderedDict([('tweet', 'It turns out even the whistleblower\'s lawyer has acknowledged this coup against President @realDonaldTrump, even going as far as to say "We will get rid of him."\n\nHis open animosity against @POTUS reeks of partisanship &amp; a coordinated effort to impeach the President at all costs. https://t.co/0Xbd0bx1NM'), ('dates', '2019-11-07 17:02:52'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.747641384601593)])
OrderedDict([('tweet', '👻TRICK: In January, Dems promised to get things done for the American people w/their new majority, only to focus 110% of their energy on impeaching @realDonaldTrump.\n\n🎃TREAT: Our economy is booming, people have more $$ in their pockets, &amp; the terrorist leader of ISIS is dead.'), ('dates', '2019-10-31 19:45:25'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.5416900515556335)])
OrderedDict([('tweet', 'Today, Dems passed a resolution to continue their sham impeachment inquiry into President @realDonaldTrump. It has been conducted in secrecy &amp; behind-closed-doors for five weeks – &amp; now they want us to legitimize it? I don’t think so.\n\nMy full statement: https://t.co/JEHX82opFJ https://t.co/nvBxNm2F9b'), ('dates', '2019-10-31 16:14:22'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.688583254814148)])
OrderedDict([('tweet', 'This is a #Halloween witch hunt against President @realDonaldTrump.\n\nDespite Dems\' closed-door theatrics, most people know these so-called "impeachable offenses" rattling around the basement of the Capitol are no more real than the goblins that will show up at their door tonight. https://t.co/cUqcSFwnTN'), ('dates', '2019-10-31 14:57:28'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.8541949391365051)])
OrderedDict([('tweet', "Isn't it ironic that Dems picked #Halloween to attempt to legitimize their spooky basement impeachment process on @realDonaldTrump?\n\nCongress is not intended to be a house of horrors. Halloween or not, it's scary to everyday Americans the way Dems have abused this process."), ('dates', '2019-10-31 14:37:30'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.9860369563102722)])
OrderedDict([('tweet', "Take a look - House Democrats' radical agenda and obsession with impeachment should scare Americans more than any #Halloween costume ever could. https://t.co/IAGdjdVy62"), ('dates', '2019-10-29 21:14:10'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.8924976587295532)])
OrderedDict([('tweet', 'Years of ridiculous regulations from Sacramento have prevented even a hint of proper forest management.\n\nPresident @realDonaldTrump had it right last summer: “There is no reason for these massive, deadly &amp; costly forest fires in CA except that forest management is so poor.” https://t.co/DA4n178243'), ('dates', '2019-10-29 20:20:04'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.6163176894187927)])
OrderedDict([('tweet', 'While Dems focus all of their waking hours on impeaching President @realDonaldTrump, we could be passing the #USMCA to provide support for manufacturers &amp; small businesses across the country. \n\nCongress should drop the hyper-partisanship &amp; pass the #USMCANow https://t.co/80whoN6mVm'), ('dates', '2019-10-28 20:57:13'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.7088661193847656)])
OrderedDict([('tweet', "Democrats have been attempting to undermine @POTUS &amp; remove him from office since the day he was elected. These closed-door impeachment proceedings are no different.\n\nWhen 75% of the House - &amp; by extension 75% of Americans - have no say in the matter, that's a major problem. https://t.co/P8dKewW42W"), ('dates', '2019-10-24 20:53:03'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.9194488525390625)])
OrderedDict([('tweet', 'Democrats: Searching for any possible reason to impeach @realDonaldTrump &amp; undo the results of the 2016 election - &amp; doing so behind closed-doors.\n\n@POTUS: Overseeing an economic BOOM for working Americans and rejuvenating the U.S. economy.\n\nhttps://t.co/O14XtgT7SE'), ('dates', '2019-10-24 20:05:57'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.6822417974472046)])
OrderedDict([('tweet', "Dems continue to deny Republican Members of Congress access to their closed-door proceedings in the basement of the U.S. Capitol.\n\nNever before has an impeachment investigation been totally hidden from the American public.\n\nIt's wrong &amp; we'll continue fighting for transparency. https://t.co/XvySIYdnCL"), ('dates', '2019-10-23 17:48:08'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.8899368643760681)])
OrderedDict([('tweet', "For 2.5 years, Dems have tried to damage President @realDonaldTrump, &amp; it’s all behind closed-doors while the public is left in the dark.\n\nI'm in the SCIF right now w/@HouseGOP while Adam Schiff hides with the witness. We demand transparency &amp; we're prepared to stay here all day."), ('dates', '2019-10-23 17:41:45'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.8104503750801086)])
OrderedDict([('tweet', 'Moments ago, I joined many of my @HouseGOP colleagues in demanding more transparency in Democrats’ secretive impeachment investigation.\n\nWhy are Adam Schiff &amp; Speaker Pelosi conducting this process behind closed doors? \n\nThe American public deserves access. https://t.co/kf2hlA3iEN'), ('dates', '2019-10-23 15:11:44'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.6119307279586792)])
OrderedDict([('tweet', "Democrats clearly haven't run a fair &amp; transparent impeachment process. I know it, my @HouseGOP colleagues know it, &amp;, most importantly, the American people know it.\n\n@SteveScalise's rule change would allow all Members to participate in this #ParodyImpeachment. #StoptheSchiffShow https://t.co/FoSHVo3Imr"), ('dates', '2019-10-22 22:10:34'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.9490314722061157)])
OrderedDict([('tweet', "Millions of Americans have been left behind by the #ACA. That's why I'm working with my colleagues in the @RepublicanStudy Committee to protect people with pre-existing conditions and empower patients to choose the healthcare option that best suits their family. https://t.co/0PsMfOLrvz"), ('dates', '2019-10-22 21:09:45'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.7198623418807983)])
OrderedDict([('tweet', 'Tonight, I joined @SteveScalise, @RepAndyBiggsAZ, &amp; @HouseGOP in voting to censure Rep. Adam Schiff. \n\nOf course, Democrats blocked the measure in the House. Chairman Schiff’s dishonesty &amp; lack of transparency with the American people will likely continue. https://t.co/vuGKEm2DLj'), ('dates', '2019-10-21 23:31:09'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.7118576765060425)])
OrderedDict([('tweet', 'Today, I attempted to attend Democrats’ closed-door impeachment hearing, but was promptly “asked” to leave. \n\nThis is what a secretive, partisan, &amp; unfair process looks like. They’ve broken all precedent &amp; violated due process. \n\nThe American people deserve transparency. https://t.co/v4aWJYLvrW'), ('dates', '2019-10-17 15:57:19'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.6475638151168823)])
OrderedDict([('tweet', "I, like many Americans, am deeply troubled by the lack of transparency in the Dems' impeachment process. Adam Schiff has repeatedly &amp; intentionally misled the American people.\n\nI join @GOPLeader, @RepAndyBiggsAZ, &amp; many of my colleagues in calling for his censure in the House. https://t.co/hMjIPdtNG4"), ('dates', '2019-10-16 17:26:56'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.7314069867134094)])
OrderedDict([('tweet', "On the same day @SpeakerPelosi decided not to let the House vote on the impeachment inquiry, Dems literally attempted to silence @SteveScalise by cutting his mic on the House floor.\n\nYou just can't make this stuff up - Dems don't want Americans to have a voice in this process. https://t.co/XSGbtbMdjY"), ('dates', '2019-10-16 15:50:19'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.9730903506278992)])
OrderedDict([('tweet', 'While @KingJames, @SteveKerr, &amp; seemingly the entire @NBA shamefully refuse to stand up to Chinese authoritarianism for fear of it affecting their future business investments, Congress is not sitting idly by. \n\nWe voted last night to stand with the protestors in Hong Kong. https://t.co/M3mT6mPWzV'), ('dates', '2019-10-16 14:51:21'), ('file_name', 'Doug LaMalfa.csv'), ('pct_true', 0.7009008526802063)])
OrderedDict([('tweet', "I'm thankful that @GOPLeader is focused on the growing threat from China. In addition to covering up the spread of #COVID19, they led a propaganda effort to blame the U.S. &amp; are quickly militarizing the South China Sea. The new China Task Force will hold the #CCP accountable."), ('dates', '2020-05-08 19:16:49'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.7248979210853577)])
OrderedDict([('tweet', 'This is outrageous! Volunteers who have gone to NY to help fight the #COVID19 pandemic should not have to pay NY state taxes. @realDonaldTrump is correct, we should not bailout states that continue to have bloated, out of control budgets.\n\nhttps://t.co/NAdIRw0ia0'), ('dates', '2020-05-06 18:45:58'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.5843309760093689)])
OrderedDict([('tweet', 'Schiff joins a growing number of Democrats and liberal media pundits on the #CCP bandwagon. \n\nHe has moved from leading the Soviet-style shampeachment against @realDonaldTrump to protecting the Chinese coverup of #COVID19. https://t.co/zRa7AwAqIU'), ('dates', '2020-05-05 18:44:44'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.7799098491668701)])
OrderedDict([('tweet', 'It comes as no surprise that Weibo (Chinese twitter), which is essentially state-run media, is critical of @realDonaldTrump.\n\nPolitico is the latest to spread propaganda and spout #CCP talking points. https://t.co/uXuJLcjbuI'), ('dates', '2020-05-04 22:05:47'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.6682955026626587)])
OrderedDict([('tweet', 'Regardless of what Democrats and their media allies say, the second allocation of #PPP has processed 2.2 million loans with an average loan size of $79,000. This is another indicator that the program is assisting our struggling small businesses and not large corporations.'), ('dates', '2020-05-04 19:22:17'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8764213919639587)])
OrderedDict([('tweet', 'If Pelosi truly wanted bipartisan oversight of the #COVID__19 response, she would not have put Maxine Waters on the committee. Waters called for the impeachment of @POTUS just 2 weeks into his presidency. This is nothing but the next attempt to impeach @realDonaldTrump.'), ('dates', '2020-04-30 18:40:47'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9698731303215027)])
OrderedDict([('tweet', 'It is now abundantly clear that @GenFlynn was setup by Comey and others who were out to get @realDonaldTrump. Flynn served this country admirably. We must hold Comey and the politically driven cronies at the FBI accountable.'), ('dates', '2020-04-30 16:33:09'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8787376284599304)])
OrderedDict([('tweet', "The select committee on the #COVID19 response is yet another political hitjob on @realDonaldTrump. This committee would be led by Rep. Clyburn, the savior of Biden's campaign &amp; the Rep who said the coronavirus is a perfect time to restructure things to fit their liberal agenda."), ('dates', '2020-04-23 19:27:13'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8151911497116089)])
OrderedDict([('tweet', 'Instead of providing additional funds for the Paycheck Protection Program when @realDonaldTrump requested, Speaker Pelosi put politics over people and delayed the program an additional two weeks.  These #PelosiLayoffs could have been avoided. https://t.co/DIHr77T1Kh'), ('dates', '2020-04-23 14:32:05'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.6509319543838501)])
OrderedDict([('tweet', 'Last week, Speaker Pelosi stated that there was “no data” to prove small businesses need more funds from the #PaycheckProtectionProgram. Senate Dems then blocked additional funding, and today due to their negligence the program has run out of funds.'), ('dates', '2020-04-16 19:22:52'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.826015830039978)])
OrderedDict([('tweet', "Pelosi and Schumer have allowed the #PaycheckProtectionProgram to run out of money. I'm astounded that Democrats would play partisan games while everyday Americans are struggling. We must increase funding for the PPP immediately."), ('dates', '2020-04-16 16:15:54'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9629300832748413)])
OrderedDict([('tweet', 'Despite the significant demand for this program, the #PaycheckProtectionProgram is scheduled to run out of money this week. Speaker Pelosi and Leader Schumer continue to block more funding for this program that is providing emergency relief to the American people.'), ('dates', '2020-04-15 20:35:13'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.5898745059967041)])
OrderedDict([('tweet', 'Democrats should be ashamed that they continue blocking additional job-saving funding. We must pass a clean bill allocating more funds to the #PaycheckProtectionProgram. Last week alone, more than $200 billion was processed to protect over 15 million small business jobs.'), ('dates', '2020-04-13 18:28:32'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9004668593406677)])
OrderedDict([('tweet', 'Once again Democrats prefer to play politics with the #COVIDー19 response. We have small businesses that are depending on loans from the #PaycheckProtectionProgram, and we must ensure that they have the resources they need. We cannot afford more delays from Schumer and Pelosi. https://t.co/Gofn49Mkbl'), ('dates', '2020-04-09 15:59:51'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.5754472613334656)])
OrderedDict([('tweet', "Due to China's lies and misinformation, #COVIDー19 was allowed to spread unchecked. This is why I have joined my colleague @RepJimBanks in a resolution that calls on the #CCP to announce that the virus originated in China, end detainment of the Uyghurs &amp; end forced labor."), ('dates', '2020-04-01 18:19:22'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8767054677009583)])
OrderedDict([('tweet', 'The Chinese Communist Party is a totalitarian regime that has a history of deceit and oppression. It comes as no surprise that they have lied about the numbers of individuals affected by #COVIDー19. The #CCP and China must be held accountable.\n\nhttps://t.co/Ch0QgyFv4L'), ('dates', '2020-04-01 16:14:42'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9748817682266235)])
OrderedDict([('tweet', 'The third #COVID19 spending bill should have given full tax deductibility for contributions made to nonprofits. Our nonprofits are struggling, but Speaker Pelosi once again managed to put her interests ahead of the American people.\n\nhttps://t.co/saywiffFbw'), ('dates', '2020-03-31 20:39:27'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.652558445930481)])
OrderedDict([('tweet', "After supporting the first two #COVID19 bills, I could not support the third $2.2 trillion bill. Aid should be targeted &amp; only go to individuals who need it. It shouldn't go to gov bureaucrats who are still receiving their taxpayer-funded salaries and haven't lost their jobs."), ('dates', '2020-03-30 18:47:15'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.505241334438324)])
OrderedDict([('tweet', 'It seems like the Democrats see an opportunity in this crisis.\n \nThey are not here to help the American people, but to hold an emergency relief bill hostage until they get their radical wish list.\n\nUnbelievable.'), ('dates', '2020-03-23 21:18:34'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9030194282531738)])
OrderedDict([('tweet', 'Pelosi\'s "coronavirus" bill:\n-Bailout all current debt at USPS\n-Same-day voter registration \n-Expands union collective bargaining\n-Offsets airline emissions by 2025\n-$15 min wage\n\nWhile @senatemajldr is working on bipartisan solutions, Pelosi pushes her radical proposals.'), ('dates', '2020-03-23 21:03:56'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.7691571712493896)])
OrderedDict([('tweet', '#SenateDemocats are once again playing partisan games by blocking the #COVIDー19 supplemental bill. The lives and livelihood of Americans are at risk, but Democrats still refuse to act. The American people deserve better.'), ('dates', '2020-03-23 18:22:24'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9580385088920593)])
OrderedDict([('tweet', 'While people are suffering, Democrats are delaying the process &amp; politicizing a bill that should be used to support small businesses, health care professionals &amp; Americans who are struggling. Pelosi and Schumer should not use a pandemic to force through their liberal wishlist.'), ('dates', '2020-03-23 16:08:25'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9854534864425659)])
OrderedDict([('tweet', 'The short answer is NO @realDonaldTrump did not dissolve the pandemic response office. We must fight back against Chinese disinformation and complicit left-wing media outlets who are attempting to politize a pandemic.\n\nhttps://t.co/9fcUFJUgtr'), ('dates', '2020-03-16 21:10:54'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8881087303161621)])
OrderedDict([('tweet', 'Sen. Schumer is such a partisan hack. He and Speaker Pelosi are playing politics while @realDonaldTrump is trying to safeguard Americans. https://t.co/hyVeSZRwQ5'), ('dates', '2020-03-13 20:22:34'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.7833448648452759)])
OrderedDict([('tweet', 'The Chinese propaganda machine is at it again. Their campaign of disinformation is startling, and their communist government has covered up information and facts from the beginning of the outbreak. \n\nhttps://t.co/0QUIDXUv2h'), ('dates', '2020-03-13 15:32:13'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.710820734500885)])
OrderedDict([('tweet', '🚨Breaking🚨 House Democrats just voted overwhelmingly against a GOP motion claiming that Soleimnani posed a deadly threat to Americans. I would like to remind my colleagues that this is the terrorist who has the blood of more than 600 fallen soldiers on his hands. Unbelievable!'), ('dates', '2020-03-11 21:41:05'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8207206726074219)])
OrderedDict([('tweet', 'We cannot continue to allow sanctuary cities to blatantly disregard federal law. These lawless cities promote illegal immigration &amp; threaten American sovereignty. Thank you @realDonaldTrump for putting the safety of Americans first. https://t.co/iIOSzmcOw1'), ('dates', '2020-03-05 15:34:28'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.889483630657196)])
OrderedDict([('tweet', 'Great news! @WVGovernor signed the commonsense #BornAlive act into law. It is disappointing that House Dems refuse to protect the lives of babies born after a failed abortion. @realDonaldTrump, despite constant Democrat opposition, remains the strongest fighter for life! #ProLife'), ('dates', '2020-03-03 15:28:54'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9055771231651306)])
OrderedDict([('tweet', 'Medicare-for-all is a socialist takeover of our health care system that will eliminate employer-sponsored health insurance. It will cost taxpayers $32 trillion over 10 years &amp; will stifle our healthcare innovation, leading to longer wait times for everyday Americans.'), ('dates', '2020-02-28 21:06:58'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8503565788269043)])
OrderedDict([('tweet', "Further proof of the radicalism of House Dems: \nAOC continues to push the socialist Green New Deal on the House Floor. This policy would cost nearly $75,000 per household in year one &amp; would crush American families &amp; @realDonaldTrump's booming economy. \n\nhttps://t.co/Bwww3XqMN7"), ('dates', '2020-02-27 18:31:53'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.7613986134529114)])
OrderedDict([('tweet', 'BIG WIN for @realDonaldTrump, allowing him to withhold money from sanctuary cities breaking federal law. While these dangerous cities promote illegal immigration, @POTUS is keeping America safe by implementing strong immigration &amp; public safety policies.\n\nhttps://t.co/JG97iGe5dv'), ('dates', '2020-02-26 21:32:36'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.7992027401924133)])
OrderedDict([('tweet', 'Instead of bipartisanship, Democrats have pursued a baseless impeachment against @realDonaldTrump &amp; have pushed radical socialist proposals for their far-left base. Their policies of Medicare-for-all, the green new deal &amp; free college will bankrupt America &amp; destroy our economy.'), ('dates', '2020-02-26 18:26:40'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9984363913536072)])
OrderedDict([('tweet', "It is disheartening that Democrats don't want to provide care to babies who survive abortions. This is just the latest form of their increasingly radical leftist base. \n\nMeanwhile, @realDonaldTrump has stated that he wants to end late-term abortion. The contrast is shocking. https://t.co/q7LHT57ZIC"), ('dates', '2020-02-25 22:17:49'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9139373898506165)])
OrderedDict([('tweet', 'Pelosi refuses to even bother proposing a budget. She knows that the radical ideas of her far-left socialist base will bankrupt America. \n\nBottom line: DC Democrats think they know how to spend your money better than you do. They are the greatest threat to our booming economy. https://t.co/8K2ISGVeoJ'), ('dates', '2020-02-24 20:57:36'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8539496660232544)])
OrderedDict([('tweet', 'Democrats are attempting to take away our God-given rights protected in the second amendment. I will always stand firm against gun-grabbing measures by members of the Democratic left. I’m thankful @realDonaldTrump supports the rights enumerated in the Constitution. #2A'), ('dates', '2020-02-21 03:02:39'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9459056258201599)])
OrderedDict([('tweet', "While @realDonaldTrump continues to keep America safe, Democrats prefer to defend radical terrorists. @POTUS's actions have kept America strong and safe."), ('dates', '2020-02-20 23:57:15'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.7873956561088562)])
OrderedDict([('tweet', ".@realDonaldTrump's blue-collar boom is the largest economic expansion in history. @POTUS &amp; @VP have created thousands of jobs in our district alone. Their pro-growth and pro-jobs agenda has led to record low unemployment throughout Colorado and the USA. https://t.co/ddWh4RJmDj"), ('dates', '2020-02-20 20:07:58'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.5737977027893066)])
OrderedDict([('tweet', "The comparison between House Dems and @HouseGOP is stunning. Dems can't be bothered to propose a budget, because all they care about is removing @realDonaldTrump from office. Pelosi once said budget proposals show what you value, well I'm still waiting to see those values. https://t.co/Iu63hFWqHU"), ('dates', '2020-02-12 16:37:29'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8398985862731934)])
OrderedDict([('tweet', "For a second straight year, House Democrats have failed one of their primary responsibilities, producing a budget to fund the government. @realDonaldTrump's budget will restore fiscal responsibility while also strengthening our military and protecting our national security."), ('dates', '2020-02-11 17:32:43'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.6933975219726562)])
OrderedDict([('tweet', '.@SenCoryGardner is 100% right, House Dems should not be able to weaponize the impeachment process &amp; it should never be used for policy disagreements. @realDonaldTrump should not be impeached months before an election.'), ('dates', '2020-02-05 22:32:24'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8198941349983215)])
OrderedDict([('tweet', 'House Dems attempted to turn the world\'s most powerful legislative body into their personal impeachment forum. @senatemajldr is right, "the Framers knew the country would need a firewall to keep partisan flames from scorching our Republic." \n\n@realDonaldTrump has been acquitted'), ('dates', '2020-02-05 22:14:42'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.9124333262443542)])
OrderedDict([('tweet', 'On Article 2 of this #shampeachment, the Senate has voted that @realDonaldTrump is NOT GUILTY as charged. @POTUS is acquited on both charges. What a massive waste of time.'), ('dates', '2020-02-05 21:33:03'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.7754825353622437)])
OrderedDict([('tweet', 'BREAKING...The Senate has voted that @realDonaldTrump is NOT GUILTY as charged on Article 1 of this #shampeachment'), ('dates', '2020-02-05 21:20:09'), ('file_name', 'Doug Lamborn.csv'), ('pct_true', 0.8654607534408569)])
OrderedDict([('tweet', 'The Consumer Protections Against Surprise Medical Bills Act is a pro-patient bill. When families are faced with a medical crisis, the last thing you want to receive is a surprise bill in the mail.'), ('dates', '2020-02-14 15:28:40'), ('file_name', 'Dusty Johnson.csv'), ('pct_true', 0.617320716381073)])
OrderedDict([('tweet', 'Rep. Blumenauer on how House Transportation Bill creates "Un-Safe Routes to School" [VIDEO] http://t.co/QmFfiaEV'), ('dates', '2012-02-15 20:02:29'), ('file_name', 'Earl Blumenauer.csv'), ('pct_true', 0.6967880725860596)])
OrderedDict([('tweet', "The WHO covered-up for China and their suppression of the truth. I introduced legislation to investigate the WHO's COVID-19 response because the American people deserve answers and Congress must hold the WHO accountable. https://t.co/d4ZUi2f1E7"), ('dates', '2020-05-04 19:30:22'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.9701655507087708)])
OrderedDict([('tweet', 'Congress must investigate the extent of China’s deception and hold them accountable. The American people deserve answers!\n\nhttps://t.co/1553G0iYo7'), ('dates', '2020-04-28 21:25:39'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.8611434698104858)])
OrderedDict([('tweet', 'Democrats are holding critical support for small businesses hostage. Hardworking Americans need Congress to act NOW! https://t.co/Mb8E8DP6br'), ('dates', '2020-04-18 17:06:35'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.9569743275642395)])
OrderedDict([('tweet', 'Democrats are playing politics and holding critical funding for small businesses hostage. Congress needs to approve more funding for the Paycheck Protection Program now! https://t.co/rjXUwCEutI'), ('dates', '2020-04-17 02:35:45'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.9258666038513184)])
OrderedDict([('tweet', 'The WHO failed at responding to the pandemic and covered for China’s lies as they suppressed information about COVID-19. The world has now been turned upside down &amp; @realDonaldTrump made the right call to hold taxpayer funding until they are in a position to get their job done.'), ('dates', '2020-04-15 21:30:19'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.9325478076934814)])
OrderedDict([('tweet', 'The Paycheck Protection Program was created to help small businesses while keeping hardworking Americans employed. It has been so popular we will need to replenish the funding, but Democrats continue to play politics. We need to fund this lifeline for small businesses NOW! https://t.co/OfZ1GutkHT'), ('dates', '2020-04-14 22:39:19'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.5985097289085388)])
OrderedDict([('tweet', 'We are in the middle of a global pandemic, American lives have been completely disrupted, many workers don’t know where their next paycheck is coming from...and Democrats are using this health crisis to promote their liberal agenda. https://t.co/QIEZqmIvOV'), ('dates', '2020-03-24 01:57:01'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.9632536172866821)])
OrderedDict([('tweet', 'We should have voted on a funding bill last week, but Democrats needed more time to politicize this public health emergency. \n\nThis is not the time for political games. We must come together and pass a clean funding bill to address the #coronavirus ASAP. https://t.co/1fsS0iD6bb'), ('dates', '2020-03-04 17:05:13'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.8728716373443604)])
OrderedDict([('tweet', 'The impeachment sham is OVER and @realDonaldTrump is VINDICATED. The Senate did the right thing today. Now, it’s time to move on and get back to work for the American people. https://t.co/R8kNCJfv5y'), ('dates', '2020-02-05 22:43:01'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.7554526925086975)])
OrderedDict([('tweet', "House Democrats fundamentally failed to prove their impeachment case in the House. Now, they want to try again in the Senate. That's not how it works. https://t.co/SkVLERtzrw"), ('dates', '2020-01-22 20:27:02'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.7466979026794434)])
OrderedDict([('tweet', "Throughout the entire impeachment process, House Dems decried the urgency of impeaching @realDonaldTrump then let it sit on the Speaker's desk for a month.\n\nSince day one, the Dem's impeachment sham has been based on political games and emotions instead of fact. It's shameful. https://t.co/43nH6w7AJh"), ('dates', '2020-01-15 22:43:56'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.7589189410209656)])
OrderedDict([('tweet', 'Dems just passed a non-binding messaging resolution to rebuke @realDonaldTrump for eliminating a terrorist with American blood on his hands. We should show the world we are united and stand strong against the leading state sponsor of terrorism. Instead, Dems continue to divide.'), ('dates', '2020-01-09 23:43:34'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.9124775528907776)])
OrderedDict([('tweet', "There is no question about it, Soleimani was a terrorist with American blood on his hands. \n\nWhile Iran chanted 'Death to America,' previous administrations cowered. Instead, @realDonaldTrump is using a decisive and tactical strategy against escalating Iranian aggression."), ('dates', '2020-01-03 15:41:34'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.7799095511436462)])
OrderedDict([('tweet', 'Today is the day Washington Democrats have been waiting for since @realDonaldTrump was elected. They will finally bring their baseless articles of impeachment to the House floor. \n\nToday is a day that will give this country a black eye. https://t.co/PvdaoIgcjk'), ('dates', '2019-12-18 17:18:52'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.516553521156311)])
OrderedDict([('tweet', 'A reminder that Democrats have made it very clear why they want to vote to impeach @realDonaldTrump tomorrow. A direct quote from my colleague across the aisle - "I\'m concerned that if we don\'t impeach this president, he will get reelected." https://t.co/GlQZVcpcEd'), ('dates', '2019-12-17 23:25:36'), ('file_name', 'Earl L. "Buddy" Carter.csv'), ('pct_true', 0.5491253733634949)])
OrderedDict([('tweet', 'Just voted to pass and send straight to the President the bipartisan Senate-passed resolution “to direct the removal of United States Armed Forces from hostilities against the Islamic Republic of Iran that have not been authorized by Congress.” https://t.co/5ygSBYpTsw?'), ('dates', '2020-03-11 23:23:14'), ('file_name', 'Ed Case.csv'), ('pct_true', 0.6149325370788574)])
OrderedDict([('tweet', 'The House passed H.R. 8, the Bipartisan Background Checks Act which I cosponsored, over a year ago. Despite over 38,000 more Americans dead from gun violence since and public support at 90%, the Senate majority still won’t act. What a tragedy on so many levels. #EndGunViolence https://t.co/fg8AuTTeEU'), ('dates', '2020-03-02 16:49:48'), ('file_name', 'Ed Case.csv'), ('pct_true', 0.7643161416053772)])
OrderedDict([('tweet', 'The #COVID19 crisis has only exacerbated the public safety risk for cannabis businesses. And now many of them are facing disruptions in their supply chain &amp; normal operations. They should be eligible for relief just like any other legal, legitimate business during this pandemic. https://t.co/HFnF2HdfWe'), ('dates', '2020-05-08 16:55:20'), ('file_name', 'Ed Perlmutter.csv'), ('pct_true', 0.5081642866134644)])
OrderedDict([('tweet', 'Did you know President Trump’s new proposed budget cuts directly affect Medicare, Medicaid, and their safety net programs such as SNAP? #TrumpBudget'), ('dates', '2020-02-12 20:16:42'), ('file_name', 'Eddie Bernice Johnson.csv'), ('pct_true', 0.7220327258110046)])
OrderedDict([('tweet', 'The law is clear: President Trump does NOT have legal authority to block grant Medicaid. This latest attack on the ACA will only result in lower quality of care for fewer people. #ProtectOurCare'), ('dates', '2020-02-06 19:52:42'), ('file_name', 'Eddie Bernice Johnson.csv'), ('pct_true', 0.9149601459503174)])
OrderedDict([('tweet', 'Michael Flynn twice pleaded guilty to lying to the FBI, but Trump Attorney General Bill Barr is doing the President’s political bidding and dropping the charges. This assault on the rule of law is outrageous. \n\nBarr should resign. \nhttps://t.co/ByBZzhQZcs'), ('dates', '2020-05-08 15:50:17'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.804884672164917)])
OrderedDict([('tweet', "We fell short of overriding Trump’s veto, but the Senate signaled once again that only Congress has the power to decide when we go to war.\n\nI led the charge against Trump's unauthorized killing of Soleimani. I'll continue my fight to ensure Trump doesn’t start a war with Iran. https://t.co/tPqZiGmwCM"), ('dates', '2020-05-07 19:46:42'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.8730207085609436)])
OrderedDict([('tweet', 'President Trump and Sec Pompeo are eager to blame China about spread of #COVID19 but have been unwilling to provide basic information about their own response. If Pompeo has evidence for his claim that the coronavirus originated in a Chinese lab, he should present it immediately.'), ('dates', '2020-05-07 00:11:58'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.6717897057533264)])
OrderedDict([('tweet', 'Mohammad bin Salman jails his own people without cause and ordered the brutal murder of a U.S. resident. He should not get any U.S. help in his dangerous pursuit of a nuclear weapon. Glad the GAO recommends more transparency on U.S.-Saudi 123 nuclear cooperation agreements. https://t.co/TAIqTXrPav'), ('dates', '2020-05-05 14:29:13'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.5712771415710449)])
OrderedDict([('tweet', "We can’t let the pandemic and lack of monitoring manpower be an excuse for climate crimes. Temporarily cleaner air won't make up for lasting damage caused by accelerating deforestation.\n \nMy legislation would deter officials from recklessly enabling it. \nhttps://t.co/X1T67mrKn1 https://t.co/fUeF0qSHcF"), ('dates', '2020-05-01 19:35:22'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.6596251130104065)])
OrderedDict([('tweet', 'Trump’s fossil fuel cronies lobbied and are going to take money that was meant to help businesses survive the #COVID19 pandemic in order to bail themselves out of $200 billion in existing debt. It’s unacceptable, unjust, and we cannot let this stand.\nhttps://t.co/DLWNMAn2XZ'), ('dates', '2020-04-30 19:34:43'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.8278782367706299)])
OrderedDict([('tweet', 'Today, @RepRoKhanna and I introduced the No Unconstitutional War with North Korea Act to send the message that no matter who is at the helm in North Korea, President Trump must NOT return to his threats of “fire and fury.” A war with North Korea would be catastrophic.'), ('dates', '2020-04-28 18:40:25'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.6321662664413452)])
OrderedDict([('tweet', 'The Paycheck Protection Program is meant to be a lifeline for struggling small businesses, not an ATM for large corporations.\n\n@SenAmyKlobuchar and I are calling for strong supervision at Treasury to stop abuse and ensure money is only going to the businesses that truly need it.'), ('dates', '2020-04-27 15:52:30'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.5141663551330566)])
OrderedDict([('tweet', 'Rohingya refugees who fled atrocities in Burma are already vulnerable. #COVID19 makes the situation even more dire. Denying them access to info during COVID could mean denying them life-saving services/guidance. Bangladesh should restore telecommunications access in the camps.'), ('dates', '2020-04-24 21:02:30'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.5631078481674194)])
OrderedDict([('tweet', 'Months into the pandemic, Trump still hasn’t delivered necessary PPE to frontline workers.\n\nJust 24 hours after oil markets tank, he’s already drawing up a bailout.\n\nWe cannot rescue reckless execs because of Trump’s corrupt priorities. We must protect workers.\n\n#NoBigOilBailout. https://t.co/Bgm7Q8tPhH'), ('dates', '2020-04-21 15:32:30'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.7223609089851379)])
OrderedDict([('tweet', 'Venezuela is undergoing a devastating human rights crisis which the onset of #COVID19 could make catastrophic. Yet the Trump administration has inexcusably failed to protect the approximately 200,000 Venezuelan nationals who could benefit from TPS in the U.S. #VenezuelaTPSNow'), ('dates', '2020-04-15 18:13:16'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.9583821892738342)])
OrderedDict([('tweet', 'Countries must not use #COVID19 as a cover to persecute LGBTI people. I denounce these attempts by the Ugandan government to target the LGBTI community and call on the Ugandan government to immediately release people who have been wrongfully imprisoned.\nhttps://t.co/hV046iTPo9'), ('dates', '2020-04-14 17:46:12'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.5432108640670776)])
OrderedDict([('tweet', 'The response to #COVID19 will shape our society for decades.\n\nThe wealthy have made their priorities clear: consolidate power and leave working people behind.\n\nThose fighting for the health and economic security of ordinary Americans must also make our priorities clear. \n\nTHREAD: https://t.co/rvf0scUUjI'), ('dates', '2020-04-09 23:28:16'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.5741710662841797)])
OrderedDict([('tweet', 'The American people deserve to know how the government and companies are using their data to address the #COVID19 pandemic. Congress must enact privacy protections because in moments of crisis, we can’t disregard or tamper with Americans’ fundamental civil liberties.'), ('dates', '2020-04-09 16:02:21'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.960922122001648)])
OrderedDict([('tweet', "We shouldn't accept the declarations by some in power who say we have to stray from the guiding principles &amp; civil liberties that make us who we are. They are wrong. We don't have to forgo all privacy in a pandemic nor watch a surveillance state take root.\nhttps://t.co/1Rx6j6TzpX"), ('dates', '2020-04-08 19:18:06'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.9014105796813965)])
OrderedDict([('tweet', 'As authoritarians exploit the pandemic to erode rule of law, we must continue to call out Beijing’s efforts to assert unlawful maritime claims in the South China Sea. The United States will not turn a blind eye to military coercion of Southeast Asian countries. https://t.co/C06UWs7Cnt'), ('dates', '2020-04-06 21:46:34'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.6199958324432373)])
OrderedDict([('tweet', 'Gutting clean air &amp; water rules would kill countless Americans under normal circumstances. But the Trump admin would add to the already horrifying death count of #COVID19 by using it as cover to attack environmental safeguards.\n\n@SenWarren &amp; I want answers\nhttps://t.co/IqUfaQY170'), ('dates', '2020-04-06 15:50:58'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.519638180732727)])
OrderedDict([('tweet', 'People with underlying health conditions like asthma are at a higher risk from #COVID19. The Trump EPA needs to halt all regulatory rollbacks that make air quality worse and could make people more exposed to pollution and toxic chemicals, especially as we fight this pandemic.'), ('dates', '2020-04-03 18:08:00'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.6400964260101318)])
OrderedDict([('tweet', 'TV providers use a lack of transparency to gouge consumers with hidden fees to the tune of $450/year per consumer. @RepAnnaEshoo &amp; I urge the FCC to enforce our law so consumers can know exactly what they are signing up for, &amp; can change their mind without exorbitant penalties. https://t.co/S0lq7YotWM'), ('dates', '2020-04-03 16:53:57'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.6196087598800659)])
OrderedDict([('tweet', '“Accidentally” sending medical equipment and protective gear overseas while he denied the threat of #COVID19 isn’t Trump incompetence, it’s Trump criminality. Instead of mobilizing industry to make equipment, Trump makes excuses. And people are dying.  \nhttps://t.co/F2XmTR7cVJ'), ('dates', '2020-04-03 01:24:41'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.5751984119415283)])
OrderedDict([('tweet', 'As Americans practice social distancing, President Trump is bringing the Big Oil CEOs together to the White House for a closed door meeting. We cannot allow them to coordinate a Big Oil bailout or abuse of the $500 billion corporate slush fund.\nhttps://t.co/QH93Q9wDUr'), ('dates', '2020-04-02 14:19:52'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.5144889950752258)])
OrderedDict([('tweet', 'President Trump has repeatedly misled Americans about #COVID19 for political gain. The public needs unbiased information. @SenSanders, @SenWarren, and I are asking the National Academy of Sciences to serve as an apolitical source of information outside of executive influence. https://t.co/q8KXGZwl23'), ('dates', '2020-04-01 22:18:26'), ('file_name', 'Edward J. Markey.csv'), ('pct_true', 0.5170512795448303)])
OrderedDict([('tweet', 'I support the Iran War Powers Resolution the Senate voted on today, but DC residents had no vote on this matter of war and peace, even though our residents have always funded and fought in our nation’s wars. #DCStatehood'), ('dates', '2020-05-07 23:14:27'), ('file_name', 'Eleanor Holmes Norton.csv'), ('pct_true', 0.7014224529266357)])
OrderedDict([('tweet', 'I wrote AG Barr protesting his callous treatment of immigration employees by not allowing them to #telework like most federal employees. \n\nPartial closure of office 4 times in 2 weeks for #COVID19 and many violations of CDC guidelines demand telework for employee safety.'), ('dates', '2020-04-30 12:36:43'), ('file_name', 'Eleanor Holmes Norton.csv'), ('pct_true', 0.6831779479980469)])
OrderedDict([('tweet', 'At her press conference today, @SpeakerPelosi said #DC was “cruelly discriminated against” when the 3rd Republican-initiated #COVIDー19 bill deliberately shorted DC by $750 million.\n\nHer continuing, spontaneous outspoken support will help me retrieve our funds.'), ('dates', '2020-04-02 21:46:41'), ('file_name', 'Eleanor Holmes Norton.csv'), ('pct_true', 0.7132779955863953)])
OrderedDict([('tweet', 'This is why we must always put workers first -- Senate and House Democrats have expanded unemployment benefits and direct cash assistance for Americans. \n\nNow, I am fighting to ensure D.C. is NOT left out of full funding in the next package.\n\n#FundDCtoFight coronavirus. https://t.co/IwnmvVKlce'), ('dates', '2020-03-30 19:55:24'), ('file_name', 'Eleanor Holmes Norton.csv'), ('pct_true', 0.6288105249404907)])
OrderedDict([('tweet', 'Under the House version of the 3rd coronavirus bill, D.C. would get $11 million in new security funding. \n\nIn these unprecedented times, we appreciate that our security needs are being addressed.\n\nThis gives us a head start in getting these funds into the final bill.'), ('dates', '2020-03-24 16:54:31'), ('file_name', 'Eleanor Holmes Norton.csv'), ('pct_true', 0.6268894076347351)])
OrderedDict([('tweet', '(1 of 3): One month ago, the House Oversight Committee voted to send the #DCstatehood bill to the House floor for the first time since 1993, when I got the first-ever floor vote on the bill in the House.'), ('dates', '2020-03-11 13:17:00'), ('file_name', 'Eleanor Holmes Norton.csv'), ('pct_true', 0.6868144273757935)])
OrderedDict([('tweet', 'This loss to DC is a loss to the American people: The Bureau of Land Management has lost more than half its Washington area employees who were needlessly slated to move out West to enforce a controversial plan to relocate staff. https://t.co/RccQhtebLm'), ('dates', '2020-03-06 20:16:58'), ('file_name', 'Eleanor Holmes Norton.csv'), ('pct_true', 0.5042118430137634)])
OrderedDict([('tweet', 'This recent incident proved Trump’s words have serious public health consequences. While it’s good that we passed my poison control center funding bill last year, they should not be fixing his mistakes. https://t.co/2R1G8Z3Mvl'), ('dates', '2020-05-01 15:14:46'), ('file_name', 'Eliot L. Engel.csv'), ('pct_true', 0.521699070930481)])
OrderedDict([('tweet', 'This is beyond irresponsible by the Arizona #GOP. \n\nTrying to confuse the media and public during a pandemic for political gain is bad enough—trying to do it by mimicking front-line health workers is abhorrent. \n\nNational @GOP should condemn this disgusting tactic. https://t.co/t4SQz7KaLF'), ('dates', '2020-04-27 16:18:37'), ('file_name', 'Eliot L. Engel.csv'), ('pct_true', 0.754766583442688)])
OrderedDict([('tweet', 'This is classic @RealDonaldTrump. Seeking to claim credit for the #COVID19 checks by ordering his name signed where the memo should be. This vanity act nearly upended the process, and for what? @POTUS needs to stop playing these shameful games with people’s lives. #stimuluscheck https://t.co/6qWAJIuGfJ'), ('dates', '2020-04-16 19:48:43'), ('file_name', 'Eliot L. Engel.csv'), ('pct_true', 0.7727072238922119)])
OrderedDict([('tweet', 'This was never a hoax or conspiracy. It was a dire threat to this country that the President ignored. That’s the reason the US is now the epicenter of this crisis. \n\nBlaming the World Health Organization will accomplish nothing &amp; cutting off resources will only make things worse. https://t.co/SLdFq6ujb9'), ('dates', '2020-04-15 01:01:29'), ('file_name', 'Eliot L. Engel.csv'), ('pct_true', 0.7969872355461121)])
OrderedDict([('tweet', 'The administration has the authority to stop vital protective equipment from flowing overseas. But the President hasn’t done enough and is putting American lives at risk.\n \nhttps://t.co/UqUAnzzsvp'), ('dates', '2020-04-04 17:42:01'), ('file_name', 'Eliot L. Engel.csv'), ('pct_true', 0.5250097513198853)])
OrderedDict([('tweet', 'During a #publichealth crisis, the #Trump Administration is gutting clean car standards which will dramatically increase air pollution and harm public health. \n\nOnce again, the #GOP is putting oil industry profits ahead of the American people. Shameful. https://t.co/dcRdJR7VLv'), ('dates', '2020-03-31 21:16:50'), ('file_name', 'Eliot L. Engel.csv'), ('pct_true', 0.6960713267326355)])
OrderedDict([('tweet', 'The Communist Party of China intentionally withheld critical information on the spread of #COVID19 that could have saved thousands of lives. And to this day, China continues to spread misinformation regarding this health crisis.'), ('dates', '2020-05-07 15:15:00'), ('file_name', 'Elise M. Stefanik.csv'), ('pct_true', 0.9324485063552856)])
OrderedDict([('tweet', 'It is inexcusable that Speaker Pelosi and Senator Schumer blocked critical funding for our small businesses who are facing unprecedented challenges during #COVID19.  I urge Democrats to quit playing political games with the livelihoods of so many hardworking families in #NY21.'), ('dates', '2020-04-16 13:40:49'), ('file_name', 'Elise M. Stefanik.csv'), ('pct_true', 0.9063764214515686)])
OrderedDict([('tweet', "The President's veto of this legislation is disappointing but not surprising.\n\nWhat is more important, and will have lasting impact, is that Congress has taken a bipartisan step to reassert its constitutional role in matters of peace and war. https://t.co/8AKkyh07rH"), ('dates', '2020-05-07 00:26:41'), ('file_name', 'Elissa Slotkin.csv'), ('pct_true', 0.8229444026947021)])
OrderedDict([('tweet', 'Mr. President, as you consider efforts to secure our supply chain, don’t make this a choice between punishing China or placating China.\n\nWe should care less about what China thinks and more about what helps keep the American people safe. https://t.co/WKZZqIV5n0'), ('dates', '2020-04-29 20:15:26'), ('file_name', 'Elissa Slotkin.csv'), ('pct_true', 0.7783554196357727)])
OrderedDict([('tweet', 'I believe strongly in the right of all Americans to protest their government, and that includes today’s protesters in their cars in Lansing –– even if I don’t agree with their premise.'), ('dates', '2020-04-15 23:50:52'), ('file_name', 'Elissa Slotkin.csv'), ('pct_true', 0.5937196016311646)])
OrderedDict([('tweet', 'Gov. Gretchen Whitmer has done nothing but stand up for people of our state, especially our front-line health providers. It’s troubling for the president to suggest that his administration would be less responsive to any state because of differences with its governor.'), ('dates', '2020-03-28 01:32:45'), ('file_name', 'Elissa Slotkin.csv'), ('pct_true', 0.525761067867279)])
OrderedDict([('tweet', 'Walmart has a responsibility to keep its workers safe from COVID-19, and it failed in Massachusetts. The entire MA delegation &amp; I want @Walmart to explain what changes the retail giant will be making to prevent more store outbreaks across the Commonwealth. https://t.co/N1CFyPUbq8'), ('dates', '2020-05-09 15:05:19'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.5592358112335205)])
OrderedDict([('tweet', 'Handing out a $500 million paint job contract to a supporter is corruption. And Trump’s monument to hate at the border – in black or any other color – is a giant waste of money while Americans are dying from this administration’s failures. https://t.co/bkw2tKQyOM'), ('dates', '2020-05-08 00:32:38'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.8445870876312256)])
OrderedDict([('tweet', 'William Barr is not the President’s lawyer – he is the country’s lawyer. His political interference is blatantly corrupt and he should resign.'), ('dates', '2020-05-07 22:20:03'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.6943982243537903)])
OrderedDict([('tweet', 'Amazon has fired at least 4 workers who raised concerns about COVID-19 safety in Amazon warehouses. This looks like a clear case of whistleblower retaliation. My Senate colleagues &amp; I want @Amazon to show us their policies that workers allegedly violated. https://t.co/MlTAzEXkwh'), ('dates', '2020-05-07 18:44:47'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.7370976805686951)])
OrderedDict([('tweet', "Today the Supreme Court will hear arguments on the Trump-Pence rules that would let virtually any employer be exempted from the ACA's birth control benefit. We cannot allow the administration to roll back the clock &amp; undermine people’s right to basic health care. #HandsOffMyBC https://t.co/ZsSixVhzuD"), ('dates', '2020-05-06 14:23:18'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.808082103729248)])
OrderedDict([('tweet', 'At least 81 Worcester Walmart employees have COVID-19. Now at least 10 Walmart employees in Quincy have COVID-19 too &amp; one worker has died. Whatever @Walmart claims they are doing to protect the health &amp; safety of their workers is clearly not enough. https://t.co/msq81dMfjw'), ('dates', '2020-05-06 00:22:37'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.515657901763916)])
OrderedDict([('tweet', "On Friday, 23 Worcester @Walmart employees had tested positive for COVID-19. Now they're up to 81. This is why Congress must pass an Essential Workers Bill of Rights: because workers can’t rely on big business to protect them. #StandWithEssentialWorkers https://t.co/nW5CIUQjZ7"), ('dates', '2020-05-03 23:19:51'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.5415351986885071)])
OrderedDict([('tweet', "We know COVID-19 has ravaged nursing homes. But assisted living facilities have many similar risks &amp; fewer rules &amp; reporting requirements. @SenMarkey, @RepMaloney &amp; I want assisted living companies to tell us how they're protecting residents from COVID-19. https://t.co/euDgMCggoM"), ('dates', '2020-05-01 20:19:03'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.9123181104660034)])
OrderedDict([('tweet', 'President Trump has made it clear that he’s putting himself &amp; his rich buddies first during this pandemic. @SenBlumenthal, @RepJayapal, @RepSarbanes, &amp; I have laid out strong oversight &amp; anti-corruption provisions for the next coronavirus relief package. https://t.co/GKToRDg4iD https://t.co/lilaMVwa7t'), ('dates', '2020-05-01 17:29:31'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.813403844833374)])
OrderedDict([('tweet', 'This is unacceptable. My colleagues &amp; I have repeatedly urged the Trump admin not to leave Indian Country behind. The federal government is failing tribal nations &amp; falling far short of its legal &amp; moral obligations. This desperately needed relief must be sent without more delay. https://t.co/6Z0inSSEG6'), ('dates', '2020-04-30 21:51:08'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.7551679611206055)])
OrderedDict([('tweet', 'President Trump still doesn’t have a plan to get our health care workers the COVID-19 supplies they need. But @JanSchakowsky &amp; I do. We have a bill to publicly manufacture PPE, drugs, &amp; other medical supplies needed to fight this pandemic. https://t.co/XmNue8wKTJ'), ('dates', '2020-04-30 16:58:13'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.5875323414802551)])
OrderedDict([('tweet', 'We can’t allow our economic recovery to become one more rigged game for the rich &amp; powerful. That’s why @AOC &amp; I have a bill to stop predatory mergers &amp; acquisitions during the pandemic to help level the playing field for small businesses. https://t.co/d7gjzmnFwK'), ('dates', '2020-04-30 00:38:08'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.7657148838043213)])
OrderedDict([('tweet', 'My new bill with @AOC is pretty simple: giant corporations &amp; private equity firms shouldn’t be able to exploit this crisis to kill small companies &amp; startups. And you know what? The overwhelming majority of Americans agree. https://t.co/j9glfQQdAH'), ('dates', '2020-04-29 01:48:57'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.5049210786819458)])
OrderedDict([('tweet', 'The international prohibition on weapons going to/from Iran ends in October. To extend this arms embargo, the Trump admin is suddenly arguing that the US is a party to the same Iran Deal it abandoned. That makes no sense. Make up your mind, @SecPompeo. https://t.co/cQ0uanYEYr'), ('dates', '2020-04-28 23:46:29'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.7359339594841003)])
OrderedDict([('tweet', "The @DeptofDefense's failure to respond quickly &amp; aggressively to COVID-19 has put our servicemembers at risk, strained military families, hurt morale, &amp; undermined military readiness. The Pentagon must do better, Secretary @EsperDoD. https://t.co/jtm2S28jHm"), ('dates', '2020-04-27 21:49:06'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.6963661909103394)])
OrderedDict([('tweet', "It's outrageous that we don't know what secret favors Trump might be getting from @DeutscheBank, or what favors the bank may be getting in return. The American people need to know the President is working for them, not for his personal financial interests. https://t.co/QABExjTZFa"), ('dates', '2020-04-26 14:07:31'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.9396162033081055)])
OrderedDict([('tweet', '26 million people are unemployed. Frontline workers are wearing garbage bags. But the Pentagon can divert billions to Trump’s racist border wall &amp; nobody blinks an eye. I bet the Pentagon has some money in the couch cushions for masks or tests. Have they tried looking? https://t.co/L1Fh9it9Ic'), ('dates', '2020-04-25 17:48:52'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.9139280319213867)])
OrderedDict([('tweet', 'The PPP was created to help devastated small businesses, but banks are playing favorites, serving the wealthy &amp; well-connected first &amp; leaving mom-and-pops behind. @NydiaVelazquez I want the @SBAOIG &amp; @USTreasury OIG to investigate this waste &amp; abuse. https://t.co/GXpEYqgJIe'), ('dates', '2020-04-24 20:39:28'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.7683876752853394)])
OrderedDict([('tweet', 'We’re in a public health &amp; economic crisis. Families &amp; small businesses need help. But the @DeptofDefense gave out a $569M contract to a GOP donor to build the President’s monument to hate at the border. @BennieGThompson &amp; I want @DoD_IG to investigate. https://t.co/ozN0y1yaMP'), ('dates', '2020-04-24 20:11:41'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.7977089285850525)])
OrderedDict([('tweet', "Trump fired the US’s top vaccine scientist for calling baloney on the President's unproven miracle cure for COVID-19. We should be listening to scientists, not Trump's cronies. Americans are dying because of the President’s corruption &amp; incompetence. https://t.co/Zpth6f4Nn9"), ('dates', '2020-04-22 23:37:15'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.8927790522575378)])
OrderedDict([('tweet', "Trump shouldn’t be using his 2020 electoral map to make life-or-death decisions &amp; distributing medical supplies based on which friends &amp; allies need a favor. We want an investigation into Trump's bungled COVID-19 response. https://t.co/qkd0Ko6ZQu"), ('dates', '2020-04-22 15:01:29'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.5537071228027344)])
OrderedDict([('tweet', 'Massachusetts health providers shouldn’t have to hide their masks in food service trucks &amp; split up on the highway so federal agents don’t seize their shipments. The @NEJM shouldn’t sound like a Hollywood movie script. This has to stop, President Trump. https://t.co/xWoxRkOOsn'), ('dates', '2020-04-20 14:18:25'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.6083512902259827)])
OrderedDict([('tweet', 'MA has the third most COVID-19 cases in the country. Our cities &amp; counties need help. But the Fed’s lending program will only send money to cities with 1 million people or more, so Boston (and every other MA city &amp; town) don’t qualify. This has to change. https://t.co/18wlnpdFfk'), ('dates', '2020-04-18 20:28:59'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.5231926441192627)])
OrderedDict([('tweet', "It’s 200 days until Election Day &amp; Congress hasn’t addressed the health &amp; safety of America’s voters during this pandemic. There's no time for partisanship. I wrote a plan 10 days ago to protect our elections. Congress should commit to these proposals now. https://t.co/Vzb42seNnQ"), ('dates', '2020-04-17 18:15:31'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.6810279488563538)])
OrderedDict([('tweet', "This is an important report on how companies like 3M &amp; Honeywell are influencing the @USChamber to push Trump to not use the DPA to produce protective equipment. The Chamber must stop putting members' short-term profits above the nation's medical needs. https://t.co/j7oWgw6oCe"), ('dates', '2020-04-17 15:57:14'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.7391917705535889)])
OrderedDict([('tweet', "Yet again, the Trump admin is putting profits of @EPAAWheeler's former &amp; future lobbying clients ahead of communities' health &amp; safety. This time, during a pandemic, they're gutting mercury pollution safeguards that would save thousands of lives each year. https://t.co/9MGT3rxCVx"), ('dates', '2020-04-17 00:18:11'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.5271376967430115)])
OrderedDict([('tweet', "Trump, Jared Kushner, &amp; his shadow task force of corporate cronies shouldn’t be making COVID-19 policy decisions that line their pockets. It's obscene that the Trump family &amp; those close to them still won’t put the country’s interests before their own. https://t.co/HQEypzvBux"), ('dates', '2020-04-16 17:23:44'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.733355700969696)])
OrderedDict([('tweet', 'President Trump’s multitude of failures have resulted in a public health catastrophe &amp; an unimaginable economic collapse. Scapegoating the @WHO, pulling public health funds from countries in need, &amp; ceding our global leadership to China will not change those facts. https://t.co/vSeZ18Kf2G'), ('dates', '2020-04-15 01:30:16'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.925892174243927)])
OrderedDict([('tweet', "COVID-19 doesn't discriminate, but generations of structural racism have left people of color more vulnerable to this virus. The CDC should #releasethedata – &amp; @RepPressley, @RepKarenBass, @RepRobinKelly, @RepBarbaraLee &amp; I have a new bill to mandate it. https://t.co/BOW6wKGCFc"), ('dates', '2020-04-14 20:20:21'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.7386595606803894)])
OrderedDict([('tweet', 'Even as research shows air pollution worsens the deadly effects of the coronavirus, Trump &amp; former coal lobbyist @EPAAWheeler are working to weaken clean air safeguards &amp; let polluters keep polluting. This is unacceptable &amp; puts even more lives at risk. https://t.co/RxmG7Nk2Ec'), ('dates', '2020-04-14 18:19:28'), ('file_name', 'Elizabeth Warren.csv'), ('pct_true', 0.5305461287498474)])
OrderedDict([('tweet', 'Despite a global pandemic which has taken the lives of over 75,000 people, the President STILL wants to take health care away from more than 20 million Americans.\n \nThe height of irresponsibility.\nhttps://t.co/SZ5kpenQw8'), ('dates', '2020-05-08 16:24:20'), ('file_name', 'Emanuel Cleaver.csv'), ('pct_true', 0.5757097005844116)])
OrderedDict([('tweet', 'The @USPS is the most popular government agency, ensures seniors and veterans can receive life-saving medications, employs 650,000 Americans, and costs the taxpayers nothing.\n \nWhy does the GOP want to kill it? https://t.co/cas2C9Lezh'), ('dates', '2020-05-04 17:42:18'), ('file_name', 'Emanuel Cleaver.csv'), ('pct_true', 0.6340200304985046)])
OrderedDict([('tweet', '30 million Americans have lost their jobs in the past 6 weeks.\n \n1 in 5 American workers are currently out of a job.\n \nFood banks are seeing mile-long lines.\n \nCongress and the Administration must do more. https://t.co/KNlFGXCvtM'), ('dates', '2020-04-30 15:50:18'), ('file_name', 'Emanuel Cleaver.csv'), ('pct_true', 0.633481502532959)])
OrderedDict([('tweet', 'We cannot allow #coronavirus relief to become a partisan circus as the Senate Majority Leader would like.\n \nMissouri needs relief just like New York and Florida and California and Michigan.\n \nEnough with the politics, we must work together #ForThePeople\nhttps://t.co/WUsCzdl0Wm'), ('dates', '2020-04-24 16:05:26'), ('file_name', 'Emanuel Cleaver.csv'), ('pct_true', 0.5325385928153992)])
OrderedDict([('tweet', 'The Senate Majority Leader has no problem bailing out corporations, but when it comes to state and local governments—you know, the people who pay our EMTs, firefighters, and teachers—well, good luck on your own. https://t.co/VwVRF8isqe'), ('dates', '2020-04-22 18:55:52'), ('file_name', 'Emanuel Cleaver.csv'), ('pct_true', 0.5712290406227112)])
OrderedDict([('tweet', 'Minority-owned small businesses make up HALF of all small businesses created in the last decade. They are a critical component of the American economy, and they deserve their fair share of the #PaycheckProtectionProgram! https://t.co/pP4dea33Y7'), ('dates', '2020-04-21 17:59:53'), ('file_name', 'Emanuel Cleaver.csv'), ('pct_true', 0.5108979344367981)])
OrderedDict([('tweet', 'After the 2008 crisis, 3.5 million Social Security beneficiaries and veterans did not receive their stimulus payments because of bureaucratic red tape. Over 120 of my colleagues and I urged the Trump admin not to make the same mistake during the #COVID19 Crisis. https://t.co/dBUL6cslSt'), ('dates', '2020-04-06 16:31:27'), ('file_name', 'Emanuel Cleaver.csv'), ('pct_true', 0.7826606035232544)])
OrderedDict([('tweet', 'Democrats are fighting to ensure that workers across America are protected during this public health crisis. We will continue to fight for common sense unemployment insurance that helps Americans weather the storm.\nhttps://t.co/QGVvehKcXO'), ('dates', '2020-04-06 15:24:28'), ('file_name', 'Emanuel Cleaver.csv'), ('pct_true', 0.5210731625556946)])
OrderedDict([('tweet', 'These numbers are staggering, and why Democrats fought so hard to beef up unemployment insurance in the CARES Act. It’s critical that we continue to get aid to communities ASAP. Also clear we need to prepare for the 4th phase of #coronavirus relief right now. https://t.co/NsC1uftVjS'), ('dates', '2020-04-02 18:51:38'), ('file_name', 'Emanuel Cleaver.csv'), ('pct_true', 0.6403543949127197)])
OrderedDict([('tweet', 'The Senate’s initial #CARES Act was a trickle-down corporate relief package. Democrats fought for and won crucial changes that put workers and #FamiliesFirst\nhttps://t.co/2WS2Uv9AxN'), ('dates', '2020-03-30 18:52:33'), ('file_name', 'Emanuel Cleaver.csv'), ('pct_true', 0.5742014050483704)])
OrderedDict([('tweet', 'This is welcome news for US farmers! However, we are not out of the woods and we should continue to ensure that China upholds their end of the trade deal. #Trade #China https://t.co/dZo5pHCg1w'), ('dates', '2020-05-08 18:17:07'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.8864948153495789)])
OrderedDict([('tweet', '. @phillyrich1 has put together a timeline of #COVIDー19 that everyone should take a look at. China was buying *so much* PPE, they actually had a trade deficit. For those who choose to run their propaganda about what has taken place, history is watching:\n\nhttps://t.co/cophw5fdaX https://t.co/ghxqGu8zgz'), ('dates', '2020-05-04 23:11:12'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.736178994178772)])
OrderedDict([('tweet', '. @GOPLeader outlines what has become painfully clear over the last several weeks by Communist leaders in China. Their motives have been clear and so must our resolve be as equally determined. Our collective safety and national security depend on it!\nhttps://t.co/1EEzNndzrf'), ('dates', '2020-04-28 00:24:13'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.5966213345527649)])
OrderedDict([('tweet', 'For years, misguided people, groups, and even some nations, have actively advocated for and participated in the boycott, divest, and sanction - BDS - of Israel. Their motives are punitive and based largely on their own anti-semitism. China, on the other hand... (1/x)'), ('dates', '2020-04-26 15:44:42'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.8087969422340393)])
OrderedDict([('tweet', 'The President and @stevenmnuchin1 are exactly right about asking for money back from big companies and entities like Harvard University who have significant means of funding that small businesses do not. Taking advantage of a program isn’t good form - leave it to the small ones.'), ('dates', '2020-04-22 12:19:02'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.8056914210319519)])
OrderedDict([('tweet', 'Just over six months ago, I gave an interview about China and challenges I brought up about the integrity of our supply chain. I also mentioned the challenges posed by Chinese nationals who had infiltrated higher learning institutions...'), ('dates', '2020-04-16 22:53:13'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.7442640066146851)])
OrderedDict([('tweet', 'I guess the Speaker and Chuck Schumer will be okay with this and playing an active role in denying funds to business owners everywhere. And, for what? Political points they think they can score? Ruining someone’s livelihood must be “acceptable” to them. https://t.co/FfWZNuhrpQ'), ('dates', '2020-04-16 21:37:45'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.5529345870018005)])
OrderedDict([('tweet', 'America, here is your split screen as we fight the affects of #Covid_19 with political elites who are happier with a freezer full of ice cream than they are with business owners staying afloat. https://t.co/Xv88v9PTrc'), ('dates', '2020-04-16 15:30:26'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.6155203580856323)])
OrderedDict([('tweet', '.@RepJimBanks has introduced legislation to stop this from happening, but it likely will need action from the President to stop this from happening. It’s unconscionable we would be investing in Chinese companies with all we know. This must stop!\nhttps://t.co/NXGj55L8kl'), ('dates', '2020-04-16 14:46:34'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.5283101797103882)])
OrderedDict([('tweet', 'The President is making the right call to halt funding to @WHO in light of misleading information they pushed out and the enabling of the Chinese government they have done to cover up the origins and nature of the #COVID19 virus.'), ('dates', '2020-04-14 22:23:48'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.6591021418571472)])
OrderedDict([('tweet', 'Yet again, Senate Democrats are holding up important funds for small business owners across America. I know that several hundred applications are trying to be filed in AR1 and yet they want to stall. Shameful and disgusting. Leave it for another day, @SenSchumer.'), ('dates', '2020-04-09 15:41:32'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.5002947449684143)])
OrderedDict([('tweet', '. @SenTomCotton was admonished and ridiculed for what has (sadly) come true with #Covid_19. I agreed with him that news coming out of China was suspect and should not be trusted, at all. Now, more evidence he wasn’t peddling “conspiracy theories”\nhttps://t.co/NPHqmvuiil'), ('dates', '2020-04-04 00:12:03'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.5574938058853149)])
OrderedDict([('tweet', 'If the Speaker cares one bit about our country and the raging #COVIDー19 pandemic, she will pause her so-called select commission on the President (political games/theater) and instead focus on the real threat to our country/world - the Chinese Government:\nhttps://t.co/hNggddAk3r'), ('dates', '2020-04-03 13:47:30'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.8521587252616882)])
OrderedDict([('tweet', 'The Speaker is failing yet again at leading the People’s House at a time of crisis. Instead of a partisan commission to oversee the Exec Branch/Agencies (House Committees already do this function), she instead needs to focus solely on our main threat and that of the world...'), ('dates', '2020-04-02 16:34:19'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.7216376066207886)])
OrderedDict([('tweet', 'Today’s jobless claims reminds us that the veracity and economic lethality that #COVIDー19 has unleashed knows no boundaries or demographics it will not damage. We must all pull together in this fight so our communities, families, and friends have a better future.#InItTogether'), ('dates', '2020-04-02 13:39:21'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.9230668544769287)])
OrderedDict([('tweet', '. @WHO you failed the entire world when you peddled these lies. You’ve continued to be a propaganda arm for China at a time when countless people are dying all over the world. Never again will your words matter for the safety and protection of mankind. https://t.co/fqJZrRnVJy'), ('dates', '2020-04-02 11:53:21'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.7957696914672852)])
OrderedDict([('tweet', 'It’s time we fully investigate the extent by which China has lied, covered up, and allowed the spread of the #COVIDー19 virus to ravage the world.\n\nI want to establish a Select Commission on China so we can fully expose the threat they pose to America and the world. https://t.co/pfOZgoliRh'), ('dates', '2020-04-01 16:15:35'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.9319846034049988)])
OrderedDict([('tweet', 'Sadly, there are already unscrupulous individuals who are trying to take advantage of those who have had damage to homes/structures. There is no such thing as “FEMA Approved” and anyone who contends this should be reported. (1/3)'), ('dates', '2020-03-29 14:20:51'), ('file_name', 'Eric A. "Rick" Crawford.csv'), ('pct_true', 0.5537090301513672)])
OrderedDict([('tweet', 'What were we up against as Intel @HouseDemocrats sought to understand Trump-Russia connections? People like Trey Gowdy telling Jared Kushner he didn’t have to answer our questions. Then, surprise, Kushner stops. @HouseIntel transcripts. https://t.co/ODBtjiLUqu'), ('dates', '2020-05-08 15:57:02'), ('file_name', 'Eric Swalwell.csv'), ('pct_true', 0.5745165944099426)])
OrderedDict([('tweet', 'In this time of crisis, we should #ExpandSocialSecurity—not rip it away from those who need it most. Let’s help seniors &amp; our most vulnerable by passing the Social Security 2100 Act rather than a payroll tax cut to help the wealthy and corporations. The choice should be easy.'), ('dates', '2020-05-07 18:24:05'), ('file_name', 'Eric Swalwell.csv'), ('pct_true', 0.6986469626426697)])
OrderedDict([('tweet', 'These citizens are protesting they cannot leave home as they *checks notes* stand unimpeded in a public capitol, and *checks more notes* with no one trying to remove them, while they *let me see my notes* are heavily armed. https://t.co/aglKzaJOR7'), ('dates', '2020-04-30 20:34:58'), ('file_name', 'Eric Swalwell.csv'), ('pct_true', 0.8183478713035583)])
OrderedDict([('tweet', 'So @realDonaldTrump calls himself a “wartime president.” Well, every “wartime president” visited the front lines of a battlefield. Why can’t Trump visit a hospital and thank nurses? Or root on a shift change at a grocery store? Visit a gravesite? Why not? It’s always about him.'), ('dates', '2020-04-22 02:09:04'), ('file_name', 'Eric Swalwell.csv'), ('pct_true', 0.6067407131195068)])
OrderedDict([('tweet', 'He wants us to think our pandemic is the fault of immigrants &amp; other nations. But we’re in this crisis- 42k-plus deaths &amp; a faltering economy as other nations did better- due to his failures of foresight &amp; leadership, &amp; his ongoing failure to take responsibility &amp; accept reality. https://t.co/9Q4CIbPJMw'), ('dates', '2020-04-21 02:50:54'), ('file_name', 'Eric Swalwell.csv'), ('pct_true', 0.9386357069015503)])
OrderedDict([('tweet', 'You know what’s “dumb” — @realDonaldTrump calling the #coronavirus a “hoax” &amp; failing to take actions that would have saved lives. #DumbDonald has made us sicker and poorer. Thankfully, @SpeakerPelosi is putting public health above her personal interests. We will defeat this. https://t.co/tNlTOrxbmQ'), ('dates', '2020-04-19 19:54:17'), ('file_name', 'Eric Swalwell.csv'), ('pct_true', 0.8322206735610962)])
OrderedDict([('tweet', 'Congressman @davidcicilline and I are leading more than 100 Dems and Republicans who want access to funding in the next relief bill to support the small local news outlets that provide essential information regarding public health and safety, local events, and democracy. https://t.co/8Pl5VPLTRV'), ('dates', '2020-04-29 22:22:39'), ('file_name', 'F. James Sensenbrenner, Jr..csv'), ('pct_true', 0.5773040652275085)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram is officially out of money. Now is not the time to play partisan politics. Workers are struggling through no fault of their own. Wisconsin’s small businesses are counting on Congress to do the right thing &amp; replenish these funds now! https://t.co/Q85udKECbr'), ('dates', '2020-04-16 17:42:37'), ('file_name', 'F. James Sensenbrenner, Jr..csv'), ('pct_true', 0.9037694334983826)])
OrderedDict([('tweet', 'Rep. Sensenbrenner: Democrats Hypocrisy Exposed as Urgent Impeachment Can Now Wait https://t.co/jOH4rA3cZc'), ('dates', '2019-12-19 18:17:00'), ('file_name', 'F. James Sensenbrenner, Jr..csv'), ('pct_true', 0.5786983966827393)])
OrderedDict([('tweet', 'Rep. Sensenbrenner will be voting no on these articles of #impeachment. https://t.co/qaTYWGWzyZ'), ('dates', '2019-12-18 19:12:11'), ('file_name', 'F. James Sensenbrenner, Jr..csv'), ('pct_true', 0.5254112482070923)])
OrderedDict([('tweet', 'Rep. Sensenbrenner statement on articles of impeachment:\n\n“Democrats have wanted to impeach President Trump since January 20, 2017 and realized their dream would come true last November.'), ('dates', '2019-12-10 15:50:52'), ('file_name', 'F. James Sensenbrenner, Jr..csv'), ('pct_true', 0.8959134817123413)])
OrderedDict([('tweet', 'Democrats are rushing through this #ShamImpeachment because Americans have seen through their broken and unfair process. https://t.co/KBpMc91dzL'), ('dates', '2019-12-09 17:01:36'), ('file_name', 'F. James Sensenbrenner, Jr..csv'), ('pct_true', 0.934891939163208)])
OrderedDict([('tweet', 'Democrats have sought to impeach President Trump since the day he took office. https://t.co/r4xlCUalMK'), ('dates', '2019-12-04 15:30:00'), ('file_name', 'F. James Sensenbrenner, Jr..csv'), ('pct_true', 0.7998983860015869)])
OrderedDict([('tweet', 'Rep. Sensenbrenner: "@WSJopinion highlights the problems that small businesses are facing after last year’s Wayfair decision. https://t.co/VghhDxB6CA'), ('dates', '2019-08-13 18:32:19'), ('file_name', 'F. James Sensenbrenner, Jr..csv'), ('pct_true', 0.7572460770606995)])
OrderedDict([('tweet', '#EarthDay is about celebrating the planet and recognizing our responsibility to protect it. \n\nHouse Democrats are working to #ActOnClimate because we want a healthy Earth not just for a day, but for generations. 🌎 https://t.co/jmY8r21q1u'), ('dates', '2020-04-22 14:47:58'), ('file_name', 'Filemon Vela.csv'), ('pct_true', 0.6170319318771362)])
OrderedDict([('tweet', 'Fears will continue to mount @POTUS\n and those fears will turn into the economy crashing, #coronavirus spreading, and a country in panic\n\nWill you do the right thing for the American people and invest in combating this health crisis\n\nor\n\n...use those funds to Build a wall \U0001f974 https://t.co/WqkcvSUWUt'), ('dates', '2020-02-28 18:22:42'), ('file_name', 'Filemon Vela.csv'), ('pct_true', 0.5543187260627747)])
OrderedDict([('tweet', 'Typical-Maduro regime places 1 of its most corrupt &amp; criminal officials in a critical leadership role as the economy collapses further amid the ongoing health crisis. The int’l community should soundly denounce this action &amp; stand with the Venezuelan people in demanding democracy https://t.co/zck25wUtBc'), ('dates', '2020-04-28 18:18:37'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5413678884506226)])
OrderedDict([('tweet', 'Small businesses are hurting and the original PPP funds have been exhausted. Hopefully leaders of both  parties will quickly work out an additional funding plan. https://t.co/f9V5FnOjiL'), ('dates', '2020-04-16 17:27:31'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.832984447479248)])
OrderedDict([('tweet', 'Our Nation faces an unexpected &amp; unprecedented challenge that has led to tremendous loss of life &amp; financial hardship in our community &amp; throughout America. As Americans, we know that we will triumph over this deadly disease &amp; regain our economic momentum https://t.co/GcFVFv62i6'), ('dates', '2020-04-13 17:34:30'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5732397437095642)])
OrderedDict([('tweet', 'I support this move to exert maximum pressure on the illegitimate #Maduro regime. Maduro’s gov’t relies on illicit activities/oppression of the Venezuelan people to maintain power. I urge all countries to join the US in taking necessary steps to hold the Maduro regime accountable https://t.co/huqVpLoUMy'), ('dates', '2020-03-26 20:27:54'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5414730906486511)])
OrderedDict([('tweet', 'Is it any wonder why Congress has such low approval ratings? \xa0Personally trading on inside info, like\xa0Senator Burr,\xa0is unacceptable. https://t.co/cFxhmk9abd'), ('dates', '2020-03-21 19:59:47'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5195361375808716)])
OrderedDict([('tweet', 'Is it any wonder why Congress has such low approval ratings? Trading on inside info, as appears has happened here, is unacceptable. The behavior along with people who do it, need to go.https://t.co/cFxhmk9abd'), ('dates', '2020-03-21 19:50:10'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.8719429969787598)])
OrderedDict([('tweet', 'We are in a gobal\xa0pandemic-Congress should utilize modern technology\xa0to permit\xa0remote voting. Votes\xa0on spending &amp; response\xa0to the pandemic\xa0are critical &amp; will need quick action.\xa0Remote voting is a way to\xa0effectuate\xa0social distancing &amp;\xa0follow\xa0proper health procedures #RemoteVote'), ('dates', '2020-03-18 22:49:34'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.680849015712738)])
OrderedDict([('tweet', 'This unjust imprisonment of US citizens by the #Maduro regime should be vigorously opposed by the US and our democratic partners. We must maintain pressure on the corrupt Maduro regime in order to facilitate a democratic transition in #Venezuela. (1/2) https://t.co/rwgr8BVQgS'), ('dates', '2020-02-07 19:13:26'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.71991366147995)])
OrderedDict([('tweet', 'Putin’s proposed constitutional reforms to limit his Presidential successor’s powers is a sign that he has no plans to relinquish power after his term expires. Regardless of what happens in 2024, he will continue to rule over the Russian Federation. https://t.co/XAsPVQn9gv'), ('dates', '2020-01-15 20:25:26'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.7512491941452026)])
OrderedDict([('tweet', "I condemn the continued efforts of the Maduro regime to block Juan Guaido and opposition lawmakers’ entry to the Nat'l Assembly. We must continue to increase pressure on Maduro to step down &amp; support efforts of interim President Guaido &amp; the democratically-elected Nat'l Assembly."), ('dates', '2020-01-07 17:50:29'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.7658910155296326)])
OrderedDict([('tweet', 'Ortega’s police force cutting off water and power to a Catholic church is a blatant attack on religious freedom in #Nicaragua. I strongly condemn the #Ortega regime’s intimidation against freedom of worship #SOSMasaya https://t.co/0PV6lquYWQ'), ('dates', '2019-11-19 19:35:44'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5508691072463989)])
OrderedDict([('tweet', 'Chinese espionage on American campuses is a grave threat to our national and economic security. My bill, the SHEET Act (HR 3071), introduced with @SenTedCruz would protect our open academic environment and prevent this sort of malign behavior https://t.co/3fudY4l5X6'), ('dates', '2019-11-05 16:24:30'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5133407115936279)])
OrderedDict([('tweet', 'I urge the President to reverse his decision of removing our troops, and to send a strong message to #Turkey – along with our other #NATO partners in Europe – that we support the #Kurds who have been fighting with us.'), ('dates', '2019-10-10 18:12:29'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5658913254737854)])
OrderedDict([('tweet', 'The recent occurrence of violence in #Ecuador is disturbing; the United States should openly support democratic actors there and the @Lenin Moreno government’s efforts to resolve this crisis peacefully.'), ('dates', '2019-10-09 14:21:24'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.967271089553833)])
OrderedDict([('tweet', 'Thank you to our @OAS_official partners who voted for sanctions against the oppressive Maduro regime under the #RioTreaty. Pressure that Western Hemisphere nations impose is critical to holding Maduro and his allies accountable for their abuses against the Venezuelan people. https://t.co/p0BW2GL20X'), ('dates', '2019-09-24 19:48:40'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5117079019546509)])
OrderedDict([('tweet', "I have introduced legislation in Congress that would protect consumers and require drug companies to include prices in TV ads. @SpeakerPelosi let's take up HR3327. @POTUS supports this concept. https://t.co/9HYxLD3PPc"), ('dates', '2019-07-09 13:58:28'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5701276659965515)])
OrderedDict([('tweet', 'The Administration’s rule requiring drug companies to include list prices in TV ads will create a market-based solution to lower prices. I introduced legislation to codify this rule so that Americans know upfront how much these medications cost.'), ('dates', '2019-06-19 19:34:38'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.8030914068222046)])
OrderedDict([('tweet', 'Prescription drug prices have skyrocketed over the last decade and many Americans can’t afford the medications they need. President @realDonaldTrump is finally doing something about it.'), ('dates', '2019-06-19 19:34:26'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.7712132334709167)])
OrderedDict([('tweet', 'While attention in Latin America has been focused on the crisis in Venezuela, a similar crisis has taken place in Nicaragua. We must respond to show corrupt leaders like Daniel Ortega that the US won’t stand by while illegitimate regimes crush democracy and neglect their people. https://t.co/8WURglDmVH'), ('dates', '2019-06-11 18:11:46'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.6493558883666992)])
OrderedDict([('tweet', 'The people of Venezuela are making history today as they stand up for freedom and democracy through #operacionlibertad . The United States stands with these brave men and women in their struggle to end oppression and restore their democracy. #EstamosUnidosVE'), ('dates', '2019-04-30 16:17:09'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.7626128196716309)])
OrderedDict([('tweet', 'Ecuadorian President Moreno was right to revoke asylum for Julian Assange. This is not a case of 1st Amendment rights, but of collusion with Chelsea Manning (real collusion, not the false Trump-Russia narrative) to obtain classified data from the U.S. government.'), ('dates', '2019-04-12 17:15:25'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.748150110244751)])
OrderedDict([('tweet', "Attempting to infiltrate sensitive gov’t computer networks &amp; illegally obtain classified information is not protected by the 1st Amendment right to free speech. Assange put American lives in danger &amp; I applaud Ecuadorian President Moreno for revoking Assange's claim to asylum."), ('dates', '2019-04-12 16:17:51'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5873180627822876)])
OrderedDict([('tweet', 'On 9/11, Islamic terrorists killed 2,996 people on American soil. Rep. Omar referring to this act of terror as "some people did something" is offensive to me &amp; to the victims who lost their lives that day, and is just the latest in a series of disturbing comments from this rep.'), ('dates', '2019-04-10 12:23:20'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5436883568763733)])
OrderedDict([('tweet', 'The United States does not need to risk the Gulf of Mexico to be energy secure. We have more reserves now than Russia or Saudi Arabia, and we are net exporters of energy. We don’t need offshore drilling on our coast. #ProtectOurCoast (3/4)'), ('dates', '2019-04-02 16:01:55'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.7117334008216858)])
OrderedDict([('tweet', 'Offshore drilling would undermine our tourism and recreation-based economy. Any repeat of the BP oil spill disaster would devastate our multi-billion dollar tourist-based economy. There is no place for offshore drilling in Florida. #ProtectOurCoast (2/4)'), ('dates', '2019-04-02 16:01:33'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.6012417674064636)])
OrderedDict([('tweet', 'Reports show more Russian military forces have arrived in Venezuela to reinforce Maduro’s illegitimate socialist regime. As @SecPompeo of the @StateDept said, we “will not stand idly by as Russia exacerbates tensions in Venezuela.”\n https://t.co/lPYQ4baGCj via @dailycaller'), ('dates', '2019-03-26 13:14:58'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.693693995475769)])
OrderedDict([('tweet', 'The sovereignty of Georgia is under threat from Russia and its allies; the United States must stand with our allies to deter future Russian aggression against its neighbors. I introduced a resolution today to express this critical support by Congress.'), ('dates', '2019-01-31 19:47:56'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.7958242297172546)])
OrderedDict([('tweet', 'Hoy el presidente Trump sancionó a la industria petrolera en #Venezuela y congeló los activos conectados al petróleo Venezolano. Esta decisión detiene el flujo de ingresos al régimen criminal ilegítimo de Maduro y salvaguarda a los recursos para el futuro del pueblo Venezolano.'), ('dates', '2019-01-28 23:46:30'), ('file_name', 'Francis Rooney.csv'), ('pct_true', 0.5500964522361755)])
OrderedDict([('tweet', "If China surpasses us in critical innovation, it will have significant implications for our national security &amp; our way of life.\n\nWe’re working with universities &amp; agencies to protect our academic research enterprise and maintain America's scientific &amp; technological leadership. https://t.co/CejBwGDdkQ"), ('dates', '2020-05-04 18:56:09'), ('file_name', 'Frank D. Lucas.csv'), ('pct_true', 0.8319387435913086)])
OrderedDict([('tweet', 'U.S. adversaries are spreading #COVID19 disinformation, exploiting communities across our country.\n\nDisinformation has killed people across the globe. Russia, China, and Iran must stop this deadly spread of disinformation.\nhttps://t.co/KxMgoujmn1'), ('dates', '2020-04-23 14:30:00'), ('file_name', 'Frank D. Lucas.csv'), ('pct_true', 0.5540015697479248)])
OrderedDict([('tweet', 'Days after Democrats shut down emergency support for America’s small businesses, the Senate has just passed additional funding for the #PaycheckProtectionProgram. \n\nIt’s time for the House to put politics aside and provide much needed support for America’s Main Street businesses.'), ('dates', '2020-04-21 22:52:40'), ('file_name', 'Frank D. Lucas.csv'), ('pct_true', 0.7159467935562134)])
OrderedDict([('tweet', 'The American people do not need funding for climate change studies or reporting requirements on work-life balance activities. The American people need help now! \n\nIt’s time for the Senate to pass the bipartisan drafted #CARESAct!\nhttps://t.co/2xWCW39mjP'), ('dates', '2020-03-24 13:00:00'), ('file_name', 'Frank D. Lucas.csv'), ('pct_true', 0.85438472032547)])
OrderedDict([('tweet', 'Instead of listening to experts, Trump treated this crisis like a failed reality TV show that prioritized economic gain over the health of Americans. House Democrats will continue to push for a national testing strategy to safely reopen the economy and keep Americans healthy. https://t.co/EyjTKWgCQo'), ('dates', '2020-05-10 17:30:00'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.8389893174171448)])
OrderedDict([('tweet', 'The coronavirus pandemic has exacerbated the financial strain our communities and residents faced after Superstorm Sandy. That is why I introduced a bill with @RepAndyKimNJ &amp; @SenatorMenendez to help homeowners and towns still recovering from Sandy. \nhttps://t.co/ZPWPf4WF8B'), ('dates', '2020-05-09 23:00:00'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.5063552856445312)])
OrderedDict([('tweet', 'The coronavirus pandemic has negatively impacted the commercial and charter fishing industries in New Jersey. That’s why I pushed for federal funding to help these industries weather the ongoing economic storm. https://t.co/qfMXyZAgqO'), ('dates', '2020-05-09 19:15:00'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.5200412273406982)])
OrderedDict([('tweet', 'Trump’s hypocrisy is clear as he rejects a widespread testing strategy for Americans even as he is tested daily. Democrats will continue to push for a comprehensive national testing plan. \nhttps://t.co/9jyVXK7vd7'), ('dates', '2020-05-09 18:23:28'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.8268649578094482)])
OrderedDict([('tweet', 'Trump’s unnecessary border wall project has already wasted millions in taxpayer dollars with no changes to border security. He now wants to throw away even more taxpayer money on a $500 million paint job in the midst of a public health crisis. \nhttps://t.co/X0dbO1q1cZ'), ('dates', '2020-05-08 00:30:00'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.6722825169563293)])
OrderedDict([('tweet', 'Despite new research confirming that air pollution has a dire — even deadly — effect on human health, particularly now in the midst of a respiratory illness pandemic, the Trump Admin has continued its barrage of environmental rollbacks undeterred.'), ('dates', '2020-05-07 19:49:50'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.5989291071891785)])
OrderedDict([('tweet', 'Even as the #coronavirus pandemic threatens the health of millions of Americans, Trump will stop at nothing to destroy the Affordable Care Act. Millions of Americans rely on the ACA’s benefits and protections. This callousness must end.\nhttps://t.co/nbCJQBFkl1'), ('dates', '2020-05-07 15:49:52'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.6630995869636536)])
OrderedDict([('tweet', 'Our nation cannot effectively combat this terrible #coronavirus pandemic in the weeks and months ahead without significant testing, contact tracing and surveillance. That’s why I’m holding a forum with @EnergyCommerce tomorrow.\nhttps://t.co/VXLmHv83zF'), ('dates', '2020-05-07 14:00:41'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.5681621432304382)])
OrderedDict([('tweet', 'Sad that Trump Administration refused to participate in virtual world summit on vaccines. US must continue to exercise leadership on global health issues particularly during a pandemic. We are all in this together. \nhttps://t.co/my4ZBryHBH'), ('dates', '2020-05-06 23:48:00'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.6154600977897644)])
OrderedDict([('tweet', "The Supreme Court heard oral arguments today over the Trump Administration's attempt to gut the protections of the ACA's birth control guarantee. We cannot go back to a time when women's employers got to decide whether they had coverage for birth control. https://t.co/jlrIToqBDi"), ('dates', '2020-05-06 21:54:37'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.9242852926254272)])
OrderedDict([('tweet', 'China must stop using the coronavirus as a propaganda tool. If they wish to assist other countries in fighting the pandemic, they should not condition their help with economic threats. Everyone needs to recognize that we are all in this together. \nhttps://t.co/Ap5XEv5XvO'), ('dates', '2020-05-03 20:17:00'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.9124006628990173)])
OrderedDict([('tweet', 'Trump keeps saying that any American can get tested, which is clearly false. But, he tests himself and the White House staff on a regular basis. Meanwhile, the rest of the country is lacking testing capacity. \nhttps://t.co/4UMfp31XC0'), ('dates', '2020-05-02 21:53:00'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.7511643171310425)])
OrderedDict([('tweet', 'United States has more confirmed COVID-19 cases and deaths than any other country and the pandemic continues to spread. Yet our President insists that the crisis is nearly over. Our President acts like an ostrich with his head in the ground. https://t.co/Nsky3zrMxj'), ('dates', '2020-05-01 14:23:28'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.5891083478927612)])
OrderedDict([('tweet', 'Leader McConnell’s reckless idea to allow state and localities to go bankrupt will only lead to cuts in services when Americans need them the most.  There must be support for states in the next stimulus package. \nhttps://t.co/fa9kQTag5O'), ('dates', '2020-04-27 20:15:00'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.774570643901825)])
OrderedDict([('tweet', 'The GOP tax scam spurred large companies to use up cash on stock buy backs and dividends. The result was that companies didn’t have much cash when the coronavirus exposed them to an economic downturn. Now they expect bailouts from the federal government. \nhttps://t.co/VeCTuR95K0'), ('dates', '2020-04-25 21:30:00'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.5858319997787476)])
OrderedDict([('tweet', 'After the #NetNeutrality repeal comment period debacle, it was clear the @FCC’s cybersecurity practices had failed. @USGAO agrees and found a disturbing lack of security that places the Commission’s information systems at risk. https://t.co/iypAFuwf5A'), ('dates', '2020-04-24 16:54:23'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.7108883857727051)])
OrderedDict([('tweet', "The commercial and recreational fishing industry in New Jersey has felt the devastating economic impacts of the COVID-19 pandemic. That's why the entire New Jersey Congressional delegation has demanded swift action from the Trump Administration. \nhttps://t.co/8TUh8NWp9J"), ('dates', '2020-04-24 14:22:46'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.5458793640136719)])
OrderedDict([('tweet', "The President continues his assault on the ACA in the midst of a public health emergency. Trump must reopen enrollment for the ACA marketplace now and heed Democrats' plans to expand Medicaid and increase subsidies to help with premiums.\nhttps://t.co/TD3JmrWXco"), ('dates', '2020-04-21 18:35:27'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.8998110294342041)])
OrderedDict([('tweet', 'Democrats have a plan to help people who lose health insurance. We want to reopen enrollment for the ACA, increase subsidies to help pay premiums, expand Medicaid and eliminate junk plans that do not cover essential benefits.  \nhttps://t.co/9RK35HxFE2'), ('dates', '2020-04-19 16:01:12'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.7493515610694885)])
OrderedDict([('tweet', 'Shameful that the President is putting lives at risk and fomenting opposition to his own plans by encouraging mass gatherings that protest science-based stay-at-home orders. https://t.co/9pRTme7Ye6'), ('dates', '2020-04-18 23:14:55'), ('file_name', 'Frank Pallone, Jr..csv'), ('pct_true', 0.6041867136955261)])
OrderedDict([('tweet', 'The D.C. swamp’s efforts to tear down President @realDonaldTrump by any means necessary are finally being exposed and justice is coming to those whose lives they ruined in their relentless, unfounded pursuits.\n\nhttps://t.co/z60AlNSlLW'), ('dates', '2020-05-07 19:43:45'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.6255092620849609)])
OrderedDict([('tweet', 'Adam Schiff and his allies have been trying to tear down President @realDonaldTrump by any means necessary since before he was elected:\n❌Mueller investigation based on the debunked Steele dossier\n❌Limitless investigatory scope\n❌A sham impeachment\n\nhttps://t.co/7PJhijvCmk'), ('dates', '2020-05-07 14:18:18'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.7317606806755066)])
OrderedDict([('tweet', 'By not releasing these transcripts, Chairman Schiff is once again deliberately withholding from the American people critical information as he continues his pattern of selective leaks, misleading narratives, divisive behavior and outright lies. https://t.co/RaCYBwZSB0'), ('dates', '2020-05-05 21:09:05'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.5727643966674805)])
OrderedDict([('tweet', 'Without the House back in Washington:\n❌Speaker Pelosi can continue to draft “far-reaching and transformative” legislation in secret\n❌China can continue not being held accountable for exacerbating this pandemic https://t.co/t0LS5FVuuB'), ('dates', '2020-05-05 00:30:46'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.6802247166633606)])
OrderedDict([('tweet', 'The left’s first attempt to unseat President Trump was seen by the American people for what it was: a partisan sham.\n\nThis Coronavirus committee is no different.\n\nWe deserve better than the baseless, politically-motivated investigations led by Speaker Pelosi and her allies. https://t.co/tJwi0ilbJA'), ('dates', '2020-04-30 02:04:54'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.5428142547607422)])
OrderedDict([('tweet', 'Speaker Pelosi refuses to bring the House back Washington to perform its critical work. Instead, she\'s working in secret on a bill she has already described as "far-reaching," "transformative," and "expensive."\n\nThe American people deserve better.\n\nhttps://t.co/8gOTaVMmxc'), ('dates', '2020-04-29 16:38:58'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.6007276773452759)])
OrderedDict([('tweet', 'President @realDonaldTrump just signed the legislation we passed yesterday to help small businesses and families. This should have happened two weeks ago, but Speaker Pelosi and Sen. Schumer shamefully injected politics into the process and hurt our economic recovery.\n\nMore ⬇ https://t.co/jpMpFPwBXx'), ('dates', '2020-04-24 17:31:34'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.600745677947998)])
OrderedDict([('tweet', "We already have an Oversight Committee. We do not need Speaker Pelosi to create more redundant bureaucracy aimed at more partisan investigations.\n\nThe American people didn't buy into her sham impeachment and they won't buy into this blatantly political move. https://t.co/OCWMoCy7nI"), ('dates', '2020-04-23 15:42:55'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.9805662631988525)])
OrderedDict([('tweet', 'Glad we are reauthorizing funding for the Paycheck Protection Program and Economic Injury Disaster Loan Program today, but we should have done this two weeks ago. \n\nBy injecting partisan politics into this process, Speaker Pelosi and Leader Schumer cost Americans their jobs. https://t.co/kW71hyo4jD'), ('dates', '2020-04-23 14:53:06'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.5827717781066895)])
OrderedDict([('tweet', 'Speaker Pelosi said just three days ago: “Leaders take responsibility.”\n\nOver the past week, she’s orchestrated:\n❌Partisan political games\n❌An unnecessary delay in reauthorizing PPP funding\n❌Thousands more Americans losing their jobs\n\nYet she won’t take responsibility. https://t.co/XJy1Yuq5lA'), ('dates', '2020-04-22 16:04:31'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.7139415740966797)])
OrderedDict([('tweet', "The Paycheck Protection Program was incredibly popular and should have been replenished over a week ago, but Speaker Pelosi and Sen. Schumer injected partisan politics into the process and hurt families and small businesses\n\nMore from last night's interview on @CSPANWJ 🔽 https://t.co/8dNsPKH77n"), ('dates', '2020-04-22 13:34:49'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.7758409380912781)])
OrderedDict([('tweet', "The Chinese government's cover-up of #COVID19 and campaign of disinformation exacerbated the effects of this crisis across the globe.\n\nThe left delaying additional funding for the Paycheck Protection Program and EIDL has exacerbated the economic pain felt by American families. https://t.co/moX6Ez5n3R"), ('dates', '2020-04-21 17:25:30'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.6306891441345215)])
OrderedDict([('tweet', '"We cannot allow our nation’s small businesses and families to suffer as a result of Democrats’ partisan games."\n\nMore 🔽\n\nhttps://t.co/6MGdDm6AXW'), ('dates', '2020-04-17 15:53:37'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.7358515858650208)])
OrderedDict([('tweet', "While small businesses need help &amp; the successful Paycheck Protection Program might run out of money tomorrow, @SpeakerPelosi:\n❌Congratulates Dems for blocking additional funding\n❌Takes a victory lap on partisan behavior\n\nLet's stop the partisan games and help small businesses! https://t.co/9PBrVB6tnq"), ('dates', '2020-04-15 15:35:46'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.7642431259155273)])
OrderedDict([('tweet', 'Small businesses employ nearly half of all Americans and the bipartisan Paycheck Protection Program has helped many get by during the #COVID19 crisis. @SpeakerPelosi and @SenSchumer need to stop playing partisan games and help us authorize more money for this in-demand program. https://t.co/QViRO4QOT4'), ('dates', '2020-04-14 17:52:12'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.6233965158462524)])
OrderedDict([('tweet', "🚨I'l be on the @WilkowMajority on @SiriusXMPatriot around 2:40 p.m. talking about efforts to hold China and the World Health Organization responsible for exacerbating this pandemic through inaction, propaganda, and a campaign of disinformation. Tune in!🚨"), ('dates', '2020-04-09 18:31:06'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.9031193852424622)])
OrderedDict([('tweet', "For reasons beyond understanding, the @WHO acted as a silent partner in China's disinformation efforts instead of protecting the lives of millions across the world, including hundreds of thousands of American citizens. \n\nWe should be defund them until @DrTedros resigns! https://t.co/KRotzkJpGO"), ('dates', '2020-04-07 17:26:08'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.6436508893966675)])
OrderedDict([('tweet', '“The Chinese Communist regime has made this problem exponentially worse by not being truthful about the contagiousness and dangerousness of COVID-19, instead engaging in a campaign of blame-shifting, misinformation, and propaganda.”\n\nMore in @DailyCaller \n\nhttps://t.co/OaoqEeyQX3'), ('dates', '2020-04-02 21:28:52'), ('file_name', 'Fred Keller.csv'), ('pct_true', 0.9302106499671936)])
OrderedDict([('tweet', 'China up to this point has stonewalled investigations looking at what may be the origins of this pandemic. We’re going to need these answers. Hear more of today’s interview with @1450WHTC: https://t.co/8oBtcCh9Zd https://t.co/jJlB1D1Ut1'), ('dates', '2020-05-05 20:42:38'), ('file_name', 'Fred Upton.csv'), ('pct_true', 0.669347882270813)])
OrderedDict([('tweet', "We can't allow #PPP funding to run out again. The #PPP is a solemn promise from Congress to America’s small biz. We sent a bipartisan letter to @SpeakerPelosi, @GOPLeader, @SenSchumer, @senatemajldr, @MarkMeadows, &amp; @stevenmnuchin1 making our view clear: https://t.co/A9I1cgRs7S https://t.co/obbkHJ2nhv"), ('dates', '2020-04-26 13:50:23'), ('file_name', 'Fred Upton.csv'), ('pct_true', 0.6722031235694885)])
OrderedDict([('tweet', 'The coronavirus crisis has opened our eyes to the real heroes among us, &amp; the @USPS is no exception. These folks are braving the elements and delivering mail across the country, &amp; I want to sincerely thank them for all that they’re doing to keep us connected. #InItTogether #MI06'), ('dates', '2020-04-18 19:32:06'), ('file_name', 'Fred Upton.csv'), ('pct_true', 0.7330019474029541)])
OrderedDict([('tweet', 'When the @SBAgov first launched the #PPP, I suspected that the original $350 billion would not last long. Small businesses in SW MI desperately need relief, &amp; my car is “pointed out” should I need to drive to D.C. and vote for more funding for this critical program. #MI06'), ('dates', '2020-04-16 19:00:48'), ('file_name', 'Fred Upton.csv'), ('pct_true', 0.805329442024231)])
OrderedDict([('tweet', '#PPP loans for small businesses have officially run dry. It’s time for Congress to act swiftly and replenish this critical relief fund for American workers, families, and small businesses. We don’t have time to play politics, and neither does our country. #InItTogether #MI06'), ('dates', '2020-04-16 14:53:49'), ('file_name', 'Fred Upton.csv'), ('pct_true', 0.5681040287017822)])
OrderedDict([('tweet', 'Appalled to read over the weekend some state leaders are not imposing strict social distancing. Calling on all Governors to follow the CDC social distancing guidelines. This is a national emergency - get with the program. We need to flatten the curve. #MI06 #InItTogether'), ('dates', '2020-04-06 15:35:08'), ('file_name', 'Fred Upton.csv'), ('pct_true', 0.9136603474617004)])
OrderedDict([('tweet', 'People should not have to put their lives on the line during a public health crisis to exercise the precious right to vote. RT if you agree. #COVID19 https://t.co/uEx8MBQPWn'), ('dates', '2020-05-07 22:30:24'), ('file_name', 'Frederica S. Wilson.csv'), ('pct_true', 0.595420777797699)])
OrderedDict([('tweet', "The #COVID19 pandemic is quickly dwindling our state's budget as more people than ever seek aid. We must all do our part by filling out our #Census2020 forms to ensure that Florida gets its fair share of federal funding. Trust me, we're going to need it! https://t.co/C1m9DyOiT5"), ('dates', '2020-05-04 15:02:23'), ('file_name', 'Frederica S. Wilson.csv'), ('pct_true', 0.6168537735939026)])
OrderedDict([('tweet', 'Sadly, there are people who will try to take advantage of others during this trying time. Please beware of phone scams! The IRS will never call to request your personal or financial information.\nhttps://t.co/yZui0bBFSX'), ('dates', '2020-04-27 14:00:42'), ('file_name', 'Frederica S. Wilson.csv'), ('pct_true', 0.501678466796875)])
OrderedDict([('tweet', 'Deporting people to Haiti in the midst of a global pandemic is both inhumane and unsafe. Continuing these flights will likely contribute to the spread of #COVID19 in the impoverished nation where many people do not have access to basic health care or clean water. https://t.co/LVzhflqRgJ'), ('dates', '2020-04-24 15:33:24'), ('file_name', 'Frederica S. Wilson.csv'), ('pct_true', 0.612941563129425)])
OrderedDict([('tweet', '#MitchAntoinette would rather deny aid to firefighters, police &amp; EMS than deliver the relief they need to keep fighting the coronavirus. This is a disgusting remark to make when American lives are on the line nationwide. https://t.co/yv6kfY5VwF'), ('dates', '2020-04-23 20:17:34'), ('file_name', 'Frederica S. Wilson.csv'), ('pct_true', 0.6375146508216858)])
OrderedDict([('tweet', "It's a disgrace that multi-billion dollar businesses are receiving #COVID19 payouts while hardworking Americans can't get the unemployment benefits they've earned!\nhttps://t.co/NmuPx0aFwl"), ('dates', '2020-04-20 19:53:02'), ('file_name', 'Frederica S. Wilson.csv'), ('pct_true', 0.8235200047492981)])
OrderedDict([('tweet', 'Too many small businesses are missing out on #PPPLoans. @housedemocrats are working to ensure #smallbiz in rural and underserved communities have access to this money. That’s why any #Smallbizrelief bill must empower community lenders https://t.co/yUW6MYs7hk'), ('dates', '2020-04-15 15:46:00'), ('file_name', 'Frederica S. Wilson.csv'), ('pct_true', 0.5261797904968262)])
OrderedDict([('tweet', 'One year ago, @HouseDemocrats made history by passing #RH1, the #ForThePeopleAct—a transformative anti-corruption and clean elections bill. But to maintain the status quo in Washington, @senatemajldr continues to black the bill. It’s time for a Senate vote on #HR1! https://t.co/WtDwORyUr8'), ('dates', '2020-03-09 15:27:54'), ('file_name', 'G. K. Butterfield.csv'), ('pct_true', 0.6297259330749512)])
OrderedDict([('tweet', 'On the House floor speaking on #Medicaid—I’m proud to stand with Democrats fighting back against the Trump Administration’s illegal Medicaid block grant scheme, which threatens health care for millions of Americans. #ProtectOurCare https://t.co/dQM5iBLPF0'), ('dates', '2020-02-06 20:35:10'), ('file_name', 'G. K. Butterfield.csv'), ('pct_true', 0.8749987483024597)])
OrderedDict([('tweet', '#FactCheck-Claiming to protect Americans with pre-existing conditions,Trump and his Administration have repeatedly sought to undermine protections offered by the ACA through executive orders and the courts. He is seeking to strike down the law and its protections entirely.'), ('dates', '2020-02-05 02:47:23'), ('file_name', 'G. K. Butterfield.csv'), ('pct_true', 0.8801838755607605)])
OrderedDict([('tweet', 'PPP. Who benefited so far?\n\nThe program kept millions of American workers on the payroll &amp; is helping small businesses across a wide range of industries.\n\nTonight, the US House funded an additional $310 billion to PPP. More Americans to keep jobs &amp; businesses to keep doors open. https://t.co/FEeFWPBWTW'), ('dates', '2020-04-23 22:48:45'), ('file_name', 'Garret Graves.csv'), ('pct_true', 0.7646786570549011)])
OrderedDict([('tweet', 'On #YomHaShoah we remember the 6 million Jewish women, men and children &amp; millions more killed by the Nazis in the Holocaust. I stand in solidarity w/ the Jewish community in Michigan, across the US &amp; world. We must #NeverForget &amp; keep fighting anti-Semitism &amp; all forms of hate.'), ('dates', '2020-04-21 01:01:21'), ('file_name', 'Gary C. Peters.csv'), ('pct_true', 0.5342651605606079)])
OrderedDict([('tweet', 'After pushing the Administration, Social Security recipients, Supplemental Security Income beneficiaries &amp; veterans now will automatically receive stimulus checks.  \nhttps://t.co/aJAJy07EBP'), ('dates', '2020-04-19 21:44:44'), ('file_name', 'Gary C. Peters.csv'), ('pct_true', 0.5166540145874023)])
OrderedDict([('tweet', "This public health crisis shows just how fragile our nation’s medical supply chain is.\n\nI'll continue pressing the Trump Administration to take action and eliminate drug shortages so Michiganders have access to the medicine they need. https://t.co/4rhUTl22jM"), ('dates', '2020-04-18 19:12:26'), ('file_name', 'Gary C. Peters.csv'), ('pct_true', 0.5706872940063477)])
OrderedDict([('tweet', 'Adam Schiff lied to his colleagues, the media, and the American public. If there is one ounce of integrity among any Democrats in the House or Senate, they will hold him accountable for it. Full statement:\nhttps://t.co/ixrIWhjPhr'), ('dates', '2020-05-08 17:08:14'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.8578585982322693)])
OrderedDict([('tweet', 'Valuable time has been lost over the past few days because of how Pelosi and her Democrat colleagues have exploited the #COVID19 crisis to try to advance their own partisan agenda. The American people need help NOW. https://t.co/wdfoVrSv3K'), ('dates', '2020-03-25 20:07:00'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.8126503825187683)])
OrderedDict([('tweet', 'Democrats are embracing socialist policies that would undo progress made for American families in the last three years. Socialism promises "free" healthcare or a "guaranteed" income, but in reality, it takes away freedom and choice from citizens and gives government undue power. https://t.co/DZZbYp99vf'), ('dates', '2020-02-26 22:57:41'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.9639225006103516)])
OrderedDict([('tweet', "Tonight, I voted no on the #PROAct. If this bill is pro anything, it's pro-union bosses, not pro-American or pro-freedom. It hurts workers, the economy, employers, and individual right to association by preempting state right-to-work laws. Full statement: https://t.co/sZFTflWX3t"), ('dates', '2020-02-07 02:42:41'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.9443560242652893)])
OrderedDict([('tweet', 'Instead of deliberately debating constitutional war powers, Democrats continue to play games at the expense of our veterans. \n\n#HR550 #AUMF'), ('dates', '2020-01-30 21:16:04'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.7925443649291992)])
OrderedDict([('tweet', 'The Trump hotel is one more political football for Democrats. Paying market rate for a hotel room is not what the founders had in mind when seeking to limit foreign influence from emoluments. Watch my remarks on this in a hearing today below: @TransportGOP https://t.co/NfLSiS8PLw'), ('dates', '2020-01-28 22:34:38'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.6168097853660583)])
OrderedDict([('tweet', "After 28 days, Speaker Pelosi has transmitted the articles of impeachment to the Senate. The Democrats' #ShamImpeachment\ncase is clearly not urgent and has clearly been partisan and political from the start. \n\nhttps://t.co/XP4G358jj5"), ('dates', '2020-01-15 21:50:00'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.8538364171981812)])
OrderedDict([('tweet', "It should now be clear to #Iran that the US will defend its own. @POTUS has learned from others' mistakes &amp; made clear we will respond to threats against US citizens.\n\nThe killing of Soleimani defies what America’s enemies have come to expect: https://t.co/xj3IMfaaVC - @washtimes"), ('dates', '2020-01-09 20:42:26'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.5635921359062195)])
OrderedDict([('tweet', 'House Democrats have now unveiled #ArticlesOfImpeachment against President Trump, despite continued lack of evidence. This is an embarrassing day for the House of Representatives and just one more phase in what has been a blatant miscarriage of justice from the start.'), ('dates', '2019-12-10 21:27:57'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.8070701360702515)])
OrderedDict([('tweet', "The #ShamImpeachment hearings are not only a partisan effort to forcibly remove the president, but are also causing neglect of other issues with real-world implications. Democrats' obsession with impeachment has tangible, negative effects on Americans’ everyday lives. #SchiffShow https://t.co/4Bt38XfPHF"), ('dates', '2019-11-21 21:26:05'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.8923579454421997)])
OrderedDict([('tweet', 'In today’s #ImpeachmentHearings, @HouseGOP Members were interrupted and silenced when they tried to ask questions. The President has been denied the right to defend himself, and now Republicans are limited in their participation.'), ('dates', '2019-11-15 22:02:30'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.621721088886261)])
OrderedDict([('tweet', 'After weeks of dishonesty, Democrats finally held a vote on a resolution that would continue a sham #ImpeachmentInquiry against POTUS. The resolution is merely an attempt to save political face and legitimize an already poisoned process:\nhttps://t.co/PyR2tXGjte'), ('dates', '2019-10-31 18:15:28'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.8358705639839172)])
OrderedDict([('tweet', 'President Trump has taken executive action to sever the link between Medicare and Social Security. My bill, the Retirement Freedom Act, would give seniors the freedom to opt out of Medicare Part A without losing their Social Security benefits. \n\nhttps://t.co/WDHxcUS4xU'), ('dates', '2019-10-04 03:00:36'), ('file_name', 'Gary J. Palmer.csv'), ('pct_true', 0.8475637435913086)])
OrderedDict([('tweet', '"Illegal immigrants convicted of violent crimes should be deported."\n\nAmid the endless posturing and finger-pointing dominating politics these days, there are still some things that ought to transcend partisan division.\n\nThis is one of them. \n\nhttps://t.co/F77TrerysF'), ('dates', '2019-10-18 16:28:30'), ('file_name', 'George Holding.csv'), ('pct_true', 0.5464820265769958)])
OrderedDict([('tweet', 'I just read the transcript. There was no quid pro quo and nothing that justifies impeachment.  \n \nI think most folks have had enough of this circus. \n\nThe politicians need to quit posturing and finger-pointing and get back to work.'), ('dates', '2019-09-25 17:33:17'), ('file_name', 'George Holding.csv'), ('pct_true', 0.5100995898246765)])
OrderedDict([('tweet', 'This budget deal hikes spending by over $300 billion at a time when our nation is already drowning in $23 trillion in debt. We cannot keep kicking this can down the road.\n\nThis is typical Washington politics and I can’t support it.'), ('dates', '2019-07-25 22:18:27'), ('file_name', 'George Holding.csv'), ('pct_true', 0.513544499874115)])
OrderedDict([('tweet', "Do you want government bureaucrats controlling all aspects of your healthcare?\n\nBecause that is exactly what will happen under Democrats’ #MedicareForAll plan.\n\nYou'll be left with no choices, no options, and no alternatives. https://t.co/myAORUwTdt"), ('dates', '2019-06-12 12:19:19'), ('file_name', 'George Holding.csv'), ('pct_true', 0.7519984245300293)])
OrderedDict([('tweet', '#MedicareforAll would kick 5,324,500 North Carolinians off their private healthcare plans and throw them into a one-size-fits-all government-run program. https://t.co/zrhKwZHvNw'), ('dates', '2019-06-11 17:38:36'), ('file_name', 'George Holding.csv'), ('pct_true', 0.6680253148078918)])
OrderedDict([('tweet', 'Democrat’s #MedicareForAll plan abolishes virtually all private insurance in the U.S. 180 million Americans would be kicked off their current plans and forced into a one-size-fits-all government run system. \n \nNo choices, no options, &amp; no alternatives.\n \nhttps://t.co/ppf1zl9L6F'), ('dates', '2019-06-11 15:03:46'), ('file_name', 'George Holding.csv'), ('pct_true', 0.7654600739479065)])
OrderedDict([('tweet', "Today is the 1yr anniversary of @POTUS's decision to scrap the #IranDeal - an agreement that appeased &amp; empowered one of the world’s most dangerous regimes. If Iran wants the sanctions removed they must eliminate their nuclear weapons program &amp; cease their support of terrorism."), ('dates', '2019-05-08 15:59:49'), ('file_name', 'George Holding.csv'), ('pct_true', 0.8561691045761108)])
OrderedDict([('tweet', 'We are in the midst of an economic boom most people thought was impossible.\n\nUnemployment is at record lows, wages are rising &amp; millions of new jobs have been created.\n\nWhy would we ever want to go back to the days of high taxes and big government? https://t.co/YEM8RWeMs1'), ('dates', '2019-05-02 17:31:59'), ('file_name', 'George Holding.csv'), ('pct_true', 0.6376492381095886)])
OrderedDict([('tweet', 'Democrat’s decision not to propose a budget is irresponsible and negligent.\n \nA failure to budget is a failure to govern.\n \nhttps://t.co/0hzijFX7NK'), ('dates', '2019-04-03 19:41:26'), ('file_name', 'George Holding.csv'), ('pct_true', 0.5955030918121338)])
OrderedDict([('tweet', "Here's why I voted against the Democrats' resolution: Securing our border means less illegal drugs coming into our country, and it also means fewer criminals will come into our country by crossing the border illegally."), ('dates', '2019-02-27 03:14:05'), ('file_name', 'George Holding.csv'), ('pct_true', 0.8926084041595459)])
OrderedDict([('tweet', 'The “compromise” bill increases spending &amp; includes several nonsensical provisions that could potentially threaten public safety. Not to mention the fact that we were given less than 24 hours to read a 1169-page bill. This is just typical Washington politics &amp; I can’t support it.'), ('dates', '2019-02-15 00:56:50'), ('file_name', 'George Holding.csv'), ('pct_true', 0.5713590383529663)])
OrderedDict([('tweet', 'Today, I proudly voted to denounce &amp; condemn antisemitism at home &amp; abroad. There is no place in our society, or in any society, for this hateful &amp; poisonous ideology. The American ppl stand united in our unequivocal support for the Jewish people &amp; our close friend &amp; ally Israel.'), ('dates', '2019-02-13 23:08:37'), ('file_name', 'George Holding.csv'), ('pct_true', 0.8260782361030579)])
OrderedDict([('tweet', "Today, I introduced a Constitutional amendment imposing #termlimits on Congress. Term limits will end politics-as-usual in Washington &amp; help fix D.C.'s broken culture.  It’s time for elected officials to start making a difference, not a career.  \n\n&gt;&gt;https://t.co/QdHJ9s07kj https://t.co/s0UWyupIz2"), ('dates', '2019-01-15 22:47:46'), ('file_name', 'George Holding.csv'), ('pct_true', 0.5513001084327698)])
OrderedDict([('tweet', 'White supremacy is a hateful and racist ideology that has absolutely no place in our society. Period.'), ('dates', '2019-01-15 17:25:47'), ('file_name', 'George Holding.csv'), ('pct_true', 0.6735272407531738)])
OrderedDict([('tweet', 'This partial #shutdown epitomizes the worst of Washington. The politicians in Congress are more interested in scoring political points than rolling up their sleeves and getting to work. Too much politics, too little common sense.'), ('dates', '2019-01-03 19:21:09'), ('file_name', 'George Holding.csv'), ('pct_true', 0.9099050760269165)])
OrderedDict([('tweet', 'Seventy-Seven years ago today, a devastating &amp; unprovoked attack took the lives of over 2400 Americans &amp; thrust the US into the WWII. Today, we pause to remember those lost at Pearl Harbor &amp; honor the members of the Greatest Generation who answered the call to serve. #PearlHarbor https://t.co/xqLj7GywjE'), ('dates', '2018-12-07 19:57:29'), ('file_name', 'George Holding.csv'), ('pct_true', 0.5195816159248352)])
OrderedDict([('tweet', 'Trump rewards a partisan donor by installing him at #USPS. The Postal Service is in crisis and needs real leadership and someone with knowledge of the issues. \n\nThis crony doesn’t cut it. https://t.co/V3mGOIcnRW'), ('dates', '2020-05-07 01:02:28'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.5246804356575012)])
OrderedDict([('tweet', 'Trump thinks ingesting chemical disinfectants might help kill the #COVID19 virus. \n\nHis degenerating resort to quackery (see sunlight, and malaria medicines) is dangerously unbalanced, and Republican defense of the very stable genius is increasingly untenable.'), ('dates', '2020-04-24 16:28:38'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.6485834121704102)])
OrderedDict([('tweet', 'While this bill is improved and provides more funding for the Paycheck Protection Program, I have a clear message for Treasury and the Banks. These funds are to help small businesses survive – it’s not a reward program for your friends with the biggest bank account. \n\nDO BETTER.'), ('dates', '2020-04-21 21:20:06'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.6613315939903259)])
OrderedDict([('tweet', 'While the country is in deep pain Trump exploits the pandemic to implement his reactionary policies on immigration, EPA regulations, federal employee rights, postal service dismantlement and Tax cuts for the rich.'), ('dates', '2020-04-21 13:30:32'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.9678581953048706)])
OrderedDict([('tweet', 'I support PPP but we need to ensure it works for all businesses. Too many were left out of the lending process. Congress must also provide states/localities funding and flexibility. They are the safety net in America and that net is being torn asunder by a revenue collapse.'), ('dates', '2020-04-17 13:32:02'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.6936681270599365)])
OrderedDict([('tweet', 'This ruling is repugnant In the midst of a pandemic. The logic is tendentious at best and requires the reader to suspend the ugly reality of illness and death all around. Today the Supreme Court, motivated by partisan interest, has blood on its hands. \n\nhttps://t.co/6jql7IxoEl'), ('dates', '2020-04-07 14:01:08'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.7777988314628601)])
OrderedDict([('tweet', "There is no justifiable reason to penalize immigrants for trying to survive a pandemic. \n\nI led a group of House Dems in urging @DHSgov to allow immigrants to receive #COVID19 aid with no effect on their statuses under Trump's shameful Public Charge Rule.\n\nhttps://t.co/Jt7Od88wn6 https://t.co/0kHelzXtVS"), ('dates', '2020-04-01 19:49:00'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.7520087361335754)])
OrderedDict([('tweet', 'We are in the midst of the worst pandemic in 100 years and America is bleeding. \n\nThis bill is not perfect. There is much I would add, and there are provisions I would certainly delete. But now is not the time to make the perfect the enemy of the good. We must pass the CARES Act. https://t.co/cYZlOON2vg'), ('dates', '2020-03-27 15:31:27'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.6947160959243774)])
OrderedDict([('tweet', "My office asked the NSC for its “pandemic playbook” in early February. We were told it was impossible to share it. They weren't even sure they could find a copy.\n\nThis President’s insistence on rejecting lessons learned from the Obama Administration has cost lives yet again. https://t.co/aJbFG4yy2T"), ('dates', '2020-03-26 14:25:53'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.5744914412498474)])
OrderedDict([('tweet', 'A band of Trumpers are holding up the stimulus because they think it is too generous to lower income families. \n\nA new low in their shameful hypocrisy after having just given the wealthy a GIANT $2 trillion tax cut last year. \n\nWe’re talking $600 for the unemployed.'), ('dates', '2020-03-25 23:44:44'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.5003204345703125)])
OrderedDict([('tweet', 'Mr. President: your delusions aren’t a substitute for reality. We’re in the midst of a deadly pandemic here. \n\nAnd it’s growing. \n\nHow about self-quarantining this Easter and letting the rest of us cope with a crisis?'), ('dates', '2020-03-24 21:50:59'), ('file_name', 'Gerald E. Connolly.csv'), ('pct_true', 0.5864580273628235)])
OrderedDict([('tweet', 'We need to fix this broken system. Reports of big banks not supporting our small businesses are troubling. As Congress works to get more funding for PPP and EIDL, Treasury needs to issue clearer guidance so our small businesses can get better support.\nhttps://t.co/AtGJRdGhFa'), ('dates', '2020-04-17 15:28:49'), ('file_name', 'Gilbert Ray Cisneros, Jr..csv'), ('pct_true', 0.7575880885124207)])
OrderedDict([('tweet', 'China is spreading propaganda and (likely) false information about #COVID19. The World Health Organization seems to be complacent. We need to find out why.\n\nhttps://t.co/DTjVbBHd3L'), ('dates', '2020-05-05 15:52:42'), ('file_name', 'Glenn Grothman.csv'), ('pct_true', 0.7681757211685181)])
OrderedDict([('tweet', 'May is Asian Pacific American Heritage Month. This includes honoring my friends in the Hmong community, who have a rich history and vibrant culture throughout #Wisconsin. #APAHM https://t.co/ukoOXyEkdC'), ('dates', '2020-05-04 15:37:07'), ('file_name', 'Glenn Grothman.csv'), ('pct_true', 0.5429785847663879)])
OrderedDict([('tweet', "We're getting conflicting reports from China on the extent of #COVID19. I, along with my colleagues, wrote a letter to Oversight Cmte. Chair Maloney outlining the need to quickly hold hearings in search of the truth, &amp; why this matters for America.\n\nhttps://t.co/K2rH03ca5p"), ('dates', '2020-04-14 15:33:31'), ('file_name', 'Glenn Grothman.csv'), ('pct_true', 0.5422161817550659)])
OrderedDict([('tweet', 'This week, the House passed a “corrections” bill that put bandages on the paid leave bill related to #COVID19 that was hastily passed last Saturday at 1am.\n\nThis is an acknowledgement that the bill, which we were only given 30 minutes to read, was too rushed!'), ('dates', '2020-03-18 19:00:38'), ('file_name', 'Glenn Grothman.csv'), ('pct_true', 0.5557692646980286)])
OrderedDict([('tweet', 'Once again, the U.N. shows it anti-Israel bias through its recently released blacklist of Israeli and international companies. The UN should end its divisive behavior and focus on lasting peace prospects between Israel and the Palestinians.\n\nhttps://t.co/H0Kn3DbdEQ'), ('dates', '2020-02-14 22:35:00'), ('file_name', 'Glenn Grothman.csv'), ('pct_true', 0.9294188022613525)])
OrderedDict([('tweet', 'Fentanyl is an extremely dangerous drug, 10x more deadly than heroin. Just a few grains can kill you.\n\nI was glad to see President Trump sign S.3201 into law, which I voted for in the House. This will keep dangerous Fentanyl a Schedule 1 drug and tight restrictions in place.'), ('dates', '2020-02-11 17:46:36'), ('file_name', 'Glenn Grothman.csv'), ('pct_true', 0.7049611210823059)])
OrderedDict([('tweet', 'Today, the U.S. Senate acquitted @realdonaldtrump. Will this put an end to Democrats’ impeachment obsession and allow Congress to get back to work?\n\nI hope so.'), ('dates', '2020-02-05 23:41:26'), ('file_name', 'Glenn Grothman.csv'), ('pct_true', 0.9869460463523865)])
OrderedDict([('tweet', 'As a result of our unprecedented efforts, illegal crossings are down 75 percent since May - dropping 8 straight months in a row.  And as the wall goes up, drug seizures rise, and border crossings go down. -@realdonaldtrump #SOTU'), ('dates', '2020-02-05 03:05:03'), ('file_name', 'Glenn Grothman.csv'), ('pct_true', 0.6067637205123901)])
OrderedDict([('tweet', 'I am calling for bipartisan legislation that achieves the goal of dramatically lowering prescription drug prices.  Get a bill to my desk, and I will sign it into law without delay. @realdonaldtrump #SOTU'), ('dates', '2020-02-05 02:47:52'), ('file_name', 'Glenn Grothman.csv'), ('pct_true', 0.617496132850647)])
OrderedDict([('tweet', "Too many people have lost their jobs through no fault of their own due to #COVID19. I wholeheartedly supported today's legislation to let these individuals know we’re going to make sure they can pay their bills and support their families. https://t.co/ypVeNUcG0u"), ('dates', '2020-04-23 22:16:53'), ('file_name', 'Glenn Thompson.csv'), ('pct_true', 0.6514788866043091)])
OrderedDict([('tweet', 'Today, the Senate votes on the #BornAlive bill, legislation that would require medical care for babies who have survived abortion. Despite House Democrats repeatedly blocking these efforts to #EndInfanticide, I will always fight for life. We must be a voice for the voiceless. https://t.co/9dbV1ZdAnp'), ('dates', '2020-02-25 21:47:26'), ('file_name', 'Glenn Thompson.csv'), ('pct_true', 0.738387942314148)])
OrderedDict([('tweet', "#Impeachment has been sucking the air out of the room for far too long. Now that this stunt is in the rear view mirror, let's get back to the people's work. https://t.co/Q1Ij3cfQtD"), ('dates', '2020-02-05 21:46:32'), ('file_name', 'Glenn Thompson.csv'), ('pct_true', 0.7421368360519409)])
OrderedDict([('tweet', 'On the Floor to highlight my joining #EPA &amp; #ACE last week in #Pittsburgh announcing the #NavigableWatersProtection Rule that replaces the flawed 2015 #WOTUS Rule.\nWith clearly defined state/federal regulations our farmers can focus providing us the #food, fiber &amp; energy we need!'), ('dates', '2020-01-28 14:59:16'), ('file_name', 'Glenn Thompson.csv'), ('pct_true', 0.5041451454162598)])
OrderedDict([('tweet', "Trump's proposed payroll tax cut will harm Social Security and greatly benefit large corporations. We should be putting money into the hands of those who need it most during #COVID19: seniors, communities of color, and people with disabilities - NOT corporations."), ('dates', '2020-05-07 19:02:36'), ('file_name', 'Grace F. Napolitano.csv'), ('pct_true', 0.9637411236763)])
OrderedDict([('tweet', 'This administration is hypocritical at every turn. #DACA is at risk because Trump terminated it. The President can drop the DACA case before the Supreme Court and extend protections today. Otherwise, these are empty, insincere words from the VP. #ProtectTheDream #HomeIsHere https://t.co/guWkUW3h9o'), ('dates', '2020-04-10 18:33:54'), ('file_name', 'Grace F. Napolitano.csv'), ('pct_true', 0.7857289910316467)])
OrderedDict([('tweet', 'Addressing challenges posed by climate change, like unpredictable drought cycles &amp; water supply, is critical here in Southern CA, where we know the drought is not over! We must prepare our communities &amp; move forward/modernize our nation’s water-related infrastructure. #WRDA2020 https://t.co/AojDGFpV3u'), ('dates', '2020-03-06 19:10:51'), ('file_name', 'Grace F. Napolitano.csv'), ('pct_true', 0.6251907348632812)])
OrderedDict([('tweet', "Women’s rights should not depend on the current political agenda, whims of Congress, or who is sitting in the White House. These basic and fundamental rights should and must be guaranteed. That's why we need the #ERANOW! https://t.co/y1F9Zez5JX"), ('dates', '2020-02-13 15:57:16'), ('file_name', 'Grace F. Napolitano.csv'), ('pct_true', 0.6157827973365784)])
OrderedDict([('tweet', '.@Pix11News reports on my resolution to denounce anti-Asian sentiment and the need to combat increased xenophobia and hate crimes caused by the outbreak of the #coronavirus. #AAPI https://t.co/CGZRFZL7bX'), ('dates', '2020-05-06 21:07:39'), ('file_name', 'Grace Meng.csv'), ('pct_true', 0.5487707257270813)])
OrderedDict([('tweet', 'Once again, @realDonaldTrump is spreading xenophobia &amp; fear as he uses #coronavirus to further his anti-immigrant agenda &amp; deflect from his handling of the crisis. We continue to stand w/ our hard working immigrants, many of whom are on the front lines responding to this crisis. https://t.co/s0VskBDKPC'), ('dates', '2020-04-21 13:56:45'), ('file_name', 'Grace Meng.csv'), ('pct_true', 0.7740288972854614)])
OrderedDict([('tweet', 'On the eve of #YomHaShoah we honor the 6 million Jewish lives that were killed during the Holocaust. We will never forget this dark period in our history and recommit to speak out whenever and wherever hatred and bigotry rises. #HolocaustRemembranceDay'), ('dates', '2020-04-20 23:12:11'), ('file_name', 'Grace Meng.csv'), ('pct_true', 0.5734391212463379)])
OrderedDict([('tweet', 'Thanks to @SenKamalaHarris, @SenDuckworth, @maziehirono for announcing the Senate version of my resolution to condemn discrimination against Asian Americans caused by the #COVID19 crisis. Congress must take a strong stand against this xenophobia &amp; bigotry. https://t.co/YBt28ICThD'), ('dates', '2020-04-05 16:46:15'), ('file_name', 'Grace Meng.csv'), ('pct_true', 0.6852923631668091)])
OrderedDict([('tweet', 'The #COVID19 pandemic is shining a spotlight on why we need to support women &amp; families with equal pay, paid leave &amp; sick days, and affordable child care. #EqualPayDay'), ('dates', '2020-04-01 00:59:18'), ('file_name', 'Grace Meng.csv'), ('pct_true', 0.5682281851768494)])
OrderedDict([('tweet', 'Calling #coronavirus Chinese virus, Wuhan virus, Kung-flu is not only disgusting, it threatens the safety of Asian Americans. Congress must take a strong stand against the rise in anti-Asian sentiment &amp; today I’ve intro’d a resolution that would do that. https://t.co/EvyzqWlN1B'), ('dates', '2020-03-25 22:01:47'), ('file_name', 'Grace Meng.csv'), ('pct_true', 0.9536147117614746)])
OrderedDict([('tweet', '👏It\'s👏called👏the👏#coronavirus👏\n \n@realDonaldTrump must stop saying "Chinese virus" or "Wuhan virus."\n \nAsian Americans are being verbally &amp; physically assaulted. \n \nMr. President, your words have deadly consequences that affects their lives. #AAPIs https://t.co/x1Vrle7Eqf'), ('dates', '2020-03-23 18:16:16'), ('file_name', 'Grace Meng.csv'), ('pct_true', 0.726711630821228)])
OrderedDict([('tweet', '.@SpeakerPelosi &amp; House Dems are in the middle of an historic power-grab. \n\nCongress has met in person for 231 yrs, but w/ Congress no longer meeting in person, it’s clear Pelosi is using this crisis to bypass Republican input so she can push her agenda &amp; attack @realDonaldTrump.'), ('dates', '2020-05-07 18:48:18'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.9087540507316589)])
OrderedDict([('tweet', 'We have doctors, nurses &amp; grocery store workers out there working safely &amp; securely to support our nation through this unprecedented challenge, but I guess @SpeakerPelosi and the Democrats don’t think the House of Representatives has a duty to serve the people. https://t.co/yP0wtgKmAV'), ('dates', '2020-05-01 16:02:53'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.5398486256599426)])
OrderedDict([('tweet', 'Hoosier business owners &amp; employers are suffering because of #COVID19. They rely on #PPP to help them stay afloat during this unprecedented challenge our country faces. D.C. Democrats must STOP blocking funding to #PPP - the political games are unacceptable.'), ('dates', '2020-04-18 17:25:48'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.8161035776138306)])
OrderedDict([('tweet', 'The American people need action. Right now our nation needs leadership, not politics. It is shocking &amp; completely irresponsible for @SpeakerPelosi &amp; @SenSchumer to torpedo a bipartisan bill that would provide relief for families &amp; our suffering economy plagued by #Coronavirus. https://t.co/ta6ZfKeFgM'), ('dates', '2020-03-23 01:34:33'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.8896657824516296)])
OrderedDict([('tweet', 'There must be consequences for #sanctuarycities that thumb their nose at Congress &amp; endanger the lives of their citizens, when all the while they have their hands out for federal $$$ from hardworking American taxpayers. This is the right move from @realDonaldTrump. https://t.co/dtK0dMWK1o'), ('dates', '2020-03-05 19:10:05'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.8275146484375)])
OrderedDict([('tweet', "I'm disgusted by @HouseDemocrats transparent attempt to use the #EqualRightsAmendment to push their extreme abortion agenda. Not only is the #ERA anti-life &amp; anti-women, but could force taxpayers to fund unlimited, unrestricted abortions. We cannot relent in the fight for life."), ('dates', '2020-02-13 16:25:24'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.8636428713798523)])
OrderedDict([('tweet', 'The Deep State has been trying to impeach @realDonaldTrump since day one. \n\nToday, @SpeakerPelosi’s partisan circus came to an end. \n\nJustice for the American People who voted our President into the White House.\n\nNow, Congress, let’s get back to the People’s work already!'), ('dates', '2020-02-05 21:33:21'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.7794030904769897)])
OrderedDict([('tweet', 'The American economy is booming, unemployment rates are the lowest in the history of our nation, and wages are growing for middle class families. With #USMCA signed and the Phase One trade deal with China negotiated, Hoosiers in my district are thriving.'), ('dates', '2020-02-05 03:44:46'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.5188310742378235)])
OrderedDict([('tweet', '🚨🚨🚨 @realDonaldTrump has signed the #USMCA, sending the already booming Trump economy into the stratosphere.\n\nAmerican farmers and manufacturers put their trust in President Trump and he kept his promise to put them first. An historic day!'), ('dates', '2020-01-29 19:26:44'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.8809220194816589)])
OrderedDict([('tweet', "I applaud @realDonaldTrump's repeal of Obama-era #WOTUS. Our farmers can now worry less about gov overreach on their own property &amp; instead focus on growing their farms &amp; small businesses. We finally have an Administration that puts farmers first. For Hoosiers that's a big deal💯"), ('dates', '2020-01-23 21:18:26'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.662611722946167)])
OrderedDict([('tweet', 'Americans aren’t fooled by @SpeakerPelosi’s continuing sham impeachment. Just more political games. Instead of working on behalf of the People, her impeachment agenda continues. A total waste of time &amp; taxpayer money. I will continue to put Indiana’s Sixth District first.'), ('dates', '2020-01-15 15:25:14'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.91453617811203)])
OrderedDict([('tweet', 'The War Powers resolution is ill-timed and reckless political theatre from @SpeakerPelosi with the sole purpose of undermining Commander-in-Chief @realDonaldTrump.  A resolution like this emboldens our enemies and sends the wrong message to our armed forces abroad.'), ('dates', '2020-01-09 18:17:28'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.8172860741615295)])
OrderedDict([('tweet', 'Democrats have been holding American farmers hostage for over a year by blocking passage of the USMCA to deny @realdonaldtrump &amp; working families a victory. They had to score a bogus political point in their impeachment vote before actually getting to work for the People.'), ('dates', '2019-12-19 21:48:27'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.9939560294151306)])
OrderedDict([('tweet', 'Democrats &amp; their allies in the fake news media have been trying to impeach @realDonaldTrump since the day he took office.'), ('dates', '2019-12-18 22:59:35'), ('file_name', 'Greg Pence.csv'), ('pct_true', 0.8311946988105774)])
OrderedDict([('tweet', 'This executive order hurts Arizona, and only serves to demonize and scapegoat immigrants. Our nation is in crisis and rather than put all of his effort behind protecting the American people, the President has taken his eye off the ball. My full statement: https://t.co/q8ilLfERNw'), ('dates', '2020-04-23 00:18:11'), ('file_name', 'Greg Stanton.csv'), ('pct_true', 0.6401432752609253)])
OrderedDict([('tweet', "The Navajo Nation has been disproportionately impacted by COVID-19. In fact, it's lost more people to the virus than 14 states. Tribes need to put all resources possible directly toward combating COVID-19, and they can't bear an additional financial burden right now. https://t.co/P4BT7vHonA"), ('dates', '2020-04-22 21:02:29'), ('file_name', 'Greg Stanton.csv'), ('pct_true', 0.5549463629722595)])
OrderedDict([('tweet', 'The @DeptofDefense decision to approve only 3,000 National Guard Members is woefully inadequate to handle the magnitude of this pandemic. Arizona faces a serious health crisis and needs the expertise and support of @AZNationalGuard to curb the effects of COVID-19.'), ('dates', '2020-03-25 22:51:08'), ('file_name', 'Greg Stanton.csv'), ('pct_true', 0.5479710698127747)])
OrderedDict([('tweet', 'We are in the midst of an unprecedented public health crisis and #Arizona cannot afford to wait until it’s too late to take meaningful, life-saving action on #COVID19. We are not doing enough. There are three more things Arizona can – and must – do *right now*:'), ('dates', '2020-03-17 23:46:02'), ('file_name', 'Greg Stanton.csv'), ('pct_true', 0.5520489811897278)])
OrderedDict([('tweet', 'Today, on\xa0#NationalDayofPrayer, I pray for all Oregonians &amp; Americans but especially those who are suffering from the \xa0coronavirus, our health care workers and first responders taking care of the sick,\xa0and the many unemployed and facing financial insecurity. #InItTogether'), ('dates', '2020-05-07 23:33:44'), ('file_name', 'Greg Walden.csv'), ('pct_true', 0.6109068989753723)])
OrderedDict([('tweet', 'These are unprecedented times, but this is unacceptable. Oregonians are struggling to feed their families and waiting too long to have their claims processed. Congress &amp; @realDonaldTrump provided states w/ the resources. Oregon needs to deliver. https://t.co/bpLS8wUL9l'), ('dates', '2020-04-29 16:09:26'), ('file_name', 'Greg Walden.csv'), ('pct_true', 0.7120553851127625)])
OrderedDict([('tweet', '.@repblumenauer is right. How did OR not see this coming &amp; be better prepared? Congress gave states flexibility &amp; money. The Trump Admin delivered. Now, those Oregonians who lost their jobs face long lines, long delays &amp; maybe less help.  What a mess. https://t.co/34SxWXDNe0 https://t.co/gejhqvshQV'), ('dates', '2020-04-09 16:07:58'), ('file_name', 'Greg Walden.csv'), ('pct_true', 0.6522337794303894)])
OrderedDict([('tweet', 'Shameful. Absolutely shameful. After needlessly delaying passage of the CARES Act less than two weeks ago, @SpeakerPelosi and @SenSchumer are at it again – delaying critical support to a nation that’s hurting.'), ('dates', '2020-04-09 15:56:24'), ('file_name', 'Greg Walden.csv'), ('pct_true', 0.8400269150733948)])
OrderedDict([('tweet', 'Americans are sick. People are hurting. Savings are being wiped out. Small businesses are closing in waves. People are being laid off. Hospitals need resources. Medical professionals need supplies. #COVID19'), ('dates', '2020-03-23 20:43:24'), ('file_name', 'Greg Walden.csv'), ('pct_true', 0.9077943563461304)])
OrderedDict([('tweet', "Local restaurants have been devastated by COVID, and they're likely to continue hurting even as our nation begins to open. I led a letter to leadership outlining what measures Congress can take next to address their specific needs. Read more here: https://t.co/LTIzWf6tMh https://t.co/SYk4b46x8D"), ('dates', '2020-05-06 21:16:07'), ('file_name', 'Gregory W. Meeks.csv'), ('pct_true', 0.5882255434989929)])
OrderedDict([('tweet', "This is an absurd attempt to scapegoat and rewrite history. This administration was briefed by its own intelligence about coronavirus, and ignored the ugly truth for political reasons. Cutting funding to the WHO doesn't erase that, it just makes a global pandemic worse. https://t.co/BSZdnZrZgX"), ('dates', '2020-04-14 22:41:19'), ('file_name', 'Gregory W. Meeks.csv'), ('pct_true', 0.8265660405158997)])
OrderedDict([('tweet', "Structural inequities that have existed for small businesses long before COVID is now hurting their ability to receive aid. Dems are working to make sure they don't slip through the cracks. \n\nThat isn't hostage taking, that's called negotiation. Read:\nhttps://t.co/RLW1CvWKF4"), ('dates', '2020-04-13 17:28:07'), ('file_name', 'Gregory W. Meeks.csv'), ('pct_true', 0.7290768623352051)])
OrderedDict([('tweet', "This is vile. My thoughts are with this poor woman. In a vain attempt to deflect blame, Trump's dangerous rhetoric is putting people in harm's way and it needs to stop. https://t.co/4SBJPweoto"), ('dates', '2020-04-07 17:51:17'), ('file_name', 'Gregory W. Meeks.csv'), ('pct_true', 0.8774269819259644)])
OrderedDict([('tweet', "There are people in my community who are dying -  DYING - while this administration flippantly tells state's to figure it out for themselves. There is no leadership from the top, only lies, and I'm tired of it. https://t.co/LMgZ36h2ZJ"), ('dates', '2020-04-07 17:32:10'), ('file_name', 'Gregory W. Meeks.csv'), ('pct_true', 0.776604413986206)])
OrderedDict([('tweet', 'Small business owners are contacting my office about their #PPPloan applications being rejected by @BankofAmerica and @Chase because they don’t have existing credit accounts with the banks.  This requirement is not in the #StimulusBill, and banks should remove them immediately.'), ('dates', '2020-04-03 18:50:08'), ('file_name', 'Gregory W. Meeks.csv'), ('pct_true', 0.9144638180732727)])
OrderedDict([('tweet', "Horrific reports out of Kabul yesterday as ISIS extremists targeted Sikh worshippers. I stand with religious minorities, including Sikhs worldwide. Religious freedom is a human right - we must push back against those who'd attack peaceful practitioners.\nhttps://t.co/TFTwBbKky6"), ('dates', '2020-03-26 17:28:36'), ('file_name', 'Gregory W. Meeks.csv'), ('pct_true', 0.7041593194007874)])
OrderedDict([('tweet', "Not comforting to see POTUS burying his head in the sand during a crisis, thinking that will save face. It was a fair question and tracks with people's experiences. \n\nTests were greatly delayed. Tests are what we'll need to control the situation and start returning to normal. https://t.co/6WhgoItDej"), ('dates', '2020-03-20 17:23:02'), ('file_name', 'Gregory W. Meeks.csv'), ('pct_true', 0.5898923873901367)])
OrderedDict([('tweet', "Many constituents have asked about an extension of the tax filing deadline.  President Trump's Executive Order has just pushed back the tax filing date to July 15. 2020."), ('dates', '2020-03-20 15:14:51'), ('file_name', 'Gus M. Bilirakis.csv'), ('pct_true', 0.6684300303459167)])
OrderedDict([('tweet', 'Pennsylvania’s fight against the #COVID19 pandemic has upended our economy and rendered one in four workers jobless, all while putting our seniors in care homes at greater risk because of a serious policy flaws by Gov. Tom Wolf. https://t.co/Wu9wCcaqwI'), ('dates', '2020-05-09 20:27:42'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.5695889592170715)])
OrderedDict([('tweet', 'Today, I introduced the TRAP Act to repeal the 1799 Logan Act, a law recently used by @Comey’s politicized FBI to set up General Flynn.\n \nThe Logan Act is an archaic &amp; useless law. In 221 years of existence, only 2 people have been prosecuted under the Act &amp; both were acquitted. https://t.co/czy8aWVZ3D'), ('dates', '2020-05-08 15:37:39'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.6711545586585999)])
OrderedDict([('tweet', "Michael Flynn was an innocent man set up by @Comey &amp; Strzok. Their hatred for @realdonaldtrump led them to overstep their boundaries in search of taking down the president.\n\nAfter 3 years of Democrats destroying Flynn’s reputation, I'm glad there's finally vindication &amp; justice. https://t.co/WNi551pnf2"), ('dates', '2020-05-07 19:22:50'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.5030005574226379)])
OrderedDict([('tweet', 'The 1799 Logan Act, recently used by Jim @Comey’s politicized FBI to set up @realdonaldtrump’s former NSA Michael Flynn, is nothing more than a political weapon. I will be introducing legislation to repeal the Logan Act and end abuse of this unconstitutional statute.'), ('dates', '2020-05-06 18:40:29'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.6009411215782166)])
OrderedDict([('tweet', 'It’s past time Congress gets to the bottom of how this pandemic began &amp; China’s failed response. Speaker Pelosi sat on the CCP’s misdeeds &amp; refuses to condemn them for their role in the spread of #COVID19. I’m glad @gopleader is holding China accountable.\nhttps://t.co/fzca2rmArp'), ('dates', '2020-05-06 15:00:19'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.8191298246383667)])
OrderedDict([('tweet', 'Not sure @Politico realizes that twitter is banned for essentially all Chinese citizens. “Chinese twitter” is purely CCP propaganda and Politico is playing right into their hands. https://t.co/HwiaD40E5y https://t.co/4aZtKc3ivX'), ('dates', '2020-05-05 19:11:18'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.8568886518478394)])
OrderedDict([('tweet', 'The bottom line of the Democrats’ behavior is this: They hate @realDonaldTrump so badly that they would rather side with the Chinese Communist Party than defend Americans and defend our spending. \nhttps://t.co/GiIqBunwEb'), ('dates', '2020-05-05 15:08:01'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.9396948218345642)])
OrderedDict([('tweet', "The new Joe Biden saga shows another instance of the Democrats' hypocrisy. For the past two weeks, Speaker Pelosi and Democrats have glossed over these accusations.\n\nDemocrats and the mainstream media only believe the victim if it fits within their narrative. https://t.co/3YhXSs5aLH"), ('dates', '2020-05-01 23:29:29'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.7911942005157471)])
OrderedDict([('tweet', '#COVID19 is reinforcing the need to reevaluate our supply chains. It’s unacceptable our country relies so heavily on the CCP for our nations’ critical pharmaceuticals &amp; PPE.\n\nIt’s time to bring manufacturing back to America.'), ('dates', '2020-04-30 20:20:13'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.795252799987793)])
OrderedDict([('tweet', "These new FBI files prove there was a clear &amp; orchestrated plan to set up General Flynn, showing just how deep the hatred of @realDonaldTrump ran at @Comey's FBI. \n \nComey &amp; his merry band of never-Trumpers’ politicization of our nation’s intel community is deeply troubling. https://t.co/7dvL16XVxi"), ('dates', '2020-04-30 17:39:34'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.5018101334571838)])
OrderedDict([('tweet', "China is trying to take control of the World Health Organization and other international organizations. \n\nWe need to ensure @WHO isn't just doing the bidding of China, but actually working to promote public health and stop pandemics. https://t.co/VBr9HAajDf"), ('dates', '2020-04-28 23:00:00'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.8971220850944519)])
OrderedDict([('tweet', 'Congressional Democrats’ plan to combat #COVID19:\n\nDelay funding for small businesses for weeks ☑️\n\nBlame @realdonaldtrump for everything ☑️\n\nContinue funding a propaganda wing of the CCP ☑️\n\nDems would rather fund Chinese propaganda than help small businesses survive COVID-19. https://t.co/FOG2KiMZkF'), ('dates', '2020-04-28 21:18:24'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.8648251891136169)])
OrderedDict([('tweet', 'Coddling the Chinese Communist Party is @drtedros &amp; @who’s number one priority. They have shown that appeasing the CCP is more important than saving lives. https://t.co/DDoikrKAws'), ('dates', '2020-04-26 21:36:43'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.9558374285697937)])
OrderedDict([('tweet', 'China continues to buy more influence at WHO. \n\nTheir puppet, @DrTedros, did exactly what the Chinese Communist Party wanted and now his organization is being rewarded with an extra $30 million in funding. https://t.co/EbEmnfj6uu'), ('dates', '2020-04-24 18:20:22'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.9624945521354675)])
OrderedDict([('tweet', 'Today the House will pass the Paycheck Protection Program &amp; Health Care Enhancement Act. \n\n@WhiteHouse requested replenishment for this vital program 16 days ago. \n\n@SpeakerPelosi &amp; Congressional Dems twiddled their thumbs for 2 weeks while hardworking Americans missed paychecks.'), ('dates', '2020-04-23 16:17:18'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.5115479230880737)])
OrderedDict([('tweet', "Most Americans know someone or have a loved one who has been affected by Alzheimer’s. Alzheimer's is the 6th leading cause of death in the US and we must continue to fund research to find a cure. Thats why I’m supporting the Younger-Onset Alzheimer’s Act. https://t.co/I63Kh02TJk"), ('dates', '2020-04-22 18:42:40'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.5147239565849304)])
OrderedDict([('tweet', 'While I’m glad the Senate finally passed further #PaymentProtectionProgram funding, this is something that should’ve been done a week ago. \n\nDemocrats continue to drag their feet when it comes to getting meaningful aid to small businesses &amp; the Americans who work for them.'), ('dates', '2020-04-21 22:10:42'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.690760612487793)])
OrderedDict([('tweet', 'Pelosi needs to stop blaming @realdonaldtrump &amp; start getting money to our struggling small businesses. They are filing for bankruptcy daily while she delays  #PaycheckProtectionProgram replenishment. \n\nWorkers need relief ASAP. It’s time to put the American people over politics. https://t.co/4ik5iHwTFq'), ('dates', '2020-04-20 16:05:24'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.8543792963027954)])
OrderedDict([('tweet', "It's time Congress get back to work.\n\nIt is shameful that we are not in DC leading the country through this crisis. We were elected to pass laws that help the American people and that's exactly what Congress should be doing right now. https://t.co/MnWuWulVSz"), ('dates', '2020-04-17 21:10:37'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.6706435680389404)])
OrderedDict([('tweet', "Congressional Democrats will take any opportunity to oppose &amp; attack @realdonaldtrump.\n \nThey’ve gone as far as siding with the Chinese Communist Party in defense of WHO's failures to contain #COVID19.\n \nThey played right into Putin's hands, now they're playing right into Xi's. https://t.co/ltSDL3rpjr"), ('dates', '2020-04-17 15:47:05'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.9350963234901428)])
OrderedDict([('tweet', "We need a nuanced and balanced approach to reopen our economy during #COVID19. All areas are not the same and we cannot paint this response with a broad stroke.\n\nUnder @realdonaldtrump's leadership, states and governors have a responsible road map to get America working again. https://t.co/GhHlQVekny"), ('dates', '2020-04-16 22:36:43'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.5121232867240906)])
OrderedDict([('tweet', 'Funds allocated to small businesses &amp; their employees during #COVID19 are completely gone.\n \nOnly one thing stands in the way of more funding: Democrats in Congress.\n \nDems need to put their reckless political games aside to pass this critical #PaycheckProtectionProgram funding.'), ('dates', '2020-04-16 15:59:35'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.7668653130531311)])
OrderedDict([('tweet', 'We’re the United States. A virus is not going to kill the American Dream. We’ve been through two world wars, we’ve been through the Spanish Influenza, we’ll keep marching. \n\nThe United States cannot be stopped by a virus. #InItTogether https://t.co/ShlnSp1vwh'), ('dates', '2020-04-10 13:23:19'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.5238735675811768)])
OrderedDict([('tweet', "My resolution has 2 requirements:\n\n1⃣ WHO Director-General @DrTedros must step down. He's either grossly negligent or complicit in helping the CCP cover-up #COVID19. \n\n2⃣ If WHO wants funding, they must commit to an investigation on how the CCP &amp; WHO mishandled this pandemic. https://t.co/9EtuUhc8MK"), ('dates', '2020-04-10 00:06:33'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.5012770295143127)])
OrderedDict([('tweet', 'The Chinese Communist Party lied from the start about #COVID19 and they jailed doctors for speaking up about it.\n \n@WHO was at best negligent in challenging the CCP’s narrative even when friends like Taiwan warned back in December that human-to-human transmission was possible. https://t.co/0aT4xnZ7VP'), ('dates', '2020-04-08 19:33:55'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.9299837946891785)])
OrderedDict([('tweet', 'We should not be funding an organization that abandoned its mission by putting China’s interests above global health and safety. @DrTedros must go ASAP.\nhttps://t.co/1AzIQdWWnB'), ('dates', '2020-04-07 19:46:38'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.609525740146637)])
OrderedDict([('tweet', "Instead of working to save lives around the world, @WHO stood by and downplayed the severity of the virus to avoid offending Chinese officials. Their actions to cover up China's failed #COVID19 response must be investigated. https://t.co/xhy26GcM5P"), ('dates', '2020-04-07 16:48:52'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.8350905179977417)])
OrderedDict([('tweet', "Today, I introduced legislation calling on Congress to withhold federal funds from the @WHO until @DrTedros resigns.\n \nAmerica is the WHO’s largest contributor. It’s not right that Americans' hard-earned taxpayer dollars are used to propagate the CCP’s lies. https://t.co/WAaDSXaQDB"), ('dates', '2020-04-07 16:07:24'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.5671856999397278)])
OrderedDict([('tweet', "The World Health Organization helped the Chinese Communist Party hide the threat of #COVID19. Had China acted 3 weeks sooner, the # of cases could've been reduced by 95%. I'll be introducing a bill to hold the WHO accountable for their negligence &amp; deceit. https://t.co/KjKCOGP6W2"), ('dates', '2020-04-06 14:02:00'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.8948094844818115)])
OrderedDict([('tweet', 'If the Chinese Communist Party was transparent about the threat posed by #COVID19 from the onset, the rest of the world would be much better prepared to combat the virus. Retweet if you agree. https://t.co/ILJEehSzPQ'), ('dates', '2020-04-04 01:08:44'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.6218023896217346)])
OrderedDict([('tweet', 'New evidence shows us what most have suspected all along: The Chinese Communist Party has been lying about its #COVID19 data leaving the rest of the world ill-prepared to fight the eventual pandemic.\nhttps://t.co/lOH4wf3ZHb'), ('dates', '2020-04-01 17:00:18'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.9156918525695801)])
OrderedDict([('tweet', "The Phase 3 #stimulusbill has passed the Senate. By putting money straight into Americans' pockets, we’re sending relief directly to those who need it most. \n\nThis is a win for all Americans. We can overcome anything when we come together. #GreatAmericanComeback #InItTogether"), ('dates', '2020-03-26 13:24:11'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.8700324892997742)])
OrderedDict([('tweet', 'We need to call out the Chinese Communist Party for lying to the world about the coronavirus and failing to contain it in Wuhan. Today I discussed with @Richzeoli ⬇️ https://t.co/AJfh5ngesk'), ('dates', '2020-03-25 20:13:40'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.7277970314025879)])
OrderedDict([('tweet', 'GOP and Democrat Senators worked together to create the bipartisan CARES Act. @SpeakerPelosi tanked the whole bill because it was drafted by McConnell instead of her. \n\n#SenateDemocrats and Pelosi would rather see a great bill die than see Republicans get a win. https://t.co/GMIuUHN7G1'), ('dates', '2020-03-24 20:19:00'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.7239789366722107)])
OrderedDict([('tweet', 'The Chinese government purposefully misled the public from the very start of the cornonavirus. Instead of warning the world about the deadly disease, they chose to cover it up and lie about its lethality. https://t.co/nDVit12aFD'), ('dates', '2020-03-24 13:35:14'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.7742989659309387)])
OrderedDict([('tweet', 'Speaker Pelosi &amp; #SenateDemocrats are holding hostage critical relief for Americans for policies that have nothing to do with coronavirus:\n \n❌Strangling airlines with zero carbon emission requirements\n\n❌ Collective bargaining for federal workers\n\n❌ Same day voter registration'), ('dates', '2020-03-23 21:10:35'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.7367106676101685)])
OrderedDict([('tweet', 'Dems claim there’s no oversight in the stimulus, but the bill forbids stock buybacks for the duration of the loans and makes sure they maintain existing employees. \n\nDems using a crisis to push their radical agenda by adding tax credits for solar &amp; wind energy is unthinkable.'), ('dates', '2020-03-23 18:36:28'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.9916439652442932)])
OrderedDict([('tweet', '#SenateDemocrats are delaying economic stability to the American people. It’s time to end these political games. https://t.co/JyYzQ6c6Oa'), ('dates', '2020-03-23 17:18:15'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.5093111991882324)])
OrderedDict([('tweet', 'The Democrats blocking critical coronavirus relief is incredibly disappointing and irresponsible. \n\nSpeaker Pelosi and Senate Dems are playing partisan political games while many Americans wonder how they will pay their bills and put food on the table. Americans need help now.'), ('dates', '2020-03-23 00:05:16'), ('file_name', 'Guy Reschenthaler.csv'), ('pct_true', 0.9688781499862671)])
OrderedDict([('tweet', 'Flynn is guilty. He confessed his guilt (twice). \n\nBarr asking for charges dropped for a Trump crony is flat-out corrupt. \n\nIf Mitch McConnell and the GOP-controlled Senate weren’t complicit in the Admin’s corruption, they would demand his resignation. https://t.co/1m2uUnMCjU'), ('dates', '2020-05-07 23:44:10'), ('file_name', 'Gwen Moore.csv'), ('pct_true', 0.9102007150650024)])
OrderedDict([('tweet', 'Republicans blocked ACA payments, which caused your premium to increase. \n\nNow, the SCOTUS ruled that their efforts to undermine the ACA were wrong.\n\nDo you trust Republicans with your health care? https://t.co/9F2jVal5n8'), ('dates', '2020-04-27 22:25:52'), ('file_name', 'Gwen Moore.csv'), ('pct_true', 0.6219145655632019)])
OrderedDict([('tweet', "I remember a time not so long ago when @realDonaldTrump was concerned about politicians using coronavirus hysteria to push their policy objectives. While Dems wanted to help struggling families, Trump just gave America's wealthiest polluters a free pass. \nhttps://t.co/NHioSExjrH"), ('dates', '2020-03-27 21:08:58'), ('file_name', 'Gwen Moore.csv'), ('pct_true', 0.9297559857368469)])
OrderedDict([('tweet', 'Republicans are using a public health crisis to deny women autonomy over their own bodies. This is dangerous and plain wrong. Abortion is essential health care. https://t.co/99pd7HtAwv'), ('dates', '2020-03-25 20:57:38'), ('file_name', 'Gwen Moore.csv'), ('pct_true', 0.5849971771240234)])
OrderedDict([('tweet', 'I stopped by an essential, family-run business today for a service, and the owners wanted to know what is wrong with Nancy Pelosi. Their PPP loan application is in, but because Nancy Pelosi and Chuck Schumer have blocked more funding, they won’t be receiving a loan anytime soon.'), ('dates', '2020-04-17 15:18:09'), ('file_name', 'H. Morgan Griffith.csv'), ('pct_true', 0.5960292220115662)])
OrderedDict([('tweet', 'Here we go again. Are the Democrats blocking more aid for small businesses unless they can get taxpayer dollars for unrelated agenda items like the Kennedy Center? https://t.co/25DQmmHPVH'), ('dates', '2020-04-09 14:51:21'), ('file_name', 'H. Morgan Griffith.csv'), ('pct_true', 0.8308399319648743)])
OrderedDict([('tweet', 'I just voted on the House floor to condemn Bernie Sanders’ positive comments on Fidel Castro. The Castro regime was built on political repression and economic misery, and Senator Sanders should stop trying to minimize this shameful record.'), ('dates', '2020-02-27 19:58:58'), ('file_name', 'H. Morgan Griffith.csv'), ('pct_true', 0.7820132970809937)])
OrderedDict([('tweet', 'Brexit was a choice made by the people of the U.K. exercising their rights as a sovereign nation. As such, it must be respected.\n\nWhile the economic and political relationship between the U.K. and Europe changes, the Special Relationship between the U.K. and the U.S. endures.'), ('dates', '2020-01-31 15:01:28'), ('file_name', 'H. Morgan Griffith.csv'), ('pct_true', 0.5593515634536743)])
OrderedDict([('tweet', 'Democrats want the people of Marion to help shoulder the tax burden in Manhattan and the people of Stuart to subsidize the tax burden of living in San Francisco. That is unfair to the people of the 9th District and most of America. My statement on SALT ⬇️ https://t.co/1EzWtXNXwE'), ('dates', '2019-12-19 21:38:05'), ('file_name', 'H. Morgan Griffith.csv'), ('pct_true', 0.597850501537323)])
OrderedDict([('tweet', 'House Democrats today abused the power of impeachment. I am sorry they did so, for the millions of Americans who voted for President Trump and for the Constitution that has been damaged. https://t.co/oACi0GWQLJ'), ('dates', '2019-12-19 02:01:53'), ('file_name', 'H. Morgan Griffith.csv'), ('pct_true', 0.9030241966247559)])
OrderedDict([('tweet', 'I will be voting no on impeachment today. A few people under the Bristol sign might disagree, but I have found no impeachable offenses, just partisan politics. https://t.co/Dl5svWig9R'), ('dates', '2019-12-18 14:39:07'), ('file_name', 'H. Morgan Griffith.csv'), ('pct_true', 0.711927056312561)])
OrderedDict([('tweet', 'The Democrat impeachment resolution is an attempt to ratify their ongoing faulty process. \n\nIt is not a serious document, and it will not provide the transparency and fairness required of an impeachment inquiry. https://t.co/xzOBcHNrUB'), ('dates', '2019-10-30 21:01:41'), ('file_name', 'H. Morgan Griffith.csv'), ('pct_true', 0.6046551465988159)])
OrderedDict([('tweet', 'Here is the resolution being rushed by Democrats to the floor without any Republican input to ratify their “impeachment inquiry.” They are not capable of running a fair, transparent, and impartial process. https://t.co/UiPnHCXoBw'), ('dates', '2019-10-29 19:27:56'), ('file_name', 'H. Morgan Griffith.csv'), ('pct_true', 0.7387363314628601)])
OrderedDict([('tweet', 'The #COVID19 pandemic is a public health crisis. \n\nTens of thousands have died.\n\nMillions w/ pre-existing health conditions are at greater risk. \n\nTrump administration continues its effort to eradicate healthcare protections in court. \n\nThey. Must. Be. Stopped. #ProtectOurCare'), ('dates', '2020-04-28 21:21:11'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5457906723022461)])
OrderedDict([('tweet', 'Calls to poison control hotlines throughout America have apparently increased significantly. \n\nIgnorance in high places is a dangerous thing.\n\nHouse Dems are pushing evidence and data-based solutions to the #coronavirus pandemic.\n\n#ForThePeople'), ('dates', '2020-04-27 14:20:41'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.9629747867584229)])
OrderedDict([('tweet', 'Thousands are dying.\n\nMillions are unemployed.\n\nRome is burning.\n\nMcConnell thinks states should declare bankruptcy.\n\nHouse Dems are fighting to end this pandemic and revive the economy #ForThePeople. https://t.co/2tlAUfh9fc'), ('dates', '2020-04-23 17:01:47'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.8570724725723267)])
OrderedDict([('tweet', 'America produced an unprecedented amount of warplanes, tanks and battleships in the effort to crush Nazi Germany. \n\nBut now we can’t manufacture enough cloth masks and cotton swabs to defeat #coronavirus?\n\nLeadership matters.\n\nFDR had it. \n\nIt’s missing right now.'), ('dates', '2020-04-21 14:03:14'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.9023967385292053)])
OrderedDict([('tweet', 'People are dying. \n\nBills are mounting.\n\nFamilies are hurting.\n\nAnd all this guy seems to care about is a stock market comeback.\n\nSAD.'), ('dates', '2020-04-16 18:16:09'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.821802020072937)])
OrderedDict([('tweet', 'We are in the midst of a devastating pandemic.\n\nAmericans are dying.\n\nMillions of jobs lost.\n\nGrow Up. https://t.co/da9BfrJtgH'), ('dates', '2020-04-14 22:36:46'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5527468919754028)])
OrderedDict([('tweet', 'What does Mitch McConnell not get?\n\nWe live in a democracy. \n\nNot a dictatorship. \n\nHe alone doesn’t get to decide how pandemic relief is spent. \n\nHouse Dems are fighting #ForThePeople ✊🏾👊🏾💪🏾. https://t.co/SsvW5TjN5p'), ('dates', '2020-04-14 13:34:14'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.6054564714431763)])
OrderedDict([('tweet', 'The poor, the sick and the afflicted in vulnerable communities have a 🎯 on their back. \n\nState and local budgets are collapsing. \n\nThat hurts the least, lost and left behind.\n\nCongress must step up and direct assistance to where it’s needed most.\n\n🙏🏽✊🏾👊🏾🇺🇸🇺🇸🇺🇸 https://t.co/u8Z1oZTAAM'), ('dates', '2020-04-11 18:01:57'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5195580124855042)])
OrderedDict([('tweet', 'House Dems passed the #CARESAct and sent it to the President’s desk approximately two weeks ago.\n\nBut the American people have still not received the direct payment checks we authorized.\n\n#WhereIsTheMoney'), ('dates', '2020-04-09 12:52:47'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7322932481765747)])
OrderedDict([('tweet', 'Stop the xenophobia.\n\nStop the anti-Asian violence.\n\nStop the inhumanity.\n\nThe only way we will defeat the #COVID19 pandemic is TOGETHER.\n\n#FlattenTheCurve https://t.co/EnquPFOO2C'), ('dates', '2020-04-06 18:15:33'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7020270228385925)])
OrderedDict([('tweet', 'With man this is impossible, but not with God; all things are possible with God. Mark 10:27\n\n#PalmSunday'), ('dates', '2020-04-05 18:16:08'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.702809751033783)])
OrderedDict([('tweet', 'The #coronavirus pandemic is ravaging the nation. \n\nPeople desperately need help.\n\nBut Trump continues his effort to take away healthcare from more than 100 million Americans w/ pre-existing conditions.\n\nDrop. The. Lawsuit. Donald.\n\n#ProtectOurCare'), ('dates', '2020-04-03 12:56:02'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.8292194604873657)])
OrderedDict([('tweet', 'People are hurting throughout America #coronavirus.\n\nThe President is melting down.\n\nHouse Dems are focused on direct payments to everyday Americans, unemployment assistance to displaced workers and supporting small businesses.\n\n#ForThePeople https://t.co/fPVuKBf43C'), ('dates', '2020-04-02 14:45:18'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.8435558080673218)])
OrderedDict([('tweet', 'The #coronavirus relief package is law and the groveling has begun.\n\nFor years the major cruise lines have sailed under foreign flags to avoid paying the U.S. corporate tax rate.\n\nThey now want the American taxpayer to bail them out. \n\nHow can I say this politely?\n\nHELL NO.'), ('dates', '2020-03-27 22:14:54'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.6425325274467468)])
OrderedDict([('tweet', 'There is no economy if the Coronavirus overwhelms, sickens and kills the American people.\n\nWhy is that so complicated for the President to understand?\n\n#SocialDistancing'), ('dates', '2020-03-24 17:07:03'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7700676321983337)])
OrderedDict([('tweet', 'What does Mitch McConnell not get about this moment?\n\nHouse Dems are in the majority.\n\nWe were elected to stand up #ForThePeople.\n\nThere will be no corporate slush fund.\n\nGet. Over. It.'), ('dates', '2020-03-23 17:22:18'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.9026718735694885)])
OrderedDict([('tweet', 'The Senate majority may consider Congress a wholly-owned subsidiary of the Trump administration.\n\nHouse Dems don’t get down like that.\n\nWe will always fight #ForThePeople.'), ('dates', '2020-03-22 22:45:08'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.8907756209373474)])
OrderedDict([('tweet', 'We must stand up for all displaced workers (and we will).\n\nBut let’s get something straight.\n\nThe major cruise lines sail under foreign flags to avoid paying the U.S. corporate tax rate.\n\nAnd now some want the American taxpayer to bail them out?\n\nGet. Lost.'), ('dates', '2020-03-20 12:06:45'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7324912548065186)])
OrderedDict([('tweet', 'Just so we’re clear.\n\nThe Constitution gives the House sole power to initiate spending on behalf of the American people.\n\nNot the Senate. And certainly not this President.\n\nHouse Dems will prioritize displaced workers, families &amp; small businesses #Stimulus.\n\nPeriod. Full 🛑.'), ('dates', '2020-03-18 16:20:15'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.8801973462104797)])
OrderedDict([('tweet', 'Trump claims that the Coronavirus is under control. \n\nIt is not. \n\nThe President needs to either step up and speak the truth #SocialDistancing.\n\nOr get out of the way.\n\nFACTS.'), ('dates', '2020-03-16 14:49:39'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7718718647956848)])
OrderedDict([('tweet', 'The Trump administration is incompetent and corrupt.\n\nThat has always been the case.\n\nWhy did so many turn a blind eye to the obvious malignancy at the White House?\n\nGREED.\n\n#GOPTaxScam'), ('dates', '2020-03-12 14:20:45'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.8462331891059875)])
OrderedDict([('tweet', 'President Trump will speak to the nation this evening.\n\nQuestion.\n\nDo you still believe the worldwide coronavirus pandemic is a Democratic hoax?'), ('dates', '2020-03-12 00:07:48'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.6444988250732422)])
OrderedDict([('tweet', 'Trump sycophants could care less about the facts related to COVID-19.\n\nThe #Coronavirus is not contained.\n\nIt is spreading like wildfire.\n\nStop lying to the American people.'), ('dates', '2020-03-09 12:40:52'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7643811702728271)])
OrderedDict([('tweet', 'Americans are dying from the #Coronavirus.\n\nThe stock market is in free fall.\n\nTornadoes have ravaged parts of the Southeast.\n\nAnd Donald Trump is back on vacation in Florida. \n\nMake America Great Again? What. A. Fraud.'), ('dates', '2020-03-07 13:51:09'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7569718956947327)])
OrderedDict([('tweet', 'The so-called President and his sycophants are apparently circulating a fake census form on Facebook.\n\nSHRED IT.\n\nSomething is very wrong with these people. \n\nBut justice delayed will not be justice denied.\n\nThese crooks will be held accountable - one way or the other.'), ('dates', '2020-03-05 18:46:31'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5455206036567688)])
OrderedDict([('tweet', 'Gentrification is a problem.\n\nDisplacement is a problem.\n\nOverdevelopment is a problem.\n\nWe must address the #AffordableHousingCrisis with the fierce urgency of now. https://t.co/9oGthWX0v8'), ('dates', '2020-03-03 15:10:01'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5868662595748901)])
OrderedDict([('tweet', 'Supremes will hear another challenge to the Affordable Care Act.\n\nTrump is obsessed with taking away your healthcare.\n\nHouse Dems are committed to strengthening the ACA and protecting more than 100 mil Americans w/ preexisting conditions.\n\nWe will prevail again.\n\n#ForThePeople.'), ('dates', '2020-03-02 17:58:32'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.6652359366416931)])
OrderedDict([('tweet', 'The Coronavirus has killed more than 2900 people.\n\nThis is a global epidemic.\n\nIt’s not a hoax.\n\nThe American people don’t need reckless rhetoric. \n\nWe. Need. Results.'), ('dates', '2020-03-01 16:22:52'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5139326453208923)])
OrderedDict([('tweet', 'Mick Mulvaney just acknowledged what we all knew to be true.\n\nWashington Republicans don’t really care about so-called fiscal responsibility.\n\nThey just pretend to do so whenever a Democrat is in the White House.\n\nSo they can cut Social Security and Medicare.\n\nSHAMELESS.'), ('dates', '2020-02-20 20:52:53'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.9581100344657898)])
OrderedDict([('tweet', 'The GOP continues to disgrace itself. \n\nWealthy and well-connected white collar criminals pardoned.\n\nRepublicans remain silent.\n\nHouse Dems continue to fight for the least, the lost and the left behind.\n\n#ForThePeople'), ('dates', '2020-02-19 16:15:52'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.9882617592811584)])
OrderedDict([('tweet', 'Manufacturing jobs are down.\n\nHealthcare costs are up.\n\nWages remain stagnant.\n\nThat’s why House Democrats are fighting hard #ForThePeople.'), ('dates', '2020-02-16 16:00:10'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.6483013033866882)])
OrderedDict([('tweet', 'Women deserve the full protection of the United States Constitution.\n\nThat’s why House Dems passed the Equal Rights Amendment resolution.\n\nAnd I proudly voted YES.\n\n#ERANow'), ('dates', '2020-02-14 20:32:59'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.6495622396469116)])
OrderedDict([('tweet', 'This guy is delusional.\n\nToo late for a clinical intervention.\n\nThe American people must decisively make clear such behavior is not acceptable.\n\n#DefendOurDemocracy https://t.co/ErcEhnfMw0'), ('dates', '2020-02-14 15:20:35'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.665732741355896)])
OrderedDict([('tweet', 'House Dems are the only thing standing between the Trump Budget and devastating cuts targeting everyday Americans.\n\nHe fights for the wealthy, the well-off and the well-connected.\n\nWe fight #ForThePeople. \n https://t.co/kF5MVn3juu'), ('dates', '2020-02-11 18:53:47'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5026069283485413)])
OrderedDict([('tweet', 'The Republican Governor of New Hampshire just said Donald Trump does not define the party.\n\nFake News.\n\nTrump IS the Republican Party.\n\nThe so-called “verdict” in the United States Senate proved that last week.\n\nSAD.'), ('dates', '2020-02-11 03:15:52'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7181506156921387)])
OrderedDict([('tweet', 'Trump claims he is fighting for you.\n\nHe is lying.\n\nThe Trump budget cuts more than $1 trillion from Medicare and Medicaid.\n\nTogether we will stop 🛑 him.'), ('dates', '2020-02-10 18:33:40'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5004425644874573)])
OrderedDict([('tweet', 'There are more than 300 million guns in America.\n\nThey threaten our children.\n\nAnd the law enforcement community.\n\nHouse Dems have acted to address America’s gun violence epidemic.\n\nIt’s time for the Senate to #DoYourJob.'), ('dates', '2020-02-09 17:42:52'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5574208498001099)])
OrderedDict([('tweet', 'Nixon resigned.\n\nClinton apologized.\n\nTrump holds a hate-filled rally.\n\nBe not discouraged.\n\nHouse Dems will continue our fight to lower drug prices and #DefendOurDemocracy.'), ('dates', '2020-02-06 20:56:52'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7656511664390564)])
OrderedDict([('tweet', 'House Dems are fighting to protect people with pre-existing conditions. President Trump is trying to rip away those healthcare protections from more than 100 million Americans. \n\nWe’re not buying the fraudulent narrative that you’re selling. Period. Full 🛑🛑🛑 https://t.co/Zd5xQ6vEoz'), ('dates', '2020-02-05 05:18:01'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7514509558677673)])
OrderedDict([('tweet', 'House Dems are fighting to protect people with pre-existing conditions. \n\nPresident Trump is trying to rip away those protections.\n\nWe must stop him. \n\nWe will stop him #ForThePeople.'), ('dates', '2020-02-05 00:02:56'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.7152178883552551)])
OrderedDict([('tweet', 'Everyday Americans are still struggling check to check.\n\n#GOPTaxScam is a spectacular failure.\n\nThe economy has suffered a manufacturing recession.\n\nAnd President Trump is back in Florida golfing.\n\nWhat happened to doing the people’s business?'), ('dates', '2020-02-01 17:34:35'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.724949061870575)])
OrderedDict([('tweet', 'The American people should decide an American election.\n\nNot the Ukrainians. Not the Russians. Not the Chinese.\n\nThe American people.\n\nPresident Trump corruptly abused his power by soliciting foreign interference.\n\nHe. Must. Be. Held. Accountable. https://t.co/IjB2cHj3ZX'), ('dates', '2020-01-30 05:05:44'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.8834450840950012)])
OrderedDict([('tweet', 'Is shamelessness a superpower?\n\nPerhaps.\n\nBut TRUTH is our Kryptonite.'), ('dates', '2020-01-28 17:00:07'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.6784902811050415)])
OrderedDict([('tweet', 'The full House is back in session this week.\n\nDems are working to fix our crumbling bridges, roads, tunnels, airports as well as mass transportation system.\n\nAnd create millions of good-paying jobs with a real infrastructure plan.\n\nFor. The. People.'), ('dates', '2020-01-28 13:05:01'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5851123332977295)])
OrderedDict([('tweet', 'Donald Trump withheld $391 million in security aid to cheat in the 2020 election.\n\nThe President argues no evidence of a corrupt shakedown scheme exists.\n\nOK.\n\nSo why are you trying to stop John Bolton from testifying?\n\nFACTS.'), ('dates', '2020-01-28 04:00:00'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.8299756050109863)])
OrderedDict([('tweet', 'The Truth Matters. Facts Matter. Witnesses Matter.\n\nJohn Bolton must testify.\n\nThe American people deserve a #FairTrial. https://t.co/zH7lOAZ8rB'), ('dates', '2020-01-27 13:48:33'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5841024518013)])
OrderedDict([('tweet', 'Note to those critics allegedly concerned with decorum.\n\nThis is the President of the United States.\n\nThe next time you think to criticize House Dems for using sharp language.\n\nSpare us the FAKE outrage. https://t.co/eXkh5LNgN7'), ('dates', '2020-01-26 20:16:22'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.5580748319625854)])
OrderedDict([('tweet', 'House Dems are fighting to lower prescription drug prices #ForThePeople. \n\nPresident Trump jammed #GOPTaxScam down our throats.\n\n83% of the benefits went to the wealthiest 1% (and the debt exploded).\n\nNow he wants to cut your Social Security.\n\nThey. Have. No. Shame.'), ('dates', '2020-01-25 13:57:26'), ('file_name', 'Hakeem S. Jeffries.csv'), ('pct_true', 0.6185086965560913)])
OrderedDict([('tweet', "1.3 million retirees are in danger of losing their hard-earned retirement benefits due to the multi-employer pension crisis, and COVID-19 is straining these struggling pension plans even further. \n\nIt's time to fix the multi-employer pension crisis. \nhttps://t.co/DEpIGMuUQq"), ('dates', '2020-04-30 17:39:01'), ('file_name', 'Haley M. Stevens.csv'), ('pct_true', 0.6399784684181213)])
OrderedDict([('tweet', 'We can never mend the pain &amp; suffering that the Armenian people experienced 105 years ago, but we take today to recognize the #ArmenianGenocide. 1.5 million Armenians were killed by the Turkish Ottoman Empire, &amp; millions more were displaced, robbed, and horrendously mistreated.'), ('dates', '2020-04-24 20:09:29'), ('file_name', 'Haley M. Stevens.csv'), ('pct_true', 0.6472086906433105)])
OrderedDict([('tweet', 'It has been one year since the House passed the Violence Against Women Act. Today, I’m calling on the Senate to pass this crucial legislation, because survivors have waited long enough. #VAWA4ALL'), ('dates', '2020-04-07 17:57:44'), ('file_name', 'Haley M. Stevens.csv'), ('pct_true', 0.8710786700248718)])
OrderedDict([('tweet', 'We are facing worst health crisis in nearly a century and unprecedented economic devastation on top of it. \n\nThis is hitting American families as a double whammy. \n\nThe Congress is committed to not letting anyone fail and I remain committed to everyone in #MI11 during this time. https://t.co/m16d5kXucT'), ('dates', '2020-04-02 15:45:04'), ('file_name', 'Haley M. Stevens.csv'), ('pct_true', 0.5687670707702637)])
OrderedDict([('tweet', 'We cannot have a healthy economy without healthy people. \n\nWe cannot know who is healthy without testing. \n\nWhere are the tests?'), ('dates', '2020-04-28 14:38:46'), ('file_name', 'Harley Rouda.csv'), ('pct_true', 0.5392555594444275)])
OrderedDict([('tweet', 'In the middle of a pandemic, @RealDonaldTrump is undoing an Obama-era policy that protects LGBTQ+ Americans from health care discrimination. \n\nMr. President — stop legislating hate and start getting us some damn tests.\n\nhttps://t.co/WrGo5BoFcX'), ('dates', '2020-04-24 18:18:06'), ('file_name', 'Harley Rouda.csv'), ('pct_true', 0.5388771295547485)])
OrderedDict([('tweet', 'Scammers are targeting millions of Americans claiming to work for the IRS, selling fraudulent #COVID19 tests, and soliciting donations for fake charities via phone. \n\nThe @FCC and @TheJusticeDept must act to protect the safety of the Americans they serve. https://t.co/MywMIGPQPp'), ('dates', '2020-04-17 14:18:03'), ('file_name', 'Harley Rouda.csv'), ('pct_true', 0.5106487274169922)])
OrderedDict([('tweet', 'Those who fail to learn from history are doomed to repeat it—the United States cannot afford for the Administration to replicate its failed response in Puerto Rico with a disease like #COVID19. \n\nhttps://t.co/HZ6d3KVmXW'), ('dates', '2020-03-30 17:30:10'), ('file_name', 'Harley Rouda.csv'), ('pct_true', 0.7508984804153442)])
OrderedDict([('tweet', 'America first, huh? \n\nSenators were sounding the alarm. Washington state had confirmed cases. \n\nPresident Trump called #COVID19 a hoax and sent 17.8 tons of medical supplies...to China. \n\nhttps://t.co/fWWT9gP11L'), ('dates', '2020-03-30 13:56:04'), ('file_name', 'Harley Rouda.csv'), ('pct_true', 0.7058928608894348)])
OrderedDict([('tweet', 'Americans need clean air &amp; clean drinking water now more than ever. \n\nThis is a horribly misguided directive from an Administration hell bent on protecting corporate interests NOT people. \n\nUsing #COVID19 as cover to permit pollution is a cheap political stunt. https://t.co/6Pqdmsmb5z'), ('dates', '2020-03-26 23:40:04'), ('file_name', 'Harley Rouda.csv'), ('pct_true', 0.6092198491096497)])
OrderedDict([('tweet', '.@RealDonaldTrump — drop the #AffordableCareAct lawsuit. Fighting to take away health insurance from 20 million Americans and put 135 million Americans with pre-existing conditions at risk during a PANDEMIC is disgraceful.'), ('dates', '2020-03-23 18:21:59'), ('file_name', 'Harley Rouda.csv'), ('pct_true', 0.6293787360191345)])
OrderedDict([('tweet', 'We need more tests. \n\nWe need tests for our senior citizens. We need tests for the immunosuppressed. We need tests for working Americans. We need tests for folks with symptoms. \n\nInformation is power, and right now the Trump Administration is powerless to stop #COVID19.'), ('dates', '2020-03-14 18:11:48'), ('file_name', 'Harley Rouda.csv'), ('pct_true', 0.5818215608596802)])
OrderedDict([('tweet', 'Governor @JayInslee is right — misinformation and mismanagement is unacceptable in the midst of a public health threat like #COVID19. \n\nThis Administration should be held accountable for its failures and be pushed to do better for the American people. https://t.co/X17aZH2opp'), ('dates', '2020-03-06 23:35:15'), ('file_name', 'Harley Rouda.csv'), ('pct_true', 0.5849071741104126)])
OrderedDict([('tweet', ".@realDonaldTrump lied in his SOTU address. \n\nHe isn't fighting to #ProtectOurCare — he's trying to kick 20 million Americans off their current health care plan and leave 135 million Americans with pre-existing conditions without coverage. https://t.co/QgFDHJXfAN"), ('dates', '2020-03-02 16:41:51'), ('file_name', 'Harley Rouda.csv'), ('pct_true', 0.9494559168815613)])
OrderedDict([('tweet', 'One single no vote on the CARES Act will not stop this critical COVID-19 relief package. It would be a disappointing and unnecessary delay, but I want to assure Kentuckians that this bill will pass despite those who choose to put politics above the American people. #PassTheBill'), ('dates', '2020-03-27 14:46:16'), ('file_name', 'Harold Rogers.csv'), ('pct_true', 0.858531653881073)])
OrderedDict([('tweet', '(3/3)...President Trump rightfully stood firm against tyranny, sending a clear message that no nation should tolerate “Iran’s destructive and destabilizing behavior,” and I pray that we can continue forward with efforts to deescalate tensions in the Middle East.'), ('dates', '2020-01-09 03:13:16'), ('file_name', 'Harold Rogers.csv'), ('pct_true', 0.8141652345657349)])
OrderedDict([('tweet', 'I joined House Republicans tonight in a vote to censure &amp; condemn Rep. Schiff for misleading the American people to advance the Democrats’ partisan &amp; secretive impeachment process against @realDonaldTrump. Rep. Schiff should be held accountable for leading this shameful scheme.'), ('dates', '2019-10-22 00:34:40'), ('file_name', 'Harold Rogers.csv'), ('pct_true', 0.9072731137275696)])
OrderedDict([('tweet', "It’s no secret that Purdue Pharma, fueled by greed, relentlessly and recklessly marketed OxyContin in the U.S., even when it became clear the drug was fueling addiction and overdose deaths. Learn more about Purdue's influence on the @WHO in my new joint report w/ @RepKClark https://t.co/SMjJuhv1zn"), ('dates', '2019-05-22 14:36:05'), ('file_name', 'Harold Rogers.csv'), ('pct_true', 0.6581322550773621)])
OrderedDict([('tweet', '#EarthDay is about celebrating the planet and recognizing our responsibility to protect it. House Democrats are working to #ActOnClimate because we want a healthy Earth not just for a day, but for generations. https://t.co/4lVrFSFRUZ'), ('dates', '2020-04-22 14:09:08'), ('file_name', 'Henry C. "Hank" Johnson, Jr..csv'), ('pct_true', 0.6413068771362305)])
OrderedDict([('tweet', 'The #coronavirus can impact all people; regardless of race, gender, nationality or religion. But Asian Americans are facing rising hate in light of .@realDonaldTrump and others’ racist rhetoric. It’s time for this to end—hate has no home here. #NoHate @NewAmericanLd'), ('dates', '2020-04-14 18:00:10'), ('file_name', 'Henry C. "Hank" Johnson, Jr..csv'), ('pct_true', 0.8886902928352356)])
OrderedDict([('tweet', '.@realDonaldTrump and the Trump administration is rushing to construct its racist, expensive, ineffective border wall and pushing anti-immigrant hysteria.\n\nThe result: The destruction of sacred, historic Native American sites that can’t be replaced.#HonorTheSacred #NoBorderWall'), ('dates', '2020-02-26 17:02:11'), ('file_name', 'Henry C. "Hank" Johnson, Jr..csv'), ('pct_true', 0.8812330365180969)])
OrderedDict([('tweet', "We know the future for all immigrants &amp; #DACA recipients remains uncertain in America. We stand with you no matter what. The House did its job, now it's time for the Senate to act. #ToImmigrantsWithLove #DreamAndPromise https://t.co/w4REHZSay8"), ('dates', '2020-02-14 15:38:37'), ('file_name', 'Henry C. "Hank" Johnson, Jr..csv'), ('pct_true', 0.68617844581604)])
OrderedDict([('tweet', 'Did you know President Trump’s new proposed budget cuts directly affect Medicare, Medicaid, and their safety net programs such as SNAP? #TrumpBudget'), ('dates', '2020-02-12 21:08:16'), ('file_name', 'Henry C. "Hank" Johnson, Jr..csv'), ('pct_true', 0.7220327258110046)])
OrderedDict([('tweet', 'It is dangerous &amp; unacceptable for border wall construction to continue during this pandemic. That’s why @MartinHeinrich &amp; I led a bicameral letter to @DHS_Wolf, AG Barr, &amp; @EsperDoD, urging them to halt ongoing wall construction at the border as the U.S. responds to COVID-19. https://t.co/3tV94IAOZK'), ('dates', '2020-04-15 21:12:47'), ('file_name', 'Henry Cuellar.csv'), ('pct_true', 0.6062881350517273)])
OrderedDict([('tweet', 'The State Department has issued a Level 4 – Do Not Travel advisory. U.S. citizens are to avoid all international travel and any U.S. citizens that are stranded abroad need to start seeking arrangements for their return home.'), ('dates', '2020-04-02 16:30:35'), ('file_name', 'Henry Cuellar.csv'), ('pct_true', 0.8266124129295349)])
OrderedDict([('tweet', "Trump's Pentagon wants more money while small businesses go without.\n \nNot on my watch.\n \nDOD has gotten over $750 billion in FY20 and it shouldn't get a cent more!"), ('dates', '2020-05-08 18:37:31'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.7740503549575806)])
OrderedDict([('tweet', '20.7 million Americans lost their jobs in April.\n\n58,000 Americans died.\n\nMeanwhile, the stock market had its best month since 1987.\n\nWe need to start valuing human life over corporate profits.'), ('dates', '2020-05-08 13:35:52'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.6884992122650146)])
OrderedDict([('tweet', 'Trump’s tax giveaway for real estate investors: $135 billion\n \nEstimated cost of funding universal testing: $50 billion\n \nDon’t tell me we don’t have the resources to combat this virus. This administration is choosing to prioritize wealth over the health &amp; safety of the country. https://t.co/dVX5h713E9'), ('dates', '2020-05-05 18:03:34'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.5083895325660706)])
OrderedDict([('tweet', 'Over 9.2 million Americans have lost their health insurance during this pandemic. \n\n9.2 million have no safety net if they get sick. \n\nThis is why we need Medicare for All.'), ('dates', '2020-05-04 20:54:27'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.8104338645935059)])
OrderedDict([('tweet', 'The president and Stephen Miller are exploiting a global pandemic to ban LEGAL immigration. \n\nThis xenophobic policy is their attempt to change the fabric of America and advance their anti-immigrant agenda.\n\nNot on our watch. https://t.co/ufzA5e7cjR'), ('dates', '2020-04-27 17:48:56'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.5362023711204529)])
OrderedDict([('tweet', 'Rent is due on May 1st. \n\nThe American people need help NOW.'), ('dates', '2020-04-24 21:26:57'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.5757865905761719)])
OrderedDict([('tweet', 'Trump is trying to escalate tensions with Iran during a pandemic. \n\nNO WAR WITH IRAN. https://t.co/UvofDAooXD'), ('dates', '2020-04-22 20:14:34'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.5170820355415344)])
OrderedDict([('tweet', 'This pandemic is a reminder that our existence on Earth is fragile. \n\nDegraded environments and pollution has increased our vulnerability to contracting the coronavirus/other viruses. \n\nClimate change will make us more vulnerable to future pandemics if we fail to act. #EarthDay'), ('dates', '2020-04-22 19:16:28'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.7157090306282043)])
OrderedDict([('tweet', 'Trump will ban immigration but allow some Southern states to loosen restrictions. \n\nThis has nothing to do with our safety and everything to do with his blatant xenophobia.  \n\nPathetic.'), ('dates', '2020-04-21 13:52:13'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.9141740202903748)])
OrderedDict([('tweet', 'On Yom HaShoah, we honor and remember the 6 million Jews brutally murdered during the Holocaust. \n \nWe must never forget this genocide and reject antisemitism and all forms of hate that give rise to violence. #NeverAgain'), ('dates', '2020-04-20 20:46:44'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.6006327271461487)])
OrderedDict([('tweet', 'Coronavirus is disproportionally killing people of color. \n\nWe need to prioritize funding to combat racial health disparities in the next COVID-19 package.'), ('dates', '2020-04-16 17:30:16'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.5109543204307556)])
OrderedDict([('tweet', 'Millions of Americans are one paycheck away from financial disaster. \n\nThe Department of Treasury is encouraging banks to seize stimulus checks from people with outstanding debt during this pandemic. \n\nShameful.'), ('dates', '2020-04-15 17:49:40'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.7800259590148926)])
OrderedDict([('tweet', '10 million people have lost their jobs in the last two weeks.\n\nWe need to put an end to the immoral system that ties healthcare coverage to employment.\n\nWe need Medicare for All.'), ('dates', '2020-04-03 13:00:00'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.5486104488372803)])
OrderedDict([('tweet', 'Jan 22:\nReporter: Are there worries about a pandemic? \n\nTrump: No, not at all. We have it totally under control. \n\nNow:\nTrump: I’ve always known this is a real pandemic.                               \n\n🤔\nhttps://t.co/qpjdKsi7W2'), ('dates', '2020-04-02 16:07:19'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.706804096698761)])
OrderedDict([('tweet', "6.6 million Americans filed for unemployment this week.\n\nWe can't just put a band-aid on this crisis. We need:\n\n-Universal and monthly cash assistance\n-Rent and mortgage moratorium\n-Student debt cancellation\n-Free testing AND treatment for all"), ('dates', '2020-04-02 14:26:23'), ('file_name', 'Ilhan Omar.csv'), ('pct_true', 0.7163458466529846)])
OrderedDict([('tweet', '(1/4) I understand that for many college students, the #COVID19 pandemic has erased their sources of income and left them in a tough spot. That is why the #CARESAct that Congress passed in March included nearly $31 billion in funding for overall education aid.'), ('dates', '2020-05-08 02:31:21'), ('file_name', 'J. French Hill.csv'), ('pct_true', 0.7259823083877563)])
OrderedDict([('tweet', "The #coronavirus exposed our massive dependence on health supplies from #China. \n\nToday, I spoke with @BobSellersTV on @Newsmax about how my legislation - the SAVE Act - is designed to diversify the U.S. health care supply chain and break America's reliance on Chinese goods. https://t.co/Uv9risaScU"), ('dates', '2020-05-07 01:28:54'), ('file_name', 'J. French Hill.csv'), ('pct_true', 0.6142446398735046)])
OrderedDict([('tweet', "Tonight marks the end of #YomHashoa, #HolocaustRemembranceDay. \n\nWe will never forget the millions of Jews, Poles, Serbs, and Romani citizens, who were brutally murdered by the Nazis. \n\nEarlier this month, I honored the 75th anniversary of the #Buchenwald death camp's liberation. https://t.co/dDHpzQSMhf"), ('dates', '2020-04-21 22:37:31'), ('file_name', 'J. French Hill.csv'), ('pct_true', 0.7319058179855347)])
OrderedDict([('tweet', 'What we’re missing due to #COVID19 coverage: This morning Xi doubled down on the misery in #HongKong. He arrested 14 voices for democracy who demanded China live up to its “promise” of 1 country, 2 systems. Xi has no intension of honoring this treaty or any other. #HK #freethe14'), ('dates', '2020-04-18 16:27:18'), ('file_name', 'J. French Hill.csv'), ('pct_true', 0.9305216670036316)])
OrderedDict([('tweet', "We don’t want to rely on countries like #China for critical supplies during a global #crisis. This morning, I joined @HughHewitt to discuss how the SAVE Act reduces our nation's reliance on foreign made medical equipment while keeping #Arkansans and #Americans safe. https://t.co/4WgUQ46dsl"), ('dates', '2020-04-17 23:08:08'), ('file_name', 'J. French Hill.csv'), ('pct_true', 0.5340732336044312)])
OrderedDict([('tweet', 'Dear @SpeakerPelosi - while you were you were on late night tv pitching chocolate ice cream, I’m receiving texts like this: “There are hundreds of us who did not get our #PPP applications in quick enough and we are now in a bind....” CC: @GOPLeader @realDonaldTrump'), ('dates', '2020-04-17 01:20:00'), ('file_name', 'J. French Hill.csv'), ('pct_true', 0.9271132946014404)])
OrderedDict([('tweet', "Over the last few weeks, we've seen countless stories outlining the shortages of #ventilators and #PPE which our medical professionals desperately need. \n\nMy bill, the SAVE Act, makes the United States less dependent on #China, and other foreign countries, for medical supplies. https://t.co/vTVakdmBnM"), ('dates', '2020-04-11 00:46:06'), ('file_name', 'J. French Hill.csv'), ('pct_true', 0.6239055395126343)])
OrderedDict([('tweet', 'We’ve known for years that Assad used chemical weapons against his own people and today’s @OPCW report confirmed it has continued. @SecPompeo, we must hold #Assad accountable and we must support the current ceasefire for the safety of innocents in #Idlib. https://t.co/MwfMbkaoCc'), ('dates', '2020-04-08 19:10:42'), ('file_name', 'J. French Hill.csv'), ('pct_true', 0.8562033772468567)])
OrderedDict([('tweet', "Great interview between @RepMcCaul and @ShannonBream on @foxnewsnight last night. \n\nChina's Communist Party is wreaking havoc across the world and assigning blame to everyone but themselves. \n\nRemember when they told @WHO that #COVID19 wasn't contagious? https://t.co/mEuTTLhSNA"), ('dates', '2020-03-25 16:02:02'), ('file_name', 'J. French Hill.csv'), ('pct_true', 0.6997325420379639)])
OrderedDict([('tweet', 'I just introduced the Leave No Taxpayer Behind Act because taxpaying immigrants were left out of the $2 trillion coronavirus relief bill. These taxpayers work in critical sectors of our economy, like agriculture, &amp; contribute greatly to our country.'), ('dates', '2020-04-03 20:43:48'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.6093557476997375)])
OrderedDict([('tweet', 'Very disappointed SCOTUS is allowing @POTUS to continue oppressing vulnerable asylum seekers.\n\nThese refugees have traveled thousands of miles to escape violence and death in their home countries.\n\nWe must stand up for families without a voice seeking a better life in the USA. https://t.co/97veGSc2RG'), ('dates', '2020-03-11 18:40:24'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.6979044675827026)])
OrderedDict([('tweet', '🚨🚨The President is once again going after American’s healthcare by attacking Medicaid.\n\nThis administration is again trying to take away lifesaving health care from seniors &amp; families.\n\nI will not allow this Admin to undermine the promise of affordable, dependable health care. https://t.co/kRuw10zWer'), ('dates', '2020-01-30 19:18:51'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.727287232875824)])
OrderedDict([('tweet', 'This is a sad day. I do not take impeachment lightly. I’ve had the opportunity to vote to impeach the President before and have voted NO on all prior occasions.'), ('dates', '2019-12-13 15:47:01'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.5026293396949768)])
OrderedDict([('tweet', 'Sadly, the evidence shows our President solicited the intervention of a foreign nation in our elections.\xa0\n\nHe urged foreign interference in exchange for a White House meeting &amp; nearly $400m taxpayer dollars in military aid.\n\nWatch my questioning here. \n👇🏽👇🏽👇🏽 https://t.co/JHoBxhhT9U'), ('dates', '2019-12-10 00:47:00'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.5905141234397888)])
OrderedDict([('tweet', 'Today, we remember the transgender people throughout our communities whose lives have been taken simply for living their identity.\n\nToo many lives have been lost. These attacks must end. \n\n#TDOR'), ('dates', '2019-11-20 21:03:57'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.7258998155593872)])
OrderedDict([('tweet', 'For the past few weeks, Republican and Democratic Members of the House Intelligence, Foreign Affairs, and Oversight Committees have investigated allegations surrounding President Trump. These allegations are serious, and the House is taking them seriously.'), ('dates', '2019-10-31 14:57:19'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.615338146686554)])
OrderedDict([('tweet', 'No one is above the law. For the past nine months, I have joined my colleagues on the House Judiciary Committee in investigating the many abuses of power levied against this President.'), ('dates', '2019-09-24 21:57:51'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.7747420072555542)])
OrderedDict([('tweet', 'When @HouseDemocrats introduced our landmark anti-corruption bill, H.R. 1 -  The For The People Act, some of our colleagues denounced it, claiming states—not the federal government—have 100% control over their elections.\n\nWell? What changed? https://t.co/6gxWBShcLJ'), ('dates', '2019-09-19 21:51:41'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.6743793487548828)])
OrderedDict([('tweet', 'Hurricane #Dorian is careening towards Florida but @realDonaldTrump would rather deport working families than protect yours. \n\nInstead of preparing, he raided @FEMA’s emergency fund to fuel his deportation machine.\n\n2018 saw 14 disasters &amp; cost $300b. How will we rebuild in 2019?'), ('dates', '2019-08-28 20:05:02'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.6833714842796326)])
OrderedDict([('tweet', '.@realDonaldTrump announced he will end ALL foreign aid, worsening the global refugee crisis. \n\nMr. President. Our neighbors’ houses are on fire. We can either help put out the fire and rebuild their economies or expect them to migrate north. \n\nWhat’s it going to be?'), ('dates', '2019-08-19 15:12:28'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.5551623106002808)])
OrderedDict([('tweet', '.@realDonaldTrump has released his new “public charge” rule. \n\nThis rule forces struggling families to choose between feeding their kids or becoming a US citizen.\n\nThis is the Admin’s latest attempt to limit legal immigration &amp; redefines who we are—a nation of immigrants.'), ('dates', '2019-08-12 16:45:52'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.6292079091072083)])
OrderedDict([('tweet', '.@Equifax lost 150 million Americans’ data. This settlement is a step in the right direction, but consumers need new laws to protect them. \n\nI introduced the Cyber Breach Notification Act to ensure Americans are notified if their data is stolen so they can protect their future. https://t.co/ohbZNXtuCs'), ('dates', '2019-07-23 00:21:57'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.7207825779914856)])
OrderedDict([('tweet', 'Very disappointed my amendment to grant citizenship to non-citizens who honorably serve in the #USA Military was rejected. \n\nThese brave servicemembers don the uniform and risk their lives for each of us. They have earned the right to call themselves Americans. #NDAA'), ('dates', '2019-07-10 18:32:28'), ('file_name', 'J. Luis Correa.csv'), ('pct_true', 0.5854685306549072)])
OrderedDict([('tweet', 'While the partisan rancor has taken the headlines, the reality is - throughout the Coronavirus crisis, President @realDonaldTrump’s Administration has delivered for Michigan. https://t.co/NuAMUGbULp'), ('dates', '2020-04-30 13:59:41'), ('file_name', 'Jack Bergman.csv'), ('pct_true', 0.8247143030166626)])
OrderedDict([('tweet', 'If this isn’t partisan politics, I don’t know what is! The impeachment votes today represent the worst of Washington, D.C. - yet another reason my constituents are so disillusioned with the process and disappointed with the 116th Congress.  \n\n#ShamImpeachment https://t.co/gG7pNsIUSV'), ('dates', '2019-12-18 21:16:53'), ('file_name', 'Jack Bergman.csv'), ('pct_true', 0.8878637552261353)])
OrderedDict([('tweet', 'Withholding paychecks from Members of Congress who fail to pass appropriations is an important step to prevent government shutdowns, which hurt the economy and millions of everyday Americans. But it’s also an important step to promote fiscal responsibility\nhttps://t.co/OpdkYW8Ln2'), ('dates', '2019-12-13 17:21:55'), ('file_name', 'Jack Bergman.csv'), ('pct_true', 0.7547115087509155)])
OrderedDict([('tweet', 'U.S. citizens have no legal recourse against foreign governments that target them online. The Foreign Sovereign Immunities Act shields bad actors from being sued by private citizens. \n\nI’ve introduced legislation to fix that —&gt;\n\nhttps://t.co/ytpziIzAmp'), ('dates', '2019-11-19 22:32:57'), ('file_name', 'Jack Bergman.csv'), ('pct_true', 0.7626652717590332)])
OrderedDict([('tweet', 'While Democrats focus entirely on ousting President @realDonaldTrump, I will remain laser-focused on fighting for the needs of the constituents of the First District and working together to accomplish our goals. https://t.co/XSud3fKu0U'), ('dates', '2019-09-26 22:29:10'), ('file_name', 'Jack Bergman.csv'), ('pct_true', 0.5595349669456482)])
OrderedDict([('tweet', 'With no major political accomplishments to show since the 2018 elections, Democratic leadership has chosen to push meaningless resolutions to score cheap political points, further dividing our nation.'), ('dates', '2019-07-16 23:47:06'), ('file_name', 'Jack Bergman.csv'), ('pct_true', 0.5858197212219238)])
OrderedDict([('tweet', 'Today the Senate overwhelmingly passed a bipartisan bill to address the crisis on our southern border. \n\nNow is the time for action, not political games. The House should take up this legislation immediately and send it to President Trump’s desk for his signature.'), ('dates', '2019-06-26 20:47:07'), ('file_name', 'Jack Bergman.csv'), ('pct_true', 0.56281977891922)])
OrderedDict([('tweet', 'Clearly, #PPP is not working to benefit the #smallbiz it was intended to save. Republicans blocked PPP fixes, but they can still insist on oversight: Congress must investigate &amp; hold hearings to ensure transparency, prevent further exploitation, &amp; hold bad actors accountable.'), ('dates', '2020-04-28 12:43:36'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.623389482498169)])
OrderedDict([('tweet', 'Outrageous: Millions of creditworthy small businesses are being shut out of #PPP &amp; left empty-handed while @SBAgov allows big banks to exploit the system to beef up their balance sheets &amp; prioritize aid to their most preferred clients. Is this why GOP leaders refused to fix PPP?'), ('dates', '2020-04-28 12:15:39'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.9560765624046326)])
OrderedDict([('tweet', 'As Pres Trump feuds w/ governors &amp; seeks to limit their ability to spend #COVID19 relief funds, @SenateDems are WAITING FOR REPUBLICANS TO STEP UP for THEIR STATES &amp; GET the Trump Admin to remove needless restrictions. Read our letter: https://t.co/9hxKODuKUb'), ('dates', '2020-04-26 16:42:41'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.5072113275527954)])
OrderedDict([('tweet', 'Dems are trying to save lives &amp; livelihoods from #COVID19, but Sen. GOP sees pandemic as opportunity to force states into bankruptcy &amp; slash state &amp; local services. This partisan ploy is a needless drag on US economy &amp; threatens finances of every tax payer https://t.co/K0J61SrFeZ'), ('dates', '2020-04-23 14:35:44'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.9547597765922546)])
OrderedDict([('tweet', "Our sailors are professionals who demonstrate great skill, courage, &amp; discipline on a daily basis. They already had all the authority needed to deal w/ harassment &amp; hostile vessels. This tweet is not a change in ROE, it's an attempt to distract Americans.\nhttps://t.co/EVoCAUAGkz"), ('dates', '2020-04-22 15:18:29'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.7446733117103577)])
OrderedDict([('tweet', "A month ago, Pres Trump promised 27 million tests &amp; then delivered only 4 million. Trump Admin's failure set U.S. back significantly. But today, Senate Democrats delivered a $25B boost to ramp up testing &amp; help accelerate a safer reopening of the economy.\nhttps://t.co/83HVMi500P"), ('dates', '2020-04-22 02:03:24'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.762847900390625)])
OrderedDict([('tweet', 'Slowly but surely, Congressional Republicans seem to be waking up to fact that #PPP needs important fixes. Dems had a plan in place last week to enhance PPP &amp; boost funding, but GOP blocked it. I urge them to start working w/ us to help save #smallbiz.\nhttps://t.co/1IWG4Hgn6O'), ('dates', '2020-04-20 19:33:26'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.846744954586029)])
OrderedDict([('tweet', "FACTS:\n✔️There is bipartisan support for fixing, enhancing &amp; expanding #PPP &amp; #EIDL.\n✔️ There is bipartisan frustration Trump Admin isn't getting $$$ out the door. \n✔️ If Sen McConnell allowed up or down vote on bipartisan plan to increase $$$ for PPP &amp; EIDL it could pass today."), ('dates', '2020-04-16 19:21:42'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.553898274898529)])
OrderedDict([('tweet', 'Small businesses are pleading with Congress to fix the Economic Injury Disaster Loan (#EIDL) program. Congress needs to boost EIDL support &amp; help Main Street businesses survive. Now is time for action &amp; problem-solving, not  partisanship &amp; more red tape.'), ('dates', '2020-04-11 23:01:18'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.7730229496955872)])
OrderedDict([('tweet', "Nobody wants to hear this, but we're still in the early stages of the battle vs. #COVID19. We can get it under control. But to stop it from returning, we can't allow our defenses to crumble. We must invest more in testing, hospitals &amp; health workers, #PPE, research, &amp; vaccine."), ('dates', '2020-04-09 14:36:01'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.5423685908317566)])
OrderedDict([('tweet', 'States, hospitals, families, &amp; #smallbiz are all pleading for additional federal help. Congress &amp; Trump must listen &amp; act. Safeguarding the health &amp; well-being of ALL Americans -- not just the powerful -- requires smart &amp; comprehensive strategy. The Admin must do a better job!'), ('dates', '2020-04-09 14:26:04'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.72687166929245)])
OrderedDict([('tweet', 'Sen. McConnell is performing a bit of political theater, requesting only $250B for #smallbiz.\nDems will counter, seeking $250B for Small biz + $150B for states + $100B for hospitals.\nGOP will object.\nThen real work of negotiating &amp; legislating begins...'), ('dates', '2020-04-09 14:13:14'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.7518908381462097)])
OrderedDict([('tweet', "Acting Sec. #Modly mishandled the situation &amp; I concur with SECDEF's decision to accept his resignation. I've already asked SECDEF &amp; DOD IG for investigation that includes an assessment of the actions of Navy leadership, both civilian &amp; military, &amp; what role, if any, WH played."), ('dates', '2020-04-07 20:26:50'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.5324221253395081)])
OrderedDict([('tweet', "BREAKING: Pres Trump seems to be engaged in a mass-culling of government watchdogs, which is a real threat to good gov't &amp; independent oversight. Pres Trump must explain why he fired Mr. Fine &amp; I will continue seeking answers &amp; ensuring real accountability. #TrumpIGMassacre"), ('dates', '2020-04-07 18:19:13'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.6037845015525818)])
OrderedDict([('tweet', 'Pres Trump’s lack of coordination &amp; failure to fully utilize #DPA in midst of #COVID19 pandemic means states bid against each other for scarce medical equipment &amp; #PPE, driving up costs for all &amp; making it harder for states w/ most urgent need to access life-saving gear. #PPENow'), ('dates', '2020-04-05 18:28:12'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.7009890675544739)])
OrderedDict([('tweet', 'This is an abuse of power: In seeking personal revenge against the intelligence community’s chief watchdog, Pres Trump is attempting to further politicize U.S. intelligence. And ultimately, that puts U.S. national security at risk.\nhttps://t.co/zJH8Jl9mWY'), ('dates', '2020-04-04 03:23:37'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.5866338014602661)])
OrderedDict([('tweet', 'This is an unhealthy decision by Pres Trump. Our pandemic response is only as strong as the most vulnerable link in the chain &amp; Trump’s decision to deny millions health coverage makes everyone more vulnerable.\nhttps://t.co/e3qVe749cI'), ('dates', '2020-04-01 15:29:43'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.5909757614135742)])
OrderedDict([('tweet', 'When Americans need health coverage most, the #AffordableCareAct has been there as a literal lifeline. With millions of workers losing their jobs -- &amp; access to employer-provided health insurance -- now is NOT the time to scrap the law. We must #ProtectOurCare. #ThanksObamacare'), ('dates', '2020-03-31 21:27:20'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.7140684723854065)])
OrderedDict([('tweet', 'RI’s #coronavirus disaster declaration has been officially granted, paving way for additional federal help to bolster RI’s emergency protective measures. This is something the Congressional delegation has been advocating for weeks. Glad Trump Admin granted request. #COVID19'), ('dates', '2020-03-31 01:17:06'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.7172531485557556)])
OrderedDict([('tweet', 'It took a lot of hard work &amp; team work, but Democrats successfully turned a corporate bailout proposal into a landmark #PeopleFirst emergency rescue bill that will help combat #COVID19 &amp; support the health &amp; well-being of all Americans. #ForThePeople #CARESAct'), ('dates', '2020-03-26 03:42:45'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.8502452373504639)])
OrderedDict([('tweet', "We're making progress on an emergency #coronavirus rescue plan. But show votes on McConnell’s $500B slushfund only slows things down. Dems are committed to helping all hardworking Americans, protecting taxpayers, delivering $ for states &amp; hospitals, &amp; saving our economy."), ('dates', '2020-03-23 17:43:00'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.6329078078269958)])
OrderedDict([('tweet', 'Helping health workers, hospitals, the unemployed, small businesses &amp; states is key. It’s time for Sen McConnell to tear up his $500B #TrumpSlushFund &amp; put #WorkersFirst. Congress must do the right thing in the midst of a pandemic &amp; put people &amp; Main Street ahead of Wall Street.'), ('dates', '2020-03-22 20:43:34'), ('file_name', 'Jack Reed.csv'), ('pct_true', 0.6683573722839355)])
OrderedDict([('tweet', "The Trump Administration's claim that releasing the Mueller material “would irrevocably lift their secrecy and possibly frustrate the government’s ability to seek further review,” is code for more coverup!  What is @POTUS so afraid of?\n \nhttps://t.co/YZb9495KFQ"), ('dates', '2020-05-07 19:46:45'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.6222430467605591)])
OrderedDict([('tweet', 'The cruise industry has spent billions on share repurchasing to line the pockets of execs &amp; shareholders at the expense of their workers &amp; while dodging U.S. taxes &amp; polluting our water &amp; air. Companies that evade U.S. laws &amp; taxes must not get a blank check from taxpayers now. https://t.co/68KGRdoWzm'), ('dates', '2020-04-27 13:43:14'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.6928300857543945)])
OrderedDict([('tweet', 'These disgusting tweets are disqualifying for any job in government let alone Assistant Secretary for Public Affairs at HHS. CAPUTO MUST BE CANNED IMMEDIATELY.  \n\nhttps://t.co/7fP4aqsoHc'), ('dates', '2020-04-23 18:37:21'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.7685497403144836)])
OrderedDict([('tweet', 'The fact that Trump the “businessman” cared more about the health and survival of Americans than President Trump speaks volumes to the subterranean moral depths that he has sunk. https://t.co/gVmwbExgao'), ('dates', '2020-04-22 15:08:05'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.5397322177886963)])
OrderedDict([('tweet', "This Holocaust Remembrance Day, we honor 6 million Jews murdered during one of humanity's darkest times. We remember this atrocity amid a new rise in antisemitism - 35% of American Jews say they have suffered anti-Semitism in the last 5 years. We must stand united against evil!"), ('dates', '2020-04-21 22:01:00'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.608883261680603)])
OrderedDict([('tweet', 'While @POTUS’s shameless corruption is on full display over his resistance to meaningful election reform, my colleagues &amp; I are fighting for no-excuse vote by mail &amp; $ so that Americans don’t have to risk their lives to vote in an election under attack from malign foreign actors!'), ('dates', '2020-04-16 20:14:00'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.9228642582893372)])
OrderedDict([('tweet', 'Friends &amp; family are dying. Our children are afraid, missing school &amp; unable to be w/friends or grandparents. Our economy has been devastated. We have more unemployed Americans now than we had in the Great Depression. @SpeakerPelosi is right, we need @POTUS to tell the truth! https://t.co/d5VBlGM5Eh'), ('dates', '2020-04-15 21:56:19'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.5139798521995544)])
OrderedDict([('tweet', 'He only knows how to play the blame game &amp; he keeps doing it til he succeeds. He blamed Obama, Pelosi &amp; the governors. It didn’t stick. So now he’s blaming the WHO. He doesn’t even know what \u2066WHO\u2069 does. Does he want another virus to invade the US? https://t.co/N7t9k1KWPH'), ('dates', '2020-04-15 03:56:05'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.5276677012443542)])
OrderedDict([('tweet', 'McConnell says hospitals &amp; local/state governments can wait for $ - accuses Dems of not taking his deal because we want more. What we want is for our docs &amp; nurses on the front lines to not have to sacrifice their health, time with their families &amp; loved ones, &amp; their LIVES.'), ('dates', '2020-04-10 15:44:41'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.8238471150398254)])
OrderedDict([('tweet', 'Violence against women has been repulsed by Ds and Rs for over 24 years...until @senatemajldr Mitch McConnell. Since COVID-19, there’s been a surge in domestic and sexual violence. The House has the passed bill. Now Senate must act to save lives! #VAWA4All'), ('dates', '2020-04-07 20:46:14'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.9202077388763428)])
OrderedDict([('tweet', 'The Roosevelt is in crisis. Sailors &amp; families are scared. They deserve to be protected &amp; reassured. Instead, Acting Sec Nav Modly fires Capt Crozier &amp; insults him to his crew, all while the Navy falls further behind targets to get Sailors tested/off ship. https://t.co/mKCjItUjMA'), ('dates', '2020-04-06 23:55:18'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.7281145453453064)])
OrderedDict([('tweet', "This is unconscionable! Profiteers are using loopholes in state laws to price gouge buyers seeking life saving PPE. We have federal profiteering laws dating back to WWII that must be used &amp; enforced! I'm also working on new legislation to fill any gaps. https://t.co/87xzQGPQp5"), ('dates', '2020-04-03 19:19:41'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.5342485308647156)])
OrderedDict([('tweet', 'Tweet-Jared Kushner is out of his lane again. The National Strategic Stockpile belongs to the AMERICAN PEOPLE. The law - it goes to state, local, tribal, territorial responders who need them during an emergency. Ignorance is bliss but shouldn’t stand at the presidential dais.'), ('dates', '2020-04-03 17:29:05'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.7288509011268616)])
OrderedDict([('tweet', 'Automatic rebate payments for Social Security beneficiaries will go a long way towards helping older Americans &amp; others, but we still have work ahead of us to ensure that this lifeline reaches those who need it most. We can’t let SSI recipients &amp; veterans fall through the cracks!'), ('dates', '2020-04-02 20:15:26'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.7377120852470398)])
OrderedDict([('tweet', 'Sent letter to @BetsyDeVosED w/ 49 Members demanding @usedgov halt plans to gut Title IX civil rights in the midst of the #COVIDー19 pandemic. The last thing schools &amp; survivors need during closures &amp; a public health and economic crisis is this dangerous rule! https://t.co/tpKlhdfYMZ'), ('dates', '2020-04-02 15:23:46'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.5063656568527222)])
OrderedDict([('tweet', 'How dare @POTUS call @SpeakerPelosi a sick puppy. What’s sick is he can’t or won’t work w/Democrats. We’re one nation under siege. Stop calling people names. Take some criticism. Fix the problem! We need TESTING, PPE, VENTILATORS. Stop quibbling about numbers &amp; save some lives.'), ('dates', '2020-03-30 22:13:27'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.9313084483146667)])
OrderedDict([('tweet', "To those trying to capitalize on a time of national crisis to advance their war on women, we in the @ProChoiceCaucus will NOT let them use the COVID-19 pandemic as an excuse to strip women of their constitutional rights to abortion &amp; reproductive health care. We won't go back!"), ('dates', '2020-03-29 15:00:00'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.6654720902442932)])
OrderedDict([('tweet', 'Once again we are reminded that @POTUS cares only about himself. No one wants to see his John Hancock on a check. They want their money in the bank. Waiting 3 weeks longer, costing more $ to print and mail is not leadership it’s ego. https://t.co/DdExfJvGxQ'), ('dates', '2020-03-28 20:52:54'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.6632335186004639)])
OrderedDict([('tweet', 'Reckless. POTUS making noises that he will NOT listen to doctors when opening the economy up is a recipe for this pandemic to extend into the summer and guarantee a new century Great Depression. @POTUS - don’t do it!'), ('dates', '2020-03-24 14:48:27'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.894261360168457)])
OrderedDict([('tweet', 'Leading a letter to hold the cruise industry accountable. They pay little in taxes (like 1%), pollute the environment, &amp; stiff passengers who have to cancel cruises w/the bill. If they want bailout $$$ from the U.S. government, they have to play by our rules &amp; uphold our values! https://t.co/78ZmRzxZqE'), ('dates', '2020-03-23 15:10:37'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.7251375913619995)])
OrderedDict([('tweet', 'POTUS is dangerous. He says one thing, “I’ve invoked the Defense Production Act!” then does another. Nothing. Companies must be required to make PPE. We are endangering our health care workers and ourselves. https://t.co/Y75UGwwKs3'), ('dates', '2020-03-22 16:00:53'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.732881486415863)])
OrderedDict([('tweet', 'The biggest threat to America right now is not COVID-19 but @POTUS continuing to spew lies out about what is happening relative to COVID-19.'), ('dates', '2020-03-20 03:41:57'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.7380719780921936)])
OrderedDict([('tweet', 'Now, maybe @POTUS’s understands WHY we need a WH office of global health. Nationalism doesn’t work with viruses. https://t.co/lH7YkjHfKe'), ('dates', '2020-03-15 17:14:59'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.5340241193771362)])
OrderedDict([('tweet', 'Trump just commuted the 35 yr sentence of a Medicare fraudster who perpetrated 1 of the worst crimes ever, bilking the system millions of Americans rely on of millions of $ by taking advantage of people w/Alzheimer’s/dementia! What could be his rationale? https://t.co/XJzj6Wwx5h'), ('dates', '2020-03-12 14:53:10'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.7961151003837585)])
OrderedDict([('tweet', 'Until we know the extent of the community spread of the coronavirus, I call on ALL candidates for President to stop holding public rallies &amp; large scale events. The CDC has been absolutely clear, people should not congregate in large groups. The candidates must lead by example!'), ('dates', '2020-03-09 17:32:13'), ('file_name', 'Jackie Speier.csv'), ('pct_true', 0.6603409051895142)])
OrderedDict([('tweet', 'PPP has helped many Hoosier small businesses save jobs. But others have been waiting for loans they urgently need while Democrats in Congress played games with workers’ livelihoods. https://t.co/en0dN4JTrI'), ('dates', '2020-04-23 20:15:30'), ('file_name', 'Jackie Walorski.csv'), ('pct_true', 0.8662892580032349)])
OrderedDict([('tweet', 'No small business should be arbitrarily excluded from #COVID19 relief. \n\nThat’s why @RonWyden and I are calling for the legal marijuana small businesses to have equal access to @SBAGov loans and grants.\n\nhttps://t.co/Ujm9I7zAhR'), ('dates', '2020-05-03 14:49:01'), ('file_name', 'Jacky Rosen.csv'), ('pct_true', 0.5266618728637695)])
OrderedDict([('tweet', 'It’s unacceptable that Betsy DeVos is excluding Dreamers from #COVID19 emergency financial aid. \n\n@SenCortezMasto and I are calling for Dreamers to be included in critical COVID-19 aid, so they have access to the same resources that ALL students need during this difficult time. https://t.co/QuNBrc0vKo'), ('dates', '2020-04-29 16:18:27'), ('file_name', 'Jacky Rosen.csv'), ('pct_true', 0.5030496716499329)])
OrderedDict([('tweet', '.@SenCortezMasto and I are calling on the President to condemn increased incidents of hate against the AAPI community during #COVID19. \n\nWe must be clear and unequivocal that the enemy is not the AAPI community, but rather a virus that endangers us all. https://t.co/HQ6KXVRBl4'), ('dates', '2020-04-27 16:02:20'), ('file_name', 'Jacky Rosen.csv'), ('pct_true', 0.787017285823822)])
OrderedDict([('tweet', 'I led along with @SenCortezMasto &amp; @RepSusieLee bicameral letters to ensure that ALL small businesses that Congress made eligible for federal relief, but which are being left out by the SBA, can access @SBAGov loans. https://t.co/X09zpeFodw'), ('dates', '2020-04-22 14:07:56'), ('file_name', 'Jacky Rosen.csv'), ('pct_true', 0.5216177105903625)])
OrderedDict([('tweet', '.@SenCortezMasto and I are calling for data transparency and fair lending practices for @SBAGov’s PPP program. \n\nToo often, minority-owned businesses struggle to access capital and are left behind. \n\nWe have to ensure ALL small businesses can access this crucial funding. https://t.co/fFZPQdT6Zk'), ('dates', '2020-04-22 02:42:25'), ('file_name', 'Jacky Rosen.csv'), ('pct_true', 0.8001366853713989)])
OrderedDict([('tweet', 'As we honor #YomHaShoah this week, we sadly recognize anti-Semitic violence increased by 18% last year, and now #COVID19 is fueling a new wave of anti-Semitism. \n\nWe must push back against those exploiting this pandemic to advance bigotry and hate.\n\nhttps://t.co/Jhsnvj8ZFE'), ('dates', '2020-04-21 20:27:59'), ('file_name', 'Jacky Rosen.csv'), ('pct_true', 0.6637576222419739)])
OrderedDict([('tweet', 'Sadly, it’s been reported there has been a surge of racist and xenophobic incidents against Asian Americans during #COVID19. \n\nI joined a letter calling for our federal agencies to address these unacceptable attacks. #NoAAPIHate https://t.co/28yO0Xlv5R'), ('dates', '2020-04-15 17:08:19'), ('file_name', 'Jacky Rosen.csv'), ('pct_true', 0.5865817070007324)])
OrderedDict([('tweet', 'We don’t need bureaucratic red tape slowing potentially effective treatments for the novel coronavirus. \n\nThat’s why I introduced the Cure the Coronavirus Act, that would authorize the FDA to speed up review of drugs that prevent or treat COVID-19. https://t.co/Ony5jjaV6m'), ('dates', '2020-03-08 20:43:00'), ('file_name', 'Jaime Herrera Beutler.csv'), ('pct_true', 0.6413119435310364)])
OrderedDict([('tweet', 'I strongly oppose recent reckless calls to remove the four lower Snake River dams. It would devastate our regional economy and communities in SW WA &amp; I will continue my fight for common-sense legislation that protects our precious salmon population. https://t.co/rqrJQ1OIfl'), ('dates', '2020-02-26 01:32:00'), ('file_name', 'Jaime Herrera Beutler.csv'), ('pct_true', 0.6342323422431946)])
OrderedDict([('tweet', '(1/3) I voted against today’s War Powers Resolution. Qasem Soleimani was responsible for providing improvised explosive devices to our enemies to kill and maim over 600 American servicemembers – leaving more Gold Star families to mourn their sons and daughters.'), ('dates', '2020-01-09 23:12:47'), ('file_name', 'Jaime Herrera Beutler.csv'), ('pct_true', 0.7603922486305237)])
OrderedDict([('tweet', 'As we watch our @GOP friends get outraged over Pelosi’s people-centric counteroffer to McConnell’s $500 billion corporate slush fund, remember 10 years ago when Obama’s economic rescue plan ($787bn total) got ZERO @GOP votes in the House and almost ZERO @GOP votes in the Senate.'), ('dates', '2020-03-24 13:31:20'), ('file_name', 'James A. Himes.csv'), ('pct_true', 0.5887518525123596)])
OrderedDict([('tweet', 'It is reckless for the President to suggest that injecting disinfectant could treat coronavirus. \n\nPlease seek medical advice from a health professional, not the President. https://t.co/tKuOvqHuCp'), ('dates', '2020-04-24 18:32:43'), ('file_name', 'James E. Clyburn.csv'), ('pct_true', 0.5001450181007385)])
OrderedDict([('tweet', 'While the April 15 tax deadline is delayed this year, we should not delay action to repair the 2017 GOP tax law’s massive giveaway to the wealthy and corporations.\n\nWe must work together to create a tax code that works for hard working Americans – not against them. #TaxDay'), ('dates', '2020-04-15 15:00:15'), ('file_name', 'James E. Clyburn.csv'), ('pct_true', 0.6322064995765686)])
OrderedDict([('tweet', 'This President repeatedly downplayed the seriousness of this pandemic to serve his own interests.  \n\nWe must stop tolerating his untruths and rely instead on the accuracy of experts. \n\nWe cannot have 50 response plans. \n\nThe president abdicated his leadership role at the start.'), ('dates', '2020-04-15 01:09:05'), ('file_name', 'James E. Clyburn.csv'), ('pct_true', 0.7233135104179382)])
OrderedDict([('tweet', 'This pandemic shows lack of health care for anyone threatens everyone. I urge SC Republicans to confront this emergency by joining 36 other states in expanding Medicaid. \n\nThis would cover nearly 200,000 uninsured South Carolinians &amp; protect all 5 million. https://t.co/JxEH7cW0X2'), ('dates', '2020-03-26 21:40:48'), ('file_name', 'James E. Clyburn.csv'), ('pct_true', 0.5371959805488586)])
OrderedDict([('tweet', "Today, the House joined the Senate in voting to affirm that the President can't go to war w/ Iran w/o authorization from Congress.\n\nThe Constitution is clear—only Congress has the power to declare war.\n\nThe President must uphold the Constitution &amp; sign this legislation into law."), ('dates', '2020-03-11 21:47:47'), ('file_name', 'James E. Clyburn.csv'), ('pct_true', 0.8396864533424377)])
OrderedDict([('tweet', 'We are in the center of a crisis and Idaho families and small businesses need relief NOW. Deeply disappointed my Democrat colleagues chose politics over the urgent needs of the American people. #COVID19'), ('dates', '2020-03-22 23:54:26'), ('file_name', 'James E. Risch.csv'), ('pct_true', 0.9503785967826843)])
OrderedDict([('tweet', 'For years, US taxpayers’ $ has gone to China’s state-run bio-agent lab. It’s right for the NIH to conduct research but we do not need to send our money to China to do it. My colleagues &amp; I are urging leadership to ensure no COVID-19 stimulus funds end up at this notorious lab. https://t.co/bg7Fq94VeI'), ('dates', '2020-04-22 16:20:44'), ('file_name', 'James Lankford.csv'), ('pct_true', 0.9593067765235901)])
OrderedDict([('tweet', 'It’s unfortunate that Democrats cannot set aside political gain to ensure small businesses and workers have the relief they need right now. Tonight #PaycheckProtectionProgram will run out of money leaving so many small businesses who need help.'), ('dates', '2020-04-15 22:04:37'), ('file_name', 'James Lankford.csv'), ('pct_true', 0.8328491449356079)])
OrderedDict([('tweet', 'Small businesses &amp; nonprofits are the backbone of this country. Since the #PaycheckProtectionProgram is quickly running out of money for small biz &amp; nonprofits, we should provide additional funding--all Republicans have agreed, but we are waiting on Senate Dems to sign off today'), ('dates', '2020-04-09 15:39:40'), ('file_name', 'James Lankford.csv'), ('pct_true', 0.5780301094055176)])
OrderedDict([('tweet', "This is ridiculous. We are in the middle of a public health crisis. #COVID has claimed nearly 75,000 American lives. And President Trump's answer is to take health care away from people and eliminate protections for people w/ pre-existing conditions. https://t.co/yDZBab4oOR"), ('dates', '2020-05-07 17:05:39'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.8250940442085266)])
OrderedDict([('tweet', 'American citizens are too often being detained as de facto hostages in business disputes or to coerce family members to return to #China—this is shocking and unacceptable behavior by the Chinese government and a clear violation of international law. https://t.co/EJHJFKQgXl'), ('dates', '2020-05-05 17:46:27'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.8997272849082947)])
OrderedDict([('tweet', "Let's recap. Here's who needs a bailout:\n✅ Hungry families\n✅ Small businesses\n✅ Essential workers\n✅ Disadvantaged communities\n\nHere's who doesn't:\n✅Wealthy CEOs &amp; Fortune 500 companies w/ access to huge lines of credit &amp; millions in cash on hand."), ('dates', '2020-04-27 23:14:00'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.639363169670105)])
OrderedDict([('tweet', 'Why is anybody surprised that this is the man who told the American people to drink bleach?\n\nPlease only take medical advice from your physician. https://t.co/NMIh1tANvO'), ('dates', '2020-04-24 13:39:39'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.5582892298698425)])
OrderedDict([('tweet', '#MitchAntoinette would rather deny aid to firefighters, police &amp; EMS than deliver the relief they need to keep fighting the coronavirus. This is a disgusting remark to make when American lives are on the line. https://t.co/puBU5RKG6O'), ('dates', '2020-04-23 23:45:26'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.6316910982131958)])
OrderedDict([('tweet', "Trump &amp; Senate GOP: It's up to the states to organize a #COVID19 response.\n\nAlso Trump &amp; Senate GOP: We won't provide funding for state &amp; local #COVID19 response.\n\nConfused? That's because it makes no sense at all."), ('dates', '2020-04-22 19:01:14'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.8507845401763916)])
OrderedDict([('tweet', '10 years ago the preventable #DeepwaterHorizon spill spewed 210 million gallons of oil into the Gulf of Mexico.\n\nNow? Trump lets Big Oil call the shots w/ a pro-polluter agenda &amp; the risk of another disaster has skyrocketed.\n\nWe should #ProtectOurCoast, not corporate profits. https://t.co/jmkhgO2qpa'), ('dates', '2020-04-20 17:33:32'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.7394682765007019)])
OrderedDict([('tweet', '1/ In the midst of a global pandemic, it is unconscionable that the witch hunt has continued against #HongKong pro-democracy leaders. Human rights &amp; rule of law abuses in HK are worsening &amp; the Trump Admin needs to robustly implement &amp; enforce the HK Human Rights &amp; Democracy Act. https://t.co/M1KqCRoOaf'), ('dates', '2020-04-18 15:54:46'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.8872859477996826)])
OrderedDict([('tweet', 'Trump couldn\'t legally sign the checks, so he put his name in the "memo" field.\n\nImagine the kind of sad, egotistical person who would turn this awful crisis into an absurd campaign stunt by demanding their name go into the memo field of a check that was approved by Congress. https://t.co/RAvj8dwdpM'), ('dates', '2020-04-15 02:08:21'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.5415990352630615)])
OrderedDict([('tweet', 'This weekend — one year since the ouster of Omar al-Bashir and his regime — we remember the millions of Sudanese people who suffered and were murdered, tortured and imprisoned during his genocidal dictatorship.\nhttps://t.co/V45MflU8sJ'), ('dates', '2020-04-12 23:33:15'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.5099071264266968)])
OrderedDict([('tweet', "The president says we're at war. When America goes to war, we don’t have 50 different strategies. We have one coordinated response. But the lack of coordination at this White House has states competing against each other like this is the hunger games.\nhttps://t.co/uU6joA60cg"), ('dates', '2020-04-06 12:45:00'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.6780508160591125)])
OrderedDict([('tweet', 'The president “invoked” the Defense Production Act, but the White House still hasn’t used its full authority.\n\nThere‘s a difference between invoking a law and using it just like there’s a difference between talk and action.\n\nWe need action. We need it now.\nhttps://t.co/JBHZCqch7U'), ('dates', '2020-04-05 01:53:39'), ('file_name', 'James P. McGovern.csv'), ('pct_true', 0.6909533143043518)])
OrderedDict([('tweet', 'Indiana small businesses have received almost $7.5 billion in #PPP funding. But many more businesses were unable to get their application approved due to a lack of funds.  It’s far past time for Pelosi and Schumer to stop the obstruction and pass a clean funding bill for #PPP.'), ('dates', '2020-04-20 17:31:08'), ('file_name', 'James R. Baird.csv'), ('pct_true', 0.707199215888977)])
OrderedDict([('tweet', '.@IRS continues to make coronavirus stimulus payments. Many have gotten their money automatically, but SSI &amp; VA benefits recipients who have dependent children under 17 &amp; did not submit a 2018/2019 tax return must file by May 5 to get their full payment. https://t.co/mJV5DIzP6L'), ('dates', '2020-05-03 22:31:31'), ('file_name', 'James R. Langevin.csv'), ('pct_true', 0.6943909525871277)])
OrderedDict([('tweet', "We can't leave states, cities &amp; towns behind &amp; ignore the livelihoods of firefighters, police officers &amp; teachers. I’m glad that @senatemajldr has backed off the ridiculous position that states should declare bankruptcy. https://t.co/LSCbW6HrKC"), ('dates', '2020-04-28 15:14:48'), ('file_name', 'James R. Langevin.csv'), ('pct_true', 0.699752151966095)])
OrderedDict([('tweet', 'Many Americans are facing financial uncertainty due to #coronavirus – not b/c of actions they’ve taken. Those struggling to make ends meet cannot afford to take a credit hit, which is why I’ve cosponsored the Disaster Protection for Workers’ Credit Act.\nhttps://t.co/rnVOUcpOe6'), ('dates', '2020-04-09 16:32:55'), ('file_name', 'James R. Langevin.csv'), ('pct_true', 0.6472810506820679)])
OrderedDict([('tweet', '“Swastikas and Confederate flags, nooses and automatic rifles do not represent who we are...” Solidarity with Governor Whitmer and Michiganders facing down rabid gun-toting Trump-inflamed zealots. https://t.co/MH1fayoPBU'), ('dates', '2020-05-02 18:04:31'), ('file_name', 'Jamie Raskin.csv'), ('pct_true', 0.6275362968444824)])
OrderedDict([('tweet', 'Did you know that 1990 was the first Asian-Pacific American Heritage Month? It started as as Asian-Pacific American Heritage Week in 1978, but Congress passed a public law expanding the observance to the full month of May. Read more via @librarycongress: https://t.co/RPKVP7pBME'), ('dates', '2020-05-01 15:28:18'), ('file_name', 'Jamie Raskin.csv'), ('pct_true', 0.8084421753883362)])
OrderedDict([('tweet', 'The anti-Asian bigot &amp; racist Caputo must be fired today. His deranged, profane &amp; hate-filled polemics disqualify him for public employment &amp; he should quickly return to the rock he lives under. Asian Americans don’t deserve this appalling insult &amp; neither do the rest of us. https://t.co/LagL11rRem'), ('dates', '2020-04-23 20:07:46'), ('file_name', 'Jamie Raskin.csv'), ('pct_true', 0.9727857708930969)])
OrderedDict([('tweet', "Trump's proposed payroll tax cut would harm Social Security and benefit corporations and the wealthy. I joined @RepJohnLarson on the Emergency Social Security Benefits Improvement Act to ensure we are helping our seniors through #COVID19. #ExpandSocialSecurity"), ('dates', '2020-05-07 17:06:17'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.8723350167274475)])
OrderedDict([('tweet', 'We should be strengthening #SocialSecurity, not giving a payroll tax cut to corporations and the wealthy. That is why I joined @RepJohnLarson on the Emergency Social Security Benefits Improvement Act, to help our most vulnerable during the #COVID19 pandemic. #ExpandSocialSecurity'), ('dates', '2020-05-07 17:05:11'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.6514921188354492)])
OrderedDict([('tweet', 'Trump may have abandoned his responsibility to lead in the midst of this pandemic, but our local governments are on the front lines. Why won’t McConnell put #FamiliesFirst and let Congress make sure communities nationwide have the resources they need to keep up the fight?'), ('dates', '2020-05-06 19:23:15'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.9248433113098145)])
OrderedDict([('tweet', 'Xenophobia and racism will not excuse the failure of this Administration to act swiftly to prevent the spread of the coronavirus in the U.S.'), ('dates', '2020-04-23 20:12:38'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.6079947352409363)])
OrderedDict([('tweet', 'Today, we celebrate #EarthDay2020 yet those who protect it are illegally behind bars in #Honduras. @PJdeHonduras has the freedom of #Guapinol HRDs in their hands and must ensure their immediate release.'), ('dates', '2020-04-22 17:40:32'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.5273735523223877)])
OrderedDict([('tweet', 'The cruelty is the point. \n\nTrump’s announcement that he will suspend immigration to the U.S. is simply another racist attempt to scapegoat immigrants during the COVID-19 pandemic.  \n\n#ImmigrationBan'), ('dates', '2020-04-21 19:05:44'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.9417926073074341)])
OrderedDict([('tweet', 'Another shameful example of failed leadership. Clearly @SBA under @realDonaldTrump can’t handle another blank check. So it’s critical for Congress to ensure that any PPP refill includes guardrails that actual small businesses get the relief they badly need https://t.co/kUIu6O7CPS'), ('dates', '2020-04-20 20:25:57'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.5355719923973083)])
OrderedDict([('tweet', 'This is criminal! \n\n@DHSgov @DHSMcAleenan @ICEgov @realDonaldTrump - stop these deportations now. If we can’t handle this pandemic, what effect do you think it will have on countries with weaker health care infrastructures. https://t.co/WmxgRF4nHe'), ('dates', '2020-04-18 18:16:30'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.5329400300979614)])
OrderedDict([('tweet', 'Trump’s @EPA refused to strengthen air quality during a pandemic caused by a RESPIRATORY disease. The pollution they‘re allowing disproportionately impacts Black &amp; low-income people already bearing the brunt of #COVID19. My Illinois colleagues &amp; I want @EPAAWheeler to do better. https://t.co/aXIOMDOIGB'), ('dates', '2020-04-16 21:31:52'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.6816465258598328)])
OrderedDict([('tweet', 'Minority-owned businesses traditionally have a harder time getting loans and may be neglected by big banks offering #PPPloans. @housedemocrats are fighting to get them #Smallbizrelief through community lenders. https://t.co/k7tq5X5NaQ'), ('dates', '2020-04-15 16:03:09'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.6725424528121948)])
OrderedDict([('tweet', "We all need to understand THE FACTS of this pandemic. @SpeakerPelosi perfectly outlined those facts and how @realDonaldTrump's actions, or lack thereof, got us to this point. #COVID19 #coronavirus \nhttps://t.co/94iloOVGQr"), ('dates', '2020-04-15 13:47:38'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.72414231300354)])
OrderedDict([('tweet', 'For over 365 days the Senate had a chance but failed to act on legislation to support survivors of domestic violence &amp; sexual assault. Unacceptable! I stand w/ survivors and call on the Senate to pass VAWA (S. 2843) today! #VAWA4ALL'), ('dates', '2020-04-07 18:43:00'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.8069123029708862)])
OrderedDict([('tweet', 'For millions of Americans, sheltering in place due to #COVID19 can be dangerous, particularly those in #DomesticAbuse situations. It’s time Senators stood with survivors and passed #VAWA4ALL'), ('dates', '2020-04-07 18:42:09'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.7284702062606812)])
OrderedDict([('tweet', 'Today we’re marking over 1 year since the House passed the Violence Against Women Reauthorization Act w/ strong bipartisan support. But the Senate has failed to act. It’s time the Senate reauthorized #VAWA without delay! #VAWA4ALL'), ('dates', '2020-04-07 18:41:33'), ('file_name', 'Janice D. Schakowsky.csv'), ('pct_true', 0.9537598490715027)])
OrderedDict([('tweet', "SMALL BUSINESSES: The Paycheck Protection Program (PPP) is starting back up Monday at 10:30am after Congress passed $310b in new PPP funding.\n\nWe've put out a guide to help you navigate the PPP, and we have a small business team on staff ready to work with you &amp; your business. 1/"), ('dates', '2020-04-26 22:08:55'), ('file_name', 'Jared F. Golden.csv'), ('pct_true', 0.5658187866210938)])
OrderedDict([('tweet', 'Many Mainers are facing layoffs and small businesses are shutting their doors. We need to be honest about what’s happening in our economy and acknowledge that things will get worse before they get better. \n\n1/'), ('dates', '2020-03-21 12:29:23'), ('file_name', 'Jared F. Golden.csv'), ('pct_true', 0.6540718078613281)])
OrderedDict([('tweet', 'When we started 2019, @SBAgov had no office focused solely on rural small businesses. That’s a problem for our district here in Maine, which is one of the most rural in the country. Our small businesses need the same access to SBA services as small biz in cities. 1/'), ('dates', '2020-02-06 00:04:04'), ('file_name', 'Jared F. Golden.csv'), ('pct_true', 0.7874075174331665)])
OrderedDict([('tweet', 'The USMCA should go down in history for what it is — a status quo trade deal that doubles down on the same disastrous policies of the past few decades.\n\nRead my op-ed in the @sunjournal here: https://t.co/OrCd03aped\n\n#mepolitics'), ('dates', '2020-02-03 14:13:54'), ('file_name', 'Jared F. Golden.csv'), ('pct_true', 0.8241678476333618)])
OrderedDict([('tweet', "This week is the 10th anniversary of Citizens United — the 2010 Supreme Court decision that has allowed special interests to flood our political system with dark money.\n\nThe tentacles of the Citizens United decision are everywhere. It's time to #EndCitizensUnited"), ('dates', '2020-01-24 15:56:13'), ('file_name', 'Jared F. Golden.csv'), ('pct_true', 0.5563591718673706)])
OrderedDict([('tweet', "This week, the House will vote on two articles of impeachment against the president. I've reached a decision and I want Mainers to hear it straight from me. \n\nPlease take the time to read and consider what I have to say in its entirety. #mepolitics https://t.co/SgFQ4NLvf3"), ('dates', '2019-12-17 23:01:10'), ('file_name', 'Jared F. Golden.csv'), ('pct_true', 0.5798361301422119)])
OrderedDict([('tweet', "This week, the House will vote on the Lower Drug Costs Now Act, which would bring down Rx drug costs for millions of Americans.\n\nBig Pharma &amp; dark money groups are spending millions to spread misleading info about the bill. Why you shouldn't believe them: https://t.co/0OWijkB72T"), ('dates', '2019-12-11 13:47:35'), ('file_name', 'Jared F. Golden.csv'), ('pct_true', 0.7444290518760681)])
OrderedDict([('tweet', 'For more than 10 months, I’ve worked with Democrats &amp; Republicans to make sure the defense authorization bill, the NDAA, keeps our country safe, looks out for servicemembers &amp; their families, and supports Maine jobs. 1/4 #mepolitics'), ('dates', '2019-12-10 23:33:36'), ('file_name', 'Jared F. Golden.csv'), ('pct_true', 0.8808273673057556)])
OrderedDict([('tweet', 'This week, the House votes on HR 3, the #LowerDrugCosts Now Act. Big Pharma is spending millions on misleading ads against the bill b/c they know it would lead to lower drug prices.\n\nLet’s set the record straight. Here’s what H.R. 3 would (&amp; wouldn’t) do for affordable Rx drugs. https://t.co/Dn9jYjTZMY'), ('dates', '2019-12-09 23:29:37'), ('file_name', 'Jared F. Golden.csv'), ('pct_true', 0.7602350115776062)])
OrderedDict([('tweet', 'While Trump &amp; his allies are bailing out already failing oil companies, prominent lenders are taking a stand thanks to public pressure. Keep it up! \n\nPollution is bad for business, &amp; it’s time all banks rule out drilling in the Arctic. #ProtectTheArctic https://t.co/Nb54l84N9G'), ('dates', '2020-05-06 13:43:21'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.7487643957138062)])
OrderedDict([('tweet', '3.8 million Americans filed for unemployment just last week—over 30 mil people have now lost their jobs. Congress needs to get more relief to families, businesses, communities, and states NOW. https://t.co/AqoWIRmNEA'), ('dates', '2020-04-30 21:05:00'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.6304423213005066)])
OrderedDict([('tweet', '.@OSHA_DOL is charged with protecting US workers &amp; ensuring safe workplaces. But in the middle of the worst safety crisis in recent history, the Trump admin is refusing to take action. We must protect workers on the frontlines of the fight against #COVID19. #WorkersMemorialDay https://t.co/uxEvDfGIhc'), ('dates', '2020-04-29 00:02:18'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.5374776721000671)])
OrderedDict([('tweet', 'It’s not a novel concept. Any large allocation of $ invites misuse. Yesterday’s vote to create an oversight subcommittee for #COVID19 spending follows the lessons from Truman’s WWII spending oversight committee, &amp; will ensure taxpayer funds are used appropriately #ForThePeople.'), ('dates', '2020-04-24 23:02:00'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.6289225816726685)])
OrderedDict([('tweet', 'I don’t care what your politics are – this is disgusting. Firefighters, police &amp; EMS are leading the fight to save American lives while countless other local/state employees are still on the job. @senatemajldr wants to deny them aid. #MitchAntoinette https://t.co/KgbEpouaji'), ('dates', '2020-04-24 01:35:13'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.7348891496658325)])
OrderedDict([('tweet', 'Shameful from the @senatemajldr. States and local governments are providing essential services during this pandemic and their budgets are hurting just like other employers. We have a responsibility to support them just as we have the private sector. https://t.co/1k4kPAtkZJ'), ('dates', '2020-04-23 16:37:56'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.5554457902908325)])
OrderedDict([('tweet', 'Once again, President Trump has broken his promises to the American people. We need more resources to protect our health workers, provide for the community, and begin the path to recovery.\nhttps://t.co/IgM19JsHvC'), ('dates', '2020-04-22 15:38:39'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.5380611419677734)])
OrderedDict([('tweet', 'The Trump admin trying every tactic to help the oil industry. Trump needs to prioritize increased testing, small business relief, and families -- not use this crisis as a veil to prop up Big Oil. #PeopleOverPolluters\nhttps://t.co/XRP4w9uhxX'), ('dates', '2020-04-21 15:27:46'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.5284938812255859)])
OrderedDict([('tweet', 'Despite what you might have heard from the IRS, Social Security beneficiaries will automatically get their CARES Act payments. That’s how Congress designed this program, &amp; we pushed hard to make sure the IRS delivers Congress’ relief to America’s seniors. https://t.co/MHQVih15cR'), ('dates', '2020-04-08 20:33:20'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.6208865642547607)])
OrderedDict([('tweet', "Trump's failure to act on #Coronavirus has put Native communities at risk. Lives will be lost that should have been saved. @HouseDemocrats passed 3 relief packages w/ major aid for Indian Country. Fed agencies must work w/ Tribes NOW to stop the #CoronavirusIndianCountry outbreak https://t.co/O94hfbQ8Gw"), ('dates', '2020-04-02 18:03:38'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.5963262319564819)])
OrderedDict([('tweet', 'We should not let our very real disputes w. the Iranian govt prevent us from treating the people of Iran with compassion. @realdonaldtrump needs to ease sanctions that will damage the Iranian health care system &amp; could contribute to the spread of #COVID19.\nhttps://t.co/4NGd0Mf3b4'), ('dates', '2020-04-02 14:51:04'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.5590925216674805)])
OrderedDict([('tweet', 'We can’t leave our front-line fighters without the supplies they need to save American lives. The Medical Supply Chain Emergency Act will force the President to federalize American manufacturing &amp; end the supply shortage. We need the #DefenseProductionActNOW. https://t.co/BCj2mqMVHj'), ('dates', '2020-03-27 18:11:00'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.5203908681869507)])
OrderedDict([('tweet', "Trump's still trying to strip health care from millions &amp; the McConnell bill provides a slush fund for favored industries. Outrageous. We need tests, PPEs, &amp; we can’t just bail out favored industries if we’re actually going to protect the US from #COVID19.\nhttps://t.co/AHhPp3EG4U"), ('dates', '2020-03-23 19:07:42'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.7601557970046997)])
OrderedDict([('tweet', 'This is shameful and illegal. Elected officials should serve the American people, not their own self interests. \n\nThese Republican Senators must answer to the millions of Americans who are suffering, and we need to know whether anyone in the White House made similar trades. https://t.co/Zn5aATQpCG'), ('dates', '2020-03-20 15:44:12'), ('file_name', 'Jared Huffman.csv'), ('pct_true', 0.9447494745254517)])
OrderedDict([('tweet', "75 years ago, Allied forces accepted Nazi Germany's unconditional surrender – ending WWII in Europe.\n\nAlong the front lines and on the home front, a generation of Americans stepped up to protect our democracy. Forever grateful for the brave women &amp; men who fought in WWII #VEDay75 https://t.co/nycypMUUgJ"), ('dates', '2020-05-08 23:16:00'), ('file_name', 'Jason Crow.csv'), ('pct_true', 0.5253853797912598)])
OrderedDict([('tweet', 'We need to confront this crisis with bold action. This week, I introduced legislation to create the Health Force -- this bill would employ Americans who may have lost their job from the crisis and invest that energy and talent into combating this disease. https://t.co/MEUszRec9w'), ('dates', '2020-05-08 20:56:06'), ('file_name', 'Jason Crow.csv'), ('pct_true', 0.6486114263534546)])
OrderedDict([('tweet', 'Without a unified response to coronavirus, the President has left states scrambling in a zero-sum game where there are no winners. \n\nToday, I introduced legislation requiring the president to utilize the Defense Production Act to mobilize a federal response to the pandemic. https://t.co/Y7UC2RnAT5'), ('dates', '2020-04-29 15:47:49'), ('file_name', 'Jason Crow.csv'), ('pct_true', 0.5346616506576538)])
OrderedDict([('tweet', 'The @SBAgov’s Paycheck Protection Program has just run out of money, leaving business owners who are trying to pay their employees stranded.\n\nWe need to fund the PPP and other SBA programs -- there are small businesses counting on our support.'), ('dates', '2020-04-16 15:41:44'), ('file_name', 'Jason Crow.csv'), ('pct_true', 0.8306488394737244)])
OrderedDict([('tweet', 'Leadership means taking responsibility.\n\nCAPT Crozier knows that, he spoke up to protect his sailors. And it may have cost him his job.\n\nWe don’t play politics with the military, especially when lives are on the line.\n\nThe Navy owes our sailors, country, &amp; Congress an explanation https://t.co/VltTL6lERR'), ('dates', '2020-04-02 22:34:58'), ('file_name', 'Jason Crow.csv'), ('pct_true', 0.7313766479492188)])
OrderedDict([('tweet', '75 years ago today, the Nazi regime surrendered to the Allied Powers, marking the end of WWII in Europe. \n\nToday, we celebrate the brave patriots who fought back Nazi tyranny to preserve our liberty and freedom for future generations. #VEDay75 https://t.co/o5qXoJN33w'), ('dates', '2020-05-09 00:27:39'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.8566228151321411)])
OrderedDict([('tweet', 'Nancy Pelosi has stalled the government’s response to the #coronavirus since day one. \n\nWhen @POTUS first asked Congress for funding to expand testing, Pelosi refused to hold a vote on it for 10 days.\n\nWhat did she have time to hold a vote on instead?\n\nBanning flavored tobacco.'), ('dates', '2020-05-05 21:13:57'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.9573785066604614)])
OrderedDict([('tweet', 'While people around the country are fighting to return to work - Speaker Pelosi decided it best to keep half of Congress closed while she still gets paid.\n\nGuess she wants to spend more time video chatting with late-night tv personalities. \n\nCompletely ridiculous!'), ('dates', '2020-05-03 15:12:58'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5728930234909058)])
OrderedDict([('tweet', "According to one estimate, Missouri has lost $44 billion from #coronavirus shutdowns, a global pandemic that happened because of China’s deceit &amp; coverup.\n\nThis is absolutely unacceptable.\n\nI'm glad Missouri is taking the lead &amp; holding #China accountable. https://t.co/8bqTEAFHpr"), ('dates', '2020-04-22 13:25:01'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.8771154284477234)])
OrderedDict([('tweet', 'It’s clear that China covered up the coronavirus, which allowed it to spread across the world \n\nThey must be held responsible for their reckless behavior.'), ('dates', '2020-04-20 02:20:45'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.9310061931610107)])
OrderedDict([('tweet', "For seven long days, Washington Democrats have blocked $250 billion in much-needed assistance for small businesses and their workers.\n\nYesterday, the fund finally ran out of money, and there's still no sense of urgency from Chuck Schumer or Nancy Pelosi. https://t.co/QcBSdo30GY"), ('dates', '2020-04-17 17:31:10'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7274796962738037)])
OrderedDict([('tweet', 'It’s unfortunate that some see a national crisis as a chance to take advantage of their fellow citizens. Thankfully, @realdonaldtrump took the first steps this week to implement my bill to protect American consumers from price gouging.\n\nMORE ⬇️https://t.co/8V6Nb1hBOF'), ('dates', '2020-03-25 19:16:16'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5168728232383728)])
OrderedDict([('tweet', 'While Nancy Pelosi wants to pass her partisan agenda, I’ve been on the phones with the people I represent in southern Missouri, listening to their concerns and hearing how the federal government can help. \n\nThose are the ideas we should be including! https://t.co/TYFBd7UNEF'), ('dates', '2020-03-24 20:00:30'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5172819495201111)])
OrderedDict([('tweet', 'Today’s #FISA bill doesn’t do enough to reign in recent abuses or protect Americans from targeting based on their political beliefs. \n\nIt could have. \n\nMy amendment added additional layers of due process protections from this kind of abuse. \n\nSadly, it was BLOCKED by House Dems. https://t.co/pPX5la9gDQ'), ('dates', '2020-03-11 21:55:06'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5627518892288208)])
OrderedDict([('tweet', 'It’s clear: both Republicans AND Democrats want Nancy Pelosi and her Budget Committee to do their job and pass a budget.\n\nLast week, 17 Democrats sent a letter to @HouseBudgetDems asking the Budget Committee to do one thing: its job.\n\nMORE ⬇️ https://t.co/8lB0DCqQuU'), ('dates', '2020-03-11 17:54:46'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.8093494772911072)])
OrderedDict([('tweet', 'China has exploited our farmers and workers again and again and again. \n\nDecades have gone by with no change. And now, when President Trump decides to fight back?\n\nJust bickering and politics from Democrats. Disgraceful.\n\nMORE ⬇️ https://t.co/zMoFVUyRLR'), ('dates', '2020-03-10 12:59:45'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.9456311464309692)])
OrderedDict([('tweet', 'This week, the courts said that @POTUS can withhold funding from #SanctuaryCities.\n\nIt’s common sense. \n\nNo taxpayer should subsidize these liberal policies that incentivize illegal immigration by offering criminals asylum from law enforcement\n\nMORE ⬇️\nhttps://t.co/5VBWLrJis4'), ('dates', '2020-03-07 23:13:27'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.616874635219574)])
OrderedDict([('tweet', 'Even when the health and safety of the American people is at stake, Democrats in Washington can’t get over their personal hatred of the President.\n\nA writer at the New York Times recently called it the “TrumpVirus.”\n\nIt’s unbelievable! https://t.co/ACaOSS37Ip'), ('dates', '2020-03-04 22:22:32'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.6270653605461121)])
OrderedDict([('tweet', 'Whenever the President does ANYTHING, Democrats in Washington still find a way to complain.\n\nThey need to get over their issues and start working with the President on behalf of the American people - just like Republicans are doing. \n\nMORE ⬇️ https://t.co/HWkJUGsHfP'), ('dates', '2020-03-01 02:53:34'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.8596347570419312)])
OrderedDict([('tweet', 'After decades of politicians and presidents talking about farmers’ \nproblems but getting nothing done, we finally have a champion in President \nTrump who is fighting for our interests.\n\nDemocrats’ response? Just more and more complaining like always.\n\nWATCH ⬇️ https://t.co/OkpXTVvYHk'), ('dates', '2020-02-27 18:52:36'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5420625805854797)])
OrderedDict([('tweet', "Nancy Pelosi and House Ds won't propose a budget. \n\nIs that because they have no policies to propose?\n\nOr is it because they don't want to show America those policies and the soul-crushing taxes they'd levee on middle class families to pay for it? ⬇️ https://t.co/xaDdh1C9HL"), ('dates', '2020-02-15 18:52:22'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.947580099105835)])
OrderedDict([('tweet', 'The irony of Speaker Pelosi criticizing @POTUS’s budget proposal this week is that her party didn’t even propose a budget last year. \n\nIt’s because they don’t want the American people to know how much they plan to spend on the Green New Deal or their government run health care. https://t.co/xPtdQQrUFy'), ('dates', '2020-02-12 23:32:57'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.8524184226989746)])
OrderedDict([('tweet', 'The @UN’s list is shameful and sickening. \n\nIt’s not only horribly anti-Semitic, but it undermines @POTUS’s recently proposed peace plan by attempting to force concessions from Israel. \n\nIt’s clear the UN has been corrupted by the disgusting #BDS movement. https://t.co/soP45r7kD9'), ('dates', '2020-02-12 22:47:53'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7573239803314209)])
OrderedDict([('tweet', 'The irony of Speaker Pelosi ripping up @POTUS’s speech is that’s exactly what the U.S. Senate is going to do to her articles of impeachment tomorrow.'), ('dates', '2020-02-05 04:09:36'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.6838720440864563)])
OrderedDict([('tweet', 'Washington Democrats apparently don’t support pre-existing conditions or protecting social security for seniors. They couldn’t be bothered to stand for that or investing in our children’s education. #SOTU'), ('dates', '2020-02-05 02:49:08'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.926150918006897)])
OrderedDict([('tweet', 'Unbelievable that Democrats in Washington won’t stand for historic low unemployment or more jobs for women. Instead Nancy just shakes her head. #SOTU'), ('dates', '2020-02-05 02:27:34'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.9471778869628906)])
OrderedDict([('tweet', 'President Trump understands how precious all lives are, including those of the unborn. His administration issued a rule to prevent taxpayer-funded abortions and cut funding from the UN because of its support for coercive abortions. #3YearsofTrump'), ('dates', '2020-01-26 17:09:21'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.8801030516624451)])
OrderedDict([('tweet', 'Speaker Pelosi and her liberal lieutenants have shredded the Constitution in their desperate attempt to get rid of President Trump. And Democrat Senators Bernie Sanders, Elizabeth Warren, and others are complicit. Watch my full remarks here: https://t.co/T1TaMCBtjR'), ('dates', '2020-01-25 15:00:06'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7757219076156616)])
OrderedDict([('tweet', 'Drugs and illegal aliens stream into our country when the border is left unsecured. That’s why President Trump has led the fight to build the border wall so we can re-establish control over our sovereign territory.\n#3YearsofTrump https://t.co/bbUkCAXuw0'), ('dates', '2020-01-23 02:25:19'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5263328552246094)])
OrderedDict([('tweet', '.@POTUS Trump made it a national priority to renegotiate unfair trade deals to finally put American farmers, ranchers, and manufacturers front and center.\nJust this past week, he signed the #PhaseOne deal with China at the White House.\n\n#3YearsofTrump'), ('dates', '2020-01-21 22:15:00'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.6586620807647705)])
OrderedDict([('tweet', 'President Trump has fought back against China for our farmers.\n\nHis strong leadership, coupled with a booming U.S. economy, forced China to agree to fairness and access for our farmers. \n\nMORE ⬇️ https://t.co/MRgdJG67Ek'), ('dates', '2020-01-11 21:25:06'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7129555940628052)])
OrderedDict([('tweet', ".@SpeakerPelosi, a fair trial starts with requiring those Senators running for President to recuse themselves from the process. They are in an undeniable position to benefit from the outcome of the trial which constitutes a clear and obvious conflict of interest. Don't you agree? https://t.co/XYOoxtKN7n"), ('dates', '2020-01-09 22:39:01'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.8239110708236694)])
OrderedDict([('tweet', 'You have already shown the American People that you are not the best person to be giving advice on how to conduct impeachment proceedings. Maybe you should let the Senate handle this one...you know like the Constitution says. https://t.co/1ZGjMFn0Bc'), ('dates', '2020-01-07 01:13:38'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5405821204185486)])
OrderedDict([('tweet', 'Iran never complied with the disastrous Iran Nuclear deal negotiated by the Obama Administration. President Trump was right to walk away from it and work to protect America, our values, and our allies against the largest state sponsor of terrorism. https://t.co/ruFb7wPkAf'), ('dates', '2020-01-05 20:59:06'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7864388823509216)])
OrderedDict([('tweet', 'Speaker Pelosi knows her #impeachmentsham was a fraud. Her work was so sloppy and negatively received that Senate Democrats are asking her to not send the #ArticlesOfImpeachment over to the Senate because they lack any evidence and it is not something they want to vote on.'), ('dates', '2020-01-02 22:02:20'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7951363921165466)])
OrderedDict([('tweet', 'The Iranian regime is desperate because President Trump’s maximum pressure campaign is working. They attacked and killed an American abroad last week and are now lashing out by directing a mob to attack our embassy in Baghdad. https://t.co/EGe91i3cn4'), ('dates', '2019-12-31 16:38:50'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7690078020095825)])
OrderedDict([('tweet', 'President @realDonaldTrump has spent his presidency fighting to secure better market access for our farmers.\n\nLast month, China doubled their imports of U.S. soybeans following the “Phase One” trade deal.\n\nThese are real results delivered by @POTUS Trump. https://t.co/jkGsJRv25f'), ('dates', '2019-12-27 20:28:21'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.6953795552253723)])
OrderedDict([('tweet', 'We should not be surprised at all that @SpeakerPelosi, @RepAdamSchiff and the rest of the Left voted to impeach President @realDonaldTrump. \n\nIt’s what they‘ve done against five of our last six Republican Presidents.\n\nThey’re the party of #impeachment! https://t.co/GBKqnLbJeI'), ('dates', '2019-12-26 19:55:02'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7383236885070801)])
OrderedDict([('tweet', '.@SpeakerPelosi played politics with the #USMCA.\n\nShe held it hostage for a full year just to try and claim at the last minute that her party was not entirely distracted by impeaching President @realDonaldTrump.\n\nMy full remarks from @WaysandMeansGOP ⬇️ https://t.co/pt9FPgIY1d'), ('dates', '2019-12-23 22:44:04'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5117501020431519)])
OrderedDict([('tweet', 'The Democrats have introduced articles of impeachment against five of the last six Republican Presidents. Their most recent attempt shows that they will do whatever is necessary to remove a President who gets in the way of their partisan agenda. \n\nMore ⬇️ https://t.co/NoWdOnu0qH'), ('dates', '2019-12-21 23:17:27'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.9801205992698669)])
OrderedDict([('tweet', "The President did absolutely NOTHING wrong, but that didn't stop Democrats from impeaching him. \n\nThey hate @realDonaldTrump and the historic results he's delivered for the American People. \n\nSad! https://t.co/o24n0ERQm5"), ('dates', '2019-12-20 20:12:47'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.9252631664276123)])
OrderedDict([('tweet', "Quick Recap:\n\n1. Radical Dems say they didn’t come to Congress to impeach @POTUS @realDonaldTrump. They did\n\n2. Radical Dems say yesterday was a “solemn” day. They couldn't even contain their excitement on the House Floor.\n\nDisgusting and Disgraceful. https://t.co/ox36zXG30v"), ('dates', '2019-12-19 19:40:30'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.9481217861175537)])
OrderedDict([('tweet', 'ICYMI: I just spoke on the House Floor against this #impeachment circus. \n\n"Democrats have introduced articles of impeachment in five out of our last six Republican presidents. They are the party of impeachment. The Democrats are the party of impeachment."\n\nFull Remarks ⬇️ https://t.co/F8efsDNAwR'), ('dates', '2019-12-18 21:55:41'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.6585859656333923)])
OrderedDict([('tweet', 'Democrats blocked my motion that would have given all members time to speak on this infamous impeachment.\n\nThey do not want Republican voices to be heard because they know we are right: President @realDonaldTrump did nothing wrong! https://t.co/d2PTbvaNxt'), ('dates', '2019-12-18 16:38:01'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.9442119002342224)])
OrderedDict([('tweet', 'Just this week, @SpeakerPelosi admitted that they have been trying to #impeach @realDonaldTrump since he was elected.\n \nThis again confirms how they’ve been unable and unwilling to accept the results of the 2016 election. https://t.co/Yy5E9iUv7W'), ('dates', '2019-12-14 16:50:42'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.6780262589454651)])
OrderedDict([('tweet', '#China has agreed to increase imports of U.S. goods and services by at least $200 billion.\n \nThey will also purchase upwards of $50 billion in U.S. #agricultural products.  \n \nThese are huge wins for our workers and farmers. \n\nThank you @realDonaldTrump!'), ('dates', '2019-12-13 20:10:10'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5491117835044861)])
OrderedDict([('tweet', "This landmark agreement was only possible because of @realDonaldTrump's willingness to stand up for American businesses, workers, and farmers who have been taken advantage of for years by China's and its abusive economic and trading policies. https://t.co/X2Txwjolod"), ('dates', '2019-12-13 19:58:27'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.8097452521324158)])
OrderedDict([('tweet', 'The lack of fact-based arguments from Democrats during #ImpeachmentDebate last night and this morning show just how much hate they have for @realDonaldTrump. \n\nThey are the party of impeachment.'), ('dates', '2019-12-12 17:30:55'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5920988321304321)])
OrderedDict([('tweet', 'This politically motivated impeachment of @realDonaldTrump has been devoid of fairness, rife with bias &amp; it threatens to shake the foundations of our democracy.\n\nAny chance for a fair trial starts with Senators running for President to recuse themselves!⬇️ https://t.co/ks8ReQ5ujx'), ('dates', '2019-12-09 20:15:38'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.8107757568359375)])
OrderedDict([('tweet', 'Chairman Nadler is putting himself before country. He is too concerned about pleasing his liberal base to satisfy their appetite for removing @realDonaldTrump. \n\nHe represents the 2nd smallest district in the country (14.25 miles). \n\nHe speaks for NY elites, not all Americans.'), ('dates', '2019-12-09 14:37:19'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7033401131629944)])
OrderedDict([('tweet', "It’s clear @SpeakerPelosi wants to impeach @realDonaldTrump because she doesn’t like him.\n\nI introduced legislation to ensure the Senate trial isn't another smear campaign.\n\nIt's simple, no Senator should participate who is running against our President. https://t.co/QAri0XQVMz"), ('dates', '2019-12-05 21:01:17'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7514462471008301)])
OrderedDict([('tweet', '.@SpeakerPelosi officially announced Articles of Impeachment against @POTUS @realDonaldTrump. \n\nThis is a sad day for America. We are formally impeaching the President of the United States regardless of truth or facts. \n\nI continue to stand with our President @realDonaldTrump!'), ('dates', '2019-12-05 15:24:19'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5519581437110901)])
OrderedDict([('tweet', "Tomorrow's #ImpeachmentHearing will have 3 Democrat witnesses, but only 1 for Republicans. \n\nWhy are they continuing with their sham process?\n\nBecause they need to control all the facts to maintain their narrative.\n\nThey want to impeach @realDonaldTrump at all costs."), ('dates', '2019-12-03 19:34:23'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.6603319048881531)])
OrderedDict([('tweet', 'While the Left is wasting our time with this latest, pointless #ImpeachmentHearing, Republicans are working. Later this week, we will confirm yet another Trump appointee to the federal bench and flip the 11th Circuit. This means more judges following the law, not inventing it. https://t.co/RlMLo3wGTu'), ('dates', '2019-11-20 17:40:45'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.5890043377876282)])
OrderedDict([('tweet', 'To respond to @RepAdamSchiff’s erroneous statement: We are not “upset that Trump got caught." We are upset that you, and the rest of the Left, have been plotting this politically motivated #impeachment since the moment @POTUS @realDonaldTrump took office. https://t.co/TF5sZ89RPm'), ('dates', '2019-11-20 15:33:27'), ('file_name', 'Jason Smith.csv'), ('pct_true', 0.7355550527572632)])
OrderedDict([('tweet', '.@realDonaldTrump’s threats to withhold critical assistance for @USPS over political motivations are egregious &amp; unacceptable. We cannot play politics amid this crisis. Americans rely on USPS for medication, voting, stimulus checks, tax refunds and more.'), ('dates', '2020-05-10 14:01:26'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.7695927619934082)])
OrderedDict([('tweet', 'Americans have never hesitated to step up &amp; give back during times of national crisis. This pandemic is no exception, and Congress can boost our response to COVID-19 and support our recovery by expanding national service programs with America’s best and brightest. #Stand4Service'), ('dates', '2020-05-08 17:10:24'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.9124444127082825)])
OrderedDict([('tweet', "The President's dangerous, partisan policies compromise women’s health in the most vulnerable communities around the world. Now, they are undermining #COVID19 response efforts. We're calling on admin to reverse these policies NOW so orgs have resources to provide lifesaving care. https://t.co/zFI1taMN9M"), ('dates', '2020-05-06 13:54:39'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.7777869701385498)])
OrderedDict([('tweet', 'Small business relief programs were intended for small businesses that are truly hurting - not\xa0big\xa0corporations. Now on Senate floor, I’m calling for the Senate to pass legislation that will shed light on these programs &amp; hold the Trump admin accountable.\xa0https://t.co/NXmhms6mvk'), ('dates', '2020-05-05 18:18:19'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.6869291067123413)])
OrderedDict([('tweet', 'COVID-19 is a global problem and needs a global solution. The U.S. shouldn’t go it alone. Contributing to the global effort to create a vaccine doesn’t diminish our own efforts – in fact, it magnifies the work already underway in American labs and hospitals. https://t.co/wL34v5g25Y'), ('dates', '2020-05-05 15:05:22'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.5679409503936768)])
OrderedDict([('tweet', 'Granite Staters are hurting which is why I’m calling on Mitch McConnell to prioritize COVID-19 response legislation rather than more votes on unqualified, right-wing judges. There are so many needs in our communities – no time should be wasted!'), ('dates', '2020-05-05 14:44:32'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.9395286440849304)])
OrderedDict([('tweet', 'The Trump administration continues to raid funding that Congress provided for our national security and divert it to his border wall. This is unconstitutional and dangerous. It also sets a horrible precedent.  These projects are critical to deterring Russian aggression in Europe. https://t.co/knSo3r0K8a https://t.co/jArf1N46pd'), ('dates', '2020-04-29 23:11:43'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.5906318426132202)])
OrderedDict([('tweet', 'Our local governments urgently need access to COVID-19 relief funds, but the Trump admin has put in place bureaucratic restrictions &amp; Mitch McConnell is wrong: bankruptcy is not an option. I’m calling on the admin to reverse course &amp; allow local governments to access CARES funds. https://t.co/GLfJ2Jctkm'), ('dates', '2020-04-27 19:13:38'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.6987702250480652)])
OrderedDict([('tweet', 'China’s military provocations against our allies and partners in the Asia-Pacific amid a global crisis are concerning. @marcorubio and I are calling on @EsperDoD to maintain our military’s readiness in the Pacific &amp; ensure the U.S. stands ready to fulfill its defense commitments. https://t.co/pNFvUewl6l'), ('dates', '2020-04-23 22:37:53'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.6993746161460876)])
OrderedDict([('tweet', 'Today on #YomHaShoah, we mourn &amp; remember the six million Jewish lives taken &amp; all those viciously murdered in the Holocaust. It’s on all of us to speak up &amp; fight back against antisemitism &amp; all forms of hatred &amp; bigotry wherever they appear. #NeverAgain #HolocaustRemembranceDay'), ('dates', '2020-04-21 22:22:56'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.5159920454025269)])
OrderedDict([('tweet', 'Unbelievable. The Trump admin should be working to strengthen, not weaken, proposed rules designed to protect Americans from exposure to harmful PFAS chemicals. The admin needs to listen to EPA scientists who understand the need to enact these important protections without delay. https://t.co/vn1zKnrTsA'), ('dates', '2020-04-21 13:07:24'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.5490568280220032)])
OrderedDict([('tweet', 'Far too many Granite Staters and Americans living in rural communities are in danger of losing their homes as a result of COVID-19. Congress needs to stand up for these families and ensure rural housing programs get the funding they need in upcoming COVID-19 response legislation. https://t.co/tZhUq5KKWR'), ('dates', '2020-04-19 16:30:11'), ('file_name', 'Jeanne Shaheen.csv'), ('pct_true', 0.6281051635742188)])
OrderedDict([('tweet', "If the people over at the FBI &amp; DOJ involved in this hit job against President Trump don't go to jail over this, this will happen again and again, until our Republic is but a shell of its former self. People have to be prosecuted over this! No talk, we need action!"), ('dates', '2020-05-08 12:32:17'), ('file_name', 'Jeff Duncan.csv'), ('pct_true', 0.8276492357254028)])
OrderedDict([('tweet', "I'm convinced that these nefarious actors decided that that even if they were unsuccessful in fabricating evidence against the President, that the negative coverage created by the media and the distraction it caused to his presidency made it all worth the effort."), ('dates', '2020-05-08 12:31:09'), ('file_name', 'Jeff Duncan.csv'), ('pct_true', 0.6101078391075134)])
OrderedDict([('tweet', 'I wrote the AG with concern over media reports of DOJ "suspending" a variety of constitutional protections. Not only has that not come to pass on the federal level, but AG is firing a warning shot against Mayors &amp; Governors who have taken certain restrictions too far. Thank you! https://t.co/c1GpPoLG4x'), ('dates', '2020-05-07 17:17:29'), ('file_name', 'Jeff Duncan.csv'), ('pct_true', 0.7506713271141052)])
OrderedDict([('tweet', 'Why did it take so long to get more #PPP funding? The answer is simple, &amp; it\'s not a partisan answer, it is simply the truth. Democrats decided to use #PPP as "leverage” for their agenda. We\'re talking about people\'s livelihoods here! This was not the time to play political games https://t.co/J6LVE99vek'), ('dates', '2020-04-23 18:34:11'), ('file_name', 'Jeff Duncan.csv'), ('pct_true', 0.7172120213508606)])
OrderedDict([('tweet', 'Joined @SenMcSallyAZ &amp; @RepMattGaetz in this important effort. \n\n“We’re sure you agree that taxpayers’ money should not be sent to a dangerous Chinese state-run bio-agent lab that lacks any meaningful oversight from US authorities &amp; is run by adversaries w/ history of lab leaks” https://t.co/GjEVNStVOX'), ('dates', '2020-04-22 15:53:41'), ('file_name', 'Jeff Duncan.csv'), ('pct_true', 0.517465353012085)])
OrderedDict([('tweet', 'Unacceptable - Democrats continue to block funding for #PPP which is a lifeline for #SmallBiz. https://t.co/21EOjNU5Cd'), ('dates', '2020-04-16 15:01:02'), ('file_name', 'Jeff Duncan.csv'), ('pct_true', 0.6780679225921631)])
OrderedDict([('tweet', '.@POTUS did the right thing by halting funds to WHO! They played a crucial role in covering up Chinese Communist Party’s botched #COVID19 response &amp; aided in downplaying severity. If anyone believes the “info” out of Chinese gov, they are ignoring the long history of propaganda. https://t.co/hG8bCN5K3h'), ('dates', '2020-04-15 15:31:02'), ('file_name', 'Jeff Duncan.csv'), ('pct_true', 0.88181471824646)])
OrderedDict([('tweet', 'Chuck Schumer and Nancy Pelosi need to lay off the identity politics with these aid packages and support bipartisan policies that will help ALL Americans, not select groups. We are all in this together! Rising tides lift all boats!'), ('dates', '2020-04-10 16:37:50'), ('file_name', 'Jeff Duncan.csv'), ('pct_true', 0.5818004608154297)])
OrderedDict([('tweet', 'We know the coronavirus came from Wuhan &amp; we know that since it emerged, the Chinese gov has been anything but truthful about it. Thanks to @RepJimBanks for leading the charge w/ this res to condemn these actions – it’s time for accountability &amp; the TRUTH.\nhttps://t.co/YYDCENWD6y'), ('dates', '2020-04-09 19:08:53'), ('file_name', 'Jeff Duncan.csv'), ('pct_true', 0.5314187407493591)])
OrderedDict([('tweet', 'Communities across #Nebraska have been hurt by #coronavirus -related tax revenue loss. I’m sure that no one wants to see police officers and firefighters laid off. The FLEX Bill rightly opens previously approved federal relief funds to help. @Leirion4Lincoln @Jean_Stothert'), ('dates', '2020-05-01 19:40:59'), ('file_name', 'Jeff Fortenberry.csv'), ('pct_true', 0.5631741285324097)])
OrderedDict([('tweet', 'This is an unprecedented crisis, requiring an unprecedented response. With this battle plan from our government—and the sacrifice and courage of the American people—we will fight back, we will help our sick, and we will restore our great nation. I support the bill. #coronavirus'), ('dates', '2020-03-27 17:18:21'), ('file_name', 'Jeff Fortenberry.csv'), ('pct_true', 0.5447216033935547)])
OrderedDict([('tweet', 'In a divided Congress, you should know things like this still happen. Had a very strong bipartisan discussion last night about nuclear arms, non-proliferation, and a new response to #Russia'), ('dates', '2020-01-28 22:59:20'), ('file_name', 'Jeff Fortenberry.csv'), ('pct_true', 0.53876793384552)])
OrderedDict([('tweet', 'I don’t want a war with Iran. The President has stated the same. Harsh reality compels us to continue to clench a fist––while holding out a hand––to find the space for authentic relations, even if it starts with small steps. https://t.co/6U5u6E95id #Iran'), ('dates', '2020-01-11 03:20:50'), ('file_name', 'Jeff Fortenberry.csv'), ('pct_true', 0.6164407134056091)])
OrderedDict([('tweet', 'The President was clear: Iran cannot threaten us and the community of responsible nations. However, he importantly pointed to the possibility of mutually improved relations between our countries. The House will meet with the Administration within the hour on these questions #Iran'), ('dates', '2020-01-08 16:56:05'), ('file_name', 'Jeff Fortenberry.csv'), ('pct_true', 0.678046703338623)])
OrderedDict([('tweet', "Finally! A House bill I supported to help stop the pernicious epidemic of robocalls just passed the Senate and is on its way to the President's desk for signature. If there is any issue that can unite America, this is it. #Robocalls"), ('dates', '2019-12-19 20:22:39'), ('file_name', 'Jeff Fortenberry.csv'), ('pct_true', 0.6401832699775696)])
OrderedDict([('tweet', 'We have a crisis in America––the high cost of prescription drugs. The Democrat bill will not solve it. The Republican bill has no chance of passage. One area of huge price growth is insulin. My one-line bill solves the problem: Manufacturers Can Sell Insulin Direct to Patients.'), ('dates', '2019-12-10 22:31:15'), ('file_name', 'Jeff Fortenberry.csv'), ('pct_true', 0.7242612838745117)])
OrderedDict([('tweet', "I have agreed to cosponsor Protecting America's Food and Agriculture Act.  The bill authorizes U.S. Customs and Border Protection to hire more ag inspectors in order to help prevent an outbreak of African swine fever in the U.S."), ('dates', '2019-11-19 19:53:50'), ('file_name', 'Jeff Fortenberry.csv'), ('pct_true', 0.5779041647911072)])
OrderedDict([('tweet', 'Americans are living through a national economic collapse the likes of which we haven’t seen since the Great Depression. Congress needs to stop posturing and pass a #NoShutOffs utility moratorium NOW. https://t.co/hMKMY9kdQ5'), ('dates', '2020-05-08 20:46:43'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.6666968464851379)])
OrderedDict([('tweet', 'How do you tell people to #StayHomeStaySafe and then cut off their water? Their electricity? Their gas? A lot of people are hurting right now and Congress has the power to help them. The House and Senate need to pass #NoShutOffs utility moratorium legislation NOW. https://t.co/hMKMY9kdQ5'), ('dates', '2020-05-07 21:00:45'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.6633222699165344)])
OrderedDict([('tweet', 'Once again we see that Trump is a puppet of the oil and gas industry, pushing the Fed into changing key rules at their request. We should use this moment to pivot to clean, renewable energy, not do political favors for the fossil fuel industry. https://t.co/pC8JT6g14m'), ('dates', '2020-05-01 21:06:21'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.6916208863258362)])
OrderedDict([('tweet', 'This is what @realdonaldtrump doesn’t understand: We can’t just decide not to be a part of the world. By relinquishing our global leadership role, Trump is creating a massive power vacuum, and handing power and influence directly to China.\nhttps://t.co/YthMQPsvIv'), ('dates', '2020-04-29 19:55:56'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.8582500219345093)])
OrderedDict([('tweet', 'Expelling a 16-year-old girl who has suffered extreme abuse without providing her rights for asylum proceedings is something we could never have expected from any U.S. gov’t. This deportation needs to stop. She must be afforded full rights &amp; gracious treatment worthy of the USA. https://t.co/zknKTmB6nt'), ('dates', '2020-04-24 04:34:52'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.5653519630432129)])
OrderedDict([('tweet', "This is chilling. Leaders retaliate against scientists for telling the truth in authoritarian regimes, not in democracies. They are risking disrupting vaccine development to protect the President's ego. https://t.co/SjH2iQsEp1"), ('dates', '2020-04-23 00:51:14'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.6798388361930847)])
OrderedDict([('tweet', 'We all know the playbook by now. Whenever Trump wants to change the subject, he says something racist/draconian about immigration. He has failed the response to coronavirus &amp; has no plan for testing/tracing. Americans know it, his poll numbers are falling—so voila, same old song. https://t.co/epPCAmuFJp'), ('dates', '2020-04-21 15:25:18'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.8940491080284119)])
OrderedDict([('tweet', 'Donald Trump just funneled HALF A BILLION DOLLARS OF TAXPAYER MONEY to political allies during an election year to pursue his No. 1 campaign pet project, but still doesn’t have a plan for nationwide coronavirus testing so we can end this crisis.\nhttps://t.co/xw0INdLrBj'), ('dates', '2020-04-20 14:25:01'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.6240248084068298)])
OrderedDict([('tweet', 'This is a huge problem for getting through the pandemic: The Admin doesn’t have their act together for testing and contact tracing, which is essential in tackling the public health challenge and reopening the economy. Big, big problem. Leadership is MIA. https://t.co/eQRvLejGaP'), ('dates', '2020-04-17 23:10:34'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.598862886428833)])
OrderedDict([('tweet', 'After thousands of Americans have died, and millions more lost their jobs, Trump risks delaying direct payments to families just so he can put his name on them. These checks aren’t a Trump Hotel—they’re vital assistance that needs to get to Americans ASAP.\nhttps://t.co/nB5oMPz3BR'), ('dates', '2020-04-16 01:40:28'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.8133575916290283)])
OrderedDict([('tweet', 'Determining whether or not the assistance funded by Congress is "too generous" isn\'t the Secretary’s job. Rs &amp; Ds came together to pass this unprecedented package to save American workers, businesses, &amp; families, &amp; now it\'s his job to get that $$$ out as fast as humanly possible. https://t.co/B2L0806NrV'), ('dates', '2020-04-11 01:08:47'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.5783404111862183)])
OrderedDict([('tweet', 'THREAD: American lives &amp; livelihoods depend on this administration getting it right w/ the $2 trillion emergency aid funds Congress has made available. What we’ve seen so far is deeply concerning. (1/)'), ('dates', '2020-04-07 23:54:47'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.6118841171264648)])
OrderedDict([('tweet', 'Wisconsin’s GOP has blocked the ability of Wisconsin citizens to vote without risking their lives.  \n\nConclusion: they don’t value the fundamental right of their citizens to vote; and they choose partisan gain over their citizens’ health. \nhttps://t.co/ICXVkRV9hc'), ('dates', '2020-04-07 17:56:18'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.632477343082428)])
OrderedDict([('tweet', 'The 700,000 Americans living in DC don’t have a voice in the Senate to fight for them &amp; were short-changed more than $700 million in the emergency aid bill. No community should be left behind, which is why I’m joining this fight on behalf of DC residents: https://t.co/hgIHUbWQ38 https://t.co/ubqjGnaZI7'), ('dates', '2020-04-03 19:12:18'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.5500178933143616)])
OrderedDict([('tweet', "Millions of Americans' lives have been thrown into chaos through no fault of their own. We need to throw lifelines out to struggling families, and we need to get those lifelines out now.\nhttps://t.co/qdGyJvhXur"), ('dates', '2020-04-02 16:24:39'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.9026987552642822)])
OrderedDict([('tweet', 'Spoke w/ @SEIU @OregonGovBrown @RepBonamici &amp; others about unconscionable reality @realDonaldTrump has put frontline workers in. Instead of leaving them to risk their lives w/ makeshift half-measures, he MUST use the Defense Production Act to meet the urgent need for PPE. https://t.co/ZLvsKV5dcU'), ('dates', '2020-03-30 21:03:51'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.7467645406723022)])
OrderedDict([('tweet', "This is OUTRAGEOUS. We're trying to pass aid for millions of Americans disastrously affected by this crisis, &amp; @SenateGOP is playing politics w/ Medicaid by excluding disability providers, community-based nursing homes, mental health providers, and more?!\nhttps://t.co/QH2sgreUGz"), ('dates', '2020-03-23 19:08:19'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.8990116715431213)])
OrderedDict([('tweet', 'Instead of bringing a bipartisan bill to the floor, @SenateGOP proposed a bill that prioritizes taking care of politically-connected special interests instead of the people who need help the most. (1/)'), ('dates', '2020-03-22 23:34:44'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.6219711303710938)])
OrderedDict([('tweet', 'Very concerned about state of Corona 3 bill. We can’t shortchange hospitals and health care workers, cities and states, and small businesses in favor of a big slush fund for huge corporations. Need to take care of workers and families.'), ('dates', '2020-03-22 18:37:58'), ('file_name', 'Jeff Merkley.csv'), ('pct_true', 0.9252221584320068)])
OrderedDict([('tweet', 'Our government should never stop anyone from practicing their faith or exercising their rights to operate a business if it can be done in a safe manner. https://t.co/E1NyF4sWmP'), ('dates', '2020-05-05 22:08:17'), ('file_name', 'Jefferson Van Drew.csv'), ('pct_true', 0.5984331965446472)])
OrderedDict([('tweet', 'We have relied on China for far too long to provide cheap labor and goods at the expense of America and our workers. We must bring that supply-chain home and end the export gap we have; It’s time to hold China accountable for their actions.\nhttps://t.co/bQKBY4j0kD'), ('dates', '2020-04-24 21:34:20'), ('file_name', 'Jefferson Van Drew.csv'), ('pct_true', 0.9361801147460938)])
OrderedDict([('tweet', 'I strongly disagree with the president’s decision to withdraw U.S. troops from Syria. President Erdogan of Turkey wasted no time on his offensive in the region, and we must work in a bipartisan manner to immediately stop Turkey from killing our allies. https://t.co/ifbpfAFrul'), ('dates', '2019-10-15 20:18:01'), ('file_name', 'Jefferson Van Drew.csv'), ('pct_true', 0.8781341314315796)])
OrderedDict([('tweet', 'https://t.co/0a1A09wiQ1\n\nI strongly disagree with the President’s decision to withdrawal U.S. troops from northeastern Syria. Our Kurd allies have been instrumental in our fight against ISIS. Our allies must know America always fulfills its responsibilities.'), ('dates', '2019-10-08 21:00:18'), ('file_name', 'Jefferson Van Drew.csv'), ('pct_true', 0.8422600626945496)])
OrderedDict([('tweet', 'Unnecessarily postponing the disbursement of stimulus checks to feed the President’s ego shows a callous disregard for the dire circumstances Americans find themselves in. Families are depending on these checks—they need to get in their hands without delay.https://t.co/g5kZ78QRDv'), ('dates', '2020-04-15 15:55:45'), ('file_name', 'Jennifer Wexton.csv'), ('pct_true', 0.7087368369102478)])
OrderedDict([('tweet', 'This is outrageous! \n\nFlynn PLEADED GUILTY to lying to investigators.  The evidence against him is overwhelming.  Now, a politicized DOJ is dropping the case.\n\nThe decision to overrule the special counsel is without precedent and warrants an immediate explanation.\n\n1/2 https://t.co/MxRyWkCMPw'), ('dates', '2020-05-07 19:06:32'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.6422179937362671)])
OrderedDict([('tweet', 'Listen to SCOTUS arguments re Trump rule that lets your boss decide your birth control coverage. BC shouldn’t be about where you work; it’s about freedom to make your own choices. That’s why I led House Dems in an amicus brief to tell Trump #HandsOffMyBC.\nhttps://t.co/UW2YsdqRDn'), ('dates', '2020-05-06 15:20:26'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.5344609022140503)])
OrderedDict([('tweet', 'More than a year ago, @HouseJudiciary issued a subpoena for the grand jury information redacted from the Mueller Report. Two courts have already agreed that DOJ has no legal basis to withhold this material, and today the Admin’s attempt to delay and keep this information... 1/2 https://t.co/8qVfZshfmu'), ('dates', '2020-05-01 20:50:01'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.6808769106864929)])
OrderedDict([('tweet', "I don’t often find myself in agreement with \u2066@GeorgeWill\u2069, but this is absolutely right: the President cannot ignore the Founders' system of checks and balances, and Congress must have power to enforce its subpoenas in court.  https://t.co/ZWk9vhuxsI"), ('dates', '2020-04-23 20:02:27'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.5780459642410278)])
OrderedDict([('tweet', 'Vulnerable small businesses across our nation have been unable to receive federal assistance because the SBA enabled large financial institutions to prioritize their wealthiest customers. This is unacceptable, and Democrats are working to put an end to it.\nhttps://t.co/YTBNCqEblL'), ('dates', '2020-04-23 19:34:59'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.6162340641021729)])
OrderedDict([('tweet', 'The last thing cities + states need are lawsuits from the AG. DOJ cannot sue local/state gov’ts with frivolous Commerce Clause claims. AG Barr should focus on voting rights and stopping hate crimes, not intimidating local officials who are saving lives.\n\nhttps://t.co/EfGvxE0ZYU'), ('dates', '2020-04-22 19:50:47'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.6391679048538208)])
OrderedDict([('tweet', 'Absolutely infuriating. Blood Centers blame @AABB, @AABB blames the @US_FDA, @US_FDA blames Blood Centers - all while people die because we don’t have enough convalescent plasma to treat those in need. The Trump admin’s incompetence is killing people. https://t.co/RBa2pwgD94'), ('dates', '2020-04-18 22:08:35'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.5227964520454407)])
OrderedDict([('tweet', '"Unlike any President in modern history, President Trump has engaged in offensive and unjustified attacks against Inspectors General, criticizing them for following the law, and retaliating against them for telling the truth”\n\nhttps://t.co/PdCdAjcYHP'), ('dates', '2020-04-13 17:43:41'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.5002425909042358)])
OrderedDict([('tweet', 'The Trump Admin suspended asylum processing without providing any legal justification. While I know that certain restrictions are necessary to protect public health, those measures must be legal. If the President’s actions are legal, he must be able to explain why to Congress. https://t.co/KY4wCRQl7D'), ('dates', '2020-04-10 23:15:27'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.6032071709632874)])
OrderedDict([('tweet', "Samaritan Purse has a long history of anti-LGBTQ discrimination. While we desperately need medical assistance, we can never abandon our values. Hate has no place in NY. That's why I wrote a letter seeking answers about how they're running their Central Park field hospital. https://t.co/c5ZQ2k08ID"), ('dates', '2020-04-10 20:41:45'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.5530188679695129)])
OrderedDict([('tweet', "The Trump Administration is in court trying to strip women's access to birth control through their health care. That's wrong—I led over 180 Members on a brief to the Supreme Court calling for that access to be protected.\n\nRead our amicus curiae brief here:\nhttps://t.co/SsLWlHVLou https://t.co/RlqSf2dIBH"), ('dates', '2020-04-09 22:17:49'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.5318865180015564)])
OrderedDict([('tweet', "While the COVID-19 crisis rages, the President fired the IG in charge of overseeing the federal government's response. This should outrage everyone. The President cannot manipulate a national emergency to settle personal, political scores. Americans will not stand for it. https://t.co/WvUxhlnhDD"), ('dates', '2020-04-07 23:29:49'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.8070651888847351)])
OrderedDict([('tweet', 'The House coronavirus bill makes sure airlines that take taxpayer money can’t then turn around and file for bankruptcy, like they did after 9-11.  The @HouseDemocrats bill is #ForThePeople. The Republican Senate bill is not.'), ('dates', '2020-03-24 19:21:10'), ('file_name', 'Jerrold Nadler.csv'), ('pct_true', 0.7338682413101196)])
OrderedDict([('tweet', 'I couldn’t agree more. California’s first responders are on the frontlines of this pandemic. @FCC’s refusal to extend the #NetNeutrality comment deadline despite current circumstances is a slap in the face to the people putting their lives on the line to serve our communities. https://t.co/7fvkkLu8Kz'), ('dates', '2020-04-20 20:52:57'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.5045185685157776)])
OrderedDict([('tweet', 'This is NOT a Chinese virus – it’s a disease that affects the global community. This kind of language only serves to stoke fear and promote a racist and blatantly false narrative. https://t.co/kLpe6TQAri'), ('dates', '2020-03-10 22:13:40'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.6636227369308472)])
OrderedDict([('tweet', 'More backward logic from the Trump Administration: While our nation faces a looming climate crisis, the #TrumpBudget proposes to slash EPA funding by 26.5%, cut DOE Energy Efficiency and Renewable Energy funding by 74%, &amp; eliminate the Advanced Research Projects Agency. https://t.co/V3wDq4zKTW'), ('dates', '2020-02-11 18:16:28'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.5445902347564697)])
OrderedDict([('tweet', 'While corporate profits are soaring, most Americans are working too hard for too little. The #PROAct strengthens workers’ power to organize and negotiate for a fair share of the profits they produce. https://t.co/msuFkMX1cp'), ('dates', '2020-02-06 22:47:37'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.7539286017417908)])
OrderedDict([('tweet', '#CitizensUnited opened a floodgate of corporate spending in politics and caused a wave of dark money to wash over our elections, drowning out the voices of the American people.'), ('dates', '2020-02-06 18:19:24'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.7605757713317871)])
OrderedDict([('tweet', 'The Trump Administration is trying to throw people off the #SocialSecurity rolls with their proposed rule for disability insurance. Americans EARN these benefits! That’s why I joined 100+ of my colleagues calling on them to halt this disastrous proposal! #NoSocialSecurityCuts'), ('dates', '2020-01-30 21:08:46'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.8171173334121704)])
OrderedDict([('tweet', "The president's lawyers have argued that his office affords him unprecedented power and by extension, anyone running for office cannot be held accountable for breaking the law if it’s in an effort to get elected."), ('dates', '2020-01-30 18:09:39'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.6332951784133911)])
OrderedDict([('tweet', 'The balance of power in our nation is derived from our Constitution. Anyone who votes to leave this president in office will be an accomplice to the collapse of our country’s very foundation. https://t.co/M54wdXRJRR'), ('dates', '2020-01-29 20:37:12'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.5414935350418091)])
OrderedDict([('tweet', 'This sham trial proves that the GOP is officially the party of Trump. Both House and Senate Republicans have shown that they are governed by the fear of crossing their leader, rather than the oath they swore to uphold. This is a sad statement about the future of our democracy.'), ('dates', '2020-01-22 18:44:11'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.8867971301078796)])
OrderedDict([('tweet', "If the president's actions bare no consequence, future presidents may act without constraint and American democracy will be at an end. Therefore, I will uphold my sworn duty to defend the Constitution and vote to impeach this president. https://t.co/MHTVgPk2UT"), ('dates', '2019-12-16 23:10:47'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.7004498243331909)])
OrderedDict([('tweet', 'There is nothing more sacred in our democracy than the right to vote. That’s why @HouseDemocrats passed #HR4 to put an end to egregious voter suppression tactics and #RestoreTheVote #ForThePeople.'), ('dates', '2019-12-06 19:07:07'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.6637916564941406)])
OrderedDict([('tweet', "Our children shouldn't have to go to school in fear. The House has passed multiple commonsense gun safety bills, and it's past time for the Senate to act. #EnoughIsEnough\nhttps://t.co/S0XKlK1Di0"), ('dates', '2019-11-14 17:42:24'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.755283534526825)])
OrderedDict([('tweet', "Over 4 million Californians have diabetes &amp; struggle with medical costs that are approximately 2.3x higher than those without. Last year, @CDCgov provided more than $6 million for prevention &amp; education programs, &amp; I'll continue to support this critical funding. #CountMeInADA"), ('dates', '2019-11-12 23:11:37'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.7743811011314392)])
OrderedDict([('tweet', "This administration wants to allow widespread discrimination in our health care system, jeopardizing many Americans' care – including women &amp; the LGBTQ community. We can’t allow this. That’s why I’m proud to cosponsor the #PutPatientsFirst Act to stop this harmful rule. https://t.co/mWCTsoMsnN"), ('dates', '2019-11-12 23:00:59'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.7293127179145813)])
OrderedDict([('tweet', "President Trump is intentionally and willfully condemning future generations to a world of catastrophic climate change and conflict. Our withdrawal from the Paris Climate Accord will abandon America's leadership at a time when the world needs it most. https://t.co/8QhlgIhtmA"), ('dates', '2019-11-05 18:58:05'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.632507860660553)])
OrderedDict([('tweet', 'With a former coal lobbyist at the helm, the EPA has been reduced to a shell of what it once was as President Trump continues to ram through a big-business agenda that will ultimately make our water less safe for use and consumption. https://t.co/7wZJjCaR6W'), ('dates', '2019-09-17 16:33:38'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.760490894317627)])
OrderedDict([('tweet', 'Last Congress, Republicans opened 19 million acres of federally protected, untouched Alaska wilderness to big oil. Opening the Arctic Refuge would destroy one of the last unspoiled places on Earth. This is no place for drilling &amp; today I was proud to vote to #ProtectTheArctic. https://t.co/1yfr3eUGTw'), ('dates', '2019-09-12 17:19:32'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.5844331979751587)])
OrderedDict([('tweet', 'Thousands of #Dreamers in my district &amp; across the country need permanent protection from deportation. While the Trump administration threatens their legal status, forcing them to live in fear, @TheDemocrats are working to #ProtectTheDream for all #DACA, #TPS &amp; #DED recipients. https://t.co/HyokyMm3Sr'), ('dates', '2019-09-05 16:21:40'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.7043386697769165)])
OrderedDict([('tweet', 'While the President is playing political games and backpedaling on his commitments, gun violence continues to devastate communities across our nation. It’s been 175 days since the House passed the bipartisan #HR8 and commonsense #HR1112. The Senate must #DoSomething.'), ('dates', '2019-08-20 17:39:12'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.8003562092781067)])
OrderedDict([('tweet', "For 7 years, #DACA has successfully protected 800K young people from deportation. These #Dreamers are our neighbors, family, &amp; friends. But the Trump administration is threatening their future in the U.S. That's why @HouseDemocrats are continuing to fight to #ProtectTheDream."), ('dates', '2019-08-15 18:28:56'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.6695549488067627)])
OrderedDict([('tweet', "Americans are sick &amp; tired of the demands &amp; desires of special interest coming before their needs. We need a constitutional amendment to end the culture of corruption that is fueled by our #campaignfinance system. That's why my proposal would eliminate PACs &amp; ban #darkmoney. https://t.co/m37gruIiK4"), ('dates', '2019-08-02 19:15:21'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.7715060710906982)])
OrderedDict([('tweet', 'Over a million Americans are at risk of losing their pensions if the House fails to pass the #ButchLewisAct - putting their families’ financial security and future at risk. No one should live in fear that their hard-earned retirement savings will collapse.'), ('dates', '2019-07-24 21:18:25'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.9496515989303589)])
OrderedDict([('tweet', "Today, the House votes to #PassthePASTAct, which would end the inhumane practice of soring #horses by intentionally inflicting pain to their feet in order to win prizes. I'm proud to cosponsor H.R. 693 &amp; call on the Senate to swiftly pass this bill &amp; end this senseless cruelty."), ('dates', '2019-07-24 18:42:17'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.8122356534004211)])
OrderedDict([('tweet', "Science is under attack. This administration has prohibited federal scientists from speaking about climate change, &amp; climate data has been scrubbed from many gov't websites. We must protect their work to safeguard public health &amp; our environment. #ScienceNotSilence #ForThePeople https://t.co/Mm1rAXRwfk"), ('dates', '2019-07-17 19:59:29'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.8307480216026306)])
OrderedDict([('tweet', 'This president has shown himself over and over to be a racist. These attacks have only unified Democrats and reminded the American people why they decided to change the makeup of Congress in 2018.'), ('dates', '2019-07-16 02:58:22'), ('file_name', 'Jerry McNerney.csv'), ('pct_true', 0.977401852607727)])
OrderedDict([('tweet', 'It is clear that Congress needs to act to provide consumers and companies with a clear federal standard that lays out robust protections for consumers’ personal data. \n\nToday, I introduced the Consumer Data Privacy and Security Act to create a new federal standard. #DataPrivacy https://t.co/BGAmqBo3ZI'), ('dates', '2020-03-12 15:51:27'), ('file_name', 'Jerry Moran.csv'), ('pct_true', 0.5169806480407715)])
OrderedDict([('tweet', "We must all act responsibly and put the health of the American public above profits. Airlines are deeply impacted by this crisis, but price-gouging customers is not the solution. I'm glad Frontier abandoned its plan to sell $39 social distancing upgrade. \n\nhttps://t.co/doZQXK8Ilg"), ('dates', '2020-05-07 16:10:07'), ('file_name', 'Jesús G. "Chuy" García.csv'), ('pct_true', 0.5666133761405945)])
OrderedDict([('tweet', 'The incompetence Trump has shown throughout this pandemic has cost thousands of lives. Dissolving the #COVIDTaskForce will not make the virus disappear.  We must hold this Administration accountable. https://t.co/cT1VkO51eV'), ('dates', '2020-05-05 23:55:35'), ('file_name', 'Jesús G. "Chuy" García.csv'), ('pct_true', 0.851208508014679)])
OrderedDict([('tweet', 'Voters aren’t just asking us to hold the Chinese Communist Party accountable, they’re demanding it. Listen to my interview with @BreitbartNews 👇 https://t.co/TqfSOLZqjO'), ('dates', '2020-05-09 11:29:03'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.8900033235549927)])
OrderedDict([('tweet', 'The China Task Force is going to be critical to Congress’s effort to combat malign Chinese Communist Party’s influence in the United States. Together, we will work with President Trump to hold China accountable and stop them from taking advantage of us!\nhttps://t.co/oObNP6mBCJ'), ('dates', '2020-05-08 18:53:58'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7995079755783081)])
OrderedDict([('tweet', 'We can’t let the Chinese Communist Party profit from a pandemic of their own making. Glad to join @MariaBartiromo this morning to discuss the new House GOP China Task Force and my bill stopping Chinese takeovers of critical American companies. https://t.co/mYuHh5xd7U'), ('dates', '2020-05-08 15:02:52'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.925758421421051)])
OrderedDict([('tweet', 'Very disappointed w/ former #CCP skeptic,@SpeakerPelosi for bailing on the House China Task Force. The China threat is too high-stakes. We MUST work together to confront it. @tonykatz\n\nhttps://t.co/a1pzDQCkK2'), ('dates', '2020-05-07 21:48:25'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7146908640861511)])
OrderedDict([('tweet', 'Rep. Jayapal explains the Democrats’ spaghetti on the wall approach in a @voxdotcom interview yesterday: “The leverage is that there is enormous suffering.”\n \nThis is looter logic. The city is burning – let’s take what we can get! https://t.co/pIbjTH1ZuK'), ('dates', '2020-05-07 19:16:26'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.8364194631576538)])
OrderedDict([('tweet', '#China’s taken our jobs, given us this virus and isn’t holding up their end of the trade deal. @realDonaldTrump will have every right to hold their feet to the fire in future negotiations. https://t.co/UdPrBDer6E'), ('dates', '2020-05-07 16:44:47'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.9133608341217041)])
OrderedDict([('tweet', 'This discovery is why #China needs to cooperate with an international investigation to determine just how long the novel coronavirus was spreading through Wuhan before Dr. Li Wenliang and others discovered it in late December.\n \nhttps://t.co/XJm9hWfvmv'), ('dates', '2020-05-06 19:27:29'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.8325985074043274)])
OrderedDict([('tweet', 'In US, gov’t derives its power from the people, not the other way around. Officials should pay careful attention to their constituents, and the consensus emerging is we cannot remain in this posture any longer. We must find ways to safely return to work.\n \nhttps://t.co/aCAFTDdT8m'), ('dates', '2020-05-06 15:31:16'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.6225161552429199)])
OrderedDict([('tweet', '.@RepAdamSchiff stop obstructing Congress and release the Russia probe transcripts! If you continue to block these documents from the public, that would be an abuse of power! https://t.co/oTTU7U9jUa'), ('dates', '2020-05-06 15:21:08'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.516403079032898)])
OrderedDict([('tweet', 'This is why so many Americans wanted @realDonaldTrump in the White House to begin with. He understands the #China threat and is making sure they don’t take advantage of us anymore.\n\nhttps://t.co/eHAZ9zJo1o'), ('dates', '2020-05-06 14:02:24'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.8140643835067749)])
OrderedDict([('tweet', 'Alarming #China is preparing for armed confrontation. Great time to deter Chinese aggression by investing in our building up our Navy. 1) it improves nat security 2) it gets Americans back to work &amp; will improve economy just like WWII manufacturing boom.\n \nhttps://t.co/HiTLR3guHH'), ('dates', '2020-05-04 15:26:52'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.6073670983314514)])
OrderedDict([('tweet', '#China hid the severity of COVID-19 and the true nature of this virus from us to stockpile medical supplies. We are now suffering from a virus that we could’ve helped them contain, maybe in Wuhan, if they had agreed to allow @CDCgov to help.\nhttps://t.co/WPm6sSblk3'), ('dates', '2020-05-04 11:55:38'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.929761528968811)])
OrderedDict([('tweet', ".@NIH's recent #CCP bio lab funding controversy got people's attention, rightfully. But it fits w/ pattern of NIH sending taxpayer funds to shady #CCP projects. I wrote @NIHDirector, urging him to take the threat of improper foreign influence seriously.\nhttps://t.co/ACQihwVo8a"), ('dates', '2020-04-30 18:47:24'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.5040194988250732)])
OrderedDict([('tweet', 'Voters aren’t just asking for us to hold #China accountable for giving us this virus, they’re demanding it. @LouDobbs @FoxBusiness https://t.co/IzrgfSzdII'), ('dates', '2020-04-30 15:50:02'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7884315252304077)])
OrderedDict([('tweet', 'Congressional Democrats already told us they’re taking advantage of a crisis to restructure things to fit their socialist vision.\n\nAlternatively, GOP is trying to provide temporary relief to preserve the strong Trump economy we had before the pandemic.\nhttps://t.co/fDGOK2IydS'), ('dates', '2020-04-30 15:32:55'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7072934508323669)])
OrderedDict([('tweet', 'Hoosier taxpayers should never be on the hook to bail out many years of mismanagement and poor leadership in states like Illinois.'), ('dates', '2020-04-30 15:10:43'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.5255548357963562)])
OrderedDict([('tweet', 'For years, bad actors like China or Russia find gaps in our supply chains and exploit them. As this pandemic has made clear, our supply chains are as vulnerable as ever. We must act now to secure them. Read my latest op-ed at @RCDefense: https://t.co/B89gncEci9'), ('dates', '2020-04-30 13:54:22'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.5263551473617554)])
OrderedDict([('tweet', 'I agree, Mr. President. American taxpayers shouldn’t have to pay for the damage this virus has caused, the Chinese Communist Party should pay. They could’ve stopped it. https://t.co/tQrQUsIovF'), ('dates', '2020-04-29 18:43:43'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.8981037735939026)])
OrderedDict([('tweet', 'We need a robust and open debate amongst the medical community now more than ever. Why are YouTube censors—who presumably know nothing about science—choosing which docs are “right?” Dystopian! https://t.co/0yCUhRPbuy'), ('dates', '2020-04-29 00:01:56'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.5089203119277954)])
OrderedDict([('tweet', "Chinese gov't officials took to Twitter again Monday and claimed the novel coronavirus began in the U.S.\n \nIt’s vitally important for Congress to push back on these lies and pass the Hold China Accountable resolution (H.Res. 907) I and 50+ MOC sponsored.\nhttps://t.co/5sw45oHmyZ"), ('dates', '2020-04-28 13:52:41'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.6818785667419434)])
OrderedDict([('tweet', "The EU just publically &amp; internally released a report on Russia &amp; #CCP COVID disinfo. But the public report is less critical of China than the internal version 😡\n\nSure seems that int'l orgs, like @WHO &amp; EU, are more prone to #CCP pressure than nations. 🤔\nhttps://t.co/mnFn1sXSdD"), ('dates', '2020-04-27 15:32:23'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.5661186575889587)])
OrderedDict([('tweet', ".@RepAdamSchiff hates transparency. He won’t allow release of transcripts because they show that the president did nothing wrong.\n \nNow, he wants to lead an investigation into the president’s handling of COVID-19. It's another witch hunt! https://t.co/HK3qJrUWsC"), ('dates', '2020-04-27 14:58:36'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.9310394525527954)])
OrderedDict([('tweet', 'This is gross and offensive. I co-lead a bipartisan task force on HASC that examines the #China threat. We should all be more concerned with the threat the Chinese regime poses to a) the U.S. b) Chinese citizens, especially #Uighurs c) #HongKong/#Tawian d) and rest of the world. https://t.co/2rr97gA3gE'), ('dates', '2020-04-27 14:17:06'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.528545618057251)])
OrderedDict([('tweet', '#China continues to spread disinformation that the novel coronavirus originated in the United States.\n \nThe U.S. Congress must set the record straight and pass H.Res 907, the Hold China Accountable resolution, intro’d by me and @MarshaBlackburn.\n \nhttps://t.co/8A5o2zkdSs'), ('dates', '2020-04-23 17:28:22'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.9033140540122986)])
OrderedDict([('tweet', "On April 9, Senate Democrats blocked additional PPP funding the WH requested &amp; @SpeakerPelosi cheered.\n \nLast Thursday, PPP funding ran out &amp; millions lost their jobs.\n\nWe're finally voting on it today... what did they gain for withholding aid for political purposes?"), ('dates', '2020-04-23 17:10:00'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.8535804748535156)])
OrderedDict([('tweet', 'Congress must offset debt impacts of #COVID19 relief legislation by passing a new, fiscally responsible budget that shrinks government to a more proper size. Good news! We already have two drafted budgets that we could pass right now! 👇'), ('dates', '2020-04-23 16:17:29'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7156152129173279)])
OrderedDict([('tweet', 'Disturbing to see anti-Semitic coronavirus conspiracy theories floating around. Congress must defend the truth about the origin of this virus. Let’s pass the Hold China Accountable resolution intro’d by me and @MarshaBlackburn and set the record straight.\nhttps://t.co/Ccgcwr15sE'), ('dates', '2020-04-22 13:51:04'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7691009044647217)])
OrderedDict([('tweet', 'Read more about what @RepGallagher &amp; I have been doing to reverse USA federal pension investments into dangerous #CCP &amp; #PLA companies helping China’s military that aren’t in best interest of OUR national security at the link👇. Insane that we would allow this at @tsp4gov!!! https://t.co/LVdw2O59sa'), ('dates', '2020-04-22 12:59:56'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7255217432975769)])
OrderedDict([('tweet', 'We have the opportunity to prevent billions of dollars in Americans’ retirement savings from going to Chinese companies affiliated with the PLA. This administration should take it. Why would we help an adversary build up their military?? https://t.co/We5gn1pZO2'), ('dates', '2020-04-22 11:42:42'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.5561345815658569)])
OrderedDict([('tweet', 'Some apps are fun, others are counterintelligence threats. Americans should know which is which before they hit the download button. \n \nThat’s why I’ve introduced a bill that would require warnings before consumers download apps like @tiktok_us or FaceApp. https://t.co/Y65F1CUssX'), ('dates', '2020-04-21 17:38:16'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.5888627767562866)])
OrderedDict([('tweet', '.@Facebook is removing posts organizing stay at home protests, arguing recent assemblies don’t comply with government orders.\n\nBut the document that gives our government its authority says otherwise.'), ('dates', '2020-04-20 22:18:05'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.748954176902771)])
OrderedDict([('tweet', 'The #CCP threatened they’d place export controls on lifesaving medical supplies and “plunge” US “into the mighty sea of the coronavirus.” Now, they’re following through.\n\nThis should infuriate everyone. We must fix our supply chains + not do biz w/ #China.\nhttps://t.co/FA39OJx0CZ'), ('dates', '2020-04-17 14:33:48'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.5098732709884644)])
OrderedDict([('tweet', 'Yet another reason Chinese Communist Party is unfit for global leadership. \nhttps://t.co/INLNzWR5ep'), ('dates', '2020-04-15 20:05:07'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.9521493315696716)])
OrderedDict([('tweet', 'Insane. China lied to us…said it didn’t spread person to person, said it was just like the flu, jailed doctors warning of the virus, encouraged travel for Lunar New Year, blocked CDC from studying the virus…@AP confirms China knew and did nothing. https://t.co/T1hegowfyA https://t.co/mWeMsCkicK'), ('dates', '2020-04-15 18:03:42'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.8722835779190063)])
OrderedDict([('tweet', 'Just agreed to join @Lancegooden’s Stop COVID Act which would allow DoJ to investigate China and file litigation if they’re found responsible for the release of the virus. More evidence released every day confirms they could have stopped it, but they didn’t. https://t.co/tFITBmn9Ji'), ('dates', '2020-04-15 17:34:23'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.6594085693359375)])
OrderedDict([('tweet', '.@CDCDirector, we pay @WHO 10X more than China. Yet @DrTedros smears Taiwan &amp; repeats #CCP disinfo. So often we pay the lion\'s share to int\'l orgs, then get shafted. Why?\n\nBc/ till Pres. Trump, we did nothing about it. Our leaders just rolled over &amp; said, "still a great partner." https://t.co/lfYBh3blC5'), ('dates', '2020-04-15 16:48:19'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.6849325299263)])
OrderedDict([('tweet', 'Any Phase 4 deal MUST include incentives for American businesses to move home from #China. They are NOT an ally. They ARE an adversarial gov’t that threatened to place export controls on lifesaving medical supplies and “plunge" the U.S. "into the mighty sea of the coronavirus.”'), ('dates', '2020-04-15 14:48:18'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.537372350692749)])
OrderedDict([('tweet', "It's clear: #China gave us this virus. The Chinese Communist Party’s cover-up will cost us lives and livelihoods. They must be held accountable and be made to pay reparations. They, not Americans, should be shouldering the cost of this virus.\nhttps://t.co/T1hegoNQq8"), ('dates', '2020-04-15 14:07:55'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.97636878490448)])
OrderedDict([('tweet', 'China took our jobs and gave us this virus. It’s time to hold them accountable for their failure and #MakeChinaPay. One way we can do it is through the int’l courts...#China willfully violated health regs designed to prevent a pandemic. Glad to join @LouDobbs to discuss. https://t.co/b1pcns4To6'), ('dates', '2020-04-13 23:34:47'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.887144923210144)])
OrderedDict([('tweet', 'Surprised @CNN hasn\'t deleted this yet. It\'s an interesting snapshot. \n\nOne day after this tweet, @realDonaldTrump announces a travel ban from China... inexplicably panned as "racist." \n\nWhile the president worked to protect us from the virus, some twaddled. https://t.co/Rrt2LlOQ3I'), ('dates', '2020-04-13 18:00:52'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7425863146781921)])
OrderedDict([('tweet', '“Early on we did not get correct information [from #China], and the incorrect information was propagated right from the beginning,” Dr. Fauci said. \n\nThe #CCP told us it was just like the seasonal flu. We now know that’s a lie. https://t.co/1vtUWYxvzj'), ('dates', '2020-04-13 12:08:01'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.5618730187416077)])
OrderedDict([('tweet', '“China’s state-media political advertising on Facebook has drawn as many as 109 million views over the past 14 months... That reach is far larger than the est. 40 million impressions Russia’s IRA obtained in its disinfo op around 2016”\n\U0001f928\nhttps://t.co/PLVt8btIB4'), ('dates', '2020-04-10 20:01:11'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.9281706809997559)])
OrderedDict([('tweet', 'PSA: To all concerned about treatment of minorities during this pandemic, the Chinese Communist Party kept Uighur Muslims in concentration camps that became hotbeds for the disease and forced them to go work in shuttered factories. Shielding #CCP from criticism enables them!'), ('dates', '2020-04-10 15:47:43'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.9409599304199219)])
OrderedDict([('tweet', 'If the #CCP had been honest from day one, if they had collaborated with the CDC, if they had provided correct data, if they hadn’t censored their citizens—if they acted responsibly.... but they didn’t. And now Americans are suffering. \n \nhttps://t.co/yOUhoST4SQ'), ('dates', '2020-04-10 14:05:19'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7564003467559814)])
OrderedDict([('tweet', 'If China’s leaders hadn’t become embarrassed by the outbreak and tried to cover up its spread, the world may have had a better chance to prepare for this or even contain it in Wuhan or #China. #MakeChinaPay via @NRO https://t.co/iduPwkzsL3'), ('dates', '2020-04-10 01:27:59'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.854216992855072)])
OrderedDict([('tweet', 'Rather than succumb to #CCP propaganda, the int’l community must hold China accountable. If the @UN can’t even acknowledge the consequences of #CCP lies and repression, it’s completely lost its purpose. \n\n@StateDept \nhttps://t.co/zcucSzmyBv'), ('dates', '2020-04-09 18:11:39'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.9507973194122314)])
OrderedDict([('tweet', 'Dems in Congress are blocking #COVID relief ...AGAIN. It’s not a time for partisan games. Absolutely shameful. https://t.co/OrphTiaK2T'), ('dates', '2020-04-09 16:08:49'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.84029221534729)])
OrderedDict([('tweet', "Regulations demand #China provide timely and transparent health info as requested within 24 hours. China rejected requests from WHO &amp; CDC in late Jan and Feb to study the new virus without explanation. They must face consequences! If UN can't do this, what is it for?? https://t.co/C0QH5tD3f6"), ('dates', '2020-04-09 15:00:41'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.6335656642913818)])
OrderedDict([('tweet', '77% of Americans blame the #CCP for worsening the COVID19 pandemic. And we’re not the only country that’s angry... India is too! \n\nExpect to see more, similar articles, as the CCP’s global propaganda effort faulters. \n\nhttps://t.co/H168jlRt0Z'), ('dates', '2020-04-09 15:00:06'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7461273074150085)])
OrderedDict([('tweet', "#China violated the 2005 Int'l Health Regulations designed to prevent global pandemics. @SecPompeo should bring a case against them at the ICJ. If they fail to submit themselves, we can induce Beijing to make things right by blocking it from the @wto. https://t.co/Qt7OsHVKR2"), ('dates', '2020-04-09 14:18:02'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.8497868776321411)])
OrderedDict([('tweet', "It won't get the attention it deserves, but a Congressional investigation found the Obama admin misled Congress, sidestepped legal reqs &amp; blocked career DOD officials' involvement during the Taliban 5-Berghdal prisoner exchange. 😡\nhttps://t.co/L0vNzFP5Ov"), ('dates', '2020-04-08 19:53:54'), ('file_name', 'Jim Banks.csv'), ('pct_true', 0.7435393929481506)])
OrderedDict([('tweet', 'Michael Flynn lied to the FBI, admitted to it, and pleaded guilty. Trump himself fired Flynn for lying to VP Pence. The Trump administration sunk to a new low &amp; undermined our entire judicial system and the rule of law. This is a travesty of justice. https://t.co/hZtk1nBidG'), ('dates', '2020-05-08 17:05:08'), ('file_name', 'Jim Cooper.csv'), ('pct_true', 0.8865067958831787)])
OrderedDict([('tweet', 'I have opposed the Trump administration’s Title IX changes from the beginning. Rolling back the Obama-era Title IX protections for sexual assault victims is dangerous &amp; makes students less safe. Rolling back the protections during a pandemic defies logic. https://t.co/Du3xMSXh8B'), ('dates', '2020-05-06 20:34:00'), ('file_name', 'Jim Cooper.csv'), ('pct_true', 0.6295881271362305)])
OrderedDict([('tweet', 'The #FreedomRiders risked their lives for our freedom. They risked their futures for our future. They faced down violence and prison for our civil rights. There are no better role models, no better heroes, and we honor their journey today. https://t.co/xOhR5QSvvG'), ('dates', '2020-05-04 23:36:00'), ('file_name', 'Jim Cooper.csv'), ('pct_true', 0.5781352519989014)])
OrderedDict([('tweet', '.@SBAgov must ensure minority-owned businesses are not shut out of this program &amp; amend guidance on PPP to reaffirm lending institutions’ obligation to comply with fair lending laws. Congress needs to act and vote on another wave of funding for PPP. https://t.co/tgjTS9aQ5K'), ('dates', '2020-04-17 18:37:35'), ('file_name', 'Jim Cooper.csv'), ('pct_true', 0.9027894735336304)])
OrderedDict([('tweet', 'This is everyone’s worst nightmare about government—it doesn’t work when you need it the most. Congress did its job; now the Administration should follow. https://t.co/Avf6RfEQET'), ('dates', '2020-04-03 21:15:43'), ('file_name', 'Jim Cooper.csv'), ('pct_true', 0.5989893674850464)])
OrderedDict([('tweet', 'The EPA’s reckless decision to roll back enforcement actions during the #COVID19 pandemic puts all Americans’ health at risk &amp; creates a dangerous precedent for future national emergencies.'), ('dates', '2020-03-31 21:09:08'), ('file_name', 'Jim Cooper.csv'), ('pct_true', 0.8701237440109253)])
OrderedDict([('tweet', 'Coronavirus is an unprecedented pandemic, and this is an unprecedented bill. It is the largest economic relief bill passed in America’s history.'), ('dates', '2020-03-27 22:37:42'), ('file_name', 'Jim Cooper.csv'), ('pct_true', 0.6638488173484802)])
OrderedDict([('tweet', "On this Int'l #HolocaustRemembranceDay, 75 years after the liberation of Auschwitz-Birkenau, where nearly 1 million Jews were killed, we passed #HR943 with strong bipartisan support."), ('dates', '2020-01-28 01:10:36'), ('file_name', 'Jim Costa.csv'), ('pct_true', 0.6635465025901794)])
OrderedDict([('tweet', 'At every turn, the American people have picked up the tab for the President’s trade war, which has strained hardworking American families and farmers. https://t.co/eWFjGGGoK5'), ('dates', '2020-01-15 19:04:41'), ('file_name', 'Jim Costa.csv'), ('pct_true', 0.5871515274047852)])
OrderedDict([('tweet', 'Millions of Americans live with a pre-existing condition, including 16 million in California alone! Thanks to ACA, they’re protected from discrimination – but that will go away if Republicans win their anti-health care lawsuit. #ProtectOurCare https://t.co/e5VHcCv92g'), ('dates', '2020-01-13 21:32:28'), ('file_name', 'Jim Costa.csv'), ('pct_true', 0.6349368691444397)])
OrderedDict([('tweet', 'It’s time for @SpeakerPelosi and @SenSchumer to stop playing politics with people’s livelihoods and our economy. We need to increase funding for the #PaycheckProtectionProgram so we can keep Americans employed and our businesses afloat as we continue fighting #COVID19. https://t.co/fy8RA4KaZ0'), ('dates', '2020-04-16 17:54:35'), ('file_name', 'Jim Hagedorn.csv'), ('pct_true', 0.6422809362411499)])
OrderedDict([('tweet', 'A pattern? \n\nObama’s IRS targeted conservatives before the 2012 election. \n\nObama’s FBI targeted President Trump and his allies before the 2016 election. \n\nScary!'), ('dates', '2020-05-09 17:00:41'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.694503128528595)])
OrderedDict([('tweet', 'CLAPPER in 2017: “I never saw any direct empirical evidence that the Trump campaign or someone in it was plotting/conspiring with the Russians to meddle with the election.”\n\nBut Adam Schiff and the Democrats put us through three years of the Mueller probe anyways? \n\nRidiculous. https://t.co/1JeKF4U48a'), ('dates', '2020-05-08 00:20:55'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.5619581341743469)])
OrderedDict([('tweet', 'Here we go again. \n\nImpeachment round 2. \n\nDemocrats never miss a chance to attack the Trump Administration. https://t.co/o8E1giWZdK'), ('dates', '2020-05-07 15:06:14'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6709838509559631)])
OrderedDict([('tweet', 'January 31: President Trump restricts travel from China, saving countless lives. \n\nJanuary 31: Adam Schiff is busy arguing to the Senate that they should remove President Trump from office.\n\nPresident Trump acted early to combat #coronavirus. \n\nWhy didn’t Adam Schiff? https://t.co/zSYPzfKiwg'), ('dates', '2020-04-27 18:04:41'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6013590693473816)])
OrderedDict([('tweet', "Why won't Adam Schiff release declassified interviews from his Russian collusion witch hunt? \n\nBecause he knows the President did nothing wrong. \nhttps://t.co/GTBZg7GSke"), ('dates', '2020-04-27 14:08:09'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.7466425895690918)])
OrderedDict([('tweet', "President Trump is draining the Swamp. But the Swamp is fighting back.\n\n• 2016: FBI spies on campaign\n• 2017-2019: Mueller investigation\n• 2019-2020: Schiff's sham impeachment\n\nYesterday, House Dems establish committee to attack President on #coronavirus.\n\nWhen will it stop? https://t.co/I0Ucvh0VTn"), ('dates', '2020-04-24 15:48:44'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6299804449081421)])
OrderedDict([('tweet', "The federal government already has 8 different entities conducting oversight of the #coronavirus. But Democrats want a 9th. \n\nWhy? \n\nBecause the others aren't partisan and political. https://t.co/lzfyHvvhAy"), ('dates', '2020-04-23 16:24:20'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.7599781155586243)])
OrderedDict([('tweet', 'Why is Harvard getting a $8.6 million #coronavirus stimulus when they have a $40 billion endowment?\n\nThe President is right to demand the money back!\n\nIn fact, every penny that isn’t going to students or businesses forced to shut down should be returned. \n\nhttps://t.co/g9enlRNsHz'), ('dates', '2020-04-22 16:05:19'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.7304189801216125)])
OrderedDict([('tweet', 'House Democrats are now considering #coronavirus legislation that PAYS states to let criminals out of prison. \n\nBut those same Democrats also argue that gun stores and your #2ndAmendment rights are “non-essential” to protect yourself. \n\nYou can’t make this up! https://t.co/GaW18YK5T7'), ('dates', '2020-04-16 20:54:58'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.9007545113563538)])
OrderedDict([('tweet', 'There are #coronavirus task forces doing great work. \n\nBut there’s one task force that’s missing in action: the U.S. Congress! \n\nWhere’s Speaker Pelosi’s plan to get back to work?'), ('dates', '2020-04-15 18:39:09'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6055606007575989)])
OrderedDict([('tweet', 'Two things Democrats never miss:\n\n1. An opportunity to use a crisis to advance their left-wing agenda.\n\n2. Any chance to attack the President.\n\nBut what’s most scary about this #coronavirus crisis?\n\nThe threats to our fundamental liberties at times like these. https://t.co/OKtOSd5Okb'), ('dates', '2020-04-14 17:22:08'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6493119597434998)])
OrderedDict([('tweet', "Why won't Speaker Pelosi quit criticizing President Trump and tell us what her plan is to bring Congress back to work? \n\nIt's time for the Speaker to lead by example. https://t.co/Lu1lq0wOvu"), ('dates', '2020-04-14 14:29:57'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.8359841108322144)])
OrderedDict([('tweet', 'The FBI got played—the Steele Dossier was Russian disinformation. \n\nWe suspect that Comey, McCabe, Baker, Strzok, and Page knew all along. \n\nBut they didn’t care because they were out to get the President. \nhttps://t.co/VmrzX0sN14'), ('dates', '2020-04-11 17:38:14'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6938306093215942)])
OrderedDict([('tweet', 'If the #WHO wants to spread Chinese propaganda about #coronavirus, then the United States shouldn’t fund the WHO with our taxpayer dollars. \n\nPeriod. \nhttps://t.co/6xpHfWQrsx'), ('dates', '2020-04-10 21:33:13'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6588795781135559)])
OrderedDict([('tweet', 'House Democrats are already demanding that illegal immigrants receive benefits in the next #coronavirus aid package. \n\nSeriously?\n\nThat can’t happen while so many American families are struggling.'), ('dates', '2020-04-08 19:27:32'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.8427979350090027)])
OrderedDict([('tweet', 'Of course Adam Schiff is complaining about ICIG Atkinson being fired. He was Schiff’s key impeachment enabler.\n\n- Let the Whistleblower lie about meeting with Schiff’s staff\n- Didn’t disclose the fact that the WB worked for Biden\n\nSchiff still won’t release Atkinson’s transcript!'), ('dates', '2020-04-04 14:46:04'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.8786388635635376)])
OrderedDict([('tweet', 'Speaker Pelosi just announced a new committee to oversee the #coronavirus crisis. \n\nThat’s ridiculous! \n\nWe already have an Oversight Committee in Congress.'), ('dates', '2020-04-03 13:54:03'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.5521828532218933)])
OrderedDict([('tweet', 'Great townhall from President Trump. He and healthcare experts laid out the path forward and how we will defeat #COVIDー19 as a country.\n\nDemocrats should bring that same spirit to negotiating. But instead they can’t seem to see past the left-wing agenda.'), ('dates', '2020-03-24 19:30:35'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.5413424372673035)])
OrderedDict([('tweet', 'Adam Schiff was so desperate to attack the President that he leaked the phone records of a journalist, the President’s attorney, and a Republican member of Congress.\n\nNow an FCC Commissioner says this process was “secret” and “partisan.”\n\nhttps://t.co/n144Lwi0vI'), ('dates', '2020-03-13 16:40:03'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6018930673599243)])
OrderedDict([('tweet', "The FBI lied to the FISA Court 17 times to spy on Carter Page.\n\nThis FISA reform bill increases protections of Americans' civil liberties. https://t.co/Gzwl2lxOIc"), ('dates', '2020-03-11 20:38:43'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6179804801940918)])
OrderedDict([('tweet', 'The Comey FBI lied to the FISA Court 17 times.\n\nThey spied on Carter Page while he was exercising his right to affiliate with the political campaign of Donald Trump.\n\nWhen secret courts undermine the First Amendment, we gotta change the law. FISA reform now.'), ('dates', '2020-03-06 21:48:50'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6915868520736694)])
OrderedDict([('tweet', '"Jordan, @RepMarkMeadows say Schumer\'s attempts to \'intimidate\' the Supreme Court won\'t work"\n\n@FoxFriends https://t.co/uNTt1v7cpP'), ('dates', '2020-03-05 14:12:00'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6645240187644958)])
OrderedDict([('tweet', '2016: Comey’s partisan and political FBI lied to the FISA Court 17 times to spy on then-candidate Trump.\n\n2020: President Trump is working with us to fix FISA. Comey—still partisan and political.\n\nhttps://t.co/4xOdrC1SdT'), ('dates', '2020-03-04 17:16:38'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.8575916290283203)])
OrderedDict([('tweet', 'Dems and the media blame President Trump for the Coronavirus and Vice President Pence for HIV.\n\nJust another Thursday. https://t.co/D3ZaqFo55F'), ('dates', '2020-02-27 21:53:43'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.9088796377182007)])
OrderedDict([('tweet', 'Comey’s FBI misled the FISA Court 17 times.\n\nWe can’t simply reauthorize the system that allowed those lies and omissions to happen.\n\nNow is our chance to fix it.'), ('dates', '2020-02-26 15:20:19'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.7008828520774841)])
OrderedDict([('tweet', 'FISA Reform markup tomorrow in Judiciary.\n\nRemember, AG Bill Barr used the term "political surveillance" to describe what took place. That should never happen in America!\n\nThat\'s why we need to make changes. https://t.co/VnvDkC86ON'), ('dates', '2020-02-25 22:37:50'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6951966881752014)])
OrderedDict([('tweet', 'Unfortunately, the Socialist Left is not an aberration. That’s who the Democrats are today.\n\nBut America is about Freedom! It’s what makes us the greatest nation ever.'), ('dates', '2020-02-25 19:04:34'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.972585916519165)])
OrderedDict([('tweet', 'For years, Democrats and the mainstream press have attacked President Trump.\n\n@CPAC will lay out how it happened, how we fought back, and what comes next.\n\nhttps://t.co/wykvAg7LRl'), ('dates', '2020-02-25 14:59:04'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.7734354138374329)])
OrderedDict([('tweet', 'So many problems outlined in IG Horowitz’s FISA report.\n\nEven the FBI admitted their agents were sharing too much sensitive info with Dossier author Chris Steele!\n\nWhy won’t @RepJerryNadler bring the Inspector General before the House Judiciary Committee?\n\nhttps://t.co/3z3edupLUC'), ('dates', '2020-02-19 16:01:35'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6427876353263855)])
OrderedDict([('tweet', 'Charged for false statements: Flynn (R), Stone (R), Papadapoulos (R).\n\nIG found 4 false statements from Andy McCabe (D). No charges and welcomed as a hero by left-wing media.'), ('dates', '2020-02-15 23:47:57'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.5180332660675049)])
OrderedDict([('tweet', 'Democrats call on Barr to resign. REALLY?\n\nBill Barr is fighting to hold the FBI accountable for lying to the FISA Court SEVENTEEN times.'), ('dates', '2020-02-13 16:39:08'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.9851260185241699)])
OrderedDict([('tweet', 'Democrats just cast another net in their never-ending fishing expedition.\n\nImpeachment is over! Democrats should stop attacking the Administration and start focusing on solving problems for the American people.\n\nhttps://t.co/h2DKUEMHU0'), ('dates', '2020-02-13 14:50:47'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.960053563117981)])
OrderedDict([('tweet', 'President Trump’s budget sets the right priorities:\n\n-Funds the wall\n-Supports our military\n-Cuts wasteful foreign aid\n-Enacts work requirements for welfare recipients\n\nMost importantly, it sets us on a path to a balanced budget.'), ('dates', '2020-02-10 15:42:39'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6016601920127869)])
OrderedDict([('tweet', 'Democrats are focused on attacking President Trump.\n\nTonight, the President will focus on how we’re winning again:\n\n-Our booming economy\n-Lowering healthcare costs for American families\n-Securing our border\n-Protecting America’s national security'), ('dates', '2020-02-04 17:58:04'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6927071809768677)])
OrderedDict([('tweet', 'Schiff says he has no regrets about this impeachment sham.\n\nREALLY??\n\nHe doesn’t regret:\n-Saying “we have not spoken directly with the ‘Whistleblower’”\n- Saying “we... look forward to the ‘Whistleblower’s’ testimony”\n-His misleading “parody” of the President’s call'), ('dates', '2020-02-03 23:09:42'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.8179177045822144)])
OrderedDict([('tweet', 'Dems run the same old play.\n\nSomeone leaks to the New York Times what a draft of Bolton’s book supposedly says.\n\nThis last-ditch effort doesn’t alter the four fundamental facts of the case.'), ('dates', '2020-01-27 19:11:57'), ('file_name', 'Jim Jordan.csv'), ('pct_true', 0.6895089149475098)])
OrderedDict([('tweet', 'Sorry to interrupt this romance between @realDonaldTrump &amp; #BigOil but my @HouseDemocrats colleagues &amp; I have a problem with @USDOT &amp; @EPA’s dangerous cuts to emissions standards &amp; the harm posed to our climate &amp; public health.\n\nAnd that’s exactly what we wrote to @SecElaineChao https://t.co/Cd5GKZX27q'), ('dates', '2020-04-23 15:56:51'), ('file_name', 'Jimmy Gomez.csv'), ('pct_true', 0.6835635304450989)])
OrderedDict([('tweet', 'People are dying. #HealthcareHeroes are exhausted. #WorkingFamilies are desperate for relief.\n\nAnd @realDonaldTrump won’t shut up about television ratings.\n\nIf only he cared about every American the same way he cares about himself. \n\nIt doesn’t get more tone deaf than this. https://t.co/bsoegQVoVS'), ('dates', '2020-04-21 18:57:01'), ('file_name', 'Jimmy Gomez.csv'), ('pct_true', 0.617774248123169)])
OrderedDict([('tweet', 'And there you go, folks. When things go bad for @realDonaldTrump, he always goes back to his anti-immigrant playbook.\n\nAnd yet, he refuses to:\n\n▪️Provide states with adequate testing\n\n▪️Tell his supporters to stop violating #SocialDistancing guidelines by his own health officials https://t.co/DncFhAWZjI'), ('dates', '2020-04-21 04:13:21'), ('file_name', 'Jimmy Gomez.csv'), ('pct_true', 0.5236866474151611)])
OrderedDict([('tweet', '21 years ago today, 13 innocent lives were tragically lost at #Columbine.\n\nWe will never forget this tragedy or these victims, and we will never relent in our fight to ensure #NotOneMore life is taken by senseless gun violence. \n\n#EnoughIsEnough https://t.co/7XfDoGFdlg'), ('dates', '2020-04-20 20:59:58'), ('file_name', 'Jimmy Gomez.csv'), ('pct_true', 0.7410604953765869)])
OrderedDict([('tweet', 'This is a stupid decision, one based on politics and a refusal to take responsibility for @realDonaldTrump’s inaction to adequately contain the spread of #COVID19.\n\nAnd worst of all, it’s going to cost people their lives.\n\nI couldn’t think of a more foolish move. https://t.co/bVOcvMNfaw'), ('dates', '2020-04-14 23:20:10'), ('file_name', 'Jimmy Gomez.csv'), ('pct_true', 0.6659979820251465)])
OrderedDict([('tweet', 'These @WhiteHouse #Coronavirus Task Force briefings have become spectacles.\n\nRather than educating people about the #COVID19 threat, @realDonaldTrump uses this platform as a campaign event, weaponizing them to vilify his critics and mirroring those like @seanhannity of @FoxNews. https://t.co/p5xKN85K8a'), ('dates', '2020-04-14 20:07:58'), ('file_name', 'Jimmy Gomez.csv'), ('pct_true', 0.5355800986289978)])
OrderedDict([('tweet', 'Now more than ever, we need #POTUS to carry himself with integrity, employ good judgement &amp; demonstrate sensible leadership.\n\nThis letter to @SenSchumer is evidence that those traits are impossible for @realDonaldTrump to exhibit. He must take this #PublicHealth crisis seriously. https://t.co/L12oBjvsmp'), ('dates', '2020-04-03 17:05:38'), ('file_name', 'Jimmy Gomez.csv'), ('pct_true', 0.6272953748703003)])
OrderedDict([('tweet', 'Rep. King is right &amp; Senator McConnell is deadly wrong.  Clearly, the Majority Leader doesn’t want to help state and local governments to pay their employees, cops, firefighters, and healthcare in the middle of a pandemic.  (1/4) https://t.co/IunwWluYdz'), ('dates', '2020-04-23 21:55:47'), ('file_name', 'Jimmy Panetta.csv'), ('pct_true', 0.8313831686973572)])
OrderedDict([('tweet', 'The President wants to reopen the economy with a boom.  But if the economy opens too soon, the only boom will be another explosion of the COVID-19 virus. \nThe administration needs to mobilize the DPA to ramp up testing and deploy more contact tracing.\nhttps://t.co/bfTSReoqXz'), ('dates', '2020-04-17 19:17:54'), ('file_name', 'Jimmy Panetta.csv'), ('pct_true', 0.5661178231239319)])
OrderedDict([('tweet', "Michael Flynn already pled guilty to the FBI. That can't just be dismissed, unwritten or undone.  \n\nBill Barr is the most corrupt Attorney General in history. He acts like Trump’s henchman instead of America’s top seeker of justice.\n\nBarr should testify before Congress or resign."), ('dates', '2020-05-08 14:41:15'), ('file_name', 'Joaquin Castro.csv'), ('pct_true', 0.8566532135009766)])
OrderedDict([('tweet', "It's unconscionable that hardworking Americans who desperately need this assistance simply won't receive it just because they're married to an immigrant. I'm fighting this injustice so that all mixed-families get what they deserve.\n\nhttps://t.co/3QPBDuk8bQ"), ('dates', '2020-05-06 14:24:54'), ('file_name', 'Joaquin Castro.csv'), ('pct_true', 0.5594478845596313)])
OrderedDict([('tweet', 'Trump Admin needs to take action to hold the meatpacking industry accountable for labor abuses.\n\nMeatpacking workers are predominantly Latino, often undocumented or refugees without economic and political power.\n\nWe must protect these essential workers. https://t.co/Bmcj49RCFp'), ('dates', '2020-04-30 02:40:45'), ('file_name', 'Joaquin Castro.csv'), ('pct_true', 0.682847797870636)])
OrderedDict([('tweet', 'Given reports of the communist regime’s coverup &amp; mishandling of this virus, I’m proud to have signed this letter authored by @SenMcSallyAZ &amp; @RepMattGaetz to guarantee taxpayer money is not funding Chinese state-run labs. https://t.co/gkvMliLZeh'), ('dates', '2020-04-27 19:01:52'), ('file_name', 'Jodey C. Arrington.csv'), ('pct_true', 0.7186098098754883)])
OrderedDict([('tweet', 'There is no telling the amount of human lives &amp; costs that would have been saved if not for #China &amp; the @WHO Director General working in coordination to cover this up.\n\nTheir lack of transparency has hurt the entire world.\n\nMy interview w/ @SteveScully: https://t.co/joDx6AA1Mg https://t.co/lcwQXtbnt8'), ('dates', '2020-04-26 18:24:15'), ('file_name', 'Jodey C. Arrington.csv'), ('pct_true', 0.5916082262992859)])
OrderedDict([('tweet', 'The Saudi’s production scheme is destroying American energy!  \n\nNow they want to dump more on US market &amp; take what’s left of our storage capacity?!  \n\n\u2066@POTUS\u2069, we need a big barrel of #AmericaFirst action! https://t.co/SEtRi69AXy'), ('dates', '2020-04-22 15:06:56'), ('file_name', 'Jodey C. Arrington.csv'), ('pct_true', 0.5414752960205078)])
OrderedDict([('tweet', 'I agree with my colleague in the Senate, @LindseyGrahamSC. \n\nChina must be held accountable for the deaths &amp; infections caused by their systemic deception &amp; intentional disinformation campaign &amp; the WHO, being soft on China, exacerbated the problem.\n\nRead: https://t.co/vyWl6TmdHG'), ('dates', '2020-04-14 18:36:34'), ('file_name', 'Jodey C. Arrington.csv'), ('pct_true', 0.6928464770317078)])
OrderedDict([('tweet', 'Right now, our focus has to be response &amp; relief, but as we recover, we must hold China &amp; the complicit @WHO accountable for their #COVIDCoverUp.\n\nThis is expected from the communist regime, but not organizations for which we fund &amp; carry the water.\n\nMore w/ @mattmartinradio: https://t.co/MPpASpaEsr'), ('dates', '2020-04-09 16:36:07'), ('file_name', 'Jodey C. Arrington.csv'), ('pct_true', 0.8798094391822815)])
OrderedDict([('tweet', 'We can’t afford to have this bogged down w/ non-germane policies, like union pension bailouts, that are irrelevant to disaster relief leg.\n\nThis move is irresponsible &amp; will prolong the process, hurting our nation’s families &amp; small businesses.\n\nMore w/@JohnFBachman on @newsmax: https://t.co/RFCFeBmoOp'), ('dates', '2020-03-23 17:01:06'), ('file_name', 'Jodey C. Arrington.csv'), ('pct_true', 0.6008544564247131)])
OrderedDict([('tweet', 'China has been taking us for a ride for years, but thanks to the #PhaseOne China trade agreement, farmers in West Texas can finally see greener pastures ahead.\n\nGod bless @realDonaldTrump for having the political will to stand up to China &amp; ensure our farmers are treated fairly. https://t.co/hA0nzBUXuG'), ('dates', '2020-02-26 19:35:50'), ('file_name', 'Jodey C. Arrington.csv'), ('pct_true', 0.8605506420135498)])
OrderedDict([('tweet', 'Democratic Socialist proposals will double down on big government polices we saw fail in the previous Administration.\n\nThe @HouseGOP &amp; @realDonaldTrump are restoring freedom from high regulations &amp; a burdensome tax code, letting people keep more of their hard-earned money. https://t.co/2KQJkwAklv'), ('dates', '2020-02-25 15:53:41'), ('file_name', 'Jodey C. Arrington.csv'), ('pct_true', 0.9065136313438416)])
OrderedDict([('tweet', 'Anne &amp; Jane -\n\nRepublicans are red, Democrats are blue; there’s no one I admire more than you.\n\nUnending love to both my sweets; like the steady stream of @realDonaldTrump’s tweets.\n\nNobody can measure, no money can buy; my heart for you like a West Texas sky.\n\n-JA #ValentinesDay https://t.co/CnaW6xJ1Q6'), ('dates', '2020-02-14 19:51:49'), ('file_name', 'Jodey C. Arrington.csv'), ('pct_true', 0.5618007183074951)])
OrderedDict([('tweet', 'Freedom &amp; boundless opportunity are the core of American ideals.\n\nSocialism is a failed ideology that will rob our citizens of their freedom &amp; destroy our country.\n\nWe must condemn socialism in every form lest we consign the next generation of Americans as wards of the state. https://t.co/gyof11sORk'), ('dates', '2020-02-12 13:10:47'), ('file_name', 'Jodey C. Arrington.csv'), ('pct_true', 0.7070080041885376)])
OrderedDict([('tweet', "This was the set up of the century.\n\nThe liberal media refuses to cover it but what about @OversightDems?\n\nChairwoman @RepMaloney— you can’t pretend like this didn't happen.\n\nAllow us to subpoena @Comey and his cronies for a hearing. https://t.co/upGfpgLHyo"), ('dates', '2020-05-09 19:14:26'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.8744527101516724)])
OrderedDict([('tweet', 'Constitutional rights are not waived for a pandemic, an emergency, or any reason.\n\nConstitutional rights are the bedrock of our country. Erode them, and you erode the foundation of America.\n\nElected officials abusing their power must be held accountable. https://t.co/mmN3FJnrDo'), ('dates', '2020-05-06 16:45:21'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.6218734383583069)])
OrderedDict([('tweet', "Disgraced @RepAdamSchiff has repeatedly lied and mislead the public.\n\nSchiff needs to release the transcripts — he won't.\n\n@SpeakerPelosi's refusal to remove Schiff as House Intel Chairman tells you everything you need to know about how much she values truth and honesty. https://t.co/Ctupw4zkRx"), ('dates', '2020-05-06 13:59:08'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9211300611495972)])
OrderedDict([('tweet', "#COVID19 has reminded Americans of @SpeakerPelosi's true colors:\n\n➡️Taking advantage of the crisis by delaying small business aid in a shameful attempt to push her liberal policy agenda\n\n➡️Leading from behind, and refusing to come back to work\n\nIt's time to reopen the House. https://t.co/1m2LA4cFgo"), ('dates', '2020-05-04 13:46:54'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.683621883392334)])
OrderedDict([('tweet', "This pandemic proved that government officials shouldn't be allowed to become oppressive dictators.\n\nEmergencies should never be used as an excuse to destroy our civil liberties.\n\nThese petty tyrants should be held accountable.  https://t.co/ULPeUrSpdh"), ('dates', '2020-05-04 13:25:00'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.7442013025283813)])
OrderedDict([('tweet', 'As more information emerges that #MikeFlynn was set up by the FBI, will the Dems and media who joined efforts to destroy him now come to his defense? \n\nWill they apologize? \n\nAbsolutely not. Dems only defend one kind of American - their own kind.\n\nEveryone else is a deplorable. https://t.co/K8ut6Z1WPj'), ('dates', '2020-05-01 13:07:06'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.7931903004646301)])
OrderedDict([('tweet', "The new #MichaelFlynn documents bolster longstanding concerns on how @Comey &amp; his leadership cabal ran the FBI.\n\nThis raises new questions:\n🚨 Where was Director Wray?\n🚨 Why didn't he release this sooner?\n🚨 Why did it take AG Barr getting involved for this to come to light? https://t.co/RiuPJFoh0i"), ('dates', '2020-04-30 18:38:26'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.5269759893417358)])
OrderedDict([('tweet', 'BREAKING: Failing News Network endorses Communist totalitarianism.\n\nEven the Communist Party of China gets more favorable coverage than @realDonaldTrump at CNN. \n\nThey’re an insult to the news industry and to freedom loving Americans! https://t.co/pk9H53BWjQ'), ('dates', '2020-04-30 01:40:42'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.858281135559082)])
OrderedDict([('tweet', 'It’s no surprise that @SpeakerPelosi gave control of the Select Committee to a member of her own leadership team who said the virus is a "tremendous opportunity to restructure things to fit our vision."\n\nThis is about preventing the reelection of @realDonaldTrump, plain &amp; simple. https://t.co/A0oeLeCERV'), ('dates', '2020-04-29 23:55:46'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.655198335647583)])
OrderedDict([('tweet', 'Disgraced @RepAdamSchiff recently claimed lives would\'ve been saved if the bogus impeachment of @POTUS had been successful.\n\nThe radical Democrats will use the #COVID19 "Select Committee" to lay the road for impeachment 2.0 — anything to prevent @realDonaldTrump\'s reelection! https://t.co/RewT9BEQPZ'), ('dates', '2020-04-28 23:39:26'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9589470624923706)])
OrderedDict([('tweet', "Manipulating the Bible to serve the Democrats' agenda is outrageously hypocritical.\n\nWhat about the Bible's teachings regarding life, equitable justice, morality, etc?\n\nYou can't spin Scripture! https://t.co/JzUGyO5WoC"), ('dates', '2020-04-27 15:14:57'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.7822133898735046)])
OrderedDict([('tweet', "Democrats assumed guilt against Brett Kavanaugh with no corroborative evidence.\n\nNow, they're giving @JoeBiden a complete pass in spite of evidence.\n\nOnce again, the Left subverts justice in their own interest.\n\nThe double-standard is a gross perversion in favor of lawlessness!"), ('dates', '2020-04-27 13:01:22'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9117376208305359)])
OrderedDict([('tweet', '.@AOC wants you to forfeit your job &amp; paycheck when we reopen the economy.\n\nMillions have spent the last few weeks just trying to keep their heads above water.\n\nNow the socialists want everyone to help push their agenda by joining a national boycott.\n\nThe Left has lost its mind. https://t.co/Vriib7Q4WS'), ('dates', '2020-04-22 20:32:56'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.6907228231430054)])
OrderedDict([('tweet', "It's time to accept that many of the #COVID19 projections used to justify the economic shutdown were way off the mark.\n\nWe need to investigate why the models were inaccurate — and to hold accountable those who got it wrong.\n\nhttps://t.co/UzRIKjhlB9"), ('dates', '2020-04-21 18:51:43'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.5055296421051025)])
OrderedDict([('tweet', 'The same politicians &amp; media outlets who were outraged over @realDonaldTrump\'s early life-saving action to restrict China travel are "outraged" again.\n\nThese people have no shame.\n\n@POTUS is making the right call to help workers &amp; save lives by temporarily halting immigration. https://t.co/n4tQlinlLy'), ('dates', '2020-04-21 16:55:36'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9442497491836548)])
OrderedDict([('tweet', 'Democrats are more concerned with the well-being of illegal aliens than American citizens.\n\nWhile Americans are forced to stay at home &amp; out of work, @HouseDemocrats &amp; @SenateDems want to release detained illegal aliens — AND give them stimulus checks!\n\nInexplicable insanity!'), ('dates', '2020-04-17 21:13:19'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9852295517921448)])
OrderedDict([('tweet', "While @SpeakerPelosi wishes to lecture Americans from her million dollar mansion and $24,000 refrigerator, business owners across America are struggling.\n\nSmall businesses are the backbone &amp; life line of the American economy.\n\nThey need Congress's help. \n\nEnd the games. https://t.co/3bSdxfOObB"), ('dates', '2020-04-16 18:50:42'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.8169968128204346)])
OrderedDict([('tweet', 'Democrats are sacrificing American small business owners to advance their liberal agenda.\n\nPPP loan funding is exhausted.\n\nMeanwhile @SpeakerPelosi &amp; @SenSchumer are playing political games. Every hour they delay drives more companies out of business &amp; more employees out of work. https://t.co/mKH2cYcuZY'), ('dates', '2020-04-16 17:33:11'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9173293113708496)])
OrderedDict([('tweet', 'We know China lied &amp; concealed data, enabling #COVID19 to spread internationally.\n\nAnd we know China hoarded medical equipment, worsening shortages here.\n\nNow this bombshell...\n\nIf true, Communist China not only exacerbated this #pandemic — it caused it.\n\nhttps://t.co/GbtmWdtkSl'), ('dates', '2020-04-16 01:38:04'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9908189177513123)])
OrderedDict([('tweet', 'Enough is enough.\n\nChina lied. People died.\n\nThe one who deserves praise for their response to coronavirus is @realDonaldTrump.\n\n@POTUS acted early to restrict travel from China. He was criticized by Democrats, the media, the WHO &amp; China — all of whom now have zero credibility. https://t.co/XaOy6eKGjN'), ('dates', '2020-04-15 19:14:21'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9653818011283875)])
OrderedDict([('tweet', '.@SpeakerPelosi is the saddest excuse for a leader.\n\nShe played political games when people &amp; businesses needed help. Shameful.\n\nAnd now she has the audacity to tout her role in delaying relief aid. Despicable.\n\nEnd the games, Madam Speaker. https://t.co/wmDxNL30G6'), ('dates', '2020-04-15 15:25:17'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.6144854426383972)])
OrderedDict([('tweet', ".@realDonaldTrump is right to withhold @WHO funding.\n\nThe World Health Organization aided &amp; abetted China's #COVID propaganda campaign, spread misinformation, and resisted Trump's travel restrictions on China.\n\nIt's simple:\n\nWe get to demand answers because we pay the bills. https://t.co/SeiZs158m7"), ('dates', '2020-04-15 14:38:35'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.6238042712211609)])
OrderedDict([('tweet', ".@Judgenap is spot on. Civil liberties don't disappear because there's a virus.\n\nAcross America, we're seeing inexcusably aggressive actions from state &amp; local officials — often w/ no real purpose or oversight.\n\nPandemic policies must submit to Constitutional rights. https://t.co/0FQySHh0wB"), ('dates', '2020-04-14 14:48:48'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.743040144443512)])
OrderedDict([('tweet', 'America pays the World Health Organization\'s bills and we\'re owed answers:\n\n➡️Why deny there was evidence of human-to-human transmission?\n\n➡️Why heap bogus praise on China for its "transparency"?\n\n➡️Why oppose @realDonaldTrump\'s China travel restrictions?\n\nhttps://t.co/DekGAMEsG9'), ('dates', '2020-04-11 00:56:27'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.552897572517395)])
OrderedDict([('tweet', ".@WHO inexcusably aided China's #coronavirus propaganda campaign, spent weeks telling the world there was no evidence of human-to-human transmission, and even pushed back against @realDonaldTrump's early travel restrictions on China.\n\n@POTUS is absolutely right to demand answers. https://t.co/IcgA6H7IsY"), ('dates', '2020-04-08 18:21:10'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9579939842224121)])
OrderedDict([('tweet', 'Communist China cannot be trusted to share accurate information on #Coronavirus.\n\nEven now China is waging a misinformation campaign about the outbreak.\n\nWhy would we trust China to be our supplier of strategic resources like medical equipment?\n\nWe need to wake up! #AmericaFirst https://t.co/PWBgeDtRPE'), ('dates', '2020-04-08 16:56:29'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.972639262676239)])
OrderedDict([('tweet', 'This is preposterous.\n\n@SpeakerPelosi wants a new special committee — not the Oversight Committee — to oversee the federal #coronavirus response.\n\nWho\'s to lead it?\n\nHouse Dem @WhipClyburn, who says the virus is "a tremendous opportunity to restructure things to fit our vision." https://t.co/IYBNLCeDbQ'), ('dates', '2020-04-03 15:52:07'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.5245260000228882)])
OrderedDict([('tweet', "FISA is broken. Democrats &amp; Deep Staters can no longer deny it.\n\nNew DOJ IG findings confirm there's a systematic problem in FISA.\n\nThe IG found problems with every single FISA application reviewed!\n\nThe surveillance on @realDonaldTrump's campaign was the tip of the iceberg. https://t.co/igwxH9o67J"), ('dates', '2020-04-01 15:53:35'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.8407988548278809)])
OrderedDict([('tweet', "3 coronavirus bills so far:\n1️⃣$8.3 billion\n2️⃣$100+ billion\n3️⃣$2.3 trillion\n\nWe're spending at an unprecedented speed to combat COVID-19.\n\nBefore we race to a 4th &amp; 5th bill — price tags unknown — we need to put our financial house in order.\n\nEnact a Balanced Budget Amendment!"), ('dates', '2020-03-31 14:49:18'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.577928364276886)])
OrderedDict([('tweet', ".@SpeakerPelosi delayed the last stimulus bill to push her radical liberal agenda.\n\nNow she's doing it again! She wants to jam infrastructure and who knows what else into a fourth #coronavirus bill.\n\nNovel concept... emergency coronavirus bills should be focused on coronavirus!"), ('dates', '2020-03-31 13:08:08'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.695294201374054)])
OrderedDict([('tweet', "This is outrageous.\n\n@SpeakerPelosi accuses @realDonaldTrump of delay after she held up the phase 3 stimulus bill for days in an attempt to jam her radical socialist agenda into the bill.\n\nMadam Speaker, you put the lives and well-being of Americans at risk &amp; that's on YOU! https://t.co/1gF3YwmE2A"), ('dates', '2020-03-29 18:14:56'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9773181676864624)])
OrderedDict([('tweet', "We have to balance smart economic policy &amp; smart health policy.\n\nMost small businesses don't have the capital to survive another couple of days.\n\nWe have to start looking at when &amp; how to reopen the country, or many Americans won't have jobs to return to.\n\nhttps://t.co/2MuJooaGFx"), ('dates', '2020-03-24 20:47:50'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.7429128885269165)])
OrderedDict([('tweet', '.@SpeakerPelosi\'s Phase 3 bill is more focused on advancing a liberal agenda than fighting the virus or helping the economy.\n\nShe insists on "not letting a good crisis go to waste."\n\nHer proposal?\n\nOne small step for #coronavirus aid; one huge leap for socialism &amp; bankruptcy. https://t.co/ph7Wotv09E'), ('dates', '2020-03-24 12:57:37'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.5345392823219299)])
OrderedDict([('tweet', '#Coronavirus began in China &amp; the Communists lied about it — blocking critical news &amp; data from being shared w/ the West.\n\nHow many thousands across the globe would have been saved if China had been transparent &amp; forthright?\n\nNow the Left-wing media wants to defend China. Unreal.'), ('dates', '2020-03-20 22:09:45'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9924142956733704)])
OrderedDict([('tweet', '.@JoeBiden\n➡️Favors China over America.\n➡️Defended China about where the virus originated.\n➡️Criticized President @realDonaldTrump’s life-saving travel restrictions on China.\n\nQuit playing politics &amp; be part of the solution rather than part of the problem.\n\n#AllInThisTogether'), ('dates', '2020-03-19 13:58:48'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9944871068000793)])
OrderedDict([('tweet', '.@SteveScalise is spot on.\n\n@realDonaldTrump is working to keep us safe as America braces for the #CoronavirusPandemic.\n\nMeanwhile @SpeakerPelosi and her Democrats want to dismantle @POTUS\'s ability to implement travel restrictions.\n\nPelosi must pull her "No Ban Act" immediately. https://t.co/nkOgEiMxAA'), ('dates', '2020-03-12 13:39:34'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.8548082709312439)])
OrderedDict([('tweet', '.@realDonaldTrump acted early to stop the spread of #coronavirus by declaring a public health emergency &amp; putting travel restrictions from China on Jan 31.\n\nThen, Democrats and the media accused @POTUS of fear-mongering and xenophobia.\n\nThe President’s early actions saved lives! https://t.co/DGIC5u96Ut'), ('dates', '2020-03-11 16:04:11'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.8984329104423523)])
OrderedDict([('tweet', "When the House considers reauthorizing FISA, remember:\n\n➡️DOJ IG found gross incompetence &amp; bias in the FBI's handling of FISA\n\n➡️FISA Court said they can no longer 100% trust FBI FISA applications\n\n➡️Democrats have refused to allow a public hearing on any of this\n\nUNACCEPTABLE."), ('dates', '2020-03-10 12:00:42'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.9087253212928772)])
OrderedDict([('tweet', 'There is a rising wave of violence by the Left in America.\n\nNumerous assaults on @realDonaldTrump supporters and ANTIFA riots in the streets.\n\nI fear this wave will only escalate as the Left continues to lash out.\n\n@chuckschumer should call it out and work to stop it — or resign. https://t.co/A0kLu88r55'), ('dates', '2020-03-09 12:11:32'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.5753523707389832)])
OrderedDict([('tweet', 'Democrat candidates drop out, Pelosi says: "America is misogynistic!"\n\nDemocrats should examine their candidates.\n\nCrooked Hillary &amp; Pocahontas are not the leaders America wants. A woman will be president one day, just not those two.\n\nTheir problem isn\'t misogyny, it\'s mistrust! https://t.co/QT5XebIfEC'), ('dates', '2020-03-07 13:37:55'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.8311806321144104)])
OrderedDict([('tweet', "Seems Mitt Romney hasn't figured out yet what we do here.\n\nCongress's job is to conduct oversight &amp; investigate corruption in our government.\n\nBut Romney shows an uncanny attachment to delusion.\n\nMitt, here's a fact:\n\nAmerica is sick of politicians who won't address corruption! https://t.co/O47jKvJZ3d"), ('dates', '2020-03-06 14:16:06'), ('file_name', 'Jody B. Hice.csv'), ('pct_true', 0.5504870414733887)])
OrderedDict([('tweet', "The House voted to replenish the #EIDL program w/ $60 billion AND to give farmers access, but @USTreasury &amp; @SBAgov haven't provided a portal for them to get the funding. They need to do so fast, and they should help move farmers to the front. More here: https://t.co/O3Nv4AGHt1"), ('dates', '2020-04-28 20:30:00'), ('file_name', 'Joe Courtney.csv'), ('pct_true', 0.5855135321617126)])
OrderedDict([('tweet', 'POTUS’s disastrous #budget proposal severs critical lifelines for millions of Americans, makes $1.1 TRILLION in cuts to Medicaid, Medicare &amp; health programs, slashes investments in America’s workforce &amp; eliminates bedrock programs that fund America’s public education system. https://t.co/6EtNBoyxpL'), ('dates', '2020-02-11 22:43:11'), ('file_name', 'Joe Courtney.csv'), ('pct_true', 0.7982503771781921)])
OrderedDict([('tweet', "I helped write the ACA provision back in 2009 that banned discrimination against Americans with pre-ex conditions. POTUS's claim that he’s “protecting” the ban is deceitful. He’s suing to roll back the ban on #PreEx exclusions, endangering the health care of 130 million people. https://t.co/ZfUGHhKRtM"), ('dates', '2020-02-05 17:17:25'), ('file_name', 'Joe Courtney.csv'), ('pct_true', 0.5901942253112793)])
OrderedDict([('tweet', 'COVID-19 has forced us to openly acknowledge what many have known for a long time. We must use this as an opportunity to address long-term health disparities for African Americans. https://t.co/1gi67JJJ5Y'), ('dates', '2020-04-27 19:05:37'), ('file_name', 'Joe Cunningham.csv'), ('pct_true', 0.8461852073669434)])
OrderedDict([('tweet', 'On this day 10 years ago, the largest offshore oil spill in world history dumped 5 million barrels of oil into the Gulf of Mexico.11 people died. \n\n10 years after the #DeepwaterHorizon disaster, we need to #ProtectOurCoast more than ever.'), ('dates', '2020-04-20 18:39:25'), ('file_name', 'Joe Cunningham.csv'), ('pct_true', 0.5225318670272827)])
OrderedDict([('tweet', 'This is unacceptable. As South Carolina businesses struggle, Congress cannot treat this program like a political football. It’s time to put politics aside, act swiftly, and get more funding and relief into the hands of Lowcountry businesses. \nhttps://t.co/zrHN3F2vdY'), ('dates', '2020-04-16 17:27:52'), ('file_name', 'Joe Cunningham.csv'), ('pct_true', 0.705386221408844)])
OrderedDict([('tweet', 'Novaa was pretty upset about turning 5 in isolation, so the Burton Fire Department &amp; Beaufort County Sheriff’s Office turned up the volume at her party! What a wonderful display of community and hospitality: https://t.co/Ny5gIqD69T'), ('dates', '2020-04-15 17:02:13'), ('file_name', 'Joe Cunningham.csv'), ('pct_true', 0.5861629843711853)])
OrderedDict([('tweet', 'TSA employees protect the millions of Americans who travel through our airports each day.  The House just passed bipartisan legislation that gives them the same rights and protections as other federal employees – including increased salaries and 12 weeks of paid parental leave.'), ('dates', '2020-03-05 18:33:47'), ('file_name', 'Joe Cunningham.csv'), ('pct_true', 0.8287724852561951)])
OrderedDict([('tweet', 'It is unthinkable that in the middle of a healthcare crisis, Republican Attorneys General would still seek to eliminate healthcare for 800,000 West Virginians and millions of Americans with pre-existing conditions. https://t.co/vq0hzVvjyo https://t.co/49WsUcN5WL'), ('dates', '2020-05-08 19:01:26'), ('file_name', 'Joe Manchin, III.csv'), ('pct_true', 0.5501009821891785)])
OrderedDict([('tweet', 'Today I urged Senate Majority Leader Mitch McConnell to focus the Senate’s efforts in returning to Washington, DC on COVID-19 pandemic related issues and oversight of COVID-19 related legislation enacted by Congress. MORE: https://t.co/OtDF5gx4Qy'), ('dates', '2020-04-28 23:18:30'), ('file_name', 'Joe Manchin, III.csv'), ('pct_true', 0.5255189538002014)])
OrderedDict([('tweet', 'When career prosecutors resigned in February, many of us called for Bill Barr to testify immediately. \n\n3 months later &amp; he remains the only AG in nearly 30 years to not testify before the House Judiciary Committee.\n\nWhat more will it take for Congress to take this seriously? https://t.co/ErG0gSmTiM'), ('dates', '2020-05-07 19:15:26'), ('file_name', 'Joe Neguse.csv'), ('pct_true', 0.5127440094947815)])
OrderedDict([('tweet', '.@realDonaldTrump is absolutely right to hold U.S. aid to the WHO after the organization spent the last six months parroting Chinese propaganda &amp; covering up critical data about coronavirus transmission. Read my full statement ➡️ https://t.co/BfYEHxmczP'), ('dates', '2020-04-16 21:31:39'), ('file_name', 'Joe Wilson.csv'), ('pct_true', 0.7513875365257263)])
OrderedDict([('tweet', 'China’s efforts to conceal the origin of coronavirus can not be tolerated, it is important that China immediately close live wildlife markets. These markets have exposed the people of China to life-threatening diseases, putting their citizens &amp; families across the world at risk. https://t.co/Ip5RsObCW7'), ('dates', '2020-04-15 16:20:13'), ('file_name', 'Joe Wilson.csv'), ('pct_true', 0.979496419429779)])
OrderedDict([('tweet', "This is what happens when the House Majority rushes a bill to a vote without allowing all members the opportunity to review. I look forward to reading the revised text, hopefully this time small businesses won't be left behind. https://t.co/rIFoEgIA6A"), ('dates', '2020-03-16 15:09:15'), ('file_name', 'Joe Wilson.csv'), ('pct_true', 0.5469806790351868)])
OrderedDict([('tweet', "Congress has been occupied because of the Dems' obsession with impeaching @realDonaldTrump. I am thankful that today the Senate voted to acquit &amp; Congress can finally get back to work for the American people. I am grateful that POTUS will now always be credited as #acquitted."), ('dates', '2020-02-05 21:44:37'), ('file_name', 'Joe Wilson.csv'), ('pct_true', 0.7784093022346497)])
OrderedDict([('tweet', 'Regardless of the Democrat’s insulting false impeachment inquiry, @realDonaldTrump has worked toward his pro-jobs, pro-family, and pro-America agenda. POTUS continues his record of promises made, promises kept. https://t.co/tudeDiNyot'), ('dates', '2020-02-04 17:29:26'), ('file_name', 'Joe Wilson.csv'), ('pct_true', 0.7612504959106445)])
OrderedDict([('tweet', 'Dirty cops and crooked politicians do not, in fact, go well together. Between Attorney General Barr and himself, Trump finally got something right. https://t.co/wTMUriB982'), ('dates', '2020-05-09 18:09:53'), ('file_name', 'John A. Yarmuth.csv'), ('pct_true', 0.6630162596702576)])
OrderedDict([('tweet', 'The 30+ million Americans who have been laid off don’t need smartass quips, they need help. And those actually in “hell” are pleading for gloves, masks, gowns &amp; more to care for their patients. Is this the “compassionate conservatism” I’ve long heard about? https://t.co/ZaF70EXdQ5'), ('dates', '2020-05-08 20:21:04'), ('file_name', 'John A. Yarmuth.csv'), ('pct_true', 0.5088998079299927)])
OrderedDict([('tweet', 'Whatever bizarro world lies beyond the Twilight Zone, Trump went there with his disinfectant idea. I can’t believe I’m saying this, but America ignore the President and don’t ingest or inject cleaning products.\n#MadScientistPOTUS #TheLysolSolution'), ('dates', '2020-04-24 14:35:24'), ('file_name', 'John A. Yarmuth.csv'), ('pct_true', 0.7655831575393677)])
OrderedDict([('tweet', 'Covering the President’s daily briefings is endangering the public. The broadcast media must report on what he says, but they should stop this start to finish coverage. There is too little useful information, and a lot of misinformation, to warrant this type of coverage.'), ('dates', '2020-04-06 17:40:35'), ('file_name', 'John A. Yarmuth.csv'), ('pct_true', 0.7475275993347168)])
OrderedDict([('tweet', 'The President floating worst-case scenario numbers in order to take credit for “only” hundreds of thousands of American lives lost is...what? Is there a word for that type of disgraceful strategy? This isn’t a PR problem, POTUS. It’s a crisis and Americans are dying.'), ('dates', '2020-03-29 23:53:27'), ('file_name', 'John A. Yarmuth.csv'), ('pct_true', 0.8903290033340454)])
OrderedDict([('tweet', 'We need to #ExpandSocialSecurity and put money in the hands of those who need it most during #COVID19 : seniors, communities of color, and people with disabilities - not corporations.'), ('dates', '2020-05-07 17:45:00'), ('file_name', 'John B. Larson.csv'), ('pct_true', 0.54475337266922)])
OrderedDict([('tweet', "Trump's proposed payroll tax cut will harm Social Security and greatly benefit large corporations. I believe we need to help our most vulnerable, seniors, and #ExpandSocialSecurity."), ('dates', '2020-05-07 17:15:00'), ('file_name', 'John B. Larson.csv'), ('pct_true', 0.6069102883338928)])
OrderedDict([('tweet', "We should be strengthening #SocialSecurity, not giving a payroll tax cut to corporations and the wealthy. That's why I introduced the Emergency Social Security Benefits Improvement Act, to help our most vulnerable during the #COVID19 pandemic. #ExpandSocialSecurity"), ('dates', '2020-05-07 17:00:00'), ('file_name', 'John B. Larson.csv'), ('pct_true', 0.8821492791175842)])
OrderedDict([('tweet', "Students are not in classrooms but thousands will take a day-long vow of silence, bringing attention to the abuse aimed at their LGBTQ classmates. This harassment hurts student's education and must stop. #NationalDayofSilence"), ('dates', '2020-04-24 17:37:14'), ('file_name', 'John B. Larson.csv'), ('pct_true', 0.8695018887519836)])
OrderedDict([('tweet', 'There are 30 mil. Americans out of work through no fault of their own, &amp; that’s why it is important that the Senate is in session. We can use targeted, temporary, &amp; specifically focused legislation to help those impacted by #coronavirus. https://t.co/81dzA30FUC'), ('dates', '2020-05-04 17:03:56'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.6679672598838806)])
OrderedDict([('tweet', 'We should never be dependent on China for anything that is in the vital interest of the United States. @realDonaldTrump is going to continue to hold China accountable, &amp; I will continue to work on this with him. https://t.co/3XPWilhdBv'), ('dates', '2020-05-01 17:16:19'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.9743254780769348)])
OrderedDict([('tweet', '.@realDonaldTrump’s leadership helped secure another $320 bil for small businesses that will save middle-class paychecks. His phased plan for reopening will get Americans safely back to work. The House should pass the #PPP bill without delay so @POTUS can sign it into law ASAP. https://t.co/f9MR2gT9gx'), ('dates', '2020-04-22 20:00:26'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.5473107099533081)])
OrderedDict([('tweet', 'Two weeks ago, Senate Republicans put forward a bill to prevent the #PaycheckProtectionProgram from running out of money. Last week, it officially ran out of money because @SpeakerPelosi &amp; @SenSchumer blocked funding &amp; burned precious time for America’s small businesses. https://t.co/3DM8TNy5II'), ('dates', '2020-04-21 21:30:19'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.8315419554710388)])
OrderedDict([('tweet', 'The @SenSchumer &amp; @SpeakerPelosi paycheck blockade must end now. The #PaycheckProtectionProgram has ensured millions of Americans keep getting paid during this crisis. We need further funding to continue this vital work. https://t.co/YkXB0QvbaS'), ('dates', '2020-04-16 13:07:59'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.83915776014328)])
OrderedDict([('tweet', 'America’s small businesses are the backbone of our nation, but they face a life-or-death situation in the midst of the #coronavirus. Starting today, small businesses can take part in the #PaycheckProtectionProgram that the #CARESAct established. https://t.co/FjDUkXHAWg'), ('dates', '2020-04-03 14:36:50'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.7353348731994629)])
OrderedDict([('tweet', '.@SpeakerPelosi came back to Congress with no urgency &amp; no real understanding of the needs of Americans. Republicans are ready to act &amp; to finish this tonight. The American people need relief, &amp; @SpeakerPelosi &amp; @SenSchumer are wasting time that we as a nation don’t have. https://t.co/ekTUGMMOse'), ('dates', '2020-03-24 23:27:06'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.7957033514976501)])
OrderedDict([('tweet', 'You are either helping right now, or hurting. Pelosi’s handpicked partisan ploys are helping no one. She’s hurting American families by prioritizing airplane emissions, same-day voter registration&amp;wind energy tax credits over #coronavirus relief. #StopTheGamesNancy, #PassTheBill'), ('dates', '2020-03-24 14:00:30'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.984805166721344)])
OrderedDict([('tweet', "We need to make sure when Americans wake up tomorrow, they don't have the same fear about #coronavirus &amp; their family's wellbeing. We need decisive action today. The time for politics is beyond us. We need to vote today, &amp; we need to pass this today. https://t.co/bYdL6g9de7"), ('dates', '2020-03-23 22:06:43'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.6017721891403198)])
OrderedDict([('tweet', 'Tonight, Democrats blocked emergency coronavirus legislation drafted in a bipartisan way, to provide immediate relief for American families. Democrats need to put partisan politics aside and join us in passing emergency funding to help Americans and stabilize our economy now.'), ('dates', '2020-03-23 02:24:57'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.8290675282478333)])
OrderedDict([('tweet', 'Congress will stop at nothing to assure that America and Americans survive and revive and thrive. We’re working on legislation that will bolster our economy and help small businesses stay viable. Together, we will defeat the #coronavirus. https://t.co/CPK0qS6bV0'), ('dates', '2020-03-22 00:11:44'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.7391138672828674)])
OrderedDict([('tweet', 'Spoke to @foxandfriends this morning on how Congress’ coronavirus legislation is a rescue operation for our country. The American people expect Congress to act &amp; to do it in a bipartisan way. We need all hands on deck to deal with &amp; recover from the #coronavirus. https://t.co/hwr24R1l7T'), ('dates', '2020-03-20 13:34:43'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.5779322385787964)])
OrderedDict([('tweet', 'Sen. Schumer’s statements today re: two #SCOTUS justices are outrageous, uncalled for and out of bounds. Chief Justice Roberts is right, we cannot tolerate political violence or threats of harassment. We should be looking to elevate our debates, not lower them. https://t.co/48zZfSR7mZ'), ('dates', '2020-03-04 23:18:28'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.7965658903121948)])
OrderedDict([('tweet', 'We need to find common ground on health care. The American people don’t want Democrats’ radical health care schemes that take insurance away from 180 million Americans who get it from work &amp; give “free” insurance to illegal immigrants while raising our taxes to pay for it. https://t.co/rt7zgn3RQT'), ('dates', '2020-02-27 19:10:00'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.9372007250785828)])
OrderedDict([('tweet', 'Economic freedom and free markets: Those are the engines of our economy. Socialism is an enemy of our economy. I think the American people will take economic freedom over socialism any day. https://t.co/5kCQYPRD5d'), ('dates', '2020-02-12 17:14:22'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.5494457483291626)])
OrderedDict([('tweet', 'Democrats are in disarray. For all their anger and outrage, they’ve failed to tap into all of the great things happening in America. Republicans, however, have a results-driven agenda. The economic renaissance we’re seeing is a direct result of Republican pro-growth policies. https://t.co/KcMC8cEmLx'), ('dates', '2020-02-11 21:49:52'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.897510826587677)])
OrderedDict([('tweet', 'The time for political stunts is over. The Senate had a fair trial and clear acquittal. Republicans stayed true to the Constitution. Now the Senate gets back to work for the American people. https://t.co/zN9iJjI3ma'), ('dates', '2020-02-05 23:15:40'), ('file_name', 'John Barrasso.csv'), ('pct_true', 0.8015154600143433)])
OrderedDict([('tweet', 'Recognizing the critical need to help small businesses survive the financial hardships caused by the coronavirus, Congress has provided significant funding for the #PaycheckProtectionProgram. READ my latest column about how its helping Arkansas #smallbiz.\n\nhttps://t.co/V8r6w8K9vM'), ('dates', '2020-04-27 18:27:42'), ('file_name', 'John Boozman.csv'), ('pct_true', 0.6508843302726746)])
OrderedDict([('tweet', 'Why, if Senate Ds didn’t want to return to work in the Capitol, didn’t they agree to let this non controversial nomination go by unanimous consent or voice vote? They can’t have it both ways. https://t.co/2AKmGKLy6r'), ('dates', '2020-05-05 00:09:39'), ('file_name', 'John Cornyn.csv'), ('pct_true', 0.7206800580024719)])
OrderedDict([('tweet', 'Your CDC won’t allow CA to use its own capable labs to test for Coronavirus and has provided just 200 test kits for a state of 40 million people. That inaction is a horrific disservice to public health and safety and we will not stand for it. https://t.co/TfKXnGxxFj'), ('dates', '2020-02-28 05:34:13'), ('file_name', 'John Garamendi.csv'), ('pct_true', 0.5978581309318542)])
OrderedDict([('tweet', "And yet, @SpeakerPelosi and Democrat leaders continue to play politics with people's lives, refusing to quickly replenish the #PPP. Over 5 million people filed for unemployment this week, and that number will continue to grow unless we provide relief. We need to fund this NOW. https://t.co/sswuvkhdSb"), ('dates', '2020-04-16 15:37:10'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.9420763850212097)])
OrderedDict([('tweet', "This #Covid_19 pandemic shows we have to be concerned about global health. This didn't start here, but it's here, and look at the impact it has had. Global partnerships are important, but we must hold their feet to the fire on performing correctly. #WHO https://t.co/2hZzGx4maM"), ('dates', '2020-04-16 00:08:13'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.5828866958618164)])
OrderedDict([('tweet', 'Senate Democrats once again voted against giving families &amp; small businesses immediate financial relief. In fact, they chose to abandon a bipartisan relief package once @SpeakerPelosi showed up with an irrelevant &amp; unnecessary list of political demands. Here are a few examples:'), ('dates', '2020-03-23 21:36:03'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.9665083289146423)])
OrderedDict([('tweet', 'Disappointing to see @SenSchumer leading the Democrats to vote down a bipartisan #coronavirus relief package yesterday. Now, while we are all back in our districts, @SpeakerPelosi is in D.C. attempting to create her own legislation for a political wish list.'), ('dates', '2020-03-23 15:24:28'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.6568752527236938)])
OrderedDict([('tweet', "The international horse meat trade is a dirty business. Horses are often shipped to other countries to be killed &amp; sold globally. Even more troubling, many haven't been cleared by any health authority, &amp; some carry diseases that make their meat unsuitable for human consumption."), ('dates', '2020-03-02 21:39:24'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.6845022439956665)])
OrderedDict([('tweet', ".@realDonaldTrump has kept his promise to protect America's national security. Terrorists have been brought to justice, our military has been rebuilt, and strong America-first policies have improved our standing throughout the world. #SOTU"), ('dates', '2020-02-05 03:53:02'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.9258103370666504)])
OrderedDict([('tweet', "To those who accuse @realDonaldTrump of assassinating a general - he didn't. He killed a terrorist. As long as Soleimani was living and breathing, he posed an imminent threat to U.S. lives. Americans are safer now that he is dead. #SOTU2020"), ('dates', '2020-02-05 03:15:30'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.8740153312683105)])
OrderedDict([('tweet', '.@realDonaldTrump positions America to face any threat to our national security — no matter the frontier. The commission of the Space Force improves America’s military readiness and helps protect our critical infrastructure. #SOTU2020'), ('dates', '2020-02-05 03:10:46'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.8823023438453674)])
OrderedDict([('tweet', '.@realDonaldTrump is strengthening our borders. Sanctuary cities defy the rule of law, put additional strain on our resources, &amp; endanger our brave law enforcement officers. We must continue to build the wall, support ICE, and welcome legal immigrants with open arms. #SOTU2020'), ('dates', '2020-02-05 02:48:33'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.6408706307411194)])
OrderedDict([('tweet', "Socialism is incompatible with the American way of life. It destroys the individual freedom &amp; liberty that make our country great. I'm proud to support American capitalism which has lifted millions out of poverty &amp; offers the opportunity to live the American dream. #SOTU2020"), ('dates', '2020-02-05 02:31:58'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.8930774927139282)])
OrderedDict([('tweet', "Under @realDonaldTrump's leadership, we secured a new free trade agreement with our North American allies &amp; let Americans keep more of their hard-earned money. The American economy is working better than ever for ALL Americans. #SOTU"), ('dates', '2020-02-05 02:12:44'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.6102222204208374)])
OrderedDict([('tweet', 'On the heels of the Dow Jones closing over 29,000 for the first time in history, @realDonaldTrump just signed phase one of a major trade deal with China that will help secure our intellectual properties and grow our economy even further! #ChinaTradeDeal'), ('dates', '2020-01-15 21:45:11'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.7704607844352722)])
OrderedDict([('tweet', 'Now that @SpeakerPelosi has finally agreed to transmit her bogus #impeachment articles to the Senate, I hope the House can now tackle the many real issues facing the American people. #ShamImpeachment'), ('dates', '2020-01-15 18:58:53'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.9830307364463806)])
OrderedDict([('tweet', 'I was in the same briefing as you, @RepJayapal, and this is absolutely false. You and your squad of Ayatollah sympathizers are spreading propaganda that divides our nation and strengthens our enemies. #Iran https://t.co/iJJfZAtOTp'), ('dates', '2020-01-08 21:41:42'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.7583099603652954)])
OrderedDict([('tweet', 'Americans at home and abroad are safer today because our adversaries know @realDonaldTrump will strongly respond to any acts of violence against U.S. citizens. #Iran https://t.co/rWLr4bvse5'), ('dates', '2020-01-08 17:31:04'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.7022759914398193)])
OrderedDict([('tweet', 'Putin is definitely laughing at us. This division and political hatred is exactly what he wanted. And yet, Democrats push forward their partisan impeachment of @realDonaldTrump. https://t.co/eREq9PJgul'), ('dates', '2019-12-18 18:40:14'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.8115219473838806)])
OrderedDict([('tweet', 'This week, Congress considers H.R. 3, a partisan bill that would put the federal government in charge of setting prescription drug prices. H.R. 3 jeopardizes American innovation &amp; patient access to care. In fact, the CBO estimates we could see 15 fewer cures over the next decade.'), ('dates', '2019-12-10 18:02:04'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.9388167858123779)])
OrderedDict([('tweet', '.@SpeakerPelosi just announced that yesterday\'s hearing with "leading American constitutional scholars" gives Congress no choice but to impeach @realDonaldTrump. But let\'s take a look at who exactly testified yesterday at the #ImpeachmentHoax ...'), ('dates', '2019-12-05 17:33:29'), ('file_name', 'John H. Rutherford.csv'), ('pct_true', 0.7132585048675537)])
OrderedDict([('tweet', 'Today @RepFletcher &amp; Rep. @michaelcburgess intro’d the House bill of our SPR legislation. This leg. to purchase U.S. crude oil supports a crucial industry, its workers &amp; the American taxpayers who will benefit from greater economic &amp; national security. https://t.co/QErc0vkw5r'), ('dates', '2020-04-21 22:56:15'), ('file_name', 'John Hoeven.csv'), ('pct_true', 0.7457053661346436)])
OrderedDict([('tweet', 'Made the case to @ENERGY @SecBrouilette for #NDenergy priorities - maintaining pressure on the Saudi regime to end its oil price war, reversing the Washington State crude-by-rail law &amp; ensuring a fair value for coal as reliable baseload power.'), ('dates', '2020-03-28 13:04:14'), ('file_name', 'John Hoeven.csv'), ('pct_true', 0.6705822348594666)])
OrderedDict([('tweet', 'We need stability in the oil markets during these challenges with COVID-19 to ensure we don’t lose long-term capacity in this essential industry. That’s why we’ve been pressing to end Saudi’s price war with Russia, including in today’s call with the Saudi Ambassador…'), ('dates', '2020-03-18 20:53:43'), ('file_name', 'John Hoeven.csv'), ('pct_true', 0.5302686095237732)])
OrderedDict([('tweet', 'Voted to confirm Interior Deputy Secretary MacGregor today. Considering the vast amount of federal &amp; tribal land in #NorthDakota, having the right people in place at @Interior is key for advancing our state’s priorities.'), ('dates', '2020-02-25 22:34:44'), ('file_name', 'John Hoeven.csv'), ('pct_true', 0.6615452170372009)])
OrderedDict([('tweet', 'For weeks, the WHO allowed the Communist Party of China to lie to the world about #coronavirus. We must hold this organization accountable for its role in the coronavirus health and economic crisis. https://t.co/D0UEEkYlYS'), ('dates', '2020-04-21 00:31:05'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.9757722616195679)])
OrderedDict([('tweet', 'Nearly 70,000 small businesses in PA have been approved for #PaycheckProtectionProgram loans - keeping hardworking Americans paid, supporting PA families, &amp; protecting our small businesses.\n\nDemocrats must stop blocking PPP funding. Workers &amp; small businesses are counting on us. https://t.co/AAhkO2v7kc'), ('dates', '2020-04-17 21:34:32'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.6296616792678833)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram already has helped 36,600+ of Pennsylvania’s small businesses - supporting hardworking families and safeguarding our economy. \n\nDemocrats must stop playing games with Americans’ paychecks. https://t.co/HanGHLoYna'), ('dates', '2020-04-16 15:11:20'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.5448270440101624)])
OrderedDict([('tweet', "36,600+ small businesses in PA have been approved for forgivable #PaycheckProtectionProgram loans.\n\nIt's despicable that Dems are blocking additional PPP funding, and I urge Speaker Pelosi &amp; Leader Schumer to stop playing games with Americans’ paychecks. https://t.co/TIZBsAbsHx https://t.co/iZExDUU4mB"), ('dates', '2020-04-15 21:30:36'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.8232218027114868)])
OrderedDict([('tweet', '.@POTUS @realDonaldTrump is 100% right to hold @WHO accountable for its gross mishandling of the #coronavirus outbreak. While the WHO propagated lies from the Chinese Communist Party, we lost valuable time &amp; opportunity to save lives. https://t.co/cdkysRPjnz'), ('dates', '2020-04-14 22:57:40'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.8366895914077759)])
OrderedDict([('tweet', "It’s despicable that Democrats blocked funding to expand the #PaycheckProtectionProgram &amp; help our small businesses get the help they need to pay hardworking Americans. \n\nThis is not the time for politics - it's time for us to work together to help our nation weather this storm."), ('dates', '2020-04-09 20:16:49'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.7972403168678284)])
OrderedDict([('tweet', 'The Chinese government must be held accountable for its role in the #coronavirus pandemic. Proud to join with @RepJimBanks and bipartisan Members of Congress to condemn China for its gross mishandling of this crisis. https://t.co/73FD1ATuzq'), ('dates', '2020-03-26 19:12:26'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.6390448808670044)])
OrderedDict([('tweet', 'Unfortunately, bad actors are taking advantage of the #coronavirus outbreak and targeting Americans with scams online and over the phone. Get the latest tips from the @FTC to guard against these malicious scams. https://t.co/INniP6anXJ'), ('dates', '2020-03-24 23:13:29'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.5290191769599915)])
OrderedDict([('tweet', 'The second failed vote in the Senate on a coronavirus relief package is more proof that Speaker Pelosi &amp; Senator Schumer put politics before the American people. This legislation is needed now to help millions of families &amp; small businesses impacted by the coronavirus outbreak.'), ('dates', '2020-03-23 22:23:06'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.6619969606399536)])
OrderedDict([('tweet', "It's unconscionable that Speaker Pelosi would play politics &amp; walk away from bipartisan talks with the Senate on coronavirus aid. Pennsylvanians need results to help families &amp; small businesses navigate this challenge.\n\nAmericans are counting on Congress to get this done."), ('dates', '2020-03-23 12:38:46'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.881571352481842)])
OrderedDict([('tweet', "Disappointed that @HouseDemocrats once again blocked the #BornAlive Abortion Survivors Protection Act. Ending infanticide shouldn't be a political issue - it shouldn't even be a question. \n\nWe won't stop fighting to protect the lives of children who are born alive. https://t.co/4L0I4ebv7Z"), ('dates', '2020-02-28 18:50:23'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.5980295538902283)])
OrderedDict([('tweet', "Today I voted to condemn Sen. Bernie Sanders' disgraceful comments praising Fidel Castro and his communist regime in Cuba. Our nation is facing a stark choice between freedom and oppression. The American people should know where we stand  – with freedom or with Senator Sanders. https://t.co/kIdnacjnAg"), ('dates', '2020-02-27 20:44:17'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.9290414452552795)])
OrderedDict([('tweet', 'A "one-size-fits-all," government-run health care system would eliminate patients\' choices and rob families of innovative cures. America simply can\'t afford House Democrats\' radical, socialist agenda. https://t.co/rVCi5bKQXj'), ('dates', '2020-02-26 17:39:37'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.5832268595695496)])
OrderedDict([('tweet', 'It is shameful that 41 Democratic Senators voted against this commonsense legislation to protect children who are #BornAlive - and the Speaker continues to shut down @HouseGOP requests for a vote in the House. \n\nAll human life is worthy of dignity and our protection. https://t.co/JzryYdImNk'), ('dates', '2020-02-26 15:51:38'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.6592763662338257)])
OrderedDict([('tweet', 'From the $93 trillion Green New Deal to "one-size-fits-all" Medicare-for-None, the Left is peddling a radical agenda that Pennsylvanians don\'t want and can\'t afford. \n\nThis is just the beginning of their socialist agenda. https://t.co/ccc28Gnckc'), ('dates', '2020-02-25 13:42:46'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.5658724308013916)])
OrderedDict([('tweet', "Today marks the end of the long and shameful path of House Democrats' #ImpeachmentSham. After months of investigations conducted at a steep cost to the American people, President @realDonaldTrump has been acquitted for life. \n\nIt's time to move forward. https://t.co/am7wiH9sZA"), ('dates', '2020-02-05 22:18:34'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.7108821272850037)])
OrderedDict([('tweet', '"One-size-fits-all" health care is one-size-fits-NONE. A radical, socialist health care agenda that stifles lifesaving innovation is the WRONG path for our nation. #SOTU #MedicareForNONE'), ('dates', '2020-02-05 02:46:55'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.8566539883613586)])
OrderedDict([('tweet', 'Here’s the bottom line: Rather than pursue the truth, Democrats have been obsessed with impeaching President @realDonaldTrump because they do not like him and do not respect the legitimate votes of those who elected him. The American people rightfully see through this sham. https://t.co/Jta8Gw0iBM'), ('dates', '2020-01-25 21:41:10'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.9232425093650818)])
OrderedDict([('tweet', 'Time and time again, House Democrats prove that their #ImpeachmentSham is - and always has been - motivated by petty politics. The American people deserve better. https://t.co/91Mj2FH1cL'), ('dates', '2020-01-23 03:00:43'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.565322756767273)])
OrderedDict([('tweet', 'Even as House Democrats continue their partisan impeachment sham, our @POTUS @realDonaldTrump is delivering on his promises to PA-13 and our nation. https://t.co/t5U9HhMTjI'), ('dates', '2020-01-15 18:30:51'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.8408636450767517)])
OrderedDict([('tweet', 'Our @POTUS @realdonaldtrump acted decisively to defend our nation and our servicemembers abroad from Iranian threats and provocations. It is concerning to see some House Democrats try to weaken our strategy of #PeaceThroughStrength when it is needed most. https://t.co/Mjc8sY1uKt'), ('dates', '2020-01-08 22:12:04'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.5396525859832764)])
OrderedDict([('tweet', 'It\'s been 20 days since House Democrats passed their shameful Articles of Impeachment.\n\nThe Speaker continues to keep these "urgent" Articles from moving to the Senate.\n\nTime and time again, Democrats prove that the #ImpeachmentSham is nothing more than a political stunt.'), ('dates', '2020-01-07 21:49:08'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.9749560952186584)])
OrderedDict([('tweet', 'On this dark day for our nation, I voted NO on the shameful Articles of Impeachment before the House. The American people aren’t fooled by House Democrats’ political attacks on our @POTUS @realDonaldTrump. https://t.co/GMIMG9wlue'), ('dates', '2019-12-19 01:58:36'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.9073255062103271)])
OrderedDict([('tweet', "This shameful #ImpeachmentSham never has been about facts. From the start, this process has been rooted in Democrats' politically-motivated efforts to sabotage our duly and democratically-elected @POTUS @realDonaldTrump. \n\nThe American people know the truth."), ('dates', '2019-12-18 18:05:20'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.9575785994529724)])
OrderedDict([('tweet', 'From the beginning of this #ImpeachmentSham, the only bipartisan vote has been against moving forward. \n\nThe American people see through this charade for what it is - a politically-motivated effort to remove our democratically and duly-elected @POTUS @realDonaldTrump from office. https://t.co/fAIHnu9HxG'), ('dates', '2019-12-17 22:59:21'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.7017630934715271)])
OrderedDict([('tweet', 'The Judiciary Committee’s advancement of two Articles of Impeachment - that lack specific crimes - on a party line vote is a new low for House Democrats. \n\nThis process has revealed the Democrats’ deep-seeded disdain for @realDonaldTrump and those who elected him.'), ('dates', '2019-12-14 00:43:50'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.8132904767990112)])
OrderedDict([('tweet', '#OTD 232 years ago, Pennsylvania became the second state to join the the Union. From the First Continental Congress to the Battle of Gettysburg, PA has been home to the events that have shaped our nation. Today, we celebrate Pennsylvanians’ legacy and service.'), ('dates', '2019-12-12 14:28:25'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.5162762403488159)])
OrderedDict([('tweet', 'Rather than point to facts, the sham impeachment inquiry has revealed the depth of House Democrats’ disdain for President Trump and their disregard of the legitimate votes of the American people, including the constituents of PA-13. Read my full statement: https://t.co/3R9SboaVQY'), ('dates', '2019-12-10 17:25:05'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.823096513748169)])
OrderedDict([('tweet', "Despite failing to meet their own requirements for an impeachment inquiry, House Democrats continue their obsessive pursuit to attack our duly and democratically-elected President. \n\nThe American people won't be fooled by this #impeachmentSHAM. https://t.co/o9cevVpaO1"), ('dates', '2019-12-09 16:04:20'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.9358313679695129)])
OrderedDict([('tweet', "During the Trump Administration, our nation has experienced record-breaking economic growth. Nearly two years after President Trump signed the Tax Cuts and Jobs Act into law, it's time for Congress to make these tax cuts permanent for the hardworking families of PA-13. #TCJA2Year https://t.co/s9uI4YfF5O"), ('dates', '2019-12-05 23:03:58'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.6517992615699768)])
OrderedDict([('tweet', 'Rather than secure the southern border, stop illicit drugs from harming our community, or pass the USMCA, House Democrats remain obsessed with impeaching our duly-elected President. \n\nThe #DoNothingDemocrats are squandering good opportunities to serve the American people.'), ('dates', '2019-12-03 22:51:38'), ('file_name', 'John Joyce.csv'), ('pct_true', 0.8043220043182373)])
OrderedDict([('tweet', 'It’s not the federal govt’s job to bail out local and state officials who spent recklessly.\n\nBUT giving those govts the flexibility to do maximum good for their communities—without putting taxpayers on the hook again—is a no-brainer.\n\nhttps://t.co/2eMzSrerFg'), ('dates', '2020-05-05 19:37:42'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.7279371619224548)])
OrderedDict([('tweet', 'Louisianians have lost their homes, businesses and health as a result of tornadoes that recently devastated Monroe and other communities. \n\nMy colleagues and I are asking POTUS to help hurting people get back on their feet by making a major disaster declaration. https://t.co/vATaE5GoBc'), ('dates', '2020-04-30 17:17:20'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.506826639175415)])
OrderedDict([('tweet', 'Glad to see @SEC_News publicly address the threat China’s emerging market poses to U.S. investors. \n\nEducating Americans about phony companies on our exchanges is good—but keeping deceitful companies out of our markets would be better.\n \nhttps://t.co/eL6HXXkWYV https://t.co/frQN0ea9C1'), ('dates', '2020-04-22 19:25:49'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.7036026120185852)])
OrderedDict([('tweet', 'Democrats’ delay on #SmallBiz funding is wildly irresponsible. \n\nThe preening elites in DC and NY think they’re smarter and more virtuous than the rest of America—but this is no time for them to push Pelosi’s radical agenda. \n\nNow is when we save lives and jobs. Focus up. https://t.co/pbu4cZtPr3'), ('dates', '2020-04-21 13:07:06'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.9141391515731812)])
OrderedDict([('tweet', "America has lost 22M jobs because of the #coronavirus pandemic, but we can save a lot if we refuel the #PaycheckProtectionProgram fast.\n\nIt's the worst possible time for Democrats to try to take advantage of the American people by tethering their pay to unrelated issues. https://t.co/gJIfkRrEFY"), ('dates', '2020-04-18 02:00:49'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.6015312075614929)])
OrderedDict([('tweet', 'Before Speaker Pelosi’s Democrats let the #PaycheckProtectionProgram SHUT DOWN yesterday, the Small Business Administration approved 26,635 loans for more than $5.1 billion in Louisiana alone. \n \nHow many jobs are we losing while Democrats refuse to fund #SmallBiz?'), ('dates', '2020-04-17 19:32:10'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.8332168459892273)])
OrderedDict([('tweet', 'Congress could pass clean #coronavirus relief bills . . . if Speaker Pelosi would agree to it. \n\nRight now, though, Democrats are sacrificing jobs by holding the #PaycheckProtectionProgram hostage. https://t.co/lj5jqmHsvQ'), ('dates', '2020-04-17 12:33:30'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.7972402572631836)])
OrderedDict([('tweet', 'Failing to refuel the #PaycheckProtectionProgram is a quick way to kill jobs and close down job creators for the long haul. \n\nCongress must inject an additional $250 billion into this program immediately—with no strings attached.\n\nhttps://t.co/1lYKoS59rr'), ('dates', '2020-04-16 23:02:35'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.5253714919090271)])
OrderedDict([('tweet', 'China’s leadership has a very casual relationship with the truth.\n\nChina has some explaining to do about the origin of the #coronavirus, and Congress and the entire world need to investigate this. https://t.co/dbPHgN4pUg'), ('dates', '2020-04-15 13:00:10'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.9576126337051392)])
OrderedDict([('tweet', '.@realDonaldTrump is right about the World Health Organization. If you turned President Xi upside down and shook him, the WHO would fall out of his pocket. \n\nChina lied, a lot of people have died, and the cover up is not right. https://t.co/ItFlwTbdXr'), ('dates', '2020-04-09 04:06:09'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.7527425289154053)])
OrderedDict([('tweet', 'Another Chinese company wildly exaggerated its success while being traded publicly in the U.S.\n \nThe Holding Foreign Companies Accountable Act would help protect American investors from this kind of flimflam as our people face a host of China’s lies.\n \nhttps://t.co/ExTT5M3u3z'), ('dates', '2020-04-02 21:33:37'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.804362952709198)])
OrderedDict([('tweet', 'Democrats took advantage of the #coronavirus pandemic to fund their pet projects and thought the American people wouldn’t notice—but they did. https://t.co/TzKt6hKTiO'), ('dates', '2020-03-29 20:22:45'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.7399635314941406)])
OrderedDict([('tweet', 'With this latest Democrat prank, Congress hit rock bottom and started to dig.\n\nWhat Speaker Pelosi and Sen. Schumer have done is breathtakingly irresponsible. Pelosi’s proposal is left-of-Lenin, and Dem obstruction has left Americans waiting for relief they needed yesterday. https://t.co/dMXAaN4i7k'), ('dates', '2020-03-25 02:19:10'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.939503014087677)])
OrderedDict([('tweet', "Unfortunately, China was more interested in its image than the world's health so they sat on the truth about the #coronavirus. \n\nI'm asking @realDonaldTrump to declare Louisiana a major disaster so that we can get major relief for workers and families. https://t.co/xRPZSGgvGJ"), ('dates', '2020-03-24 20:34:27'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.5368340611457825)])
OrderedDict([('tweet', 'This $31M grant from @fema brings resources to slow the spread of the #coronavirus.\n\nBut Louisiana workers need more help from Congress, yet Democrats still refuse to let us move forward. \n\nWhy? Democrats want wind &amp; solar tax credits more than jobs.\n\nhttps://t.co/n5Bt4ytEEJ'), ('dates', '2020-03-24 16:35:22'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.512239396572113)])
OrderedDict([('tweet', "Reality is calling, but Democrats keep hanging up. \n\nIn the middle of the #coronavirus health crisis, Democrats demand Congress offsets carbon emissions or else they'll strangle Americans economically. \n\nWe have days—not months—to get help to businesses &amp; workers. End the games. https://t.co/KZAxbdtWVi"), ('dates', '2020-03-24 01:24:52'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.8268822431564331)])
OrderedDict([('tweet', 'Democrats are holding funding for hospitals &amp; salaries for workers hostage while they try to resurrect the Green New Deal. \n\nOur priority should be people, but today Democrats are playing vampire politics—sucking hope &amp; help from Americans who need it now more than ever.'), ('dates', '2020-03-23 16:11:51'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.9660458564758301)])
OrderedDict([('tweet', 'Louisianians, like Americans, are victors. We are not victims.\n\nRight now, the Senate is working out a way to get money to businesses and people who are laid off to keep them going until our economy can get back to normal.  \n\nWe will get through the #coronavirus together. https://t.co/PiBfkmBMJY'), ('dates', '2020-03-21 01:48:54'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.6613706946372986)])
OrderedDict([('tweet', 'While @realDonaldTrump takes the bold step of filling up our strategic reserve as we fight the #coronavirus, we also need to remember the offshore oil workers and independent operators who support America’s economy as well as our energy independence.'), ('dates', '2020-03-19 19:48:30'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.6088678240776062)])
OrderedDict([('tweet', "Americans could be investing in China-based companies without realizing that that comes w/ the risk of fraud, espionage, human rights abuse—and funding China's Communist Party. \n\n@ChrisVanHollen &amp; I are asking the SEC to tell the public about these risks.\n\nhttps://t.co/ozwaH0HMbo"), ('dates', '2020-03-06 17:37:17'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.9677987098693848)])
OrderedDict([('tweet', 'A divided @FCC chose to give away $15B to foreign satellite companies when American taxpayers want that money for rural broadband.\n\nThese foreign satellite firms want all 4 feet and their snout in the taxpayer trough, but the FCC shouldn’t be helping them.\nhttps://t.co/wpUwRziYnU'), ('dates', '2020-02-28 18:02:55'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.6611142754554749)])
OrderedDict([('tweet', 'At 20 weeks in the womb, babies feel pain. \n \nThe Pain-Capable Unborn Child Protection Act would protect unborn children from painful, premature death.\n \nIt’s shameful that America is STILL 1 of the 7 countries that allow on-demand abortions after 20 weeks. #ProLife'), ('dates', '2020-02-25 22:14:48'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.60272616147995)])
OrderedDict([('tweet', 'Americans deserve to know when people lobbying Washington are corrupt. \n \nThe JACK Act has made it easy for Americans to find out when swamp creatures break the law. \n\nhttps://t.co/4X6YmWUIQ3'), ('dates', '2020-02-20 21:54:51'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.5365862846374512)])
OrderedDict([('tweet', 'Speaker Pelosi’s attempt to normalize #impeachment and to turn it into a routine political weapon was a mistake. \n\nIn fact, it was reckless. \n\nA country as great as ours deserves better, and so do her people.\n\nMy full statement: https://t.co/ktSQ3oyNTy'), ('dates', '2020-02-05 22:27:36'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.6476866602897644)])
OrderedDict([('tweet', "America is respected around the world again because the president is restoring a strong, decisive national defense. \n\nPeace often comes through strength, and America’s enemies know that they don’t get to write the rules any longer. \n\nAmerica doesn't dance to their tune. #SOTU"), ('dates', '2020-02-05 02:34:26'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.5996452569961548)])
OrderedDict([('tweet', "Reports of Bolton’s book don't answer the question: Why did POTUS ask for an investigation?\n\nThe Ukrainian government has been organically &amp; historically corrupt, &amp; Hunter Biden’s windfall from Burisma told the world America’s foreign policy can bought like a sack of potatoes. https://t.co/sIHbIFvtsK"), ('dates', '2020-01-30 15:38:43'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.8766942024230957)])
OrderedDict([('tweet', "Iran's Ahmad Hamzeh is taking up Soleimani’s mantle. \n\nAmerica should answer his threats against our president and citizens by adding him to the Specially Designated Nationals and Blocked Persons list.\n \nWe aren’t buying the poison Hamzeh’s selling. https://t.co/b1CnXSwDAk"), ('dates', '2020-01-28 23:01:50'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.5582907199859619)])
OrderedDict([('tweet', 'Yesterday, House Democrats showed more zeal than wisdom in their accusations of a cover-up. \n\nThe reality is both sides will be treated fairly in the Senate—equal time and opportunity for the prosecution and defense. https://t.co/VK1RFHj74C'), ('dates', '2020-01-22 15:26:57'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.5232028365135193)])
OrderedDict([('tweet', 'This weekend, the Taiwanese people—in the face of constant Chinese aggression and threats—showed a watching world what free and fair democratic elections look like. \n \nMay we listen to the Taiwanese voices rejecting the lies peddled by the Communist Party of China.'), ('dates', '2020-01-13 19:48:58'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.9190462827682495)])
OrderedDict([('tweet', 'America is not the bad guy here. Iran won’t stop its provocative acts. We just want Iran to stop trying to hurt us, stop trying to hurt our friends and stop trying to have a nuclear weapon. https://t.co/HQBGboQg05'), ('dates', '2020-01-10 13:39:28'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.9290347695350647)])
OrderedDict([('tweet', 'We were able to save American taxpayers billions of dollars by making sure the FCC held a public auction for C-band spectrum. A public auction, rather than a private sale, ensures the competition that our rural families depend on to access 5G. \nhttps://t.co/uZ5c4AJM87'), ('dates', '2019-12-08 15:30:00'), ('file_name', 'John Kennedy.csv'), ('pct_true', 0.5499370098114014)])
OrderedDict([('tweet', "There are forces in America today trying to take us back to another time and another place, but with the passage of this bill, we're saying we're not going back. We're going forward. #HR4 #VotingRightsAdvancementAct"), ('dates', '2019-12-06 16:36:01'), ('file_name', 'John Lewis.csv'), ('pct_true', 0.858898401260376)])
OrderedDict([('tweet', "There is no such thing as an illegal human being. We may have all come here on different ships, but we're in the same boat now."), ('dates', '2019-07-12 21:19:41'), ('file_name', 'John Lewis.csv'), ('pct_true', 0.7182316184043884)])
OrderedDict([('tweet', 'Combating climate change is not a Democratic or Republican issue. It is the question of preserving this little piece of real estate that we call Earth. Congress cannot stand on the sidelines. The time to act is now. https://t.co/1b798vhJUc'), ('dates', '2019-05-15 16:02:20'), ('file_name', 'John Lewis.csv'), ('pct_true', 0.8639023900032043)])
OrderedDict([('tweet', 'I am sick and outraged over the murder of journalist and American permanent resident Jamal Khashoggi. The President and his administration have failed to uphold their duty to the Constitution and to the rule of law by turning a blind-eye to this demonic act.'), ('dates', '2018-10-18 17:25:13'), ('file_name', 'John Lewis.csv'), ('pct_true', 0.5641502737998962)])
OrderedDict([('tweet', 'Have courage—raw courage—to stand up and speak up for those left out and left behind. #goodtrouble'), ('dates', '2018-07-10 14:30:57'), ('file_name', 'John Lewis.csv'), ('pct_true', 0.6406074166297913)])
OrderedDict([('tweet', 'I never thought I would live to see a day when the Supreme Court of the United States would again make a decision as inhumane as Dred Scott v. Sandford or Korematsu v. United States. #StandWithMuslims #NoMuslimBanEver'), ('dates', '2018-06-26 21:03:33'), ('file_name', 'John Lewis.csv'), ('pct_true', 0.6214747428894043)])
OrderedDict([('tweet', 'The frantic, desperate release of this memo leads us down a dangerous path of political spin and accusations instead of thoughtful, responsible, congressional oversight.'), ('dates', '2018-02-02 20:05:00'), ('file_name', 'John Lewis.csv'), ('pct_true', 0.5122144222259521)])
OrderedDict([('tweet', 'I never thought that I would witness the dismantling of the checks and balances of our country, and I never imagined that Republicans would cast aside the national security interests of our nation.'), ('dates', '2018-02-02 20:04:39'), ('file_name', 'John Lewis.csv'), ('pct_true', 0.6377660036087036)])
OrderedDict([('tweet', 'The @StateDept has issued a Level 4 Global Travel Advisory, which strongly cautions Americans abroad that if they do not make efforts to return to the U.S. in the immediate future, the ability of the State Department to provide repatriation assistance cannot be guaranteed.'), ('dates', '2020-04-02 02:05:06'), ('file_name', 'John P. Sarbanes.csv'), ('pct_true', 0.5708589553833008)])
OrderedDict([('tweet', 'Over the last several weeks, Congress has acted swiftly to address the coronavirus crisis by helping shore up America’s health care system and by providing financial support to American workers and small businesses impacted by the pandemic. #COVID_19'), ('dates', '2020-03-27 17:50:07'), ('file_name', 'John P. Sarbanes.csv'), ('pct_true', 0.7943318486213684)])
OrderedDict([('tweet', 'Every day that the Paycheck Protection Program goes without funding more people lose their livelihoods. The Democrats’ decision to play politics with small business funds is reprehensible. Fund #PPP now.  https://t.co/rlOUQjpJ9H'), ('dates', '2020-04-17 17:01:22'), ('file_name', 'John R. Carter.csv'), ('pct_true', 0.7556734085083008)])
OrderedDict([('tweet', "It's been 10 days since I called for more $ for the Paycheck Protection Program &amp; today #PPP ran out of money. Now is not the time for partisan games. Livelihoods hang in the balance. I urge my Democrat colleagues to stop the games,  pass a clean #PPP funding bill today. https://t.co/wAx51qyirk"), ('dates', '2020-04-16 16:01:36'), ('file_name', 'John R. Carter.csv'), ('pct_true', 0.6788948774337769)])
OrderedDict([('tweet', "The nuclear deal gave Iran, the leading state-sponsor of terrorism, billions of dollars and it used that money to terrorize people around the world. Iran's aggression &amp; hostility towards America &amp; Israel is why I rejected this bad deal when Pres. Obama proposed it 3 years ago."), ('dates', '2018-05-08 19:09:52'), ('file_name', 'John R. Moolenaar.csv'), ('pct_true', 0.5583677887916565)])
OrderedDict([('tweet', 'The House has passed a bill extending CHIP for 6 yrs. The extension is the longest extension in the history of CHIP and means more than 100,000 MI kids will have health insurance. The legislation keeps govt open, funds natl security, and pays our troops. Dems should support it.'), ('dates', '2018-01-19 01:04:38'), ('file_name', 'John R. Moolenaar.csv'), ('pct_true', 0.5996209383010864)])
OrderedDict([('tweet', 'FBI Director Wray is right: counterfeiting evidence to conduct illegal surveillance is utterly unacceptable.\n\nBad actors abused FISA tools to try taking down the Trump campaign and presidency. I trust Durham’s criminal investigation will provide accountability and justice. https://t.co/WqFOmPLKcC'), ('dates', '2020-02-10 16:16:31'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.5628237128257751)])
OrderedDict([('tweet', 'We warned them...As Schiff and House Democrats argue today for more witnesses and evidence, let’s remind everyone what we told them about the need for more witnesses and evidence the day Schiff rushed a vote on his “overwhelming” articles of impeachment. #badstrategy #backfired https://t.co/ZQWaBNNv4b'), ('dates', '2020-01-31 20:08:22'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.6737908124923706)])
OrderedDict([('tweet', 'His latest Trump frame job falling apart, Schiff tells Senators yesterday this impeachment is a clear case of “bribery.” He just neglected to mention why “bribery” got destroyed and then ditched over in the House. https://t.co/1zLVA6yNTQ'), ('dates', '2020-01-30 12:46:09'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.672925591468811)])
OrderedDict([('tweet', "Contrary to Democrats' “it’s debunked” assertion, Ambassador Yovanovitch admitted the Hunter Biden-Burisma pay-to-play board position was so concerning to the Obama administration that Burisma was the only Ukrainian company she was briefed on for her Senate confirmation. https://t.co/JvRRlNfkAz"), ('dates', '2020-01-27 23:05:00'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.925464391708374)])
OrderedDict([('tweet', 'The President’s defense is effectively unwinding how Democrats misrepresented the testimony of witnesses like Tim Morrison, who while listening in during the Trump-Zelensky call, heard no “demands”, no bribery, no extortion and nothing improper. https://t.co/CvVrwea3bE'), ('dates', '2020-01-27 21:31:48'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.8644524812698364)])
OrderedDict([('tweet', 'Every time things don’t go the Democrats’ way, they try to change the narrative and selectively leak information. No witness testimony can change the fact that this president was deprived of due process. https://t.co/yvyXADXVXU'), ('dates', '2020-01-27 18:35:29'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.8254068493843079)])
OrderedDict([('tweet', 'During the first three days of the Senate impeachment trial, Democrats told the American people that their star witness, Ambassador Sondland, unequivocally confirmed under oath that there was quid pro quo.'), ('dates', '2020-01-26 16:10:25'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.9833089709281921)])
OrderedDict([('tweet', 'This impeachment fails factually, legally and constitutionally. I expect to hold all Republicans on the impeachment vote, and even pick up some Democrats. https://t.co/vYlGuCmnQK'), ('dates', '2020-01-23 02:50:22'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.7801313996315002)])
OrderedDict([('tweet', "The Democrats' arguments during the Senate trial have been a painfully dull rerun of the House proceedings. I'll be discussing more on impeachment with @TuckerCarlson at about 8:20 p.m. ET on @FoxNews."), ('dates', '2020-01-22 23:01:10'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.7098843455314636)])
OrderedDict([('tweet', "Obstruction of Congress? To even allege it, is an admission of constitutional illiteracy. The Founders had a term they used for the Democrats' obstruction of Congress. They called it: Separation of Powers. https://t.co/e8bKy6Cllb"), ('dates', '2019-12-19 01:50:04'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.6894769072532654)])
OrderedDict([('tweet', 'Acting as both judge and prosecutor, Democrats tell America they don’t need any more evidence or testimony before impeaching the President. So it’s fair to impeach him on that evidence and testimony—but it’s not enough  evidence and testimony for a Senate trial? Great strategy.'), ('dates', '2019-12-17 02:14:33'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.8174738883972168)])
OrderedDict([('tweet', 'The president took the unprecedented step of releasing a transcript so everyone could see the truth. \n\nHere’s another transcript everyone should see: sworn testimony confirming that the whistleblower didn’t tell the truth both verbally and in writing. But Schiff won’t release it. https://t.co/uRCvDPK1rg'), ('dates', '2019-12-12 17:35:57'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.7723702192306519)])
OrderedDict([('tweet', "Alexander Hamilton said the greatest danger of impeachment would be depriving a president of due process and using impeachment politically by the party that has the most votes in the House. Sadly, that day has come. Today's Democrats are the Founders' worst nightmare. https://t.co/tIGs8F6QA1"), ('dates', '2019-12-12 15:45:03'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.7380388975143433)])
OrderedDict([('tweet', 'The IG’s role is to find facts, not to necessarily make conclusions. The facts in Horowitz’s report say that the FBI’s senior leadership kept an investigation going based on an entirely unreliable dossier even though they knew they didn’t have a valid basis to do so. https://t.co/cTdLi6VaGS'), ('dates', '2019-12-10 00:36:28'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.6745366454124451)])
OrderedDict([('tweet', 'Democrats want to impeach Trump without allowing a single question to be asked of a single witness about why IG Atkinson wasn’t told the truth about contacts between the whistleblower and Schiff. They’ll be held accountable for this by their voters next November. https://t.co/ax1Gy3KsRC'), ('dates', '2019-12-09 22:56:54'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.9737939834594727)])
OrderedDict([('tweet', 'Speaker Pelosi announces that Democrats will move forward with articles of impeachment just days before the IG is set to release his findings regarding allegations of surveillance abuses against the Trump campaign. Not a coincidence.\nhttps://t.co/QpSawxMSRu'), ('dates', '2019-12-06 01:34:48'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.7521181702613831)])
OrderedDict([('tweet', 'Professor Turley, who voted against Trump in 2016, confirmed under oath today from a legal perspective that there was no bribery, no extortion, no obstruction of justice and no abuse of power. What false charge can Democrats possibly pursue next? https://t.co/pxSazjKFyA'), ('dates', '2019-12-04 22:29:35'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.5881741046905518)])
OrderedDict([('tweet', 'Sondland admits Trump told him “I want no quid pro quo.” Also admits his opinion that there was quid pro quo is based on a “presumption” – and is directly contradicted by Volker and Morrison. Reasonable people having different conclusions isn’t compelling evidence to impeach. https://t.co/aYaxH7O2xc'), ('dates', '2019-11-20 20:43:59'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.8080905079841614)])
OrderedDict([('tweet', 'Why won’t Schiff allow Hunter Biden to testify before his Committee, even after Yovanovitch confirmed under oath today that his involvement with Burisma posed a legitimate concern? https://t.co/XqEUhz6iLh'), ('dates', '2019-11-15 21:52:48'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.6729261875152588)])
OrderedDict([('tweet', 'Listen to the liberal media and House Dems change their story after Taylor’s testimony is clear that nobody in the Ukrainian government was aware of a military hold at the time of the Trump-Zelensky call. Quid pro quo was impossible on July 25, so let’s make it September instead. https://t.co/ZlqDm8NptX'), ('dates', '2019-11-06 21:09:23'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.8242806792259216)])
OrderedDict([('tweet', 'It’s more than just bias—it’s an actual legal conflict of interest. Schiff is using his authority as a Chairman presiding over an impeachment inquiry to prevent the investigation and discovery of facts about his own actions or the actions of his staff. https://t.co/7R8Xxdfj1A'), ('dates', '2019-11-01 15:53:28'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.755814790725708)])
OrderedDict([('tweet', 'This is impeachment effort 3.0 by Schiff. After twice falsely accusing the President of treason with Russia and obstructing justice, Democrats put Schiff in charge of impeachment 3.0 with someone who became a whistleblower after meeting with Schiff’s staff. https://t.co/9BNiFOuFbV'), ('dates', '2019-11-01 15:17:28'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.9549652934074402)])
OrderedDict([('tweet', 'Looking forward to discussing the impact of Tim Morrison’s testimony on the Democrats’ impeachment inquiry today at 9 a.m. ET on Fox News.\n\nHis testimony was devastating to the false narrative that anything illegal or improper occurred on the President’s July 25 call.'), ('dates', '2019-11-01 11:45:25'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.6958683729171753)])
OrderedDict([('tweet', 'Morrison’s testimony today is devastating to the false Democrat narrative that anything illegal or improper happened on the July 25 Trump-Zelensky call. It’s also devastating to the credibility of certain other witnesses who have asserted there was anything illegal or improper.'), ('dates', '2019-10-31 18:03:51'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.9748655557632446)])
OrderedDict([('tweet', 'The Democrats will continue leaking snippets of testimony out of context to make it seem like something sinister or nefarious is happening in their impeachment inquiry, instead of letting the American people see it for themselves and make their own determinations. https://t.co/tG6PPPHMb4'), ('dates', '2019-10-18 15:49:05'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.9556683301925659)])
OrderedDict([('tweet', 'The American people should be insulted by the Democrats’ attempt to influence the outcome of the 2020 election through a sham "impeachment inquiry" that was designed as a smear campaign against President Trump. I support the administration\'s refusal to tolerate or participate... https://t.co/lvPbpzlawx'), ('dates', '2019-10-09 19:25:15'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.9382504224777222)])
OrderedDict([('tweet', 'Since the ICIG has no evidence whatsoever about the relationship and contacts between the whistleblower and Schiff’s staff, the investigation has to get that information from other fact witnesses. Who also happen to be running the investigation.'), ('dates', '2019-10-04 19:15:03'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.5213151574134827)])
OrderedDict([('tweet', 'So Schiff claims outrage and alleges a White House "cover-up" to not notify Congress of the whistleblower complaint when his own staff knew and advised the whistleblower before the complaint was filed? Yeah, sure, this isn’t a sham.  \n\nhttps://t.co/GE6bmDnXYB'), ('dates', '2019-10-02 19:49:27'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.5014433860778809)])
OrderedDict([('tweet', 'After the transcript showed nothing improper, President Zelensky said publicly there was nothing improper. Now Democrats want an investigation because a whistleblower with no first-hand knowledge knows better than Zelensky does about what he really meant to say. Absurd. https://t.co/e2CQFiQIVL'), ('dates', '2019-09-30 22:13:16'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.6472237706184387)])
OrderedDict([('tweet', 'The whistleblower was wrong to assert that the President’s actions constituted "a serious or flagrant problem, abuse, or violation of law or Executive Order.” \n\nI guess that’s what happens when you base a whistleblower complaint off secondhand information and media reports. https://t.co/C6z8Xx3eKE'), ('dates', '2019-09-26 18:52:49'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.5734023451805115)])
OrderedDict([('tweet', 'The Democrats continue to make false accusations about crimes that didn’t occur and promise evidence that doesn’t exist – and they can never deliver – this time at the expense of our national security. https://t.co/vHnhGQPVhC'), ('dates', '2019-09-25 17:13:44'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.8123216032981873)])
OrderedDict([('tweet', 'In their desire to undo the 2016 election and destroy President Trump, Democrats have today unequivocally and irreparably harmed our national security and compromised an important ally. Instead of forming an impeachment line, Democrats should consider forming an apology line.'), ('dates', '2019-09-25 14:38:57'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.9914308190345764)])
OrderedDict([('tweet', 'After changing the Judiciary Committee rules to circumvent the Constitution on the initiation of impeachment proceedings, Democrats are still failing miserably. #PartyofImpeachment https://t.co/1Eh3XIVSh9'), ('dates', '2019-09-17 20:23:34'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.5899255871772766)])
OrderedDict([('tweet', "Chairman Nadler's current argument on impeachment is entirely inconsistent with what he was saying back in 1998 with respect to the Clinton impeachment hearing. He wants to change the rules because the rules need to be changed to get Donald Trump. https://t.co/dPXBQxNVSy"), ('dates', '2019-09-12 16:34:00'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.628330647945404)])
OrderedDict([('tweet', 'Looking forward to joining @AmericaNewsroom on @FoxNews at 9 a.m. ET to discuss the confusion over the Democrats’ impeachment strategy, as the House Judiciary Committee holds a markup today to formalize procedures for an impeachment investigation. @BillHemmer @SandraSmithFox'), ('dates', '2019-09-12 12:07:07'), ('file_name', 'John Ratcliffe.csv'), ('pct_true', 0.7375767230987549)])
OrderedDict([('tweet', 'Joshua 8:35 There was not a word of all that Moses had commanded which Joshua did not read before all the assembly with the women the little ones and the strangers who were living among them.'), ('dates', '2018-03-02 12:22:43'), ('file_name', 'John Shimkus.csv'), ('pct_true', 0.5764610171318054)])
OrderedDict([('tweet', 'Leviticus 18:24 Do not defile yourselves by any of these things; for by all these the nation’s which I am casting out before you have become defiled.'), ('dates', '2018-02-21 12:53:00'), ('file_name', 'John Shimkus.csv'), ('pct_true', 0.5289446711540222)])
OrderedDict([('tweet', 'That a political operative could hire someone to compile a dossier, which is then used by federal law enforcement agencies to go to the FISA Court to spy on American citizens, is really scary. https://t.co/Fz0AmAXHob'), ('dates', '2018-02-09 15:53:08'), ('file_name', 'John Shimkus.csv'), ('pct_true', 0.5389047265052795)])
OrderedDict([('tweet', 'China should be held accountable for under-reporting &amp; lying about #COVID19 in their country. \n\nAt a minimum, the world should ban the use of Huawei technology. If countries use Chinese tech in their #5G networks, China will be able to surveil &amp; spy on people around the world. https://t.co/cLvp7AgrZx'), ('dates', '2020-05-07 17:08:44'), ('file_name', 'John Thune.csv'), ('pct_true', 0.9476156234741211)])
OrderedDict([('tweet', "Remember❗️there's no amount of federal spending in Washington that can substitute for a dynamic economy. We'll do whatever it takes to respond to this crisis &amp; get America back on its feet, but to fully recover, we need to address the health emergency &amp; safely reopen the economy. https://t.co/DSqW1LPIEO"), ('dates', '2020-04-27 21:55:15'), ('file_name', 'John Thune.csv'), ('pct_true', 0.5193039178848267)])
OrderedDict([('tweet', "Additional #PPP funding has finally passed the Senate. After Dems delayed funding for nearly a week &amp; openly put politics ahead of workers, they've changed course &amp; supported this #SmallBizRelief program. Now, this program is one step closer to helping more #SmallBiz stay afloat."), ('dates', '2020-04-21 21:53:50'), ('file_name', 'John Thune.csv'), ('pct_true', 0.6547973155975342)])
OrderedDict([('tweet', 'Question: What’s one way the world can hold China accountable for its role in spreading the #COVID19 pandemic?\n \nAnswer: By keeping Huawei equipment out of the world’s 5G networks.\n \n“There are no easy answers, but there are simple answers.” Ronald Reagan'), ('dates', '2020-04-18 15:22:36'), ('file_name', 'John Thune.csv'), ('pct_true', 0.8383438587188721)])
OrderedDict([('tweet', 'It’s crazy to me that Washington Democrats are playing politics with Americans’ lives. This is a crisis, and we need to put the fire out. https://t.co/sQgBz0Y5Mu'), ('dates', '2020-04-17 17:58:18'), ('file_name', 'John Thune.csv'), ('pct_true', 0.6854613423347473)])
OrderedDict([('tweet', '#China has been spreading misinformation about #COVID19 from day one. This should be a wakeup call to the world about how China operates: a complete lack of transparency and censorship of facts. This is going to take an international response to hold them accountable. https://t.co/pFvKHZKM2r'), ('dates', '2020-04-17 16:28:02'), ('file_name', 'John Thune.csv'), ('pct_true', 0.907209038734436)])
OrderedDict([('tweet', 'The #PPP is out of money. There’s a simple fix. Congress could increase funds allowing @SBAgov to reopen this program. It should be an easy vote. I hope Dems don’t force America’s small biz &amp; their employees to endure more anxiety &amp; uncertainty than they’re already feeling.'), ('dates', '2020-04-16 19:17:34'), ('file_name', 'John Thune.csv'), ('pct_true', 0.925434947013855)])
OrderedDict([('tweet', 'I’m having bad déjà vu. \nWas this really only three weeks ago?\nAnd now we’re back here *again* with Democrats blocking relief during a #GlobalPandemic. This isn’t an abstract policy debate – these are peoples’ jobs and Americans’ livelihoods.\nThat’s what we’re trying to protect. https://t.co/ldvpQp9MQO'), ('dates', '2020-04-16 17:09:10'), ('file_name', 'John Thune.csv'), ('pct_true', 0.5594662427902222)])
OrderedDict([('tweet', '22 million Americans have filed for unemployment since the #CoronavirusOutbreak hit our country. Hopefully, this serves as a stark wake-up call for Democrats who are standing in the way of additional funding for a program that’s helping save jobs &amp; keep small businesses open.'), ('dates', '2020-04-16 16:43:11'), ('file_name', 'John Thune.csv'), ('pct_true', 0.5507382154464722)])
OrderedDict([('tweet', 'This never needed to happen.\n\n@SenateGOP was ready to approve more relief funds last week — Dems stepped in to block.\n\n*Once again* Dems are holding important relief hostage to push other items. Small businesses are the backbone of our country, &amp; right now they need our help. https://t.co/9Op5PQj96W'), ('dates', '2020-04-16 15:19:24'), ('file_name', 'John Thune.csv'), ('pct_true', 0.7671607136726379)])
OrderedDict([('tweet', '#PPP is running out of money because so many people depend on it. It’s helped small biz across the country &amp; kept millions of Americans employed. All we have to do is add funding. That should be a simple vote. But Dems are objecting &amp; trying to push other items. We must act now. https://t.co/ECwDxCCpk5'), ('dates', '2020-04-15 23:35:20'), ('file_name', 'John Thune.csv'), ('pct_true', 0.562967836856842)])
OrderedDict([('tweet', 'Dems’ doomsday claims over the collapse of the internet when the @FCC removed “Net Neutrality” rules in favor of an open internet have, again and again, been proven false. Here’s yet another example👇🏻https://t.co/sPAWVNs1Kc'), ('dates', '2020-04-15 16:45:44'), ('file_name', 'John Thune.csv'), ('pct_true', 0.7704022526741028)])
OrderedDict([('tweet', '.@SBAgov estimates that 5.7 million American jobs have been saved by the #PaycheckProtectionProgram created by the Senate. The first loan was issued only 4 days ago. Let’s keep more Americans on the job. \n\nIf this program needs more funding, there’s bipartisan support for that. https://t.co/wJCphwEv20'), ('dates', '2020-04-07 22:47:55'), ('file_name', 'John Thune.csv'), ('pct_true', 0.61148601770401)])
OrderedDict([('tweet', '#JobsReport confirms #COVID is hitting American workers &amp; small biz hard. Many businesses are shutting down at no fault of their own, making it hard to pay workers. #SmallBiz relief loans enacted by Congress are now available. Info👇🏻 https://t.co/PYMsbsaHHZ'), ('dates', '2020-04-03 17:05:12'), ('file_name', 'John Thune.csv'), ('pct_true', 0.761488139629364)])
OrderedDict([('tweet', 'Many have been skeptical of China’s lack of transparency surrounding #COVID19, and new reporting confirms their concealment.\n\nOur focus is on our domestic response, saving lives &amp; helping American families. Accurate data informs these efforts. https://t.co/azayT0WbTf'), ('dates', '2020-04-01 18:26:05'), ('file_name', 'John Thune.csv'), ('pct_true', 0.622637927532196)])
OrderedDict([('tweet', 'Illegal robocallers are trying to take advantage of the coronavirus crisis. But the @FCC is taking important steps (required by my #TRACEDAct) that will help protect consumers from these bad actors now and in the future.'), ('dates', '2020-03-31 18:04:02'), ('file_name', 'John Thune.csv'), ('pct_true', 0.7566752433776855)])
OrderedDict([('tweet', 'Dems are calling the bill they’re blocking a “bailout” even though the businesses we’re trying to help are struggling due to no fault of their own – they were shut down – and employ MILLIONS of Americans. \n\nHow many Americans have to lose their jobs before Dems allow us to act? https://t.co/ZyufJsLZ9r'), ('dates', '2020-03-23 19:38:31'), ('file_name', 'John Thune.csv'), ('pct_true', 0.9955978989601135)])
OrderedDict([('tweet', 'Dems are saying over &amp; over this bill is a "bailout" for American businesses. It’s not. The money they’re talking about is in the form of *loans*. It has to be repaid. This is about Americans who are hurting. We’re in a position to do something about it. It’s time we do. https://t.co/oXaSqIXzR8'), ('dates', '2020-03-23 18:12:24'), ('file_name', 'John Thune.csv'), ('pct_true', 0.9584479928016663)])
OrderedDict([('tweet', "Last night was probably one of the saddest days I've seen in my time in the United States Senate. \n\nOur country is in a crisis, and the Democrats are still playing a political game. We don't have the luxury of time in this fight against COVID."), ('dates', '2020-03-23 15:53:20'), ('file_name', 'John Thune.csv'), ('pct_true', 0.5996227860450745)])
OrderedDict([('tweet', 'To recap: Democrats just blocked a bill that would provide direct cash payments to Americans who need it most – like hourly workers. \n\nNot to mention funding to help support health care workers who are on the front lines. \n\nI hope Dems can put politics aside and work with us.'), ('dates', '2020-03-23 01:32:28'), ('file_name', 'John Thune.csv'), ('pct_true', 0.8757444620132446)])
OrderedDict([('tweet', "Our hearts are with all people striving for freedom, especially those living under an oppressive Iranian regime that seeks the USA's demise. It's shameful that House Democrats refuse to consider @GOPLeader’s resolution, which I cosponsored, backing freedom for the Iranian people."), ('dates', '2020-01-15 21:33:05'), ('file_name', 'John W. Rose.csv'), ('pct_true', 0.8591487407684326)])
OrderedDict([('tweet', "I'm proud to cosponsor @RepMarkMeadows' resolution admonishing Speaker Pelosi for playing political games by not sending the articles of impeachment to the Senate. The constitution does not grant the Speaker the power to dictate the terms of a Senate impeachment trial."), ('dates', '2020-01-08 16:02:54'), ('file_name', 'John W. Rose.csv'), ('pct_true', 0.8161643147468567)])
OrderedDict([('tweet', 'This morning, House Judiciary passed both articles of impeachment against @POTUS. Disappointingly, this was a party-line vote. There has been no vote scheduled for the full House, but the people of #TN6 know where I stand. I support President Trump.\nhttps://t.co/JhghMasuD2'), ('dates', '2019-12-13 16:18:38'), ('file_name', 'John W. Rose.csv'), ('pct_true', 0.7655593156814575)])
OrderedDict([('tweet', 'Yesterday, @POTUS signed an Executive Order to strengthen protections for American Jews and express zero-tolerance for anti-Semitism. I stand with President Trump in condemning targeted attacks on American Jews and any acts of anti-Semitism, including those on college campuses.'), ('dates', '2019-12-12 19:12:11'), ('file_name', 'John W. Rose.csv'), ('pct_true', 0.8479070067405701)])
OrderedDict([('tweet', 'What we have seen today is not surprising. For three years, House Democrats have planned to impeach @POTUS. If the full House considers these Articles of Impeachment, I will be voting "no."\n\nRead my full statement here ⬇️\n\nhttps://t.co/JhghMasuD2'), ('dates', '2019-12-10 22:58:10'), ('file_name', 'John W. Rose.csv'), ('pct_true', 0.8372206687927246)])
OrderedDict([('tweet', "Today, the #DoNothingDemocrats are amplifying their #1 goal: impeach President Trump. They have focused on this for three years.\n\nHouse Dems should be focused on #USMCA, funding the government, and securing the border.\n\nYet, Speaker Pelosi's partisan investigation continues. https://t.co/Grd5O5iauU"), ('dates', '2019-11-13 20:47:20'), ('file_name', 'John W. Rose.csv'), ('pct_true', 0.6717310547828674)])
OrderedDict([('tweet', 'Speaker Pelosi has yet again betrayed the American people by championing policy that would move away from historic precedent &amp; enact unique constraints on Members in the minority during an impeachment inquiry. This #KangarooCourt is just another attempt to undo the 2016 election. https://t.co/gjvDXzrNeF'), ('dates', '2019-10-31 13:51:52'), ('file_name', 'John W. Rose.csv'), ('pct_true', 0.8936702013015747)])
OrderedDict([('tweet', 'House Dems are disregarding House rules and ignoring the safeguards and fairness we expect as Americans as they continue their efforts to undo the 2016 election of President Trump. They’re making decisions behind closed doors while hiding the facts, even from Members of Congress. https://t.co/9shKdXTHcw'), ('dates', '2019-10-22 22:40:39'), ('file_name', 'John W. Rose.csv'), ('pct_true', 0.6305374503135681)])
OrderedDict([('tweet', 'House Speaker Nancy Pelosi has now caved to the far-left members of her party in calls for an impeachment inquiry. The people of #TN6 can rest assured, though: I will not be distracted. I will continue working with our President to defend our #TennesseeValues. https://t.co/JpO48PixYQ'), ('dates', '2019-09-25 18:02:29'), ('file_name', 'John W. Rose.csv'), ('pct_true', 0.5840693116188049)])
OrderedDict([('tweet', 'The price of meat in grocery stores continues to go up while independent ranchers are seeing their margins go way down. This just doesn’t add up—that’s why I’m calling on @TheJusticeDept to open an investigation into price fixing in cattle markets. https://t.co/RhzuRUrfQE'), ('dates', '2020-04-27 17:42:23'), ('file_name', 'Jon Tester.csv'), ('pct_true', 0.6491320133209229)])
OrderedDict([('tweet', 'Montanans should be worried about keeping their families healthy, not standing in line at the DMV to fill out government paperwork. The deadline to obtain a REAL ID to fly commercially has been delayed—helping folks remain in compliance with the law while keeping families safe.'), ('dates', '2020-03-24 00:23:05'), ('file_name', 'Jon Tester.csv'), ('pct_true', 0.5757864713668823)])
OrderedDict([('tweet', 'When will Treasury and SBA release loan forgiveness rules for the #PaycheckProtectionProgram? \n\nWe’re now well into the 8-week period for many of these loans, and Iowa’s businesses and lenders are still in the dark on this.'), ('dates', '2020-05-07 00:19:25'), ('file_name', 'Joni Ernst.csv'), ('pct_true', 0.7095198035240173)])
OrderedDict([('tweet', 'Why in the world is the US gov’t going to invest our servicemembers’ retirement in companies in China? We need to put an end to that.\n \n@POTUS can take action ASAP to stop this, and I’m working with @MarcoRubio on a bill to ban this from ever happening again.'), ('dates', '2020-05-01 17:30:45'), ('file_name', 'Joni Ernst.csv'), ('pct_true', 0.6620761156082153)])
OrderedDict([('tweet', 'We can’t have teams like the Lakers getting slam dunks at the expense of small biz under the #PaycheckProtectionProgram. \n\nThankfully they gave the $$ back &amp; the admin is taking steps to ensure biz who are in need—like those here in Iowa—are prioritized. https://t.co/rYkMsljozL'), ('dates', '2020-04-30 16:08:04'), ('file_name', 'Joni Ernst.csv'), ('pct_true', 0.6107848882675171)])
OrderedDict([('tweet', '#COVID19 has caused Iowa’s colleges to dramatically change the way they teach our students.\n \nI spoke w/presidents of Iowa’s community colleges today about challenges they face. A common need: increased access to broadband – something I’ve been pushing for, &amp; will continue to.'), ('dates', '2020-04-28 00:01:45'), ('file_name', 'Joni Ernst.csv'), ('pct_true', 0.7099770903587341)])
OrderedDict([('tweet', 'American tax $$ shouldn’t be going to China’s wet markets. And they shouldn’t be going to China’s Wuhan lab either. \n\nToday, I joined my colleagues in calling to end this funding &amp; express my deep concern over past federal funding to the controversial Chinese state-run Wuhan lab. https://t.co/ElTGEPSgj6'), ('dates', '2020-04-22 20:35:19'), ('file_name', 'Joni Ernst.csv'), ('pct_true', 0.8947277069091797)])
OrderedDict([('tweet', 'Small businesses across Iowa are feeling the economic impact of COVID-19. Iowa’s local newspapers and radio and TV stations—which provide critical reporting for our communities—are facing those same challenges. They should benefit from the #PaycheckProtectionProgram as well.'), ('dates', '2020-04-21 15:33:19'), ('file_name', 'Joni Ernst.csv'), ('pct_true', 0.7481633424758911)])
OrderedDict([('tweet', '#COVID19 has harmed the renewable fuels industry right here in Iowa. There are those who would use this public health crisis as an excuse to undercut the #RFS. Our farmers &amp; biofuels producers need relief now – not another gut punch. https://t.co/iAbY72DrTK'), ('dates', '2020-04-17 14:50:13'), ('file_name', 'Joni Ernst.csv'), ('pct_true', 0.7907194495201111)])
OrderedDict([('tweet', 'The World HEALTH Organization has failed to hold Communist China accountable for covering up the largest global HEALTH pandemic in a lifetime.\n \nIowans deserve answers. The world deserves answers. I look forward to hearing from the @WHO Director-General. https://t.co/7sysD8SIS9'), ('dates', '2020-04-16 13:22:57'), ('file_name', 'Joni Ernst.csv'), ('pct_true', 0.8284256458282471)])
OrderedDict([('tweet', "We’re seeing alarming cases of people claiming to be the federal government, the IRS, or public health organizations to gain access to people's sensitive information. It's wrong - and it's dangerous.\n\nWe've compiled some FAQs to help protect #Roc families: https://t.co/hs4NCLchGy"), ('dates', '2020-05-04 23:10:02'), ('file_name', 'Joseph D. Morelle.csv'), ('pct_true', 0.5555328726768494)])
OrderedDict([('tweet', 'Americans need every penny in their stimulus checks. Reports that banks may be allowed to seize this money, meant to support struggling families, are deeply troubling. @RepJudyChu &amp; I are calling on Treasury to rectify this immediately. Read more here. ⬇️ https://t.co/TjAOiViUsz'), ('dates', '2020-04-16 19:47:35'), ('file_name', 'Joseph D. Morelle.csv'), ('pct_true', 0.542611300945282)])
OrderedDict([('tweet', 'Let’s set the record straight: we need Medicare For All and we need guaranteed access to counsel in civil cases. \n\nNot one or the other. \n\nBoth.'), ('dates', '2020-05-09 19:50:08'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.5680335760116577)])
OrderedDict([('tweet', 'Not a single American family should be forced to fight for their home against big banks without a lawyer by their side.'), ('dates', '2020-05-08 23:07:39'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.6189783811569214)])
OrderedDict([('tweet', 'It should surprise absolutely no one that GOP Senators are trying to demonize immigrants to distract from their leader’s failure to protect the American people. https://t.co/WRL2Tx1nyK'), ('dates', '2020-05-08 19:31:43'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.7598181962966919)])
OrderedDict([('tweet', "For a guy who has tried to erase President Obama's legacy at every possible turn, Trump reached a major milestone today. \n\nAll those jobs President Obama created are gone."), ('dates', '2020-05-08 18:22:36'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.7544811367988586)])
OrderedDict([('tweet', 'That this Administration would use a global pandemic that has killed over 70,000 Americans as cover to starve sexual violence survivors of justice should shock us. \n\nBut it doesn’t. \n\nBecause their mission has been to give power to the powerful all along. https://t.co/tGV1YRwa4l'), ('dates', '2020-05-07 19:19:16'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.5474060773849487)])
OrderedDict([('tweet', 'We can’t let anyone go without health care during this crisis, or any time going forward. \n\nThat’s why I’m joining @RepJayapal to introduce a bill that would expand Medicare to the millions left unemployed during this pandemic and cap out-of-pocket costs.'), ('dates', '2020-05-01 17:42:15'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.6387693881988525)])
OrderedDict([('tweet', "A healthcare system more concerned with profits than patients was never equipped to confront a pandemic like COVID-19.\n\nBecause of our nation's failure to guarantee universal healthcare, millions of people are out of jobs and health care as coronavirus ravages their communities."), ('dates', '2020-05-01 15:00:52'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.5637624263763428)])
OrderedDict([('tweet', '60,000 Americans have died due to this virus.\n\nInstead of offering an ounce of empathy to their families, this President has decided there’s only one real victim here.\n\nHimself.'), ('dates', '2020-04-30 20:07:29'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.678395688533783)])
OrderedDict([('tweet', 'Over 60 people have contracted coronavirus directly linked to voting in Wisconsin.\n\nThis is a public safety and civil rights issue. We can’t force people to choose between exercising their constitutional right to vote and protecting their health.\n\nWe need national vote by mail.'), ('dates', '2020-04-30 16:20:30'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.8277077674865723)])
OrderedDict([('tweet', 'Our nation’s largest financial institutions cannot be allowed to hide behind profits and politics as small businesses suffer. \n\nIf banks are prioritizing large loans over small businesses that are struggling to survive, the American public has a right to know.'), ('dates', '2020-04-24 20:35:31'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.8918660283088684)])
OrderedDict([('tweet', '50,000 Americans have died in a global health pandemic. \n\nAnd this hateful Administration is dedicating their time to denying LGBTQ patients access to health care. https://t.co/L25LTQEsdm'), ('dates', '2020-04-24 16:28:36'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.7264678478240967)])
OrderedDict([('tweet', "Small business owners with criminal records should not be denied critical relief to keep their businesses afloat.\n\nThat's why @RepBeatty &amp; I are calling on Congressional leadership to Ban The Box for small biz funding. https://t.co/T7aPvaEePW"), ('dates', '2020-04-22 18:28:01'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.5695171356201172)])
OrderedDict([('tweet', 'COVID-19 didn’t target communities of color and low-income families. Our policies did. \n\nThis Earth Day we need to fight for environmental justice for the frontline communities that bear the brunt of our inaction on climate change.'), ('dates', '2020-04-22 16:49:59'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.5454753637313843)])
OrderedDict([('tweet', 'Combatting stubborn educational inequities requires bridging the digital divide that leaves too many students behind, not just during a global health pandemic, but every single day.'), ('dates', '2020-04-21 17:02:21'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.8639792799949646)])
OrderedDict([('tweet', 'He rips infants from their arms. He sends them home to certain death. \n\nHe tears their families apart, exploits their labor, savages their livelihoods, and now --\n\nHe blames immigrants for a pandemic HE chose to ignore before it was too late.'), ('dates', '2020-04-21 14:38:25'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.5757264494895935)])
OrderedDict([('tweet', 'Justice in this country demands a reckoning with our past. \n\nThat’s why we must not just legalize marijuana but expunge past possession records and provide clemency. \n\nNow.'), ('dates', '2020-04-20 18:29:18'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.6481584310531616)])
OrderedDict([('tweet', 'We need to Ban The Box for Covid-19 small business loans.\n\nNo small business owner should be denied critical relief because of a past offense as minor as marijuana possession.'), ('dates', '2020-04-17 16:20:04'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.6673504710197449)])
OrderedDict([('tweet', 'Small businesses will not survive unless we fix the Treasury Department’s flawed implementation of relief loans and the barriers this Administration has built. \n\nAny future COVID-19 package must focus on getting money directly to small businesses and their workers. https://t.co/E1SzU3TpNI'), ('dates', '2020-04-15 22:07:50'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.7398693561553955)])
OrderedDict([('tweet', 'Harvard researchers are projecting that we may have to practice social distancing until 2022.\n\nThere is absolutely no reason we should not institute national vote by mail right now.\n\nThis is not a blip in the radar. It’s our new normal. Let’s be prepared. No debate. Vote by mail.'), ('dates', '2020-04-15 14:36:33'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.5409592986106873)])
OrderedDict([('tweet', 'Only this President would try to solve one pandemic by leaving us unprepared for the next one.'), ('dates', '2020-04-15 00:09:27'), ('file_name', 'Joseph P. Kennedy III.csv'), ('pct_true', 0.6267538666725159)])
OrderedDict([('tweet', 'We cannot let our local governments go bankrupt. NY and NJ are the states with the two largest #COVID19 caseloads in the country and we need federal assistance. (1/2)\nhttps://t.co/yNAkYIkVmi'), ('dates', '2020-04-28 15:02:17'), ('file_name', 'Josh Gottheimer.csv'), ('pct_true', 0.6953896880149841)])
OrderedDict([('tweet', 'THREAD:\n\n@senatemajldr is saying our local communities should go bankrupt — the same localities who employ 1000s of first responders &amp; front-line workers helping fight this pandemic.'), ('dates', '2020-04-23 21:37:35'), ('file_name', 'Josh Gottheimer.csv'), ('pct_true', 0.9310426712036133)])
OrderedDict([('tweet', 'On Yom HaShoah, we remember the six million Jews killed by the Nazis during the Holocaust. We must always stand strong against anti-Semitism, hatred, and intolerance in all forms. #NeverForget https://t.co/crKCb2kZGw'), ('dates', '2020-04-20 23:52:58'), ('file_name', 'Josh Gottheimer.csv'), ('pct_true', 0.7280048727989197)])
OrderedDict([('tweet', 'What are the @SBAgov and Treasury doing? The PPP program I voted to establish is for SMALL BUSINESSES. These big corporations should never have been eligible in the first place. \nhttps://t.co/LWEvjsCL4Y'), ('dates', '2020-04-29 19:00:01'), ('file_name', 'Josh Harder.csv'), ('pct_true', 0.7278791666030884)])
OrderedDict([('tweet', 'Today we remember the millions of Assyrians and Armenians who were killed in the genocide of 1915 and commit ourselves to never letting anything like it happen again.'), ('dates', '2020-04-24 22:15:00'), ('file_name', 'Josh Harder.csv'), ('pct_true', 0.6636741757392883)])
OrderedDict([('tweet', 'I couldn’t disagree with Mitch McConnell more. \n\nIf Congress doesn’t help, cities and counties may have to lay off firefighters and law enforcement. It’ll risk all of the services we rely on every day. \nhttps://t.co/qBd8M5wp0n'), ('dates', '2020-04-22 23:12:10'), ('file_name', 'Josh Harder.csv'), ('pct_true', 0.521580696105957)])
OrderedDict([('tweet', 'The world has been turned upside down by this pandemic – bureaucrats in Washington need to get with the times. @scoecomm has to put together this big report, &amp; that’s fine, but they should be given the time they need. We can’t afford to lose this funding.\nhttps://t.co/lmO6ME3PvS'), ('dates', '2020-04-07 21:00:00'), ('file_name', 'Josh Harder.csv'), ('pct_true', 0.5712594389915466)])
OrderedDict([('tweet', "The modern global economy weakens American workers and has empowered China’s rise. That must change. It's time to pursue policies that are good for American workers and American families. Abolishing the World Trade Organization is a good place to start. https://t.co/vFc2kY20Et"), ('dates', '2020-05-07 21:05:41'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.8138415217399597)])
OrderedDict([('tweet', "Coronavirus is not only a public health crisis. With 30M Americans unemployed, it's also an economic crisis. And it has exposed a hard truth about the modern global economy: it weakens American workers and has empowered China’s rise. That must change. https://t.co/Pcgp8ctkVd"), ('dates', '2020-05-06 22:06:54'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.7524305582046509)])
OrderedDict([('tweet', 'Senator Josh Hawley writes in @nytopinion: "There is no going back to the world before #COVID19. And in truth, we shouldn’t want to. That world had not worked for too many Americans for far too long." https://t.co/Pcgp8ctkVd'), ('dates', '2020-05-06 19:12:07'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.8161171078681946)])
OrderedDict([('tweet', "We’re not going to have an economy to open if nobody has jobs. Congress should be focused on getting Americans their jobs back. We don't have to continue down this path. https://t.co/ftOSR0EkKa"), ('dates', '2020-05-06 14:03:08'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.6433253884315491)])
OrderedDict([('tweet', 'The @UN decision to partner with Tencent, a glorified surveillance arm of Beijing, is stupid and dangerous.\n\nA new bill from Sen. Hawley &amp; @SenTedCruz would prevent US officials from using tech from companies that cooperate with the Chinese Communist Party https://t.co/vZ3fg8sT4g'), ('dates', '2020-05-04 20:08:33'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.919647216796875)])
OrderedDict([('tweet', "We shouldn't be doing state and local bailouts. Fiscally irresponsible states shouldn't be using this as an opportunity to get free money from the taxpayer — because it isn't free! Congress need to be focused on getting people back to work. https://t.co/1TtKCCiP37"), ('dates', '2020-05-04 14:31:43'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.7222000956535339)])
OrderedDict([('tweet', 'NEW THIS AM — Senators Blackburn, McSally, and Rubio have cosponsored Senator Josh Hawley’s bill to prohibit universities from receiving funds from the CARES Act if they have an endowment larger than $10B.\n\nReps. Jordan, Cline intro House companion bill. https://t.co/r1SlZ5p5ST'), ('dates', '2020-05-01 14:09:33'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.5073235034942627)])
OrderedDict([('tweet', 'The domination of a select few companies in the US meatpacking industry is a serious cause for concern. The consolidation of this industry has cause a supply chain problem. Monopoly is bad for farmers, bad for ranchers, bad for America. https://t.co/LIEWzFtl6Z'), ('dates', '2020-04-30 17:34:40'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.5162417888641357)])
OrderedDict([('tweet', "The coronavirus pandemic continues to highlight growing competition and security problems in our food supply. The recent closing of just three pork plants has shut down a staggering 15% of America’s pork production.\n\nIt's time the @FTC investigate. https://t.co/yNsBwS2Ya6"), ('dates', '2020-04-29 13:16:47'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.6449201703071594)])
OrderedDict([('tweet', 'These practices are alarming for America’s small businesses even under ordinary circumstances.\n\nBut at a time when most small retail businesses must rely on Amazon because of coronavirus-related shutdowns, predatory data practices threaten these businesses’ very existence. https://t.co/RmBPhKRlIV'), ('dates', '2020-04-28 15:14:53'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.8813954591751099)])
OrderedDict([('tweet', "Today, Senators Hawley and Blunt sent a letter to @USTreasury &amp; @SBAgov urging them to ensure that smaller colleges and universities won't be disqualified from receiving emergency PPP funding due to their participation in the federal work-study program. https://t.co/fKUgvUcpxr"), ('dates', '2020-04-27 22:47:49'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.5520132780075073)])
OrderedDict([('tweet', "Sen. Hawley has legislation to hold China accountable for silencing whistleblowers &amp; withholding critical information. \n\nIntro'd April 14, his bill would strip China of immunity in US courts and create a cause of action against the Chinese Communist Party. https://t.co/pjtpRP2j7i"), ('dates', '2020-04-27 22:06:27'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.9801326990127563)])
OrderedDict([('tweet', "Workers shouldn't be forced to bear the burden of this pandemic. They haven't done anything wrong.\n\nIn an interview with @MarkReardonKMOX, Hawley lays out the next steps Congress should take for coronavirus relief. It all starts with rehiring Americans who have lost their jobs. https://t.co/S4eUE5VFdr"), ('dates', '2020-04-27 20:35:03'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.5608446598052979)])
OrderedDict([('tweet', "China lied to the world.\n\nChina knew what they were doing.\n\nChina knew that this was going to be a pandemic.\n\nChina didn't take steps to stop it.\n\nChina needs to be held accountable. https://t.co/0c4KK1yPfs"), ('dates', '2020-04-24 19:39:45'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.9900141954421997)])
OrderedDict([('tweet', "Sen. Hawley's legislation would prohibit universities from receiving CARES Act funds if they have an endowment larger than $10 billion, unless they first spend some of their own money on coronavirus-related financial assistance for students. \n\nDetails here https://t.co/H7h0jELru8 https://t.co/RAzLepn59g"), ('dates', '2020-04-23 16:38:17'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.5355225801467896)])
OrderedDict([('tweet', 'Sen. Hawley has legislation to hold China accountable for silencing whistleblowers &amp; withholding critical information. His bill would strip China of its immunity in US courts and create a private right of action against the Chinese Communist Party. https://t.co/pjtpRP2j7i'), ('dates', '2020-04-21 14:30:06'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.9913837909698486)])
OrderedDict([('tweet', 'China covered up the origins of this pandemic, gagged doctors, forged statistics &amp; cost the world time that could have saved thousands of lives, trillions of dollars\n\nSen. Hawley is calling for the Beijing to be held liable for civil claims in U.S. courts https://t.co/EB3jBg3KGj'), ('dates', '2020-04-20 17:59:54'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.8706881999969482)])
OrderedDict([('tweet', '"We know the truth isn\'t that this was just some happenstance occurrence that China didn\'t know anything about," Hawley said. "We know they knew all about this, we know they suppressed the evidence..." https://t.co/yzWVDMcV7z'), ('dates', '2020-04-20 16:56:52'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.8181236386299133)])
OrderedDict([('tweet', 'Secretary of the Army tells Sen. Hawley there is no logical response to the irrational accusation that US brought coronavirus to Wuhan.\n\nBut the next logical step the US should take is holding China to account, and having them pay for the costs of their lies and deception. https://t.co/eJv4Er7MIo'), ('dates', '2020-04-16 17:01:18'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.5746966004371643)])
OrderedDict([('tweet', "Senator Hawley's new legislation would allow every citizen in this country who has been affected by the coronavirus to sue Beijing. The Chinese Communist Party must be held to account for its lies and deception. https://t.co/z2YMyJhCug"), ('dates', '2020-04-16 14:30:26'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.6799734830856323)])
OrderedDict([('tweet', 'China covered up the origins of this pandemic, gagged doctors, forged statistics &amp; cost the world time that could have saved thousands of lives, trillions of dollars\n\nSen. Hawley is calling for the Beijing to be held liable for civil claims in U.S. courts [https://t.co/btwrxri2DH'), ('dates', '2020-04-14 16:12:26'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.8929065465927124)])
OrderedDict([('tweet', "We need to know exactly what China knew and when it knew.\n\nThe Chinese Communist Party's attempts to hide the virus allowed it to spread and prevented the US and other nations from protecting ourselves sooner.\n\nIt is time to hold Beijing accountable. https://t.co/xazdtUd3KV"), ('dates', '2020-04-14 13:03:39'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.9887582063674927)])
OrderedDict([('tweet', "Because the government has taken the step of closing the economy to protect public health, Congress should in turn protect jobs in this country through this crisis.\n\nWe can't afford to tweak existing programs and hope for the best. We must think differently and be bold. https://t.co/mV8eyy45oK"), ('dates', '2020-04-10 17:06:55'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.6645240187644958)])
OrderedDict([('tweet', 'Because the government has taken the step of closing the economy to protect public health, Congress should in turn protect jobs in this country through this crisis.\n\nWe must get unemployment down NOW to secure workers and families, and help businesses get ready to restart. https://t.co/G2dReGO9qf'), ('dates', '2020-04-09 13:20:58'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.5246757864952087)])
OrderedDict([('tweet', "Where are our supply chains? — China\n\nOur medical devices, where are they made? — China\n\nOur pharmaceuticals? — China\n\nWho does Big Tech want to do business with? — China\n\nIt's time we ask what kind of economy have we allowed to be created &amp; what is it doing for American workers. https://t.co/9N1z0dqOGb"), ('dates', '2020-04-06 18:07:28'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.9226592183113098)])
OrderedDict([('tweet', 'Make no mistake: China is trying to turn the coronavirus pandemic to their advantage.\n\nSen. Hawley: "The need for us to laser focus on China’s economic and military ambitions is going to be more urgent once we beat this pandemic, not less.” https://t.co/1Babg1R5jN'), ('dates', '2020-04-06 14:48:45'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.8014583587646484)])
OrderedDict([('tweet', "Sen. Hawley lays out how Beijing is using the coronavirus crisis as an opportunity to further their quest for world power.\n\nAnd why it's time we fundamentally rethink our relationship with China &amp; start putting American workers first by bringing jobs &amp; manufacturing back home. https://t.co/HA1MoAXppP"), ('dates', '2020-04-02 15:44:06'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.724477231502533)])
OrderedDict([('tweet', "We need to know exactly what China knew, when it knew it &amp; how the Chinese Communist Party's decisions to try &amp; hide the virus allowed it to spread &amp; prevent the US &amp; other nations from protecting ourselves sooner.\n\nIt is time to hold Beijing accountable. https://t.co/xazdtUd3KV"), ('dates', '2020-03-31 21:23:17'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.9765510559082031)])
OrderedDict([('tweet', "Sen. Hawley's call for China to be held to account for its lies &amp; deception that helped unleash coronavirus on the world has growing support.\n\n@JohnCornyn, @SenTomCotton &amp; @SenMcSallyAZ have all joined his resolution to start an international investigation https://t.co/tSAp9rMihU"), ('dates', '2020-03-26 20:29:41'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.7077580094337463)])
OrderedDict([('tweet', 'We need an international investigation to make it clear to the world where this #coronavirus pandemic started.\n\nThe Chinese government lied about it, jailed the whistleblower doctors, and cost the world thousands of lives and billions of dollars.\n\nChina must be held responsible. https://t.co/zKtTBqxEOz'), ('dates', '2020-03-25 15:24:37'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.8270203471183777)])
OrderedDict([('tweet', 'Watch live as Senator Hawley joins @seanhannity shortly to talk about his resolution calling for a full international investigation into China’s cover-up in the spread of the coronavirus.'), ('dates', '2020-03-25 01:18:34'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.6236497759819031)])
OrderedDict([('tweet', "What do we know about the Chinese Communist Party's coronavirus cover-up in its early days?\n\n+ Suppressed the real news\n+ Silenced whistleblower doctors\n+ At least one of those doctors lost his life\n+ Delayed global response to this virus by weeks\n+ Thousands of lives lost https://t.co/NIUPwQelXq"), ('dates', '2020-03-24 22:14:48'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.8154874444007874)])
OrderedDict([('tweet', "It's time for an international investigation into China's lies about the origins of #coronavirus.\n\nChina should be held to account and they should foot the bill for what the world, including the United States, is now suffering. https://t.co/kRc4ILrfsX"), ('dates', '2020-03-24 17:08:43'), ('file_name', 'Josh Hawley.csv'), ('pct_true', 0.9134654402732849)])
OrderedDict([('tweet', 'Several million U.S. citizens reside in the territories  and too often these citizens are ignored in federal government responses. This has been particularly in the case of #PuertoRico. We must take care of ALL Americans.'), ('dates', '2020-03-19 22:28:25'), ('file_name', 'José E. Serrano.csv'), ('pct_true', 0.6494056582450867)])
OrderedDict([('tweet', 'Today, I voted in favor of the No War Against Iran Act &amp; repealing an outdated #AUMF Against Iraq Resolution of 2002. This would constrain the President’s military actions against Iran &amp; reassert Congress’s Constitutional authority to declare war. #NoWarWithIran #StopEndlessWar'), ('dates', '2020-01-30 21:12:49'), ('file_name', 'José E. Serrano.csv'), ('pct_true', 0.8803170323371887)])
OrderedDict([('tweet', 'American elections should be decided by Americans. Period.\nMy @HouseGOP colleagues need to decide whether it is ok to pressure foreign governments to undermine American democracy for personal political gain. #TruthExposed'), ('dates', '2019-10-10 16:56:05'), ('file_name', 'José E. Serrano.csv'), ('pct_true', 0.745585560798645)])
OrderedDict([('tweet', '#PuertoRico’s debt should not be paid on the backs of hardworking Puerto Ricans who are losing their jobs, retirement pensions &amp; homes as a result of the economic crisis, all while wealthy corporations &amp; individuals enjoy dubious tax breaks.'), ('dates', '2019-10-02 19:54:10'), ('file_name', 'José E. Serrano.csv'), ('pct_true', 0.9567514061927795)])
OrderedDict([('tweet', 'Meanwhile @HouseGOP @SenateGOP sit idly by as @realDonaldTrump sacks another IG because he doesn’t want anyone that exposes failures &amp; tells the truth.\n\nhttps://t.co/CAJOZjUu9j'), ('dates', '2020-05-02 12:53:19'), ('file_name', 'Joyce Beatty.csv'), ('pct_true', 0.7176175117492676)])
OrderedDict([('tweet', '.@GOP has no problems bailing out BIG business but reluctant to help cities, counties, &amp; smaller communities. #Priorities https://t.co/2iP9UgUnjt'), ('dates', '2020-04-30 18:00:00'), ('file_name', 'Joyce Beatty.csv'), ('pct_true', 0.6167765855789185)])
OrderedDict([('tweet', '22,000,000 Americans are unemployed and struggling to make ends meet, but @realDonaldTrump is putting his name on #COVID19 checks. #Priorities https://t.co/kMEY9dN2zB'), ('dates', '2020-04-16 17:30:00'), ('file_name', 'Joyce Beatty.csv'), ('pct_true', 0.739837646484375)])
OrderedDict([('tweet', 'Millions of Americans, including children, were denied stimulus checks because of their spouses’ or parent’s immigration status.\n\nThe administration cannot allow families to go hungry due to a discriminatory policy punishing citizens for loving or being born to an immigrant.'), ('dates', '2020-05-01 20:00:11'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.7929847836494446)])
OrderedDict([('tweet', 'Trump has a global responsibility to preserve policies that combat climate change.\nDenying the climate change crisis poses a serious threat to our youth and nation’s future.\nhttps://t.co/slD9MGjWkM'), ('dates', '2020-04-02 19:50:04'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.6887444257736206)])
OrderedDict([('tweet', 'We must #ProtectDueProcess and ensure #JudicialIndependence for immigration courts. The Trump Administration’s dangerous efforts to turn the system into a deportation machine undermines our global credibility and puts asylum seekers in danger. #CourtsInCrisis'), ('dates', '2020-02-19 20:17:00'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.8781991004943848)])
OrderedDict([('tweet', 'President Trump wants to spend $2 BILLION on his useless border wall. Meanwhile, the #TrumpBudget proposes cuts to education, Medicaid, SNAP, &amp; other programs that help Americans secure their future. This Administration has its priorities backward.'), ('dates', '2020-02-19 00:07:37'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.5511530637741089)])
OrderedDict([('tweet', 'The UN’s release of blacklisted companies is politically motivated and will only promote discriminatory boycotts.\n\nBDS’ objective is to eliminate the State of Israel; this is merely their first step. We must push back to prevent the demonization of our democratic ally, Israel.'), ('dates', '2020-02-13 20:33:23'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.7245252728462219)])
OrderedDict([('tweet', 'Over the past three years, President Trump has made every effort to further enrich the wealthy, while nearly 13 million Americans are working more than one job to make ends meet.'), ('dates', '2020-02-05 05:40:16'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.8042526245117188)])
OrderedDict([('tweet', 'President Trump has pushed forward unacceptable policies based on hatred and fear. \nThree years ago, the President kicked off his administration with an unconstitutional and discriminatory Muslim Travel Ban.'), ('dates', '2020-01-27 17:46:00'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.9067672491073608)])
OrderedDict([('tweet', 'President Trump has escalated violence in Iran without briefing Congress.  The Administration must brief Congress on their next steps to help protect the best interest of the American People, our troops, and national security.'), ('dates', '2020-01-03 23:31:57'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.815212607383728)])
OrderedDict([('tweet', 'One year ago, the American people sent House Democrats to Washington with one mission: Fight #ForThePeople. We had an impactful 2019 serving #CA51 constituents and look forward to continuing our fight in 2020. #YearInReview https://t.co/AK159VAJaU'), ('dates', '2020-01-03 18:36:18'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.6815572381019592)])
OrderedDict([('tweet', 'The Trump Administration owes Congress answers -- we need to know how many people endured #FamilySeparation and the plan to reunify every child with their loved ones. It’s time to end this cruel government-sanctioned child abuse. #HumanRightsDay\n\nhttps://t.co/1Am5h56fmb'), ('dates', '2019-12-10 19:10:09'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.6956512331962585)])
OrderedDict([('tweet', 'President Trump’s irrational act to remove our troops from Syria, is not only betraying and endangering our allies, but is putting our national security at risk by inviting a possible ISIS resurgence. https://t.co/u8dwH45W5N'), ('dates', '2019-10-17 17:46:26'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.6880034804344177)])
OrderedDict([('tweet', 'President Trump wants to discriminate against immigrant populations by denying green cards based on who they believe might need public assistance in the future as part of his #WarOnWorkingFamilies. https://t.co/5AWze9k7VH'), ('dates', '2019-10-08 17:01:52'), ('file_name', 'Juan Vargas.csv'), ('pct_true', 0.6015315055847168)])
OrderedDict([('tweet', "This is an incredibly irresponsible message on the day we posted our highest unemployment #s since the Great Depression. There's still a lot of May left. We are not through this crisis. Businesses need support now. Families are struggling now. We cannot abandon American workers. https://t.co/sQu4f6P91S"), ('dates', '2020-05-08 17:09:55'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.6517312526702881)])
OrderedDict([('tweet', "After 400 days of obstruction and lies, it's clear the President is disobeying the law &amp; hiding his tax returns because he wants to hide information from the public. It shouldn't take a court ruling for a President to be honest. https://t.co/XDu47zj8XW"), ('dates', '2020-05-08 14:50:12'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.6953216791152954)])
OrderedDict([('tweet', 'Cutting the payroll tax = cutting social security. \n\nPeople are suffering. We should not be robbing from their future. https://t.co/l8pJ130Fnw'), ('dates', '2020-05-06 14:08:20'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.5067920088768005)])
OrderedDict([('tweet', "The same ‘pro-life’ party that protects gun maker profits over children's lives also thinks some people should die so that some businesses can reopen. \n\nThis makes it clear that, when they say they are ‘pro-life’, they mean they’re just pro-controlling women’s bodies. https://t.co/SnUXCBesQy"), ('dates', '2020-05-04 16:06:24'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.8470006585121155)])
OrderedDict([('tweet', "Just as we saw with Puerto Rico, Donald Trump is happy to let people live or die based on whether or not he thinks they voted for him. A pandemic is no time to play politics with people’s lives. \n\nBy the way, California pays more to the Federal gov't than we receive. https://t.co/4YdT88MCb7"), ('dates', '2020-04-29 22:16:47'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.6338584423065186)])
OrderedDict([('tweet', "The PPP was created to help all businesses survive this crisis. But when Treasury opened lending up to the big banks, most small businesses got left out. Today, the @HouseSmallBiz committee is holding a hearing to hear from Members about how to get more aid where it's needed. https://t.co/NCRxfrUjQ7"), ('dates', '2020-04-23 15:49:15'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.588383138179779)])
OrderedDict([('tweet', 'Immigrant families must be able to obtain the care and help they need during this public health emergency without fear or confusion. The public charge rule and enforcement action around sensitive locations deter them from seeking medical care. #NationOfImmigrants'), ('dates', '2020-04-20 19:52:14'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.6168646216392517)])
OrderedDict([('tweet', "I'm glad Republicans are calling this out as well. Treasury opened PPP up to big banks w/out any rules for equitable lending. That's why, last week, House Dems sent a letter urging Treasury clarify that PPP should be available to ALL who need it, not just the biggest customers. https://t.co/PeyX518njF"), ('dates', '2020-04-20 19:38:52'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.607421875)])
OrderedDict([('tweet', "That's exactly what happened @Kasie. And it's why we're pushing SBA and Treasury to ensure banks lend to all businesses. The federal gov't assumes all risk for these loans so there's no reason to limit them to just the biggest businesses. https://t.co/xnKzBtoJI9 https://t.co/56BaP8chtu"), ('dates', '2020-04-17 20:16:17'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.5410745143890381)])
OrderedDict([('tweet', "Congress passed the PPP so that businesses could keep paying employees. But big banks turned many small businesses away, even their longtime customers. That's why today, we're urging @SBAJovita &amp; @stevenmnuchin1 to ensure equitable lending from banks! https://t.co/ywPk2Z5uel"), ('dates', '2020-04-16 21:52:36'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7057735919952393)])
OrderedDict([('tweet', "We don't have a border with China. Also, the fact that you can't distinguish between China &amp; Chinese AMERICANS puts Asian American lives at risk. @SpeakerPelosi stood with the #AAPI community as you were stoking xenophobia and downplaying the threat from coronavirus entirely. https://t.co/Q0OHATUsP8"), ('dates', '2020-04-16 19:44:15'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.9848323464393616)])
OrderedDict([('tweet', 'When coronavirus hit, people blamed Asian Americans. Chinese restaurants were the first to close. Physical and verbal assaults were rising. @SpeakerPelosi did the right thing and stood up to discourage xenophobia. The President would do well to follow her example, not mock it. https://t.co/3qvNbXkndp'), ('dates', '2020-04-16 13:48:50'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.925597608089447)])
OrderedDict([('tweet', 'Politicians are using pandemic fears to push an anti-abortion agenda, violating reproductive rights. The Supreme Court must protect access to time-sensitive, essential abortion. #AbortionisEssential https://t.co/5XVKrRVWbO'), ('dates', '2020-04-13 16:33:48'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7682427763938904)])
OrderedDict([('tweet', "Confused by this tweet without context? Well, today, the President rolled back clean emissions standards for vehicles - against even the wishes of car makers. But polluting the air is unpopular so he's doing what he does and is lying to you, claiming to have done the opposite https://t.co/Pe8onXn1L1"), ('dates', '2020-03-31 21:25:03'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.6857746243476868)])
OrderedDict([('tweet', 'Every time President Trump refers to the "Chinese virus," he is stoking anger and fear, which puts AAPI\'s at risk. My @MSNBC interview today on xenophobia and what more Congress can do to help us through this crisis. https://t.co/myKROvZBQT'), ('dates', '2020-03-31 15:28:21'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.6598886847496033)])
OrderedDict([('tweet', 'The President admitted on national TV today that part of his reelection strategy is by disenfranchising Democrats. In a democracy we should not fear letting the people – all the people – vote. https://t.co/6uXIwXrzGy'), ('dates', '2020-03-30 18:47:55'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.8278490900993347)])
OrderedDict([('tweet', "EPA's regulations are meant to keep people safe and healthy, particularly from respiratory illnesses. It's irresponsible &amp; unforgivable to use a pandemic as an excuse to make more people sick so oil companies can profit. https://t.co/xHrETMM6CM"), ('dates', '2020-03-27 15:01:25'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.8153160214424133)])
OrderedDict([('tweet', "Global &amp; US health experts say labeling a disease by geography is harmful. But it's clear this administration is willing to endanger Asian Americans, hamper the global response, and hold back the economy just to direct anger and blame away from Trump. https://t.co/2aY7yYL6Yz"), ('dates', '2020-03-25 17:39:52'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.528700590133667)])
OrderedDict([('tweet', 'Thank you President Trump for this important reminder.\n\nBut it wouldn\'t be necessary if you hadn\'t repeatedly rejected the advice of experts who said your use of "Chinese virus" was inciting violence &amp; prejudice. If you want to protect Asian Americans, stop using the term https://t.co/pz55fiqB60'), ('dates', '2020-03-23 21:54:51'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.6029530763626099)])
OrderedDict([('tweet', 'Every time Trump insists on using the term "Chinese virus," he is encouraging these exact types of attacks. He is inciting violence to avoid blame for his own failures. We warned that this is what follows when you reject expert advice &amp; blame whole groups: https://t.co/0PJW52LgaV'), ('dates', '2020-03-23 15:23:42'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.5192475318908691)])
OrderedDict([('tweet', 'Calling it “Chinese Virus”:\n❌ gets more people tested\n❌ provides hospitals with ventilators \n❌ prepares communities to respond \n❌ is a more accurate name\n✅ endangers Asian Americans by stoking fear and anger (according to the CDC) https://t.co/q96f4F5UXY'), ('dates', '2020-03-20 16:23:36'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.5218769311904907)])
OrderedDict([('tweet', "Our whole economy is being impacted by #coronavirus, &amp; so we need a response just as big. When Democrats next discuss our stimulus plans, I plan to bring up the need to support gig &amp; freelance workers too - like artists &amp; technicians who've lost work in the entertainment industry"), ('dates', '2020-03-20 12:23:40'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.6549782752990723)])
OrderedDict([('tweet', 'No matter how much he calls it the "Chinese virus," no matter how many hate crimes he incites, Donald Trump cannot hide from his own failure to address this crisis from the beginning. Bigotry will not produce more tests. Racism does not provide ventilators to hospitals.'), ('dates', '2020-03-19 15:52:38'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7504470348358154)])
OrderedDict([('tweet', 'You are the only ones trying to divide us. The WHO said don’t do this. The CDC said don’t do this. Your own HHS Secretary said don’t do this. You’re choosing something that experts say is wrong and dangerous because you WANT division &amp; bigotry. https://t.co/R2XN9Pp1WG'), ('dates', '2020-03-18 22:46:39'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7456873655319214)])
OrderedDict([('tweet', "They know it has a name. They're intentionally changing it now to a racist and xenophobic name because people are understandably concerned, so Republicans are trying to direct fear &amp; anger at Chinese people, not Donald Trump. https://t.co/UMeMRWNO1x"), ('dates', '2020-03-18 12:53:22'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7610747218132019)])
OrderedDict([('tweet', "It's not about China, it's about the Chinese people here - or even those who just look Chinese - who will be subjected to violence and prejudice. \n \nPointing to something from 100 years ago doesn't matter b/c naming conventions changed to avoid xenophobia &amp; improve public health https://t.co/QU8oOpEd93"), ('dates', '2020-03-18 12:52:34'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.8996318578720093)])
OrderedDict([('tweet', 'I\'m noticing that it\'s mostly white men who are so quick to tell us that "Chinese virus" isn\'t a racist term. Maybe because they\'re not on the receiving end of the violence and prejudice it incites.'), ('dates', '2020-03-17 21:24:28'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7460013628005981)])
OrderedDict([('tweet', 'Stop calling it Chinese Virus! By ignoring your own experts, you are putting people in danger from racism and violence. https://t.co/RU7JmHwFxk'), ('dates', '2020-03-17 00:16:29'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7183731198310852)])
OrderedDict([('tweet', 'This is right across from my office. Racism and xenophobia of this kind have absolutely no place in my district or our country. It’s exactly why I have demanded that our elected officials act responsibly and immediately stop calling COVID-19  the “Chinese” or “Wuhan” Coronavirus. https://t.co/y7LJU1XzKS'), ('dates', '2020-03-14 18:55:49'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7764216065406799)])
OrderedDict([('tweet', 'China didn\'t "unleash" anything. A virus spread, as viruses do. Blaming China &amp; insisting on calling this the "wuhan" virus even though every medical expert said not to, is putting people\'s lives in danger. Stop politicizing this &amp; put people first! https://t.co/RLM8SXJhdq'), ('dates', '2020-03-12 22:24:20'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.9894118905067444)])
OrderedDict([('tweet', "At this point, there's just no other way to say it. \nRepublicans are promoting racism. \nThey are putting people's lives in danger.\nThey have been warned by health experts not to. \nAnd they do not seem to care. \nThey are being reckless with people's lives. https://t.co/vxzE3Kaft2"), ('dates', '2020-03-12 15:24:59'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7657510042190552)])
OrderedDict([('tweet', "The House right now is debating my #NOBANAct to repeal President Trump's cruel #MuslimBans. But, true to form, Republicans are being dishonest about it. My bill in NO WAY hampers our #coronavirus response. In fact, there's explicit language about protecting public health."), ('dates', '2020-03-11 16:36:45'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7129674553871155)])
OrderedDict([('tweet', "HHS @SecAzar said don't do it. \nThe @CDCgov said don't do it.\nThe World Health Organization said don't do it all the way back in 2015. \nLabeling a virus by country/ethnicity puts people at risk. So my question to the Republicans is: Why are you choosing to ignore the experts?"), ('dates', '2020-03-11 16:17:45'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.5967058539390564)])
OrderedDict([('tweet', "This is a poor way to respond when told your words are putting people's lives in danger. @CDCgov and @HHSGov agree. And it's also not true. I sent @GOPLeader &amp; all members a letter urging them not to do this in February. But instead, he doubles down. https://t.co/hLMnh2EUY4"), ('dates', '2020-03-10 23:09:19'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.5614567399024963)])
OrderedDict([('tweet', 'Racist and wrong. Your own medical experts in your own government say that a wall would not stop #covid-19. They also say that calling it the "china virus", as below, puts people in danger. Listen to your experts. Keep people safe - don\'t stoke xenophobia. https://t.co/y6UuBzy1aH'), ('dates', '2020-03-10 17:36:44'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7008329033851624)])
OrderedDict([('tweet', 'Either Rep. Scalise has not read my bill, or he has and is being dishonest to you about it. Repealing the Muslim Bans would in no way impact our #Covid-19 response. My bill just requires that any bans be based on evidence (an actual pandemic), not prejudice ("ban all Muslims"). https://t.co/sQCFWgkH5l'), ('dates', '2020-03-10 17:10:29'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.8556152582168579)])
OrderedDict([('tweet', 'This is reckless. Why do you insist on disagreeing with HHS and CDC - both of which have said to not do this. “Stigma is the enemy of public health” – CDC. Your words do not help us understand this threat, they only serve to spread xenophobia and make all of us less safe. https://t.co/tckRbLh7nk'), ('dates', '2020-03-10 13:31:42'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.7699640989303589)])
OrderedDict([('tweet', 'ICE wants to "flood the streets" not in search of criminals and not in search of threats, but in search of immigrants. This is not about law enforcement, it\'s about terrorizing immigrant communities, which will leave us all less secure. Shame! https://t.co/ypqXzhQCgT'), ('dates', '2020-03-05 19:57:50'), ('file_name', 'Judy Chu.csv'), ('pct_true', 0.6442740559577942)])
OrderedDict([('tweet', '50 years ago, Americans celebrated the first #EarthDay in support of a healthier planet. As we celebrate #EarthDayatHome, we’re living through the consequences of not adequately preparing for a global health pandemic. We can’t make the same mistake with the #ClimateCrisis. https://t.co/fECS6fh5xi'), ('dates', '2020-04-22 15:54:07'), ('file_name', 'Julia Brownley.csv'), ('pct_true', 0.5898789763450623)])
OrderedDict([('tweet', 'There are many inaccurate rumors circulating on social media that seek to take advantage of the COVID-19 pandemic. Please beware of such scams!'), ('dates', '2020-04-16 22:11:52'), ('file_name', 'Julia Brownley.csv'), ('pct_true', 0.8162623047828674)])
OrderedDict([('tweet', 'In 2017, few believed that @realDonaldTrump could successfully negotiate new trade terms with China or renegotiate NAFTA to better serve America’s farmers and ranchers. \n\nToday, those critics have no ground to stand on. https://t.co/N2Vz75C1Sb'), ('dates', '2020-01-15 21:35:08'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.5967562198638916)])
OrderedDict([('tweet', 'For the last year, Democrats have focused on a partisan impeachment process, but @POTUS kept fighting for #USMCA. Today, we get the deal done. #USMCAnow https://t.co/7eJRy6Z9rj'), ('dates', '2019-12-19 20:10:22'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.7922847270965576)])
OrderedDict([('tweet', '"They hate this president. They hate those of us who voted for him. They think we\'re stupid. They think we made a mistake. They think Hillary Clinton should be president..." – \n@RepChrisStewart https://t.co/v8t0RGhuPE'), ('dates', '2019-12-18 21:41:38'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.930401086807251)])
OrderedDict([('tweet', '"Democrats understand they are offering the weakest case for impeachment since Andrew Johnson, that the public isn’t convinced, and so they simply want to get it over with." https://t.co/XDf3zAYQcu'), ('dates', '2019-12-13 14:49:11'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.9494408369064331)])
OrderedDict([('tweet', 'The Democrats’ obsession with impeachment has halted all normal legislative business. Congress is unable to perform our basic duties of appropriations, funding the military, and passing the #USMCA. The American people deserve better. It is time to get back to work.'), ('dates', '2019-10-23 21:42:30'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.9309329390525818)])
OrderedDict([('tweet', 'It is one thing for the Chinese Communist government to manipulate the lives of our farmers and ranchers for their political gain. It is another thing entirely for the United States Congress to do that to our own citizens.'), ('dates', '2019-09-19 14:47:24'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.8604309558868408)])
OrderedDict([('tweet', 'From today’s @HouseAgGOP hearing: It is shameful that Democrats have chosen to use farmers and ranchers as leverage simply because they don’t like this president.'), ('dates', '2019-09-19 14:46:40'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.897599458694458)])
OrderedDict([('tweet', 'Democrats are threatening to harm rural America in an attempt to damage @realDonaldTrump by preventing USDA from providing support to ag producers. They are playing political games w/ farmers in a time of unprecedented challenge. https://t.co/7dAPLbsYnJ'), ('dates', '2019-09-13 13:32:06'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.9596415758132935)])
OrderedDict([('tweet', '#USMCA ensures Canada will eliminate discriminatory grading of U.S. and eliminates country of origin statements on Canada’s quality grade certificates. @TexasWheat #Rally4USMCA #USMCAnow'), ('dates', '2019-09-12 14:43:32'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.9011125564575195)])
OrderedDict([('tweet', 'American farmers and ranchers across the country have praised #USMCA for its outstanding market access and modern, science-based sanitary and phytosanitary standards that are the strongest achieved in any trade agreement, which will keep our families safer. #USMCAnow #Rally4USMCA'), ('dates', '2019-09-12 14:33:57'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.9057432413101196)])
OrderedDict([('tweet', 'Democrats now have a choice. I hope they choose to work with us on policies that will benefit the American people, rather than falling further down the ridiculous rabbit hole of attacking the President and pursuing impeachment as their top priority.\n\nhttps://t.co/UDLOV7FXt1'), ('dates', '2019-05-23 13:21:40'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.9283614158630371)])
OrderedDict([('tweet', 'Anti-Semitism, racism, white supremacy, and hateful speech have no place in the House of Representatives. House Democrats must take bigotry of all kinds seriously, not just instances that fit their progressive liberal agenda. https://t.co/zgKdIsXu1Y'), ('dates', '2019-03-08 14:30:11'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.8663924932479858)])
OrderedDict([('tweet', 'Tax dollars should not serve as a campaign slush fund for politicians, which is why I am voting no on the Democrat Politician Protection Act #NOTforthePeopleAct #HR1'), ('dates', '2019-03-06 16:57:55'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.9201145768165588)])
OrderedDict([('tweet', "And @SpeakerPelosi won't even allow a vote to be held in the House. It's unfathomable that Democrats are okay with children being murdered after they are brought into this world https://t.co/Fh4QXypS5y"), ('dates', '2019-02-27 14:13:08'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.8393123745918274)])
OrderedDict([('tweet', 'For years, China has employed illegal trade practices that have driven jobs out of America and hurt our workers. The President is right – we must protect our workers by making fair trade a priority #SOTU'), ('dates', '2019-02-06 02:51:54'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.8518842458724976)])
OrderedDict([('tweet', 'The Chinese government’s objective is to drive American companies out of business and move their technology and jobs to China at any cost, including IP theft and illegal trade practices. https://t.co/DeyvSvynss'), ('dates', '2019-01-23 16:47:40'), ('file_name', 'K. Michael Conaway.csv'), ('pct_true', 0.7472707629203796)])
OrderedDict([('tweet', "Attorney General Bill Barr has corrupted our justice system by acting more like Trump's personal attorney than the nation’s chief law enforcement officer. I'll say it again: Barr must resign."), ('dates', '2020-05-09 19:22:03'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.752310037612915)])
OrderedDict([('tweet', "Trump is questioning the accuracy of the COVID-19 death toll because it ultimately highlights his failure of leadership during this pandemic. The American people deserve better than this.\n\nI'm pushing FEMA and HHS to ensure an accurate death toll. https://t.co/IlD0YSuCmf"), ('dates', '2020-05-08 02:06:33'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.5792992115020752)])
OrderedDict([('tweet', 'I’m tired of seeing large corporations bailed out with taxpayer dollars while our smallest businesses just down the street are on the brink of collapse.\n\nYesterday, I introduced the Saving Our Street Act to create a $125 billion fund to help micro businesses—not Wall Street.'), ('dates', '2020-05-07 20:54:33'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.555559515953064)])
OrderedDict([('tweet', 'The American people are tired of taxpayer funds going to big corporations while neighborhood businesses continue to struggle. My new bill is about saving the micro businesses just down the street—not Wall Street. https://t.co/e3gsZ5aHZ0'), ('dates', '2020-05-06 23:48:54'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.7704096436500549)])
OrderedDict([('tweet', "We're seeing a sharp increase in threats targeting Asian-American and other minority communities—no doubt fueled by Trump's xenophobic statements.\n\nThat's why @SenatorDurbin and I are pushing Senate leadership for emergency funding to address hate crimes during the pandemic."), ('dates', '2020-05-05 16:00:37'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.9831778407096863)])
OrderedDict([('tweet', "Here's how likely minority-owned businesses are to be DENIED from PPP loans:\n\nBlack-owned: 95%\nLatino-owned: 91%\nNative Hawaiian or Pacific Islander-owned: 91%\nAsian-owned: 75%\n\nThis is not what an equitable COVID-19 response looks like. I'm working to fix it."), ('dates', '2020-05-05 00:02:09'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.6176036596298218)])
OrderedDict([('tweet', "So we're clear about Mitch McConnell's priorities this week: he's pushing to confirm Justin Walker, a 37 y/o conservative ideologue, to the DC Circuit Court.\n\nAnd on top of that, we already know this nominee has attacked SCOTUS's ruling upholding the ACA. https://t.co/gms4c2HEPg"), ('dates', '2020-05-04 18:26:15'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.5198457837104797)])
OrderedDict([('tweet', 'Absolutely unacceptable that Trump is prioritizing bailing out Big Oil companies while dragging his feet to support millions of unemployed people, workers, small businesses, and state and local governments. The American people are tired of this.'), ('dates', '2020-05-04 01:51:17'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.9752285480499268)])
OrderedDict([('tweet', 'Right now, Attorney General Bill Barr has DOJ lawyers—who work for taxpayers and the federal government—in the Supreme Court trying to dismantle the Affordable Care Act. Remember that.'), ('dates', '2020-05-03 02:22:31'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.6942813396453857)])
OrderedDict([('tweet', "Private debt collectors should not line their pockets with people’s $1,200 direct payments while families struggle to feed their babies and pay the rent.\n\nGovernors are stepping up to prevent this—I'm pushing Secretary Mnuchin to prohibit this nationwide. https://t.co/5EJgJwtT5p"), ('dates', '2020-05-02 01:10:03'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.5002099871635437)])
OrderedDict([('tweet', 'Reminder that Mitch McConnell is bringing the Senate back into session next week with nothing on the schedule to address the coronavirus crisis. Instead, he wants to confirm more judges. In the middle of a pandemic.'), ('dates', '2020-04-30 18:07:27'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.5683243274688721)])
OrderedDict([('tweet', 'This is an outrage. More than 55,000 Americans have died from coronavirus and Senate Republicans still want to prioritize stacking our courts with ideologues. We won’t stand for this. https://t.co/04xpBJZZxe'), ('dates', '2020-04-29 16:52:30'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.8650227189064026)])
OrderedDict([('tweet', '26 million Americans have lost their income—no one should profit off their pain. Countless people are relying on credit cards just to put food on the table. We need a suspension of credit card interest, fees, and penalties—now.'), ('dates', '2020-04-28 16:16:45'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.9308564066886902)])
OrderedDict([('tweet', "Now is not the time to be giving away money to a small group of wealthy Americans. It's outrageous that Senate Republicans continue to support a tax break for this group as part of COVID-19 relief.\n\nI cosponsored a bill last week to repeal the tax giveaway."), ('dates', '2020-04-27 16:00:04'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.9416396617889404)])
OrderedDict([('tweet', 'Marijuana small businesses employ more than 240,000 workers and should be allowed to access coronavirus relief funds too. My colleagues and I are pushing to ensure they’re not left out of Congress’s next relief package.'), ('dates', '2020-04-24 15:27:02'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.5395880937576294)])
OrderedDict([('tweet', "People across the country are going hungry and debt collectors have the gall to try to seize the $1,200 direct payments. Unacceptable.\n\nI applaud this move by Governor @GavinNewsom and it's precisely why I'm pushing for it nationwide. Mnuchin needs to act on this immediately. https://t.co/oRfpWpk7P9"), ('dates', '2020-04-24 14:05:26'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.6850947141647339)])
OrderedDict([('tweet', "Since January 2017, I've been asking the Trump administration this very question—will they keep our country's promise that DACA recipients' personal information won't be used for immigration enforcement?\n\nThese young people deserve to know. https://t.co/dCzqSE7axR"), ('dates', '2020-04-22 23:57:14'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.6576189994812012)])
OrderedDict([('tweet', "There's no sugar coating this. People will die because of Trump’s commitment to rolling back major anti-pollution measures, and communities of color are already suffering the most. This is why we fight. #EarthDay https://t.co/CFOfJlaJMM"), ('dates', '2020-04-22 19:20:49'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.8624448776245117)])
OrderedDict([('tweet', "Black Americans shouldn't fear being targeted by law enforcement for wearing face masks during a pandemic. I'm pushing Attorney General Barr and FBI Directory Wray to require racial bias training for police. https://t.co/HtUGDxJJo5"), ('dates', '2020-04-21 13:38:44'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.7325571179389954)])
OrderedDict([('tweet', "Trump failed to take this crisis seriously from day 1. His abandonment of his role as president has cost lives. And now, he's shamelessly politicizing this pandemic to double down on his anti-immigrant agenda.\n\nEnough, Mr. President. The American people are fed up. https://t.co/pT5edXBIT5"), ('dates', '2020-04-21 03:07:53'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.6861546039581299)])
OrderedDict([('tweet', 'This pandemic isn’t close to being over, and yet the authorization for National Guard troops on the frontlines of responding to COVID-19 will soon expire. We need them to continue the fight in California and around the country.\n\nI’m demanding Trump extend the authorization.'), ('dates', '2020-04-20 23:24:47'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.5971461534500122)])
OrderedDict([('tweet', "Yesterday's unemployment numbers were staggering. 22 million Americans have now lost their jobs because of this pandemic.\n\nTo Mitch McConnell: Congress has to get back to work. People need to eat and keep a roof over their heads. That means ensuring monthly cash payments."), ('dates', '2020-04-18 00:25:15'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.5459343194961548)])
OrderedDict([('tweet', "Unacceptable that debt collectors are cashing in on people's $1,200 direct payments. Americans are in dire need of money to pay their rent and put food on the table—this is absurd.\n \nI'm leading my colleagues today in a push to end this. https://t.co/PQFBe69ogh"), ('dates', '2020-04-17 16:51:16'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.7751864194869995)])
OrderedDict([('tweet', "This pandemic has blown the lid off something many of us have known our entire lives: there are deep racial disparities in our country’s health care system.\n\nThis Black Maternal Health Week, it's critical that we speak truth about the ongoing crisis of Black maternal mortality. https://t.co/YHlJUCRCn6"), ('dates', '2020-04-15 18:09:06'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.6226397156715393)])
OrderedDict([('tweet', 'Black Americans are infected and dying from COVID-19 at alarming rates, and we need demographic data to understand the full scope of the issue.\n\nProud to join my colleagues on a bill today to require the federal government to collect and report this data.'), ('dates', '2020-04-14 21:56:17'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.6051598787307739)])
OrderedDict([('tweet', "Too many minority-owned businesses are being left out of coronavirus relief programs. And the truth is that our neighborhoods won't look the same without these businesses.\n\nThe administration must do more to prioritize relief for them. https://t.co/VbqZ8Qobe7"), ('dates', '2020-04-14 14:11:11'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.5148766040802002)])
OrderedDict([('tweet', 'Any week now, SCOTUS may rule on DACA and upend the lives of thousands of young people. It’s unconscionable to issue a ruling when over 200,000 DACA recipients are essential workers in this crisis, including 29,000 in the health care industry.\n\nSCOTUS must delay its ruling.'), ('dates', '2020-04-11 16:47:19'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.5550681352615356)])
OrderedDict([('tweet', 'Millions of people have just lost their jobs and are relying on credit cards to feed themselves and their families. No one should profit off of this pain in the middle of a pandemic.\n\nThat’s why I’m calling for a suspension of credit card interest, fees, and penalties.'), ('dates', '2020-04-10 23:44:47'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.8203740119934082)])
OrderedDict([('tweet', 'This pandemic is exploiting long existing environmental injustices all across the country. Clean air is a human right but too many communities are suffering from toxic air pollution. We must fight for clean air for all. https://t.co/JewwDkJ9U7'), ('dates', '2020-04-09 15:51:15'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.5795185565948486)])
OrderedDict([('tweet', "I'm calling for an immediate suspension of credit card interest, fees, and penalties for the duration of this crisis.\n\nNo one should profit off of Americans who have just lost their income. Charging exorbitant interest, fees, and penalties during this pandemic is simply immoral."), ('dates', '2020-04-08 15:31:41'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.664853036403656)])
OrderedDict([('tweet', "Earlier today, Trump removed the lead Inspector General charged with monitoring how the administration spends $2 trillion in COVID-19 relief.\n\nThis is yet another deeply troubling example of Trump and his administration attempting to evade accountability. We won't stand for it. https://t.co/MITRK7P9ls"), ('dates', '2020-04-08 02:19:20'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.6717520356178284)])
OrderedDict([('tweet', "Trump's intentional failure to appoint inspectors general makes clear his hostility to being held accountable. This is especially irresponsible w/ the unprecedented scale of the federal response to COVID-19.\n\nI'm demanding Trump move quickly to fill IG vacancies. This can't wait."), ('dates', '2020-04-04 17:45:09'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.7044460773468018)])
OrderedDict([('tweet', 'The firing of the Intelligence Community’s IG is dangerous. It is further evidence Trump will do anything he can to escape oversight and will punish anyone who isn’t “loyal.”\n\nEven in the midst of a deadly pandemic, the president is obsessed with settling scores over impeachment. https://t.co/qvqu4QPQ3Z'), ('dates', '2020-04-04 14:35:24'), ('file_name', 'Kamala D. Harris.csv'), ('pct_true', 0.6533580422401428)])
OrderedDict([('tweet', 'On Friday, the Trump administration vetoed a United Nations resolution that called for a global ceasefire in conflicts to collectively focus on the COVID-19 pandemic.\n\nWe have the most deaths and the most infections in the world.\n\nWhat will it take for this to be taken seriously?'), ('dates', '2020-05-10 16:35:01'), ('file_name', 'Karen Bass.csv'), ('pct_true', 0.6979476809501648)])
OrderedDict([('tweet', 'We are in the midst of a global pandemic.\n\nNearly one in every four deaths WORLDWIDE is an American and yet here you are, attacking the Speaker because she has the courage, leadership and competence the nation needs right now.\n\nThis is the leadership needed in the White House. https://t.co/IyUQ0Q7Zwm'), ('dates', '2020-04-19 19:30:31'), ('file_name', 'Karen Bass.csv'), ('pct_true', 0.6355007290840149)])
OrderedDict([('tweet', 'No one should die in jail from COVID-19 because they were driving on a suspended license due to an unpaid traffic ticket.'), ('dates', '2020-04-17 01:37:03'), ('file_name', 'Karen Bass.csv'), ('pct_true', 0.5871973633766174)])
OrderedDict([('tweet', 'No one should die in jail from COVID-19 because they couldn’t post a $1,000 bail.'), ('dates', '2020-04-06 14:20:10'), ('file_name', 'Karen Bass.csv'), ('pct_true', 0.6993016004562378)])
OrderedDict([('tweet', 'Remember when we had a President we could be proud of? 8 years and no scandals, no indictments, no articles of impeachment — seems like a long time ago but it was only about 3 years ago. \n\nDon’t forget what a real President is like. November is in only 9 months. #PresidentsDay https://t.co/mpR2gZ2NDi'), ('dates', '2020-02-17 19:28:04'), ('file_name', 'Karen Bass.csv'), ('pct_true', 0.6458348035812378)])
OrderedDict([('tweet', 'How will history judge this President for proposing a 26% cut to the Environmental Protection Agency as the world burns?'), ('dates', '2020-02-14 03:31:00'), ('file_name', 'Karen Bass.csv'), ('pct_true', 0.569063127040863)])
OrderedDict([('tweet', 'This is who Donald Trump is -- a corrupt politician who has violated the Constitution and feels emboldened by his phony exoneration. \n\nThis President is a clear and dangerous threat to our democracy. https://t.co/xU0f1Y8XtW'), ('dates', '2020-02-12 15:32:07'), ('file_name', 'Karen Bass.csv'), ('pct_true', 0.7222689390182495)])
OrderedDict([('tweet', 'Over a million Americans are sick and Trump wants to "terminate health care." That says it all. https://t.co/NNzhgvCqmV'), ('dates', '2020-05-06 22:24:03'), ('file_name', 'Katherine M. Clark.csv'), ('pct_true', 0.6100624799728394)])
OrderedDict([('tweet', "Once again, the 'fake news' is coming from the White House. Trump's refusal to acknowledge the need for #PPE and ensure we have adequate medical supplies continues to put our essential workers in danger and could delay our recovery.\nhttps://t.co/foNvylCEXv"), ('dates', '2020-04-26 17:52:13'), ('file_name', 'Katherine M. Clark.csv'), ('pct_true', 0.5630666613578796)])
OrderedDict([('tweet', "Small businesses feeling the most pain have been shut out of #PPP and left empty-handed. That's why @HouseDemocrats are fighting for REAL relief that will go directly to our community banks and Main Street shops. Don't believe the #GOP spin. #forthepeople\nhttps://t.co/Xv3DpqgibU"), ('dates', '2020-04-21 17:24:31'), ('file_name', 'Katherine M. Clark.csv'), ('pct_true', 0.8478730916976929)])
OrderedDict([('tweet', 'More than 40,000 Americans have lost their lives to #Covid_19. \n \nTrump is once again relying on division to distract us from his failure to provide the testing, PPE and tracing that is necessary to save lives and reopen our economy.\n \nWe’re not falling for it.'), ('dates', '2020-04-21 14:49:43'), ('file_name', 'Katherine M. Clark.csv'), ('pct_true', 0.779872477054596)])
OrderedDict([('tweet', 'As Americans are fighting for their lives against #Covid_19, Trump decides it is a good time to release deadly neurotoxins into our air and water. His indifference to the health and safety of Americans is literally breathtaking. \nhttps://t.co/buliLEdj44'), ('dates', '2020-04-17 18:46:54'), ('file_name', 'Katherine M. Clark.csv'), ('pct_true', 0.5399541258811951)])
OrderedDict([('tweet', "There are still not enough tests and the Administration doesn't have an acceptable national testing strategy. \n\nAnd without universal testing, reopening comes with major risks."), ('dates', '2020-05-04 17:34:01'), ('file_name', 'Kathleen M. Rice.csv'), ('pct_true', 0.6274224519729614)])
OrderedDict([('tweet', "Too many small businesses are still struggling to get help. House Democrats are pushing for a #SmallBizRelief package that expands access and provides more funding to the @SBAgov's disaster and #PPP loan programs. https://t.co/TF4rlWOalt"), ('dates', '2020-04-15 16:07:16'), ('file_name', 'Kathleen M. Rice.csv'), ('pct_true', 0.6270201206207275)])
OrderedDict([('tweet', "This public health crisis is a reminder that no one should\xa0ever be denied health care or discriminated against simply for being themselves.\xa0\n\xa0\nThat's why I'm proud to support the fight for trans rights and recognize today as #TransDayOfVisibility."), ('dates', '2020-04-01 02:58:02'), ('file_name', 'Kathleen M. Rice.csv'), ('pct_true', 0.5248599648475647)])
OrderedDict([('tweet', 'Gov DeSantis &amp; @FLDEO are failing our neighbors. He has the authority to increase the # of weeks &amp;expand unemployment benefits. He has refused to do so. Our system was designed to fail &amp;minimal effort has been made to ensure it works. The gov must do more.\nhttps://t.co/gE7bUwk8W3'), ('dates', '2020-05-01 20:34:00'), ('file_name', 'Kathy Castor.csv'), ('pct_true', 0.5039267539978027)])
OrderedDict([('tweet', 'Without oversight, the biggest risk here is not Americans cheating the government; it’s the government cheating the American people, by not using taxpayer dollars to help those who need it most. https://t.co/pTTzyoHBIm'), ('dates', '2020-04-30 16:30:14'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.6180612444877625)])
OrderedDict([('tweet', '“Now is not the time for guessing games. Tens of thousands of Americans are dead, and millions are out of work in part because Trump dragged his feet and misled the public about what his administration was doing and when.”\n\nMore from me in the @washingtonpost ⬇️⬇️ https://t.co/P45hGDuIgQ'), ('dates', '2020-04-29 03:36:59'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.5868146419525146)])
OrderedDict([('tweet', 'Here are the facts:\n\n1) President Trump has the authority to use the DPA to meet a number of critical needs—including #COVID19 tests.\n\n2) Congress allocated $1B to the DPA fund for that exact reason.\n\n3) 40,000 Americans died before President Trump used the DPA to order tests.'), ('dates', '2020-04-21 19:34:42'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.6673355102539062)])
OrderedDict([('tweet', 'Staffing companies backed by private equity are\n❌ cutting physician pay,\n❌ lying about doing it, and\n❌ retaliating against clinicians who speak up.\n\n@SenWarren and I are calling out those who put profit margins ahead of our frontline healthcare workers. https://t.co/LOPx189bSr'), ('dates', '2020-04-15 20:42:07'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.6072795987129211)])
OrderedDict([('tweet', 'The President should have started using the Defense Production Act weeks ago, across our economy. If he had, we wouldn’t be choosing between export bans and global partnership. Another unforced error that will cost lives. https://t.co/VVmoPb6gay'), ('dates', '2020-04-03 22:15:56'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.5078839659690857)])
OrderedDict([('tweet', 'As this pandemic threatens to crush more and more Americans with medical debt, @CFPBDirector is pushing policies that put patients’ private medical information at risk. I’m following up to demand answers for American families—repeated sidestepping is simply not good enough. https://t.co/Zw0p7Yhsd2'), ('dates', '2020-04-02 16:18:21'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.8837906718254089)])
OrderedDict([('tweet', 'This isn’t about playing politics. Especially during a crisis, the American people need to have confidence that our government is doing right by their families.\n\nThat’s why I’ve asked for a spot on the Congressional commission that oversees the $500 billion corporate bailout. https://t.co/ng1ROiwbAf'), ('dates', '2020-04-01 22:31:38'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.617320716381073)])
OrderedDict([('tweet', "Price gouging during this national public health emergency is absolutely repugnant. We've all got to step up and hold companies accountable to keep Americans safe, and I'm glad to see state Attorneys General taking this on. https://t.co/e10JnHCRkX"), ('dates', '2020-03-25 19:39:29'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.6512022614479065)])
OrderedDict([('tweet', 'People are concerned about paying their bills and staying safe during this pandemic—they shouldn’t also have to worry about companies profiting off their personal data.\n\n@RepMalinowski and I are conducting oversight of the tech giant behind a new #Covid_19 triage website. https://t.co/OCdiwqA9Vr'), ('dates', '2020-03-18 16:05:40'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.7025436758995056)])
OrderedDict([('tweet', 'This is a gross message to send, @JetBlue. The precautions the CDC is recommending, like teleworking and social distancing, will save lives, but not if abused by corporations looking to make a buck. This isn’t the time for jokes that downplay the seriousness of this pandemic. https://t.co/XZ8KvowoUT'), ('dates', '2020-03-12 21:56:17'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.7856299877166748)])
OrderedDict([('tweet', 'I refuse to put patients’ health at risk so that insurance companies can use COVID-19 to pad their profits. Patients with private insurance shouldn’t have to choose between lab tests for this virus and putting food on the table.'), ('dates', '2020-03-06 23:32:29'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.5970141887664795)])
OrderedDict([('tweet', 'There’s no excuse for the Senate’s failure to pass legislation to finally allow the Department of Health &amp; Human Services to negotiate for #LowerDrugCosts. People are struggling, and this is how we can help them. https://t.co/NWbRuK1csJ'), ('dates', '2020-03-04 14:18:39'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.7027703523635864)])
OrderedDict([('tweet', 'Intimate partner violence is an issue that affects Americans of all ages. This #TeenDVMonth, I renew my calls for the Senate to renew #VAWA—for the 1.5 million high schoolers affected by dating violence every year.'), ('dates', '2020-02-25 19:45:35'), ('file_name', 'Katie Porter.csv'), ('pct_true', 0.6898571252822876)])
OrderedDict([('tweet', 'Congress must immediately put more money into the #PaycheckProtectionProgram, which keeps our small businesses open and millions of Americans paid - there should not be partisan fighting over that. Read more in my press release here: https://t.co/MsIhsU5QpI https://t.co/1rQMl9RgG9'), ('dates', '2020-04-18 14:31:52'), ('file_name', 'Kay Granger.csv'), ('pct_true', 0.619592010974884)])
OrderedDict([('tweet', 'When the #coronavirus was diagnosed, China covered up its scale, making the crisis even worse. Read more on how China threatens our way of life and how #Texas plays a key role in countering foreign threats like China: https://t.co/E9GP06OPbb https://t.co/IodZ0ogdbp'), ('dates', '2020-04-04 15:00:00'), ('file_name', 'Kay Granger.csv'), ('pct_true', 0.9266571998596191)])
OrderedDict([('tweet', "China and Russia are stirring up trouble in America's backyard and threatening our way of life. As they try to gain ground in Latin America, I am committed to working with @SOUTHCOM’s Admiral Craig Faller to ensure he has the resources needed to defend our nation and allies. https://t.co/0VPTOHyzb0"), ('dates', '2020-03-05 20:25:21'), ('file_name', 'Kay Granger.csv'), ('pct_true', 0.6634982228279114)])
OrderedDict([('tweet', "In the past days we got confirmation that both the Mueller investigation and the @GenFlynn investigation ignored exonerating evidence and actively pushed baseless narratives, just to target @realDonaldTrump\n\nNow we're going down the same path, during a crisis. https://t.co/FSOksUPCIE"), ('dates', '2020-05-09 01:41:04'), ('file_name', 'Kelly Armstrong.csv'), ('pct_true', 0.5413784980773926)])
OrderedDict([('tweet', 'We need structural change, not plugging the hole. Too many workers and consumers depend on the @USPS for us to keep the Postal Service on the path to insolvency. https://t.co/dKxJhCHnIT'), ('dates', '2020-05-08 16:26:32'), ('file_name', 'Kelly Armstrong.csv'), ('pct_true', 0.6336380243301392)])
OrderedDict([('tweet', 'Democrats’ COVID19 Oversight Committee is Impeachment 2.0—we can’t afford that during a crisis.\n\nI have confidence in @GOPLeader ‘s  appointments of @SteveScalise @Jim_Jordan @RepWalorski @RepBlaine I’m especially glad there’s at least one doctor @repmarkgreen on the committee.'), ('dates', '2020-05-07 15:53:24'), ('file_name', 'Kelly Armstrong.csv'), ('pct_true', 0.5337769389152527)])
OrderedDict([('tweet', "For almost four years, Democrats claimed that President @realDonaldTrump colluded with Russia. That's been disproven several times over.\n\nIt's well past time to see where these unfounded allegations originated. It's time for @RepAdamSchiff to release these transcripts. https://t.co/hV7wJqjEwS"), ('dates', '2020-05-05 17:15:22'), ('file_name', 'Kelly Armstrong.csv'), ('pct_true', 0.6190561056137085)])
OrderedDict([('tweet', 'Broad verdict? You mean through state-approved posts on a state-run platform in a country which oppresses its citizens’ freedom of speech. This story does nothing more than help peddle China’s misinformation campaign...embarrassing. https://t.co/ZgbzSWyYqP'), ('dates', '2020-05-05 13:04:05'), ('file_name', 'Kelly Armstrong.csv'), ('pct_true', 0.5089655518531799)])
OrderedDict([('tweet', 'Absolutely abhorrent. \n\nHundreds of thousands of deaths are on China’s hands because of its incomprehensible, selfish deception.\n\nChina must be held accountable.\n\nhttps://t.co/N0ZiTs5S0I'), ('dates', '2020-05-04 22:08:13'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.8393890261650085)])
OrderedDict([('tweet', 'Proud to join this effort. Chinese laboratories that pose security concerns and lack proper oversight, like the Wuhan Institute of Virology, should not benefit from American taxpayer funding. \n\nThat’s why we’re asking Senate &amp; House leadership to cut off this funding. https://t.co/Kji0v4kC8T'), ('dates', '2020-04-23 15:48:39'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.6146045923233032)])
OrderedDict([('tweet', "China must pay the price for its role in COVID-19, which has cost American lives, wrecked our economy, pushed our health care system to the brink, &amp; devastated families &amp; communities.\n \n@realDonaldTrump must hold China accountable. I'm confident he will.\n\nhttps://t.co/NdC0W0TbJJ"), ('dates', '2020-04-22 17:39:19'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.9565032124519348)])
OrderedDict([('tweet', "Despite Democrats' sad - but not surprising - political games, #PPP is getting a much-needed replenishment &amp; the front lines are getting more money to keep saving lives and #slowthespread.\n \nTo Georgia's small businesses still awaiting aid: Help is FINALLY on the way. https://t.co/neh4cNalhZ"), ('dates', '2020-04-21 22:13:10'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.506040096282959)])
OrderedDict([('tweet', "I challenge Democrats to explain to the 861,000+ Georgians who filed jobless claims in the last month WHY they won't replenish #PPP.\n \n#PPP has run out. MILLIONS of people still need help. Why are @SenSchumer &amp; @SpeakerPelosi holding the program hostage? https://t.co/iC2mawmsL1"), ('dates', '2020-04-17 14:53:54'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.9672607779502869)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram has run dry. Why?\n \nBecause @SenSchumer &amp; @SpeakerPelosi blocked $251 billion in relief funding to push their own political agenda.\n \nDemocrats continue to play politics while small businesses are closing and 22 million Americans are now jobless. https://t.co/KWF1JoAWIC'), ('dates', '2020-04-16 15:35:20'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.9708733558654785)])
OrderedDict([('tweet', 'The American people should know the truth about why #PPP funding is running dry today: Democrats picked politics over hardworking families.\n \nInstead of providing $3.5 billion per hour in relief, loans will stop thanks to Democrats.\n \nThis is unacceptable.https://t.co/BxzzpgM00g'), ('dates', '2020-04-15 21:43:20'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.8256919980049133)])
OrderedDict([('tweet', 'While Senate Dems blocked more funding for #smallbiz loans, I was speaking with small business owners from @CatoosaG County and the @VLCCHAMBER. It’s clear they need more relief to keep their employees on payroll and our economy stable. Let’s stop playing games &amp; get it done.'), ('dates', '2020-04-10 00:50:07'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.5375967025756836)])
OrderedDict([('tweet', 'I’ve said this time and time again, now is not the time for politics. Democrats are continuing to play political games at the expense of American workers. https://t.co/krbi8mn7nA'), ('dates', '2020-04-09 15:28:50'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.5977700352668762)])
OrderedDict([('tweet', 'China’s stats on #COVIDー19 are based on misinformation. It’s clearer than ever the Chinese Communist Party is concealing the numbers and data behind the nature of the #coronavirus.\n \nHow many more deaths will occur because of China’s deception? https://t.co/gbTkKBcsLN'), ('dates', '2020-04-04 20:12:40'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.9804428219795227)])
OrderedDict([('tweet', 'You are either helping right now, or hurting. Pelosi’s handpicked partisan ploys are helping no one. She’s hurting American families by prioritizing airplane emissions, same day voter registration&amp;wind energy tax credits over #coronavirus relief. #StopTheGamesNancy, #PassTheBill'), ('dates', '2020-03-24 14:00:55'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.9826825857162476)])
OrderedDict([('tweet', 'Democrats said NO to relief. “Not right now” to $1200 checks, adding $600/week to unemployment checks, hospital funding, small business loans and greater CERTAINTY. \n\nNothing is distracting me - not even the fake news - from fighting for all Georgians. https://t.co/cBjNVJZGOT'), ('dates', '2020-03-23 22:37:29'), ('file_name', 'Kelly Loeffler.csv'), ('pct_true', 0.6345782279968262)])
OrderedDict([('tweet', 'It is time for those in charge to admit that they have taken things to the extreme.\n\nThere is no shame in admitting that we might have acted wrongly out of compassion, but it is unforgivable if we continue to punish Americans when we know better. https://t.co/g0DjuhvJ5V'), ('dates', '2020-05-01 15:01:09'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.506354570388794)])
OrderedDict([('tweet', 'It’s the FBI’s job to pursue the truth, not vendettas. It is now Congress’ job to investigate this indefensible behavior. https://t.co/UYogLqsEsY'), ('dates', '2020-04-30 15:26:35'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.5945157408714294)])
OrderedDict([('tweet', 'Our nation is facing difficult times and the American people are hurting. \n\nCongress must ensure that we aren’t leaving small counties behind as we get our country back on her feet. https://t.co/spbFzhaV5O https://t.co/Kwiz5k3Zym'), ('dates', '2020-04-21 16:45:12'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.5313571691513062)])
OrderedDict([('tweet', 'The WHO must be held accountable for its role in covering up the Chinese Communist Party’s failed response to #COVID_19. \n\n@realDonaldTrump is right to halt funding for this corrupt organization. https://t.co/TsrPztqHbs'), ('dates', '2020-04-14 23:17:47'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.6044343709945679)])
OrderedDict([('tweet', 'The CCP’s irresponsible handling of the coronavirus and its false propaganda have worsened this global health crisis.\n\nIt’s time we as a Congress take a strong stand against China’s nefarious actions. \n\nhttps://t.co/GxqDGok9iJ'), ('dates', '2020-04-02 13:20:12'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.8188342452049255)])
OrderedDict([('tweet', 'I’ve said this since Day 1: We cannot let fear push our nation over the brink. \n\nAuthoritarian limits on business activity are hurting American workers &amp; small businesses. We need to get healthy people back to work &amp; focus resources on the most vulnerable. https://t.co/erYV2p1SBr'), ('dates', '2020-03-24 17:09:14'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.6472283005714417)])
OrderedDict([('tweet', 'We need to reduce our dependence on China and prioritize the American people. \n\nThankfully, we have a President in the White House who is skeptical of China’s increasing dominance. @realDonaldTrump continues to combat China’s unfair trade practices and always puts #AmericaFirst. https://t.co/q4dgjv81oh'), ('dates', '2020-03-21 16:04:00'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.9361220002174377)])
OrderedDict([('tweet', 'Over 80 percent of pharmaceuticals sold in American stores come from a country mounting a full-on misinformation campaign against the U.S. \n\nWe need to reevaluate our dependency on China and move toward self-reliance. https://t.co/f9Qt4y0pnN'), ('dates', '2020-03-18 14:09:52'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.503919243812561)])
OrderedDict([('tweet', 'Nancy Pelosi squandered days trying to push wishlist items on us rather than engaging in a meaningful bipartisan discussion. This bill is a 110-page, multi-billion dollar boondoggle shoved on us at the stroke of midnight.'), ('dates', '2020-03-14 05:06:50'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.5492551922798157)])
OrderedDict([('tweet', 'Our government should not spy on Americans without a warrant, it’s that simple.\n\nAmericans should feel confident their privacy is protected and that neither the government nor any agent affiliated with the government are abusing constitutional rights.\nhttps://t.co/OkzWL4tYqi'), ('dates', '2020-03-12 14:36:15'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.7640696167945862)])
OrderedDict([('tweet', 'You can’t just slap a new coat of paint on a house with structural problems and call it a day, but that’s basically what H.R. 6172 does for FISA.\n\nThe bill does nothing to stop the government from spying on U.S. citizens without a warrant. The American people deserve better.'), ('dates', '2020-03-11 21:29:12'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.873160719871521)])
OrderedDict([('tweet', 'WATCH: Today, Democrats on the Judiciary Committee rejected an opportunity to come together and condemn far-left-wing violence in their domestic terrorism bill.\n\nPurveyors of hate and terror, regardless of ideology, should be equally condemned for their violent acts. https://t.co/zp7zJndYMY'), ('dates', '2020-03-11 17:10:00'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.5521433353424072)])
OrderedDict([('tweet', 'You’re right @toddstarnes, we have to start taking Democrats at their word.\n\n“Hell yes we’re going to take your AR-15, your AK-47” means one thing: our constitutional right to carry is at risk with Beto O’Rourke in charge of Joe Biden’s gun control efforts.\n\n🎙 LISTEN: https://t.co/vjuS90GaF4'), ('dates', '2020-03-09 21:04:00'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.6362205147743225)])
OrderedDict([('tweet', '.@BetoORourke thinks it’s okay for the government to trample on my constitutional rights but it’s wrong for me to oppose tyranny. \n\nEver wonder why you’re unemployed Beto? https://t.co/8qy51SUpSV'), ('dates', '2020-03-07 15:40:26'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.6381801962852478)])
OrderedDict([('tweet', 'Since day 1, Democrats have politicized the coronavirus. The president’s initial $2.5 billion request was a thoughtful proposal to address our coronavirus response needs.'), ('dates', '2020-03-04 22:14:15'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.7825497984886169)])
OrderedDict([('tweet', 'Juror Tomeka Hart’s political bias leads me to ask the question, does the partisan divide in the nation’s political landscape make it impossible for us to have a fair legal system?\n\nhttps://t.co/c4tKLwr0cQ'), ('dates', '2020-03-04 15:05:27'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.695863664150238)])
OrderedDict([('tweet', 'We must lay down our partisan swords to respond to the coronavirus and do what’s best for the country.\n\nThat’s why I signed a letter urging @SpeakerPelosi and @LeaderHoyer against tying a public health crisis to the upcoming FISA re-authorization. https://t.co/YofHvL2h7a'), ('dates', '2020-03-02 15:21:33'), ('file_name', 'Ken Buck.csv'), ('pct_true', 0.5067753791809082)])
OrderedDict([('tweet', 'It was China’s initial cover up and inadequate reaction to this virus that allowed for it to grow into a pandemic of this magnitude. We can no longer be dependent upon this communist regime for supplies essential for our response. Couldn’t have said it better myself, @GOPLeader. https://t.co/P1qOd58Jes'), ('dates', '2020-04-28 19:06:47'), ('file_name', 'Ken Calvert.csv'), ('pct_true', 0.6806471347808838)])
OrderedDict([('tweet', 'Democrats’ insistence on using critical resources such as the #PaycheckProtectionProgram as leverage to further their agenda is despicable. There is no political opportunity to take advantage of here, we MUST put American’s first. https://t.co/oQzXQD2RvV'), ('dates', '2020-04-21 18:53:25'), ('file_name', 'Ken Calvert.csv'), ('pct_true', 0.7490025162696838)])
OrderedDict([('tweet', 'Dems need to stop blocking the PPP program because it is keeping small businesses going during these times of uncertainty, read about some of the many success stories: https://t.co/21jd8Aa9AU'), ('dates', '2020-04-17 19:16:54'), ('file_name', 'Ken Calvert.csv'), ('pct_true', 0.6168392896652222)])
OrderedDict([('tweet', 'We need to replenish #PPP NOW. Dems are holding up funding in order to ADD funds to accounts that have NOT been depleted. That’s the difference. To a small biz owner, &amp; their workers, it’s the difference between financial hope or financial despair https://t.co/SL6g9ijUU1'), ('dates', '2020-04-17 15:17:40'), ('file_name', 'Ken Calvert.csv'), ('pct_true', 0.6683237552642822)])
OrderedDict([('tweet', 'In this time of great challenge, uncertainty, and upheaval, we must have the ability to conduct the business of the House of Representatives in a public and transparent way by holding committee hearings, debating, marking-up and voting on legislation.'), ('dates', '2020-04-28 21:59:52'), ('file_name', 'Kendra S. Horn.csv'), ('pct_true', 0.5891419649124146)])
OrderedDict([('tweet', "Our small businesses are job creators and engines of opportunity. When they suffer so do our workers and families. I'm proud to introduce the Too Small To Fail Act with @RepChrisPappas. Our bill would provide immediate grant assistance and zero interest loans to small businesses. https://t.co/l6DJoIFCSG"), ('dates', '2020-03-23 17:27:03'), ('file_name', 'Kendra S. Horn.csv'), ('pct_true', 0.5034422278404236)])
OrderedDict([('tweet', "It's been one year since I voted to pass #HR1 - an anti-corruption clean elections bill to empower American voters over big-money donors. Our democracy should always put people first. Now it's time for the Senate to take action and pass the #ForThePeople Act! https://t.co/kLuBfIfZXD"), ('dates', '2020-03-10 18:44:40'), ('file_name', 'Kendra S. Horn.csv'), ('pct_true', 0.686248779296875)])
OrderedDict([('tweet', "I've heard from too many Oklahomans forced to choose between putting food on the table and affording prescription medications. We can't allow people to fall through the cracks of our health care system. The Senate must take up the Lower Drug Costs Now Act. https://t.co/wCz57erVGf"), ('dates', '2020-03-04 18:48:00'), ('file_name', 'Kendra S. Horn.csv'), ('pct_true', 0.5322462916374207)])
OrderedDict([('tweet', "The devil is in the details. While the NASA budget preview includes an encouraging increase, it's nowhere near the $5-6 billion that NASA's Bridenstine said is needed for a 2024 Moon landing. When we talk about plans as big as returning to the moon, the details matter."), ('dates', '2020-02-08 00:17:07'), ('file_name', 'Kendra S. Horn.csv'), ('pct_true', 0.5482293963432312)])
OrderedDict([('tweet', 'It’s time to fix our nation’s infrastructure. We have to bring America up to speed by expanding broadband. We have to fix our nation’s roads and electrical grid. We must invest in water and space infrastructure. This isn’t a GOP or Dem issue, it’s an American issue. #SOTU'), ('dates', '2020-02-05 02:59:07'), ('file_name', 'Kendra S. Horn.csv'), ('pct_true', 0.7971017360687256)])
OrderedDict([('tweet', 'Unfortunately, Democrats on the Ways and Means committee unanimously rejected my amendment today to allow seniors and the chronically ill to PERMANENTLY deduct their medical expenses if they exceed 7.5% of their income.'), ('dates', '2019-06-20 20:52:16'), ('file_name', 'Kenny Marchant.csv'), ('pct_true', 0.9349365830421448)])
OrderedDict([('tweet', 'Millions of Americans will spend less time and money filing their taxes.The Joint Committee on Tax estimates 18 million people will itemize their deductions for 2018, as 46.5 million did the previous year. People can now choose how to use more of their hard earned resources. https://t.co/Qq0tXQi8uf'), ('dates', '2018-06-15 22:00:00'), ('file_name', 'Kenny Marchant.csv'), ('pct_true', 0.6558422446250916)])
OrderedDict([('tweet', 'After constituents in #TX24 had their newborn child’s Social Security card stolen out of their mailbox, we found that acquiring a new SSN for their child was incredibly difficult. This commonsense, bipartisan legislation addresses this important issue for our nation’s children. https://t.co/SCQh2OEBLl'), ('dates', '2018-04-11 14:51:44'), ('file_name', 'Kenny Marchant.csv'), ('pct_true', 0.5913368463516235)])
OrderedDict([('tweet', 'This new bill will put a much needed $320 billion in funding in the crucial #PaycheckProtectionProgram and bring money to our local small businesses who need it most – after they waited days for Democrats to stop holding them hostage.'), ('dates', '2020-04-22 13:11:23'), ('file_name', 'Kevin Brady.csv'), ('pct_true', 0.5392481684684753)])
OrderedDict([('tweet', 'The Paycheck Protection Program has officially run out of money.\n\nDemocrats are too concerned with playing politics to work with us to pass more funding.\n\nBecause of that, small businesses and workers across the country will be left high and dry.'), ('dates', '2020-04-16 15:49:21'), ('file_name', 'Kevin Brady.csv'), ('pct_true', 0.6968302130699158)])
OrderedDict([('tweet', 'WHO: NO REFORMS, NO MONEY.\n\n@realDonaldTrump right to temp halt funding for corrupt #WHO. Their pro-China bias &amp; withholding crucial COVID-19 data cost lives in U.S. &amp; the world. \n\nWorld needs credible, timely health data on this &amp; future health crises. \n\n#WHO failing. https://t.co/DkUHU6SvYU'), ('dates', '2020-04-14 23:06:42'), ('file_name', 'Kevin Brady.csv'), ('pct_true', 0.9331668019294739)])
OrderedDict([('tweet', 'SMALL BUSINESS HELD HOSTAGE\n\nToday: 1,202,118 small businesses have received over $250 bil in #PPPLoans. \n\nBut the money is running out quickly.  \n\nRT if #Democrats should stop holding hostage #PresidentTrump ask for $250b more for small biz $$. \n\n#FundNow\n#StandWithSmallBusiness https://t.co/LtzEkzqHKv'), ('dates', '2020-04-14 21:37:05'), ('file_name', 'Kevin Brady.csv'), ('pct_true', 0.6286662220954895)])
OrderedDict([('tweet', 'No Data?\n\n- 960,000 small biz got #PPPLoans = $232 b.\n- Over 2/3 of PPP loan fund exhausted; likely run out this week. \n- Tens of thousands of small biz face closure. \n\nYet, @SpeakerPelosi blocking $251 b more emergency #PPPLoans, says “no data” they need it.\n\n#Fiddling\n#Sad'), ('dates', '2020-04-14 14:26:21'), ('file_name', 'Kevin Brady.csv'), ('pct_true', 0.5652120113372803)])
OrderedDict([('tweet', 'WHY THE MEDIA PANIC? \n\n#President focused on controlling #COVID19, helping workers keep jobs &amp; biz survive.\n\nAlso, gathering expert advice on when &amp; how to re-open U.S. economy: \n- as health conditions permit\n- safely for workers &amp; customers\n- working w/state &amp; local leaders.'), ('dates', '2020-04-10 22:14:33'), ('file_name', 'Kevin Brady.csv'), ('pct_true', 0.5313852429389954)])
OrderedDict([('tweet', "This is not the time to play partisan games - the lives and livelihoods of Americans are at stake. \n\nWe need more funding now; politics can wait, small businesses can't. https://t.co/mrjkIg3mKE"), ('dates', '2020-04-09 14:58:55'), ('file_name', 'Kevin Brady.csv'), ('pct_true', 0.654837965965271)])
OrderedDict([('tweet', 'Conservatives Should Back CARES\n\nThis = cash-flow crisis for #Biz\n\nCustomers disappeared due to virus - now layoffs/closures.\nInjecting $$ into local biz, small &amp; large, stops cash-flow crisis. \nPreserves biz &amp; jobs thru crisis.\nSupports supply-side economy by keeping biz intact.'), ('dates', '2020-03-26 22:39:55'), ('file_name', 'Kevin Brady.csv'), ('pct_true', 0.5647321343421936)])
OrderedDict([('tweet', 'As every sector of our economy struggles to survive #COVID19 and seeks financial stability from the federal government, environmental extremists are using the pandemic to accelerate their goal of putting America’s energy jobs in the grave. https://t.co/EyUU5O4Shm'), ('dates', '2020-05-08 16:44:00'), ('file_name', 'Kevin Cramer.csv'), ('pct_true', 0.5902476906776428)])
OrderedDict([('tweet', 'China and its state media’s handling of #COVID19 demonstrate the dangers of communism.\n\nI’m concerned some of our media isn’t as concerned by it as they should be. https://t.co/7pCtAwxoeB'), ('dates', '2020-05-05 21:30:00'), ('file_name', 'Kevin Cramer.csv'), ('pct_true', 0.8756074905395508)])
OrderedDict([('tweet', 'One thing is certain: China squelched important information about #COVID19 which could have prevented the spread of the virus.\n\nThe free world needs to remember this when we deal with these communists in the future. https://t.co/qTuJGhwKFU'), ('dates', '2020-05-05 20:30:00'), ('file_name', 'Kevin Cramer.csv'), ('pct_true', 0.6455572247505188)])
OrderedDict([('tweet', 'With the world experiencing a supply glut, why would the United States accept more oil? \n\nWe have no business letting Saudi tankers unload on our shores. \n\nWe are energy independent, we should act accordingly. https://t.co/DsnjQIsc0s'), ('dates', '2020-04-21 22:00:00'), ('file_name', 'Kevin Cramer.csv'), ('pct_true', 0.7924557328224182)])
OrderedDict([('tweet', 'Shame on companies for rewarding bad behavior by taking this crude as US oil producers suffer.\n\nThe Saudis flooded the market, American companies shouldn’t help them by taking it.\n\n@realDonaldTrump should keep every option on the table, embargo or tariff. https://t.co/gJcQXdCU5v'), ('dates', '2020-04-18 16:01:42'), ('file_name', 'Kevin Cramer.csv'), ('pct_true', 0.8009364604949951)])
OrderedDict([('tweet', "The incoming Saudi tankers are going to make a dire situation worse.\n\nThe supply glut is Saudi Arabia's fault, let’s not force American oil producers to suffer for it.\n\n@realDonaldTrump shouldn’t let these tankers unload on our soil. \n\nAmerica first! https://t.co/jkPss6XeLc"), ('dates', '2020-04-18 14:10:29'), ('file_name', 'Kevin Cramer.csv'), ('pct_true', 0.6169950366020203)])
OrderedDict([('tweet', 'The blame lies at the feet of Democrats who would rather exploit a crisis to advance their extreme partisan agenda than fund a wildly popular program they helped create.\n\nDemocrats: stop your reckless behavior and support a clean appropriation for the #PaycheckProtectionProgram. https://t.co/ulcQRJXEvq'), ('dates', '2020-04-16 15:25:20'), ('file_name', 'Kevin Cramer.csv'), ('pct_true', 0.9765783548355103)])
OrderedDict([('tweet', 'These reports are encouraging &amp; a step in the right direction. \n\nNo matter what, the mess Saudi Arabia &amp; Russia made may take years to clean up. https://t.co/MCTJhUiHX6'), ('dates', '2020-04-09 20:07:51'), ('file_name', 'Kevin Cramer.csv'), ('pct_true', 0.5247833728790283)])
OrderedDict([('tweet', 'Here we go again.\n\nLest they let the country forget about their disgraceful acts just weeks ago, Minority Leader @SenSchumer &amp; @SenateDems are once again holding help hostage and exploiting #COVID19 to advance @SpeakerPelosi’s extreme, left-wing ‘vision.’ https://t.co/RiUuEAf2Lt'), ('dates', '2020-04-09 15:01:34'), ('file_name', 'Kevin Cramer.csv'), ('pct_true', 0.9524000287055969)])
OrderedDict([('tweet', "Russia &amp; Saudi Arabia are both bad actors in the oil dispute but the Saudis are the most disappointing.\n\nThey pretend to be an ally, but they're using #COVID19 to bankrupt American shale.\n\nLet's pull our troops out of the Kingdom if they don't stop waging war on American jobs. https://t.co/xayDLu5kLY"), ('dates', '2020-04-08 16:45:00'), ('file_name', 'Kevin Cramer.csv'), ('pct_true', 0.6084199547767639)])
OrderedDict([('tweet', "I'm alarmed by the far-left policies @SpeakerPelosi continues to sneak into coronavirus relief bills, including anti-Second Amendment gun control measures.  Check out my thoughts on this in the @dcexaminer --&gt; \nhttps://t.co/r07MpFh8io"), ('dates', '2020-05-07 15:44:16'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.6391928195953369)])
OrderedDict([('tweet', "Yesterday, PPP ran out of $. We saw this coming &amp; tried to prevent it by passing add'l funds but @SenateDems blocked it. @SpeakerPelosi even bragged about it. \nThis is a complete forfeiture of leadership, leaving small biz leaders behind when they need us most."), ('dates', '2020-04-17 15:49:26'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.5624855756759644)])
OrderedDict([('tweet', '#DearNancyPelosi,\n\nDo you find it hypocritical to call out poison pills in the Senate bill (which there are none - it was a very bipartisan process) when just 2 weeks ago, we had to wait 48+ hours to vote bc you littered the legislation with poison pills that had to be removed?'), ('dates', '2020-03-24 19:08:40'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.5362344980239868)])
OrderedDict([('tweet', '#DearNancyPelosi, \n\nWhy did you tank a bipartisan stimulus package that could have passed the House &amp; been signed by @realdonaldtrump ASAP? Was it the direct cash to suffering Americans or the much-needed aid to medical workers on the frontlines of #Covid19 that put you off?'), ('dates', '2020-03-24 17:00:02'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.8937492370605469)])
OrderedDict([('tweet', '#DearNancyPelosi, \n\nWhen the American people are looking to our leaders to protect them, why are you focusing on expanding protections for illegal immigrants &amp; restricting our border agents from keeping our country safe?'), ('dates', '2020-03-24 16:23:36'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.9288496971130371)])
OrderedDict([('tweet', '#DearNancyPelosi,\n\nElection Day is more than 7 months away. When you could focus on helping American small businesses survive this government-imposed restriction on non-essential services, why are you wasting energy on same-day voter registration requirements?'), ('dates', '2020-03-24 15:51:51'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.7635821104049683)])
OrderedDict([('tweet', '#DearNancyPelosi, \n\nThe #GreenNewDeal was overwhelmingly shot down in the Senate last year. Why are you caving to a freshman socialist and sneaking GND provisions into what should be a #Covid19 relief package?'), ('dates', '2020-03-24 15:51:05'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.9239538311958313)])
OrderedDict([('tweet', '#DearNancyPelosi,\n\nWith millions of Americans out of work and wondering how to feed their families, why are you increasing federal payments to Planned Parenthood?'), ('dates', '2020-03-24 15:50:29'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.7030943036079407)])
OrderedDict([('tweet', '#DearNancyPelosi,\n\nWhere were you while the Senate worked day &amp; night – with a strongly bipartisan task force – to create a stimulus package that provides aid directly to the people most impacted by #Covid19?'), ('dates', '2020-03-24 15:49:52'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.6431220769882202)])
OrderedDict([('tweet', '#DearNancyPelosi,\n\nAirline workers are in a crisis, the industry is facing unprecedented losses. Instead of providing resources for those losing their jobs, why are you wasting $100 MILLION on research into sustainable aviation fuels?'), ('dates', '2020-03-24 15:49:11'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.9193189740180969)])
OrderedDict([('tweet', '#DearNancyPelosi,\n\nWith #Covid19 spreading through our country, why do you want to divert $35 MILLION to the Kennedy Center in Washington, DC?'), ('dates', '2020-03-24 15:29:17'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.7748672366142273)])
OrderedDict([('tweet', '#DearNancyPelosi, \n\nWhat good does a bailout of the Post Office do to the millions of Americans out of work and suffering from #Covid19?'), ('dates', '2020-03-24 15:23:07'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.8546692728996277)])
OrderedDict([('tweet', 'Proud to join @RepJimBanks @sethmoulton &amp; a bipartisan coalition in the House as a cosponsor of this resolution condemning the Chinese Communist Party not only for misleading the world on #Covid19, but pushing blame on the US with a propaganda campaign.\n\nhttps://t.co/SlIvkYuRJX'), ('dates', '2020-03-24 14:26:25'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.7128034830093384)])
OrderedDict([('tweet', "A disagreement btwn the Middle Eastern oil cartel &amp; Russia caused the largest stock market drop since '08. America must become more than energy-independent, we need to be energy-dominant. Thanks to @realDonaldTrump's pro-America policies, we are well on our way. #gopHEAT"), ('dates', '2020-03-10 19:59:29'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.7808805108070374)])
OrderedDict([('tweet', 'Looks like some of my colleagues slept in this morning! At the @housebudgetGOP hearing, the only Democrat to show up was the chairman. Shows exactly where their priorities are when it comes to  our constitutional responsibility to create a federal budget.'), ('dates', '2020-02-27 19:21:55'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.5496048331260681)])
OrderedDict([('tweet', "Last night, I spoke on the House floor on the need to protect American workers, give them freedom &amp; independence. I was booed &amp; heckled by Democrats.\n\nThey would rather protect illegal immigrants &amp; give them union memberships (when they shouldn't be working here in the 1st place)"), ('dates', '2020-02-07 18:33:05'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.6326218843460083)])
OrderedDict([('tweet', 'There is nothing quite as empowering as a job. No federal program has the power to pull a person out of poverty, no government handout can help a person find independence and success. Only through gainful employment can a person find true stability.\n\nhttps://t.co/TTMK2YZsQm'), ('dates', '2019-12-04 20:34:53'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.6079419851303101)])
OrderedDict([('tweet', "House Democrats refuse to make their impeachment inquiry legitimate because they know it's founded on false pretenses. \n\nTo treat @SteveScalise like this is shameful. Impeachment is a bipartisan process, not a one-sided quest for revenge. https://t.co/tHQPN7e9Fx"), ('dates', '2019-10-16 13:50:19'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.7411244511604309)])
OrderedDict([('tweet', "Today I watched a room of Democrats try to rationalize why it's better for bureaucrats to be centralized in the Swamp instead of   moving the Bureau of Land Management in the heart of the West - where 99% of the public lands they manage are. #DrainTheSwamp"), ('dates', '2019-09-10 16:38:03'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.9237464666366577)])
OrderedDict([('tweet', "It's time to act on the #USMCA. Pelosi has no reason to hold up the vote - except to spite @realDonaldTrump. \n\nThis deal is GOOD for workers, GOOD for business, and GOOD for America. It's time to pass the #USMCAnow! https://t.co/JNziIJCbM8"), ('dates', '2019-09-10 14:28:26'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.7437489628791809)])
OrderedDict([('tweet', "Today with @housebudgetGOP, we had yet another assault on OK's #1 job sector - energy.\n\nThe majority has focused so much time into this, we've nearly forgotten our committee's purpose - maybe that's why we still don't have a budget?\n#gopHEAT"), ('dates', '2019-07-24 17:41:16'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.7184191346168518)])
OrderedDict([('tweet', '"This committee has become a mockery of democracy and a mockery of good government." - @RepRobBishop\n\nHe\'s absolutely right. The Democrats on @NatResources don\'t care about moving our country forward - none of these bills have a shot at becoming law.'), ('dates', '2019-07-17 14:38:42'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.7091081738471985)])
OrderedDict([('tweet', "The Senate passed a bipartisan  border security package yesterday 84-8. It includes funding for border agents to end the crisis at our border. If the House can't pass the same bill, we shouldn't be going home this afternoon &amp; we absolutely shouldn't get a month recess in August."), ('dates', '2019-06-27 14:39:11'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.7516034245491028)])
OrderedDict([('tweet', 'We debated supplemental funding for the border in the House tonight. \n\nRemember, it’s been 56 days since President Trump declared an emergency and asked for Congress to pass funding.'), ('dates', '2019-06-25 23:51:17'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.5536739826202393)])
OrderedDict([('tweet', 'Another week has passed &amp; still no vote on the #USMCA in the House.'), ('dates', '2019-06-21 16:43:09'), ('file_name', 'Kevin Hern.csv'), ('pct_true', 0.5238037705421448)])
OrderedDict([('tweet', "How many lives have been destroyed because of communist China's lies?\n\nAnd yet Democrats have no plan to hold China accountable. They don't even have a plan to reconvene Congress."), ('dates', '2020-05-09 12:45:37'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.9987014532089233)])
OrderedDict([('tweet', 'Our country has neglected the warning signs from the Chinese Communist Party for far too long. So I am forming the China Task Force, led by @RepMcCaul.\n\nDemocrats were set to join but bailed with no explanation. Our country simply cannot wait any longer.\nhttps://t.co/T2GyqVli0B'), ('dates', '2020-05-07 18:31:15'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.9002492427825928)])
OrderedDict([('tweet', 'For years, China has conspired to overtake our universities, our press, and our pharmaceuticals.\n\nNever again. https://t.co/gjiUWVSTCq'), ('dates', '2020-05-06 20:26:26'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.7167916893959045)])
OrderedDict([('tweet', "Businesses across America are beginning to reopen their doors, but Speaker Pelosi refuses to get the House of Representatives back to work.\n\nCongress CAN and SHOULD operate in a safe and effective manner, and Republicans have a plan to restore the People's voice. https://t.co/5LEEdA0lL2"), ('dates', '2020-05-06 14:39:33'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.6442899107933044)])
OrderedDict([('tweet', "Congress cannot stay shut down in the midst of a pandemic.\n\nWe need a safe return to regular order—not more yes or no votes on big-spending bills that are crafted behind closed doors in the Speaker's office. https://t.co/yRz1T6M2cI"), ('dates', '2020-05-04 18:21:24'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.5828271508216858)])
OrderedDict([('tweet', "Mark my words → Speaker Pelosi's coronavirus committee is just Impeachment 2.0."), ('dates', '2020-04-29 22:50:50'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.6810052394866943)])
OrderedDict([('tweet', "The American people deserve elected leaders who will work day in and day out. That's not what they're getting from the Democrat-controlled House of Representatives. https://t.co/khkeUS2uzv"), ('dates', '2020-04-29 17:45:17'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.7886341214179993)])
OrderedDict([('tweet', "The Democrats have no plan—whether it's to hold communist China accountable or to reconvene Congress itself.\n\nThe American people deserve better. https://t.co/28Ta8stLa6"), ('dates', '2020-04-29 13:07:19'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.9936404824256897)])
OrderedDict([('tweet', "If there is one thing that should unite us as Americans, it's this. → We should never again allow ourselves to be dependent on Communist China for essential medical supplies. https://t.co/Ho5iplgWwp"), ('dates', '2020-04-28 17:25:00'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.743421733379364)])
OrderedDict([('tweet', 'Across the world, people have lost grandparents. Parents. Siblings. Children. All because China lied.\n\nNever again. Communist China must be held accountable. https://t.co/UxdxWAqk3i'), ('dates', '2020-04-27 23:37:00'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.8394079208374023)])
OrderedDict([('tweet', 'Unacceptable. → China’s medical supply monopoly encourages labor and environmental abuses that exacerbate an unfair global economic advantage. \n\nWe can never again leave America’s medical fate in the hands of another nation. https://t.co/JaLUlRRLH9'), ('dates', '2020-04-27 12:32:39'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.9233909845352173)])
OrderedDict([('tweet', '𝐍𝐞𝐯𝐞𝐫 𝐚𝐠𝐚𝐢𝐧. We can never again put ourselves in a position where we have to depend on Communist China for essential equipment. https://t.co/Tn8Ilt3Bpt'), ('dates', '2020-04-26 18:16:00'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.6692094206809998)])
OrderedDict([('tweet', 'While President @realDonaldTrump works to save America, Democrats have only been concerned with:\n⦾ Impeachment\n⦾ Investigations\n⦾ Ice cream'), ('dates', '2020-04-25 13:22:36'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.8702986836433411)])
OrderedDict([('tweet', 'Democrats have attempted to block nearly every action President Trump has taken to protect the American people from COVID-19.\n\nFor weeks, they put partisan politics ahead of public health. https://t.co/GR8PZkmT36'), ('dates', '2020-04-24 23:37:00'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.9538155198097229)])
OrderedDict([('tweet', "While President Trump worked to keep Americans safe from coronavirus,\n➡️ Democrats criticized the ban on flights from China.\n➡️ The WHO was saying it didn't transmit from person to person.\n➡️ Nancy Pelosi invited everyone to congregate in the streets of San Francisco. https://t.co/8XGxr3jni4"), ('dates', '2020-04-24 18:35:29'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.7773199677467346)])
OrderedDict([('tweet', 'Democrats are voting to create a political committee led by Joe Biden\'s top supporter—who wants to leverage COVID19 to "restructure" America.\n\nCongress already created 3 panels for Coronavirus funding oversight. Dems don\'t care—they just want more investigations.\n\nPure politics. https://t.co/lTmxh3qKiU'), ('dates', '2020-04-23 20:26:00'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.880608320236206)])
OrderedDict([('tweet', '4.4 million. That’s how many Americans lost their jobs in the last week—the exact time span that Democrats let the Paycheck Protection Program run out of money.\n\nIt didn’t have to be this way. These are Pelosi Layoffs.'), ('dates', '2020-04-23 12:40:09'), ('file_name', 'Kevin McCarthy.csv'), ('pct_true', 0.9598315358161926)])
OrderedDict([('tweet', "The House just voted to facilitate the ratification of the Equal Rights Amendment (ERA). Women deserve the same protections under the law as men. It's that simple. We need #ERANow!"), ('dates', '2020-02-13 16:48:31'), ('file_name', 'Kim Schrier.csv'), ('pct_true', 0.7042494416236877)])
OrderedDict([('tweet', 'Seventy-five years ago, Auschwitz was liberated, and we pledged “Never Again.” But now, the forces of hatred, bigotry, and antisemitism that led to the Holocaust are re-emerging all over the world.'), ('dates', '2020-01-27 20:44:08'), ('file_name', 'Kim Schrier.csv'), ('pct_true', 0.7199057936668396)])
OrderedDict([('tweet', 'Qassim Soleimani was without question an enemy of the US, with American blood on his hands. Managing the multiple threats posed by Iran and its proxies requires diplomatic finesse, coordination with our allies, and an appreciation for unintended consequences.'), ('dates', '2020-01-04 01:56:39'), ('file_name', 'Kim Schrier.csv'), ('pct_true', 0.786516010761261)])
OrderedDict([('tweet', 'The president is trying to distract the American people with delusional statements about missing masks in New York hospitals. We need real leadership, not these kind of conspiracy theories. https://t.co/FNlPR2gCfx'), ('dates', '2020-03-30 18:36:16'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.8400090932846069)])
OrderedDict([('tweet', 'We can give Americans the help and relief that they need, but sadly, Senate Republicans are beholden to donors and corporate interests, not the needs of the people. https://t.co/aOJhJRMS3S'), ('dates', '2020-03-23 17:07:01'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.6832091808319092)])
OrderedDict([('tweet', 'We need a stimulus package that is fair and equitable. What the Senate voted on today wasn’t even close to what the American people need. https://t.co/JbqByF7jxE'), ('dates', '2020-03-23 01:14:54'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.6029505729675293)])
OrderedDict([('tweet', 'Once again, the Trump administration is misusing the federal government for political retribution. There is no factual basis for this policy—its true design is to punish New York. https://t.co/7DwtVhVdGE'), ('dates', '2020-02-06 17:00:21'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.6885854601860046)])
OrderedDict([('tweet', 'The Senate failed the American people today. \n \nDespite overwhelming evidence that President Trump abused his power and obstructed Congress, Senate Republicans voted to acquit him, putting personal ambitions and political futures ahead of the Constitution they swore to uphold.'), ('dates', '2020-02-05 21:33:36'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.961173951625824)])
OrderedDict([('tweet', 'The Trump administration just announced a horrible plan to gut Medicaid, their latest attempt to restrict access to affordable health care.\n\nPeople across the country have spoken loud and clear – they like Medicaid and don’t want to see it cut. We must fight back. https://t.co/HV3F5NCgMr'), ('dates', '2020-01-30 18:36:39'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.6001219749450684)])
OrderedDict([('tweet', 'The House managers of this impeachment trial have made a compelling case that the president willfully abused the power of his office. Here’s why that’s so important: https://t.co/mYdOPJTU8J'), ('dates', '2020-01-23 23:30:00'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.5845642685890198)])
OrderedDict([('tweet', "Congress must assert its war powers authority before it's too late. \n\nThat's why I'm sponsoring @SenSanders's No War With Iran Act to prohibit the president from taking any further military action involving Iran, unless he notifies Congress immediately and obtains authorization. https://t.co/SOV2gw7O4l"), ('dates', '2020-01-09 21:27:03'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.5688202381134033)])
OrderedDict([('tweet', "It’s been two years since President Trump’s FCC repealed #NetNeutrality. Now, massive corporations can discriminate against content they don’t like and restrict access for American consumers. We need a free and open internet—let's demand a vote on the #SaveTheInternet Act."), ('dates', '2019-12-10 18:31:04'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.8034525513648987)])
OrderedDict([('tweet', "After today's reporting by the @washingtonpost suggesting that administration officials—and potentially military officials—misled the American public about the Afghan War, I'm calling for hearings to address these concerning revelations and review our strategy in Afghanistan. https://t.co/inBmq9zNKT"), ('dates', '2019-12-09 21:56:02'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.5491910576820374)])
OrderedDict([('tweet', 'These explosive documents and revelations shed a light on nearly two decades of unending conflict in Afghanistan—and make clear that we need fundamental reform to put a stop to our country’s forever wars.\n\nCongress needs to take back its war powers.\n\nhttps://t.co/EtUjWlm0YV'), ('dates', '2019-12-09 17:53:25'), ('file_name', 'Kirsten E. Gillibrand.csv'), ('pct_true', 0.6828134059906006)])
OrderedDict([('tweet', 'Students defrauded by for-profit colleges are often left with crippling debt, useless degrees and none of the opportunities they were promised. This is wrong and today the House of Representatives reversed an unnecessarily cruel rule, enacted by Secretary DeVos.'), ('dates', '2020-01-16 19:05:45'), ('file_name', 'Kurt Schrader.csv'), ('pct_true', 0.890268087387085)])
OrderedDict([('tweet', 'I have long opposed U.S. intervention in the Middle East. I cosponsored and voted for a resolution today that would prevent the President from taking further military action in Iran without Congressional approval.'), ('dates', '2020-01-09 23:05:22'), ('file_name', 'Kurt Schrader.csv'), ('pct_true', 0.8271979093551636)])
OrderedDict([('tweet', 'Hoy es el Día de Apreciación a los Cónyuges Militares. Gracias a todos quienes aman y apoyan a nuestros valientes miembros del servicio.'), ('dates', '2020-05-08 18:56:00'), ('file_name', 'Kyrsten Sinema.csv'), ('pct_true', 0.820640504360199)])
OrderedDict([('tweet', 'Today, Senate Democrats blocked additional funding to help small businesses keep paychecks coming to hundreds of thousands of workers across the country. \nThis is no time for Senate Democrats to play political games.'), ('dates', '2020-04-09 19:06:18'), ('file_name', 'Lamar Alexander.csv'), ('pct_true', 0.7605423331260681)])
OrderedDict([('tweet', 'Attacking @TVAnews doesn’t do one thing to solve the pandemic and has no place in federal COVID-19 response legislation. \n\nTVA does not receive one dollar in federal taxpayer subsidies or federal appropriations.'), ('dates', '2020-04-09 01:38:20'), ('file_name', 'Lamar Alexander.csv'), ('pct_true', 0.7812439203262329)])
OrderedDict([('tweet', 'In the @wherald: “…There is a step that Congress should take to have an immediate positive impact and reassure patients seeking care that they will not be stuck with a surprise medical bill that could bankrupt their family.” \n\nhttps://t.co/jQisHeBVk9'), ('dates', '2020-04-02 14:35:26'), ('file_name', 'Lamar Alexander.csv'), ('pct_true', 0.7899365425109863)])
OrderedDict([('tweet', 'Current and former officials at the U.S. DOJ who are responsible for railroading Gen. Michael Flynn in an attempt to take down a duly elected President now need to be held accountable for Prosecutorial misconduct. If this can happen to Gen. Flynn it could happen to any of us.'), ('dates', '2020-05-07 19:57:03'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.6930432915687561)])
OrderedDict([('tweet', 'For weeks the media/Dems criticized @realDonaldTrump saying he doesn’t have a “National” plan for Covid 19 (he does through CDC, FDA, NIH etc.) Now that he is developing an economic “reopening” plan they say he doesn’t have the authority to tell states what to do! Which is it?'), ('dates', '2020-04-14 16:39:39'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.9952329993247986)])
OrderedDict([('tweet', 'Yet again, Washington Democrats are unwilling to put politics aside and work with Republicans to quickly provide needed support for the American people during this health care crisis. Employers and employees across Indiana and the nation are hurting through no fault of their own.'), ('dates', '2020-04-09 19:48:31'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.766821563243866)])
OrderedDict([('tweet', 'So the @whca removed the reporter from a conservative news outlet from the briefing rotation at the WH (they were invited by the Admin. ?)and allowed a reporter with known ties to the Communist Chinese government in? She promptly blurted out communist propaganda on world wide TV.'), ('dates', '2020-04-07 12:09:32'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.9867610931396484)])
OrderedDict([('tweet', 'These unprecedented times call for unity to help Hoosiers and all Americans who are struggling medically and/or financially through no fault of their own, which is why I support the CARES Act. https://t.co/Wd2y47QT7x'), ('dates', '2020-03-27 16:13:10'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.5582839250564575)])
OrderedDict([('tweet', 'When @realDonaldTrump gave a goal of Easter to begin opening up our country the media and liberals criticized him. How will they respond when Cuomo says essentially the same thing? https://t.co/55RyxNFPJB'), ('dates', '2020-03-26 22:45:33'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.6552256941795349)])
OrderedDict([('tweet', 'Democrats are intentionally misleading the American people by politicizing the coronavirus. Instead of playing politics, we need to work together to support the professional health experts working to contain this virus &amp; protect the American people. https://t.co/tNKHbDphhZ'), ('dates', '2020-03-04 15:23:24'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.9609823226928711)])
OrderedDict([('tweet', 'Today marks the end of a sham impeachment process that  began the moment President Trump was elected to office because Speaker Pelosi and Washington Democrats simply cannot accept the results of the 2016 election.\nhttps://t.co/4fmE9UieN3 https://t.co/NONV1ACFD5'), ('dates', '2020-02-05 21:43:41'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.5448183417320251)])
OrderedDict([('tweet', 'Democrats know that President Trump will continue to appoint conservative judges to the federal courts including the Supreme Court if he is re-elected. Democrats are in a panic over this situation and are willing to railroad the President out of office to stop him. Shameful.'), ('dates', '2020-01-25 17:24:26'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.9811041355133057)])
OrderedDict([('tweet', 'I just finished listening to the President’s legal team present to the U.S. Senate. It is clear that House Democrats do not have a case and are attempting to have the President removed from office because they don’t like him and don’t believe they can beat him at the ballot box.'), ('dates', '2020-01-25 17:15:04'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.6468628644943237)])
OrderedDict([('tweet', 'Speaker Nancy Pelosi slammed these sham articles of impeachment through the House at breakneck speed. Her actions are not justified, her reasons are not sound, and her attempt to interfere in the Senate process is tantamount to a child throwing a fit https://t.co/B78k5S0Dq5'), ('dates', '2020-01-15 18:50:33'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.7576118111610413)])
OrderedDict([('tweet', 'Current Democrat Members of Congress, Democrat Presidential candidates and former Obama administration officials are now going as far as defending terrorists and supporting Iran over the U.S. in order to criticize President Trump. Let that sink in.'), ('dates', '2020-01-05 17:17:42'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.814470648765564)])
OrderedDict([('tweet', 'The President acted appropriately to protect Americans and our allies in the Middle East. This action did not require Congressional authorization. Soleimani was a terrorist and was planning future attacks. Iran should realize America’s resolve to defend ourselves and stand down.'), ('dates', '2020-01-03 20:48:36'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.7770376801490784)])
OrderedDict([('tweet', 'Speaker Pelosi is now holding the articles of impeachment and not sending them to the Senate. What are the Dems afraid of? I thought President Trump was an imminent threat to the Constitution and our Democracy? Politics plain and simple. Give me a break! See you next November.'), ('dates', '2019-12-19 19:48:37'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.8718083500862122)])
OrderedDict([('tweet', 'This is a sad and dangerous moment in our history as impeachment is being used to undo the will of the American people and silence the voices of millions of Americans in the process. https://t.co/7NBVuJKlKg'), ('dates', '2019-12-18 23:02:48'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.8050824999809265)])
OrderedDict([('tweet', 'According to @SpeakerPelosi, she would only move forward with impeachment if:\n- There was something compelling,\n- And overwhelming,\n- And bipartisan.\n \nWith none of these conditions applying, it is clear even by her standards that this impeachment is a political sham.'), ('dates', '2019-12-13 20:22:11'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.5987690687179565)])
OrderedDict([('tweet', 'House Democrat impeachment press conference today\n3 CA Reps\n3 NY Reps\n1 MA Rep\n0 Middle America Reps\nCoastal Dems think they can control the entire nation &amp; forget about citizens in mid America. If Dems get their way &amp; eliminate the Electoral College your voices will be silenced. https://t.co/bYivfD2BJR'), ('dates', '2019-12-10 17:27:39'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.5845900774002075)])
OrderedDict([('tweet', 'House Democrats have been obsessed with impeaching President Trump since he was elected in 2016. Their blatant refusal to accept the will of the American people in that election undermines our democracy and effectively silences the voices of millions of Americans. https://t.co/okg97Tec6k'), ('dates', '2019-12-10 15:57:06'), ('file_name', 'Larry Bucshon.csv'), ('pct_true', 0.8342745304107666)])
OrderedDict([('tweet', 'No one should feel unsafe in their community—especially not in their own home. During #COVID19 we must ensure that those most vulnerable to #domesticviolence are safe. It’s time the Senate reauthorized Violence Against Women Act without delay! #VAWA'), ('dates', '2020-04-07 20:40:53'), ('file_name', 'Lauren Underwood.csv'), ('pct_true', 0.5486269593238831)])
OrderedDict([('tweet', 'All 53 interview transcripts of the House’s Trump-Russia probe were unanimously approved for release by @HouseIntelComm in Sept 2018. All have been signed off on by the Intel Community to protect classified info. Why aren’t these transcripts being released, @RepAdamSchiff?'), ('dates', '2020-05-06 15:18:22'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.623843252658844)])
OrderedDict([('tweet', 'Today the US Senate is going back into session in DC. The House is not but should. It’s time and it’s not too late for the powers that be to change her mind.'), ('dates', '2020-05-04 11:51:18'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.6545978784561157)])
OrderedDict([('tweet', 'On #YomHaShoah or #HolocaustRemembranceDay we honor the innocent lives of 6 million Jews &amp; millions of others who were tragically murdered &amp; the survivors who suffered so much. Let us renew our pledge to never allow this darkest form of evil to ever rise again. https://t.co/BggFqNe2he'), ('dates', '2020-04-20 23:38:00'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.6057801842689514)])
OrderedDict([('tweet', 'While @realDonaldTrump believes the enemy is coronavirus, @SpeakerPelosi believes the enemy is Pres Trump. In order to defeat COVID-19, we must have a clear head &amp; united, common purpose to confront and end this pandemic. #FundPPP https://t.co/pp50vZ12aB'), ('dates', '2020-04-18 14:45:28'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.7693800926208496)])
OrderedDict([('tweet', 'The Paycheck Protection Program is running dry and many small businesses will never reopen again if Congress doesn’t immediately get back to work to provide the necessary support. \n\nHalf of the US workforce is made up of small business employees.\n\nCongress needs to do its job!'), ('dates', '2020-04-16 01:54:33'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.5143685340881348)])
OrderedDict([('tweet', 'In the future, between the national stockpile, state stockpiles, and an emergency domestic manufacturing plan, never again should the United States rely on a foreign government for ventilators or Personal Protective Equipment (PPE) to respond to a pandemic.'), ('dates', '2020-04-14 22:35:17'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.7247252464294434)])
OrderedDict([('tweet', 'First we must get through the challenge immediately before us, but never again should the US rely on China or any other country for that matter to provide PPE/ventilators to respond to a pandemic. We should stockpile enough of both to be fully prepared for a nationwide pandemic.'), ('dates', '2020-04-02 01:57:27'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.6415573358535767)])
OrderedDict([('tweet', 'The Speaker should sit this one out trying to spend this critical time playing the coronavirus blame game.\n\nI vividly remember that the same day Pres Trump first met with his coronavirus Task Force on January 29, 2020, Congress was being hijacked with her sham impeachment trial. https://t.co/UaQVrnfFwn'), ('dates', '2020-03-30 03:04:27'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.6624788045883179)])
OrderedDict([('tweet', 'This lingering bitterness is ugly and not helpful and must end. We need to all be Americans on the same team here.\n\nTo combat coronavirus, we cannot be Republicans or Democrats first.\n\nWe have to Americans First. https://t.co/P3VomOIjpu'), ('dates', '2020-03-27 21:05:54'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.5388087630271912)])
OrderedDict([('tweet', "Gov. Cuomo just said the stimulus package could've &amp; should've provided additional support for the NYS budget. \n\nHe is right.\n\nHere's the context not mentioned:\n\nMcConnell offered the FMAP language Cuomo asked for &amp; Schumer blocked it, resulting in the loss of SIX BILLION for NY."), ('dates', '2020-03-26 16:17:02'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.7248348593711853)])
OrderedDict([('tweet', "Long Island has more coronavirus cases than any state in the US. We aren't Republicans or Democrats. We are Long Islanders, New Yorkers &amp; Americans first. Our message to ANYONE consumed by partisan politics is to knock if off &amp; be part of a solely coronavirus focused solution!"), ('dates', '2020-03-23 22:50:28'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.5705116987228394)])
OrderedDict([('tweet', "In @NYGovCuomo's last presser, he mentioned how NY would lose $6B in fed $ if the Senate didn't tweak its new bill. He's correct. It would be counterintuitive &amp; awful policy to penalize NY for reigning in Medicaid costs. It wouldn't make any sense.\nhttps://t.co/aXNrAF9nYH"), ('dates', '2020-03-22 04:06:41'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.6504241824150085)])
OrderedDict([('tweet', 'This WaPo WH reporter @jdawsey1 is deliberately leaving out here the rest of Dr Fauci’s sentence/statement contradicting what this reporter is trying to imply w/this cherry picked part of a sentence. \n\nIs there a better time than now to STOP this garbage? #InItTogether https://t.co/jIgOoMxuqT'), ('dates', '2020-03-21 19:59:20'), ('file_name', 'Lee M. Zeldin.csv'), ('pct_true', 0.6446585059165955)])
OrderedDict([('tweet', "May is Asian Pacific American Heritage Month when we celebrate the contributions &amp; heritage of this diverse group of Americans. At a time when there's so much prejudice towards minorities, it's also important to dispel misinformation and instead lift up #AAPI stories. #APAHM2020"), ('dates', '2020-05-01 20:49:49'), ('file_name', 'Linda T. Sánchez.csv'), ('pct_true', 0.5040621757507324)])
OrderedDict([('tweet', 'The #COVID19 pandemic underscores just how deadly #HealthDisparities are for people of color in this country. We need to learn the lessons of this crisis and take a data-driven approach towards #HealthEquity. #HEAA2020'), ('dates', '2020-04-28 19:08:42'), ('file_name', 'Linda T. Sánchez.csv'), ('pct_true', 0.6496140360832214)])
OrderedDict([('tweet', 'Too many small businesses had problems accessing loans through the PPP program. Democrats demanded changes so that these dollars get to the hardworking men and women who need them. Tomorrow, the House will vote on an interim relief package to provide them with additional support. https://t.co/MmEbVePc3h'), ('dates', '2020-04-22 15:12:37'), ('file_name', 'Linda T. Sánchez.csv'), ('pct_true', 0.7247766256332397)])
OrderedDict([('tweet', "75 years after the Allies liberated Auschwitz, #WeRemember the more than 6 million lives taken and recommit ourselves to #NeverForget. During difficult times, it's so important to stand up against bigotry and injustice.\n\n#HolocaustRemembranceDay"), ('dates', '2020-04-21 19:22:16'), ('file_name', 'Linda T. Sánchez.csv'), ('pct_true', 0.8678650259971619)])
OrderedDict([('tweet', 'The U.S. has more #COVIDー19 cases than any other country. Instead of working with Congress to help families get back to work or with states to ramp up testing, Trump is halting all legal immigration. Failing to meet this challenge is much more dangerous than any immigrant.'), ('dates', '2020-04-21 13:15:57'), ('file_name', 'Linda T. Sánchez.csv'), ('pct_true', 0.7989974617958069)])
OrderedDict([('tweet', 'Americans are getting their stimulus checks later because the President wants his name on it. I support @WaysMeansCmte Oversight Subcommittee Chairman @repjohnlewis &amp; Select Revenue Measures Subcommittee Chairman @RepThompson in asking what we all want to know—WHY? https://t.co/qp5A7dLuCJ'), ('dates', '2020-04-16 15:04:59'), ('file_name', 'Linda T. Sánchez.csv'), ('pct_true', 0.5949304699897766)])
OrderedDict([('tweet', 'President Trump is delaying checks to the neediest Americans by demanding that his signature be printed on them. We are in the middle of a deadly pandemic and people are struggling, yet this is what he’s focused on right now. We deserve better. https://t.co/N61rVJx86N'), ('dates', '2020-04-15 16:57:20'), ('file_name', 'Linda T. Sánchez.csv'), ('pct_true', 0.7054101824760437)])
OrderedDict([('tweet', 'Suspending funding to the @WHO during a pandemic is reckless and dangerous. Instead of blaming the WHO, we need to be focusing on solutions that SAVE LIVES and NOT jeopardize the health of our country and the world.\nhttps://t.co/e5f3aysMQB'), ('dates', '2020-04-15 15:20:00'), ('file_name', 'Linda T. Sánchez.csv'), ('pct_true', 0.618211567401886)])
OrderedDict([('tweet', 'There are thousands of #Dreamers working on the frontlines of the #CoronavirusOutbreak.\n\nIf the Trump Administration is truly “inspired” by #DACA recipients who are essential workers, the President should drop the DACA case at SCOTUS &amp; extend protections.\n\n#ProtectTheDream'), ('dates', '2020-04-10 18:11:01'), ('file_name', 'Linda T. Sánchez.csv'), ('pct_true', 0.8003165125846863)])
OrderedDict([('tweet', 'I think the Supreme Court will strike down President Obama’s immigration action. https://t.co/ftdtkDocA1'), ('dates', '2016-01-19 14:56:26'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.5906532406806946)])
OrderedDict([('tweet', "This wasn't an academic debate.  There were real people holding real jobs who were hurt by the closing of EXIM. #EximBank"), ('dates', '2015-12-04 02:17:56'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.9541367292404175)])
OrderedDict([('tweet', '4) No prohibition on sending American forces on the ground to combat ISIL.'), ('dates', '2015-12-03 17:00:40'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.5183047652244568)])
OrderedDict([('tweet', '2) No geographic limits are placed on American military in the fight against ISIL.'), ('dates', '2015-12-03 16:59:50'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.709844708442688)])
OrderedDict([('tweet', 'We don’t have enough American troops inside of Iraq to destroy ISIL anytime soon.'), ('dates', '2015-11-29 15:49:02'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.774937093257904)])
OrderedDict([('tweet', 'About to go on CBS @FaceTheNation from Baghdad about the threat posed by radical Islam.'), ('dates', '2015-11-29 15:43:23'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.5959713459014893)])
OrderedDict([('tweet', 'AUMF BACKGROUND #2) No geographic limits are placed on American military or intelligence services in the fight against ISIL.'), ('dates', '2015-11-18 17:45:14'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.6224763989448547)])
OrderedDict([('tweet', 'The American people\nare ready to take on radical Islam because they see it for what it is -- a threat to our homeland and way of life.'), ('dates', '2015-11-16 19:55:57'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.9064962863922119)])
OrderedDict([('tweet', "If we don't destroy ISIL they will grow stronger. And it's clear -- they want to kill Americans."), ('dates', '2015-11-06 13:11:07'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.5269252061843872)])
OrderedDict([('tweet', 'President Obama is putting fifty brave Americans at risk without a clear strategy of how to degrade and destroy ISIL. #Syria'), ('dates', '2015-10-30 18:11:58'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.7336575984954834)])
OrderedDict([('tweet', 'ISIL is not going to be intimidated by this move. In fact, ISIL will see this as yet another sign of President Obama’s weakness.  #Syria'), ('dates', '2015-10-30 18:10:49'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.8654303550720215)])
OrderedDict([('tweet', 'For those who want to avoid responsibility for their actions regarding #Benghazi, Trey Gowdy has become their worst nightmare.'), ('dates', '2015-10-22 14:08:13'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.5528548955917358)])
OrderedDict([('tweet', '.@TGowdySC -- For those who want to avoid responsibility for their actions regarding #Benghazi, Trey Gowdy has become their worst nightmare.'), ('dates', '2015-10-20 16:08:45'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.9045570492744446)])
OrderedDict([('tweet', '.@tgowdy -- I have complete confidence Trey Gowdy will not stop investigating until all the facts about what happened in Benghazi are known.'), ('dates', '2015-10-20 16:05:50'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.5007429718971252)])
OrderedDict([('tweet', 'From past experiences, I know the media &amp; many Americans will reflexively blame the GOP should a government shutdown occur.'), ('dates', '2015-09-24 19:24:22'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.7052775621414185)])
OrderedDict([('tweet', 'We do NOT currently have 290 members of the House of Representatives, along with 67 Senators, who will vote to override a presidential veto.'), ('dates', '2015-09-24 19:21:44'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.8388948440551758)])
OrderedDict([('tweet', 'Instead we should set our sights on winning the war &amp; that only comes about by electing a President &amp; Congress committed to protecting life.'), ('dates', '2015-09-24 19:20:01'), ('file_name', 'Lindsey Graham.csv'), ('pct_true', 0.7009778618812561)])
OrderedDict([('tweet', 'New data shows that air quality is getting worse, yet the EPA is continuing to rollback basic public health protections. This is irresponsible and dangerous.\nhttps://t.co/jvcP8c85FM'), ('dates', '2020-05-05 15:30:25'), ('file_name', 'Lisa Blunt Rochester.csv'), ('pct_true', 0.5593277215957642)])
OrderedDict([('tweet', 'We must listen to our scientists and public health experts. EPA’s *own* public health experts say limiting air pollution will save American lives.'), ('dates', '2020-04-16 14:23:33'), ('file_name', 'Lisa Blunt Rochester.csv'), ('pct_true', 0.5318955183029175)])
OrderedDict([('tweet', '*Attention Delaware*\n \nUnfortunately, scammers are taking advantage of the anxiety consumers are feeling during the #COVID19 pandemic. During a State of Emergency in Delaware, price gouging is illegal.'), ('dates', '2020-04-01 20:42:17'), ('file_name', 'Lisa Blunt Rochester.csv'), ('pct_true', 0.5005273818969727)])
OrderedDict([('tweet', 'We can’t leave our front line workers fighting #COVID19 without the supplies they need to save American lives. The Medical Supply Chain Emergency Act will force the President to federalize American manufacturing &amp; end the supply shortage. We need the #DefenseProductionActNOW. https://t.co/lCzGc5L8jc'), ('dates', '2020-03-27 18:55:58'), ('file_name', 'Lisa Blunt Rochester.csv'), ('pct_true', 0.6102443933486938)])
OrderedDict([('tweet', 'The #COVID19 pandemic has had a devastating impact on Alaska’s seafood sector and our world-class fisheries. Alaska produces about 60 percent of our nation’s seafood, and families across the country are reliant on Alaskan seafood products to keep their families fed.'), ('dates', '2020-05-05 00:24:42'), ('file_name', 'Lisa Murkowski.csv'), ('pct_true', 0.5076474547386169)])
OrderedDict([('tweet', 'Good news on our efforts to press the Trump administration to issue guidance that better accommodates seasonal employers devastated by the coronavirus pandemic—today the @USTreasury issued a rule change that takes into account the uniqueness of seasonal businesses and workers.'), ('dates', '2020-04-27 22:44:40'), ('file_name', 'Lisa Murkowski.csv'), ('pct_true', 0.5321366786956787)])
OrderedDict([('tweet', 'Barely two weeks ago, Congress unanimously agreed that Alaska Native Corporations should be eligible for funding under the CARES Act and we expect Treasury to uphold and follow the law.'), ('dates', '2020-04-16 23:16:55'), ('file_name', 'Lisa Murkowski.csv'), ('pct_true', 0.9018523097038269)])
OrderedDict([('tweet', 'Congress must be united to counter the threat posed by China. Earlier this year, Democrats walked away from establishing a bipartisan task force.\n\nInstead of launching attacks against @GOPLeader &amp; @HouseGOP, why won’t they join us to defend the nation from the lies of the CCP? https://t.co/Lp8RXEEwNr'), ('dates', '2020-05-07 18:56:45'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.82869952917099)])
OrderedDict([('tweet', "There is no question that because of the Chinese Communist Party's lies, because of their cover-up, because of the way that they’ve operated, they caused COVID-19 to spread globally and they’ve got to be held accountable. https://t.co/mnF4errzaP"), ('dates', '2020-05-04 16:41:06'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.9843369126319885)])
OrderedDict([('tweet', "Speaker Pelosi still refuses to consider any plan to allow the House to operate or function remotely. Sidelining and paralyzing the People's House at a moment of national crisis is unacceptable. It's time for the House to do its job."), ('dates', '2020-04-29 19:15:25'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.8011384010314941)])
OrderedDict([('tweet', 'The Chinese Communist Party cannot be allowed to blackmail the United States. We must lead the world in holding them accountable https://t.co/DJhUnW6aYb https://t.co/j4fdwTWKX3'), ('dates', '2020-04-23 22:20:08'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.604265034198761)])
OrderedDict([('tweet', 'Speaker Pelosi and Congressional Democrats have been playing political games since the COVID-19 crisis began and they are hurting American families. More funds are desperately needed for the Paycheck Protection Program now. https://t.co/DNzG6UF9nM'), ('dates', '2020-04-15 22:53:31'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.6243846416473389)])
OrderedDict([('tweet', 'As long as the World Health Organization is doing the bidding of the Chinese Communist Party, they’re costing lives not saving lives. We must hold the CCP accountable for its lies and disinformation efforts about the spread of coronavirus. https://t.co/mFGMt8Ttal'), ('dates', '2020-04-15 18:51:37'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.742479145526886)])
OrderedDict([('tweet', 'Nancy Pelosi and Senate Democrats are once again blocking small business relief. This is despicable. The American people are going to hold Democrats accountable for blocking money to people who are hurting https://t.co/FIctvld2ea'), ('dates', '2020-04-09 15:21:13'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.9567893147468567)])
OrderedDict([('tweet', 'Democrats continue to block legislation that would provide crucial emergency relief to Americans who are suffering from the coronavirus pandemic and economic crisis. They are holding hostage lifesaving aid in order to push their “Green New Deal” agenda. https://t.co/TbiYy5Wh9a https://t.co/xRjYjAhNpq'), ('dates', '2020-03-23 23:26:45'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.9219628572463989)])
OrderedDict([('tweet', "In the face of a global health crisis and a global economic crisis, Democrat leaders have proven that they can't be trusted to\xa0put the priorities of the American people first."), ('dates', '2020-03-23 18:03:13'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.7571614980697632)])
OrderedDict([('tweet', 'The U.K. government’s decision to allow Huawei into its 5G network will embed the Chinese Communist Party into the United Kingdom’s telecom infrastructure. That poses a clear national security risk to the U.K. and the U.S. \nI hope our ally’s misguided decision is reversed. https://t.co/HvhM5CCgNL'), ('dates', '2020-03-10 18:23:19'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.9357362389564514)])
OrderedDict([('tweet', 'Our rural communities depend on small refineries to support thousands of jobs. But a recent court ruling could put those jobs at risk. The EPA must take action and protect the role small refineries play in securing U.S. energy independence. Learn more: https://t.co/n4gICQdxwz'), ('dates', '2020-03-04 20:27:02'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.5048837065696716)])
OrderedDict([('tweet', 'Socialism extinguishes freedom. There’s nobody who understands this more than people whose families came to the United States from Cuba. https://t.co/f4jhWyoesT https://t.co/icnjU0fYaY'), ('dates', '2020-02-27 15:23:52'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.5291973352432251)])
OrderedDict([('tweet', 'Let’s get something straight Bernie: socialism is authoritarianism. There aren’t enough literacy programs in the world that will ever change this undeniable fact. https://t.co/zlQi9LKz2J'), ('dates', '2020-02-25 15:56:12'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.6925100088119507)])
OrderedDict([('tweet', 'Between their embrace of socialism and obsession with impeachment, the Democrats have demonstrated that they absolutely cannot be trusted. https://t.co/MzbLCPfC1n'), ('dates', '2020-02-11 16:12:59'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.820556104183197)])
OrderedDict([('tweet', 'The President Has Been Acquitted; The American People Will Hold Democrats Accountable For Abandoning The Constitution https://t.co/ei2lMr9TgD https://t.co/tC1bZOulah'), ('dates', '2020-02-05 22:00:26'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.7257474064826965)])
OrderedDict([('tweet', "The Democrats' impeachment sham and embrace of socialism proves they cannot be trusted with power. Listen to my full interview with @guypbenson below: https://t.co/qKYuC71NHC"), ('dates', '2020-02-04 14:28:59'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.5921069979667664)])
OrderedDict([('tweet', 'House Democrats: “Our evidence is overwhelming and compelling.”\n\nAlso House Democrats: “The Senate is engaging in a cover-up if they rely on our evidence and don’t call more witnesses.”\n\n#EndTheSham'), ('dates', '2020-01-31 11:28:45'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.8584425449371338)])
OrderedDict([('tweet', "WATCH: The proposals the House voted on earlier today would undermine President Trump's Constitutional authority and would weaken America's national security. https://t.co/24xTbwOtmj"), ('dates', '2020-01-30 21:45:18'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.7567635774612427)])
OrderedDict([('tweet', 'As we commemorate Holocaust Remembrance Day, which comes 75 years after the liberation of Auschwitz, we honor the millions of Jews brutally murdered by the Nazis while rededicating ourselves to opposing anti-Semitism in all its forms so that this never happens again. #WeRemember https://t.co/DuySR1uGhj'), ('dates', '2020-01-27 18:15:51'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.5306451916694641)])
OrderedDict([('tweet', 'This morning the White House legal team began taking apart the House Democrats’ impeachment case, methodically and completely. \n\nAnd, they reminded the American people, once again, that when @RepAdamSchiff is talking he’s lying.'), ('dates', '2020-01-25 17:50:41'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.9026005268096924)])
OrderedDict([('tweet', 'The disastrous Iran deal negotiated by the Obama Admin. was designed to allow them to cheat. The U.S. should once-and-for-all end all civil-nuclear waivers, which Iran has long exploited. Read my statement w/ @SenTedCruz, @GrahamBlog &amp; @SenTomCotton here: https://t.co/OF06tPAg5a'), ('dates', '2020-01-23 16:31:21'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.5773740410804749)])
OrderedDict([('tweet', 'Huawei is controlled by the Chinese Communist Party and its efforts to infiltrate 5G are nefarious at their core. Allowing the company into the U.K.’s 5G networks would pose a national security threat that could not be mitigated or contained. https://t.co/bS4AhL9aaa'), ('dates', '2020-01-21 15:52:57'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.7956165075302124)])
OrderedDict([('tweet', "Rep. Elissa Slotkin's resolution is a partisan effort to undermine the President and America's national security. Listen to my full interview with @DaveAkerlyOnAir on WILS-Lansing here: https://t.co/oKeVcMtcP3"), ('dates', '2020-01-09 16:47:19'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.5649397969245911)])
OrderedDict([('tweet', 'Now that Iran has backed out of the catastrophic Obama nuclear deal, there are no more pretenses of compliance. The U.S. should once and for all end all civil-nuclear waivers, which Iran has long exploited. Read my statement with @SenTedCruz &amp; @GrahamBlog: https://t.co/sHjQjztfw7'), ('dates', '2020-01-07 15:18:15'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.6287674903869629)])
OrderedDict([('tweet', "House Democrats' partisan, reckless, dangerous articles of impeachment threaten our Republic. Tonight, I spoke on the House floor about why I am voting no. https://t.co/c8CAtJRXCO"), ('dates', '2019-12-19 00:35:36'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.6886751055717468)])
OrderedDict([('tweet', "The American people deserve to know precisely which members of the House are voting to impeach a duly-elected President. The Democrats don't want that. They just rejected my motion for a manual call of the roll.\n\nThey're afraid because this process has been a sham from the start. https://t.co/iQRshbglou"), ('dates', '2019-12-18 16:00:54'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.8712472319602966)])
OrderedDict([('tweet', 'The Democrats are trying to remove the President from office based on presumptions, not based on facts, not based on evidence. It’s an affront to the Constitution. It’s an affront to their oath of office. https://t.co/A6GZOVRwfX'), ('dates', '2019-12-10 21:35:00'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.8604211211204529)])
OrderedDict([('tweet', 'The American people will hold the Democrats accountable for failing to provide for the defense of the nation. https://t.co/lja3e9utkO'), ('dates', '2019-12-04 20:11:17'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.8854105472564697)])
OrderedDict([('tweet', 'Speaker Pelosi has said time and again that if the House is going to move forward on impeachment, the evidence needs to be clear, compelling, and bipartisan. The Democrats fail on every single one of those counts. https://t.co/83g9ExlnUX'), ('dates', '2019-12-04 01:22:54'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.5691359043121338)])
OrderedDict([('tweet', 'The House just passed yet another short-term funding bill despite the invariable harm it inflicts on our national defense. Instead of shifting focus to properly funding our military, the Democrats are still choosing to prioritize partisan impeachment.'), ('dates', '2019-11-19 20:25:45'), ('file_name', 'Liz Cheney.csv'), ('pct_true', 0.8158218860626221)])
OrderedDict([('tweet', "75 years ago, Allied forces accepted Nazi Germany's unconditional surrender, ending #WWII in Europe. Brave people from all across #TX07, our country, and the world brought down fascism and totalitarianism in Europe—and for that, I'm grateful. #VEDay https://t.co/IP7jTO47iH"), ('dates', '2020-05-08 19:38:18'), ('file_name', 'Lizzie Fletcher.csv'), ('pct_true', 0.7946828007698059)])
OrderedDict([('tweet', 'FDA’s “anything goes” policy for #COVID19 antibody testing has been an invitation for fraud, with tests whose results currently offer little value, while misinforming important risk management decisions. (1/2)'), ('dates', '2020-05-05 21:44:32'), ('file_name', 'Lloyd Doggett.csv'), ('pct_true', 0.5073436498641968)])
OrderedDict([('tweet', 'Outrageous that many American citizens are being denied relief checks solely because they file a joint tax return with a spouse who is not a citizen.\n\nYet another way Trump’s bigotry dictates denying help for some families, to the detriment of all of us. https://t.co/EqJwN5hcfl'), ('dates', '2020-04-28 21:05:12'), ('file_name', 'Lloyd Doggett.csv'), ('pct_true', 0.7144169807434082)])
OrderedDict([('tweet', 'Chilling reality is that Trump’s delay, denial of the imminent pandemic cost American lives.\n\n@FinancialTimes Edward Luce: “Epidemiologists say that if the US shutdown had taken place two weeks earlier, 90 per cent of the deaths would have been prevented.”\nhttps://t.co/3iVOdwkdms'), ('dates', '2020-04-28 13:42:43'), ('file_name', 'Lloyd Doggett.csv'), ('pct_true', 0.6006928086280823)])
OrderedDict([('tweet', 'GA governor opened tattoo parlors on Friday amidst this pandemic.\n\nWhat would be appropriate tattoo? A skull and crossbones ☠️ with “Born to Endanger Healthcare Workers &amp; Neighbors”?\n\nFor the governor, a Trump arm tattoo to match Trump denial seemingly tattooed on his brain?'), ('dates', '2020-04-27 19:17:04'), ('file_name', 'Lloyd Doggett.csv'), ('pct_true', 0.5013806223869324)])
OrderedDict([('tweet', 'One-two punch: Even as we fight a deadly respiratory illness, President Trump’s EPA increases air pollution by undoing limits on mercury and other harmful pollutants from power plants—despite request from utilities to keep the rule in place.\nhttps://t.co/UkVUSK08oG'), ('dates', '2020-04-27 00:27:02'), ('file_name', 'Lloyd Doggett.csv'), ('pct_true', 0.7668132781982422)])
OrderedDict([('tweet', 'On the 50th anniversary of #EarthDay, we must remain vigilant against Trump using the pandemic to cover his weakening of auto emission and power plant standards. \n\nWith our lungs already under attack by #COVID19 Trump makes a bad problem worse. #ouronlyplanet #EarthDay2020'), ('dates', '2020-04-22 13:56:25'), ('file_name', 'Lloyd Doggett.csv'), ('pct_true', 0.9083759188652039)])
OrderedDict([('tweet', 'This is unacceptable. Congress must swiftly authorize additional funding for the Paycheck Protection Program. Sen. Schumer and Speaker Pelosi are playing political games, instead of supporting small businesses, the engine of the American economy. https://t.co/KmWV10mIvx'), ('dates', '2020-04-16 16:59:18'), ('file_name', 'Lloyd Smucker.csv'), ('pct_true', 0.7689881324768066)])
OrderedDict([('tweet', 'Small businesses need support right now and once again Sen. Schumer &amp; Speaker Pelosi are playing partisan politics. We cannot allow their partisan wish list to cause this critical small business support to run out. https://t.co/F99BbixL83'), ('dates', '2020-04-10 16:07:27'), ('file_name', 'Lloyd Smucker.csv'), ('pct_true', 0.9374604821205139)])
OrderedDict([('tweet', 'The American people are counting on Congress to provide relief to their families and to support small businesses. The bill in the Senate was crafted through bipartisan discussion. This is no time to play politics, it’s time for the Senate to pass the third phase #COVID19 relief. https://t.co/EEcUx8tgYI'), ('dates', '2020-03-24 13:39:18'), ('file_name', 'Lloyd Smucker.csv'), ('pct_true', 0.6832650899887085)])
OrderedDict([('tweet', 'This is unconscionable. Delays in aid will impact families and businesses across the country. The American people deserve better. We’re #InItTogether and Sen. Schumer and Speaker Pelosi need to stop putting their partisan politics above country. https://t.co/GGVYmnrkcX'), ('dates', '2020-03-23 18:45:36'), ('file_name', 'Lloyd Smucker.csv'), ('pct_true', 0.88524329662323)])
OrderedDict([('tweet', 'These aren’t just statistics, they are the lives &amp; livelihoods of Americans that are feeling the financial hardships caused by #COVID19. \n\nCongress must go big in our next relief package to put more money into the pockets of Americans who are struggling! \n\nhttps://t.co/Ea8w4Ov9a3'), ('dates', '2020-05-08 16:45:00'), ('file_name', 'Lois Frankel.csv'), ('pct_true', 0.8143102526664734)])
OrderedDict([('tweet', "#DearBetsy, a global pandemic has turned students' lives upside-down. @BetsyDeVosED should be spending her time ensuring students have the support they need to succeed in school, not rolling back the rights of survivors.\n\n🗣 Now is not the time to release the #TitleIX rule! https://t.co/mKo71jOrrH"), ('dates', '2020-05-05 17:34:07'), ('file_name', 'Lois Frankel.csv'), ('pct_true', 0.5146660208702087)])
OrderedDict([('tweet', '.@GovRonDeSantis is minimizing the deaths of over 1000 Floridians--so Trumpian!'), ('dates', '2020-04-29 21:38:57'), ('file_name', 'Lois Frankel.csv'), ('pct_true', 0.6253584027290344)])
OrderedDict([('tweet', "This administration always falls back on xenophobia when things don't go to plan—it’s shameful. \n \n@realDonaldTrump, blaming immigrants will not keep Americans safe from #COVID19. \n \nWe need tests &amp; leadership, not finger pointing &amp; thinly-veiled racism https://t.co/8cDkoQSbdX"), ('dates', '2020-04-21 14:34:19'), ('file_name', 'Lois Frankel.csv'), ('pct_true', 0.6529778242111206)])
OrderedDict([('tweet', 'We are in the middle of a pandemic. While Congress is working in a bipartisan way to pass measures that will protect American’s lives &amp; livelihoods, @realDonaldTrump is shamefully putting partisan politics over the safety &amp; health of our communities. https://t.co/eoT30hj8fZ'), ('dates', '2020-04-08 15:30:06'), ('file_name', 'Lois Frankel.csv'), ('pct_true', 0.8559167981147766)])
OrderedDict([('tweet', 'Trump’s claims that states have all the supplies they need to fight the #coronavirus pandemic couldn’t be further from the truth. Just talk to anyone who works in a hospital – the lack of protective equipment means they are putting their lives on the line every day to save ours. https://t.co/cF4IIvfgaO'), ('dates', '2020-04-06 22:43:08'), ('file_name', 'Lois Frankel.csv'), ('pct_true', 0.7857150435447693)])
OrderedDict([('tweet', 'The #coronavirus pandemic is bearing down on Florida and our #healthcare system is taking a direct hit. The additional $13.4 million in critical @cdcgov funding directly to Florida will be used to strengthen our #publichealth infrastructure &amp; save lives.\n\nhttps://t.co/AvhzhBPM4o'), ('dates', '2020-04-06 21:35:37'), ('file_name', 'Lois Frankel.csv'), ('pct_true', 0.5593490600585938)])
OrderedDict([('tweet', 'Shameful – the state of Texas is using a pandemic as an excuse to block women from getting an abortion.\n\nThis is a political ploy and it will deprive women of necessary medical care. \n\nhttps://t.co/98tkMPrc7r'), ('dates', '2020-03-31 20:05:21'), ('file_name', 'Lois Frankel.csv'), ('pct_true', 0.5622916221618652)])
OrderedDict([('tweet', "I'm disturbed by Texas' decision to politicize the response to #coronavirus, using this pandemic to control women's bodies.\n\nThey are playing politics with people's lives.\n\nI stand with the abortion providers suing the state over this harmful decision.\n\nhttps://t.co/fW7mpstwWM"), ('dates', '2020-03-30 16:12:28'), ('file_name', 'Lois Frankel.csv'), ('pct_true', 0.5392976403236389)])
OrderedDict([('tweet', 'Even as we grapple with this unprecedented global pandemic, @BetsyDeVosED continues to push her anti-student, anti-survivor agenda with no regard for the human cost.\n\nHer decision does a tremendous disservice victims of sexual assault and harassment on college campuses. #TitleIX https://t.co/aANHdkBo1y'), ('dates', '2020-05-06 22:11:53'), ('file_name', 'Lori Trahan.csv'), ('pct_true', 0.7393613457679749)])
OrderedDict([('tweet', '"PPP is a life-saver for many small businesses and I greatly appreciate the fact that the SBA has had to stand-up this brand new $660 billion program very fast. SBA has essentially had to build this plane while you’re flying it." - Rep. Trahan #TrahanTownHall'), ('dates', '2020-04-30 20:25:21'), ('file_name', 'Lori Trahan.csv'), ('pct_true', 0.6392456889152527)])
OrderedDict([('tweet', '"This pandemic has highlighted the importance of nursing homes having robust plans for emergency preparedness and infectious disease control." - Rep. Trahan #TrahanTownHall'), ('dates', '2020-04-30 20:21:34'), ('file_name', 'Lori Trahan.csv'), ('pct_true', 0.6837655901908875)])
OrderedDict([('tweet', '"I have drafted legislation -- called the Pandemic Production Act -- which would allow the federal government to partner with industry the way we do for our national defense." - Rep. Trahan #TrahanTownHall'), ('dates', '2020-04-30 20:16:35'), ('file_name', 'Lori Trahan.csv'), ('pct_true', 0.7029241323471069)])
OrderedDict([('tweet', 'The American people agree: we should not have to choose our health over our democracy. We need to enact #VotebyMail and the time is now. \nhttps://t.co/09LYS9Qn1t'), ('dates', '2020-04-29 01:53:21'), ('file_name', 'Lori Trahan.csv'), ('pct_true', 0.564723551273346)])
OrderedDict([('tweet', "#COVID19 has placed workers, families &amp; small business owners under incredible stress. No one should have to choose between paying their medical bills or rent. That's why this week @RepLBR, @BillPascrell &amp; I introduced the Coverage for COVID-19 Treatment Act."), ('dates', '2020-04-26 19:08:33'), ('file_name', 'Lori Trahan.csv'), ('pct_true', 0.8679664731025696)])
OrderedDict([('tweet', "I'd like to know how I'm holding up a bill from going to the Senate that hasn't been written yet? Most Senators are not even back in DC yet. #FakeNews"), ('dates', '2020-03-16 18:21:12'), ('file_name', 'Louie Gohmert.csv'), ('pct_true', 0.5180898308753967)])
OrderedDict([('tweet', '#Coronavirus funding should be a CLEAN bill. No FISA reauthorization, no price fixing on drugs. Democrats should stop playing politics with the health of the American people.'), ('dates', '2020-03-04 17:22:27'), ('file_name', 'Louie Gohmert.csv'), ('pct_true', 0.9544304013252258)])
OrderedDict([('tweet', 'Countless families in #CA40 are struggling over how to pay their rent or mortgage during this crisis. That’s why I’m cosponsoring #HR6515, which will suspend your obligation to pay your residential rent or mortgage during the #COVID19 emergency.'), ('dates', '2020-05-01 23:33:24'), ('file_name', 'Lucille Roybal-Allard.csv'), ('pct_true', 0.5704920291900635)])
OrderedDict([('tweet', 'President Trump is trying to divert attention from his mishandling of the coronavirus pandemic by once again using the pretext of the pandemic to attack immigrants and immigration. At the same time, he is continuing to undermine state efforts to minimize the impact of COVID-19.'), ('dates', '2020-04-21 17:33:25'), ('file_name', 'Lucille Roybal-Allard.csv'), ('pct_true', 0.6369705200195312)])
OrderedDict([('tweet', 'He firmado una carta urgiendo a la Administración de Trump que traduzca y distribuya de inmediato orientación pública sobre el coronavirus en español y otros idiomas. Todos los estadounidenses necesitan información actualizada en su idioma para guiarlos a través de esta pandemia.'), ('dates', '2020-04-11 22:05:37'), ('file_name', 'Lucille Roybal-Allard.csv'), ('pct_true', 0.529229462146759)])
OrderedDict([('tweet', 'All people deserve to have their voice truly heard.\n\n150 years ago, the 15th Amendment was ratified, affirming that no American could be denied the right to vote because of their race.\n\nWe must continue the fight to protect the right to vote &amp; ensure all Americans have a voice.'), ('dates', '2020-03-30 22:01:00'), ('file_name', 'Lucy McBath.csv'), ('pct_true', 0.7279202938079834)])
OrderedDict([('tweet', '“The term ‘net neutrality’ is misused. Most of us believe in a free and open Internet. But under the label of ‘net neutrality’ in 2015, the Obama-Era FCC imposed additional and unnecessary regulations on Internet Service Providers (ISPs)." (1/2)'), ('dates', '2019-04-10 15:58:52'), ('file_name', 'Mac Thornberry.csv'), ('pct_true', 0.7732784152030945)])
OrderedDict([('tweet', '“Since Democrats took the majority, the House has spent its time considering hyper-partisan legislation and Sense of Congress resolutions that will never be signed by the President."  (1)'), ('dates', '2019-04-04 18:10:45'), ('file_name', 'Mac Thornberry.csv'), ('pct_true', 0.9872068762779236)])
OrderedDict([('tweet', 'On H.R. 1 - “House Democrats try to frame this bill as a way to empower American voters; however, hidden in the text are outrageous regulations and more federal bureaucracy. (1 of 2)'), ('dates', '2019-03-08 17:00:12'), ('file_name', 'Mac Thornberry.csv'), ('pct_true', 0.6343493461608887)])
OrderedDict([('tweet', 'We are battling a pandemic that claimed more than 75,000 lives. Yet Trump is pushing to suspend healthcare for 20M Americans. People are dying, Mr. President,\xa0throw out your case, there could not be a worse time to tear down the ACA &amp; its protections. \n\nhttps://t.co/lhp1cFR7JF'), ('dates', '2020-05-08 16:24:19'), ('file_name', 'Madeleine Dean.csv'), ('pct_true', 0.9407769441604614)])
OrderedDict([('tweet', 'We must save our smallest businesses from bankruptcy.  Which is why @RepDerekKilmer and I introduced the Restore America’s Main Street Act to provide direct cash payments to these businesses — we are hopeful it will be included in the next CARES package.\n\nhttps://t.co/8gFPYkMGRo'), ('dates', '2020-05-06 17:23:31'), ('file_name', 'Madeleine Dean.csv'), ('pct_true', 0.8419962525367737)])
OrderedDict([('tweet', "$1200 is NOT enough. Americans won't survive this crisis with a single payment. \n\nRent payments haven't stopped. Adult children are home from college. Folks are struggling to keep food on the table.\n\nAmericans need multiple cash payments of $2000. I won't stop fighting for you. https://t.co/LzHvpJsrhk"), ('dates', '2020-04-17 18:01:53'), ('file_name', 'Madeleine Dean.csv'), ('pct_true', 0.6179311871528625)])
OrderedDict([('tweet', 'An inferno for which he was repeatedly warned but chose instead to peddle conspiracies about. This President is misguided and looks to blame rather than focus on healing our nation. I’m ashamed of his behavior. https://t.co/N6PyqFCCzZ'), ('dates', '2020-04-15 01:27:43'), ('file_name', 'Madeleine Dean.csv'), ('pct_true', 0.5406309366226196)])
OrderedDict([('tweet', 'Over 10 million workers have filed for unemployment. Many will lose their paychecks &amp; their employer-based health care. The President must reopen the ACA marketplace immediately. And we must pursue universal health care — I believe a well-funded public option is essential.'), ('dates', '2020-04-08 21:23:44'), ('file_name', 'Madeleine Dean.csv'), ('pct_true', 0.6779845356941223)])
OrderedDict([('tweet', "Don't let Senate Republicans confuse you. Democrats want to get immediate relief to American workers.\n\nWe want transparency to ensure the funds go to American families and Main Street — not Wall Street buybacks."), ('dates', '2020-03-23 17:24:17'), ('file_name', 'Madeleine Dean.csv'), ('pct_true', 0.5827008485794067)])
OrderedDict([('tweet', 'Minority-owned businesses traditionally have a harder time getting loans and may be neglected by big banks offering #PPPloans. @housedemocrats are fighting to get them #Smallbizrelief through community lenders. https://t.co/qqOvpZJ7Ru'), ('dates', '2020-04-15 18:07:03'), ('file_name', 'Marc A. Veasey.csv'), ('pct_true', 0.6725424528121948)])
OrderedDict([('tweet', 'If the Trump Admin is “inspired” by #DACA recipients who are essential workers then it should stop using them as political pawns. DACA is at risk because Trump terminated it.\nTrump can drop the DACA case at SCOTUS &amp; extend protections while we recover from this crisis. https://t.co/0FFITqE8ZW'), ('dates', '2020-04-10 19:12:39'), ('file_name', 'Marc A. Veasey.csv'), ('pct_true', 0.5197246670722961)])
OrderedDict([('tweet', 'President Trump’s budget proposal aims to cut $5 billion from public education in 2021 alone. This is unacceptable. Public schools are the foundation of success for young people across the country. #PublicSchoolProud  #PublicSchoolsWeek'), ('dates', '2020-02-25 20:59:28'), ('file_name', 'Marc A. Veasey.csv'), ('pct_true', 0.5411376953125)])
OrderedDict([('tweet', 'Today, Trump’s #PublicCharge rule to deny green cards to  immigrants who are on or might one day need public benefits like Medicaid will take effect. Families shouldn’t have to choose between staying in our country or receiving life-saving medical care. https://t.co/YP0SbRw8tj'), ('dates', '2020-02-24 16:46:45'), ('file_name', 'Marc A. Veasey.csv'), ('pct_true', 0.7536478042602539)])
OrderedDict([('tweet', "It’s a shame that the president’s budget proposal turns its back on American families. \n\nTrump's budget slashes billions from SNAP, limits lifesaving medical research, and reduces funding for the Centers for Disease Control and Prevention by $678 million. https://t.co/XZtlH8RFWm"), ('dates', '2020-02-10 21:08:15'), ('file_name', 'Marc A. Veasey.csv'), ('pct_true', 0.5345912575721741)])
OrderedDict([('tweet', 'Medicaid is one of the most important tools we have to help people struggling with opioid addiction, but the Trump Administration is trying to cut benefits for millions of Americans in midst of an epidemic. #ProtectOurCare'), ('dates', '2020-02-06 19:33:28'), ('file_name', 'Marc A. Veasey.csv'), ('pct_true', 0.6759653687477112)])
OrderedDict([('tweet', 'Every year, over 36,000 Americans are killed in acts of gun violence. That is not normal. \n\nThis #GunViolenceSurvivorsWeek, the Senate must stop their inaction and start saving lives by taking up the common-sense gun violence prevention legislation the House passed last year.'), ('dates', '2020-02-06 17:53:23'), ('file_name', 'Marc A. Veasey.csv'), ('pct_true', 0.5264940857887268)])
OrderedDict([('tweet', "The President’s claims that he is improving our health care are #FakeNews. In reality, he's gutting access to health care at every turn. That is why I am proud to lead today’s resolution fighting back against his latest move to cut Medicaid through an illegal block-grant scheme."), ('dates', '2020-02-06 13:44:27'), ('file_name', 'Marc A. Veasey.csv'), ('pct_true', 0.6409579515457153)])
OrderedDict([('tweet', 'The President is trying to dismantle the ACA and strip protections for preexisting conditions from millions of Americans in the middle of a pandemic. https://t.co/zxSyqyIm2A'), ('dates', '2020-05-07 15:21:00'), ('file_name', 'Marcia L. Fudge.csv'), ('pct_true', 0.6060250997543335)])
OrderedDict([('tweet', 'College students have been pushed out of their housing, leaving many homeless or forced into unstable living.  @BetsyDeVosED should be spending her time ensuring students have the support they need to succeed in school, not rolling back protections for survivors. https://t.co/peJr42xjPj'), ('dates', '2020-05-06 16:24:06'), ('file_name', 'Marcia L. Fudge.csv'), ('pct_true', 0.5234896540641785)])
OrderedDict([('tweet', "When farmers needed assistance because of the President's trade wars, the Administration rolled out a bailout program that favored large farms. We cannot allow @USDA to repeat this mistake as small and medium-sized farmers struggling through the current pandemic seek relief. https://t.co/nC3bCPUnoe"), ('dates', '2020-05-04 17:13:26'), ('file_name', 'Marcia L. Fudge.csv'), ('pct_true', 0.6885712742805481)])
OrderedDict([('tweet', 'Too often, workers are forced to choose between their jobs and their safety.  \n\nNever has this been more apparent than during the coronavirus pandemic.  \n\nEssential workers deserve strong workplace protections, including PPE, and hazard pay. #WorkerMemorialDay'), ('dates', '2020-04-28 16:55:58'), ('file_name', 'Marcia L. Fudge.csv'), ('pct_true', 0.5557222962379456)])
OrderedDict([('tweet', 'It’s been one year since the House voted to reauthorize and expand the Violence Against Women Act. \n\nThis bill supports survivors and prevents domestic violence, but still sits on Leader McConnell’s desk collecting dust.  \n\nWe need to reauthorize #VAWA now. https://t.co/DTNFOpbEXQ'), ('dates', '2020-04-04 19:35:32'), ('file_name', 'Marcia L. Fudge.csv'), ('pct_true', 0.6870830059051514)])
OrderedDict([('tweet', 'The President’s call to “reopen” the country by Easter is arbitrary and dangerous to public health. Health experts and our ability to #FlattenTheCurve are what dictate the timeline – not the President.  https://t.co/tqAjvqZkBW'), ('dates', '2020-03-26 19:39:03'), ('file_name', 'Marcia L. Fudge.csv'), ('pct_true', 0.6667834520339966)])
OrderedDict([('tweet', 'The continued reference to the coronavirus as the “Chinese Virus” and “Wuhan Virus” is not only blatantly xenophobic, it is dangerous to the Asian American community.  Proud to support @RepGraceMeng’s resolution condemning anti-Asian sentiment racism related to #COVID19. https://t.co/8mvGVoBo5b'), ('dates', '2020-03-26 18:46:46'), ('file_name', 'Marcia L. Fudge.csv'), ('pct_true', 0.7006816864013672)])
OrderedDict([('tweet', 'Senators Rubio, @SenTomCotton, @SenSasse, &amp; @RepLizCheney introduced a bill to rename the street outside the Chinese embassy in #WashingtonDC “Li Wenliang Plaza.” \n\nThe bill honors the late Dr. Li Wenliang,  who warned the world about the #Covid_19 before the CCP silenced him.'), ('dates', '2020-05-07 17:19:39'), ('file_name', 'Marco Rubio.csv'), ('pct_true', 0.7777289748191833)])
OrderedDict([('tweet', "Sen. Rubio &amp; @SenatorMenendez issued a statement after the @USGAO published the findings of an investigation they requested into the Trump Administration’s civil nuclear cooperation negotiations with #SaudiArabia, as well as any negotiation by the executive branch since Dec. '09."), ('dates', '2020-05-04 22:15:00'), ('file_name', 'Marco Rubio.csv'), ('pct_true', 0.7316951155662537)])
OrderedDict([('tweet', "Australian FM @MarisePayne &amp; @SenatorWong rightfully called for an int'l inquiry into the origins of #COVID_19, then #Beijing threatened to boycott Aussie goods. Democracies &amp; responsible states should stand w/ Australia &amp; push back against China’s bullying and economic coercion. https://t.co/AeLjPyvxVT"), ('dates', '2020-04-29 13:40:48'), ('file_name', 'Marco Rubio.csv'), ('pct_true', 0.5896043181419373)])
OrderedDict([('tweet', 'On the 105th anniversary of the #ArmenianGenocide, we remember the 1.5 million Armenians who were murdered at the hands of the Ottoman Empire.'), ('dates', '2020-04-24 13:28:52'), ('file_name', 'Marco Rubio.csv'), ('pct_true', 0.6473567485809326)])
OrderedDict([('tweet', 'Democrats held #PPP funding for #smallbiz hostage for nearly two weeks. Their reckless partisanship accomplished nothing, and endangered countless small businesses who need capital to stay alive and pay their workers. https://t.co/Nf52dXykai'), ('dates', '2020-04-22 00:26:47'), ('file_name', 'Marco Rubio.csv'), ('pct_true', 0.9393165707588196)])
OrderedDict([('tweet', 'Durante el #COVID_19, #Maduro continúa reprimiendo a la prensa libre. La semana pasada,Eduardo Galindo fue detenido arbitrariamente y luego liberado. La inescrupulosidad del régimen incluso los ha llevado a entrar a las casas de familiares de periodistas venezolanos en el exilio.'), ('dates', '2020-04-21 21:46:29'), ('file_name', 'Marco Rubio.csv'), ('pct_true', 0.7241269946098328)])
OrderedDict([('tweet', 'In recent weeks, #Beijing has moved to tighten its grip on illegally claimed territory in the South China Sea. The Chinese Communist Party is exploiting the #COVID_19 crisis to make moves in #HongKong and #SouthChinaSea. https://t.co/m1P7AqYkuO'), ('dates', '2020-04-20 21:21:50'), ('file_name', 'Marco Rubio.csv'), ('pct_true', 0.6736041307449341)])
OrderedDict([('tweet', "Rising authoritarianism in Central/Eastern Europe coincides w/ a huge increase in Russian/Chinese influence. The U.S. shouldn't sit idly by as we watch democracy slip away from our allies. Where is U.S. leadership on this? Why is Trump AWOL as Russia/China expand their influence? https://t.co/6EfXYXNNmM"), ('dates', '2020-05-07 17:24:05'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.6863086819648743)])
OrderedDict([('tweet', "Northern Ohio has many large communities whose residents, families, &amp; businesses pay taxes too. They shouldn't be shortchanged during a pandemic. President Trump has the power to fix this – and he should do so immediately. My Oped in the @clevelanddotcom 👇https://t.co/tk1CjKb1e9"), ('dates', '2020-05-01 21:19:34'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.6496784687042236)])
OrderedDict([('tweet', 'Over 45,000 Americans have died. More die everyday. We need widespread testing to safely reopen the economy. President Trump’s calls to open the economy are irresponsible, go against public health expertise, and will extend the amount and length of suffering Americans face. https://t.co/uPtx2r0GRZ'), ('dates', '2020-04-22 17:52:10'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.6750500798225403)])
OrderedDict([('tweet', 'The facts speak for themselves. The Trump Administration oversaw a big business restaurant and hotel chain cash grab on hard working taxpayers. Again. Now we know why they resisted transparency. \nhttps://t.co/d3tu993QYf'), ('dates', '2020-04-20 20:47:43'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.5503719449043274)])
OrderedDict([('tweet', 'Truly disappointing that Americans will have to wait for their relief payment so the President can enrich his brand. Even as our country and American families battle a global public health and economic emergency, personal image will come first. https://t.co/M9G6rErRc1'), ('dates', '2020-04-15 02:07:22'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.6586199402809143)])
OrderedDict([('tweet', 'This is why we need President Trump to immediately use the Defense Production Act to its FULL capacity. This country must ramp up production of medical equipment. Time is running out and Trump’s reluctance risks the well being of thousands of Americans. https://t.co/4v0fiAhEXu'), ('dates', '2020-04-02 19:33:04'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.7266591191291809)])
OrderedDict([('tweet', 'The vast number of Americans who’ve filed for unemployment in recent weeks is absolutely staggering. 10M Americans are recently unemployed and the number is growing every hour. These unprecedented times will take unprecedented action to get through it. https://t.co/bjbqLerOLl'), ('dates', '2020-04-02 17:25:39'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.5192040801048279)])
OrderedDict([('tweet', 'The burden President Trump is asking of our seniors, which would likely result in people leaving their homes during a pandemic to seek help from a tax adviser, is a complete and utter leadership failure of this Administration. https://t.co/NtFS4z6tPU'), ('dates', '2020-04-01 22:03:16'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.5492182374000549)])
OrderedDict([('tweet', 'Its wrong for President Trump to force seniors to jump through hoops and fill out unnecessary paperwork to receive their coronavirus stimulus checks, which Congress authorized on a bipartisan basis. Trump and Mnuchin should reverse course and not pass the buck to seniors.'), ('dates', '2020-04-01 22:00:38'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.7309190630912781)])
OrderedDict([('tweet', 'We protect the economy by defeating COVID 19 &amp; #FlatteningTheCurve, not pretending the threat doesn’t exist. Upending our national response would overwhelm hospitals, put lives at risk, &amp; deal a greater blow to our economy. Trump should listen to science &amp; public heath experts. https://t.co/nTP9hpOOrC'), ('dates', '2020-03-23 19:30:53'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.6913999319076538)])
OrderedDict([('tweet', "Tonight, Trump will hold a townhall in PA, so friendly reminder that he's working tirelessly to repeal the ACA and kick 741,000 Ohioans off their healthcare, undermine workers' right to organize, dissolve support for public education, &amp; roll back vital environmental protections."), ('dates', '2020-03-05 21:47:57'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.625950276851654)])
OrderedDict([('tweet', "Budgets are more than just fiscal documents, they are a reflection of our values. But President Trump’s budget was not written with working people in mind – instead, it's a laundry list of corporate handouts that literally leave working people in the cold. https://t.co/QqXVD6KuEM"), ('dates', '2020-02-10 23:17:17'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.6072820425033569)])
OrderedDict([('tweet', 'The fact that Senate Republicans won’t call a single witnesses should anger every American. They are trading the long term health of our democracy for political self-interest. It’s shameful. https://t.co/eGiCm66zcB'), ('dates', '2020-01-31 17:10:08'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.844905436038971)])
OrderedDict([('tweet', "This week is the 47th Anniversary of Roe v. Wade, yet Trump is marking the occasion by addressing March For Life, the 1st president to do so. \n\nThis is another sign that women's Constitutional right to make their own healthcare decisions is more at risk than it has been in years. https://t.co/l8qjnTco6t"), ('dates', '2020-01-24 20:50:30'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.5738258361816406)])
OrderedDict([('tweet', "Trump said he's willing to cut Social Security and Medicare to reduce the deficit, which has skyrocketed under his presidency thanks to the tax cuts he gave billionaires and corporations. Social Security and Medicare must be strengthened, not rolled back! https://t.co/IBn2rxrW36"), ('dates', '2020-01-24 19:52:00'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.9031481742858887)])
OrderedDict([('tweet', 'Our elections should be driven by and for the people, not big moneyed special interests. That’s why I’ve introduced a Constitutional Amendment nullifying Citizens United and giving power over our democracy back to the voters. #EndCitizensUnited https://t.co/6AeiilPxFy'), ('dates', '2020-01-17 21:39:08'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.5044829249382019)])
OrderedDict([('tweet', 'USMCA would be the law of the land for decades, yet it doesn’t include a single provision to address climate change or protect the environment. The climate crisis is here and the American people can’t afford another NAFTA-style, do-nothing trade deal. https://t.co/XFyya0vSoA'), ('dates', '2020-01-17 19:50:41'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.6983656883239746)])
OrderedDict([('tweet', 'GAO finds President Trump broke the law to advance his own political interests risking national security. American’s know Republican efforts to block a real Senate trial with witnesses is a cover-up to protect the President.  https://t.co/Sky3vzTrNK'), ('dates', '2020-01-16 17:12:33'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.9317749738693237)])
OrderedDict([('tweet', 'Today, the House voted to send two Articles of Impeachment to the Senate. I voted in favor.\n\nNow, the Senate must uphold its Constitutional obligation to carry out a fair trail with witnesses &amp; documents.\n\nThe American people deserve truth. Not a cover up.\nhttps://t.co/EuNqitjhdh'), ('dates', '2020-01-16 00:54:58'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.788720428943634)])
OrderedDict([('tweet', 'Reminder: The President &amp; his allies in Congress blocked every effort to prevent Russian interference in our democracy following 2016. Today, Mitch McConnell continues to block House-passed efforts to secure our elections &amp; hold Russia accountable. Why?\nhttps://t.co/PZ7jfGMCrn'), ('dates', '2020-01-14 19:10:25'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.821940004825592)])
OrderedDict([('tweet', 'The American people do not want a war with Iran. The people of Northern Ohio do not want a war with Iran. The Trump Admin must immediately work to deescalate tensions and provide Congress a full briefing on the Administration’s strategy. No more endless wars. #NoWarWithIran https://t.co/9EzpWBK8Bo'), ('dates', '2020-01-03 21:59:36'), ('file_name', 'Marcy Kaptur.csv'), ('pct_true', 0.8540876507759094)])
OrderedDict([('tweet', "It's been 12 weeks since the first case of #COVID19 was reported in our country.\n \nAnd this administration still does not have a comprehensive national testing strategy.\n \nThat's unacceptable."), ('dates', '2020-05-06 20:56:26'), ('file_name', 'Margaret Wood Hassan.csv'), ('pct_true', 0.5411584377288818)])
OrderedDict([('tweet', "As a global pandemic rages on, @senatemajldr wants the Senate to confirm Justin Walker – who has vehemently opposed Americans' right to health care – to the D.C. Circuit Court. That's outrageous. I'm calling on my colleagues to #StopWalker because #courtsmatter."), ('dates', '2020-05-05 18:52:39'), ('file_name', 'Margaret Wood Hassan.csv'), ('pct_true', 0.9110456109046936)])
OrderedDict([('tweet', 'Today is #YomHaShoah – also known as #HolocaustRemembranceDay. We must continue to tell the stories of the 6 million Jews and countless others who were murdered during the Holocaust and recommit to the fight against anti-semitism and hate.'), ('dates', '2020-04-21 18:20:43'), ('file_name', 'Margaret Wood Hassan.csv'), ('pct_true', 0.628776490688324)])
OrderedDict([('tweet', 'Amid the #COVID19 outbreak, domestic violence has spiked.\n\nThe House passed the Violence Against Women Reauthorization Act over a year ago. It’s long past time for the Senate to pass #VAWA to protect victims and survivors of domestic violence.\nhttps://t.co/suCNIFjcPO'), ('dates', '2020-04-07 17:06:44'), ('file_name', 'Margaret Wood Hassan.csv'), ('pct_true', 0.5166812539100647)])
OrderedDict([('tweet', 'The Senate should focus on #COVID19 related matters &amp; oversight. I urged @SenateMajLdr to hold hearings on our testing capabilities and capacity, #PPP implementation, &amp; other issues related to the COVID-19 public health &amp; economic emergencies.\n\nhttps://t.co/PKYhSdwd17'), ('dates', '2020-05-04 20:51:27'), ('file_name', 'Maria Cantwell.csv'), ('pct_true', 0.719092071056366)])
OrderedDict([('tweet', "The president is wrong. The 'pan' in pandemic means it doesn't stop at borders. We need to have a global response to stop the virus.\n\nhttps://t.co/bKEfcjF9yZ"), ('dates', '2020-04-16 17:56:09'), ('file_name', 'Maria Cantwell.csv'), ('pct_true', 0.8150036931037903)])
OrderedDict([('tweet', 'There has been a surge of reports of discrimination against Asian Americans related to the #COVID19 pandemic, but no concerted effort from federal agencies to address this problem.\n\nhttps://t.co/3c62tjjPWJ'), ('dates', '2020-04-13 17:47:49'), ('file_name', 'Maria Cantwell.csv'), ('pct_true', 0.5717313885688782)])
OrderedDict([('tweet', 'It’s been over one year since the House passed the Violence Against Women Act, and Mitch McConnell still hasn’t brought up the bill for a vote in the Senate. It’s long past time to reauthorize #VAWA\n\nhttps://t.co/PfvEsTLRwz'), ('dates', '2020-04-07 19:02:01'), ('file_name', 'Maria Cantwell.csv'), ('pct_true', 0.7819547653198242)])
OrderedDict([('tweet', 'The #COVID19 pandemic has spurred a nationwide shortage of donated blood, but @US_FDA continues to enforce a discriminatory donor deferral policy. I urged FDA to update its blood donor policies in favor of ones that are grounded in science &amp; allow all healthy Americans to donate. https://t.co/4SvgbQB2bh'), ('dates', '2020-03-28 16:21:36'), ('file_name', 'Maria Cantwell.csv'), ('pct_true', 0.6182996034622192)])
OrderedDict([('tweet', 'Taiwan’s exemplary response to #COVID19 has been marked by transparency &amp; deftness, in striking contrast to Communist China’s deception &amp; failures. The US &amp; world would benefit from including #Taiwan in international organizations, diplomacy &amp; multilateral decision-making. https://t.co/DmnbTJXwXr'), ('dates', '2020-05-06 14:44:12'), ('file_name', 'Mario Diaz-Balart.csv'), ('pct_true', 0.8131501078605652)])
OrderedDict([('tweet', 'Today, during a call with the appropriations committee, \n@stevenmnuchin1 reiterated the federal government’s efforts to ensure that #PPP funding goes to those who truly need and deserve it. We will not tolerate those who try to cheat U.S. taxpayers.'), ('dates', '2020-04-29 18:28:37'), ('file_name', 'Mario Diaz-Balart.csv'), ('pct_true', 0.5383554697036743)])
OrderedDict([('tweet', 'Democratic Leadership must stop using the pandemic as leverage to advance their agenda. They’ve allowed the #PPP to hit its funding cap, preventing small businesses from accessing much-needed aid. This is not a time to play partisan politics— the American people need solutions!'), ('dates', '2020-04-20 17:12:42'), ('file_name', 'Mario Diaz-Balart.csv'), ('pct_true', 0.9149318933486938)])
OrderedDict([('tweet', '.@realDonaldTrump is right to suspend funding to @WHO while we assess how it became a mouthpiece for Communist China’s misinformation on Wuhan #Coronavirus. When lives depended on accurate info, #WHO failed spectacularly.'), ('dates', '2020-04-15 13:53:32'), ('file_name', 'Mario Diaz-Balart.csv'), ('pct_true', 0.9942566752433777)])
OrderedDict([('tweet', 'Until we know the full truth of what happened, the U.S. must continue to press for answers no matter how long it takes. My prayers are with #BobLevinson and his longsuffering family who have fought courageously for 13 years to bring their loved one home. https://t.co/fXTmhVeNC6'), ('dates', '2020-03-26 19:30:47'), ('file_name', 'Mario Diaz-Balart.csv'), ('pct_true', 0.5632016062736511)])
OrderedDict([('tweet', 'The charges announced by @TheJusticeDept today reveal wide-ranging corruption and criminality throughout the highest levels of the #Maduro regime. \n\nThese criminals who have plundered #Venezuela’s resources and engaged in drug trafficking must be brought to justice. https://t.co/onP2EOSQJg'), ('dates', '2020-03-26 17:03:16'), ('file_name', 'Mario Diaz-Balart.csv'), ('pct_true', 0.541657567024231)])
OrderedDict([('tweet', 'Congressman DeSaulnier’s friend and @OversightDems colleague @RepAOC drops by #VirtualOfficeHours to discuss who qualifies for pandemic unemployment insurance, how to access it, &amp; the communities left behind that Congress must continue to fight for. https://t.co/N4NY5jlTZI'), ('dates', '2020-04-30 21:38:11'), ('file_name', 'Mark DeSaulnier.csv'), ('pct_true', 0.6165112853050232)])
OrderedDict([('tweet', '.@CCCounty has issued an order requiring anyone working at or visiting an essential business, seeking medical attention, or riding public transit to wear face coverings to reduce the spread of coronavirus.'), ('dates', '2020-04-17 20:03:38'), ('file_name', 'Mark DeSaulnier.csv'), ('pct_true', 0.5893939137458801)])
OrderedDict([('tweet', 'The Social Security Administration is advising Social Security recipients who did not file tax returns in 2018 or 2019 to file soon to help speed up the individual rebate they may be eligible for. For more information, please visit https://t.co/ZoocdZPcTH. \n-Team DeSaulnier'), ('dates', '2020-04-13 16:59:33'), ('file_name', 'Mark DeSaulnier.csv'), ('pct_true', 0.6548172831535339)])
OrderedDict([('tweet', 'The President is not treating #Covid_19 with the urgency it warrants. We need to scale up testing, establish emergency paid leave, &amp; protect health care workers immediately. Instead of declaring a national emergency, @realDonaldTrump has spent the morning tweeting attacks &amp; lies. https://t.co/i6aUd38EBX'), ('dates', '2020-03-13 15:32:31'), ('file_name', 'Mark DeSaulnier.csv'), ('pct_true', 0.6571840047836304)])
OrderedDict([('tweet', 'Today, @CoCoHealth issued new guidance that Contra Costa residents should avoid mass gatherings in an attempt to reduce the spread of COVID-19 &amp; @CCCounty declared a state of emergency.'), ('dates', '2020-03-10 23:06:29'), ('file_name', 'Mark DeSaulnier.csv'), ('pct_true', 0.8012058734893799)])
OrderedDict([('tweet', 'The President continues to contradict what his own public health officials are saying about the coronavirus. If we want to help Americans &amp; the economy, the public needs accurate information and a well-coordinated government response. https://t.co/6Iu6RuyAeo'), ('dates', '2020-03-09 16:24:51'), ('file_name', 'Mark DeSaulnier.csv'), ('pct_true', 0.5720227956771851)])
OrderedDict([('tweet', 'This is completely unacceptable. Speaker Pelosi was just touting we hadn’t even “spent a third” of the PPP “so we have time to negotiate." 7 days later…PPP funds are GONE. We repeatedly warned Dems about this &amp; instead, they blocked a bill last week to renew PPP funding. https://t.co/L2mHlZpmB2'), ('dates', '2020-04-16 17:20:59'), ('file_name', 'Mark E. Amodei.csv'), ('pct_true', 0.8791587948799133)])
OrderedDict([('tweet', 'Good news is, more than $1B in relief has gone out to NV small businesses. Unfortunately, our small gaming entities are being refused help by SBA &amp; Treasury, a decision that completely flies in the face of everything the CARES Act stands for. More here: https://t.co/qKHb23dFzw'), ('dates', '2020-04-16 00:08:48'), ('file_name', 'Mark E. Amodei.csv'), ('pct_true', 0.5027863383293152)])
OrderedDict([('tweet', 'Voted NO today on the #ExIm Bank bill. I’ve always been supportive of the Bank’s mission, but this partisan bill includes a bundle of unnecessary provisions that distract from the Bank’s ability to ensure U.S. businesses remain competitive. Full statement: https://t.co/ooyjANjNht'), ('dates', '2019-11-15 18:57:54'), ('file_name', 'Mark E. Amodei.csv'), ('pct_true', 0.5201504826545715)])
OrderedDict([('tweet', 'Just voted against Speaker Pelosi’s last ditch attempt to legitimize this unprecedented impeachment process that has been conducted behind closed doors for weeks now with little inclusion of minority committee members. My full statement here: https://t.co/jBA9nEfmRI'), ('dates', '2019-10-31 17:35:48'), ('file_name', 'Mark E. Amodei.csv'), ('pct_true', 0.6719518303871155)])
OrderedDict([('tweet', '(1/2) I voted against tonight’s legislation because it’s not responsible to advance bills that have been deemed dead on arrival in the Senate, eliminate funding for border security, &amp; cut millions of dollars for House-passed funding priorities unique to Western states like ours'), ('dates', '2019-01-04 03:19:24'), ('file_name', 'Mark E. Amodei.csv'), ('pct_true', 0.8299065828323364)])
OrderedDict([('tweet', "75 years ago today, Eisenhower signed our victory in the shape of Nazi Germany's surrender papers. VE Day, the triumph of the free world over a horrific, totalitarian regime, is still a powerful reminder today: America does not give in."), ('dates', '2020-05-08 21:11:14'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.639029860496521)])
OrderedDict([('tweet', "China's actions to cover up their handling of the COVID-19 outbreak have shed new light on China's hegemonic intentions. It's time for America to act. \n\nRead my op-ed in @RCDefense: \n\nhttps://t.co/LMlFnaW1Vh"), ('dates', '2020-05-07 18:12:28'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.6073060631752014)])
OrderedDict([('tweet', 'In Tennessee, we are a fiscally-responsible state. We have the lowest per-capita debt in the Nation and no income tax. @SpeakerPelosi and @SenSchumer—state bailouts are unfair to well-run states like Tennessee. \nhttps://t.co/konhsw7gdQ'), ('dates', '2020-05-06 20:25:04'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.6511504650115967)])
OrderedDict([('tweet', 'While 30M Americans have filed for unemployment across the country, @SpeakerPelosi and @SenSchumer continue to use this crisis to scope out every opportunity to introduce their laundry list of increased spending and state bailouts instead of negotiating in a bipartisan manner.'), ('dates', '2020-05-06 18:16:58'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.7742605209350586)])
OrderedDict([('tweet', 'Poorly-managed states across the country are pressing for $500 billion in bailout funds, blaming COVID-19 for this need. When in reality, decades of waste and fraud are the real reason for their shortfalls. Why should Tennesseans bankroll their budget failures?'), ('dates', '2020-05-06 15:50:43'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.686092734336853)])
OrderedDict([('tweet', "The U.S. is currently at a crossroads. We can continue to allow China's control of our companies, or, we can explore new avenues like my SOS ACT—to address this issue and ensure American control over our most vital national security and defense systems."), ('dates', '2020-05-06 14:30:36'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8091122508049011)])
OrderedDict([('tweet', 'China has a long track record of swooping in with reams of cash to make strategic purchases of companies in the critical infrastructure, tech, and defense sectors. In 2018, Chinese entities invested some $25 billion in the United States.'), ('dates', '2020-05-05 21:34:22'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5755318999290466)])
OrderedDict([('tweet', 'As Congress and the Trump Administration consider avenues to hold China accountable for their actions in the spread of COVID-19, my new SOS ACT would incentivize investors to support industries that are vital to our national security hit hard by the current economic crisis.'), ('dates', '2020-05-05 18:52:15'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8856245279312134)])
OrderedDict([('tweet', 'This weekend, @SecPompeo said he\'s seen "enormous evidence" the coronavirus originated in a Wuhan lab. As the list of evidence piles up against communist China, there is no question we must hold China accountable. \n\nhttps://t.co/Ue8tZr5ush'), ('dates', '2020-05-04 21:30:21'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.6629564762115479)])
OrderedDict([('tweet', "China is looking to buy greater influence, and at a time when U.S. companies are particularly vulnerable, we need to act now to maintain control of technology critical to our national security. The SOS ACT is the first step in blocking China's acquisition of strategic assets."), ('dates', '2020-05-04 20:15:08'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.936678409576416)])
OrderedDict([('tweet', "China's rulers are hoping a cartoon will rewrite history. The reality is, they did not warn the world. They silenced Taiwan in December from warning the West. They hid the virus' severity so they could hoard supplies."), ('dates', '2020-05-04 17:38:19'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9697467684745789)])
OrderedDict([('tweet', 'The SOS ACT would use $10 billion already appropriated in the CARES Act to protect essential American systems from Chinese state acquisition. We cannot allow the communist dictatorship that spread this virus to also benefit from it, at the cost of our national security.'), ('dates', '2020-05-02 18:10:05'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.721495509147644)])
OrderedDict([('tweet', 'There are many things beyond our control in the coronavirus crisis, but what is in our control is our response. We can learn from the mistakes we made by rewiring our supply lines to no longer be dependent on communist China—clearly, they are not a reliable source.'), ('dates', '2020-05-01 19:20:08'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5925471186637878)])
OrderedDict([('tweet', 'Communist China has clearly mislead the entire globe—and in doing so, shed light on our supply chain. We must secure our systems against China, which is why I introduced the #SOSACT. \n\nI discuss this further with @alexsalvinews: https://t.co/3fefkPssvk'), ('dates', '2020-05-01 15:20:47'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9843575954437256)])
OrderedDict([('tweet', 'As we come up on the other side of this crisis, there is one player who stands out: China. We cannot continue to treat the Chinese Communist Party as if it had our best interests in mind. They concealed the virus from the world, at the cost of lives. They are not our friends.'), ('dates', '2020-04-30 23:11:02'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8650620579719543)])
OrderedDict([('tweet', "Many of our vital companies are struggling thanks to China's concealment of COVID-19. Now, China wants to take advantage of the crisis they caused and gobble up more of our supply chain. That's why I introduced the #SOSACT, to protect our supply lines against China's tactics."), ('dates', '2020-04-29 20:05:07'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9707189798355103)])
OrderedDict([('tweet', 'Communist China wants to take advantage of this economic crisis that their virus caused by buying up companies that are vital to our national defense. We must pass the #SOSACT to secure our systems against China’s predatory tactics.'), ('dates', '2020-04-29 18:04:54'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9928597211837769)])
OrderedDict([('tweet', 'China has already cost American taxpayers over $2 trillion as a result of this virus which they concealed from the world. Here’s a list of ways we’re pushing back against China’s dictatorial tactics—'), ('dates', '2020-04-28 18:04:37'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.978549063205719)])
OrderedDict([('tweet', '.@SpeakerPelosi\'s new \'oversight\' committee is nothing more than a continuation of the Dem leadership\'s tired, endless attacks on @POTUS. 26M Americans have lost their jobs—and all the left cares about is the "tremendous opportunity" to restructure things to fit their vision.'), ('dates', '2020-04-27 20:35:04'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8764483332633972)])
OrderedDict([('tweet', "Communist China is buying up companies that are critical to America's security—Congress must act now to stop China's predatory buying. \n\nMore from my discussion with @cvpayne: https://t.co/EVqkGrFD6k"), ('dates', '2020-04-27 19:24:10'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9801506996154785)])
OrderedDict([('tweet', 'The CCP accuses the U.S. of biowarfare, blames the origination of COVID-19 on our troops, and now—the State Department confirms that Russia and Iran are joining forces in spreading disinformation on the origin of the virus. \n\nhttps://t.co/ZbdZUTkwOj'), ('dates', '2020-04-24 22:18:49'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9416149258613586)])
OrderedDict([('tweet', "I'm heading back to Tennessee today—Congress needs to get back to work. While @SpeakerPelosi has us on recess, Xi's CCP is not taking a break. That's why I introduced a new bill this week to prevent China's acquisition of critical American companies."), ('dates', '2020-04-24 19:05:07'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.88459712266922)])
OrderedDict([('tweet', "9 in 10 American adults see China's power and influence in America as a threat, according to a recent Pew survey. 62% say it's a major threat. It's becoming more clear to every American by the day—we must hold China accountable. https://t.co/pyPPV2NDaB"), ('dates', '2020-04-22 21:05:05'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5389382839202881)])
OrderedDict([('tweet', "Now that @SpeakerPelosi doesn't have the votes to ram proxy voting through, she's pivoted to having us vote on creating the new select committee. We already have an Oversight Committee—let's not turn the rest of this year into another year of endless investigations."), ('dates', '2020-04-22 18:35:07'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8690181374549866)])
OrderedDict([('tweet', "For two weeks, knowing loans for American small businesses were running out, @SpeakerPelosi and @SenSchumer kept delaying and delaying the bill. The agreement to replenish the PPP is LONG overdue. I'm glad to be back in DC to vote on it."), ('dates', '2020-04-22 17:24:13'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.6905252933502197)])
OrderedDict([('tweet', 'I join my colleagues in urging Congressional leadership to ensure no COVID-19 stimulus funds end up at the WIV, China’s state-run lab in Wuhan. For years, U.S. taxpayer dollars have been given to Wuhan’s lab, and @POTUS is cutting those grants: https://t.co/Ma140hWt5O'), ('dates', '2020-04-21 23:54:09'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5046440958976746)])
OrderedDict([('tweet', 'We knew China wanted to control the #coronavirus narrative when they tried to blame Wuhan’s deadly outbreak on U.S. soldiers. Doesn’t make this report from our intel community that Chinese operatives are actively distorting the story any less disturbing.\n\nhttps://t.co/mrleZCPH2Z'), ('dates', '2020-04-21 19:30:16'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9815125465393066)])
OrderedDict([('tweet', "Beijing's propaganda machine continues to churn out false and insulting stories blaming the coronavirus—which originated in China, and possibly in a Wuhan lab—on the U.S. The Arab world is just their latest target. https://t.co/gl11OGByy7"), ('dates', '2020-04-21 16:01:10'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.7615817189216614)])
OrderedDict([('tweet', "Of course the Wuhan lab is going to deny any report that suggest COVID-19 started at the WIV. That doesn't make the story any less true. In fact, it's expected from the same government that kicked out American journalists and hid the virus from the world. https://t.co/9A9TOS4Zs6"), ('dates', '2020-04-21 13:45:14'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5167425870895386)])
OrderedDict([('tweet', "It's the beginning of yet another week and we still don't have new funding measures for the PPP. It's past time for @SpeakerPelosi and @SenSchumer to come to the table for the common good. America's small businesses need relief."), ('dates', '2020-04-20 18:13:59'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5990553498268127)])
OrderedDict([('tweet', "It's not the Chinese people. It's their communist dictators. The same ones that killed hundreds, maybe thousands in Tiananmen Square. \n\nhttps://t.co/cnyRSf6OEE"), ('dates', '2020-04-18 21:00:42'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8879406452178955)])
OrderedDict([('tweet', "Don't be fooled—the Chinese Communist Party knew the destruction this virus would cause in December. Now, with over 2 million cases worldwide, this pandemic has caused utter destruction. The CCP must be held responsible for this deceit."), ('dates', '2020-04-18 14:45:09'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9672350287437439)])
OrderedDict([('tweet', "Remember, the reason we shut down our sense of normalcy was to save lives. Now, those lives are at risk as millions of American families file for unemployment each week. Our businesses can't wait. Dem leadership must fund the PPP, before it's too late."), ('dates', '2020-04-17 20:35:08'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5121544599533081)])
OrderedDict([('tweet', "More on why I'm calling on Congress to cover the expenses of any American company moving back home from China. We can, and should pay for this by using tariffs President Trump levied on China. It's time to bring our companies back to the U.S. https://t.co/6daBkdtuLJ"), ('dates', '2020-04-17 19:30:18'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5536491870880127)])
OrderedDict([('tweet', "Helping America's Main Street shouldn't be a partisan issue. How many more families and their small businesses have to suffer before @SpeakerPelosi and @SenSchumer realize this?"), ('dates', '2020-04-17 18:45:11'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8735634684562683)])
OrderedDict([('tweet', 'Americans are suffering. Their livelihoods have vanished—and yet, @SpeakerPelosi and @SenSchumer continue to drag their feet instead of passing essential funding to the PPP. This is a national disgrace.'), ('dates', '2020-04-17 16:35:08'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9854875802993774)])
OrderedDict([('tweet', "Since the discovery of the virus in December, China's communist regime has taken extraordinary efforts to hide its origination and early spread. Today, Wuhan revised its death toll—which increased by 50%."), ('dates', '2020-04-17 15:43:06'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9760639667510986)])
OrderedDict([('tweet', "China's coverup of the coronavirus—which we're now learning likely originated in a Wuhan lab, and not wet markets as they told us—is yet another item on a long list of lies from the Chinese Communist Party."), ('dates', '2020-04-17 14:45:12'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9889483451843262)])
OrderedDict([('tweet', 'Beijing is now blocking American companies from exporting PPE and test kits from their Chinese factories. This is exactly what we feared would happen, and why I urged Congress to expense American companies to move back home—stat. https://t.co/8FheY7kUrq'), ('dates', '2020-04-16 23:15:05'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.7150423526763916)])
OrderedDict([('tweet', 'The money in the PPP program has just run out. Unbelievable that @SpeakerPelosi and @SenSchumer are still holding additional funding hostage. We are talking about American livelihoods.'), ('dates', '2020-04-16 22:42:42'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9300667643547058)])
OrderedDict([('tweet', 'Every day, the world sees more sickness and death from Xi’s virus—and every day, the world sees more of the Chinese Communist Party’s true colors. It’s time for the Free World to stand together and hold China’s leaders accountable. https://t.co/9nmw4snHxX'), ('dates', '2020-04-16 00:18:31'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.7821619510650635)])
OrderedDict([('tweet', "The WHO's massive mismanagement and coverup of the coronavirus at the behest of the CCP is more than enough reason for @POTUS to cut their funding. There's absolutely no reason for taxpayers to spend $400M+ each year for an organization that reports to China's corrupt regime."), ('dates', '2020-04-15 21:00:22'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8951651453971863)])
OrderedDict([('tweet', 'Chinese officials waited SIX days after secretly determining they were facing a pandemic—letting Wuhan host a mass banquet for millions to celebrate the Lunar New Year—before warning the public. Unacceptable. https://t.co/tVNW3FEJ0y'), ('dates', '2020-04-15 19:45:07'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8773799538612366)])
OrderedDict([('tweet', 'Almost $270B of the CARES Act original $350B for small biz has already been processed and will run out by the end of the week, yet, the Democratic leadership refuses to appropriate more without dangerous strings attached.'), ('dates', '2020-04-15 17:06:53'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5148360729217529)])
OrderedDict([('tweet', "While the left and media are obsessed with @realDonaldTrump’s response in the early months of the virus, let's remember what they were all doing—investigating him and holding up Congress with a sham impeachment."), ('dates', '2020-04-15 14:45:16'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.836000919342041)])
OrderedDict([('tweet', "Enough is enough. It's time to help bring American businesses home, which is why I am urging Congressional leadership to consider the use of existing tariffs levied on China to move our companies back to the U.S. \n\nhttps://t.co/IsHcTUW8zo"), ('dates', '2020-04-15 13:30:32'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5153371095657349)])
OrderedDict([('tweet', 'From their response to the virus, collusion with the WHO, and manipulation of the PPE, the world sees China’s communist leaders for who they are. \n\nhttps://t.co/CoB8S740qC'), ('dates', '2020-04-15 00:49:04'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8042986989021301)])
OrderedDict([('tweet', 'Good. If the WHO wants to keep getting money from American taxpayers, it shouldn’t be carrying water for the Chinese Communist Party.'), ('dates', '2020-04-14 23:34:04'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8003988265991211)])
OrderedDict([('tweet', 'Between December 31—when cases were first reported—and January 31—some 430,000 people were on direct flights from China to the U.S. If @POTUS hadn’t ignored the WHO, who knows how many more Americans would be infected by now?'), ('dates', '2020-04-14 20:27:52'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.963429868221283)])
OrderedDict([('tweet', "We should temporarily allow American companies that move back from China to expense 100% of the moving costs. And, while, I'm personally no fan of tariffs, we should pay for it with the existing tariffs President Trump levied on China, as long as they remain."), ('dates', '2020-04-13 19:19:19'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.883842945098877)])
OrderedDict([('tweet', 'Taiwan warned the WHO about human-to-human transmission, but two weeks later the WHO said they didn’t see any evidence of it. The WHO chose to ignore Taiwan’s warnings in favor of repeating China’s lies instead.'), ('dates', '2020-04-10 20:00:23'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9755844473838806)])
OrderedDict([('tweet', '17 million Americans lost their jobs over the last 3 weeks. Want to know how @SenSchumer responded? By blocking new aid to small businesses. \n\nIt’s critical that we support the PPP to save jobs &amp; protect livelihoods—but Dems stood in the way of money that’s needed now. Shameful.'), ('dates', '2020-04-10 18:30:19'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.913518488407135)])
OrderedDict([('tweet', 'Why does the U.S. fund an organization that serves the interests of the CCP? The WHO delayed declaring a health emergency, downplayed the danger of traveling to China, &amp; disseminated Chinese propaganda despite China’s coverup of a virus that has unleashed untold suffering. https://t.co/jw6ciWqDrf'), ('dates', '2020-04-09 23:16:54'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9239055514335632)])
OrderedDict([('tweet', 'Funding for small businesses in the PPP program is running out, yet today @SpeakerPelosi and @SenSchumer insisted on blocking additional relief for Americans unless they could tack on provisions that expedited their own agenda. Shameful.'), ('dates', '2020-04-09 20:59:38'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8972135186195374)])
OrderedDict([('tweet', "I agree w/ my Democrat colleagues that colleges &amp; universities that need relief funds should get them, &amp; that predatory universities should not. But it's laughable to think that the privileged schools at which Sen. Warren used to teach might also benefit. https://t.co/emOOaY3PQj"), ('dates', '2020-04-09 13:58:18'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.749669075012207)])
OrderedDict([('tweet', "China continued with wet markets knowing the health risk after the SARS outbreak. The CCP's deceit has caused global suffering, and they must be held accountable. \n\nJoined @cvpayne on @FoxBusiness to discuss: https://t.co/B4bjNPgiQ4"), ('dates', '2020-04-07 20:30:11'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.7636004090309143)])
OrderedDict([('tweet', "The House already has an Oversight Committee—I'm on it. We also wrote 3 oversight provisions into the CARES Act. @SpeakerPelosi is just trying to use this global crisis to investigate President Trump, yet again. It never ends! https://t.co/dpuWjSZdHQ"), ('dates', '2020-04-07 15:31:34'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.5599105358123779)])
OrderedDict([('tweet', 'Not only has China\'s deceit caused unbelievable suffering throughout the world—especially in Italy—they are now advancing a PR campaign of "charity" while stooping to the lowest of lows. The CCP must be held accountable—&amp; the Free World must ensure it. https://t.co/WsjT8Plw8v'), ('dates', '2020-04-07 01:36:05'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8513045907020569)])
OrderedDict([('tweet', "China's lies have severely crippled our Nation and the globe's actions to combat COVID-19. Untold numbers of people have suffered needlessly, and the global economy is under catastrophic duress– it's unconscionable."), ('dates', '2020-04-05 18:30:17'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9974045157432556)])
OrderedDict([('tweet', "The United States must refute China's misinformation and hold the Chinese Communist Party accountable for its misconduct. 🔽\nhttps://t.co/yIPHPSX8ci"), ('dates', '2020-04-03 23:00:43'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9752334356307983)])
OrderedDict([('tweet', "Still unsure if China is misreporting their numbers? 600,000 Chinese citizens are back on lock down. This blatant dishonesty from the Chinese Communist Party is utterly unacceptable. It's the reason we're in this crisis in the first place."), ('dates', '2020-04-03 22:30:14'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.9922602772712708)])
OrderedDict([('tweet', "We're in the middle of a global crisis. Americans are suffering. And @SpeakerPelosi is forming yet another committee to investigate President @realDonaldTrump. The insanity never ends."), ('dates', '2020-04-03 20:20:04'), ('file_name', 'Mark E. Green.csv'), ('pct_true', 0.8709427714347839)])
OrderedDict([('tweet', 'What’s happening in the meat industry?\n\n1⃣ Giant monopolies control most of the supply\n2⃣ So COVID cases are a threat to the entire supply chain\n3⃣ Workers aren’t protected, exposing thousands to COVID\n4⃣ Trump doesn’t care if workers get sick and die. https://t.co/3MO17r14xX'), ('dates', '2020-05-08 20:25:33'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.5413903594017029)])
OrderedDict([('tweet', "We cannot allow unchecked corporations &amp; private equity firms to profit off of this pandemic.\n\nI joined @davidcicilline &amp; others to demand a moratorium on corporate mergers during this crisis.\n\nInequality is prevalent across our country, we shouldn't allow a crisis to deepen it. https://t.co/rzoWrmbtBj"), ('dates', '2020-05-05 19:01:54'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.5986278653144836)])
OrderedDict([('tweet', 'A global pandemic is not the moment to silence the voices of the few experts in the room.\n\nCongress and the American people need answers.\n\n5 years ago I remember when Republicans screamed for accountability from the president—where are they now?\nhttps://t.co/Tm4tJSJIUM'), ('dates', '2020-05-05 14:45:13'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.6364175081253052)])
OrderedDict([('tweet', "This president has no conscience.\n\nIn public, it's reopen, liberate &amp; MAGA.\n\nIn private, he knows we're far behind on testing and massive rallies &amp; reopening states now threatens the health of our entire nation.\n\nThis president's addiction to lying is costing thousands of lives. https://t.co/hYD7eOHMlC"), ('dates', '2020-05-04 16:54:45'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.8187404870986938)])
OrderedDict([('tweet', "Imagine if all the energy @senatemajldr put into defending businesses who don't protect their workers was put into...protecting workers? \n\nIt boggles my mind that Republicans rush to defend bad behavior, instead of you know—protecting human lives. https://t.co/uFc9OaKUHB"), ('dates', '2020-05-04 13:47:55'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.6135634183883667)])
OrderedDict([('tweet', 'The last month has exemplified how President Trump failed us all.\n\nDenial of a problem until too late.\n\nLeaving states on their own.\n\nHawking unproven drugs.\n\nTrusting corporate advisors over doctors.\n\nWe need a president that puts our lives over his ego.\nhttps://t.co/x59S1x2MsW'), ('dates', '2020-05-03 21:32:39'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.6491928696632385)])
OrderedDict([('tweet', "Over 3k meat industry workers have COVID-19.\n\nBut @realDonaldTrump wants to keep plants open w/out protecting workers.\n\n@RepDavidEPrice &amp; I are demanding workers get mandatory testing, PPE &amp; enforceable standards.\n\nWe can't protect food supply chains w/out protecting workers. https://t.co/kTL1M0fNx7"), ('dates', '2020-05-01 16:23:10'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.6504375338554382)])
OrderedDict([('tweet', "This is a global pandemic &amp; Trump thinks it's the time to reward political allies &amp; punish his Democratic enemies?\n\nWe're all facing a crisis.\n\nWe need leadership that serves everyone, every state, every territory.\n\nFailing one of us is failing us all.\nhttps://t.co/PCU3PN4TMI"), ('dates', '2020-04-29 13:17:59'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.6121799349784851)])
OrderedDict([('tweet', 'Leave it to this administration to try and legalize healthcare discrimination during a global pandemic.\n\nThis time their anti-LGBTQ bigotry threatens public health.\n\nEveryone should have access to testing and treatment.\n\n@HHSGov must put an end to this NOW.https://t.co/U7zYa9XjhW'), ('dates', '2020-04-24 20:50:30'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.5551190972328186)])
OrderedDict([('tweet', "So many flags here.\n\n1. 'Optics' is not why we're worried about your assault rifles &amp; Confederate flags...\n\n2. WI was never in the Confederacy.\n\n3. This rally is a public health threat.\n\n4. You forgot to have them keep Nazi flags home too, or was that intentional oversight? https://t.co/G8snI1U8GN"), ('dates', '2020-04-23 22:55:55'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.8201558589935303)])
OrderedDict([('tweet', 'Another briefing by this President &amp; I can no longer keep track of whose fault he thinks this pandemic is.\n\nWHO? Governors? China? Obama?\n\nA good leader owns up to their faults.\n\nAnd the truth is Trump’s response to this pandemic has been a failure—period.https://t.co/dlXGz3WXje'), ('dates', '2020-04-14 23:59:27'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.8123117089271545)])
OrderedDict([('tweet', 'Trump is halting funding to the @WHO right now?\n\nDuring a global pandemic?\n\nBecause he thinks THEY mismanaged this crisis? \n \nWe have over 600,000 confirmed COVID-19 cases in the US.\n\nWill this man ever own up to his failures? https://t.co/jUOcYExDjc'), ('dates', '2020-04-14 22:57:39'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.683182418346405)])
OrderedDict([('tweet', "The inequities exposed by this pandemic are not new—we just can’t hide from them anymore.\n\nProtect workers.\n\nRaise wages. \n\nFix our healthcare system.\n\nThese problems won’t go away with this pandemic, so let's do something about it.\nhttps://t.co/wnw1SnGIFz"), ('dates', '2020-04-14 18:57:53'), ('file_name', 'Mark Pocan.csv'), ('pct_true', 0.5930788516998291)])
OrderedDict([('tweet', "We can’t just “wait and see” what happens to the millions who’ve lost their jobs and health insurance in the middle of a pandemic. They need paychecks now.\n \nThe next coronavirus bill should include #PaycheckSecurity for rank-and-file workers who've lost their jobs. https://t.co/CDut9GqSn0"), ('dates', '2020-05-08 18:07:05'), ('file_name', 'Mark R. Warner.csv'), ('pct_true', 0.6472991108894348)])
OrderedDict([('tweet', 'The Senate is back in session next week, but don’t expect any work on the coronavirus crisis.\n \nNo hearings. No oversight. Just packing the courts with more conservative judges.\n \nThe American people deserve better. https://t.co/tqgLzXNz4g'), ('dates', '2020-04-29 15:32:37'), ('file_name', 'Mark R. Warner.csv'), ('pct_true', 0.6115795373916626)])
OrderedDict([('tweet', "Good. We've been telling the Trump administration that big corporations are crowding out small businesses and their workers who desperately need this assistance. Glad we are starting to get somewhere. https://t.co/WdGY6Ok0Sx"), ('dates', '2020-04-23 16:33:18'), ('file_name', 'Mark R. Warner.csv'), ('pct_true', 0.5770182013511658)])
OrderedDict([('tweet', 'In the midst of a national emergency, it is unconscionable that the President is once again attempting to undermine the integrity of the intelligence community by firing yet another intelligence official, the ICIG, simply for doing his job.'), ('dates', '2020-04-04 02:57:24'), ('file_name', 'Mark R. Warner.csv'), ('pct_true', 0.7885261178016663)])
OrderedDict([('tweet', 'This is not the first step Congress has taken to deal with the COVID-19 pandemic, nor will it be the last. This bill provides significant financial relief to our families and businesses struggling with the effects of widespread closures and other public health measures. https://t.co/8mr5ECpFY7'), ('dates', '2020-03-26 03:56:43'), ('file_name', 'Mark R. Warner.csv'), ('pct_true', 0.5462780594825745)])
OrderedDict([('tweet', 'How about instead of trying to keep the public in the dark about unemployment data, we focus on trying to help people who are losing jobs and paychecks because of the coronavirus.\n \nWe will get through this, but fudging the numbers isn’t going to help. https://t.co/J7YN3h4sSq'), ('dates', '2020-03-20 13:44:52'), ('file_name', 'Mark R. Warner.csv'), ('pct_true', 0.6199358701705933)])
OrderedDict([('tweet', 'Attacking the press and stifling accurate coverage of the Chinese Communist Party’s failings while the world is distracted by a pandemic is a deeply cynical move and shows you exactly what kind of regime President Xi is running. https://t.co/8Frbs1ISqj'), ('dates', '2020-03-18 19:53:52'), ('file_name', 'Mark R. Warner.csv'), ('pct_true', 0.9600569605827332)])
OrderedDict([('tweet', 'The President continues to claim he supports protecting people with preexisting conditions, yet he continues to pursue a lawsuit that will rip protections away from millions of Americans. \nhttps://t.co/EOGfA9m3N2'), ('dates', '2020-03-02 17:16:58'), ('file_name', 'Mark R. Warner.csv'), ('pct_true', 0.7036781311035156)])
OrderedDict([('tweet', 'The truth is – an overwhelming majority of Americans support background checks for gun sales. And one full year after House passage of #HR8, it’s long past time to bring this commonsense bill up for a Senate vote. #BackgroundChecksSaveLives'), ('dates', '2020-02-27 19:12:00'), ('file_name', 'Mark R. Warner.csv'), ('pct_true', 0.6560948491096497)])
OrderedDict([('tweet', 'Workers are getting sick and dying from #COVID19 because OSHA is refusing to issue emergency standards to keep workers safe.\n\nNo one should have to risk their life at work. The Trump Administration must step up and protect workers. #WorkersMemorialDay https://t.co/IggjRsTfh0'), ('dates', '2020-04-28 21:52:23'), ('file_name', 'Mark Takano.csv'), ('pct_true', 0.5788716673851013)])
OrderedDict([('tweet', "Long-term exposure to pollution has a significant effect on #COVID19 mortality rates. This is why we can't let Trump get away with rolling back environmental protections. His attack on clean air threatens communities of color that are bearing the brunt of this crisis. #EarthDay https://t.co/EEi1LrddfQ"), ('dates', '2020-04-22 15:51:14'), ('file_name', 'Mark Takano.csv'), ('pct_true', 0.7100767493247986)])
OrderedDict([('tweet', 'There has been a surge in reports of hate and harassment against Asian Americans. This is exactly why we called on Trump and other political leaders to stop using racist terms to refer to #COVID19. Everyone must condemn this. https://t.co/6mcKCUbSMX'), ('dates', '2020-04-17 18:16:06'), ('file_name', 'Mark Takano.csv'), ('pct_true', 0.8092656135559082)])
OrderedDict([('tweet', "Trump is treating the economic relief Americans desperately need like a big campaign stunt. He keeps making the response to #COVID19 all about him and failing to exert the leadership we need to get through this crisis. For him, it's always Trump First.\nhttps://t.co/CKT6puDWoJ"), ('dates', '2020-04-15 14:10:53'), ('file_name', 'Mark Takano.csv'), ('pct_true', 0.5541782975196838)])
OrderedDict([('tweet', ".@realDonaldTrump's claims about suicide are reckless and harmful to people experiencing a mental health crisis — including our veterans. The President should be more careful and considerate when addressing the nation, especially during these difficult times. https://t.co/eImvlc6ZuM"), ('dates', '2020-03-24 18:23:11'), ('file_name', 'Mark Takano.csv'), ('pct_true', 0.6205664873123169)])
OrderedDict([('tweet', "General Flynn was improperly targeted by the DOJ in yet another baseless attempt to ruin the lives of those supportive of @realDonaldTrump.\n\nNow that he's been exonerated, who will be held accountable for this egregious smear campaign of a war hero?"), ('dates', '2020-05-08 01:52:05'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.5117149353027344)])
OrderedDict([('tweet', 'Americans across our nation should never live in fear that without video evidence, crimes will go unseen, voices won’t be heard, and countless innocent lives will become forgotten.\n\nJustice for Ahmaud Arbery needs to be ensured - and served. #JusticeForAhmaud'), ('dates', '2020-05-07 02:30:52'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.6156927943229675)])
OrderedDict([('tweet', 'China’s malpractice and delay unquestionably threatened Americans’ safety and security. We must hold them accountable for these atrocities.\n\nhttps://t.co/s85tYfzCL5'), ('dates', '2020-05-04 13:24:01'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.8686298131942749)])
OrderedDict([('tweet', 'The govt cannot &amp; will never be what drives our economic recovery – only our small businesses can do that.\n\nEvery dollar we spend out of thin air gives China more control over our debt, economy, &amp; supply chain. Any more funding should be limited until we get America back to work. https://t.co/RLi5kwuc9S'), ('dates', '2020-04-30 18:08:25'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.6922621726989746)])
OrderedDict([('tweet', 'The DOJ improperly targeted @realDonaldTrump + his staff. Flynn is a war hero and they destroyed his life because they didn’t like the President.\n\nTime and time again, DOJ has acted in an embarrassing and detrimental fashion. Until someone goes to jail, this behavior won’t stop.'), ('dates', '2020-04-30 16:50:07'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.5267606973648071)])
OrderedDict([('tweet', "NCAA doesn't need anti-trust exemptions. They have spent decades using their lawyers to keep young men and women from receiving basic rights, even as they grew to a $1B-a-year organization. I'm sure those same lawyers can help them navigate this w/o congressional intervention. https://t.co/FUQWxA4ptu"), ('dates', '2020-04-29 15:31:29'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.7706888318061829)])
OrderedDict([('tweet', 'The next time Speaker Pelosi and Majority Leader Hoyer want to accuse @realDonaldTrump of not having a plan to get America back to work, I would like them to remember that they didn’t have a plan to get 435 people back to Congress next week. https://t.co/tuMfF72Ako'), ('dates', '2020-04-28 17:43:15'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.6810204982757568)])
OrderedDict([('tweet', 'Senate Democrats + Speaker Pelosi blocked a clean bill for more PPP funds, but just blaming them is unacceptable.\n\nMcConnell + Pelosi need to call every Member back on Monday to put their priority on the record: keeping people employed + businesses open or their partisan agenda.'), ('dates', '2020-04-17 16:52:20'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.8944418430328369)])
OrderedDict([('tweet', 'PPP is now exhausted and it’s because of politicians thinking their agenda is more important than keeping people employed. This program has already helped more than 24,000 small businesses in North Carolina keep their workers paid. People’s livelihoods are not bargaining chips. https://t.co/BmP0v0rqN1'), ('dates', '2020-04-16 18:15:16'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.7519562840461731)])
OrderedDict([('tweet', '"China’s attempt to walk a line between alerting the public and avoiding panic set the stage for a pandemic that has infected almost 2 million people and taken more than 126,000 lives.” https://t.co/l4zox6TCja'), ('dates', '2020-04-15 14:10:06'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.7195006012916565)])
OrderedDict([('tweet', "China's lies and disinformation are unacceptable. We will hold them accountable for their role in the spread of the #COVID19 pandemic that has cost American lives and shut down our economy. https://t.co/iYUxMwAoac"), ('dates', '2020-04-09 18:55:56'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.9724618792533875)])
OrderedDict([('tweet', 'Keeping American family-owned businesses open and paying their employees is not a bargaining chip for Democrats in Washington.\n\nSmall business owners are not hostages.\n\nPeople are hurting and all some seem to care about is using their pain for a political agenda. It must end now! https://t.co/1hjI3xaXns'), ('dates', '2020-04-09 16:14:01'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.8426774740219116)])
OrderedDict([('tweet', 'I guess Clyburn wasn’t bluffing when he stated that COVID-19 pandemic "is a tremendous opportunity to restructure things to fit our [House Democrats] vision.” https://t.co/NZv8mV2D5x'), ('dates', '2020-04-02 16:51:00'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.557485044002533)])
OrderedDict([('tweet', 'China’s lack of transparency created this. Now we are asking American taxpayers to foot the bill? Not hardly! \n\nI am working on a plan to make the Communist Party of China pay for this stimulus package. More to come soon.'), ('dates', '2020-03-26 18:39:22'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.982093095779419)])
OrderedDict([('tweet', 'These policies fail on their merits, so partisan hacks in DC are trying to exploit the suffering of Americans and the current crisis to achieve a wish list.\n\nENOUGH.\n\nRepublicans, Democrats and Independents in North Carolina don’t care about your agenda right now! https://t.co/Gkyal3yEOj'), ('dates', '2020-03-23 15:13:11'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.9100139737129211)])
OrderedDict([('tweet', 'People are scared. People are hurting. Jobs are being lost. Retirements are being destroyed.\n\nYet DC Democrats just killed legislation they helped write to lift up US families and small businesses. Why? Partisan bombs and gamesmanship. \n\nEveryone needs to get over themselves. https://t.co/JgLstZ9LD9'), ('dates', '2020-03-23 14:54:47'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.9054127335548401)])
OrderedDict([('tweet', 'What has been in America for awhile is freedom – something you despise. \n\nWe have our hands full with your failures right now, but we will get through it. \n\nWhen we do, we’ll be in touch with your communist party, @zlj517. https://t.co/x2IoWXVp2i'), ('dates', '2020-03-22 19:20:13'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.6889608502388)])
OrderedDict([('tweet', 'For too long, economic policies have encouraged companies to shift manufacturing of vital medicines and equipment to China.\n\nA lesson from the #Coronavirus is that we must be less reliant on China and keep our supply chains secure during times of crisis. Our safety depends on it.'), ('dates', '2020-03-20 15:12:57'), ('file_name', 'Mark Walker.csv'), ('pct_true', 0.663455605506897)])
OrderedDict([('tweet', 'Over 35,000 businesses in OK have already received funding from the #PaycheckProtectionProgram to keep their doors open &amp; employees at work. But there are thousands more who have been waiting &amp; need help while Speaker Pelosi &amp; Senator Schumer played politics w/ their livelihoods.'), ('dates', '2020-04-23 22:17:57'), ('file_name', 'Markwayne Mullin.csv'), ('pct_true', 0.5494222044944763)])
OrderedDict([('tweet', '#TBT to last Thursday when Democrats blocked additional funding for small businesses to keep their workers employed. Now, the Paycheck Protection Program is out of money and millions of small businesses won’t be able to get help.'), ('dates', '2020-04-16 18:23:21'), ('file_name', 'Markwayne Mullin.csv'), ('pct_true', 0.7958829402923584)])
OrderedDict([('tweet', '99% of Oklahoma’s businesses are small businesses and it’s appalling Speaker Pelosi and Senator Schumer are leaving them out to dry. https://t.co/siFWPbTZaZ'), ('dates', '2020-04-16 17:09:45'), ('file_name', 'Markwayne Mullin.csv'), ('pct_true', 0.7285359501838684)])
OrderedDict([('tweet', "It is despicable that Speaker Pelosi and Senator Schumer are blocking critical additional funding for small businesses and the millions of people who depend on them for a paycheck. Stop playing politics with people's livelihoods! \n\nhttps://t.co/8a4N2UJaAU"), ('dates', '2020-04-15 21:06:27'), ('file_name', 'Markwayne Mullin.csv'), ('pct_true', 0.8302410840988159)])
OrderedDict([('tweet', 'Between COVID-19 decreasing demand and foreign actors intentionally disrupting the global oil market, American oil producers and the jobs they support are hurting.'), ('dates', '2020-04-08 20:13:32'), ('file_name', 'Markwayne Mullin.csv'), ('pct_true', 0.7526340484619141)])
OrderedDict([('tweet', 'Now is not the time to play political games and push partisan agendas. We need to do what is necessary to keep the American people safe. https://t.co/WnZVmMGmd3'), ('dates', '2020-03-04 17:30:03'), ('file_name', 'Markwayne Mullin.csv'), ('pct_true', 0.5264894962310791)])
OrderedDict([('tweet', 'Left-wing Democrats just blocked legislation that ensures newborn babies who survive an abortion receive medical care. I am appalled and disgusted. If I was there to vote today, I would have voted YES to protect innocent babies.'), ('dates', '2020-02-28 17:18:33'), ('file_name', 'Markwayne Mullin.csv'), ('pct_true', 0.8747348785400391)])
OrderedDict([('tweet', 'From day one, this has been entirely politically driven &amp; it’s no surprise that @realDonaldTrump was acquitted today. Throughout this entire impeachment process one fact has remained the same: President Trump has not committed a single impeachable offense.'), ('dates', '2020-02-05 21:50:37'), ('file_name', 'Markwayne Mullin.csv'), ('pct_true', 0.7652983069419861)])
OrderedDict([('tweet', 'Authoritarian regimes like China, Russia &amp; Turkey have exploited COVID-19 to consolidate power &amp; silence critics. @SenMarkey &amp; I introduced the #ProtectingHumanRightsDuringPandemicAct to ensure the United States is at the forefront of the fight to protect human rights.'), ('dates', '2020-05-08 17:26:00'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9203385710716248)])
OrderedDict([('tweet', 'The coronavirus does not respect geographic borders. It is a global pandemic and therefore the effort to combat it must also be global. Yet China is focused on blocking Taiwan from attending the @WHO summit – all because Taiwan stands up to the #CCP. https://t.co/BnnpDcVfbG'), ('dates', '2020-05-07 16:24:45'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.8039100170135498)])
OrderedDict([('tweet', 'Adam Schiff cried for transparency when he claimed the Trump campaign colluded with Russia.\n\nNow, he refuses to release the transcripts from the House Intelligence Committee’s Russia probe.\n\nWhat is Adam Schiff hiding this time?\nhttps://t.co/02E3ou0IGq'), ('dates', '2020-05-07 02:04:06'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.6643003225326538)])
OrderedDict([('tweet', 'The Chinese Government spent the early days of the COVID-19 outbreak destroying testing samples, intimidating doctors, and expelling journalists who refused to publish Communist Party propaganda.\n \nThey lied to everyone about how dangerous this virus is, &amp; they did it on purpose. https://t.co/z1MRe5tnt9'), ('dates', '2020-05-07 00:56:37'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9174385666847229)])
OrderedDict([('tweet', 'We can’t shake hands right now because China and the @WHO sealed the deal with a handshake to keep the coronavirus a secret. https://t.co/6t8bEdPrTK'), ('dates', '2020-05-06 19:13:49'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9331603646278381)])
OrderedDict([('tweet', 'Discussing my Stop COVID Act with @kevcirilli, which will hold Communist China accountable for lying to the world. https://t.co/4N1xA370Wl'), ('dates', '2020-05-06 16:39:28'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.885878324508667)])
OrderedDict([('tweet', 'China lied to the world and failed to #StopCOVID. \n\nAmericans should be able to sue the Chinese Communist Party and its minions for the damage they’ve inflicted. #CCP'), ('dates', '2020-05-06 14:36:07'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.999434769153595)])
OrderedDict([('tweet', 'I introduced the Stop COVID Act to give all Americans the ability to sue China in U.S. courts for the damage they’ve inflicted on our country. \nhttps://t.co/hje18SMzu7'), ('dates', '2020-05-05 17:35:58'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.6684310436248779)])
OrderedDict([('tweet', 'Why is Pelosi not at work?\n\nIs she cooking up backroom deals while keeping her House members in the dark?'), ('dates', '2020-05-04 23:59:27'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9666234850883484)])
OrderedDict([('tweet', 'The American people deserve the ability to hold China directly accountable for their dishonest behavior.  \n\nEnough is enough. https://t.co/6snWbq8DSy'), ('dates', '2020-05-04 15:31:24'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.831662654876709)])
OrderedDict([('tweet', '.@realDonaldTrump China should pay for the damages they’ve caused.\n \nMy Stop COVID Act will allow Americans to hold China financially accountable in US Court. https://t.co/lx1uVYHW7o'), ('dates', '2020-05-01 17:04:59'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9877534508705139)])
OrderedDict([('tweet', 'China is a reckless and dangerous communist regime. \n\nWe must hold them accountable. https://t.co/9qTqYvHgoq'), ('dates', '2020-04-30 23:12:57'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.8978291153907776)])
OrderedDict([('tweet', 'The Democrats failed at impeachment and will stop at nothing to bring down @realDonaldTrump. https://t.co/Hcy0yt8alD'), ('dates', '2020-04-30 22:14:30'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.6453706622123718)])
OrderedDict([('tweet', 'China’s goal is to run your life. \n \nPower is what they are seeking. https://t.co/9eEHyyi4Ec'), ('dates', '2020-04-30 18:45:47'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.5015413761138916)])
OrderedDict([('tweet', 'We can’t trust the Chinese government.\n\nWe need to independently investigate what role the Wuhan lab and communist party played in the COVID-19 outbreak. https://t.co/eYc881HNI8'), ('dates', '2020-04-30 16:40:35'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.900631308555603)])
OrderedDict([('tweet', '✅@realDonaldTrump banned \ninternational flights from hotspots in January\n\n❌China permitted international flights, spreading the virus to other countries\n\n✅@realDonaldTrump is rapidly expanding testing for Americans each day.\n\n❌China destroyed testing and evidence of COVID-19'), ('dates', '2020-04-29 23:52:41'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.6765552163124084)])
OrderedDict([('tweet', 'Newsflash, @CNN:\n—“Modern China” is a brutal communist country\n\n—The “Beijing Model” is to lie about the virus &amp; death toll\n\n—The “empowered” #CCP silenced doctors &amp; destroyed testing kits\n\n𝙎𝙩𝙤𝙥 𝙥𝙪𝙗𝙡𝙞𝙨𝙝𝙞𝙣𝙜 𝘾𝙝𝙞𝙣𝙚𝙨𝙚 𝙥𝙧𝙤𝙥𝙖𝙜𝙖𝙣𝙙𝙖. https://t.co/vyz9JCPOm1 https://t.co/3P3TLyQRLv'), ('dates', '2020-04-29 19:36:46'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.8849973082542419)])
OrderedDict([('tweet', 'If Communist China spent half the effort it does issuing threats and pushing its propaganda on preventing the spread of COVID-19, we likely would not be in the midst of a global pandemic. https://t.co/jDyfLOvctx'), ('dates', '2020-04-29 18:24:42'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9477073550224304)])
OrderedDict([('tweet', 'Communist China destroyed early COVID-19 testing samples, silenced doctors, and expelled American journalists. \n\nMy Stop COVID Act will empower Americans to sue China in US courts for the costs of the virus. https://t.co/7QrBo610kT'), ('dates', '2020-04-28 19:06:51'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9647036790847778)])
OrderedDict([('tweet', 'China has launched an all-out campaign to deflect from their atrocious coronavirus response. We should commend EU partners like Germany who are transparent about Chinese efforts and reject their strong-arm tactics, while encouraging nations like France to resist #CCP propaganda.'), ('dates', '2020-04-28 17:40:06'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9679839611053467)])
OrderedDict([('tweet', 'China has cost the US trillions in economics damages, caused millions of Americans to lose their jobs, ripped families apart, and forced loved ones to die alone. \n\n@realDonaldTrump is right to seek damages from the Communist regime. https://t.co/s8xbfl3zaP'), ('dates', '2020-04-28 15:12:05'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.971611499786377)])
OrderedDict([('tweet', 'The audacity of @DrTedros is surreal. \n\n@WHO and Communist China are why COVID-19 turned into a global pandemic. \n\nGod only knows how many more people would be dead if the world listened to the @WHO. https://t.co/3wYxMjlLqF'), ('dates', '2020-04-27 19:22:31'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9489272236824036)])
OrderedDict([('tweet', '#ChinaLiedPeopleDied, and now the communist regime must be held accountable. My Stop COVID Act will give American small business owners the ability to take the #CCP to court in the US.\n \nhttps://t.co/fe8x9wdI3W'), ('dates', '2020-04-27 17:47:31'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.8866276144981384)])
OrderedDict([('tweet', 'China’s Communist government is grasping at straws. \n\nThey are desperate to hide their role in spreading the coronavirus. \n\nThe #CCP is threatening economic retaliation against Australia if the country pursues an independent investigation of the outbreak. https://t.co/JtaMJF52Lc'), ('dates', '2020-04-27 14:24:10'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9889312386512756)])
OrderedDict([('tweet', 'We need answers from @WHO @DrTedros:\n \nWhat took place when you met with Xi Jinping on January 28?\n \nWhy didn’t you recommend restricting Chinese travel and trade on January 30?\n \nhttps://t.co/tRryCgwB1B'), ('dates', '2020-04-26 22:18:18'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.5506047606468201)])
OrderedDict([('tweet', 'The EU is self-censoring itself to appease the Chinese government. \n\nAfter Chinese diplomats threatened retaliation, @eu_eeas officials rewrote a report that criticized the #CCP for its global disinformation campaign. Our global partners must not kowtow to Communists in Beijing.'), ('dates', '2020-04-26 17:07:11'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9829792380332947)])
OrderedDict([('tweet', 'In Illinois, liberal politicians cut sweetheart deals with corrupt union bosses to keep each other in power.\n\nNow, the Democrat run state is using the guise of corona to ask Washington for a taxpayer bailout for their Big Labor pals. @senatemajldr'), ('dates', '2020-04-25 12:39:11'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9606820940971375)])
OrderedDict([('tweet', 'My Stop COVID Act with @SenMcSallyAZ and @Lancegooden puts the power squarely in the hands of the American people and gives them the opportunity to sue China for spreading the coronavirus. https://t.co/6bzjdDYWgG'), ('dates', '2020-04-23 19:24:05'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.552278995513916)])
OrderedDict([('tweet', 'NY and other left wing states have been terribly mismanaged for years. Kowtowing to labor unions by approving generous taxpayer funded benefits in exchange for votes has left them with billion dollar shortfalls. Blame yourself @NYGovCuomo, not @senatemajldr.'), ('dates', '2020-04-23 17:19:33'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.8586286902427673)])
OrderedDict([('tweet', 'The Chinese Communist Party has cost us:\n \n—Trillions in economic damage\n \n—More than 22 million American jobs\n \n—Over 180,000 people world wide\n \n𝙒𝙚 𝙣𝙚𝙚𝙙 𝙖𝙘𝙘𝙤𝙪𝙣𝙩𝙖𝙗𝙞𝙡𝙞𝙩𝙮.\nhttps://t.co/E2KAZ7Hqhm'), ('dates', '2020-04-23 14:59:02'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9379499554634094)])
OrderedDict([('tweet', 'Who’s the biggest liar of 2020?'), ('dates', '2020-04-23 00:08:31'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.5623598098754883)])
OrderedDict([('tweet', 'Nancy Pelosi must have a PhD in revisionist history. \n\nWhile the Democrats spent days blocking funding for small businesses, American dreams went down the drain and workers were laid off. https://t.co/mfVLnjPPO7'), ('dates', '2020-04-22 18:25:46'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9287968277931213)])
OrderedDict([('tweet', 'My Stop COVID Act gives Americans the opportunity to take Communist China to court and demand accountability for the regime’s lies and deceit. https://t.co/zENvzqTiev'), ('dates', '2020-04-22 16:02:30'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.8563169836997986)])
OrderedDict([('tweet', 'It is alarming that @YouTube now relies on the @WHO’s biased recommendations to decide what video content to take down.\n\xa0\nThe @WHO gave inaccurate information about the spread of the virus, still parrots Chinese propaganda, and fought @realDonaldTrump’s China travel ban. https://t.co/eTG9JG2eRs'), ('dates', '2020-04-22 12:14:05'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.882496178150177)])
OrderedDict([('tweet', '.@WHO is led by a pro-China communist sympathizer. It’s no surprise @DrTedros has defended China’s handling of the virus.'), ('dates', '2020-04-21 23:04:16'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9975569248199463)])
OrderedDict([('tweet', 'While @DrTedros was Ethiopia’s foreign minister, Communist China donated generously to his country. \n\nThey strongly backed his campaign for @WHO director-general.\n \nNow, @DrTedros is likely repaying the favor by covering up China’s lies and deceit.'), ('dates', '2020-04-21 17:06:41'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9608186483383179)])
OrderedDict([('tweet', 'Tennessee small business owners are frustrated with @SpeakerPelosi and @SenSchumer because they’re using small businesses as a bargaining chip. Fund the @SBA loan program now!'), ('dates', '2020-04-21 12:22:07'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9370549917221069)])
OrderedDict([('tweet', '.@WHO director-general @DrTedros is accused of covering up three health epidemics in Ethiopia.\n \n@WHO helped Beijing cover up the Chinese coronavirus pandemic.\n \nWhat else is the @WHO hiding?'), ('dates', '2020-04-21 00:51:05'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.8215868473052979)])
OrderedDict([('tweet', 'Every minute that @SenSchumer and @SpeakerPelosi deny funding for small businesses, another American’s dream goes down the drain. Show our millions of small business owners some respect.'), ('dates', '2020-04-20 23:38:09'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.929686963558197)])
OrderedDict([('tweet', 'We cannot accept the narrative the #CCP wants us to believe. \n \nWe cannot rely on the Chinese government to be honest.\n \nWe cannot allow Beijing to escape accountability.\n \nAmericans must be able to seek justice &amp; make China pay the costs of its cover up.\nhttps://t.co/qlcHNolCDE'), ('dates', '2020-04-20 19:09:27'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.7295488119125366)])
OrderedDict([('tweet', 'It’s time to hold Beijing accountable. \n\nThe Stop COVID Act will allow Americans to sue Communist China and seek compensation if they suffered harm resulting from the regime’s role in spreading this virus.\n@Lancegooden\nhttps://t.co/qlcHNolCDE'), ('dates', '2020-04-20 17:14:22'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9742926359176636)])
OrderedDict([('tweet', 'My Stop COVID-19 Act will make China legally and financially liable for unleashing the COVID-19 infection on our country. @SenMcSallyAZ @Lancegooden\n \n#ChinaLiedPeopleDied'), ('dates', '2020-04-20 14:35:11'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.8630555272102356)])
OrderedDict([('tweet', 'China’s Communist Party must face consequences for spreading the coronavirus. \n\nThe costs have been devastating: trillions of dollars in economic damage, more than 22 million American jobs lost, and over 150,000 deaths worldwide and counting. \n\nhttps://t.co/eLIax5xVJl'), ('dates', '2020-04-20 00:45:11'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9727069139480591)])
OrderedDict([('tweet', '.@DrTedros @WHO is a pro-China communist sympathizer accused of covering up health epidemics in the past.\n \nIn Ethiopia, @DrTedros was part of the brutal authoritarian regime that ruled the country with an iron fist.'), ('dates', '2020-04-19 21:21:55'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9816312193870544)])
OrderedDict([('tweet', 'China knew about the coronavirus, hid it from the world, and bought massive amounts of personal protective equipment off the market. \n\nNow, they are sitting on this stockpile and profiteering from it. \nhttps://t.co/yccRfvVQgu'), ('dates', '2020-04-19 19:54:14'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.6003815531730652)])
OrderedDict([('tweet', '#ChinaLiedPeopleDied\n \nChina cost the USA trillions of dollars in damages and wrecked our economy.\n \nNow, they must pay the price.\n \nI introduced the Stop COVID-19 Act to allow Americans to sue Communist China and seek justice in US courts. @SenMcSallyAZ @Lancegooden'), ('dates', '2020-04-18 21:59:53'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9965341091156006)])
OrderedDict([('tweet', 'We’ve already seen China use the guise of the coronavirus pandemic to expel journalists, arrest dissidents, and target Tibetans and Uyghurs. Now, Beijing is expanding their efforts, arresting over a dozen pro-Democracy activists in Hong Kong. https://t.co/a0if6ZYm6d'), ('dates', '2020-04-18 16:49:36'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.974528968334198)])
OrderedDict([('tweet', 'Since the beginning of the COVID-19 outbreak, the Chinese Communist Party has gone to great lengths to cover up, deflect, and blame others for the fallout. \n\nThe @WHO pushed China’s propaganda, which helped the virus and misinformation about it spread.'), ('dates', '2020-04-17 18:33:23'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9792178869247437)])
OrderedDict([('tweet', 'China’s Communist Party lied to the world about how the coronavirus virus spread from Wuhan.\n \nThey wreaked death and havoc on millions of families. \n \nThey destroyed American businesses and jobs. \n \nNo longer will the malevolent regime be able to escape justice in our courts.'), ('dates', '2020-04-17 15:48:37'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9547195434570312)])
OrderedDict([('tweet', 'This is the least surprising thing I’ve seen on twitter. China’s chief propagandist @zlj517 retweeting @WHO @DrTedros. https://t.co/im0U38vQCY'), ('dates', '2020-04-17 01:01:03'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.7935971021652222)])
OrderedDict([('tweet', '.@WHO is one of the biggest offenders of posting misinformation.\n \nThey ignored information that the coronavirus could be spread from person to person.\n \nThey lashed out at @realDonaldTrump for restricting travel.\n \nThey pushed #CCP propaganda about China’s COVID-19 case count.'), ('dates', '2020-04-16 20:58:49'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.8956946134567261)])
OrderedDict([('tweet', '.@SenMcSallyAZ &amp; I are introducing the Stop COVID Act to make China liable for unleashing COVID-19 on our country. Americans victimized by Chinese Communist Party lies &amp; deceit deserve the opportunity to take China to court in the U.S. &amp; demand compensation. @LanceGooden'), ('dates', '2020-04-16 20:06:20'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.9959477782249451)])
OrderedDict([('tweet', 'There is only one program that is out of funds — the #PaycheckProtectionProgram. \n\nBut Democrats are fixated on pumping endless taxpayer dollars into areas assistance hasn’t even been distributed!'), ('dates', '2020-04-16 19:08:45'), ('file_name', 'Marsha Blackburn.csv'), ('pct_true', 0.6636885404586792)])
OrderedDict([('tweet', 'China is our adversary. We need to confront them economically by bringing our manufacturing home, and we need to hold them accountable for their handling of the coronavirus. https://t.co/4bxSHEg8nW'), ('dates', '2020-05-06 21:33:59'), ('file_name', 'Martha McSally.csv'), ('pct_true', 0.895661473274231)])
OrderedDict([('tweet', "Taxpayer $ shouldn't be sent to a dangerous Chinese state-run bio-agent lab that lacks any meaningful oversight from U.S. authorities &amp; is run by adversaries w/ a history of lab leaks.\n\nI led a letter w/ @RepMattGaetz to Senate &amp; House leadership to ensure this funding is cut off https://t.co/528Q05Qac4"), ('dates', '2020-04-22 15:39:11'), ('file_name', 'Martha McSally.csv'), ('pct_true', 0.6112635731697083)])
OrderedDict([('tweet', 'I’m partnering with @MarshaBlackburn to introduce legislation that will allow Americans to file lawsuits against the Chinese Communist Party for its role in perpetuating the global spread of the coronavirus. The Chinese government must be held accountable. https://t.co/bMydIvxTwy'), ('dates', '2020-04-20 17:31:48'), ('file_name', 'Martha McSally.csv'), ('pct_true', 0.6805317997932434)])
OrderedDict([('tweet', 'The Chinese government must be held accountable for the pain its inflicted across the U.S. Our legislation will allow Americans to file lawsuits against Beijing for its role in perpetuating the coronavirus. https://t.co/aTrDFdtO5O'), ('dates', '2020-04-16 21:33:59'), ('file_name', 'Martha McSally.csv'), ('pct_true', 0.5646713376045227)])
OrderedDict([('tweet', 'There is zero doubt that the Chinese communist government has American blood on its hands. The NIH must immediately stop deploying American tax dollars to China for this dangerous research. https://t.co/skkRQ6DpBa'), ('dates', '2020-04-16 18:42:30'), ('file_name', 'Martha McSally.csv'), ('pct_true', 0.985353410243988)])
OrderedDict([('tweet', '⏰ Congress needs to pass additional funding for the Paycheck Protection Program—and soon. 1M small biz loans have been approved, but there are 30M small businesses in our nation who need help. \n \nIt’s time to put politics aside and put American workers &amp; small businesses first. https://t.co/6yi6OXTVlJ'), ('dates', '2020-04-15 17:22:02'), ('file_name', 'Martha McSally.csv'), ('pct_true', 0.5104278326034546)])
OrderedDict([('tweet', "#COVID19 is disproportionately impacting Tribal nations. That's why I've introduced legislation to eliminate the cost-share requirements tied to emergency protective measures, which will allow Tribal governments to access 100% of the funding they need. https://t.co/vh460WSUoY"), ('dates', '2020-05-08 14:28:01'), ('file_name', 'Martin Heinrich.csv'), ('pct_true', 0.5993119478225708)])
OrderedDict([('tweet', "Tomorrow, the Supreme Court will hear cases challenging a Trump admin rule that exempts employers from providing the #ACA birth control benefit.\n \nWomen's health decisions shouldn't depend on where they work. Ideological views shouldn't interfere with their choices. #HandsOffMyBC"), ('dates', '2020-05-05 19:34:17'), ('file_name', 'Martin Heinrich.csv'), ('pct_true', 0.8441349267959595)])
OrderedDict([('tweet', 'If McConnell calls the Senate into session next week, it should not be to pack the courts with his unqualified nominees.\n\nThe Senate should be holding public oversight hearings on shortages of testing supplies and PPE and why small business loans were given to huge corporations.'), ('dates', '2020-04-28 17:47:50'), ('file_name', 'Martin Heinrich.csv'), ('pct_true', 0.7359660863876343)])
OrderedDict([('tweet', 'Funny, I don’t remember Majority Leader Mitch McConnell being concerned about deficits when he shoveled billions in tax breaks to his donors. https://t.co/LJzDHVzmbw'), ('dates', '2020-04-22 15:43:30'), ('file_name', 'Martin Heinrich.csv'), ('pct_true', 0.505023717880249)])
OrderedDict([('tweet', "It's been over a year since the House passed #VAWA. As many stay home due to #COVID19, victims of domestic violence are being presented with a dangerous situation. It's time for the Senate to stand up for these survivors and vote to reauthorize VAWA immediately. #VAWA4ALL"), ('dates', '2020-04-07 19:38:29'), ('file_name', 'Martin Heinrich.csv'), ('pct_true', 0.5482960343360901)])
OrderedDict([('tweet', 'Good. Congress made it explicitly clear in the law that Social Security beneficiaries do not need to take any additional action in order to receive their payments. https://t.co/ncX01QKLJK'), ('dates', '2020-04-02 02:06:44'), ('file_name', 'Martin Heinrich.csv'), ('pct_true', 0.5513309836387634)])
OrderedDict([('tweet', 'This naked power grab shows why Attorney General Barr has no credibility and should resign. \n \nhttps://t.co/DU4FmKrC3B'), ('dates', '2020-03-22 15:29:47'), ('file_name', 'Martin Heinrich.csv'), ('pct_true', 0.6627124547958374)])
OrderedDict([('tweet', 'This afternoon, the Senate put partisan politics aside to pass the emergency medical and immediate economic relief measures Americans are counting on as we confront this unprecedented public health crisis.'), ('dates', '2020-03-18 23:55:52'), ('file_name', 'Martin Heinrich.csv'), ('pct_true', 0.68451988697052)])
OrderedDict([('tweet', 'Ron Johnson is turning the Homeland Security Committee into an instrument of Russian disinformation.'), ('dates', '2020-03-10 18:13:53'), ('file_name', 'Martin Heinrich.csv'), ('pct_true', 0.7279551029205322)])
OrderedDict([('tweet', 'Dr. Fauci does not work for the White House. He works for the American people. The American people deserve to hear from Dr. Fauci. https://t.co/98DCo94jKq'), ('dates', '2020-05-01 23:39:58'), ('file_name', 'Mary Gay Scanlon.csv'), ('pct_true', 0.5004886984825134)])
OrderedDict([('tweet', 'We have lost 50,000 lives to #COVID19 in first three months of this pandemic. We don’t need to lose more lives to dangerously bad advice. https://t.co/IrBgRnDVjk'), ('dates', '2020-04-25 00:34:35'), ('file_name', 'Mary Gay Scanlon.csv'), ('pct_true', 0.5492716431617737)])
OrderedDict([('tweet', 'The infuriating part of this is that GOP members of Congress are defying science and medical advice, and placing others at risk. https://t.co/wXSPu6HQqm'), ('dates', '2020-04-23 17:05:46'), ('file_name', 'Mary Gay Scanlon.csv'), ('pct_true', 0.7303171753883362)])
OrderedDict([('tweet', 'House Democrats have been working to lower drug prices, protect our clean water &amp; air, and end gun violence. We passed the #ForThePeople Act last year to rein in big money in politics so we can make progress these issues &amp; more. It’s way past time for a vote in the Senate.'), ('dates', '2020-03-08 22:13:00'), ('file_name', 'Matt Cartwright.csv'), ('pct_true', 0.5760261416435242)])
OrderedDict([('tweet', "Why did Trey Gowdy go on TV in May 2018 and say the FBI did what our fellow Americans would have wanted them to do? And that it had nothing to do with @realDonaldTrump?\n\nWhen we wanted subpoena power, it was Paul Ryan and Gowdy who wouldn't give it to us. https://t.co/EbS5O9wbWG https://t.co/sQDmo2ohkt"), ('dates', '2020-05-09 01:47:00'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.6796687841415405)])
OrderedDict([('tweet', "The Democrats lied to the American people.\n\nThere was no collusion. Obama's FBI set a trap for Michael Flynn to go after @realDonaldTrump.\n\nIt was all a setup. And the more we learn, the more we're proven right. https://t.co/WlwMhJedr7"), ('dates', '2020-05-08 23:09:38'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.8808543682098389)])
OrderedDict([('tweet', "It's great to see honor restored to General Flynn's good name.\n\nThe boomerang has come back on many House Democrats - @RepAdamSchiff and @SpeakerPelosi have egg on their face.\n\nAdam Schiff was caught in a lie and then used an impeachment over Ukraine to cover up that lie. https://t.co/14iO9yFOLn"), ('dates', '2020-05-07 22:08:22'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.6496008038520813)])
OrderedDict([('tweet', 'Instead of bailing out Chinese businesses in the United States, we should seize their assets and put them in receivership to pay damages to Americans who have suffered as a consequence of the Wuhan Virus. https://t.co/BEt3ArOrnZ'), ('dates', '2020-05-06 19:37:50'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.8647170662879944)])
OrderedDict([('tweet', "Why are we three years away from @realDonaldTrump's election still learning about the things our own corrupt government did to try and derail him?\n\nThis should have been out in the open way before now. We need to show the American people how deep the plot was to go after @POTUS. https://t.co/mCR2jbJ48g"), ('dates', '2020-05-06 16:19:56'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.8093451261520386)])
OrderedDict([('tweet', 'China stole our health. Now they threaten our national security. \n\nhttps://t.co/ATgPw7lPKm'), ('dates', '2020-05-06 13:43:42'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.6760134696960449)])
OrderedDict([('tweet', "China is massively expanding a Trojan horse spying operation in the US.\n\nYour local police may be unknowingly helping them.\n\nOur own Department of Homeland Security wrote that this is the type of information we're worried China might sell to terrorists.\n\nhttps://t.co/BdcBY05KqF"), ('dates', '2020-05-05 14:32:41'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7632757425308228)])
OrderedDict([('tweet', 'Since COVID-19, DJI is openly bragging that they have just donated these Trojan Horse gifts to states and municipalities.\n\nI believe the Attorney General should immediately ground the entire fleet of DJI drones used by US law enforcement. https://t.co/wg9n5LytIu'), ('dates', '2020-05-05 03:20:00'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.8060617446899414)])
OrderedDict([('tweet', 'BREAKING: China has infected our law enforcement agencies.\n\nChinese drones are being used by some US police to enforce social distancing...and likely sending information to China.\n\nChina stole our health. Now they threaten our national security. \n\nThis must end now. https://t.co/zTU4WmVNKQ'), ('dates', '2020-05-05 00:27:31'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.8380035161972046)])
OrderedDict([('tweet', "Why are we three years away from President @realDonaldTrump's 2016 election still learning about the things our own corrupt government did to try and derail him? \n\nThis should have been out in the open way before now. https://t.co/zeNtsQsIwj"), ('dates', '2020-05-04 15:18:04'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7182784080505371)])
OrderedDict([('tweet', '“They stopped people going into China, but they didn’t stop people going into the USA and all over the world...so what’s that all about?\n\nIn other words, they knew they had a problem. I think they were embarrassed by the problem.” -@realDonaldTrump\n\n#TrumpTownHall https://t.co/ZnlPTndz9K'), ('dates', '2020-05-04 00:50:20'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.9817768931388855)])
OrderedDict([('tweet', '“It got out. China should’ve stopped it.”\n\n“[China] didn’t want to have us see it. Maybe they were worried about competence, maybe they were worried about something else.” -@realDonaldTrump \n\n#COVID19 #TrumpTownHall https://t.co/awdSOQQT5Y'), ('dates', '2020-05-04 00:03:42'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.9568573832511902)])
OrderedDict([('tweet', 'The Mueller investigation was a joke.\n\nThere was no collusion. There was no basis to put our country through this investigation.\n\nHow small do these Democrats, prosecutors, and @Comey look now, given all they did to try to railroad President @realDonaldTrump without facts? https://t.co/2Quy6GwHol'), ('dates', '2020-05-03 01:36:32'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7293295860290527)])
OrderedDict([('tweet', "In @SpeakerPelosi's vision for America, we wouldn't be visiting the prisoners. The prisoners would be visiting us.\n\nDemocrats want to move cell block C into our neighborhoods. \n\nInstead of releasing illegal aliens into our communities, we should deport them immediately. https://t.co/zs6b2JLqSu"), ('dates', '2020-04-30 20:02:16'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.8734483122825623)])
OrderedDict([('tweet', 'It is unconscionable that at a time when we should be working together as Americans, Democrats are working to socially engineer our country and advance their radical legislative priorities.\n\nThe PANDEMIC Act puts #AmericaFirst \n\nhttps://t.co/VvoYz8LCYO'), ('dates', '2020-04-30 15:53:40'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.8234483003616333)])
OrderedDict([('tweet', "As Republicans and @POTUS were working to deliver provisions to Americans, the Left was trying to gain leverage for their agenda.\n\nWe shouldn't just be horrified that they're using COVID-19 for leverage, we should be frightened by the radical agenda they're trying to advance. https://t.co/NaQ6a7m6GL"), ('dates', '2020-04-30 14:02:54'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7487291097640991)])
OrderedDict([('tweet', 'Democrats are taking advantage of a national pandemic by working to advance their radical agenda at the expense of Americans’ safety and well-being.\n\nThe PANDEMIC Act puts Americans first by requiring that all illegal aliens be deported during a pandemic.\n\nhttps://t.co/Zo1Vq0DMAE'), ('dates', '2020-04-29 20:17:46'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.817240834236145)])
OrderedDict([('tweet', "Instead of releasing illegal aliens, we should deport them immediately. That's why I filed the PANDEMIC Act.\n\nIt's what Mexico does- they're curbing illegal immigration by immediately deporting people during COVID-19.\n\nWe should love our country as much as Mexico loves theirs. https://t.co/UEEuf1aaLr"), ('dates', '2020-04-29 01:01:36'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.804190993309021)])
OrderedDict([('tweet', "TONIGHT: I'll be on @TuckerCarlson to unveil my response to the Democrats' radical, America Last immigration agenda they are trying to advance during the COVID-19 pandemic. TUNE IN to @FoxNews at 8:15pm ET/7:15pm CT"), ('dates', '2020-04-28 22:24:11'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.5870046615600586)])
OrderedDict([('tweet', "Americans deserve to know who's fighting for what. \n\nDemocrats are fighting for illegal aliens, upgrades to the Kennedy Center, and bailouts for blue states.\n\nRepublicans are trying to leverage the strength of this economy for our people, our workers, and our businesses. https://t.co/js5Jt8TuOL"), ('dates', '2020-04-28 14:35:16'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7578344345092773)])
OrderedDict([('tweet', 'Chinese Virus - Impairs Public Health\n\nChinese Drones - Impairs our Liberty \n\nI’m against both.\n\nWhy aren’t some local governments? https://t.co/3bDRRpcrnf'), ('dates', '2020-04-27 19:06:56'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.6233993172645569)])
OrderedDict([('tweet', "If this pandemic has taught us anything, it's a wake-up call about our relationship with China.\n\nIn defeating the Wuhan virus, we should not invite China to have a greater role in our country or potentially risk a public health crisis becoming a national security risk. https://t.co/YAxS3TYnAs"), ('dates', '2020-04-27 17:54:20'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7830368280410767)])
OrderedDict([('tweet', "Some municipalities in America have used Chinese drones with video surveillance to spy on people in their own backyards.\n\nWe're starting to see some real infringements on people's liberties, particularly at the local government level. https://t.co/HpihhPeSd0"), ('dates', '2020-04-27 14:34:25'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7737920880317688)])
OrderedDict([('tweet', "As Americans, we're not going to lose to this virus by fundamentally altering the rights and liberties that have built the most prosperous human society in all of history.\n\n@GovMikeHuckabee @HuckabeeOnTBN @TBN https://t.co/APl6zAXWP1"), ('dates', '2020-04-26 00:25:00'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.509297251701355)])
OrderedDict([('tweet', 'The Trump Doctrine is one that does not seek to engage us in yet another forever war.\n\nBut when Americans are in danger, President @realDonaldTrump has been very clear that he will respond. Iran better get their act together because President Trump means what says. https://t.co/EepRTu2IZv'), ('dates', '2020-04-24 18:18:47'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7166412472724915)])
OrderedDict([('tweet', 'American blood should not be used to avenge foreign oil or foreign vessels. But when Americans are in danger, President @realDonaldTrump has been very clear that he will respond. https://t.co/DTAv1WIeNw'), ('dates', '2020-04-23 19:20:31'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7422254681587219)])
OrderedDict([('tweet', "Under President @realDonaldTrump's doctrine, we're not going to engage in endless foreign wars, but we will take out terrorists.\n\nhttps://t.co/GjXkFbGIHw"), ('dates', '2020-04-23 14:27:55'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.6453361511230469)])
OrderedDict([('tweet', "Under President @realDonaldTrump's doctrine, we won't engage in endless foreign wars but we will take out terrorists.\n\nThe message is very clear to any Iranian official who would engage in terrorism:\n\nEither get your act together, or get prepared to meet General Soleimani. https://t.co/tNcWSaf5nv"), ('dates', '2020-04-23 01:54:59'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.6211893558502197)])
OrderedDict([('tweet', 'My governor, @GovRonDeSantis, has taken a lot of incoming hate from the national media, but Florida is using science, innovation, and data to fight back against COVID-19 instead of draconian measures that shut down the economy and zap the will of our fellow Americans. https://t.co/8IZTkd0Ned'), ('dates', '2020-04-21 20:12:21'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7733142375946045)])
OrderedDict([('tweet', 'We need more Democrats who are willing to work with President @realDonaldTrump and a lot less like the Democratic leadership who are just trying to score political points. https://t.co/sGUaldAa9F'), ('dates', '2020-04-21 14:28:13'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.676105797290802)])
OrderedDict([('tweet', "There is a dangerous globalist element to the establishment elite's thinking, but Americans can take pride in the fact that President @realDonaldTrump isn't going to overpay for bad science from people who shill for China.\n\nReal leaders ask tough questions and strike balances. https://t.co/mGuPJqm3e0"), ('dates', '2020-04-20 15:14:43'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.878204345703125)])
OrderedDict([('tweet', 'It is unacceptable that Democrats have chosen to use the COVID-19 pandemic as an opportunity to socially reengineer our country to conform to their agenda.\n\nWe should be coming together as a country and putting #AmericaFirst.\n\n(@BreitbartNews @SeanMoran3)\n\nhttps://t.co/70CTaz7wJ7'), ('dates', '2020-04-18 15:46:42'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.890497088432312)])
OrderedDict([('tweet', ".@DevinNunes has been an American patriot telling the truth on the deep state's corruption in the Russia witch hunt, and he's absolutely right:\n\nPeople should be charged with crimes for obstructing his investigation. #WitchHuntExposed https://t.co/TQmigLh5zI"), ('dates', '2020-04-16 19:04:19'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.6534475088119507)])
OrderedDict([('tweet', 'Russia tried to discredit @realDonaldTrump by using spies funded and directed by the Russian government to co-mingle with the DNC\'s dossier.\n\nBut Mueller still claimed the Steele Dossier was "not his purview." \n\nUnbelievable. https://t.co/0sDVSZSlw7'), ('dates', '2020-04-16 15:14:33'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.6522804498672485)])
OrderedDict([('tweet', 'At best, Americans are funding people who are lying to us.\n\nAt worst, we’re funding people who we knew had problems handling pathogens, who then birthed a monster virus onto the world. https://t.co/0ASWpe6vzQ'), ('dates', '2020-04-15 00:44:18'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.7976120710372925)])
OrderedDict([('tweet', 'If Democrats cared about carefully handling taxpayer money, they wouldn’t have spent $350 million of it on programs to resettle refugees worldwide, at a time when half the world’s population is on lockdown.\n\nhttps://t.co/QXzyWfKwAI'), ('dates', '2020-04-14 18:02:10'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.8289279341697693)])
OrderedDict([('tweet', "What's the next move for the progressive Left?\n\nWhile Bernie wasn't able to drag our country into a socialist revolution, he's still actively trying to revolutionize the Democratic party. https://t.co/GKXmf0foSo"), ('dates', '2020-04-13 16:43:58'), ('file_name', 'Matt Gaetz.csv'), ('pct_true', 0.5216615796089172)])
OrderedDict([('tweet', 'When I put on uniform, we didn’t fight for blue or red states—we defended the United States. Governors from both parties are asking to be reimbursed for fighting the pandemic. That’s not a bailout—that’s what the federal government is there for.\nhttps://t.co/JrMgPQ0RfW'), ('dates', '2020-05-05 17:55:42'), ('file_name', 'Max Rose.csv'), ('pct_true', 0.5917003154754639)])
OrderedDict([('tweet', "UPDATE: After some back-and-forth and unnecessary confusion, the Treasury Dept. has confirmed that seniors on Social Security who don't file taxes will still be eligible for COVID relief checks--just as the CARES Act intended.\n\nREAD MORE: https://t.co/Rk8RBwS3Kx\n\n-Team Max"), ('dates', '2020-04-02 19:21:57'), ('file_name', 'Max Rose.csv'), ('pct_true', 0.5219479203224182)])
OrderedDict([('tweet', 'We can’t be reactive, we have to be proactive. That is why @HeyNowJO and I worked to secure these tents to ensure RUMC is prepared for any surge in patients.\nhttps://t.co/b4AOew5OWx'), ('dates', '2020-03-17 18:48:29'), ('file_name', 'Max Rose.csv'), ('pct_true', 0.5542235374450684)])
OrderedDict([('tweet', 'If pushing #Hydroxychloroquine wasn’t enough, Trump is now suggesting disinfectant &amp; ultraviolet light as a COVID19 treatment! Trump is endangering the lives of people who believe &amp; follow his advice! He needs to shut up &amp; be denied TV time w/ his unhinged cures &amp; remedies!'), ('dates', '2020-04-25 00:55:05'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8668097853660583)])
OrderedDict([('tweet', 'Gov Kemp followed the advice of Trump &amp; his protestors &amp; “liberated” his state &amp; opened up the businesses that have the most personal contact w/ individuals in a state w escalating COVID19 sickness &amp; death. His actions will cause more infections. Stupid Kemp should know better!'), ('dates', '2020-04-25 00:53:48'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7131888270378113)])
OrderedDict([('tweet', 'GSA better not renegotiate Trump’s lease on his DC hotel. That property belongs to the fed govt! He should never have gotten that lease. What about every other fed bldg leased by other businesses? Will GSA reduce rent on those leases? GSA better stand up to this corrupt bully!'), ('dates', '2020-04-25 00:46:11'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7009460926055908)])
OrderedDict([('tweet', 'Trump, when are you going to say a word of sympathy &amp; condolences to the families &amp; friends of the 50k coronavirus victims who’ve died in our country &amp; even the others who’ve died around the world? Have you no empathy? Have you no compassion or decency?'), ('dates', '2020-04-25 00:39:52'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5816401243209839)])
OrderedDict([('tweet', 'Trump, stop politicking! Most of the mayors &amp; governors agree they are desperate for testing. Do not believe this country can open up businesses w/o testing, testing, testing! The real test numbers must be revealed!'), ('dates', '2020-04-19 19:01:53'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5378425121307373)])
OrderedDict([('tweet', 'People beware! Trump is slow walking America to dictatorship. Press attacks, owns AG Barr, controlling our courts including Supreme Court, firing inspector generals, complimenting Putin &amp; Kim Jong Un, now trying to defund US Postal Service, &amp; opposing vote by mail!'), ('dates', '2020-04-15 02:59:53'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8590927124023438)])
OrderedDict([('tweet', 'The statistics &amp; data are emerging about #COVID19 racial disparities, including infections &amp; death. This Admin had better not play around w/a lack of testing, lack of sufficient PPE at hospitals in these communities, &amp; lack of support for the homeless &amp; healthcare resources.'), ('dates', '2020-04-15 02:50:51'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5010315179824829)])
OrderedDict([('tweet', 'Why did Trump decide to defund drive-thru testing in the States, disadvantaging minority communities? Trump HHS reason was unless the states need it. What kind of question is that when testing is lacking all over w/ blacks &amp; minorities dying in huge numbers? We’re fighting back!'), ('dates', '2020-04-15 02:48:48'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8896492719650269)])
OrderedDict([('tweet', 'Would Trump dare use #coronavirus as an opportunity to reward big GOP donors &amp; place people in our country in a vulnerable position b/c the PPE hasn’t been available to the states? Very suspicious! Are these new GOP companies getting PPE from what Kushner called “our stockpile”?'), ('dates', '2020-04-06 21:13:19'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.9534600377082825)])
OrderedDict([('tweet', 'Is Trump using the federal government to block the states from being able to purchase PPE or forcing them to sale to the federal stockpile? Many states can’t get equip b/c Trump is outbidding them — but then he’s not sending the supplies to all states...WHY?'), ('dates', '2020-04-06 21:12:17'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.9502136707305908)])
OrderedDict([('tweet', 'What the hell, Jared Kushner! What do you mean our stockpile vs state stockpile of health equipment? This is the UNITED STATES OF AMERICA! This is a CRISIS! U &amp; your inept father in law have bungled the response. You better get the PPE, respirators, ventilators to the states NOW!'), ('dates', '2020-04-06 00:16:40'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.6565057039260864)])
OrderedDict([('tweet', 'Do not listen to 45 when he suggests untested #Hydroxychloroquine to treat #COVID19 - listen to Fauci. Don’t listen to 45 when he says heat will kill the virus; or he won’t wear a mask; or when he calls hospital workers thieves, or refuses natl lockdown! 45 just can’t be trusted!'), ('dates', '2020-04-06 00:09:25'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5682176351547241)])
OrderedDict([('tweet', "Trump, you incompetent idiot! You sent 18 tons of PPE to China early but ignored warnings &amp; called COVID19 concerns a hoax. You've endangered doctors, nurses, aids, orderlies, &amp; janitors - all risking their lives to save ours. Pray 4 forgiveness for the harm that you're causing!"), ('dates', '2020-03-30 21:05:36'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.6038702130317688)])
OrderedDict([('tweet', "Trump, stop congratulating yourself! You’re a failure &amp; you've mishandled this #COVID19 disaster! You're not knowledgeable &amp; you don’t know more than experts &amp; generals. Your ignorance &amp; incompetence are appalling &amp; you continue to demonstrate that every time you open your mouth!"), ('dates', '2020-03-30 21:01:54'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.6344188451766968)])
OrderedDict([('tweet', "The Senate bailout bill must protect employees! There should be no firings, no layoffs, no cut in pay, no cut hrs, or diminishing health benefits/pensions. The bailout must not increase pay for CEOs, give bonuses, or allow for stock buybacks! America's workers MUST come first."), ('dates', '2020-03-22 21:22:08'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5942007899284363)])
OrderedDict([('tweet', 'Trump is duplicitous &amp; causes confusion every time he opens up his lying mouth! He claimed to invoke the Defense Prod. Act, but later said he was waiting for the "worst case scenario." How many more people have to get sick or DIE until Trump exercises the full power of the DPA?'), ('dates', '2020-03-22 21:20:33'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.9011178612709045)])
OrderedDict([('tweet', 'The way Dr. Fauci bowed his head in shame &amp; disgust listening to Trump spout his Fox News talking points is how all of us feel. People in this country are suffering enough. Why would media subject the American public to Trump’s daily LIES? Fauci should lead all pressers. Period!'), ('dates', '2020-03-22 21:19:32'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8969767093658447)])
OrderedDict([('tweet', 'Shout out to @GavinNewsom &amp; @NYGovCuomo! This is what real leadership looks like. They are kicking ass &amp; doing so w/ empathy &amp; respect, taking care of our citizens while Trump is trying to make us believe he changed his mind about calling the #Coronavirus a hoax!'), ('dates', '2020-03-22 21:19:04'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8088183999061584)])
OrderedDict([('tweet', 'Trump’s swipe @Yamiche is yet another attack on a woman raising serious ?s. Trump can’t stand being challenged by women but he’s especially disrespectful towards women of color. @PBS Yamiche’s question was legit. Why did Trump shut down pandemic office? Answer: Plain ignorance!'), ('dates', '2020-03-16 00:06:10'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8991897702217102)])
OrderedDict([('tweet', "Rep. Nunes' latest rant, urging people to go out like there's no pandemic, should be ignored! He is one of Trump's biggest sycophants in Congress! Listen to health experts, @CDCgov, &amp; your local officials. Take #coronavirus seriously &amp; practice social distancing."), ('dates', '2020-03-16 00:00:30'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8026143312454224)])
OrderedDict([('tweet', 'I don’t trust Trump or his sycophants! Trump had been partying it up at Mar-a-Lago &amp; at all these places w/ people who now have #coronavirus! He should follow @cdc guidance and self quarantine for 14 days, just like the public is advised to do.'), ('dates', '2020-03-15 23:55:19'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5540573000907898)])
OrderedDict([('tweet', 'More than 2000 former DOJ officials courageously organized letter explaining why Barr’s interference in Roger Stone’s sentencing is dangerous &amp; a threat to our democracy. Trump doesn’t seem to care that it’s rumored he has mafia associations, but he sure acts like a mafia boss!'), ('dates', '2020-02-18 22:33:35'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5492136478424072)])
OrderedDict([('tweet', 'Trump using AG Barr to get his criminal friends out of jail is the tip of the iceberg. I’ll say it again: Trump will pardon Flynn, Manafort, &amp; Stone b/c the GOP cowards in the Sen bowed to his feet. Repubs now stand for crime, corruption, autocracy, &amp; Russia! They are PATHETIC!'), ('dates', '2020-02-18 22:28:48'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7044174075126648)])
OrderedDict([('tweet', 'It’s not enough for Barr to resign. I want him out of DOJ, disbarred, disgraced, &amp; investigated for obstruction of justice. Barr sabotaged Mueller report, was in on Ukraine plot, &amp; now he wants to free Trump’s criminal friends. In my estimation, they ALL belong in jail!'), ('dates', '2020-02-18 22:25:13'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7698412537574768)])
OrderedDict([('tweet', 'Barr is Trump’s puppet, acting as his personal lawyer rather than the people’s atty. He’s undermining our democracy by interfering in the sentencing of Roger Stone &amp; seeking a review of Flynn’s case. Barr claims this is ordinary procedure - he’s an ordinary LIE! He’s got to go!'), ('dates', '2020-02-17 02:23:27'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.910006582736969)])
OrderedDict([('tweet', 'The 4 pro career DOJ attys who investigated Roger Stone recommended 7-9 yrs incarceration b/c Stone is guilty of witness tampering &amp; lying. He is well known as a dirty trickster &amp; political terrorist. This is typical of Trump’s friends. Don’t be surprised if Trump pardons him!'), ('dates', '2020-02-17 02:17:14'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7042545080184937)])
OrderedDict([('tweet', 'Many of this crooked President’s supporters are among the most vulnerable beneficiaries of Medicare &amp; Medicaid. Yet, Trump’s budget proposes nearly $1 tr in cuts to ea. of these prgrms to pay tax cuts for the rich. Don’t be tricked. Trump will cut these prgrms if he’s reelected!'), ('dates', '2020-02-17 02:14:10'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7636211514472961)])
OrderedDict([('tweet', 'America needs to wake up. GOP is willing to lie, cheat, &amp; steal for power. They have no shame &amp; no guts. They can’t bring themselves to remove a lying criminal con man, impeached president, in an election year, but they can STEAL a Supreme Court seat in an election year! B.S.!'), ('dates', '2020-02-01 16:43:41'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.9843534827232361)])
OrderedDict([('tweet', 'I’ve never seen a more insufferable &amp; pathetic group of Senators than those in the GOP. They’ve chosen to strap themselves to a corrupt con man they KNOW is guilty. They’re all cowards. None of them belong in the Senate. They should go home and grow a backbone!'), ('dates', '2020-02-01 16:42:14'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7275002002716064)])
OrderedDict([('tweet', 'Recent attacks on the Jewish community are proof that anti-Semitism is on the rise once again. We must never forget the victims of the Holocaust and continue to confront hate and bigotry. #HolocaustMemorialDay'), ('dates', '2020-01-28 01:23:41'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8381512761116028)])
OrderedDict([('tweet', 'What a sad time in American history when Trump &amp; #MoscowMitch can collude to fast-track a SHAM trial when the facts are so clear Trump abused his power. Trump has no defense &amp; never disputed our facts. No wonder polls show most Americans think Trump should be removed from office.'), ('dates', '2020-01-21 18:42:35'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8677527904510498)])
OrderedDict([('tweet', 'The demonstrations around the country today must not be ignored. Americans do not support the Impeached President’s needless escalation w/ Iran, nor do we want a senseless war. Peace advocates must not let up! Let’s keep showing our power &amp; sending Trump a message: #NoWarWithIran'), ('dates', '2020-01-10 00:31:01'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7662033438682556)])
OrderedDict([('tweet', 'Many are outraged that the impeached prez nearly caused a war w/ Iran &amp; want Congress to intervene.\n\nHouse Dems just passed a bill that limits any further military force against Iran by unhinged Trump w/out approval.\n\nAmerica needs a new president, not another war. #NoWarWithIran'), ('dates', '2020-01-10 00:25:48'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.9385764598846436)])
OrderedDict([('tweet', 'No matter how much #MoscowMitch wants to dismiss articles or acquit Trump, so much info is revealed every day showing how the impeached president tried to get WH officials to break the law in his plot to extort Ukraine in exchange for an attack on Biden. Trump is guilty as hell.'), ('dates', '2020-01-07 01:20:25'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.6377153992652893)])
OrderedDict([('tweet', 'Everyone knows the impeached President can’t be trusted. Is he lying about imminent danger? Did Trump make the decision on his own because he thinks he knows more than his generals? Why should we believe him when he, Pompeo, &amp; Pence can’t keep their lies straight?'), ('dates', '2020-01-06 19:29:03'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.9676457643508911)])
OrderedDict([('tweet', 'Only a reckless, cowardly, impeached international laughing stock would order a deadly drone strike from a golf resort  before running off to a rally to be worshipped by the few evangelicals who still support him. A real President would’ve immediately shared info w/ Congress.'), ('dates', '2020-01-04 20:55:36'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.6883426904678345)])
OrderedDict([('tweet', 'WAPO documented 15000+ lies told by Trump since he took office. How can the American people trust him when he lies all the time? He lied about Obama’s birth certificate and tweeted in 2011 that Obama would “start a war with Iran” to get reelected. Is that what Trump is doing now?'), ('dates', '2020-01-04 20:54:52'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.9674450755119324)])
OrderedDict([('tweet', 'This prez walked away from the Iran Nuclear Deal for peace, threatened the "end of Iran," killed top Iranian general w/out AUMF from Congress &amp; abandoned our Kurdish allies fighting ISIS. Americans deserve better answers rather than dangerous disruption in the entire Middle East!'), ('dates', '2020-01-03 17:17:52'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.6168853044509888)])
OrderedDict([('tweet', "All this talk about Pelosi's timing is futile. We've done our job. #ConManDon has been impeached. When will #MoscowMitch stop conspiring w/ Trump &amp; work w/ Schumer on bipartisan rules &amp; witnesses so that the Speaker can select managers to present our strong case in the Senate?"), ('dates', '2019-12-23 02:16:18'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.9580869674682617)])
OrderedDict([('tweet', "Revelation by former WH officials proves what we've known all along: Trump is #PutinsPuppet. Trump repeated Putin's talking point that Ukraine, not Russia, interfered in the ‘16 election. Mark my words. If the Senate doesn't remove him, Trump will invite Putin to the WH next yr."), ('dates', '2019-12-21 02:52:54'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.6707412600517273)])
OrderedDict([('tweet', 'Finally the day has come where #impeachment will be voted on in the House of Representatives. If possible, take time to listen to the debate. \n\nDemocrats have done our work and laid out the facts: Trump abused his power &amp; obstructed Congress. We’re on the correct side of history.'), ('dates', '2019-12-18 15:53:48'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5939427018165588)])
OrderedDict([('tweet', 'Shout out to millennials, GenX, GenZ for staying w/ me as I fought for Trump’s impeachment. We stand here w/ Speaker Pelosi &amp; the “Big 6” announcing the articles of impeachment. Stay hopeful that GOP Senators will live up to their so-called patriotism. No one is above the law! https://t.co/CUCROTIKUE'), ('dates', '2019-12-12 16:25:42'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5193714499473572)])
OrderedDict([('tweet', 'Lindsey, how can you say you love Biden? You don’t have a clue what love means. If you loved him, you wouldn’t be lying &amp; using all this energy trying to harm him &amp; protect the lying Prez who you really love b/c you want him to save your Senate seat. You don’t know what love is!'), ('dates', '2019-11-26 14:18:21'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7458363771438599)])
OrderedDict([('tweet', 'Rick Perry (TX) has lost his mind! He said Trump was “chosen” by God. Is that the same God who told him to go to Ukraine &amp; give Zelensky a list of people to “trust” which led to 2 of his supporters getting an oil deal from Ukraine, despite being outbid? Subpoena Rick Perry too!'), ('dates', '2019-11-26 14:14:49'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.6800405383110046)])
OrderedDict([('tweet', 'Never in my wildest dreams did I think the GOP would become the spokespersons for Russian propaganda. Dr. Fiona Hill (fmr. WH advisor to Trump) chastised Repubs &amp; used her expertise to confirm they’re simply being Putin’s stooges. Drain the swamp! Impeachment is the only answer!'), ('dates', '2019-11-25 17:01:24'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5263867974281311)])
OrderedDict([('tweet', 'When you have all US intel agencies  confirming Russia led by Putin hacked the DNC &amp; interfered in US election &amp; Russian apologists like Sen. Kennedy trying to blame Ukraine, you know our democracy is now being dangerously undermined by the GOP - not just Russia. We must impeach!'), ('dates', '2019-11-25 17:00:38'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.6115959882736206)])
OrderedDict([('tweet', 'The Republicans have no credible defense for Trump’s attempted bribery of Zelensky in Ukraine. They are failing in their WEAK and LYING efforts to support Trump because of their desperation and fear of him and bedamn our democracy!Impeachment is the only answer.'), ('dates', '2019-11-25 16:59:51'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.9928557872772217)])
OrderedDict([('tweet', 'Trump brags about not settling &amp; winning lawsuits. Lied again! A judge just ordered him to pay $2 mil in damages (to REAL nonprofits) for using Trump Foundation to pay debts, aid his campaign &amp; buy a $10K self portrait! He stole the $$$ and got caught! One of many crimes!'), ('dates', '2019-11-09 00:45:05'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7159301042556763)])
OrderedDict([('tweet', 'Can you believe racist Jefferson Beauregard Sessions is running to come back to the Senate? Sessions has no respect for himself in the way he\'s singing Trump\'s praises who embarrassed him, called him a "dumb southerner", &amp; threw him under the bus. What a mindless wonder!'), ('dates', '2019-11-08 22:10:38'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8674625754356384)])
OrderedDict([('tweet', "Sondland lied in his early testimony defending Trump and now that he sees the facts are unfolding about this crooked president, he has changed his lies &amp; confirmed a quid pro quo. He's only telling the truth now to save his hide! You can run but you can't hide."), ('dates', '2019-11-08 21:57:20'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.8594578504562378)])
OrderedDict([('tweet', 'GOP scrambling &amp; can\'t get their lies straight. First they say no quid pro quo when Trump w/held mil. aid from Ukraine for dirt on Biden. Then Mulvaney says this happens all the time &amp; "get over it." Lies getting entangled. Now we know Trump atty Giuliani is the real SecOfState.'), ('dates', '2019-11-08 21:57:02'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.885596752166748)])
OrderedDict([('tweet', "Misuse! Manipulation! Discrimination! Criminal Activity! Just a few of Facebook’s egregious offenses &amp; now they have the nerve to launch a #BigTechTakeover in the form of a currency that could rival the U.S. Dollar?! Is it 10 o'clock yet? Mr. Zuckerberg &amp; I have a lot to discuss."), ('dates', '2019-10-23 14:12:17'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5154448747634888)])
OrderedDict([('tweet', 'No one is above the law including Mark Zuckerberg. Facebook has allowed election interference, released private data, violated civil rights laws, among other offenses &amp; now they’re trying to launch a #BigTechTakover by creating this mysterious #ZuckBuck? Not on my watch! @FSCDems'), ('dates', '2019-10-23 14:11:24'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7460771799087524)])
OrderedDict([('tweet', "Can you believe it? This brazen president is inviting the 7 most powerful countries in the world to hold its conference at his FAILING hotel in FL, the Trump Doral, disregarding the emoluments clause that's already the subject of two lawsuits?"), ('dates', '2019-10-18 18:40:33'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7892779111862183)])
OrderedDict([('tweet', "The more we learn, the more flabbergasted we are. Can you believe Pompeo was on the call w/ Trump when he was shaking down Ukraine's pres &amp; attempting to find dirt on Biden? Was Pompeo also involved in hiding transcripts on a classified server? Impeachment will uncover the truth."), ('dates', '2019-10-01 19:13:53'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.6253108978271484)])
OrderedDict([('tweet', "It’s certainly past time for Giuliani's docs/texts to have been subpoenaed by our committees. His brazen lies, outrageous distortions, &amp; attempts to do anything Trump wants him to do have got to be stopped. His unintended revelations will help us solidify Trump's impeachment."), ('dates', '2019-10-01 17:50:58'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5971564054489136)])
OrderedDict([('tweet', "AG Barr is the highest law enforcement officer &amp; the worst. He's supposed to be the people’s lawyer, but he’s just Trump’s puppet. He's been caught soliciting foreign countries' help in undermining his FBI &amp; our Intel agencies. He should be Trump’s companion w/impeachment."), ('dates', '2019-10-01 17:24:47'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7881270051002502)])
OrderedDict([('tweet', "I'm calling on the GOP to stop Trump's filthy talk of whistleblowers being spies &amp; using mob language implying they should be killed. Impeachment is not good enough for Trump. He needs to be imprisoned &amp; placed in solitary confinement. But for now, impeachment is the imperative."), ('dates', '2019-10-01 17:19:28'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.9211834669113159)])
OrderedDict([('tweet', "Trump is so irresponsible &amp; so hungry for power &amp; control, he would dare imply that a civil war will ensue if he's impeached. He is dog whistling to his white supremacists to create fear &amp; intimidation b/c he knows he is going to be impeached. He knows he deserves impeachment."), ('dates', '2019-10-01 17:17:04'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.638545572757721)])
OrderedDict([('tweet', "Trump &amp; his sycophants tried to cover up the transcripts re: his phone call to Ukraine's president by hiding the info on a server reserved for highly classified info. It didn't work. They've been caught red handed. Not just thinking about impeachment, impeachment is on the way."), ('dates', '2019-09-27 16:00:50'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.7210907340049744)])
OrderedDict([('tweet', 'Do you know Trump called the whistleblower, and others who disclosed his impeachable activities, spies and insinuated they should be killed? Trump is a crook and a hoodlum!'), ('dates', '2019-09-27 15:55:47'), ('file_name', 'Maxine Waters.csv'), ('pct_true', 0.5799641013145447)])
OrderedDict([('tweet', 'Bill Barr doesn’t care if we see his corruption right out in the open because in his America, the crime isn’t lying to the FBI, it’s telling the truth about @realDonaldTrump.'), ('dates', '2020-05-08 00:46:42'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.9186112284660339)])
OrderedDict([('tweet', 'The Chinese Exclusion Act.\nThe Japanese Incarceration Camps.\nAnd now @realDonaldTrump’s “Chinese virus”.\n \nThis country’s long history of anti-Asian racism just got its newest addition. Enough is enough.\nhttps://t.co/o0jdoksTID'), ('dates', '2020-04-24 22:25:49'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.5770621299743652)])
OrderedDict([('tweet', 'There’s no lack of competition for who’s the biggest suck up in @realDonaldTrump’s cabinet, but Bill Barr wins hands down. Even with thousands of lives on the line, Barr’s priority is to help Trump politically. We need an Attorney General not a Trump General. Resign already.'), ('dates', '2020-04-22 19:04:19'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.817283034324646)])
OrderedDict([('tweet', 'We have a president that takes “no responsibility” for his failures on #COVID19. Now he’s trying to distract us from his incompetence with yet another attack on immigrants.\n\nWe don\'t need another "executive order.” We need executive leadership @realDonaldTrump refuses to provide.'), ('dates', '2020-04-21 19:05:41'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.7750491499900818)])
OrderedDict([('tweet', "I know seniors who get their Social Security through USPS.\nI know people who get their medications through USPS.\n\n@realDonaldTrump is so afraid of people mailing in the Census or ballots, he's willing to let an essential institution close their doors. We must #SaveThePostOffice. https://t.co/bpXoYJO995"), ('dates', '2020-04-19 19:54:40'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.5646520256996155)])
OrderedDict([('tweet', 'It’s been more than a year since @DHSGov said they were monitoring the conditions in Venezuela while 200,000 Venezuelans in America waited. Forcing them back to a human rights crisis amid a pandemic could be catastrophic. \n \nIt’s time for the Senate to extend #VenezuelaTPSNow.'), ('dates', '2020-04-17 14:57:50'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6207777261734009)])
OrderedDict([('tweet', 'I opposed Andrew Wheeler as EPA Administrator because he cared more about the fossil fuel industry than public health.\n\nToday the EPA issued an outrageous rule to weaken limits on mercury emissions from power plants, harming child development and making the air dirtier. https://t.co/9C0PQQ0Jcn'), ('dates', '2020-04-16 18:36:21'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.575870156288147)])
OrderedDict([('tweet', ".@realDonaldTrump shook-down a foreign leader, stonewalled Congress and bullied whistleblowers. In the midst of a pandemic, he's abusing his power to take revenge on the IG who did his job to inform Congress of whistleblower's complaint.\n\nWe need a President not a petty dictator. https://t.co/vgju9omrE4"), ('dates', '2020-04-06 16:16:54'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8840583562850952)])
OrderedDict([('tweet', 'With the nominations of Justin Walker and Cory Wilson, @realDonaldtrump and @SenMajLdr are sending a clear message on what their priorities are.\n \nThey’re pushing anti-health care nominees at a time Americans are facing a public health crisis unlike anything we’ve ever seen. https://t.co/J7Zb4khV6M'), ('dates', '2020-04-04 00:11:15'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.5519796013832092)])
OrderedDict([('tweet', 'The racist rhetoric from the highest level of our government has contributed to a disturbing rise in hate-crimes targeting Asian Americans.\n\nCalling the #COVID19Pandemic the ‘Wuhan virus’, ‘Kung flu’, or ‘Chinese virus’ isn’t clever or funny, it’s wrong and deeply harmful. https://t.co/IZCxUocBc2'), ('dates', '2020-04-02 19:33:54'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.7560623288154602)])
OrderedDict([('tweet', 'Under the cover of coronavirus, @realDonaldTrump is rolling back fuel economy standards, against the advice of scientists and even the auto industry.\n \nWe’re going through an economic crisis and yet this move would cost drivers at the pump to benefit the oil industry. https://t.co/kF68ASmmjD'), ('dates', '2020-03-31 21:32:42'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.532844603061676)])
OrderedDict([('tweet', 'During a public health emergency when it’s clearer than ever that we all need health care, @realDonaldTrump is racing to put Cory Wilson – an extremely partisan, anti-Affordable Care Act nominee – on the federal circuit court where Trump is fighting to undermine the ACA. https://t.co/MfxJFIlswI'), ('dates', '2020-03-31 17:22:24'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.5233595371246338)])
OrderedDict([('tweet', 'It is shameful and unbelievably selfish for Republicans to hold up this bill that’s going to help working people in our country.\n\nIt’s amazing that some are saying we’re giving too much for these people who are already getting so little. https://t.co/EV55tmiBcx'), ('dates', '2020-03-25 19:58:05'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8309410810470581)])
OrderedDict([('tweet', 'Mitch McConnell’s bill demonstrates that Senate Republicans are not taking this pandemic seriously. It’s past time for them to get serious about passing a bill that does much more for American workers, families, and the most vulnerable.'), ('dates', '2020-03-22 23:31:24'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8932763338088989)])
OrderedDict([('tweet', 'It’s already been a year since the House passed the Dream and Promise Act. The Senate must take action to protect #DACA, #TPS, and #DED recipients. \n\nIn the meantime, DACA recipients should renew their DACA because their #HomeIsHere.'), ('dates', '2020-03-12 18:56:23'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8063961267471313)])
OrderedDict([('tweet', 'The coronavirus is already in the United States. Mr. President, where are the test kits? Still no straight talk from @realDonaldTrump. Auwe!'), ('dates', '2020-03-12 02:39:06'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6139281392097473)])
OrderedDict([('tweet', "The fake moral outrage from the right wing won't distract us from the fact that Chief Justice Roberts and his conservative majority on the Supreme Court are poised to dismantle a woman's Constitutional right to an abortion under Roe V. Wade. https://t.co/jiUNaPHZJq"), ('dates', '2020-03-04 23:32:00'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8586253523826599)])
OrderedDict([('tweet', 'According to this court, there’s no Congressional oversight of the executive branch by subpoena, giving license to this lawless president to continue covering up his criminal conduct. https://t.co/YQsCdBuNRq'), ('dates', '2020-02-29 00:12:03'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6502664685249329)])
OrderedDict([('tweet', 'It’s been a full year since the House passed #HR8 requiring background checks on all gun sales.\n \nWhile the bill sits on Mitch McConnell’s desk, over 38000 Americans have died from gun violence. It’s time for him to do his job and bring it to a vote in the Senate. #EndGunViolence'), ('dates', '2020-02-27 20:11:29'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8445631265640259)])
OrderedDict([('tweet', "The Supreme Court's latest decision effectively immunizes a border patrol agent who shot an unarmed teenager in the face, killing him. \n\nThis decision will only further embolden the Border Patrol to act with impunity. DHS must be subject to accountability and oversight. https://t.co/E3J3QcHqv0"), ('dates', '2020-02-27 15:42:08'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.5416862368583679)])
OrderedDict([('tweet', "It's not enough for @realDonaldTrump to purge anyone in his administration who is not totally loyal to him. He's now going so far as to further control the Supreme Court with his baseless calls for Justices Ginsburg and Sotomayor to recuse themselves. https://t.co/qAMz5QkuBR"), ('dates', '2020-02-26 14:28:14'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.749967098236084)])
OrderedDict([('tweet', '.@realDonaldTrump is engaging in wishful thinking if he thinks the coronavirus has been contained. The President should care about the health of his own citizens, but instead he’s more concerned about the stock market and any negative effect this will have on his re-election. https://t.co/KfgZOKVhll'), ('dates', '2020-02-25 23:59:03'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.712905764579773)])
OrderedDict([('tweet', 'Our nation should never repeat the internment of Japanese Americans during WWII. 78 years later, @realDonaldTrump has ignored our history and is abusing his power to target people based on ethnicity.\n \nOn this #DayOfRemembrance, we must stand together and fight for all people.'), ('dates', '2020-02-19 22:17:00'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.5800021886825562)])
OrderedDict([('tweet', "Since I called for AG Barr to resign in 2019, he's shown no sign of independence as he continues to act as the attorney for @realDonaldTrump, not the people.\n\nWith this administration it seems the rule of law, our independent judiciary, and checks and balances are out the window. https://t.co/xVlTFE6PF5"), ('dates', '2020-02-18 20:59:49'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6282821893692017)])
OrderedDict([('tweet', 'Immigration judges should be allowed to be fair and independent, not instruments of @realDonaldTrump’s cruel, anti-immigrant campaign. \n \nThe Trump administration’s assault on immigration courts undermines the ability of asylum seekers and immigrants to get a fair hearing. https://t.co/AeI1VSLvCh'), ('dates', '2020-02-18 15:39:17'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.5229742527008057)])
OrderedDict([('tweet', 'Decades after the Vietnam War ended, our veterans are still suffering from exposure to Agent Orange. @realDonaldTrump claims he cares, but is delaying these critical benefits - it’s time to stop stonewalling. https://t.co/QtA2QnALc3'), ('dates', '2020-02-14 18:56:02'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6913366913795471)])
OrderedDict([('tweet', 'The President and his handmaiden Bill Barr are politicizing the Justice Department to go after his enemies.\n\nTo watch my Republican colleagues twist themselves into a pretzel to lend cover to @realDonaldTrump is showing the American people how bankrupt the Republican party is. https://t.co/SDOqfgqMVg'), ('dates', '2020-02-13 15:02:59'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.7336665391921997)])
OrderedDict([('tweet', ".@realDonaldTrump’s reckless, impulsive actions are bringing us perilously close to war with Iran.\n\nIt's time for Congress to reassert its Constitutional authority by demanding the President seek explicit authorization prior to any military action against Iran. https://t.co/G5cKrz0I6i"), ('dates', '2020-02-13 01:07:44'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.7814542055130005)])
OrderedDict([('tweet', 'If infanticide is already illegal in our country, why did the Senate Judiciary Committee meet today? The answer is simple.\n\nThis is the latest in a long effort by Republicans to take control of women’s bodies and prevent us from exercising our constitutional right to an abortion. https://t.co/kicLLQq6Xy'), ('dates', '2020-02-11 19:36:11'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6932412981987)])
OrderedDict([('tweet', 'Are you a woman, a person who needs health care, a senior on Medicare, a new mom receiving SNAP benefits, an American living with a disability, or a student struggling to afford college? Chances are, @realDonaldTrump’s budget would screw you over. Join me in fighting back.'), ('dates', '2020-02-11 01:02:04'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.5474961400032043)])
OrderedDict([('tweet', 'Moments after the impeachment trial, Senator McConnell raced to put anti-voting rights judicial nominee Andrew Brasher in a lifetime seat for the circuit court in Alabama. Brasher has defended discriminatory gerrymandering and fought to gut the Voting Rights Act. #StopBrasher'), ('dates', '2020-02-07 21:59:28'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8581488132476807)])
OrderedDict([('tweet', '.@realDonaldTrump is attacking our nation’s monuments which are culturally and environmentally sensitive lands, for what? To pad his ego and the pockets of the fossil fuel industry. https://t.co/LIj1BK7UxO'), ('dates', '2020-02-07 18:38:21'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8182550072669983)])
OrderedDict([('tweet', 'I will vote to convict and remove President Donald Trump for abusing his power and obstructing Congress.\n\nIt’s time for the Senate to uphold its Constitutional responsibility by convicting this president and holding him accountable. https://t.co/7kgoQH7kmV'), ('dates', '2020-02-05 19:32:16'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.7052416205406189)])
OrderedDict([('tweet', 'The current #StateofHealthCare is under attack. @realDonaldTrump has used all three branches of government to sabotage our health care system and threaten coverage for vulnerable Americans.\n\nHealth care is a right, not a privilege for the wealthy.'), ('dates', '2020-02-05 01:31:09'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.842349112033844)])
OrderedDict([('tweet', 'Republicans are arguing as if we can’t do our jobs if it’s an election year. We have a constitutional responsibility to conduct an impeachment trial – it has nothing to do with an election. That’s not how this is supposed to work. https://t.co/bJ2bC3bWfz'), ('dates', '2020-02-03 16:00:18'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.7002490162849426)])
OrderedDict([('tweet', 'Now that Senate Republicans have let @realDonaldTrump off the hook, he’s focusing on:\n \n✓ attacking immigrants\n✓ cutting Social Security\n✓ getting rid of protections for those with pre-existing conditions\n \nStay tuned as he becomes more emboldened than ever.'), ('dates', '2020-02-01 01:00:07'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.856178879737854)])
OrderedDict([('tweet', "The first Social Security check was issued 80 yrs ago today. But this week @realDonaldTrump made clear he's going after this lifeline at the expense of our seniors to pay for the GOP’s tax cuts for the wealthy. Hardworking Americans earned these benefits and deserve nothing less. https://t.co/TZBKEytYkC"), ('dates', '2020-01-31 22:20:58'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6005485653877258)])
OrderedDict([('tweet', 'Our country is in great danger. @realDonaldTrump already had few constraints on his behavior and now there will be even fewer. He’s been let off the hook by Republicans. But he’s not going to be set free by the American people.'), ('dates', '2020-01-31 04:41:18'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8463383913040161)])
OrderedDict([('tweet', 'While the focus is on impeachment, @realDonaldTrump is attacking one of our oldest conservation laws to give industries a free pass to kill birds. Nearly 3 billion birds across hundreds of species have been lost in North America. We must protect the Migratory Bird Treaty Act. https://t.co/Az3p5FgLqM'), ('dates', '2020-01-30 23:39:50'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6140541434288025)])
OrderedDict([('tweet', 'We are witnessing the coronation of @realDonaldTrump with Mitch McConnell holding the crown and Republicans holding his train. https://t.co/AI8rR0qK9p'), ('dates', '2020-01-30 17:05:49'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.7650710940361023)])
OrderedDict([('tweet', ".@realDonaldTrump is attacking the ACA right now — injecting the market w/ chaos and hurting those with pre-existing conditions.\n\nSince he's pushed the Supreme Court to delay the case, millions of Americans are left in fear of losing healthcare coverage. https://t.co/IbgFLEUPFK"), ('dates', '2020-01-29 23:33:34'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.982397735118866)])
OrderedDict([('tweet', "Republicans have taken us down the rabbit hole of irrelevant distractions. They're trying to distract from the truth - the President used his powers to shake down a foreign government. John Bolton is ready and willing to testify to that, but do Republicans want to hear it? https://t.co/GdiYHYlrB2"), ('dates', '2020-01-28 17:13:54'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.9316967129707336)])
OrderedDict([('tweet', 'The Supreme Court gave @realDonaldTrump’s cruel public charge rule the green light today. This rule was intended to confuse and scare immigrant families, and it’s working. The people who will suffer the most are the poor and vulnerable among us – particularly children. https://t.co/elbJzuG85b'), ('dates', '2020-01-28 02:21:18'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.562932550907135)])
OrderedDict([('tweet', 'As part of his virulent anti-immigrant agenda, @realDonaldTrump issued the Muslim Ban 3 years ago. Since then countless families and communities have been separated because of this cruel policy. America must move past this painful chapter in our history by passing the #NoBanAct. https://t.co/uzGGOrAXpZ'), ('dates', '2020-01-27 20:12:46'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8195633888244629)])
OrderedDict([('tweet', 'John Bolton’s first person testimony is crucial to the impeachment trial. Republicans have been saying they wanted a direct witness and here he is. Republican Senators need to face the truth and make a decision of conscience. https://t.co/nfUVZ9SAF9'), ('dates', '2020-01-27 17:27:10'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6656846404075623)])
OrderedDict([('tweet', "In 2016, Trump campaigned on protecting Medicare, Medicaid, and Social Security. Now he's proposing cuts to address budget deficits driven by his tax handout to billionaires and corporations. Rather than cutting, we should be working to strengthen and expand them for our seniors. https://t.co/G4SguBZnrA"), ('dates', '2020-01-23 20:25:28'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6714106202125549)])
OrderedDict([('tweet', 'Republican senators can’t sit still because they refuse to face the facts –@realDonaldTrump shook down a vulnerable ally for his personal political gain. In other words, the truth hurts. https://t.co/bO8HWOczt7'), ('dates', '2020-01-23 17:22:33'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.7245966792106628)])
OrderedDict([('tweet', 'Last night, when Republicans voted down every effort on our part to call witnesses and documents – that shows me where their heads are at. They may be wringing their hands, but they’re voting right down the line with Mitch McConnell to protect this lying President. https://t.co/x3JaSkJ3Qz'), ('dates', '2020-01-23 02:54:11'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.6775199174880981)])
OrderedDict([('tweet', "47 years ago, #RoeVWade affirmed a woman’s constitutional right to safe, legal abortion on her own terms. Yet, extreme politicians and judges are working to undermine the right to choose. We must fight these attacks on reproductive rights - because we can't and won't go back."), ('dates', '2020-01-22 20:35:46'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8757714033126831)])
OrderedDict([('tweet', 'The President’s lawyer claimed the President “is a man of his word.” But we know @realDonaldTrump has lied 16,000 times. How can you give credence to somebody who says a whopper like that? Yet Republicans are voting down the line to protect this out of control, lawless President. https://t.co/bn0lDybuf2'), ('dates', '2020-01-22 14:10:13'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8775168061256409)])
OrderedDict([('tweet', 'If @realDonaldTrump is allowed to get away with what he did, who is he going to shake down next? Who is he going to bribe next to do his political dirty work? https://t.co/DzwGgGJj8i'), ('dates', '2020-01-21 21:51:02'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.5113656520843506)])
OrderedDict([('tweet', 'Mitch McConnell and @realDonaldTrump are totally on the same page about using the impeachment trial as a coverup. Any Republican who goes along with the McConnell resolution is complicit. The American people demand a fair trial. https://t.co/5kMmYd1NHz'), ('dates', '2020-01-21 17:05:11'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.8495882153511047)])
OrderedDict([('tweet', '10 years ago, the Supreme Court allowed special interests to flood our elections and silence our voices. Corporations shouldn’t have more power than every day Americans in our democracy. It’s time to #EndCitizensUnited and put the power back in the hands of the American people.'), ('dates', '2020-01-21 16:36:53'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.5982330441474915)])
OrderedDict([('tweet', 'Republicans are going on and on about how there was no due process. Instead of letting Mitch McConnell whitewash the impeachment process, why don’t we do the American people a service and actually have a fair trial. https://t.co/nniICz4ks7'), ('dates', '2020-01-20 02:55:58'), ('file_name', 'Mazie K. Hirono.csv'), ('pct_true', 0.7640970349311829)])
OrderedDict([('tweet', "In this time of crisis and economic uncertainty, I’m disappointed the @EPA did not stand up for small oil refineries in Wyoming. The decision not to appeal hurts small refineries &amp; puts workers' jobs at risk &amp; harms the communities that rely on them. https://t.co/GPT0Q3QwXr"), ('dates', '2020-03-25 20:03:19'), ('file_name', 'Michael B. Enzi.csv'), ('pct_true', 0.7654255032539368)])
OrderedDict([('tweet', 'Why isn’t there more concern for the employees out of work? We have a bill to provide enormous financial assistance, but my colleagues across the aisle are holding it up in order to forward their own agenda on unrelated issues. Rome is burning and Congress is fiddling! https://t.co/mCxfHhysnT'), ('dates', '2020-03-24 22:09:17'), ('file_name', 'Michael B. Enzi.csv'), ('pct_true', 0.5819905996322632)])
OrderedDict([('tweet', 'Less than three years ago, the U.S. suspended all Brazilian beef imports because of public health concerns. I’m concerned that the recent decision to lift this ban could lead to more issues and that these beef imports won’t be properly monitored. https://t.co/WHXW4fELKT'), ('dates', '2020-02-28 20:22:57'), ('file_name', 'Michael B. Enzi.csv'), ('pct_true', 0.7301495671272278)])
OrderedDict([('tweet', 'Medicare patients with hemophilia and other bleeding disorders require specialized treatments to recover after being hospitalized, but they currently face challenges when they try to receive this critical care.'), ('dates', '2020-02-27 19:51:34'), ('file_name', 'Michael B. Enzi.csv'), ('pct_true', 0.5182334184646606)])
OrderedDict([('tweet', 'Emerging cell-culture technologies may reshape the food industry in the coming years. Existing food safety laws were drafted long before these technologies were contemplated.'), ('dates', '2019-12-16 21:14:23'), ('file_name', 'Michael B. Enzi.csv'), ('pct_true', 0.650050938129425)])
OrderedDict([('tweet', 'Government shutdowns do not benefit anyone and actually end up costing taxpayers more money. I cosponsored legislation that would prevent future government shutdowns. It is time to end unnecessary shutdowns for good. https://t.co/Mlpp7SoAWB'), ('dates', '2019-12-11 15:24:46'), ('file_name', 'Michael B. Enzi.csv'), ('pct_true', 0.6741315722465515)])
OrderedDict([('tweet', 'We are celebrating an incredible landmark today – 150 years ago, Wyoming led the way by granting women the right to vote and hold office. The Equality State is proud to have taken this important step 50 years before the enactment of the 19th Amendment. https://t.co/q2ohLvhEk5'), ('dates', '2019-12-10 14:30:46'), ('file_name', 'Michael B. Enzi.csv'), ('pct_true', 0.6131958961486816)])
OrderedDict([('tweet', 'Americans deserve fiscal transparency when it comes to government spending. We need this bipartisan budget reform bill more than ever. #brokenbudget https://t.co/XMGPgMsaJq'), ('dates', '2019-11-06 19:31:35'), ('file_name', 'Michael B. Enzi.csv'), ('pct_true', 0.5428656935691833)])
OrderedDict([('tweet', "Contrary to some outdated reports, I cosponsored @LindseyGrahamSC's resolution. While the House has the right to craft its own rules, I don't believe it's following its own precedent or going about the process in a manner that's fair to the president and the House minority party."), ('dates', '2019-10-29 19:21:41'), ('file_name', 'Michael B. Enzi.csv'), ('pct_true', 0.8838587403297424)])
OrderedDict([('tweet', "The jailing of Shelley Luther was completely unnecessary. \n\nSmall biz owners have been hit hard by #COVID19 and putting them behind bars when they are trying to save their life's work should not happen. \n\nRT if you agree. https://t.co/ePiKxpiPqv"), ('dates', '2020-05-08 22:27:46'), ('file_name', 'Michael C. Burgess.csv'), ('pct_true', 0.5733383297920227)])
OrderedDict([('tweet', 'The world needs more heroes like Dr. Li Wenliang. China must be held accountable for silencing this hero and for the damage caused by their dishonesty. https://t.co/XuvGPQBgat'), ('dates', '2020-05-08 17:57:01'), ('file_name', 'Michael C. Burgess.csv'), ('pct_true', 0.5196446776390076)])
OrderedDict([('tweet', "It was obvious this virus was a bigger concern than China was letting the world know when the Chinese government  decided to shut down a city of over 10 million. \n\nThe damage inflicted on the world has been much greater due to China's dishonesty. https://t.co/q8Sc1WMvLi"), ('dates', '2020-05-05 20:45:26'), ('file_name', 'Michael C. Burgess.csv'), ('pct_true', 0.8802750110626221)])
OrderedDict([('tweet', 'The economy is sick because of virus. Congress took the steps to help American small businesses through this public health crisis. https://t.co/lesYllLZ2k'), ('dates', '2020-05-05 00:51:43'), ('file_name', 'Michael C. Burgess.csv'), ('pct_true', 0.688895046710968)])
OrderedDict([('tweet', 'The economy is sick because of this #Coronavirus and the federal government has taken action to ensure it thrives in the future by starting to reopen in phases. https://t.co/qV2tmnQrl2'), ('dates', '2020-04-28 01:04:00'), ('file_name', 'Michael C. Burgess.csv'), ('pct_true', 0.6041635274887085)])
OrderedDict([('tweet', "It's important to understand how this pandemic started as the world fights to defeat #COVID19 \n\nChina has no reason not to be honest right now. In fact, the Chinese government has a duty to share that information with us. \nhttps://t.co/xFH2Twdmlq https://t.co/1o7koo17dV"), ('dates', '2020-04-26 23:54:20'), ('file_name', 'Michael C. Burgess.csv'), ('pct_true', 0.659993052482605)])
OrderedDict([('tweet', 'It has been over a week since Senate Dems blocked vital funding for the #PPP . Now the program has run out of money. \n\n@SteveScalise and I want to hear your story about why #PPP is so important for you and your small business. \nhttps://t.co/KcOBC0hgoo'), ('dates', '2020-04-17 16:02:27'), ('file_name', 'Michael C. Burgess.csv'), ('pct_true', 0.6561118364334106)])
OrderedDict([('tweet', "The #PPP funding has run dry after Democrats blocked additional funding when their partisan demands weren't met. Our nation's small businesses need help NOW. \n\nEven Obama's own SBA Chief agrees.\nhttps://t.co/0St4kiFPXE"), ('dates', '2020-04-16 23:01:50'), ('file_name', 'Michael C. Burgess.csv'), ('pct_true', 0.8370033502578735)])
OrderedDict([('tweet', 'This is not the time for partisan games. Congress needs to be working together to defeat #COVID19 not trying to push their own partisan agenda. https://t.co/jF6iDS0EsP'), ('dates', '2020-04-16 02:30:24'), ('file_name', 'Michael C. Burgess.csv'), ('pct_true', 0.7690762281417847)])
OrderedDict([('tweet', 'Democrats continue to play partisan games. \n \nAmerican businesses need relief. \n \n@SBAgov needs funding NOW. https://t.co/hQ137qpiaA'), ('dates', '2020-04-15 04:09:57'), ('file_name', 'Michael C. Burgess.csv'), ('pct_true', 0.825534999370575)])
OrderedDict([('tweet', "Millions in taxpayer dollars sent to the WHO, as they've seemingly turned a blind eye to COVID-19 propaganda from the Chinese Community Party and at times parroted the same talking points.\n\nWhy should the U.S. fund an entity that does not further the interests of our republic? https://t.co/Kg1cFfw96F"), ('dates', '2020-04-30 18:11:11'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.5937656164169312)])
OrderedDict([('tweet', 'The government of China has taken advantage of the good will the U.S. has shown for years. Now the Chinese Communist Party has manipulated the coronavirus pandemic at every turn — in spite of the heartbreaking human toll. \n\nDiscussed the latest with @OANN:\nhttps://t.co/fESJzfp05x'), ('dates', '2020-04-24 15:00:35'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.8607994914054871)])
OrderedDict([('tweet', "The Paycheck Protection Program is serving as a lifeline to our nation’s small businesses and workers during this pandemic. Now it's run dry. Congress should prioritize American small businesses and families instead of political agendas. What’s the hold up, Speaker Pelosi?"), ('dates', '2020-04-16 17:34:17'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.7809435129165649)])
OrderedDict([('tweet', 'The partisan effort to impeach the president began long ago. \n\nFor months, Washington has wasted American taxpayer dollars, putting politics above the needs of the country. It’s past time for Congress to get things done for the American people so that we can move forward.'), ('dates', '2020-02-05 22:11:44'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.8561025261878967)])
OrderedDict([('tweet', "Impeachment proceedings have never been about a honest search for truth, and today, after withholding her Articles of Impeachment for weeks, Speaker Pelosi's finally sending them to the Senate.\n\nLet's end this charade &amp; get back to doing the work we were sent to Washington to do. https://t.co/JLZyDaHXyE"), ('dates', '2020-01-15 15:16:08'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.9252070188522339)])
OrderedDict([('tweet', 'Despite unfair and, even at times, secret hearings, this impeachment investigation uncovered no evidence that the president committed impeachable offenses. This is a single-party effort in Congress to remove a sitting president. (1/2)'), ('dates', '2019-12-19 01:47:32'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.737216591835022)])
OrderedDict([('tweet', "Every day it’s only more obvious that Washington is corrupt and broken.\n\nI voted against the so-called impeachment resolution. It doesn't grant rights to the minority or guarantee transparency for the American people. Our country should not continue down this path."), ('dates', '2019-10-31 19:29:45'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.657532811164856)])
OrderedDict([('tweet', 'I support resolution to censure Rep. Adam Schiff because I believe that he should stop holding secret hearings in the basement of the Capitol and instead give the American people the transparency they deserve. He must be held accountable for refusing access to the public.'), ('dates', '2019-10-22 13:22:46'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.7092342972755432)])
OrderedDict([('tweet', 'Impeachment proceedings should be deliberate, defined, and transparent. The "impeachment" proceedings happening in Washington right now are being conducted in a top-secret room in the basement of the Capitol.\n\nThe American people deserve access to the complete story. https://t.co/apoxK3lpLp'), ('dates', '2019-10-17 01:31:19'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.76800537109375)])
OrderedDict([('tweet', 'There is no end to the border crisis in sight. Congress is allowing cartels to massively profit because we refuse to close off the avenues they are using to smuggle migrants. We must act now.\n \n-Fix the Flores Settlement\n-Build the border wall\n-Reform broken asylum laws https://t.co/MeEem4LsLO'), ('dates', '2019-07-12 17:20:46'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.6081949472427368)])
OrderedDict([('tweet', 'U.S. ally Israel is under attack—both in its own homeland and in ours. Even in Congress, anti-Semitic statements have become a regular occurrence. I made the case today @DailySignal that America must continue to #StandWithIsrael against foreign threats and anti-Semitism here. https://t.co/QjfvHdfcNd'), ('dates', '2019-05-16 20:49:02'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.8447023630142212)])
OrderedDict([('tweet', 'Israel is a committed ally and a reliable friend of the United States, one which shares our commitment to peace and liberty. America must continue both to support Israel against foreign threats and to stand up against anti-Semitism here at home. #StandWithIsrael'), ('dates', '2019-05-15 20:07:22'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.7918388843536377)])
OrderedDict([('tweet', 'The national debt has been spiraling out of control, and it’s Congress’ duty to fix the problem. Responsible spending is the first step. That’s why I cannot in good conscience support busting the budget caps and passing the buck to our kids and grandkids.'), ('dates', '2019-05-08 16:34:32'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.7061984539031982)])
OrderedDict([('tweet', 'After a thorough investigation, and despite the wishful thinking pushed by the President’s detractors in Congress and the media over the past two years, Special Counsel Mueller’s report has confirmed that there was no collusion with Russia on the part of the President or his 1/'), ('dates', '2019-03-25 22:16:19'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.7482045888900757)])
OrderedDict([('tweet', "In today's “compromise” – Congress could not find $5.7 billion for a barrier that Customs and Border Protection has requested to help end the humanitarian crisis on our border.\n\nYet the bill contains more than $12 billion in international assistance.\n\n#CongressIsBroken"), ('dates', '2019-02-14 22:06:28'), ('file_name', 'Michael Cloud.csv'), ('pct_true', 0.9315662980079651)])
OrderedDict([('tweet', 'People’s lives have been upended by this pandemic. @SenJackReed, @RepDonBeyer, and I released a plan to use automatic triggers to ensure those who have lost their jobs receive sustained government assistance to help them through this crisis. https://t.co/Qif1Xd0wpr'), ('dates', '2020-05-09 15:07:08'), ('file_name', 'Michael F. Bennet.csv'), ('pct_true', 0.6423638463020325)])
OrderedDict([('tweet', '75 years ago, the US and allies liberated Europe, ending a grueling campaign that had stretched from El Alamein to the Ardennes. Today, we honor these men and women not only by remembering their sacrifice, but recommitting to the values for which they fought. #VEDay75'), ('dates', '2020-05-08 16:28:23'), ('file_name', 'Michael F. Bennet.csv'), ('pct_true', 0.5507562160491943)])
OrderedDict([('tweet', "This pandemic has taken an enormous toll on @USPS. As an institution that's always been there for us – we must not allow our dedicated postal workers to weather the storm alone.\n \nCongress must act now to ensure long-term financial security for the USPS. https://t.co/DckamY5yYp"), ('dates', '2020-05-07 23:32:31'), ('file_name', 'Michael F. Bennet.csv'), ('pct_true', 0.5240573287010193)])
OrderedDict([('tweet', 'Every American should be concerned when the Department of Justice puts the interests of the president and his cronies ahead of the American people. The Trump Administration’s assault on the rule of law continues. https://t.co/6aE8j2vUNW'), ('dates', '2020-05-07 20:42:29'), ('file_name', 'Michael F. Bennet.csv'), ('pct_true', 0.5331545472145081)])
OrderedDict([('tweet', "Not having access to the internet today means not having access to a classroom.\n\nNo senator would accept that for their children. And we can't accept it for America's children. https://t.co/6ffVu6ECvf"), ('dates', '2020-04-30 17:05:49'), ('file_name', 'Michael F. Bennet.csv'), ('pct_true', 0.5119438171386719)])
OrderedDict([('tweet', 'Attacking immigrants and spreading fear won’t protect us from this virus.\n \nUniversal testing and supplies and equipment will.'), ('dates', '2020-04-21 03:50:57'), ('file_name', 'Michael F. Bennet.csv'), ('pct_true', 0.502761721611023)])
OrderedDict([('tweet', 'Withdrawing U.S. funding from the World Health Organization in the midst of a global pandemic is one of the most irrational things the president could do.\n \nInstead of blaming the WHO, the president should take responsibility and reclaim U.S. global leadership.'), ('dates', '2020-04-15 21:30:10'), ('file_name', 'Michael F. Bennet.csv'), ('pct_true', 0.5922118425369263)])
OrderedDict([('tweet', "President Trump's decision to remove the inspector general overseeing the $2.2 trillion CARES Act sent a clear and chilling message across government:\n \nThose who raise fact-based concerns and truths within the administration will be silenced."), ('dates', '2020-04-14 20:21:54'), ('file_name', 'Michael F. Bennet.csv'), ('pct_true', 0.5405223369598389)])
OrderedDict([('tweet', 'Just as @USTreasury acted to ensure those on Social Security and Disability will receive cash payments without having to jump through hoops, the same must be done for those receiving Supplemental Security Income (SSI) and Veterans benefits.'), ('dates', '2020-04-03 20:42:51'), ('file_name', 'Michael F. Bennet.csv'), ('pct_true', 0.5137460827827454)])
OrderedDict([('tweet', '"It doesn\'t take a pandemic to kick so many women and families into economic ruin. But maybe it takes a pandemic for the rest of the country to take notice." - Lilly Ledbetter\n \nClosing the wage gap is common sense. And it\'s needed now more than ever. https://t.co/mEhOgfRe0B'), ('dates', '2020-04-01 03:19:38'), ('file_name', 'Michael F. Bennet.csv'), ('pct_true', 0.5495989918708801)])
OrderedDict([('tweet', '.@SenateMajLdr doesn’t think we need this legislation? Black Americans are STILL being killed by violent racists. In February, I voted for the Emmett Till Anti-Lynching Act to make lynching a federal crime. @SenateMajLdr standing in the way of justice for so many victims. https://t.co/X1zfDw3YIW'), ('dates', '2020-05-07 16:10:51'), ('file_name', 'Michael F. Doyle.csv'), ('pct_true', 0.6438788771629333)])
OrderedDict([('tweet', '.@BetsyDeVosED just finalized a rule that undermines existing Title IX protections for survivors of sexual assault.'), ('dates', '2020-05-06 18:30:56'), ('file_name', 'Michael F. Doyle.csv'), ('pct_true', 0.8716281652450562)])
OrderedDict([('tweet', 'The Paycheck Protection Program was supposed to help small and minority-owned businesses during #COVID19, but it’s missed the mark. The Trump Administration’s implementation has left CDFIs and smaller lenders behind and many small businesses are now in even more dire straits.'), ('dates', '2020-04-30 00:17:30'), ('file_name', 'Michael F. Doyle.csv'), ('pct_true', 0.805295467376709)])
OrderedDict([('tweet', 'Woke up to news of ANOTHER @realdonaldtrump #FridayNightMassacre. @POTUS’ goal is clear: punishing anyone who does their job and reveals potential wrongdoing by this administration &amp; intimidating anyone else from exposing illegal behavior in the future. \n\nhttps://t.co/SFLGbj4YFg'), ('dates', '2020-04-04 11:59:39'), ('file_name', 'Michael F. Doyle.csv'), ('pct_true', 0.5684535503387451)])
OrderedDict([('tweet', 'Territories are under the plenary power of Congress. The House represents the People, the Senate represents the Republic. \n\nTerritories have Delegates in the House; the role of Territories as a part of the Republic, should also Senate representation.\n\nhttps://t.co/XUen9XCdDC'), ('dates', '2019-12-22 04:21:48'), ('file_name', 'Michael F. Q. San Nicolas.csv'), ('pct_true', 0.6712453365325928)])
OrderedDict([('tweet', 'Border Wall will diverting over $749 million of current Guam buildup funding to the $3.6 billion border wall. Diversion of these funds will certainly setback the military buildup on Guam. #Guam #borderwall #DOD'), ('dates', '2019-05-01 13:04:24'), ('file_name', 'Michael F. Q. San Nicolas.csv'), ('pct_true', 0.6627262830734253)])
OrderedDict([('tweet', 'Democrats continue to use COVID-19 as an excuse to push their partisan agenda, but this is not a time to let up on pro-life protections. I joined a group of Republicans to introduce legislation to defend the Hyde Amendment. Taxpayer dollars should never be used to fund abortions.'), ('dates', '2020-05-09 15:01:59'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.9822826981544495)])
OrderedDict([('tweet', 'The Democrats’ Subcommittee on the Coronavirus Crisis will become another partisan circus like the impeachment sham. Republicans are investigating the true root of the coronavirus outbreak, the Chinese Communist Party.\n \nhttps://t.co/aGsWzCBJ5f'), ('dates', '2020-05-07 21:56:59'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.9115161299705505)])
OrderedDict([('tweet', 'Hard-working Americans are losing their jobs and Democrats are worried about “losing leverage.” Democrats blocked funds for businesses to pay their employees in order to gain political points for their far-left agenda. This is wrong. https://t.co/NQ6Fgro2AJ'), ('dates', '2020-04-21 18:51:23'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.8369775414466858)])
OrderedDict([('tweet', 'Over 20,000 Mississippi businesses have received loans through the Paycheck Protection Program so they can keep their doors open. Americans need this program, but by holding up additional funding for this program, Democrats have pressured businesses to close.'), ('dates', '2020-04-20 17:32:16'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.7899758219718933)])
OrderedDict([('tweet', '.@SpeakerPelosi &amp; @SenSchumer are putting the future of thousands of small businesses at risk by obstructing funds needed to keep these businesses afloat. Democrats need to stop blocking these funds. America needs its small businesses.'), ('dates', '2020-04-16 15:19:32'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.863082230091095)])
OrderedDict([('tweet', 'I cosponsored legislation to withhold WHO funding &amp; I stand w/ President Trump to hold this organization accountable for their mismanagement of the coronavirus response. I  also called for an investigation into the WHO’s missteps in the COVID-19 response.\n\nhttps://t.co/FTCSR9HIRb'), ('dates', '2020-04-15 00:17:10'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.7079361081123352)])
OrderedDict([('tweet', 'A national emergency is not a tool for Democrats to force through their progressive policies. Nancy Pelosi, Chuck Schumer and the Democratic Party need to set aside their political agenda and instead focus on combatting this crisis and helping Americans. https://t.co/0utdC95F15'), ('dates', '2020-03-23 18:34:25'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.583057701587677)])
OrderedDict([('tweet', 'I am deeply troubled that Democrats refuse to recognize the simple fact that every child is precious. Today, Democrats blocked our bill, the #BornAlive Abortion Survivors Protection Act, that would have protected the lives of children born alive following an attempted abortion. https://t.co/cSjpVU70TF'), ('dates', '2020-02-28 19:52:16'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.6562916040420532)])
OrderedDict([('tweet', 'The Socialism experiment has been run numerous times and it always ends in devastation, while conservative economic policies have created prosperity across our globe. Democrats and Republicans should agree: America will never be a Socialist country. https://t.co/GS3lMV1Q8Q'), ('dates', '2020-02-28 17:01:00'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.6758151650428772)])
OrderedDict([('tweet', 'The #NewWayForwardAct is a dangerous piece of legislation that would require law enforcement to be lenient on criminals if they are in the U.S. illegally. As a member of @HomelandGOP, I will continue to fight to secure our borders &amp; protect our communities from criminal activity. https://t.co/MJinzgHrWW'), ('dates', '2020-02-14 17:05:05'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.6092827320098877)])
OrderedDict([('tweet', "Impeachment by the numbers:\n\n• 18 witnesses have testified\n• 30 times Schiff said they had “made their case”\n• 0 evidence of wrongdoing\n\nDemocrats have wasted tax dollars &amp; time in their attempt to undo the 2016 election. Enough is enough. It's time to end the #impeachmentsham"), ('dates', '2020-01-31 21:42:03'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.987184464931488)])
OrderedDict([('tweet', 'This weekend, tens of thousands of Americans will gather in Washington to take a stand for our unborn children. Their voices are heard loud &amp; clear by many Members of Congress, &amp; it is my hope we can do more to convince all Members that every child has a right to life. #ProLife https://t.co/fC6b0AeZ00'), ('dates', '2020-01-24 16:54:44'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.5525715351104736)])
OrderedDict([('tweet', 'Iran tensions have been subdued, and the Democrats’ impeachment has been exposed as a partisan sham. Tune in to @supertalk tomorrow morning at 7:20 am to get the latest news as I discuss these issues and many more with @paulgalloshow.'), ('dates', '2020-01-24 00:00:00'), ('file_name', 'Michael Guest.csv'), ('pct_true', 0.5249754190444946)])
OrderedDict([('tweet', 'It appears you were ill-informed Bob. Sharing this manipulated video is beneath you &amp; we expect more from journalists. For the record my full quote said “Covid-19 isn’t a Republican problem or a Democrat problem, it’s an American problem.”\n\n#StrongerTogetherEvenWhenApart https://t.co/pAPwbeQzSB'), ('dates', '2020-05-09 01:20:32'), ('file_name', 'Michael K. Simpson.csv'), ('pct_true', 0.6637093424797058)])
OrderedDict([('tweet', 'Today is the 75th anniversary of victory in Europe. Nazi German was defeated and freedom rings today because of the heroes sacrifice. \n\n#VEDay75 https://t.co/eGT3KefWw2'), ('dates', '2020-05-08 16:01:58'), ('file_name', 'Michael K. Simpson.csv'), ('pct_true', 0.6052728891372681)])
OrderedDict([('tweet', 'The #PPP has been a lifesaver for millions of American workers. I am disappointed @SpeakerPelosi  &amp; House Dems took so long to replenish it. I will be voting YES for additional funding for this program because American jobs and small business depend on it. https://t.co/TEdZ6DWSOz'), ('dates', '2020-04-22 21:30:36'), ('file_name', 'Michael K. Simpson.csv'), ('pct_true', 0.8537853360176086)])
OrderedDict([('tweet', 'The  #PaycheckProtectionProgram is officially out of funds. Small businesses throughout Idaho are counting on Congress to step up. I’m calling on @SpeakerPelosi to stop playing politics and let this money get to the people who actually need it. https://t.co/V3uB7gcnrI'), ('dates', '2020-04-16 16:49:24'), ('file_name', 'Michael K. Simpson.csv'), ('pct_true', 0.5764762163162231)])
OrderedDict([('tweet', 'Russia killed #INFtreaty by repeatedly violating it, placing Europe &amp; US at risk with its new nuclear missiles &amp; weapons. A treaty can’t be with one party. Russia failed to become compliant within a 6 month deadline, bringing into question our nuclear arms agreements.'), ('dates', '2019-08-02 13:11:20'), ('file_name', 'Michael R. Turner.csv'), ('pct_true', 0.6395286917686462)])
OrderedDict([('tweet', 'Legislating against our Commander in Chief’s ability to defend our country and its allies is a threat to our national security and dangerously emboldens Iran and its proxy terrorist organizations Hamas and Hezbollah which threaten our ally Israel.'), ('dates', '2019-06-25 17:45:57'), ('file_name', 'Michael R. Turner.csv'), ('pct_true', 0.5549798607826233)])
OrderedDict([('tweet', 'That’s why it was so important for @GOPLeader to launch the #ChinaTaskForce-to hold the #CCP accountable not only for their #COVID19 coverup, but to address their malign actions globally. Why won’t Dem leaders let their members stand up for what they know is right &amp; join us? https://t.co/15fhRvh4YA'), ('dates', '2020-05-09 13:58:12'), ('file_name', 'Michael T. McCaul.csv'), ('pct_true', 0.801634669303894)])
OrderedDict([('tweet', 'When faced w/ an impending pandemic once again in 2019, the #CCP failed to report, allowing #COVID19 to spread, killing 100s of 1,000s of people &amp; devastating the global economy. The WHO blindly followed the CCP, delaying necessary action to protect people around the globe. https://t.co/JL8v84MFsE'), ('dates', '2020-05-04 17:08:18'), ('file_name', 'Michael T. McCaul.csv'), ('pct_true', 0.5224696397781372)])
OrderedDict([('tweet', 'Today’s oil market news is yet another devastating blow to TX’s resilient oil industry. It’s vital that Congress expeditiously funds the SPR and looks at every other feasible option to alleviate this unprecedented burden.'), ('dates', '2020-04-20 22:14:09'), ('file_name', 'Michael T. McCaul.csv'), ('pct_true', 0.6590980291366577)])
OrderedDict([('tweet', 'China committed one of the worst cover-ups in human history. \n\nREAD👇🏼\n\nhttps://t.co/MSzwcRUo9G'), ('dates', '2020-03-30 13:59:17'), ('file_name', 'Michael T. McCaul.csv'), ('pct_true', 0.5063771605491638)])
OrderedDict([('tweet', 'This is an unprecedented moment – the #CoronavirusOubreak will go down in history as one of the worst crises our nation has faced: people are losing loved ones, their jobs, and small businesses built over a lifetime are failing. (1/3)'), ('dates', '2020-03-27 16:25:19'), ('file_name', 'Michael T. McCaul.csv'), ('pct_true', 0.78701251745224)])
OrderedDict([('tweet', 'Today, we saw the shocking, but not surprising, news. A staggering 3.3 million Americans have filed for unemployment as a direct result of the economic damage caused by the #Coronavirus.'), ('dates', '2020-03-26 19:59:39'), ('file_name', 'Michael T. McCaul.csv'), ('pct_true', 0.8332464098930359)])
OrderedDict([('tweet', 'The threat posed by #China’s Communist Party isn’t a partisan issue. It’s an issue ALL Americans should care about.\n\nThe #CCP is working overtime to take America down.\n\nWe must act to protect our national security. https://t.co/5eqdv9xCm7'), ('dates', '2020-05-09 18:28:35'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.9453349113464355)])
OrderedDict([('tweet', 'Saying the #Taliban’s spring offensive aimed at overthrowing the #Afghan govt is keeping the “Peace Deal” is a farce.\n\nIf the Taliban were truly serious about peace, they would enter a ceasefire &amp; release Mark #Frerichs IMMEDIATELY! https://t.co/0DnOSPcudk'), ('dates', '2020-05-08 19:12:59'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.6551812291145325)])
OrderedDict([('tweet', 'We cannot let #China and its communist government continue on the path to a global takeover. Looking forward to joining my House colleagues on the China Task Force to protect our freedoms and our national security. https://t.co/jbigdL7wm7'), ('dates', '2020-05-07 15:24:17'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.8441370725631714)])
OrderedDict([('tweet', "Not surprising that #China's government opposes an investigation into the origins of #COVID19. \n\nWhen the world finds out the #CCP's irresponsible behavior caused this pandemic to spread, #China's entire attempt at global dominance will crumble. https://t.co/FzssIKBIco"), ('dates', '2020-05-07 14:33:15'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.9806864857673645)])
OrderedDict([('tweet', "Whether it's through college students or professors, the #CCP is stealing our research &amp; technology to later use against us.\n\n#China is trying to take America down by cheating their way to the top.\n\nOur national security &amp; way of life are in danger. We have to put a stop to this! https://t.co/qr1xokksFg"), ('dates', '2020-05-07 13:00:01'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.7165713906288147)])
OrderedDict([('tweet', "While American universities are home to some of the brightest scholars, they also house a dark underbelly of Chinese espionage &amp; intellectual theft that puts our country in danger.\n\nWe must put an end to #China's nefarious plan to steal its way to the top. https://t.co/PRfmHHnlBm"), ('dates', '2020-05-06 14:40:54'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.7484331727027893)])
OrderedDict([('tweet', 'America’s military shouldn’t have their retirement funds paying for #China’s mass surveillance and human rights abuses.\n\nWe cannot allow ANY of these funds to be funneled to the #CCP. We have to put a stop to this practice immediately! https://t.co/ikDePA7KpR'), ('dates', '2020-05-05 12:22:15'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.9274052977561951)])
OrderedDict([('tweet', "#China's Communist regime is actively buying influence to steal technology &amp; research from our nation's universities.\n\nWe cannot allow this rampant spying &amp; theft in our colleges. We owe it to the American people to protect our country from bad actors and their communist ideals! https://t.co/0fTlt2i805"), ('dates', '2020-05-04 18:20:09'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.9721570611000061)])
OrderedDict([('tweet', 'It is a complete disgrace that our military &amp; federal employees’ retirement plans will help fund #China’s Communist regime.\n\nThe #CCP is a threat to American values &amp; a danger to our way of life.\n\nAs a member of the military for 24 years, I won’t stand for this! https://t.co/iAflroFh08'), ('dates', '2020-05-01 00:00:00'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.859956681728363)])
OrderedDict([('tweet', 'The last thing our military should be investing their retirement plans on is #China’s Communist regime. \n\nWe must stop this horrible decision and remove and replace ALL the board members who approved it! https://t.co/z8dOVP5nQZ'), ('dates', '2020-04-30 20:46:14'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.7352278828620911)])
OrderedDict([('tweet', "Citrus greening came from China &amp; devastated Florida's citrus groves. We shouldn't expose their crops to any potential new pests &amp; diseases. \n\nJoined my FL colleagues to urge @USDA to reverse the decision to import new fruits from #China and #Mexico. We stand with FL growers! https://t.co/XasRgMCM86"), ('dates', '2020-04-30 13:30:00'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.6606127023696899)])
OrderedDict([('tweet', "It's a disgrace to our military that their retirement savings will be used to fund #China's human rights abuses, mass surveillance &amp; quest for global domination.\n\nAs a member of the U.S. military for 24 years &amp; counting, I will not stand for this. \nhttps://t.co/n7ou71UEBd"), ('dates', '2020-04-28 17:31:05'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.7466570138931274)])
OrderedDict([('tweet', 'According to the #CCP, Americans are “lying through their teeth” &amp; playing "tricks" to deflect blame on the origin of #COVID19.\n \nThere\'s no blame to deflect. This pandemic started in #China and because of their propaganda &amp; lies, thousands died worldwide.\nhttps://t.co/SFWPS5lrak'), ('dates', '2020-04-28 16:46:31'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.9088950753211975)])
OrderedDict([('tweet', "#China's Communist Party doesn't want to be our friend.\n\nThey want to dominate the West with their communist ideals — and if China has its way, the entire world will be subservient to their New World Order with China in charge.\n\nWake up, America. We have to protect ourselves. https://t.co/N4krZREON5"), ('dates', '2020-04-25 17:47:27'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.9894346594810486)])
OrderedDict([('tweet', 'Our brave men and women in uniform can’t have their retirement funds going to #China’s Communist Party. \n\nWorking hard to put a stop to this crazy practice! @PeteHegseth https://t.co/6n32iTAzAE'), ('dates', '2020-04-25 14:23:38'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.6020298600196838)])
OrderedDict([('tweet', 'For years, America has been dependent on a drug supply chain that starts in #China.\n\n#COVID19 has shown us the dangers of this vulnerability.\n\nWe have to bring our drug supply back to the US and put America first again. https://t.co/ocCtP6oQT6'), ('dates', '2020-04-24 13:35:02'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.7908411026000977)])
OrderedDict([('tweet', 'Email House members received just now. The Senate is taking action tomorrow. But the House isn’t meeting until Thursday at the earliest?\n\nThat’s TOO LATE. \n\nWhile our small businesses in Florida are suffering and @SpeakerPelosi and the dithering Dems are in no rush to help them! https://t.co/5V6Rar8Rw4'), ('dates', '2020-04-21 17:54:01'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.7536153197288513)])
OrderedDict([('tweet', '#COVID19 is a wake-up call for America that the government of #China is not our friend.\n\nTheir propaganda, lack of transparency &amp; influence is dangerous to the world. We have to cut off the #CCP.\n\nWe cannot allow the CCP to have stake in American industries anymore. https://t.co/goFKddDID7'), ('dates', '2020-04-21 15:19:27'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.8362009525299072)])
OrderedDict([('tweet', 'Democrats are AWOL on #PPP for small businesses. @RepBrianMast I’m in Washington, ready to join you in the foxhole (six feet away of course) to fight for this funding! Let’s roll. 👊🇺🇸 @realDonaldTrump https://t.co/0wHluX5X2K'), ('dates', '2020-04-17 18:28:13'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.6794513463973999)])
OrderedDict([('tweet', "This is the message small businesses see when they try to apply for help via the #PaycheckProtectionProgram.\n\xa0\nCongress is not scheduled to be back in DC until 5/4 \u2060— 17 days away.\n\xa0\nOur small business owners &amp; employees can't wait that long.\n\nThey're depending on us to act NOW. https://t.co/jd8sWjVSqB"), ('dates', '2020-04-16 14:54:01'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.7274383306503296)])
OrderedDict([('tweet', "#China's government downplayed the #COVID19 pandemic &amp; the world looked to the @WHO for guidance on how to respond.\n\nInstead of sounding the alarm, the WHO catered to #CCP propaganda &amp; downplayed the severity of this virus.\n\nI stand with President @realDonaldTrump to #DefundWHO. https://t.co/Ha4X746jEM"), ('dates', '2020-04-15 19:07:30'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.887057363986969)])
OrderedDict([('tweet', "#China's government waited SIX DAYS before warning the public of a pandemic. \n\nThat's six days too long. \n\nThe #CCP's coverup of #COVID caused unnecessary disease and death worldwide. They must be held accountable! https://t.co/FMuuDYvEY1"), ('dates', '2020-04-15 18:53:40'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.781655490398407)])
OrderedDict([('tweet', 'On 12/30 a Chinese doctor warned of a deadly SARS-like illness. He was punished for it.\n\nIn Jan., China knew #COVID was spreading. Yet WHO Director Tedros praised #China’s handling of the virus.\n\nWHO blessed the CCP response to this pandemic. The world is suffering because of it. https://t.co/GU3x3nyXe2'), ('dates', '2020-04-14 23:33:33'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.7082546949386597)])
OrderedDict([('tweet', "#China's government buys influence in underdeveloped countries by lending them money for development projects.\n\nIn Ethiopia, this gave them power over officials like @DrTedros, who is now complicit in #CCP's propaganda at the @WHO.\n\nThe CCP is danger to the health of the world. https://t.co/lxjgPM4h7I"), ('dates', '2020-04-13 18:17:54'), ('file_name', 'Michael Waltz.csv'), ('pct_true', 0.898522138595581)])
OrderedDict([('tweet', '75 years ago, the Allies declared victory over Nazi Germany in WWII. Today, we remember and honor the bravery and sacrifices made by our Greatest Generation, who fought to defend freedom around the world. #VEDay75 https://t.co/IxRLIG0au5'), ('dates', '2020-05-08 18:48:57'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.7704270482063293)])
OrderedDict([('tweet', "Millions of job creators &amp; their employees are relying on the #PPP. It's all that's left for many that are struggling to survive. I was proud to vote for additional funding because hardworking Americans shouldn’t be left in limbo at the very moment they are most in need of help. https://t.co/nuMVcHSol3"), ('dates', '2020-04-23 22:28:47'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.5525359511375427)])
OrderedDict([('tweet', 'Speaker Pelosi is pushing to create a Select Committee on the Coronavirus Crisis. She wants more bureaucracy, more partisan grandstanding, and another means to attack the president.  The federal government already has eight oversight functions over the COVID-19 response:'), ('dates', '2020-04-23 17:20:02'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.8621302247047424)])
OrderedDict([('tweet', 'The #PPP has run dry, leaving millions of job creators w/o the emergency funding needed to stay afloat. We knew this day was coming, but Pelosi &amp; Schumer were willing to let businesses &amp; employees suffer while they played politics &amp; killed extra funding. https://t.co/SCDgReXK7y https://t.co/kuxlOvVSm5'), ('dates', '2020-04-16 20:51:09'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.7623408436775208)])
OrderedDict([('tweet', 'The PPP is a desperately needed lifeline for millions of businesses. But funds are expected to run out this week. When Republicans advanced legislation to fund the PPP, how did Speaker Pelosi react? She congratulated Schumer for blocking the bill &amp; killing the additional funding. https://t.co/toMvXHkxmi'), ('dates', '2020-04-15 17:03:27'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.720999002456665)])
OrderedDict([('tweet', 'Families are scared; small businesses are struggling; hospitals are fighting an uphill battle. While the bill wasn’t perfect, the consequences of inaction were too high not to support it. We have a long way to go but the more certainty we can restore, the quicker we’ll get there. https://t.co/PooBGUiFkh'), ('dates', '2020-03-27 17:54:08'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.8037675023078918)])
OrderedDict([('tweet', 'Not only did Speaker Pelosi intentionally disrespect @POTUS by ripping up his speech, she insulted the Americans whose inspiring stories were included in those pages. Her actions reflected poorly on the entire House and deserved a rebuke by her colleagues. I voted to do so today. https://t.co/CR053CCEko'), ('dates', '2020-02-06 19:21:17'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.501289963722229)])
OrderedDict([('tweet', 'The Senate put the final nail in the coffin for impeachment today, reaching the same conclusion so many Americans knew all along: the Democrats’ motives were partisan and the evidence was paltry. \n\nRead my full statement ↓ \nhttps://t.co/zfYcwM9Z85'), ('dates', '2020-02-05 23:06:14'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.6588248014450073)])
OrderedDict([('tweet', 'There’s a big difference though, @nathanlgonzales: I protested the Democrat majority in IL giving us mere minutes to review a 200 page pension reform bill before voting on it. Speaker Pelosi deliberately tried to insult the leader of our country behind his back on national TV. https://t.co/mCYADLPYAw'), ('dates', '2020-02-05 17:07:38'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.690683901309967)])
OrderedDict([('tweet', "Not even a month after Iran launched an attack on US troops, Dems voted to limit @POTUS's ability to keep our nation safe by blocking him from taking action against Iran, even if intelligence shows an attack on the US or our allies is imminent. I voted NO on tying POTUS's hands."), ('dates', '2020-01-30 19:03:14'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.9281344413757324)])
OrderedDict([('tweet', "Things are looking up for America’s farmers – @POTUS signed #USMCA this morning! Under the President's leadership and with bipartisan support in Congress, we’ve taken big step forward on behalf of the American people. #USMCAWin https://t.co/9r2Yo8zdQl"), ('dates', '2020-01-29 19:36:09'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.7175480127334595)])
OrderedDict([('tweet', 'For months, Speaker Pelosi has been saying that impeachment is an urgent national interest. Yet, 21 days after the House voted on the articles of impeachment she is still refusing to send them to the Senate. Even DC Democrats have had enough.\n \nhttps://t.co/7sDVd3R0su'), ('dates', '2020-01-08 23:46:52'), ('file_name', 'Mike Bost.csv'), ('pct_true', 0.8090423941612244)])
OrderedDict([('tweet', "We can’t keep borrowing trillions. Our public debt wasn't projected to reach the level we're at now until years in the future, and yet most in Congress continue to shrug off trillion dollar deficits. \n\nThat should advance the discussion of how we address our systemic debt issue. https://t.co/3MlxXOmIEY"), ('dates', '2020-05-05 15:56:07'), ('file_name', 'Mike Braun.csv'), ('pct_true', 0.6259080171585083)])
OrderedDict([('tweet', "We must be careful that we don't get over the tipping point when it comes to the economic damage. The tough thing about it is we can't give one inch back to the disease. Because if we backslide there, then it has a snowball effect into the economy.\nhttps://t.co/LwS5frfdwV"), ('dates', '2020-04-27 14:04:10'), ('file_name', 'Mike Braun.csv'), ('pct_true', 0.5530301332473755)])
OrderedDict([('tweet', "As a Main Street entrepreneur, it's sad that Senate Democrats have blocked $250 billion of additional aid for small businesses. The PPP fund will soon run out of money and more businesses will close their doors while Democrats block additional relief.\nhttps://t.co/A0R2KZFlg4"), ('dates', '2020-04-09 16:49:20'), ('file_name', 'Mike Braun.csv'), ('pct_true', 0.7571210861206055)])
OrderedDict([('tweet', '“Sadly I think Nancy Pelosi flew back to town to commandeer our bipartisan task group negotiations because Rahm Emanuel was right: ‘Never let a crisis go to waste.’” @FRCdc \nhttps://t.co/pGOHp0rcHG'), ('dates', '2020-03-24 14:51:36'), ('file_name', 'Mike Braun.csv'), ('pct_true', 0.5412455797195435)])
OrderedDict([('tweet', 'You are either helping right now, or hurting. Pelosi’s handpicked partisan ploys are helping no one. She’s hurting American families by prioritizing airplane emissions, same day voter registration&amp;wind energy tax credits over #coronavirus relief. #StopTheGamesNancy, #PassTheBill'), ('dates', '2020-03-24 14:26:24'), ('file_name', 'Mike Braun.csv'), ('pct_true', 0.9826825857162476)])
OrderedDict([('tweet', 'There was no need for this to be delayed - sadly @SpeakerPelosi and @SenSchumer hijacked the bipartisan negotiations. We’re covering small business &amp; workers with liquidity &amp; larger businesses with loans - not bailouts or grants - so they can pay their employees. This is urgent. https://t.co/7ub6izKh4Q'), ('dates', '2020-03-23 13:44:11'), ('file_name', 'Mike Braun.csv'), ('pct_true', 0.8887912034988403)])
OrderedDict([('tweet', 'From Top Gun to Red Dawn, no American icon is safe from Chinese Communist Censorship in our movies today. My modest proposal to @JonahDispatch? A disclaimer before the film starts, stating that the content has been approved by the CCP. https://t.co/IkSFZnZWCF'), ('dates', '2020-04-29 13:35:51'), ('file_name', 'Mike Gallagher.csv'), ('pct_true', 0.6343762874603271)])
OrderedDict([('tweet', 'This crisis has accelerated a bipartisan recognition that we need to re-evaluate our relationship with China. But are other countries coming to that realization? On this week’s NEW Look podcast I’m joined by @TomTugendhat to discuss geopolitics &amp; the UK’s response to this virus. https://t.co/soT54mdkPR'), ('dates', '2020-04-28 17:22:33'), ('file_name', 'Mike Gallagher.csv'), ('pct_true', 0.6089025735855103)])
OrderedDict([('tweet', 'It’s inexcusable that Speaker Pelosi allowed funding for the PPP to run dry. Small businesses deperately need these funds, and we should reconvene ASAP to get this done.\n\nEvery day we delay funding the PPP is a day we risk seeing more hardworking Wisconsinites miss a paycheck. https://t.co/sNPPuEz8f0'), ('dates', '2020-04-17 15:28:38'), ('file_name', 'Mike Gallagher.csv'), ('pct_true', 0.6245607137680054)])
OrderedDict([('tweet', 'During this time of crisis, we are painfully coming to the realization that our pharmaceutical supply chains are far too reliant on China. Today I spoke with @TeamCavuto about my bill to move these critical supply chains out of China and bring this manufacturing back home. https://t.co/EMY5LXxX40'), ('dates', '2020-04-10 21:05:07'), ('file_name', 'Mike Gallagher.csv'), ('pct_true', 0.6921244859695435)])
OrderedDict([('tweet', '.@jack cannot sidestep the question as to why he allows the CCP to block the Chinese public from using Twitter while simultaneously using the platform to spread propaganda around the world. The American and Chinese people deserve an answer.\n\nhttps://t.co/iByqUEP5JH'), ('dates', '2020-04-08 23:43:19'), ('file_name', 'Mike Gallagher.csv'), ('pct_true', 0.8924666047096252)])
OrderedDict([('tweet', 'Great joining @LRacheldG on the @DailySignal podcast to discuss why its critical we move our pharmaceutical supply chains out of China, and additional actions Congress should take to combat the Coronavirus. Listen to the full interview here: https://t.co/FlpGnUG3Zn'), ('dates', '2020-04-08 17:09:46'), ('file_name', 'Mike Gallagher.csv'), ('pct_true', 0.5689730644226074)])
OrderedDict([('tweet', "The world needs to know the full scope of the Chinese Communist Party's role in\xa0the present catastrophe. After protecting for sources and methods, as much of this report as possible needs to be declassified in order to hold CCP leaders to account. https://t.co/onXL1ARm8w"), ('dates', '2020-04-01 16:40:56'), ('file_name', 'Mike Gallagher.csv'), ('pct_true', 0.5413264632225037)])
OrderedDict([('tweet', "The Justice Dept.'s decision to drop the case against @GenFlynn is welcome news, but Congress still has a responsibility to get answers about the FBI's conduct during its investigation. More on how @Jim_Jordan &amp; I are leading the charge for an explanation⬇️https://t.co/FzWHodBqIr"), ('dates', '2020-05-07 23:13:34'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.6669356822967529)])
OrderedDict([('tweet', "America has paid dearly during this pandemic, both in lives and livelihoods. If that is found to be a direct result of China’s actions, the Chinese government must be held accountable. Congress needs to be ready and have the backbone to enact @RepublicanStudy's recommendations. https://t.co/uCNV3vuRaG"), ('dates', '2020-05-04 13:03:53'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.7355340719223022)])
OrderedDict([('tweet', "Tune out the MSM. @realDonaldTrump's innovative response to COVID-19 has set a new precedent for presidential leadership during a national health crisis. History will show that cutting red tape, private sector partnerships &amp; letting states lead was key to overcoming this pandemic https://t.co/K35EZ2xbSB"), ('dates', '2020-05-04 12:33:42'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.5903576016426086)])
OrderedDict([('tweet', 'Our state has been hit hard by COVID-19, the oil &amp; gas crisis, &amp; even a series of violent storms--but Louisianians never give up &amp; we always pull together.\n\nThe great people of Vernon Parish have shown once again why hope prevails here. #Louisianastrong https://t.co/3aOxTf4ZKJ'), ('dates', '2020-04-29 15:55:20'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.560064971446991)])
OrderedDict([('tweet', "America has paid dearly during this pandemic, both in lives and livelihoods. If that is found to be a direct result of China’s actions, the Chinese government must be held accountable. Read  @RepublicanStudy's proposal to do just that ➡️ https://t.co/H47GlxqrLD https://t.co/q16mA4GdBH"), ('dates', '2020-04-28 19:30:39'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.5702837109565735)])
OrderedDict([('tweet', 'Leave it to @SpeakerPelosi to turn a public health crisis into a political tool to attack @realDonaldTrump.\n\xa0\nNow, she has created yet another subcommittee for overseeing the coronavirus crisis, when we ALREADY have 8 committees tasked with oversight on the response.'), ('dates', '2020-04-23 23:00:42'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.8459713459014893)])
OrderedDict([('tweet', 'In the last 7 hours, over 35,000 #PPPloans went unprocessed because @SpeakerPelosi &amp; @SenSchumer are blocking more funding. We are fighting to save jobs and small businesses. What have they fought for?\xa0\xa0\n\n… a taxpayer bailout for newspaper pension plans.\n\nhttps://t.co/jWyJCpqry0'), ('dates', '2020-04-16 22:03:42'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.6884089112281799)])
OrderedDict([('tweet', 'In the last 5 hours, over 25,000 #PPPloans went unprocessed because @SpeakerPelosi &amp; @SenSchumer are blocking more funding. We are fighting to save jobs and small businesses. What have they fought for? \n\n... $34 million for the Kennedy Center.\n\nhttps://t.co/jWyJCpqry0'), ('dates', '2020-04-16 20:03:04'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.6124687194824219)])
OrderedDict([('tweet', 'In the last 4 hours, over 20,000 #PPPloans went unprocessed because @SpeakerPelosi &amp; @SenSchumer are blocking more funding. We are fighting to save jobs &amp; small businesses. What have they fought for? \n\n... Prohibiting states from requiring an ID to vote. \n\nhttps://t.co/jWyJCpqry0'), ('dates', '2020-04-16 19:01:37'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.6853638291358948)])
OrderedDict([('tweet', 'In the last three hours, over 15,000 #PPPloans went unprocessed because @SpeakerPelosi &amp; @SenSchumer are blocking more funding. \n\nWe are fighting to save jobs and small businesses. What have they fought for during this pandemic?\n\n...$300 million for PBS\n\nhttps://t.co/9DdXIH43cg'), ('dates', '2020-04-16 18:21:56'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.6187753081321716)])
OrderedDict([('tweet', 'In the last two hours, over 10,000 #PPPloans went unprocessed because @SpeakerPelosi &amp; @SenSchumer are blocking more funding. \n\nWe are fighting to save jobs and small businesses. What have they fought for during this pandemic?\n\n...carbon offsets\n\nhttps://t.co/9DdXIH43cg'), ('dates', '2020-04-16 17:13:54'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.5754110217094421)])
OrderedDict([('tweet', 'It is unconscionable that the Democratic leadership continues to block funds our small businesses desperately need as a means to fulfill their political agenda. \n\n@SpeakerPelosi &amp; @SenSchumer - a national crisis is no time to play politics. Stop the games &amp; let us fund the #PPP https://t.co/Q52lpsIFzF'), ('dates', '2020-04-16 15:27:03'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.8875951766967773)])
OrderedDict([('tweet', "It's an undisputed fact that the WHO parroted Chinese misinformation on COVID-19. For an org that so many nations depend upon to put its relationship with China above the health &amp; safety of billions is unconscionable. @realDonaldTrump is right to cut funds https://t.co/KSCqOx8BbM"), ('dates', '2020-04-15 19:32:20'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.8671754002571106)])
OrderedDict([('tweet', 'The Paycheck Protection Program has brought nearly $4 BILLION into La., saving thousands of small businesses and even more jobs. The PPP is working but Congress must act to help those businesses still waiting in line for help. I urge Dem leadership to stop blocking these funds.'), ('dates', '2020-04-15 18:07:25'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.6483155488967896)])
OrderedDict([('tweet', "#PPP is already saving Louisiana businesses &amp; many others are waiting in line for loans. It's unconscionable that the Democratic leadership would block additional funding just to fulfill their political agenda.\n\n@SpeakerPelosi a national crisis is NOT the time to play politics."), ('dates', '2020-04-14 19:14:13'), ('file_name', 'Mike Johnson.csv'), ('pct_true', 0.8928772211074829)])
OrderedDict([('tweet', 'Rahm Emanuel once said “never allow a crisis to go to waste.”  Chuck Schumer &amp; Nancy Pelosi are following that advice as they obstruct relief for workers &amp; businesses hurt by coronavirus shutdowns. Why? To ram through a series of progressive pet projects!\n\nhttps://t.co/QkWpWAIbr1'), ('dates', '2020-03-24 13:22:38'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.7582666873931885)])
OrderedDict([('tweet', 'America is the most prepared nation on earth to confront #coronavirus and the threat to the general public remains low. Congress must act to keep it that way.\n\nDemocrats should stop playing politics with funding that will help state &amp; local governments keep our communities safe. https://t.co/zLQcyyfPJS'), ('dates', '2020-03-04 16:35:42'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.7174330949783325)])
OrderedDict([('tweet', 'Communist regimes have inflicted misery and death on millions of people. This ideology has no redeeming quality and should be resolutely condemned. \n\nHouse Democrats just sided with Bernie Sanders and Fidel Castro over freedom-starved Cubans. https://t.co/GPXkybYsX3'), ('dates', '2020-02-27 22:03:11'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.9538917541503906)])
OrderedDict([('tweet', "Medicare for All is really Medicare for None. It would destroy the program seniors depend on &amp; limit healthcare options for everyone.\n\nAmericans want us to build on what works in our healthcare system &amp; fix what doesn't, not replace their plans &amp; providers with D.C. bureaucrats. https://t.co/G2vikxNwIR"), ('dates', '2020-02-27 17:13:54'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.7181926369667053)])
OrderedDict([('tweet', "The Tax Cuts &amp; Jobs Act lowered tax rates so job creators would return to America after decades of fleeing to more tax-friendly shores. That's exactly what is happening.\n\nDemocrats say these businesses don't pay enough in taxes, but they won't pay anything if they aren't here!"), ('dates', '2020-02-11 18:08:21'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.8658854365348816)])
OrderedDict([('tweet', "The Senate acquittal of @POTUS @realDonaldTrump means the disgraceful abuse of Congress' impeachment power by House Democrats is over. I call on them to heal the wounds their reckless behavior has caused by working with us on the priorities of the American people going forward."), ('dates', '2020-02-05 21:33:38'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.9222269654273987)])
OrderedDict([('tweet', "Sad that Speaker Pelosi ripped up @POTUS @realDonaldTrump's #SOTU2020 speech, which offered a bold policy vision, touted our economy, honored our nation's heroes, &amp; more.\n\nWhy do this? Because she won't accept that policies her party has long-derided are making America strong."), ('dates', '2020-02-05 16:00:01'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.9357219934463501)])
OrderedDict([('tweet', '“The United States of America should be a sanctuary for law-abiding Americans, not criminal aliens.” -@POTUS\n\nI support this president’s fight to secure America’s borders and keep Americans safe. #SOTU2020'), ('dates', '2020-02-05 03:06:23'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.737125813961029)])
OrderedDict([('tweet', 'This has been the most productive week in trade policy in decades. Yesterday, @POTUS @realDonaldTrump signed the Phase One trade agreement with China and today the Senate will pass USMCA.\n\nProud to stand behind this president as he fights for American workers!'), ('dates', '2020-01-16 14:35:39'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.5997704863548279)])
OrderedDict([('tweet', "When President Obama ordered the elimination of Osama bin Laden, Republicans cheered his decision. Last night, Democrats showed more concern over the 2020 election than the loss of American lives. @POTUS @Realdonaldtrump was right to eliminate one of the world's top terrorists. https://t.co/xYBoBGG9gA"), ('dates', '2020-01-10 14:57:17'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.7190927863121033)])
OrderedDict([('tweet', ".@POTUS @realDonaldTrump was right to eliminate Qasem Soleimani, one of one of the world's most dangerous terrorists. Sadly, House Democrats have turned this win for America into a political spectacle. I voted against their partisan war powers resolution tonight. https://t.co/XJcuYPuEOg"), ('dates', '2020-01-09 23:30:00'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.9113783836364746)])
OrderedDict([('tweet', "After years of aggression and the last administration's appeasement policy, America is confronting the world’s largest state sponsor of terror under @POTUS @realDonaldTrump’s leadership. Iran’s attacks on Americans and our allies will not be tolerated."), ('dates', '2020-01-08 21:52:02'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.5679318904876709)])
OrderedDict([('tweet', "Last night, Democrats impeached @POTUS @realDonaldTrump. Today, the House passes a trade deal that's one of his top priorities. What a difference a day makes.\n\n#USMCA is a big win for America, a Christmas present for workers &amp; farmers wrapped in red, white, &amp; blue wrapping paper!"), ('dates', '2019-12-19 17:06:38'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.5588489174842834)])
OrderedDict([('tweet', 'Irony: Dems accuse @realDonaldTrump of abuse of power while blatantly abusing theirs w/ a partisan impeachment of an elected president. Their self-righteous references to oaths &amp; the Constitution obscure the obvious intent: political vengeance.\n\nI voted against both articles.'), ('dates', '2019-12-19 01:53:13'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.9590928554534912)])
OrderedDict([('tweet', '.@POTUS to @SpeakerPelosi: You are the ones interfering in America’s elections, subverting America’s democracy, &amp; obstructing justice. You are the ones bringing pain &amp; suffering to our Republic for your own selfish, personal, political, &amp; partisan gain.\n\nFact check: True!'), ('dates', '2019-12-18 14:23:59'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.8519448637962341)])
OrderedDict([('tweet', "Another promise made and kept despite the Do-Nothing-Democrats' impeachment charade. @POTUS @realDonaldTrump and @USTradeRep are delivering for American workers, farmers, and businesses. This new trade deal w/ China is another great win for America. 🇺🇸\n\nhttps://t.co/igfWr6pmQa"), ('dates', '2019-12-13 20:44:29'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.718304455280304)])
OrderedDict([('tweet', 'Democrats have made it crystal clear all along that the drive for impeachment is about political vengeance &amp; damaging @POTUS @realDonaldTrump ahead of the 2020 election.\n\nI will vote against both articles of impeachment on the House floor.'), ('dates', '2019-12-13 15:21:48'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.9118577241897583)])
OrderedDict([('tweet', 'The impeachment screenplay is getting bad reviews, but Speaker Pelosi insists the show must go on even though it meets none of her own criteria.\n\nDemocrats were always going to do this. They wrote the final scene the day @POTUS @realDonaldTrump was elected. https://t.co/0MioTRxuOq'), ('dates', '2019-12-09 17:45:27'), ('file_name', 'Mike Kelly.csv'), ('pct_true', 0.8268471360206604)])
OrderedDict([('tweet', 'What happened to Gen. Flynn was a travesty. This is why we need real FISA reform when the Senate debates reauthorization next week. https://t.co/TJqdabM8ij'), ('dates', '2020-05-07 20:11:50'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.5299298763275146)])
OrderedDict([('tweet', 'It is black letter law that a government may not single out religious activity for adverse treatment by government. That’s what Louisville did, and Judge Walker was right to overturn the city’s ban. He’ll make a great judge on the DC Circuit. https://t.co/SEUdUjpWZG'), ('dates', '2020-05-06 15:43:20'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.6913639903068542)])
OrderedDict([('tweet', '"we are supportive of funding to respond to the Coronavirus and protect our frontline workers, but we cannot allow states &amp; localities to get a blank check from American taxpayers to fund areas of their budget that have nothing to do with the Coronavirus." https://t.co/MqBqYOZZiQ'), ('dates', '2020-04-30 19:38:11'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.5839365124702454)])
OrderedDict([('tweet', 'I don’t know a member of Congress more principled or dedicated to the Constitution than @RepThomasMassie.  @SpeakerPelosi delayed this process repeatedly and sent the House into recess before its job was complete.  Big mistake!'), ('dates', '2020-03-27 19:35:11'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.5806335210800171)])
OrderedDict([('tweet', 'The Coronavirus is a real emergency and the federal government can and should act to mitigate the economic pain from this situation. Unfortunately, this House response bill is Rube Goldberg machine of unfunded mandates and tax benefits that will only end up hurting workers.'), ('dates', '2020-03-18 20:30:59'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.5702896118164062)])
OrderedDict([('tweet', 'We all agree workers affected by the Coronavirus need financial assistance. The only question is how to finance it. The House bill forces businesses to front the money. Instead our amendment would give workers cash directly from govt.\nhttps://t.co/xzg0oZlAaI'), ('dates', '2020-03-18 17:26:24'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.5869541168212891)])
OrderedDict([('tweet', 'The coronavirus threat is bigger than politics. But so are the American people. Over the next few weeks, we’re going to prove it. That’s how we’re going to beat this thing. Together.\nhttps://t.co/4nQMqLsqR2'), ('dates', '2020-03-15 17:11:39'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.6460193395614624)])
OrderedDict([('tweet', 'Congress must rein in FISA surveillance in the wake of abuses | Washington Examiner. My friend @tolmanbrett argues effectively for reform of FISA and the PATRIOT Act.\nhttps://t.co/scaWWgH2tw'), ('dates', '2020-03-11 16:26:21'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.7186816930770874)])
OrderedDict([('tweet', 'Congress must rein in FISA surveillance in the wake of abuses | Washington Examiner.                                       My friend Brett Tolman argues effectively for reform of FISA and the PATRIOT Act. https://t.co/scaWWgH2tw'), ('dates', '2020-03-11 15:18:36'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.7264474034309387)])
OrderedDict([('tweet', 'Even where the House FISA / PATRIOT Act deal claims to offer modest protection for political candidates and elected officials, it doesn’t provide the same protection for the American people!  @realDonaldTrump has asked for real FISA reforms, not fake ones. Sad!'), ('dates', '2020-03-11 01:58:09'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.7651583552360535)])
OrderedDict([('tweet', 'The #deepstate spied on @realDonaldTrump. PATRIOT Act §215 makes it easier for the feds to spy on Americans. FISA should be used to stop foreign terrorists, not spy on U.S. citizens. Absent major FISA reform, we should let 215 expire this week! #Veto215'), ('dates', '2020-03-09 19:10:35'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.7174782752990723)])
OrderedDict([('tweet', 'The FBI improperly spied on the Trump campaign in 2016. We must ensure the bureau can’t do it in 2020 or ever again. Because if the FBI can unfairly target a presidential campaign, imagine what it can do to regular Americans.\nhttps://t.co/XHmCnKWoBm'), ('dates', '2020-03-03 14:39:32'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.6704742908477783)])
OrderedDict([('tweet', 'Three provisions of FISA are set to expire on March 15th. Congress should not reauthorize them without substantially reforming FISA. Our intelligence-gathering agencies need to focus on foreign agents and terrorists, and should not be spying on U.S. citizens. Do you agree?'), ('dates', '2020-02-26 19:41:34'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.6546344757080078)])
OrderedDict([('tweet', 'Would President Bush have been subject to impeachment charges based on his handling of NSA surveillance, detention of combatants, or use of waterboarding? https://t.co/L7FQibZPDR'), ('dates', '2020-01-30 23:47:58'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.5350189208984375)])
OrderedDict([('tweet', 'Under the standard embraced by the House managers, would President Obama have been subject to impeachment charges based on his handling of the Benghazi attack, the Bergdahl swap, or DACA? https://t.co/PlfvH8HNxg'), ('dates', '2020-01-30 23:47:27'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.5540807247161865)])
OrderedDict([('tweet', 'The House Managers have argued aggressively that the president’s actions contravened U.S. foreign policy. Isn’t it the president’s place – certainly more than the place of career civil servants – to conduct foreign policy? https://t.co/U1Iu0ITTKi'), ('dates', '2020-01-29 19:46:50'), ('file_name', 'Mike Lee.csv'), ('pct_true', 0.5612435340881348)])
OrderedDict([('tweet', 'This is outrageous, the @USPS is an essential service that we can’t afford to lose. \n\nInstead of offering severely needed financial assistance, President Trump is threatening to veto any bill that will give them relief.\n\nhttps://t.co/nQQbFxPK8A'), ('dates', '2020-04-28 14:53:26'), ('file_name', 'Mike Levin.csv'), ('pct_true', 0.7084571719169617)])
OrderedDict([('tweet', "10 years ago, the largest offshore oil disaster in world history dumped 5 million barrels of oil into the Gulf of Mexico. 11 people died.\n\nTrump's pro-polluter agenda = unsafe working conditions &amp; fewer environmental protections. We must #ProtectOurCoast, not Big Oil's profits. https://t.co/J5ypZNaUFn"), ('dates', '2020-04-20 22:42:22'), ('file_name', 'Mike Levin.csv'), ('pct_true', 0.7517968416213989)])
OrderedDict([('tweet', 'Weakening regulations on the release of mercury and other toxic metals is incredibly dangerous. \n\nThe Trump Admin is once again proving they favor the fossil fuel industry over Americans’ health. #EnvironmentalPollutionAgency\n\nhttps://t.co/RN0OBuqmwG'), ('dates', '2020-04-16 21:43:23'), ('file_name', 'Mike Levin.csv'), ('pct_true', 0.6656774878501892)])
OrderedDict([('tweet', "While the U.S. faces its worst crisis in modern history, the Trump Administration is still attacking marginalized communities. This week they plan to end protections for the #LGBTQ community. As a member of the @LGBTEqCaucus, I'm working to ensure that everyone means everyone."), ('dates', '2020-04-27 22:16:32'), ('file_name', 'Mike Quigley.csv'), ('pct_true', 0.5364670753479004)])
OrderedDict([('tweet', "President Trump is looking for anyone to blame for this crisis other than himself. The reality is that the President and the President alone is responsible for his White House's failed response. https://t.co/GS45RbGdls"), ('dates', '2020-04-21 17:07:00'), ('file_name', 'Mike Quigley.csv'), ('pct_true', 0.586889922618866)])
OrderedDict([('tweet', 'The #DeepwaterHorizon oil spill could have been prevented if Big Oil had chosen to #ProtectOurCoast rather than its own profits. \n\nThis week marks 10 years after this disaster and Trump just keeps rolling back protections to help the industry, not you and your family. https://t.co/rSRManzmyg'), ('dates', '2020-04-21 13:59:47'), ('file_name', 'Mike Quigley.csv'), ('pct_true', 0.7407923340797424)])
OrderedDict([('tweet', "Governor Pritzker's agreement with China is a great example of why we need a strong and centralized public health system. Governors should not have to compete with each other or other countries for PPE or masks. We should be working together to combat COVID-19. https://t.co/XSWYXEpjUS"), ('dates', '2020-04-16 20:15:00'), ('file_name', 'Mike Quigley.csv'), ('pct_true', 0.5339459180831909)])
OrderedDict([('tweet', "The President wants anyone responsible for holding him accountable to be someone he can count on to be loyal. He's left the door wide open for ongoing corruption in his own administration. We need an independent Inspector General to allocate the $2 trillion stimulus efficiently. https://t.co/Eu9FQRubkk"), ('dates', '2020-04-10 18:59:00'), ('file_name', 'Mike Quigley.csv'), ('pct_true', 0.7352562546730042)])
OrderedDict([('tweet', "As the coronavirus pandemic continues, my colleagues and I fear that the EPA is hastily putting forward rules that the public cannot digest and comment on in a timely manner. I, along with @RepMcNerney and more are urging the EPA to prioritize the public's comment period. https://t.co/NOBgLxetkz"), ('dates', '2020-04-08 23:58:00'), ('file_name', 'Mike Quigley.csv'), ('pct_true', 0.7355315685272217)])
OrderedDict([('tweet', "The EPA has a responsibility to hold companies accountable for their pollution. Suspending monitoring of pollution by companies will damage the progress we've made in protecting our environment. The EPA should reimplement monitoring immediately. https://t.co/wOs65M9Qwu"), ('dates', '2020-04-07 23:45:00'), ('file_name', 'Mike Quigley.csv'), ('pct_true', 0.852574348449707)])
OrderedDict([('tweet', "Between all the twists and turns of #TigerKing are reminders of how dangerous big cats are. If even one had escaped, first responders could have faced a horrifying situation. Passing the #BigCactPublicSafetyAct isn't just about saving the animals, it's about saving people too. https://t.co/KPnqG1iKMz"), ('dates', '2020-04-06 18:45:00'), ('file_name', 'Mike Quigley.csv'), ('pct_true', 0.5328977108001709)])
OrderedDict([('tweet', 'This action should’ve been taken over 2 wks ago when @realDonaldTrump requested more money for PPP, or it could’ve happened last wk when the program ran out of money. Instead, Speaker Pelosi wasted time with her ridiculous partisan political games.  #AL03\nhttps://t.co/IK8FvCP3EZ'), ('dates', '2020-04-23 23:01:19'), ('file_name', 'Mike Rogers.csv'), ('pct_true', 0.8845188617706299)])
OrderedDict([('tweet', "I voted against the Democrats' Select Committee just now because it is part of the Democrats' Trump Derangement Syndrome Checklist:  \n\nRussia, Russia, Russia ✔️\n\nMueller Witch Hunt ✔️\n\nSham Impeachment ✔️\n\nPelosi’s Select Committee ✔️"), ('dates', '2020-04-23 20:59:47'), ('file_name', 'Mike Rogers.csv'), ('pct_true', 0.8400598168373108)])
OrderedDict([('tweet', 'Nancy Pelosi &amp; Chuck Schumer continue to block the replenishment of the Paycheck Protection Program (PPP) of the CARES Act for solely selfish partisan political reasons.'), ('dates', '2020-04-13 23:31:38'), ('file_name', 'Mike Rogers.csv'), ('pct_true', 0.8442608714103699)])
OrderedDict([('tweet', 'The Paycheck Protection Plan has provided a safety net for our small businesses during these unprecedented times and I was extremely disappointed to see Senate Democrats block a unanimous consent request to provide standalone emergency funding for this critical program.'), ('dates', '2020-04-09 18:08:37'), ('file_name', 'Mike Rogers.csv'), ('pct_true', 0.6189268827438354)])
OrderedDict([('tweet', "Hey #AL03, I know you are waiting for Washington to act to help American families, but Speaker Pelosi won't let us do anything until her demands are met like..."), ('dates', '2020-03-23 21:48:41'), ('file_name', 'Mike Rogers.csv'), ('pct_true', 0.5598546862602234)])
OrderedDict([('tweet', 'Today, House Democrats once again refused to allow a vote to protect the most innocent newborn babies for sickening abortionists.  Why does Speaker Pelosi continue to block the #BornAlive Act? Their pro-abortion Socialist agenda is disgusting.'), ('dates', '2020-02-28 17:41:32'), ('file_name', 'Mike Rogers.csv'), ('pct_true', 0.9796115756034851)])
OrderedDict([('tweet', "The actions taken by China’s government to cover-up the coronavirus and its spread aren’t just negligence – they are criminal in nature. China must be held accountable. Read more in this week's column: https://t.co/bX1nzL3ZLk"), ('dates', '2020-05-08 13:23:22'), ('file_name', 'Mike Rounds.csv'), ('pct_true', 0.8691443800926208)])
OrderedDict([('tweet', "China's cover-up of the COVID-19 crisis isn't just negligence, it's criminal in nature. We must hold China accountable. https://t.co/5ptGrIHfYi"), ('dates', '2020-05-07 14:19:58'), ('file_name', 'Mike Rounds.csv'), ('pct_true', 0.9326947331428528)])
OrderedDict([('tweet', "This isn't right. Leaving these types of small businesses out of important COVID-19 recovery efforts would be crippling for SD. I'm urging the SBA and Department of Treasury to allow gaming operations to participate in the newly-created Paycheck Protection Program. https://t.co/dX9pctaz5z"), ('dates', '2020-04-04 21:47:39'), ('file_name', 'Mike Rounds.csv'), ('pct_true', 0.7896341681480408)])
OrderedDict([('tweet', 'The federal government has a responsibility to act in emergency situations, such as this, to protect lives and safeguard our country from undue harm. Tonight, the Senate fulfilled this important duty.'), ('dates', '2020-03-26 03:50:14'), ('file_name', 'Mike Rounds.csv'), ('pct_true', 0.8654021620750427)])
OrderedDict([('tweet', 'You’re either helping right now, or hurting. Pelosi’s handpicked partisan ploys are helping no one. She’s hurting American families by prioritizing airplane emissions, same day voter registration &amp; wind energy tax credits over #coronavirus relief. #StopTheGamesNancy, #PassTheBill'), ('dates', '2020-03-24 14:00:21'), ('file_name', 'Mike Rounds.csv'), ('pct_true', 0.9741449952125549)])
OrderedDict([('tweet', 'American families and businesses are suffering. Now is not the time to make liberal demands that have nothing to do with addressing this urgent crisis. https://t.co/Or3y5QQSUW'), ('dates', '2020-03-23 15:41:30'), ('file_name', 'Mike Rounds.csv'), ('pct_true', 0.773232102394104)])
OrderedDict([('tweet', '“Businesses who have purchased business interruption insurance in good faith and are forced to close their doors because of a major event and through no fault of their own deserve to have their claims honored.” Read more in @NapaRegister. https://t.co/FeK5UAvG0Q'), ('dates', '2020-04-15 23:00:13'), ('file_name', 'Mike Thompson.csv'), ('pct_true', 0.6407190561294556)])
OrderedDict([('tweet', 'I coauthored @RepGraceMeng’s resolution condemning anti-Asian sentiment related to Coronavirus. Such hateful rhetoric endangers the lives of our Asian-American neighbors and friends and I was proud to stand strong against it. Read the resolution here: https://t.co/34x2k5SCxQ'), ('dates', '2020-03-30 21:00:00'), ('file_name', 'Mike Thompson.csv'), ('pct_true', 0.8156057596206665)])
OrderedDict([('tweet', '.@SenateMajLdr McConnell: This Pandemic Came From China and the Chinese Communist Party’s Tactics Have Made It Worse \nhttps://t.co/Tn9M3KbKQK'), ('dates', '2020-05-07 15:54:25'), ('file_name', 'Mitch McConnell.csv'), ('pct_true', 0.6842550039291382)])
OrderedDict([('tweet', '.@senatemajldr McConnell: The core business of our nation does not stop while we fight the coronavirus. This week, Senate committees are conducting oversight and holding hearings. (1/2)'), ('dates', '2020-05-06 15:09:57'), ('file_name', 'Mitch McConnell.csv'), ('pct_true', 0.5929610133171082)])
OrderedDict([('tweet', "China’s main objective: become the global superpower militarily, economically, and geopolitically. To stop the spread of China's authoritarian reach, free nations must link arms and offer a choice: Abide by the global order or lose access to our markets. https://t.co/YUB1XdrrpO"), ('dates', '2020-04-29 15:04:49'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.7447757124900818)])
OrderedDict([('tweet', 'The #COVID19 pandemic originated in China, and through cover-up efforts, the virus became global. Attitudes toward China have changed by virtue of this pandemic. America must partner with likeminded nations to counter China’s objectives and aggression. https://t.co/YUB1XdrrpO'), ('dates', '2020-04-28 16:15:13'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.9224907755851746)])
OrderedDict([('tweet', '.@WHO appears to have shown remarkable deference to the Chinese government throughout the #COVID19 global pandemic. The @UN must immediately convene an independent panel of experts to review and assess the WHO’s pandemic response, and recommend reforms. https://t.co/YntgdUjzUh'), ('dates', '2020-04-24 19:58:18'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.6494439840316772)])
OrderedDict([('tweet', '.@CDCgov has put significant effort toward data collection and sharing in the midst of the COVID-19 pandemic. But, despite having a world-class public health system, the lack of comprehensive, real-time #COVID19 data has hindered our ability to monitor and mitigate its spread.'), ('dates', '2020-04-23 21:16:03'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.5744157433509827)])
OrderedDict([('tweet', "#COVID19 has exposed China's dishonesty for the world to see. America must lead like-minded nations in developing a unified strategy to counter China’s predation and corruption. https://t.co/gJchtn4Ioa"), ('dates', '2020-04-23 20:45:27'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.8055379986763)])
OrderedDict([('tweet', 'The Social Security and Medicare Trustees annual report released today outlines a disturbing reality—Medicare will become insolvent in the next 6 years and Social Security will be in the same boat by 2035. All of this before the impacts of #COVID19 are even assessed. https://t.co/aPNLOgPKmp'), ('dates', '2020-04-22 20:00:36'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.8259741067886353)])
OrderedDict([('tweet', 'It is well-documented that wet markets in China have been the source of a number of worldwide health problems. We are urging China to immediately cease the operation of these markets to protect the international community from additional health risks. https://t.co/qWJniOYHRw'), ('dates', '2020-04-09 22:32:50'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.6847690343856812)])
OrderedDict([('tweet', "The Chinese Communist Party is spreading dangerous &amp; dishonest claims that the US is responsible for the #COVID19 pandemic—a pandemic that originated due to its own incompetence. We're asking for the creation of a NSC task force to counter this propaganda. https://t.co/R6M9lQdruY"), ('dates', '2020-03-23 22:49:09'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.9524295330047607)])
OrderedDict([('tweet', 'It’s unacceptable that adversaries like Russia and China have more assets than we do in the Arctic. I urged @DHSgov Acting Secretary Wolf to prioritize providing our @USCG with increased security equipment, such as polar icebreakers. https://t.co/ZXk5jWz83X'), ('dates', '2020-03-06 18:26:44'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.5125290751457214)])
OrderedDict([('tweet', "China uses its Belt and Road Initiative to invest in global mineral production, leaving developing countries beholden to the CCP. My Energy Bill amendment would rein in China's economic aggression by establishing a mechanism to monitor these investments. https://t.co/uUFEyLW57g"), ('dates', '2020-03-05 17:29:36'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.7786852717399597)])
OrderedDict([('tweet', 'It’s been 20 days since we raised this exact concern to @FDATobacco Director Zeller. Flavored e-cigarettes are still on the market, addicting our kids to nicotine. The @US_FDA must put the health of our children ahead of tobacco companies’ profits and eliminate vaping flavors. https://t.co/ZyJIC0xQ4g'), ('dates', '2020-03-04 21:41:57'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.5219258069992065)])
OrderedDict([('tweet', 'Our government must partner with pharmaceutical companies to produce a large number of #Coronavirus vaccines once a viable vaccine is found. We can’t rely on countries like China for the manufacturing of pharmaceutical products that are critical to our national security. https://t.co/a6cCVCrGJU'), ('dates', '2020-03-03 21:05:32'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.6037399768829346)])
OrderedDict([('tweet', 'An authoritarian China is perhaps the single greatest threat to freedom around the world. Tune in as I discuss the importance of linking arms with our allies to confront growing Chinese aggression with @AndersFoghR at the @UofDenver’s @JosefKorbel School. https://t.co/3YSLVdGz0c'), ('dates', '2020-02-29 01:05:59'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.5924745798110962)])
OrderedDict([('tweet', 'China engages in cowboy capitalism—attacking the economies of free nations through predatory pricing, cyber theft, forced technology transfer, and more. I spoke at @MunSecConf about the threat China poses to international free trade. https://t.co/iJ481aTKHe'), ('dates', '2020-02-21 18:55:55'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.8546733856201172)])
OrderedDict([('tweet', "The Administration must unequivocally communicate its posture toward Libya on the global stage, at a time when Libya's civil war poses a threat to US interests. Clarifying the US-Libya strategy, including defining who our friends are, will help protect our regional interests. https://t.co/ZvZSGK71Gh"), ('dates', '2020-02-12 18:38:10'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.6798329949378967)])
OrderedDict([('tweet', 'In the midst of the Coronavirus, Taiwan’s exclusion from the @WHO greatly inhibits the ability for a coordinated, global response. This is yet another ploy by the CCP to isolate Taiwan from the global community.'), ('dates', '2020-01-31 15:49:43'), ('file_name', 'Mitt Romney.csv'), ('pct_true', 0.9244542717933655)])
OrderedDict([('tweet', '.@Revelare: I agree 100%. I have publicly encouraged Gov. Ivey to kill her shutdown orders. She does not heed my advice. Notwithstanding our disagreement, Governor Ivey’s new #shutdown order is not as bad as her prior shutdown order. Not good. Just not as bad.\n\n#ReOpenAmerica https://t.co/CsGuaJDog7'), ('dates', '2020-05-08 16:41:24'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.5021503567695618)])
OrderedDict([('tweet', 'https://t.co/HytYVdeo0c\n\n#SanFrancisco #Socialist #Democrats #Coronavirus response: give away FREE drugs &amp; alcohol. These “leaders” also let #IllegalAliens LAWFULLY vote in elections.\n\nElsewhere in America, you are jailed for returning to work.\n\nHas the world turned upside down?'), ('dates', '2020-05-08 16:28:47'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.7847892642021179)])
OrderedDict([('tweet', 'https://t.co/uoj5ha1Fn8\n\n#CNN #MSNBC &amp; other #FakeNews media won’t air this story. It’s not in #Socialist #Democrat campaign agenda.\n\n#Covid_19 hasn’t made USA border security problem go away.\n\n#IllegalAliens kill 2,000+ Americans each year... &amp; infect officers with #Coronavirus.'), ('dates', '2020-05-08 15:04:36'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.9020606875419617)])
OrderedDict([('tweet', 'Bob. Why do I comment? Many reasons. 2 stand out:\n\n#AndrewCuomo &amp; #NewYork have worst #coronavirus death rate in USA. Cuomo must be held accountable for deadly mismanagement.\n\nCuomo wants America bail outs for decades of NY financial mismanagement.\n\nTruth reduces bailout risks. https://t.co/XzvoAbOQrM'), ('dates', '2020-05-08 13:58:42'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.790809154510498)])
OrderedDict([('tweet', '.@realDonaldTrump is right, America must get back to work. The government should act as a partner and advisor, but defer to citizens the liberty and freedom of making their own decisions. https://t.co/hbQ81LDE6n'), ('dates', '2020-05-05 20:38:26'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.5372223258018494)])
OrderedDict([('tweet', 'https://t.co/xtbKWTJ1NK\n\n.@AOC for rent strike. The result?\n\nApartments are not maintained &amp; fall into bad disrepair. Condemnations result. Property taxes not paid, hurting local govts. People lose places to rent &amp; live in.\n\nDUMB!\n\n#AOC= #Socialist #Democrat leader = Dangerous!'), ('dates', '2020-05-01 11:33:14'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.8924453854560852)])
OrderedDict([('tweet', 'https://t.co/9gyIG45dzr\n\n.@SpeakerPelosi seeks to BORROW ANOTHER $1 TRILLION for states &amp; cities that shut themselves down!\n\nCongress already appropriated #Pandemic $$ to states: $11 billion to Alabama for #Covid19...more than Alabama’s education &amp; general fund budgets COMBINED!'), ('dates', '2020-05-01 02:55:51'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.5274004340171814)])
OrderedDict([('tweet', 'https://t.co/tIR2BCwC32\n\n#Socialist #Democrat #Congressmen support #proxy voting by a 20 to 1 margin.\n\nDems give #SpeakerPelosi their proxy, who then casts their votes. Dems GET PAID IN FULL without showing up for work at Capitol Hill.\n\nSAD!\n\nCOWARDLY! \n\nAMERICA - FIGHT BACK!!'), ('dates', '2020-04-28 04:05:47'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.8705065846443176)])
OrderedDict([('tweet', "Tennessee Valley professionals aren't letting COVID-19 stand in the way of completing @NASA_SLS— the most powerful rocket ever. Progress continues despite the challenges of telework on this rocket that will eventually transport astronauts to the Moon &amp; Mars. https://t.co/WfnyOXVqFU"), ('dates', '2020-04-14 15:44:56'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.5120868682861328)])
OrderedDict([('tweet', 'NAZI revisionists like fambonijones are disgusting!\n\nNAZI is short for “National SOCIALIST German Workers’ Party”.\n\nThe NAZI party platform from 1920 &amp; after advocated socialist policies. NAZI socialism differed only mildly from China &amp; USSR.\n\nSOCIALISM KILLS LIBERTY &amp; PEOPLE! https://t.co/UogHvTZkSn https://t.co/ZSQrHIAHN3'), ('dates', '2020-04-04 16:10:04'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.9859578013420105)])
OrderedDict([('tweet', 'Today I voted against HR 6172/FISA reauthorization. FISA used to illegally &amp; secretly spy on Americans for partisan political gain.\n\nFiring ALL FISA personnel sends message even SWAMP understands. THEN create replacement to protect Americans from terrorists &amp; OUR OWN GOVERNMENT! https://t.co/sg3SwSH466'), ('dates', '2020-03-11 21:02:17'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.7243597507476807)])
OrderedDict([('tweet', '#coronavirus/#COVID2019 causes #Congress to consider recessing. https://t.co/YvLwm7FdOT\n\nYes, COVID-19 is highly contagious.  But, unless you are old or have immune deficiency, the threat is VERY low.\n\nCongress must stand strong as America’s example not cut &amp; run like snowflakes!'), ('dates', '2020-03-09 16:08:00'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.5091832876205444)])
OrderedDict([('tweet', 'H.R. 2474 HOUSE VOTE (1) denies Americans their secret ballot vote right on unionizing &amp; (2) forces American workers to pay union dues or be fired.\n\nH.R. 2474 PASSES House 224-194. Sadly, Dictatorial Socialism beats American freedom.\n\nUnlike Dems, I VOTED WITH the American flag! https://t.co/hwTMpi4UFd'), ('dates', '2020-02-07 02:13:06'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.5296664834022522)])
OrderedDict([('tweet', 'Socialist Dems push H.R. 2474 to (1) deny Americans their right to a secret ballot vote on unionizing &amp; (2) force American workers to pay union dues or be fired.\n\nThe battle lines are clear: Dictatorial Socialism or American freedom.\n\nUnlike Dems, I vote with the American flag! https://t.co/9Hk80ZARhB'), ('dates', '2020-02-07 01:05:32'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.8735576272010803)])
OrderedDict([('tweet', '#MittRomney “convict @realDonaldTrump” logic is backwards.\nRomney: Biden acts are “ugly” but “No crime was alleged by the defense”. Requiring 100% proof BEFORE investigating is silly. Investigation purpose: find “acquit or convict” proof. If already have proof, why investigate? https://t.co/WEhFGyoSxK'), ('dates', '2020-02-05 20:11:59'), ('file_name', 'Mo Brooks.csv'), ('pct_true', 0.7192906737327576)])
OrderedDict([('tweet', 'The Trump Administration must not take away health care from tens of millions of Americans in the middle of a pandemic. The President should end his lawsuit seeking to dismantle the ACA and the crucial pre-existing condition protections it includes. https://t.co/cPY2S8B0bG'), ('dates', '2020-05-06 15:15:14'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.8342925310134888)])
OrderedDict([('tweet', 'Where there’s money, there’s mischief. Congress will not back away from our responsibility to ensure the resources we have made available are being used properly as our nation confronts this public health crisis. #FamiliesFirst @NewDay https://t.co/PJ3GonMRQY'), ('dates', '2020-04-30 19:32:30'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.7482718825340271)])
OrderedDict([('tweet', 'Congressional Democrats are fighting to put #FamiliesFirst – and those values will guide us as we move forward. #CNNSOTU https://t.co/x2iwMXMlYi'), ('dates', '2020-04-26 15:22:11'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.5340645909309387)])
OrderedDict([('tweet', '.@HouseDemocrats are committed to fixing the flaws in the Paycheck Protection Program that have kept too many small businesses from gaining access. #FamiliesFirst @ThisWeekABC https://t.co/hBS5hg2N95'), ('dates', '2020-04-19 21:44:09'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.5761678218841553)])
OrderedDict([('tweet', 'Leaders take responsibility. The President must stop attempting to shift blame and start making the science-based decisions we need to confront this crisis. #FamiliesFirst #FNS https://t.co/h06ZnT4vG7'), ('dates', '2020-04-19 20:50:48'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.6985626220703125)])
OrderedDict([('tweet', 'The coronavirus is moving swiftly, and our communities cannot afford for us to wait. House Democrats will continue to work relentlessly and in a bipartisan way to lift up American families and workers. #FamiliesFirst https://t.co/Uy3JyoySqn'), ('dates', '2020-04-03 17:09:13'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.5377635955810547)])
OrderedDict([('tweet', 'We need to be doing more to ensure Americans have the health care they need. Instead, the President is continuing his mission to steal coverage from millions &amp; tear protections away from the more than 130 million Americans living with pre-existing conditions. #AMR #FamiliesFirst https://t.co/ragUalVgHP'), ('dates', '2020-04-02 23:31:54'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.7173306345939636)])
OrderedDict([('tweet', 'While the President has made clear he will ignore oversight provisions in the #CARESAct, Acting IG Fine must exercise his full oversight authority to ensure that the Admin uses this relief package to help workers, not to pad the pockets of corporations &amp; the wealthiest few.'), ('dates', '2020-03-30 23:52:58'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.7872859239578247)])
OrderedDict([('tweet', 'Ten years ago, Democrats made the single largest investment in student aid in U.S. history with the Student Aid &amp; Fiscal Responsibility Act. Last week, we secured student borrower relief in the CARES Act &amp; will prioritize student debt throughout this crisis. #FamiliesFirst'), ('dates', '2020-03-30 21:13:23'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.8605937361717224)])
OrderedDict([('tweet', 'The President’s attempts to avoid oversight are a surprise to no one. Rest assured: the Congress will exercise its oversight responsibilities. #FamiliesFirst #TRMS https://t.co/qczjTzJOy0'), ('dates', '2020-03-28 16:59:17'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.8197410106658936)])
OrderedDict([('tweet', 'Congressional Democrats have turned the Senate Republicans’ bill upside down, transforming it from trickle down proposal into a relief plan that puts #FamiliesFirst. #SitRoom https://t.co/42EEjFBC8H'), ('dates', '2020-03-26 02:53:10'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.9003133177757263)])
OrderedDict([('tweet', 'Congressional Democrats are united in our commitment to using all tools available to fight the coronavirus outbreak like the health crisis it is &amp; make sure workers are protected from the loss of a paycheck or that no family falls into financial ruin because of this pandemic.'), ('dates', '2020-03-19 22:27:18'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.6318755745887756)])
OrderedDict([('tweet', 'Bigoted statements which spread misinformation and blame Asians and the Asian American community for #coronavirus make us all less safe. @GOPLeader must delete this tweet and apologize immediately. https://t.co/twzCcVAWDH'), ('dates', '2020-03-10 22:58:58'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.7803630828857422)])
OrderedDict([('tweet', 'We need the #ForThePeople Act to ensure that we have a government of the people — not a government of the money.'), ('dates', '2020-03-10 15:42:15'), ('file_name', 'Nancy Pelosi.csv'), ('pct_true', 0.6453886032104492)])
OrderedDict([('tweet', 'We need the #truth and the truth isn’t partisan.\n \nIt speaks volumes that the Trump Administration won’t let Dr. Fauci, the leading expert of #COVID19, take questions from the House.\n \nWhat is @realDonaldTrump #Afraid #DrFauci will say?\n\nhttps://t.co/8PiNRDW1kz'), ('dates', '2020-05-05 19:17:47'), ('file_name', 'Nanette Diaz Barragán.csv'), ('pct_true', 0.8237403631210327)])
OrderedDict([('tweet', 'The President is lying again. @realDonaldTrump Your failure to act early and quick is among one reason the U.S. has hit a million cases. #COVID19 #FactsMatter #TruthMatters https://t.co/CiYsaQ9fBg'), ('dates', '2020-04-29 04:31:18'), ('file_name', 'Nanette Diaz Barragán.csv'), ('pct_true', 0.7468603253364563)])
OrderedDict([('tweet', 'Was the Trump Admin’s lack of transparency intentional so they could divert #SmallBusiness relief money to big corporations?\n\nThis is exactly why @HouseDemocrats are fighting to set standards, promote transparency &amp; expand opportunities for underserved small biz in the next bill. https://t.co/P1lYjTU07y'), ('dates', '2020-04-21 16:14:33'), ('file_name', 'Nanette Diaz Barragán.csv'), ('pct_true', 0.7726134657859802)])
OrderedDict([('tweet', 'Today, @RepFletcher &amp; I introduced a bill to protect #Medicaid\n\nMillions of Americans would lose access to healthcare if Trump Admin’s #MFAR is implemented. It is a backdoor attempt to weaken Medicaid.\n\nWe cannot lose #healthcare during a global pandemic\n\nhttps://t.co/v0S0Gmr0hE'), ('dates', '2020-04-17 17:08:53'), ('file_name', 'Nanette Diaz Barragán.csv'), ('pct_true', 0.7540984749794006)])
OrderedDict([('tweet', 'Fuel economy rollbacks\nUnsafe particulate matter levels\nNow, weaker mercury standards\n\nTrump Admin makes clear it favors fossil fuel industry over public health\n\nIs the EPA Administrator a coal lobbyist or something? \nOh.  \n#ProtectPublicHealth #BeyondCoal\nhttps://t.co/xfffbC5Krk'), ('dates', '2020-04-16 17:44:57'), ('file_name', 'Nanette Diaz Barragán.csv'), ('pct_true', 0.7494812607765198)])
OrderedDict([('tweet', 'Speaker Pelosi has created a “House Select Committee on the Coronavirus Crisis,” but there are already EIGHT mechanisms to oversee the govt’s response. \n\nWhy does she want to add her own select committee? #FL02 #COVID02'), ('dates', '2020-04-23 17:37:13'), ('file_name', 'Neal P. Dunn.csv'), ('pct_true', 0.7934341430664062)])
OrderedDict([('tweet', 'The @WHO was negligent &amp; potentially complicit in the Chinese coverup of the #Coronavirus, which has so far claimed the lives of 26,000 Americans &amp; brought the world to the brink of economic meltdown.'), ('dates', '2020-04-15 23:48:33'), ('file_name', 'Neal P. Dunn.csv'), ('pct_true', 0.9306432604789734)])
OrderedDict([('tweet', 'Today’s vote exonerates @realDonaldTrump and sends a clear message to House Democrats that they’ve wasted Congress’ time and the time of the American people who are fed up with this never-ending partisan witch-hunt.'), ('dates', '2020-02-05 23:19:06'), ('file_name', 'Neal P. Dunn.csv'), ('pct_true', 0.952387809753418)])
OrderedDict([('tweet', 'In response to House Democrats chanting "HR 3" - your legislation will crush medical innovation and produce fewer cures to deadly diseases. #SOTU2020'), ('dates', '2020-02-05 02:50:15'), ('file_name', 'Neal P. Dunn.csv'), ('pct_true', 0.7148821949958801)])
OrderedDict([('tweet', 'I agree @POTUS @realDonaldTrump - “we will never let socialism destroy American health care.” #SOTU2020'), ('dates', '2020-02-05 02:43:59'), ('file_name', 'Neal P. Dunn.csv'), ('pct_true', 0.8088534474372864)])
OrderedDict([('tweet', 'This weekend, the United States Embassy in Iraq was attacked by mortar fire from Iranian backed militias, and today House Democrats voted to remove @realDonaldTrump ability to respond to such attacks. House Democrats are being complete hypocrites.'), ('dates', '2020-01-30 19:15:13'), ('file_name', 'Neal P. Dunn.csv'), ('pct_true', 0.8355380892753601)])
OrderedDict([('tweet', '.@SpeakerPelosi knows the Articles of Impeachment against @realDonaldTrump are clearly a sham and will fail in the Senate. She has chosen to bypass our governing rules and go rogue by not transferring them to @senatemajldr. https://t.co/yxpwTNR6uW'), ('dates', '2020-01-09 18:21:57'), ('file_name', 'Neal P. Dunn.csv'), ('pct_true', 0.8814716935157776)])
OrderedDict([('tweet', 'Americans are counting on their government to show up for them in their time of need &amp; Trump is instead sabotaging the critical work Inspectors General do to maximize the use of taxpayer dollars &amp; protect against fraud &amp; corruption. My statement here⤵️https://t.co/mkILkyeraT'), ('dates', '2020-04-08 00:37:57'), ('file_name', 'Nita M. Lowey.csv'), ('pct_true', 0.728786289691925)])
OrderedDict([('tweet', 'You can’t fight an enemy you can’t see. That’s why the #FamiliesFirst Act makes #coronavirus testing free. Congress must pass this legislation immediately.'), ('dates', '2020-03-13 15:27:14'), ('file_name', 'Nita M. Lowey.csv'), ('pct_true', 0.8188468813896179)])
OrderedDict([('tweet', 'The Trump Administration would have us believe their education budget proposal empowers states and districts with flexibility, but the numbers don’t lie. This budget request would eliminate 41 programs and cut education funding by $6.2 billion, and our students deserve better. https://t.co/px8Q28Im6o'), ('dates', '2020-02-27 23:45:25'), ('file_name', 'Nita M. Lowey.csv'), ('pct_true', 0.6282753348350525)])
OrderedDict([('tweet', 'I am horrified to see the deployment of tactical ICE agents in NY. Yet again Trump is trying to punish our great state because we won’t deport our neighbors. We will not stand down and will not be intimidated by this radical administration. \nhttps://t.co/cnHSBui9HL'), ('dates', '2020-02-14 22:05:29'), ('file_name', 'Nita M. Lowey.csv'), ('pct_true', 0.6014954447746277)])
OrderedDict([('tweet', 'Weeks after the US Embassy in Iraq was attacked, Trump is suggesting a nearly 11% cut to Embassy Security. His willingness to endanger our brave diplomats is reckless &amp; puts Americans at risk. I’m eagerly awaiting an explanation for these callous cuts.\n \nhttps://t.co/9hOgZEI3sC'), ('dates', '2020-02-12 01:54:48'), ('file_name', 'Nita M. Lowey.csv'), ('pct_true', 0.7934920787811279)])
OrderedDict([('tweet', 'The #TrumpBudget puts New Yorkers at risk. It’s no surprise Trump’s SOTU speech wrote checks his budget was never designed to cash.'), ('dates', '2020-02-11 20:51:04'), ('file_name', 'Nita M. Lowey.csv'), ('pct_true', 0.667603075504303)])
OrderedDict([('tweet', 'One day after @SpeakerPelosi took the next step to #DefendOurDemocracy, GAO confirmed what @HouseDemocrats have long understood: Tump illegally withheld security assistance to Ukraine.\n\nIt’s up to the Senate.\n \nIf Trump has nothing to hide➡️he shouldn’t be afraid of a fair trial'), ('dates', '2020-01-17 14:50:45'), ('file_name', 'Nita M. Lowey.csv'), ('pct_true', 0.594871997833252)])
OrderedDict([('tweet', 'President Trump is again stealing national security funding to pay for his border wall. This reckless decision:\n\n❌ Makes America less safe\n❌ Burdens military families\n❌ Tramples Constitutional norms\n\nMy full statement w/@RepVisclosky &amp; @RepDWStweets: https://t.co/HtTbtwtIak'), ('dates', '2020-01-14 15:19:18'), ('file_name', 'Nita M. Lowey.csv'), ('pct_true', 0.5591939687728882)])
OrderedDict([('tweet', 'The #ACA has helped 62M Americans get birth control w/o out-of-pocket costs, but #SCOTUS is deciding whether your boss or university has the right to take that away. \n\nThe @realDonaldTrump admin. needs to keep its hands off basic health care. #HandsOffMyBC'), ('dates', '2020-05-06 17:29:30'), ('file_name', 'Norma J. Torres.csv'), ('pct_true', 0.8962869644165039)])
OrderedDict([('tweet', "Rates of violence against women and girls in #IndianCountry are 10 TIMES higher than the national average. We cannot continue to let #MMIW slip through the cracks &amp; I introduced Savanna's Act w/ @RepNewhouse &amp; @RepDebHaaland to help find justice for these women. #SilentCrisis"), ('dates', '2020-05-05 21:05:22'), ('file_name', 'Norma J. Torres.csv'), ('pct_true', 0.6811047792434692)])
OrderedDict([('tweet', 'Sadly, some Americans have yet to wakeup to the harsh reality that the people that matter the most during a pandemic are often the least appreciated and least protected! #CoronaLessons https://t.co/8fji3I1TfQ'), ('dates', '2020-04-21 20:38:50'), ('file_name', 'Norma J. Torres.csv'), ('pct_true', 0.6433977484703064)])
OrderedDict([('tweet', 'We are all hurting b/c of the #StayAtHome order. Americans want to go back to work &amp; Americans have the right to free speech. What we cannot do is sacrifice #PublicHealth to open the economy too soon. Doing so puts lives at unnecessary risk. https://t.co/wg09NQkPVA'), ('dates', '2020-04-20 22:47:51'), ('file_name', 'Norma J. Torres.csv'), ('pct_true', 0.5379493832588196)])
OrderedDict([('tweet', "These #Trump deportations are spreading #COVID19 to an already unstable region and only causing more migrants to flee to the US. From the start, @POTUS’ deportation force has outsourced challenges to unprepared countries—and now they're exporting death.\nhttps://t.co/tTuwmJxsd4"), ('dates', '2020-04-16 22:05:15'), ('file_name', 'Norma J. Torres.csv'), ('pct_true', 0.7700807452201843)])
OrderedDict([('tweet', 'In a decision that completely defies logic, @realDonaldTrump moved to end funding for @WHO in the midst of a global pandemic w/ over 2M #COVID19 cases worldwide. Is it too much to ask for our President to prioritize global health over pleasing conservative ideologues? Evidently. https://t.co/UC6FX6POP9'), ('dates', '2020-04-15 21:04:22'), ('file_name', 'Norma J. Torres.csv'), ('pct_true', 0.5785307288169861)])
OrderedDict([('tweet', 'This is deeply disturbing. That our government would deny a green card to someone volunteering on the frontline of the #COVID19 battle makes no sense. Monitoring this closely. If this is part of @realDonaldTrump’s anti-immigrant policies it’s appalling.\n\nhttps://t.co/UgRIv9gKEH'), ('dates', '2020-05-07 00:18:37'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.5444833040237427)])
OrderedDict([('tweet', 'Today marks the start of Asian Pacific American Heritage month.  Asian Americans are a critical component of the economic success and cultural tapestry of New York city. Let’s celebrate this #APAHM together as we honor their profound commitment to our nation.'), ('dates', '2020-05-01 21:02:42'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.6290116906166077)])
OrderedDict([('tweet', "Deeply concerned this activist in #PuertoRico, who has been feeding the hungry, was detained for protesting to call for PR gov't to distribute food &amp; open school cafeterias. These people were exercising their 1st amendment rights - safely - from cars. https://t.co/2nnSdeUHde"), ('dates', '2020-04-30 19:04:54'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.644445538520813)])
OrderedDict([('tweet', "Another appalling instance of anti-Asian bias. New Yorkers, let’s battle #Covid19 together. We will not let the virus of hate overcome our city's spirit of love and unity.\nhttps://t.co/6XSPDyUcQY"), ('dates', '2020-04-28 20:29:00'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.7406102418899536)])
OrderedDict([('tweet', 'Proud to stand with @SenWarren to send a message to the Trump Administration that skirting the law by putting Wall Street over Main Street is unacceptable. \n\nWe are asking the @SBAOIG &amp; @USTreasury OIG to investigate problems plaguing #PPPloans. #PPP https://t.co/2z5dILZYzl'), ('dates', '2020-04-24 21:06:04'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.6904779076576233)])
OrderedDict([('tweet', '#MitchAntoinette would rather deny aid to firefighters, police &amp; EMS than deliver the relief they need to keep fighting the coronavirus. This is a disgusting remark to make when American lives are on the line nationwide. https://t.co/VhM1F1aqez'), ('dates', '2020-04-23 18:04:36'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.6375146508216858)])
OrderedDict([('tweet', 'The President’s efforts to divide us, stoke hate and foment fear will not distract from his abysmal response to this pandemic. This illegal, authoritarian move must and will be stopped. https://t.co/7HZ2YQzpTl'), ('dates', '2020-04-21 13:45:30'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.5441527366638184)])
OrderedDict([('tweet', 'The President’s obvious efforts to divide us, stoke hate and foment fear will not distract from his abysmal response to this pandemic. This illegal, authoritarian move must and will be stopped.'), ('dates', '2020-04-21 10:55:08'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.7862749099731445)])
OrderedDict([('tweet', 'Sen. @marcorubio recently said the PPP program is not a Wall Street package. I agree. That’s why before we pour more taxpayer money into it, we need guardrails &amp; transparency to protect #SmallBiz interests and stop loans from reaching big corporations. https://t.co/6Ag7Pp9tML'), ('dates', '2020-04-17 14:02:04'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.8870800733566284)])
OrderedDict([('tweet', "We need protections to ensure #PPP funds are going to actual small businesses, so giant companies don't game the system, deplete the funds and leave behind small, main street businesses.\n\nhttps://t.co/T9vRPq3R6I"), ('dates', '2020-04-16 16:42:21'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.656286895275116)])
OrderedDict([('tweet', 'Minority-owned businesses traditionally have a harder time getting loans and may be neglected by big banks offering #PPPloans. @housedemocrats are fighting to get them #Smallbizrelief through community lenders. https://t.co/NFpGKJycHD'), ('dates', '2020-04-15 15:30:00'), ('file_name', 'Nydia M. Velázquez.csv'), ('pct_true', 0.6725424528121948)])
OrderedDict([('tweet', 'Over the years our nation’s anti-corruption laws have been slowly gutted, largely by court decisions, and it couldn’t be coming at a worse time. Congress needs to reverse that trend.  Now.  I’m working to do just that.  https://t.co/bvZGsqaZGK'), ('dates', '2020-05-07 17:58:41'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.6134247779846191)])
OrderedDict([('tweet', 'Instead of focusing on the pandemic engulfing our nation, Republicans on the Sen. Judiciary Comm. this AM are acting like it’s business as usual &amp; pushing ahead with a divisive judicial nominee for a seat that ISN’T EVEN VACANT UNTIL SEPTEMBER: https://t.co/sB7qU4eMOJ'), ('dates', '2020-05-06 15:55:05'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.6027648448944092)])
OrderedDict([('tweet', "Sometimes you don't need to interpret the Constitution; you just need to read it. I hope the independent judiciary does exactly that &amp; affirms that Congress, NOT the Pres., has the power of the purse. Read my comment on the ongoing arguments before the DC Appeals Court: https://t.co/4NR4KuRldl"), ('dates', '2020-04-28 15:12:54'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.6081351637840271)])
OrderedDict([('tweet', 'The White House refused to join a meeting of world leaders to accelerate work on tests, drugs and vaccines against #COVID19. Why? Because it was sponsored by #WHO, Trump’s latest scapegoat. We need leaders who put science and facts over politics and ego. https://t.co/pYc0O5aqO6'), ('dates', '2020-04-27 17:52:01'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.5546838641166687)])
OrderedDict([('tweet', 'The world needs U.S. leadership to fight the greatest threat since WWII but instead this Administration pontificates and shifts blame for its abysmal failures. The result? China’s influence in WHO will grow and the world learns it cannot count on America.https://t.co/D4EnJ4Qpps'), ('dates', '2020-04-23 23:48:46'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.9052925109863281)])
OrderedDict([('tweet', 'We all know what this immigration ban is about. It has NOTHING to do w/protecting Americans &amp; EVERYTHING to do w/scapegoating immigrants to rile up his base. This admin is a 1 trick pony: cast blame &amp; misdirection to distract from their own failures. https://t.co/i8GpD8SRdL'), ('dates', '2020-04-23 18:39:21'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.5393277406692505)])
OrderedDict([('tweet', 'This is exactly why I just asked AG Barr what authority he believes they have to overrule governors &amp; state health officials. They must not be threatening legal action based on a secret interpretation of the law. Any legal basis must be public! https://t.co/38C0yKArxF https://t.co/dFNUKicStI'), ('dates', '2020-04-21 19:58:58'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.6460050344467163)])
OrderedDict([('tweet', 'No president should rely on a secret body of law to consider unprecedented, unconstitutional actions. Not during a pandemic, or anytime. I’m asking the Trump Admin for any justification for Trump’s baseless assertion of “total” authority over the states. Read it here: https://t.co/NSvT0JWkGQ'), ('dates', '2020-04-21 16:46:34'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.5558604001998901)])
OrderedDict([('tweet', 'Does the President realize that his attempt to distract from his response to the COVID-19 pandemic by shutting down legal immigration would shut our doors to many of the VERY healthcare professionals who are caring for our sick? Stop doing policy and government by tweets!'), ('dates', '2020-04-21 15:10:30'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.8932210803031921)])
OrderedDict([('tweet', 'Not only has Pres Trump cut off funding for the World Health Org in the midst of a pandemic that has infected millions worldwide, he has cut off funding for the Pan–American Health Org that combats the virus in THIS hemisphere. It’s a dangerous political action.'), ('dates', '2020-04-18 20:13:02'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.8563778400421143)])
OrderedDict([('tweet', 'Pres Trump shamelessly blames #WHO at a time when he praised China and dismissed the virus as nothing to worry about. This is no time for hypocrisy, rewriting history, and evading responsibility. We need to work together to combat the greatest global health threat in a century.'), ('dates', '2020-04-16 17:51:33'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.8734651207923889)])
OrderedDict([('tweet', 'Some false claims are more dangerous than others. This is exactly why the Framers did not leave us defenseless in the face a president’s delusions of autocracy. They left us a Constitution. (And it’s well past time that this president read it, respect it, and follow it.)'), ('dates', '2020-04-14 10:04:38'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.8157712817192078)])
OrderedDict([('tweet', 'It has been one year since the House passed the VAWA reauthorization. For decades VAWA has been a bipartisan effort. It’s past time for the Senate to pass the bipartisan House bill, and respond to the current and critical needs of ALL survivors. #VAWA4ALL'), ('dates', '2020-04-08 13:16:43'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.808729887008667)])
OrderedDict([('tweet', 'Today @JusticeOIG found the FBI routinely fails to disclose all relevant evidence in FISA cases. @SenMikeLee and I are working to fix this &amp; other issues. There’s an opportunity for real reform when the Senate takes up FISA in the weeks ahead &amp; I hope our colleagues will join us. https://t.co/5b7nDptavA'), ('dates', '2020-03-31 21:01:30'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.6836665868759155)])
OrderedDict([('tweet', 'Bragging abt TV ratings while people die in escalating #s reveals a level of narcissism I’ve never before seen in a President. This isn’t some reality TV show. This is REALITY. Ratings mean nothing to the 1000s of Americans who’ve already lost loved ones\nhttps://t.co/hAgFe9Mavd'), ('dates', '2020-03-29 21:59:00'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.8282688856124878)])
OrderedDict([('tweet', 'In a crisis, the Guard never lets Vermonters down. And we should never let our Guard down. https://t.co/eTIM5Po5oA'), ('dates', '2020-03-24 21:58:18'), ('file_name', 'Patrick J. Leahy.csv'), ('pct_true', 0.7581422924995422)])
OrderedDict([('tweet', 'The COVID-19 virus poses a significant threat to public health. The Senate Sergeant at Arms made the tough, but right, decision to close Senate offices to the public.'), ('dates', '2020-03-12 20:46:28'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.5843572616577148)])
OrderedDict([('tweet', "The Chinese gov. hacked &amp; stole the private data of millions of Americans, including over 5.5 million Pennsylvanians. This reckless, unacceptable, &amp; brazen disregard for the rule of law must not stand. I fully support @TheJusticeDept's indictments today. https://t.co/X9GmINEaCs"), ('dates', '2020-02-10 20:20:40'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.7995734214782715)])
OrderedDict([('tweet', '75 years ago, allied forces liberated Auschwitz-Birkenau, a place that will forever embody the evil and horror of the Holocaust. This #HolocaustRemembranceDay we must recommit ourselves to ensuring that such an atrocity never happen again. #WeRemember https://t.co/USmSry41E6'), ('dates', '2020-01-27 16:59:33'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.8029038906097412)])
OrderedDict([('tweet', 'These stats are scary: the unilateral banning of natural gas fracking by a future president would devastate the U.S./PA economies. To help safeguard against this, the Senate should pass my measure making clear that no president has the authority to ban fracking on private land. https://t.co/1Ti22PwDgK'), ('dates', '2019-12-19 19:07:42'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.8101268410682678)])
OrderedDict([('tweet', "VWs &amp; Toyotas don't pose a nat. security threat. The window to levy 232 tariffs on foreign autos is closed. If @CommerceGov wants to impose punitive taxes on Americans who buy foreign cars, they must launch a new investigation. Congress should reassert authority over 232 tariffs. https://t.co/X8M9MxsZQk"), ('dates', '2019-12-03 22:33:58'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.7988274693489075)])
OrderedDict([('tweet', 'By the account of anyone who watched the heinous video that circulated on social media last weekend, these two individuals viciously tortured an innocent animal and should be prosecuted to the fullest extent of the law. https://t.co/nzj7vEL9kl'), ('dates', '2019-12-03 13:39:30'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.7501479983329773)])
OrderedDict([('tweet', "The elections in Hong Kong this past weekend were a stunning rebuke of the communist Chinese regime. It's proof that the vast majority of Hong Kong residents believe in, &amp; want to preserve, the democratic freedoms they've had all these years. It's time for Beijing to take notice. https://t.co/QVBX7FOgpJ"), ('dates', '2019-11-25 19:44:18'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.7920278906822205)])
OrderedDict([('tweet', 'President Trump is right to confront China on its egregious trade practices but tariffs also hurt American workers and consumers. Further proof: a new study showing that 1.5 million American jobs are at risk because of the trade war with China: https://t.co/jd1Ha3DC4Z'), ('dates', '2019-11-13 00:38:28'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.8817328810691833)])
OrderedDict([('tweet', 'For decades, the federal government has operated a Soviet-style program propping up U.S. sugar farmers. This is bad for confectionery workers &amp; American consumers. @SenatorShaheen &amp; I have a bipartisan bill to fix this. Read more from @PGdanielmoore: https://t.co/tZmUBe1QA4'), ('dates', '2019-11-05 17:28:46'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.8283131718635559)])
OrderedDict([('tweet', 'The president’s decision to withdraw U.S. forces from the northern Syrian border poses a significant threat to our national security and risks reversing the progress made in the region to destroy ISIS.'), ('dates', '2019-10-07 17:19:18'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.6696305871009827)])
OrderedDict([('tweet', 'EPA and DOE have affirmed that the RFS disproportionately hurts refineries and workers.\n \nThere have been just two federal mandates requiring Americans to buy a product whether they want or not. One is corporate welfare for ethanol. The other is Obamacare. https://t.co/EGTlF3puRE'), ('dates', '2019-10-02 20:50:57'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.9497119188308716)])
OrderedDict([('tweet', 'China continues to undermine American drug laws by turning a blind eye to dealers who export this junk to the US. The Senate should put my and @SenDougJones bipartisan bill on the floor to punish countries, like China, that do not adhere to narcotics control standards. https://t.co/lDjLzgknBh'), ('dates', '2019-10-01 20:51:21'), ('file_name', 'Patrick J. Toomey.csv'), ('pct_true', 0.9911670088768005)])
OrderedDict([('tweet', 'Instead of conducting meaningful oversight, the Democrats’ plan to create a select committee to review the admin’s #coronavirus response will inevitably become another partisan investigation into @realDonaldTrump. We need transparency &amp; accountability, not political games. https://t.co/775luznrGp'), ('dates', '2020-04-23 17:46:51'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.733776867389679)])
OrderedDict([('tweet', 'It was clear, funding for the #PPP was going to run out quickly. But rather than work with us, Democrats blocked funding while Speaker Pelosi falsely claimed there was no evidence that PPP even needed more money. https://t.co/4LxBqka5tN'), ('dates', '2020-04-16 15:12:28'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.7750385403633118)])
OrderedDict([('tweet', 'Small businesses are an important part of #NC10 communities, which is why it is disappointing Congressional Democrats continue to oppose a bill that would add crucial funding to the #PaycheckProtectionProgram. We must continue working to support American workers and families. https://t.co/S05XDH9kPm'), ('dates', '2020-04-14 21:30:00'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.7929946184158325)])
OrderedDict([('tweet', "There is no time for @HouseDemocrats &amp; @SenateDems’s political games. The American people are suffering &amp; need our help now. Passage of the #CARESAct would provide vitally important #coronavirusrelief for Americans immediately. Let's get it done. https://t.co/iJsVG2rwD6"), ('dates', '2020-03-23 20:30:40'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.7351921200752258)])
OrderedDict([('tweet', 'Democrats are playing politics with the health and safety of the American people.\n \n#ICYMI, Senate Democrats voted down even considering a bipartisan bill that would provide direct assistance to millions of Americans in this critical time → https://t.co/mEtzMzRyXu'), ('dates', '2020-03-23 14:56:24'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.8335698246955872)])
OrderedDict([('tweet', 'Horrible policy that would be harmful to hardworking Americans just like the FTT. We should be helping America’s small businesses &amp; startup employees achieve their dreams by closely linking their hard work to ownership not taxing their equity away before they can even get started https://t.co/Fl9SKSVXrx'), ('dates', '2020-02-28 17:42:49'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.7123628258705139)])
OrderedDict([('tweet', 'This is something Republicans and Democrats can actually agree on. Instead of impeachment and endless investigations we should be working together to stop the growing threat of Huawei and China as a whole → https://t.co/rAgPeZLnQE'), ('dates', '2020-02-17 18:22:42'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.8922450542449951)])
OrderedDict([('tweet', 'During the #SOTU, President @realDonaldTrump highlighted the positive impact our booming economy is having on American workers. The January jobs report is more proof of this good news. Watch this video from @USTreasury to find out why → https://t.co/64Ey91yiQ1'), ('dates', '2020-02-09 18:00:00'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.5281989574432373)])
OrderedDict([('tweet', 'As this graphic demonstrates, the Democrats’ impeachment sham was never about the facts, instead it has always been about their desperation to undo the results of the 2016 election. https://t.co/CiYGtHZPhJ'), ('dates', '2020-02-08 16:00:00'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.7608004212379456)])
OrderedDict([('tweet', 'After months of claiming they had overwhelming evidence against the President, Democrats at the Senate impeachment trial are calling for more witnesses. This proves what we have known all along: they are willing to do whatever it takes to overturn the will of the American people. https://t.co/PN4aGRVOdM'), ('dates', '2020-01-28 20:24:52'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.669571042060852)])
OrderedDict([('tweet', 'The entire impeachment process has been a politically-motivated sham that has kept Congress from doing actual work that would benefit the American people. Read my statement on voting against sending the articles of impeachment to the Senate here: https://t.co/y7LSQ2lt37'), ('dates', '2020-01-15 21:03:57'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.5377141833305359)])
OrderedDict([('tweet', 'Speaker Pelosi and @HouseDemocrats continue with their fiercely partisan impeachment process. I will be voting against these politically-motivated attacks against @realDonaldTrump whenever they come before the full House. https://t.co/K7p7vfP9Ux'), ('dates', '2019-12-13 15:26:47'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.5971211194992065)])
OrderedDict([('tweet', 'American workers have waited long enough, the time to pass the #USMCA is now. From enabling our economy to continue to grow to the jobs it will help create, there is no excuse not to pass this trade deal and give a win to these hardworking men and women → https://t.co/fsNtLM0Fzd'), ('dates', '2019-12-10 21:02:33'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.5474026799201965)])
OrderedDict([('tweet', 'At every turn, House Democrats have ignored facts and created a sham process just to get the results they have wanted since the 2016 election. The articles of impeachment announced today are no different → https://t.co/nXTf7M15qt'), ('dates', '2019-12-10 19:05:50'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.6368821859359741)])
OrderedDict([('tweet', 'Despite what some House Democrats may claim they have been trying to undermine @POTUS since before he took office. The impeachment hearings are just the latest in a long list of excuses they are using to try and overturn the will of the American people → https://t.co/zgSgBdQSgw'), ('dates', '2019-12-09 20:34:12'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.8971808552742004)])
OrderedDict([('tweet', 'House Democrats don’t have any evidence so why are they still refusing to end the sham impeachment process? Because they are willing to ignore the facts as long as they are able to achieve their goal of removing President Trump from office → https://t.co/GPOD6swotB'), ('dates', '2019-12-04 18:37:38'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.9151384234428406)])
OrderedDict([('tweet', 'Over the past 6 months the people of Hong Kong have protested against China’s efforts to undermine their democracy. I applaud @POTUS for signing legislation supporting Hong Kong. These bills send a strong message to China: we will always support freedom → https://t.co/RMNo0yLxRZ'), ('dates', '2019-12-02 16:41:43'), ('file_name', 'Patrick T. McHenry.csv'), ('pct_true', 0.7557653188705444)])
OrderedDict([('tweet', 'We do need a vaccine. We do need tests. We do not need the President contradicting public health experts. https://t.co/0Ji7CLSYKT'), ('dates', '2020-05-08 21:50:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.5037418007850647)])
OrderedDict([('tweet', 'I can’t believe it’s 2020 &amp; we still have to fight against barriers that make it harder for women to get the care they need. Birth control is health care—and women shouldn’t have to pay extra, ask permission, or overcome obstacles to access it. #FreeThePill'), ('dates', '2020-05-08 20:45:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.599522590637207)])
OrderedDict([('tweet', 'Update: the Trump Administration still plans to move forward with its health care lawsuit and continue its efforts to take away your health care—once again putting politics over patients and families. https://t.co/oMVh8ARsWj'), ('dates', '2020-05-06 21:31:26'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.5922901630401611)])
OrderedDict([('tweet', '2 months since President Trump said "anybody who wants a test can get a test”—&amp; it’s still not true. I fought to require that the Admin finally provide a plan to nationally ramp up testing in our last coronavirus bill, &amp; they need to deliver.\nhttps://t.co/LdSAkXrme9'), ('dates', '2020-05-06 18:05:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.5256043672561646)])
OrderedDict([('tweet', 'If President Trump &amp; Republicans refuse to stop their partisan health care lawsuit—even during a global pandemic—it will be the ultimate proof they care more about politics than patients &amp; families who could be hurt by their relentless attacks. https://t.co/G3RfudFwMR'), ('dates', '2020-05-06 00:55:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.6855728030204773)])
OrderedDict([('tweet', "It's simple: no one's ideology is more important than a woman's personal health care decisions. And yet, since day one President Trump and VP Pence have been pushing backward policies to block access to birth control and roll back #ReproRights. #HandsOffMyBC https://t.co/brWljDlzyz"), ('dates', '2020-05-06 00:10:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.5185719728469849)])
OrderedDict([('tweet', "Mitch McConnell brought back the Senate during a global pandemic to jam through more of President Trump's extreme judicial nominees who want to dismantle the ACA. Justin Walker made clear he wouldn't think twice about ruling to take away your health care if confirmed. #StopWalker"), ('dates', '2020-05-05 18:50:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.9533872008323669)])
OrderedDict([('tweet', "ICYMI: Over the weekend President Trump moved to replace an independent government official investigating his Admin's response to the coronavirus pandemic—continuing his clear pattern of retaliation against those who tell the truth.\nhttps://t.co/7lRRJeYIgg"), ('dates', '2020-05-04 20:54:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.604780912399292)])
OrderedDict([('tweet', 'The very health care protections Republicans are trying to take away with their ongoing, partisan lawsuit have been critical as our nation responds to the coronavirus pandemic. Trump &amp; the GOP should stop their health care attacks immediately. \nhttps://t.co/zxL0Z9IPf6'), ('dates', '2020-05-03 20:15:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.7954069375991821)])
OrderedDict([('tweet', 'We all know the President hasn’t told people the truth about this virus or his Admin’s response, and late last night, he moved to replace an independent government official who did. The President cannot be above oversight. https://t.co/Rfr795wXQ7'), ('dates', '2020-05-02 17:29:08'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.6631991863250732)])
OrderedDict([('tweet', ".@BetsyDeVosED pushing Dreamers &amp; other vulnerable students out of needed relief from the CARES Act is a cruel decision that isn't authorized in the law. She should immediately reverse this shameful, arbitrary guidance. https://t.co/eiTyLkBgcg"), ('dates', '2020-05-01 22:31:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.5024240612983704)])
OrderedDict([('tweet', 'Congress cannot stand by and let thousands of workers lose their jobs and allow local governments to suspend vital services at a time when communities need them the most. I will keep fighting to get cities, counties, and states the support they need. \nhttps://t.co/Ovbzxt78zJ'), ('dates', '2020-05-01 18:45:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.5386477708816528)])
OrderedDict([('tweet', "Jared Kushner’s comments completely disregard the thousands who have died and the millions of Americans whose lives have been upended by this crisis. It's an absurd attempt to distract from the Trump Admin’s failure to ramp up our nation’s testing capacity in any meaningful way. https://t.co/mQr3ykjvdg"), ('dates', '2020-04-29 20:35:56'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.9054743051528931)])
OrderedDict([('tweet', 'The Trump Admin’s failure to address testing at a national level has resulted in states &amp; hospitals scrambling for necessary equipment to respond to the coronavirus. We are all on the same team—&amp; it’s past time this administration starts acting like it. \nhttps://t.co/Lnv8fIgSRR'), ('dates', '2020-04-28 21:35:30'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.6215237975120544)])
OrderedDict([('tweet', "President Trump's testing blueprint does nothing new and will accomplish nothing new. Perhaps most pathetically, it attempts to shirk obviously federal responsibilities by assigning them solely to states instead. https://t.co/FY1D56kLat"), ('dates', '2020-04-27 22:32:43'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.5477816462516785)])
OrderedDict([('tweet', "Reports that the Trump Administration is using the cover of the pandemic to consider aggressive immigration policies that aren't in the best interests of children in its care are shameful. @rosadelauro &amp; I pressed @SecAzar to protect these children.\nhttps://t.co/WmK7g8Pzmp"), ('dates', '2020-04-24 19:45:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.522874653339386)])
OrderedDict([('tweet', "We already know President Trump's border wall is reckless and fueled by bigotry. Building it during a pandemic—and putting workers' health at risk in the process—makes it even more shameful. \nhttps://t.co/sb7S4JDPnV"), ('dates', '2020-04-24 18:50:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.7486866116523743)])
OrderedDict([('tweet', 'We need to be listening to experts and science, not pushing them aside. A global pandemic is not the time to shuffle personnel, or contradict and remove experts for wanting to do their job well. These reports are incredibly disturbing and I will be pushing for answers. https://t.co/KceFh09GCU'), ('dates', '2020-04-22 21:39:24'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.6234816908836365)])
OrderedDict([('tweet', "It's absolutely shameful that extreme politicians would take advantage of this crisis to push their anti-abortion agenda. Abortion is essential health care and these cruel, partisan attacks must stop."), ('dates', '2020-04-22 20:30:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.9221825003623962)])
OrderedDict([('tweet', "On today's call, @SenateDems pushed VP Pence on testing &amp; got no answers. The Administration is still refusing to act with urgency to ramp up our nation's testing capacity—&amp; shifting blame onto governors is another attempt to point fingers rather than solve problems."), ('dates', '2020-04-18 00:02:57'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.7462379932403564)])
OrderedDict([('tweet', 'The Trump Administration’s early missteps and lack of urgency to ramp up coronavirus testing have put us way behind. If we want to get back to any semblance of normal life, we need to get back on track—and I laid out a plan to do just that. \nhttps://t.co/mOTJnCpUFN'), ('dates', '2020-04-17 16:31:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.6002963185310364)])
OrderedDict([('tweet', ".@SecAzar must allocate CARES Act funds to those who desperately need it on the frontlines of this pandemic without delay. Congress didn't intend these funds to be used as a bandaid so President Trump can avoid taking steps to expand access to care.\nhttps://t.co/QIXRG7mYdI"), ('dates', '2020-04-16 17:45:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.7510705590248108)])
OrderedDict([('tweet', 'From the start of the coronavirus pandemic, the Trump Administration has failed to build up the testing infrastructure we need. Our country can’t wait—and now @SenateDems have a roadmap to ramp up testing nationwide.\nhttps://t.co/AbmW2P4D1w'), ('dates', '2020-04-16 16:33:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.557195782661438)])
OrderedDict([('tweet', "It's shameful that extreme, right-wing politicians would use the coronavirus pandemic to push through their harmful, anti-abortion agenda. Abortion is health care and it’s reckless to try to take that away, especially at a time like this. \nhttps://t.co/HEF1oYHV5i"), ('dates', '2020-04-13 19:05:00'), ('file_name', 'Patty Murray.csv'), ('pct_true', 0.9305528998374939)])
OrderedDict([('tweet', 'It will be interesting to see how the Democrats blame Russia for these court documents from 1996... https://t.co/Jzh3HjZNvz'), ('dates', '2020-05-08 13:24:06'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.6558789610862732)])
OrderedDict([('tweet', 'It’s time to hold the Obama-Biden officials and dirty cops accountable for that they did to this American hero. https://t.co/kpm52Dnqlc'), ('dates', '2020-05-07 23:44:32'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.7403712868690491)])
OrderedDict([('tweet', 'Absolute vindication of @GenFlynn! We must now hold the corrupt and dirty cops who got us to this point accountable for their wrongdoing!'), ('dates', '2020-05-07 19:55:04'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.7056193947792053)])
OrderedDict([('tweet', "The Obama-Biden FBI used Democrat-funded Russian disinformation as a basis to illegally spy on @realDonaldTrump's campaign. Then, Comey caused sensitive information to leak to the media. Now, we're learning the extent of the FBI's entrapment of @GenFlynn.  https://t.co/IU4pumhO8W"), ('dates', '2020-05-05 01:17:11'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.7002790570259094)])
OrderedDict([('tweet', 'What happened to @GenFlynn is a national disgrace. We MUST hold those responsible accountable.'), ('dates', '2020-05-01 15:33:33'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.7248919010162354)])
OrderedDict([('tweet', 'Flashback to the Kavanaugh hearings and the circus the left and their media allies made over a high school yearbook. But Senate records are out of the question? https://t.co/NusAGdFrLi'), ('dates', '2020-05-01 14:47:36'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.5933475494384766)])
OrderedDict([('tweet', "Democrats don't care about women, they care about power and they've proven they will do anything and everything to obtain it. https://t.co/wVrcChEHup"), ('dates', '2020-04-30 21:02:37'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.8721369504928589)])
OrderedDict([('tweet', 'Never forget how low the Democrats and mainstream media went to try ruining Brett Kavanaugh’s life. Where are they now? https://t.co/KGI3E8BGha'), ('dates', '2020-04-30 19:40:03'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.7028850317001343)])
OrderedDict([('tweet', 'Bombshell documents reveal General Flynn was set up by the FBI. We must hold the Comey Cabal accountable, including McCabe, Yates, McCord, Strzok, and Pientka! https://t.co/rbVXwq0pA5'), ('dates', '2020-04-29 23:58:27'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.7184166312217712)])
OrderedDict([('tweet', 'We cannot truly put #AmericaFirst unless we suspend H-1B, H4, L1, B1, B2, Optional Practical Training Program, and other guestworker admissions. We must promote wages and opportunities for American workers during this period of rising unemployment.'), ('dates', '2020-04-28 22:18:12'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.7940837740898132)])
OrderedDict([('tweet', 'When @realDonaldTrump banned travel from China, saving countless American lives, Nancy Pelosi and Democrats were trying to tie the president’s hands and prevent him from keeping Americans safe. https://t.co/viHSei5e6z'), ('dates', '2020-04-28 21:02:37'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.9499631524085999)])
OrderedDict([('tweet', 'China sends more students to American universities than anyone else in the world. At the same time, many American graduates can’t land jobs in their respective fields. Time to put #AmericaFirst! https://t.co/LCd97Gx7z3'), ('dates', '2020-04-28 16:55:52'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.9136752486228943)])
OrderedDict([('tweet', 'The current pandemic that our nation faces illustrates the challenge of organizing a whole-of-government response to an evolving biological threat. The U.S. must quickly implement changes that could speed adoption of the National Biodefense Strategy.\nhttps://t.co/eBLc9KySQF'), ('dates', '2020-04-28 15:48:01'), ('file_name', 'Paul A. Gosar.csv'), ('pct_true', 0.7135193347930908)])
OrderedDict([('tweet', 'For two decades, Kurds have fought on frontlines alongside US troops against terror. We should remain in a position to prevent unnecessary conflict between two of our traditional allies. Abandoning the Kurds could lead to devastating loss of life and the potential return of ISIS.'), ('dates', '2019-10-08 04:35:01'), ('file_name', 'Paul Cook.csv'), ('pct_true', 0.675846517086029)])
OrderedDict([('tweet', 'Whether deriding racial minorities at home or cavorting with Nazis and white supremacists abroad, Rep. Steve King brings dishonor to himself, the party of Abraham Lincoln, and his nation. We must reconsider whether Mr. King has the moral authority to continue serving in Congress. https://t.co/pUNn6XOpbE'), ('dates', '2019-01-11 16:25:57'), ('file_name', 'Paul Cook.csv'), ('pct_true', 0.5977849364280701)])
OrderedDict([('tweet', "Yesterday's horrific terror attack in Pennsylvania, shows why it's critical that we fight hatred wherever we see it. All Americans should be concerned about any rise in Anti-Semitism. America stands with the victims and their families during this tragic time."), ('dates', '2018-10-28 15:17:10'), ('file_name', 'Paul Cook.csv'), ('pct_true', 0.503391683101654)])
OrderedDict([('tweet', 'Terrible news as Chinese checkbook diplomacy steals away Burkina Faso from our ally, Taiwan. Congress is united in its support for the people of Taiwan against these bullies in Beijing.'), ('dates', '2018-05-25 23:14:45'), ('file_name', 'Paul Cook.csv'), ('pct_true', 0.8112984895706177)])
OrderedDict([('tweet', "I’m deeply disappointed in Senator Schumer and Senate Democrats for shutting down government. The CR provided health care for over 9 million children and crucial funding for our military and national defense. Here's my full statement: https://t.co/mOnnXritlQ #SchumerShutdown"), ('dates', '2018-01-20 16:47:06'), ('file_name', 'Paul Cook.csv'), ('pct_true', 0.5419865846633911)])
OrderedDict([('tweet', "Disappointed in @usdot decison about Norwegian Air. I'll continue to stand with our US airlines to ensure fair competition."), ('dates', '2016-04-15 20:46:55'), ('file_name', 'Paul Cook.csv'), ('pct_true', 0.5293827652931213)])
OrderedDict([('tweet', '.@realDonaldTrump and his Administration have coordinated public-private partnerships that have supplied #Michigan with Personal Protective Equipment for our frontline healthcare workers. https://t.co/kjZQqMiYUp'), ('dates', '2020-05-01 20:53:54'), ('file_name', 'Paul Mitchell.csv'), ('pct_true', 0.5574783086776733)])
OrderedDict([('tweet', "While businesses struggle and workers lose their jobs the Dems play politics. \n\nWho says it isn't business as usual in DC? #PaycheckProtectionProgram\n\nhttps://t.co/qKxNql7gYL"), ('dates', '2020-04-15 19:06:38'), ('file_name', 'Paul Mitchell.csv'), ('pct_true', 0.6434465646743774)])
OrderedDict([('tweet', 'It is beyond disappointing to see Democrats in the Senate block vital additional funding for the Paycheck Protection Program. \n\nOur small businesses desperately need this program so they can keep people employed. It’s time to stop the political games and start helping people.'), ('dates', '2020-04-09 15:27:47'), ('file_name', 'Paul Mitchell.csv'), ('pct_true', 0.7613772749900818)])
OrderedDict([('tweet', 'The Senate’s #Coronavirus relief bill includes provisions that Republicans and Democrats agree on, provisions that would quickly help workers, families, and small businesses. Unfortunately, Speaker Pelosi has abandoned this approach in favor of pushing partisan priorities.'), ('dates', '2020-03-23 21:28:44'), ('file_name', 'Paul Mitchell.csv'), ('pct_true', 0.8571484088897705)])
OrderedDict([('tweet', 'Soon the House will vote on H.J.Res 79, which would remove the deadline for ratification of the #ERA. Congress does not have the constitutional authority to retroactively remove the deadline for a proposed constitutional amendment, which is why I oppose H.J.Res. 79.'), ('dates', '2020-02-13 15:20:27'), ('file_name', 'Paul Mitchell.csv'), ('pct_true', 0.7886064052581787)])
OrderedDict([('tweet', 'The economy is doing great, and Michiganders are feeling the positive effects. Since @realDonaldTrump was elected 102,000 jobs have been created in #Michigan—including 17,000 manufacturing jobs. This is a huge victory for Michigan’s 10th district and the entire state. #SOTU'), ('dates', '2020-02-05 02:12:14'), ('file_name', 'Paul Mitchell.csv'), ('pct_true', 0.6047995090484619)])
OrderedDict([('tweet', 'Alexander Hamilton’s fears have been realized, and #impeachment has become weaponized. I do not believe President Trump’s actions constitute high crimes and misdemeanors and therefore voted against both Articles of Impeachment.'), ('dates', '2019-12-19 01:55:14'), ('file_name', 'Paul Mitchell.csv'), ('pct_true', 0.8036172986030579)])
OrderedDict([('tweet', 'Disturbing that the Administration is trying to uproot environmental protections during #COVID19, hoping no one will notice. We noticed, and we strongly condemn these actions that obstruct the public’s basic right to participate in the rulemaking process. https://t.co/dNiXTiLZ7y'), ('dates', '2020-04-16 14:40:42'), ('file_name', 'Paul Tonko.csv'), ('pct_true', 0.8081551194190979)])
OrderedDict([('tweet', 'We cannot leave our frontline workers without the supplies they need to save American lives. The Medical Supply Chain Emergency Act forces the President to federalize American manufacturing &amp; end the supply shortage. We need the #DefenseProductionActNOW\nhttps://t.co/Sj1cIEa0e1'), ('dates', '2020-03-27 18:56:11'), ('file_name', 'Paul Tonko.csv'), ('pct_true', 0.5930067896842957)])
OrderedDict([('tweet', "We cannot fight #COVID19—or any crisis—without science.\n\nYet, that's exactly what EPA intends to do by ignoring scientific &amp; medical studies with conclusions they don’t like. If public health experts &amp; scientists are sidelined, we all will pay the price.\nhttps://t.co/BI9ScJl0Hc"), ('dates', '2020-03-26 20:21:41'), ('file_name', 'Paul Tonko.csv'), ('pct_true', 0.8206358551979065)])
OrderedDict([('tweet', 'Trump Admin is back again with an EPA rule that will let them pick studies they like and throw away ones they don’t, under the guise of ‘open data’. Science doesn’t work that way. \n\nLet’s trust science &amp; facts, not politics, to keep our air &amp; water safe. \nhttps://t.co/6twp7sKwP4'), ('dates', '2020-03-04 18:18:58'), ('file_name', 'Paul Tonko.csv'), ('pct_true', 0.800708532333374)])
OrderedDict([('tweet', "Utah recently became the 19th state to ban conversion “therapy.” Congress must end this dangerous, hateful practice nationwide. That's why I support @RepSeanMaloney’s Prohibition of Medicaid Funding for Conversion Therapy Act. Our #LGBTQ kids deserve better."), ('dates', '2020-02-05 19:55:33'), ('file_name', 'Paul Tonko.csv'), ('pct_true', 0.8005372285842896)])
OrderedDict([('tweet', 'On the 75th anniversary of the liberation of Auschwitz, #WeRemember the millions of victims who lost their lives &amp; pledge to fight anti-Semitism, racism and hate wherever it takes root. \n#HolocaustMemorialDay #NeverForget \n\nhttps://t.co/o82n7p4bNF'), ('dates', '2020-01-27 20:28:23'), ('file_name', 'Paul Tonko.csv'), ('pct_true', 0.7151039242744446)])
OrderedDict([('tweet', 'The president delayed using the DPA for tests and protective equipment, but is now using it to ensure profits for large companies at the expense of a primarily Latino workforce. This is a dangerous, racist decision that will cost lives. https://t.co/flEipzcVui'), ('dates', '2020-04-28 18:44:10'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.6208582520484924)])
OrderedDict([('tweet', 'This isn’t the time for misinformation or “hunches” from the president. This is the time to empower experts and listen to what the science is telling us. https://t.co/l2kcrcKjgy'), ('dates', '2020-03-05 17:13:46'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.5610311031341553)])
OrderedDict([('tweet', "Whether they want to admit it or not, the Trump Administration's #RemainInMexico policy puts asylum seekers and refugees in harm's way. These are people fleeing unimaginable violence in their home countries. They should not be subjected to that same violence when they reach ours. https://t.co/WaCJmp0eqc"), ('dates', '2020-02-26 21:26:35'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.7396706938743591)])
OrderedDict([('tweet', "Borrowers shouldn't be weighed down by student loan debt after they've been defrauded by for-profit colleges. Today, the House voted to reverse Secretary DeVos' latest attempt to put for-profit institutions ahead of students. https://t.co/Nn8rqiCgo5"), ('dates', '2020-01-16 20:30:57'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.6859822273254395)])
OrderedDict([('tweet', 'The president is trying to take money from the men and women of our military to build a wall that’s already proven to be useless. Remind me—how does this make us safer, @realDonaldtrump?\nhttps://t.co/i382raZDBu'), ('dates', '2020-01-14 22:52:42'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.5443902015686035)])
OrderedDict([('tweet', 'The president has spent most of his time in office trying to take health care coverage away from millions of Americans, including those with pre-existing conditions. We’re not going to let that happen.\nhttps://t.co/Z1meHvJE5U'), ('dates', '2020-01-13 21:11:58'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.7462121248245239)])
OrderedDict([('tweet', 'From lowering health care costs to standing up for veterans, the House has passed more than 275 bipartisan bills that Senator McConnell refuses to consider. House Democrats are working #ForThePeople, it’s time for the Senate to do the same. https://t.co/J5cfR9ZJtr'), ('dates', '2019-12-04 18:09:44'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.5814902782440186)])
OrderedDict([('tweet', 'Dreamers aren’t pawns for the president to play political games with. If @realDonaldTrump was serious about #DACA, he could take action to protect the program today. https://t.co/WfHrBvmh6W'), ('dates', '2019-10-09 18:45:11'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.5263129472732544)])
OrderedDict([('tweet', 'This is disgusting. The president has no sense of the values that made our country great. https://t.co/dWQWR8uXl8'), ('dates', '2019-10-02 15:50:39'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.8426403999328613)])
OrderedDict([('tweet', 'No one is above the law. The House should respond to the latest allegations against the president by opening an impeachment inquiry immediately. https://t.co/zH3GMpwB97'), ('dates', '2019-09-24 18:18:55'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.6898900270462036)])
OrderedDict([('tweet', 'We should call this what it is—a partisan power grab designed to hurt Americans. The president is risking the health of California families and adding to the climate crisis. If he won’t join us in preventing deadly pollution, automakers should meet these standards voluntarily. https://t.co/tsN7x7IbIb'), ('dates', '2019-09-18 18:38:47'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.7001510858535767)])
OrderedDict([('tweet', "Today marks 18 years since a cowardly act of terror took the lives of 2,977 Americans. We will #NeverForget the lives of those who were lost, or the heroism of those who rushed into harm's way to save the lives of others.  https://t.co/5Eh4GWUKEC"), ('dates', '2019-09-11 14:06:37'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.6156182885169983)])
OrderedDict([('tweet', "The president's comments this weekend were bigoted and racist, and so is the president."), ('dates', '2019-07-15 12:37:26'), ('file_name', 'Pete Aguilar.csv'), ('pct_true', 0.6275761127471924)])
OrderedDict([('tweet', '#OTD 75 years ago, victory in Europe was declared when Nazi Germany unconditionally surrendered to the Allied Powers. We will never forget the bravery &amp; sacrifice made by all of our veterans who fought to preserve democracy &amp; defeat the Nazi regime. #VEDay https://t.co/VCkrnfiS90 https://t.co/hhix964UBz'), ('dates', '2020-05-08 16:01:54'), ('file_name', 'Pete Olson.csv'), ('pct_true', 0.8419471383094788)])
OrderedDict([('tweet', '894 days have passed since the #Citgo6 were wrongfully imprisoned in Venezuela.\n\n894 days have passed since they have held their families.\n\n894 days have passed since they have set foot in America.\n\nWe will #NeverGiveUp until they are released back to the USA! https://t.co/CECkCnITGG'), ('dates', '2020-05-03 15:40:14'), ('file_name', 'Pete Olson.csv'), ('pct_true', 0.6834034323692322)])
OrderedDict([('tweet', 'The #Citgo6 have not set foot on American soil in 886 days.\n\nThey have missed over two years worth of birthdays, anniversaries and holidays with their loved ones because of their wrongful imprisonment.\n\nWe will #NeverGiveUp until they are reunited with their families in the USA! https://t.co/NIPHpzR5VN'), ('dates', '2020-04-25 15:36:24'), ('file_name', 'Pete Olson.csv'), ('pct_true', 0.6086154580116272)])
OrderedDict([('tweet', 'American workers are desperately hurting, hospitals are in need of critical resources and small businesses are in dire straits. As a fiscal conservative, I’m always wary of large spending bills, but this is a lifeline America needs. My full statement: https://t.co/BIg3ZIJq1v https://t.co/BptV1rH8MT'), ('dates', '2020-04-23 22:24:55'), ('file_name', 'Pete Olson.csv'), ('pct_true', 0.580594539642334)])
OrderedDict([('tweet', '883 days ago, the #Citgo6 were arrested and wrongly imprisoned in Venezuela.\n\nThe 883-day-long ordeal that Jorge, Tomeu, Alirio Jose, Jose Luis, Gustavo and Jose are enduring must end. They must be released back to their families in the USA! #NeverGiveUp https://t.co/zrWIgTQjBu'), ('dates', '2020-04-22 15:31:51'), ('file_name', 'Pete Olson.csv'), ('pct_true', 0.5538849234580994)])
OrderedDict([('tweet', 'COVID-19 has exposed the dangers of reliance on China for the medications so many Americans depend on. That’s why I introduced legislation to require federal agencies to purchase drugs made in the USA &amp; reduce dependence on nations like China. Learn more: https://t.co/JJiabM5s87 https://t.co/dqemNzupZ6'), ('dates', '2020-05-05 17:09:00'), ('file_name', 'Pete Stauber.csv'), ('pct_true', 0.6808570623397827)])
OrderedDict([('tweet', 'It has been 4 days since Democrat leadership let the Paycheck Protection Program run out of funding. This program has served as a vital lifeline for small biz and workers, so Democrats must abandon their political games and join Republicans to replenish its funding ASAP. https://t.co/mbYWhZCa16'), ('dates', '2020-04-20 16:29:38'), ('file_name', 'Pete Stauber.csv'), ('pct_true', 0.7730041146278381)])
OrderedDict([('tweet', '.@GOPLeader is absolutely right. For too long, we have relied on nations that do not have our best interests at heart. \n\nWe need to build support for a supply chain that is made in America, manufactured in America, and mined in America! https://t.co/H1Fw0oU66C'), ('dates', '2020-04-09 03:52:23'), ('file_name', 'Pete Stauber.csv'), ('pct_true', 0.540248453617096)])
OrderedDict([('tweet', ".@POTUS's decision to suspend travel from Europe was the right call as a large number of coronavirus cases are now originating there. Plz remember that American citizens &amp; legal residents can return from Europe during the travel restriction after they go through screening."), ('dates', '2020-03-12 19:59:38'), ('file_name', 'Pete Stauber.csv'), ('pct_true', 0.7994753122329712)])
OrderedDict([('tweet', "Too many Americans can't afford lifesaving medications, so @realDonaldTrump is correct to call on Congress to pass bipartisan legislation that will lower the cost of prescription drugs! #SOTU2020"), ('dates', '2020-02-05 02:55:43'), ('file_name', 'Pete Stauber.csv'), ('pct_true', 0.7423729300498962)])
OrderedDict([('tweet', 'Under @realDonaldTrump, 58,000 jobs have been created in Minnesota and unemployment is at 3.3 percent. We are experiencing the best economy of our lifetime! #SOTU2020'), ('dates', '2020-02-05 02:13:34'), ('file_name', 'Pete Stauber.csv'), ('pct_true', 0.6617919206619263)])
OrderedDict([('tweet', 'Trump wants to spend $500 million just to PAINT his border wall black—which will do nothing to stop illegal immigration—because he thinks it looks better. \n \nImagine the PPE we could purchase for healthcare providers &amp; frontline workers with $500 million.\n\nhttps://t.co/8zZPuwh2UW'), ('dates', '2020-05-08 17:31:10'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.5494422912597656)])
OrderedDict([('tweet', 'For decades, the GOP has tried to destroy @usps. Millions of Americans rely on this vital public service—we cannot let it go bankrupt.\n\nMy bill would save the postal service more than $5 billion a year and put them on track towards financial solvency. \nhttps://t.co/LK1u7mWlRx'), ('dates', '2020-05-05 23:02:05'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.804973304271698)])
OrderedDict([('tweet', 'In the midst of the #COVID19 crisis, Trump is blatantly continuing his toxic anti-environmental agenda. On the 50th anniversary of #EarthDay, I am committed to fighting his radical climate-denier policies for a greener, healthier future. \n \nhttps://t.co/29Hwx9kCvY'), ('dates', '2020-04-22 17:24:09'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.6318823099136353)])
OrderedDict([('tweet', '.@speakerpelosi speaks the truth: Even though Donald Trump was warned in advance of this pandemic, he ignored it. His recklessness and narcissism caused unnecessary deaths, economic uncertainty, and a crisis that will take our nation years to recover from. https://t.co/JZZKrSxXgy'), ('dates', '2020-04-15 21:19:18'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.7412382364273071)])
OrderedDict([('tweet', "Cutting off funding to the World Health Organization in the midst of a pandemic isn't just petty--it's downright dangerous. How many people will die while Trump tries to pin the blame for his disastrous response to #COVID19 on anybody but himself?\nhttps://t.co/OUYrl5thMG"), ('dates', '2020-04-15 19:07:27'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.699701189994812)])
OrderedDict([('tweet', 'In the wake of disasters like #COVID19, scammers may solicit money or financial info by phone or email. The IRS will NEVER call to verify your financial info for relief payments. Be on the lookout for schemes, and learn more by visiting https://t.co/MPQFMyenby'), ('dates', '2020-04-03 23:31:20'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.5290592312812805)])
OrderedDict([('tweet', 'After the 2008 crisis, 3.5 million Social Security beneficiaries and veterans did not receive their stimulus payments because of bureaucratic red tape. Today, I urged the Trump administration to not make this same mistake during the #COVID-19 crisis. https://t.co/hQN34oPu2F'), ('dates', '2020-04-03 21:15:16'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.7177289724349976)])
OrderedDict([('tweet', "Our nation is in crisis. Trump's singular focus ought to be stopping the spread of #COVID19 &amp; our economic recovery.Instead, he's rolling back fuel efficiency standards, adding billions of tons of CO2 emissions and putting public health further at-risk. https://t.co/4H0jVJ4iUZ"), ('dates', '2020-03-31 17:24:33'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.601040780544281)])
OrderedDict([('tweet', 'Trump is *finally* beginning to use the Defense Production Act, something he should have done weeks ago. Delay has meant health care workers have gone without critical protective equipment. We need to use the DPA to its full extent. Anything less is unacceptable. #DPANow'), ('dates', '2020-03-27 21:08:15'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.8169019222259521)])
OrderedDict([('tweet', 'The GOP #COVID19 relief package echoes the Wall Street bailout of 2008, including a $500B #slushfund with no accountability and no oversight. I opposed the bailout in 2008 and will oppose anything that resembles it today. We need to put working families first.'), ('dates', '2020-03-23 22:07:37'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.6115391850471497)])
OrderedDict([('tweet', 'Trump has invoked the Defense Production Act but says he doesn’t yet want to order domestic production of urgently-needed medical and protective equipment. Hospitals and front-line workers need these critical tools NOW. What is he waiting for? https://t.co/yZiyx2BHVx'), ('dates', '2020-03-19 17:12:31'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.680181622505188)])
OrderedDict([('tweet', ".@HUDgov will end foreclosures until next month--a good 1st step but not nearly enough time for families impacted by this crisis. I've urged the administration to suspend foreclosures on federally-assisted housing until the crisis abates entirely. https://t.co/NXXP6cbEjn"), ('dates', '2020-03-18 20:33:02'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.5224719643592834)])
OrderedDict([('tweet', "The nation is facing unprecedented circumstances. That's why I'm pushing for credit agencies to stop adverse reporting &amp; mortgage lenders to halt foreclosures during this crisis. No one should have to worry about losing the roof over their head during a national emergency."), ('dates', '2020-03-18 16:59:26'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.8451545238494873)])
OrderedDict([('tweet', "It's been a year since the House passed #HR1, a democracy reform &amp; anti-corruption bill that would end big money's dominance in our elections &amp; protect voting rights. Thanks to @senatemajldr, it's languished in the Senate ever since. Mitch, it's time to do your job. #ForThePeople"), ('dates', '2020-03-10 20:54:51'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.9074781537055969)])
OrderedDict([('tweet', 'In 2016, Trump promised to never touch Social Security &amp; Medicare.\n \nFour years later? "We\'ll be cutting" these critical earned benefits.\n \nHis priorities are clear: his latest budget cuts hundreds of billions from Medicaid, Medicare, and Social Security. https://t.co/m3Q9acyAqF'), ('dates', '2020-03-06 17:06:07'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.7160673141479492)])
OrderedDict([('tweet', 'Flash boys are a threat to our economy. My Wall Street Tax Act would impose a modest tax on financial trades to rein in their abusive practices and generate hundreds of billions to invest in an economy that works for everyone, not just the wealthiest. https://t.co/fuwPLWdyho'), ('dates', '2020-03-04 14:37:27'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.5267317295074463)])
OrderedDict([('tweet', 'It’s been one year since the House passed the commonsense #HR8, the Bipartisan Background Checks Act – the first major gun violence prevention reform bill in more than a decade. It’s beyond time for the GOP stonewalling to end. #BackgroundChecksSaveLives #EndGunViolence'), ('dates', '2020-02-27 20:26:23'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.8174507021903992)])
OrderedDict([('tweet', 'Trump’s #PublicCharge rule puts thousands—especially U.S.-citizen children—at risk, &amp; creates unprecedented wealth &amp; age tests. Immigration officers can label someone a public charge if they think they will access benefits in the future. This is wrong. https://t.co/pbapskcvzQ'), ('dates', '2020-02-24 19:46:37'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.5528928637504578)])
OrderedDict([('tweet', "Trump's politicization of the DOJ undermines the rule of law and our democracy’s foundations. AG Barr’s interference in a DOJ investigation as a favor to President Trump is reminiscent of an autocracy, not a democracy. https://t.co/FtGDAyqwfw"), ('dates', '2020-02-13 22:50:42'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.6878758072853088)])
OrderedDict([('tweet', 'The #TrumpBudget is reckless and cruel, with trillions of $ cuts to health care, education, housing, SNAP, environmental protection, and more. These cuts are disastrous for working Americans, families, and seniors who rely on them to survive. https://t.co/g3NLTu8tVO'), ('dates', '2020-02-11 00:25:38'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.5147264003753662)])
OrderedDict([('tweet', 'A new @EconomicPolicy report details how hard Oregon has been hit as a result of trade deficits with China. We need a trade deal that actually addresses the fundamental flaws with the U.S.-China trade relationship and creates a more level playing field for U.S. workers. https://t.co/3Pqx3rnZ4D'), ('dates', '2020-01-31 22:52:55'), ('file_name', 'Peter A. DeFazio.csv'), ('pct_true', 0.6116121411323547)])
OrderedDict([('tweet', 'I am disturbed that the Administration has hesitated to implement these tariffs. For years, China has made promises to reduce their steel overcapacity and combat other unfair trade practices. The time for action is now. https://t.co/HT2kHURDub'), ('dates', '2018-05-30 12:14:47'), ('file_name', 'Peter J. Visclosky.csv'), ('pct_true', 0.518494725227356)])
OrderedDict([('tweet', 'On a legal and level playing field, American steelworkers can compete with anyone in the world, and the U.S. government has a responsibility to ensure that a level playing field is maintained every day. It is imperative that the President act and act now. https://t.co/Y7FIHAH0pY'), ('dates', '2018-03-08 17:05:31'), ('file_name', 'Peter J. Visclosky.csv'), ('pct_true', 0.5001018047332764)])
OrderedDict([('tweet', 'DACA participants work legally, pay taxes, and contribute to the US economy. They do not deserve to be in this situation of uncertainty and fear. I support the DREAM Act and it is past time that Congress allows this important bill to be debated and voted upon.'), ('dates', '2017-11-20 18:12:35'), ('file_name', 'Peter J. Visclosky.csv'), ('pct_true', 0.6869557499885559)])
OrderedDict([('tweet', 'Time for my fellow GOP Senators and Reps to knock off lies that NY wants fed $ for bailout &amp; pension funds. All we want is compensation for COVID-19. For cops, fire and healthcare workers. Almost 20,000 dead in NYS. Not Blue State issue. Red, White &amp; Blue.'), ('dates', '2020-05-05 16:38:17'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.7477049231529236)])
OrderedDict([('tweet', 'Proud to go on national television to condemn @senatemajldr McConnell’s disgraceful remark that states like NY &amp; NJ should go bankrupt rather than receive federal funds to pay cops, firefighters &amp; healthcare workers risking their lives to fight Coronavirus! Morally Indefensible!! https://t.co/naG05O3kQg'), ('dates', '2020-04-23 18:54:48'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.6725598573684692)])
OrderedDict([('tweet', '.@senatemajldr McConnell’s dismissive remark that States devastated by Coronavirus should go bankrupt rather than get the federal assistance they need and deserve is shameful and indefensible. To say that it is “free money” to provide funds for...'), ('dates', '2020-04-23 00:18:13'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8931466341018677)])
OrderedDict([('tweet', 'Neither FBI or Mueller found any evidence of connection btwn Russia &amp; Trump campaign. Only collusion was Russian Intelligence giving phony intel to Christopher Steele for Clinton campaign which inadvertently became Russian beneficiary. Too bad Progressives can’t accept the truth!'), ('dates', '2020-04-15 00:27:01'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.5361505150794983)])
OrderedDict([('tweet', 'Recent declassification of IG report reveals disgraceful abuses of power by FBI in its scandalous investigation of non-existent Russia collusion w Trump campaign. Baseless from start. Almost as shameful is silence by TDS Progressives who continue ignore this threat to democracy.'), ('dates', '2020-04-14 23:26:15'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8444709777832031)])
OrderedDict([('tweet', 'The World Health Organization (WHO) has been irresponsible and shameful in covering up for China and allowing the pandemic to spread. Reforms must be made. The Director-General must resign. U.S. aid must be reduced.'), ('dates', '2020-04-13 13:16:10'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.6247788071632385)])
OrderedDict([('tweet', 'Essential that China’s criminal negligence and responsibility for the pandemic be investigated and that the United States end dependence on China for pharmaceuticals and medical supplies.'), ('dates', '2020-04-11 13:06:32'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.978004515171051)])
OrderedDict([('tweet', '.@BernieSanders finds good things to say about Communist Castro but attacks U.S. ally @netanyahu. Sanders is really dangerous for America’s national security. What is wrong with him?'), ('dates', '2020-02-26 12:14:05'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.9347813129425049)])
OrderedDict([('tweet', '.@BernieSanders praises Communist tyrant Fidel Castro for his “literacy program.” How about killing, torture &amp; imprisonment of his political opponents? Or his persecution of religion? You gotta be kidding! No one could really believe that teaching literacy offsets evil brutality!'), ('dates', '2020-02-25 02:42:46'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.9499025940895081)])
OrderedDict([('tweet', 'Attorney General Barr is a lawyer’s lawyer of absolute integrity. Any Administration would be proud and privileged to have him in its service.'), ('dates', '2020-02-14 16:52:46'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.5195246934890747)])
OrderedDict([('tweet', 'NY “No Bail” law is dangerous and indefensible. Now @BernieSanders wants to have NO Bail for entire country! How many innocent people have to be assaulted and murdered before progressives wise up?!?'), ('dates', '2020-02-08 15:25:57'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8147725462913513)])
OrderedDict([('tweet', 'Time for progressives and media friends to knock off the whining about @POTUS Trump’s legitimate anger after 2 years of shameful Russia probe - enabled by media - and inherently flawed impeachment inquisition. Progressives &amp; media love to attack...'), ('dates', '2020-02-07 15:50:16'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.9452000856399536)])
OrderedDict([('tweet', '75th anniversary of liberation of Auschwitz. Every American must remember the Holocaust and denounce all anti-Semitism whether deadly attacks against Jews or acts of vandalism on synagogues, the BDS movement or statements by Members of Congress. Never Again!!'), ('dates', '2020-01-27 13:30:19'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8115589618682861)])
OrderedDict([('tweet', 'I have again spoken out against House Dems shameful rush to impeach @POTUS Trump b/c he was allegedly a clear &amp; present\xa0danger &amp; then waiting a month before sending Articles of Impeachment to Senate. Ultimate hypocrisy &amp; abuse of power by Dems. Will uphold Constitution &amp; vote NO. https://t.co/xjbgmUW8aP'), ('dates', '2020-01-14 16:09:56'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.7798672914505005)])
OrderedDict([('tweet', 'Voted against Democrat War Powers scheme. Totally counterproductive. Don’t undermine @POTUS Trump when USA needs unity vs. Iran. Don’t blame America first!'), ('dates', '2020-01-10 03:19:46'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.9301514625549316)])
OrderedDict([('tweet', 'Outstanding Democrat and American @JoeLieberman asks “Why can’t the Democratic party’s candidates simply admit Soleimani’s death makes Americans safer?” Time for Americans to stand together and reject knee jerk Trump Derangement Syndrome.'), ('dates', '2020-01-06 15:45:12'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.9332640171051025)])
OrderedDict([('tweet', 'Quoting @mgoodwin_nypost:\n“In a sane country, which America used to be, there would be shared satisfaction over the elimination of Soleimani. Yet Dems apparently outsourced their reactions to robots.” Too bad Blame America First crowd can’t get over TDS &amp; do what’s right for USA.'), ('dates', '2020-01-05 21:06:23'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.9597448706626892)])
OrderedDict([('tweet', 'Vital that Americans unite behind @POTUS Trump decision to attack and kill Iranian terrorist Soleimani. No time for political sniping.'), ('dates', '2020-01-03 03:28:27'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8798535466194153)])
OrderedDict([('tweet', 'When will the mainstream media apologize for enabling and/or covering for flagrant FBI abuses against @POTUS Trump described by IG Horowitz. Shameful violations by FBI Brass and abdication by media. Running out of time for redemption!'), ('dates', '2019-12-12 02:20:49'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.9697401523590088)])
OrderedDict([('tweet', 'Articles of Impeachment vs. @POTUS Trump announced today are shameless, baseless abuse of Congressional power by House Democrats. Liberal media will enable &amp; support. I will defend Constitution &amp; vote NO. Can’t yield to progressive mob rule.'), ('dates', '2019-12-10 14:29:27'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.9375144243240356)])
OrderedDict([('tweet', '.@SpeakerPelosi call for impeachment is a reckless abuse of power intended to overturn the 2016 election. Constitution vests power in the America people not progressive mobs. @POTUS Trump 2020 election guaranteed.'), ('dates', '2019-12-05 14:21:46'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8464347720146179)])
OrderedDict([('tweet', 'Absolutely disgraceful that @FLOTUS was repeatedly booed at Youth Opioid Summit in Baltimore. If any issue should be free from abusive, rude behavior, it should be protecting kids from opioids. Has the left no sense of decency?!?'), ('dates', '2019-11-26 18:26:16'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.5988882780075073)])
OrderedDict([('tweet', 'After weeks of secret testimony and now 2 days of public hearings, the Democrats have not produced any evidence of anything impeachable. Terrible abuse of Congressional power and hypocrisy by progressive politicians and allies in media.'), ('dates', '2019-11-16 15:17:36'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8828061819076538)])
OrderedDict([('tweet', '.@KingJames disgraces himself by dismissing Hong Kong activists and ignoring China’s oppression. Time for @NBA to show some guts. Proud that House passed legislation last night affirming support for Hong Kong.'), ('dates', '2019-10-16 14:57:04'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.6274049878120422)])
OrderedDict([('tweet', 'Strongly disagree with @POTUS Trump decision to allow Erdogan to invade Syria. Betrays Kurds, strengthens ISIS and endangers American homeland.'), ('dates', '2019-10-07 13:14:36'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8626508116722107)])
OrderedDict([('tweet', 'House Democrats must release Volker testimony. Texts show no threats or quid pro quo. Also time for Dems to provide due process, end lynch mob mentality and put up or shut up.'), ('dates', '2019-10-05 12:26:34'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8400812745094299)])
OrderedDict([('tweet', 'Nothing remotely impeachable in transcript. Ukrainian President brought up Giuliani before @POTUS Trump mentioned Biden. No quid pro quo. Pursuing impeachment is indefensible.'), ('dates', '2019-09-25 15:38:35'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8005209565162659)])
OrderedDict([('tweet', 'IG Report on Comey leaking government documents and violating law, procedures and guidelines is devastating. Shows Comey’s bias vs. @POTUS Trump. Expect next IG Report on FISA abuse to be even more scathing.'), ('dates', '2019-08-29 15:26:32'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.5543524026870728)])
OrderedDict([('tweet', 'Have great respect for #NYPD Commissioner O’Neill but disagree with decision to terminate Officer Pantaleo. Cops have to make split second decisions. Pantaleo’s decision was understandable under circumstances.'), ('dates', '2019-08-19 16:53:20'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.5772398710250854)])
OrderedDict([('tweet', 'Not being Italian-American I have no idea whether “Fredo” is an ethnic slur. (We Irish have our own problems.) But I support @ChrisCuomo confronting jerk who insults him in public especially when he’s w family. Love that Queens Fighting Spirit Chris! Illegitimi Non Carborundum!'), ('dates', '2019-08-13 23:58:03'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.8392201662063599)])
OrderedDict([('tweet', 'Shameful and irresponsible for @SenBooker to blame @POTUS Trump for El Paso and Dayton mass murders. Just as would be wrong to blame Presidents Clinton and Obama for Columbine or Sandy Hook.'), ('dates', '2019-08-04 15:52:28'), ('file_name', 'Peter T. King.csv'), ('pct_true', 0.9454877972602844)])
OrderedDict([('tweet', 'Last night the House acted decisively to protect public health and the financial security of families impacted by the #coronavirus outbreak. The Senate and President must act now to put these protections into law. #vtpoli  \n\nSpecifically, the bill includes:'), ('dates', '2020-03-14 14:07:03'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.7948631644248962)])
OrderedDict([('tweet', 'There is no excuse for keeping health experts in the dark about #coronavirus. President Trump needs to allow the experts to do their jobs without his political interference. #vtpoli https://t.co/z0GqnEcvhZ'), ('dates', '2020-03-11 21:48:49'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.5102391242980957)])
OrderedDict([('tweet', 'President Trump wants to raid funds from a program that helps vulnerable families heat their homes to make up for his slow and inadequate #coronavirus response. We can fight this public health threat without shutting off people’s heat. #LIHEAP #vtpoli https://t.co/ATWHo9gNJY'), ('dates', '2020-02-26 22:35:09'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.5655664205551147)])
OrderedDict([('tweet', 'Today, the House finally reaffirmed its constitutional responsibility regarding the use of military force. I fought hard for two bills that will stop President Trump from getting us into a war with Iran as a result of his erratic use of military force. #NoWarWithIran #vtpoli'), ('dates', '2020-01-30 20:48:04'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.8310248255729675)])
OrderedDict([('tweet', 'On the 47th anniversary of #RoevWade, women’s rights are under attack across the country. I am fighting to block outrageous attacks on women’s health and proud that Vermont is a leader in protecting women’s rights. #vtpoli'), ('dates', '2020-01-22 21:13:52'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.6431984305381775)])
OrderedDict([('tweet', '10 years ago the #CitizensUnited decision opened the floodgate of corporate &amp; unregulated dark money in our elections. The House-passed #ForThePeopleAct will keep billionaires &amp; corporations with blank checks out of our political process. Time for the Senate to act. #vtpoli https://t.co/hQpAzqZ3cp'), ('dates', '2020-01-21 19:25:57'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.7176259160041809)])
OrderedDict([('tweet', 'Violence against the LGBTQ+ community is appalling and should be prosecuted to the full extent of the law. On Transgender Day of Remembrance, we remember those in the transgender community who have lost their lives due to hate, violence, and transphobia. #TDOR #vtpoli https://t.co/q2cLS4JXne'), ('dates', '2019-11-20 22:10:01'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.6225895881652832)])
OrderedDict([('tweet', 'Climate change is not a theory. Its effects are being felt now in Vermont. We must tackle this threat urgently and decisively. Time is of the essence. #vtpoli https://t.co/abqOGKrAoL'), ('dates', '2019-09-17 14:53:29'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.5906271934509277)])
OrderedDict([('tweet', 'Why in the world is Medicare prohibited from negotiating bulk price drug discounts? Big Pharma is ripping off taxpayers and seniors! Congress must empower Medicare to negotiate lower drug prices. #vtpoli #StopRXGreed https://t.co/2gChZhQwSV'), ('dates', '2019-09-10 17:36:24'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.9043018817901611)])
OrderedDict([('tweet', 'Trump may not have pulled the trigger but his hateful rhetoric is providing the powder. These massacres are hate crimes. The president is debasing his office with coded but transparent hate speech in our name. Hate speech has consequences: hate crimes. #vtpoli'), ('dates', '2019-08-04 18:44:58'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.9617074728012085)])
OrderedDict([('tweet', 'What’s more “sickening” @senatemajldr is that you and @realDonaldTrump hold the keys to common sense gun reforms. The nation is reeling. Lead on this or get out of the way! #vtpoli https://t.co/q46ioouR4n'), ('dates', '2019-08-04 18:41:11'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.6358544826507568)])
OrderedDict([('tweet', 'Tonight the House voted to condemn President Trump’s racist tweets. Attacking these four Members of Congress was dangerous, appalling and a complete repudiation of the best aspirations of the American people. #vtpoli https://t.co/c8VU9vzjzs'), ('dates', '2019-07-16 22:57:50'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.9372565150260925)])
OrderedDict([('tweet', 'The Trump Administration has embraced a lawsuit being argued right now in the 5th Circuit that would repeal health care coverage for Americans with preexisting conditions. They have no plan to replace it. #ProtectOurCare https://t.co/czdBToMmsc'), ('dates', '2019-07-09 19:45:43'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.7211431860923767)])
OrderedDict([('tweet', 'Those who posted these vile comments should not be wearing the uniform of the @CBP. DHS leaders must come before Congress to explain how they will hold these employees accountable &amp; ensure that CBP culture does not condone this despicable behavior. https://t.co/6MN4CQyWDt'), ('dates', '2019-07-03 17:56:00'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.7692433595657349)])
OrderedDict([('tweet', '“Americans do not want another war in the Middle East. Congress must assert its clear authority under the Constitution to check this erratic President Trump and the ideologues in his administration who seem hell-bent to going to war with Iran.” WATCH HERE: https://t.co/2J42JQr5eA'), ('dates', '2019-06-27 15:55:31'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.8613332509994507)])
OrderedDict([('tweet', 'Today the Trump EPA gutted the #CleanPowerPlan, replacing it with a head-in-the-sand #DirtyPowerScam. Shameful. This cannot stand. #vtpoli  https://t.co/369er8yXe2'), ('dates', '2019-06-19 19:14:46'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.5616856813430786)])
OrderedDict([('tweet', 'In spite of the White House suppressing written @StateDept testimony on climate change for @HouseIntel, hearing witnesses made it quite clear: The Trump Administration is endangering the world by ignoring climate change. #vtpoli https://t.co/0WrGMTyIpP'), ('dates', '2019-06-10 20:19:58'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.7782379984855652)])
OrderedDict([('tweet', '75 years ago U.S. &amp; allied forces landed in Normandy to “bring about the destruction of the German war machine, the elimination of Nazi tyranny over the oppressed peoples of Europe, and security for ourselves in a free world.” Today we remember their bravery &amp; sacrifice. #DDay75 https://t.co/z4aDc23eAh'), ('dates', '2019-06-06 18:50:52'), ('file_name', 'Peter Welch.csv'), ('pct_true', 0.7598037123680115)])
OrderedDict([('tweet', 'Here are5⃣reasons why we need the #PaycheckGuaranteeAct. Take a look and RT to agree.\n \n1⃣It gets millions of workers who have lost their jobs off of unemployment and back on payroll—quickly getting direct support into the hands of workers. And it would be retroactive to March 1.'), ('dates', '2020-05-09 20:04:26'), ('file_name', 'Pramila Jayapal.csv'), ('pct_true', 0.5311720371246338)])
OrderedDict([('tweet', 'Unprecedented crisis deserves unprecedented action, not half measures &amp; exclusion. People struggling with healthcare, unable to pay bills, shuttering businesses. My #PaycheckGuaranteeAct gives workers paychecks &amp; supports survival of biz. We CAN do this.  https://t.co/BgwEgCbws0'), ('dates', '2020-05-09 04:04:40'), ('file_name', 'Pramila Jayapal.csv'), ('pct_true', 0.697138786315918)])
OrderedDict([('tweet', "Trump's payroll tax cut would harm Social Security while benefiting corporations and the wealthy. We must stop him and #ExpandSocialSecurity. I'm proud to join @RepJohnLarson on the Emergency Social Security Benefits Improvement Act to ensure we help our seniors through #COVID19."), ('dates', '2020-05-07 21:30:49'), ('file_name', 'Pramila Jayapal.csv'), ('pct_true', 0.8599727749824524)])
OrderedDict([('tweet', "Not only is the Trump Administration denying how massive this crisis is, but they're denying our local leaders crucial information about how to beat it.\n \nThis incompetence will cost our communities more lives. https://t.co/FLZYCir351"), ('dates', '2020-05-07 19:45:00'), ('file_name', 'Pramila Jayapal.csv'), ('pct_true', 0.5415166020393372)])
OrderedDict([('tweet', 'When European countries have guaranteed the paychecks of workers during this time of need, they have effectively been able to stop mass unemployment. Mass unemployment is a policy choice – and a bad one. It’s time to pass my #PaycheckGuaranteeAct.\nhttps://t.co/gp4OfUM59O'), ('dates', '2020-05-07 16:00:01'), ('file_name', 'Pramila Jayapal.csv'), ('pct_true', 0.5379419326782227)])
OrderedDict([('tweet', 'I’ve heard from small businesses who are worried that PPP’s preconditions may add to their debts while failing to help their employees. My #PaycheckGuaranteeAct would use the existing tax info businesses have with IRS to guarantee 100% of every paycheck. https://t.co/0FO3YsxkE9'), ('dates', '2020-05-05 19:30:00'), ('file_name', 'Pramila Jayapal.csv'), ('pct_true', 0.5133898854255676)])
OrderedDict([('tweet', 'Kudos to @GovInslee for letting science &amp; public health guide the way out of this crisis. This is absolutely the right decision. We’re not in the clear here in #WA &amp; we cannot let up on following social distancing &amp; public health guidance. #StayHomeSaveLives https://t.co/6XwHHhyCjS'), ('dates', '2020-05-02 00:39:32'), ('file_name', 'Pramila Jayapal.csv'), ('pct_true', 0.7038211822509766)])
OrderedDict([('tweet', 'This is incredibly cruel. US citizens married to undocumented immigrants WHO FILE THEIR TAXES with an ITIN number are not eligible for stimulus checks. That makes absolutely no sense whatsoever. BOTH are paying taxes and BOTH should receive checks. https://t.co/Lw8CFAwi1C'), ('dates', '2020-04-28 23:37:09'), ('file_name', 'Pramila Jayapal.csv'), ('pct_true', 0.8174407482147217)])
OrderedDict([('tweet', 'My investigation found that as the Administration instituted its travel ban, Americans fleeing coronavirus outbreaks in Asian and Europe were not rigorously screened on their return, limiting our ability to contain the disease or treat those who had it. https://t.co/CWIXB3ZYVt'), ('dates', '2020-05-08 15:16:45'), ('file_name', 'Raja Krishnamoorthi.csv'), ('pct_true', 0.6566162109375)])
OrderedDict([('tweet', 'I opened an investigation into this very issue last month. Consumers are purchasing standard travel insurance products, thinking they will be covered if they cancel their trip because of coronavirus, but companies are refusing to cover those claims. https://t.co/O5jLhf4A64'), ('dates', '2020-04-29 19:01:24'), ('file_name', 'Raja Krishnamoorthi.csv'), ('pct_true', 0.5693537592887878)])
OrderedDict([('tweet', 'The #FDA has not responded to my letters and while they have taken authorized 8 companies,  there are still more than 150 companies selling junk tests on the market. https://t.co/jQB336niO6'), ('dates', '2020-04-29 16:31:34'), ('file_name', 'Raja Krishnamoorthi.csv'), ('pct_true', 0.6203493475914001)])
OrderedDict([('tweet', "My @OversightDems subcommittee just released preliminary staff findings showing major flaws in the Administration’s handling of COVID-19 tests: the FDA doesn't know if the serological tests on the market work even though these tests are central to WH plans to reopen the country. https://t.co/brKUnrX3FX"), ('dates', '2020-04-24 21:44:42'), ('file_name', 'Raja Krishnamoorthi.csv'), ('pct_true', 0.5359612107276917)])
OrderedDict([('tweet', "It's telling that the the White House thinks the President is behaving more responsibly in suggesting that consuming or injecting disinfectants could cure coronavirus than they think the media and experts are in pointing out that you should ABSOLUTELY NOT DO THAT. https://t.co/xvDkcsFKNn"), ('dates', '2020-04-24 19:08:48'), ('file_name', 'Raja Krishnamoorthi.csv'), ('pct_true', 0.8785876631736755)])
OrderedDict([('tweet', 'While we’re passing another stimulus package, it won’t be the last we need to get through this crisis. If the Senate Majority Leader blocks the next package, he’s not just blocking a piece of legislation, he’s blocking our country’s way forward. https://t.co/Hksn74y28f'), ('dates', '2020-04-23 19:01:01'), ('file_name', 'Raja Krishnamoorthi.csv'), ('pct_true', 0.5923582911491394)])
OrderedDict([('tweet', 'This is unacceptable. Small businesses owners are struggling tremendously, they shouldn’t have to worry about their personal information being stolen too. https://t.co/CvwicBKJKZ'), ('dates', '2020-04-23 11:00:59'), ('file_name', 'Raja Krishnamoorthi.csv'), ('pct_true', 0.5196165442466736)])
OrderedDict([('tweet', 'I can tell you who doesn’t love to see negative oil prices: the millions of Americans whose livelihoods depend on the exploration, extraction, refinement, transportation, marketing, sale, and use of petroleum products.'), ('dates', '2020-04-21 17:12:22'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.8819456100463867)])
OrderedDict([('tweet', ".@realDonaldTrump &amp; @SecretarySonny understand groceries don't just magically appear on store shelves, but involve months of advanced planning &amp; diverse supply chains. Our farmers need support to avoid bankruptcy &amp; imminent collapse of these supply chains. https://t.co/nSwa81wz4M"), ('dates', '2020-04-18 15:14:04'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.5588820576667786)])
OrderedDict([('tweet', 'Only Pelosi &amp; Schumer would take a bipartisan program that is working &amp; try to kill it in favor of their own selfish ambitions. They prefer Americans be jobless &amp; reliant on the government dole, rather than employed with benefits &amp; ready to go back to work https://t.co/ZVEpr95Kzb'), ('dates', '2020-04-16 15:59:06'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.8590883016586304)])
OrderedDict([('tweet', 'House Democrats have started yet another partisan witch hunt to investigate @realDonaldTrump’s execution of the CARES Act after the Dems diverted millions from Americans in crisis. Her thought processes are both delusional &amp; reckless. I once again call for her removal as Speaker.'), ('dates', '2020-04-03 12:21:59'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.9346145987510681)])
OrderedDict([('tweet', '.@realDonaldTrump’s proactive response to the Wuhan coronavirus will limit incoming sources of infection, identify&amp;treat the infected, &amp; protect vulnerable populations from future infection. If China had been transparent &amp; taken outside help to begin with, we’d be better off now.'), ('dates', '2020-03-12 17:41:29'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.5863243937492371)])
OrderedDict([('tweet', "The Senate should take immediate and drastic action to address Senator Schumer's words. This is the kind of incendiary rhetoric that leads to tragic situations like what happened with Congressman Steve Scalise."), ('dates', '2020-03-05 23:12:30'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.6382424831390381)])
OrderedDict([('tweet', 'The New Way Forward Act shows how far off the deep end Nancy Pelosi’s Democrats have gone by decriminalizing illegal immigration &amp; bringing back those deported. We should finish building the wall, crack down on sanctuary cities, and enforce existing laws. https://t.co/umjlPWsAo1'), ('dates', '2020-02-19 12:35:43'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.6251251101493835)])
OrderedDict([('tweet', 'The American people deserve to know how much money was wasted on the sham impeachment of @realDonaldTrump, which is why I joined @RepByrne in support of the Support Taxpayer Accountability for Impeachment (SHAM) Act to call for an audit of funds spent on this charade. https://t.co/MamZFUeyYQ'), ('dates', '2020-02-07 12:26:41'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.5870845317840576)])
OrderedDict([('tweet', "President @realDonaldTrump's acquittal is great news for the American people. Democrats led by Nancy Pelosi and Adam Schiff have spent the better part of a year chasing this sham of an impeachment, a year which could better have been spent supporting the President’s agenda."), ('dates', '2020-02-05 21:41:37'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.8641653060913086)])
OrderedDict([('tweet', "I spoke to @KALBtv5 last night about the President's new trade deal with China and the Democrats' ongoing impeachment charade. They've been trying to unseat him for three years, but he just keeps on WINNING! https://t.co/jaC7U6HsuI"), ('dates', '2020-01-16 16:01:52'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.6171835064888)])
OrderedDict([('tweet', "So, if @realDonaldTrump had notified Pelosi and Congress before taking decisive life-saving action, I truly believe the terrorist Soleimani would still be alive and killing Americans. Congress moves at a snail's pace - thankfully, President Trump moves at light speed."), ('dates', '2020-01-06 17:49:07'), ('file_name', 'Ralph Lee Abraham.csv'), ('pct_true', 0.8647626042366028)])
OrderedDict([('tweet', 'This is truly scary. We knew #china owned state media, but it turns out they can also control the United States’ media. \n\nComrade #bloomberg is not alone, this goes for every outlet mindlessly repeating CCP talking points and “official” #covid figures. https://t.co/ndoiAO8TSI'), ('dates', '2020-04-14 14:23:36'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.8906804323196411)])
OrderedDict([('tweet', 'Leave it to the @WHO to ignore Taiwan’s warnings about the Chinese COVID-cover-up, only to accuse Taiwan of racism without a shred of evidence. \n\nNO MORE FUNDING WITHOUT ACCOUNTABILITY! https://t.co/Hjds1qvAgv'), ('dates', '2020-04-09 20:04:50'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.681534469127655)])
OrderedDict([('tweet', 'Transparency and accountability saves lives.\xa0China’s authoritarian and closed system has cost the world great suffering. \n\nThe Chinese government should be held responsible for its censorship and false propaganda.\n\n#WeWontForget https://t.co/pAMRkEHxpv'), ('dates', '2020-04-03 01:48:39'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.8562665581703186)])
OrderedDict([('tweet', 'In the midst of a crisis, Nancy Pelosi &amp; Co. are actively engaged in mindless obstruction. \n\nHer 1,119-page version of Bill #3 in the House is truly outrageous. Check out a few of her baked in items. \n\nhttps://t.co/TDh1h6M8ar'), ('dates', '2020-03-23 23:39:22'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.6662985682487488)])
OrderedDict([('tweet', '.realDonaldTrump: "The U.S. should be a sanctuary for law-abiding Americans, not criminal aliens...  If you come illegally, you will now be promptly removed."\n\nDemocrats (again): silence, shaking heads, nervous chatter.'), ('dates', '2020-02-05 03:08:38'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.8224073648452759)])
OrderedDict([('tweet', "Imagine siding with a theocratic regime murdering and torturing its own people just because you can't admit that  @realDonaldTrump did the right thing by eliminating a known terrorist #IranProtests2020 \n\nhttps://t.co/9lsbxGQz8e"), ('dates', '2020-01-13 16:25:23'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.6976550817489624)])
OrderedDict([('tweet', "Democrats have chosen to make this day one of the darkest in American politics. It's ridiculous.\n\nThis has never been about some benign phone call. Instead, it's a petulant attempt to damage a President who stands opposed to their far-left dreams for this country.\n\nI'm with #45."), ('dates', '2019-12-18 18:46:42'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.9480401277542114)])
OrderedDict([('tweet', 'If Democrats cared what the American people thought of this #ImpeachmentScam, this poll would give them reason to pause.\n\nBut then again, if Dems cared, they wouldn’t be trying to overturn the people’s vote in the first place.\n\nhttps://t.co/xlf0lDYJzn'), ('dates', '2019-12-16 23:45:56'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.9808743596076965)])
OrderedDict([('tweet', "Does your pocket feel heavier? That's because it is.\n\nNumbers don't lie folks: President Trump's economy has benefited American families. https://t.co/L1qCnCjttT"), ('dates', '2019-10-30 14:35:34'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.6680747270584106)])
OrderedDict([('tweet', "If the Majority truly cared about the will of the American people, they'd make the transcripts public and allow for the people's representatives to be in attendance. No closed doors, no confidential rules, no secrets.  The #SchiffShow needs to end. https://t.co/oGzdyuVHnq"), ('dates', '2019-10-23 16:34:03'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.6698171496391296)])
OrderedDict([('tweet', 'This is what placing people first looks like. The Palmetto State is prospering with HISTORIC LOW levels of #unemployment thanks to @HouseGOP Pro-Growth Policies. https://t.co/mzbNCj198b'), ('dates', '2019-10-18 20:53:41'), ('file_name', 'Ralph Norman.csv'), ('pct_true', 0.6036154627799988)])
OrderedDict([('tweet', 'The fake news can’t stand that some people might not need to submit to the new authoritarianism of the left because they are immune to coronavirus. Modern science disagrees.'), ('dates', '2020-05-05 23:03:49'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.5853533744812012)])
OrderedDict([('tweet', 'It’s important that @realDonaldTrump veto any fake FISA reform that doesn’t prevent this secret court from investigating political campaigns. No presidential candidate of either party should be investigated by this unconstitutional court.'), ('dates', '2020-05-05 19:40:42'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.6287093758583069)])
OrderedDict([('tweet', 'America should not tolerate abhorrent human rights abusers or use your tax dollars to support them. \n\n“Washington should end both civilian and military aid and press the regime to reform its people’s rights and liberties. It is bad enough to tolerate...” https://t.co/FEsgtoKU1r'), ('dates', '2020-04-23 19:30:36'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.83849036693573)])
OrderedDict([('tweet', 'I believe there exists too much danger for nearly half-a-trillion-dollar  spending bills being passed without Congress being in session. A handful of individuals in leadership shouldn’t have this much power. All of Congress should weigh in. They should return at once.'), ('dates', '2020-04-21 21:15:36'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.7985981106758118)])
OrderedDict([('tweet', 'Usually in a totalitarian state they first shut down dissent, then they shut down religion. Gov. Beshear did it backwards, but still the same result.  \n\nKentuckians plan COVID-19 restrictions protest at the Capitol https://t.co/juAF3DvlLn'), ('dates', '2020-04-17 14:33:37'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.6797493100166321)])
OrderedDict([('tweet', 'The constitution doesn’t allow the federal gov’t to become the ultimate regulator of our lives because they wave a doctor’s note. Powers not delegated are RESERVED to states &amp; the PEOPLE. If we dispense with constitutional restraints, we will have more to worry about than a virus'), ('dates', '2020-04-14 15:57:20'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.5315278768539429)])
OrderedDict([('tweet', 'The Constitution guarantees an individual right to keep and bear arms. The protections afforded by the 2nd Amendment are indispensable and they have served our nation well through tranquil times as well as turbulent ones. Our circumstances are no different https://t.co/7fHEoQSYHD'), ('dates', '2020-04-01 13:02:41'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.548790454864502)])
OrderedDict([('tweet', 'Hearing various airlines refused to rescue stranded Americans around the world, including Morocco! Americans are willing to pay for tickets. Airlines are demanding that @StateDept charter flights instead. Why refuse paying customers &amp; ask Congress for a bailout in the same week?'), ('dates', '2020-03-19 22:10:05'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.8569447994232178)])
OrderedDict([('tweet', 'Scandinavians don’t “feel the Bern” — they want Bernie to tell the truth that Nordic success came from capitalism. Exactly the point that \u2066@KelleyAshbyPaul\u2069 and I make in The Case Against Socialism.\n\nBernie Sanders is wrong on democratic socialism... https://t.co/yazeNJ3wIz'), ('dates', '2020-03-16 22:50:08'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.8191559910774231)])
OrderedDict([('tweet', 'Hey \u2066@realDonaldTrump\u2069: Adam Schiff is playing you on FISA reform. Don’t settle for fake reform!  \n\nAdam Schiff FISA reform intervention protects FBI power - Washington Times https://t.co/H1MgZfwhiN'), ('dates', '2020-03-16 17:50:10'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.5991669297218323)])
OrderedDict([('tweet', 'As @realDonaldTrump has repeatedly stated, we should not reauthorize these expiring surveillance powers without real reform. https://t.co/hMtMx7w6Ij'), ('dates', '2020-03-12 22:40:09'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.6841139793395996)])
OrderedDict([('tweet', 'Adam Schiff “strongly” supports the fake FISA reforms. \u2066@realDonaldTrump\u2069 - do you need any more evidence they are selling you a bill of goods? Real reform or VETO.  \n\nHouse passes FISA reform over bipartisan objections - Washington Times https://t.co/d2ZJed17jV'), ('dates', '2020-03-12 21:41:22'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.611504316329956)])
OrderedDict([('tweet', "The ‘Deal' on FISA is weak sauce diluted &amp; made impotent by A.G. Barr.  None of the reforms prevent secret FISA court from abusing the rights of Americans.  None of the reforms prevent a President of either party from a politically motivated investigation.  Big Disappointment!"), ('dates', '2020-03-10 22:29:50'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.744013249874115)])
OrderedDict([('tweet', 'When I asked James comey after the Pulse mass shooting if in retrospect the FBI concluded too soon that Mateen was not a threat. The arrogant Comey refused to admit any mistake &amp; maintained the opinion that Mateen was not a threat. Comey: tell that to the 49 people who died. https://t.co/JSyduiUTjm'), ('dates', '2020-03-06 01:00:45'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.5171027779579163)])
OrderedDict([('tweet', 'Note to Bernie bro socialists: Sweden is NOT a socialist nation. Read Johan Norberg’s great report on free markets in Sweden. Kelley and I quote Norbert in our recent book: The Case Against Socialism. https://t.co/4kgWCh8Gvg'), ('dates', '2020-03-03 14:33:24'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.5830960273742676)])
OrderedDict([('tweet', 'No surprise that A.G. Barr supports reauthorizing Patriot Act without legislative reforms. To protect all Americans from domestic surveillance we absolutely must block the FISA court from spying on Americans! \n https://t.co/OfUEKeft8x'), ('dates', '2020-03-02 19:30:36'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.8817195296287537)])
OrderedDict([('tweet', 'Only government could be so stupid as to deny a merger of bankrupt coal companies trying to re-form. What kind of moron wants to prevent mergers in an industry that has been losing market share for a decade and struggling to survive extinction? https://t.co/brLXVc5R4U'), ('dates', '2020-02-26 20:54:15'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.8032402396202087)])
OrderedDict([('tweet', 'A.G. Barr advocating for ‘clean renewal’ of Patriot Act without any legislation to reform FISA is a disservice to @realDonaldTrump and should be roundly defeated.  \n\nThe secret FISA court should be forbidden from allowing spying on political campaigns ever again — period!'), ('dates', '2020-02-26 16:57:37'), ('file_name', 'Rand Paul.csv'), ('pct_true', 0.8231453895568848)])
OrderedDict([('tweet', 'Pelosi needs to put down the ice cream, put down her ego, stop putting down @realDonaldTrump &amp; get back to work so we can help Americans &amp; our hurting businesses. And this has been “put down in black &amp; white.” Speaker Pelosi: are you listening? https://t.co/oKQILWAeFe'), ('dates', '2020-04-20 16:14:11'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.7585126161575317)])
OrderedDict([('tweet', 'Sad, isn’t it? We put a really good program in place with guidelines that were all hammered out. Obviously it was a big success and more money is needed. It only takes a one page bill to add more money. But the Democrats are playing politics. What else is new? #GetRealPelosi https://t.co/BJCwI4IMAC'), ('dates', '2020-04-17 15:31:41'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.5113611221313477)])
OrderedDict([('tweet', 'Nancy Pelosi &amp; D’s impeached President Trump for “holding up aid to Ukraine.” Now she &amp; Schumer are holding up aid to Americans. What should be done with them? @realDonaldTrump'), ('dates', '2020-03-24 23:37:59'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.811825692653656)])
OrderedDict([('tweet', 'He is doing a great job, Kristie! Despite being maligned, attacked &amp; basically slandered by the Liberals &amp; the Fake News Media. Yet our President perseveres. Other countries hate him: China, Russia...etc because he’s making America great again. Yet he perseveres @realDonaldTrump https://t.co/lmAFPJTr0x'), ('dates', '2020-03-24 11:41:19'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.5247513055801392)])
OrderedDict([('tweet', 'Americans stand with the people of #Iran as they raise their voices against an oppressive regime. \n\n#SOTU2020'), ('dates', '2020-02-05 03:17:45'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.6582540273666382)])
OrderedDict([('tweet', 'President Trump: “The United States should be a sanctuary for law abiding Americans not criminal aliens.” Yet the Democrats won’t stand and clap. Where does their allegiance lie?'), ('dates', '2020-02-05 03:03:40'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.9268196821212769)])
OrderedDict([('tweet', '-120,000 criminal aliens charged with nearly 10,000 burglaries\n-5,000 sexual assaults\n-45,000 violent assaults\n-2,000 murders\n\nIgnoring these statistics to further a political agenda is not putting America first! Our job is to protect our citizens! \n\n#SOTU'), ('dates', '2020-02-05 03:01:11'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.7352296113967896)])
OrderedDict([('tweet', 'Democrats won’t even stand for putting America First... they must be waiting on the Iowa caucuses just to see which one of their socialistic candidates is “first.” Must really be sad to hitch your wagon to a socialistic philosophy that is doomed to fail! Americans will remember!'), ('dates', '2020-02-05 02:38:48'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.9565989971160889)])
OrderedDict([('tweet', 'Right again Mr President! Problem could be that the DNCC hasn’t decided who the’re gonna cheat against like they did Sanders last time. Our country &amp; Republicans thoug have almost 4 years of success thanks to you, Mr President! How can the D’s compete with that? @realDonaldTrump https://t.co/dVplXs0pB4'), ('dates', '2020-02-04 11:39:36'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.7148797512054443)])
OrderedDict([('tweet', 'And the truth is, the burden of proof lies with the Democrats and they failed to prove their case. 1/3 https://t.co/zo5kYXmtEF'), ('dates', '2020-02-03 20:02:15'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.6020727753639221)])
OrderedDict([('tweet', 'Adam Schiff says: Senators: there is a storm blowing through this Capitol. Darn right there is! The Crazy Socialist D’s have been shouting “Impeach 45!” Or “We have to impeach this president because he might get re-elected.”'), ('dates', '2020-01-31 21:40:46'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.6888809204101562)])
OrderedDict([('tweet', '.@RepAdamSchiff says  a trial without   witnesses is no trial at all. That’s because when they prosecuted their “case” in the House they gave the Republicans a chance to call witnesses? O right - NOT! And now he wants “the jury” to call more witnesses? 1/3'), ('dates', '2020-01-31 21:35:28'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.8400818705558777)])
OrderedDict([('tweet', 'Watching the Democrat Zoo in the Senate. Sylvia Garcia said the the Senators were actually “judges.” Said she was a lawyer &amp; a judge. Sooooo if the Senators were the “judges”... who/what is John Roberts? The Bailiff? She may need to retake law courses.'), ('dates', '2020-01-24 21:38:07'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.8925366997718811)])
OrderedDict([('tweet', 'Those Dems are elated that they have a chance to thwart the will of the people &amp; overturn an election they never accepted. “D” in this instance stands for Delusional, Dastardly, Devilish, Dumbfounded, Dishonest, Downtrodden, Disconnected &amp; Dismayed. Darn! @realDonaldTrump https://t.co/CPsImANQO5'), ('dates', '2020-01-21 13:58:29'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.767981231212616)])
OrderedDict([('tweet', 'Our great president is under attack by a bunch of crazy Democrats! Here is yet one more video that shows that they know that this is improper. Shame on them. What could we have gotten done if the Democrats hadn’t been so obstructionist? https://t.co/etK4CV1Ms7'), ('dates', '2020-01-20 04:32:32'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.7411647439002991)])
OrderedDict([('tweet', 'Sad, but the D’s seem giddy at the proposition of undoing the 2016 election AND the will of Americans. Maxine Waters, Al Green &amp; others (especially the fake news) have made it no secret that their aim was &amp; is to take out a duly elected president. @realDonaldTrump #NotOnOurWatch https://t.co/0sZX9VN04o'), ('dates', '2020-01-16 20:33:02'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.7343431711196899)])
OrderedDict([('tweet', 'While @SpeakerPelosi was preoccupied handing out custom pens after signing impeachment articles to her henchmen (bizarre), @realDonaldTrump was signing a trade deal with China.\n\n@CNN\u2069 even thinks it’s weird &amp; unusual. \n\nWatch: https://t.co/G2VsicZZPt'), ('dates', '2020-01-16 18:06:10'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.8573729991912842)])
OrderedDict([('tweet', '“Pelosi &amp; Company’s” days are numbered. Americans are seeing them for who they are. Our great President is persevering in spite of them! Look how far we’ve come under his leadership. How much further could we have come with just a little help from the D’s? Sad! @realDonaldTrump https://t.co/jokK3SmStN'), ('dates', '2020-01-15 19:45:25'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.5183252096176147)])
OrderedDict([('tweet', 'Alec Baldwin lecturing on “12 cylinder economy” issues, &amp; morality?How “rich” is that? Not as rich as “out of touch actor” who lectures working Americans:beneficiaries of Trump’s great economy! Hollywood actors don’t have a clue! Especially Baldwin! @realDonaldTrump @AlecBaldwin https://t.co/s4D1f96O99'), ('dates', '2020-01-15 01:15:14'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.5386829376220703)])
OrderedDict([('tweet', 'Isn’t it odd Lopez can say something “crazy” &amp; some say stupid; (welcome to the Liberal Mindset - They would let the ship sink, just because they can’t be the Capt!) But when Conservatives &amp; the GOP exercise free speech... that’s a different story! Thank God for @realDonaldTrump https://t.co/BWnaliMmFI'), ('dates', '2020-01-10 21:04:00'), ('file_name', 'Randy K. Weber, Sr..csv'), ('pct_true', 0.9415581226348877)])
OrderedDict([('tweet', 'Trump is now blaming the @WHO for his disastrous #COVIDー19 response. Yes, it needs to do more. That’s why I‘ve pushed to increase its funding, as he should instead of threatening to withold it. We cannot make it through this crisis without their help.\nhttps://t.co/8cmM5ad2M9'), ('dates', '2020-04-08 15:08:29'), ('file_name', 'Rashida Tlaib.csv'), ('pct_true', 0.7371988892555237)])
OrderedDict([('tweet', "Trump is wrong again when he is focused on another tax cut for business as an economic stimulus. It won't work. I agree with @Claudia_Sahm, stimulus payments to individuals are the best way to help during an economic downturn. https://t.co/ppttJUG0EZ"), ('dates', '2020-03-10 14:57:13'), ('file_name', 'Rashida Tlaib.csv'), ('pct_true', 0.5106803178787231)])
OrderedDict([('tweet', 'People carrying the burden of student debt are under immense financial strain during this time. Glad to see @CAgovernor sign this EO to stop debt collectors taking your COVID-19 related financial assistance like unemployment benefits or stimulus payments. https://t.co/wD8TlHaiDH'), ('dates', '2020-04-29 22:29:00'), ('file_name', 'Raul Ruiz.csv'), ('pct_true', 0.743007481098175)])
OrderedDict([('tweet', 'Repealing the #CleanPowerPlan is another attempt from the Trump Administration to undermine clean air standards that protect our health. I joined @RepPaulTonko &amp; @RepHuffman in filing an amicus brief against this repeal. Breathing clean air is a common good for everyone.'), ('dates', '2020-04-29 02:10:00'), ('file_name', 'Raul Ruiz.csv'), ('pct_true', 0.5769797563552856)])
OrderedDict([('tweet', 'Today is #HolocaustRemembranceDay—a time we pause to remember the more than six million Jews who died at the hands of the Nazis. While we can’t hold in-person ceremonies right now, you can watch ceremonies and testimonials online here. \nhttps://t.co/rpoyZ7hIdk\n#YomHaShoah'), ('dates', '2020-04-22 00:41:00'), ('file_name', 'Raul Ruiz.csv'), ('pct_true', 0.8173563480377197)])
OrderedDict([('tweet', "Trump is using a pandemic to weaken our environmental laws without public oversight, and he isn’t sparing the Grand Canyon. I have a bill to permanently protect this iconic site from uranium mining. \n\nIt's time for the Senate to pass it. https://t.co/LDEIsyH6n0"), ('dates', '2020-05-08 16:25:07'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.6547255516052246)])
OrderedDict([('tweet', "Is Trump capable of appointing anyone who's not a typical crony to positions of power? USPS is going through a major crisis and need real leadership AND funding, not typical Trump posturing. https://t.co/ANkcrpq3j0"), ('dates', '2020-05-07 23:55:07'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.6070927381515503)])
OrderedDict([('tweet', "Perhaps it's the lack of workplace protections &amp; access to testing that is responsible for the outbreaks in meatpacking plants--not the workers themselves. The excuses the Trump Admin makes for their lack in any coherent #COVID19 strategy are ridiculous.\n https://t.co/Br8UCx0pLR"), ('dates', '2020-05-07 21:40:03'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.5068767666816711)])
OrderedDict([('tweet', "Even in the midst of a pandemic, Trump can't help but waste money on his useless wall. While hospitals need PPE and small businesses need funds to survive, Trump is trying to spend over $500 million to paint his vanity wall black.  https://t.co/KGhcMeqfUx"), ('dates', '2020-05-07 18:55:10'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.8793927431106567)])
OrderedDict([('tweet', "#SCOTUS is hearing cases on the ACA's birth control benefit. \n\nUnder Trump-Pence rules, a woman could lose birth control coverage otherwise required by law because her boss has a personal religious objection to it. \n\nYour boss’s beliefs shouldn’t dictate the care you get. https://t.co/wZX3YeuQQU"), ('dates', '2020-05-06 19:40:06'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.8204827308654785)])
OrderedDict([('tweet', 'The Trump Administration is using a pandemic to shove through an unpopular policy that pollutes Native American communities. \n\nCommunities want to keep their land, water, and air safe--not polluted with impunity. \n\nInterior must extend the comment period. https://t.co/tsx5sOu3ng'), ('dates', '2020-05-01 13:36:29'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.6592290997505188)])
OrderedDict([('tweet', "This is what we were afraid of. \n\nMassive loans without accountability to giant corporations that have proved time and time again that they are not accountable to their workers. This is why we're demanding strong oversight of these funds. https://t.co/Uk8BWSi5YW"), ('dates', '2020-04-29 20:35:07'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.6609519720077515)])
OrderedDict([('tweet', "The PPE shortage is not 'fake news' as the President like to call it. \n\nThe fact is that PPE is not arriving to our hospital workers who desperately need it. Instead of complaining about coverage, Trump should be working to ensure our workers are safe. https://t.co/6nGR7Wc72g"), ('dates', '2020-04-27 19:05:09'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.8323920965194702)])
OrderedDict([('tweet', "This isn't about protecting jobs. It's about xenophobia. \n\nPresident Trump's response to #COVID19 is a national disgrace, and now he's using the crisis to implement his anti-immigrant agenda. We should instead be focused on access to testing and treatment. https://t.co/Av5LgAn018"), ('dates', '2020-04-21 19:30:15'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.7345170974731445)])
OrderedDict([('tweet', 'On this day 10 years ago,11 people died in the largest offshore oil spill in world history that dumped more than 5 million barrels of oil into the Gulf of Mexico.\n\nThis can never happen again. \n\nWe must end Trump’s greedy, dangerous “energy dominance” agenda and #ProtectOurCoast. https://t.co/zcJWWE26hV'), ('dates', '2020-04-20 19:30:40'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.6919406056404114)])
OrderedDict([('tweet', 'In the same way that Trump weakened our health care system &amp; refused to listen to experts on #Coronavirus, he’s spent the last 3 years weakening safety regulations.\n\n#DeepwaterHorizon could have been prevented if Big Oil had chosen to #ProtectOurCoast rather than its own profits. https://t.co/g5daWoJ0sH'), ('dates', '2020-04-20 16:44:58'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.8141420483589172)])
OrderedDict([('tweet', "In the rush to reopen the country, we cannot allow Trump's failed response to the outbreak write the playbook. Our strategy should be based on science and protecting lives--not political considerations. https://t.co/E6RpIYALBv"), ('dates', '2020-04-17 23:20:04'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.5075847506523132)])
OrderedDict([('tweet', 'In the midst of a pandemic, it is now more important than ever that we ensure that everyone is covered and has access to the healthcare they need during this time. \n\nMeanwhile the Trump Administration is still hell-bent on dismantling the ACA in court. https://t.co/2E1852hMzB'), ('dates', '2020-04-14 19:35:07'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.5266162753105164)])
OrderedDict([('tweet', "Stories like Ingrid's are all too common. She's undocumented, pays taxes using an ITIN, but is blocked from receiving any relief under the current #COVID19 relief. My legislation will help workers like her make it through this crisis.  https://t.co/2Yhuv5usAH"), ('dates', '2020-04-08 23:20:05'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.5304344892501831)])
OrderedDict([('tweet', 'Simply unacceptable. \n\nHospitals are raising the alarm on the challenges they face to Administration.\n\nInstead of acknowledging their concern, Trump says the report is wrong. \n\nWe need real leadership to protect healthcare workers.  https://t.co/hIyCgdNx9p'), ('dates', '2020-04-07 20:25:10'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.5408583283424377)])
OrderedDict([('tweet', 'It is shameful that the EPA is using #COVID19 as an excuse to allow polluters more leeway to dirty our air an water. \n\nThese regulations help protect us and should remain in place. https://t.co/z47sttICZU'), ('dates', '2020-03-28 00:35:04'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.7644444704055786)])
OrderedDict([('tweet', "Indian Country has been ignored for far too long, and we can't allow #COVID19 to spread through their communities and not provide them with the resources they need to combat it. \n\nI spoke with IHS yesterday to discuss their response plan.  https://t.co/X5MWFQN8Ix"), ('dates', '2020-03-25 16:05:13'), ('file_name', 'Raúl M. Grijalva.csv'), ('pct_true', 0.7173084020614624)])
OrderedDict([('tweet', 'The results of this investigation reaffirm Congressional intent &amp; vindicate the thousands of small business owners left out in the cold by SBA’s mismanagement of the Paycheck Protection Program. https://t.co/H9KtLMFSd0'), ('dates', '2020-05-08 23:51:55'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.6251819729804993)])
OrderedDict([('tweet', 'The facts are clear, Flynn was convicted—twice. I’m asking for an immediate IG investigation into DOJ dropping the case &amp; unprecedented politicization of federal law enforcement—further decimating its integrity. https://t.co/KBatfCX3BT'), ('dates', '2020-05-08 19:07:33'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.5830255150794983)])
OrderedDict([('tweet', 'A war against an insidious global pandemic is now unavoidable but Americans should be spared an unnecessary military conflict that harms our national interests without protecting our national security. The Constitution is clear: the power to declare war lies with Congress. https://t.co/tsKrYYf0Cw'), ('dates', '2020-05-07 23:09:26'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.7415570616722107)])
OrderedDict([('tweet', 'Never mind that Flynn pleaded guilty to lying to investigators, AG Barr’s DOJ will drop the case. Democracies become autocratic when officials in power use crises to cover up their flagrant disrespect for the rule of law. https://t.co/LcGN1MVucT'), ('dates', '2020-05-07 21:41:45'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.9287941455841064)])
OrderedDict([('tweet', 'Unsafe on any seas. Cruise lines need far reaching health care upgrades &amp; reforms before they set sail again—with passenger lives at stake. They also fail fundamental environmental, labor &amp; other regulations. Great column by @gailcollins. https://t.co/SvUXVYKg2g'), ('dates', '2020-05-07 14:15:51'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.5843467712402344)])
OrderedDict([('tweet', 'For what seems like the millionth time, the Trump/Pence admin is trying to get between a person’s body &amp; their reproductive rights. Today’s SCOTUS case has the potential to let bosses block access to birth control &amp; dismantle a critical ACA protection.  \nhttps://t.co/a7TrxGXi3k'), ('dates', '2020-05-06 21:48:02'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.5737302303314209)])
OrderedDict([('tweet', 'Judicial nominee Justin Walker believes the ACA should be dismantled. His beliefs would hurt real people, like Conner, a 9 year old CT boy, who would have his lifesaving health care ripped away from him if Walker’s “principles” capture our courts. #StopWalker https://t.co/H2f7t0bhTZ'), ('dates', '2020-05-06 16:54:11'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.815960705280304)])
OrderedDict([('tweet', 'Bosses, businesses, &amp; schools shouldn’t dictate if people can access birth control. If Trump/Pence win this SCOTUS case hundreds of thousands of people could lose birth control coverage through the ACA. https://t.co/Hh5gp95U0H https://t.co/AH1rhdrPfs'), ('dates', '2020-05-05 19:23:18'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.7182474732398987)])
OrderedDict([('tweet', 'In the middle of a global pandemic McConnell’s seemingly singular focus is to rush through the nomination of his anti-health care crony—Justin Walker. Forcing through judges who will gut the ACA &amp; cost millions access to health care is not only dangerous, but deadly. #StopWalker'), ('dates', '2020-05-05 17:11:46'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.9253716468811035)])
OrderedDict([('tweet', 'No need to shut the courthouse door to reopen the economy. McConnell’s blanket legal shield for lawbreakers has an unacceptable price—key Constitutional rights. Victims deserve their day in court. https://t.co/zjj5WeG9k8'), ('dates', '2020-05-05 15:50:02'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.683129072189331)])
OrderedDict([('tweet', 'Staggering news—Trump admin’s own petrifying death toll projections. The Senate should be doing a COVID 4 crisis relief package, not confirming judges, especially a McConnell protégé who wants to decimate the ACA &amp; American health care. https://t.co/Buuo01xiBz'), ('dates', '2020-05-04 20:50:42'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.7525859475135803)])
OrderedDict([('tweet', 'Instead of real remedies for the economic &amp; health care crisis, Republicans are pushing a special, sweetheart legal shield—shutting courthouse doors to victims of flagrant wrongdoing. Businesses that follow the law have nothing to fear. https://t.co/9SihZoPyxa'), ('dates', '2020-05-04 14:09:21'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.7069471478462219)])
OrderedDict([('tweet', 'McConnell’s priority in a deadly health care crisis is pushing confirmation of a judge who is passionately committed to destroying American health care. Incredibly, that’s why we’re all going back to DC tomorrow. https://t.co/aPYnZP6oRN'), ('dates', '2020-05-04 00:09:35'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.7744967341423035)])
OrderedDict([('tweet', 'Trump’s serial IG purges must be stopped as part of a broad oversight package in COVID 4 legislation. @SenWarren &amp; my proposals will help forestall fraud &amp; favoritism—like HHS’s IG tried to do here— &amp; assure taxpayer dollars reach Americans who need them. https://t.co/wdXfNj0pPn'), ('dates', '2020-05-03 15:17:41'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.5837374329566956)])
OrderedDict([('tweet', 'McConnell seems ready to offer sweeping broad immunity from accountability—a blanket shield for reckless, rapacious wrongdoing. Victims deserve their day in court. Congress must defend core Constitutional rights. https://t.co/SJEO9eWUmr'), ('dates', '2020-04-28 17:54:22'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.6117957234382629)])
OrderedDict([('tweet', 'Just what America needs, according to McConnell—bankruptcy for states &amp; straight jackets for lawyers. The American people want real solutions to this hideous hardship &amp; heartbreak, right now. https://t.co/TcFFFCkMpZ'), ('dates', '2020-04-28 12:02:09'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.6578046679496765)])
OrderedDict([('tweet', 'Both Republicans &amp; Democrats agree that McConnell’s message is pathetic &amp; pernicious. By forcing states to go bankrupt, he’d be looking the other way as police, firemen, first responders, bus drivers, &amp; other state employees lose their jobs. https://t.co/sPGfn4Nser'), ('dates', '2020-04-25 14:44:24'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.8080044984817505)])
OrderedDict([('tweet', 'Now I maybe know why small businesses have called me exasperated about receiving no PPP loans. We need an immediate IG investigation to ascertain if banks/SBA improperly played favorites. Right now—so Congress can address it in COVID4. https://t.co/o5qugQpT2Y'), ('dates', '2020-04-23 18:09:03'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.8451927900314331)])
OrderedDict([('tweet', 'Ousting experts in the middle of a pandemic for believing in science &amp; tested methods is unjustifiable. There is no quick fix to this virus &amp; anyone who panders those dangerous myths are putting people’s lives at risk.  https://t.co/E2fE8ylbR6'), ('dates', '2020-04-23 01:27:13'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.6721321940422058)])
OrderedDict([('tweet', 'Reckless risk to servicemembers—inadequate COVID-19 testing—just admitted to Senate Armed Services Committee on a call today. Pentagon leaders say testing is “constrained." No path to sufficient numbers.'), ('dates', '2020-04-22 22:23:59'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.7994765043258667)])
OrderedDict([('tweet', 'DeVos denying aid to DACA students hurts &amp; hinders these young people &amp; America. Our nation needs their talents &amp; training. She emulates Trump’s most insidious instincts. https://t.co/o3k0AYoJFi'), ('dates', '2020-04-22 18:54:37'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.689984142780304)])
OrderedDict([('tweet', 'Barr drags down both public health &amp; rule of law by threatening “stay at home” orders &amp; comparing them to “house arrest.” He says he might sue states—who’ve been leaders during COVID-19—seemingly acting as a Trump political operative, not law enforcer. https://t.co/DJz5y9a8X8'), ('dates', '2020-04-22 14:22:52'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.5957764983177185)])
OrderedDict([('tweet', 'My father was one of the lucky few to escape Nazi Germany, but millions of Jews faced grave horrors. On Yom HaShoah, #WeRemember the lives lost to immense brutality &amp; bigotry. The Holocaust must never be forgotten &amp; anti-Semitism—&amp; all forms of hate—must be condemned. https://t.co/yF9vSe0gbP'), ('dates', '2020-04-22 00:55:18'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.8381414413452148)])
OrderedDict([('tweet', 'Yet again impulsive demagoguery not real policy-making. Trump’s racism &amp; xenophobia will not stop COVID-19 nor hide his complete mishandling of this deadly virus. I’ll continue to fight for our immigrant communities &amp; against Trump’s hateful agenda. https://t.co/i8mCwK3Odx'), ('dates', '2020-04-21 22:25:25'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.9156174659729004)])
OrderedDict([('tweet', 'Today, the Trump Admin inexcusably extended their order to exploit COVID-19 &amp; expel asylum seekers. As my colleagues &amp; I have been making clear for weeks, this pandemic does not give the Admin a free pass to violate Constitutional rights or operate outside the law. https://t.co/YeoqYeIKOi'), ('dates', '2020-04-20 21:58:35'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.5224105715751648)])
OrderedDict([('tweet', 'Congressional Republicans &amp; Trump’s SBA need to put the SMALL back in this small business relief program. Big guys are sucking millions from PPP, leaving the most deserving unserved. https://t.co/PeJG4B6Z4k'), ('dates', '2020-04-20 18:34:19'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.6621896028518677)])
OrderedDict([('tweet', 'Enforcers must stop scammers &amp; bottom feeders from exploiting COVID-19 &amp; endangering health. False pitches &amp; sky-high price hikes should be halted &amp; prosecuted.https://t.co/OI7GGkFsZ2'), ('dates', '2020-04-17 13:02:24'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.6539557576179504)])
OrderedDict([('tweet', 'Banks seizing CARES checks must stop. Get a grip on banking greed. Congress must do it if Trump Treasury officials won’t. This money was meant for food, medicine, rent &amp; other struggling families’ needs. Not overdrawn accounts. Banks need to care. https://t.co/OPI7i9m0PS'), ('dates', '2020-04-16 18:53:55'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.5622671246528625)])
OrderedDict([('tweet', 'Instead of Trump falsely blaming the WHO, or his faux review, America needs a Pandemic Review Commission—an independent, bipartisan investigation just like after 9/11. https://t.co/9E0u67xHga'), ('dates', '2020-04-15 14:59:16'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.8036881685256958)])
OrderedDict([('tweet', 'Unacceptable. During a pandemic the last thing we should do is underfund &amp; undermine the WHO. They warned about an imminent, insidious threat, which Trump &amp; his Admin chose to ignore. \nhttps://t.co/unV0nyjVLr'), ('dates', '2020-04-15 00:37:49'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.8347828984260559)])
OrderedDict([('tweet', 'Having failed to provide critical medical equipment, the least the Trump Admin can do is cover states’ cost share. If not, Congress must act to waive states’ 25% funding share in the next relief package. Red state or blue it’s real money.'), ('dates', '2020-04-13 14:16:23'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.7938980460166931)])
OrderedDict([('tweet', 'Once we overcome this heartbreaking crisis, the American people deserve to understand how &amp; why we were so unprepared. I’m announcing legislation with @RepAdamSchiff, @SenFeinstein, @SenKamalaHarris, &amp; @SenMarkey creating a nonpartisan, independent investigation to do just that.'), ('dates', '2020-04-10 19:40:14'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.5158308148384094)])
OrderedDict([('tweet', 'Trump wants to kill the messenger—the WHO. It warned about an imminent, insidious pandemic. Trump dismissed &amp; demeaned the message &amp; now seeks to decimate WHO. Unacceptable. https://t.co/keAb73wgSc'), ('dates', '2020-04-08 23:57:50'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.9126290678977966)])
OrderedDict([('tweet', 'This blatant cover-up scheme is outrageous. It has no purpose except to derail&amp;defang oversight. To my Republican colleagues who’ve built careers advocating spending oversight: you must take a stand as Trump sidelines scrutiny over the largest relief funding bill in our history. https://t.co/j3mf1n5xq0'), ('dates', '2020-04-07 19:34:52'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.8531501889228821)])
OrderedDict([('tweet', 'I am calling on FTC to investigate @zoom_us. Zoom’s pattern of security failures &amp; privacy infringements should have drawn the FTC’s attention &amp; scrutiny long ago. Advertising privacy features that do not exist is clearly a deceptive act.'), ('dates', '2020-04-07 13:05:52'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.8882561922073364)])
OrderedDict([('tweet', 'Modly should be removed unceremoniously for these shocking remarks—especially after failing to protect sailors’ safety &amp; health. He has betrayed their trust. DoD IG must immediately launch an investigation. https://t.co/RAZTWit0oU'), ('dates', '2020-04-06 18:37:07'), ('file_name', 'Richard Blumenthal.csv'), ('pct_true', 0.5189356207847595)])
OrderedDict([('tweet', '30 years ago today – after President Reagan famously charged Gorbachev to “Tear down this wall!” - the Berlin Wall fell, signaling the beginning of the end of the Cold War.\n \nA piece of the wall now lies at the President’s feet in the Capitol. A tribute to his enduring legacy. https://t.co/zLVctJH6vn'), ('dates', '2019-11-09 18:19:00'), ('file_name', 'Richard Burr.csv'), ('pct_true', 0.5119851231575012)])
OrderedDict([('tweet', 'Russia is waging an information warfare campaign against the U.S. that didn’t start and didn’t end with the 2016 election. Their goal is to sow societal discord and erode confidence in the machinery of government.\n \nRead the new Senate Intel report here: https://t.co/4avncSiWWu'), ('dates', '2019-10-08 16:53:55'), ('file_name', 'Richard Burr.csv'), ('pct_true', 0.5332646369934082)])
OrderedDict([('tweet', "Most Americans agree: Government employees fired for misconduct should not be rehired. However, that's not always the case at the IRS. The agency has a long history of rehiring employees who were previously terminated for offenses like fraud and falsification of documents. 1/3"), ('dates', '2019-07-02 17:36:22'), ('file_name', 'Richard Burr.csv'), ('pct_true', 0.5856478810310364)])
OrderedDict([('tweet', 'During this unprecedented time, the Senate has been able to advance another agreement to help combat this deadly disease. Although my colleagues and I still have urgent work to do, this is a step in the right direction.'), ('dates', '2020-03-18 20:25:58'), ('file_name', 'Richard C. Shelby.csv'), ('pct_true', 0.566092848777771)])
OrderedDict([('tweet', "Throughout the impeachment proceedings in the Senate, I've carefully listened to arguments from both the House managers &amp; the President's counsel to fairly evaluate whether or not @POTUS's actions meet the Constitution’s high bar for an impeachable offense."), ('dates', '2020-01-31 22:49:12'), ('file_name', 'Richard C. Shelby.csv'), ('pct_true', 0.7036086916923523)])
OrderedDict([('tweet', 'Over the last 12 wks, House Dems engineered a political circus &amp; distracted us from the important work we’re here to accomplish for the American ppl. Their impeachment effort was rushed, unfair, &amp; careless – &amp; their decision to impeach POTUS last night seems completely unfounded.'), ('dates', '2019-12-19 15:40:29'), ('file_name', 'Richard C. Shelby.csv'), ('pct_true', 0.9418513178825378)])
OrderedDict([('tweet', 'Exciting news! Science &amp; technology leader, @LeidosInc, just announced its decision to acquire #Huntsville’s @Dynetics. Look forward to the cutting-edge national security &amp; defense work, including hypersonics, space solutions, &amp; more, which will result from this new relationship.'), ('dates', '2019-12-17 22:05:18'), ('file_name', 'Richard C. Shelby.csv'), ('pct_true', 0.5009364485740662)])
OrderedDict([('tweet', 'We are all painfully aware that the American people are worried about their health, jobs, the economy, &amp; what life will look like after this emergency ends. It is our responsibility to give our communities the confidence that we will get through this crisis. https://t.co/gclsrR2AAh'), ('dates', '2020-04-23 16:11:41'), ('file_name', 'Richard E. Neal.csv'), ('pct_true', 0.5524415373802185)])
OrderedDict([('tweet', 'If the Supreme Court rules against the #ACA, tens of millions of people will lose their health insurance and patients will be unable to access the critical treatments and medicines they need. https://t.co/ql5kTDH3lh'), ('dates', '2020-03-02 16:20:17'), ('file_name', 'Richard E. Neal.csv'), ('pct_true', 0.6813604235649109)])
OrderedDict([('tweet', 'Reminder: Democrats want to safeguard protections for people with pre-existing conditions, while President Trump and Republicans support a lawsuit that will rip those protections away. https://t.co/jovTG6dyOC'), ('dates', '2020-02-27 16:32:18'), ('file_name', 'Richard E. Neal.csv'), ('pct_true', 0.93232661485672)])
OrderedDict([('tweet', "Today I spoke w @NikkiHaley to discuss #COVID19 and #China. I value her perspective as an Amb &amp; Gov &amp; look forward to working w my @HouseCommerce colleagues on legislation to protect cyber security &amp; America's manufacturing sector. Thx you @cathymcmorris for setting up the call."), ('dates', '2020-05-06 21:57:41'), ('file_name', 'Richard Hudson.csv'), ('pct_true', 0.5950813293457031)])
OrderedDict([('tweet', 'Unfortunately bad actors have tried to use the #coronavirus crisis to prey on people through scams and price gouging. The Combating Pandemic Scams Act will ensure the public is aware of scams to better protect themselves and their loved ones. #InItTogether https://t.co/trOAdiYFvA'), ('dates', '2020-04-07 17:43:16'), ('file_name', 'Richard Hudson.csv'), ('pct_true', 0.6236828565597534)])
OrderedDict([('tweet', 'While protecting public health is our top priority, no government or law enforcement agency should use the #coronavirus to infringe upon our 2nd Amendment rights. I call on Sheriff Baker to follow NC law and continue to process pistol permits. #ncpol\n\nhttps://t.co/f41C4Kuasx'), ('dates', '2020-03-24 22:38:08'), ('file_name', 'Richard Hudson.csv'), ('pct_true', 0.7390809059143066)])
OrderedDict([('tweet', 'This is what President Trump is focused on right now? 70,000+ Americans have died from #COVID19, &amp; more are getting sick every day.  \n\nLet’s work on stopping this virus &amp; saving lives. Painting the President’s vanity wall can wait.  \nhttps://t.co/jfTTTQTFyY'), ('dates', '2020-05-07 20:21:30'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.7175050973892212)])
OrderedDict([('tweet', "The Constitution is clear - the power to declare war lies with Congress.\n\nDisappointed in my Republican colleagues for voting to uphold the President’s veto of the War Powers Act I intro'd w/@timkaine, handing this all-important Congressional power over to the executive branch."), ('dates', '2020-05-07 18:39:07'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.7525123953819275)])
OrderedDict([('tweet', 'Tens of thousands of Americans have died from #COVID19. Over 1.2M have been infected.  \n \nAnd yet, even in the midst of this pandemic, the President &amp; his party refuse to end efforts to dismantle the ACA &amp; rip health care away from millions of Americans. It would be a disaster. https://t.co/bEccYBs1xs'), ('dates', '2020-05-06 22:05:53'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.7902087569236755)])
OrderedDict([('tweet', "This country cannot afford a housing crisis on top of #COVID19. I'm cosponsoring @SenJackReed's bill to establish a $75B Housing Assistance Fund that would provide working families with a lifeline to keep the lights on &amp; avoid preventable foreclosures or evictions."), ('dates', '2020-05-06 21:34:39'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.5941689014434814)])
OrderedDict([('tweet', 'The Senate Judiciary Committee should not waste time on hearings for unqualified right-wing judicial nominees in the midst of this pandemic. @JudiciaryDems are calling for hearings on issues under the Committee’s jurisdiction that are directly related to #COVID19. https://t.co/pzre560x7i'), ('dates', '2020-05-06 13:29:55'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.6907275319099426)])
OrderedDict([('tweet', 'Last week, after Pres. Trump unveiled his reckless executive order to endanger workers by keeping meat plants open during the pandemic, I wrote IL meat processing companies to ask for info on how they plan to protect their workers from #COVID19 infections. https://t.co/IqfpEUOtv1'), ('dates', '2020-05-04 20:41:06'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.5006324052810669)])
OrderedDict([('tweet', '#COVID19 has put on full display the major structural inequities &amp; discrimination in our health care system that have disproportionally impacted minority communities for generations. https://t.co/Cgq4QzScuQ'), ('dates', '2020-05-01 18:02:06'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.6133729815483093)])
OrderedDict([('tweet', 'Important decision by the 7th Circuit – it reaffirms that the Department of Justice can’t withhold vital Byrne-JAG violence prevention funds in its attempt to force communities to adopt the Trump Administration’s anti-immigrant policies. https://t.co/6S3umxrEko'), ('dates', '2020-05-01 15:38:18'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.8241190314292908)])
OrderedDict([('tweet', 'No responsible public health expert would back up what Jared Kushner said yesterday. \n\nWe are by no means on the other side of #COVID19 -- and if we open back up too soon, we will end up right back where we started. We cannot afford to let that happen. https://t.co/UeFwctQBSG'), ('dates', '2020-04-30 21:22:17'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.5440854430198669)])
OrderedDict([('tweet', "#COVID19 affects all residents. And sadly, there are many stories like Ms. Mendez's. That's why I'm proud to cosponsor @maziehirono’s Coronavirus Immigrant Families Protection Act, which expands eligibility of Economic Impact Payments to those w/ an ITIN. https://t.co/MYOiEXnS0s"), ('dates', '2020-04-30 16:26:48'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.6627737283706665)])
OrderedDict([('tweet', 'The President knows Russia is coming to interfere in our elections yet again, but he continually refuses to take this threat seriously. Dems stand ready to prevent future efforts by Russia or others to influence &amp; disrupt our elections. I invite Pres Trump &amp; his party to join us. https://t.co/Oxrq2BnDwV'), ('dates', '2020-04-29 17:45:30'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.5561196804046631)])
OrderedDict([('tweet', 'For years, the Department of Justice has tried to withhold Byrne-JAG violence prevention funds from local communities unless the communities adopt the Administration’s preferred immigration policies. The law is clear that DOJ does not have the authority to do this.'), ('dates', '2020-04-27 23:14:28'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.9142507314682007)])
OrderedDict([('tweet', 'Where would America be in this crisis without millions of immigrants and first generation Americans who are essential workers on the frontlines of this pandemic? \n\nPresident Trump should stop attacking immigrants and spend his valuable time attacking this virus. https://t.co/bET69g85W5'), ('dates', '2020-04-23 22:42:32'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.6949123740196228)])
OrderedDict([('tweet', 'Why is @SecretarySonny saying that ethanol is "not our jurisdiction" and therefore excluded from $19B of @USDA CARES Act funding? \n\nUnder President Obama, USDA helped ethanol. What changed? Those in charge. https://t.co/RT4E37M4Dg'), ('dates', '2020-04-23 21:53:01'), ('file_name', 'Richard J. Durbin.csv'), ('pct_true', 0.5028404593467712)])
OrderedDict([('tweet', 'Fascinating article about how Chinese government began censoring COVID-19 impacts as soon as December 30.  Censored Contagion: How Information on the Coronavirus is Managed on Chinese Social Media - The Citizen Lab https://t.co/yHQ831Pq3u'), ('dates', '2020-04-16 18:03:12'), ('file_name', 'Rick Larsen.csv'), ('pct_true', 0.8643062114715576)])
OrderedDict([('tweet', 'Americans agree we shouldn’t be using their hard earned tax dollars to help bailout fiscally irresponsible states. \n\nIt’s time these poorly-managed states learn to live within their means like Florida does. https://t.co/CxSIwCg7Br'), ('dates', '2020-05-09 20:11:07'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.5999289155006409)])
OrderedDict([('tweet', 'Taiwan is a trusted partner that tried to warn the @WHO of the coronavirus when Communist China was silencing Chinese doctors and suppressing information. \n\nLines have been drawn between supporting freedom or authoritarian regimes. We will be watching where the world comes down. https://t.co/tpYdLZaL6E'), ('dates', '2020-05-08 01:10:41'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9228715300559998)])
OrderedDict([('tweet', 'We must hold Communist China accountable. China steals our jobs, our technology &amp; is responsible for this pandemic that’s killed hundreds of thousands around the world. \n\nThe most important thing we can do is\xa0stop buying Chinese products and #BuyAmerican. https://t.co/88XvNbDHh5'), ('dates', '2020-05-07 14:12:47'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9847692251205444)])
OrderedDict([('tweet', 'Absurd to lock up an innocent businesswoman simply trying to keep her business afloat and food on the table for her family. \n\nWe can responsibly open our economy back up and keep Americans safe without this kind of unacceptable government overreach. https://t.co/B9omMJn43n'), ('dates', '2020-05-07 02:09:19'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.5168020725250244)])
OrderedDict([('tweet', 'The IOC is faced with an important decision.\n\nThe @Olympics are an incredible opportunity to allow the world’s best athletes to represent their countries on the global stage. The 2022 Olympics shouldn’t be hosted by Communist China &amp; President Xi, who doesn’t value human rights. https://t.co/F0FR7088bd'), ('dates', '2020-05-06 16:21:44'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.8760286569595337)])
OrderedDict([('tweet', 'Unacceptable that Chinese drones are being used by law enforcement to enforce social distancing.\n\nGiving the CCP easy access into our lives is dangerous &amp; a security threat. I urge my colleagues to pass my American Security Drone Act &amp; put an end to this. https://t.co/9gpivZlOpu'), ('dates', '2020-05-05 18:55:15'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.6336746215820312)])
OrderedDict([('tweet', 'Thank you @RepTedYoho for introducing this bill in the House.\n\nCommunist China has been one of the worst human rights violators &amp; is now responsible for the spread of the #Coronavirus. The 2022 Olympics should be rebid &amp; moved to a nation that respects human rights &amp; freedom. https://t.co/0Wlaw7WSyq'), ('dates', '2020-05-05 17:00:42'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.8659723401069641)])
OrderedDict([('tweet', 'Communist China is determined to increase their control over as much of the world as they can, even if it means using a devastating pandemic as an opportunity to do so.\n \nWe can’t turn a blind eye to Communist China. https://t.co/wjdQrC9FX7'), ('dates', '2020-05-05 15:05:33'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9750831127166748)])
OrderedDict([('tweet', 'Florida taxpayers should not be responsible for bailing out poorly-managed states like NY, CA and IL.\n\nThe federal government is and will continue to reimburse states for #Coronavirus expenses. But states need to take responsibility for their own budgets. https://t.co/T8ff21X9ch'), ('dates', '2020-05-04 02:25:01'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.5234794616699219)])
OrderedDict([('tweet', 'More evidence for what some of us have been saying for months. \n\nCommunist China tried to cover up and hide the #Coronavirus from the world. The consequences have been devastating. Communist China must be held responsible for their deceit. https://t.co/QdufquCNSB'), ('dates', '2020-05-02 13:48:05'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9834217429161072)])
OrderedDict([('tweet', 'While Communist China’s grip on #HongKong tightens during this epidemic, the people of Hong Kong’s determination and will is not shattered.\n \nThe fight for human rights and autonomy continues each day. https://t.co/87ybq2NLNf'), ('dates', '2020-05-01 17:57:20'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.7743191123008728)])
OrderedDict([('tweet', 'Communist China’s lack of transparency led to this devastating pandemic. They need to be held accountable.\n \nThe best way we can do it is to stop buying products from Communist China. It’s time we bring our jobs back to the US. https://t.co/KVgIfi59fv'), ('dates', '2020-05-01 13:54:37'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9659153819084167)])
OrderedDict([('tweet', 'The Federal government should continue to reimburse state &amp; local governments for expenses related to #Coronavirus. But FL taxpayers shouldn’t bail out poorly-managed state budgets.\n\n@NYGovCuomo needs to answer for NY’s decades of poor fiscal management. https://t.co/Mma2VQYSpj'), ('dates', '2020-04-30 15:41:11'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.5547634363174438)])
OrderedDict([('tweet', 'The best way to hold Communist China accountable is to buy American products, but that’s not always easy to do. \n\nWould you find it helpful if online retailers told you where a product was made before you purchased it?'), ('dates', '2020-04-29 21:25:21'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.8953391313552856)])
OrderedDict([('tweet', 'The Cuban people have suffered nearly 6 decades of atrocities &amp; oppression inflicted by the regime – and @JoeBiden’s embrace of Obama’s appeasement policies toward the Castro dictatorship did nothing to help.\n \nBiden should instead stand with Cubans in their fight for freedom. https://t.co/jEaU9Z0DbQ'), ('dates', '2020-04-29 18:53:12'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9324455261230469)])
OrderedDict([('tweet', 'Glad that @RealDonaldTrump is taking action to hold Communist China and the @WHO accountable. \n\nWe need more information and transparency. What did they know and when did they know it? https://t.co/GThbcwKtR5'), ('dates', '2020-04-29 15:59:29'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.8640426397323608)])
OrderedDict([('tweet', "Federal Authorities have seized more than $450 million in assets belonging to Maduro's cronies since 2017.\n \nThe US will not welcome or do business with anyone who supports @NicolasMaduro. We will continue to work until every corrupt Venezuelan official is held responsible. https://t.co/U48MhlWEf6"), ('dates', '2020-04-28 22:50:24'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.5445346236228943)])
OrderedDict([('tweet', 'Communist China has been stealing our jobs and our technology, and now their actions have led to this global pandemic.\n \nWe need to hold Communist China accountable and that can start by simply buying American and supporting American businesses. https://t.co/8QT5ObPFLd'), ('dates', '2020-04-28 18:09:11'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9980100989341736)])
OrderedDict([('tweet', 'It’s irresponsible &amp; reckless to take $$ from American taxpayers &amp; use it to save liberal politicians from the consequences of their poor choices.\n\nAmerican families make responsible budgetary every day. So does FL. It’s time for NY, IL &amp; CA do the same. More in my @WSJ Op-Ed ⬇️ https://t.co/CxAK5sTE1J'), ('dates', '2020-04-27 23:20:20'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.6768667101860046)])
OrderedDict([('tweet', '.@LindseyGrahamSC is completely right. If our nation has learned one thing from this pandemic it is that we should not be relying on Communist China for any of our supplies. \n \nWe should be working to build up our own stock pile. https://t.co/VllNHuDJ1X https://t.co/OTXsSyrCiu'), ('dates', '2020-04-27 21:46:16'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.6556352376937866)])
OrderedDict([('tweet', "While the global community continues to combat the #Coronavirus, we still need answers about the role the @WHO played in aiding Communist China.\n\nMy colleagues &amp; I have requested that taxpayer $$ be held back from the organization until we're able to get the answers we deserve. https://t.co/kfQsFABS71"), ('dates', '2020-04-27 16:00:01'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.5537906885147095)])
OrderedDict([('tweet', 'This is not the time for the EU to cower in the face of Communist China’s intimidation. \n\nCommunist China lied and attempted to cover up the #Coronavirus from the global community and the results have been devastating worldwide. They must answer for it. https://t.co/u4Z7PRHATS'), ('dates', '2020-04-25 17:28:32'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9918040037155151)])
OrderedDict([('tweet', 'Before anymore taxpayer $$ is sent to fund the @WHO, Americans deserve answers about what they knew and their role in helping Communist China cover up the #Coronavirus.\n\nMy colleagues and I have requested that funding be placed on hold until an investigation is completed. https://t.co/a7LvD0KZz4'), ('dates', '2020-04-25 15:29:13'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.718271017074585)])
OrderedDict([('tweet', 'Today, @SenRonJohnson, @SenThomTillis, @SteveDaines, @SenKevinCramer &amp; I requested that the Senate Appropriations Committee hold any funding for the @WHO pending a Congressional investigation into their role in helping Communist China cover up info about the #Coronavirus.\nRead ⬇️ https://t.co/t802jOr1TX'), ('dates', '2020-04-24 19:14:04'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.6588510870933533)])
OrderedDict([('tweet', 'It is unacceptable that millions of taxpayer dollars are not getting to the small businesses truly in need. Instead, big companies, some of which haven’t been negatively impacted by this crisis, are getting taxpayer $$.\n \nIts time they #givebackthefunds.'), ('dates', '2020-04-23 20:41:57'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.6909379959106445)])
OrderedDict([('tweet', 'Huawei is a bad actor &amp; a pawn for Communist China, which has done nothing but lie about the #Coronavirus &amp; put many lives at risk.\n\nBack in January, I wrote an Op-Ed on my concerns with the UK allowing Huawei in &amp; I’m glad they’re now seeing the risks. https://t.co/WiAqrLYpob https://t.co/wk6nFQhh6y'), ('dates', '2020-04-21 19:49:00'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.6850652098655701)])
OrderedDict([('tweet', 'We need to be focusing on the small businesses that are struggling &amp; the Americans that have lost their jobs, not the wealthy &amp; big corporations.\n \nCongress needs to make sure that taxpayer dollars are going to those who need it most. https://t.co/isFzaedYeF'), ('dates', '2020-04-20 22:39:30'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.5683469772338867)])
OrderedDict([('tweet', 'Millions have lost their job. Small businesses are suffering &amp; trying to stay alive. Meanwhile, Harvard, with its $40B endowment, just got $9M in taxpayer money.\n\nThe CARES Act was necessary to save our economy. This kind of waste from the federal government IS NOT NECESSARY. https://t.co/TnGEaZXHs5'), ('dates', '2020-04-20 16:00:33'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.7901132106781006)])
OrderedDict([('tweet', 'Communist China has consistently lied &amp; covered up the #Coronavirus and poses a national security threat to the United States. \n\nNo American media should be promoting their propaganda, especially their attempts to slander our military. This is unacceptable by @CNN. https://t.co/NMpt8ztIDG'), ('dates', '2020-04-19 23:04:26'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9971076846122742)])
OrderedDict([('tweet', 'This is a clear attempt by the CCP to silence those speaking up for human rights and democracy.\n\nThe people of #HongKong deserve their autonomy and violent acts like these will not go unnoticed. https://t.co/u5BR5HuQxf'), ('dates', '2020-04-19 20:58:25'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.7860915660858154)])
OrderedDict([('tweet', 'In times of crisis we can’t be reliant on foreign nations to provide basic supplies, especially Communist China. American companies are ready and able to answer the call. \n\nThat’s why I introduced a bill to rebuild our American-made stockpile of PPE. https://t.co/VllNHuDJ1X https://t.co/HHfPEDEeeB'), ('dates', '2020-04-19 17:07:17'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.5861933827400208)])
OrderedDict([('tweet', 'Think about this for a second. This virus originated in Communist China and the Chinese Communist Party’s lies helped it spread around the world. \n\nNow we’re using drones made by a Chinese company and backed by the CCP to enforce social distancing. This is crazy! https://t.co/pJDGoviRis'), ('dates', '2020-04-18 19:10:23'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.8320629000663757)])
OrderedDict([('tweet', 'Even in the midst of a pandemic Communist China continues to target pro-democracy leaders in #HongKong.\n\nThis is unacceptable. The people of #HongKong are simply fighting for their basic human rights! https://t.co/cNF3IxTvvk'), ('dates', '2020-04-18 14:10:29'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9626964926719666)])
OrderedDict([('tweet', '“Didn’t look”? Really? \n\nThe @WHO isn’t even trying to hide their role in helping Communist China cover up the early spread of #Coronavirus. Their careless actions have been detrimental to the global response to the outbreak, and they must now answer for it. https://t.co/5AgtLBt5X3'), ('dates', '2020-04-17 14:02:15'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.8967662453651428)])
OrderedDict([('tweet', 'Communist China has been lying to us from the start, and their response to the #Coronavirus has only further confirmed that. \n\nIt’s time Americans start focusing on buying American goods and bringing jobs back to America. We can’t keep giving China any more advantages over us. https://t.co/SAFIwuxbNV'), ('dates', '2020-04-16 23:43:08'), ('file_name', 'Rick Scott.csv'), ('pct_true', 0.9958145022392273)])
OrderedDict([('tweet', 'On this day 75 years ago, the Allied forces accepted Nazi Germany’s surrender and claimed Victory in Europe, which changed the course of World War II. Today we remember the service and sacrifice of those who helped secure V-E Day. https://t.co/AJCLsIYKdr'), ('dates', '2020-05-08 12:21:00'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.686611533164978)])
OrderedDict([('tweet', 'Today the Senate is back in session, but Speaker Pelosi has not released any plan on when the House will safely return. Meanwhile, @GOPLeader, @HouseAdmnGOP and @RulesReps have stepped up and crafted a clear plan to get Congress back to work. https://t.co/9mSALM56Cv'), ('dates', '2020-05-04 16:58:12'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.7056005001068115)])
OrderedDict([('tweet', '.@GOPLeader is right, we cannot depend on China for essentials like PPE and medicine. It’s time to take back control of our supply chain. https://t.co/ZEmXD8zb63'), ('dates', '2020-05-01 14:04:37'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.5970487594604492)])
OrderedDict([('tweet', '.@realDonaldTrump is right to suspend funding to the World Health Organization until we fully understand their role in the coronavirus outbreak and how they allowed China to downplay the severity of the situation.'), ('dates', '2020-04-16 19:22:02'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.933891773223877)])
OrderedDict([('tweet', 'Congress must show the American people that in times of crisis we are not Republicans or Democrats – we are all Americans first.'), ('dates', '2020-03-24 23:55:08'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.9277094602584839)])
OrderedDict([('tweet', 'This week President @realDonaldTrump released a pro-worker, pro-family budget that:\n✅ Balances\n✅ Cuts wasteful spending\n✅ Shrinks the size of the federal government\n \nBut we still haven’t seen a budget from Speaker Pelosi and House Democrats!'), ('dates', '2020-02-13 17:08:54'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.6160934567451477)])
OrderedDict([('tweet', 'This is exactly why we can’t lose the lock and dam and cede more control over our water flows with the Corps’ misguided rock weir plan. https://t.co/ZEIbKbwV3L'), ('dates', '2020-02-09 00:24:22'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.6375190615653992)])
OrderedDict([('tweet', "Tonight House Dems passed the PRO Act, which diminishes the rights of American workers &amp; benefits union bosses. The bill outright bans states' Right-to-Work laws. I offered an amendment to strike that language and protect #RightToWork but it was defeated.\n https://t.co/Obx3gaGw7R"), ('dates', '2020-02-07 02:32:32'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.6642216444015503)])
OrderedDict([('tweet', 'Great news! The Senate upheld the Constitution by voting to acquit President Trump of the impeachment articles sent over by the House Democrats. Now that this sham is over it’s time to get to work for the American people. https://t.co/Rc7y8wbpoe'), ('dates', '2020-02-05 22:22:15'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.6889408826828003)])
OrderedDict([('tweet', 'We all know Nancy Pelosi is having a tough week - @realDonaldTrump gave a great #SOTU last night and her impeachment charade is unraveling at the seams today. But ripping the speech is beneath the dignity of the office she holds.'), ('dates', '2020-02-05 14:47:18'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.7588824033737183)])
OrderedDict([('tweet', 'Across the world today, #WeRemember the more than six million Jewish lives senselessly taken in the Holocaust. 75 years after the liberation of Auschwitz, we must vow #NeverAgain and come together to condemn anti-Semitism in any form. #HolocaustRemembranceDay https://t.co/Wqz7Uzhya8'), ('dates', '2020-01-27 17:07:26'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.6265547275543213)])
OrderedDict([('tweet', 'Today the Senate began impeachment trial proceedings. Just a reminder that House Democrats completely failed to produce any evidence proving that President Trump did anything wrong. Now the Senate must reject this sham for what it is: A baseless political smear. https://t.co/b8M2Iap1Ul'), ('dates', '2020-01-21 20:05:34'), ('file_name', 'Rick W. Allen.csv'), ('pct_true', 0.9469369649887085)])
OrderedDict([('tweet', '7 weeks ago, we had the same number of COVID deaths as South Korea.\xa0\n\nOur abysmal failure to manufacture and distribute tests will forever haunt this country.\xa0\nhttps://t.co/6Yzzumhc9g'), ('dates', '2020-05-08 13:44:00'), ('file_name', 'Ro Khanna.csv'), ('pct_true', 0.5274742841720581)])
OrderedDict([('tweet', 'I support extending the UN arms embargo on Iran.\n\nBut\xa0Trump must use existing arms sanctions laws that stop weapons from going to Iran AND work to negotiate new regional arms control agreements.\nhttps://t.co/5ZsGPFmMqe'), ('dates', '2020-05-07 21:44:30'), ('file_name', 'Ro Khanna.csv'), ('pct_true', 0.8595073819160461)])
OrderedDict([('tweet', "No one should have to risk their life in a meat processing plant for America to eat.\n \nThat's why I'm introducing @SenBooker’s Farm System Reform Act in the House, to fight back against corporate Ag consolidation and protect family farmers. https://t.co/Wm1Rn6bsGf"), ('dates', '2020-05-07 13:58:47'), ('file_name', 'Ro Khanna.csv'), ('pct_true', 0.5152986645698547)])
OrderedDict([('tweet', 'Congress cannot bail out fossil fuel companies. CARES was supposed to benefit small businesses, not Big Oil. Proud to cosponsor @RepBarragan’s bill. https://t.co/YdOLehKsHJ'), ('dates', '2020-05-06 13:55:11'), ('file_name', 'Ro Khanna.csv'), ('pct_true', 0.569623589515686)])
OrderedDict([('tweet', 'We should be testing 1 million Americans a day.\n\nThe Defense Production Act gives the federal government that power, but Trump continues to drag his feet.\n\nSo @SenSanders and I are introducing legislation to put another $75 billion toward producing masks, swabs, and gloves. https://t.co/Y5Pw8tLGun'), ('dates', '2020-05-04 15:20:12'), ('file_name', 'Ro Khanna.csv'), ('pct_true', 0.6744943261146545)])
OrderedDict([('tweet', "Another thread on the national debt -&gt;\n\xa0\nThe Fed owns 40% of all US public debt than any other entity—more than the 30% China or any other foreign governments own. \n\nThat's how, in just weeks, Treasury issued millions of $1,200 checks. 1/x"), ('dates', '2020-04-20 20:58:48'), ('file_name', 'Ro Khanna.csv'), ('pct_true', 0.89383864402771)])
OrderedDict([('tweet', 'American workers, not corporations, need our support the most right now.\xa0\n\nCongress should be putting all our efforts toward making sure folks can still put food on the table and stay in their homes.'), ('dates', '2020-04-18 17:01:00'), ('file_name', 'Ro Khanna.csv'), ('pct_true', 0.5524162650108337)])
OrderedDict([('tweet', 'A one-time, $1200 check is not going to pay the bills.\n\nIf the Trump administration believes big corporations deserve weekly + monthly loans, why can’t it do the same for working families?\n\n@RepTimRyan and I want to change that. https://t.co/K5UufpHwOm'), ('dates', '2020-04-11 15:29:00'), ('file_name', 'Ro Khanna.csv'), ('pct_true', 0.5338971018791199)])
OrderedDict([('tweet', 'Last yr we released a bombshell report shining a light on China’s rampant theft of our taxpayer-funded research &amp; intellectual property. Their actions undermine our higher ed system &amp; endanger our national security. This further scrutiny is well-warranted. https://t.co/LKx0WBYthF'), ('dates', '2020-05-10 00:11:59'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.5472627282142639)])
OrderedDict([('tweet', 'DOJ just dropped the Gen. Flynn case; evidence shows they never should have pursued it in the 1st place. \n\nDOJ &amp; FBI need to thoroughly review their past operations after this miscarriage of justice &amp; the recent DOJ IG report showing FISA abuse in the Carter Page investigation.'), ('dates', '2020-05-08 18:48:49'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.6456812620162964)])
OrderedDict([('tweet', 'China needs to be held accountable for deceiving the global community and covering up the severity of #COVID19. https://t.co/VFkAu4SarB'), ('dates', '2020-05-07 23:19:33'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.7344692945480347)])
OrderedDict([('tweet', 'The SBA should not deny crucial aid to small business owners with past criminal records. \n\nThese individuals have turned their lives around, started a small businesses, created jobs &amp; contribute to their communities. https://t.co/h26ay5hZMf'), ('dates', '2020-05-01 18:46:41'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.8895537853240967)])
OrderedDict([('tweet', 'Today @SenatorCardin &amp; I called on the SBA to allow small biz owners w/ criminal records to apply for the Paycheck Protection Program.\n\nWe should not deny assistance to ppl who have turned their lives around, started a small biz, created jobs &amp; contribute to their communities.'), ('dates', '2020-04-30 18:20:23'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.7459704279899597)])
OrderedDict([('tweet', 'As we continue to fight the #coronavirus pandemic, we can’t lose sight of another health crisis that has hit Ohio hard: the addiction epidemic.\n \nWe must keep our eye on the ball &amp; ensure we don’t lose ground in the fight against addiction. https://t.co/JR1yYrN1sG'), ('dates', '2020-04-29 17:35:42'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.5123471617698669)])
OrderedDict([('tweet', 'Millions of Americans are working remotely; Congress should too. We have vital work to do and we can’t allow this virus to stop us from completing it. https://t.co/kdee86Fbrd'), ('dates', '2020-04-25 20:52:06'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.5478888750076294)])
OrderedDict([('tweet', '.@realDonaldTrump just signed the additional COVID funding into law, injecting $320B into the #PaycheckProtectionProgram.\n\nThis program has been a lifeline for Ohio #smallbiz, providing roughly 60K loans totaling more than $14B.\n\nI look forward to it getting back up &amp; running.'), ('dates', '2020-04-24 16:55:51'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.5361768007278442)])
OrderedDict([('tweet', 'On Yom HaShoah - Holocaust Remembrance Day - we remember the 6 million Jews brutally murdered by the Nazi regime and reaffirm our commitment to fighting anti-Semitism and hate everywhere. #HolocaustRemembranceDay https://t.co/zAtqo5ciD1'), ('dates', '2020-04-21 22:00:53'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.7514060139656067)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram has already issued 1M+ loans to small businesses, saving millions of jobs nationwide.\n \nBut yesterday the program ran out of money. And Democrats have blocked a measure to replenish it.\n \nLet’s stop playing politics &amp; fund this vital lifeline now. https://t.co/OIYDnYeFg2'), ('dates', '2020-04-17 17:11:21'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.8388581871986389)])
OrderedDict([('tweet', 'The Paycheck Protection Program has run out of money. This is no time for partisan politics. \n\nCongress must replenish the funds today. \n\nMillions of jobs are on the line. https://t.co/0BUnEP52Vn'), ('dates', '2020-04-16 15:36:46'), ('file_name', 'Rob Portman.csv'), ('pct_true', 0.5744267106056213)])
OrderedDict([('tweet', 'Today, on #HolocaustRemembranceDay, #WeRemember the 6 million Jews who were murdered by the Nazis. Never again can we allow anti-Semitism and genocide to take root in our world. https://t.co/S59v1kM6oe'), ('dates', '2020-01-27 22:46:16'), ('file_name', 'Rob Woodall.csv'), ('pct_true', 0.9232515096664429)])
OrderedDict([('tweet', 'Yesterday, Republicans &amp; Democrats passed the #NDAA for the 59th consecutive year.\n\nThis bill reaffirms our nation’s defense priorities, ensures the livelihood of the men and women of our Armed Forces, keeps America safe, &amp; preserves our global leadership.\nhttps://t.co/5jPnMDdI0x'), ('dates', '2019-12-12 14:18:45'), ('file_name', 'Rob Woodall.csv'), ('pct_true', 0.6695938110351562)])
OrderedDict([('tweet', 'Today marks the 78th anniversary of the attack on Pearl Harbor. \n\nWe remember the bravery and courage of the 2,403 Americans who lost their lives defending our nation on that infamous day. \n\nTheir sacrifice will never be forgotten. #PearlHarborRemembranceDay https://t.co/vfifOGm8SO'), ('dates', '2019-12-07 17:02:54'), ('file_name', 'Rob Woodall.csv'), ('pct_true', 0.5160114765167236)])
OrderedDict([('tweet', 'Looks like Democratic leadership is at it again, creating some big government spending bill behind closed doors. This type of political game is not what the country needs right now. https://t.co/vo1BUlMOh2'), ('dates', '2020-05-07 19:11:28'), ('file_name', 'Robert B. Aderholt.csv'), ('pct_true', 0.5274120569229126)])
OrderedDict([('tweet', 'While the national media narrative seems to constantly highlight that the United States has the most COVID-19 cases, it’s even more important to put things in perspective.  We have conducted the most testing and our per capita death rate is lower than many other countries. https://t.co/RvSErlimWY'), ('dates', '2020-05-06 14:56:18'), ('file_name', 'Robert B. Aderholt.csv'), ('pct_true', 0.6248884201049805)])
OrderedDict([('tweet', 'The Chinese Communist Party has been working for years to become a global pharmacy. And after blatantly lying and withholding information about COVID-19, I think it’s time we implement Buy America policies in the medical supply and pharmaceutical industry. https://t.co/aSCrA7SrZ7'), ('dates', '2020-04-17 15:29:49'), ('file_name', 'Robert B. Aderholt.csv'), ('pct_true', 0.9178469181060791)])
OrderedDict([('tweet', "Enough is enough, we need to put political games aside. The PPP fund is depleted and Americans are counting on us to do something. I'm urging Democratic Leadership to support and pass the Senate replenishment bill. Now. \n\nhttps://t.co/odmvHSskQQ"), ('dates', '2020-04-16 21:48:48'), ('file_name', 'Robert B. Aderholt.csv'), ('pct_true', 0.8482604622840881)])
OrderedDict([('tweet', 'The COVID-19 epidemic requires fast action and relief to small businesses and families. This is not the time to write a partisan wish list in the Phase 3 stimulus. Here are some of the ridiculous proposals included:'), ('dates', '2020-03-24 13:20:19'), ('file_name', 'Robert B. Aderholt.csv'), ('pct_true', 0.5609257221221924)])
OrderedDict([('tweet', 'I voted to condemn Senator Bernie Sanders’ comments praising communist Fidel Castro. A literacy program instituted to help spread propaganda is no bright spot in a dark regime that executed and jailed those who sought freedom.'), ('dates', '2020-02-27 22:58:31'), ('file_name', 'Robert B. Aderholt.csv'), ('pct_true', 0.9571294784545898)])
OrderedDict([('tweet', "75 years ago, Allied forces accepted Nazi Germany's unconditional surrender – ending #WWII in Europe.\n\nThankful for the service and sacrifices of all the brave women and men who fought during this conflict. #VEDay75 https://t.co/q8kW8PN1Ck"), ('dates', '2020-05-08 20:15:56'), ('file_name', 'Robert C. "Bobby" Scott.csv'), ('pct_true', 0.5446605682373047)])
OrderedDict([('tweet', 'It is profoundly disturbing that President Trump is using his authority to force workers into dangerous conditions without providing enforceable safety standards to protect them from #COVID19.\nhttps://t.co/5WtBTrLaaf'), ('dates', '2020-04-28 21:04:45'), ('file_name', 'Robert C. "Bobby" Scott.csv'), ('pct_true', 0.538031280040741)])
OrderedDict([('tweet', "Today is #WorkersMemorialDay. We are in the middle of a global pandemic. Workers are falling ill and dying from #COVID19. But President Trump's @OSHA_DOL has failed to issue an emergency standard to keep our workers safe."), ('dates', '2020-04-28 15:15:26'), ('file_name', 'Robert C. "Bobby" Scott.csv'), ('pct_true', 0.6412919163703918)])
OrderedDict([('tweet', 'President Trump’s refusal to give millions of uninsured and underinsured Americans the opportunity to sign up for comprehensive health coverage in the midst of a global pandemic will have dire consequences for people across this country. \nhttps://t.co/T8RBDNy0UH'), ('dates', '2020-04-01 21:05:28'), ('file_name', 'Robert C. "Bobby" Scott.csv'), ('pct_true', 0.5282990336418152)])
OrderedDict([('tweet', '#OnThisDay in 1945, the course of World War II changed when the Allied forces accepted Nazi Germany’s surrender and claimed Victory in Europe. Today, we remember the service and sacrifice of all who helped secure #VEDay. #VEDay75 https://t.co/cuu4I61Y69'), ('dates', '2020-05-08 16:28:43'), ('file_name', 'Robert E. Latta.csv'), ('pct_true', 0.5423027276992798)])
OrderedDict([('tweet', "The reality is, many Ohioans in #OH5 who live in rural areas don’t have reliable internet connectivity. The Broadband DATA Act, which I led in Congress, is now law &amp; will spur an invisible transformation for rural communities so they aren't left behind &gt;&gt; https://t.co/dcU858YRg6"), ('dates', '2020-04-28 21:18:00'), ('file_name', 'Robert E. Latta.csv'), ('pct_true', 0.6050106883049011)])
OrderedDict([('tweet', '.@HouseCommerce members @USRepMikeDoyle, Rep. Gianforte, @RepMcEachin &amp; I urge @TheJusticeDept &amp; @fcc to go after robocallers aggressively using the tools Congress has given them through the #bipartisan #TRACEDAct to deter &amp; enforce violations of the law effectively. Our letter⤵️ https://t.co/BXB8Y8rdF7'), ('dates', '2020-04-02 18:33:36'), ('file_name', 'Robert E. Latta.csv'), ('pct_true', 0.5399146676063538)])
OrderedDict([('tweet', 'Thank you to Homeland Security Investigations – who has continued to protect us from fraud as we fight the #COVID19 pandemic. They have partnered with the private sector to stop fraud schemes and counterfeit materials from entering the US supply chain. https://t.co/kh8Y572WJO'), ('dates', '2020-05-06 18:29:05'), ('file_name', 'Robert J. Wittman.csv'), ('pct_true', 0.6614532470703125)])
OrderedDict([('tweet', 'Freedom of the press is a fundamental human right, a pillar of democracy, and an indispensable check on authoritarian overreach. \n\nFrom Wuhan, China, to towns in the U.S, to conflict zones in Venezuela, journalists are risking their lives to report on the #COVID19 pandemic.'), ('dates', '2020-05-03 17:21:51'), ('file_name', 'Robert Menendez.csv'), ('pct_true', 0.741923451423645)])
OrderedDict([('tweet', "This #WorldPressFreedomDay, we can't forget about the local news outlets giving lifesaving information to people across New Jersey. They've been hit hard by #COVID19, just like any small biz. That's why @SenateDems have asked @ombpress to direct federal ad $ to help support them. https://t.co/AssrXZw6wv"), ('dates', '2020-05-03 17:20:31'), ('file_name', 'Robert Menendez.csv'), ('pct_true', 0.5188046097755432)])
OrderedDict([('tweet', 'On #YomHaAtzmaut, we not only honor &amp; commemorate the 72nd anniversary of Israel, but we honor the values that the US &amp; Israel hold dear—hope over fear, democracy over despotism, human rights over hatred.\n\nThese values keep the US-Israel bond unbreakable. #IsraeliIndependenceDay'), ('dates', '2020-04-29 14:30:50'), ('file_name', 'Robert Menendez.csv'), ('pct_true', 0.9066866636276245)])
OrderedDict([('tweet', "I led a letter last year demanding that the President NOT redirect congressionally-appropriated counter-Russia funds to his barbaric border wall. \n\nLooks like he's at it again. Will Trump still say that no Admin has been tougher on Russia after this $1B gift to his friend Putin? https://t.co/amLlTe1nNH"), ('dates', '2020-04-28 23:53:59'), ('file_name', 'Robert Menendez.csv'), ('pct_true', 0.5133987665176392)])
OrderedDict([('tweet', "Minority communities have been disproportionately hit by #COVID19 and minority-owned small businesses need the same access to assistance as any big business.\n\nWe're calling on the Trump Admin to expand #PPP opportunities so they can get it. https://t.co/AwlRg0WM0r"), ('dates', '2020-04-27 22:47:26'), ('file_name', 'Robert Menendez.csv'), ('pct_true', 0.6524761319160461)])
OrderedDict([('tweet', 'Our states did not choose to face the #COVID19 pandemic. \n\nThey did not choose to have this deadly virus claim thousands of American lives.\n\nThey did not choose to have their economies sink into a devastating recession. https://t.co/F07r93DyLG'), ('dates', '2020-04-23 20:53:28'), ('file_name', 'Robert Menendez.csv'), ('pct_true', 0.7751097083091736)])
OrderedDict([('tweet', 'Suspending immigration months after the #coronavirus started spreading in our communities is not a public health decision. \n\nPresident Trump needs to understand the enemy is not immigrants, but rather a virus that endangers us all.  https://t.co/Pitd4v7rKU'), ('dates', '2020-04-21 16:19:43'), ('file_name', 'Robert Menendez.csv'), ('pct_true', 0.7514166235923767)])
OrderedDict([('tweet', 'The same President who wants to “drain the swamp” is turning Washington into his cesspool of corruption. https://t.co/XqUXdImMhY'), ('dates', '2020-05-07 21:49:49'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.6658977270126343)])
OrderedDict([('tweet', 'The complaint raises serious concerns that the Trump Administration is playing political games with the health of the American people. If confirmed, these allegations involve the worst kind of cronyism—that which puts human life in jeopardy. https://t.co/N5DhF8q2DN'), ('dates', '2020-05-05 23:06:05'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.7064294815063477)])
OrderedDict([('tweet', "The President has been trying to gut key provisions of the #ACA since assuming office. Denying millions of women access to vital care like contraception is an outrageous assault on women’s health. I'm standing with you through this fight, @PAAttorneyGen. https://t.co/vHj7kPkJAY"), ('dates', '2020-05-05 20:44:05'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.6546844244003296)])
OrderedDict([('tweet', '"Trump — whose decision-making has been guided largely by his reelection prospects... provided justification to pivot to where he preferred to be: cheering an economic revival rather than managing a catastrophic health crisis." https://t.co/BU1cQD2aMA'), ('dates', '2020-05-05 14:15:56'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.6474702954292297)])
OrderedDict([('tweet', "Over 30 million Americans have filed for unemployment since this crisis started, yet rent is still due tomorrow. So here's a reminder that Democrats fought for a nationwide freeze on evictions in the last #COVID19 relief bill but Republicans refused to negotiate on this."), ('dates', '2020-04-30 20:44:37'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.8674056529998779)])
OrderedDict([('tweet', 'The first iteration of the PPP program did not do nearly enough for truly small and minority-owned businesses. Since Democrats stepped in, the legislation passed yesterday will ensure resources genuinely help family, women, veteran and minority-owned Main Street businesses.'), ('dates', '2020-04-22 20:23:00'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.5341994166374207)])
OrderedDict([('tweet', 'I commend @PAAttorneyGen for calling on USDA to suspend its rulemaking that would cut assistance for over 3 million people. There’s never a good time to cut food assistance for those who need it most, but amidst a pandemic, we should only be strengthening our social safety net. https://t.co/ENwr5gQqF5'), ('dates', '2020-04-22 16:51:40'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.6031177043914795)])
OrderedDict([('tweet', '#EarthDay reading: in the midst of #COVID19, the President is moving forward with his dangerous anti-environmental agenda—but a pandemic involving a severe respiratory illness is definitely not the time to cut back on clean air protections.  https://t.co/qelS1czbsO'), ('dates', '2020-04-22 15:45:12'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.8405541181564331)])
OrderedDict([('tweet', 'On #YomHaShoah we honor the 6 million Jews &amp; millions of others murdered in the Holocaust. We have a responsibility to combat anti-Semitism, hate &amp; division, which is why I’m a member of @SenJackyRosen &amp; @SenatorLankford’s Senate Bipartisan Task Force for Combating Anti-Semitism.'), ('dates', '2020-04-20 22:19:24'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.7353354096412659)])
OrderedDict([('tweet', 'The President has fallen short of HIS OWN pandemic goals, and now he is trying to rewrite history in order to absolve himself of blame. This Nation’s testing inadequacies are a direct result of President Trump’s actions, not the states. https://t.co/NosHYShSNH'), ('dates', '2020-04-18 17:08:53'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.870011568069458)])
OrderedDict([('tweet', 'It is outrageous that scammers are seeking to capitalize on the public anxiety of COVID-19. Please stay alert for these malicious phone scams and don’t give your personal or financial information to a caller you don’t know or trust—you don’t have to pay a dime for your payment. https://t.co/CnJtl2TVkF'), ('dates', '2020-04-16 21:28:24'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.5741580128669739)])
OrderedDict([('tweet', '.@realDonaldTrump claims he has the #COVID19 pandemic "totally under control." Our reality tells a different story. The President\'s response throughout this pandemic has been a case study of misinformation, misplaced blame and an inexcusable failure in leadership (THREAD)'), ('dates', '2020-04-16 19:54:51'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.6480653882026672)])
OrderedDict([('tweet', "Reminder: The problem isn't just that millions of Americans are without health care coverage during a global pandemic. Thanks to the Trump Admin’s efforts to undermine the ACA &amp; push junk health plans, many Americans have insurance that will NOT cover #COVID19 testing/treatment."), ('dates', '2020-04-16 17:15:33'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.7517743110656738)])
OrderedDict([('tweet', '#COVID19 is particularly dangerous for those with pre-existing conditions, yet Republicans are in court trying to destroy pre-existing condition protections... in the middle of a pandemic.\n\n@morethanmySLE and I have a message for Republicans: #ProtectOurCare and #DropTheLawsuit https://t.co/yPejU516Nr'), ('dates', '2020-04-15 15:32:52'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.778969407081604)])
OrderedDict([('tweet', 'Scammers are taking advantage of the mass confusion and anxiety caused by #COVID19 so I want to be clear: The government will NOT call to ask you for your Social Security number, bank account or credit card number in order to send your relief check. Anyone that does is a scammer.'), ('dates', '2020-04-10 19:39:28'), ('file_name', 'Robert P. Casey, Jr..csv'), ('pct_true', 0.790082573890686)])
OrderedDict([('tweet', 'For years, the Senate has confirmed our nation’s most senior national security leaders. We can’t allow this or any President to use the “acting” distinction to skirt Congressional oversight, hence my National Security Transparency Act. https://t.co/uY5UcuTxuu'), ('dates', '2020-02-27 21:46:22'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.8772383332252502)])
OrderedDict([('tweet', 'In the year that #HR8 has sat on @SenateMajLdr’s desk, 232 of his Kentucky constituents have been killed by guns. His inaction costs American lives. He must stand up to the #NRA and #DoSomething! https://t.co/42sekwpzmm'), ('dates', '2020-02-27 19:23:15'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.5437976121902466)])
OrderedDict([('tweet', 'You can’t make these headlines up. This Administration has had its priorities misplaced &amp; failed the American people since Day One. https://t.co/m9MHOaMP56'), ('dates', '2020-02-25 19:59:22'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.6682745218276978)])
OrderedDict([('tweet', 'The #NRA has long opposed scientific research into gun violence. The reason is clear: they don’t want the American people to have the facts, they only want to spread propaganda. #HR1114 https://t.co/K5o9JAgb89'), ('dates', '2020-02-24 18:44:02'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.7207587361335754)])
OrderedDict([('tweet', 'In 2018, 39,741 Americans lost their lives to #gunviolence. While the House passed commonsense gun safety bills nearly a year ago, President @realDonaldTrump &amp; @SenateGOP remain too beholden to the #NRA’s donations to act &amp; save lives. #AtWhatCost https://t.co/yTfpcObzfO'), ('dates', '2020-02-13 19:22:10'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.8759579062461853)])
OrderedDict([('tweet', 'You can’t actively contribute to a reckless administration and then apologize afterward. Sad to see so many follow President Trump into infamy after careers of dedicated service. https://t.co/lzhrnkLXFh'), ('dates', '2020-02-13 17:50:04'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.5584519505500793)])
OrderedDict([('tweet', 'It was difficult and frustrating to sit through the #SOTU as the President outright lied, over and over. The Trump Economy is not magic, it’s the continuation of the Obama Economy. #ThanksObama https://t.co/iq3MMBYUVs'), ('dates', '2020-02-12 17:16:40'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.8034413456916809)])
OrderedDict([('tweet', 'Because the American people know when they are being lied to and they don’t like it. https://t.co/P76G8HKf2Q'), ('dates', '2020-02-06 22:59:43'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.6034013628959656)])
OrderedDict([('tweet', 'Hardworking Americans deserve the right to organize. For too long, we’ve watched our middle class erode as wealthy corporations and their lobbyists undermine workers’ ability to organize. NOT ANYMORE! I’m proud to support the #PROAct'), ('dates', '2020-02-06 21:12:37'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.8944891691207886)])
OrderedDict([('tweet', 'Beyond disappointed to watch long respected members of @SenateGOP destroy their legacies by putting party before country. We are all Americans first, not Republicans or Democrats.'), ('dates', '2020-01-31 15:14:30'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.7911314964294434)])
OrderedDict([('tweet', 'Agree with him or not, Amb. Bolton has been a fixture of national security for decades. The Senate MUST hear from him! #BoltonMustTestify'), ('dates', '2020-01-27 21:47:33'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.5315549373626709)])
OrderedDict([('tweet', '#OTD 75 years ago, liberation came to Auschwitz. As we reflect on the horrors of the Holocaust, we recommit to #NeverAgain and to standing up to hatred and bigotry in all its forms. #HolocaustRemembranceDay'), ('dates', '2020-01-27 17:51:09'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.5407842993736267)])
OrderedDict([('tweet', 'Social Security is not a handout. Americans worked hard for decades and paid into the system. We deserve to get what we paid for! https://t.co/dpamwqgXfk'), ('dates', '2020-01-27 16:24:21'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.5044829845428467)])
OrderedDict([('tweet', 'The Trump Administration blatantly lied to the American people by saying “there were no casualties.” That statement was completely false; the American people deserve the truth when it comes to our national security. https://t.co/DaRzF28fcl'), ('dates', '2020-01-24 20:47:45'), ('file_name', 'Robin L. Kelly.csv'), ('pct_true', 0.9608631134033203)])
OrderedDict([('tweet', 'The @IRSnews has announced that Social Security retirement, survivors, and disability beneficiaries will begin receiving their Economic Impact Payments this week.\n\nTo date the IRS has issued more than 88.1 million payments out of 150 million total payments that will be issued.'), ('dates', '2020-04-27 18:00:00'), ('file_name', 'Rodney Davis.csv'), ('pct_true', 0.5800826549530029)])
OrderedDict([('tweet', "Speaker Pelosi is proposing a bill that includes many policies unrelated to the Coronavirus pandemic. We need to pass a bill that focuses every effort on providing relief to Americans &amp; businesses struggling, not one party's political wishlist."), ('dates', '2020-03-23 22:27:50'), ('file_name', 'Rodney Davis.csv'), ('pct_true', 0.6709098219871521)])
OrderedDict([('tweet', '.@SenateGOP fought to provide targeted relief for the American people, while @SpeakerPelosi and House Democrats unveiled a laundry list of unrelated partisan demands.\n\nHere’s what @SenateGOP kept out of this week’s #CARESAct:\n\n(THREAD)'), ('dates', '2020-03-26 19:17:43'), ('file_name', 'Roger F. Wicker.csv'), ('pct_true', 0.9511787295341492)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram gave TX small businesses over $28 billion to help keep their doors open.\n\nFor two weeks, Democrats held additional funds hostage because they didn’t want to lose “leverage.”\n\nI’m proud to stand with Main Street by voting for more relief today. https://t.co/OXhzZx6gNd'), ('dates', '2020-04-23 14:03:23'), ('file_name', 'Roger Williams.csv'), ('pct_true', 0.6890265345573425)])
OrderedDict([('tweet', 'Small businesses and families should NEVER have been forced to wait on the Democrats’ political wish list. \n\nPrecious time has already been lost — we need to replenish funds for the #PaycheckProtectionProgram immediately. https://t.co/evFaLYfdcj'), ('dates', '2020-04-21 21:52:18'), ('file_name', 'Roger Williams.csv'), ('pct_true', 0.5771737694740295)])
OrderedDict([('tweet', 'Millions of Americans are losing their jobs and small businesses are on the verge of closing their doors. \n\nMeanwhile, Democrats are concerned about “leverage.” https://t.co/j79YEPrBrI'), ('dates', '2020-04-21 14:45:44'), ('file_name', 'Roger Williams.csv'), ('pct_true', 0.8986086845397949)])
OrderedDict([('tweet', 'Democrats are playing games while small businesses desperately need more money NOW in the #PaycheckProtectionProgram. My full statement: https://t.co/1uLkaSSNh7. https://t.co/eKBkdah3wZ'), ('dates', '2020-04-16 17:10:11'), ('file_name', 'Roger Williams.csv'), ('pct_true', 0.6666755676269531)])
OrderedDict([('tweet', 'Democrats just blocked additional emergency funding for the #PaycheckProtectionProgram. \n\nSmall businesses need additional funds in the PPP now — there’s no time to waste on political games.'), ('dates', '2020-04-09 15:34:12'), ('file_name', 'Roger Williams.csv'), ('pct_true', 0.9461271166801453)])
OrderedDict([('tweet', 'The Paycheck Protection Program is a lifeline intended for small businesses impacted by COVID-19. Intentionally fraudulent applications, not mistakes, are unacceptable and take away from Kansans who really need it.'), ('dates', '2020-05-06 20:23:59'), ('file_name', 'Ron Estes.csv'), ('pct_true', 0.5455319881439209)])
OrderedDict([('tweet', 'Since it’s implementation, the Paycheck Protection Program has been popular with small businesses in Kansas. We need a clean bill to add more funding to this program immediately.\n\nIt’s time that Congress does the right thing to keep people employed and small businesses open. https://t.co/BUk227TEY0'), ('dates', '2020-04-16 20:26:14'), ('file_name', 'Ron Estes.csv'), ('pct_true', 0.6498730182647705)])
OrderedDict([('tweet', 'The Paycheck Protection Program offers a lifeline to small businesses, allowing 15 million Americans to keep their jobs. Now it needs more funding.\n \nSpeaker Pelosi &amp; Leader Schumer need to stop playing politics with our small businesses &amp; allow Congress to pass a clean bill. https://t.co/VnenszcYOO'), ('dates', '2020-04-16 17:49:36'), ('file_name', 'Ron Estes.csv'), ('pct_true', 0.7852356433868408)])
OrderedDict([('tweet', "This morning's #coronavirus briefing was encouraging as the FDA, scientists, medical professionals, &amp; innovators are working around the clock to develop &amp; approve safe &amp; effective anti-viral treatments. This is only possible because of unleashed American ingenuity. #InItTogether"), ('dates', '2020-03-19 17:25:05'), ('file_name', 'Ron Estes.csv'), ('pct_true', 0.7609521746635437)])
OrderedDict([('tweet', 'Small businesses impacted by the #coronavirus crisis, like this Kimberly daycare, are who #PPP is supposed to help. I continue to push to make sure PPP is helping these businesses and is not used up by others who aren’t hurt by the crisis. https://t.co/DHm4bZFNPb'), ('dates', '2020-04-29 23:49:50'), ('file_name', 'Ron Johnson.csv'), ('pct_true', 0.6335242986679077)])
OrderedDict([('tweet', 'You are either helping right now, or hurting. Pelosi’s handpicked partisan ploys are helping no one. She’s hurting American families by prioritizing airplane emissions, same day voter registration&amp;wind energy tax credits over #coronavirus relief. #StopTheGamesNancy, #PassTheBill'), ('dates', '2020-03-24 15:03:13'), ('file_name', 'Ron Johnson.csv'), ('pct_true', 0.9826825857162476)])
OrderedDict([('tweet', 'America is in the midst of a national crisis that is no one’s fault. But staying focused on these 3 goals I outline in @WSJopinion will help ensure the massive amount of federal  #coronavirus spending will be used as efficiently and effectively as possible.https://t.co/fc0nRnxqRF'), ('dates', '2020-03-21 00:00:00'), ('file_name', 'Ron Johnson.csv'), ('pct_true', 0.7143204808235168)])
OrderedDict([('tweet', 'As @SecPompeo has made clear, Crimea is Ukraine. Six years after Russia’s illegal attempt to annex Crimea, Vladimir Putin should know that the United States will continue to answer Russian aggression with strength and resolve. https://t.co/6oxB3b94pI'), ('dates', '2020-02-27 19:58:48'), ('file_name', 'Ron Johnson.csv'), ('pct_true', 0.7515180706977844)])
OrderedDict([('tweet', "Disappointed that Democrat colleagues would seek to undermine legitimate oversight being done by @ChuckGrassley and me by selectively leaking information to the press. We won't be deterred. https://t.co/qVS1vxtRvl"), ('dates', '2020-02-07 21:57:30'), ('file_name', 'Ron Johnson.csv'), ('pct_true', 0.8502311706542969)])
OrderedDict([('tweet', "Just last year, @SpeakerPelosi said that any impeachment ‘would have to be so clearly bipartisan in terms of acceptance of it.’ \n\nYet that's exactly what House Democrats passed and why I voted to acquit @realDonaldTrump this afternoon. https://t.co/HDSBgw7stI"), ('dates', '2020-02-05 21:44:54'), ('file_name', 'Ron Johnson.csv'), ('pct_true', 0.8073023557662964)])
OrderedDict([('tweet', 'The Democrat Iowa caucus debacle is a perfect metaphor for Democrat policy proposals for America - they would be a total disaster.'), ('dates', '2020-02-04 12:47:09'), ('file_name', 'Ron Johnson.csv'), ('pct_true', 0.7502004504203796)])
OrderedDict([('tweet', 'To use the House managers’ own argument on witnesses:\n \n“The American people deserve a fair impeachment.”\n“The Constitution deserves a fair impeachment.”\n“The president deserves a fair impeachment.”'), ('dates', '2020-02-01 00:33:27'), ('file_name', 'Ron Johnson.csv'), ('pct_true', 0.6815001368522644)])
OrderedDict([('tweet', 'For more than 2 years businesses &amp; working families have been footing the bill for tariffs implemented in reckless trade wars. Now these tariffs are impacting the ability for our health care workers to get the medical supplies they need to fight #COVID19. Now more than ever we...'), ('dates', '2020-03-24 23:17:21'), ('file_name', 'Ron Kind.csv'), ('pct_true', 0.7305964231491089)])
OrderedDict([('tweet', '10 years ago today, the Supreme Court made the disastrous decision of allowing dark money to influence our elections with its ruling in Citizens United v. FEC. https://t.co/ttNODoOTj8'), ('dates', '2020-01-21 19:26:42'), ('file_name', 'Ron Kind.csv'), ('pct_true', 0.5079972147941589)])
OrderedDict([('tweet', "Wisconsinites know that sticking to a budget is about discipline &amp; priorities. Meanwhile, leaders of both parties have passed reckless tax cuts for billionaires &amp; bills that aren't paid for, causing the deficit to balloon. It's irresponsible, unacceptable, &amp; must stop. https://t.co/aOpAuORX9A"), ('dates', '2020-01-13 22:33:37'), ('file_name', 'Ron Kind.csv'), ('pct_true', 0.6872947216033936)])
OrderedDict([('tweet', 'There is no doubt that Soleimani was an enemy of the United States and has been responsible for the death of thousands—including many of our own men and women in uniform. However, this Administration’s decision escalated an already tense situation with Iran, bringing us closer to'), ('dates', '2020-01-03 22:29:26'), ('file_name', 'Ron Kind.csv'), ('pct_true', 0.6657902002334595)])
OrderedDict([('tweet', 'I’m pleased to share that today, Congress and the Administration came to a bipartisan agreement on USMCA. For months, my colleagues and I have worked and advocated for a trade deal that would ensure a level playing field for'), ('dates', '2019-12-10 21:56:32'), ('file_name', 'Ron Kind.csv'), ('pct_true', 0.5841255784034729)])
OrderedDict([('tweet', 'This is what happens when Congress passes massive unpaid for tax cuts for billionaires and special interests at the expense of Wisconsin’s future. We can’t keep racking up bills on our nation’s credit card and expecting our kids to foot the bill. https://t.co/kAhIBZkCn1'), ('dates', '2019-10-25 19:30:54'), ('file_name', 'Ron Kind.csv'), ('pct_true', 0.5185128450393677)])
OrderedDict([('tweet', '84 years ago today, FDR signed The Social Security Act of 1935 into law, giving older Americans stability and security. #SocSec84 https://t.co/GFfLzkvfRM'), ('dates', '2019-08-14 20:21:24'), ('file_name', 'Ron Kind.csv'), ('pct_true', 0.5196995139122009)])
OrderedDict([('tweet', "Tariffs are taxes on WI's farmers, workers, &amp; families, &amp; that’s exactly who is going to feel it after this escalation."), ('dates', '2019-08-01 19:37:20'), ('file_name', 'Ron Kind.csv'), ('pct_true', 0.5192541480064392)])
OrderedDict([('tweet', 'On #HolocaustRemembranceDay, we remember the millions of lives lost because of hate - and recommit to standing against bigotry and antisemitism every day. #NeverAgain'), ('dates', '2019-05-02 21:28:20'), ('file_name', 'Ron Kind.csv'), ('pct_true', 0.8650580644607544)])
OrderedDict([('tweet', "On May 8, 1945, Allied forces declared victory in Europe over Nazi Germany in #WWII.\n\nOur fathers &amp; grandfather's generation met and defeated tyranny &amp; oppression. The biggest threat to the current generation is the #CCP that controls the most populous country on earth. #VEDay75 https://t.co/WrdLWidf3N"), ('dates', '2020-05-08 21:03:25'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.5391539335250854)])
OrderedDict([('tweet', 'I applaud @GOPLeader for creating the China Task Force &amp; I am confident that my fellow Texan @RepMcCaul will hold Communist China accountable for their COVID-19 coverup.\n\nThe list of wrongdoings committed by the #CCP is extensive, but one thing is clear, they lied &amp; people died. https://t.co/oqx83yjjOY'), ('dates', '2020-05-07 21:51:23'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.5009254217147827)])
OrderedDict([('tweet', 'Communist China is STILL blocking U.S. scientists and health experts from researching the origin of coronavirus.\n\nI agree with @SecPompeo. We need access to those labs to figure out what happened. The longer the #CCP refuses to cooperate, the more lives will be lost. https://t.co/IeKGDzr0NL'), ('dates', '2020-05-05 18:08:08'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.9408468008041382)])
OrderedDict([('tweet', 'Remember when Communist China tried to blame our military for COVID-19? \n\nThe #CCP is attempting to re-write history &amp; shift blame to the United States for the origin of COVID-19. \n\nWe all know the truth. This virus originated in Wuhan. Period. https://t.co/ghTvFJXy6H'), ('dates', '2020-04-29 14:58:36'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.9361303448677063)])
OrderedDict([('tweet', 'Our taxpayer dollars should not be sent to dangerous bio-agent labs in Wuhan ran by Communist China. To make matters worse, these labs have no US oversight &amp; a history of lab leaks. \n\nI sent a letter led by @SenMcSallyAZ and @RepMattGaetz urging leadership to end this funding.👇 https://t.co/SXhGNgS1BP'), ('dates', '2020-04-27 14:43:16'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.907457709312439)])
OrderedDict([('tweet', 'Don\'t be fooled, this ninth "Impeachment 2.0" committee created by Speaker Pelosi, and led by Joe Biden\'s biggest supporter, is just another attempt by Democrats to use their House majority to go after @realDonaldTrump.\n\nMy full statement ⬇ https://t.co/5dYd7gbKy2'), ('dates', '2020-04-23 19:23:42'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.5505156517028809)])
OrderedDict([('tweet', 'I could not agree more with my colleague @Jim_Jordan. \n\nAfter blocking small business funding for two weeks, House Democrats want to use taxpayer dollars for a political hack job while our nation continues to fight a global pandemic. https://t.co/p4l3GClV3N'), ('dates', '2020-04-23 16:48:06'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.5688111186027527)])
OrderedDict([('tweet', 'Democrats blocked #PPP funding for 2 weeks ⤵️\n\n4/7 → @POTUS requests PPP funding\n4/9 → Senate Dems block efforts for PPP funding\n4/16 → PPP runs out of money\n4/23 → The House will finally pass more funding\n\nThe result: 4.4 million Americans filing for unemployment. Shameful. https://t.co/3O3lZyuzU9'), ('dates', '2020-04-23 15:17:52'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.7288123369216919)])
OrderedDict([('tweet', 'Texas has processed 134,737 #PaycheckProtectionProgram loans. That’s 134,737 small businesses that are able to keep their employees on the payroll during this crisis. \n\nSmall businesses need more #PPP funding and they need it NOW. Millions of American jobs hang in the balance.'), ('dates', '2020-04-21 16:28:57'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.8915278911590576)])
OrderedDict([('tweet', 'On Yom Hashoah, we remember the over 6 million Jews and millions of others that were murdered during the Holocaust. Wherever we are in the world, we must always stand up against all forms of antisemitism, bigotry, and hatred. \n\nWe will #NeverForget. https://t.co/Y4oNCI3rnd'), ('dates', '2020-04-21 14:23:21'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.522014319896698)])
OrderedDict([('tweet', 'On Friday, @RepChrisSmith &amp; I introduced legislation that holds Communist China accountable for using the #WHO as a mouthpiece to spread lies &amp; false propaganda. \n\nMore info👇\nhttps://t.co/mi9QpU4OA1'), ('dates', '2020-04-20 17:39:38'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.8770228028297424)])
OrderedDict([('tweet', '5.2 million Americans filed for unemployment in the last week, and the #PaycheckProtectionProgram is officially out of money. \n\nIt’s time for Democrats to stop holding American jobs hostage. Our workers and small businesses deserve better than this. https://t.co/oNBTbXE5Eo'), ('dates', '2020-04-16 15:15:43'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.8082834482192993)])
OrderedDict([('tweet', 'Sadly, this headline should not be surprising to anyone. \n\nThe Chinese Communist Party stayed silent for a week, waited until 3000 ppl had been infected, before alerting the global community of the COVID-19 outbreak. \n\nWe must hold the #CCP accountable for these reckless actions. https://t.co/kHHj7i6Alc'), ('dates', '2020-04-15 14:13:05'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.9457513093948364)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram is working for our small biz. Over 1 million loans have been approved &amp; $240+ billion in loans have been processed. \n\nThe longer Democrats continue to play politics with additional funding for the #PPP, the more American jobs they put at risk.'), ('dates', '2020-04-14 18:01:53'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.7570850253105164)])
OrderedDict([('tweet', 'The Chinese Communist Party relies on total and complete control of media &amp; speech. Their lack of transparency since day one of the #COVID19 crisis has led to deaths in the U.S. and around the globe. \n\nThe #CCP lied to the world and they need to be held accountable.'), ('dates', '2020-04-07 15:49:57'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.9576034545898438)])
OrderedDict([('tweet', 'I am proud to be a cosponsor on @RepJimBanks &amp; @SethMoulton’s resolution condemning the Chinese Communist Party for their handling of #COVID19. \n\nThe blatant lies coming out of China have led to the loss of American lives. Enough is enough. The #CCP needs to be held accountable. https://t.co/nunula6r2B'), ('dates', '2020-03-24 20:28:28'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.6892701387405396)])
OrderedDict([('tweet', 'Twice in one day, Democrats blocked:\n\n❌ Resources for health care professionals &amp; hospitals\n❌ Stability for small businesses\n❌ Direct payments to Americans\n\nThe American people deserve better than the partisan politics we saw today from @SenSchumer and @SpeakerPelosi.'), ('dates', '2020-03-23 22:17:05'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.5820193290710449)])
OrderedDict([('tweet', 'The public health and economic stability of our nation are at risk, and the American people are looking to Congress for leadership.\n \nInstead, Senate Dems are playing partisan politics while American families and businesses continue to suffer. We need to take action. https://t.co/yqf1qcgLYt'), ('dates', '2020-03-23 17:27:43'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.8313189148902893)])
OrderedDict([('tweet', 'Since day one Democrats have politicized the #coronavirus. They are adding poison pills to the bill that will hold up innovation putting the safety of Americans at risk. \n\nWe need a CLEAN funding bill and we need it NOW. https://t.co/DLR3znhN7P'), ('dates', '2020-03-04 17:17:44'), ('file_name', 'Ron Wright.csv'), ('pct_true', 0.9488041400909424)])
OrderedDict([('tweet', 'We are 183 days away from the 2020 election and Mitch McConnell is still refusing to expand #VoteByMail through November so that Americans won’t be forced to choose between their health and exercising their constitutional right to vote. https://t.co/IVXrlxeS27'), ('dates', '2020-05-04 14:01:28'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.6832846999168396)])
OrderedDict([('tweet', "As usual, big corporations are getting richer and minority-owned small businesses are getting left behind. Congress must be doing more to ensure minority and women-owned businesses don't fall through the cracks during this economic crisis. https://t.co/T8fRQhAZd8"), ('dates', '2020-05-01 19:24:37'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.7742398977279663)])
OrderedDict([('tweet', 'Thirty MILLION Americans have lost their jobs in a little over a month and Republicans are still debating the merits of unemployment assistance. Instead of turning this into partisan floor debate get to work helping American families. https://t.co/It90qmDKwp'), ('dates', '2020-05-01 16:49:37'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.8931471705436707)])
OrderedDict([('tweet', 'Over 30 million Americans have lost their jobs through no fault of their own. Meanwhile Republicans in Congress and Trump officials are STILL claiming that workers are taking advantage of unemployment benefits for their own gain. Let that sink in. https://t.co/RFSTKgaJX8'), ('dates', '2020-04-30 14:02:41'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.9473618865013123)])
OrderedDict([('tweet', 'The same Republicans who voted in favor of blowing a $2 trillion hole in the deficit in 2017 are now crying foul as Democrats fight to extend a lifeline for the millions of Americans that have lost their jobs because of #COVID19.'), ('dates', '2020-04-27 20:40:44'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.9611399173736572)])
OrderedDict([('tweet', 'We are 190 days away from the 2020 election and Mitch McConnell is still refusing to expand #VoteByMail through November so that Americans won’t be forced to choose between their health and exercising their constitutional right to vote. https://t.co/4XNrLtwN4h'), ('dates', '2020-04-27 14:19:50'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.6832846999168396)])
OrderedDict([('tweet', "These numbers won't stop rising any time soon. This is all the more reason for Congress to continue working on the next bipartisan #COVID19 relief package to give American workers the help they need to weather this economic storm. America's economic future depends on it. https://t.co/7r83Y9P9S4"), ('dates', '2020-04-23 12:46:09'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.5763576030731201)])
OrderedDict([('tweet', 'My parents fled Nazi Germany and were welcomed to this country with open arms. These days the same can’t be said for many who flee violence for a safer life in America. We must remain united against those that sow divisiveness through hatred and fear.  #HolocaustRemembranceDay'), ('dates', '2020-04-21 15:05:44'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.9876695275306702)])
OrderedDict([('tweet', 'We are 197 days away from the 2020 election and Mitch McConnell is still refusing to expand #VoteByMail through November so that Americans won’t be forced to choose between their health and exercising their constitutional right to vote. https://t.co/uLjgGgZyUE'), ('dates', '2020-04-20 13:54:06'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.6832846999168396)])
OrderedDict([('tweet', "#COVID19 poses a unique risk to seniors and those in nursing homes. It’s unacceptable that @CDCgov and @CMSgov won't disclose the number of cases in nursing homes after @SenBobCasey and I pressed for answers. We must do better to protect America’s seniors from this virus."), ('dates', '2020-04-17 16:09:31'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.6149788498878479)])
OrderedDict([('tweet', '22 MILLION American workers have lost their jobs. Meanwhile Trump’s Labor Secretary is so out of touch that he thinks workers will become “dependent” on assistance. Trump officials need to stop worrying about an outbreak of laziness and get the unemployment checks out. https://t.co/vgznOCBe8M'), ('dates', '2020-04-16 14:25:22'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.9667196273803711)])
OrderedDict([('tweet', 'This money should be going directly to Americans to help them pay rent and put food on the table. Not shady debt collectors. I joined a bipartisan group of senators in demanding that the Trump administration ensure this money gets into the right hands. https://t.co/oe9FjyxLLk'), ('dates', '2020-04-15 15:23:04'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.608487606048584)])
OrderedDict([('tweet', ".@SenSchumer and I led Senate Democrats in pressing Trump's Labor Department to clarify guidelines and requirements so workers who have lost their jobs because of #COVID19 get the help they need as soon as possible. This can’t wait.\nhttps://t.co/N0NhW0BDg2"), ('dates', '2020-04-14 14:01:32'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.663472592830658)])
OrderedDict([('tweet', 'We are 204 days away from the 2020 election and Mitch McConnell is still refusing to expand #VoteByMail through November so that Americans won’t be forced to choose between their health and exercising their constitutional right to vote. https://t.co/8vCXkzRk9f'), ('dates', '2020-04-13 13:23:35'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.6832846999168396)])
OrderedDict([('tweet', 'Mitch McConnell’s opposition to any and all proposals that would expand access to voting for Americans isn’t new. It goes back decades. Here’s a thread on his long history of voting against voting:'), ('dates', '2020-04-10 17:51:34'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.6576762795448303)])
OrderedDict([('tweet', 'Regardless of party the majority of Americans want #VoteByMail. Donald Trump won’t let them because he’s afraid Republicans can’t win elections on their own merit. https://t.co/e5QgD6eHKH'), ('dates', '2020-04-08 18:25:03'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.6052457690238953)])
OrderedDict([('tweet', 'We are 211 days away from the 2020 election and Mitch McConnell is still refusing to expand #VoteByMail through November so that Americans won’t be forced to choose between their health and exercising their constitutional right to vote. https://t.co/cHwQx8nM21'), ('dates', '2020-04-06 14:06:12'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.5509462952613831)])
OrderedDict([('tweet', "Workers who have lost their jobs can't afford to wait for help. @SenSanders @SenSchumer and I are asking the Labor Department to expedite distribution of supercharged unemployment we secured in the #COVID19 relief package to get Americans the help they urgently need. https://t.co/RxV4V8R6Ql"), ('dates', '2020-04-01 18:49:36'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.5465133786201477)])
OrderedDict([('tweet', 'Not even a global pandemic will stop Donald Trump from putting his political agenda above Americans’ health care needs. https://t.co/O0mSlX6xJs'), ('dates', '2020-04-01 17:08:00'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.6698365807533264)])
OrderedDict([('tweet', 'NEW: Earlier this week I discovered a crucial error in Trump administration guidance for the #COVID19 relief bill that would block states like Oregon from getting additional Medicaid dollars for home-based care for seniors and patients with disabilities.'), ('dates', '2020-03-31 17:33:19'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.5845905542373657)])
OrderedDict([('tweet', 'We are 218 days away from the 2020 election and Mitch McConnell is still refusing to expand #VoteByMail through November so that Americans won’t be forced to choose between their health and exercising their constitutional right to vote.'), ('dates', '2020-03-30 15:06:36'), ('file_name', 'Ron Wyden.csv'), ('pct_true', 0.7522128224372864)])
OrderedDict([('tweet', 'The American people deserve the truth. We do not have a national testing plan. We do not have a national contact tracing plan. We do not have PPE. The American people are scared, and they want answers. https://t.co/kUw6CBHd9M'), ('dates', '2020-05-06 16:10:26'), ('file_name', 'Rosa L. DeLauro.csv'), ('pct_true', 0.9364026784896851)])
OrderedDict([('tweet', 'Tomorrow, the Supreme Court will hear arguments on Trump administration led rules that would allow employers to deny birth control coverage under the ACA. Access to birth control should not depend on where you live, work, or go to school. #HandsOffMyBC https://t.co/umvvd7FpzW'), ('dates', '2020-05-05 20:49:28'), ('file_name', 'Rosa L. DeLauro.csv'), ('pct_true', 0.5000404119491577)])
OrderedDict([('tweet', 'The White House’s decision is letting politics overtake public health. Proud to stand with @NitaLowey in condemning the White House’s partisan games. https://t.co/iKpwUSebRw'), ('dates', '2020-05-02 16:35:54'), ('file_name', 'Rosa L. DeLauro.csv'), ('pct_true', 0.6896407008171082)])
OrderedDict([('tweet', 'On Yom HaShoah we honor the lives of the millions of Jews who were killed by the Nazis in Germany. Families and communities were ripped apart, and the world witnessed genocide on a horrific scale. We must continue our efforts to call out anti-Semitism whenever we see it. https://t.co/dau0MAXlxc'), ('dates', '2020-04-20 20:01:44'), ('file_name', 'Rosa L. DeLauro.csv'), ('pct_true', 0.8373735547065735)])
OrderedDict([('tweet', 'Even in the midst of the COVID-19 pandemic—which has killed tens of thousands of Americans and demands the undivided attention of our federal government—the Trump administration remains focused on inflicting trauma on unaccompanied children. https://t.co/yxq1Bs043c'), ('dates', '2020-04-16 15:22:41'), ('file_name', 'Rosa L. DeLauro.csv'), ('pct_true', 0.603188693523407)])
OrderedDict([('tweet', 'The American people deserve the truth from their elected officials, and I am glad @SpeakerPelosi is willing to tell it. The truth will get us through this painful health and economic crisis, and it will save lives. -Rosa https://t.co/4coXqa1ej7'), ('dates', '2020-04-15 00:44:41'), ('file_name', 'Rosa L. DeLauro.csv'), ('pct_true', 0.5047529935836792)])
OrderedDict([('tweet', 'By denying millions of uninsured Americans the opportunity to sign up for healthcare, President Trump is putting American lives at risk. During uncertain times, our federal government should help people get the care they need—not make it more difficult. https://t.co/rtmd24X4lM'), ('dates', '2020-04-01 21:21:53'), ('file_name', 'Rosa L. DeLauro.csv'), ('pct_true', 0.5182941555976868)])
OrderedDict([('tweet', 'COVID-19 is the biggest economic and health crisis our country has ever faced. Nurses and doctors are in ICUs trying to save our humanity against this awful pandemic. The hour is dark, but Congress is ready to help get families, workers, and the country to the dawn of recovery. https://t.co/PK7xVzyWl6'), ('dates', '2020-03-27 13:48:26'), ('file_name', 'Rosa L. DeLauro.csv'), ('pct_true', 0.6479915976524353)])
OrderedDict([('tweet', 'Businesses are closing and people are losing their jobs right now because of the coronavirus pandemic. Working people and families are facing an unprecedented threat to their economic security. https://t.co/svsCJfMKLo'), ('dates', '2020-03-20 22:31:42'), ('file_name', 'Rosa L. DeLauro.csv'), ('pct_true', 0.7384068369865417)])
OrderedDict([('tweet', "Over 30+ million Americans out of work, #SmallBusinesses shut down, communities struggling with #COVID19, and this is what Speaker Pelosi is worried about?!?\n\nShame some can't put politics aside, not even during a national pandemic. Witchhunt 3.0:  https://t.co/ApwPikJJZC"), ('dates', '2020-04-30 14:05:16'), ('file_name', 'Ross Spano.csv'), ('pct_true', 0.603498101234436)])
OrderedDict([('tweet', 'China HAS TO be held responsible for #COVID19. Their intentional deception has lead to over 218k+ deaths worldwide and 58k+ deaths in the US, including in #FL15. \n\nI commend President @realDonaldTrump for stating he will halt funding to the @WHO-- They too, must answer. https://t.co/jvxV28s3pT'), ('dates', '2020-04-29 14:30:05'), ('file_name', 'Ross Spano.csv'), ('pct_true', 0.6230087280273438)])
OrderedDict([('tweet', "While Democrats were discussing ice cream flavors last week, another 4.4 million Americans lost their jobs. Unacceptable. Today, I will vote 'Yes' to replenish #PPP.\n\nToday, House Dems are also pushing impeachment 3.0 against @realDonaldTrump.  H.Res 935-- I will be voting NO."), ('dates', '2020-04-23 15:21:52'), ('file_name', 'Ross Spano.csv'), ('pct_true', 0.8588300347328186)])
OrderedDict([('tweet', 'Rep. Ross Spano: CARES Act gives needed relief to small business, citizens | Commentary https://t.co/Mj4ncPlmMP'), ('dates', '2020-04-09 16:56:58'), ('file_name', 'Ross Spano.csv'), ('pct_true', 0.6414117217063904)])
OrderedDict([('tweet', '🚨The American people need to know what Speaker Pelosi is putting in her version of the bill 🚨\n\nStill going through it myself, but this pretty much sums it up. Take a listen for yourself: https://t.co/WjMnwDV9BE'), ('dates', '2020-03-24 03:17:39'), ('file_name', 'Ross Spano.csv'), ('pct_true', 0.6453672647476196)])
OrderedDict([('tweet', 'Let’s be clear as to what happened yesterday:\n\nDemocrats said NO to providing  assistance to millions of Americans &amp; #SmallBusinesses affected by COVID19. To top it off, the @nytimes (MSM) tried to cover for them. ⤵️\n\nIts time for leadership, not politics: https://t.co/pgadhnQcog https://t.co/Mexfv3Jast'), ('dates', '2020-03-23 13:28:34'), ('file_name', 'Ross Spano.csv'), ('pct_true', 0.6342024803161621)])
OrderedDict([('tweet', "75 years ago today, Nazi Germany surrendered to Allied Forces, marking WWII's end in Europe."), ('dates', '2020-05-08 17:09:58'), ('file_name', 'Roy Blunt.csv'), ('pct_true', 0.9770384430885315)])
OrderedDict([('tweet', "It's been 4 days since Democrats let the #PaycheckProtectionProgram run out of funds. Here’s an example of why this program is so vital to Missouri’s businesses &amp; why it needs additional funds immediately https://t.co/hwNJTSvUgS"), ('dates', '2020-04-20 23:31:00'), ('file_name', 'Roy Blunt.csv'), ('pct_true', 0.569023847579956)])
OrderedDict([('tweet', 'Delaying additional funding for #PPP is reckless. Democrats are playing partisan games while delaying paychecks to Americans during a pandemic. https://t.co/UwQ3spLwSV'), ('dates', '2020-04-17 19:30:00'), ('file_name', 'Roy Blunt.csv'), ('pct_true', 0.9153587818145752)])
OrderedDict([('tweet', 'Air travel is down an estimated 96%, leaving Missouri airports with huge losses as a result of the #coronavirus pandemic.'), ('dates', '2020-04-14 18:47:44'), ('file_name', 'Roy Blunt.csv'), ('pct_true', 0.7898337841033936)])
OrderedDict([('tweet', 'The bipartisan #PaycheckProtectionProgram is working &amp; making a big impact for #smallbiz across the country. It’s already saved an estimated 15 million small business jobs. But Senate Dems are blocking additional funding for this program to play politics. Enough is enough. #PPP'), ('dates', '2020-04-14 16:26:08'), ('file_name', 'Roy Blunt.csv'), ('pct_true', 0.7429016828536987)])
OrderedDict([('tweet', 'Senate Democrats are back to partisan games - this time blocking a bipartisan bill that would provide small businesses &amp; the people who work for them additional relief from the #coronavirus impact. Keep in mind 99.4% of Missouri businesses are small businesses'), ('dates', '2020-04-09 16:24:03'), ('file_name', 'Roy Blunt.csv'), ('pct_true', 0.6277998685836792)])
OrderedDict([('tweet', 'The PPP is failing too many small local businesses. Large corporations shouldn’t be allowed to jump the line &amp; receive millions while smaller family-owned businesses are left out. My plan would provide across-the-board checks to every business &amp; nonprofit w under 50 employees. https://t.co/d6cmjcWMDO'), ('dates', '2020-04-29 16:45:54'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.7310371398925781)])
OrderedDict([('tweet', 'PPP is failing small businesses who need it most. \n\nWe need to get money directly to those who are being left behind.\n\nMy plan would level the playing field by immediately sending checks so businesses &amp; nonprofits can pay employees &amp; continue as the backbone of our economy. https://t.co/RF1XJMtT4O'), ('dates', '2020-04-29 14:09:38'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.6504341959953308)])
OrderedDict([('tweet', 'This admin &amp; @Treasury have decided to follow the law &amp; allow small Tribal businesses to access the PPP after I pushed them to for weeks. They must now also follow the law &amp; stop trying to divert Tribal govt relief aid to for-profit corporations. https://t.co/tJpgS8DCQb'), ('dates', '2020-04-24 17:18:34'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.7628078460693359)])
OrderedDict([('tweet', '.@ArizonaDES told my staff it may take "a few weeks" &amp; there’s "no timeline" for processing unemployment payments for thousands of workers that Congress provided in the CARES Act. People won’t be able to buy food &amp; could lose their homes by then. Gov. Ducey must answer for this.'), ('dates', '2020-04-22 18:59:29'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.7293427586555481)])
OrderedDict([('tweet', 'We can’t safely “reopen” the country until we have adequate testing. We still don’t have that. President Trump didn’t listen to scientists and doctors, and he didn’t take early steps to produce and distribute rapid national testing. We’re paying for that now.'), ('dates', '2020-04-17 00:43:14'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.5566629767417908)])
OrderedDict([('tweet', 'Underserved small businesses aren’t getting a fair shake. States/localities are bearing the brunt of the crisis. Hospitals are teetering on financial collapse &amp; saving lives w/o PPE. It’s clear what Congress is obligated to do &amp;  it should have been done a week ago. https://t.co/S9cn44Lsym'), ('dates', '2020-04-16 20:42:31'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.868656575679779)])
OrderedDict([('tweet', 'We need to broaden eligibility of SBA loans so tribally owned businesses, small community health clinics, gyms, nonprofits, and others have access. #SmallBizRelief'), ('dates', '2020-04-15 17:39:14'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.7478860020637512)])
OrderedDict([('tweet', 'Cut red tape! The Administration shouldn’t impose unnecessary hurdles on receiving SBA loans. Let’s prioritize smaller, underserved businesses that don’t have a legion of lawyers to help them apply. #SmallBizRelief'), ('dates', '2020-04-15 15:43:27'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.6960704326629639)])
OrderedDict([('tweet', 'Nobody should ever have to worry about medical bills, but certainly not during a pandemic. Today I introduced a bill with @RepGilCisneros to allow anyone with #COVID19 to be automatically eligible for Medicaid to cover related treatment.'), ('dates', '2020-04-07 15:54:17'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.5915030837059021)])
OrderedDict([('tweet', 'A Captain punished for saving his sailors’ lives but putting too many Admirals on the CC line? Smells like @USNavy brass BS, @SECNAV. https://t.co/wiipGipAb7'), ('dates', '2020-04-03 04:14:42'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.5332737565040588)])
OrderedDict([('tweet', 'Tribal communities will suffer disproportionately from this pandemic. Native communities have higher rates of pre-existing conditions &amp; Tribal health providers are chronically under-resourced. #CoronavirusIndianCountry 1/ https://t.co/jMUJRQV3pG'), ('dates', '2020-04-02 18:27:08'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.5832513570785522)])
OrderedDict([('tweet', 'While Mashpee is preparing to respond to a global pandemic, Trump is violating their ancestral right to tribal lands which they have inhabited for millennia. \n\nhttps://t.co/oNUd0XYskd'), ('dates', '2020-03-29 16:04:33'), ('file_name', 'Ruben Gallego.csv'), ('pct_true', 0.5912390351295471)])
OrderedDict([('tweet', "A Southampton study suggests the # of cases could have been reduced by 95% had China moved to contain the virus 3 weeks sooner. The @WHO shouldn't have downplayed the severity of this to appease Chinese officials. I'm cosponsoring @GReschenthaler bill to investigate this further. https://t.co/lkZrqxdkAc"), ('dates', '2020-04-13 19:27:49'), ('file_name', 'Russ Fulcher.csv'), ('pct_true', 0.680577278137207)])
OrderedDict([('tweet', "In Idaho we're already seeing the benefits of the US-China Phase One Agreement signed by @realDonaldTrump last month. The @USDA @SecretarySonny announced today that the agreement will allow immediate access to China for U.S. potatoes for processing.\n\nhttps://t.co/E6Ya9AdCyh"), ('dates', '2020-02-25 22:24:38'), ('file_name', 'Russ Fulcher.csv'), ('pct_true', 0.5216013789176941)])
OrderedDict([('tweet', 'It is unfortunate that democrats rejected an opportunity for a bipartisan Authorization for Use of Military Force (AUMF) bill. \n\nInstead, it has been shoved into a bill originally intended to honor veterans (Merchant Mariners of WWII Congressional Gold Medal Act) in order to...'), ('dates', '2020-01-30 16:58:15'), ('file_name', 'Russ Fulcher.csv'), ('pct_true', 0.8875073194503784)])
OrderedDict([('tweet', 'If POTUS wants to keep plants open, he must prioritize workers: PPE, safety standards, hazard pay, healthcare.\n\nMany are immigrants who Trump disparages and tries to deport. Now he needs their essential work and will risk safety without clear protections. Can’t have it both ways. https://t.co/CTms9qU3Xz'), ('dates', '2020-04-29 02:02:57'), ('file_name', 'Salud O. Carbajal.csv'), ('pct_true', 0.500321090221405)])
OrderedDict([('tweet', "In the middle of a global crisis, this administration chose to sneak by even more provisions that weaken the health of communities everywhere.\n\nThis is a poor use of administrative power. I'll keep fighting for strong, commonsense solutions to address climate change. https://t.co/3Vbydfyx3M"), ('dates', '2020-04-05 17:13:10'), ('file_name', 'Salud O. Carbajal.csv'), ('pct_true', 0.5341384410858154)])
OrderedDict([('tweet', "Farmers in North Missouri have taken a devastating one-two punch. Last year, we endured record-setting floods. This year, COVID-19 has turned markets on their head. That's why I'm urging the USDA to make assistance to farmers effective, fair, and fast. https://t.co/kudxruCYAL"), ('dates', '2020-04-30 20:00:01'), ('file_name', 'Sam Graves.csv'), ('pct_true', 0.5973714590072632)])
OrderedDict([('tweet', 'We need to replenish the Paycheck Protection Program now—no more political horse-trading, no more delays, &amp; no strings attached. Small businesses, the workers they employ, &amp; the families those workers provide for are all depending on us to get this done. https://t.co/7eVQEtSYPa'), ('dates', '2020-04-17 23:22:47'), ('file_name', 'Sam Graves.csv'), ('pct_true', 0.8071480989456177)])
OrderedDict([('tweet', 'This coronavirus pandemic has posed unprecedented challenges for workers, families, and small businesses—but Americans have responded with unprecedented patriotism, making sacrifices and working together to get through this. https://t.co/lMGsSP6rIk'), ('dates', '2020-03-31 16:00:01'), ('file_name', 'Sam Graves.csv'), ('pct_true', 0.5242664217948914)])
OrderedDict([('tweet', 'The coronavirus bill that Speaker Pelosi tried to ram through yesterday had almost no support, even from within her own party. \n\nShe had to shelve her partisan wish list and start negotiating a better deal that will help the American people through this coronavirus outbreak.'), ('dates', '2020-03-13 16:00:31'), ('file_name', 'Sam Graves.csv'), ('pct_true', 0.8474767804145813)])
OrderedDict([('tweet', 'I just voted against ANOTHER attempt by House majority to stop President Trump from eliminating dangerous terrorists like Qasem Soleimani—who was responsible for murdering hundreds of American service members and slaughtering of thousands of innocent civilians.'), ('dates', '2020-03-11 21:59:00'), ('file_name', 'Sam Graves.csv'), ('pct_true', 0.935255765914917)])
OrderedDict([('tweet', 'When FEMA makes a mistake, FEMA should pay the price for that mistake—not honest disaster victims who are just trying to rebuild their lives. My bill will hold FEMA accountable for their mistakes. https://t.co/PychI00SWu'), ('dates', '2020-03-10 16:37:08'), ('file_name', 'Sam Graves.csv'), ('pct_true', 0.8512244820594788)])
OrderedDict([('tweet', 'Last week I introduced the Preventing Disaster Revictimization Act. It’s a short, simple bill that would hold FEMA accountable for their mistakes—instead of allowing FEMA to punish, threaten, &amp; harass honest disaster victims who are awarded aid because of a mistake made by FEMA. https://t.co/CArVIA7MTE'), ('dates', '2020-03-03 19:00:00'), ('file_name', 'Sam Graves.csv'), ('pct_true', 0.6067512631416321)])
OrderedDict([('tweet', 'We’ve kicked this can down the road long enough. We need to fix our roads and bridges—and we can’t let petty partisan politics get in the way. https://t.co/aA9KMEkONq'), ('dates', '2020-02-14 19:00:00'), ('file_name', 'Sam Graves.csv'), ('pct_true', 0.6212424039840698)])
OrderedDict([('tweet', 'It is unfortunate that the desire to impeach the President runs so deep that even issues related to public buildings have become partisan.\n \nIt’s ridiculous that the majority is trying to slam this President for a lease negotiated under the last administration. https://t.co/PMlzNufJUj'), ('dates', '2020-01-28 21:00:00'), ('file_name', 'Sam Graves.csv'), ('pct_true', 0.6568819284439087)])
OrderedDict([('tweet', "The COVID-19 outbreak has changed our lives &amp; affected every aspect of our nation. We are facing an extreme public health emergency &amp; we have invested billions into helping those who've been hit the hardest by the virus’ impact. https://t.co/JH6LABX5Ho"), ('dates', '2020-03-27 17:52:45'), ('file_name', 'Sanford D. Bishop, Jr..csv'), ('pct_true', 0.6635729670524597)])
OrderedDict([('tweet', 'While the Pres. has a duty to protect Americans from threats, by using military force when necessary, using it w/o a threat or Congressional approval is not only reckless, it is a violation of the Constitution.'), ('dates', '2020-01-09 23:11:42'), ('file_name', 'Sanford D. Bishop, Jr..csv'), ('pct_true', 0.7557824850082397)])
OrderedDict([('tweet', 'American ag has a labor crisis. U.S. workers stopped working in ag jobs &amp; farmers became dependent on foreign workers to keep their farms running and operating. H.R. 5038 requires applicants to undergo background checks &amp; pass criminal/ntl sec bars. 1/2'), ('dates', '2019-12-11 23:14:31'), ('file_name', 'Sanford D. Bishop, Jr..csv'), ('pct_true', 0.5992093086242676)])
OrderedDict([('tweet', 'This rule only serves to punish our most vulnerable citizens &amp; during the holiday season adds insult to injury. Instead of helping these Americans, the Administration has decided to disparage them as lazy &amp; undeserving. #HandsOffSNAP https://t.co/E8QCZyGQ1s'), ('dates', '2019-12-04 20:34:09'), ('file_name', 'Sanford D. Bishop, Jr..csv'), ('pct_true', 0.5852893590927124)])
OrderedDict([('tweet', "This CR is vital to ensure Americans aren't hurt by a gov shutdown. Americans shouldn’t have to pay the price when Congress can’t agree on funding. While I support the CR, I urge the Senate to pass the House-passed approps bills &amp; get them to the President.https://t.co/CQgGZg3ZzX"), ('dates', '2019-09-19 20:15:51'), ('file_name', 'Sanford D. Bishop, Jr..csv'), ('pct_true', 0.7352253198623657)])
OrderedDict([('tweet', 'Once again, Republicans have found a way to prolong the suffering of millions of Americans by blocking H.R. 2157. This disaster relief is vital to the livelihoods of millions and should not be used as a political pawn.'), ('dates', '2019-05-28 18:12:33'), ('file_name', 'Sanford D. Bishop, Jr..csv'), ('pct_true', 0.8268802762031555)])
OrderedDict([('tweet', 'Serious - but, sadly, not surprising - developments with the China lab COVID 19 stories.  Full investigations at highest levels must continue.'), ('dates', '2020-04-16 18:20:48'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.7717896699905396)])
OrderedDict([('tweet', 'When Donald Trump warned the country of coronavirus from China and declared the U.S. would work to defeat it — Nancy Pelosi tore up his speech. She was too busy with impeachment to help Donald Trump protect the country, which he has worked hard to do despite these childish antics https://t.co/IBFAB8Nxdq'), ('dates', '2020-04-01 18:56:32'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.8073838949203491)])
OrderedDict([('tweet', 'While Donald Trump was responding to coronavirus, Nancy Pelosi was handing out commemorative impeachment pens. Very early, he formed the Coronavirus Task Force, restricted travel from danger zones, and declared a public health emergency. https://t.co/A8m0QVoPYM'), ('dates', '2020-03-31 23:32:31'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.7260830402374268)])
OrderedDict([('tweet', 'When Donald Trump was dealing with coronavirus, Nancy Pelosi was stuck on impeachment. Even just a few weeks ago, the Speaker had scheduled more open-borders legislation for a vote but withdrew it as top scientists lauded travel restrictions President implemented very early https://t.co/pZDnGPTtKV'), ('dates', '2020-03-31 14:57:59'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.7998402714729309)])
OrderedDict([('tweet', 'The President has already shown remarkable foresight and energy in implementing an America First agenda that reduces our country’s dependence on unreliable China, where the Communist Party created the conditions for coronavirus and concealed the extent. https://t.co/U8OdpJIoY7'), ('dates', '2020-03-30 19:32:58'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.9001625180244446)])
OrderedDict([('tweet', 'The Liberal Media media is determined to prevent you from watching the President of the United States deliver updates about coronarvirus. Would anybody trust these reporters to report his daily briefings accurately? The briefings are a great source of information!'), ('dates', '2020-03-29 23:03:13'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.8568302392959595)])
OrderedDict([('tweet', 'The federal government had a unproductive stranglehold on testing. But once President Trump lifted restrictions, opening testing to private labs and states, testing accelerated rapidly. https://t.co/pNPICpuDOD'), ('dates', '2020-03-27 15:04:17'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.6834271550178528)])
OrderedDict([('tweet', 'All too common attitude on the Left and in liberal media — praising communist regime that spread coronavirus around the globe, while attacking the President who warned of China, impeaching him during initial response, and spending big on negative TV ads rather than assisting.'), ('dates', '2020-03-25 19:02:35'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.9525318741798401)])
OrderedDict([('tweet', 'At a time when we’re trying to get businesses and employees back up and running, Nancy Pelosi thought now would be a great time to saddle them with MORE COSTS as part of the Green New Deal, which the Resistance is trying to shove into emergency coronavirus relief. Unbelievable.'), ('dates', '2020-03-24 14:27:30'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.6250560283660889)])
OrderedDict([('tweet', 'This is pretty low. After pursuing another failed impeachment — as the President was leading our national response to coronavirus — Democrats immediately began pouring money into negative attack ads. All politics, all the time.  Resistance. Obstruction. We will succeed anyway! https://t.co/fFYf5ZP92I'), ('dates', '2020-03-22 14:24:31'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.8334690928459167)])
OrderedDict([('tweet', 'It’s not just that CCP (Chinese Communist Party) hid coronavirus from world and did little to stop spread. Primary problem is CCP’s flouting of environmental and health standards, in order to undercut competing economies, that create conditions for succession of outbreaks.'), ('dates', '2020-03-19 11:49:53'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.9060555100440979)])
OrderedDict([('tweet', 'This is some of the best evidence the Chinese government (CCP) lied about the extent of the coronavirus disease and delayed an effective response — and why Donald Trump restricted travel from China so early. The regime, spewing blatantly false propaganda, is not to be trusted. https://t.co/Tn599W506d'), ('dates', '2020-03-19 03:51:29'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.5089579820632935)])
OrderedDict([('tweet', 'Chinese Communist Party buried information, punished doctors and reporters who revealed it, and allowed Wuhan Virus to spread across the world, only the latest virus to originate in China where lax labor, health and environmental standards help it to undercut responsible nations https://t.co/Uhb6aowvD3'), ('dates', '2020-03-17 20:46:59'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.7334691286087036)])
OrderedDict([('tweet', 'Open-borders types don’t want us to talk about origins of coronavirus in China (just like its oppressive government.) But Chinese Communist Party created the conditions, suppressed info, disappeared doctors and reporters warning the world, and are still spreading misinformation https://t.co/xnAcy0z8eW'), ('dates', '2020-03-17 14:59:35'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.972124457359314)])
OrderedDict([('tweet', 'The latest news from the Get Trump media is that he disbanded pandemic response unit at NSC — completely false. It’s stronger and more focused after a streamlining. Regarding China, the President has been spot on. Keep following common-sense guidelines from health officials. https://t.co/uX7Mjt1XIF'), ('dates', '2020-03-16 21:22:07'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.5460604429244995)])
OrderedDict([('tweet', 'President Trump stopped flights from China and Iran very early. Slowness of some countries to act forced us to stop flights from Europe as well. Health experts commend @realDonaldTrump’s prompt response, but Far Left still wedded to open borders in midst of coronavirus. https://t.co/SYM7p5M2XF'), ('dates', '2020-03-16 13:15:57'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.8574318885803223)])
OrderedDict([('tweet', 'As our country deals with yet another viral outbreak from China, let’s also keep in mind fentanyl from Chinese chemical producers kills tens of thousands of Americans yearly. The Chinese Communist Party is a serious threat to our health and safety. Let’s start acting like it!'), ('dates', '2020-03-13 17:02:47'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.9011985063552856)])
OrderedDict([('tweet', 'Who won election promising to protect this country from China? Donald Trump. Who has done more than any recent president to secure US borders against threats including coronavirus? Donald Trump. While Resistance was chasing impeachment,he was leading the nationwide response!'), ('dates', '2020-03-13 16:57:25'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.9246649742126465)])
OrderedDict([('tweet', 'China is the source of many viral outbreaks, because the communist regime disregards basic health and safety in pursuit of economic dominance, and the country has also monopolized production of essential medicines. This is a bad one-two punch, but the world is finally waking up.'), ('dates', '2020-03-12 00:08:29'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.8018490672111511)])
OrderedDict([('tweet', 'Coronavirus only the latest global outbreak to originate in Communist China. Ignoring basic health and safety, and mistreating its own people, are just a few ways the chronic trade cheat undercuts more responsible countries in the global economy.'), ('dates', '2020-03-11 23:19:35'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.8920187950134277)])
OrderedDict([('tweet', 'President Trump campaigned and won promising to address problems Communist China poses. He won on a promise to secure our borders against deadly fentanyl from China and other health hazards. He has shown foresight and acted quickly to stop the coronavirus originating in China.'), ('dates', '2020-03-11 19:37:08'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.829288899898529)])
OrderedDict([('tweet', "Timing of Democrats' open-borders bill today could not be worse. Donald Trump's quick action to restrict or ban travel, while they were focused on another failed impeachment, undoubtedly prevented coronavirus spread in this country. Secure borders are healthy borders!"), ('dates', '2020-03-10 19:30:49'), ('file_name', 'Scott DesJarlais.csv'), ('pct_true', 0.7669442892074585)])
OrderedDict([('tweet', "There's something fundamentally wrong with #PPPloans going to publicly traded firms when so many small businesses are struggling to simply apply. Glad to see that action is being taken to correct this error. \nhttps://t.co/6bS0qxEhnN"), ('dates', '2020-04-24 18:51:00'), ('file_name', 'Scott H. Peters.csv'), ('pct_true', 0.5560570955276489)])
OrderedDict([('tweet', 'In 2010, the #DeepwaterHorizon blowout caused the worst oil spill in history. 10 years later, the Trump admin. seeks to weaken safety measures. We passed 3 anti-drilling bills in 2019 and will continue to work to protect the planet &amp; coastal communities. https://t.co/YXNJadhVNo'), ('dates', '2020-04-20 18:21:00'), ('file_name', 'Scott H. Peters.csv'), ('pct_true', 0.6117221117019653)])
OrderedDict([('tweet', "Our armed forces shouldn't have to worry about debt collection for them or their families while deployed. Today, the House passed the Fair Debt Practices for Servicemembers Act to protect those in uniform from facing these kinds of coercive threats."), ('dates', '2020-03-03 01:38:00'), ('file_name', 'Scott H. Peters.csv'), ('pct_true', 0.5719996094703674)])
OrderedDict([('tweet', 'The CCP’s lies, cover-ups, and overall abhorrent response to the COVID-19 pandemic is nothing new for this oppressive, deceiving regime. America must step up and hold the CCP accountable. Thanks, @MariaBartiromo, for the discussion this morning. https://t.co/1C6fpyXBPg'), ('dates', '2020-05-05 19:57:21'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.6571721434593201)])
OrderedDict([('tweet', 'The Chinese Communist Party (CCP) lied, covered up and attempted to blame America for COVID-19 in an attempt to preserve its own image. The CCP must be held accountable for its negligence and deadly behavior. https://t.co/uTCqs8YEsv'), ('dates', '2020-04-28 18:21:55'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.9790562987327576)])
OrderedDict([('tweet', 'The Communist Party of China and their co-conspirator, the World Health Organization, are responsible for the spread of COVID-19. The CPC took aggressive steps to hide COVID-19 – censoring media, detaining doctors, and destroying testing samples. They must be held responsible!'), ('dates', '2020-04-20 21:49:10'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.903214156627655)])
OrderedDict([('tweet', 'As the sun sets today, Yom HaShoah begins. On this #HolocaustRemembranceDay, we honor the 6 million Jewish lives taken during the Holocaust. We’ll never forget the horrors that fell upon these souls and reaffirm our commitment to eliminate anti-Semitism in all forms. #NeverAgain https://t.co/60rPswLqAZ'), ('dates', '2020-04-20 20:06:28'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.5099911093711853)])
OrderedDict([('tweet', 'Washington Democrats already blocked additional funding once, and I implore them to put their partisan games aside and join Republicans to pass a clean funding bill to reopen PPP. Our small businesses can’t afford to wait. https://t.co/7fzolzd1TC'), ('dates', '2020-04-16 20:59:48'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.6976159811019897)])
OrderedDict([('tweet', 'Small business owners and their employees need relief now. The #PaycheckProtectionProgram funding has run out and Congress must act quickly to provide additional aid.\n\nDem leadership must stop holding job creators and workers hostage for their partisan pet projects. ACT NOW!'), ('dates', '2020-04-16 16:12:59'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.5640708804130554)])
OrderedDict([('tweet', "WHO’s botched COVID-19 investigation did a huge disservice to the American people and the int'l community.  To protect the Chinese Communist govt, WHO put the health of 7.8 bil people at risk. @realdonaldtrump is spot on — the U.S. should indefinitely withhold all WHO funding."), ('dates', '2020-04-15 18:58:35'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.98479825258255)])
OrderedDict([('tweet', 'Our Country is facing a crisis.  This is no time for politics and games.  \n\nDemocrat Leadership is risking American jobs and lives in an attempt to score political points and fulfill their liberal wish list.  STOP IT – and work on helping the American People. https://t.co/I4jsQ1wIUP'), ('dates', '2020-03-23 18:05:32'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.8321624994277954)])
OrderedDict([('tweet', 'Congress must pass a clean funding packing to combat coronavirus. \n\n- No FISA reauthorization\n- No partisan elements \n- No playing politics with public health \n\nAddressing the spread of coronavirus is not and should not become a partisan issue.'), ('dates', '2020-03-04 16:55:35'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.7213608622550964)])
OrderedDict([('tweet', "We must hold universities accountable and bring transparency to the benefits they receive from foreign governments and persons. I've introduced H.R. 3921 to do just that. \n\nFailing to do so puts our students at risk of foreign influence by China, Saudi Arabia, Qatar, and others. https://t.co/ng7sYLirFW"), ('dates', '2020-02-13 14:01:05'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.5551201105117798)])
OrderedDict([('tweet', 'The recent Coronavirus outbreak is shining a light on the oppressive, intrusive and deadly policies of the Communist Government of China. Dr. Li was arrested and silenced for trying to warn his country of this deadly outbreak. Dr. Li, I wish you Godspeed. https://t.co/uqm7J0qfSB'), ('dates', '2020-02-07 18:40:19'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.8102414011955261)])
OrderedDict([('tweet', 'BREAKING: President Trump has been acquitted on BOTH articles of impeachment. \n\nThe President will be acquitted forever.'), ('dates', '2020-02-05 21:35:00'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.8474123477935791)])
OrderedDict([('tweet', 'After months of wasted time and taxpayer money, the Senate has ended this sham impeachment. \n\nDespite having zero evidence, the House Democrats decided to drag Congress and the American People through a monthslong, partisan hit job.\n\nTime for Congress to get back to work.'), ('dates', '2020-02-05 21:32:59'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.8661254644393921)])
OrderedDict([('tweet', "The Chinese gov't continues feckless &amp; reckless international policies eerily similar to SARS failures, which is why I'm actively pressing the #CDC to proactively protect the American People with accuracy &amp; facts. #coronavirususa #coronoravirus   #KnowledgeIsPower #TruthSaves https://t.co/zHMYvWibT7"), ('dates', '2020-01-31 21:25:43'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.5818652510643005)])
OrderedDict([('tweet', 'Adam Schiff and House Democrats failed to make a case for impeachment; it’s not the Senate’s job now.  Having the Senate call additional witnesses is like asking the judge and jury in a murder trial to go and collect evidence on the accused.'), ('dates', '2020-01-28 22:26:38'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.9396324157714844)])
OrderedDict([('tweet', 'Mocking Americans is a new low for CNN. In 2014, when Russian tanks were invading Ukraine, Democrats didn’t care to look at Ukraine on a map or read the news to understand the true threat the Ukrainian people faced. Where was CNN’s outcry then?  https://t.co/siMnBGmHUK'), ('dates', '2020-01-28 21:27:42'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.8051321506500244)])
OrderedDict([('tweet', 'This is the difference between partisanship and moving the Country forward. Washington liberals continue to push their baseless impeachment articles, while @realDonaldTrump has negotiated the USMCA, the Phase One deal with China, and now a pathway for peace in the Middle East. https://t.co/gkG9UR1LMW'), ('dates', '2020-01-28 20:28:08'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.8738117218017578)])
OrderedDict([('tweet', 'Under previous regulation, 99% of Pennsylvania was defined as a “Waters of the United States” – subjecting our state to significant federal control. Under the new rule, states and localities have the flexibility to manage their land and water. This is a win for Pennsylvania! https://t.co/dIQT8Vxrrm'), ('dates', '2020-01-23 21:16:39'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.6198582649230957)])
OrderedDict([('tweet', "For years, China's communist government has blocked Taiwan’s participation in international organizations.\n\nThe recent outbreak of a deadly coronavirus in China and its spread to Taiwan is a clear reason why the US must continue to support Taiwan’s full participation in the WHO. https://t.co/kjfRR5Fsdm"), ('dates', '2020-01-22 17:54:00'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.9799525141716003)])
OrderedDict([('tweet', '“We all know that in their hearts it is what they desired all along.”  \n\nThe House Dems couldn’t hide their excitement for impeaching @realDonaldTrump when they cheered on the House floor. The American people don’t buy it. It’s a purely political sham. https://t.co/CC1SBLZHb9'), ('dates', '2020-01-15 18:47:43'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.9076203107833862)])
OrderedDict([('tweet', '#MondayMotivation: It’s been 26 days since the House voted on the “urgent” impeachment articles. @SpeakerPelosi has yet to send the articles over to the Senate. After dividing her caucus and the country, maybe she will fulfill her constitutional duty this week.'), ('dates', '2020-01-13 18:01:19'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.7552827000617981)])
OrderedDict([('tweet', 'The Democrats on the Judiciary Committee just said that foreign-policy must be left to the “professionals.” Heaven forbid the President thinks he can be involved in foreign-policy! Do you think the “professionals” will be accountable in any election?'), ('dates', '2019-12-12 21:13:57'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.9272758960723877)])
OrderedDict([('tweet', 'Democrats apparently don’t understand and refuse to accept the legitimate separation of powers. They weren’t willing to go to court but they are sure willing to abuse their own power and take the law into their own hands.'), ('dates', '2019-12-12 20:53:55'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.9510028958320618)])
OrderedDict([('tweet', 'Democrats on the Judiciary Committee are acting as though oversight means the executive branch must bow to the legislative branch. They didn’t feel that way when a President said he would use his pen and a phone.'), ('dates', '2019-12-12 20:53:30'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.9958922863006592)])
OrderedDict([('tweet', 'Democrats on the Judiciary Committee are telling you that if a President doesn’t help them railroad him, then he must be guilty of anything for which he’s accused. Aren’t you happy the judicial system doesn’t treat you that way?'), ('dates', '2019-12-12 20:52:25'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.9755973815917969)])
OrderedDict([('tweet', 'Democrats on the Judiciary Committee just admitted they can determine what evidence they choose to use to impeach the President. Agreed. Republicans just think impeachment ought to be based on High Crimes and Misdemeanors, not because Dems don’t like the President.'), ('dates', '2019-12-12 20:51:38'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.9663445353507996)])
OrderedDict([('tweet', 'It is time for U.S. Attorney Durham to investigate if Adam Schiff was actively covering up the malfeasance outlined in the IG report of Crossfire Hurricane - explain the memo. @HouseIntel @HouseIntelComm @TheJusticeDept'), ('dates', '2019-12-12 20:49:50'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.591383695602417)])
OrderedDict([('tweet', 'While the radical left is trying to remove him from Office, President Trump continues to deliver profound results for the American People.'), ('dates', '2019-12-10 22:43:56'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.8908836841583252)])
OrderedDict([('tweet', 'Before his testimony today, Jonathan Turley - Democrat - noted that Schiff’s public hearings failed to provide a complete picture and “conflicted on critical points.” Do you wonder why Democrats didn’t call HIM as their own witness?'), ('dates', '2019-12-04 16:13:26'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.8272895216941833)])
OrderedDict([('tweet', 'Mr. Feldman who just testified in favor of impeachment, called for impeachment of the President less than two months after he was elected. What was the impeachable offense, a tweet? Do you think Feldman is impartial?'), ('dates', '2019-12-04 15:56:08'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.5419490933418274)])
OrderedDict([('tweet', 'Dr. Hill told Rep. Himes that Ukrainian Ambassador Chaly’s comments about candidate Trump were inadvisable.'), ('dates', '2019-11-21 19:15:50'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.5956169366836548)])
OrderedDict([('tweet', 'Dr. Hill testified that they (she and Tim Morrison) were concerned about how LTC Vindman would handle highly charged and partisan issues.'), ('dates', '2019-11-21 19:06:50'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.7221038937568665)])
OrderedDict([('tweet', 'Dr. Hill testified that she was upset that Ambassador Sonland wasn’t coordinating with her and that he wasn’t dealing with the interagency process.'), ('dates', '2019-11-21 19:06:21'), ('file_name', 'Scott Perry.csv'), ('pct_true', 0.5352429151535034)])
OrderedDict([('tweet', 'For Speaker Pelosi, COVID-19 mitigation and recovery efforts are no longer about stabilizing the economy, helping small businesses, providing healthcare workers with essential equipment, or assisting out-of-work Americans make ends meet. https://t.co/tcFn4AdG8O'), ('dates', '2020-05-08 20:56:51'), ('file_name', 'Scott R. Tipton.csv'), ('pct_true', 0.5150407552719116)])
OrderedDict([('tweet', 'The COVID-19 pandemic is again reinforcing the importance of responsible, U.S. mineral development. During trade negotiations in June 2019, China threatened to cut-off our access to rare earth minerals. Now, the COVID-19 shines a bright light on China’s dominance...'), ('dates', '2020-05-07 17:12:49'), ('file_name', 'Scott R. Tipton.csv'), ('pct_true', 0.8655809164047241)])
OrderedDict([('tweet', '28,000 Colorado businesses have been approved for #PaycheckProtectionProgram loans. Now thousands of other businesses sit in limbo as Democrats continue to use a national crisis to pursue political goals instead of focusing on immediate needs of families. https://t.co/AhDVdFDYUB https://t.co/Yg8iUZJB5h'), ('dates', '2020-04-17 15:12:40'), ('file_name', 'Scott R. Tipton.csv'), ('pct_true', 0.6129997372627258)])
OrderedDict([('tweet', "It's shameful to put politics ahead of struggling families and businesses. That's precisely what Speaker Pelosi and Sen. Schumer are doing. We need to act immediately, there is no time to waste. #PaycheckProtectionProgram \nhttps://t.co/BQlcTr4dpw"), ('dates', '2020-04-16 17:24:57'), ('file_name', 'Scott R. Tipton.csv'), ('pct_true', 0.8665730953216553)])
OrderedDict([('tweet', "There is no 'contest' between our governments. We are working to save lives of our citizens, they are working to save their regime from the world.\n\nhttps://t.co/OqRG5w19xk"), ('dates', '2020-04-01 20:24:57'), ('file_name', 'Scott R. Tipton.csv'), ('pct_true', 0.5710858702659607)])
OrderedDict([('tweet', 'We will hold the Communist Regime of China accountable! https://t.co/vLPFTzmmFB'), ('dates', '2020-04-01 18:23:20'), ('file_name', 'Scott R. Tipton.csv'), ('pct_true', 0.7705220580101013)])
OrderedDict([('tweet', 'The coronavirus does not care what you look like or where you are from. The demonizing of Asian American communities is incorrect &amp; morally reprehensible. On Tuesday, join me to shed a light on the experiences of Asian-Americans in Illinois, and how we can stop these attacks. https://t.co/xPyoHXtIC5'), ('dates', '2020-05-02 14:04:00'), ('file_name', 'Sean Casten.csv'), ('pct_true', 0.6148756742477417)])
OrderedDict([('tweet', 'Another Republican attempt to sabotage the ACA. While the #SCOTUS ruling is good news, their efforts won’t stop. We must continue to push back against these efforts to strip millions of healthcare coverage. https://t.co/byFSUDzlHI'), ('dates', '2020-04-27 22:13:43'), ('file_name', 'Sean Casten.csv'), ('pct_true', 0.786745011806488)])
OrderedDict([('tweet', 'Disinformation spreads like wildfire on social media platforms like @Facebook. These tactics are dangerous and heightens panic in a time when we should have a united message on how to deal with #COVID19. Read the full story from @NYT https://t.co/HJMDI2m5Kl'), ('dates', '2020-04-23 20:23:39'), ('file_name', 'Sean Casten.csv'), ('pct_true', 0.7252381443977356)])
OrderedDict([('tweet', 'In the midst of a global pandemic, &amp; despite his own Admin. concluding that it could cost the US economy $22 billion, Trump is still acting to increase carbon emissions, exacerbate the #climatecrisis, &amp; make Americans pay more at the pump. Unacceptable. https://t.co/c5Fpu4Qbdg'), ('dates', '2020-03-31 16:14:39'), ('file_name', 'Sean Casten.csv'), ('pct_true', 0.8201592564582825)])
OrderedDict([('tweet', "AG Barr is corrosive to our rule of law. Dropping the case against Flynn is a decision made purely on the basis of politics - not justice.\n\nBut his weakness shouldn't reflect on the majority of DOJ employees. Their faithful service will save us from the damage Barr inflicts."), ('dates', '2020-05-09 17:42:23'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.7872527837753296)])
OrderedDict([('tweet', 'Trump just told America he wants to "terminate healthcare under Obamacare”\n\n... during an unparalleled economic &amp; healthcare crisis, when over 9.2 mil workers have lost employer-sponsored insurance &amp; up to 35 mil people could lose coverage by the end of the crisis\n\nThat\'s hateful https://t.co/bHSaGNTuv1'), ('dates', '2020-05-07 01:41:55'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.644805371761322)])
OrderedDict([('tweet', "Anyone who has kids at home right now knows that America's teachers are heroes.\n\n#TeacherAppreciationWeek should be a reminder to us all that Congress needs to put its money where its mouth is – that starts with paying educators a living wage."), ('dates', '2020-05-05 23:05:11'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.6949169635772705)])
OrderedDict([('tweet', 'Small businesses are being left in the dark - waiting weeks for updates on their applications.\n\n@SBAgov needs to step it up, cut the government red tape, and get these folks what they need to keep their employees on the payroll and pay their bills.\nhttps://t.co/PrPj9VuQJG'), ('dates', '2020-04-30 18:45:15'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.6652128100395203)])
OrderedDict([('tweet', ".@MayoClinic has a policy requiring all visitors to wear a mask. Pence ignored it. That's not the sound judgment we want to see from the head of the coronavirus task force. https://t.co/pPxrioqvC6"), ('dates', '2020-04-28 19:44:47'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.6364059448242188)])
OrderedDict([('tweet', 'People are dying and political party has got nothing to do with it. Every state on the frontlines of this fight needs federal support. Anyone who thinks otherwise is plain wrong. https://t.co/KAiCJTA4pX'), ('dates', '2020-04-23 17:54:51'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.7953078150749207)])
OrderedDict([('tweet', ".@senatemajldr will spend billions to bail out corporations but won't lift a finger for states on the frontlines of this fight.\n\nIf states don’t get support, they're going to have to lay off workers, cut benefits &amp; drop projects to stay afloat.\n\nThat's not what our economy needs. https://t.co/b2pxtSqlMY"), ('dates', '2020-04-22 20:09:30'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.5612274408340454)])
OrderedDict([('tweet', "The @EPA has used the cover of COVID-19 to undo key protections - all of which were designed to safeguard public health.\n\nThey've dismantled toxic air protections, clean car standards &amp; fine particle pollution protections.\n\nThis #EarthDay, it's time the @EPA starts doing its job."), ('dates', '2020-04-22 16:15:43'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.716185986995697)])
OrderedDict([('tweet', "There's not going to be any kind of “reopening” until we've got COVID-19 testing - and lots of it.\n\nIf the president really wants to get America back to work, he should use the DPA to streamline testing.\n\nThat's the fastest way we're going to reopen this country."), ('dates', '2020-04-16 23:01:00'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.74601811170578)])
OrderedDict([('tweet', "We can't restart our economy without testing.\n\nPrivate labs and medical facilities have helped us make some progress, but that's no substitute for leadership from the top.\n\nStates can't do this on their own.\n\nThe federal government needs to help."), ('dates', '2020-04-15 20:42:15'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.5054110884666443)])
OrderedDict([('tweet', "This is not the president's money to give. These are taxpayer dollars. Shame on him. https://t.co/i3THuGN2KM"), ('dates', '2020-04-15 17:53:22'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.5973129272460938)])
OrderedDict([('tweet', 'In the last week, the president has:\n\n– fired the IC IG who reported the Ukraine whistleblower complaint\n\n–\xa0smeared the HHS IG for reporting supply shortages at hospitals\n\n– &amp; removed the IG overseeing our $2T coronavirus package\n\nThis is a disturbing pattern of retaliation.'), ('dates', '2020-04-07 19:23:38'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.8021960854530334)])
OrderedDict([('tweet', "Our health care system is more strained than ever, but Trump &amp; his GOP allies refuse to back down from their lawsuit to take protections away from 135 million people w/ pre-existing conditions. It's irresponsible &amp; it's out of touch. #ProtectOurCare https://t.co/sK0S9dG3ZB"), ('dates', '2020-03-31 16:09:00'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.6513254642486572)])
OrderedDict([('tweet', 'Just voted to pass this $2 trillion relief package. No matter what stunts @RepThomasMassie tried to pull today, there was never a question that we were going to pass this bill.\n\nWe all know Americans are concerned, worried, and suffering. Not going to let folks down. https://t.co/7xZYD6smuO'), ('dates', '2020-03-27 21:51:25'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.5996478199958801)])
OrderedDict([('tweet', 'Congress has reauthorized the DPA over 50 times. We did that for a reason. We did that so the president could protect American lives during crises like the one we are living in right now. The president should be using the full force of this power - without hesitation. https://t.co/CDXmeNi68N'), ('dates', '2020-03-25 22:43:30'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.7689659595489502)])
OrderedDict([('tweet', 'It is indefensible that the Trump Admin &amp; 18 GOP state AGs are STILL suing to have the ACA struck down.\n\nFolks should never have to worry that the president will pull their insurance out from under them - especially not during a pandemic.\n\nNow, more than ever, we need the ACA. https://t.co/iKFwtKlGZN'), ('dates', '2020-03-23 15:43:48'), ('file_name', 'Sean Patrick Maloney.csv'), ('pct_true', 0.8355585336685181)])
OrderedDict([('tweet', 'Nobody should use this crisis to further a political agenda. Congress should pass H.R. 6425, the Protect our Post Offices Act, which will keep the post office solvent through 2022 and give us time to actually fix the underlying financial problems. \n\nhttps://t.co/tzjVB6UoIE'), ('dates', '2020-04-14 18:26:00'), ('file_name', 'Seth Moulton.csv'), ('pct_true', 0.7940037250518799)])
OrderedDict([('tweet', 'People are hurting from the economic disruption #coronavirus has caused. @MassLWD &amp; the state’s Dept. of Unemployment Assistance are hosting daily town halls to answer your questions about how to apply. https://t.co/9jGbeIVWXT'), ('dates', '2020-03-23 16:25:11'), ('file_name', 'Seth Moulton.csv'), ('pct_true', 0.5067033767700195)])
OrderedDict([('tweet', 'People are worried about #coronavirus. They deserve straight answers from their leaders. https://t.co/EqNrozjY3r'), ('dates', '2020-03-11 20:13:38'), ('file_name', 'Seth Moulton.csv'), ('pct_true', 0.5928757190704346)])
OrderedDict([('tweet', "NEW: I'm calling for the release of all the businesses that have received loans through the Paycheck Protection Program.\n\nIt's unacceptable that #KS03 small businesses can't access these loans while large corporations w/ deep pockets are having no problem. https://t.co/s2bbPJvDBV"), ('dates', '2020-04-30 19:49:31'), ('file_name', 'Sharice Davids.csv'), ('pct_true', 0.5366162657737732)])
OrderedDict([('tweet', "Our workers can't afford to miss another paycheck, and our small businesses can't afford to go without this relief. The Administration needs to provide clearer guidance, and Congress needs to provide more resources, so this program can work as intended.\nhttps://t.co/bx6cg2H3hs"), ('dates', '2020-04-10 16:39:37'), ('file_name', 'Sharice Davids.csv'), ('pct_true', 0.6707966923713684)])
OrderedDict([('tweet', 'This reversal is not only outrageous and irresponsible – it’s putting lives at risk. Rather than work to ensure the health and safety of Kansans, extremists in the state legislature have created chaos and confusion and endangered our public health.\nhttps://t.co/lHWM5wZ1ST'), ('dates', '2020-04-09 15:42:55'), ('file_name', 'Sharice Davids.csv'), ('pct_true', 0.6102281212806702)])
OrderedDict([('tweet', 'We put oversight measures in place to ensure this tax-payer funded aid is used efficiently and fairly to help our families, workers, and small businesses. But the President is already undermining those measures. We need stronger oversight &amp; enforcement. \nhttps://t.co/9iPY9HUI9b'), ('dates', '2020-04-08 15:54:00'), ('file_name', 'Sharice Davids.csv'), ('pct_true', 0.6104708313941956)])
OrderedDict([('tweet', 'Asian Pacific Americans are still being impacted by racially charged Coronavírus epithets and physical attacks. DOJ must break the silence and ramp up their investigation. We must join together in opposition to this outrageous behavior. #AsianAmerican #AsiaNewsNetwork'), ('dates', '2020-04-23 11:58:58'), ('file_name', 'Sheila Jackson Lee.csv'), ('pct_true', 0.9440622925758362)])
OrderedDict([('tweet', 'The @realdonaldtrump is once again using immigration as a distraction to his failed policies. This time it’s to coverup his failed COVID-19 response. There is already a defacto pause on immigration. This is abuse of power w/o Congress oversight &amp; smells like political showmanship'), ('dates', '2020-04-23 11:55:47'), ('file_name', 'Sheila Jackson Lee.csv'), ('pct_true', 0.8140468597412109)])
OrderedDict([('tweet', 'Any decision to halt funding to the @WHO  is destructive to the global effort of saving lives. This decision by the President to ignore global health experts, scientists, and medical experts poses a considerable threat to the lives and livelihoods of people in this nation and ...'), ('dates', '2020-04-15 23:27:40'), ('file_name', 'Sheila Jackson Lee.csv'), ('pct_true', 0.758215606212616)])
OrderedDict([('tweet', 'Today marks the sixth anniversary of the abduction of 276 #Chibokgirls. These young ladies were forced to abandon their religion and were sold into slavery in Nigeria by the extremist group, #BokoHaram. While 164 of these young ladies have been liberated, 112 still remain... https://t.co/2pDEjHSTLN'), ('dates', '2020-04-14 20:27:03'), ('file_name', 'Sheila Jackson Lee.csv'), ('pct_true', 0.5740677118301392)])
OrderedDict([('tweet', 'Shame on the Administration! Preventing Americans in dire need of healthcare from enrolling in #Obamacare. This decision to not reopen Obamacare exchanges during the #Coronanavius pandemic increases the likelihood of a dangerous and avoidable healthcare crisis...'), ('dates', '2020-04-04 01:29:00'), ('file_name', 'Sheila Jackson Lee.csv'), ('pct_true', 0.674981951713562)])
OrderedDict([('tweet', 'Americans are desperate and dying. We’re not allowing one distracted Member to stop this stupendous legislation,which is not our final bill, that will save lives,protect healthcare professionals,and provide a safety net to families,small businesses and our brothers and sisters'), ('dates', '2020-03-27 17:21:51'), ('file_name', 'Sheila Jackson Lee.csv'), ('pct_true', 0.7583817839622498)])
OrderedDict([('tweet', 'I am outraged that we are seeing an uptick in violence against Chinese Americans and also other Asian-Americans because of the horrible misrepresentation by the President calling the coronavirus a Chinese virus and by others “Kung Flu”...'), ('dates', '2020-03-26 02:10:59'), ('file_name', 'Sheila Jackson Lee.csv'), ('pct_true', 0.9965431094169617)])
OrderedDict([('tweet', 'CARES bill gave climate mischief-maker @MarathonPetroCo over $400 million; probably paid them back all their political spending in one fat giveaway. \n\nYou mad yet?  \nhttps://t.co/xpjr1cOLoa'), ('dates', '2020-05-07 20:05:37'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.6014598608016968)])
OrderedDict([('tweet', 'We should be focused on relief to frontline workers and families battling #COVID19. \n\nInstead, Mitch McConnell wants to confirm his anti-ACA protégé to rule against Americans’ health care in the midst of a global pandemic. #StopWalker'), ('dates', '2020-05-05 17:00:38'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.9052330255508423)])
OrderedDict([('tweet', 'Congratulations, @washingtonpost, on your climate coverage Pulitzer. The warnings are everywhere, but Republican ears are too stuffed with fossil fuel money to hear. Keep banging.  \nhttps://t.co/8V64y2lrwr'), ('dates', '2020-05-05 14:28:20'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.5007436275482178)])
OrderedDict([('tweet', "More right-wing nonsense that doesn't mention dark-money @FedSoc’s role selecting Justices, dark-money @judicialnetwork’s role confirming Justices, &amp; 80-case win record for big Republican donors.  \nhttps://t.co/fVBmNXUHKA"), ('dates', '2020-05-04 18:58:07'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.5651628971099854)])
OrderedDict([('tweet', 'These are the emissions of the airlines; American carriers refused to do anything on emissions in public bailout. https://t.co/gyZ9Cxd32e'), ('dates', '2020-05-02 20:59:33'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.504757285118103)])
OrderedDict([('tweet', 'Does this signal anything to anyone? Hello?? The same interests that fund Mitch McConnell &amp; his Party fund the dark money machine surrounding (&amp; stocking) the Supreme Court: an obedient Court will do unpopular things they can’t make elected Republicans do.\nhttps://t.co/ZJPFhvB7Ua'), ('dates', '2020-04-29 19:45:56'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.8608812093734741)])
OrderedDict([('tweet', 'We added more #PPP money to meet the needs of small businesses, particularly through small banks &amp; credit unions that got pushed aside by big banks. Now big banks have jammed the system with their automated applications, frustrating small banks again. @SBAgov needs to fix this.'), ('dates', '2020-04-28 19:25:14'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.7023192644119263)])
OrderedDict([('tweet', 'Coronavirus warnings were in the PDB?!\n \n"The repeated warnings were conveyed in... the President\'s Daily Brief... But the alarms appear to have failed to register with the president, who routinely skips reading the PDB...” \nhttps://t.co/C1fsbe0Mtn'), ('dates', '2020-04-28 16:06:51'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.6988261342048645)])
OrderedDict([('tweet', 'First, we need to conquer an enemy within: a corrupt, anti-science, anti-government ideology, ordinarily kept alive on the fringes by cranks, but now fomented by massive industries that resent scientific checks or public oversight that interferes with their money-making. https://t.co/XLhcpUETP3'), ('dates', '2020-04-25 14:11:23'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.8962985873222351)])
OrderedDict([('tweet', 'Republicans snuck a $195 billion #MillionairesGiveaway into a pandemic relief bill – even though it did nothing to fight COVID. @RepLloydDoggett &amp; I are introducing legislation to repeal this shameful tax break &amp; ensure that aid goes to Americans in need.\nhttps://t.co/CSyLdJqg0N'), ('dates', '2020-04-24 18:32:33'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.6345415115356445)])
OrderedDict([('tweet', 'Why is the AG threatening governors who are fighting to keep their citizens safe from COVID? Is DOJ seriously going to sue states for trying to protect citizens?   \nhttps://t.co/8IfZmLKzYD'), ('dates', '2020-04-23 14:06:31'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.7116206884384155)])
OrderedDict([('tweet', 'Let’s be clear here: it’s less that fossil fuel corporations are a “pillar of the economy” than that fossil fuel donors are the central pillar of Republican fundraising; hence the party’s creepy-crawling obedience to fossil fuel interests.   \nhttps://t.co/DDo8V8MoII'), ('dates', '2020-04-21 20:49:32'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.682819128036499)])
OrderedDict([('tweet', 'To all those faint hearts who panicked or were misled when Mitch McConnell tried to jam us on SBA funding, prepare for a MUCH better bill.'), ('dates', '2020-04-21 18:48:41'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.5782931447029114)])
OrderedDict([('tweet', 'Still no clear direction from the Trump administration on money for states, money for education, remaining money for hospitals, etc. \n\nSeems like a long damn time since we passed this bill — what’s the holdup?'), ('dates', '2020-04-20 23:13:26'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.7063426375389099)])
OrderedDict([('tweet', 'On top of the straight-up $195B giveaway to million-dollar-plus earners that Republicans snuck into CARES, there’s this problem that needs fixing. Money needs to flow to actual small businesses and their workers.\nhttps://t.co/zHyVMJ10co'), ('dates', '2020-04-20 19:17:59'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.6778936982154846)])
OrderedDict([('tweet', 'Hang tough, Chuck! Its not enough to put more money into SBA without knowing who’s been left out; still no clarity on hospitals money or states money; still no PPE adequacy.'), ('dates', '2020-04-15 20:54:51'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.7571045756340027)])
OrderedDict([('tweet', 'These are the loopy but reliably partisan judges that Republicans put on courts.  \nhttps://t.co/qc9orMX1el'), ('dates', '2020-04-15 19:48:39'), ('file_name', 'Sheldon Whitehouse.csv'), ('pct_true', 0.617438554763794)])
OrderedDict([('tweet', 'This situation could have been completely avoided. Small businesses in America are in desperate need. We should not be playing politics and denying small businesses the help they need. Congress MUST fund #PPP. https://t.co/qI15VIBSig'), ('dates', '2020-04-16 15:06:12'), ('file_name', 'Shelley Moore Capito.csv'), ('pct_true', 0.9382827281951904)])
OrderedDict([('tweet', 'This is just nonsensical. \n\nThe Trump Administration has effectively restricted our local communities from spending CARES Act relief aid on what they need most:\n\nto make up for lost revenue.\n\nLet our communities spend the money Congress already passed.\n\nhttps://t.co/2St5MYiZcE'), ('dates', '2020-05-07 15:22:31'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.5895506143569946)])
OrderedDict([('tweet', 'Time and again, President Trump has sided with Wall Street over workers.\n \nThis pandemic has put his priorities on full display.\n\nHe needs to develop a real national testing plan so people can go back to work without this staggering death toll.\nhttps://t.co/pw3puaBrfr'), ('dates', '2020-05-06 17:30:53'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.5432379841804504)])
OrderedDict([('tweet', 'Reports that Trump political appointees may have exerted improper influence and overruled career staff show how far this Admin will go to protect payday lenders over consumers.\n\n@CFPB must stop any new payday rules and restart the rulemaking process.\nhttps://t.co/evDVFaiv0a'), ('dates', '2020-05-01 22:46:11'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.5862539410591125)])
OrderedDict([('tweet', 'For weeks I called on President Trump to use the DPA to ramp up production of PPE and medical supplies.\n \nInstead, he’s taken the most anti-worker action possible:\n \nUsing the DPA to force meatpacking workers to reopen - without protections - in unsafe conditions.'), ('dates', '2020-05-01 15:49:23'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.9083128571510315)])
OrderedDict([('tweet', "We should be putting more money straight into people’s pockets with another round of direct payments. \n\n$1,200 was a good start, but it's not enough when families are still struggling to survive. \n\nThat’s why my original plan called for supporting families THROUGHOUT this crisis. https://t.co/v1J4ApYwGk"), ('dates', '2020-04-28 21:12:49'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.6552805304527283)])
OrderedDict([('tweet', 'Mitch McConnell has never hesitated to give huge handouts to corporations.\n \nBut funding our communities - our cops and firefighters and teachers? Too much to ask.\n \nWe will stand up to any Washington politician who wants to let Ohio cities go bankrupt.\n\nhttps://t.co/esxEAMOmY0'), ('dates', '2020-04-23 18:33:54'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.6888857483863831)])
OrderedDict([('tweet', "More of the same from President Trump - trying to divide Americans and incite fear to distract from the fact that he failed to prepare for this crisis. \n\nWe can't lock out a virus, especially when the President's inaction already allowed it to spread. \n\nhttps://t.co/3Z3QXXVlkA"), ('dates', '2020-04-21 14:57:44'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.7312381863594055)])
OrderedDict([('tweet', 'Straight from the Trump Administration’s playbook: deflect the blame for their awful pandemic preparedness and scapegoat journalists.\n \nDon’t forget – President Trump eliminated the entire White House pandemic team designed to deal with a crisis like this over 700 days ago. https://t.co/IwceB2efPj'), ('dates', '2020-04-14 14:43:41'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.7480912804603577)])
OrderedDict([('tweet', '52 years ago, our government acknowledged decades of injustice and inequality by passing the #FairHousingAct.\n \nCongress must do more to ensure the vision of the bill - ending discrimination in our housing system - becomes a reality. https://t.co/zT2FpAmLW2'), ('dates', '2020-04-11 13:43:32'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.6979628801345825)])
OrderedDict([('tweet', 'Yesterday, if not for a court’s injunction, the Trump Administration’s cruel rule to take food assistance away from hundreds of thousands of Americans would have gone into effect. \n\nUSDA should drop it’s appeal - too many families are hurting. \n\nhttps://t.co/xztsvbPREF'), ('dates', '2020-04-02 22:50:57'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.6974953413009644)])
OrderedDict([('tweet', 'Mitch McConnell and Senate Republicans had an opportunity to draft a bipartisan relief package that would help the people suffering the most - workers and their families. Instead, they’re doing what they did in 2008 and 2009: giving Wall Street a blank check with taxpayer money.'), ('dates', '2020-03-22 17:39:15'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.5978792905807495)])
OrderedDict([('tweet', "Despite what he's told the public, President Trump hasn’t invoked and used the full powers of the Defense Production Act. I joined @SenMarkey and 18 of our colleagues in calling on the president to fully use the extraordinary powers of the DPA NOW. https://t.co/So8UUOjPrO"), ('dates', '2020-03-20 21:59:12'), ('file_name', 'Sherrod Brown.csv'), ('pct_true', 0.5004554390907288)])
OrderedDict([('tweet', '#FamiliesFirst means protecting small businesses nationwide. That’s why Democrats fought for &amp; won an expansion of fast relief for small businesses – and made rent, mortgage and utility costs eligible for SBA loan forgiveness.'), ('dates', '2020-03-26 18:02:47'), ('file_name', 'Stacey E. Plaskett.csv'), ('pct_true', 0.8698250651359558)])
OrderedDict([('tweet', 'The Coronavirus is serious. The House will now swiftly advance a strong, strategic funding package that fully addresses the scale and seriousness of this public health crisis.'), ('dates', '2020-02-25 18:39:06'), ('file_name', 'Stacey E. Plaskett.csv'), ('pct_true', 0.5527405142784119)])
OrderedDict([('tweet', 'Yes this is America and we will take its soul back November 2020 - democracy https://t.co/LiNnTiYY26'), ('dates', '2020-02-07 23:24:18'), ('file_name', 'Stacey E. Plaskett.csv'), ('pct_true', 0.5828571319580078)])
OrderedDict([('tweet', 'I look forward to seeing the American people do what the Senate refused to.'), ('dates', '2020-02-05 21:50:44'), ('file_name', 'Stacey E. Plaskett.csv'), ('pct_true', 0.6279659271240234)])
OrderedDict([('tweet', 'While predictable it is still disappointing to see that the Senate majority put loyalty to their party before loyalty to the constitution and what is right.'), ('dates', '2020-01-31 22:53:40'), ('file_name', 'Stacey E. Plaskett.csv'), ('pct_true', 0.7189671993255615)])
OrderedDict([('tweet', "The President should be removed from office because he committed high crimes and misdemeanors. If in November the American people decide that they want him to lead our country once again, they'll re-elect him."), ('dates', '2020-01-29 19:42:26'), ('file_name', 'Stacey E. Plaskett.csv'), ('pct_true', 0.7333580255508423)])
OrderedDict([('tweet', "Aside from addressing my Republican colleagues' bad faith arguments, I spoke about America's crumbling infrastructure and how Democrats are fighting to make American infrastructure better, safer, and more resilient.  That's how Dems walk and chew gum at the same time. https://t.co/BUl0uOKcTm"), ('dates', '2020-01-09 19:33:30'), ('file_name', 'Stacey E. Plaskett.csv'), ('pct_true', 0.5272663831710815)])
OrderedDict([('tweet', 'If Republicans are truly concerned with things being done in a timely manner, they should call the Senate Majority Leader and ask him to take up the 400 bills that the House has sent to the Senate 275 of which are BIPARTISAN. https://t.co/K7Wko5j1uR'), ('dates', '2020-01-09 19:01:48'), ('file_name', 'Stacey E. Plaskett.csv'), ('pct_true', 0.69086754322052)])
OrderedDict([('tweet', 'Downplaying the seriousness of this pandemic, hiding the facts from the public, &amp; praising states for reopening before it is safe to do so will cost lives. In his rush to reopen the economy, Trump is putting public health &amp; our economic recovery at risk. https://t.co/zXoes9MlCv'), ('dates', '2020-05-08 19:22:57'), ('file_name', 'Steny H. Hoyer.csv'), ('pct_true', 0.8128663897514343)])
OrderedDict([('tweet', 'People of color are losing their jobs at higher rates during this crisis. We cannot overlook how the pandemic has exacerbated racial disparities. Dems will continue to focus on inequities as we respond to the crisis &amp; work to eliminate them moving forward. https://t.co/ufYOI8ASUt'), ('dates', '2020-05-07 15:17:13'), ('file_name', 'Steny H. Hoyer.csv'), ('pct_true', 0.8032287955284119)])
OrderedDict([('tweet', 'Today’s unemployment claims report shows 33.5 million Americans – or 1 in 5 working Americans – have sought unemployment insurance in the last seven weeks. These figures reflect the economic burden this pandemic has had on our economy and working families.'), ('dates', '2020-05-07 14:36:04'), ('file_name', 'Steny H. Hoyer.csv'), ('pct_true', 0.8322335481643677)])
OrderedDict([('tweet', "The President's repeated ousting of Inspectors General is a shameful attack on oversight and reflects his contempt for any accountability. These actions endanger our democracy and weaken our collective ability to defeat crises like the coronavirus. https://t.co/jypoUwy7nW"), ('dates', '2020-05-06 21:23:28'), ('file_name', 'Steny H. Hoyer.csv'), ('pct_true', 0.727778434753418)])
OrderedDict([('tweet', 'It is unacceptable that the WH has blocked Dr. Fauci from testifying before @AppropsDems while allowing him to testify before the Senate.\nit is clear the President is playing partisan games. https://t.co/ypV1LdBo7Z'), ('dates', '2020-05-03 01:01:22'), ('file_name', 'Steny H. Hoyer.csv'), ('pct_true', 0.7107556462287903)])
OrderedDict([('tweet', "Over 1 million #coronavirus infections.\nOver 60,000 deaths.\n30 million unemployed. \n\nIt is incomprehensible that the Trump Administration is calling that a success. The President's serious missteps have cost lives &amp; are setting back our economic recovery. https://t.co/bbfaw811wd"), ('dates', '2020-04-30 17:38:14'), ('file_name', 'Steny H. Hoyer.csv'), ('pct_true', 0.5551337003707886)])
OrderedDict([('tweet', 'What happens if state and local governments go bankrupt? Schools, law enforcement, and other essential services will go unfunded. We cannot allow that to happen during this crisis, @senatemajldr. House Dems will continue to fight for these critical funds. https://t.co/U8lHSIlT1D'), ('dates', '2020-04-24 20:30:48'), ('file_name', 'Steny H. Hoyer.csv'), ('pct_true', 0.606009304523468)])
OrderedDict([('tweet', 'The House has taken action today, but our work is far from complete. The scope of this crisis is far wider than the scope of the relief offered in today’s interim bill &amp; the House will need to take further action soon #ForThePeople. https://t.co/CzgYp1tJLi'), ('dates', '2020-04-23 23:36:16'), ('file_name', 'Steny H. Hoyer.csv'), ('pct_true', 0.528589129447937)])
OrderedDict([('tweet', 'It’s clear the state’s unemployment system has failed Floridians. Glad the Governor agrees with my call for an independent investigation. We must hold state leaders—including the Governor himself—accountable for their failure to help workers during this crisis. #FlaPol #FL07 https://t.co/JC61vkB3Dq'), ('dates', '2020-05-04 23:03:04'), ('file_name', 'Stephanie N. Murphy.csv'), ('pct_true', 0.6616603136062622)])
OrderedDict([('tweet', 'Right now America does not have the capability to effectively contact trace, which is the method used to monitor and notify people who have come in contact with people infected with COVID-19 to help ensure they take the necessary precautions and contain the spread.'), ('dates', '2020-05-04 14:20:50'), ('file_name', 'Stephen F. Lynch.csv'), ('pct_true', 0.7533681392669678)])
OrderedDict([('tweet', '(1/3) While the President announced on March 5th that the coronavirus tests were here, a month has passed &amp; some hospitals &amp; health centers still have zero tests &amp; almost no personal protective equipment (PPE). While we don’t need politics we do need to hold people accountable...'), ('dates', '2020-04-04 14:09:00'), ('file_name', 'Stephen F. Lynch.csv'), ('pct_true', 0.5620928406715393)])
OrderedDict([('tweet', 'Pleased to see the House passed H.R. 1140 last week, the Rights for Transportation Security Officers Act, which will help improve the security operation of the TSA by providing TSA officers and employees with the worker rights and protections they deserve.'), ('dates', '2020-03-09 16:13:44'), ('file_name', 'Stephen F. Lynch.csv'), ('pct_true', 0.515363335609436)])
OrderedDict([('tweet', 'This is a miscarriage of justice. #MichaelFlynn pled guilty. It’s clear that Trump is once again undermining our courts and interfering on behalf of his cohort. Bill Barr has enabled this behavior and allowed the rule of law to be replaced by the rule of Trump. #FlynnIsGuilty https://t.co/SJegKx4hrU'), ('dates', '2020-05-07 20:52:27'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.7962088584899902)])
OrderedDict([('tweet', 'Trump knows that under oath #DrAnthonyFauci and Dr.Birks will tell truth and,ipso facto, contradict #TrumpOwnsEveryDeath No respect for the #Constitution and its checks/ balance and #oversight #COVIDー19 #covid  https://t.co/g3ocjxetbt'), ('dates', '2020-05-05 02:03:48'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.8046752214431763)])
OrderedDict([('tweet', '26 million Americans are unemployed, one million have been infected and 60,000 have died. We are in the midst of a pandemic, but Mitch McConnell’s top priority is confirming judges. This is disgraceful. Get it together! #Coronavirus #CultureOfCorruption\n\nhttps://t.co/V6pacs9EAt'), ('dates', '2020-04-30 01:04:01'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.6283043026924133)])
OrderedDict([('tweet', 'No, @realDonalTrump, most people did not think this was going to “blow over.” You were warned about the #Coronavirus in more than a dozen classified briefings in January &amp; February. Despite these warnings, you continued to downplay the disease for weeks. #TrumpLies #COVID19 https://t.co/n1uzLuqQC3'), ('dates', '2020-04-28 22:02:34'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.7300801873207092)])
OrderedDict([('tweet', 'Trump sees emergency funds as a tool for rewarding political allies &amp; punishing rivals. #COVID19 doesn’t see party affiliation &amp; neither should the federal response. If Trump continues to play politics, he will only make the #CoronavirusPandemic worse.\nhttps://t.co/4in1dToEIs'), ('dates', '2020-04-27 23:53:49'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.6231842041015625)])
OrderedDict([('tweet', 'Trump has repeatedly claimed that, during an emergency, he has “the right to do a lot of things that people don’t even know about.” When challenged on this, he said the Administration would “write up papers” justifying his claim. No justification has been provided. #TrumpLies'), ('dates', '2020-04-24 22:08:02'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.7901773452758789)])
OrderedDict([('tweet', 'It’s Dangerous to the health and welfare of every America that we have a president who is not only not knowledgeable about science and foreign affairs but who has no relationship to honesty truth empathy or justice #TrumpDisinfectant #TrumpIsNotADoctor https://t.co/Hw3DHaedXj'), ('dates', '2020-04-24 18:03:48'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.6518459916114807)])
OrderedDict([('tweet', 'Shameful comment from @SenMajLdr McConnell. Instead of making sure American communities have the resources they need to fight #COVID19, #MitchAntoinette is attacking their calls for aid as “free money.” How does this help frontline workers who are risking their lives every day? https://t.co/kya6WAJltH'), ('dates', '2020-04-23 21:46:06'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.8297790288925171)])
OrderedDict([('tweet', 'This is a HUGE mistake. If @GovernorBillLee allows #Tennessee businesses to reopen next week, the #coronavirus will likely tear through our state &amp; overwhelm our hospitals. I encourage Tennesseans to #StayHome as much as possible, even after the order is lifted. #Memphis #COVID19 https://t.co/r5Je5dE352'), ('dates', '2020-04-21 18:44:03'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.5342399477958679)])
OrderedDict([('tweet', 'Shame on #TrumpVirusCoverup seeking scapegoat for his selfish politics. We need to keep #WorldHealthOrganization strong as the world needs to be one to fight common enemy #CoronavirusPandemic.If one can’t see our oneness now you are a stupid and heartless fool. #TrumpIsUseless'), ('dates', '2020-04-19 00:58:44'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.6524143218994141)])
OrderedDict([('tweet', 'It is egregious that Republicans fought for additional tax breaks for the wealthy amid a global #pandemic. I sent a letter to House leadership urging them to eliminate these provisions &amp;focus on relief for individuals, families &amp; #SmallBusinesses. #COVID19 https://t.co/LNdcwflmaV'), ('dates', '2020-04-17 15:24:16'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.6206777095794678)])
OrderedDict([('tweet', 'Trump is using federal funds to reward his political allies &amp; punish his rivals. He may be the 1st President in history to use emergency powers to hinder, rather than help, the response to a crisis. There’s no time for political games. This is a #pandemic! #CultureOfCorruption https://t.co/szw4VXMKpG'), ('dates', '2020-04-16 15:58:42'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.7046568989753723)])
OrderedDict([('tweet', 'Trump does not have “total” authority to “reopen the economy” &amp; he can’t order states to relax #SocialDistancing measures. He can’t ignore our #Constitution &amp; the basic checks and balances put in place by our founders. He’s President Trump, not #KingTrump. #CoronavirusPandemic https://t.co/yifh0LORpW'), ('dates', '2020-04-16 00:42:55'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.9580492973327637)])
OrderedDict([('tweet', '.#TrumpPressConference says no administration had such problems getting appointments approved. Amnesia! #MitchMcConnell didn’t give #BarackObama a vote on ANY  judge for 14 months. Remember #merrickgarland ? #TrumpPressBriefing #TrumpLies'), ('dates', '2020-04-15 22:44:52'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.7813026309013367)])
OrderedDict([('tweet', 'Minority-owned businesses traditionally have a harder time getting loans and may be neglected by big banks offering #PPPloans. @housedemocrats are fighting to get them #Smallbizrelief through community lenders. #Memphis #SmallBusiness\nhttps://t.co/YVa7CeAzZE'), ('dates', '2020-04-15 15:09:19'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.6345390677452087)])
OrderedDict([('tweet', 'Trump defends $25 million in Kennedy Center funding in coronavirus stimulus https://t.co/IlnV6XCWSK .This was 3 weeks ago . Today he said #Democrats put that in for THEIR politics. #TrumpPressConf #TrumpLies'), ('dates', '2020-04-14 23:23:29'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.9074917435646057)])
OrderedDict([('tweet', 'Because of #COVID19, more than 16 million Americans have submitted #unemployment claims in the past THREE WEEKS, but #Labor Secretary Eugene Scalia is using his authority to limit who qualifies for assistance. This is despicable. We are in the midst of a pandemic! https://t.co/bhtObb5iim'), ('dates', '2020-04-10 19:11:30'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.6763163208961487)])
OrderedDict([('tweet', 'Wall Street journal editorialized that Trump uses briefings for his benefit and not for giving truthful info to the public. #TrumpPressBriefing #TrumpLiesAmericansDie https://t.co/QtoUzPjpAg'), ('dates', '2020-04-09 23:18:46'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.6878889799118042)])
OrderedDict([('tweet', 'Trump war on investigator generals is a WAR on #Americans as I.G’s root out fraud and abuse and further transparency. He wants no oversight. Dictatorship don’t allow it.Our #Constitution provides for oversight!https://t.co/18TL3DhXEu'), ('dates', '2020-04-07 22:26:13'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.6603970527648926)])
OrderedDict([('tweet', 'In January, Trump learned that the #coronavirus could infect 100 million Americans and cause 1-2 million deaths. Instead of preparing, he continued downplaying the disease for weeks. He can blame the WHO, but this is his mess. He knew and he did nothing. #COVID19 https://t.co/s6Ajq8oXGu'), ('dates', '2020-04-07 19:01:27'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.7175740003585815)])
OrderedDict([('tweet', 'Senator Collins is sadly either over the hill in her blindness and weakness or never made it up the hill. She needs to retire in shame for #Kavanaugh &amp; #impeachment vote. She isn’t #prochoice pro woman or pro #truth https://t.co/T0azQ28BbO'), ('dates', '2020-04-05 16:39:23'), ('file_name', 'Steve Cohen.csv'), ('pct_true', 0.680669903755188)])
OrderedDict([('tweet', 'Extending this immigration ban will give American workers the advantage with open jobs.  This is about putting Montana and America workers first. https://t.co/gW8POoeT8e'), ('dates', '2020-05-08 14:35:25'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.6323085427284241)])
OrderedDict([('tweet', 'We must end our dependence on China for life-saving drugs &amp; PPE.  It’s time to bring these manufacturing jobs home to America. 🇺🇸 https://t.co/VLPKCsqkHy'), ('dates', '2020-05-07 21:40:48'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.564887285232544)])
OrderedDict([('tweet', 'We must end our reliance on China.  That’s why I’m pushing a bill to strengthen our American medical supply chain, and bring drug manufacturing home. https://t.co/zs34dGi8MH'), ('dates', '2020-05-06 13:33:22'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.8172517418861389)])
OrderedDict([('tweet', 'We must end our dependence on China for drug manufacturing – it’s a national health AND a national security risk.\n\nThat’s why I’m pushing to strengthen our medical supply chain here in America. https://t.co/OOGO2Mvor2'), ('dates', '2020-05-05 13:16:42'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.7651979327201843)])
OrderedDict([('tweet', "China’s cover up of COVID-19 made this pandemic worse. Our economy shut down &amp; American lives have been lost.\n\nDrug / PPE manufacturing jobs must be brought back to the U.S. We can't continue to depend on China."), ('dates', '2020-05-04 20:27:01'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.9795777201652527)])
OrderedDict([('tweet', 'New #s for MT under the @SBAgov #PPP - MT small businesses have received 13,456 loans for over $1.4 billion. \n\nThis is great, but the $$ has run out and @SenSchumer &amp; @SpeakerPelosi continue to play political games &amp; block new funding. We MUST replenish the funds NOW.'), ('dates', '2020-04-17 19:33:22'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.8426458239555359)])
OrderedDict([('tweet', '.@realDonaldTrump is right to hold the @WHO accountable for their response to the COVID-19 outbreak and its work with China.'), ('dates', '2020-04-15 17:06:37'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.9469977021217346)])
OrderedDict([('tweet', 'Relying on China for medicine or essential medical equipment is detrimental to our national security and endangers our public health.  We MUST reduce our dependence on China. https://t.co/m40tc6stpT'), ('dates', '2020-04-14 13:49:38'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.673906147480011)])
OrderedDict([('tweet', 'We need to end our dependency on China.  It’s time to bring drug and supply manufacturing and JOBS back to the US. 🇺🇸\nhttps://t.co/4duK8pYMed'), ('dates', '2020-04-13 19:01:42'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.5638033747673035)])
OrderedDict([('tweet', 'We need to work together to protect Montana workers and small businesses.  Small businesses are closing their doors &amp; workers are losing their jobs.  I’m disappointed that boosting the Paycheck Protection Program was blocked on the Senate floor.  We can’t wait any longer.'), ('dates', '2020-04-09 15:15:43'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.677695095539093)])
OrderedDict([('tweet', 'China could have saved lives but they hid the virus from the world – they must be held accountable! https://t.co/MW4iJVEFtb'), ('dates', '2020-04-07 14:04:13'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.9429869055747986)])
OrderedDict([('tweet', 'We deserve to know the truth about China’s coverup.  China must be held accountable. https://t.co/jsg73F1EQk'), ('dates', '2020-04-06 18:04:31'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.7414709329605103)])
OrderedDict([('tweet', 'Now isn’t the time for political games.  Congress needs to get its act together.  Montana workers, families, and small businesses need relief. https://t.co/jzGf2vx17T'), ('dates', '2020-03-24 15:38:57'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.6770161390304565)])
OrderedDict([('tweet', 'Millions of Americans, along w/ five of our colleagues, are quarantined because of COVID-19 right now. We are in a public health and economic crisis. We don’t have time for partisan politics. We NEED to come together and get this done for the good of our country.'), ('dates', '2020-03-23 18:15:20'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.526795506477356)])
OrderedDict([('tweet', 'Montana workers forced out of their job because of the #Coronavirus need help and relief NOW.'), ('dates', '2020-03-23 01:15:20'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.6375966668128967)])
OrderedDict([('tweet', "Our ranchers have been challenged for months, and the virus will only make that worse.  That's why I'm calling on the DOJ to investigate market manipulation in the cattle markets and beef industry.\nhttps://t.co/ZWLZhGqRhf"), ('dates', '2020-03-20 18:23:00'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.5269888639450073)])
OrderedDict([('tweet', 'No place for inappropriate and reckless threats from @SenSchumer towards 2 Supreme Court justices. He should denounce his political and dangerous comments, rather than dig deeper with a false spin.'), ('dates', '2020-03-05 03:17:24'), ('file_name', 'Steve Daines.csv'), ('pct_true', 0.8033226728439331)])
OrderedDict([('tweet', 'Flynn case moves @seanhannity to call on FBI Director Chris Wray to “Hold...Bad Apples Accountable”. And it’s not the FBI alone, but Obama Adm. DOJ, CIA, &amp; other Deep State operatives who perpetrated “the single biggest abuse of power in American history.” https://t.co/ft3nkzRz0m'), ('dates', '2020-05-05 08:16:12'), ('file_name', 'Steve King.csv'), ('pct_true', 0.7629976868629456)])
OrderedDict([('tweet', 'I was pleased to join @repmattgaetz, @senatormcsallyaz, &amp; my Iowa Senators, @ChuckGrassley and @Senjoniernst on a letter that seeks to block American tax dollars from being distributed to China’s Wuhan Institute of Virology. We know China lied, and we shouldn’t fund their labs.'), ('dates', '2020-04-22 22:35:28'), ('file_name', 'Steve King.csv'), ('pct_true', 0.5679805278778076)])
OrderedDict([('tweet', '. @RealDonaldTrump makes wise move to halt immigration to protect from COVID-19 &amp; allow for economic recovery. We have &gt;22 million illegals &amp; &gt;22 million unemployed. Somewhere in this equation there must be a solution. https://t.co/MOK03CF8lg'), ('dates', '2020-04-21 12:52:13'), ('file_name', 'Steve King.csv'), ('pct_true', 0.5750836730003357)])
OrderedDict([('tweet', '22,295 Small Business Loans have been approved for Iowa through the SBA’s #PaycheckProtectionProgram. Over $3.7 Billion PPP$’s have been loaned to Iowans to save small businesses and their employees! Funds will soon be gone, yet Democrats are blocking $250 billion.'), ('dates', '2020-04-15 20:00:49'), ('file_name', 'Steve King.csv'), ('pct_true', 0.5423310995101929)])
OrderedDict([('tweet', 'The @RealDonaldTrump is right &amp; knows @TuckerCarlson is right. Elitists &amp; globalists sold the U.S. out to China. Our economic leaders convinced America that we would build an information, finance and service economy and outsource manufacturing to China. https://t.co/OZDbuYp50N'), ('dates', '2020-02-19 04:34:11'), ('file_name', 'Steve King.csv'), ('pct_true', 0.8256676197052002)])
OrderedDict([('tweet', 'These geniuses have no concept of freedom of speech. @RandPaul has the right to freedom of speech, too.https://t.co/7kEdh4SaRP'), ('dates', '2020-01-31 05:38:34'), ('file_name', 'Steve King.csv'), ('pct_true', 0.5141992568969727)])
OrderedDict([('tweet', 'Pundits continually reference the question before the Senate as, “Should the President be impeached?” He HAS been impeached—by the House. The question before the Senate is, “Should he be removed from office?” #impeachment I say “NO!”'), ('dates', '2020-01-28 01:09:32'), ('file_name', 'Steve King.csv'), ('pct_true', 0.7924041748046875)])
OrderedDict([('tweet', 'We should have added “eagle egg” to the list with “unborn child”. We might have peeled off a few more Democrats. https://t.co/wPCyJKTsHX'), ('dates', '2020-01-10 19:34:32'), ('file_name', 'Steve King.csv'), ('pct_true', 0.5173951387405396)])
OrderedDict([('tweet', 'Pelosi’s strategy in refusing to message the Articles of Impeachment to the Senate is, in part, an effort to delay @realDonaldTrump trial until after his State of the Union Address. Pelosi cannot stand the thought of Trump’s Declaration of Exoneration in her face on national TV.'), ('dates', '2020-01-10 14:14:06'), ('file_name', 'Steve King.csv'), ('pct_true', 0.9766641855239868)])
OrderedDict([('tweet', 'House Democrats might want to send the Iranians pallets of cash like Appeaser-In-Chief Obama did, but I stand with letting @realdonaldtrump, our Commander-in-Chief, make the tough calls and take the swift &amp; certain actions necessary to protect our nation. https://t.co/J9QdZXGGhx'), ('dates', '2020-01-09 23:44:14'), ('file_name', 'Steve King.csv'), ('pct_true', 0.8326051235198975)])
OrderedDict([('tweet', 'We had a classified briefing today on #Iran. I am bound to confidentiality.  I support President Trump’s decision to take out Sulaimani. But, this is the first time I have hoped briefers would NOT answer questions b/c Dems would use the answers to empower &amp; encourage our enemies.'), ('dates', '2020-01-09 02:04:34'), ('file_name', 'Steve King.csv'), ('pct_true', 0.6854727268218994)])
OrderedDict([('tweet', 'Since the controversial “coup attempt” in 2016, human rights violations have been increasing in #Turkey, including this awful reality—nearly 2,000 babies and children under age 8 have had to live in prison and 864 remain behind bars with their mothers.  What would Ataturk say?'), ('dates', '2020-01-08 22:23:08'), ('file_name', 'Steve King.csv'), ('pct_true', 0.8398281335830688)])
OrderedDict([('tweet', 'It’s a new year @JDScholten &amp; you are still ducking &amp; dodging. Every Democrat but one voted FOR impeachment. You are FOR #impeachment by default. You WILL have to answer the question: “Do you support impeachment?”@SimonRadio1776'), ('dates', '2020-01-08 00:55:52'), ('file_name', 'Steve King.csv'), ('pct_true', 0.6249449253082275)])
OrderedDict([('tweet', 'It’s another new day @JDScholten &amp; you are still ducking &amp; dodging. Every Democrat but one voted FOR impeachment. You are FOR #impeachment by default. You WILL have to answer the question. @SCJBretH'), ('dates', '2019-12-24 01:01:42'), ('file_name', 'Steve King.csv'), ('pct_true', 0.5139947533607483)])
OrderedDict([('tweet', 'Democrats did impeach @realDonaldTrump because they hate him. Bigger reason: shield themselves from a reckoning b\\c they weaponized Obama Executive Branch; DOJ, FBI, CIA, State, &amp; FISA Courts, illegally surveilled Trump campaign, transition, &amp; presidency. https://t.co/CfkmU7hS0R'), ('dates', '2019-12-22 00:03:19'), ('file_name', 'Steve King.csv'), ('pct_true', 0.9716737270355225)])
OrderedDict([('tweet', 'Andy, If the Speaker can refuse to message passed articles of  impeachment to the Senate, thereby effectively vetoing the will of the House, it would also be the case for every bill, even discharge petitions. It’s a never been tested theory that cannot withstand strict scrutiny. https://t.co/rgkWbpYRrw'), ('dates', '2019-12-21 02:46:58'), ('file_name', 'Steve King.csv'), ('pct_true', 0.6237558722496033)])
OrderedDict([('tweet', 'I just voted against the “Sore Loser Democrats” and both of their sham Articles of Impeachment. In doing so, I stand firmly with the Constitution and with @realDonaldTrump. My full statement is linked. https://t.co/JITvieqUH9 #IA04'), ('dates', '2019-12-19 03:05:10'), ('file_name', 'Steve King.csv'), ('pct_true', 0.7175692319869995)])
OrderedDict([('tweet', '#Impeachment debate continues w/Democrats preaching on the rule of law. Stupid is as stupid does. They categorically reject rule of law. Open borders, pro-amnesty Hilary exempt from the law, etc. @realDonaldTrump has broken NO laws &amp; Democrats have charged no violation of law.'), ('dates', '2019-12-18 23:55:50'), ('file_name', 'Steve King.csv'), ('pct_true', 0.9944211840629578)])
OrderedDict([('tweet', 'Democrats weaponized FBI, DOJ, Obama POTUS, the Intellegence Community, etc. Their crimes are treasonous and broader in scope than any in history. So bad they had to resort to impeachment of @realDonaldTrump in an attempt to cover up their crimes.'), ('dates', '2019-12-18 19:03:00'), ('file_name', 'Steve King.csv'), ('pct_true', 0.9810739159584045)])
OrderedDict([('tweet', 'Pelosi is preaching about the “republic” as if they were anything but enemies of their newly discovered word. She says this is a “national civics lesson.” Every future president will be subject to impeachment purely for spiteful partisan political reasons.'), ('dates', '2019-12-18 17:17:17'), ('file_name', 'Steve King.csv'), ('pct_true', 0.8378815650939941)])
OrderedDict([('tweet', 'Snark is no substitute for a response. Iowans now know you would vote to impeach @RealDonaldTrump &amp; remove him from office but lack the courage of your convictions to admit it. #IA04 https://t.co/14GUoKCGIx'), ('dates', '2019-12-17 21:08:37'), ('file_name', 'Steve King.csv'), ('pct_true', 0.8453234434127808)])
OrderedDict([('tweet', 'The voters of Iowa #4 want to know; are you, @JDScholten, for or against impeaching our Constitutionally elected President? The 3 Iowa D’s have announced they will vote yes &amp; I stand with @RealDonaldTrump.  No more stalling.'), ('dates', '2019-12-17 20:00:15'), ('file_name', 'Steve King.csv'), ('pct_true', 0.5694206357002258)])
OrderedDict([('tweet', 'Joe Biden confessed to that which Democrats accuse @RealDonaldTrump. If Biden is elected president, how would Congress not be compelled to impeach him?'), ('dates', '2019-12-12 00:10:04'), ('file_name', 'Steve King.csv'), ('pct_true', 0.7262450456619263)])
OrderedDict([('tweet', 'Democrats will try to impeach @RealDonaldTrump for “abuse of power” and “obstruction of Congress”. Democrats ARE GUILTY of “Abuse of Congress”. If they succeed, every POTUS is impeachable for all time.'), ('dates', '2019-12-10 16:16:50'), ('file_name', 'Steve King.csv'), ('pct_true', 0.972545862197876)])
OrderedDict([('tweet', 'Atty Gen Barr’s insightful speech to the @FedSoc. “...after...Trump won...opponents inaugurated...“The Resistance,” and they rallied around an explicit strategy of using every tool and maneuver available to sabotage the functioning of his Administration.“ https://t.co/Ov2aXTQRFj'), ('dates', '2019-11-16 17:07:36'), ('file_name', 'Steve King.csv'), ('pct_true', 0.6735187768936157)])
OrderedDict([('tweet', 'Last night, @realdonaldtrump told @seanhannity that The New York Times is a "fake newspaper" he doesn\'t want in the White House. Trump\'s right, and everyone but @GOPleader McCarthy knows it. Restore my committees, Kevin, and stop reading the NYT. https://t.co/AxdmEX2F6K #MAGA'), ('dates', '2019-10-22 14:53:41'), ('file_name', 'Steve King.csv'), ('pct_true', 0.5976780652999878)])
OrderedDict([('tweet', "It’s now clear: Comey's FBI baselessly destroyed Gen. Flynn's life to try to take down @realDonaldTrump.\n\nThis was a set up from the start. It was never about Russia or collusion.\n\nWhere's the media outrage? If they can do it to the President and a general, they can do it to you. https://t.co/iTTp7XfX0N"), ('dates', '2020-05-09 18:35:30'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.7284381985664368)])
OrderedDict([('tweet', "We've seen this playbook before.\n\nAll Pelosi and House Democrats do is harass President Trump and try to rig 2020 against him.\n\nIf you thought they were going to let a global pandemic stop them, think again.\n\nThey're already using Coronavirus to harass @realDonaldTrump some more. https://t.co/TVV9eLQ997"), ('dates', '2020-05-08 23:00:47'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8773268461227417)])
OrderedDict([('tweet', 'Outrageous. → First action Coronavirus committee Dems took was sending harassing letters to companies that followed the PPP law to pay workers—a law they all voted for.\n\nDems spend more time shaming and investigating American workers and job creators than they do China. Telling.'), ('dates', '2020-05-08 20:33:55'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9538468718528748)])
OrderedDict([('tweet', '🚨 Here we go again. Dems are gearing up for another witch hunt of @realDonaldTrump with the Coronavirus Subcommittee.\n\nThis virus attacked our nation. We must use this committee to fight back—not play politics with more pointless partisan investigations. https://t.co/FJ3g1IG8Dk'), ('dates', '2020-05-07 22:22:21'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.7094782590866089)])
OrderedDict([('tweet', "Dems want to turn Coronavirus into Impeachment 2.0 with their Select Subcommittee on the Coronavirus Crisis.\n\nWe won't let them do this again. I'm honored to be named the lead Republican on it—and will fight to stop it from turning into yet another witch hunt of @realDonaldTrump."), ('dates', '2020-05-07 14:18:22'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9265961050987244)])
OrderedDict([('tweet', "We've seen enough.\n\nComey's crooked cops:\n• Spied on Trump's campaign\n• Plotted to entrap Gen. Flynn\n• Tried to take down @realDonaldTrump after the people elected him President\n\nNow everyone who abused their power to push their own political agendas must be held accountable. https://t.co/cXtbzWxwBo"), ('dates', '2020-05-05 22:18:44'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.5240186452865601)])
OrderedDict([('tweet', '⚠️ Trump Derangement Syndrome: Schiff calls @realDonaldTrump holding China accountable a "continuing disservice."\n \nNo surprise that the man who wasted time on a sham impeachment &amp; the Russian collusion hoax instead of focusing on China is using Communist China\'s talking points. https://t.co/0ZoRcvlEYt'), ('dates', '2020-05-05 18:33:58'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9905835390090942)])
OrderedDict([('tweet', "Congress should be working to:\n– Help workers &amp; families\n– Hold China accountable\n– Get our economy up &amp; running\n– Move our medical supply chain to America\n\nBut Pelosi's House Dems still won't show up to work—even as @realDonaldTrump &amp; the Senate are working in DC. Unacceptable."), ('dates', '2020-05-05 16:05:02'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9854174852371216)])
OrderedDict([('tweet', "Let's be clear about what's happening:\n\n→ America is facing the biggest crisis of our time.\n→ Essential workers are showing up to their jobs.\n→ @realDonaldTrump &amp; the Senate are in DC working.\n\nBut Pelosi’s House Dems are sitting at home.\n\nRT to tell them to GET BACK TO WORK."), ('dates', '2020-05-04 18:34:54'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8128758668899536)])
OrderedDict([('tweet', ".@DevinNunes &amp; @Jim_Jordan had it right from the start: Pro-Hillary agents in Comey's FBI abused their power to try to take down @realDonaldTrump.\n\nThis was never really about Russia or collusion—just attacking Trump.\n\nRT if you agree → These dirty cops must be held accountable. https://t.co/hpO0vyy130"), ('dates', '2020-05-01 22:37:45'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8713589906692505)])
OrderedDict([('tweet', "🚨 Deep State Exposed → Evidence proves anti-Trump agents in Comey's FBI plotted to take down @realDonaldTrump and @GenFlynn using Russian disinformation.\n\nThat's the very definition of a dirty cop.\n\nNow it's time for JUSTICE. We can't allow this to happen to anyone ever again. https://t.co/hqTZWOvF7e"), ('dates', '2020-05-01 14:30:03'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.6415652632713318)])
OrderedDict([('tweet', 'And there it is. Hillary Clinton admits Dems are using the Coronavirus crisis to try to advance their radical socialist agenda:\n\n"This would be a terrible crisis to waste.”\n\nWith all the destruction this virus has caused, all Dems can think of is how they capitalize on it. Sick. https://t.co/RSnCwK9ybQ'), ('dates', '2020-04-29 00:11:41'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8271211385726929)])
OrderedDict([('tweet', 'Congress has a growing list to work on—but Pelosi would rather draft bills in secret than get back to work &amp; lead America.\n\nEssential workers everywhere are doing their jobs. @realDonaldTrump is doing his. The Senate will be working in DC next week. Pelosi’s House should be too.'), ('dates', '2020-04-28 21:55:47'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8147681951522827)])
OrderedDict([('tweet', 'Communist China lied to the world about Coronavirus and refused to let our experts in early to help stop the spread. Their cover-up caused global suffering and destroyed the entire world economy.\n\nChina must be held accountable. https://t.co/T7tO3H8XNB'), ('dates', '2020-04-28 18:20:48'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9789246320724487)])
OrderedDict([('tweet', "Nearly 500 frustrated small biz owners wrote to me last week about Dems' job-killing delays.\n\nWATCH → @PressSec @KayleighMcEnany slams Pelosi &amp; Schumer for their obstruction that hurt workers &amp; small businesses—like one in Alabama that told me they were on the brink of layoffs. https://t.co/OxJxw8jQOD"), ('dates', '2020-04-27 16:55:19'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8483392000198364)])
OrderedDict([('tweet', 'Absolutely disgusting headline from POLITICO.\n\nAmericans are losing their lives, jobs, and savings—yet some in the media are more worried about finding the ways this could hurt @realDonaldTrump in the 2020 election. https://t.co/mSLVnW7jI4'), ('dates', '2020-04-23 20:25:22'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8225530982017517)])
OrderedDict([('tweet', '4.4 million Americans lost their jobs last week while Pelosi blocked funds for the Paycheck Protection Program.\n\nThis lifeline should have never been cut off. Today we will replenish it—but some small businesses may never recover from the completely avoidable damage done by Dems. https://t.co/sKU9KHS4Fg'), ('dates', '2020-04-23 19:23:35'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8405951857566833)])
OrderedDict([('tweet', "We already have 8 entities overseeing COVID-19.\n\nBut Pelosi wants another committee—led by one of Biden's biggest supporters.\n\nDon't let her tell you otherwise: This is a political scheme to spend more taxpayer money harassing @realDonaldTrump and trying to rig the 2020 election. https://t.co/xpbzuG7GaR"), ('dates', '2020-04-23 18:02:42'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8519495725631714)])
OrderedDict([('tweet', 'The Paycheck Protection Program is a lifeline for America\'s small businesses—it\'s already saved 30 MILLION jobs.\n\nIt should have never expired. But Dems blocked it to play politics and treat it as "leverage" for a week.\n\nGood News → The House will vote to replenish it tomorrow. https://t.co/DnLdNW3xwI'), ('dates', '2020-04-22 20:54:06'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.6968109011650085)])
OrderedDict([('tweet', 'Biden: "Look at the institutional changes we can make"\nWhip Clyburn: "Opportunity to restructure things to fit our vision"\nRep Jayapal: Worried about leverage\n\nRepublicans are working to save lives/jobs. Dems are using a pandemic to push their socialist agenda.\n\nQuite a contrast.'), ('dates', '2020-04-22 18:17:58'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.6756215691566467)])
OrderedDict([('tweet', '🚨 Biden reveals plan to use this pandemic to advance Dems\' radical agenda:\n\n"Because of this COVID crisis...Look at the institutional changes we can make.”\n\nPeople are losing their lives, jobs, and savings—but Dems are shamelessly more worried about their political priorities. https://t.co/8i0LpP3mzH'), ('dates', '2020-04-22 15:05:18'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9369258284568787)])
OrderedDict([('tweet', 'Nancy Pelosi needlessly harmed hardworking Americans across the country with her partisan games and delays—including this San Francisco restaurant and their employees IN HER OWN DISTRICT.\n\nThis didn’t have to happen. Small businesses will never forget what she did. https://t.co/q0GK5LEfny'), ('dates', '2020-04-22 00:01:55'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8336506485939026)])
OrderedDict([('tweet', "Dems knew the Paycheck Protection Program would run dry—yet chose to block more funding to play politics for over a week while American workers and small businesses suffered.\n\nIt's about time they agreed to replenish the program—it should've happened a lot sooner."), ('dates', '2020-04-21 21:09:44'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9546846747398376)])
OrderedDict([('tweet', 'WATCH → Democrat Rep. Jayapal says she\'s worried about "giving away leverage" by supporting more aid for small businesses to pay their employees.\n\nAmerican workers are losing their jobs. Small businesses are closing permanently.\n\nThis is about people\'s livelihoods, not leverage. https://t.co/Fky7BUTtBp'), ('dates', '2020-04-21 12:16:41'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.7802276015281677)])
OrderedDict([('tweet', 'Oil &amp; gas workers are losing their livelihoods.\n\nRep. Ocasio-Cortez\'s response was "you absolutely love to see it."\n\nThen she deleted it because she doesn\'t want people to see the truth: Dems are willing to sacrifice people\'s jobs &amp; livelihoods for their radical socialist agenda. https://t.co/0uMnlRVM6a'), ('dates', '2020-04-21 00:02:01'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.903526246547699)])
OrderedDict([('tweet', "→ 11 days since Dems blocked Paycheck Protection Program funding\n→ 4 days since funds ran completely dry\n→ 22 million Americans already unemployed\n\nAmerican workers and small businesses won't forget what Chuck Schumer and Nancy Pelosi did. https://t.co/jH8ZVCkxdI"), ('dates', '2020-04-20 16:47:22'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9868796467781067)])
OrderedDict([('tweet', 'Schumer &amp; Pelosi pulled the rug out from under small businesses when they blocked aid to cover payroll.\n\nNow those businesses are having to lay people off &amp; close their doors—possibly forever.\n\nThis was avoidable. Dems must agree to replenish the Paycheck Protection Program ASAP. https://t.co/haII4dvjZ3'), ('dates', '2020-04-19 22:26:15'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8609693050384521)])
OrderedDict([('tweet', 'Chuck Schumer &amp; Nancy Pelosi are:\n• Using a pandemic to play petty Washington games\n• Holding funding for paychecks for small business employees hostage to their unrelated partisan agenda\n• Making economic destruction worse\n\nRT if you agree → America deserves better.'), ('dates', '2020-04-19 20:12:19'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.7954516410827637)])
OrderedDict([('tweet', 'This Louisiana small business needs Paycheck Protection Program aid now. They have a message for Chuck Schumer &amp; Nancy Pelosi:\n\n"The Democrats need to stop playing games with people\'s lives from their million dollar homes with their financially secure futures. ENOUGH IS ENOUGH!" https://t.co/voxg84N6Se'), ('dates', '2020-04-19 18:30:49'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8499796986579895)])
OrderedDict([('tweet', 'Reminder ↓\n\nRepublicans tried to replenish the Paycheck Protection Program a week before funds ran out—no gimmicks, no games.\n\nBut Chuck Schumer and Nancy Pelosi said NO.\n\nNow—because of them—workers and small businesses could lose everything. Some may never recover.\n\nShameful.'), ('dates', '2020-04-19 16:07:41'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9922871589660645)])
OrderedDict([('tweet', 'Hundreds of small business owners have written to me about the unbearable decisions they now face without access to the Paycheck Protection Program aid Schumer and Pelosi are blocking.\n\nThis Georgia jeweler will have to lay off 4 people—leaving them unable to cover their rent ↓ https://t.co/gRRM4DDEaQ'), ('dates', '2020-04-19 00:28:31'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.5760483741760254)])
OrderedDict([('tweet', "RT to stand with the small businesses being hurt by Chuck Schumer and Nancy Pelosi's despicable partisan games and tell them to QUIT blocking critical Paycheck Protection Program funding NOW.\n\nEnough is enough. Millions of jobs are on the line."), ('dates', '2020-04-18 19:11:24'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9470199346542358)])
OrderedDict([('tweet', "Small businesses across America now face desperate circumstances—many won't survive—because Chuck Schumer and Nancy Pelosi purposefully let Paycheck Protection Program funds run dry while they played more of their petty partisan games.\n\nThis suffering was completely avoidable. https://t.co/jLiK2R25oy"), ('dates', '2020-04-18 17:58:01'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9638349413871765)])
OrderedDict([('tweet', "Don't let Dems &amp; the media spin this differently: Schumer &amp; Pelosi are the ONLY ones blocking aid for workers/small biz.\n\nEach day Dems block Paycheck Protection Program funds results in hundreds of possibly permanent business closures &amp; the destruction of countless livelihoods."), ('dates', '2020-04-18 14:31:22'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9564967751502991)])
OrderedDict([('tweet', 'Small businesses are begging Chuck Schumer &amp; Nancy Pelosi to quit holding Paycheck Protection Program funds hostage to their unrelated partisan demands.\n\nLivelihoods are at stake. What Dems are doing is a complete scandal.\n\nSee pleas from across America ↓ https://t.co/q2NEWhmlF7'), ('dates', '2020-04-17 22:58:56'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.8298982977867126)])
OrderedDict([('tweet', "🚨 America's essential workforce is being impacted by Chuck Schumer and Nancy Pelosi's dangerous political games.\n\nThis training institute in Oklahoma won't be able to keep paying their instructors without the Paycheck Protection Program relief Democrats are blocking ↓ https://t.co/y4R9LqRGIB"), ('dates', '2020-04-17 21:54:40'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.5794466137886047)])
OrderedDict([('tweet', '→ Small businesses are barely hanging on\n→ 22 million Americans already lost their jobs\n→ Millions more rely on small biz for their paychecks\n\nAnd yet Nancy Pelosi and Chuck Schumer are BLOCKING Paycheck Protection Program funding to play petty partisan games.\n\nUnforgivable. https://t.co/phnPd1ZLh7'), ('dates', '2020-04-17 15:59:15'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9377813339233398)])
OrderedDict([('tweet', "How many more Americans have to lose their jobs, businesses, and savings before Dems will quit holding funds for the Paycheck Protection Program hostage to their unrelated partisan demands?\n\nEnough is enough. Small businesses like Foy Distribution in Georgia can't afford to wait. https://t.co/tbdnODyZh5"), ('dates', '2020-04-17 14:40:35'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.89072185754776)])
OrderedDict([('tweet', "Real hardworking Americans are the ones paying the price of Democrats' completely partisan obstruction of Paycheck Protection Program funding.\n\nAmerican workers and businesses can't afford more of their petty Washington games.\n\nTake their word for it ↓ https://t.co/Nsb2mr5g8Q"), ('dates', '2020-04-17 01:15:05'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.6426862478256226)])
OrderedDict([('tweet', 'What Dems want to end:\n– 1.6 MILLION loans approved\n– $350 BILLION in aid guaranteed\n– Millions of American jobs saved\n\nThe Paycheck Protection Program was WORKING—now Dems are blocking funds.\n\nRT if you’ve had enough of them putting their partisan demands over workers in need.'), ('dates', '2020-04-16 22:07:18'), ('file_name', 'Steve Scalise.csv'), ('pct_true', 0.9269548058509827)])
OrderedDict([('tweet', 'Now that the Senate passed the #USMCA, Ohioans are one step closer to seeing the benefits. Partnerships with Canada and Mexico already support over 249,000 jobs in #Ohio. This will strengthen small businesses, #ag and #manufacturing policies, and innovate the digital marketplace. https://t.co/7AgJkQgKc8'), ('dates', '2020-01-17 18:54:06'), ('file_name', 'Steve Stivers.csv'), ('pct_true', 0.5632207989692688)])
OrderedDict([('tweet', 'The 𝐠𝐨𝐨𝐝: Kansas small businesses received 26,245 #PPP loans = $4.29 billion\n\nThe 𝐛𝐚𝐝: these funds have been empty since last Thursday\n\nThe 𝐮𝐠𝐥𝐲: Dems are refusing to replenish the funds and save American jobs\n\nWe must pass additional #PPP funding now!\n\n#KS02 #ksleg https://t.co/4ZRscsEa6S'), ('dates', '2020-04-20 23:29:21'), ('file_name', 'Steve Watkins.csv'), ('pct_true', 0.5508506298065186)])
OrderedDict([('tweet', '40 years ago the United States defeated the Communist Soviet Union in what was later known as the Miracle on Ice.\n\n40 years later and Americans still haven’t gotten tired of kicking communism.\n\n#KS02 #ksleg \n\n https://t.co/ynPKF7FY9O'), ('dates', '2020-02-23 01:22:56'), ('file_name', 'Steve Watkins.csv'), ('pct_true', 0.7823007702827454)])
OrderedDict([('tweet', "The four facts remain the same despite #BookTourBolton\n▪️ No quid pro quo\n▪️ No pressure\n▪️ No knowledge of aid withheld\n▪️ Aid was released\n\n@POTUS @realDonaldTrump is innocent and his defense team has completely dismantled the Dems' argument. \n\n#KS02 #ksleg https://t.co/QBI83Naujj"), ('dates', '2020-01-27 22:50:39'), ('file_name', 'Steve Watkins.csv'), ('pct_true', 0.8941813707351685)])
OrderedDict([('tweet', "Democrat's partisan resolution ties the hands of our Commander-In-Chief @realDonaldTrump and undermines his ability to defend Americans abroad. \n\nThis is unacceptable and why I'm voting no on the resolution. \n\n#KS02 #ksleg https://t.co/gTXx8XA7oo"), ('dates', '2020-01-09 21:44:57'), ('file_name', 'Steve Watkins.csv'), ('pct_true', 0.7200592160224915)])
OrderedDict([('tweet', "🚨Senate passed more funds for #PPP🚨\n\nThe House plans to take up the bill on Thursday. I'm in DC &amp; stand ready for legislative business.\n\nWhile I'm glad Congress will replenish this program, PPP funding should have NEVER been blocked by Dems or allowed to run dry to begin with."), ('dates', '2020-04-21 21:52:50'), ('file_name', 'Steve Womack.csv'), ('pct_true', 0.515457272529602)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram is a bipartisan program. Replenishment of funds shouldn\'t be made political, but Democrats are seemingly focused on "leverage." We are in the midst of a crisis and people\'s businesses and jobs are on the line. The PPP needs more funding now. https://t.co/Qust8yPZH7'), ('dates', '2020-04-21 15:05:56'), ('file_name', 'Steve Womack.csv'), ('pct_true', 0.8538098335266113)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram has exhausted its funding. Republicans have been sounding the alarm, while Democrats blocked additional funds. The consequences are real. Just look at the below quote from WSJ article. Congress must act NOW to support small businesses &amp; workers. https://t.co/tjDMICrRLR'), ('dates', '2020-04-16 02:23:04'), ('file_name', 'Steve Womack.csv'), ('pct_true', 0.7179419994354248)])
OrderedDict([('tweet', "The #PaycheckProtectionProgram has been a lifeline for #smallbusinesses in #AR3 and across the US. With #PPP running out, it's inexcusable that Chuck Schumer &amp; Speaker Pelosi continue to block additional aid. American workers &amp; businesses deserve action. Stop the political games. https://t.co/UtSq1Pu3uh"), ('dates', '2020-04-15 19:08:16'), ('file_name', 'Steve Womack.csv'), ('pct_true', 0.7223458290100098)])
OrderedDict([('tweet', 'Today, we say #NeverAgain as we mourn the lives lost to evil in the Holocaust 75 years ago. This #HolocaustRemembranceDay, we stand united with the Jewish community against hatred, bigotry and anti-Semitism. #YomHaShoah https://t.co/pJDPduZcUh'), ('dates', '2020-04-21 18:19:42'), ('file_name', 'Steven Horsford.csv'), ('pct_true', 0.6034842729568481)])
OrderedDict([('tweet', 'These stories are heartbreaking--the reality for many Americans during this #COVID19 pandemic. This is why I introduced the Workers Health Coverage Protection Act to provide 100 percent coverage of COBRA insurance plans for those who have lost their jobs.\n\nhttps://t.co/HzDwhvAlIm'), ('dates', '2020-04-20 14:57:10'), ('file_name', 'Steven Horsford.csv'), ('pct_true', 0.5232549905776978)])
OrderedDict([('tweet', "Yesterday, 3.5 million people likely lost their employer-sponsored healthcare. The president's decision not to allow special enrollment under the #ACA is inhumane. Many of these American families also won’t be eligible for Medicaid. The time to act is NOW.\nhttps://t.co/8rsqpu1UMr"), ('dates', '2020-04-02 17:37:22'), ('file_name', 'Steven Horsford.csv'), ('pct_true', 0.7840644717216492)])
OrderedDict([('tweet', "China lied to the world. They must be held accountable. \n\nI've joined several lawmakers calling on @SecPompeo &amp; AG Barr to bring a case against China to the International Court of Justice. \n\nMORE: https://t.co/UlxLRXP0Z0"), ('dates', '2020-04-21 21:02:05'), ('file_name', 'Steven M. Palazzo.csv'), ('pct_true', 0.8239356875419617)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram is currently not accepting new applications because of a lapse in appropriations. Even though Democrats have delayed this vital program, I remain committed to supporting our small businesses through it all. \n\nhttps://t.co/Hbf1SZCv9b'), ('dates', '2020-04-20 16:39:29'), ('file_name', 'Steven M. Palazzo.csv'), ('pct_true', 0.507340669631958)])
OrderedDict([('tweet', "The #PaycheckProtectionProgram is out of money. This didn't have to happen. \n\nDems needlessly snubbed America's small businesses by choosing to block additional funding for the PPP.\n\nAmerica's small businesses are counting on Congress to do the right thing &amp; authorize more funds. https://t.co/EbCHNREvwa"), ('dates', '2020-04-16 16:19:00'), ('file_name', 'Steven M. Palazzo.csv'), ('pct_true', 0.9625285863876343)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram runs out of funding today. \n\nCongressional Democrats chose to continue playing their political games instead of supporting our small businesses who are in serious need of relief.  \n\nThe program that has been a lifeline for many is on hold.'), ('dates', '2020-04-16 14:59:02'), ('file_name', 'Steven M. Palazzo.csv'), ('pct_true', 0.9308084845542908)])
OrderedDict([('tweet', 'Democrats flipped this emergency package from an insufficient plan that left behind hospitals and health workers and did nothing to aid the survival of the most vulnerable small businesses on Main Street. \nhttps://t.co/mksOJ4WOWZ'), ('dates', '2020-04-21 22:24:26'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.8647850751876831)])
OrderedDict([('tweet', "It's outrageous that struggling Americans have to wait for relief checks because Trump needs to see his name on their checks. How many more stunts must Americans endure to feed the President’s ego?\nhttps://t.co/pFJ2u1AujX"), ('dates', '2020-04-15 20:25:08'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.6306370496749878)])
OrderedDict([('tweet', 'The last amendment to our constitution took over 200 years to be ratified. It had no expiration date. Yet the #EqualRightsAmendment was given an arbitrary deadline for ratification. The House will vote to remove that deadline &amp; move the #ERA closer to becoming the 28th Amendment.'), ('dates', '2020-02-12 18:54:55'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.5078626275062561)])
OrderedDict([('tweet', 'A budget is supposed to reflect our values. The budget put forth by President Trump continues the broken promises of his administration. His budget slashes:\n\n❌ Medicare by $500 billion\n❌ Medicaid by $900 billion\n❌ Life saving medical research by $3 billion\n⬇️⬇️⬇️⬇️⬇️'), ('dates', '2020-02-11 19:55:25'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.7171669602394104)])
OrderedDict([('tweet', '75 years after the liberation of Auschwitz-Birkenau, we remember &amp; mourn the more than 6 million Jews and millions of other persecuted groups murdered in the Holocaust. We must continue to stand up against Anti-Semitism &amp; hatred. We will never forget. #NeverAgain #WeRemember'), ('dates', '2020-01-27 16:00:31'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.970595121383667)])
OrderedDict([('tweet', 'Yes, "right" does matter as my colleague Rep. Adam Schiff eloquently puts it. "The framers couldn\'t protect us from ourselves if right and truth don\'t matter...No constitution can protect us if right does not matter anymore."\n#DefendOurDemocracy https://t.co/0ZYaelculw'), ('dates', '2020-01-24 17:49:27'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.6437981128692627)])
OrderedDict([('tweet', "It's clear that the Senate is not following the Clinton impeachment precedent. There needs to be a fair trial. What is the President afraid of? #DefendOurDemocracy https://t.co/jCwyr4webB"), ('dates', '2020-01-21 19:39:34'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.7176240682601929)])
OrderedDict([('tweet', "With the House set to send impeachment articles to the Senate, it's important to note that every impeachment trial in US history has heard witnesses. That's what a trial is. No witnesses and no documents is what a cover-up looks like. #DefendOurDemocracy\nhttps://t.co/2xINbm1Azi"), ('dates', '2020-01-15 16:40:51'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.5164189338684082)])
OrderedDict([('tweet', 'There is no doubt that Qasem Soleimani was responsible for countless deaths and violence, what is not clear is the Trump Administration’s justification and the strategy for the aftermath of this dangerous escalation.'), ('dates', '2020-01-03 18:30:14'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.8113610148429871)])
OrderedDict([('tweet', 'After weeks of investigation and testimony from courageous and committed public servants, it’s clear President Trump put his political and personal priorities before our national security. For Congress to sit idly by would not only violate our oath but send a message to this...'), ('dates', '2019-12-10 19:33:10'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.5654253959655762)])
OrderedDict([('tweet', 'Fighting for American workers! The House will vote to reauthorize the Export-Import Bank, which has been a powerful force for creating good-paying jobs. In fact, 2 million jobs in the last decade! This will also ensure small businesses can compete on a global scale.'), ('dates', '2019-11-14 18:18:27'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.575989305973053)])
OrderedDict([('tweet', 'The President endangered our national security for his personal political gain. I just voted to move the impeachment inquiry to the next phase. The American people will begin to hear firsthand about the abuse of power committed by President Trump. #DefendOurDemocracy'), ('dates', '2019-10-31 18:46:25'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.9733162522315979)])
OrderedDict([('tweet', "Just voted to condemn President Trump's disastrous decision that has added to an already dangerous situation in the Middle East. https://t.co/JHrMz1YCAo"), ('dates', '2019-10-16 19:32:21'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.5702884793281555)])
OrderedDict([('tweet', 'The House has passed dozens of major bills - such as to lower health care &amp; prescription drug prices, end corruption, raise wages, and address gun violence - only to have Senate Majority Leader McConnell block them in the Senate. Time for Senate obstruction to end. #ForThePeople https://t.co/KD12VTWeIm'), ('dates', '2019-10-11 17:22:33'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.651651918888092)])
OrderedDict([('tweet', 'Just voted to cancel President Trump\'s "national emergency" and block him from talking money from our military to fund his unnecessary border wall. Our servicemembers and their families should not be pawns in political fights, especially just to satisfy a campaign promise.'), ('dates', '2019-09-27 18:08:24'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.6971513032913208)])
OrderedDict([('tweet', "It is time to open an impeachment inquiry. To not move forward would make Congress complicit in the President's behavior. President Trump’s actions surrounding Ukraine epitomize his disregard for checks and balances as well as normal practices and protocols."), ('dates', '2019-09-24 21:12:04'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.922278881072998)])
OrderedDict([('tweet', "Two years ago Heather Heyer was killed in Charlottesville when a neo-Nazi drove his car into people protesting white supremacy and hatred. Let's honor her by never giving up the struggle against hate. https://t.co/MGoTpMD9R1"), ('dates', '2019-08-12 21:29:42'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.5959388613700867)])
OrderedDict([('tweet', 'The American people are tired of waiting for the Senate to do something - anything - to keep people safe. Leader McConnell needs to call the Senate back and do their job. #DoSomething #EnoughIsEnough \n\nhttps://t.co/Uvja4TJBHw https://t.co/s5m0XbU3H4'), ('dates', '2019-08-07 19:53:36'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.5238394737243652)])
OrderedDict([('tweet', "Every American deserves to have their voice heard. Despite enactment of the Voting Rights Act over 50 yrs ago, these rights continue to be eroded. The House passed #HR1 to strengthen voters' rights, end gerrymandering, and lift up the voices of the American people. #ForThePeople https://t.co/ZHYHY7S4Fx"), ('dates', '2019-08-06 22:56:00'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.5219801068305969)])
OrderedDict([('tweet', "Californians' lives should not be at risk because of another state's lax gun laws, which is why I'm an original cosponsor of the Assault Weapons Ban Act. It's time to reinstate the nationwide assault weapons ban.\nhttps://t.co/q3W9cEcS50"), ('dates', '2019-08-06 21:44:43'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.7734074592590332)])
OrderedDict([('tweet', "The House is doing its job and passed legislation (#HR8 &amp; #HR1112) to end gun violence. It's time for the Senate to do its job and bring these bills to the Senate floor for a vote. #HonorWithAction"), ('dates', '2019-08-04 23:22:50'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.8703672289848328)])
OrderedDict([('tweet', "We need to stand up for animals who cannot stand up for themselves and end horse soring for good. It is a cruel practice that should have been outlawed decades ago. As a proud cosponsor of the PAST Act, I'm pleased to see the House pass this bill today. #PassthePASTAct"), ('dates', '2019-07-25 20:06:09'), ('file_name', 'Susan A. Davis.csv'), ('pct_true', 0.6734738349914551)])
OrderedDict([('tweet', 'Maine has received $1.25 billion in direct assistance so far through the CARES Act, but our state &amp; local governments will need additional help to offset the sharp revenue decrease caused by COVID-19.'), ('dates', '2020-05-08 19:20:32'), ('file_name', 'Susan M. Collins.csv'), ('pct_true', 0.5978754758834839)])
OrderedDict([('tweet', 'COVID-19 has greatly affected Maine’s potato farmers, who have lost nearly half of their business with the shutdown of restaurants &amp; could see orders reduced next year. To help mitigate losses, the Delegation is urging USDA to directly purchase Maine potatoes from farmers. https://t.co/hcTEkmPfse'), ('dates', '2020-04-21 12:32:27'), ('file_name', 'Susan M. Collins.csv'), ('pct_true', 0.584892749786377)])
OrderedDict([('tweet', 'It is unconscionable that criminals are seeking to exploit the current public health and economic crisis, yet scams related to COVID-19 are proliferating. I spoke with @newscentermaine about ways Mainers can protect themselves. https://t.co/WGJ6MiiEaw'), ('dates', '2020-04-08 11:45:26'), ('file_name', 'Susan M. Collins.csv'), ('pct_true', 0.6579975485801697)])
OrderedDict([('tweet', 'Demand for the Payroll Protection Program I co-authored, which provides forgivable loans so small businesses can survive and send paychecks to their employees, is so strong that I have urged the White House and my colleagues to prepare to increase the funding.'), ('dates', '2020-04-07 14:38:31'), ('file_name', 'Susan M. Collins.csv'), ('pct_true', 0.5067780017852783)])
OrderedDict([('tweet', 'Social Security beneficiaries have received letters claiming their payments will be suspended while SSA offices are closed amid COVID-19 unless they call a number given by con artists. Beneficiaries are then misled into providing personal info or payment.\n\nhttps://t.co/wudFPv4ByU'), ('dates', '2020-04-07 14:10:20'), ('file_name', 'Susan M. Collins.csv'), ('pct_true', 0.516215443611145)])
OrderedDict([('tweet', 'During this public health crisis &amp; time of economic hardship, criminals are seeking to steal from hardworking Americans. Small businesses are being told that they must pay a fee to apply for a Paycheck Protection Program loan, a program I co-authored.\n\nhttps://t.co/HmE4i4I80u'), ('dates', '2020-04-06 16:24:43'), ('file_name', 'Susan M. Collins.csv'), ('pct_true', 0.7414526343345642)])
OrderedDict([('tweet', "We are in the midst of a crisis in our country caused by the coronavirus.  When it comes to Americans’ health and saving Americans’ jobs and our small businesses, we don't have another day. We don't have another hour. We don't have another minute to delay acting. https://t.co/xbeagCoLvQ"), ('dates', '2020-03-23 18:06:48'), ('file_name', 'Susan M. Collins.csv'), ('pct_true', 0.8500779867172241)])
OrderedDict([('tweet', 'Today we remember the victory over Nazi Germany was the result of many nations working together w/ a shared purpose. We remember &amp; thank the brave men &amp; women who fought for our freedom &amp; protected us. #VEDay75 https://t.co/MlmxjX8fMG'), ('dates', '2020-05-08 14:35:39'), ('file_name', 'Susan W. Brooks.csv'), ('pct_true', 0.7066928148269653)])
OrderedDict([('tweet', 'Today, we commemorate the 6 million Jews &amp; millions of others who were killed in the Holocaust. We must stand UNITED in peace, community &amp; tolerance to overcome hatred &amp; discrimination in ALL forms. TOGETHER we can ensure that the past is never forgotten. #HolocaustRemembranceDay https://t.co/UAeJ7xECsb'), ('dates', '2020-04-21 15:14:47'), ('file_name', 'Susan W. Brooks.csv'), ('pct_true', 0.6875592470169067)])
OrderedDict([('tweet', 'As a strong believer in #bipartisanship time is not our side. We can’t be fighting ideological battles at this time. Yesterday, Senate Democrats blocked key aid from reaching the American people. They blocked direct payments to Americans, money for our small businesses,...'), ('dates', '2020-03-24 16:49:54'), ('file_name', 'Susan W. Brooks.csv'), ('pct_true', 0.9372115731239319)])
OrderedDict([('tweet', ".@BetsyDeVosED, a pandemic has turned students' lives upside-down. We should be spending our time ensuring students have the support they need to succeed in school, not rolling back the rights of survivors.\n\nNow is not the time to release the #TitleIX rule.https://t.co/kxG8B5lC0R"), ('dates', '2020-05-06 19:31:05'), ('file_name', 'Susan Wild.csv'), ('pct_true', 0.7187359929084778)])
OrderedDict([('tweet', "As we remember the 6 million lives stolen during the Holocaust, may we #NeverForget one of history's darkest hours—never repeating it and always speaking out against hate &amp; anti-Semitism. #HolocaustRememberanceDay #YomHaShoah https://t.co/Zp3x7uL3jb"), ('dates', '2020-04-21 23:59:53'), ('file_name', 'Susan Wild.csv'), ('pct_true', 0.6457910537719727)])
OrderedDict([('tweet', 'This is why we need to define “impacted areas” not by the number of #COVID19 cases, but by economic devastation, such as unemployment. Nevada needs to be at the top of the list in the next package. That’s what I’m fighting for. https://t.co/nMATHHp240'), ('dates', '2020-04-27 13:43:25'), ('file_name', 'Susie Lee.csv'), ('pct_true', 0.506846010684967)])
OrderedDict([('tweet', "Gaming small businesses should've never been left out of @SBAgov’s small business relief efforts. I, along with the bipartisan Nevada delegation, have been pushing the administration for weeks to fix this, ... 1/2 https://t.co/vvG1MPqYGc"), ('dates', '2020-04-24 15:52:27'), ('file_name', 'Susie Lee.csv'), ('pct_true', 0.8269547820091248)])
OrderedDict([('tweet', "The Paycheck Protection Program was enacted to help ALL small businesses.\n\nBut the administration's guidelines have left countless Nevadans and Americans without relief.\n\nLet’s make sure no Americans are left behind during #COVID19. @HouseSmallBiz https://t.co/44XAaYo48E"), ('dates', '2020-04-23 16:43:27'), ('file_name', 'Susie Lee.csv'), ('pct_true', 0.5493720769882202)])
OrderedDict([('tweet', 'Too many small businesses are missing out on #PPPLoans. We need to ensure #smallbiz in rural and underserved communities have access to this money. That’s why any #Smallbizrelief bill must empower community lenders. https://t.co/L8IWLyRmSX'), ('dates', '2020-04-16 01:30:00'), ('file_name', 'Susie Lee.csv'), ('pct_true', 0.5561358332633972)])
OrderedDict([('tweet', '700 @Alcoa workers in my district are losing their jobs and livelihood. It’s long past time for President Trump to effectively deal with China’s aluminum excess capacity that depresses prices and hurts workers. https://t.co/zyTG41JFQz'), ('dates', '2020-05-05 15:07:40'), ('file_name', 'Suzan K. DelBene.csv'), ('pct_true', 0.7403877377510071)])
OrderedDict([('tweet', 'Americans have a right to their representation in Congress and @RepMcGovern @RulesDemocrats proposal to allow proxy voting during #COVID19 guarantees that right. This is a historic crisis and as it continues we need to take historic action so Congress can respond quickly.'), ('dates', '2020-04-21 20:12:33'), ('file_name', 'Suzan K. DelBene.csv'), ('pct_true', 0.5496652722358704)])
OrderedDict([('tweet', 'Anti-choice politicians around the country are exploiting the #COVID19 pandemic to try and limit abortion services. I condemn attempts to deny patients access to comprehensive care. https://t.co/tmANVW2YLL'), ('dates', '2020-03-27 22:31:45'), ('file_name', 'Suzan K. DelBene.csv'), ('pct_true', 0.6848239898681641)])
OrderedDict([('tweet', 'The ongoing anti-Asian rhetoric when discussing COVID-19 &amp; the resulting racist attacks on the Asian-American community are unacceptable. I joined @RepGraceMeng in denouncing anti-Asian sentiments &amp; condemn racism in all its forms. https://t.co/ht1QMIoFf4'), ('dates', '2020-03-26 21:50:37'), ('file_name', 'Suzan K. DelBene.csv'), ('pct_true', 0.8846973180770874)])
OrderedDict([('tweet', 'Any day now the Supreme Court could rule on President Trump’s plan to end #DACA. This decision by the administration is heartless and, if upheld, would be catastrophic for America’s efforts in fighting #COVID19.'), ('dates', '2020-04-24 14:50:36'), ('file_name', 'Suzanne Bonamici.csv'), ('pct_true', 0.9128629565238953)])
OrderedDict([('tweet', 'We’re in the middle of an unprecedented health &amp; econ crisis. Working families are struggling to pay bills &amp; keep food on the table. Instead of showing empathy, the President is threatening the World Health Organization &amp; turning direct payment checks into a political stunt. https://t.co/AwQjW54VuC'), ('dates', '2020-04-15 03:22:35'), ('file_name', 'Suzanne Bonamici.csv'), ('pct_true', 0.512017011642456)])
OrderedDict([('tweet', 'Esto es inhumano! We should not be denying direct economic relief to 1.2 million Americans in need just because they’re married to an immigrant.\n\nVamos a seguir luchando para que todos reciban la ayuda económica que necesitan durante esta pandemic.\n\nhttps://t.co/wmZJl5phF1'), ('dates', '2020-05-07 00:57:42'), ('file_name', 'Sylvia R. Garcia.csv'), ('pct_true', 0.754877507686615)])
OrderedDict([('tweet', 'We cannot allow for @ICEgov facilities to be turned into Petri dishes for infection because this administration has not taken action. Migrants under U.S. custody deserve better. Their lives are worth saving. https://t.co/oHRCFHCrIS'), ('dates', '2020-05-05 22:27:32'), ('file_name', 'Sylvia R. Garcia.csv'), ('pct_true', 0.6710522174835205)])
OrderedDict([('tweet', "75 years ago, Allied forces accepted Nazi Germany's unconditional surrender – ending #WWII in Europe.\n\nWe in the Central Valley are forever grateful to our service members who fought during this conflict.\n\nTheir bravery and their sacrifices will never be forgotten. #VEDay75 https://t.co/obiwT2FBop"), ('dates', '2020-05-08 22:01:00'), ('file_name', 'TJ Cox.csv'), ('pct_true', 0.6853160262107849)])
OrderedDict([('tweet', 'Two years ago, some of my House colleagues voted to repeal our health care. If it had become law, it would have left 23 MILLION *more* Americans without coverage, raised premiums by 20 percent, and gutted protections for people with pre-existing conditions. #ProtectOurCare https://t.co/DDkixmY0vo'), ('dates', '2020-05-08 19:00:00'), ('file_name', 'TJ Cox.csv'), ('pct_true', 0.767692506313324)])
OrderedDict([('tweet', "The structure of PPP allowed bigger businesses to vacuum up the lion’s share of the first $350B package before small businesses got a cent. That's ridiculous. \n\nhttps://t.co/VFmH5Pdo6c"), ('dates', '2020-04-17 19:50:53'), ('file_name', 'TJ Cox.csv'), ('pct_true', 0.5896049737930298)])
OrderedDict([('tweet', 'The COVID-19 crisis is leaving fatalities &amp; job losses throughout our nation. \n\nWe’re fighting back by investing $200 million in expanding telehealth—a priority I’ve personally been pushing since day one.\n\nhttps://t.co/RSAZnkekq0'), ('dates', '2020-04-06 14:30:00'), ('file_name', 'TJ Cox.csv'), ('pct_true', 0.5162628293037415)])
OrderedDict([('tweet', "Trump wants to take away your health care during a deadly pandemic. And he's lying about protecting people with pre-existing health conditions.\n\nThat’s it. That’s the tweet. https://t.co/ac8vjK4F0w"), ('dates', '2020-05-06 20:06:26'), ('file_name', 'Tammy Baldwin.csv'), ('pct_true', 0.8578221201896667)])
OrderedDict([('tweet', 'Trump has promoted hydroxychloroquine as a treatment for COVID-19 despite the fact it is not an approved treatment. He has sent 335,800 tabs to Milwaukee and I want to know why, where they went and who has them. https://t.co/wVRxmU9nID'), ('dates', '2020-04-30 19:27:54'), ('file_name', 'Tammy Baldwin.csv'), ('pct_true', 0.6213793158531189)])
OrderedDict([('tweet', 'Today I introduced the Medical Supply Transparency and Delivery Act with @ChrisMurphyCT, @SenSchumer and 43 Senate Democrats to require @POTUS to unlock the full power of the DPA to mobilize a federal response to COVID19 that increases production of testing supplies and #PPE.'), ('dates', '2020-04-29 14:00:07'), ('file_name', 'Tammy Baldwin.csv'), ('pct_true', 0.5053645968437195)])
OrderedDict([('tweet', "The White House's broken supply chain and distribution process for medical supplies has left our VA workers without the #PPE they need to protect themselves and treat our veterans. This is a disgraceful failure. https://t.co/ruBrv17u9f"), ('dates', '2020-04-24 17:27:48'), ('file_name', 'Tammy Baldwin.csv'), ('pct_true', 0.5106812119483948)])
OrderedDict([('tweet', 'Profits should be shared with the workers who create them. It’s just wrong for big corporations to reward the wealthy or top executives with more stock buybacks, while closing facilities &amp; laying off workers.\nhttps://t.co/GdMQ01xDZ0'), ('dates', '2020-04-10 17:13:00'), ('file_name', 'Tammy Baldwin.csv'), ('pct_true', 0.7127950191497803)])
OrderedDict([('tweet', 'President Trump doesn’t want you to know that the HHS Inspector General has concluded that our hospitals and health care workers don’t have the medical supplies they need to treat patients and save lives. His denial of this reality will cost lives. https://t.co/mCO4a1CiQY'), ('dates', '2020-04-08 20:55:00'), ('file_name', 'Tammy Baldwin.csv'), ('pct_true', 0.5936288833618164)])
OrderedDict([('tweet', 'It’s heartless for President Trump to lock out the uninsured &amp; under-insured from having a chance to purchase a quality, affordable health insurance plan during a pandemic. Trump should reopen ACA marketplaces now so people can get the health care coverage they need now. https://t.co/wBcCuV9bPW'), ('dates', '2020-04-08 17:32:04'), ('file_name', 'Tammy Baldwin.csv'), ('pct_true', 0.5365222692489624)])
OrderedDict([('tweet', 'A conservative majority on the Wisconsin Supreme Court ruled against a safe election and for putting lives at risk.\n \nGOP appointed justices on the #SCOTUS ruled against a fair election and for a massive disenfranchisement of Wisconsin voters.\n\nWho is on your side? #CourtsMatter'), ('dates', '2020-04-07 00:11:30'), ('file_name', 'Tammy Baldwin.csv'), ('pct_true', 0.6886711716651917)])
OrderedDict([('tweet', 'Asian Americans are still facing racism and xenophobia related to the #COVID19 pandemic.\n\nThat’s one reason why @SenatorDurbin &amp; I joined @maziehirono in calling on @TheJusticeDept to take concrete steps to address the surge in hate crimes &amp; discrimination against this community.'), ('dates', '2020-05-05 17:53:01'), ('file_name', 'Tammy Duckworth.csv'), ('pct_true', 0.9335499405860901)])
OrderedDict([('tweet', "If protesters want to protest something, protest the Trump Administration's failure to provide test kits and #PPE.\n\nThis President has not done his job and continues to put politics in front of the health and well-being of Americans. \n\nShame on him. https://t.co/D0Xp8532qP"), ('dates', '2020-05-03 17:32:40'), ('file_name', 'Tammy Duckworth.csv'), ('pct_true', 0.6735673546791077)])
OrderedDict([('tweet', 'Trump is a failed Commander-in-Chief.\n\nNo real Commander would ever endanger the lives of his troops just for his own personal ego.\n\nThe man has no shame. https://t.co/6H4HnmLq3c'), ('dates', '2020-04-26 22:16:56'), ('file_name', 'Tammy Duckworth.csv'), ('pct_true', 0.6490306258201599)])
OrderedDict([('tweet', 'I never thought the phrase “speaking truth to power” would be interchangeable with saying “don’t drink bleach,” but these are strange times indeed. https://t.co/orc9PUV7se'), ('dates', '2020-04-24 00:29:22'), ('file_name', 'Tammy Duckworth.csv'), ('pct_true', 0.5182419419288635)])
OrderedDict([('tweet', 'Throughout the #COVID19 pandemic, the admin has failed to make something clear: the enemy is not the Asian or Asian-American community, but rather a virus that endangers us all. \n\nMy colleagues &amp; I wrote to Trump to remind him of this https://t.co/SReOSOzyPd'), ('dates', '2020-04-22 17:27:36'), ('file_name', 'Tammy Duckworth.csv'), ('pct_true', 0.8328238129615784)])
OrderedDict([('tweet', 'The deadly #COVIDー19 pandemic does not discriminate between individuals who have health insurance coverage and those who don’t. \n\nDuring this public health crisis, it’s in every American’s interest that we ensure those who need care are able to access it—and afford it.'), ('dates', '2020-04-21 19:45:28'), ('file_name', 'Tammy Duckworth.csv'), ('pct_true', 0.6980478763580322)])
OrderedDict([('tweet', 'Donald Trump’s basic responsibility is to keep Americans safe. The rising death count shows that he’s failing.\n \nInstead of spending his time deflecting blame, he should start *listening* to Governors &amp; giving them the supplies they need to fight the #COVID19 public health crisis https://t.co/k6jGVuyFqM'), ('dates', '2020-04-21 01:53:30'), ('file_name', 'Tammy Duckworth.csv'), ('pct_true', 0.5548675060272217)])
OrderedDict([('tweet', 'By allowing the Paycheck Protection Program to run out of money &amp; refusing to negotiate a deal with Democrats, Senate Republicans are pitting small businesses against hospitals and health care workers. \n\nRead my full statement on this https://t.co/eOcxDLKbhM'), ('dates', '2020-04-17 19:02:51'), ('file_name', 'Tammy Duckworth.csv'), ('pct_true', 0.6817776560783386)])
OrderedDict([('tweet', 'I’m troubled by reports that small businesses across the country are not receiving their Emergency Economic Injury Grant (EEIG) advances, despite the fact that they are eligible entities and have taken the necessary steps.'), ('dates', '2020-04-15 18:25:03'), ('file_name', 'Tammy Duckworth.csv'), ('pct_true', 0.7336676120758057)])
OrderedDict([('tweet', 'American taxpayers, whose dollars fund a significant part of the World Health Organization’s budget, deserve to know to what extent the W.H.O. amplified Chinese Communist Party propaganda. https://t.co/VFoqJxp8ay'), ('dates', '2020-05-08 15:37:33'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.8546193838119507)])
OrderedDict([('tweet', 'The UN must renew the arms embargo on the Iranian regime.\n\nIran is the #1 exporter of terrorism and we have to keep maximum pressure on this rogue state. https://t.co/pNq2NKeJkt'), ('dates', '2020-05-04 19:16:57'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.5067237615585327)])
OrderedDict([('tweet', "I fully support a complete investigation into the origins and causes of the Coronavirus in China. \n\nThe Chinese Communist Party's lies and misinformation should be fully exposed. https://t.co/ZJDwrorRYS"), ('dates', '2020-04-23 15:20:26'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.5724937915802002)])
OrderedDict([('tweet', 'It has become painfully clear that China can no longer be trusted. \n\nWe have to pivot away from China and toward national self-reliance so we are never caught flat-footed again. https://t.co/MEzZqAyS0k'), ('dates', '2020-04-22 14:56:21'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.8861181735992432)])
OrderedDict([('tweet', "Proud to join @RepJimBanks on this. China's Communist Party must be held accountable. https://t.co/GLJzaC0JMt"), ('dates', '2020-04-21 13:38:32'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.5171794891357422)])
OrderedDict([('tweet', "The Chinese Communist Party cannot be trusted. \n\nIt's time we rethink our reliance on China for medical supplies and pharmaceuticals. https://t.co/qYCniQbcOi"), ('dates', '2020-04-14 19:41:31'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.8414775133132935)])
OrderedDict([('tweet', 'As we continue to fight the coronavirus, we have to start reducing our reliance on China for medical supplies and pharmaceuticals.\n\nI introduced a bill with @RepGallagher, @RepLizCheney, and @RepStefanik that helps do that. https://t.co/Kcz2enbkdE'), ('dates', '2020-04-13 13:34:42'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.5167001485824585)])
OrderedDict([('tweet', 'The Chinese Communist Party:\n \n- Silenced doctors.\n\n- Ordered the destruction of research samples.\n\n- Underreported the number of infections &amp; deaths.\n\n- Pushed propaganda that the U.S. military created COVID-19.\n \nWe have to reduce our reliance on China. They cannot be trusted. https://t.co/2mn3h9OGFU'), ('dates', '2020-04-08 18:57:04'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.7852669358253479)])
OrderedDict([('tweet', "Mecklenburg County has 444 cases of COVID-19, which would have been preventable had China not lied, and the first reaction is to take political cheap shots. That's not helping anybody. https://t.co/fSoVVsE3zX"), ('dates', '2020-04-01 18:37:53'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.7729001045227051)])
OrderedDict([('tweet', 'Speaker Pelosi delayed the aid package a week while she pushed her liberal wishlist.\n\nNow she’s taking political pot shots during a crisis. \n\nShe’s not helping. https://t.co/kGIvZ3RJe1'), ('dates', '2020-03-29 14:22:31'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.5319015383720398)])
OrderedDict([('tweet', 'Speaker Pelosi thinks she can use the cover of a global pandemic to pass her far-left agenda? \n\nNot a chance. The American people see right through this charade.'), ('dates', '2020-03-24 02:18:12'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.86175936460495)])
OrderedDict([('tweet', 'Looks like Speaker Pelosi tossed a political grenade into the coronavirus relief negotiations tonight. \n\nWorkers and small businesses should be the focus here, not her liberal wishlist. https://t.co/HJMK3nPuaZ'), ('dates', '2020-03-23 00:58:07'), ('file_name', 'Ted Budd.csv'), ('pct_true', 0.7171663641929626)])
OrderedDict([('tweet', 'Elected officials in the US don’t have the right to demand apologies from their citizens. Shelley Luther refused to apologize for trying to earn a living and feed her family, and I was proud to support her yesterday when she re-opened her salon in accordance with #TX law. https://t.co/HRLnu8UppO'), ('dates', '2020-05-09 19:04:27'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.544445276260376)])
OrderedDict([('tweet', 'American taxpayers SHOULD NOT be forced to bail out states for irresponsibility and fiscally reckless policies that long preceded the #CoronavirusPandemic. #NoStateBailouts https://t.co/oK5rPv2H96'), ('dates', '2020-05-08 20:38:28'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.5145516395568848)])
OrderedDict([('tweet', 'Democrats are trying to eliminate blue-collar jobs by pressuring banks to discriminate against oil, gas &amp; coal industries. I joined @SenKevinCramer &amp; @SenDanSullivan in urging @POTUS to take a stand for our energy producers &amp; the hardworking Americans whose jobs are on the line. https://t.co/s24dxH7TW7'), ('dates', '2020-05-07 23:06:40'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9086189270019531)])
OrderedDict([('tweet', 'Americans are feeling the devastation caused by the #CoronavirusPandemic as another 3.2M workers have filed for unemployment. As we continue working to defeat this virus, we must focus on safely re-opening our economy so people can return to their jobs &amp; lives outside the home. https://t.co/DMM37dGsj0'), ('dates', '2020-05-07 20:15:18'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.5715773701667786)])
OrderedDict([('tweet', 'Why is @washingtonpost allowing the Chinese Communist Party to publish its propaganda?\n \nWe now have proof the CCP knew the #CoronavirusOutbreak could lead to a global pandemic &amp; spent weeks trying to cover it up. They have blood on their hands and they must be held accountable. https://t.co/sUjgVPJ25S'), ('dates', '2020-05-07 19:09:26'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.978935182094574)])
OrderedDict([('tweet', 'Democrats continue to wage war against America’s blue-collar workers. Denying energy producers access to capital will add hundreds of thousands of more workers to the unemployment rolls on top of 33.5M Americans already out of work because of #COVID19.\nhttps://t.co/0ebOcPshcK https://t.co/igUDN3uphu'), ('dates', '2020-05-07 16:45:48'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.6657317876815796)])
OrderedDict([('tweet', 'MY MESSAGE TO HOLLYWOOD: We’re not going to facilitate Chinese Communist censorship. My SCRIPT Act will force Hollywood studios censoring films in search of bigger profits to choose between the assistance they need from the American government and the money they want from China. https://t.co/8Ql3InF8Te'), ('dates', '2020-05-07 01:48:03'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9579622149467468)])
OrderedDict([('tweet', "The global devastation caused by the Chinese Communist Party’s #CoronavirusCoverup should be a wake-up call to the entire world. China's censorship led to the loss of hundreds of thousands of innocent lives, and the officials responsible must be held accountable. https://t.co/PFnCQSv76P"), ('dates', '2020-05-06 22:19:35'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.910304605960846)])
OrderedDict([('tweet', ".@TencentGlobal and @Huawei are espionage operations for the Chinese Communist Party, masquerading as telecommunication companies. I'm introducing a bill to help stop China’s surveillance in the US by prohibiting federal employees from using these platforms for official business. https://t.co/Je4fh0sU6T"), ('dates', '2020-05-06 21:33:59'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9892483949661255)])
OrderedDict([('tweet', 'We cannot let this country fall into another Great Depression. There are too many politicians in Washington, including @SpeakerPelosi, who are not focused on getting the economy moving again. We need to keep people safe, follow sound science, but also get people back to work. https://t.co/HRLkqErpNn'), ('dates', '2020-05-06 15:53:39'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.7732012271881104)])
OrderedDict([('tweet', 'As a result of the #CoronavirusPandemic, our country has experienced unprecedented economic devastation. But that is not a reason for states like Texas to bailout states that have been fiscally irresponsible since long before this crisis hit. https://t.co/egc5SEcrFu'), ('dates', '2020-05-06 14:56:04'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.5869636535644531)])
OrderedDict([('tweet', 'Hollywood is fully complicit in China’s censorship. This week I introduced the SCRIPT Act, legislation that will force Hollywood studios to choose between the assistance they need from the American government and the dollars they want from China. https://t.co/WOGONXgsRo'), ('dates', '2020-05-06 02:43:28'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.749282717704773)])
OrderedDict([('tweet', '.@TuckerCarlson: "Studios in California have censored and rewritten American films to please China...Senator Ted Cruz is infuriated by it and says it is time for the federal government to punish studios that grovel before their Chinese masters." https://t.co/KFim3r8EYG'), ('dates', '2020-05-06 01:37:22'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.7415457963943481)])
OrderedDict([('tweet', 'This is why having defense capabilities in space is so critical: @NASA has identified an asteroid that will fly by Earth in 2029 and is working with @SpaceForceDoD to mitigate potential threats to Americans and the rest of the world. https://t.co/ediROxULgq'), ('dates', '2020-05-05 22:27:32'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.605480432510376)])
OrderedDict([('tweet', 'China is the greatest geopolitical threat the US faces over the next century. China covered up the #coronavirus outbreak and lied about it to the world. That’s why I’m introducing four bills to address the threat China poses to our national security and public health. https://t.co/ocIkcqOFSp'), ('dates', '2020-05-04 22:24:48'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9427219033241272)])
OrderedDict([('tweet', 'US intel officials have confirmed the CCP “intentionally concealed the severity” of #coronavirus from the world while stockpiling their own medical supplies. The people responsible for this deadly coverup must be held accountable. I’m introducing legislation to do just that. https://t.co/bmCSvqPznG'), ('dates', '2020-05-04 21:39:39'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.6968164443969727)])
OrderedDict([('tweet', 'Once we recover from the #CoronavirusPandemic, we must reevaluate our relationship with China. The Chinese government hid the #coronavirus outbreak from the world, which has cost hundreds of thousands of lives and millions of jobs. China must be held accountable. https://t.co/JOZSquOWKI'), ('dates', '2020-05-04 20:58:11'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9100580215454102)])
OrderedDict([('tweet', '"For too long, Hollywood has been complicit in China\'s censorship and propaganda in the name of bigger profits," Sen. Cruz says. https://t.co/RlcNaFI0b7 via @Gundisalvus @DailySignal'), ('dates', '2020-05-04 15:23:51'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.673778235912323)])
OrderedDict([('tweet', 'China is the single greatest geopolitical threat facing the US over the next century. This week I will be introducing several bills to combat Chinese espionage, stop Chinese censorship and propaganda, and hold China accountable for endangering millions of lives. https://t.co/ciqpqCj7CQ'), ('dates', '2020-05-03 21:41:59'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9166285991668701)])
OrderedDict([('tweet', 'The most important national security and foreign policy consequence of this #CoronavirusPandemic is that there is going to be a fundamental reassessment of the US relationship with China. China is America’s greatest geopolitical threat. We need to start acting like it. https://t.co/CGYsnLdpmd'), ('dates', '2020-05-03 16:49:14'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.7850880026817322)])
OrderedDict([('tweet', 'From NY to KY, we are seeing a disturbing trend of "pandemic authoritarianism" emerge across the country. I\'m urging AG Barr to investigate the local officials using #COVID19 as an opportunity to restrict freedom &amp; specifically target communities of faith. https://t.co/U4kgRSyWII'), ('dates', '2020-04-30 22:58:13'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.5095512270927429)])
OrderedDict([('tweet', '.@foxnews: Cruz bill would bar US officials from using Chinese-backed tech, stop taxpayer dollars for related UN contracts https://t.co/046vW6twkU'), ('dates', '2020-04-30 21:00:42'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.6473733186721802)])
OrderedDict([('tweet', 'RELEASE: Cruz calls on Attorney General Barr to investigate state and local officials who target and discriminate against religious communities under the guise of protecting public health.'), ('dates', '2020-04-30 19:25:07'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.5463709235191345)])
OrderedDict([('tweet', 'The aggression and defensiveness of the Chinese Communist Party strongly suggests they have something to hide. Chinese officials who participated in and continue to participate in the #CoronavirusCoverup must be held accountable. https://t.co/TAyD1Z25oS'), ('dates', '2020-04-29 22:24:47'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9264716506004333)])
OrderedDict([('tweet', "For too long, Hollywood has been complicit in China's censorship and propaganda in the name of bigger profits. When Congress returns, I’ll introduce legislation to cut off Hollywood from @DeptofDefense assistance if those studios censor their films for screening in China."), ('dates', '2020-04-29 19:05:29'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9813786745071411)])
OrderedDict([('tweet', 'I have long-called for U.S. companies to stand up for freedom and stop kowtowing to China. My SCRIPT Act will ensure Hollywood studios will do just that, by forcing them to choose between the assistance they need from the American government and the dollars they want from China. https://t.co/3esGpHVqj0'), ('dates', '2020-04-29 16:17:00'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.7955312132835388)])
OrderedDict([('tweet', 'For too long Hollywood has been complicit in China’s censorship &amp; propaganda in the name of bigger profits. My SCRIPT Act will be a wakeup call, forcing Hollywood studios to choose b/n the assistance they need from the US gov &amp; the $$ they want from China.\nhttps://t.co/Jl4F6woEbu'), ('dates', '2020-04-29 00:10:09'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9074956178665161)])
OrderedDict([('tweet', 'Excellent piece by @michaelsobolik on how Chinese Communist Party’s censorship allowed the #CoronavirusOutbreak to spread into a global pandemic and endanger lives, and why the U.S. must hold CCP officials accountable for suppressing medical information.\nhttps://t.co/I3ibezgpzd'), ('dates', '2020-04-28 23:21:25'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9213145971298218)])
OrderedDict([('tweet', "We're facing two simultaneous crises: a public health crisis that has caused more than 50,000 Americans to lose their lives and an economic crisis that has caused 26 million Americans to lose their jobs. We need to focus on saving Americans from both crises. https://t.co/5NsEU3hGwP"), ('dates', '2020-04-25 00:15:39'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.8085077404975891)])
OrderedDict([('tweet', 'The Chinese Communist Party should not be able to set up shop in Mexico &amp; blanket America with propaganda. As\xa0@kredo0\xa0first reported, when Congress returns I will intro a bill to close\xa0@FCC\xa0loopholes being used by stations connected to CCP-controlled media, including Phoenix TV.'), ('dates', '2020-04-24 22:36:43'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.9682379364967346)])
OrderedDict([('tweet', 'We have always known China’s censorship was a direct threat to human rights. We now know it also poses a direct threat to U.S. national security, our economy, and global public health. https://t.co/BWcpM4VU8x'), ('dates', '2020-04-24 16:05:31'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.8242016434669495)])
OrderedDict([('tweet', "The @WHO appears to have shown remarkable deference to China throughout\xa0the #CoronavirusPandemic. I joined @SenatorRisch &amp; colleagues in a letter to UN Sec Gen @antonioguterres requesting the UN convene an independent panel of experts to review WHO's response to\xa0#COVID19."), ('dates', '2020-04-24 15:36:41'), ('file_name', 'Ted Cruz.csv'), ('pct_true', 0.6203334331512451)])
OrderedDict([('tweet', 'During Asian Pacific American Heritage Month, it’s more important than ever to stand in solidarity with the #AAPI community and denounce the alarming rise in anti-Asian bigotry we are seeing due to #COVID19. This is unacceptable and must end. #StopAAPIHate #APAHM2020'), ('dates', '2020-05-01 20:52:25'), ('file_name', 'Ted Lieu.csv'), ('pct_true', 0.5896995067596436)])
OrderedDict([('tweet', 'The U.S. needs to break its silence &amp; recognize the #ArmenianGenocide. Last year we overwhelmingly passed a resolution to recognize the systematic killing of 1.5 million Armenians under the Ottoman Empire in 1915. Today we remember those who perished &amp; pledge #NeverAgain. https://t.co/PdROOJxmb9'), ('dates', '2020-04-24 21:30:35'), ('file_name', 'Ted Lieu.csv'), ('pct_true', 0.508421003818512)])
OrderedDict([('tweet', '#MitchAntoinette is attacking state and local governments desperately seeking aid to fight #COVID19, saying he would prefer they go bankrupt instead of giving them emergency funds. This is a disgusting remark to make when American lives are on the line. https://t.co/oVi4m2zF7r'), ('dates', '2020-04-24 14:00:56'), ('file_name', 'Ted Lieu.csv'), ('pct_true', 0.8792957663536072)])
OrderedDict([('tweet', 'On this #HolocaustRemembranceDay, #WeRemember the 6 million Jews and the millions of others murdered during the Holocaust. We say #NeverAgain and recommit to combating hatred, antisemitism, and injustice wherever we encounter it. #YomHaShoah'), ('dates', '2020-04-21 23:10:14'), ('file_name', 'Ted Lieu.csv'), ('pct_true', 0.7269203066825867)])
OrderedDict([('tweet', 'We’re calling for an investigation into whether @SECNAV acted inappropriately by relieving Captain Brett Crozier who expressed concern about #coronavirus spreading among servicemembers on the USS Theodore Roosevelt. Our letter with @RepRubenGallego ⬇️ https://t.co/WO9ERFLUXz'), ('dates', '2020-04-03 14:59:44'), ('file_name', 'Ted Lieu.csv'), ('pct_true', 0.6382112503051758)])
OrderedDict([('tweet', 'We can’t leave our front-line fighters without the supplies they need to save American lives. The Medical Supply Chain Emergency Act will force the President to federalize American manufacturing &amp; end the supply shortage. We need the #DefenseProductionActNOW. https://t.co/oSyO67OFym'), ('dates', '2020-03-27 18:15:16'), ('file_name', 'Ted Lieu.csv'), ('pct_true', 0.5203908681869507)])
OrderedDict([('tweet', 'Today marks the 10-year anniversary of the #ACA, which has provided millions of Americans with health care coverage. Despite Republicans’ ongoing efforts to repeal the #ACA, @HouseDemocrats continue the fight to #ProtectOurCare #ForThePeople. #ACAanniversary'), ('dates', '2020-03-23 15:02:22'), ('file_name', 'Ted Lieu.csv'), ('pct_true', 0.9629669189453125)])
OrderedDict([('tweet', 'Throughout this crisis, #Taiwan has repeatedly demonstrated the value of transparency &amp; responsible democratic governance in the face of a sweeping #pandemic. @HouseGOP @HouseForeignGOP'), ('dates', '2020-05-08 20:40:00'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.8268919587135315)])
OrderedDict([('tweet', 'Today I introduced the House companion to @SenRickScott bill calling for the IOC to rebid the 2022 #WinterOlympics in #Beijing. The global community should not reward a country with a horrendous #HumanRights record with the honor of hosting major world sporting events. @iocmedia'), ('dates', '2020-05-05 15:57:35'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.8635861277580261)])
OrderedDict([('tweet', '#DHS intel report confirms “#China held off informing [WHO] the #coronavirus was a contagion most of January” so it could hoard medical supplies from abroad. Then they send faulty &amp; broken medical supply shipments to countries in need in a so-called “politics of generosity”'), ('dates', '2020-05-04 19:28:55'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.880425214767456)])
OrderedDict([('tweet', 'China’s appointment to the @UN_HRC given its own horrible record on #HumanRights is beyond ridiculous. Proud to cosponsor @repSteube bill H.Res.919 calling out the #UN for this misguided decision &amp; expect that they recognize the millions of oppressed people in #China.'), ('dates', '2020-04-29 14:40:18'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.9383319616317749)])
OrderedDict([('tweet', '#SmallBusinesses are the real engines that drive our economy &amp; they are in need of capital through the #PayrollProtectionProgram. W/ #PPP out of funds @SpeakerPelosi is more concerned w/ playing politics than working with us to pass more funding. https://t.co/DFLslX5W9x'), ('dates', '2020-04-16 16:35:23'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.6654336452484131)])
OrderedDict([('tweet', '.@POTUS is right to call out #WHO for heavy reliance on #China &amp; embrace of their propaganda machine. We must reevaluate relationships with any multilateral organization that blindly follows malicious misinformation campaigns. https://t.co/zXV1OAJcjQ @HouseForeignGOP'), ('dates', '2020-04-08 13:38:41'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.6603387594223022)])
OrderedDict([('tweet', 'Here\'s what #China &amp; the #CCP have been pedaling during the #coronoavirus crisis.\n\n🚫Expulsion of international journalists\n🚫Faulty medical equipment\n🚫Debunked conspiracy theories\n🚫Falsified COVID-19 case numbers\n\nAll this to win a "propaganda battle." What we need is #truth.'), ('dates', '2020-04-02 01:13:11'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.7758805155754089)])
OrderedDict([('tweet', '#China could have been honest when #coronavirus first hit #Wuhan. \n\nThey were not.\n\nA lesson for all nations: never trust information from an authoritarian state. https://t.co/WAnsAGnYMu'), ('dates', '2020-04-01 21:43:21'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.5999496579170227)])
OrderedDict([('tweet', 'For too long, the #Maduro regime has subjected the #Venezuelan people to instability &amp; poverty &amp; aided efforts to destroy American communities through the illegal drug trade. I applaud the @TheJusticeDept actions in holding Maduro &amp; his partners accountable. @HouseForeignGOP'), ('dates', '2020-03-26 18:48:23'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.9296130537986755)])
OrderedDict([('tweet', 'Recent reports are evidence that #Taiwan can’t rely on #China or Intl organizations they are barred from in times of crisis. In order for Taiwan’s Pop. of 23 million to have a voice on global health, they must have a seat at the table. https://t.co/xRdJDmqzcJ @HouseGOP @WHO'), ('dates', '2020-03-24 14:03:38'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.8814256191253662)])
OrderedDict([('tweet', 'Unbelievable that China’s govt is supporting conspiracy claims that the U.S. is the real source of #coronavirus &amp; is concealing the true extent of the spread within our country. \nhttps://t.co/iamJMNRlGF @HouseGOP @HouseForeignGOP'), ('dates', '2020-03-05 15:02:59'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.5199403762817383)])
OrderedDict([('tweet', 'So far, #China has reported over 72,000 cases of #coronavirus while #Taiwan reports just over 20. But the @WHO classifies both as “very high risk” by lumping them together as one country.'), ('dates', '2020-02-20 22:04:19'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.8693103790283203)])
OrderedDict([('tweet', 'Yet another red flag for why we should stay away from #Huawei. The US &amp; partners should focus on meeting long-term security risks over seeking comfort &amp; convenience in the short-term. @HouseGOP @HouseForeignGOP #China https://t.co/Y9wkLYgp02'), ('dates', '2020-02-11 21:06:25'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.5168981552124023)])
OrderedDict([('tweet', '#DOJ charged 4 Chinese #hackers w/stealing the personal info of millions of Americans from #Equifax. The U.S. will not sit back &amp; watch as the security &amp; privacy of its citizens are routinely violated by #China. The #PLA must be held accountable. https://t.co/8iwWMeIPfc @HouseGOP'), ('dates', '2020-02-10 15:28:13'), ('file_name', 'Ted S. Yoho.csv'), ('pct_true', 0.9056801795959473)])
OrderedDict([('tweet', 'Today marks 10 years since Pres. Obama signed the Affordable Care Act into law. \n\nThe ACA has meant:\n✅ Insurance companies can’t charge more for pre-existing conditions;\n✅ Women are charged the same as men;\n✅ More than 200,000 in AL have gained coverage.\n\n#ProtectOurCare'), ('dates', '2020-03-10 13:57:54'), ('file_name', 'Terri A. Sewell.csv'), ('pct_true', 0.5965924859046936)])
OrderedDict([('tweet', "The recent footage of Ahmaud Arbery's death is heartbreaking. @TheJusticeDept must conduct a full investigation into his death and hold his killers accountable. \n \nAhmaud and his family deserve justice. #JusticeForAhmaud"), ('dates', '2020-05-07 19:47:59'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.5671153664588928)])
OrderedDict([('tweet', "This Admin. does not get to act with impunity. Silencing the voices of public health experts is entirely unacceptable. We should be putting the nation's health first.\n\nWe don't want Trump-filtered facts. We just want the facts.\n\nhttps://t.co/GHxjeeRRfH"), ('dates', '2020-05-05 22:15:00'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.8018485307693481)])
OrderedDict([('tweet', "Pleased to see Germany take this step to ban terror org Hezbollah. For years, I've led the charge in Congress urging Euro partners to designate Hezbollah in its entirety. More countries &amp; EU taking this step will help stop Hez funding &amp; operations abroad. \nhttps://t.co/ft2m1sBEvf"), ('dates', '2020-05-01 19:46:48'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.5980249047279358)])
OrderedDict([('tweet', "It's been one year since the Jewish community of Poway lost Lori Gilbert-Kaye to a horrific act of anti-Semitism.\n\nMay her memory serve as a blessing &amp; a reminder that while hateful &amp; bigoted people will use even deadly means to intimidate us, we will not let them win. https://t.co/cBmVXAh8ye"), ('dates', '2020-04-28 14:02:23'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.6440543532371521)])
OrderedDict([('tweet', "The new Social Security Trustees report does not reflect the impact of COVID19.\n \nCongress must act to protect seniors &amp; people w/ disabilities &amp; expand Social Security benefits. These most vulnerable Americans shouldn't have their earned benefits cut.\n\nhttps://t.co/s7twuyeOyr"), ('dates', '2020-04-24 17:28:16'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.6685448884963989)])
OrderedDict([('tweet', 'Gov DeSantis’ comments yesterday were dangerously wrong. COVID-19 has killed young people.\n\n+ opening schools isn’t just about student safety. It’s about families &amp; teachers &amp; bus drivers &amp; janitors &amp; staff.\n\nFloridians know this. The governor should, too.\nhttps://t.co/5ET80dwrGI'), ('dates', '2020-04-12 00:13:46'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.5607674717903137)])
OrderedDict([('tweet', '16.5 million people have filed for unemployment over the past three weeks. \n\nMany others have been stuck on the phone, online, or resorted to paper applications with Florida DEO.\n\nWe must get workers the benefits they need. https://t.co/TWr4a60ClG'), ('dates', '2020-04-09 19:53:37'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.7222529053688049)])
OrderedDict([('tweet', 'The Admin. is unfairly penalizing small business owners who previously committed a minor criminal offense from seeking CARES Act loans to protect their employees &amp; businesses. \n\nA person who served their time &amp; is contributing to our economy, should be able to apply for relief. https://t.co/eWiULw4VWW'), ('dates', '2020-04-09 14:28:28'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.813929557800293)])
OrderedDict([('tweet', 'Using the COVID-19 crisis to push a destructive anti-choice agenda is unacceptable. \n \nOur communities are currently experiencing a global pandemic. Our focus should be on improving public health, not restricting people from exercising their basic healthcare rights. https://t.co/cMQ374PnIj'), ('dates', '2020-04-09 13:30:43'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.5157250165939331)])
OrderedDict([('tweet', 'Between anti-Semitic "Zoombombers" &amp; racist attacks on Asian Americans, bigots are trying to seize on our fears during this crisis.\n\nBut we cannot allow them to divide us in a moment when we need to be united against this public health threat.\n\nLet\'s fight this virus together. https://t.co/Ji48rMkGYz'), ('dates', '2020-04-01 16:32:44'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.8697539567947388)])
OrderedDict([('tweet', '"God is spreading it in your synagogues!”\nMore hateful vitriol from the antisemitic propaganda website TruNews.\n\nThis dangerous language should be condemned - and these guys should never get another invitation from the White House. https://t.co/A6qE51a71N'), ('dates', '2020-03-30 16:06:55'), ('file_name', 'Theodore E. Deutch.csv'), ('pct_true', 0.5165050029754639)])
OrderedDict([('tweet', 'Small businesses across the nation just lost their lifeline because of @SpeakerPelosi’s political games, and she’s now busy picking out what flavor of gourmet ice cream she’s going to have. Grateful for my Dem colleagues like @SenatorSinema who are putting country before party. https://t.co/QR9b0WEdiv'), ('dates', '2020-04-16 19:01:57'), ('file_name', 'Thom Tillis.csv'), ('pct_true', 0.8303771018981934)])
OrderedDict([('tweet', 'Disgusting to see these companies price gouge essential medical supplies, taking advantage of health care providers who desperately need them. My legislation, the Ending Price-Gouging During Emergencies Act, would punish these companies with criminal prosecution and big fines. https://t.co/0wwAP3Xj92'), ('dates', '2020-03-27 23:25:15'), ('file_name', 'Thom Tillis.csv'), ('pct_true', 0.7818393707275391)])
OrderedDict([('tweet', 'In response to meat shortages, empty suits are saying “we have a demand problem, not a supply problem.”  \n\nI’m telling the same suits to get ready for TAR AND FEATHERS if they’re going to blame consumers for the brittle food supply politicians created at the request of lobbyists.'), ('dates', '2020-05-10 12:48:30'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.6300512552261353)])
OrderedDict([('tweet', 'Maybe when the Congressmen sitting at home eating ice cream out of their $24,000 refrigerators can\'t get ice cream anymore they\'ll reconvene Congress &amp; say "instead of throwing money at the problem let\'s solve it."\n\nThanks for having me on @lfs6b. \n\nhttps://t.co/u1Kjehl4Qf'), ('dates', '2020-05-07 22:50:56'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.5083593130111694)])
OrderedDict([('tweet', 'To originate the CARES act in the Senate &amp; thereby violate the spirit of the origination clause (Constitution says revenue bills must originate in the House), the Senate took a bill that passed the House last year, gutted the entire bill, &amp; substituted their bill as an amendment. https://t.co/64gKolYZYx'), ('dates', '2020-05-06 05:56:29'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.8134799599647522)])
OrderedDict([('tweet', 'Allowing grocery shelves to go empty while farmers have to destroy animals because Chinese and Brazilian owned meat packers can’t process the meat is an American tragedy!\n\nStop over-regulating American farmers and processors. \n\nPass the PRIME Act!\n#AmericaEatAgain https://t.co/YIkrl7JPyH'), ('dates', '2020-05-05 22:27:18'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.6959298849105835)])
OrderedDict([('tweet', "Everybody has two senators &amp; a representative. Call them &amp; tell them to sponsor the PRIME Act. Tell them you don't want meat shortages &amp; don't want animals being wasted in landfills.\n\nThanks for having me on @toddstarnes \n\nMiss the interview? Listen here: https://t.co/pKGIrEGqWE"), ('dates', '2020-05-05 21:48:47'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.6530943512916565)])
OrderedDict([('tweet', 'These Senators were sponsors of the PRIME Act before the meat shortages began! \n\nThe PRIME Act can reduce our dependence on multinational meat processors while empowering farmers and American owned and operated processors.\n@senangusking \n@randpaul \n@senalexander \n@marshablackburn'), ('dates', '2020-05-05 15:46:59'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.9090014696121216)])
OrderedDict([('tweet', 'If you’re under the impression that the government has fixed problems with the food supply then you’ve been duped.\n\nExpect higher prices, more imported food of questionable origin, and domestic supply disruptions for at least three months.\n\nCongress should pass the PRIME Act now.'), ('dates', '2020-05-04 15:11:19'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.5302357077598572)])
OrderedDict([('tweet', 'Thousands of animals will be killed &amp; wasted today instead of feeding families. Meanwhile Congress takes an extended vacation.\n\nPass the PRIME Act now to allow small American owned meat processors to catch the ball that the Chinese, Brazilian, &amp; multinational processors dropped.'), ('dates', '2020-05-03 13:35:16'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.8609610795974731)])
OrderedDict([('tweet', 'The PRIME act would allow small American owned meat processors to take up much of the slack as giant multinational meat packers shut down. The alternative to not passing the PRIME Act is meat shortages, wasted animals, and bankrupt farmers.\n\nhttps://t.co/Yo2ud5AARf'), ('dates', '2020-05-03 02:06:31'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.7181776762008667)])
OrderedDict([('tweet', 'Dozens of Republicans, Democrats, and Independents support this bill in both the House and the Senate.  Let’s put politics aside and pass the PRIME Act before more animals are unnecessarily wasted in land fills and rendering plants. https://t.co/sVwWcZFBvk'), ('dates', '2020-05-02 23:38:20'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.6435307264328003)])
OrderedDict([('tweet', '"Rep. Thomas Massie is gaining bipartisan support for his PRIME Act. The bill has been sitting in the House &amp; the Senate for nearly a year, meaning it could be fast tracked to @realDonaldTrump if the political will is there."\n\nhttps://t.co/P4fuM5GoQ7'), ('dates', '2020-05-01 20:33:03'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.5677100419998169)])
OrderedDict([('tweet', 'When a government seeks to suspend the right to keep and bear arms during a crisis, the government becomes the crisis. https://t.co/nbqzt22AEZ'), ('dates', '2020-04-30 23:41:52'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.5761162638664246)])
OrderedDict([('tweet', 'If Congress isn’t going to convene until the bill is ready to vote on...\n\nwho the hell is writing the bill?'), ('dates', '2020-04-28 19:52:02'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.7144892811775208)])
OrderedDict([('tweet', "“From parroting the Chinese communist government's lies about COVID-19 to wrongly warning people against wearing masks, the WHO has badly mishandled its communications about the pandemic. The organization really needs to get its act together.”\n\n https://t.co/4zxks02bWr"), ('dates', '2020-04-28 12:57:27'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.7840529084205627)])
OrderedDict([('tweet', 'Why we haven’t hit the debt limit?\n\nLast October, congress voted to SUSPEND the debt limit until after the elections!\n\nI offered an amendment to rename that bill “A bill to kick the can down the road and for other purposes.” I couldn’t possibly have imagined those other purposes. https://t.co/7p5ccSRDLm'), ('dates', '2020-04-28 12:09:26'), ('file_name', 'Thomas Massie.csv'), ('pct_true', 0.7211255431175232)])
OrderedDict([('tweet', 'In February, I voted to approve a bipartisan resolution that would require @realDonaldTrump to seek congressional approval before our nation is dragged into another endless war in the Middle East.\n \nI believed that was a good idea then and I still do today.'), ('dates', '2020-05-07 18:27:32'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.5270066857337952)])
OrderedDict([('tweet', 'The families behind these devastating numbers are living a nightmare.\n\nWith the Senate back in session, our focus must be on getting relief to these Americans who find themselves in dire straits through no fault of their own. https://t.co/YrsF4nEzxF'), ('dates', '2020-05-07 14:33:17'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.570582389831543)])
OrderedDict([('tweet', 'He’s not even trying to hide it anymore. \n\nDuring a pandemic that has already killed more than 72,000 Americans, @realDonaldTrump says he’ll continue his push to rip health care away from millions. https://t.co/oFfKc7h4gk'), ('dates', '2020-05-06 20:41:03'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.8031681180000305)])
OrderedDict([('tweet', 'It comes as little surprise that @EPA’s Science Advisory Board still has major concerns with EPA\'s so-called "Strengthening Transparency in Regulatory Science rule."\n\nThis rule would result in the suppression of scientific research and the censorship of scientists and experts.'), ('dates', '2020-04-28 22:21:17'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.7045444846153259)])
OrderedDict([('tweet', "We can’t manage what we can’t measure.\n \nIt's why @BillCassidy and I are asking @HHSGov and @CDCgov to use existing reportable disease framework to track and log two things: #COVID19 infections and individuals who may be immune."), ('dates', '2020-04-22 16:54:43'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.6416062116622925)])
OrderedDict([('tweet', '.@SenatorTimScott, @SenatorMenendez, @SenThomTillis and I are urging the Chinese government to work with @StateDept to quickly address new trade requirements that are needlessly preventing life-saving medical equipment from reaching the U.S. as our nation fights #COVID19. https://t.co/patwwMsMLA'), ('dates', '2020-04-18 22:18:00'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.5284550786018372)])
OrderedDict([('tweet', 'Today’s report is devastating. In just 3 weeks, at least 17 million of our fellow Americans have lost their jobs through no fault of their own.\n \nBehind these numbers are our moms, dads, brothers, sisters and neighbors who find themselves in dire straits they never imagined. https://t.co/KDLIMGohPU'), ('dates', '2020-04-09 16:50:43'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.5450608730316162)])
OrderedDict([('tweet', 'This is dangerous.\n \nYou have to wonder: Even in the midst of a global pandemic, why is @realDonaldTrump so preoccupied with removing the independent watchdogs whose job it is to identify wrongdoing and root out corruption?\n \nhttps://t.co/APjLyx7mSU'), ('dates', '2020-04-07 17:33:28'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.8761425018310547)])
OrderedDict([('tweet', 'This is unacceptable.\n \nThe President of the United States continues to drag his feet on using his full wartime powers that would require our nation’s companies to produce the life-saving equipment those on the frontlines so badly need right now. https://t.co/mjDfvV4Y1T'), ('dates', '2020-04-03 17:48:04'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.8524751663208008)])
OrderedDict([('tweet', "Businesses need certainty and predictability to be successful. Regulatory uncertainty and litigation is the very outcome that U.S. automakers sought to avoid – and for America's automotive industry, that's the only thing this rule is guaranteed to deliver. https://t.co/ZqrfUuDT1d"), ('dates', '2020-03-31 20:48:29'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.7641850113868713)])
OrderedDict([('tweet', 'The Trump Administration has produced and today unveiled a fundamentally flawed rule that utterly fails to uphold any of the promises it has been making for the past three years. https://t.co/OELmBiAb9q'), ('dates', '2020-03-31 19:48:29'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.6135950088500977)])
OrderedDict([('tweet', "This is dangerous.\n\nGovernors are asking for more equipment because they need it — immediately. They're the ones hearing directly from experts and those on the frontlines who say they need more equipment.\n\nInstead of doubting them, work with governors to get them what they need. https://t.co/aiLFtteOQe"), ('dates', '2020-03-27 17:17:36'), ('file_name', 'Thomas R. Carper.csv'), ('pct_true', 0.7095109820365906)])
OrderedDict([('tweet', 'When the new #Coronavirus originated in Wuhan, the Chinese Communist Party covered up its severity and let it get out of control. This ignorance has thrown the world into chaos. We must hold the CCP accountable for the harm its caused the Chinese people and the global economy.'), ('dates', '2020-04-21 18:48:15'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.9927797913551331)])
OrderedDict([('tweet', 'Small businesses are hurting NOW, @SpeakerPelosi @SenSchumer...here in Tennessee and even in your home states. Your partisan political games are hurting my constituents AND yours. My colleagues &amp; I stand ready to support small businesses &amp; the people they employ NOW. #RefillPPP https://t.co/lJhS4HTnW2'), ('dates', '2020-04-18 15:04:29'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.9499512314796448)])
OrderedDict([('tweet', "Democrats' demands to attach mail-in voting and the Green New Deal to Paycheck Protection Program funds is hurting workers and small businesses everywhere. \n\nNow is not the time to cave to the far-left activists. Think of the workers and small businesses in your constituency."), ('dates', '2020-04-16 15:20:11'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.8713504076004028)])
OrderedDict([('tweet', 'Congressional Democrats delaying crucial Paycheck Protection Program funding only hurts workers and families in East Tennessee and across the country. Now is not the time to enact political agendas, small businesses and their employees must come first. https://t.co/WkbriB9bYa'), ('dates', '2020-04-09 18:12:54'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.5989806652069092)])
OrderedDict([('tweet', 'Heads up Social Security recipients: Beware of a scam letter threatening the suspension of your Social Security benefits due to the COVID-19 crisis. These letters are FAKE and @SocialSecurity has not suspended or discontinued benefits because of #COVID19. https://t.co/KbP8ksVUVm'), ('dates', '2020-03-25 19:34:27'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.6740686893463135)])
OrderedDict([('tweet', 'I’ve heard from East Tennessee workers, small businesses, independent contractors, and self-employed folks. They need help and they need it now. \n\n@SpeakerPelosi has put her political agenda and liberal wish-list ahead of the immediate needs of millions of Americans. Shameful.'), ('dates', '2020-03-24 21:15:13'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.523423433303833)])
OrderedDict([('tweet', 'Last week, Senators worked together on a bill that would reduce the economic fallout from #COVID19. \n\nToday, millions are wondering why this bipartisan bill was blocked. Twice. \n\nThis bill helps struggling Americans, but Speaker Pelosi blocked it in favor of her political agenda.'), ('dates', '2020-03-23 23:55:30'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.8289840221405029)])
OrderedDict([('tweet', 'Glad House Leadership did the right thing and put forward a Coronavirus response package free of partisan poison pills. Policy riders don’t have any business in a bill our country needs to protect public health. I voted yes. https://t.co/OgLrYZru89'), ('dates', '2020-03-04 22:13:59'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.5020470023155212)])
OrderedDict([('tweet', 'Rumor has it Speaker Pelosi and her leadership team plan to offer Coronavirus response funding and FISA re-authorization in one bill. This is ridiculous. Congress cannot afford to play politics with urgent public health matters. We need to vote on a clean Coronavirus bill. https://t.co/POtFsDgdPf'), ('dates', '2020-02-27 22:53:21'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.6722164750099182)])
OrderedDict([('tweet', "So...Castro's government-run literacy program gives him a free pass for the murder and imprisonment of his political opponents. Gotcha. \n\nEvery House Democrat needs to let their constituency know where they stand on this communist nonsense. Enough is enough. @HouseDemocrats https://t.co/9BoFor72aD"), ('dates', '2020-02-25 14:32:53'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.6703240275382996)])
OrderedDict([('tweet', "ICYMI: No federal budget should allow federal bureaucrats to abuse taxpayer dollars as their personal slush fund. I asked OMB Director @RussVought45 what the President's budget proposal does to eliminate wasteful, fraudulent spending during yesterday's @housebudgetGOP hearing. https://t.co/lAsxiD28d4"), ('dates', '2020-02-13 16:34:22'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.6033241152763367)])
OrderedDict([('tweet', "Achievable, reasonable priorities for America are clear in President Trump's budget. On the other hand, Democrats are either too afraid or unwilling to draft a budget explaining how much of your money they want to spend on their socialist, pie in the sky programs."), ('dates', '2020-02-12 16:45:43'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.9100866317749023)])
OrderedDict([('tweet', ".@RussVought45 testifying before @housebudgetGOP on President Trump's budget proposal. Democrats complain about the proposal, yet they will likely fail to produce their own budget. Are they nervous to reveal the true cost of their out of touch policies like the Green New Deal?"), ('dates', '2020-02-12 15:23:01'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.9799157381057739)])
OrderedDict([('tweet', 'The $4.67 billion disaster relief bill voted on by the House was misleading because the funds aren’t exclusively for Puerto Rico. Rather than a vote to throw more money at the problem, we should’ve started by managing the $8 billion in disaster funds we’ve already appropriated.'), ('dates', '2020-02-07 17:55:05'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.522908091545105)])
OrderedDict([('tweet', 'Our federal government responded to recent disasters in Puerto Rico by directing HUD to release over $8 billion in disaster aid. This was just a few weeks ago. We need to see if this relief is working before Congress votes to spend billions more.'), ('dates', '2020-02-07 15:24:50'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.7752327919006348)])
OrderedDict([('tweet', 'The Puerto Rican government did the right thing by firing these officials for stashing away much needed disaster aid, but before authorizing more funding, Congress needs assurance that this kind of negligence won’t happen again. https://t.co/EEHb9iG0Zt'), ('dates', '2020-02-07 14:46:14'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.834380567073822)])
OrderedDict([('tweet', "I believe in capitalism and the free market. Socialism is dangerous, and Venezuela's recent history makes that clear. Juan Guaido attending the State of the Union last night sent a powerful message that the United States will stand up to socialist tyranny around the world."), ('dates', '2020-02-05 20:53:07'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.9160995483398438)])
OrderedDict([('tweet', 'We now pause House business once again to vote on the "urgent", 28 day old, partisan impeachment articles. Meanwhile, we will not vote to stand up for the Iranian protesters basic right to free speech. \n\nThis Democrat majority\'s priorities are completely out of touch.'), ('dates', '2020-01-15 18:25:35'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.7971229553222656)])
OrderedDict([('tweet', "Why in the world are @HouseDemocrats blocking the @GOPLeader's resolution condemning this Iranian regime for obvious human rights violations? This isn't complicated. Let's stand up for the same basic right to free speech in Iran that we have here and vote on this resolution."), ('dates', '2020-01-14 21:49:46'), ('file_name', 'Tim Burchett.csv'), ('pct_true', 0.8651102185249329)])
OrderedDict([('tweet', 'First @RealDonaldTrump played favorites over which governors got supplies. Now he’s blocking the House from hearing from Dr. Fauci bc @HouseDemocrats are in the majority. No more political games, Mr. President. The American people need leadership &amp; action.\nhttps://t.co/XPshzyLDNo'), ('dates', '2020-05-05 13:46:00'), ('file_name', 'Tim Ryan.csv'), ('pct_true', 0.5637562274932861)])
OrderedDict([('tweet', 'Outrageous. We’ve lost more people to this than in Vietnam &amp; this President won’t allow a top official to testify before Congress so we can coordinate an integrated, whole-government response to save lives. I’m disgusted by this game show host President.\nhttps://t.co/VPPaGEttqi'), ('dates', '2020-05-02 03:20:43'), ('file_name', 'Tim Ryan.csv'), ('pct_true', 0.7899889349937439)])
OrderedDict([('tweet', 'Let’s get this straight. Mitch McConnell has no problem bailing out airlines and steakhouses, but he refuses to help local communities keep police officers and firefighters on the job… during a pandemic. \nhttps://t.co/bftmQAmqez'), ('dates', '2020-04-30 13:39:00'), ('file_name', 'Tim Ryan.csv'), ('pct_true', 0.5749009251594543)])
OrderedDict([('tweet', 'We’re not just combatting a virus, #COVID19 has also accelerated our country’s mental health crisis. My House &amp; Senate colleagues &amp; I are fighting to establish a 3-digit suicide helpline so folks get the help they need as quickly &amp; seamlessly as possible. https://t.co/E7gOBY5Ccz'), ('dates', '2020-04-28 13:26:00'), ('file_name', 'Tim Ryan.csv'), ('pct_true', 0.7038949728012085)])
OrderedDict([('tweet', 'Appalling that—during a worldwide pandemic that has taken the lives of tens of thousands of Americans &amp; put more than 20 million out of work —Congressional Republicans would slip in a measure to give $170 billion to millionaires &amp; billionaires. Shameful. https://t.co/wRQYJgU3Oz'), ('dates', '2020-04-20 16:24:00'), ('file_name', 'Tim Ryan.csv'), ('pct_true', 0.8781370520591736)])
OrderedDict([('tweet', 'Consumer spending is 70% of our GDP; how will small businesses survive if their customers don’t have money in their pockets? The old strategies have failed us. This is an unprecedented crisis, &amp; it is time for government to respond in an unprecedented way.\nhttps://t.co/1w3qKJyBfu'), ('dates', '2020-04-20 15:30:31'), ('file_name', 'Tim Ryan.csv'), ('pct_true', 0.5498762726783752)])
OrderedDict([('tweet', 'If we have money to bail out major corporations like the airlines, we have money to ensure men &amp; women who serve in police &amp; fire jobs can continue to serve our communities. Millions of Americans are hurting, I will not leave our local heroes out to dry. https://t.co/6AOyEVyV84'), ('dates', '2020-04-18 16:46:57'), ('file_name', 'Tim Ryan.csv'), ('pct_true', 0.6754353046417236)])
OrderedDict([('tweet', '🙅\u200d♂️ No cheating the American people out of their cash refunds. The #COVID19 crisis isn’t the time for opportunism, airlines have to follow the law.\nhttps://t.co/omomxjsdvk'), ('dates', '2020-04-09 13:20:00'), ('file_name', 'Tim Ryan.csv'), ('pct_true', 0.7994080781936646)])
OrderedDict([('tweet', 'In case you’re wondering, yes, Democrats are still blocking small business relief, and therefore paychecks for workers across the country. There’s a simple, one page bill that’s been ready to go for more than a week.'), ('dates', '2020-04-20 18:27:02'), ('file_name', 'Tim Scott.csv'), ('pct_true', 0.8370093107223511)])
OrderedDict([('tweet', '75 years ago, the forces of freedom prevailed and Nazi Germany’s surrender marked the end of WWII in Europe. Our nation will always remember the incredible sacrifices made by the Greatest Generation to secure this heroic victory. #VEDay75 https://t.co/4bzODklTAU'), ('dates', '2020-05-08 17:10:46'), ('file_name', 'Tim Walberg.csv'), ('pct_true', 0.9185093641281128)])
OrderedDict([('tweet', 'Workers and small businesses are left to wait another day as Speaker Pelosi continues to stand in the way of additional funding. We need to replenish the Paycheck Protection Program without delay. https://t.co/mF0XDRMvnP'), ('dates', '2020-04-20 23:13:56'), ('file_name', 'Tim Walberg.csv'), ('pct_true', 0.5812206864356995)])
OrderedDict([('tweet', "Michigan workers and small businesses are struggling and we need to act with urgency. People's jobs, health, and livelihoods are on the line. We don’t have time for political games."), ('dates', '2020-03-23 23:55:01'), ('file_name', 'Tim Walberg.csv'), ('pct_true', 0.7795939445495605)])
OrderedDict([('tweet', 'This is truly unbelievable. Rolling back Title IX rules (in the midst of a pandemic no less) is a reckless decision that is only going to end up hurting survivors of sexual assault and removing important protections for a fair process on college campuses across the country. https://t.co/07OAswGL4r'), ('dates', '2020-05-06 21:59:25'), ('file_name', 'Tina Smith.csv'), ('pct_true', 0.5221165418624878)])
OrderedDict([('tweet', 'Nope, not an accurate characterization, @WSJopinion. And good news! Trump heard the Democrats &amp; is moving in our direction, for the good of small business, hospitals, &amp; improved testing. Bipartisan compromise adds up to better legislation. Nice try though! https://t.co/TaFQnN9mDw'), ('dates', '2020-04-19 17:53:44'), ('file_name', 'Tina Smith.csv'), ('pct_true', 0.710830807685852)])
OrderedDict([('tweet', 'While we’ve been focused on finding COVID-19 solutions, the Trump Administration EPA has quietly rolled back rules on toxic air emissions even though the nation is suffering from a respiratory pandemic. https://t.co/JfQqf2eSXV'), ('dates', '2020-04-16 20:19:47'), ('file_name', 'Tina Smith.csv'), ('pct_true', 0.6911415457725525)])
OrderedDict([('tweet', 'Small businesses need our support now, yet Mitch McConnell is quietly trying to deny funding to non-profits who get Medicaid support. \n \nWhat does that mean? Nursing homes and providers of services for people with disabilities might not be given the funding they need to survive. https://t.co/Q7ewB8OPiY'), ('dates', '2020-03-23 20:48:57'), ('file_name', 'Tina Smith.csv'), ('pct_true', 0.8002604842185974)])
OrderedDict([('tweet', 'Ten years ago today @BarackObama signed the #ACA into law and helped millions of Americans gain access to health care. \n\nBecause of the ACA and programs like @MNsure, more Minnesotans are able to access testing and care during this global crisis. #ProtectOurCare'), ('dates', '2020-03-23 17:49:40'), ('file_name', 'Tina Smith.csv'), ('pct_true', 0.7993502020835876)])
OrderedDict([('tweet', 'No votes in the U.S. Senate today. It’s been over two days since the House passed legislation to support families during #COVIDー19. @Senatemajldr, we should have stayed in session to get it done. Even Trump supports this bill, we need action.'), ('dates', '2020-03-17 01:15:36'), ('file_name', 'Tina Smith.csv'), ('pct_true', 0.739386796951294)])
OrderedDict([('tweet', 'Wrote a letter to express unwavering support for @AusintheUS after recent backlash from Chinese Ambassador Cheng Jingye. No matter the external pressure or coercion, we will always have Australia’s back, just as Australia has always had ours. #Mateship https://t.co/3f2q5qzAbs'), ('dates', '2020-05-07 21:36:38'), ('file_name', 'Todd Young.csv'), ('pct_true', 0.5129400491714478)])
OrderedDict([('tweet', "It was great speaking with Ambassador @A_Sinodinos from @AusintheUS about our shared interest in investigating the Chinese Communist Party's role in the #coronavirus pandemic. As China threatens Australia with retaliation, allies must stand together to push for answers."), ('dates', '2020-04-28 20:00:00'), ('file_name', 'Todd Young.csv'), ('pct_true', 0.8352152705192566)])
OrderedDict([('tweet', 'I agree with @realDonaldTrump that we should refrain from bailing out fiscally irresponsible cities and states. That’s what my Government Bailout Prevention Act proposes: https://t.co/UJj1FSmj6Z https://t.co/wDQmuSMk6c'), ('dates', '2020-04-27 20:48:07'), ('file_name', 'Todd Young.csv'), ('pct_true', 0.5707018375396729)])
OrderedDict([('tweet', 'China is threatening Australia with economic pain if they don’t end their investigation into China’s responsibility for the #coronavirus. This bullying is a chilling window into what a CCP-led world looks like.\n\nIf the CCP have nothing to hide, why make this threat? https://t.co/OWrffvpT62'), ('dates', '2020-04-27 15:41:10'), ('file_name', 'Todd Young.csv'), ('pct_true', 0.9858832955360413)])
OrderedDict([('tweet', 'Need more evidence that China is trying to use the #coronavirus pandemic to advance their agenda? Look no further than Hong Kong where those fighting for democracy are being rounded up while the world is distracted by this pandemic. Where will this end? https://t.co/CifqG4Klgp'), ('dates', '2020-04-18 18:49:19'), ('file_name', 'Todd Young.csv'), ('pct_true', 0.8341199159622192)])
OrderedDict([('tweet', 'This is a deeply troubling development. State Department cables warned of a lab in Wuhan, China, doing experiments on #coronavirus back in 2018. Chinese Communist Party officials must provide answers. https://t.co/RI8fGvJhPN'), ('dates', '2020-04-14 15:26:59'), ('file_name', 'Todd Young.csv'), ('pct_true', 0.8253500461578369)])
OrderedDict([('tweet', 'Why Blame the Chinese Communist Party? Wet markets. Poor regulation. Coverups. Global Disinformation campaigns. https://t.co/7gkSgVq9Pq'), ('dates', '2020-04-09 18:37:43'), ('file_name', 'Todd Young.csv'), ('pct_true', 0.966571569442749)])
OrderedDict([('tweet', "It's unconscionable that Speaker Pelosi &amp; Leader Schumer are holding up replenishing the PPP aid Congress promised to #SmallBiz. Unless Democrats change course immediately, it is going to cost millions their jobs &amp; thousands their businesses. https://t.co/9SGqZ3kCtt"), ('dates', '2020-04-17 22:01:21'), ('file_name', 'Tom Cole.csv'), ('pct_true', 0.932805597782135)])
OrderedDict([('tweet', "The American people will neither forget nor forgive a party that tried to take advantage of their suffering to advance a left-wing political agenda. \n\nMy full statement on @SpeakerPelosi's transparently partisan, left-wing #coronavirus bill ⬇️\nhttps://t.co/ucx09FG061"), ('dates', '2020-03-23 22:24:56'), ('file_name', 'Tom Cole.csv'), ('pct_true', 0.9059057831764221)])
OrderedDict([('tweet', '75 years ago, Allied forces reclaimed a continent from Nazi tyranny. \n\nToday, communities across America and Europe remember the troops who achieved that victory at great cost so we can live in freedom.'), ('dates', '2020-05-08 13:25:25'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.5796059370040894)])
OrderedDict([('tweet', 'U.S. Marshals face the same risks as their FBI or DEA counterparts when serving abroad, but they aren’t eligible for the same pay. \n\nOur bill would ensure that USMS employees receive appropriate compensation for the dangers they face serving our nation.\n\n https://t.co/Ec8EPDn9AM'), ('dates', '2020-05-07 21:02:55'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.5884353518486023)])
OrderedDict([('tweet', "If the Chinese Communist Party is confident about the origin of this virus then I'm sure they would welcome an investigation into those origins... https://t.co/ZEcKBy0dvH"), ('dates', '2020-05-07 11:13:38'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.7882106900215149)])
OrderedDict([('tweet', 'Everyone knows the Chinese Internet is censored and unrepresentative of the Chinese people—but the media can’t pass up an opportunity to “roast” the president, even if it means publishing CCP propaganda. Shameful. https://t.co/19ce4KUYcY'), ('dates', '2020-05-05 17:51:05'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9298429489135742)])
OrderedDict([('tweet', 'The pandemic has laid bare our dependence on a hostile power, Communist China, for even basic supplies like respirator masks and medicine. \n\nWe must earn our independence, just like our Founding Fathers did before us. Read:\n\nhttps://t.co/3LIS0Bts3X'), ('dates', '2020-05-05 14:37:29'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9067994952201843)])
OrderedDict([('tweet', 'The Chinese people are the first, and worst, victims of the Chinese Communist Party both in this pandemic and going back 70+ years. ICYMI: I joined @marthamaccallum earlier tonight: https://t.co/rxjCDcJEgn'), ('dates', '2020-05-05 00:03:53'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.8685559630393982)])
OrderedDict([('tweet', 'The United States must prepare for competition with Communist China as fierce as our contest with the Soviet Union. We’ll need strategic investment and private enterprise to win, just as we did in the past. My essay for @AmerCompass:\nhttps://t.co/3LIS0Bts3X'), ('dates', '2020-05-04 23:44:51'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.5950196385383606)])
OrderedDict([('tweet', 'China unleashed this plague in the first place, but don’t expect the media to hold it accountable. It’s too busy repackaging CCP propaganda.\n\nhttps://t.co/zL9DbJLBQe'), ('dates', '2020-05-04 22:31:03'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9427515268325806)])
OrderedDict([('tweet', 'The Chinese Communist Party is arming itself with technology acquired from our labs and research universities. We need to reconsider granting Chinese nationals visas to study dual-use science and technology, especially STEM students at the graduate level. https://t.co/nUCGPb98EZ'), ('dates', '2020-04-27 15:58:31'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.8743319511413574)])
OrderedDict([('tweet', 'The Chinese Communist leadership did not want to see their relative power and standing in the world decline because this virus was contained within China so they deliberately hid it from the world. MORE: https://t.co/GGLdmJaFT6'), ('dates', '2020-04-26 15:15:18'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9593902230262756)])
OrderedDict([('tweet', 'China will try to exploit the world’s weakness in the wake of a virus it unleashed. \n\nWe cannot allow it to succeed. \n\nThe FORCE Act will greatly strengthen the U.S. position in the Indo-Pacific, allowing us to block China’s goals of regional dominance.\n\n https://t.co/bAy4cOkEGg'), ('dates', '2020-04-22 17:22:14'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9432542324066162)])
OrderedDict([('tweet', 'China continues to lie and spread disinformation to the civilized world to coverup its own incompetence, corruption, and dishonesty. https://t.co/5hvWQKNrWd'), ('dates', '2020-04-22 14:05:38'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.7162469625473022)])
OrderedDict([('tweet', 'We all want clean water, but the Obama-era WOTUS rule drastically expanded federal control over private land owners and ag producers. The Navigable Water Protection Rule will secure our nation’s clean water supply without overextending federal power. \n\nhttps://t.co/wh7e0A2nfH'), ('dates', '2020-04-21 22:28:24'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.706387460231781)])
OrderedDict([('tweet', 'No matter what the Chinese Communist Party says, given the mounting evidence, the most likely origins for the China virus are the Wuhan labs studying bats and coronavirus. \n\nhttps://t.co/nWMKiO2190'), ('dates', '2020-04-21 16:58:10'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9111016988754272)])
OrderedDict([('tweet', 'More spin from the China apologists at @WHO. \n\nWe know the Wuhan labs researched animals &amp; coronaviruses. A lab breach is the most plausible explanation of the virus origin. \n\n@WHO misdirection purports to but doesn’t address that scenario.\n\nhttps://t.co/KBcEpsEbHD'), ('dates', '2020-04-21 12:07:10'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.5112736225128174)])
OrderedDict([('tweet', '22 million Americans have lost their jobs in the last month because of the China virus. Let’s help them get back to work before we import more foreigners to compete for their jobs. https://t.co/0a03nncxeC'), ('dates', '2020-04-21 02:31:36'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.5067118406295776)])
OrderedDict([('tweet', 'From their handling of the Wuhan virus to arresting Hong Kong’s pro-democracy leaders, the Chinese Communist Party remains—and should be treated as—a threat to free people everywhere.\n\nhttps://t.co/mHrV2Uc4jT'), ('dates', '2020-04-20 14:12:20'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.742816150188446)])
OrderedDict([('tweet', "The Chinese Communist Party's decision to cover up the virus led to thousands of needless deaths and untold economic harm. \n\nIt’s only appropriate that we hold the Chinese government accountable for the damage it has caused. \n\nhttps://t.co/vVhT3uksqF"), ('dates', '2020-04-16 17:03:46'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9658800959587097)])
OrderedDict([('tweet', 'China is withholding vital medical supplies from us—supplies made by *American companies* in China. In the middle of a pandemic unleashed on the world by China.\n\nChina must pay. https://t.co/AhOaQ1TniE'), ('dates', '2020-04-16 12:12:57'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.6283468008041382)])
OrderedDict([('tweet', 'The Chinese Communist Party is responsible for every single death, every job lost, every retirement nest egg lost, from this coronavirus. And Xi Jinping must be made to pay the price. https://t.co/OLCj5Z5rrp'), ('dates', '2020-04-16 01:46:24'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.7601378560066223)])
OrderedDict([('tweet', "The Chinese Communist Party's conducting covert nuclear tests is yet another reason to unsign the Comprehensive Nuclear-Test-Ban Treaty. https://t.co/xH7SAedwn7"), ('dates', '2020-04-15 20:40:27'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9095759391784668)])
OrderedDict([('tweet', 'The Chinese Communist Party’s lack of transparency caused this pandemic, and now they’re holding important medical supplies hostage to extort praise. \n\nCriminal. \n\nhttps://t.co/4GxD1L7z3u'), ('dates', '2020-04-15 13:10:47'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9468636512756348)])
OrderedDict([('tweet', 'Beijing claims a small Vietnamese fishing boat rammed one of its coast-guard ships in the South China Sea, rather than the other way around. It’s hard to imagine a more ludicrous story. The U.S. stands with Vietnam as it defends its citizens and sovereignty from CCP aggression.'), ('dates', '2020-04-11 17:00:54'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.5049023628234863)])
OrderedDict([('tweet', '.@Max_Fisher claims—without evidence—that I believe the China virus was “produced by a Chinese weapons lab.” He should have done his homework. https://t.co/smzBE7ypdd'), ('dates', '2020-04-08 21:20:18'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.8264504671096802)])
OrderedDict([('tweet', 'Sadly, it seems Zoom is yet another American company that has sacrificed the security and privacy of Americans to China in order to save a buck. https://t.co/9uLEjB5aWz'), ('dates', '2020-04-08 16:50:15'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.7626910209655762)])
OrderedDict([('tweet', 'The Butcher of Damascus &amp; his Moscow backers think the world forgot his barbaric slaughter of civilians using chemical weapons, but his deeds are exposed &amp; his infamy will follow him to the grave. \n\nCivilized nations should punish Assad for his crimes. https://t.co/vGyMIFZi1m'), ('dates', '2020-04-08 15:26:12'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.5208033919334412)])
OrderedDict([('tweet', 'China’s fraud extends well beyond its Wuhan virus coverups. \n\nMy EQUITABLE Act would have protected Americans from Chinese deception by requiring all Chinese companies traded on American exchanges to follow the same US regulations as every other nation. \n\nhttps://t.co/sDzWbRiUr5'), ('dates', '2020-04-02 18:06:42'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9264986515045166)])
OrderedDict([('tweet', 'The IRS is forcing people on social security to file a tax return before they receive China virus relief.                   \n\nExtremely disappointed to see the IRS directly contradicting the CARES Act, blocking aid to our seniors.'), ('dates', '2020-04-01 17:52:14'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.5455387234687805)])
OrderedDict([('tweet', 'A *single mortuary* in Wuhan reportedly ordered more urns in two days than the Chinese Communist Party has reported total deaths in the whole country. \n\nI’m sure you’re shocked by evidence of Chinese lies. https://t.co/FUOhZ5JYpr'), ('dates', '2020-03-29 15:51:11'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.5440313220024109)])
OrderedDict([('tweet', 'China just shut down movie theaters nationwide *after* reopening them. \n\nSound to you like they have pandemic under control? Me neither.\n\nChina is still lying. https://t.co/dDMsyZNBVZ'), ('dates', '2020-03-29 13:11:24'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.8999286890029907)])
OrderedDict([('tweet', 'Will you get China virus relief?\n\nMost people will get a direct deposit in the next three weeks, if the IRS has your bank account info. The rest will be mailed a check. No one has to apply. \n\nDetails here:\nhttps://t.co/HzQHJVhPjZ'), ('dates', '2020-03-26 19:00:30'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.6103337407112122)])
OrderedDict([('tweet', 'Maybe @SenSchumer meant he was on his own 2-yard line, with 98 yards to go?\n\nThe Democrats need to abandon their ideological wish list &amp; join us to save American lives &amp; jobs. https://t.co/ixHomVQaVl'), ('dates', '2020-03-25 01:38:54'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.8518549203872681)])
OrderedDict([('tweet', 'Let me translate from Chuck-speak: “There’s no deal because Democrats are *still* putting a liberal wish list over American lives &amp; jobs, but I want you to blame McConnell.” https://t.co/ZG5KTZIkTg'), ('dates', '2020-03-24 16:31:10'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.8834050893783569)])
OrderedDict([('tweet', 'This is the stark truth: we have to arrest the spread of the China virus to get the economy back on its feet &amp; get life back to something like normal. https://t.co/KNUTXNIpnL'), ('dates', '2020-03-24 13:24:21'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.7373742461204529)])
OrderedDict([('tweet', '.@SenSchumer has walked the Democrats into a box canyon &amp; @SpeakerPelosi wants an ideological wish list totally unrelated to the pandemic. Meanwhile, Americans are dying, losing their jobs, &amp; seeing their retirement savings evaporate. Let me suggest some grounds for compromise:'), ('dates', '2020-03-23 22:20:06'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9959635734558105)])
OrderedDict([('tweet', 'Families and businesses need help now to survive the China virus pandemic. But @SpeakerPelosi walked away from negotiations to write her own bill, full of absurd provisions completely unrelated to the crisis at hand. Here’s what Speaker Pelosi is demanding while Americans suffer:'), ('dates', '2020-03-23 19:21:57'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9913580417633057)])
OrderedDict([('tweet', 'This report is correct. Pelosi &amp; Schumer are willing to risk your life, your job, your retirement savings for a radical, left-wing wish list that has nothing to do with this virus. Disgraceful. https://t.co/le9jlTB4KH'), ('dates', '2020-03-23 14:48:17'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.7622522115707397)])
OrderedDict([('tweet', 'Chinese Communist Party lies &amp; corruption turned a local health problem into a global pandemic, devastating lives &amp; dreams across our country. There will be a reckoning. https://t.co/CPlEohA5ez'), ('dates', '2020-03-22 13:45:30'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9731876254081726)])
OrderedDict([('tweet', 'The key word is “reported” ... don’t fall for Chinese Communist Party propaganda. Their lies inflicted this plague on the world. https://t.co/GupYyffopP'), ('dates', '2020-03-20 14:19:50'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.7114678025245667)])
OrderedDict([('tweet', '.@USTreasury made the right call in delaying tax payments 90 days while American families grapple with this crisis.\n\nWe should extend the filing deadline 90 days as well.\n\nFolks getting a refund should still file ASAP.'), ('dates', '2020-03-19 14:21:03'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.5192093253135681)])
OrderedDict([('tweet', "The Chinese Communist Party has threatened to cut off America's access to vital drugs in the midst of a pandemic caused by its own failures. It's time to pull America's supply chains for life-saving medicine out of China. https://t.co/XrBorsdynQ"), ('dates', '2020-03-19 00:43:03'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9442453980445862)])
OrderedDict([('tweet', 'The Chinese Communist Party deserves every bit of the blame it’s getting for its malicious response to the outbreak. Their own citizens—and citizens around the world—are the true victims of their crimes. We will hold them accountable. https://t.co/Y51zFKsOlD'), ('dates', '2020-03-18 15:53:30'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9537485837936401)])
OrderedDict([('tweet', 'The Chinese Communist Party stays in power by restricting access to information. It’s no surprise that a free press—particularly one reporting on China’s terrible response to the Wuhan coronavirus—has threatened CCP enough to be expelled from their shores. https://t.co/8CCAPJRLos'), ('dates', '2020-03-17 17:44:27'), ('file_name', 'Tom Cotton.csv'), ('pct_true', 0.9594538807868958)])
OrderedDict([('tweet', 'We have a cybercrime crisis in this country. For too long, Americans have been victims of a lopsided cyber battlefield that allows hackers to act without consequence. \n\n@RepJoshG &amp; I teamed up to do something about it. @FoxBusiness https://t.co/1nohWDvBOw'), ('dates', '2020-03-03 14:55:43'), ('file_name', 'Tom Graves.csv'), ('pct_true', 0.657752275466919)])
OrderedDict([('tweet', '"As a result of our unprecedented efforts, illegal crossings are down 75% since May, dropping 8 straight months in a row.  And as the wall goes up, drug seizures rise, and border crossings go down." #SOTU2020'), ('dates', '2020-02-05 03:06:23'), ('file_name', 'Tom Graves.csv'), ('pct_true', 0.6386873722076416)])
OrderedDict([('tweet', '"Whether we are Republican, Democrat, or Independent, surely we must all agree that every human life is a sacred gift from God." #SOTU2020'), ('dates', '2020-02-05 02:57:14'), ('file_name', 'Tom Graves.csv'), ('pct_true', 0.6730090379714966)])
OrderedDict([('tweet', '“America’s enemies are on the run. America’s fortunes are on the rise and America’s future is blazing bright.” #SOTU2020'), ('dates', '2020-02-05 02:10:19'), ('file_name', 'Tom Graves.csv'), ('pct_true', 0.9900043606758118)])
OrderedDict([('tweet', 'Dems continue their attempts to curb #POTUS ability to protect #US. We shld stand with our Commander in Chief and against Iranian aggression.  \n\nhttps://t.co/OSa4aaYSmW'), ('dates', '2020-01-30 19:59:39'), ('file_name', 'Tom Graves.csv'), ('pct_true', 0.8190097808837891)])
OrderedDict([('tweet', 'On #HolocaustRemembranceDay #WeRemember the innocent lives lost &amp; the generations of families affected by persecution &amp; evil. We must #NeverForget this dark time in history &amp; stand united in the fight against anti-Semitism.'), ('dates', '2020-01-27 17:51:18'), ('file_name', 'Tom Graves.csv'), ('pct_true', 0.8955875635147095)])
OrderedDict([('tweet', 'Yesterday, my friend @RepAdamSchiff quoted what I told him when he asked how we might answer those who wonder how America could allow such corruption. "This proceeding is our answer."\n\nThe Senate may be failing. But America will not. -TM https://t.co/KYTndftHku'), ('dates', '2020-01-31 19:32:08'), ('file_name', 'Tom Malinowski.csv'), ('pct_true', 0.5131797194480896)])
OrderedDict([('tweet', 'Tom joined @OversightDems to discuss why those who hold anti-Semitic views are feeling increasingly emboldened to act on their hate. https://t.co/bRB4d0IT5V'), ('dates', '2020-01-29 23:05:19'), ('file_name', 'Tom Malinowski.csv'), ('pct_true', 0.7071094512939453)])
OrderedDict([('tweet', 'Violence motivated by hate is on the rise in America, and New Jersey has seen more than our share. I will keep doing everything in my power to ensure the federal government has the right strategy and enough resources to help us fight back -TM https://t.co/fVWfwublOJ'), ('dates', '2020-01-23 01:16:24'), ('file_name', 'Tom Malinowski.csv'), ('pct_true', 0.6265550851821899)])
OrderedDict([('tweet', '"The Senate restored the vision of the American founders: that impeachment is only to be used for “treason, bribery, or other High Crimes and Misdemeanors,” and not as a partisan weapon to settle political differences..." Rep. McClintock on @KAHIradio https://t.co/fbLdODXiKF'), ('dates', '2020-02-06 18:38:34'), ('file_name', 'Tom McClintock.csv'), ('pct_true', 0.6378266215324402)])
OrderedDict([('tweet', 'Now we know just how reckless is the Democrats’ chant of “Resist by ANY means necessary.” This is a stunning abuse of power and a shameless travesty of justice...https://t.co/HB08bknTRf https://t.co/SWntXLuWmA'), ('dates', '2019-12-19 00:24:41'), ('file_name', 'Tom McClintock.csv'), ('pct_true', 0.7220160365104675)])
OrderedDict([('tweet', 'If this can become the new standard of impeachment-that Congress can impeach any President for any action whose motives opponents question...the separation of powers at the heart of our Constitution will have been severely damaged if not utterly destroyed https://t.co/SDqk8edEOQ https://t.co/ltUPdZbiTf'), ('dates', '2019-12-13 18:56:36'), ('file_name', 'Tom McClintock.csv'), ('pct_true', 0.7189589738845825)])
OrderedDict([('tweet', "The second article of impeachment says, “We're not willing to go to court. We'll take the law into our own hands.” These are the same people who tell us that no one is above the law, except, of course, for themselves. https://t.co/IcdLxu6NCg https://t.co/SH3ckT41Hh"), ('dates', '2019-12-13 16:12:03'), ('file_name', 'Tom McClintock.csv'), ('pct_true', 0.6362679600715637)])
OrderedDict([('tweet', 'This bill purports to assert a very important constitutional right: the right to trial by jury in civil actions.  But it does this by denying another very important constitutional right – the freedom of unimpaired contract...\nhttps://t.co/EqAqRBaAtv'), ('dates', '2019-09-20 16:34:31'), ('file_name', 'Tom McClintock.csv'), ('pct_true', 0.5890582203865051)])
OrderedDict([('tweet', 'Without federal government investment &amp; oversight, we will likely prolong this crisis and place significant strains on our health system, our economy, &amp; our way of life. \n\nWe need a clear federal approach to deploying COVID-19 testing. #AZ01 https://t.co/hJSmW49jdL'), ('dates', '2020-05-07 23:45:00'), ('file_name', 'Tom O’Halleran.csv'), ('pct_true', 0.5507478713989258)])
OrderedDict([('tweet', 'Uranium mining threatens the health of families, water, &amp; #AZ01 tourism. When we reopen our economies, we must prioritize areas like Grand Canyon that bring in millions of yearly visitors &amp; immense revenue, not open them up to dangerous contamination. https://t.co/9bizhg6NEE'), ('dates', '2020-04-29 01:00:00'), ('file_name', 'Tom O’Halleran.csv'), ('pct_true', 0.7001155614852905)])
OrderedDict([('tweet', 'We are hearing from businesses and lenders about the need to replenish #PaycheckProtectionProgram . Congress needs to act immediately to support this program. Small businesses and their employees are in a day by day struggle to survive while the Speaker blocks critical funding.'), ('dates', '2020-04-16 14:41:43'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.8995310068130493)])
OrderedDict([('tweet', 'With all that is happening in the world, extremists still push their agenda to halt free speech on college campuses. We are watching how a campus lecture from @benshapiro , already budgeted by the Student Assoc, will be handled by @SyracuseU. Free Speech must be protected!'), ('dates', '2020-04-13 20:07:10'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.6153045296669006)])
OrderedDict([('tweet', "We stand together opposing the Governor's very dangerous and reckless action of taking our ventilators. He is leaving our communities in a terrible position which will cost lives."), ('dates', '2020-04-03 16:18:15'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.789272665977478)])
OrderedDict([('tweet', 'We care about keeping people safe and ensuring people have fair access to testing. This is why the Problem Solvers Caucus was proud to push Congress to pass a nonpartisan, bipartisan $8.3 billion emergency funding bill to fight the coronavirus last week. (1/3)'), ('dates', '2020-03-14 10:24:31'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.6936440467834473)])
OrderedDict([('tweet', 'Ridiculous! Why is @SyracuseU blocking @seanspicer from speaking while liberal speakers are welcomed with open arms? Very unfair.\n\nWe must fight to ensure our precious 1st Amendment rights are cared for. Discrimination should not occur because of politics. https://t.co/4NNJZ32PAo'), ('dates', '2020-03-02 22:34:15'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.513540506362915)])
OrderedDict([('tweet', 'Many House Democrats are pushing a drug pricing plan which would HALT innovation for new cures. \n\nWith a vaccine for Coronavirus desperately needed, isn’t it foolish for those on the other side of the aisle to go down this path? https://t.co/aqu6fq4Wl2'), ('dates', '2020-02-27 17:52:16'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.7381940484046936)])
OrderedDict([('tweet', 'We must hold bad actors in the end of life caregiver community accountable, and I am proud to see my legislation to demand additional oversight and transparency of Medicare hospice providers advance toward the House floor. https://t.co/77dYZMNRs2'), ('dates', '2020-02-13 19:05:32'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.5553067922592163)])
OrderedDict([('tweet', '"29 days ago, a criminal alien freed by the Sanctuary City of New York was charged with the brutal rape and murder of a 92-year-old woman.  The killer had been previously arrested for assault, but under New York\'s sanctuary policies, he was set free." - @realDonaldTrump #SOTU'), ('dates', '2020-02-05 03:04:41'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.7197042107582092)])
OrderedDict([('tweet', 'Under @realDonaldTrump leadership hardworking people in New York have seen 347 thousand jobs created and 156 thousand people have been lifted out of poverty. #SOTU2020 #SOTU'), ('dates', '2020-02-05 02:18:45'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.7263606786727905)])
OrderedDict([('tweet', 'Under President Trump’s economy, previously left behind Americans are seeing a 💥BLUE COLLAR BOOM💥:\n\n✅ The lowest-income workers have seen the fastest pay increase (16 PERCENT)\n\n✅ 7 MILLION NEW JOBS AND 500,000 NEW MANUFACTURING JOBS\n\n✅ 12,000 new factories\n\n#SOTU2020 #SOTU'), ('dates', '2020-02-05 02:14:59'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.8899567127227783)])
OrderedDict([('tweet', 'One party extremist control in Albany is hurting New York. @NYGovCuomo now wants to PERMANENTLY ban fracking.\n\nOn top of that, SUNY Binghamton still has not made a firm commitment to protect free speech on campus by inviting Art Laffer back to speak! What is our state coming to? https://t.co/4VbUTpSjBB'), ('dates', '2020-01-24 16:10:29'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.8707048892974854)])
OrderedDict([('tweet', 'I am disappointed in @SenSchumer vote against USMCA. This GREAT trade deal will help our dairy farmers and manufacturers in New York. There is no explanation for him to go thumbs down on this other than him being railroaded by those in the extreme left of the Democratic Party.'), ('dates', '2020-01-16 18:09:11'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.6636582016944885)])
OrderedDict([('tweet', '"It’s clear. Substantively, the grounds for impeachment are not there. The Republicans are sticking tight. What I’m hearing, more and more, is the number of Democrats who are in a serious quagmire as to how they’re going to vote.." https://t.co/x2CiWi3jhE'), ('dates', '2019-12-17 14:09:48'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.9174648523330688)])
OrderedDict([('tweet', 'This shouldn’t be a partisan issue. President Obama signed this agreement. The House reaffirmed our commitment to Israel’s defense through inconditional ironclad aid. We should not let extremist politics blind us from the fact that Israel is our strongest ally in the Middle East. https://t.co/8ofi62P0F0'), ('dates', '2019-12-06 21:12:06'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.7299408912658691)])
OrderedDict([('tweet', 'The American people are fed up with impeachment. They want Congress to do its job and get to work to solve problems for hardworking people. \n\nIt is time for @SpeakerPelosi to put USMCA and legislation to lower drug prices on the floor for a vote! https://t.co/jvtjBa6CeG'), ('dates', '2019-11-19 19:53:23'), ('file_name', 'Tom Reed.csv'), ('pct_true', 0.8393553495407104)])
OrderedDict([('tweet', 'The Chinese government cost Americans 10 million jobs in just two weeks. \n\nThey must be held accountable for their lack of transparency, secrecy, and propaganda against the US military.'), ('dates', '2020-04-03 12:37:05'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.8634737133979797)])
OrderedDict([('tweet', 'American citizens are looking to @SpeakerPelosi, as a leader, for help. \n\nSpeaker Pelosi is withholding aide to Americans, while prioritizing wind and solar tax credits, and federal takeover of local elections. \n\nIt’s time for the games to end. Americans deserve better.'), ('dates', '2020-03-24 16:53:08'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.787818968296051)])
OrderedDict([('tweet', 'Why is gas still $2 per gallon?  \n\nOil has dropped from $52 to $22 per barrel in the last 30 days. Gas should be $1 per gallon, not $2!\n\nPeople are losing their jobs.  People are worried about financial survival.  Why are the oil companies gouging us?!?'), ('dates', '2020-03-22 22:38:37'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.8615508675575256)])
OrderedDict([('tweet', 'The House just passed $7.8 billion for emergency #Coronavirus spending. \n\nI’m happy that Democrats finally put the partisan politics aside, so we could put the health and safety of Americans first.'), ('dates', '2020-03-04 22:06:21'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.6210196614265442)])
OrderedDict([('tweet', 'It’s unconscionable that House Democrats have blocked a bill that would end Infanticide 80+ times. This should not be a partisan issue. \n\nI urge my colleagues in the Senate to stand with @realdonaldtrump in protecting the life of unborn children. #BornAliveAct'), ('dates', '2020-02-25 19:22:23'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.9656578302383423)])
OrderedDict([('tweet', '1/2: For the past five months, Democrats have wasted American tax dollars and time in their attempt to overturn the 2016 election. The President has a responsibility to make sure our foreign aid is not lost to corruption.'), ('dates', '2020-02-05 22:35:10'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.9109600782394409)])
OrderedDict([('tweet', 'Despite all the constant negativity in the media, @realDonaldTrump and Republican policies are putting the OPPORTUNITY back into this LAND OF OPPORTUNITY!\n\nLet’s build on our positive momentum in 2020!\n\nhttps://t.co/Gdzh5xC7i6'), ('dates', '2020-01-01 17:14:44'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.6125802993774414)])
OrderedDict([('tweet', 'America’s economy is strong! Today’s report shows that the economy continues to flourish under the new tax code. Just imagine what we could do if the Democrats stopped focusing on impeachment and worked with us to pass #USMCA and further the U.S. economy. https://t.co/n6cZnPfqkY'), ('dates', '2019-12-06 15:30:57'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.5417391657829285)])
OrderedDict([('tweet', 'It’s been one year since @realDonaldTrump and leaders from Canada and Mexico signed #USMCA. Speaker Pelosi has refused to bring it for a vote. She is more focused on hurting @realDonaldTrump than boosting our economy or creating American jobs.\nhttps://t.co/mj59hO7tBj'), ('dates', '2019-12-01 17:54:23'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.534989595413208)])
OrderedDict([('tweet', 'I voted NO on the sham impeachment inquiry. @HouseDemocrats don’t accept the results of the 2016 election and are blinded by hatred of @realDonaldTrump. It’s time for @SpeakerPelosi and the Dems to put country before party. Let’s work for the American People. They deserve it.'), ('dates', '2019-11-02 14:00:00'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.7685336470603943)])
OrderedDict([('tweet', 'Unemployment is at record lows. 1,064,243 jobs hang in the balance as long as #USMCA is not ratified. It would easily pass Congress but Democrats are too busy trying to impeach @realDonaldTrump. It’s time for @SpeakerPelosi to put country before party: it’s time to pass #USMCA!'), ('dates', '2019-10-30 21:29:00'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.8451828360557556)])
OrderedDict([('tweet', '. @realdonaldtrump and pro-growth Republican policies have supercharged our economy and all Americans are benefiting. Thank you President Trump! https://t.co/up6iOQ42Fg'), ('dates', '2019-10-30 19:13:53'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.5302619934082031)])
OrderedDict([('tweet', 'The impeachment process started the day after the election, and it’s time for the Democrats to put the American people before their political interests.'), ('dates', '2019-10-09 19:16:45'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.6769614219665527)])
OrderedDict([('tweet', 'It has been 313 days since the President signed the USMCA with Mexico and Canada. And yet still no vote. \n\nInstead, the Democrats continue to try to undo the 2016 election. They struck out with the Kavanaugh hearings and they struck out with the Mueller Investigation.'), ('dates', '2019-10-09 19:16:36'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.8288331031799316)])
OrderedDict([('tweet', 'President Trump has the bases loaded with record low unemployment rates, 2% GDP growth and a competitive economy once again. \n\nHowever, Speaker Pelosi is denying the American people the grand slam they deserve by not bringing USMCA to the House floor for a vote.'), ('dates', '2019-10-09 19:15:45'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.5648244619369507)])
OrderedDict([('tweet', 'As Dems continue their political warfare, @realDonaldTrump secured a trade deal with Japan today that will open new markets for $7 billion in U.S. agriculture products &amp; will set new digital trade rules. https://t.co/EBx4hUE0SZ'), ('dates', '2019-09-25 23:02:01'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.7068531513214111)])
OrderedDict([('tweet', 'It’s disappointing but not surprising that the Democrats have launched an impeachment inquiry based on an unverified second-hand account, rather than waiting 24 hours for President Trump’s voluntary release of the full transcript of the phone call.'), ('dates', '2019-09-24 22:24:34'), ('file_name', 'Tom Rice.csv'), ('pct_true', 0.9600788354873657)])
OrderedDict([('tweet', 'I am deeply disappointed that @POTUS vetoed our resolution to prevent unconstitutional war w/ Iran, and that the Senate did not override his veto to protect the Constitution.\n \nBut there is bipartisan momentum behind this effort. The American people do not want a war with Iran.'), ('dates', '2020-05-07 19:15:52'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.6918078064918518)])
OrderedDict([('tweet', 'Our resolution to stop unconstitutional war with Iran isn’t about an election—it is about standing up for the Constitution. That’s why Democrats AND Republicans voted to pass it. https://t.co/L41QJjsE8p'), ('dates', '2020-05-06 22:04:41'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.8668627142906189)])
OrderedDict([('tweet', ".@Interior MUST extend the public comment period for proposed changes to the management of Chaco Canyon's resources. We are in the midst of a pandemic, and tribes, pueblos, and communities directly impacted need a genuine opportunity to participate. https://t.co/5uWLMZLrbY"), ('dates', '2020-05-01 20:15:35'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.5271324515342712)])
OrderedDict([('tweet', 'We are urging @Interior not to allow oil and gas companies to stop paying the royalties they owe to states like New Mexico. Such a move won’t help workers, and these royalties go directly to essential public services during a pandemic. https://t.co/Q9VguclR65'), ('dates', '2020-05-01 20:00:54'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.5903703570365906)])
OrderedDict([('tweet', 'A message to @senatemajldr: the Senate should put all its focus and energy into legislation and oversight of the #COVID19 pandemic when we convene again next week.\n \nThe American people can’t afford for us to spend our time on unqualified, far-right judicial nominees.'), ('dates', '2020-04-29 19:25:33'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.940538763999939)])
OrderedDict([('tweet', 'Corporations should not get to exploit this pandemic. Big Plastic should take responsibility for their waste instead of asking for handouts. It is communities and people who truly need our help.\n \nNo handouts for Big Plastic. https://t.co/aMnMSrWAkr'), ('dates', '2020-04-28 17:13:37'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.8544456958770752)])
OrderedDict([('tweet', ".@POTUS, stop holding @USPS funding hostage. Do not play politics with a service so essential to Americans everywhere, especially in rural areas.\n \nThe post office serves everyone, every day. We can't endanger those who rely on it or the jobs of 630,000 postal workers. #SaveUSPS https://t.co/T9IhH5XeIq"), ('dates', '2020-04-24 21:20:54'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.6818616986274719)])
OrderedDict([('tweet', "We're urging the Trump admin. to extend work authorizations for #DACA &amp; #TPS recipients.\n\nDeporting a million+ people in the midst of this crisis would be cruel. And 330,000 DACA &amp; TPS recipients are on the front lines of #COVID19.\n\nLet's put people's health and wellbeing first. https://t.co/dYqW36RrA2"), ('dates', '2020-04-20 21:50:19'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.6106887459754944)])
OrderedDict([('tweet', 'The struggle for Indian Country to get their fair share of federal resources is not a new problem. That’s part of the reason why this is such a dangerous time for Native communities: for too long, they’ve been left behind by the federal government. https://t.co/51rHbvFRAc'), ('dates', '2020-04-17 16:59:55'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.8507787585258484)])
OrderedDict([('tweet', "Small businesses have been clear: the Trump admin.'s management of #PPP is seriously flawed.\n\nBut Senator McConnell is blocking us from fixing it &amp; funding it. He's also blocking funds for:\n-Hospitals\n-Health care workers\n-State &amp; local gov'ts\nThat both D &amp; R gov's say is needed."), ('dates', '2020-04-16 16:21:45'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.870522677898407)])
OrderedDict([('tweet', 'There are more than 600,000 confirmed COVID-19 cases in the U.S. But the Trump administration STILL doesn’t have a plan for widespread testing. @SenateDems are joining health care providers, mayors, governors, and the American people in asking: #WhereAreTheTests?'), ('dates', '2020-04-15 18:15:33'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.849260151386261)])
OrderedDict([('tweet', "The COVID-19 pandemic threatens to overwhelm New Mexico’s health care system. That's why the $170 million that front-line NM providers are receiving from the #CARESAct is such a big deal. READ MORE: https://t.co/cgIQNe1fTa"), ('dates', '2020-04-14 16:38:34'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.7115868330001831)])
OrderedDict([('tweet', "The president continues to remove those who are there to keep him accountable. Now, he's sidelining the independent watchdog who is integral to our COVID-19 response. When @POTUS thinks the rules don't apply to him, the American people pay the price. https://t.co/ah3b88a1fE"), ('dates', '2020-04-07 21:31:19'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.8640294671058655)])
OrderedDict([('tweet', 'We are asking @EPAAWheeler how @EPA is adjusting pollution enforcement in light of #COVID19.\n\nWe cannot allow EPA’s core functions—which are essential to protecting public health, clean air &amp; water—to fall by the wayside while we fight this crisis. The agency must be transparent. https://t.co/a9M1l3AYzj'), ('dates', '2020-04-02 19:13:18'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.6529296040534973)])
OrderedDict([('tweet', 'We are in the midst of a pandemic, and the president won’t re-open the exchange to help people get health care.\n\nWe called for a re-opening because this unprecedented crisis requires simple measures like this to get people the care they need. It’s unconscionable that he won’t. https://t.co/buCfz1ywEv'), ('dates', '2020-04-01 21:36:36'), ('file_name', 'Tom Udall.csv'), ('pct_true', 0.8244479894638062)])
OrderedDict([('tweet', '80,000 #COVID19 deaths in America. This could have been avoided had the Trump Administration responded early and with urgency.'), ('dates', '2020-05-09 23:56:48'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.6686531901359558)])
OrderedDict([('tweet', "A global pandemic is not the time to push a political agenda that interferes with people’s lives and rights.  I'm proud to join more than 150 of my colleagues in support of a woman's right to choose. https://t.co/H4Vfw2NUJG"), ('dates', '2020-05-07 17:49:07'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.7435336709022522)])
OrderedDict([('tweet', 'The Trump Administration ignored repeated warnings from scientists and did not take the #COVID19 threat seriously, and now the American people are paying the price.  \n\nhttps://t.co/QDqBebm4Aj'), ('dates', '2020-05-05 20:40:00'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.8898867964744568)])
OrderedDict([('tweet', "President @realDonaldTrump's poor leadership and inaction have contributed to the devastation the #COVID19 pandemic has caused. Although he is not directly responsible for the outbreak, his unwillingness to act early on has made it difficult to contain."), ('dates', '2020-05-04 18:20:53'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.5975054502487183)])
OrderedDict([('tweet', '"Black and Latino Californians ages 18 to 64 are dying more frequently of COVID-19 than their white and Asian counterparts relative to their share of the population."\n\nhttps://t.co/YSSdJoIxu2'), ('dates', '2020-05-01 18:40:19'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.6019940972328186)])
OrderedDict([('tweet', "#EarthDay is about celebrating the planet and recognizing our responsibility to protect it. Climate change is a real global threat and it's our responsibility to keep the Earth healthy not just for a day, but for generations. #climatechange https://t.co/yFdxSC6kR5"), ('dates', '2020-04-22 15:22:14'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.5510697960853577)])
OrderedDict([('tweet', 'This is wrong. Despite repeated calls from doctors and health care workers to re-open enrollment for the exchange amid the national public health crisis — Trump is once again choosing partisan politics over American working families. \n\nPEOPLE ARE DYING! \n\nhttps://t.co/KYYpWypq8q'), ('dates', '2020-04-01 18:49:42'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.6273927092552185)])
OrderedDict([('tweet', 'Today, I will be voting to pass the CARES Act - a $2 trillion coronavirus response package. \n\nWorking families are hurting and American businesses are suffering. The American people are looking to Congress for leadership and action.'), ('dates', '2020-03-27 15:07:46'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.7951232194900513)])
OrderedDict([('tweet', 'Over the weekend, @senatemajldr McConnell put forward a coronavirus response bill that prioritizes corporations, not American workers. The @HouseDemocrats plan we released is about protecting American families, supporting healthcare workers, and preventing the spread of #COVID19.'), ('dates', '2020-03-24 19:31:20'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.8235719203948975)])
OrderedDict([('tweet', 'This is disgusting. We’re in the middle of a national emergency and rather than bringing this country together, the Trump team is using xenophobic language to pit communities against one another. The tone at the top matters. https://t.co/ZIHgasfNAS'), ('dates', '2020-03-17 23:33:02'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.8208615779876709)])
OrderedDict([('tweet', 'The President and his administration have not been honest with the American people about the seriousness of the coronavirus pandemic. Pres. Trump and his team mismanaged this from the beginning. Mitigation would have been easier had there been a steady hand at the top. #COVID19 https://t.co/mOTeJNNPAQ'), ('dates', '2020-03-16 20:11:20'), ('file_name', 'Tony Cárdenas.csv'), ('pct_true', 0.6542951464653015)])
OrderedDict([('tweet', "In our strong economy, Hoosier businesses of all sizes are thriving, growing, and hiring, but the #1 problem I hear when I tour these businesses is they can't find enough people to fill their open high-skilled jobs. https://t.co/voWN2yQH4m"), ('dates', '2020-02-26 19:25:25'), ('file_name', 'Trey Hollingsworth.csv'), ('pct_true', 0.5721340179443359)])
OrderedDict([('tweet', "Tonight, I voted NO on the Democrats' 3-year long attempt to overturn the results of the 2016 election of @POTUS. From the start this partisan process, the only impeachment along party lines in our history, has been about political showmanship, not Americans’ priorities."), ('dates', '2019-12-19 01:36:59'), ('file_name', 'Trey Hollingsworth.csv'), ('pct_true', 0.7433529496192932)])
OrderedDict([('tweet', 'I voted against today’s Democrat resolution to deprive @POTUS of his due process and against continuing a partisan investigation. House Democrats have misappropriated a solemn constitutional duty to delegitimize an election and the 62 million voters who supported our President.'), ('dates', '2019-10-31 15:42:19'), ('file_name', 'Trey Hollingsworth.csv'), ('pct_true', 0.8765608072280884)])
OrderedDict([('tweet', 'On this day 75 years ago, Nazi Germany surrendered and liberty prevailed. Our nation remembers and thanks the Greatest Generation who served bravely in #WWII to defend freedom at home and around the world. #VEDay75 https://t.co/d6GwMjVyao'), ('dates', '2020-05-08 14:41:33'), ('file_name', 'Troy Balderson.csv'), ('pct_true', 0.6926412582397461)])
OrderedDict([('tweet', 'Since Congress replenished the #PaycheckProtectionProgram, nearly 60k OH businesses have been approved for &gt;$4.7 Billion in #PPP loans! This will go far in providing #SmallBizRelief, ensuring Ohioans can remain on payroll through #COVID19 &amp; #smallbusinesses don’t have to shutter.'), ('dates', '2020-05-05 21:48:00'), ('file_name', 'Troy Balderson.csv'), ('pct_true', 0.6017923951148987)])
OrderedDict([('tweet', "THREAD ⬇️\n\nIf AK Steel’s manufacturing of electrical steel is edged out by importers who dodge tariffs &amp; undercut prices, not only will Ohioans lose their jobs, but the integrity of the American electrical grid will be in peril. We can't allow our critical infrastructure... (1/4)"), ('dates', '2020-05-05 12:36:43'), ('file_name', 'Troy Balderson.csv'), ('pct_true', 0.8119737505912781)])
OrderedDict([('tweet', 'I fully condemn bigotry in all forms, including the horribly offensive anti-Semitic sign displayed @ Ohio statehouse. I echo @GovMikeDeWine sentiments. These remarks have no place in OH or US, especially @ a time when we should be uplifting each other, not tearing each other down https://t.co/ObVAjTJ17c'), ('dates', '2020-04-23 02:17:24'), ('file_name', 'Troy Balderson.csv'), ('pct_true', 0.5236891508102417)])
OrderedDict([('tweet', 'Small businesses in my district need the #PaycheckProtectionProgram to work for them.\n\nUnfortunately, the money for this program has run out and Nancy Pelosi let it happen.\n\nI am working to make sure we replenish the funds to support our small businesses. #ActNow'), ('dates', '2020-04-19 16:50:35'), ('file_name', 'Troy Balderson.csv'), ('pct_true', 0.796576201915741)])
OrderedDict([('tweet', 'Two weeks ago, America knew #PPP &amp; #EIDL would run dry\n\nWe are BEHIND THE CURVE re-funding these programs\n\nIt could have been prevented, but now, America’s small businesses are paying the price\n\nNancy Pelosi &amp; Chuck Schumer MUST #ActNow'), ('dates', '2020-04-18 12:02:00'), ('file_name', 'Troy Balderson.csv'), ('pct_true', 0.9781233668327332)])
OrderedDict([('tweet', "The Paycheck Protection Program has run out of money to distribute loans. Meanwhile Speaker Pelosi continues to block additional funding, playing political games with Ohio's small businesses, workers, and families. #ActNow https://t.co/nFBtmY8k06"), ('dates', '2020-04-17 19:30:09'), ('file_name', 'Troy Balderson.csv'), ('pct_true', 0.8052047491073608)])
OrderedDict([('tweet', 'While @GovHawaii does not have the power to suspend flights, I’m calling on the Governor to immediately institute a mandatory 14-day self-quarantine of anyone who travels to Hawaiʻi. https://t.co/CxC19QkZj4'), ('dates', '2020-03-21 22:00:11'), ('file_name', 'Tulsi Gabbard.csv'), ('pct_true', 0.566798210144043)])
OrderedDict([('tweet', 'The opioid crisis continues to leave devastation in its wake — Americans who have seen lives lost, families broken, a strained prison system, &amp; taxpayers footing the bill. (1/2)\nhttps://t.co/be6wvv9VOG'), ('dates', '2019-09-24 18:58:41'), ('file_name', 'Tulsi Gabbard.csv'), ('pct_true', 0.7385008931159973)])
OrderedDict([('tweet', 'Oklahoma is asking a judge to hold Johnson &amp; Johnson accountable for its role in the state’s opioid crisis, but we need better safeguards to make sure all Americans are protected from companies who put profits over people’s well-being! (1/2)\nhttps://t.co/DQ7fkRaujO'), ('dates', '2019-07-18 19:29:23'), ('file_name', 'Tulsi Gabbard.csv'), ('pct_true', 0.8706692457199097)])
OrderedDict([('tweet', 'Trump’s comments were made to foment racism and bigotry, to further divide our country for his own political gain, and put his own interests ahead of the interests of our country. (1/3) https://t.co/r4ESueGbVm'), ('dates', '2019-07-17 03:06:05'), ('file_name', 'Tulsi Gabbard.csv'), ('pct_true', 0.5905722975730896)])
OrderedDict([('tweet', 'The president’s failed leadership has cost tens of millions of Americans their jobs. Children are going hungry. Families are losing their homes.\n\nAnd yet Republicans in Congress still refuse to work to protect people. This is heartbreaking and infuriating.'), ('dates', '2020-05-08 19:19:30'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.9406796097755432)])
OrderedDict([('tweet', 'We write: this is "overt political bias, if not outright corruption."\n\nThe Inspector General should investigate this possible breach of federal law (the Hatch Act) and other violations. https://t.co/IKSVxVvB55'), ('dates', '2020-05-08 19:15:49'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.7098881602287292)])
OrderedDict([('tweet', "Let's pretend that President Trump is the “wartime” president he claims to be.\n\nHe ignored intelligence reports about a looming attack, sent troops into battle without protective gear, and more Americans have died than in Vietnam, Afghanistan and Iraq combined."), ('dates', '2020-05-07 18:39:02'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.741828441619873)])
OrderedDict([('tweet', 'President Trump says that he will reveal his “secret” health care plan after the election. Ok.\n\nRepublicans have been saying for ten years that they’ll unveil their plan to replace the Affordable Care Act “soon.”\n\nThere is no plan. https://t.co/5r4gwysL0u'), ('dates', '2020-05-07 15:30:25'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.5686209797859192)])
OrderedDict([('tweet', 'It should be noted that many of the GOP governors rushing to reopen their states may have the support of the president but they are violating the White House’s own guidance for how to reopen.'), ('dates', '2020-05-06 20:36:44'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.8807331323623657)])
OrderedDict([('tweet', 'Congress passed billions of dollars for global health program funding during covid, but this administration’s anti-woman agenda means that communities that provide reproductive health services won’t get covid funding. That’s not just wrong, it’s dangerous.'), ('dates', '2020-05-06 16:00:38'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.9099145531654358)])
OrderedDict([('tweet', 'America is the strongest, richest country on earth, but for months the president has refused to use his authority to produce protective gear, test kits, and more.\n\nWe’ve introduced new legislation to require him to do so. We’re behind on COVID. It’s time to catch up. https://t.co/Vy6KOrShhw'), ('dates', '2020-05-05 19:55:33'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.6288274526596069)])
OrderedDict([('tweet', 'What are they hiding? When the White House hides information from Congress, they’re hiding information from the American people. https://t.co/D96T4Pwnil'), ('dates', '2020-05-05 18:05:53'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.7959515452384949)])
OrderedDict([('tweet', "We can't open up too quickly and lose the progress we've been able to gain. Cases are going down in some areas because people are social distancing, wearing masks, and listening to the health experts. https://t.co/Z8kKPtTzlF"), ('dates', '2020-05-04 21:33:06'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.5357186794281006)])
OrderedDict([('tweet', 'We are still receiving reports that Florida’s unemployment system is riddled with errors and bugs.\n\nThe governor has said this is a priority, but information is not forthcoming. People need help now.'), ('dates', '2020-05-04 16:31:03'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.6453019380569458)])
OrderedDict([('tweet', 'In Abraham Lincoln, there “never was an impulse of unlawful ambition, a shadow of dishonesty, [nor] a wish that was not for the welfare of his country.”\n- The Chicago Daily Tribune'), ('dates', '2020-05-04 15:18:22'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.5592769384384155)])
OrderedDict([('tweet', 'Never let anyone pretend that this death toll was inevitable.\n\nThe president and his administration were warned, and they did nothing. https://t.co/RciROwCpN8'), ('dates', '2020-05-01 17:56:31'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.5379961133003235)])
OrderedDict([('tweet', 'When we impeached the president, we warned that if acquitted, he would abuse his power again.\n\nNow he’s doing it, with American lives on the line.\n\nEmergency relief for covid-19 CANNOT be used as a political tool. https://t.co/eA8iLzumxu'), ('dates', '2020-05-01 17:55:38'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.8161963224411011)])
OrderedDict([('tweet', 'American democracy has survived civil war, world wars, depressions and recessions, and previous pandemics. We’ll survive this.\n\nBut using this emergency as an excuse to restrict voting goes against everything we are supposed to be.'), ('dates', '2020-04-30 14:03:05'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.7360798120498657)])
OrderedDict([('tweet', '“I need you to do me a favor though.”\n\nWhen the Senate GOP shut down the impeachment trial and said that it was OK for the president to use life-and-death policy for political gain, he heard them loud and clear. https://t.co/v6OwCKMxfk'), ('dates', '2020-04-29 20:46:15'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.7334606051445007)])
OrderedDict([('tweet', 'Why is President Trump turning saving lives into a partisan issue?\n\nTo quote President Obama, “there are no red states or blue states, just the United States.”'), ('dates', '2020-04-28 20:34:50'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.9570117592811584)])
OrderedDict([('tweet', 'Florida’s Governor needs to explain why 40% of Florida unemployment insurance claims have been rejected, and why so many Floridians haven’t been able to access the federal expansion we passed a month ago. https://t.co/v2tFKV0orL'), ('dates', '2020-04-28 20:08:38'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.6653946042060852)])
OrderedDict([('tweet', 'Helping states fund critical COVID-19 responses is not a “blue state bailout."\n\nFirst, it’s the American people’s tax money, not the president’s personal bank account.\n\nSecond, this virus doesn’t care if you’re a Democrat or a Republican, and neither should we. https://t.co/8TYtWoV5os'), ('dates', '2020-04-28 18:25:23'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.5537579655647278)])
OrderedDict([('tweet', 'The president has refused to create a national strategy for COVID-19. Instead, states and municipalities are stepping up to lead, but they have limited resources. The federal government MUST assist with this response.'), ('dates', '2020-04-28 18:23:32'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.7046850919723511)])
OrderedDict([('tweet', 'With millions of Americans voting by mail, it’s an attack on our republic to try to dismantle the U.S. Postal Service.'), ('dates', '2020-04-27 17:37:09'), ('file_name', 'Val Butler Demings.csv'), ('pct_true', 0.5606491565704346)])
OrderedDict([('tweet', 'The #PaycheckProtectionProgram ran out of funds yesterday. People are losing  jobs, businesses are closing, &amp; parents are worried about providing for their families. Now is not the time for politicians to use the hardship of the American people as bargaining chips. https://t.co/w2EeQrJKtD'), ('dates', '2020-04-17 18:40:40'), ('file_name', 'Van Taylor.csv'), ('pct_true', 0.8444269299507141)])
OrderedDict([('tweet', 'For decades, the Iranian regime has been the antagonist for peace and civility in the Middle East. The strike on Soleimani was just and legal, and most importantly, ensures he will never take another American life. https://t.co/GEUR6oC7Xb'), ('dates', '2020-01-14 14:47:24'), ('file_name', 'Van Taylor.csv'), ('pct_true', 0.8267322182655334)])
OrderedDict([('tweet', 'America needs to become less dependent on China for life-saving drugs. China is not our friend. Read about my bill to ensure that we never again are forced to rely on other countries for critical drugs that keep Americans healthy. https://t.co/q02Gl15Id4'), ('dates', '2020-05-06 19:03:31'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.8959649205207825)])
OrderedDict([('tweet', 'Coronavirus has shown how dangerously reliant we are on China for life-saving medicines. China recently threatened to cut off access to these vital drugs. Today I introduced legislation in the U.S. House to rebuild drug manufacturing in America.'), ('dates', '2020-05-05 14:31:29'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.8842047452926636)])
OrderedDict([('tweet', 'Kudos to Secretary of State Pompeo for calling on China to shut down their animal "wet” markets, where coronavirus may have originated. My colleague @RepHastingsFL and I urged the White House to do so last week. https://t.co/UGlVZMivMi'), ('dates', '2020-04-23 13:18:15'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.7368481755256653)])
OrderedDict([('tweet', 'President Trump plans to suspend immigration into the United States during the coronavirus pandemic. I support his action, do you?'), ('dates', '2020-04-21 14:50:53'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.5463215112686157)])
OrderedDict([('tweet', 'My Florida colleague @RepHastingsFL and I just wrote to President Trump condemning China\'s live animal markets that serve as incubators for deadly viruses like coronavirus. Our letter urges Trump to demand that China impose strict regulation of these so-called "wet markets." https://t.co/F4aMDEX4DV'), ('dates', '2020-04-19 00:36:11'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.5444978475570679)])
OrderedDict([('tweet', "Our fellow Floridians on a cruise ship are coming home, including four of my constituents. Those sick will get the treatment they need. This is the right decision and why I urged White House COS Staff Mark Meadows to intervene. We don't leave our neighbors stranded in the ocean."), ('dates', '2020-04-02 13:18:24'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.7034197449684143)])
OrderedDict([('tweet', "Hearing that 49 Floridians stuck on a cruise ship, including four of my constituents, will be coming home. This is the right decision and why I urged White House Chief of Staff Mark Meadows to intervene. We don't leave our neighbors stranded in the ocean."), ('dates', '2020-04-01 22:51:52'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.7105385065078735)])
OrderedDict([('tweet', 'Just talked to the White House about stranded cruise ships with Floridians on board. Told the White House they need to get the Floridians off the ships, back in Florida and into quarantine for treatment if necessary. Told Gov. DeSantis the same. We take care of our people.'), ('dates', '2020-03-30 21:22:29'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.6001878976821899)])
OrderedDict([('tweet', 'As the Trump administration requests funds to address the deadly #Coronavirus, Congress cannot afford to take this threat lightly. We need to devote whatever resources are necessary to protect the public and contain this rapidly-spreading virus.'), ('dates', '2020-02-25 19:16:01'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.5469770431518555)])
OrderedDict([('tweet', 'Now that the partisan impeachment of President Trump has failed, it’s time for Congress to get back to work. Let’s join together and tackle our real challenges – starting with Washington dysfunction and a toxic political system that leaves no room for compromise.'), ('dates', '2020-02-05 21:33:03'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.9208856225013733)])
OrderedDict([('tweet', "If you needed any proof that the impeachment of President Trump was entirely political, look no further than Nancy Pelosi's petty threat to withhold sending the articles of impeachment to the U.S. Senate for a trial. She is only proving this process was a sham from the beginning."), ('dates', '2019-12-19 14:40:40'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.5806655287742615)])
OrderedDict([('tweet', 'BREAKING: The U.S. Senate just passed my proposal to slap sanctions on China for failing to curb the export of deadly Fentanyl into the U.S. It now goes to President Trump for his signature! ☑️'), ('dates', '2019-12-17 18:00:20'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.8516274690628052)])
OrderedDict([('tweet', 'My proposal to slap sanctions on China for failing to curb the export of deadly Fentanyl into the U.S. was just passed by the U.S. House as part of the National Defense Authorization Act. It’s time to hold China accountable.'), ('dates', '2019-12-11 22:45:42'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.9569831490516663)])
OrderedDict([('tweet', 'With anti-Semitism on the rise, President Trump is expected to sign an executive order today targeting anti-Semitism on college campuses. Anti-Semitism has no place in our country, especially in our schools.'), ('dates', '2019-12-11 18:55:10'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.8002902269363403)])
OrderedDict([('tweet', 'Nancy Pelosi is making an enormous mistake by launching the first partisan impeachment in history. It sets a dangerous precedent. Our Founding Fathers created the best way to remove a President --  it’s called an election and the next one is less than a year away!'), ('dates', '2019-12-05 16:13:40'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.8682674765586853)])
OrderedDict([('tweet', 'Drug overdose deaths have more than doubled in Manatee County over the past 10 months. It’s time to hold China accountable for not doing enough to curb the flow of deadly opioids like fentanyl. Congress needs to pass my “Fentanyl Sanctions Act.” https://t.co/1VbHd0o0wc'), ('dates', '2019-11-05 16:04:10'), ('file_name', 'Vern Buchanan.csv'), ('pct_true', 0.6138038039207458)])
OrderedDict([('tweet', 'Laws allowing “citizens arrests” &amp; easy access to guns coupled w/ our country’s deep roots of racism put a target on the backs of African Americans like Ahmaud.\n\nEven w/ video of his murder, it may be difficult for his family to get justice. Imagine if there hadn’t been a video. https://t.co/9f1PnG2p1S'), ('dates', '2020-05-09 14:31:28'), ('file_name', 'Veronica Escobar.csv'), ('pct_true', 0.6866087913513184)])
OrderedDict([('tweet', 'It’s shameful that the Texas AG \u2066@KenPaxtonTX\u2069 cares so little about Texans’ safety that he wants to force people to make a choice between their health and their vote in the middle of a deadly global pandemic.\n\nEnough!  Texans deserve better. https://t.co/EAA11G3aSI'), ('dates', '2020-05-04 16:29:42'), ('file_name', 'Veronica Escobar.csv'), ('pct_true', 0.7279678583145142)])
OrderedDict([('tweet', 'This is unacceptable. \n \nWhile our country is facing an unprecedented crisis, @realDonaldTrump is still attacking marginalized communities. This week his administration is planning to end protections in the #ACA for the #LGBTQ community. \n\nhttps://t.co/vI7xSmh8Nb'), ('dates', '2020-04-29 22:00:00'), ('file_name', 'Veronica Escobar.csv'), ('pct_true', 0.6898736357688904)])
OrderedDict([('tweet', 'The Trump administration continues its attacks against #DACA recipients. Dreamers are part of our communities, economy and part of the solution to combat the #COVID19 pandemic, yet they are left out of relief efforts.\n\nhttps://t.co/jftjsKX9bv'), ('dates', '2020-04-24 18:30:00'), ('file_name', 'Veronica Escobar.csv'), ('pct_true', 0.5630363821983337)])
OrderedDict([('tweet', 'Postmaster General Megan Brennan told lawmakers that @USPS is projected to lose $13 billion due to #COVID19 and an additional $54 billion over the next decade. USPS projects that it could run out of money by September if they do not obtain more funding. (1/8)'), ('dates', '2020-04-20 14:39:46'), ('file_name', 'Vicente Gonzalez.csv'), ('pct_true', 0.5744365453720093)])
OrderedDict([('tweet', '🚨🚨@StateDept advises U.S. citizens to avoid all international travel b/c of the impact of #COVID19.  In countries where departure options remain available, U.S. citizens should arrange for return to the U.S., unless they are prepared to remain abroad for an indefinite period.'), ('dates', '2020-03-19 15:15:06'), ('file_name', 'Vicente Gonzalez.csv'), ('pct_true', 0.5601880550384521)])
OrderedDict([('tweet', "The world is waking up to #China's deliberate deceit over the #coronavirus. Once again, Beijing proves it is not a trusted actor.\n\nAmerica, it's time we cut our dependency on China for our pharmaceuticals. Let's take back control of our own security.\nhttps://t.co/GlbVaFyqk3"), ('dates', '2020-05-04 14:19:20'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.9030629992485046)])
OrderedDict([('tweet', '"China’s coronavirus actions are just one of many threats it poses." @NikkiHaley is right on this!\n\nWe must reduce our dependence on #China and look for alternative supply chains for critical industries. It\'s time for America and our allies to stand up.\n\nhttps://t.co/GNs2JoUXdZ'), ('dates', '2020-05-01 18:40:02'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.8432804942131042)])
OrderedDict([('tweet', "The #coronavirus highlights our reliance on #China in many sectors essential to national security like our pharmaceutical industry.\n\nThat's why I'm leading a bill with @RepGaramendi to reduce our military's dependency on China for basic medical supplies.\nhttps://t.co/SYY0jliAXp"), ('dates', '2020-05-01 15:13:09'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.5480087399482727)])
OrderedDict([('tweet', "#China's deceit is responsible for the #coronavirus crisis, both the health crisis and the economic downfall. China must be held accountable for its spread and for making it harder for the world to respond. \n\n@SecPompeo is absolutely right on this.\nhttps://t.co/kDTfLQAwBd"), ('dates', '2020-04-30 16:41:24'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.8203786611557007)])
OrderedDict([('tweet', 'We cannot forget our farmers and ranchers who are in desperate need of relief. The #coronavirus has disrupted commodity values and drastically reshaped supply chains. \n\nOur farmers and ranchers are the very folks who help feed our country. #InItTogether https://t.co/aqzClscx2F'), ('dates', '2020-04-16 19:34:38'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.636276125907898)])
OrderedDict([('tweet', "The #PaycheckProtectionProgram runs out TODAY. @SpeakerPelosi and Sen. Minority Leader Schumer ALREADY agreed to the #PPP terms when we passed the CARES Act.\n\nThere's no reason to obfuscate and block this critical funding. It's time to put partisanship aside and pass funding NOW! https://t.co/1e56OnCfW5"), ('dates', '2020-04-16 13:40:31'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.7607982158660889)])
OrderedDict([('tweet', '#China must be held accountable for their lies and cover-up regarding #coronavirus. Their deceit has killed Americans and around the world, crippling our economy.\n\nWe MUST work to reduce our dependency on China, ESPECIALLY for our pharmaceuticals and basic medical equipment.'), ('dates', '2020-04-14 23:56:28'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.9406964778900146)])
OrderedDict([('tweet', "The #PaycheckProtectionProgram is keeping small businesses afloat and employees on the payroll as we face the #coronavirus battle.\n\nIt's time for @SpeakerPelosi, @SenSchumer and Democrats to work with @realDonaldTrump on a clean bill to bolster PPP funds.\n\nhttps://t.co/a86G385HKO"), ('dates', '2020-04-14 15:53:27'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.630505383014679)])
OrderedDict([('tweet', "I'm pleased to see the federal government's response to the #coronavirus is finally taking shape for Missouri's hospitals:\n\n5,000+ Missouri health care providers, totaling ~$618.6 million, were paid through the first disbursement for hospitals under the CARES Act."), ('dates', '2020-04-13 15:36:06'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.5217047929763794)])
OrderedDict([('tweet', "#China's lies and deceit made it harder for the world to respond to the #coronavirus. They are responsible for this pandemic and should NOT be relied on for our medicine - certainly not our military's medicine. \n\nChina is a reckless global actor and must be held accountable. https://t.co/NRFstbefId"), ('dates', '2020-04-01 17:26:20'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.9524520635604858)])
OrderedDict([('tweet', "Just as @realDonaldTrump reduced our dependence on  Middle East oil, we should also reduce our dependence on #China for our pharmaceuticals.\n\nIf #COVID19 has taught us anything, it's that China should NOT be trusted with our servicemembers' medicine.\n\nhttps://t.co/PiIBB5ZoF3"), ('dates', '2020-03-25 23:20:47'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.7205602526664734)])
OrderedDict([('tweet', "#China must be held to account for their response to the #coronavirus. The lack of transparency and their shrewd secrecy made it incredibly difficult for the world to respond to #COVIDー19.\n\nThat's why I co-sponsored this resolution from @RepJimBanks.\n\nhttps://t.co/u6eYmJyFYi"), ('dates', '2020-03-24 17:03:36'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.5449103116989136)])
OrderedDict([('tweet', 'The #CoronavirusOutbreak highlights a grave national security concern. \n\nIn a time of conflict, #China could stop our supply of generic prescription drugs and basic medical equipment. We are far too dependent on China.\n\nFrom @BryanLowry3 and @KCStar:\nhttps://t.co/mqLLCDwVVR'), ('dates', '2020-03-18 19:55:26'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.8080816268920898)])
OrderedDict([('tweet', 'We saw that @realDonaldTrump continues to take decisive steps to mitigate the #coronavirus and protect American lives. \n\nThe early travel ban on #China and decisive actions on travel have saved lives and have proven pivotal in our efforts to combat the virus.'), ('dates', '2020-03-12 01:37:56'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.5942661762237549)])
OrderedDict([('tweet', "Congress should NOT be playing politics with #coronavirus funding bills. This is not the time to fill them up with politically-loaded language and irrelevant funding requests.\n\nWe need to help keep our communities safe, not play politics with people's lives. https://t.co/x7dSLTApRH"), ('dates', '2020-03-04 16:39:22'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.6903490424156189)])
OrderedDict([('tweet', 'I voted YES on @MarioDB\'s resolution to condemn Bernie Sanders\' comments about Fidel Castro. \n\nThe House should make it clear that the "literacy program" Sanders praised was a propaganda tool to spread Marxist-socialist ideologies.\n\nSadly the House chose to not condemn communism.'), ('dates', '2020-02-27 21:09:27'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.9603351354598999)])
OrderedDict([('tweet', 'Socialism and big government takeovers don’t work. @realDonaldTrump has shown that they key to our endeavors is sustained economic growth and creating opportunities of empowerment for all Americans.\n\nCutting bureaucratic red tape and letting people keep more of their money WORKS! https://t.co/KZQpj6Qi0j'), ('dates', '2020-02-24 18:47:52'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.5826373100280762)])
OrderedDict([('tweet', 'Women are not victims who need validation. The outcomes of the #ERA are anything but pro-woman.\n\nToday I voted to uphold the Constitution, promote life, and protect women’s rights that have been so hard-fought gained by voting "NO." https://t.co/kSzubZoiht'), ('dates', '2020-02-13 21:21:43'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.5019221901893616)])
OrderedDict([('tweet', 'Now that America is moving on from an impeachment process that brought Congress to a standstill, it’s time Speaker Pelosi lets us get back to doing the work of the American people.\n\nLet’s stop the complaining and grandstanding and actually put forward a budget on the floor. https://t.co/CZ04nb9qJ1'), ('dates', '2020-02-11 21:05:00'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.8360949754714966)])
OrderedDict([('tweet', "With its stronghold on the global pharmaceutical supply chain, most generic drugs used by our military are made in #China.\n\nThis reliance on China is a grave national security threat. That's why I'm co-leading a bill in Congress to address this:\n\nhttps://t.co/rsmwxnoo5o"), ('dates', '2020-02-06 15:36:31'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.7290940284729004)])
OrderedDict([('tweet', 'Both "not guilty."\n\nNow that the Senate has acquitted @realDonaldTrump, it\'s time that our lawmakers get back to doing the work of real Americans. \n\nParticularly to my friends on the other side of the aisle, it\'s time to find common ground with President Trump.'), ('dates', '2020-02-05 21:32:56'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.6467558741569519)])
OrderedDict([('tweet', "There were names of people who died for their country in that speech, names of families who lost a loved one, people struggling with cancer, stories of a young man's aspirations, a military family reunited.\n\nIt's saddening to see that behavior. But America still marches ahead. https://t.co/FSsxjszBAD"), ('dates', '2020-02-05 15:30:05'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.6787588596343994)])
OrderedDict([('tweet', "The #IACaucusDisaster is a perfect example of how Democrats run things. \n\nGood thing this #SOTU we'll be hearing about the great strides America is making under @realDonaldTrump: a booming economy, revamping our trade deals, securing our borders, and taking out terrorists."), ('dates', '2020-02-04 14:42:57'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.7811724543571472)])
OrderedDict([('tweet', "We see yet again how the House Democrats' three-month-long impeachment sham brought the House to a screeching halt.\n\nWe're days away from the federal ban on #fentanyl expiring. The Senate unanimously passed an extension. The House hasn't acted. Total silence from Speaker Pelosi."), ('dates', '2020-01-28 19:29:06'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.9499862194061279)])
OrderedDict([('tweet', 'Yes, @GOPLeader McCarthy is absolutely right about this.\n\nTheir entire argument, which they bring up over and over again, is built on the fact that they do not want @realDonaldTrump to be President.\n\nNo one should weaponize impeachment for politics purposes. https://t.co/0cDqN92p6Y'), ('dates', '2020-01-23 20:19:52'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.5896351337432861)])
OrderedDict([('tweet', 'I’ve said this from the beginning. This entire sham impeachment against @realDonaldTrump is purely political, all about undoing the decision of the American people in 2016, and all about the next election.\n\nHouse Democrats have made that crystal clear.\n\n https://t.co/npnqMQiJyu'), ('dates', '2020-01-22 18:41:35'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.6905333399772644)])
OrderedDict([('tweet', 'While @SpeakerPelosi rammed through a dead-on-arrival healthcare bill to distract from impeachment, we introduced H.R. 19, built on 40+ reforms by Republicans AND Democrats to lower your prescription drug costs.\n\nYour healthcare is not a distraction.\n\nhttps://t.co/hDzA2Xibie'), ('dates', '2020-01-17 14:57:49'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.5777474641799927)])
OrderedDict([('tweet', 'Senate Minority Leader Chuck Schumer said that @realDonaldTrump "has got to go in the election, if not sooner." \n\nThat\'s all this impeachment sham has ever been - a political move to go against 1.4 million Missourians who voted for Trump in 2016.\n\nhttps://t.co/9hPgGOlf21'), ('dates', '2020-01-14 21:17:37'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.6170612573623657)])
OrderedDict([('tweet', "House Democrats blocked @GOPLeader's resolution in support of the Iran protesters. I applaud Leader McCarthy for trying to bring Republicans and Democrats together on this.\n\nSupporting a resolution so basic shouldn't be this difficult. The House is better than this."), ('dates', '2020-01-14 19:21:28'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.6029725074768066)])
OrderedDict([('tweet', 'This impeachment against @realDonaldTrump is a sham.\n\nThe House came to a screeching halt because they said impeachment was urgent. \n\nSo why did Speaker Pelosi play politics with the articles for nearly a month?\n\nWe need to do the work of the American people here in Washington. https://t.co/4XvsN1Mqvh https://t.co/JamlH8BvVs'), ('dates', '2020-01-11 19:27:10'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.8986944556236267)])
OrderedDict([('tweet', 'On #Iran: @realDonaldTrump made the right decision in taking out #Soleimani. \n\nThe fact that Soleimani can no longer plot violent and barbaric acts of terror against Americans, our allies, and countless civilians throughout the Middle East is something that should be celebrated. https://t.co/2DMjAeAI71'), ('dates', '2020-01-10 19:55:44'), ('file_name', 'Vicky Hartzler.csv'), ('pct_true', 0.5715102553367615)])
OrderedDict([('tweet', '75 years ago today, the tyrannical Nazi regime crumbled before the eyes of the world. We commend our veterans - and their families - for their service.\n\nBecause of your sacrifices, and determination to extinguish the flame of a deadly regime, the world’s unity is strengthened. https://t.co/k7mrpoiWIE'), ('dates', '2020-05-08 15:40:15'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.5606207847595215)])
OrderedDict([('tweet', 'For far too long, China’s encroachment on our public universities has gone unchecked. Securing America’s educational system requires us to combat the influence of bad actors like the CCP. I’m confident that @BetsyDeVosED will help lead the charge against their efforts. https://t.co/xYsucm7HxI'), ('dates', '2020-05-05 18:44:51'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.7804572582244873)])
OrderedDict([('tweet', 'House Democrats are complacent with inaction and indifference. China’s influence continues to grow. Americans are losing their jobs. Small businesses have growing weight on their shoulders. Meanwhile, House Democrats are nowhere to be found 🔎. \n\nWe cannot allow any more delay. https://t.co/BGPgldtjww'), ('dates', '2020-05-05 13:36:57'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.96103835105896)])
OrderedDict([('tweet', 'Try this on for size: the CCP is a regime built upon oppressing dissenting views at all costs. It’s not a verdict of many, it’s the verdict of a regime that quashes free speech and substitutes public discourse with propaganda. People have died at the hands of the CCP. Shameful! https://t.co/5sXRJZx3HJ'), ('dates', '2020-05-04 21:32:14'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.7443117499351501)])
OrderedDict([('tweet', "House Democrats are sending a clear message that abdicating Congress' duty to serve the American people is more of a priority than getting America back to work. Republicans are approaching things differently. We're working to protect the American people instead of sitting idly by https://t.co/jx9Uo5D2hZ"), ('dates', '2020-05-04 18:42:59'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.6967185735702515)])
OrderedDict([('tweet', 'The FBI knowingly undermined the rule of law that exists in this country, and the American people should be concerned about the dangerous precedent that this creates. \n\nThis is nothing short of a coordinated hit job against @GenFlynn. https://t.co/leecM0vPHb'), ('dates', '2020-05-01 00:04:00'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.7404674291610718)])
OrderedDict([('tweet', 'By not resuming the work of the People’s House, Democrats have forsaken the accountability that they claim to champion. I share the sentiment of the @GOPleader, the American people deserve elected leaders who do not shy away from the duties that they swore to uphold. https://t.co/ElVbWhcPWx'), ('dates', '2020-04-29 18:10:55'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.6999649405479431)])
OrderedDict([('tweet', "For years, I've fought alongside my colleagues to hold scammers accountable for unwanted calls that defraud and disrupt the American people. Very pleased to see action by the @FCC on this issue. The actions of these bad actors must not go unchecked. https://t.co/rIEMZZcJu8"), ('dates', '2020-04-29 17:38:35'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.5682134032249451)])
OrderedDict([('tweet', 'Hardworking Americans on the front lines are fighting, but Speaker Pelosi &amp; House Democrats are nowhere to be found. The American people need to see Congress in action and not sitting on the sidelines. "Where\'s Waldo"? No, it\'s "Where Are The Democrats?". https://t.co/DK9yo5BhXw'), ('dates', '2020-04-29 14:33:46'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.9491894841194153)])
OrderedDict([('tweet', 'We’re no strangers to comeback stories. America’s economic engine is not down for the count. Everyday Americans are stepping up. The private sector is helping lead the charge. Americans helping Americans. Neighbors helping neighbors. We will defeat COVID-19 together.'), ('dates', '2020-04-28 19:46:46'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.818594753742218)])
OrderedDict([('tweet', 'Let’s rewind the clocks:\n\nApril 2nd ➡️ Speaker Pelosi calls for the creation of a new committee while American lives are on the line\n\nApril 9th➡️ Senate Democrat’s block additional funding for the PPP\n\nApril 16th➡️Funding for the PPP runs dry\n\nThat, Sen. Schumer, is bungling. https://t.co/RWF9bnus1k'), ('dates', '2020-04-27 17:09:36'), ('file_name', 'Virginia Foxx.csv'), ('pct_true', 0.6210852861404419)])
OrderedDict([('tweet', 'Many corporations with alternative financing mechanisms took PPP funding first, and as a result, small businesses are having to shut their doors.\n \nI sent a letter to Secretary Steve Mnuchin asking that he have large corporations return the money.\n \nhttps://t.co/Ms9lPJ7QoV'), ('dates', '2020-05-04 19:00:30'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.6052507162094116)])
OrderedDict([('tweet', 'It seems like House Republicans are the only ones willing to go back to work. \n\nWhile the Senate reopens and Pelosi flip flops between opening dates, @GOPLeader released a plan to safely restore the voice of the people. \n\nPelosi need to stop stalling and bring the House back now.'), ('dates', '2020-05-04 14:30:34'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.5509709119796753)])
OrderedDict([('tweet', 'Some states are asking for more federal funding. Congress has already given them $765 billion.\n\nTheir debt has nothing to do with COVID-19, but their out-of-control pension systems and mismanaged legislatures.'), ('dates', '2020-05-01 14:56:35'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.7629138231277466)])
OrderedDict([('tweet', 'Dr. Li Wenliang bravely risked his safety to warn the public about COVID-19 despite the punishment he faced from Communist China for speaking up. I cosponsored the Dr. Li Wenliang Congressional Gold Medal Act to recognize his bravery and global service. \nhttps://t.co/MxE9vZgZvq'), ('dates', '2020-04-27 16:47:08'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.6862103939056396)])
OrderedDict([('tweet', "Speaker Pelosi couldn't find a way with the EIGHT existing oversight mechanisms to take down @realDonaldTrump, so she decided to try and create another one.\n\nIt didn't work with Mueller, it didn't work with Ukraine, and it isn't going to work now. Time to accept 2016 and move on."), ('dates', '2020-04-23 18:49:28'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.6285197138786316)])
OrderedDict([('tweet', "While Americans suffer, House Dems secretly drafted a plan to allow voting by proxy.\n\nWe were elected to represent the PEOPLE, not sharecrop their vote out during times of crisis. We are the proxy for the district. We don't get to pass that duty on.\n\nAmerica needs us to show up."), ('dates', '2020-04-22 15:38:10'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.7699225544929504)])
OrderedDict([('tweet', 'Why do we rely on Communist China for vital supplies like PPE? We should manufacture these products in the US and create much needed jobs for Americans. Communist China cannot be trusted!\nhttps://t.co/DkvrvP6n1F'), ('dates', '2020-04-21 22:00:00'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.9400665760040283)])
OrderedDict([('tweet', 'CCP mouthpiece publicly acknowledges the tank man in the Tiananmen Square massacre. It took 30 YEARS for Beijing to acknowledge this oppression. How long will it take them to be honest about the #COVID19 coverup? https://t.co/gRtQ3TO2C2'), ('dates', '2020-04-21 20:34:24'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.6081066727638245)])
OrderedDict([('tweet', 'As a party to the 2005 International Health Regulations, China has a duty to report information during a global public health emergency. Not only did China fail to do this, but they deliberately spread lies.\n\nThe #CCP needs to be held accountable.\n\nhttps://t.co/DDoF6QkDQM'), ('dates', '2020-04-21 00:04:20'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.706461489200592)])
OrderedDict([('tweet', 'There is no logical basis for Speaker Pelosi’s opposition to this decision. @realDonaldTrump is right to halt funding to the WHO after their incredible failures and political agenda put our citizens and global economy in harm’s way.\n\nhttps://t.co/r6MDAgYDLL via @DailyCaller'), ('dates', '2020-04-20 15:45:15'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.8734205961227417)])
OrderedDict([('tweet', '22M Americans have filed for unemployment since mid-March. Now, PPP is set to run out of funds while DC Democrats continue to play politics with people’s livelihoods?\n\nAmericans need to pay rent and provide for their families. This hold up of funding is unacceptable and shameful.'), ('dates', '2020-04-16 13:43:12'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.6628482937812805)])
OrderedDict([('tweet', 'The Paycheck Protection Program will run out of funds this Friday. Instead of funding a program they’ve already supported, DC Democrats are willing to exploit the suffering of American small business owners for political gain.\n\nSpeaker Pelosi, this isn’t a game. Time to act.'), ('dates', '2020-04-14 23:10:49'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.8910480737686157)])
OrderedDict([('tweet', 'Thank you @realDonaldTrump for holding the @WHO accountable.\n\nThe WHO spent more time criticizing the decisive early actions from President Trump’s travel ban than they did protecting lives and our global economy.\n\nUntil they can do their jobs, we are right to halt funding.'), ('dates', '2020-04-14 22:38:01'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.7689204812049866)])
OrderedDict([('tweet', "This is a drop in the bucket for the UN's failure to uphold human rights. The Chinese Communist Party openly violates freedoms of belief, expression, and privacy, and now the United Nations expects them to prosecute the violations they perpetuate? https://t.co/c5TrWlo0Ix"), ('dates', '2020-04-13 15:01:03'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.9353986978530884)])
OrderedDict([('tweet', 'Senate Democrats are AGAIN blocking much-needed relief for our families and small businesses in an attempt to force more of their liberal agenda.\n \nAmericans will not forget this shameful display of partisanship during a time of crisis.'), ('dates', '2020-04-09 15:59:54'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.8796566128730774)])
OrderedDict([('tweet', 'We all know @WHO  and where. My question is what really happened in China, when did they truly know they had an outbreak, and why were they covering it up? https://t.co/Aa44csOn2k'), ('dates', '2020-04-02 17:19:43'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.8567584156990051)])
OrderedDict([('tweet', 'China must be held responsible for the economic damages from COVID-19. That is why I introduced the Chinese Government COVID-19 Accountability Act. We should not have to pay for China’s mistakes financially. This virus has already cost us enough! https://t.co/HQiCB4dyYB'), ('dates', '2020-03-26 21:30:00'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.9718379974365234)])
OrderedDict([('tweet', 'Democrats are taking this national emergency and using it as an opportunity to push their crooked Green New Deal. Instead of focusing on public health, they are trying to push new emissions standards for airlines in the COVID-19 response bill. Shameful!'), ('dates', '2020-03-24 21:00:00'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.9869472980499268)])
OrderedDict([('tweet', 'If Communist China had been transparent about COVID-19, thousands of lives would have been saved. I am proud to cosponsor this bill led by @RepJimBanks condemning the Chinese government\nhttps://t.co/r5GYBdoxOF\n condemns-chinese-government-over-handling-of-coronavirus-response'), ('dates', '2020-03-24 16:35:00'), ('file_name', 'W. Gregory Steube.csv'), ('pct_true', 0.9934417009353638)])
OrderedDict([('tweet', 'Honored to join @MaryVought to highlight the @DNC shift: \n"[Biden\'s] campaign’s platform now speaks of “protect[ing] the constitutional right to abortion” and “repealing the Hyde Amendment.” The end result of this proposed Biden-Pelosi agenda: Taxpayer-funded abortion on demand." https://t.co/BlhrxPpkGR'), ('dates', '2020-04-30 14:16:45'), ('file_name', 'Warren Davidson.csv'), ('pct_true', 0.6546011567115784)])
OrderedDict([('tweet', "No American should lose essential liberty for fleeting security. Once surrendered, freedom is hard to reclaim.\nAlso, once big-tech has your data, it's hard to reclaim.... but it should be easy, free, and enforceable. Congress must pass comprehensive data privacy legislation. #4A https://t.co/7fdPl6sv1X"), ('dates', '2020-04-29 19:58:30'), ('file_name', 'Warren Davidson.csv'), ('pct_true', 0.5835554003715515)])
OrderedDict([('tweet', '#DefendFreedom \n“The federal government has taken too much tax money from the people, too much authority from the states, and too much liberty with the Constitution.” \n- Ronald Reagan'), ('dates', '2020-04-14 01:10:59'), ('file_name', 'Warren Davidson.csv'), ('pct_true', 0.6024595499038696)])
OrderedDict([('tweet', '"From the opening of the investigation, the FBI team kept accumulating exculpatory information. Yet rather than wind the investigation down, they ramped it up...Then it got worse. The FBI team excluded exculpatory information from its FISA application..." https://t.co/evKck9QyqM'), ('dates', '2020-04-11 16:41:47'), ('file_name', 'Warren Davidson.csv'), ('pct_true', 0.5334650278091431)])
OrderedDict([('tweet', '#4A Sadly, this is not an April Fool’s article. In Carpenter v United States, #SCOTUS clarified that warrantless searches of Americans’ geolocation data violates our protected right to privacy. Stop warrantless surveillance of American citizens. \ncc:Kansas https://t.co/qYMFlt2MLM'), ('dates', '2020-04-01 22:49:32'), ('file_name', 'Warren Davidson.csv'), ('pct_true', 0.9223959445953369)])
OrderedDict([('tweet', '#fact Unfortunately, the whole structure is actively hostile to privacy. True DL is inherently superior to centralized surveillance schemes like China’s. Privacy ≠ secrecy so warrants could still achieve discovery. Digital currency must never exclude physical cash &amp; must... https://t.co/yQkfQY4CF7'), ('dates', '2020-03-26 10:52:01'), ('file_name', 'Warren Davidson.csv'), ('pct_true', 0.7975634336471558)])
OrderedDict([('tweet', 'The Taliban, just like the Chinese Communist Party and the Kremlin, is using the #COVID19 pandemic to advance their own goals while jeopardizing the lives of innocent civilians. \n\nhttps://t.co/WYNz2L7Vjg'), ('dates', '2020-05-08 19:01:02'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7513779997825623)])
OrderedDict([('tweet', 'The Iranian regime has shown disregard for global stability and even the lives of their own people. They’ve proven they can’t be trusted. Keeping the @UN arms embargo is paramount to global security, which is why I joined @RepMcCaul in calling for its extension.'), ('dates', '2020-05-04 21:58:51'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.8432498574256897)])
OrderedDict([('tweet', 'The Chinese Communist Party is using #COVID19 to exert global power in an attempt to surpass the U.S. as a world super power. \nFor decades, many of our allies failed to realize the severity of the threat the CCP posed, but #COVID19 has been a wakeup call.\nhttps://t.co/BUhgLVEocG'), ('dates', '2020-04-29 18:18:04'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7567430734634399)])
OrderedDict([('tweet', 'The Chinese Communist Party wants to set the standards for advanced technologies around the world, stacking the deck against the U.S. &amp; our allies. We must ensure democratic, not authoritarian values guide the development of technologies like AI and 5G.\n \nhttps://t.co/DC6vtlgSf4'), ('dates', '2020-04-28 15:44:27'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.8240116238594055)])
OrderedDict([('tweet', 'Now is far from the time for China to exert their force over other countries. \n \nStop using #COVID19 to help yourselves. Your image campaign failed, your attempts to blame the U.S. failed, and this will inevitably fail as well. \n \nhttps://t.co/JPirwM0mO2'), ('dates', '2020-04-24 17:14:00'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7074751853942871)])
OrderedDict([('tweet', "The Chinese Communist Party's misinformation campaign and failures on #COVID19 will have shockwaves around the world. This can and will continue to affect their role across the globe.\n \nhttps://t.co/5qMcHmc2tF"), ('dates', '2020-04-24 16:02:00'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7914624810218811)])
OrderedDict([('tweet', "As early as February, there were messages in China trying to tie #COVID19 to the U.S. \n \nIt's outrageous the Chinese Communist Party continues to falsely blame the U.S. for #COVID19 while also spreading deadly information about the virus. \n \nhttps://t.co/ngmZcWIoIT"), ('dates', '2020-04-24 15:10:00'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.9441744685173035)])
OrderedDict([('tweet', 'Misinformation on #COVID19 has literally killed people around the world.\n \nTehran, the Kremlin &amp; the Chinese Communist Party must stop trying to hurt others and focus on helping their own citizens. \n \nhttps://t.co/DHaC0G6f1j'), ('dates', '2020-04-22 20:16:42'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.8484795689582825)])
OrderedDict([('tweet', "I'm glad a deal was finally made, but it never should have taken so long. This is partisanship rearing its ugly head. \n \nThe Paycheck Protection Program has been out of money since Thursday. Congress should have acted before that ever happened. \n \nhttps://t.co/BcDyTHkpQD"), ('dates', '2020-04-21 18:59:36'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.5329979658126831)])
OrderedDict([('tweet', 'Democrats refusing to fund the Paycheck Protection Program is outrageous.\n\nThis shouldn\'t be this hard.\n\nAs @WSJ said, "Americans will have every right to conclude that Mrs. Pelosi and Mr. Schumer are responsible for the worsening economic destruction."\n\nhttps://t.co/8Nh4YooV6v'), ('dates', '2020-04-17 16:32:55'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.8149887919425964)])
OrderedDict([('tweet', 'The Paycheck Protection Program ran out of money.\n\nWe must provide additional funding IMMEDIATELY.\n\nThese loans are critical to keeping people on payroll, on their benefits &amp; giving small businesses help.\n\nStop the politics. Texas businesses need these loans.'), ('dates', '2020-04-16 18:18:14'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.5068199038505554)])
OrderedDict([('tweet', 'Dear Chinese Communist Party, \n\nIf you really want to improve your image, own up to the fact that your actions worsened this #pandemic. \n\nTake responsibility and actually help the world. \n\nYour propaganda campaign is backfiring for a reason. \n\nhttps://t.co/ER0XbiqFM2'), ('dates', '2020-04-15 19:52:38'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.576952338218689)])
OrderedDict([('tweet', 'Another example of the Iranian gov’t lying to the world. If they want to be treated like a normal country: stop killing Americans and our allies, stop killing their own innocent citizens peacefully protesting &amp; stop lying about their nuclear program. \n\nhttps://t.co/UOP8T73R3E'), ('dates', '2020-04-10 15:07:00'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7113319039344788)])
OrderedDict([('tweet', 'American families feel uncertainty every day because of this pandemic. Just last week, 6.6 million people filed for unemployment. Families worry about their next paycheck, the next meal on the table and, worse yet, whether this illness will hit their family next.'), ('dates', '2020-04-09 16:26:15'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.5642091631889343)])
OrderedDict([('tweet', 'The Chinese Communist Party failed to be transparent in its early response to #COVID19, endangering everyone.\n\nTaiwan, however, worked to help others.\n\nThe US &amp; our allies should do more to support Taiwan in the intl. arena, including @WHO membership.\n\nhttps://t.co/qMS94tZ21a'), ('dates', '2020-04-07 13:36:40'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7062310576438904)])
OrderedDict([('tweet', "The Chinese Communist Party has undergone an aggressive campaign to cover-up the #Coronavirus' tracks, going so far as to try to blame the U.S. for its spread.\n \nThis timeline by @axios illustrates the spread of the virus and the CCP's coverup efforts:\n \nhttps://t.co/WPg1VecZGV"), ('dates', '2020-04-04 18:25:14'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.9413747191429138)])
OrderedDict([('tweet', 'From the beginning, the Chinese Communist Party was dishonest about the #coronavirus, which put the world in danger. \n \n@DerekScissors1 of @AEI is right to be questioning the #COVID19 numbers coming out of China.\n \nhttps://t.co/7YOr2RPnJx'), ('dates', '2020-04-03 22:30:00'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.947996199131012)])
OrderedDict([('tweet', 'The Chinese Communist Party isn’t the only country spreading misinformation, so is the Kremlin. They want to create chaos &amp; make it harder for the U.S. &amp; our allies to contain #COVID19\n \nThese regimes must be held responsible for their actions. https://t.co/znxQPLPSPV'), ('dates', '2020-04-01 21:50:28'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.980079710483551)])
OrderedDict([('tweet', "My friend @RepMcCaul is right. The Chinese Communist Party keeps lying about this virus. \n\nWe must hold the CCP accountable for putting the entire world's health in jeopardy. https://t.co/tLLxL4kX1A"), ('dates', '2020-03-31 17:07:19'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.9673616290092468)])
OrderedDict([('tweet', 'The Chinese Communist Party has historically hidden what goes on inside China. \n\nThey hid #COVID19, then underplayed its severity.\n \nNow, as we battle the virus here, it looks like the CCP is still lying to the rest of the world.\n \nThis is alarming. \n \nhttps://t.co/6qymgten6r'), ('dates', '2020-03-30 18:04:45'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.9192475080490112)])
OrderedDict([('tweet', "China's gov't so far in 2020:\n✅Hid a deadly virus from the rest of the world endangering the lives of billions\n✅Banned American journalists from China to hide the Chinese Communist Party's (CCP) actions\n✅Joined Russia in spreading #COVID19 lies to stoke fears amid a pandemic"), ('dates', '2020-03-24 22:34:10'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.9317500591278076)])
OrderedDict([('tweet', 'The Chinese government is threatening the health of millions, &amp; the Russians are up-to their old tricks to get people to distrust institutions by spreading global disinformation about #COVID19.\n \nWe must be vigilant when reading info about the pandemic.\n \nhttps://t.co/u6AKXmb7Lk'), ('dates', '2020-03-19 21:03:10'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.5799261927604675)])
OrderedDict([('tweet', 'Austin Tice put his life in harm’s way to report on the civil war in Syria. Sadly, he has been held captive for years because of it. I pray he is still alive &amp; will be reunited with his family. \n\n#FreeAustinTice\n\nhttps://t.co/L37Mo70xaW'), ('dates', '2020-03-19 19:27:23'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.5017949938774109)])
OrderedDict([('tweet', 'Instead of the Communist Party of China message-testing threats in their government-backed media, how about they talk about their responsibility to help the rest of the world deal with the #COVID19 pandemic that started within their borders?\n\nhttps://t.co/6yDhIdBitz'), ('dates', '2020-03-19 15:28:39'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.93537837266922)])
OrderedDict([('tweet', 'China endangered the globe by hiding the deadly, destructive nature of Coronavirus.\n \nNow, they want to ban @WSJ, @nytimes &amp; @washingtonpost from reporting on their actions.\n \nChina keeps attacking human rights &amp; putting us all in danger.\n \nWhat’s next? \n \nhttps://t.co/b7c8QK7ywd'), ('dates', '2020-03-17 20:03:57'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.980057954788208)])
OrderedDict([('tweet', 'World powers should be focused on working together to slow the spread of COVID-19 and stop this pandemic.\n \nNow is not the time for the Chinese Communist Party to spread disinformation and falsely blame the U.S. for spreading this deadly virus.\n\nhttps://t.co/UHNiLgU0yt'), ('dates', '2020-03-17 15:41:49'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.6185983419418335)])
OrderedDict([('tweet', 'The #AfghanPeaceDeal emboldens the Taliban. We need a regional policy that will not jeopardize our security and abandon our ally. Met with Ambassador @RoyaRahmani to discuss how we ensure the current Afghan government has the support it needs to root out terrorism. https://t.co/IwmkWF6R0T'), ('dates', '2020-03-10 22:52:29'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.5610063076019287)])
OrderedDict([('tweet', 'This is not the behavior of a regime committed to a peaceful nuclear program. The United States and our European allies must impose snap-back sanctions on the Iranian government to force them to change course. https://t.co/3gatlPT2FB'), ('dates', '2020-03-03 19:17:47'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7731219530105591)])
OrderedDict([('tweet', "“When you're dealing with something of this severity, transparency is important. Especially when it's a global health risk. The fact that [China] didn't allow WHO folks on the ground, they delayed that for several weeks, and it prevented us from gathering additional details.” https://t.co/rysRVKF61f"), ('dates', '2020-02-27 13:23:38'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.5593310594558716)])
OrderedDict([('tweet', 'At a time when our country is concerned with disinformation, for The New York Times to allow a terrorist responsible for the death of Americans (and who kidnapped their own reporter) to pen an op-ed with things they know are lies is outrageous.'), ('dates', '2020-02-20 20:05:16'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.6050244569778442)])
OrderedDict([('tweet', 'The lack of government transparency from the Chinese Communist Party in dealing with the coronavirus global public health emergency is contributing to the international community’s inability to resolve the crisis. @HoustonChron https://t.co/DIChy4GtP7'), ('dates', '2020-02-19 19:35:21'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.5903609991073608)])
OrderedDict([('tweet', 'American military and economic dominance is no longer guaranteed, and this will have a profound effect on our nation’s approach to foreign policy. Happy to see @UTSA and @CFR_org are hosting a constructive discussion on how to address the global challenges we face. #CFRatUTSA https://t.co/aSt8Jole5t'), ('dates', '2020-02-11 16:13:25'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.5025864243507385)])
OrderedDict([('tweet', 'China’s scheme to overtake the U.S. economy started years ago, and today’s announcement confirms what we suspected — the Equifax breach was state sponsored. Our government and the private sector must collaborate to stop these entirely preventable breaches. https://t.co/AUbPsLIxXn'), ('dates', '2020-02-10 21:52:59'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.9540959596633911)])
OrderedDict([('tweet', "You can't make this up. General Secretary Xi's communist regime detained and silenced Dr. Wenliang for speaking out on the dangers of coronavirus, and now the virus has taken his life.\n \nTheir refusal to be fully transparent has put the world at risk. https://t.co/a6UOjmdkfU"), ('dates', '2020-02-07 17:37:38'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7588335275650024)])
OrderedDict([('tweet', 'Good piece on what China will do to surpass the United States as the most important economy. We need new policies to confront this threat, like if an American company can’t do something in China, then Chinese companies shouldn’t be able to do it here. https://t.co/P1lOvlciXE'), ('dates', '2020-02-02 16:21:48'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.8849256038665771)])
OrderedDict([('tweet', "We’ve heard of protests in Hong Kong and Iran, but there have been protests in Georgia. These are freedom-loving people fighting Russian oppression and demanding that their gov't implement promised democratic reforms. I’m glad Congress is supporting them in a bipartisan way. https://t.co/DLQ4anmiqY"), ('dates', '2020-01-31 15:09:13'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.8023169636726379)])
OrderedDict([('tweet', 'China produces over 80% of the world’s rare earth minerals and they are used in everything from cell phones to advanced weapon systems. This action by the Administration to allow domestic rare earth production is a big U.S. national security victory. https://t.co/V4eRjboRD2'), ('dates', '2020-01-18 01:20:56'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.5078197121620178)])
OrderedDict([('tweet', "We must be united in condemning the Iranian regime and remember that it's separate from the Iranian people and Persian culture. \n\nLet’s stand with the Iranian people and amplify their message as they stand up against tyranny and seek their freedom. #IranProtests #FreeIran https://t.co/JyCAI1ZcIL"), ('dates', '2020-01-12 23:01:30'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.6005039811134338)])
OrderedDict([('tweet', 'China is spreading disinformation to interfere in Taiwan’s election. Are we prepared if they do it in the U.S.? https://t.co/rKDLL9LlQE'), ('dates', '2020-01-10 01:00:00'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7799069881439209)])
OrderedDict([('tweet', 'Now is not the time for a partisan exercise that the Ayatollahs could use as propaganda. Instead, Congress should be united in condemning the Iranian regime that has been attacking America and our allies for 40 years. https://t.co/5oQblwEA1C'), ('dates', '2020-01-09 22:15:00'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.9398108720779419)])
OrderedDict([('tweet', "The Iranian regime has lied to the world about their nuclear arsenal and killing 1,500 unarmed protestors. They lied to their own people about the number of American deaths in this week's missile strikes.\n \nAre they now lying about the deaths of 176 innocent civilians? https://t.co/gMRalgXeGw"), ('dates', '2020-01-09 20:25:31'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.9472849369049072)])
OrderedDict([('tweet', 'Our troops are in a dangerous world, and we must protect American lives and our allies. We also have to remember why we are seeing the increased Iranian activity in Iraq: because the Iraqi people were sick and tired of an Iranian puppet as their prime minister. https://t.co/pzamLcDiHx'), ('dates', '2020-01-08 23:54:28'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.9256160259246826)])
OrderedDict([('tweet', "We've seen that when the Iranian regime suffers consequences for their behavior, they stop that behavior. The strike on Soleimani and their response shows that they realize that they'll have to change their ways. @AmericaNewsroom https://t.co/2klFvuGf9H"), ('dates', '2020-01-08 19:01:00'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7850242257118225)])
OrderedDict([('tweet', "Let's be clear: this is not something that is new. We know the Iranian regime has been trying to kill Americans for over 40 years. When there are no consequences for bad behavior, they're going to continue doing it. @NewDay https://t.co/78fMzEElVL"), ('dates', '2020-01-07 19:01:00'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.7969217300415039)])
OrderedDict([('tweet', 'The recent attacks in New York remind us that antisemitism continues to plague our society. We must root out this senseless hate, which is why today and every day I stand with those who are #JewishandProud to combat antisemitism in all forms. @AJCGlobal'), ('dates', '2020-01-06 20:53:38'), ('file_name', 'Will Hurd.csv'), ('pct_true', 0.6942322254180908)])
OrderedDict([('tweet', 'I am calling on President Trump to take immediate action to harness the power of the Defense Production Act and put Americans to work making the lifesaving items that health care providers need to save lives – without delay.'), ('dates', '2020-03-20 15:23:12'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.5974100828170776)])
OrderedDict([('tweet', 'The House passed HR 1 a year ago today - it expands voter access, election integrity, election security, political spending rules, and ethics for the three branches of government. Sadly, Mitch McConnell continues to refuse to allow the Senate to vote on it.'), ('dates', '2020-03-09 01:29:21'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.8019552826881409)])
OrderedDict([('tweet', 'Yesterday, four career federal prosecutors assigned to the Roger Stone case resigned in protest after the President and Attorney General Barr intervened in their sentencing requests. This is so serious.'), ('dates', '2020-02-12 17:14:49'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.5533638000488281)])
OrderedDict([('tweet', 'These are some key differences between the Trump and Clinton impeachment proceedings. These are also the reasons why President Trump was impeached for obstructing Congress. #DefendOurDemocracy https://t.co/3prLvx2rt2'), ('dates', '2020-01-18 20:20:03'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.8485054969787598)])
OrderedDict([('tweet', 'The President should not be tweeting about fifty-two Iranian targets – a reference to the fifty-two Americans taken hostage in 1979 – because it will not deter a response from Iran. In fact, I believe it will act to further incite Iranian leadership. https://t.co/6wMzSL8ZHs'), ('dates', '2020-01-05 20:31:06'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.6700020432472229)])
OrderedDict([('tweet', "Trump's trade war is bad for local business. When the E.U. proposed retaliatory tariffs against our cranberry growers in 2018 I was able to leverage my relationships with leaders in Europe to stop them."), ('dates', '2019-11-27 19:04:18'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.5793818831443787)])
OrderedDict([('tweet', 'There is no place in the world for white supremacy, for racism, or for xenophobia. Our parents and grandparents - the greatest generation - fought a war on that ideal.'), ('dates', '2019-11-15 17:15:13'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.5522124767303467)])
OrderedDict([('tweet', 'Trump’s Ukrainian dirt-for-arms scandal brought on impeachment investigations, but you’re not alone if you don’t fully understand the importance of Ukraine. Here’s what happened and why it’s so important: https://t.co/cN0NXcSckA'), ('dates', '2019-11-15 12:44:56'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.6592141389846802)])
OrderedDict([('tweet', 'We have a Constitutional obligation as a Congress to protect our country. We must move on forward on a formal impeachment inquiry.'), ('dates', '2019-09-24 21:31:58'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.7529574632644653)])
OrderedDict([('tweet', 'The President has acted to coerce a foreign power to undercut our country’s national security and democracy, an action declared to be an ‘urgent matter’ by the Inspector General. The Constitution and our rule of law are at stake right now.'), ('dates', '2019-09-24 21:31:52'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.6608269810676575)])
OrderedDict([('tweet', 'The IPCC Report warns us that without cuts in carbon emissions, the world could see an average sea level rise of 62 centimeters over the course of the lifetime of people born today – that is over two feet. #ClimateCrisis'), ('dates', '2019-09-18 14:34:27'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.5200713276863098)])
OrderedDict([('tweet', 'Change happens through leadership. We wouldn’t be in this situation right now – our witnesses would be safely and happily in school pursuing their dreams and not protesting and pushing their governments to act – if everyone was already doing their part. #ClimateCrisis'), ('dates', '2019-09-18 14:33:29'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.7991661429405212)])
OrderedDict([('tweet', 'There must be leadership in addressing climate change because we, and especially your generation, can’t afford for things not to change. #ClimateCrisis'), ('dates', '2019-09-18 14:32:48'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.521312415599823)])
OrderedDict([('tweet', 'Last week, @realdonaldtrump sent a racist tweet to derail our Wampanoag bill.  This week, by an overwhelming majority, we stood up for the basic rights of not just the Mashpee Wampanoag but all Native Americans.'), ('dates', '2019-05-15 21:38:49'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.7032140493392944)])
OrderedDict([('tweet', 'Trump signed an identical bill last year. So why tweet against a bill recognizing the tribe of the first Thanksgiving? Because of his well-documented alliance with the RI casino lobbyist. A weak attempt to hide corrupt influence in a racist tweet.'), ('dates', '2019-05-08 17:39:31'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.8493334054946899)])
OrderedDict([('tweet', '.@JunckerEU declares 25% of EU budget will go to mitigate climate change. Our friends in Europe again take two steps forward while we take two back.  The US should be leading the fight against climate change, not focusing energies trying to debunk science. https://t.co/4FbTUMx4b9'), ('dates', '2019-02-22 15:11:18'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.5210094451904297)])
OrderedDict([('tweet', "The President's willingness to discard our longstanding NATO alliance without consulting Congress or our allies is disturbing. That there was discussion of this prospect alone is alarming, damages our credibility with our European allies, and leaves us less secure."), ('dates', '2019-01-16 00:17:10'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.8469495177268982)])
OrderedDict([('tweet', 'The Department of Homeland Security has many responsibilities and contributing to the Terrorist Screening Database is one of them. This is the database that TSA uses for the “No-Fly List” and the database that the Department of Justice checks against gun purchases. #TrumpShutdown'), ('dates', '2019-01-09 01:11:21'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.8857001662254333)])
OrderedDict([('tweet', 'Secretary of Defense Mattis’ resignation letter makes it clear his departure was a direct result of President Trump’s worldview and policy. His departure will create even more uncertainty during a time when our allies and coalition partners are questioning US leadership.'), ('dates', '2018-12-21 02:49:45'), ('file_name', 'William R. Keating.csv'), ('pct_true', 0.5675266981124878)])
OrderedDict([('tweet', 'After almost two weeks of delay, Democrats have finally allowed additional funding for small businesses to move forward. I am disappointed it took so long—we cannot allow political wish lists to stand in the way of help for American workers and businesses.'), ('dates', '2020-04-21 23:29:00'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.5123683214187622)])
OrderedDict([('tweet', 'Today we remember the six million Jews who were murdered during the Holocaust. We will not forget these lives taken too soon and say #NeverAgain to such hate and anti-Semitism. https://t.co/PyMywlr6ee'), ('dates', '2020-04-21 21:16:34'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.7831164002418518)])
OrderedDict([('tweet', 'In just a few short weeks, #PPP has saved an estimated 15 million jobs, but there are still millions more that need access to this vital program. Democrats are holding up additional funding when they know the money will run out by the end of the week, if not before. https://t.co/PJRXTm2P2s'), ('dates', '2020-04-15 23:39:50'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.637202799320221)])
OrderedDict([('tweet', '#PPP has provided $2.7 billion to more than 14,000 #smallbiz and non-profits in South Carolina. With money quickly running out, Democrats are holding up additional funding in exchange for political pet projects. More from @GOPLeader and @SenatorTimScott: https://t.co/16oQh7dnUR'), ('dates', '2020-04-15 17:45:58'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.6535632014274597)])
OrderedDict([('tweet', 'The U.S. is the World Health Organization’s largest contributor. Our tax dollars are funding an organization that has mishandled the #COVID19 pandemic and promoted propoganda for the Chinese Communist Party. President @realDonaldTrump is right to withhold funding from the WHO.'), ('dates', '2020-04-15 00:30:21'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.8016080856323242)])
OrderedDict([('tweet', '#PaycheckProtectionProgram is a lifeline for small businesses and workers who have been hit hard by this crisis. It is clear we need to extend funding, yet today Senate Democrats blocked legislation for additional funding. The American people need paychecks, not political games.'), ('dates', '2020-04-09 20:22:31'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.8962365984916687)])
OrderedDict([('tweet', "U.S. intelligence has now confirmed what we already knew: China covered up the seriousness of this virus for months. The world is now paying the price for their mistakes. The Communist Party of China's propaganda campaign must stop. https://t.co/nBzl0lbCWN"), ('dates', '2020-04-01 20:20:23'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.725028395652771)])
OrderedDict([('tweet', 'American businesses, workers, and families are hurting through no fault of their own. The bipartisan CARES Act which was passed by the Senate last night provides critical relief to keep our economy moving during this crisis. (1/5)'), ('dates', '2020-03-26 14:36:23'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.8685786128044128)])
OrderedDict([('tweet', 'All the partisanship and dysfunction in Washington right now are preventing South Carolina’s small businesses, families, and workers from getting the help they need during this health crisis.  Read my full statement: https://t.co/crNe18A0DY'), ('dates', '2020-03-23 21:59:13'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.5986438393592834)])
OrderedDict([('tweet', 'Congress must put politics aside and quickly pass a bipartisan bill to protect families, workers, and the American economy. Time is of the essence. Small businesses are closing. The American people are hurting. There is no time to waste.\n\nhttps://t.co/BH7PQnATGn'), ('dates', '2020-03-23 15:43:55'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.7998888492584229)])
OrderedDict([('tweet', 'It is impossible not to interpret the Senate Minority Leader’s words as anything other than a threat. Chief Justice Roberts is exactly right, this is dangerous. All Democrats should be asked whether or not they agree with Schumer. https://t.co/1jSC6ncJYC'), ('dates', '2020-03-04 23:47:00'), ('file_name', 'William R. Timmons IV.csv'), ('pct_true', 0.7664064764976501)])
OrderedDict([('tweet', 'Families should not have to fear losing their homes as COVID-19 continues to impact jobs. This AM, @HUDgov authorized @FHAgov to stop foreclosure actions &amp; evictions for the next 60 days. Please continue to reach out to my team if we can be of assistance. https://t.co/s3hz8csiFC'), ('dates', '2020-03-19 21:48:06'), ('file_name', 'Xochitl Torres Small.csv'), ('pct_true', 0.6670681238174438)])
OrderedDict([('tweet', 'The Trump-Pence rules allow employers &amp; universities to take birth control coverage away from their employees &amp; students. They leave women on their own to find &amp; pay for contraception— that’s discrimination.\n\nEveryone deserves birth control coverage no matter what. #HandsOffMyBC'), ('dates', '2020-05-05 19:14:20'), ('file_name', 'Yvette D. Clarke.csv'), ('pct_true', 0.9060622453689575)])
OrderedDict([('tweet', 'Trump is yet again using immigrants as his pawns instead of doing actual presidential work. Surprised? Nah. Disappointed? Definitely. \n\nPeople are dying and 45 won’t take any responsibility nor action to protect lives. ENOUGH. #coronavirus https://t.co/yQciFzOgpo'), ('dates', '2020-04-21 03:16:18'), ('file_name', 'Yvette D. Clarke.csv'), ('pct_true', 0.7997214794158936)])
OrderedDict([('tweet', 'We will not allow Trump to use a pandemic to push through his administration’s xenophobic, racist policies. America needs a leader—not a bully—in times of crisis like this. #NotMyPresident https://t.co/jQcCIy5q33'), ('dates', '2020-03-22 15:10:31'), ('file_name', 'Yvette D. Clarke.csv'), ('pct_true', 0.9203692674636841)])
OrderedDict([('tweet', "I'm concerned by reports that the Trump Administration is unfairly allocating funds to the benefit of big businesses, with millions for big corporations instead of Main Street.\n\nMore on my take on the rollout of the Paycheck Protection Program ⤵️\nhttps://t.co/9iq3tItQmX"), ('dates', '2020-04-17 15:42:31'), ('file_name', 'Zoe Lofgren.csv'), ('pct_true', 0.7013538479804993)])
OrderedDict([('tweet', "The Trump Administration's #MuslimBan has not made us safer.\n\nIt has weakened America's standing in the world &amp; runs contrary to our country’s moral &amp; philosophical foundation.\n\nYou can watch the #NoBan Act live debate this morning ⤵️\nhttps://t.co/gRNmgF0SCu"), ('dates', '2020-02-12 14:49:51'), ('file_name', 'Zoe Lofgren.csv'), ('pct_true', 0.6784720420837402)])]

In [0]:
df = pd.DataFrame(summary_data)
output_csv_df = df.sort_values(['pct_true'], ascending=False)

In [0]:
output_csv_df

In [0]:
output_csv_df.drop(columns=['covid_1',	'covid_2',	'covid_3', 'other_1',	'other_2',	'other_3'], inplace=True)

In [0]:
output_csv_df.rename(columns={"pct_true": "likelihood"}, inplace=True)

In [0]:
output_csv_df

In [0]:
output_csv_name = f'{save_path}/fear_tweets_may-11-2020.csv'
output_csv_df.to_csv(output_csv_name, index=False)